# PATSTAT PIZnet Jupyter Demo 

# Patent Applicants and Technology Distribution in Germany by Landkreis

This notebook analyzes patent applicants and technology distributions across Germany's NUTS Level 3 regions (Landkreise). Using SQL to query the PATSTAT database, we will load data into Pandas DataFrames and explore it interactively with Pygwalker. For optional geographical visualization, we will use Geopandas to create maps of applicants and technology distributions by Landkreis. 

### Setup and import the library modules

In [59]:
# Import necessary libraries and modules
# Install necessary libraries if not already installed
#!pip install pandas sqlalchemy pygwalker geopandas

# Import libraries
import pandas as pd
import pygwalker as pyg
from sqlalchemy import create_engine, func
import time

# Import xml lib for IPC sub group labels
from lxml import etree as ET

# (Optional) Import Geopandas for mapping if needed later
import geopandas as gpd




In [ ]:
# Import necessary libraries and modules
# Import the library module
from epo.tipdata.patstat import PatstatClient

# Intantiate the client object / TEST or PRDOD
patstat = PatstatClient(env='TEST')
# patstat = PatstatClient(env='PROD')

# Instantiate the ORM
db = patstat.orm()

# import all the tables we need
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS202_APPLN_TITLE, TLS206_PERSON, TLS207_PERS_APPLN, TLS224_APPLN_CPC, TLS231_INPADOC_LEGAL_EVENT


#### Test Query 1

List all granted applications filed at EPO (direct +PCT) having a year of filing = 2010.


In [ ]:
# Define SQL queries using ORM for analysis
# Example: Query patent applicants and technology distribution
# test query 1
q = db.query(TLS201_APPLN.appln_id, TLS201_APPLN.appln_auth, TLS201_APPLN.appln_nr, 
                           TLS201_APPLN.appln_kind, TLS201_APPLN.appln_filing_date).\
             filter(TLS201_APPLN.appln_filing_year == 2010,
                    TLS201_APPLN.appln_auth == 'EP',
                    TLS201_APPLN.granted == 'Y')
patstat.df(q)

#### Test Query 2

Create a hitlist of Chinese applicants filing patents at the EPO (direct or PCT).

In [ ]:
# Define SQL queries using ORM for analysis
# Example: Query patent applicants and technology distribution
# test query 2

q = db.query(TLS206_PERSON.psn_name, TLS206_PERSON.person_ctry_code,
                           func.count(TLS201_APPLN.appln_id).label('APPLICATIONS_AT_EPO')).\
             select_from(TLS206_PERSON).\
             join(TLS207_PERS_APPLN).join(TLS201_APPLN).\
             filter(TLS206_PERSON.person_ctry_code == 'CN',
                    TLS207_PERS_APPLN.applt_seq_nr > 0,
                    TLS207_PERS_APPLN.invt_seq_nr == 0,
                    TLS201_APPLN.appln_auth == 'EP').\
             group_by(TLS206_PERSON.psn_name, TLS206_PERSON.person_ctry_code).\
             order_by(func.count(TLS201_APPLN.appln_id).desc()).limit(100)
patstat.df(q)

### ChatGPT (GPT-4o) as a Co-Developer

Initial Prompt with mtc.berlin "ChatGPT Team" with activated memory:

I need you to help me develop SQL statements for PATSTAT based on regional NUTS codes to identify applicants and technologies based on federal states in Germany. Please act as a co-developer and be so kind to help me with a map per federal state where we create lists for applicants and technologies distribution in level 3 NUTS. We want to visualize with pygwalker library. 

Please answer in markdown and code blocks to easily copy the results for testing in a jupyter notebook. 


#### Pyg Walker settings

1. Coordinate System: Geographic
2. Mark System: Choropleth
3. Download nuts in geojson from Eurostat (https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics) 
4. Load geojson file
5. Enter Feature ID
6. Add NUTS field to Geometry ID
7. Add Row Count to Color


In [ ]:
# Import necessary libraries and modules
from sqlalchemy import func

# Define query
q = db.query(
    TLS206_PERSON.person_name,
    TLS206_PERSON.nuts.label('nuts_code'),
    TLS224_APPLN_CPC.cpc_class_symbol.label('technology_field'),  # Adding CPC class as technology field
    func.count(TLS201_APPLN.appln_id).label('appln_count')
).select_from(TLS206_PERSON).\
    join(TLS207_PERS_APPLN, TLS206_PERSON.person_id == TLS207_PERS_APPLN.person_id).\
    join(TLS201_APPLN, TLS207_PERS_APPLN.appln_id == TLS201_APPLN.appln_id).\
    join(TLS224_APPLN_CPC, TLS201_APPLN.appln_id == TLS224_APPLN_CPC.appln_id).\
    filter(
        TLS206_PERSON.nuts.startswith('DE'),   # Filter for Germany NUTS code
        TLS206_PERSON.nuts_level == 3          # Limit to NUTS level 3
    ).\
    group_by(
        TLS206_PERSON.nuts,
        TLS206_PERSON.person_name,
        TLS224_APPLN_CPC.cpc_class_symbol      # Group by CPC class as well
    ).\
    order_by(TLS206_PERSON.nuts)  # Sort by nuts_code
# Convert to DataFrame for further processing and visualization

df = patstat.df(q)

df


In [ ]:
# Execute this cell for a Tableau like visualisationin PygWalker

pyg.walk(df)

# vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["choropleth"],"coordSystem":"geographic","limit":-1,"timezoneDisplayOffset":0},"encodings":{"dimensions":[{"fid":"person_name","name":"person_name","basename":"person_name","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"nuts_code","name":"nuts_code","basename":"nuts_code","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"technology_field","name":"technology_field","basename":"technology_field","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"fid":"appln_count","name":"appln_count","basename":"appln_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","offset":0},{"fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[],"columns":[],"color":[{"fid":"appln_count","name":"appln_count","basename":"appln_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","offset":0}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[{"fid":"nuts_code","name":"nuts_code","basename":"nuts_code","semanticType":"nominal","analyticType":"dimension","offset":0}],"details":[],"filters":[],"text":[]},"layout":{"showActions":true,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":776,"height":576},"format":{},"geoKey":"NUTS_ID","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false},"geojson":{"type":"FeatureCollection","name":"NUTS_RG_20M_2013_4326.geojson","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:EPSG::4326"}},"features":[{"type":"Feature","properties":{"NUTS_ID":"AT","LEVL_CODE":0,"CNTR_CODE":"AT","NAME_LATN":"ÖSTERREICH","NUTS_NAME":"ÖSTERREICH"},"geometry":{"type":"Polygon","coordinates":[[[16.940278,48.617245498000045],[16.949778,48.535791998000036],[16.851106,48.438635],[16.842372500000067,48.35332699800006],[16.957182,48.26347999600006],[16.976203,48.172244497000065],[17.066741,48.11868149800006],[17.06741450000004,48.03143750000004],[17.160797500000058,48.00665650000008],[17.09466150000003,47.970871498000065],[17.085187500000075,47.87443749800008],[17.00994950000006,47.858343998000066],[17.07445350000006,47.80825550000003],[17.093074,47.70823599900007],[16.749031,47.68141649700004],[16.61228950000003,47.76022649600003],[16.421846,47.66470449800005],[16.652076,47.622903499000074],[16.71450550000003,47.539630998000064],[16.64622,47.446596999000064],[16.445455,47.40720949700005],[16.43376150000006,47.352918499000054],[16.483743,47.28759999600004],[16.419486500000062,47.19491549800006],[16.529331,47.12723199800007],[16.46332650000005,47.094956498000045],[16.520565500000032,47.05618149800006],[16.438882,47.02970299800006],[16.508267500000045,47.00125599800003],[16.302262,46.99842649800007],[16.113849,46.86906799800005],[15.996236,46.835398499000064],[15.984746500000028,46.752207],[16.038086,46.65614549600008],[15.786422,46.70746949900007],[15.649988,46.70575699900007],[15.508846500000061,46.616227998000056],[15.40197949900005,46.65354849800008],[15.065120500000035,46.65211249600003],[14.879666,46.61124799800007],[14.818606,46.50938299900008],[14.674580500000047,46.450687499000026],[14.565175500000066,46.37245349600005],[14.43450050000007,46.44294350000007],[13.714184999000054,46.522703499000045],[13.504249500000071,46.566303997000034],[12.731392999000036,46.63428799900004],[12.690635,46.656972],[12.477924,46.679835497000056],[12.312809,46.78386599600003],[12.273703,46.88438799800008],[12.143811,46.91377999700006],[12.121343500000023,47.00700199800008],[12.24074550000006,47.06916849800007],[12.136014,47.08066749900007],[11.747028500000056,46.96887249800005],[11.627199500000074,47.01329899900003],[11.164281500000072,46.96572249900004],[11.02225,46.76541049800005],[10.757589500000051,46.78518599900008],[10.667368500000066,46.87549049800003],[10.469651500000055,46.85490899900003],[10.489350500000057,46.93779049800003],[10.38931799900007,47.00052449900005],[10.144974500000046,46.85100949800005],[9.876148500000056,46.93463550000007],[9.879422500000032,47.01956299600005],[9.607078,47.06077449900005],[9.620580500000074,47.15164549900004],[9.530749,47.270581],[9.673370500000033,47.38151050000005],[9.55872050000005,47.54189299800004],[9.736566500000038,47.534906498000055],[9.765470500000049,47.588856],[9.967813500000034,47.54624049600005],[9.999526,47.48301699600006],[10.09147050000007,47.45907599700007],[10.09985049900007,47.35476149700003],[10.23393,47.38328599700003],[10.178353,47.270113998000056],[10.475413,47.431702],[10.431323500000076,47.50367749600008],[10.454439,47.55579699800006],[10.886199,47.53684799800004],[10.871043,47.48188199800006],[10.991202500000043,47.396130999000036],[11.016500500000063,47.396368],[11.42142750000005,47.44485199800005],[11.38380950000004,47.47220749600007],[11.410219,47.495324],[11.572426,47.514643998000054],[11.632883,47.59244599600004],[12.060662,47.61874349800007],[12.20731950000004,47.609837998000046],[12.183931,47.70037049800004],[12.264886500000046,47.73248299800008],[12.257177500000068,47.6791],[12.338045499000032,47.697087500000066],[12.575026500000035,47.63231599900007],[12.695795500000031,47.68222299800004],[12.807303,47.61022749800003],[12.77925650000003,47.579285500000026],[12.81025,47.54706299800006],[13.003285500000061,47.463374498000064],[13.046055500000023,47.52050249800004],[13.097948,47.63371699800007],[13.047965,47.709118998000065],[12.905047,47.72364349900005],[13.00389750000005,47.84796899600008],[12.875785500000063,47.962608998000064],[12.860181500000067,47.996639998000035],[12.751555,48.112810497000055],[12.94468450000005,48.20669249800005],[13.177043,48.29438999600006],[13.407838,48.372160500000064],[13.438430500000038,48.548895],[13.513369500000067,48.59097799700004],[13.727090500000031,48.51301950000004],[13.825764,48.61861649800005],[13.796107,48.71360049800006],[13.839507,48.771605],[14.115935,48.58917099800004],[14.324520500000062,48.56233949700004],[14.470069,48.64843699800008],[14.691014,48.58430199700007],[14.808667,48.77879949900006],[14.979332,48.77235049800004],[14.990445,49.009649],[15.157331,48.991779500000064],[15.160247,48.94168649800008],[15.365882999000064,48.98170699900004],[15.542449500000032,48.90769699600003],[15.753633500000035,48.852178500000036],[15.84331,48.873388498000054],[16.102884500000073,48.74539599900004],[16.648492,48.78520849800003],[16.90202350000004,48.71797799800004],[16.940278,48.617245498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT1","LEVL_CODE":1,"CNTR_CODE":"AT","NAME_LATN":"OSTÖSTERREICH","NUTS_NAME":"OSTÖSTERREICH"},"geometry":{"type":"Polygon","coordinates":[[[15.157331,48.991779500000064],[15.160247,48.94168649800008],[15.365882999000064,48.98170699900004],[15.542449500000032,48.90769699600003],[15.753633500000035,48.852178500000036],[15.84331,48.873388498000054],[16.102884500000073,48.74539599900004],[16.648492,48.78520849800003],[16.90202350000004,48.71797799800004],[16.940278,48.617245498000045],[16.949778,48.535791998000036],[16.851106,48.438635],[16.842372500000067,48.35332699800006],[16.957182,48.26347999600006],[16.976203,48.172244497000065],[17.066741,48.11868149800006],[17.06741450000004,48.03143750000004],[17.160797500000058,48.00665650000008],[17.09466150000003,47.970871498000065],[17.085187500000075,47.87443749800008],[17.00994950000006,47.858343998000066],[17.07445350000006,47.80825550000003],[17.093074,47.70823599900007],[16.749031,47.68141649700004],[16.61228950000003,47.76022649600003],[16.421846,47.66470449800005],[16.652076,47.622903499000074],[16.71450550000003,47.539630998000064],[16.64622,47.446596999000064],[16.445455,47.40720949700005],[16.43376150000006,47.352918499000054],[16.483743,47.28759999600004],[16.419486500000062,47.19491549800006],[16.529331,47.12723199800007],[16.46332650000005,47.094956498000045],[16.520565500000032,47.05618149800006],[16.438882,47.02970299800006],[16.508267500000045,47.00125599800003],[16.302262,46.99842649800007],[16.113849,46.86906799800005],[15.996236,46.835398499000064],[16.12171950000004,46.99066649900004],[16.086508,47.074209],[16.113463,47.16183849600003],[16.01515850000004,47.36712749800006],[16.17176950000004,47.42239999900005],[16.087398500000063,47.507157498000026],[15.847011,47.56778949800008],[15.821781,47.637955496000075],[15.725103,47.646455496000044],[15.719146,47.70988349900006],[15.611712,47.75108649800006],[15.374876999000037,47.824564],[15.217306,47.79602749900005],[15.08924950000005,47.741471],[14.738462,47.74872349900005],[14.709993,47.85064149900006],[14.771718,47.883778998000025],[14.493634,47.99779749600003],[14.456123,48.04303699800005],[14.479043500000046,48.10441449900003],[14.521179,48.23760949800004],[14.688370500000076,48.157290498000066],[14.828842500000064,48.189076998000075],[14.992047,48.25592899800006],[14.986449500000049,48.319307998000056],[14.943687,48.32364799900006],[14.964447,48.378512],[14.873728500000027,48.459239998000044],[14.912456500000076,48.51396149800007],[14.691014,48.58430199700007],[14.808667,48.77879949900006],[14.979332,48.77235049800004],[14.990445,49.009649],[15.157331,48.991779500000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT11","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Burgenland (AT)","NUTS_NAME":"Burgenland (AT)"},"geometry":{"type":"Polygon","coordinates":[[[17.06741450000004,48.03143750000004],[17.160797500000058,48.00665650000008],[17.09466150000003,47.970871498000065],[17.085187500000075,47.87443749800008],[17.00994950000006,47.858343998000066],[17.07445350000006,47.80825550000003],[17.093074,47.70823599900007],[16.749031,47.68141649700004],[16.61228950000003,47.76022649600003],[16.421846,47.66470449800005],[16.652076,47.622903499000074],[16.71450550000003,47.539630998000064],[16.64622,47.446596999000064],[16.445455,47.40720949700005],[16.43376150000006,47.352918499000054],[16.483743,47.28759999600004],[16.419486500000062,47.19491549800006],[16.529331,47.12723199800007],[16.46332650000005,47.094956498000045],[16.520565500000032,47.05618149800006],[16.438882,47.02970299800006],[16.508267500000045,47.00125599800003],[16.302262,46.99842649800007],[16.113849,46.86906799800005],[15.996236,46.835398499000064],[16.12171950000004,46.99066649900004],[16.086508,47.074209],[16.113463,47.16183849600003],[16.01515850000004,47.36712749800006],[16.17176950000004,47.42239999900005],[16.281081,47.45513249900006],[16.372502,47.64248549800004],[16.26904950000005,47.79640549800007],[16.377188499000056,47.82732149700007],[16.388891499000067,47.88160099600003],[16.486915500000066,47.94262049800005],[16.58028,47.888607999000044],[16.7021,48.01165199800005],[16.889998,48.07463899800007],[16.979235500000073,48.023454497000046],[17.016295,48.07352049800005],[16.971083500000077,48.10057199600004],[17.066741,48.11868149800006],[17.06741450000004,48.03143750000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT111","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Mittelburgenland","NUTS_NAME":"Mittelburgenland"},"geometry":{"type":"Polygon","coordinates":[[[16.64622,47.446596999000064],[16.445455,47.40720949700005],[16.43376150000006,47.352918499000054],[16.37763250000006,47.34620799800007],[16.281081,47.45513249900006],[16.372502,47.64248549800004],[16.421846,47.66470449800005],[16.652076,47.622903499000074],[16.71450550000003,47.539630998000064],[16.64622,47.446596999000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT112","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Nordburgenland","NUTS_NAME":"Nordburgenland"},"geometry":{"type":"Polygon","coordinates":[[[17.160797500000058,48.00665650000008],[17.09466150000003,47.970871498000065],[17.085187500000075,47.87443749800008],[17.00994950000006,47.858343998000066],[17.07445350000006,47.80825550000003],[17.093074,47.70823599900007],[16.749031,47.68141649700004],[16.61228950000003,47.76022649600003],[16.421846,47.66470449800005],[16.372502,47.64248549800004],[16.26904950000005,47.79640549800007],[16.377188499000056,47.82732149700007],[16.388891499000067,47.88160099600003],[16.486915500000066,47.94262049800005],[16.58028,47.888607999000044],[16.7021,48.01165199800005],[16.889998,48.07463899800007],[16.979235500000073,48.023454497000046],[17.016295,48.07352049800005],[16.971083500000077,48.10057199600004],[17.066741,48.11868149800006],[17.06741450000004,48.03143750000004],[17.160797500000058,48.00665650000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT113","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Südburgenland","NUTS_NAME":"Südburgenland"},"geometry":{"type":"Polygon","coordinates":[[[16.43376150000006,47.352918499000054],[16.483743,47.28759999600004],[16.419486500000062,47.19491549800006],[16.529331,47.12723199800007],[16.46332650000005,47.094956498000045],[16.520565500000032,47.05618149800006],[16.438882,47.02970299800006],[16.508267500000045,47.00125599800003],[16.302262,46.99842649800007],[16.113849,46.86906799800005],[15.996236,46.835398499000064],[16.12171950000004,46.99066649900004],[16.086508,47.074209],[16.113463,47.16183849600003],[16.01515850000004,47.36712749800006],[16.17176950000004,47.42239999900005],[16.281081,47.45513249900006],[16.37763250000006,47.34620799800007],[16.43376150000006,47.352918499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT12","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Niederösterreich","NUTS_NAME":"Niederösterreich"},"geometry":{"type":"Polygon","coordinates":[[[15.157331,48.991779500000064],[15.160247,48.94168649800008],[15.365882999000064,48.98170699900004],[15.542449500000032,48.90769699600003],[15.753633500000035,48.852178500000036],[15.84331,48.873388498000054],[16.102884500000073,48.74539599900004],[16.648492,48.78520849800003],[16.90202350000004,48.71797799800004],[16.940278,48.617245498000045],[16.949778,48.535791998000036],[16.851106,48.438635],[16.842372500000067,48.35332699800006],[16.957182,48.26347999600006],[16.976203,48.172244497000065],[17.066741,48.11868149800006],[16.971083500000077,48.10057199600004],[17.016295,48.07352049800005],[16.979235500000073,48.023454497000046],[16.889998,48.07463899800007],[16.7021,48.01165199800005],[16.58028,47.888607999000044],[16.486915500000066,47.94262049800005],[16.388891499000067,47.88160099600003],[16.377188499000056,47.82732149700007],[16.26904950000005,47.79640549800007],[16.372502,47.64248549800004],[16.281081,47.45513249900006],[16.17176950000004,47.42239999900005],[16.087398500000063,47.507157498000026],[15.847011,47.56778949800008],[15.821781,47.637955496000075],[15.725103,47.646455496000044],[15.719146,47.70988349900006],[15.611712,47.75108649800006],[15.374876999000037,47.824564],[15.217306,47.79602749900005],[15.08924950000005,47.741471],[14.738462,47.74872349900005],[14.709993,47.85064149900006],[14.771718,47.883778998000025],[14.493634,47.99779749600003],[14.456123,48.04303699800005],[14.479043500000046,48.10441449900003],[14.521179,48.23760949800004],[14.688370500000076,48.157290498000066],[14.828842500000064,48.189076998000075],[14.992047,48.25592899800006],[14.986449500000049,48.319307998000056],[14.943687,48.32364799900006],[14.964447,48.378512],[14.873728500000027,48.459239998000044],[14.912456500000076,48.51396149800007],[14.691014,48.58430199700007],[14.808667,48.77879949900006],[14.979332,48.77235049800004],[14.990445,49.009649],[15.157331,48.991779500000064]],[[16.18190050000004,48.17102999600007],[16.21809650000006,48.12806799600003],[16.576992500000074,48.14443649800006],[16.536381,48.17461149800005],[16.54698250000007,48.26299899800006],[16.43804250000005,48.31626699800006],[16.205158,48.26206449800003],[16.18190050000004,48.17102999600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT121","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Mostviertel-Eisenwurzen","NUTS_NAME":"Mostviertel-Eisenwurzen"},"geometry":{"type":"Polygon","coordinates":[[[15.507681,48.31959849800006],[15.422403,48.21082249700004],[15.516368,48.12880349800008],[15.263455500000077,47.997678499000074],[15.335960500000056,47.89001549900007],[15.16132250000004,47.84915199800008],[15.217306,47.79602749900005],[15.08924950000005,47.741471],[14.738462,47.74872349900005],[14.709993,47.85064149900006],[14.771718,47.883778998000025],[14.493634,47.99779749600003],[14.456123,48.04303699800005],[14.479043500000046,48.10441449900003],[14.521179,48.23760949800004],[14.688370500000076,48.157290498000066],[14.828842500000064,48.189076998000075],[14.992047,48.25592899800006],[14.986449500000049,48.319307998000056],[14.943687,48.32364799900006],[14.964447,48.378512],[15.1244,48.329183998000076],[15.270585,48.373744498000065],[15.337208,48.34444549800003],[15.309930500000064,48.26916099700003],[15.507681,48.31959849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT122","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Niederösterreich-Süd","NUTS_NAME":"Niederösterreich-Süd"},"geometry":{"type":"Polygon","coordinates":[[[16.388891499000067,47.88160099600003],[16.377188499000056,47.82732149700007],[16.26904950000005,47.79640549800007],[16.372502,47.64248549800004],[16.281081,47.45513249900006],[16.17176950000004,47.42239999900005],[16.087398500000063,47.507157498000026],[15.847011,47.56778949800008],[15.821781,47.637955496000075],[15.725103,47.646455496000044],[15.719146,47.70988349900006],[15.611712,47.75108649800006],[15.374876999000037,47.824564],[15.217306,47.79602749900005],[15.16132250000004,47.84915199800008],[15.335960500000056,47.89001549900007],[15.491254999000034,47.97053199800007],[15.551513999000065,48.07030649600006],[15.915857500000072,48.088535497000066],[16.388891499000067,47.88160099600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT123","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Sankt Pölten","NUTS_NAME":"Sankt Pölten"},"geometry":{"type":"Polygon","coordinates":[[[15.995652500000062,48.145711997000035],[15.915857500000072,48.088535497000066],[15.551513999000065,48.07030649600006],[15.491254999000034,47.97053199800007],[15.335960500000056,47.89001549900007],[15.263455500000077,47.997678499000074],[15.516368,48.12880349800008],[15.422403,48.21082249700004],[15.507681,48.31959849800006],[15.731362500000046,48.38419999800004],[15.841757,48.368628],[15.760172,48.313855498000066],[15.885311,48.23163649900005],[15.957077499000036,48.259554500000036],[15.995652500000062,48.145711997000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT124","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Waldviertel","NUTS_NAME":"Waldviertel"},"geometry":{"type":"Polygon","coordinates":[[[15.542449500000032,48.90769699600003],[15.753633500000035,48.852178500000036],[15.887635,48.78510949900004],[15.815353500000072,48.692703],[15.94094,48.67173399600006],[15.908326500000044,48.59815149800005],[15.73824,48.570100498000045],[15.783852,48.48615049600005],[15.731362500000046,48.38419999800004],[15.507681,48.31959849800006],[15.309930500000064,48.26916099700003],[15.337208,48.34444549800003],[15.270585,48.373744498000065],[15.1244,48.329183998000076],[14.964447,48.378512],[14.873728500000027,48.459239998000044],[14.912456500000076,48.51396149800007],[14.691014,48.58430199700007],[14.808667,48.77879949900006],[14.979332,48.77235049800004],[14.990445,49.009649],[15.157331,48.991779500000064],[15.160247,48.94168649800008],[15.365882999000064,48.98170699900004],[15.542449500000032,48.90769699600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT125","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Weinviertel","NUTS_NAME":"Weinviertel"},"geometry":{"type":"Polygon","coordinates":[[[16.940278,48.617245498000045],[16.949778,48.535791998000036],[16.851106,48.438635],[16.410054500000058,48.50807049900004],[16.360824,48.59568999700008],[16.22964150000007,48.56988449900007],[16.12616950000006,48.46326799800005],[15.783852,48.48615049600005],[15.73824,48.570100498000045],[15.908326500000044,48.59815149800005],[15.94094,48.67173399600006],[15.815353500000072,48.692703],[15.887635,48.78510949900004],[15.753633500000035,48.852178500000036],[15.84331,48.873388498000054],[16.102884500000073,48.74539599900004],[16.648492,48.78520849800003],[16.90202350000004,48.71797799800004],[16.940278,48.617245498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT126","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Wiener Umland/Nordteil","NUTS_NAME":"Wiener Umland/Nordteil"},"geometry":{"type":"Polygon","coordinates":[[[16.851106,48.438635],[16.842372500000067,48.35332699800006],[16.957182,48.26347999600006],[16.976203,48.172244497000065],[16.788922,48.11510099800006],[16.576992500000074,48.14443649800006],[16.536381,48.17461149800005],[16.54698250000007,48.26299899800006],[16.43804250000005,48.31626699800006],[16.205158,48.26206449800003],[16.18190050000004,48.17102999600007],[15.995652500000062,48.145711997000035],[15.957077499000036,48.259554500000036],[15.885311,48.23163649900005],[15.760172,48.313855498000066],[15.841757,48.368628],[15.731362500000046,48.38419999800004],[15.783852,48.48615049600005],[16.12616950000006,48.46326799800005],[16.22964150000007,48.56988449900007],[16.360824,48.59568999700008],[16.410054500000058,48.50807049900004],[16.851106,48.438635]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT127","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Wiener Umland/Südteil","NUTS_NAME":"Wiener Umland/Südteil"},"geometry":{"type":"Polygon","coordinates":[[[17.066741,48.11868149800006],[16.971083500000077,48.10057199600004],[17.016295,48.07352049800005],[16.979235500000073,48.023454497000046],[16.889998,48.07463899800007],[16.7021,48.01165199800005],[16.58028,47.888607999000044],[16.486915500000066,47.94262049800005],[16.388891499000067,47.88160099600003],[15.915857500000072,48.088535497000066],[15.995652500000062,48.145711997000035],[16.18190050000004,48.17102999600007],[16.21809650000006,48.12806799600003],[16.576992500000074,48.14443649800006],[16.788922,48.11510099800006],[16.976203,48.172244497000065],[17.066741,48.11868149800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT13","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Wien","NUTS_NAME":"Wien"},"geometry":{"type":"Polygon","coordinates":[[[16.576992500000074,48.14443649800006],[16.21809650000006,48.12806799600003],[16.18190050000004,48.17102999600007],[16.205158,48.26206449800003],[16.43804250000005,48.31626699800006],[16.54698250000007,48.26299899800006],[16.536381,48.17461149800005],[16.576992500000074,48.14443649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT130","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Wien","NUTS_NAME":"Wien"},"geometry":{"type":"Polygon","coordinates":[[[16.576992500000074,48.14443649800006],[16.21809650000006,48.12806799600003],[16.18190050000004,48.17102999600007],[16.205158,48.26206449800003],[16.43804250000005,48.31626699800006],[16.54698250000007,48.26299899800006],[16.536381,48.17461149800005],[16.576992500000074,48.14443649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT2","LEVL_CODE":1,"CNTR_CODE":"AT","NAME_LATN":"SÜDÖSTERREICH","NUTS_NAME":"SÜDÖSTERREICH"},"geometry":{"type":"Polygon","coordinates":[[[15.611712,47.75108649800006],[15.719146,47.70988349900006],[15.725103,47.646455496000044],[15.821781,47.637955496000075],[15.847011,47.56778949800008],[16.087398500000063,47.507157498000026],[16.17176950000004,47.42239999900005],[16.01515850000004,47.36712749800006],[16.113463,47.16183849600003],[16.086508,47.074209],[16.12171950000004,46.99066649900004],[15.996236,46.835398499000064],[15.984746500000028,46.752207],[16.038086,46.65614549600008],[15.786422,46.70746949900007],[15.649988,46.70575699900007],[15.508846500000061,46.616227998000056],[15.40197949900005,46.65354849800008],[15.065120500000035,46.65211249600003],[14.879666,46.61124799800007],[14.818606,46.50938299900008],[14.674580500000047,46.450687499000026],[14.565175500000066,46.37245349600005],[14.43450050000007,46.44294350000007],[13.714184999000054,46.522703499000045],[13.504249500000071,46.566303997000034],[12.731392999000036,46.63428799900004],[12.690635,46.656972],[12.724958,46.74344450000007],[12.965998,46.78955399800003],[12.841158,46.86097949800006],[12.724635999000043,46.995786499000076],[12.757816500000047,47.04582399800006],[12.656833500000062,47.09951749600003],[12.730896500000028,47.13099249900006],[13.073021500000038,47.01341099900003],[13.354519500000038,47.097104],[13.68653850000004,47.03981950000008],[13.784528499000032,46.94388249600007],[13.90357350000005,47.062910498000065],[13.883161500000028,47.109582998000064],[13.996124,47.13281049600005],[13.864178999000046,47.252169999000046],[13.607597,47.28356499800003],[13.56334350000003,47.42046250000004],[13.585662999000022,47.47479949800004],[13.722560500000043,47.46211150000005],[13.755931,47.55272999600004],[13.694059,47.62321149700006],[13.777418,47.71452899800005],[14.010598,47.70089199700004],[14.050777499000048,47.61715699700005],[14.20579250000003,47.65253799800007],[14.326396500000044,47.60755449800007],[14.738462,47.74872349900005],[15.08924950000005,47.741471],[15.217306,47.79602749900005],[15.374876999000037,47.824564],[15.611712,47.75108649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT21","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Kärnten","NUTS_NAME":"Kärnten"},"geometry":{"type":"Polygon","coordinates":[[[12.730896500000028,47.13099249900006],[13.073021500000038,47.01341099900003],[13.354519500000038,47.097104],[13.68653850000004,47.03981950000008],[13.784528499000032,46.94388249600007],[13.946724,46.94062999800008],[14.132562500000063,47.06234199700003],[14.396193,46.98069649800004],[14.844475,47.048099998000055],[15.027008,46.90973499800003],[14.976265,46.78478450000006],[15.030154,46.76978949800008],[15.065120500000035,46.65211249600003],[14.879666,46.61124799800007],[14.818606,46.50938299900008],[14.674580500000047,46.450687499000026],[14.565175500000066,46.37245349600005],[14.43450050000007,46.44294350000007],[13.714184999000054,46.522703499000045],[13.504249500000071,46.566303997000034],[12.731392999000036,46.63428799900004],[12.690635,46.656972],[12.724958,46.74344450000007],[12.965998,46.78955399800003],[12.841158,46.86097949800006],[12.724635999000043,46.995786499000076],[12.757816500000047,47.04582399800006],[12.656833500000062,47.09951749600003],[12.730896500000028,47.13099249900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT211","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Klagenfurt-Villach","NUTS_NAME":"Klagenfurt-Villach"},"geometry":{"type":"Polygon","coordinates":[[[14.23342450000007,46.70016749800004],[14.42721750000004,46.73376099800004],[14.54289,46.65617899800003],[14.456691,46.56429799600005],[14.48709,46.50598099800004],[14.43450050000007,46.44294350000007],[13.714184999000054,46.522703499000045],[13.504249500000071,46.566303997000034],[13.577491,46.65325499800008],[13.43275999900004,46.65716099800005],[13.408853,46.69774349800008],[13.454853,46.76999249900007],[13.941092500000025,46.75515949600003],[13.960948500000029,46.715656996000064],[13.920257500000048,46.69577799600006],[13.954916500000024,46.655133497000065],[14.23342450000007,46.70016749800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT212","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Oberkärnten","NUTS_NAME":"Oberkärnten"},"geometry":{"type":"Polygon","coordinates":[[[13.354519500000038,47.097104],[13.68653850000004,47.03981950000008],[13.784528499000032,46.94388249600007],[13.946724,46.94062999800008],[14.012552500000027,46.85570049700004],[14.135860999000045,46.85232949700003],[14.117759500000034,46.81616899800008],[14.207835500000044,46.77883799800003],[14.187144,46.748058498000034],[14.258765500000038,46.73786699900006],[14.23342450000007,46.70016749800004],[13.954916500000024,46.655133497000065],[13.920257500000048,46.69577799600006],[13.960948500000029,46.715656996000064],[13.941092500000025,46.75515949600003],[13.454853,46.76999249900007],[13.408853,46.69774349800008],[13.43275999900004,46.65716099800005],[13.577491,46.65325499800008],[13.504249500000071,46.566303997000034],[12.731392999000036,46.63428799900004],[12.690635,46.656972],[12.724958,46.74344450000007],[12.965998,46.78955399800003],[12.841158,46.86097949800006],[12.724635999000043,46.995786499000076],[12.757816500000047,47.04582399800006],[12.656833500000062,47.09951749600003],[12.730896500000028,47.13099249900006],[13.073021500000038,47.01341099900003],[13.354519500000038,47.097104]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT213","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Unterkärnten","NUTS_NAME":"Unterkärnten"},"geometry":{"type":"Polygon","coordinates":[[[14.844475,47.048099998000055],[15.027008,46.90973499800003],[14.976265,46.78478450000006],[15.030154,46.76978949800008],[15.065120500000035,46.65211249600003],[14.879666,46.61124799800007],[14.818606,46.50938299900008],[14.674580500000047,46.450687499000026],[14.565175500000066,46.37245349600005],[14.43450050000007,46.44294350000007],[14.48709,46.50598099800004],[14.456691,46.56429799600005],[14.54289,46.65617899800003],[14.42721750000004,46.73376099800004],[14.23342450000007,46.70016749800004],[14.258765500000038,46.73786699900006],[14.187144,46.748058498000034],[14.207835500000044,46.77883799800003],[14.117759500000034,46.81616899800008],[14.135860999000045,46.85232949700003],[14.012552500000027,46.85570049700004],[13.946724,46.94062999800008],[14.132562500000063,47.06234199700003],[14.396193,46.98069649800004],[14.844475,47.048099998000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT341","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Bludenz-Bregenzer Wald","NUTS_NAME":"Bludenz-Bregenzer Wald"},"geometry":{"type":"Polygon","coordinates":[[[9.999526,47.48301699600006],[10.09147050000007,47.45907599700007],[10.09985049900007,47.35476149700003],[10.23393,47.38328599700003],[10.178353,47.270113998000056],[10.212995,47.157583998000064],[10.134231500000055,47.083864996000045],[10.159806,46.98375099700007],[10.098220500000025,46.92735799800005],[10.144974500000046,46.85100949800005],[9.876148500000056,46.93463550000007],[9.879422500000032,47.01956299600005],[9.607078,47.06077449900005],[9.620580500000074,47.15164549900004],[9.640375,47.21488449900005],[9.708096,47.194799499000055],[9.848613500000056,47.29248549700003],[9.816834,47.31096699600005],[9.823090500000035,47.43164699700003],[9.89238350000005,47.444287],[9.854281,47.47491099800004],[9.999526,47.48301699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT342","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Rheintal-Bodenseegebiet","NUTS_NAME":"Rheintal-Bodenseegebiet"},"geometry":{"type":"Polygon","coordinates":[[[9.967813500000034,47.54624049600005],[9.999526,47.48301699600006],[9.854281,47.47491099800004],[9.89238350000005,47.444287],[9.823090500000035,47.43164699700003],[9.816834,47.31096699600005],[9.848613500000056,47.29248549700003],[9.708096,47.194799499000055],[9.640375,47.21488449900005],[9.620580500000074,47.15164549900004],[9.530749,47.270581],[9.673370500000033,47.38151050000005],[9.55872050000005,47.54189299800004],[9.736566500000038,47.534906498000055],[9.765470500000049,47.588856],[9.967813500000034,47.54624049600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE","LEVL_CODE":0,"CNTR_CODE":"BE","NAME_LATN":"BELGIQUE-BELGIË","NUTS_NAME":"BELGIQUE-BELGIË"},"geometry":{"type":"Polygon","coordinates":[[[6.020998999000028,50.75429549900008],[6.268615500000067,50.625980999000035],[6.189312500000028,50.566094],[6.192200499000023,50.52105549800007],[6.20628750000003,50.521302],[6.315556,50.49704249800004],[6.374430500000074,50.45226950000006],[6.342218500000058,50.38000550000004],[6.405028500000071,50.323308498000074],[6.176784500000053,50.23586299800007],[6.137662500000033,50.129951498000025],[6.02489950000006,50.18277949700007],[5.895269,50.112216498000066],[5.746319,49.85359499900005],[5.910688,49.662388],[5.818117,49.54631049900007],[5.734555999000065,49.545690500000035],[5.470882999000025,49.49723799800006],[5.393511,49.61711099900003],[5.153738500000031,49.717925999000045],[4.969431,49.80182599800003],[4.851578,49.793255],[4.889914,49.90920999800005],[4.793194,49.981999],[4.896794,50.13742049800004],[4.796697,50.14867799800004],[4.680489,50.06813850000003],[4.694556,49.99650949900007],[4.432494,49.94161599600005],[4.233133,49.95782849700004],[4.140853,49.978759996000065],[4.231028,50.071983498000066],[4.126875,50.13492199800004],[4.221769,50.255832500000054],[4.027774500000021,50.35833049800004],[3.710389,50.303165499000045],[3.65551,50.46173549800005],[3.615081500000031,50.490399],[3.286492,50.52756899700006],[3.245294,50.71300949700003],[3.176996,50.75616449700004],[3.098481,50.77901899900007],[3.018708500000059,50.773532998000064],[2.863276,50.708343496000055],[2.63495,50.81274399900008],[2.607036,50.91268949800008],[2.632617,50.94594549900006],[2.546011,51.08938199700003],[2.749051499000075,51.16176999800007],[3.110688500000037,51.31225950000004],[3.36721650000004,51.368134498000074],[3.380661,51.274299496000026],[3.621433500000023,51.292781998000066],[3.856339500000047,51.21105599900005],[3.977666500000055,51.22513199800005],[4.234817500000077,51.348253999000065],[4.242049,51.35396699800003],[4.243669500000067,51.37472949800008],[4.279565,51.37601749600003],[4.431206,51.363845998000045],[4.379466500000035,51.44664],[4.548236,51.473228497000036],[4.535450500000024,51.422889499000064],[4.669544,51.42638399800006],[4.759926,51.50246449700006],[4.841390500000045,51.48071299900005],[4.773122500000056,51.413783496000065],[4.927639501000044,51.39542449600003],[5.037678500000027,51.47936999800004],[5.10218,51.42900499600006],[5.071045,51.39342099800007],[5.237716500000033,51.261600498000064],[5.515765,51.29532449800007],[5.566283500000054,51.22083649700005],[5.855658500000061,51.14459949900004],[5.798274,51.05985349800005],[5.766149,51.009235499000056],[5.758272499000043,50.95479499900006],[5.644038500000022,50.87138349800006],[5.687622,50.811923998000054],[5.682000499000026,50.757446497000046],[5.892073,50.75523749800004],[6.020998999000028,50.75429549900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE1","LEVL_CODE":1,"CNTR_CODE":"BE","NAME_LATN":"RÉGION DE BRUXELLES-CAPITALE / BRUSSELS HOOFDSTEDELIJK GEWEST","NUTS_NAME":"RÉGION DE BRUXELLES-CAPITALE / BRUSSELS HOOFDSTEDELIJK GEWEST"},"geometry":{"type":"Polygon","coordinates":[[[4.476783500000067,50.820377499000074],[4.480476500000066,50.794257998000035],[4.245566500000052,50.817626999000026],[4.415738,50.90616999800005],[4.476783500000067,50.820377499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE10","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Région de Bruxelles-Capitale / Brussels Hoofdstedelijk Gewest","NUTS_NAME":"Région de Bruxelles-Capitale / Brussels Hoofdstedelijk Gewest"},"geometry":{"type":"Polygon","coordinates":[[[4.476783500000067,50.820377499000074],[4.480476500000066,50.794257998000035],[4.245566500000052,50.817626999000026],[4.415738,50.90616999800005],[4.476783500000067,50.820377499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE100","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. de Bruxelles-Capitale / Arr. van Brussel-Hoofdstad","NUTS_NAME":"Arr. de Bruxelles-Capitale / Arr. van Brussel-Hoofdstad"},"geometry":{"type":"Polygon","coordinates":[[[4.476783500000067,50.820377499000074],[4.480476500000066,50.794257998000035],[4.245566500000052,50.817626999000026],[4.415738,50.90616999800005],[4.476783500000067,50.820377499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE2","LEVL_CODE":1,"CNTR_CODE":"BE","NAME_LATN":"VLAAMS GEWEST","NUTS_NAME":"VLAAMS GEWEST"},"geometry":{"type":"Polygon","coordinates":[[[3.018708500000059,50.773532998000064],[2.945734077000054,50.79394636200004],[2.842169,50.75140399900005],[2.863276,50.708343496000055],[2.63495,50.81274399900008],[2.607036,50.91268949800008],[2.632617,50.94594549900006],[2.546011,51.08938199700003],[2.749051499000075,51.16176999800007],[3.110688500000037,51.31225950000004],[3.36721650000004,51.368134498000074],[3.380661,51.274299496000026],[3.621433500000023,51.292781998000066],[3.856339500000047,51.21105599900005],[3.977666500000055,51.22513199800005],[4.234817500000077,51.348253999000065],[4.242049,51.35396699800003],[4.243669500000067,51.37472949800008],[4.279565,51.37601749600003],[4.431206,51.363845998000045],[4.379466500000035,51.44664],[4.548236,51.473228497000036],[4.535450500000024,51.422889499000064],[4.669544,51.42638399800006],[4.759926,51.50246449700006],[4.841390500000045,51.48071299900005],[4.773122500000056,51.413783496000065],[4.927639501000044,51.39542449600003],[5.037678500000027,51.47936999800004],[5.10218,51.42900499600006],[5.071045,51.39342099800007],[5.237716500000033,51.261600498000064],[5.515765,51.29532449800007],[5.566283500000054,51.22083649700005],[5.855658500000061,51.14459949900004],[5.798274,51.05985349800005],[5.766149,51.009235499000056],[5.758272499000043,50.95479499900006],[5.644038500000022,50.87138349800006],[5.687622,50.811923998000054],[5.431686500000069,50.71980299900008],[5.236892,50.727146],[5.101852500000064,50.70865249800005],[5.019701,50.750973],[4.597269,50.76353450000005],[4.100483,50.70595549700005],[3.895736499000066,50.73294449800005],[3.815390500000035,50.75072499600003],[3.775359500000036,50.74772999800007],[3.541272500000048,50.733700498000076],[3.459731,50.76596850000004],[3.324118,50.722308998000074],[3.176996,50.75616449700004],[3.098481,50.77901899900007],[3.018708500000059,50.773532998000064]],[[4.476783500000067,50.820377499000074],[4.415738,50.90616999800005],[4.245566500000052,50.817626999000026],[4.480476500000066,50.794257998000035],[4.476783500000067,50.820377499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE21","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Antwerpen","NUTS_NAME":"Prov. Antwerpen"},"geometry":{"type":"Polygon","coordinates":[[[4.841390500000045,51.48071299900005],[4.773122500000056,51.413783496000065],[4.927639501000044,51.39542449600003],[5.037678500000027,51.47936999800004],[5.10218,51.42900499600006],[5.071045,51.39342099800007],[5.237716500000033,51.261600498000064],[5.21449750000005,51.22080599800006],[5.261073499000076,51.146964998000044],[4.981574500000022,51.03488549800005],[4.789319,51.038928998000074],[4.528663,50.992264498000054],[4.240999,51.03686899600007],[4.175792,51.10121150000003],[4.311167500000067,51.12614799800008],[4.329453500000056,51.18516549800006],[4.242049,51.35396699800003],[4.243669500000067,51.37472949800008],[4.279565,51.37601749600003],[4.431206,51.363845998000045],[4.379466500000035,51.44664],[4.548236,51.473228497000036],[4.535450500000024,51.422889499000064],[4.669544,51.42638399800006],[4.759926,51.50246449700006],[4.841390500000045,51.48071299900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE211","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Antwerpen","NUTS_NAME":"Arr. Antwerpen"},"geometry":{"type":"Polygon","coordinates":[[[4.669544,51.42638399800006],[4.775887,51.312553500000035],[4.694647,51.178406],[4.527603500000055,51.15838599600005],[4.47052,51.073035997000034],[4.311167500000067,51.12614799800008],[4.329453500000056,51.18516549800006],[4.242049,51.35396699800003],[4.243669500000067,51.37472949800008],[4.279565,51.37601749600003],[4.431206,51.363845998000045],[4.379466500000035,51.44664],[4.548236,51.473228497000036],[4.535450500000024,51.422889499000064],[4.669544,51.42638399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE212","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Mechelen","NUTS_NAME":"Arr. Mechelen"},"geometry":{"type":"Polygon","coordinates":[[[4.789319,51.038928998000074],[4.528663,50.992264498000054],[4.240999,51.03686899600007],[4.175792,51.10121150000003],[4.311167500000067,51.12614799800008],[4.47052,51.073035997000034],[4.527603500000055,51.15838599600005],[4.694647,51.178406],[4.816503,51.10392],[4.770009500000072,51.071735500000045],[4.789319,51.038928998000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE213","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Turnhout","NUTS_NAME":"Arr. Turnhout"},"geometry":{"type":"Polygon","coordinates":[[[5.10218,51.42900499600006],[5.071045,51.39342099800007],[5.237716500000033,51.261600498000064],[5.21449750000005,51.22080599800006],[5.261073499000076,51.146964998000044],[4.981574500000022,51.03488549800005],[4.789319,51.038928998000074],[4.770009500000072,51.071735500000045],[4.816503,51.10392],[4.694647,51.178406],[4.775887,51.312553500000035],[4.669544,51.42638399800006],[4.759926,51.50246449700006],[4.841390500000045,51.48071299900005],[4.773122500000056,51.413783496000065],[4.927639501000044,51.39542449600003],[5.037678500000027,51.47936999800004],[5.10218,51.42900499600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE22","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Limburg (BE)","NUTS_NAME":"Prov. Limburg (BE)"},"geometry":{"type":"Polygon","coordinates":[[[5.687622,50.811923998000054],[5.431686500000069,50.71980299900008],[5.236892,50.727146],[5.101852500000064,50.70865249800005],[5.187943,50.902702496000074],[5.042069,50.94281399800008],[5.159698,51.02414699800005],[4.981574500000022,51.03488549800005],[5.261073499000076,51.146964998000044],[5.21449750000005,51.22080599800006],[5.237716500000033,51.261600498000064],[5.515765,51.29532449800007],[5.566283500000054,51.22083649700005],[5.855658500000061,51.14459949900004],[5.798274,51.05985349800005],[5.766149,51.009235499000056],[5.758272499000043,50.95479499900006],[5.644038500000022,50.87138349800006],[5.687622,50.811923998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE221","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Hasselt","NUTS_NAME":"Arr. Hasselt"},"geometry":{"type":"Polygon","coordinates":[[[5.622395,50.999236996000036],[5.613771,50.89797599900004],[5.250804500000072,50.901511998000046],[5.286719500000061,50.78288699900003],[5.236892,50.727146],[5.101852500000064,50.70865249800005],[5.187943,50.902702496000074],[5.042069,50.94281399800008],[5.159698,51.02414699800005],[4.981574500000022,51.03488549800005],[5.261073499000076,51.146964998000044],[5.339950500000043,51.09271599700003],[5.347893500000055,51.012170998000045],[5.600551,51.06433099900005],[5.622395,50.999236996000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE222","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Maaseik","NUTS_NAME":"Arr. Maaseik"},"geometry":{"type":"Polygon","coordinates":[[[5.566283500000054,51.22083649700005],[5.855658500000061,51.14459949900004],[5.798274,51.05985349800005],[5.766149,51.009235499000056],[5.622395,50.999236996000036],[5.600551,51.06433099900005],[5.347893500000055,51.012170998000045],[5.339950500000043,51.09271599700003],[5.261073499000076,51.146964998000044],[5.21449750000005,51.22080599800006],[5.237716500000033,51.261600498000064],[5.515765,51.29532449800007],[5.566283500000054,51.22083649700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE223","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Tongeren","NUTS_NAME":"Arr. Tongeren"},"geometry":{"type":"Polygon","coordinates":[[[5.758272499000043,50.95479499900006],[5.644038500000022,50.87138349800006],[5.687622,50.811923998000054],[5.431686500000069,50.71980299900008],[5.236892,50.727146],[5.286719500000061,50.78288699900003],[5.250804500000072,50.901511998000046],[5.613771,50.89797599900004],[5.622395,50.999236996000036],[5.766149,51.009235499000056],[5.758272499000043,50.95479499900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE23","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Oost-Vlaanderen","NUTS_NAME":"Prov. Oost-Vlaanderen"},"geometry":{"type":"Polygon","coordinates":[[[4.242049,51.35396699800003],[4.329453500000056,51.18516549800006],[4.311167500000067,51.12614799800008],[4.175792,51.10121150000003],[4.240999,51.03686899600007],[4.147153,50.96939849800003],[4.052910500000053,50.78412250000008],[3.895736499000066,50.73294449800005],[3.815390500000035,50.75072499600003],[3.775359500000036,50.74772999800007],[3.541272500000048,50.733700498000076],[3.459731,50.76596850000004],[3.523682500000064,50.80499249800005],[3.466746500000056,50.90209649900004],[3.419824,50.91093499800007],[3.461246500000073,50.94248199800006],[3.446067,51.04704649700005],[3.331306,51.09887299900004],[3.41093,51.15988899600006],[3.380661,51.274299496000026],[3.621433500000023,51.292781998000066],[3.856339500000047,51.21105599900005],[3.977666500000055,51.22513199800005],[4.234817500000077,51.348253999000065],[4.242049,51.35396699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE231","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Aalst","NUTS_NAME":"Arr. Aalst"},"geometry":{"type":"Polygon","coordinates":[[[4.147153,50.96939849800003],[4.052910500000053,50.78412250000008],[3.895736499000066,50.73294449800005],[3.815390500000035,50.75072499600003],[3.872793,50.814674498000045],[3.787503500000071,50.81449499800004],[3.746936,50.907943498000066],[3.84770099900004,50.952755],[4.147153,50.96939849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE232","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Dendermonde","NUTS_NAME":"Arr. Dendermonde"},"geometry":{"type":"Polygon","coordinates":[[[4.175792,51.10121150000003],[4.240999,51.03686899600007],[4.147153,50.96939849800003],[3.84770099900004,50.952755],[3.82314150000002,51.04477299700005],[3.923106500000074,51.07601149900006],[4.033947,51.095031498000026],[4.012327500000026,51.13417050000004],[4.175792,51.10121150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE233","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Eeklo","NUTS_NAME":"Arr. Eeklo"},"geometry":{"type":"Polygon","coordinates":[[[3.856339500000047,51.21105599900005],[3.41093,51.15988899600006],[3.380661,51.274299496000026],[3.621433500000023,51.292781998000066],[3.856339500000047,51.21105599900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE234","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Gent","NUTS_NAME":"Arr. Gent"},"geometry":{"type":"Polygon","coordinates":[[[3.923106500000074,51.07601149900006],[3.82314150000002,51.04477299700005],[3.84770099900004,50.952755],[3.746936,50.907943498000066],[3.466746500000056,50.90209649900004],[3.419824,50.91093499800007],[3.461246500000073,50.94248199800006],[3.446067,51.04704649700005],[3.331306,51.09887299900004],[3.41093,51.15988899600006],[3.856339500000047,51.21105599900005],[3.977666500000055,51.22513199800005],[3.978867,51.16463850000008],[3.897331,51.14720899900004],[3.923106500000074,51.07601149900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE235","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Oudenaarde","NUTS_NAME":"Arr. Oudenaarde"},"geometry":{"type":"Polygon","coordinates":[[[3.746936,50.907943498000066],[3.787503500000071,50.81449499800004],[3.872793,50.814674498000045],[3.815390500000035,50.75072499600003],[3.775359500000036,50.74772999800007],[3.541272500000048,50.733700498000076],[3.459731,50.76596850000004],[3.523682500000064,50.80499249800005],[3.466746500000056,50.90209649900004],[3.746936,50.907943498000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE236","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Sint-Niklaas","NUTS_NAME":"Arr. Sint-Niklaas"},"geometry":{"type":"Polygon","coordinates":[[[4.311167500000067,51.12614799800008],[4.175792,51.10121150000003],[4.012327500000026,51.13417050000004],[4.033947,51.095031498000026],[3.923106500000074,51.07601149900006],[3.897331,51.14720899900004],[3.978867,51.16463850000008],[3.977666500000055,51.22513199800005],[4.234817500000077,51.348253999000065],[4.242049,51.35396699800003],[4.329453500000056,51.18516549800006],[4.311167500000067,51.12614799800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE24","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Vlaams-Brabant","NUTS_NAME":"Prov. Vlaams-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[5.159698,51.02414699800005],[5.042069,50.94281399800008],[5.187943,50.902702496000074],[5.101852500000064,50.70865249800005],[5.019701,50.750973],[4.597269,50.76353450000005],[4.100483,50.70595549700005],[3.895736499000066,50.73294449800005],[4.052910500000053,50.78412250000008],[4.147153,50.96939849800003],[4.240999,51.03686899600007],[4.528663,50.992264498000054],[4.789319,51.038928998000074],[4.981574500000022,51.03488549800005],[5.159698,51.02414699800005]],[[4.245566500000052,50.817626999000026],[4.480476500000066,50.794257998000035],[4.476783500000067,50.820377499000074],[4.415738,50.90616999800005],[4.245566500000052,50.817626999000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE241","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Halle-Vilvoorde","NUTS_NAME":"Arr. Halle-Vilvoorde"},"geometry":{"type":"Polygon","coordinates":[[[4.528663,50.992264498000054],[4.617923,50.93556599800007],[4.476783500000067,50.820377499000074],[4.415738,50.90616999800005],[4.245566500000052,50.817626999000026],[4.480476500000066,50.794257998000035],[4.597269,50.76353450000005],[4.100483,50.70595549700005],[3.895736499000066,50.73294449800005],[4.052910500000053,50.78412250000008],[4.147153,50.96939849800003],[4.240999,51.03686899600007],[4.528663,50.992264498000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE242","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Leuven","NUTS_NAME":"Arr. Leuven"},"geometry":{"type":"Polygon","coordinates":[[[4.981574500000022,51.03488549800005],[5.159698,51.02414699800005],[5.042069,50.94281399800008],[5.187943,50.902702496000074],[5.101852500000064,50.70865249800005],[5.019701,50.750973],[4.597269,50.76353450000005],[4.480476500000066,50.794257998000035],[4.476783500000067,50.820377499000074],[4.617923,50.93556599800007],[4.528663,50.992264498000054],[4.789319,51.038928998000074],[4.981574500000022,51.03488549800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE25","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. West-Vlaanderen","NUTS_NAME":"Prov. West-Vlaanderen"},"geometry":{"type":"Polygon","coordinates":[[[3.018708500000059,50.773532998000064],[2.945734077000054,50.79394636200004],[2.842169,50.75140399900005],[2.863276,50.708343496000055],[2.63495,50.81274399900008],[2.607036,50.91268949800008],[2.632617,50.94594549900006],[2.546011,51.08938199700003],[2.749051499000075,51.16176999800007],[3.110688500000037,51.31225950000004],[3.36721650000004,51.368134498000074],[3.380661,51.274299496000026],[3.41093,51.15988899600006],[3.331306,51.09887299900004],[3.446067,51.04704649700005],[3.461246500000073,50.94248199800006],[3.419824,50.91093499800007],[3.466746500000056,50.90209649900004],[3.523682500000064,50.80499249800005],[3.459731,50.76596850000004],[3.324118,50.722308998000074],[3.176996,50.75616449700004],[3.098481,50.77901899900007],[3.018708500000059,50.773532998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE251","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Brugge","NUTS_NAME":"Arr. Brugge"},"geometry":{"type":"Polygon","coordinates":[[[3.380661,51.274299496000026],[3.41093,51.15988899600006],[3.331306,51.09887299900004],[3.163075,51.06520099800008],[3.069538500000021,51.01546499700004],[3.043536,51.06066149700007],[3.046618,51.21969999700008],[3.110688500000037,51.31225950000004],[3.36721650000004,51.368134498000074],[3.380661,51.274299496000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE252","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Diksmuide","NUTS_NAME":"Arr. Diksmuide"},"geometry":{"type":"Polygon","coordinates":[[[3.043536,51.06066149700007],[3.069538500000021,51.01546499700004],[2.959001500000056,50.953148],[2.810093,50.90510549600003],[2.719744500000047,50.94966899800005],[2.76616,51.086745998000026],[2.820953,51.1026],[2.982092,51.12870399800005],[3.043536,51.06066149700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE253","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Ieper","NUTS_NAME":"Arr. Ieper"},"geometry":{"type":"Polygon","coordinates":[[[3.018708500000059,50.773532998000064],[2.945734077000054,50.79394636200004],[2.842169,50.75140399900005],[2.863276,50.708343496000055],[2.63495,50.81274399900008],[2.607036,50.91268949800008],[2.719744500000047,50.94966899800005],[2.810093,50.90510549600003],[2.959001500000056,50.953148],[3.061697500000037,50.92371749800003],[3.028050500000063,50.88455199800006],[3.102455,50.83807349800003],[3.098481,50.77901899900007],[3.018708500000059,50.773532998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE254","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Kortrijk","NUTS_NAME":"Arr. Kortrijk"},"geometry":{"type":"Polygon","coordinates":[[[3.466746500000056,50.90209649900004],[3.523682500000064,50.80499249800005],[3.459731,50.76596850000004],[3.324118,50.722308998000074],[3.176996,50.75616449700004],[3.098481,50.77901899900007],[3.102455,50.83807349800003],[3.28706550000004,50.90438449800007],[3.419824,50.91093499800007],[3.466746500000056,50.90209649900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE255","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Oostende","NUTS_NAME":"Arr. Oostende"},"geometry":{"type":"Polygon","coordinates":[[[3.043536,51.06066149700007],[2.982092,51.12870399800005],[2.820953,51.1026],[2.749051499000075,51.16176999800007],[3.110688500000037,51.31225950000004],[3.046618,51.21969999700008],[3.043536,51.06066149700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE256","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Roeselare","NUTS_NAME":"Arr. Roeselare"},"geometry":{"type":"Polygon","coordinates":[[[3.28706550000004,50.90438449800007],[3.102455,50.83807349800003],[3.028050500000063,50.88455199800006],[3.061697500000037,50.92371749800003],[2.959001500000056,50.953148],[3.069538500000021,51.01546499700004],[3.163075,51.06520099800008],[3.171675,50.952518498000074],[3.28706550000004,50.90438449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE257","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Tielt","NUTS_NAME":"Arr. Tielt"},"geometry":{"type":"Polygon","coordinates":[[[3.419824,50.91093499800007],[3.28706550000004,50.90438449800007],[3.171675,50.952518498000074],[3.163075,51.06520099800008],[3.331306,51.09887299900004],[3.446067,51.04704649700005],[3.461246500000073,50.94248199800006],[3.419824,50.91093499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE258","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Veurne","NUTS_NAME":"Arr. Veurne"},"geometry":{"type":"Polygon","coordinates":[[[2.820953,51.1026],[2.76616,51.086745998000026],[2.719744500000047,50.94966899800005],[2.607036,50.91268949800008],[2.632617,50.94594549900006],[2.546011,51.08938199700003],[2.749051499000075,51.16176999800007],[2.820953,51.1026]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE3","LEVL_CODE":1,"CNTR_CODE":"BE","NAME_LATN":"RÉGION WALLONNE","NUTS_NAME":"RÉGION WALLONNE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[6.020998999000028,50.75429549900008],[6.268615500000067,50.625980999000035],[6.189312500000028,50.566094],[6.192200499000023,50.52105549800007],[6.20628750000003,50.521302],[6.315556,50.49704249800004],[6.374430500000074,50.45226950000006],[6.342218500000058,50.38000550000004],[6.405028500000071,50.323308498000074],[6.176784500000053,50.23586299800007],[6.137662500000033,50.129951498000025],[6.02489950000006,50.18277949700007],[5.895269,50.112216498000066],[5.746319,49.85359499900005],[5.910688,49.662388],[5.818117,49.54631049900007],[5.734555999000065,49.545690500000035],[5.470882999000025,49.49723799800006],[5.393511,49.61711099900003],[5.153738500000031,49.717925999000045],[4.969431,49.80182599800003],[4.851578,49.793255],[4.889914,49.90920999800005],[4.793194,49.981999],[4.896794,50.13742049800004],[4.796697,50.14867799800004],[4.680489,50.06813850000003],[4.694556,49.99650949900007],[4.432494,49.94161599600005],[4.233133,49.95782849700004],[4.140853,49.978759996000065],[4.231028,50.071983498000066],[4.126875,50.13492199800004],[4.221769,50.255832500000054],[4.027774500000021,50.35833049800004],[3.710389,50.303165499000045],[3.65551,50.46173549800005],[3.615081500000031,50.490399],[3.286492,50.52756899700006],[3.245294,50.71300949700003],[3.176996,50.75616449700004],[3.324118,50.722308998000074],[3.459731,50.76596850000004],[3.541272500000048,50.733700498000076],[3.775359500000036,50.74772999800007],[3.815390500000035,50.75072499600003],[3.895736499000066,50.73294449800005],[4.100483,50.70595549700005],[4.597269,50.76353450000005],[5.019701,50.750973],[5.101852500000064,50.70865249800005],[5.236892,50.727146],[5.431686500000069,50.71980299900008],[5.687622,50.811923998000054],[5.682000499000026,50.757446497000046],[5.892073,50.75523749800004],[6.020998999000028,50.75429549900008]]],[[[3.018708500000059,50.773532998000064],[2.863276,50.708343496000055],[2.842169,50.75140399900005],[2.945734077000054,50.79394636200004],[3.018708500000059,50.773532998000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE31","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Brabant Wallon","NUTS_NAME":"Prov. Brabant Wallon"},"geometry":{"type":"Polygon","coordinates":[[[5.019701,50.750973],[4.983341,50.64224249800003],[4.577537999000072,50.54228599800007],[4.247219500000028,50.596118998000065],[4.177351,50.66335299700006],[4.106683500000031,50.63327449700006],[4.100483,50.70595549700005],[4.597269,50.76353450000005],[5.019701,50.750973]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE310","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Nivelles","NUTS_NAME":"Arr. Nivelles"},"geometry":{"type":"Polygon","coordinates":[[[5.019701,50.750973],[4.983341,50.64224249800003],[4.577537999000072,50.54228599800007],[4.247219500000028,50.596118998000065],[4.177351,50.66335299700006],[4.106683500000031,50.63327449700006],[4.100483,50.70595549700005],[4.597269,50.76353450000005],[5.019701,50.750973]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE32","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Hainaut","NUTS_NAME":"Prov. Hainaut"},"geometry":{"type":"MultiPolygon","coordinates":[[[[3.459731,50.76596850000004],[3.541272500000048,50.733700498000076],[3.775359500000036,50.74772999800007],[3.815390500000035,50.75072499600003],[3.895736499000066,50.73294449800005],[4.100483,50.70595549700005],[4.106683500000031,50.63327449700006],[4.177351,50.66335299700006],[4.247219500000028,50.596118998000065],[4.577537999000072,50.54228599800007],[4.588662500000055,50.321315998000046],[4.474986999000066,50.32761],[4.286857,50.25579049700008],[4.401648500000022,50.23227299900003],[4.390042,50.050178998000035],[4.432494,49.94161599600005],[4.233133,49.95782849700004],[4.140853,49.978759996000065],[4.231028,50.071983498000066],[4.126875,50.13492199800004],[4.221769,50.255832500000054],[4.027774500000021,50.35833049800004],[3.710389,50.303165499000045],[3.65551,50.46173549800005],[3.615081500000031,50.490399],[3.286492,50.52756899700006],[3.245294,50.71300949700003],[3.176996,50.75616449700004],[3.324118,50.722308998000074],[3.459731,50.76596850000004]]],[[[3.018708500000059,50.773532998000064],[2.863276,50.708343496000055],[2.842169,50.75140399900005],[2.945734077000054,50.79394636200004],[3.018708500000059,50.773532998000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE321","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Ath","NUTS_NAME":"Arr. Ath"},"geometry":{"type":"Polygon","coordinates":[[[3.775359500000036,50.74772999800007],[3.744744500000024,50.72942750000004],[3.904207,50.61555499800005],[3.74291,50.52555849700008],[3.752057500000035,50.44591149900003],[3.65551,50.46173549800005],[3.615081500000031,50.490399],[3.615301499000054,50.56082899900008],[3.713779500000044,50.589286998000034],[3.487709500000051,50.70359799700003],[3.541272500000048,50.733700498000076],[3.775359500000036,50.74772999800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE322","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Charleroi","NUTS_NAME":"Arr. Charleroi"},"geometry":{"type":"Polygon","coordinates":[[[4.577537999000072,50.54228599800007],[4.588662500000055,50.321315998000046],[4.474986999000066,50.32761],[4.301935999000023,50.37012849800004],[4.267735500000072,50.471572999000045],[4.217754,50.46539299600005],[4.186348500000065,50.52516550000007],[4.247219500000028,50.596118998000065],[4.577537999000072,50.54228599800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE323","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Mons","NUTS_NAME":"Arr. Mons"},"geometry":{"type":"Polygon","coordinates":[[[4.075613,50.440604998000026],[4.027774500000021,50.35833049800004],[3.710389,50.303165499000045],[3.65551,50.46173549800005],[3.752057500000035,50.44591149900003],[3.74291,50.52555849700008],[3.904207,50.61555499800005],[4.075613,50.440604998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE324","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Mouscron","NUTS_NAME":"Arr. Mouscron"},"geometry":{"type":"Polygon","coordinates":[[[3.018708500000059,50.773532998000064],[2.863276,50.708343496000055],[2.842169,50.75140399900005],[2.945734077000054,50.79394636200004],[3.018708500000059,50.773532998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE325","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Soignies","NUTS_NAME":"Arr. Soignies"},"geometry":{"type":"Polygon","coordinates":[[[3.895736499000066,50.73294449800005],[4.100483,50.70595549700005],[4.106683500000031,50.63327449700006],[4.177351,50.66335299700006],[4.247219500000028,50.596118998000065],[4.186348500000065,50.52516550000007],[4.217754,50.46539299600005],[4.075613,50.440604998000026],[3.904207,50.61555499800005],[3.744744500000024,50.72942750000004],[3.775359500000036,50.74772999800007],[3.815390500000035,50.75072499600003],[3.895736499000066,50.73294449800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE326","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Thuin","NUTS_NAME":"Arr. Thuin"},"geometry":{"type":"Polygon","coordinates":[[[4.474986999000066,50.32761],[4.286857,50.25579049700008],[4.401648500000022,50.23227299900003],[4.390042,50.050178998000035],[4.432494,49.94161599600005],[4.233133,49.95782849700004],[4.140853,49.978759996000065],[4.231028,50.071983498000066],[4.126875,50.13492199800004],[4.221769,50.255832500000054],[4.027774500000021,50.35833049800004],[4.075613,50.440604998000026],[4.217754,50.46539299600005],[4.267735500000072,50.471572999000045],[4.301935999000023,50.37012849800004],[4.474986999000066,50.32761]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE327","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Tournai","NUTS_NAME":"Arr. Tournai"},"geometry":{"type":"Polygon","coordinates":[[[3.541272500000048,50.733700498000076],[3.487709500000051,50.70359799700003],[3.713779500000044,50.589286998000034],[3.615301499000054,50.56082899900008],[3.615081500000031,50.490399],[3.286492,50.52756899700006],[3.245294,50.71300949700003],[3.176996,50.75616449700004],[3.324118,50.722308998000074],[3.459731,50.76596850000004],[3.541272500000048,50.733700498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE33","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Liège","NUTS_NAME":"Prov. Liège"},"geometry":{"type":"Polygon","coordinates":[[[6.020998999000028,50.75429549900008],[6.268615500000067,50.625980999000035],[6.189312500000028,50.566094],[6.192200499000023,50.52105549800007],[6.20628750000003,50.521302],[6.315556,50.49704249800004],[6.374430500000074,50.45226950000006],[6.342218500000058,50.38000550000004],[6.405028500000071,50.323308498000074],[6.176784500000053,50.23586299800007],[6.137662500000033,50.129951498000025],[6.02489950000006,50.18277949700007],[5.980197500000031,50.329852998000035],[5.866588,50.345589],[5.870176499000024,50.269374998000046],[5.721549500000037,50.26198949900004],[5.748129,50.30116649800004],[5.675838499000065,50.36851499800008],[5.406546,50.42556349800003],[5.393235,50.37936399800003],[5.221259500000031,50.41729749800004],[5.198977,50.47880599900003],[5.034275500000035,50.53176499800003],[5.028495,50.58882149800007],[4.983341,50.64224249800003],[5.019701,50.750973],[5.101852500000064,50.70865249800005],[5.236892,50.727146],[5.431686500000069,50.71980299900008],[5.687622,50.811923998000054],[5.682000499000026,50.757446497000046],[5.892073,50.75523749800004],[6.020998999000028,50.75429549900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE331","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Huy","NUTS_NAME":"Arr. Huy"},"geometry":{"type":"Polygon","coordinates":[[[5.394420500000024,50.58982850000007],[5.702544500000045,50.40395349800008],[5.675838499000065,50.36851499800008],[5.406546,50.42556349800003],[5.393235,50.37936399800003],[5.221259500000031,50.41729749800004],[5.198977,50.47880599900003],[5.034275500000035,50.53176499800003],[5.028495,50.58882149800007],[5.349207,50.633419],[5.394420500000024,50.58982850000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE332","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Liège","NUTS_NAME":"Arr. Liège"},"geometry":{"type":"Polygon","coordinates":[[[5.682000499000026,50.757446497000046],[5.819052,50.714557497000044],[5.746800500000063,50.66125850000003],[5.763645,50.60209649800004],[5.695985,50.60057849800006],[5.773152,50.56382349800003],[5.792998999000076,50.435298999000054],[5.702544500000045,50.40395349800008],[5.394420500000024,50.58982850000007],[5.349207,50.633419],[5.435543500000051,50.65160749800003],[5.431686500000069,50.71980299900008],[5.687622,50.811923998000054],[5.682000499000026,50.757446497000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE334","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Waremme","NUTS_NAME":"Arr. Waremme"},"geometry":{"type":"Polygon","coordinates":[[[5.349207,50.633419],[5.028495,50.58882149800007],[4.983341,50.64224249800003],[5.019701,50.750973],[5.101852500000064,50.70865249800005],[5.236892,50.727146],[5.431686500000069,50.71980299900008],[5.435543500000051,50.65160749800003],[5.349207,50.633419]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE335","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Verviers - communes francophones","NUTS_NAME":"Arr. Verviers - communes francophones"},"geometry":{"type":"Polygon","coordinates":[[[6.181858,50.40477749900003],[5.980197500000031,50.329852998000035],[5.866588,50.345589],[5.870176499000024,50.269374998000046],[5.721549500000037,50.26198949900004],[5.748129,50.30116649800004],[5.675838499000065,50.36851499800008],[5.702544500000045,50.40395349800008],[5.792998999000076,50.435298999000054],[5.773152,50.56382349800003],[5.695985,50.60057849800006],[5.763645,50.60209649800004],[5.746800500000063,50.66125850000003],[5.819052,50.714557497000044],[5.682000499000026,50.757446497000046],[5.892073,50.75523749800004],[6.020998999000028,50.75429549900008],[5.954752500000041,50.68423849900006],[6.189312500000028,50.566094],[6.192200499000023,50.52105549800007],[6.131211500000063,50.49954199600006],[6.181858,50.40477749900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE336","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Bezirk Verviers - Deutschsprachige Gemeinschaft","NUTS_NAME":"Bezirk Verviers - Deutschsprachige Gemeinschaft"},"geometry":{"type":"MultiPolygon","coordinates":[[[[6.176784500000053,50.23586299800007],[6.137662500000033,50.129951498000025],[6.02489950000006,50.18277949700007],[5.980197500000031,50.329852998000035],[6.181858,50.40477749900003],[6.131211500000063,50.49954199600006],[6.192200499000023,50.52105549800007],[6.20628750000003,50.521302],[6.315556,50.49704249800004],[6.374430500000074,50.45226950000006],[6.342218500000058,50.38000550000004],[6.405028500000071,50.323308498000074],[6.176784500000053,50.23586299800007]]],[[[6.268615500000067,50.625980999000035],[6.189312500000028,50.566094],[5.954752500000041,50.68423849900006],[6.020998999000028,50.75429549900008],[6.268615500000067,50.625980999000035]]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE34","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Luxembourg (BE)","NUTS_NAME":"Prov. Luxembourg (BE)"},"geometry":{"type":"Polygon","coordinates":[[[5.748129,50.30116649800004],[5.721549500000037,50.26198949900004],[5.870176499000024,50.269374998000046],[5.866588,50.345589],[5.980197500000031,50.329852998000035],[6.02489950000006,50.18277949700007],[5.895269,50.112216498000066],[5.746319,49.85359499900005],[5.910688,49.662388],[5.818117,49.54631049900007],[5.734555999000065,49.545690500000035],[5.470882999000025,49.49723799800006],[5.393511,49.61711099900003],[5.153738500000031,49.717925999000045],[4.969431,49.80182599800003],[5.110004500000059,49.925365499000065],[4.973,50.028999500000054],[5.263063500000044,50.10849],[5.234382,50.22196949800008],[5.395951500000024,50.28981399600008],[5.393235,50.37936399800003],[5.406546,50.42556349800003],[5.675838499000065,50.36851499800008],[5.748129,50.30116649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE341","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Arlon","NUTS_NAME":"Arr. Arlon"},"geometry":{"type":"Polygon","coordinates":[[[5.818117,49.54631049900007],[5.734555999000065,49.545690500000035],[5.684699500000022,49.78239449900008],[5.700612,49.82065950000003],[5.746319,49.85359499900005],[5.910688,49.662388],[5.818117,49.54631049900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE342","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Bastogne","NUTS_NAME":"Arr. Bastogne"},"geometry":{"type":"Polygon","coordinates":[[[5.980197500000031,50.329852998000035],[6.02489950000006,50.18277949700007],[5.895269,50.112216498000066],[5.746319,49.85359499900005],[5.700612,49.82065950000003],[5.63335,49.83720399600003],[5.657076500000073,49.90026849900005],[5.529507500000022,49.86990749800003],[5.496263500000055,49.903159997000046],[5.553446500000064,49.952518499000064],[5.417419500000051,50.04844649900008],[5.664053,50.110564999000076],[5.654273,50.166496497000026],[5.74285,50.19298950000007],[5.721549500000037,50.26198949900004],[5.870176499000024,50.269374998000046],[5.866588,50.345589],[5.980197500000031,50.329852998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE343","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Marche-en-Famenne","NUTS_NAME":"Arr. Marche-en-Famenne"},"geometry":{"type":"Polygon","coordinates":[[[5.675838499000065,50.36851499800008],[5.748129,50.30116649800004],[5.721549500000037,50.26198949900004],[5.74285,50.19298950000007],[5.654273,50.166496497000026],[5.664053,50.110564999000076],[5.417419500000051,50.04844649900008],[5.263063500000044,50.10849],[5.234382,50.22196949800008],[5.395951500000024,50.28981399600008],[5.393235,50.37936399800003],[5.406546,50.42556349800003],[5.675838499000065,50.36851499800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE344","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Neufchâteau","NUTS_NAME":"Arr. Neufchâteau"},"geometry":{"type":"Polygon","coordinates":[[[5.417419500000051,50.04844649900008],[5.553446500000064,49.952518499000064],[5.496263500000055,49.903159997000046],[5.529507500000022,49.86990749800003],[5.657076500000073,49.90026849900005],[5.63335,49.83720399600003],[5.700612,49.82065950000003],[5.684699500000022,49.78239449900008],[5.501007499000025,49.73444],[5.185126,49.77567299800006],[5.153738500000031,49.717925999000045],[4.969431,49.80182599800003],[5.110004500000059,49.925365499000065],[4.973,50.028999500000054],[5.263063500000044,50.10849],[5.417419500000051,50.04844649900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE345","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Virton","NUTS_NAME":"Arr. Virton"},"geometry":{"type":"Polygon","coordinates":[[[5.734555999000065,49.545690500000035],[5.470882999000025,49.49723799800006],[5.393511,49.61711099900003],[5.153738500000031,49.717925999000045],[5.185126,49.77567299800006],[5.501007499000025,49.73444],[5.684699500000022,49.78239449900008],[5.734555999000065,49.545690500000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE35","LEVL_CODE":2,"CNTR_CODE":"BE","NAME_LATN":"Prov. Namur","NUTS_NAME":"Prov. Namur"},"geometry":{"type":"Polygon","coordinates":[[[5.393235,50.37936399800003],[5.395951500000024,50.28981399600008],[5.234382,50.22196949800008],[5.263063500000044,50.10849],[4.973,50.028999500000054],[5.110004500000059,49.925365499000065],[4.969431,49.80182599800003],[4.851578,49.793255],[4.889914,49.90920999800005],[4.793194,49.981999],[4.896794,50.13742049800004],[4.796697,50.14867799800004],[4.680489,50.06813850000003],[4.694556,49.99650949900007],[4.432494,49.94161599600005],[4.390042,50.050178998000035],[4.401648500000022,50.23227299900003],[4.286857,50.25579049700008],[4.474986999000066,50.32761],[4.588662500000055,50.321315998000046],[4.577537999000072,50.54228599800007],[4.983341,50.64224249800003],[5.028495,50.58882149800007],[5.034275500000035,50.53176499800003],[5.198977,50.47880599900003],[5.221259500000031,50.41729749800004],[5.393235,50.37936399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE351","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Dinant","NUTS_NAME":"Arr. Dinant"},"geometry":{"type":"Polygon","coordinates":[[[5.393235,50.37936399800003],[5.395951500000024,50.28981399600008],[5.234382,50.22196949800008],[5.263063500000044,50.10849],[4.973,50.028999500000054],[5.110004500000059,49.925365499000065],[4.969431,49.80182599800003],[4.851578,49.793255],[4.889914,49.90920999800005],[4.793194,49.981999],[4.896794,50.13742049800004],[4.796697,50.14867799800004],[4.752530500000034,50.161784999000076],[4.799529,50.17840599800007],[4.75336150000004,50.27515099800007],[4.726381500000059,50.315197],[4.790403,50.35638799800006],[5.069949500000064,50.350883499000076],[5.221259500000031,50.41729749800004],[5.393235,50.37936399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE352","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Namur","NUTS_NAME":"Arr. Namur"},"geometry":{"type":"Polygon","coordinates":[[[5.028495,50.58882149800007],[5.034275500000035,50.53176499800003],[5.198977,50.47880599900003],[5.221259500000031,50.41729749800004],[5.069949500000064,50.350883499000076],[4.790403,50.35638799800006],[4.726381500000059,50.315197],[4.75336150000004,50.27515099800007],[4.594510500000069,50.27384549800007],[4.588662500000055,50.321315998000046],[4.577537999000072,50.54228599800007],[4.983341,50.64224249800003],[5.028495,50.58882149800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BE353","LEVL_CODE":3,"CNTR_CODE":"BE","NAME_LATN":"Arr. Philippeville","NUTS_NAME":"Arr. Philippeville"},"geometry":{"type":"Polygon","coordinates":[[[4.588662500000055,50.321315998000046],[4.594510500000069,50.27384549800007],[4.75336150000004,50.27515099800007],[4.799529,50.17840599800007],[4.752530500000034,50.161784999000076],[4.796697,50.14867799800004],[4.680489,50.06813850000003],[4.694556,49.99650949900007],[4.432494,49.94161599600005],[4.390042,50.050178998000035],[4.401648500000022,50.23227299900003],[4.286857,50.25579049700008],[4.474986999000066,50.32761],[4.588662500000055,50.321315998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG","LEVL_CODE":0,"CNTR_CODE":"BG","NAME_LATN":"BULGARIA","NUTS_NAME":"БЪЛГАРИЯ"},"geometry":{"type":"Polygon","coordinates":[[[28.578884,43.73873899900008],[28.567903500000057,43.60174999800006],[28.607464,43.53937149700005],[28.529502999000044,43.42353799700004],[28.450334,43.38072599700007],[28.150778,43.404731998000045],[28.060180500000058,43.316445],[28.013714,43.22670349800006],[27.908924,43.18582549800004],[27.946768,43.168396],[27.890018500000053,43.042132999000046],[27.88262,42.83841349700003],[27.89906699900007,42.70024550000005],[27.731260500000076,42.70910649700005],[27.633594500000072,42.57825499900008],[27.512098500000036,42.553142499000046],[27.44790450000005,42.461486998000055],[27.616647500000056,42.443859],[27.784431,42.33601749800005],[27.756605,42.24549849600004],[28.03551250000004,41.98307949800005],[27.859884500000078,42.005878497000026],[27.559395,41.904781496000055],[27.25369450000005,42.09926199800003],[27.059650500000032,42.088336997000056],[26.94922850000006,42.00021349800005],[26.561544500000025,41.92627349800006],[26.584593,41.902652998000065],[26.538608500000066,41.823482496000054],[26.369060500000046,41.82068249800005],[26.328071499000032,41.74837899700003],[26.357879,41.711104498000054],[26.16942,41.74869149700004],[26.060393,41.688516],[26.17645550000003,41.554799496000044],[26.15868850000004,41.391182496000056],[25.948626,41.320342],[25.906437500000038,41.307574997000074],[25.224698500000045,41.26463099700004],[25.17937550000005,41.31018799700007],[24.92368350000004,41.402213999000026],[24.783956,41.36018899600003],[24.607909500000062,41.42761799600004],[24.525637,41.56869999700007],[24.059744,41.52211199900006],[24.046779,41.45128599800006],[23.91613249900007,41.48162649900007],[23.62422299900004,41.375727496000025],[23.322212500000035,41.399669999000025],[23.18137150000007,41.318869498000026],[22.927591500000062,41.338539498000046],[22.965055500000062,41.35190749700007],[22.968327500000044,41.51983549800008],[22.948411500000077,41.63179649800003],[23.032124,41.72430099800005],[22.90193350000004,41.87764199900005],[22.867214,42.02219949600004],[22.510412,42.15515849700006],[22.36020650000006,42.311157],[22.520242,42.39757350000008],[22.559255,42.48028399700007],[22.439139500000067,42.56646449900006],[22.461472,42.64849049800006],[22.500916,42.73917749800006],[22.442821499000047,42.82545650000003],[22.51551050000006,42.86853849700003],[22.748399500000062,42.889787496000054],[22.783864,42.98161299900005],[22.98325250000005,43.11146499800003],[23.00621,43.19287849700004],[22.907942,43.22096799600007],[22.747454,43.38661299800003],[22.666975500000035,43.40274749800005],[22.53332850000004,43.47140699800008],[22.491855,43.63917149900004],[22.35713050000004,43.80948199800008],[22.409539,44.00423999800006],[22.536145,44.04551149900004],[22.621212,44.063376498000025],[22.613011500000027,44.16558699900003],[22.67516150000006,44.21566299600005],[22.966399500000023,44.09852499600004],[23.045898,44.06374949900004],[22.878725,43.98156750000004],[22.838719,43.877852],[22.99715450000008,43.80762699600007],[23.267777,43.846812],[23.63008,43.791259997000054],[24.112719,43.699566497000035],[24.324132,43.69956799600004],[24.501014500000053,43.762396996000064],[24.641605999000035,43.73341299800006],[25.29374050000007,43.65429399900006],[25.54464050000007,43.64298149600006],[25.67186250000003,43.691339998000046],[25.783091,43.71223699700005],[26.114924,43.97408949700008],[26.358359999000072,44.03841550000004],[26.379968,44.04295099800004],[26.906678500000055,44.133331998000074],[27.271344999000064,44.12633649800006],[27.399397500000077,44.012369998000054],[27.643537500000036,44.047972998000034],[27.69541,43.98734299900008],[27.737999,43.95341499700004],[27.916834,44.00827399700006],[27.994264500000043,43.842964497000025],[28.236143,43.75852199800005],[28.578884,43.73873899900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG3","LEVL_CODE":1,"CNTR_CODE":"BG","NAME_LATN":"SEVERNA I YUGOIZTOCHNA BULGARIA","NUTS_NAME":"СЕВЕРНА И ЮГОИЗТОЧНА БЪЛГАРИЯ"},"geometry":{"type":"Polygon","coordinates":[[[22.838719,43.877852],[22.99715450000008,43.80762699600007],[23.267777,43.846812],[23.63008,43.791259997000054],[24.112719,43.699566497000035],[24.324132,43.69956799600004],[24.501014500000053,43.762396996000064],[24.641605999000035,43.73341299800006],[25.29374050000007,43.65429399900006],[25.54464050000007,43.64298149600006],[25.67186250000003,43.691339998000046],[25.783091,43.71223699700005],[26.114924,43.97408949700008],[26.358359999000072,44.03841550000004],[26.379968,44.04295099800004],[26.906678500000055,44.133331998000074],[27.271344999000064,44.12633649800006],[27.399397500000077,44.012369998000054],[27.643537500000036,44.047972998000034],[27.69541,43.98734299900008],[27.737999,43.95341499700004],[27.916834,44.00827399700006],[27.994264500000043,43.842964497000025],[28.236143,43.75852199800005],[28.578884,43.73873899900008],[28.567903500000057,43.60174999800006],[28.607464,43.53937149700005],[28.529502999000044,43.42353799700004],[28.450334,43.38072599700007],[28.150778,43.404731998000045],[28.060180500000058,43.316445],[28.013714,43.22670349800006],[27.908924,43.18582549800004],[27.946768,43.168396],[27.890018500000053,43.042132999000046],[27.88262,42.83841349700003],[27.89906699900007,42.70024550000005],[27.731260500000076,42.70910649700005],[27.633594500000072,42.57825499900008],[27.512098500000036,42.553142499000046],[27.44790450000005,42.461486998000055],[27.616647500000056,42.443859],[27.784431,42.33601749800005],[27.756605,42.24549849600004],[28.03551250000004,41.98307949800005],[27.859884500000078,42.005878497000026],[27.559395,41.904781496000055],[27.25369450000005,42.09926199800003],[27.059650500000032,42.088336997000056],[26.94922850000006,42.00021349800005],[26.561544500000025,41.92627349800006],[26.505151,41.99440999700005],[26.538056,42.15419699800003],[26.19273350000003,42.208129996000025],[26.137642500000027,42.15459199900005],[26.15387050000004,42.041241497000044],[26.057769499000074,42.017656499000054],[25.877450500000066,42.13096949700008],[25.763608500000032,42.052389996000045],[25.609251,42.20095899900008],[25.458139500000073,42.16101249900004],[25.42784850000004,42.08947999700007],[25.348064,42.11437749700008],[25.259848500000032,42.184405500000025],[25.081143,42.183936499000026],[25.083079,42.25491649800006],[25.173759,42.27125149900007],[25.112305500000048,42.39638149700005],[25.194347,42.391169999000056],[25.153932001000044,42.45523550000007],[25.191318500000023,42.49178649800007],[25.03865650000006,42.51863649900008],[25.000570500000038,42.588310001000025],[25.007195500000023,42.73761899800007],[24.754968001000066,42.70852799800008],[24.511502,42.79306999900007],[24.385927,42.749953497000035],[24.127983999000037,42.77494799600004],[24.16750150000007,42.816318498000044],[24.16529150100007,42.92970149800004],[24.062151501000073,42.96479499900005],[24.043948501000045,43.05992899800003],[23.97150350000004,43.06225149800008],[23.62619,43.05803899800003],[23.566569,42.993892998000035],[23.412767,43.160521997000046],[23.143223,43.11142049800003],[23.00621,43.19287849700004],[22.907942,43.22096799600007],[22.747454,43.38661299800003],[22.666975500000035,43.40274749800005],[22.53332850000004,43.47140699800008],[22.491855,43.63917149900004],[22.35713050000004,43.80948199800008],[22.409539,44.00423999800006],[22.536145,44.04551149900004],[22.621212,44.063376498000025],[22.613011500000027,44.16558699900003],[22.67516150000006,44.21566299600005],[22.966399500000023,44.09852499600004],[23.045898,44.06374949900004],[22.878725,43.98156750000004],[22.838719,43.877852]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG31","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Severozapaden","NUTS_NAME":"Северозападен"},"geometry":{"type":"Polygon","coordinates":[[[22.838719,43.877852],[22.99715450000008,43.80762699600007],[23.267777,43.846812],[23.63008,43.791259997000054],[24.112719,43.699566497000035],[24.324132,43.69956799600004],[24.501014500000053,43.762396996000064],[24.641605999000035,43.73341299800006],[25.29374050000007,43.65429399900006],[25.204204,43.63763249600004],[25.209436,43.498289499000066],[25.129687500000045,43.458086998000056],[25.256122,43.35800549700008],[25.14043,43.26683049800005],[25.076597,43.17778049800006],[24.96011550000003,43.19376299700008],[24.966582499000026,43.123320497000066],[24.918223,43.10466199700005],[24.951546,43.03119649800004],[24.870264500000076,42.972207998000044],[24.88553,42.89652849600003],[24.96766750000006,42.87725499900006],[25.01459568300004,42.748554082000055],[25.007195500000023,42.73761899800007],[24.754968001000066,42.70852799800008],[24.511502,42.79306999900007],[24.385927,42.749953497000035],[24.127983999000037,42.77494799600004],[24.16750150000007,42.816318498000044],[24.16529150100007,42.92970149800004],[24.062151501000073,42.96479499900005],[24.043948501000045,43.05992899800003],[23.97150350000004,43.06225149800008],[23.62619,43.05803899800003],[23.566569,42.993892998000035],[23.412767,43.160521997000046],[23.143223,43.11142049800003],[23.00621,43.19287849700004],[22.907942,43.22096799600007],[22.747454,43.38661299800003],[22.666975500000035,43.40274749800005],[22.53332850000004,43.47140699800008],[22.491855,43.63917149900004],[22.35713050000004,43.80948199800008],[22.409539,44.00423999800006],[22.536145,44.04551149900004],[22.621212,44.063376498000025],[22.613011500000027,44.16558699900003],[22.67516150000006,44.21566299600005],[22.966399500000023,44.09852499600004],[23.045898,44.06374949900004],[22.878725,43.98156750000004],[22.838719,43.877852]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG311","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Vidin","NUTS_NAME":"Видин"},"geometry":{"type":"Polygon","coordinates":[[[22.966399500000023,44.09852499600004],[23.045898,44.06374949900004],[22.878725,43.98156750000004],[22.838719,43.877852],[22.99715450000008,43.80762699600007],[22.964666500000078,43.755926498000065],[23.058414,43.69419299800006],[22.951083,43.65839399600003],[22.747454,43.38661299800003],[22.666975500000035,43.40274749800005],[22.53332850000004,43.47140699800008],[22.491855,43.63917149900004],[22.35713050000004,43.80948199800008],[22.409539,44.00423999800006],[22.536145,44.04551149900004],[22.621212,44.063376498000025],[22.613011500000027,44.16558699900003],[22.67516150000006,44.21566299600005],[22.966399500000023,44.09852499600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG312","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Montana","NUTS_NAME":"Монтана"},"geometry":{"type":"Polygon","coordinates":[[[23.63008,43.791259997000054],[23.545374500000037,43.57758849800007],[23.636919,43.52246899800008],[23.445928500000036,43.50289749900003],[23.397329,43.44397299800005],[23.417073,43.39184649800006],[23.317432500000052,43.291535500000066],[23.47310250000004,43.224921],[23.40086750000006,43.196428498000046],[23.412767,43.160521997000046],[23.143223,43.11142049800003],[23.00621,43.19287849700004],[22.907942,43.22096799600007],[22.747454,43.38661299800003],[22.951083,43.65839399600003],[23.058414,43.69419299800006],[22.964666500000078,43.755926498000065],[22.99715450000008,43.80762699600007],[23.267777,43.846812],[23.63008,43.791259997000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG313","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Vratsa","NUTS_NAME":"Враца"},"geometry":{"type":"Polygon","coordinates":[[[24.112719,43.699566497000035],[24.324132,43.69956799600004],[24.266002,43.633482498000035],[24.027881,43.60502899900007],[24.001271,43.52460549800003],[24.072958500000027,43.451093498000034],[23.964145,43.355638498000076],[23.95151050000004,43.27449999600003],[24.028557,43.206159498000034],[24.028177,43.10793999800006],[23.97150350000004,43.06225149800008],[23.62619,43.05803899800003],[23.566569,42.993892998000035],[23.412767,43.160521997000046],[23.40086750000006,43.196428498000046],[23.47310250000004,43.224921],[23.317432500000052,43.291535500000066],[23.417073,43.39184649800006],[23.397329,43.44397299800005],[23.445928500000036,43.50289749900003],[23.636919,43.52246899800008],[23.545374500000037,43.57758849800007],[23.63008,43.791259997000054],[24.112719,43.699566497000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG314","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Pleven","NUTS_NAME":"Плевен"},"geometry":{"type":"Polygon","coordinates":[[[24.641605999000035,43.73341299800006],[25.29374050000007,43.65429399900006],[25.204204,43.63763249600004],[25.209436,43.498289499000066],[25.129687500000045,43.458086998000056],[25.256122,43.35800549700008],[25.14043,43.26683049800005],[25.01953049900004,43.378201998000065],[24.977156500000035,43.29673549800003],[24.754803500000037,43.31310949600004],[24.577855501000045,43.203880996000066],[24.387188500000036,43.27737699700003],[24.028557,43.206159498000034],[23.95151050000004,43.27449999600003],[23.964145,43.355638498000076],[24.072958500000027,43.451093498000034],[24.001271,43.52460549800003],[24.027881,43.60502899900007],[24.266002,43.633482498000035],[24.324132,43.69956799600004],[24.501014500000053,43.762396996000064],[24.641605999000035,43.73341299800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG315","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Lovech","NUTS_NAME":"Ловеч"},"geometry":{"type":"Polygon","coordinates":[[[25.14043,43.26683049800005],[25.076597,43.17778049800006],[24.96011550000003,43.19376299700008],[24.966582499000026,43.123320497000066],[24.918223,43.10466199700005],[24.951546,43.03119649800004],[24.870264500000076,42.972207998000044],[24.88553,42.89652849600003],[24.96766750000006,42.87725499900006],[25.01459568300004,42.748554082000055],[25.007195500000023,42.73761899800007],[24.754968001000066,42.70852799800008],[24.511502,42.79306999900007],[24.385927,42.749953497000035],[24.127983999000037,42.77494799600004],[24.16750150000007,42.816318498000044],[24.16529150100007,42.92970149800004],[24.062151501000073,42.96479499900005],[24.043948501000045,43.05992899800003],[23.97150350000004,43.06225149800008],[24.028177,43.10793999800006],[24.028557,43.206159498000034],[24.387188500000036,43.27737699700003],[24.577855501000045,43.203880996000066],[24.754803500000037,43.31310949600004],[24.977156500000035,43.29673549800003],[25.01953049900004,43.378201998000065],[25.14043,43.26683049800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG32","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Severen tsentralen","NUTS_NAME":"Северен централен"},"geometry":{"type":"Polygon","coordinates":[[[27.271344999000064,44.12633649800006],[27.399397500000077,44.012369998000054],[27.643537500000036,44.047972998000034],[27.69541,43.98734299900008],[27.62003050000004,43.896907499000065],[27.543196,43.917966498000055],[27.503872,43.84105249800007],[27.277629999000055,43.79992199700007],[27.20928150000003,43.74278099600008],[27.21953650000006,43.66462049800003],[27.001888500000064,43.69686750000005],[27.035250500000075,43.657460496000056],[26.800677500000063,43.552358],[26.825797,43.50165549800005],[26.715611500000023,43.45500849800004],[26.730439500000045,43.41010349800007],[26.64655,43.32641199600005],[26.48755,43.347285999000064],[26.46894,43.383894499000064],[26.50939,43.38724049800004],[26.471655,43.443182999000044],[26.367741499000033,43.43772599600004],[26.370426500000065,43.49626449900006],[26.267681500000037,43.54971399600004],[26.188978,43.51739349800005],[26.077584,43.51554499700006],[25.943785500000047,43.385800998000036],[25.95983,43.369547],[25.986506,43.369286498000065],[26.004585,43.31875449800003],[26.094859,43.291906],[26.120039500000075,43.18015199800004],[26.058638,43.11883149700003],[26.076070499000025,42.97925849600006],[26.167414500000064,42.952155498000025],[26.00579350000004,42.79730899800006],[25.837491,42.77722749800006],[25.61496150000005,42.78680849800003],[25.61923150000007,42.74802199800007],[25.24970950000005,42.74265149900003],[25.181868001000055,42.787554499000066],[25.01459568300004,42.748554082000055],[24.96766750000006,42.87725499900006],[24.88553,42.89652849600003],[24.870264500000076,42.972207998000044],[24.951546,43.03119649800004],[24.918223,43.10466199700005],[24.966582499000026,43.123320497000066],[24.96011550000003,43.19376299700008],[25.076597,43.17778049800006],[25.14043,43.26683049800005],[25.256122,43.35800549700008],[25.129687500000045,43.458086998000056],[25.209436,43.498289499000066],[25.204204,43.63763249600004],[25.29374050000007,43.65429399900006],[25.54464050000007,43.64298149600006],[25.67186250000003,43.691339998000046],[25.783091,43.71223699700005],[26.114924,43.97408949700008],[26.358359999000072,44.03841550000004],[26.379968,44.04295099800004],[26.906678500000055,44.133331998000074],[27.271344999000064,44.12633649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG321","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Veliko Tarnovo","NUTS_NAME":"Велико Търново"},"geometry":{"type":"Polygon","coordinates":[[[25.54125350000004,43.442904998000074],[25.80459,43.368007],[25.943785500000047,43.385800998000036],[25.95983,43.369547],[25.986506,43.369286498000065],[26.004585,43.31875449800003],[26.094859,43.291906],[26.120039500000075,43.18015199800004],[26.058638,43.11883149700003],[26.076070499000025,42.97925849600006],[26.167414500000064,42.952155498000025],[26.00579350000004,42.79730899800006],[25.837491,42.77722749800006],[25.61496150000005,42.78680849800003],[25.620469,42.90116499800007],[25.56272450000006,42.911019],[25.55272,42.97154199800008],[25.59481,43.019146496000076],[25.304556500000047,43.05686449800004],[25.298422,43.11149999700007],[25.124037,43.088768499000025],[25.076597,43.17778049800006],[25.14043,43.26683049800005],[25.256122,43.35800549700008],[25.129687500000045,43.458086998000056],[25.209436,43.498289499000066],[25.204204,43.63763249600004],[25.29374050000007,43.65429399900006],[25.54464050000007,43.64298149600006],[25.457617,43.57088599800005],[25.570728,43.50456],[25.54125350000004,43.442904998000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG322","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Gabrovo","NUTS_NAME":"Габрово"},"geometry":{"type":"Polygon","coordinates":[[[25.076597,43.17778049800006],[25.124037,43.088768499000025],[25.298422,43.11149999700007],[25.304556500000047,43.05686449800004],[25.59481,43.019146496000076],[25.55272,42.97154199800008],[25.56272450000006,42.911019],[25.620469,42.90116499800007],[25.61496150000005,42.78680849800003],[25.61923150000007,42.74802199800007],[25.24970950000005,42.74265149900003],[25.181868001000055,42.787554499000066],[25.01459568300004,42.748554082000055],[24.96766750000006,42.87725499900006],[24.88553,42.89652849600003],[24.870264500000076,42.972207998000044],[24.951546,43.03119649800004],[24.918223,43.10466199700005],[24.966582499000026,43.123320497000066],[24.96011550000003,43.19376299700008],[25.076597,43.17778049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG323","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Ruse","NUTS_NAME":"Русе"},"geometry":{"type":"Polygon","coordinates":[[[26.333949500000074,43.86136749800005],[26.351514500000064,43.77764799900007],[26.458777,43.76654599900007],[26.410049,43.69316649600006],[26.442371499000046,43.65547549900003],[26.16314550000004,43.63579150000004],[26.188978,43.51739349800005],[26.077584,43.51554499700006],[25.943785500000047,43.385800998000036],[25.80459,43.368007],[25.54125350000004,43.442904998000074],[25.570728,43.50456],[25.457617,43.57088599800005],[25.54464050000007,43.64298149600006],[25.67186250000003,43.691339998000046],[25.783091,43.71223699700005],[26.114924,43.97408949700008],[26.358359999000072,44.03841550000004],[26.409216,44.00947199800004],[26.389222500000074,43.948151996000036],[26.333949500000074,43.86136749800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG324","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Razgrad","NUTS_NAME":"Разград"},"geometry":{"type":"Polygon","coordinates":[[[27.001888500000064,43.69686750000005],[27.035250500000075,43.657460496000056],[26.800677500000063,43.552358],[26.825797,43.50165549800005],[26.715611500000023,43.45500849800004],[26.730439500000045,43.41010349800007],[26.64655,43.32641199600005],[26.48755,43.347285999000064],[26.46894,43.383894499000064],[26.50939,43.38724049800004],[26.471655,43.443182999000044],[26.367741499000033,43.43772599600004],[26.370426500000065,43.49626449900006],[26.267681500000037,43.54971399600004],[26.188978,43.51739349800005],[26.16314550000004,43.63579150000004],[26.442371499000046,43.65547549900003],[26.410049,43.69316649600006],[26.458777,43.76654599900007],[26.351514500000064,43.77764799900007],[26.333949500000074,43.86136749800005],[26.389222500000074,43.948151996000036],[26.574203,43.932798],[27.001888500000064,43.69686750000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG325","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Silistra","NUTS_NAME":"Силистра"},"geometry":{"type":"Polygon","coordinates":[[[27.271344999000064,44.12633649800006],[27.399397500000077,44.012369998000054],[27.643537500000036,44.047972998000034],[27.69541,43.98734299900008],[27.62003050000004,43.896907499000065],[27.543196,43.917966498000055],[27.503872,43.84105249800007],[27.277629999000055,43.79992199700007],[27.20928150000003,43.74278099600008],[27.21953650000006,43.66462049800003],[27.001888500000064,43.69686750000005],[26.574203,43.932798],[26.389222500000074,43.948151996000036],[26.409216,44.00947199800004],[26.358359999000072,44.03841550000004],[26.379968,44.04295099800004],[26.906678500000055,44.133331998000074],[27.271344999000064,44.12633649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG33","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Severoiztochen","NUTS_NAME":"Североизточен"},"geometry":{"type":"Polygon","coordinates":[[[27.916834,44.00827399700006],[27.994264500000043,43.842964497000025],[28.236143,43.75852199800005],[28.578884,43.73873899900008],[28.567903500000057,43.60174999800006],[28.607464,43.53937149700005],[28.529502999000044,43.42353799700004],[28.450334,43.38072599700007],[28.150778,43.404731998000045],[28.060180500000058,43.316445],[28.013714,43.22670349800006],[27.908924,43.18582549800004],[27.946768,43.168396],[27.890018500000053,43.042132999000046],[27.88262,42.83841349700003],[27.43479350000007,42.88179349600006],[27.27076550000004,42.97950549700005],[27.094433500000036,42.95391049800003],[26.58998850000006,42.92011849800008],[26.554392,42.92831799900006],[26.577725500000042,42.98569699900003],[26.360266500000023,43.03433750000005],[26.167414500000064,42.952155498000025],[26.076070499000025,42.97925849600006],[26.058638,43.11883149700003],[26.120039500000075,43.18015199800004],[26.094859,43.291906],[26.004585,43.31875449800003],[25.986506,43.369286498000065],[25.95983,43.369547],[25.943785500000047,43.385800998000036],[26.077584,43.51554499700006],[26.188978,43.51739349800005],[26.267681500000037,43.54971399600004],[26.370426500000065,43.49626449900006],[26.367741499000033,43.43772599600004],[26.471655,43.443182999000044],[26.50939,43.38724049800004],[26.46894,43.383894499000064],[26.48755,43.347285999000064],[26.64655,43.32641199600005],[26.730439500000045,43.41010349800007],[26.715611500000023,43.45500849800004],[26.825797,43.50165549800005],[26.800677500000063,43.552358],[27.035250500000075,43.657460496000056],[27.001888500000064,43.69686750000005],[27.21953650000006,43.66462049800003],[27.20928150000003,43.74278099600008],[27.277629999000055,43.79992199700007],[27.503872,43.84105249800007],[27.543196,43.917966498000055],[27.62003050000004,43.896907499000065],[27.69541,43.98734299900008],[27.737999,43.95341499700004],[27.916834,44.00827399700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG331","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Varna","NUTS_NAME":"Варна"},"geometry":{"type":"Polygon","coordinates":[[[28.060180500000058,43.316445],[28.013714,43.22670349800006],[27.908924,43.18582549800004],[27.946768,43.168396],[27.890018500000053,43.042132999000046],[27.88262,42.83841349700003],[27.43479350000007,42.88179349600006],[27.27076550000004,42.97950549700005],[27.094433500000036,42.95391049800003],[27.072268,43.01348199700004],[27.21754550000003,43.04237550000005],[27.163041500000077,43.10279950000006],[27.18512,43.17499399800005],[27.29541450000005,43.17975099700004],[27.240255500000046,43.27013899800005],[27.309439,43.32905649600008],[27.27156750000006,43.38120799700005],[27.345133,43.44006049800004],[27.30445799900008,43.51543449600007],[27.429463500000054,43.60026699900004],[27.507686500000034,43.497113496000054],[27.615808,43.526719497000045],[27.863004500000045,43.359661999000025],[27.937068500000066,43.400765],[28.060180500000058,43.316445]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG332","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Dobrich","NUTS_NAME":"Добрич"},"geometry":{"type":"Polygon","coordinates":[[[28.578884,43.73873899900008],[28.567903500000057,43.60174999800006],[28.607464,43.53937149700005],[28.529502999000044,43.42353799700004],[28.450334,43.38072599700007],[28.150778,43.404731998000045],[28.060180500000058,43.316445],[27.937068500000066,43.400765],[27.863004500000045,43.359661999000025],[27.615808,43.526719497000045],[27.507686500000034,43.497113496000054],[27.429463500000054,43.60026699900004],[27.302918,43.58979550000004],[27.21953650000006,43.66462049800003],[27.20928150000003,43.74278099600008],[27.277629999000055,43.79992199700007],[27.503872,43.84105249800007],[27.543196,43.917966498000055],[27.62003050000004,43.896907499000065],[27.69541,43.98734299900008],[27.737999,43.95341499700004],[27.916834,44.00827399700006],[27.994264500000043,43.842964497000025],[28.236143,43.75852199800005],[28.578884,43.73873899900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG333","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Shumen","NUTS_NAME":"Шумен"},"geometry":{"type":"Polygon","coordinates":[[[27.21953650000006,43.66462049800003],[27.302918,43.58979550000004],[27.429463500000054,43.60026699900004],[27.30445799900008,43.51543449600007],[27.345133,43.44006049800004],[27.27156750000006,43.38120799700005],[27.309439,43.32905649600008],[27.240255500000046,43.27013899800005],[27.29541450000005,43.17975099700004],[27.18512,43.17499399800005],[27.163041500000077,43.10279950000006],[27.21754550000003,43.04237550000005],[27.072268,43.01348199700004],[27.094433500000036,42.95391049800003],[26.58998850000006,42.92011849800008],[26.554392,42.92831799900006],[26.577725500000042,42.98569699900003],[26.697158,43.078543498000045],[26.662219,43.15835549800005],[26.71858650000007,43.15297899800004],[26.775474,43.23276099900005],[26.731125500000076,43.28513099800006],[26.798699,43.37185350000004],[26.730439500000045,43.41010349800007],[26.715611500000023,43.45500849800004],[26.825797,43.50165549800005],[26.800677500000063,43.552358],[27.035250500000075,43.657460496000056],[27.001888500000064,43.69686750000005],[27.21953650000006,43.66462049800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG334","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Targovishte","NUTS_NAME":"Търговище"},"geometry":{"type":"Polygon","coordinates":[[[26.730439500000045,43.41010349800007],[26.798699,43.37185350000004],[26.731125500000076,43.28513099800006],[26.775474,43.23276099900005],[26.71858650000007,43.15297899800004],[26.662219,43.15835549800005],[26.697158,43.078543498000045],[26.577725500000042,42.98569699900003],[26.360266500000023,43.03433750000005],[26.167414500000064,42.952155498000025],[26.076070499000025,42.97925849600006],[26.058638,43.11883149700003],[26.120039500000075,43.18015199800004],[26.094859,43.291906],[26.004585,43.31875449800003],[25.986506,43.369286498000065],[25.95983,43.369547],[25.943785500000047,43.385800998000036],[26.077584,43.51554499700006],[26.188978,43.51739349800005],[26.267681500000037,43.54971399600004],[26.370426500000065,43.49626449900006],[26.367741499000033,43.43772599600004],[26.471655,43.443182999000044],[26.50939,43.38724049800004],[26.46894,43.383894499000064],[26.48755,43.347285999000064],[26.64655,43.32641199600005],[26.730439500000045,43.41010349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG34","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Yugoiztochen","NUTS_NAME":"Югоизточен"},"geometry":{"type":"Polygon","coordinates":[[[26.554392,42.92831799900006],[26.58998850000006,42.92011849800008],[27.094433500000036,42.95391049800003],[27.27076550000004,42.97950549700005],[27.43479350000007,42.88179349600006],[27.88262,42.83841349700003],[27.89906699900007,42.70024550000005],[27.731260500000076,42.70910649700005],[27.633594500000072,42.57825499900008],[27.512098500000036,42.553142499000046],[27.44790450000005,42.461486998000055],[27.616647500000056,42.443859],[27.784431,42.33601749800005],[27.756605,42.24549849600004],[28.03551250000004,41.98307949800005],[27.859884500000078,42.005878497000026],[27.559395,41.904781496000055],[27.25369450000005,42.09926199800003],[27.059650500000032,42.088336997000056],[26.94922850000006,42.00021349800005],[26.561544500000025,41.92627349800006],[26.505151,41.99440999700005],[26.538056,42.15419699800003],[26.19273350000003,42.208129996000025],[26.137642500000027,42.15459199900005],[26.15387050000004,42.041241497000044],[26.057769499000074,42.017656499000054],[25.877450500000066,42.13096949700008],[25.763608500000032,42.052389996000045],[25.609251,42.20095899900008],[25.458139500000073,42.16101249900004],[25.42784850000004,42.08947999700007],[25.348064,42.11437749700008],[25.259848500000032,42.184405500000025],[25.081143,42.183936499000026],[25.083079,42.25491649800006],[25.173759,42.27125149900007],[25.112305500000048,42.39638149700005],[25.194347,42.391169999000056],[25.153932001000044,42.45523550000007],[25.191318500000023,42.49178649800007],[25.03865650000006,42.51863649900008],[25.000570500000038,42.588310001000025],[25.007195500000023,42.73761899800007],[25.01459568300004,42.748554082000055],[25.181868001000055,42.787554499000066],[25.24970950000005,42.74265149900003],[25.61923150000007,42.74802199800007],[25.61496150000005,42.78680849800003],[25.837491,42.77722749800006],[26.00579350000004,42.79730899800006],[26.167414500000064,42.952155498000025],[26.360266500000023,43.03433750000005],[26.577725500000042,42.98569699900003],[26.554392,42.92831799900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG341","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Burgas","NUTS_NAME":"Бургас"},"geometry":{"type":"Polygon","coordinates":[[[27.88262,42.83841349700003],[27.89906699900007,42.70024550000005],[27.731260500000076,42.70910649700005],[27.633594500000072,42.57825499900008],[27.512098500000036,42.553142499000046],[27.44790450000005,42.461486998000055],[27.616647500000056,42.443859],[27.784431,42.33601749800005],[27.756605,42.24549849600004],[28.03551250000004,41.98307949800005],[27.859884500000078,42.005878497000026],[27.559395,41.904781496000055],[27.25369450000005,42.09926199800003],[27.059650500000032,42.088336997000056],[26.943062500000053,42.144318500000054],[26.917073,42.34152199700003],[26.862353,42.38767999700008],[26.96973,42.43310349800004],[26.83868650000005,42.50736949900005],[26.71400950000003,42.68456149700006],[26.601556500000072,42.80721349800007],[26.58998850000006,42.92011849800008],[27.094433500000036,42.95391049800003],[27.27076550000004,42.97950549700005],[27.43479350000007,42.88179349600006],[27.88262,42.83841349700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG342","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Sliven","NUTS_NAME":"Сливен"},"geometry":{"type":"Polygon","coordinates":[[[26.577725500000042,42.98569699900003],[26.554392,42.92831799900006],[26.58998850000006,42.92011849800008],[26.601556500000072,42.80721349800007],[26.71400950000003,42.68456149700006],[26.64059050000003,42.69343249800005],[26.672502500000064,42.63743199800007],[26.55205250000006,42.56821650000006],[26.426252500000032,42.57423399800007],[26.34071,42.45414499800006],[26.232439500000055,42.46978099800003],[26.18612,42.36132699700005],[26.254705500000057,42.369687500000055],[26.175732,42.26146099600004],[25.930477500000052,42.322275999000055],[25.874343,42.445862498000054],[25.914383,42.47255899800007],[25.844794500000035,42.556874496000034],[25.91175450000003,42.582457497000064],[25.858780500000023,42.62855450100005],[25.837491,42.77722749800006],[26.00579350000004,42.79730899800006],[26.167414500000064,42.952155498000025],[26.360266500000023,43.03433750000005],[26.577725500000042,42.98569699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT22","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Steiermark","NUTS_NAME":"Steiermark"},"geometry":{"type":"Polygon","coordinates":[[[15.611712,47.75108649800006],[15.719146,47.70988349900006],[15.725103,47.646455496000044],[15.821781,47.637955496000075],[15.847011,47.56778949800008],[16.087398500000063,47.507157498000026],[16.17176950000004,47.42239999900005],[16.01515850000004,47.36712749800006],[16.113463,47.16183849600003],[16.086508,47.074209],[16.12171950000004,46.99066649900004],[15.996236,46.835398499000064],[15.984746500000028,46.752207],[16.038086,46.65614549600008],[15.786422,46.70746949900007],[15.649988,46.70575699900007],[15.508846500000061,46.616227998000056],[15.40197949900005,46.65354849800008],[15.065120500000035,46.65211249600003],[15.030154,46.76978949800008],[14.976265,46.78478450000006],[15.027008,46.90973499800003],[14.844475,47.048099998000055],[14.396193,46.98069649800004],[14.132562500000063,47.06234199700003],[13.946724,46.94062999800008],[13.784528499000032,46.94388249600007],[13.90357350000005,47.062910498000065],[13.883161500000028,47.109582998000064],[13.996124,47.13281049600005],[13.864178999000046,47.252169999000046],[13.607597,47.28356499800003],[13.56334350000003,47.42046250000004],[13.585662999000022,47.47479949800004],[13.722560500000043,47.46211150000005],[13.755931,47.55272999600004],[13.694059,47.62321149700006],[13.777418,47.71452899800005],[14.010598,47.70089199700004],[14.050777499000048,47.61715699700005],[14.20579250000003,47.65253799800007],[14.326396500000044,47.60755449800007],[14.738462,47.74872349900005],[15.08924950000005,47.741471],[15.217306,47.79602749900005],[15.374876999000037,47.824564],[15.611712,47.75108649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT221","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Graz","NUTS_NAME":"Graz"},"geometry":{"type":"Polygon","coordinates":[[[15.616364,47.00274149900008],[15.402805499000067,46.876577498000074],[15.184935500000051,47.169362496000076],[15.052846500000044,47.21131550000007],[15.048424,47.227032499000074],[15.242972500000064,47.34329099800004],[15.421807500000057,47.34459949700005],[15.46319950000003,47.338152499000046],[15.442114,47.22249850000003],[15.590146,47.18882099800004],[15.643122,47.127508497000065],[15.631656500000076,47.07291550000008],[15.730508500000042,47.00398899600003],[15.616364,47.00274149900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT222","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Liezen","NUTS_NAME":"Liezen"},"geometry":{"type":"Polygon","coordinates":[[[14.738462,47.74872349900005],[15.08924950000005,47.741471],[15.139081500000032,47.73417599800007],[15.082488,47.70281399900006],[15.065203500000052,47.61015499800004],[14.750471,47.63332199700005],[14.60294,47.42680749600004],[14.426113,47.45919949800003],[14.098427,47.262868498000046],[13.864178999000046,47.252169999000046],[13.607597,47.28356499800003],[13.56334350000003,47.42046250000004],[13.585662999000022,47.47479949800004],[13.722560500000043,47.46211150000005],[13.755931,47.55272999600004],[13.694059,47.62321149700006],[13.777418,47.71452899800005],[14.010598,47.70089199700004],[14.050777499000048,47.61715699700005],[14.20579250000003,47.65253799800007],[14.326396500000044,47.60755449800007],[14.738462,47.74872349900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT223","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Östliche Obersteiermark","NUTS_NAME":"Östliche Obersteiermark"},"geometry":{"type":"Polygon","coordinates":[[[15.847011,47.56778949800008],[15.421807500000057,47.34459949700005],[15.242972500000064,47.34329099800004],[15.048424,47.227032499000074],[14.856698,47.34676249800003],[14.613720500000056,47.378633498000056],[14.60294,47.42680749600004],[14.750471,47.63332199700005],[15.065203500000052,47.61015499800004],[15.082488,47.70281399900006],[15.139081500000032,47.73417599800007],[15.08924950000005,47.741471],[15.217306,47.79602749900005],[15.374876999000037,47.824564],[15.611712,47.75108649800006],[15.719146,47.70988349900006],[15.725103,47.646455496000044],[15.821781,47.637955496000075],[15.847011,47.56778949800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT224","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Oststeiermark","NUTS_NAME":"Oststeiermark"},"geometry":{"type":"Polygon","coordinates":[[[15.847011,47.56778949800008],[16.087398500000063,47.507157498000026],[16.17176950000004,47.42239999900005],[16.01515850000004,47.36712749800006],[16.113463,47.16183849600003],[16.086508,47.074209],[16.12171950000004,46.99066649900004],[15.996236,46.835398499000064],[15.984746500000028,46.752207],[16.038086,46.65614549600008],[15.786422,46.70746949900007],[15.649988,46.70575699900007],[15.691536499000051,46.86247999700004],[15.605505,46.89791299700005],[15.616364,47.00274149900008],[15.730508500000042,47.00398899600003],[15.631656500000076,47.07291550000008],[15.643122,47.127508497000065],[15.590146,47.18882099800004],[15.442114,47.22249850000003],[15.46319950000003,47.338152499000046],[15.421807500000057,47.34459949700005],[15.847011,47.56778949800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT225","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"West- und Südsteiermark","NUTS_NAME":"West- und Südsteiermark"},"geometry":{"type":"Polygon","coordinates":[[[15.616364,47.00274149900008],[15.605505,46.89791299700005],[15.691536499000051,46.86247999700004],[15.649988,46.70575699900007],[15.508846500000061,46.616227998000056],[15.40197949900005,46.65354849800008],[15.065120500000035,46.65211249600003],[15.030154,46.76978949800008],[14.976265,46.78478450000006],[15.027008,46.90973499800003],[14.844475,47.048099998000055],[15.052846500000044,47.21131550000007],[15.184935500000051,47.169362496000076],[15.402805499000067,46.876577498000074],[15.616364,47.00274149900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT226","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Westliche Obersteiermark","NUTS_NAME":"Westliche Obersteiermark"},"geometry":{"type":"Polygon","coordinates":[[[14.60294,47.42680749600004],[14.613720500000056,47.378633498000056],[14.856698,47.34676249800003],[15.048424,47.227032499000074],[15.052846500000044,47.21131550000007],[14.844475,47.048099998000055],[14.396193,46.98069649800004],[14.132562500000063,47.06234199700003],[13.946724,46.94062999800008],[13.784528499000032,46.94388249600007],[13.90357350000005,47.062910498000065],[13.883161500000028,47.109582998000064],[13.996124,47.13281049600005],[13.864178999000046,47.252169999000046],[14.098427,47.262868498000046],[14.426113,47.45919949800003],[14.60294,47.42680749600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT3","LEVL_CODE":1,"CNTR_CODE":"AT","NAME_LATN":"WESTÖSTERREICH","NUTS_NAME":"WESTÖSTERREICH"},"geometry":{"type":"Polygon","coordinates":[[[14.115935,48.58917099800004],[14.324520500000062,48.56233949700004],[14.470069,48.64843699800008],[14.691014,48.58430199700007],[14.912456500000076,48.51396149800007],[14.873728500000027,48.459239998000044],[14.964447,48.378512],[14.943687,48.32364799900006],[14.986449500000049,48.319307998000056],[14.992047,48.25592899800006],[14.828842500000064,48.189076998000075],[14.688370500000076,48.157290498000066],[14.521179,48.23760949800004],[14.479043500000046,48.10441449900003],[14.456123,48.04303699800005],[14.493634,47.99779749600003],[14.771718,47.883778998000025],[14.709993,47.85064149900006],[14.738462,47.74872349900005],[14.326396500000044,47.60755449800007],[14.20579250000003,47.65253799800007],[14.050777499000048,47.61715699700005],[14.010598,47.70089199700004],[13.777418,47.71452899800005],[13.694059,47.62321149700006],[13.755931,47.55272999600004],[13.722560500000043,47.46211150000005],[13.585662999000022,47.47479949800004],[13.56334350000003,47.42046250000004],[13.607597,47.28356499800003],[13.864178999000046,47.252169999000046],[13.996124,47.13281049600005],[13.883161500000028,47.109582998000064],[13.90357350000005,47.062910498000065],[13.784528499000032,46.94388249600007],[13.68653850000004,47.03981950000008],[13.354519500000038,47.097104],[13.073021500000038,47.01341099900003],[12.730896500000028,47.13099249900006],[12.656833500000062,47.09951749600003],[12.757816500000047,47.04582399800006],[12.724635999000043,46.995786499000076],[12.841158,46.86097949800006],[12.965998,46.78955399800003],[12.724958,46.74344450000007],[12.690635,46.656972],[12.477924,46.679835497000056],[12.312809,46.78386599600003],[12.273703,46.88438799800008],[12.143811,46.91377999700006],[12.121343500000023,47.00700199800008],[12.24074550000006,47.06916849800007],[12.136014,47.08066749900007],[11.747028500000056,46.96887249800005],[11.627199500000074,47.01329899900003],[11.164281500000072,46.96572249900004],[11.02225,46.76541049800005],[10.757589500000051,46.78518599900008],[10.667368500000066,46.87549049800003],[10.469651500000055,46.85490899900003],[10.489350500000057,46.93779049800003],[10.38931799900007,47.00052449900005],[10.144974500000046,46.85100949800005],[9.876148500000056,46.93463550000007],[9.879422500000032,47.01956299600005],[9.607078,47.06077449900005],[9.620580500000074,47.15164549900004],[9.530749,47.270581],[9.673370500000033,47.38151050000005],[9.55872050000005,47.54189299800004],[9.736566500000038,47.534906498000055],[9.765470500000049,47.588856],[9.967813500000034,47.54624049600005],[9.999526,47.48301699600006],[10.09147050000007,47.45907599700007],[10.09985049900007,47.35476149700003],[10.23393,47.38328599700003],[10.178353,47.270113998000056],[10.475413,47.431702],[10.431323500000076,47.50367749600008],[10.454439,47.55579699800006],[10.886199,47.53684799800004],[10.871043,47.48188199800006],[10.991202500000043,47.396130999000036],[11.016500500000063,47.396368],[11.42142750000005,47.44485199800005],[11.38380950000004,47.47220749600007],[11.410219,47.495324],[11.572426,47.514643998000054],[11.632883,47.59244599600004],[12.060662,47.61874349800007],[12.20731950000004,47.609837998000046],[12.183931,47.70037049800004],[12.264886500000046,47.73248299800008],[12.257177500000068,47.6791],[12.338045499000032,47.697087500000066],[12.575026500000035,47.63231599900007],[12.695795500000031,47.68222299800004],[12.807303,47.61022749800003],[12.77925650000003,47.579285500000026],[12.81025,47.54706299800006],[13.003285500000061,47.463374498000064],[13.046055500000023,47.52050249800004],[13.097948,47.63371699800007],[13.047965,47.709118998000065],[12.905047,47.72364349900005],[13.00389750000005,47.84796899600008],[12.875785500000063,47.962608998000064],[12.860181500000067,47.996639998000035],[12.751555,48.112810497000055],[12.94468450000005,48.20669249800005],[13.177043,48.29438999600006],[13.407838,48.372160500000064],[13.438430500000038,48.548895],[13.513369500000067,48.59097799700004],[13.727090500000031,48.51301950000004],[13.825764,48.61861649800005],[13.796107,48.71360049800006],[13.839507,48.771605],[14.115935,48.58917099800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT31","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Oberösterreich","NUTS_NAME":"Oberösterreich"},"geometry":{"type":"Polygon","coordinates":[[[14.115935,48.58917099800004],[14.324520500000062,48.56233949700004],[14.470069,48.64843699800008],[14.691014,48.58430199700007],[14.912456500000076,48.51396149800007],[14.873728500000027,48.459239998000044],[14.964447,48.378512],[14.943687,48.32364799900006],[14.986449500000049,48.319307998000056],[14.992047,48.25592899800006],[14.828842500000064,48.189076998000075],[14.688370500000076,48.157290498000066],[14.521179,48.23760949800004],[14.479043500000046,48.10441449900003],[14.456123,48.04303699800005],[14.493634,47.99779749600003],[14.771718,47.883778998000025],[14.709993,47.85064149900006],[14.738462,47.74872349900005],[14.326396500000044,47.60755449800007],[14.20579250000003,47.65253799800007],[14.050777499000048,47.61715699700005],[14.010598,47.70089199700004],[13.777418,47.71452899800005],[13.694059,47.62321149700006],[13.755931,47.55272999600004],[13.722560500000043,47.46211150000005],[13.585662999000022,47.47479949800004],[13.516169,47.496504498000036],[13.476737,47.541251998000064],[13.523298,47.61832899800004],[13.484094,47.64154149800004],[13.550868500000036,47.71411599800007],[13.436274500000025,47.73855099800005],[13.538693500000022,47.78234599800004],[13.289711500000067,47.82437249800006],[13.280289,47.954714496000065],[13.373432,47.96580850000004],[13.303931,48.00786949800005],[12.860181500000067,47.996639998000035],[12.751555,48.112810497000055],[12.94468450000005,48.20669249800005],[13.177043,48.29438999600006],[13.407838,48.372160500000064],[13.438430500000038,48.548895],[13.513369500000067,48.59097799700004],[13.727090500000031,48.51301950000004],[13.825764,48.61861649800005],[13.796107,48.71360049800006],[13.839507,48.771605],[14.115935,48.58917099800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT311","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Innviertel","NUTS_NAME":"Innviertel"},"geometry":{"type":"Polygon","coordinates":[[[13.727090500000031,48.51301950000004],[13.869247,48.42591549800005],[13.923367,48.390179498000066],[13.855284500000039,48.29029949800008],[13.993682,48.21739949800008],[13.753979,48.118048498000064],[13.459253500000045,48.106908],[13.303931,48.00786949800005],[12.860181500000067,47.996639998000035],[12.751555,48.112810497000055],[12.94468450000005,48.20669249800005],[13.177043,48.29438999600006],[13.407838,48.372160500000064],[13.438430500000038,48.548895],[13.513369500000067,48.59097799700004],[13.727090500000031,48.51301950000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT312","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Linz-Wels","NUTS_NAME":"Linz-Wels"},"geometry":{"type":"Polygon","coordinates":[[[14.521179,48.23760949800004],[14.479043500000046,48.10441449900003],[14.135374,48.09854999800007],[13.987305,48.00262249800005],[13.753979,48.118048498000064],[13.993682,48.21739949800008],[13.855284500000039,48.29029949800008],[13.923367,48.390179498000066],[13.869247,48.42591549800005],[14.03029,48.37318199900005],[14.322932500000036,48.47537549700007],[14.336628,48.42410649800007],[14.466402500000072,48.395677498000055],[14.482123,48.302628998000046],[14.417312500000037,48.30456849800004],[14.408871999000041,48.248395498000036],[14.521179,48.23760949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT313","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Mühlviertel","NUTS_NAME":"Mühlviertel"},"geometry":{"type":"Polygon","coordinates":[[[14.691014,48.58430199700007],[14.912456500000076,48.51396149800007],[14.873728500000027,48.459239998000044],[14.964447,48.378512],[14.943687,48.32364799900006],[14.986449500000049,48.319307998000056],[14.992047,48.25592899800006],[14.828842500000064,48.189076998000075],[14.688370500000076,48.157290498000066],[14.521179,48.23760949800004],[14.408871999000041,48.248395498000036],[14.417312500000037,48.30456849800004],[14.482123,48.302628998000046],[14.466402500000072,48.395677498000055],[14.336628,48.42410649800007],[14.322932500000036,48.47537549700007],[14.03029,48.37318199900005],[13.869247,48.42591549800005],[13.727090500000031,48.51301950000004],[13.825764,48.61861649800005],[13.796107,48.71360049800006],[13.839507,48.771605],[14.115935,48.58917099800004],[14.324520500000062,48.56233949700004],[14.470069,48.64843699800008],[14.691014,48.58430199700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT314","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Steyr-Kirchdorf","NUTS_NAME":"Steyr-Kirchdorf"},"geometry":{"type":"Polygon","coordinates":[[[14.479043500000046,48.10441449900003],[14.456123,48.04303699800005],[14.493634,47.99779749600003],[14.771718,47.883778998000025],[14.709993,47.85064149900006],[14.738462,47.74872349900005],[14.326396500000044,47.60755449800007],[14.20579250000003,47.65253799800007],[14.050777499000048,47.61715699700005],[14.010598,47.70089199700004],[13.999456,47.800373998000055],[14.097795,47.844956498000045],[13.991926,47.879208498000025],[13.987305,48.00262249800005],[14.135374,48.09854999800007],[14.479043500000046,48.10441449900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT315","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Traunviertel","NUTS_NAME":"Traunviertel"},"geometry":{"type":"Polygon","coordinates":[[[13.753979,48.118048498000064],[13.987305,48.00262249800005],[13.991926,47.879208498000025],[14.097795,47.844956498000045],[13.999456,47.800373998000055],[14.010598,47.70089199700004],[13.777418,47.71452899800005],[13.694059,47.62321149700006],[13.755931,47.55272999600004],[13.722560500000043,47.46211150000005],[13.585662999000022,47.47479949800004],[13.516169,47.496504498000036],[13.476737,47.541251998000064],[13.523298,47.61832899800004],[13.484094,47.64154149800004],[13.550868500000036,47.71411599800007],[13.436274500000025,47.73855099800005],[13.538693500000022,47.78234599800004],[13.289711500000067,47.82437249800006],[13.280289,47.954714496000065],[13.373432,47.96580850000004],[13.303931,48.00786949800005],[13.459253500000045,48.106908],[13.753979,48.118048498000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT32","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Salzburg","NUTS_NAME":"Salzburg"},"geometry":{"type":"Polygon","coordinates":[[[13.585662999000022,47.47479949800004],[13.56334350000003,47.42046250000004],[13.607597,47.28356499800003],[13.864178999000046,47.252169999000046],[13.996124,47.13281049600005],[13.883161500000028,47.109582998000064],[13.90357350000005,47.062910498000065],[13.784528499000032,46.94388249600007],[13.68653850000004,47.03981950000008],[13.354519500000038,47.097104],[13.073021500000038,47.01341099900003],[12.730896500000028,47.13099249900006],[12.656833500000062,47.09951749600003],[12.399521,47.15280149700004],[12.24074550000006,47.06916849800007],[12.136014,47.08066749900007],[12.081858,47.194371498000066],[12.114546500000074,47.306444],[12.469798,47.323774498000034],[12.506090500000028,47.394129500000076],[12.699591500000054,47.47788599800003],[12.64163,47.55450299600005],[12.666601,47.57862699800006],[12.575026500000035,47.63231599900007],[12.695795500000031,47.68222299800004],[12.807303,47.61022749800003],[12.77925650000003,47.579285500000026],[12.81025,47.54706299800006],[13.003285500000061,47.463374498000064],[13.046055500000023,47.52050249800004],[13.097948,47.63371699800007],[13.047965,47.709118998000065],[12.905047,47.72364349900005],[13.00389750000005,47.84796899600008],[12.875785500000063,47.962608998000064],[12.860181500000067,47.996639998000035],[13.303931,48.00786949800005],[13.373432,47.96580850000004],[13.280289,47.954714496000065],[13.289711500000067,47.82437249800006],[13.538693500000022,47.78234599800004],[13.436274500000025,47.73855099800005],[13.550868500000036,47.71411599800007],[13.484094,47.64154149800004],[13.523298,47.61832899800004],[13.476737,47.541251998000064],[13.516169,47.496504498000036],[13.585662999000022,47.47479949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT321","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Lungau","NUTS_NAME":"Lungau"},"geometry":{"type":"Polygon","coordinates":[[[13.864178999000046,47.252169999000046],[13.996124,47.13281049600005],[13.883161500000028,47.109582998000064],[13.90357350000005,47.062910498000065],[13.784528499000032,46.94388249600007],[13.68653850000004,47.03981950000008],[13.354519500000038,47.097104],[13.31779,47.158981],[13.358197,47.208531],[13.607597,47.28356499800003],[13.864178999000046,47.252169999000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT322","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Pinzgau-Pongau","NUTS_NAME":"Pinzgau-Pongau"},"geometry":{"type":"Polygon","coordinates":[[[13.046055500000023,47.52050249800004],[13.187479500000052,47.55269449800005],[13.435766500000057,47.45839749800007],[13.516169,47.496504498000036],[13.585662999000022,47.47479949800004],[13.56334350000003,47.42046250000004],[13.607597,47.28356499800003],[13.358197,47.208531],[13.31779,47.158981],[13.354519500000038,47.097104],[13.073021500000038,47.01341099900003],[12.730896500000028,47.13099249900006],[12.656833500000062,47.09951749600003],[12.399521,47.15280149700004],[12.24074550000006,47.06916849800007],[12.136014,47.08066749900007],[12.081858,47.194371498000066],[12.114546500000074,47.306444],[12.469798,47.323774498000034],[12.506090500000028,47.394129500000076],[12.699591500000054,47.47788599800003],[12.64163,47.55450299600005],[12.666601,47.57862699800006],[12.575026500000035,47.63231599900007],[12.695795500000031,47.68222299800004],[12.807303,47.61022749800003],[12.77925650000003,47.579285500000026],[12.81025,47.54706299800006],[13.003285500000061,47.463374498000064],[13.046055500000023,47.52050249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT323","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Salzburg und Umgebung","NUTS_NAME":"Salzburg und Umgebung"},"geometry":{"type":"Polygon","coordinates":[[[13.303931,48.00786949800005],[13.373432,47.96580850000004],[13.280289,47.954714496000065],[13.289711500000067,47.82437249800006],[13.538693500000022,47.78234599800004],[13.436274500000025,47.73855099800005],[13.550868500000036,47.71411599800007],[13.484094,47.64154149800004],[13.523298,47.61832899800004],[13.476737,47.541251998000064],[13.516169,47.496504498000036],[13.435766500000057,47.45839749800007],[13.187479500000052,47.55269449800005],[13.046055500000023,47.52050249800004],[13.097948,47.63371699800007],[13.047965,47.709118998000065],[12.905047,47.72364349900005],[13.00389750000005,47.84796899600008],[12.875785500000063,47.962608998000064],[12.860181500000067,47.996639998000035],[13.303931,48.00786949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT33","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Tirol","NUTS_NAME":"Tirol"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.724958,46.74344450000007],[12.690635,46.656972],[12.477924,46.679835497000056],[12.312809,46.78386599600003],[12.273703,46.88438799800008],[12.143811,46.91377999700006],[12.121343500000023,47.00700199800008],[12.24074550000006,47.06916849800007],[12.399521,47.15280149700004],[12.656833500000062,47.09951749600003],[12.757816500000047,47.04582399800006],[12.724635999000043,46.995786499000076],[12.841158,46.86097949800006],[12.965998,46.78955399800003],[12.724958,46.74344450000007]]],[[[12.264886500000046,47.73248299800008],[12.257177500000068,47.6791],[12.338045499000032,47.697087500000066],[12.575026500000035,47.63231599900007],[12.666601,47.57862699800006],[12.64163,47.55450299600005],[12.699591500000054,47.47788599800003],[12.506090500000028,47.394129500000076],[12.469798,47.323774498000034],[12.114546500000074,47.306444],[12.081858,47.194371498000066],[12.136014,47.08066749900007],[11.747028500000056,46.96887249800005],[11.627199500000074,47.01329899900003],[11.164281500000072,46.96572249900004],[11.02225,46.76541049800005],[10.757589500000051,46.78518599900008],[10.667368500000066,46.87549049800003],[10.469651500000055,46.85490899900003],[10.489350500000057,46.93779049800003],[10.38931799900007,47.00052449900005],[10.144974500000046,46.85100949800005],[10.098220500000025,46.92735799800005],[10.159806,46.98375099700007],[10.134231500000055,47.083864996000045],[10.212995,47.157583998000064],[10.178353,47.270113998000056],[10.475413,47.431702],[10.431323500000076,47.50367749600008],[10.454439,47.55579699800006],[10.886199,47.53684799800004],[10.871043,47.48188199800006],[10.991202500000043,47.396130999000036],[11.016500500000063,47.396368],[11.42142750000005,47.44485199800005],[11.38380950000004,47.47220749600007],[11.410219,47.495324],[11.572426,47.514643998000054],[11.632883,47.59244599600004],[12.060662,47.61874349800007],[12.20731950000004,47.609837998000046],[12.183931,47.70037049800004],[12.264886500000046,47.73248299800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT331","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Außerfern","NUTS_NAME":"Außerfern"},"geometry":{"type":"Polygon","coordinates":[[[10.886199,47.53684799800004],[10.871043,47.48188199800006],[10.991202500000043,47.396130999000036],[10.862247500000024,47.33685],[10.819073,47.385869497000044],[10.706494,47.361774998000044],[10.638514500000042,47.33007299700006],[10.65168250000005,47.26746049800005],[10.212995,47.157583998000064],[10.178353,47.270113998000056],[10.475413,47.431702],[10.431323500000076,47.50367749600008],[10.454439,47.55579699800006],[10.886199,47.53684799800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT332","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Innsbruck","NUTS_NAME":"Innsbruck"},"geometry":{"type":"Polygon","coordinates":[[[11.42142750000005,47.44485199800005],[11.692852500000072,47.26048499600006],[11.627199500000074,47.01329899900003],[11.164281500000072,46.96572249900004],[11.090124,46.988639497000065],[11.117531500000041,47.05087899700004],[11.042944500000033,47.18077099800007],[11.094710500000076,47.22512799800006],[10.995681500000046,47.31132549800003],[11.016500500000063,47.396368],[11.42142750000005,47.44485199800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT333","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Osttirol","NUTS_NAME":"Osttirol"},"geometry":{"type":"Polygon","coordinates":[[[12.656833500000062,47.09951749600003],[12.757816500000047,47.04582399800006],[12.724635999000043,46.995786499000076],[12.841158,46.86097949800006],[12.965998,46.78955399800003],[12.724958,46.74344450000007],[12.690635,46.656972],[12.477924,46.679835497000056],[12.312809,46.78386599600003],[12.273703,46.88438799800008],[12.143811,46.91377999700006],[12.121343500000023,47.00700199800008],[12.24074550000006,47.06916849800007],[12.399521,47.15280149700004],[12.656833500000062,47.09951749600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT334","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Tiroler Oberland","NUTS_NAME":"Tiroler Oberland"},"geometry":{"type":"Polygon","coordinates":[[[11.164281500000072,46.96572249900004],[11.02225,46.76541049800005],[10.757589500000051,46.78518599900008],[10.667368500000066,46.87549049800003],[10.469651500000055,46.85490899900003],[10.489350500000057,46.93779049800003],[10.38931799900007,47.00052449900005],[10.144974500000046,46.85100949800005],[10.098220500000025,46.92735799800005],[10.159806,46.98375099700007],[10.134231500000055,47.083864996000045],[10.212995,47.157583998000064],[10.65168250000005,47.26746049800005],[10.638514500000042,47.33007299700006],[10.706494,47.361774998000044],[10.819073,47.385869497000044],[10.862247500000024,47.33685],[10.991202500000043,47.396130999000036],[11.016500500000063,47.396368],[10.995681500000046,47.31132549800003],[11.094710500000076,47.22512799800006],[11.042944500000033,47.18077099800007],[11.117531500000041,47.05087899700004],[11.090124,46.988639497000065],[11.164281500000072,46.96572249900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT335","LEVL_CODE":3,"CNTR_CODE":"AT","NAME_LATN":"Tiroler Unterland","NUTS_NAME":"Tiroler Unterland"},"geometry":{"type":"Polygon","coordinates":[[[12.338045499000032,47.697087500000066],[12.575026500000035,47.63231599900007],[12.666601,47.57862699800006],[12.64163,47.55450299600005],[12.699591500000054,47.47788599800003],[12.506090500000028,47.394129500000076],[12.469798,47.323774498000034],[12.114546500000074,47.306444],[12.081858,47.194371498000066],[12.136014,47.08066749900007],[11.747028500000056,46.96887249800005],[11.627199500000074,47.01329899900003],[11.692852500000072,47.26048499600006],[11.42142750000005,47.44485199800005],[11.38380950000004,47.47220749600007],[11.410219,47.495324],[11.572426,47.514643998000054],[11.632883,47.59244599600004],[12.060662,47.61874349800007],[12.20731950000004,47.609837998000046],[12.183931,47.70037049800004],[12.264886500000046,47.73248299800008],[12.257177500000068,47.6791],[12.338045499000032,47.697087500000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"AT34","LEVL_CODE":2,"CNTR_CODE":"AT","NAME_LATN":"Vorarlberg","NUTS_NAME":"Vorarlberg"},"geometry":{"type":"Polygon","coordinates":[[[9.967813500000034,47.54624049600005],[9.999526,47.48301699600006],[10.09147050000007,47.45907599700007],[10.09985049900007,47.35476149700003],[10.23393,47.38328599700003],[10.178353,47.270113998000056],[10.212995,47.157583998000064],[10.134231500000055,47.083864996000045],[10.159806,46.98375099700007],[10.098220500000025,46.92735799800005],[10.144974500000046,46.85100949800005],[9.876148500000056,46.93463550000007],[9.879422500000032,47.01956299600005],[9.607078,47.06077449900005],[9.620580500000074,47.15164549900004],[9.530749,47.270581],[9.673370500000033,47.38151050000005],[9.55872050000005,47.54189299800004],[9.736566500000038,47.534906498000055],[9.765470500000049,47.588856],[9.967813500000034,47.54624049600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE134","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ortenaukreis","NUTS_NAME":"Ortenaukreis"},"geometry":{"type":"Polygon","coordinates":[[[8.222007,48.60322749800008],[8.218828,48.50795449800006],[8.275435500000071,48.47785649700006],[8.247635500000058,48.395136498000056],[8.303992,48.34909449600008],[8.32941,48.30260150000004],[8.278795500000058,48.29021449700008],[8.31294,48.20516549800004],[8.282725500000026,48.17166999700004],[8.180335,48.18392],[8.151861500000052,48.22898949800003],[7.680713,48.257266996000055],[7.739841500000068,48.321731500000055],[7.804824,48.59257949800008],[7.95963050000006,48.71858099800005],[8.222007,48.60322749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE135","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rottweil","NUTS_NAME":"Rottweil"},"geometry":{"type":"Polygon","coordinates":[[[8.737186499000074,48.377976998000065],[8.683524,48.28078749800005],[8.751237,48.15953999700008],[8.604284500000063,48.074715498000046],[8.516634500000066,48.166499997000074],[8.282725500000026,48.17166999700004],[8.31294,48.20516549800004],[8.278795500000058,48.29021449700008],[8.32941,48.30260150000004],[8.303992,48.34909449600008],[8.345293500000025,48.38446699800005],[8.494052,48.314436999000065],[8.487894499000049,48.38202349800008],[8.737186499000074,48.377976998000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE136","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwarzwald-Baar-Kreis","NUTS_NAME":"Schwarzwald-Baar-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.282725500000026,48.17166999700004],[8.516634500000066,48.166499997000074],[8.604284500000063,48.074715498000046],[8.662905,47.971078498000054],[8.588482999000064,47.944916496000076],[8.640146,47.85481199800006],[8.61233,47.801461999000026],[8.510115,47.77619049900005],[8.43903350000005,47.853638498000066],[8.319668,47.98867249800003],[8.136648,48.009147496000026],[8.109056500000065,48.063544500000035],[8.166075,48.07578999800006],[8.125374,48.13169499700007],[8.180335,48.18392],[8.282725500000026,48.17166999700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE137","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Tuttlingen","NUTS_NAME":"Tuttlingen"},"geometry":{"type":"Polygon","coordinates":[[[8.953761,48.10727899800003],[9.012045,47.93692299700007],[8.640146,47.85481199800006],[8.588482999000064,47.944916496000076],[8.662905,47.971078498000054],[8.604284500000063,48.074715498000046],[8.751237,48.15953999700008],[8.953761,48.10727899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE138","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Konstanz","NUTS_NAME":"Konstanz"},"geometry":{"type":"Polygon","coordinates":[[[9.151105,47.89498299700006],[9.128582,47.846817498000064],[9.030533,47.806520499000044],[9.174700500000029,47.732297998000035],[9.215705,47.66601899600005],[9.182192,47.655890498000076],[8.87499850000006,47.65597349700005],[8.843022,47.71226249800003],[8.77209750000003,47.718459497000026],[8.795708500000046,47.67559549900005],[8.727981500000055,47.69268049800007],[8.74010450000003,47.752789],[8.61233,47.801461999000026],[8.640146,47.85481199800006],[9.012045,47.93692299700007],[9.151105,47.89498299700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE139","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lörrach","NUTS_NAME":"Lörrach"},"geometry":{"type":"Polygon","coordinates":[[[8.018114500000024,47.84139399900005],[7.977745,47.694976],[7.874626500000034,47.63642899800004],[7.894107500000075,47.58637499800005],[7.713784500000031,47.539404999000055],[7.634097,47.56111350000003],[7.69337,47.600644998000064],[7.589039,47.58987799800008],[7.519558,47.66812299900005],[7.545990500000073,47.743573998000045],[7.889874,47.878971998000054],[8.018114500000024,47.84139399900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE13A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Waldshut","NUTS_NAME":"Waldshut"},"geometry":{"type":"Polygon","coordinates":[[[8.510115,47.77619049900005],[8.405565999000032,47.67451349800007],[8.606369500000028,47.66900549800005],[8.595602,47.605544499000075],[8.562841,47.599432497000066],[8.52144,47.633524],[8.426434500000028,47.567548997000074],[8.221265500000072,47.61551499800004],[8.01872,47.550453996000044],[7.894107500000075,47.58637499800005],[7.874626500000034,47.63642899800004],[7.977745,47.694976],[8.018114500000024,47.84139399900005],[8.183383500000048,47.782542500000034],[8.267485,47.85203799800007],[8.43903350000005,47.853638498000066],[8.510115,47.77619049900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE14","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Tübingen","NUTS_NAME":"Tübingen"},"geometry":{"type":"Polygon","coordinates":[[[9.944071500000064,48.631755498000075],[10.23077950000004,48.510510999000076],[10.133895500000051,48.454871498000045],[10.032694500000048,48.45719649800003],[9.997606,48.35011799800003],[10.095357500000034,48.16401499800003],[10.136457,48.10845949700007],[10.135728,48.02376149600008],[10.132387,48.01539999800008],[10.114954500000067,47.97625750000003],[10.104207,47.974358496000036],[10.110136500000067,47.93714999800005],[10.131928500000072,47.82008749600004],[10.064681,47.78588749900007],[10.139437500000042,47.703273496000065],[10.077292500000055,47.63927449700003],[9.840336,47.677096498000026],[9.692543,47.610768998000026],[9.55872050000005,47.54189299800004],[9.495604,47.551454998000054],[9.182192,47.655890498000076],[9.215705,47.66601899600005],[9.174700500000029,47.732297998000035],[9.030533,47.806520499000044],[9.128582,47.846817498000064],[9.151105,47.89498299700006],[9.012045,47.93692299700007],[8.953761,48.10727899800003],[8.751237,48.15953999700008],[8.683524,48.28078749800005],[8.737186499000074,48.377976998000065],[8.774055,48.416278498000054],[8.825225500000045,48.46304199700006],[8.75571050000002,48.50414],[8.768902500000024,48.52184099800007],[9.129176500000028,48.60126649800003],[9.151229500000056,48.60405549700005],[9.582475500000044,48.539146998000035],[9.58517450000005,48.536741498000026],[9.944071500000064,48.631755498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE141","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Reutlingen","NUTS_NAME":"Reutlingen"},"geometry":{"type":"Polygon","coordinates":[[[9.582475500000044,48.539146998000035],[9.58517450000005,48.536741498000026],[9.638778500000058,48.42445149800005],[9.490054,48.23270999700003],[9.39189,48.196503],[9.347566500000028,48.23950099800004],[9.187028500000054,48.280835996000064],[9.161679500000048,48.31278499800004],[9.221773,48.33676049700006],[9.12947949900007,48.37289849600006],[9.129176500000028,48.60126649800003],[9.151229500000056,48.60405549700005],[9.582475500000044,48.539146998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE142","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Tübingen, Landkreis","NUTS_NAME":"Tübingen, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.129176500000028,48.60126649800003],[9.12947949900007,48.37289849600006],[8.774055,48.416278498000054],[8.825225500000045,48.46304199700006],[8.75571050000002,48.50414],[8.768902500000024,48.52184099800007],[9.129176500000028,48.60126649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE143","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Zollernalbkreis","NUTS_NAME":"Zollernalbkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.12947949900007,48.37289849600006],[9.221773,48.33676049700006],[9.161679500000048,48.31278499800004],[9.187028500000054,48.280835996000064],[9.107458500000064,48.252091998000026],[9.186262,48.20983849700008],[9.162836500000026,48.136367998000026],[9.033456,48.16523349700003],[8.953761,48.10727899800003],[8.751237,48.15953999700008],[8.683524,48.28078749800005],[8.737186499000074,48.377976998000065],[8.774055,48.416278498000054],[9.12947949900007,48.37289849600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE144","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ulm, Stadtkreis","NUTS_NAME":"Ulm, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.032694500000048,48.45719649800003],[9.997606,48.35011799800003],[9.921303,48.312152996000066],[9.843791,48.36133149800003],[10.032694500000048,48.45719649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE145","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Alb-Donau-Kreis","NUTS_NAME":"Alb-Donau-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[10.23077950000004,48.510510999000076],[10.133895500000051,48.454871498000045],[10.032694500000048,48.45719649800003],[9.843791,48.36133149800003],[9.921303,48.312152996000066],[9.997606,48.35011799800003],[10.095357500000034,48.16401499800003],[10.00858450000004,48.151485998000055],[10.068539,48.18974099800005],[9.896037500000034,48.29301049900005],[9.744256,48.17611399900005],[9.490054,48.23270999700003],[9.638778500000058,48.42445149800005],[9.58517450000005,48.536741498000026],[9.944071500000064,48.631755498000075],[10.23077950000004,48.510510999000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE146","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Biberach","NUTS_NAME":"Biberach"},"geometry":{"type":"Polygon","coordinates":[[[10.095357500000034,48.16401499800003],[10.136457,48.10845949700007],[10.135728,48.02376149600008],[10.132387,48.01539999800008],[10.114954500000067,47.97625750000003],[10.104207,47.974358496000036],[9.604912500000069,48.002283996000074],[9.284018500000059,48.145027998000046],[9.347566500000028,48.23950099800004],[9.39189,48.196503],[9.490054,48.23270999700003],[9.744256,48.17611399900005],[9.896037500000034,48.29301049900005],[10.068539,48.18974099800005],[10.00858450000004,48.151485998000055],[10.095357500000034,48.16401499800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE147","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bodenseekreis","NUTS_NAME":"Bodenseekreis"},"geometry":{"type":"Polygon","coordinates":[[[9.409414500000025,47.821604998000055],[9.500521500000048,47.732905997000046],[9.72878,47.68110849800007],[9.692543,47.610768998000026],[9.55872050000005,47.54189299800004],[9.495604,47.551454998000054],[9.182192,47.655890498000076],[9.215705,47.66601899600005],[9.174700500000029,47.732297998000035],[9.030533,47.806520499000044],[9.128582,47.846817498000064],[9.270273,47.88019899900007],[9.409414500000025,47.821604998000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE148","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ravensburg","NUTS_NAME":"Ravensburg"},"geometry":{"type":"Polygon","coordinates":[[[9.604912500000069,48.002283996000074],[10.104207,47.974358496000036],[10.110136500000067,47.93714999800005],[10.131928500000072,47.82008749600004],[10.064681,47.78588749900007],[10.139437500000042,47.703273496000065],[10.077292500000055,47.63927449700003],[9.840336,47.677096498000026],[9.692543,47.610768998000026],[9.72878,47.68110849800007],[9.500521500000048,47.732905997000046],[9.409414500000025,47.821604998000055],[9.37659,47.879231496000045],[9.423145,47.96316249800003],[9.604912500000069,48.002283996000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE149","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Sigmaringen","NUTS_NAME":"Sigmaringen"},"geometry":{"type":"Polygon","coordinates":[[[9.347566500000028,48.23950099800004],[9.284018500000059,48.145027998000046],[9.604912500000069,48.002283996000074],[9.423145,47.96316249800003],[9.37659,47.879231496000045],[9.409414500000025,47.821604998000055],[9.270273,47.88019899900007],[9.128582,47.846817498000064],[9.151105,47.89498299700006],[9.012045,47.93692299700007],[8.953761,48.10727899800003],[9.033456,48.16523349700003],[9.162836500000026,48.136367998000026],[9.186262,48.20983849700008],[9.107458500000064,48.252091998000026],[9.187028500000054,48.280835996000064],[9.347566500000028,48.23950099800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE2","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"BAYERN","NUTS_NAME":"BAYERN"},"geometry":{"type":"Polygon","coordinates":[[[10.12726,50.559533],[10.450532,50.40185949900007],[10.600550500000054,50.33368799800007],[10.610115,50.22799499800004],[10.729202,50.23000549600005],[10.851494500000058,50.26276249800003],[10.715266500000041,50.363590997000074],[10.945718500000055,50.38646649800006],[11.115305,50.36700699600004],[11.189943,50.27118550000006],[11.259778,50.27494649800008],[11.265938,50.47942099900007],[11.423893,50.51606449800005],[11.420630500000073,50.443366499000035],[11.481568,50.43162150000006],[11.519853,50.37426849800005],[11.603290500000071,50.398766],[11.919884500000023,50.42440349800006],[12.100900500000023,50.31802799700006],[12.13994,50.27771849800007],[12.09076,50.25238450000006],[12.160679,50.21984899700004],[12.260799,50.05815649700003],[12.48988950000006,49.98176099900007],[12.470510500000046,49.94502550000004],[12.550988500000074,49.905088],[12.401524,49.75837299700004],[12.521638,49.686761999000055],[12.593778500000042,49.54219149800008],[12.644176,49.52305749900006],[12.633763500000043,49.47619499700005],[12.77881,49.34807849600003],[12.972595500000068,49.33388799800008],[13.170907999000065,49.17357950000007],[13.397147,49.05100999800004],[13.416651500000057,48.98003549800006],[13.551578500000062,48.96778749600003],[13.839507,48.771605],[13.796107,48.71360049800006],[13.825764,48.61861649800005],[13.727090500000031,48.51301950000004],[13.513369500000067,48.59097799700004],[13.438430500000038,48.548895],[13.407838,48.372160500000064],[13.177043,48.29438999600006],[12.94468450000005,48.20669249800005],[12.751555,48.112810497000055],[12.860181500000067,47.996639998000035],[12.875785500000063,47.962608998000064],[13.00389750000005,47.84796899600008],[12.905047,47.72364349900005],[13.047965,47.709118998000065],[13.097948,47.63371699800007],[13.046055500000023,47.52050249800004],[13.003285500000061,47.463374498000064],[12.81025,47.54706299800006],[12.77925650000003,47.579285500000026],[12.807303,47.61022749800003],[12.695795500000031,47.68222299800004],[12.575026500000035,47.63231599900007],[12.338045499000032,47.697087500000066],[12.257177500000068,47.6791],[12.264886500000046,47.73248299800008],[12.183931,47.70037049800004],[12.20731950000004,47.609837998000046],[12.060662,47.61874349800007],[11.632883,47.59244599600004],[11.572426,47.514643998000054],[11.410219,47.495324],[11.38380950000004,47.47220749600007],[11.42142750000005,47.44485199800005],[11.016500500000063,47.396368],[10.991202500000043,47.396130999000036],[10.871043,47.48188199800006],[10.886199,47.53684799800004],[10.454439,47.55579699800006],[10.431323500000076,47.50367749600008],[10.475413,47.431702],[10.178353,47.270113998000056],[10.23393,47.38328599700003],[10.09985049900007,47.35476149700003],[10.09147050000007,47.45907599700007],[9.999526,47.48301699600006],[9.967813500000034,47.54624049600005],[9.765470500000049,47.588856],[9.736566500000038,47.534906498000055],[9.55872050000005,47.54189299800004],[9.692543,47.610768998000026],[9.840336,47.677096498000026],[10.077292500000055,47.63927449700003],[10.139437500000042,47.703273496000065],[10.064681,47.78588749900007],[10.131928500000072,47.82008749600004],[10.110136500000067,47.93714999800005],[10.104207,47.974358496000036],[10.114954500000067,47.97625750000003],[10.132387,48.01539999800008],[10.135728,48.02376149600008],[10.136457,48.10845949700007],[10.095357500000034,48.16401499800003],[9.997606,48.35011799800003],[10.032694500000048,48.45719649800003],[10.133895500000051,48.454871498000045],[10.23077950000004,48.510510999000076],[10.278268,48.51607449800008],[10.314184,48.55628950000005],[10.268706999000074,48.703546998000036],[10.361222500000054,48.65318649900007],[10.487258,48.696662],[10.423689,48.74449599600007],[10.45700050000005,48.917141498000035],[10.409849500000064,48.977435498000034],[10.256763500000034,49.05949499900004],[10.204997499000058,49.098118998000075],[10.248023,49.14928099900004],[10.124643500000047,49.20082450000007],[10.111966500000051,49.38491049800007],[10.161698,49.39549399800006],[10.094741500000055,49.447898500000065],[10.11832750000002,49.47316949800006],[10.083623,49.542955498000026],[9.92656099900006,49.48483550000003],[9.831804500000032,49.70380349800007],[9.632460499000047,49.69318599800005],[9.648736,49.79147749900005],[9.471497499000066,49.77972650000004],[9.295673,49.740530998000054],[9.399960500000077,49.72884049700008],[9.41092,49.66350799800006],[9.103006,49.57745599900005],[9.088071,49.689591998000026],[9.150809,49.742850496000074],[9.036080500000025,49.84650399700007],[9.05008,49.866315],[9.016088500000023,49.991340500000035],[9.048820500000033,49.99226549900004],[8.990559999000027,50.067119],[9.235982500000034,50.146280499000056],[9.404984500000069,50.08773449700004],[9.519177500000069,50.10585],[9.494769500000075,50.22667199600005],[9.623150999000075,50.22903999700003],[9.745881,50.31040799900006],[9.73291450000005,50.356149498000036],[9.771871,50.424859498000046],[9.935655,50.419606499000054],[10.041338500000052,50.51646949800005],[10.12726,50.559533]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Oberbayern","NUTS_NAME":"Oberbayern"},"geometry":{"type":"Polygon","coordinates":[[[11.516278,49.03202599800005],[11.599306500000068,48.95149699800004],[11.703917500000045,48.918352998000046],[11.719435,48.78234549800004],[11.655467500000043,48.74270899700008],[11.706735,48.61664649900007],[11.910186500000066,48.58523149900003],[11.960526,48.57636799800008],[12.017122500000028,48.43067599800003],[12.165274500000066,48.42035799900003],[12.250551,48.32194599700006],[12.483791,48.42967549800005],[12.60729150000003,48.35363199900007],[12.822942500000067,48.31532549900004],[12.94468450000005,48.20669249800005],[12.751555,48.112810497000055],[12.860181500000067,47.996639998000035],[12.875785500000063,47.962608998000064],[13.00389750000005,47.84796899600008],[12.905047,47.72364349900005],[13.047965,47.709118998000065],[13.097948,47.63371699800007],[13.046055500000023,47.52050249800004],[13.003285500000061,47.463374498000064],[12.81025,47.54706299800006],[12.77925650000003,47.579285500000026],[12.807303,47.61022749800003],[12.695795500000031,47.68222299800004],[12.575026500000035,47.63231599900007],[12.338045499000032,47.697087500000066],[12.257177500000068,47.6791],[12.264886500000046,47.73248299800008],[12.183931,47.70037049800004],[12.20731950000004,47.609837998000046],[12.060662,47.61874349800007],[11.632883,47.59244599600004],[11.572426,47.514643998000054],[11.410219,47.495324],[11.38380950000004,47.47220749600007],[11.42142750000005,47.44485199800005],[11.016500500000063,47.396368],[10.991202500000043,47.396130999000036],[10.871043,47.48188199800006],[10.886199,47.53684799800004],[10.872826,47.57245899800006],[10.954276500000049,47.61652049800006],[10.715888500000062,47.705480496000064],[10.767090500000052,47.843793498000025],[10.826458,47.99526099800005],[10.75635950000003,48.04034599800008],[10.801532,48.10417049900008],[10.786499500000048,48.15013799800005],[10.848869,48.147790996000026],[10.903411,48.236676500000044],[11.03451,48.192992],[11.061886,48.27419099800005],[11.116768500000035,48.26847799800004],[11.198315,48.386679998000034],[11.17824950000005,48.41746599900006],[11.311675,48.45227299800007],[11.295411,48.47457399900003],[11.12657450000006,48.55498750000004],[11.155116,48.59589999600007],[11.022986,48.62017149600007],[10.984102,48.69280199700006],[11.005941500000063,48.821882997000046],[10.936839500000076,48.86327599800006],[11.201364500000068,49.04655149800004],[11.327119,48.997929998000075],[11.40715750000004,49.03805599800006],[11.385714,49.078858],[11.538008999000056,49.06994849800003],[11.516278,49.03202599800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE211","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ingolstadt, Kreisfreie Stadt","NUTS_NAME":"Ingolstadt, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.489323500000069,48.74500399900006],[11.43311250000005,48.69937099900005],[11.255916,48.779428497000026],[11.406287500000076,48.81724850000006],[11.499217,48.78439849700004],[11.489323500000069,48.74500399900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE212","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"München, Kreisfreie Stadt","NUTS_NAME":"München, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.412451,48.12711499900007],[11.360877,48.15808899800004],[11.402939500000059,48.20312050000007],[11.499818,48.24814599800004],[11.638246500000037,48.22613499800008],[11.62949750000007,48.179193499000064],[11.723082500000032,48.136753498000076],[11.685725,48.077502999000046],[11.508732,48.06155399900007],[11.412451,48.12711499900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE213","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rosenheim, Kreisfreie Stadt","NUTS_NAME":"Rosenheim, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.151501,47.83940499600004],[12.054313500000035,47.817710498000054],[12.138233,47.886750498000026],[12.151501,47.83940499600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE214","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Altötting","NUTS_NAME":"Altötting"},"geometry":{"type":"Polygon","coordinates":[[[12.94468450000005,48.20669249800005],[12.751555,48.112810497000055],[12.61640250000005,48.04504099800005],[12.548177500000065,48.12501849800003],[12.566205,48.158884998000076],[12.565616,48.33433599800003],[12.60729150000003,48.35363199900007],[12.822942500000067,48.31532549900004],[12.94468450000005,48.20669249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE215","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Berchtesgadener Land","NUTS_NAME":"Berchtesgadener Land"},"geometry":{"type":"Polygon","coordinates":[[[13.046055500000023,47.52050249800004],[13.003285500000061,47.463374498000064],[12.81025,47.54706299800006],[12.77925650000003,47.579285500000026],[12.807303,47.61022749800003],[12.695795500000031,47.68222299800004],[12.848618500000043,47.75491950000003],[12.728451500000062,47.83982049800005],[12.875799500000028,47.87748399800006],[12.848342500000058,47.93570149900006],[12.875785500000063,47.962608998000064],[13.00389750000005,47.84796899600008],[12.905047,47.72364349900005],[13.047965,47.709118998000065],[13.097948,47.63371699800007],[13.046055500000023,47.52050249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE216","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bad Tölz-Wolfratshausen","NUTS_NAME":"Bad Tölz-Wolfratshausen"},"geometry":{"type":"Polygon","coordinates":[[[11.623461500000076,47.92032799800006],[11.66385850000006,47.88900149800003],[11.636795,47.791997],[11.679977,47.642390498000054],[11.597967500000038,47.591787999000076],[11.632883,47.59244599600004],[11.572426,47.514643998000054],[11.410219,47.495324],[11.262775500000032,47.58044199600005],[11.283516500000076,47.66511399900003],[11.34408,47.69369749700007],[11.412085,47.74695649600005],[11.32917150000003,47.82109150000008],[11.334016500000075,47.93359799600006],[11.428582,47.97854349800008],[11.623461500000076,47.92032799800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE217","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dachau","NUTS_NAME":"Dachau"},"geometry":{"type":"Polygon","coordinates":[[[11.493183,48.440510998000036],[11.57869850000003,48.29954599800004],[11.499818,48.24814599800004],[11.402939500000059,48.20312050000007],[11.116768500000035,48.26847799800004],[11.198315,48.386679998000034],[11.17824950000005,48.41746599900006],[11.311675,48.45227299800007],[11.493183,48.440510998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE218","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ebersberg","NUTS_NAME":"Ebersberg"},"geometry":{"type":"Polygon","coordinates":[[[12.043185,48.14167449900003],[12.085656500000027,48.12234399600004],[12.098838500000056,47.99276049900004],[11.847952500000076,47.948252998000044],[11.759641500000043,47.99708049800006],[11.787904,48.057165498000074],[11.756169500000055,48.22111149600005],[12.043185,48.14167449900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE219","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Eichstätt","NUTS_NAME":"Eichstätt"},"geometry":{"type":"Polygon","coordinates":[[[11.599306500000068,48.95149699800004],[11.703917500000045,48.918352998000046],[11.719435,48.78234549800004],[11.604577500000062,48.81383],[11.489323500000069,48.74500399900006],[11.499217,48.78439849700004],[11.406287500000076,48.81724850000006],[11.255916,48.779428497000026],[11.005941500000063,48.821882997000046],[10.936839500000076,48.86327599800006],[11.201364500000068,49.04655149800004],[11.327119,48.997929998000075],[11.40715750000004,49.03805599800006],[11.385714,49.078858],[11.538008999000056,49.06994849800003],[11.516278,49.03202599800005],[11.599306500000068,48.95149699800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Erding","NUTS_NAME":"Erding"},"geometry":{"type":"Polygon","coordinates":[[[12.017122500000028,48.43067599800003],[12.165274500000066,48.42035799900003],[12.250551,48.32194599700006],[12.20220450000005,48.21772199700007],[12.043185,48.14167449900003],[11.756169500000055,48.22111149600005],[11.738186,48.27633399800004],[11.807998500000053,48.38941049600004],[12.017122500000028,48.43067599800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Freising","NUTS_NAME":"Freising"},"geometry":{"type":"Polygon","coordinates":[[[11.910186500000066,48.58523149900003],[11.960526,48.57636799800008],[12.017122500000028,48.43067599800003],[11.807998500000053,48.38941049600004],[11.738186,48.27633399800004],[11.57869850000003,48.29954599800004],[11.493183,48.440510998000036],[11.662399,48.498518999000055],[11.706735,48.61664649900007],[11.910186500000066,48.58523149900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Fürstenfeldbruck","NUTS_NAME":"Fürstenfeldbruck"},"geometry":{"type":"Polygon","coordinates":[[[11.402939500000059,48.20312050000007],[11.360877,48.15808899800004],[11.412451,48.12711499900007],[11.385188500000027,48.11202799700004],[11.136971500000072,48.097616998000035],[11.055588500000056,48.103724998000075],[11.03451,48.192992],[11.061886,48.27419099800005],[11.116768500000035,48.26847799800004],[11.402939500000059,48.20312050000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Garmisch-Partenkirchen","NUTS_NAME":"Garmisch-Partenkirchen"},"geometry":{"type":"Polygon","coordinates":[[[11.34408,47.69369749700007],[11.283516500000076,47.66511399900003],[11.262775500000032,47.58044199600005],[11.410219,47.495324],[11.38380950000004,47.47220749600007],[11.42142750000005,47.44485199800005],[11.016500500000063,47.396368],[10.991202500000043,47.396130999000036],[10.871043,47.48188199800006],[10.886199,47.53684799800004],[10.872826,47.57245899800006],[10.954276500000049,47.61652049800006],[10.978436500000043,47.71426849800008],[11.34408,47.69369749700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Landsberg am Lech","NUTS_NAME":"Landsberg am Lech"},"geometry":{"type":"Polygon","coordinates":[[[11.03451,48.192992],[11.055588500000056,48.103724998000075],[11.136971500000072,48.097616998000035],[11.149588,47.953412498000034],[10.767090500000052,47.843793498000025],[10.826458,47.99526099800005],[10.75635950000003,48.04034599800008],[10.801532,48.10417049900008],[10.786499500000048,48.15013799800005],[10.848869,48.147790996000026],[10.903411,48.236676500000044],[11.03451,48.192992]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Miesbach","NUTS_NAME":"Miesbach"},"geometry":{"type":"Polygon","coordinates":[[[11.806467,47.92574749800008],[11.948146,47.84490099700008],[11.915883,47.81053049900004],[11.989375500000051,47.77680999800003],[12.060662,47.61874349800007],[11.632883,47.59244599600004],[11.597967500000038,47.591787999000076],[11.679977,47.642390498000054],[11.636795,47.791997],[11.66385850000006,47.88900149800003],[11.623461500000076,47.92032799800006],[11.806467,47.92574749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21G","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mühldorf a. Inn","NUTS_NAME":"Mühldorf a. Inn"},"geometry":{"type":"Polygon","coordinates":[[[12.60729150000003,48.35363199900007],[12.565616,48.33433599800003],[12.566205,48.158884998000076],[12.387555500000076,48.11737999800005],[12.085656500000027,48.12234399600004],[12.043185,48.14167449900003],[12.20220450000005,48.21772199700007],[12.250551,48.32194599700006],[12.483791,48.42967549800005],[12.60729150000003,48.35363199900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21H","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"München, Landkreis","NUTS_NAME":"München, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[11.738186,48.27633399800004],[11.756169500000055,48.22111149600005],[11.787904,48.057165498000074],[11.759641500000043,47.99708049800006],[11.847952500000076,47.948252998000044],[11.806467,47.92574749800008],[11.623461500000076,47.92032799800006],[11.428582,47.97854349800008],[11.424148,48.096415498000056],[11.385188500000027,48.11202799700004],[11.412451,48.12711499900007],[11.508732,48.06155399900007],[11.685725,48.077502999000046],[11.723082500000032,48.136753498000076],[11.62949750000007,48.179193499000064],[11.638246500000037,48.22613499800008],[11.499818,48.24814599800004],[11.57869850000003,48.29954599800004],[11.738186,48.27633399800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21I","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neuburg-Schrobenhausen","NUTS_NAME":"Neuburg-Schrobenhausen"},"geometry":{"type":"Polygon","coordinates":[[[11.255916,48.779428497000026],[11.43311250000005,48.69937099900005],[11.43873,48.646866],[11.345301,48.6065],[11.295411,48.47457399900003],[11.12657450000006,48.55498750000004],[11.155116,48.59589999600007],[11.022986,48.62017149600007],[10.984102,48.69280199700006],[11.005941500000063,48.821882997000046],[11.255916,48.779428497000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21J","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Pfaffenhofen a. d. Ilm","NUTS_NAME":"Pfaffenhofen a. d. Ilm"},"geometry":{"type":"Polygon","coordinates":[[[11.719435,48.78234549800004],[11.655467500000043,48.74270899700008],[11.706735,48.61664649900007],[11.662399,48.498518999000055],[11.493183,48.440510998000036],[11.311675,48.45227299800007],[11.295411,48.47457399900003],[11.345301,48.6065],[11.43873,48.646866],[11.43311250000005,48.69937099900005],[11.489323500000069,48.74500399900006],[11.604577500000062,48.81383],[11.719435,48.78234549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21K","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rosenheim, Landkreis","NUTS_NAME":"Rosenheim, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[12.387555500000076,48.11737999800005],[12.325560500000051,48.067578996000066],[12.357393500000057,48.04042449800005],[12.327091500000051,47.977539996000075],[12.444032,47.92210049800008],[12.36387,47.869927999000026],[12.389148,47.814127496000026],[12.338045499000032,47.697087500000066],[12.257177500000068,47.6791],[12.264886500000046,47.73248299800008],[12.183931,47.70037049800004],[12.20731950000004,47.609837998000046],[12.060662,47.61874349800007],[11.989375500000051,47.77680999800003],[11.915883,47.81053049900004],[11.948146,47.84490099700008],[11.806467,47.92574749800008],[11.847952500000076,47.948252998000044],[12.098838500000056,47.99276049900004],[12.085656500000027,48.12234399600004],[12.387555500000076,48.11737999800005]],[[12.151501,47.83940499600004],[12.138233,47.886750498000026],[12.054313500000035,47.817710498000054],[12.151501,47.83940499600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21L","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Starnberg","NUTS_NAME":"Starnberg"},"geometry":{"type":"Polygon","coordinates":[[[11.385188500000027,48.11202799700004],[11.424148,48.096415498000056],[11.428582,47.97854349800008],[11.334016500000075,47.93359799600006],[11.32917150000003,47.82109150000008],[11.149588,47.953412498000034],[11.136971500000072,48.097616998000035],[11.385188500000027,48.11202799700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21M","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Traunstein","NUTS_NAME":"Traunstein"},"geometry":{"type":"Polygon","coordinates":[[[12.751555,48.112810497000055],[12.860181500000067,47.996639998000035],[12.875785500000063,47.962608998000064],[12.848342500000058,47.93570149900006],[12.875799500000028,47.87748399800006],[12.728451500000062,47.83982049800005],[12.848618500000043,47.75491950000003],[12.695795500000031,47.68222299800004],[12.575026500000035,47.63231599900007],[12.338045499000032,47.697087500000066],[12.389148,47.814127496000026],[12.36387,47.869927999000026],[12.444032,47.92210049800008],[12.327091500000051,47.977539996000075],[12.357393500000057,48.04042449800005],[12.325560500000051,48.067578996000066],[12.387555500000076,48.11737999800005],[12.566205,48.158884998000076],[12.548177500000065,48.12501849800003],[12.61640250000005,48.04504099800005],[12.751555,48.112810497000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE21N","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Weilheim-Schongau","NUTS_NAME":"Weilheim-Schongau"},"geometry":{"type":"Polygon","coordinates":[[[11.32917150000003,47.82109150000008],[11.412085,47.74695649600005],[11.34408,47.69369749700007],[10.978436500000043,47.71426849800008],[10.954276500000049,47.61652049800006],[10.715888500000062,47.705480496000064],[10.767090500000052,47.843793498000025],[11.149588,47.953412498000034],[11.32917150000003,47.82109150000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE22","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Niederbayern","NUTS_NAME":"Niederbayern"},"geometry":{"type":"Polygon","coordinates":[[[13.170907999000065,49.17357950000007],[13.397147,49.05100999800004],[13.416651500000057,48.98003549800006],[13.551578500000062,48.96778749600003],[13.839507,48.771605],[13.796107,48.71360049800006],[13.825764,48.61861649800005],[13.727090500000031,48.51301950000004],[13.513369500000067,48.59097799700004],[13.438430500000038,48.548895],[13.407838,48.372160500000064],[13.177043,48.29438999600006],[12.94468450000005,48.20669249800005],[12.822942500000067,48.31532549900004],[12.60729150000003,48.35363199900007],[12.483791,48.42967549800005],[12.250551,48.32194599700006],[12.165274500000066,48.42035799900003],[12.017122500000028,48.43067599800003],[11.960526,48.57636799800008],[11.910186500000066,48.58523149900003],[11.706735,48.61664649900007],[11.655467500000043,48.74270899700008],[11.719435,48.78234549800004],[11.703917500000045,48.918352998000046],[11.599306500000068,48.95149699800004],[11.659785,49.016129499000044],[11.869344,49.02040649700007],[12.065426500000058,48.94369549900006],[12.133691500000054,48.87679399900003],[12.097496,48.77199199700004],[12.135276,48.765911998000036],[12.382227999000065,48.84956499800006],[12.44293,48.98196449800008],[12.489241500000048,48.98061849900006],[12.479026500000032,49.03198599800004],[12.768203,49.114760998000065],[12.971182,49.16640749700008],[13.133487,49.11175099800005],[13.170907999000065,49.17357950000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE221","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Landshut, Kreisfreie Stadt","NUTS_NAME":"Landshut, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.186271,48.519960998000045],[12.019374500000026,48.511909496000044],[12.282579,48.58944549900008],[12.186271,48.519960998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE222","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Passau, Kreisfreie Stadt","NUTS_NAME":"Passau, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[13.513369500000067,48.59097799700004],[13.438430500000038,48.548895],[13.302793,48.601886498000056],[13.513369500000067,48.59097799700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE223","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Straubing, Kreisfreie Stadt","NUTS_NAME":"Straubing, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.665930500000059,48.88035399800003],[12.480274500000064,48.87040199900008],[12.557331,48.91959699800003],[12.665930500000059,48.88035399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE224","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Deggendorf","NUTS_NAME":"Deggendorf"},"geometry":{"type":"Polygon","coordinates":[[[13.195463500000074,48.86277349800008],[13.258705,48.74106299700003],[12.994285,48.60061299900008],[12.967381,48.61126549900007],[12.796515500000055,48.71480249800004],[12.764000499000076,48.797911998000075],[12.848788500000069,48.83972199800007],[12.891426500000023,48.95447099900008],[13.195463500000074,48.86277349800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE225","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Freyung-Grafenau","NUTS_NAME":"Freyung-Grafenau"},"geometry":{"type":"Polygon","coordinates":[[[13.551578500000062,48.96778749600003],[13.839507,48.771605],[13.796107,48.71360049800006],[13.258705,48.74106299700003],[13.195463500000074,48.86277349800008],[13.285471,48.87787499900003],[13.283235499000057,48.95800550000007],[13.416651500000057,48.98003549800006],[13.551578500000062,48.96778749600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE226","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kelheim","NUTS_NAME":"Kelheim"},"geometry":{"type":"Polygon","coordinates":[[[12.097496,48.77199199700004],[11.906058500000029,48.70115449900004],[11.877926,48.653469],[11.910186500000066,48.58523149900003],[11.706735,48.61664649900007],[11.655467500000043,48.74270899700008],[11.719435,48.78234549800004],[11.703917500000045,48.918352998000046],[11.599306500000068,48.95149699800004],[11.659785,49.016129499000044],[11.869344,49.02040649700007],[12.065426500000058,48.94369549900006],[12.133691500000054,48.87679399900003],[12.097496,48.77199199700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE227","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Landshut, Landkreis","NUTS_NAME":"Landshut, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[12.135276,48.765911998000036],[12.329385500000058,48.72143499800006],[12.398478,48.64511749800005],[12.348959500000035,48.571834498000044],[12.495424999000022,48.55751799900003],[12.508154,48.50297499900006],[12.470559499000046,48.48638099900006],[12.483791,48.42967549800005],[12.250551,48.32194599700006],[12.165274500000066,48.42035799900003],[12.017122500000028,48.43067599800003],[11.960526,48.57636799800008],[11.910186500000066,48.58523149900003],[11.877926,48.653469],[11.906058500000029,48.70115449900004],[12.097496,48.77199199700004],[12.135276,48.765911998000036]],[[12.186271,48.519960998000045],[12.282579,48.58944549900008],[12.019374500000026,48.511909496000044],[12.186271,48.519960998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE228","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Passau, Landkreis","NUTS_NAME":"Passau, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[13.796107,48.71360049800006],[13.825764,48.61861649800005],[13.727090500000031,48.51301950000004],[13.513369500000067,48.59097799700004],[13.302793,48.601886498000056],[13.438430500000038,48.548895],[13.407838,48.372160500000064],[13.177043,48.29438999600006],[13.081070500000067,48.37012649800005],[13.174037,48.41342749700004],[13.085387500000024,48.48898999800008],[13.10613,48.53910449700004],[12.994285,48.60061299900008],[13.258705,48.74106299700003],[13.796107,48.71360049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE229","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Regen","NUTS_NAME":"Regen"},"geometry":{"type":"Polygon","coordinates":[[[13.416651500000057,48.98003549800006],[13.283235499000057,48.95800550000007],[13.285471,48.87787499900003],[13.195463500000074,48.86277349800008],[12.891426500000023,48.95447099900008],[12.768203,49.114760998000065],[12.971182,49.16640749700008],[13.133487,49.11175099800005],[13.170907999000065,49.17357950000007],[13.397147,49.05100999800004],[13.416651500000057,48.98003549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE22A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rottal-Inn","NUTS_NAME":"Rottal-Inn"},"geometry":{"type":"Polygon","coordinates":[[[12.967381,48.61126549900007],[12.994285,48.60061299900008],[13.10613,48.53910449700004],[13.085387500000024,48.48898999800008],[13.174037,48.41342749700004],[13.081070500000067,48.37012649800005],[13.177043,48.29438999600006],[12.94468450000005,48.20669249800005],[12.822942500000067,48.31532549900004],[12.60729150000003,48.35363199900007],[12.483791,48.42967549800005],[12.470559499000046,48.48638099900006],[12.508154,48.50297499900006],[12.967381,48.61126549900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE22B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Straubing-Bogen","NUTS_NAME":"Straubing-Bogen"},"geometry":{"type":"Polygon","coordinates":[[[12.768203,49.114760998000065],[12.891426500000023,48.95447099900008],[12.848788500000069,48.83972199800007],[12.764000499000076,48.797911998000075],[12.596174500000075,48.723711999000045],[12.431198,48.766751998000075],[12.329385500000058,48.72143499800006],[12.135276,48.765911998000036],[12.382227999000065,48.84956499800006],[12.44293,48.98196449800008],[12.489241500000048,48.98061849900006],[12.479026500000032,49.03198599800004],[12.768203,49.114760998000065]],[[12.665930500000059,48.88035399800003],[12.557331,48.91959699800003],[12.480274500000064,48.87040199900008],[12.665930500000059,48.88035399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE22C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dingolfing-Landau","NUTS_NAME":"Dingolfing-Landau"},"geometry":{"type":"Polygon","coordinates":[[[12.967381,48.61126549900007],[12.508154,48.50297499900006],[12.495424999000022,48.55751799900003],[12.348959500000035,48.571834498000044],[12.398478,48.64511749800005],[12.329385500000058,48.72143499800006],[12.431198,48.766751998000075],[12.596174500000075,48.723711999000045],[12.764000499000076,48.797911998000075],[12.796515500000055,48.71480249800004],[12.967381,48.61126549900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE23","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Oberpfalz","NUTS_NAME":"Oberpfalz"},"geometry":{"type":"Polygon","coordinates":[[[12.633763500000043,49.47619499700005],[12.77881,49.34807849600003],[12.972595500000068,49.33388799800008],[13.170907999000065,49.17357950000007],[13.133487,49.11175099800005],[12.971182,49.16640749700008],[12.768203,49.114760998000065],[12.479026500000032,49.03198599800004],[12.489241500000048,48.98061849900006],[12.44293,48.98196449800008],[12.382227999000065,48.84956499800006],[12.135276,48.765911998000036],[12.097496,48.77199199700004],[12.133691500000054,48.87679399900003],[12.065426500000058,48.94369549900006],[11.869344,49.02040649700007],[11.659785,49.016129499000044],[11.599306500000068,48.95149699800004],[11.516278,49.03202599800005],[11.538008999000056,49.06994849800003],[11.385714,49.078858],[11.269556,49.18877449800004],[11.311939,49.20688949600003],[11.292171,49.26712099700006],[11.185988500000065,49.30040149600006],[11.247176,49.325352999000074],[11.326181,49.30242749800004],[11.461613999000065,49.42276449800005],[11.557755,49.419042],[11.596248,49.474094499000046],[11.510854,49.55320899900005],[11.580373500000064,49.616895499000066],[11.562709118000043,49.677272059000074],[11.630662500000028,49.76057099800005],[11.854835500000036,49.84710899700008],[11.811642500000062,49.96063449600007],[11.903691500000036,49.97902],[12.198293500000034,49.999184498000034],[12.260799,50.05815649700003],[12.48988950000006,49.98176099900007],[12.470510500000046,49.94502550000004],[12.550988500000074,49.905088],[12.401524,49.75837299700004],[12.521638,49.686761999000055],[12.593778500000042,49.54219149800008],[12.644176,49.52305749900006],[12.633763500000043,49.47619499700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE231","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Amberg, Kreisfreie Stadt","NUTS_NAME":"Amberg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.914553499000021,49.44512549800004],[11.836434,49.41182599900003],[11.778660500000058,49.46419399800004],[11.914553499000021,49.44512549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE232","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Regensburg, Kreisfreie Stadt","NUTS_NAME":"Regensburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.175445500000023,48.97284749800008],[12.033177,49.01512149800004],[12.139454,49.068055998000034],[12.175445500000023,48.97284749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE233","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Weiden i. d. Opf, Kreisfreie Stadt","NUTS_NAME":"Weiden i. d. Opf, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.143612500000074,49.70794850000004],[12.26463,49.65741449800004],[12.129003,49.61459199700005],[12.063970500000039,49.67415949600007],[12.143612500000074,49.70794850000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE234","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Amberg-Sulzbach","NUTS_NAME":"Amberg-Sulzbach"},"geometry":{"type":"Polygon","coordinates":[[[12.11549,49.54795499800008],[12.032832,49.42971399900006],[11.976839,49.43692199800006],[12.062225,49.352929998000036],[12.001068,49.342218496000044],[11.934220500000038,49.22107450000004],[11.557755,49.419042],[11.596248,49.474094499000046],[11.510854,49.55320899900005],[11.580373500000064,49.616895499000066],[11.562709118000043,49.677272059000074],[11.630662500000028,49.76057099800005],[11.641755,49.62976499800004],[11.974396500000068,49.64015249800008],[11.977528500000062,49.583756496000035],[12.11549,49.54795499800008]],[[11.914553499000021,49.44512549800004],[11.778660500000058,49.46419399800004],[11.836434,49.41182599900003],[11.914553499000021,49.44512549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE235","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Cham","NUTS_NAME":"Cham"},"geometry":{"type":"Polygon","coordinates":[[[12.633763500000043,49.47619499700005],[12.77881,49.34807849600003],[12.972595500000068,49.33388799800008],[13.170907999000065,49.17357950000007],[13.133487,49.11175099800005],[12.971182,49.16640749700008],[12.768203,49.114760998000065],[12.479026500000032,49.03198599800004],[12.321756500000049,49.09059449800003],[12.311049500000024,49.152472],[12.47927,49.27254299800006],[12.510248500000046,49.31586099700007],[12.485326,49.39394499800005],[12.543421999000032,49.46122799900007],[12.633763500000043,49.47619499700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE236","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neumarkt i. d. OPf.","NUTS_NAME":"Neumarkt i. d. OPf."},"geometry":{"type":"Polygon","coordinates":[[[11.557755,49.419042],[11.934220500000038,49.22107450000004],[11.935903500000052,49.195070498000064],[11.71246750000006,49.09641350000004],[11.659785,49.016129499000044],[11.599306500000068,48.95149699800004],[11.516278,49.03202599800005],[11.538008999000056,49.06994849800003],[11.385714,49.078858],[11.269556,49.18877449800004],[11.311939,49.20688949600003],[11.292171,49.26712099700006],[11.185988500000065,49.30040149600006],[11.247176,49.325352999000074],[11.326181,49.30242749800004],[11.461613999000065,49.42276449800005],[11.557755,49.419042]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE237","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neustadt a. d. Waldnaab","NUTS_NAME":"Neustadt a. d. Waldnaab"},"geometry":{"type":"Polygon","coordinates":[[[12.401524,49.75837299700004],[12.521638,49.686761999000055],[12.593778500000042,49.54219149800008],[12.11549,49.54795499800008],[11.977528500000062,49.583756496000035],[11.974396500000068,49.64015249800008],[11.641755,49.62976499800004],[11.630662500000028,49.76057099800005],[11.854835500000036,49.84710899700008],[12.078012,49.783974],[12.174610500000028,49.85560849900003],[12.23673350000007,49.77110150000004],[12.401524,49.75837299700004]],[[12.143612500000074,49.70794850000004],[12.063970500000039,49.67415949600007],[12.129003,49.61459199700005],[12.26463,49.65741449800004],[12.143612500000074,49.70794850000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE238","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Regensburg, Landkreis","NUTS_NAME":"Regensburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[12.311049500000024,49.152472],[12.321756500000049,49.09059449800003],[12.479026500000032,49.03198599800004],[12.489241500000048,48.98061849900006],[12.44293,48.98196449800008],[12.382227999000065,48.84956499800006],[12.135276,48.765911998000036],[12.097496,48.77199199700004],[12.133691500000054,48.87679399900003],[12.065426500000058,48.94369549900006],[11.869344,49.02040649700007],[11.659785,49.016129499000044],[11.71246750000006,49.09641350000004],[11.935903500000052,49.195070498000064],[12.098483500000043,49.14335249800007],[12.17211750000007,49.22262249700003],[12.311049500000024,49.152472]],[[12.175445500000023,48.97284749800008],[12.139454,49.068055998000034],[12.033177,49.01512149800004],[12.175445500000023,48.97284749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE239","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwandorf","NUTS_NAME":"Schwandorf"},"geometry":{"type":"Polygon","coordinates":[[[12.593778500000042,49.54219149800008],[12.644176,49.52305749900006],[12.633763500000043,49.47619499700005],[12.543421999000032,49.46122799900007],[12.485326,49.39394499800005],[12.510248500000046,49.31586099700007],[12.47927,49.27254299800006],[12.311049500000024,49.152472],[12.17211750000007,49.22262249700003],[12.098483500000043,49.14335249800007],[11.935903500000052,49.195070498000064],[11.934220500000038,49.22107450000004],[12.001068,49.342218496000044],[12.062225,49.352929998000036],[11.976839,49.43692199800006],[12.032832,49.42971399900006],[12.11549,49.54795499800008],[12.593778500000042,49.54219149800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE23A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Tirschenreuth","NUTS_NAME":"Tirschenreuth"},"geometry":{"type":"Polygon","coordinates":[[[12.550988500000074,49.905088],[12.401524,49.75837299700004],[12.23673350000007,49.77110150000004],[12.174610500000028,49.85560849900003],[12.078012,49.783974],[11.854835500000036,49.84710899700008],[11.811642500000062,49.96063449600007],[11.903691500000036,49.97902],[12.198293500000034,49.999184498000034],[12.260799,50.05815649700003],[12.48988950000006,49.98176099900007],[12.470510500000046,49.94502550000004],[12.550988500000074,49.905088]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE24","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Oberfranken","NUTS_NAME":"Oberfranken"},"geometry":{"type":"Polygon","coordinates":[[[11.481568,50.43162150000006],[11.519853,50.37426849800005],[11.603290500000071,50.398766],[11.919884500000023,50.42440349800006],[12.100900500000023,50.31802799700006],[12.13994,50.27771849800007],[12.09076,50.25238450000006],[12.160679,50.21984899700004],[12.260799,50.05815649700003],[12.198293500000034,49.999184498000034],[11.903691500000036,49.97902],[11.811642500000062,49.96063449600007],[11.854835500000036,49.84710899700008],[11.630662500000028,49.76057099800005],[11.562709118000043,49.677272059000074],[11.426542,49.61387550000006],[11.368694,49.666708],[11.280304500000057,49.60430149900003],[11.087569,49.60001499800006],[10.93238000100007,49.701340499000025],[10.927335500000027,49.768429999000034],[10.720849500000043,49.78820049900003],[10.654417500000022,49.721123498000054],[10.551436,49.755772499000045],[10.448965,49.822177],[10.50524850000005,49.87668199800004],[10.682689500000038,49.88200549800007],[10.761121499000069,49.96824199800005],[10.751062,50.020492998000066],[10.86011650000006,50.09178549600006],[10.729202,50.23000549600005],[10.851494500000058,50.26276249800003],[10.715266500000041,50.363590997000074],[10.945718500000055,50.38646649800006],[11.115305,50.36700699600004],[11.189943,50.27118550000006],[11.259778,50.27494649800008],[11.265938,50.47942099900007],[11.423893,50.51606449800005],[11.420630500000073,50.443366499000035],[11.481568,50.43162150000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE241","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bamberg, Kreisfreie Stadt","NUTS_NAME":"Bamberg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.95114,49.85599449800003],[10.830088,49.914629500000046],[10.949447500000076,49.90070749600005],[10.95114,49.85599449800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE242","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bayreuth, Kreisfreie Stadt","NUTS_NAME":"Bayreuth, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.561262500000055,49.97465049600004],[11.672539500000028,49.93489499800006],[11.554737,49.89265899800006],[11.527064,49.93654249800005],[11.561262500000055,49.97465049600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE243","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Coburg, Kreisfreie Stadt","NUTS_NAME":"Coburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.063743500000044,50.265046],[10.892022500000053,50.25061849800005],[10.953553,50.295622500000036],[11.063743500000044,50.265046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE244","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hof, Kreisfreie Stadt","NUTS_NAME":"Hof, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.97191950000007,50.31580549600005],[11.897174500000062,50.270222998000065],[11.808415500000024,50.308335998000075],[11.97191950000007,50.31580549600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE245","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bamberg, Landkreis","NUTS_NAME":"Bamberg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[11.246398,49.98956299900004],[11.184316,49.90884099900006],[11.23811,49.870473],[10.927335500000027,49.768429999000034],[10.720849500000043,49.78820049900003],[10.654417500000022,49.721123498000054],[10.551436,49.755772499000045],[10.448965,49.822177],[10.50524850000005,49.87668199800004],[10.682689500000038,49.88200549800007],[10.761121499000069,49.96824199800005],[10.751062,50.020492998000066],[10.86011650000006,50.09178549600006],[10.884773500000051,50.09314249800008],[10.993956,50.02974099800008],[11.185557,50.057525498000075],[11.147398999000075,50.021425],[11.246398,49.98956299900004]],[[10.95114,49.85599449800003],[10.949447500000076,49.90070749600005],[10.830088,49.914629500000046],[10.95114,49.85599449800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE246","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bayreuth, Landkreis","NUTS_NAME":"Bayreuth, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[11.789114,50.10324799600005],[11.903691500000036,49.97902],[11.811642500000062,49.96063449600007],[11.854835500000036,49.84710899700008],[11.630662500000028,49.76057099800005],[11.562709118000043,49.677272059000074],[11.426542,49.61387550000006],[11.368694,49.666708],[11.366949500000032,49.80339749700005],[11.23811,49.870473],[11.184316,49.90884099900006],[11.246398,49.98956299900004],[11.256807500000036,49.99995849800007],[11.38824,49.936386998000046],[11.647582500000055,50.04200250000008],[11.678085,50.12460249800006],[11.789114,50.10324799600005]],[[11.561262500000055,49.97465049600004],[11.527064,49.93654249800005],[11.554737,49.89265899800006],[11.672539500000028,49.93489499800006],[11.561262500000055,49.97465049600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE247","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Coburg, Landkreis","NUTS_NAME":"Coburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.945718500000055,50.38646649800006],[11.115305,50.36700699600004],[11.189943,50.27118550000006],[11.16854150000006,50.206447499000035],[11.198185,50.20209550000004],[10.985359500000072,50.211697496000056],[11.00927,50.179688498000075],[10.884773500000051,50.09314249800008],[10.86011650000006,50.09178549600006],[10.729202,50.23000549600005],[10.851494500000058,50.26276249800003],[10.715266500000041,50.363590997000074],[10.945718500000055,50.38646649800006]],[[11.063743500000044,50.265046],[10.953553,50.295622500000036],[10.892022500000053,50.25061849800005],[11.063743500000044,50.265046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE248","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Forchheim","NUTS_NAME":"Forchheim"},"geometry":{"type":"Polygon","coordinates":[[[11.368694,49.666708],[11.280304500000057,49.60430149900003],[11.087569,49.60001499800006],[10.93238000100007,49.701340499000025],[10.927335500000027,49.768429999000034],[11.23811,49.870473],[11.366949500000032,49.80339749700005],[11.368694,49.666708]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE249","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hof, Landkreis","NUTS_NAME":"Hof, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[12.100900500000023,50.31802799700006],[12.13994,50.27771849800007],[12.09076,50.25238450000006],[12.160679,50.21984899700004],[11.789114,50.10324799600005],[11.678085,50.12460249800006],[11.645904,50.121590999000034],[11.68976,50.16025049900003],[11.659743,50.22472699600007],[11.598383,50.22494449800007],[11.624133,50.26370299600006],[11.541264,50.265412],[11.603290500000071,50.398766],[11.919884500000023,50.42440349800006],[12.100900500000023,50.31802799700006]],[[11.97191950000007,50.31580549600005],[11.808415500000024,50.308335998000075],[11.897174500000062,50.270222998000065],[11.97191950000007,50.31580549600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE24A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kronach","NUTS_NAME":"Kronach"},"geometry":{"type":"Polygon","coordinates":[[[11.481568,50.43162150000006],[11.519853,50.37426849800005],[11.603290500000071,50.398766],[11.541264,50.265412],[11.342196,50.16031249800005],[11.198185,50.20209550000004],[11.16854150000006,50.206447499000035],[11.189943,50.27118550000006],[11.259778,50.27494649800008],[11.265938,50.47942099900007],[11.423893,50.51606449800005],[11.420630500000073,50.443366499000035],[11.481568,50.43162150000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE24B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kulmbach","NUTS_NAME":"Kulmbach"},"geometry":{"type":"Polygon","coordinates":[[[11.678085,50.12460249800006],[11.647582500000055,50.04200250000008],[11.38824,49.936386998000046],[11.256807500000036,49.99995849800007],[11.342196,50.16031249800005],[11.541264,50.265412],[11.624133,50.26370299600006],[11.598383,50.22494449800007],[11.659743,50.22472699600007],[11.68976,50.16025049900003],[11.645904,50.121590999000034],[11.678085,50.12460249800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE24C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lichtenfels","NUTS_NAME":"Lichtenfels"},"geometry":{"type":"Polygon","coordinates":[[[11.198185,50.20209550000004],[11.342196,50.16031249800005],[11.256807500000036,49.99995849800007],[11.246398,49.98956299900004],[11.147398999000075,50.021425],[11.185557,50.057525498000075],[10.993956,50.02974099800008],[10.884773500000051,50.09314249800008],[11.00927,50.179688498000075],[10.985359500000072,50.211697496000056],[11.198185,50.20209550000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE24D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wunsiedel i. Fichtelgebirge","NUTS_NAME":"Wunsiedel i. Fichtelgebirge"},"geometry":{"type":"Polygon","coordinates":[[[12.260799,50.05815649700003],[12.198293500000034,49.999184498000034],[11.903691500000036,49.97902],[11.789114,50.10324799600005],[12.160679,50.21984899700004],[12.260799,50.05815649700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE25","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Mittelfranken","NUTS_NAME":"Mittelfranken"},"geometry":{"type":"Polygon","coordinates":[[[10.927335500000027,49.768429999000034],[10.93238000100007,49.701340499000025],[11.087569,49.60001499800006],[11.280304500000057,49.60430149900003],[11.368694,49.666708],[11.426542,49.61387550000006],[11.562709118000043,49.677272059000074],[11.580373500000064,49.616895499000066],[11.510854,49.55320899900005],[11.596248,49.474094499000046],[11.557755,49.419042],[11.461613999000065,49.42276449800005],[11.326181,49.30242749800004],[11.247176,49.325352999000074],[11.185988500000065,49.30040149600006],[11.292171,49.26712099700006],[11.311939,49.20688949600003],[11.269556,49.18877449800004],[11.385714,49.078858],[11.40715750000004,49.03805599800006],[11.327119,48.997929998000075],[11.201364500000068,49.04655149800004],[10.936839500000076,48.86327599800006],[10.815371,48.90109049800003],[10.817673500000069,48.949463499000046],[10.680472500000064,48.913612998000076],[10.642254500000035,49.016631500000074],[10.409849500000064,48.977435498000034],[10.256763500000034,49.05949499900004],[10.204997499000058,49.098118998000075],[10.248023,49.14928099900004],[10.124643500000047,49.20082450000007],[10.111966500000051,49.38491049800007],[10.161698,49.39549399800006],[10.094741500000055,49.447898500000065],[10.11832750000002,49.47316949800006],[10.083623,49.542955498000026],[10.110316,49.62140849900004],[10.388013,49.634258497000076],[10.338674,49.704163996000034],[10.551436,49.755772499000045],[10.654417500000022,49.721123498000054],[10.720849500000043,49.78820049900003],[10.927335500000027,49.768429999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE251","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ansbach, Kreisfreie Stadt","NUTS_NAME":"Ansbach, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.645879500000035,49.25732449800006],[10.454278500000044,49.285394],[10.54715,49.34016599700004],[10.655179,49.319955],[10.611443,49.282352496000044],[10.645879500000035,49.25732449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE252","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Erlangen, Kreisfreie Stadt","NUTS_NAME":"Erlangen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.030211500000064,49.53537149700003],[10.987928,49.53678549800003],[10.934024,49.53446850000006],[10.932828,49.53825149600004],[10.921724,49.63509399800006],[11.04954250000003,49.60815899800008],[11.030211500000064,49.53537149700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE253","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Fürth, Kreisfreie Stadt","NUTS_NAME":"Fürth, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.99045299900007,49.43653999900005],[10.90723,49.467187498000044],[10.934024,49.53446850000006],[10.987928,49.53678549800003],[11.026713500000028,49.47748849800007],[10.99045299900007,49.43653999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE254","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nürnberg, Kreisfreie Stadt","NUTS_NAME":"Nürnberg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.030211500000064,49.53537149700003],[11.178176,49.47408899800007],[11.128309,49.442360499000074],[11.169249,49.388472497000066],[11.07981,49.33927550000004],[10.99779650000005,49.37588499600008],[10.992715500000031,49.381493498000054],[11.02621,49.40277049900004],[10.99045299900007,49.43653999900005],[11.026713500000028,49.47748849800007],[10.987928,49.53678549800003],[11.030211500000064,49.53537149700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE255","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwabach, Kreisfreie Stadt","NUTS_NAME":"Schwabach, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.07981,49.33927550000004],[11.014224,49.294058998000025],[10.976629,49.32532199700006],[10.99779650000005,49.37588499600008],[11.07981,49.33927550000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE256","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ansbach, Landkreis","NUTS_NAME":"Ansbach, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.697351500000025,49.43627349800005],[10.867839,49.35612649800004],[10.905277500000068,49.25266749600007],[10.837641,49.19775099700007],[10.655083500000046,49.13264249800005],[10.687758,49.047082],[10.642254500000035,49.016631500000074],[10.409849500000064,48.977435498000034],[10.256763500000034,49.05949499900004],[10.204997499000058,49.098118998000075],[10.248023,49.14928099900004],[10.124643500000047,49.20082450000007],[10.111966500000051,49.38491049800007],[10.161698,49.39549399800006],[10.094741500000055,49.447898500000065],[10.11832750000002,49.47316949800006],[10.239222,49.48247649800004],[10.29538,49.412622499000065],[10.697351500000025,49.43627349800005]],[[10.645879500000035,49.25732449800006],[10.611443,49.282352496000044],[10.655179,49.319955],[10.54715,49.34016599700004],[10.454278500000044,49.285394],[10.645879500000035,49.25732449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE257","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Erlangen-Höchstadt","NUTS_NAME":"Erlangen-Höchstadt"},"geometry":{"type":"Polygon","coordinates":[[[10.927335500000027,49.768429999000034],[10.93238000100007,49.701340499000025],[11.087569,49.60001499800006],[11.280304500000057,49.60430149900003],[11.191064500000039,49.56078999600004],[11.178176,49.47408899800007],[11.030211500000064,49.53537149700003],[11.04954250000003,49.60815899800008],[10.921724,49.63509399800006],[10.932828,49.53825149600004],[10.815766,49.535607500000026],[10.754441499000052,49.59155649600007],[10.781884,49.65722799900004],[10.594371,49.69161749800003],[10.654417500000022,49.721123498000054],[10.720849500000043,49.78820049900003],[10.927335500000027,49.768429999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE258","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Fürth, Landkreis","NUTS_NAME":"Fürth, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.932828,49.53825149600004],[10.934024,49.53446850000006],[10.90723,49.467187498000044],[10.99045299900007,49.43653999900005],[11.02621,49.40277049900004],[10.992715500000031,49.381493498000054],[10.867839,49.35612649800004],[10.697351500000025,49.43627349800005],[10.680367999000055,49.49417249900006],[10.815766,49.535607500000026],[10.932828,49.53825149600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE259","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nürnberger Land","NUTS_NAME":"Nürnberger Land"},"geometry":{"type":"Polygon","coordinates":[[[11.557755,49.419042],[11.461613999000065,49.42276449800005],[11.326181,49.30242749800004],[11.247176,49.325352999000074],[11.169249,49.388472497000066],[11.128309,49.442360499000074],[11.178176,49.47408899800007],[11.191064500000039,49.56078999600004],[11.280304500000057,49.60430149900003],[11.368694,49.666708],[11.426542,49.61387550000006],[11.562709118000043,49.677272059000074],[11.580373500000064,49.616895499000066],[11.510854,49.55320899900005],[11.596248,49.474094499000046],[11.557755,49.419042]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE25A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neustadt a. d. Aisch-Bad Windsheim","NUTS_NAME":"Neustadt a. d. Aisch-Bad Windsheim"},"geometry":{"type":"Polygon","coordinates":[[[10.654417500000022,49.721123498000054],[10.594371,49.69161749800003],[10.781884,49.65722799900004],[10.754441499000052,49.59155649600007],[10.815766,49.535607500000026],[10.680367999000055,49.49417249900006],[10.697351500000025,49.43627349800005],[10.29538,49.412622499000065],[10.239222,49.48247649800004],[10.11832750000002,49.47316949800006],[10.083623,49.542955498000026],[10.110316,49.62140849900004],[10.388013,49.634258497000076],[10.338674,49.704163996000034],[10.551436,49.755772499000045],[10.654417500000022,49.721123498000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE25B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Roth","NUTS_NAME":"Roth"},"geometry":{"type":"Polygon","coordinates":[[[11.169249,49.388472497000066],[11.247176,49.325352999000074],[11.185988500000065,49.30040149600006],[11.292171,49.26712099700006],[11.311939,49.20688949600003],[11.269556,49.18877449800004],[11.385714,49.078858],[11.40715750000004,49.03805599800006],[11.327119,48.997929998000075],[11.201364500000068,49.04655149800004],[10.837641,49.19775099700007],[10.905277500000068,49.25266749600007],[10.867839,49.35612649800004],[10.992715500000031,49.381493498000054],[10.99779650000005,49.37588499600008],[10.976629,49.32532199700006],[11.014224,49.294058998000025],[11.07981,49.33927550000004],[11.169249,49.388472497000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE25C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Weißenburg-Gunzenhausen","NUTS_NAME":"Weißenburg-Gunzenhausen"},"geometry":{"type":"Polygon","coordinates":[[[11.201364500000068,49.04655149800004],[10.936839500000076,48.86327599800006],[10.815371,48.90109049800003],[10.817673500000069,48.949463499000046],[10.680472500000064,48.913612998000076],[10.642254500000035,49.016631500000074],[10.687758,49.047082],[10.655083500000046,49.13264249800005],[10.837641,49.19775099700007],[11.201364500000068,49.04655149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE26","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Unterfranken","NUTS_NAME":"Unterfranken"},"geometry":{"type":"Polygon","coordinates":[[[10.12726,50.559533],[10.450532,50.40185949900007],[10.600550500000054,50.33368799800007],[10.610115,50.22799499800004],[10.729202,50.23000549600005],[10.86011650000006,50.09178549600006],[10.751062,50.020492998000066],[10.761121499000069,49.96824199800005],[10.682689500000038,49.88200549800007],[10.50524850000005,49.87668199800004],[10.448965,49.822177],[10.551436,49.755772499000045],[10.338674,49.704163996000034],[10.388013,49.634258497000076],[10.110316,49.62140849900004],[10.083623,49.542955498000026],[9.92656099900006,49.48483550000003],[9.831804500000032,49.70380349800007],[9.632460499000047,49.69318599800005],[9.648736,49.79147749900005],[9.471497499000066,49.77972650000004],[9.295673,49.740530998000054],[9.399960500000077,49.72884049700008],[9.41092,49.66350799800006],[9.103006,49.57745599900005],[9.088071,49.689591998000026],[9.150809,49.742850496000074],[9.036080500000025,49.84650399700007],[9.05008,49.866315],[9.016088500000023,49.991340500000035],[9.048820500000033,49.99226549900004],[8.990559999000027,50.067119],[9.235982500000034,50.146280499000056],[9.404984500000069,50.08773449700004],[9.519177500000069,50.10585],[9.494769500000075,50.22667199600005],[9.623150999000075,50.22903999700003],[9.745881,50.31040799900006],[9.73291450000005,50.356149498000036],[9.771871,50.424859498000046],[9.935655,50.419606499000054],[10.041338500000052,50.51646949800005],[10.12726,50.559533]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE261","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Aschaffenburg, Kreisfreie Stadt","NUTS_NAME":"Aschaffenburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[9.237795500000061,49.93480049800007],[9.10724,49.93085849800008],[9.080084,50.007749999000055],[9.237795500000061,49.93480049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE262","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schweinfurt, Kreisfreie Stadt","NUTS_NAME":"Schweinfurt, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.274649,50.05810999800008],[10.165328500000044,50.031192],[10.221836500000052,50.09134649600003],[10.274649,50.05810999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE263","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Würzburg, Kreisfreie Stadt","NUTS_NAME":"Würzburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.013277500000072,49.804249997000056],[9.982376,49.71344349800006],[9.876827,49.75309249700007],[9.872402500000021,49.82224599800003],[10.013277500000072,49.804249997000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE264","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Aschaffenburg, Landkreis","NUTS_NAME":"Aschaffenburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.404984500000069,50.08773449700004],[9.385584,50.05578349600006],[9.480371,49.94413149800005],[9.422738,49.88113799800004],[9.318481499000029,49.835274999000035],[9.237795500000061,49.93480049800007],[9.080084,50.007749999000055],[9.10724,49.93085849800008],[9.05008,49.866315],[9.016088500000023,49.991340500000035],[9.048820500000033,49.99226549900004],[8.990559999000027,50.067119],[9.235982500000034,50.146280499000056],[9.404984500000069,50.08773449700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE265","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bad Kissingen","NUTS_NAME":"Bad Kissingen"},"geometry":{"type":"Polygon","coordinates":[[[9.935655,50.419606499000054],[9.975419500000044,50.31677949600004],[10.16732599900007,50.26180999800005],[10.325891,50.30278349900004],[10.370134,50.22916199900004],[10.328379,50.14810399800007],[10.179863500000067,50.153876],[9.964013,50.05471699800006],[9.808532,50.04696249700004],[9.831141500000058,50.08539099700005],[9.623150999000075,50.22903999700003],[9.745881,50.31040799900006],[9.73291450000005,50.356149498000036],[9.771871,50.424859498000046],[9.935655,50.419606499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE266","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhön-Grabfeld","NUTS_NAME":"Rhön-Grabfeld"},"geometry":{"type":"Polygon","coordinates":[[[10.450532,50.40185949900007],[10.600550500000054,50.33368799800007],[10.610115,50.22799499800004],[10.450536500000055,50.19670849800008],[10.370134,50.22916199900004],[10.325891,50.30278349900004],[10.16732599900007,50.26180999800005],[9.975419500000044,50.31677949600004],[9.935655,50.419606499000054],[10.041338500000052,50.51646949800005],[10.12726,50.559533],[10.450532,50.40185949900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE267","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Haßberge","NUTS_NAME":"Haßberge"},"geometry":{"type":"Polygon","coordinates":[[[10.610115,50.22799499800004],[10.729202,50.23000549600005],[10.86011650000006,50.09178549600006],[10.751062,50.020492998000066],[10.761121499000069,49.96824199800005],[10.682689500000038,49.88200549800007],[10.50524850000005,49.87668199800004],[10.457201500000053,49.89507299700006],[10.514161,49.93242649600006],[10.320344500000033,50.029543998000065],[10.433131,50.07610899600007],[10.393697,50.10494250000005],[10.450536500000055,50.19670849800008],[10.610115,50.22799499800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE268","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kitzingen","NUTS_NAME":"Kitzingen"},"geometry":{"type":"Polygon","coordinates":[[[10.448965,49.822177],[10.551436,49.755772499000045],[10.338674,49.704163996000034],[10.388013,49.634258497000076],[10.110316,49.62140849900004],[10.082171500000072,49.64741549800004],[10.127669500000025,49.68680799900005],[10.04051,49.74661299600007],[10.065310500000066,49.825352],[10.189162,49.892056498000045],[10.448965,49.822177]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE269","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Miltenberg","NUTS_NAME":"Miltenberg"},"geometry":{"type":"Polygon","coordinates":[[[9.422738,49.88113799800004],[9.471497499000066,49.77972650000004],[9.295673,49.740530998000054],[9.399960500000077,49.72884049700008],[9.41092,49.66350799800006],[9.103006,49.57745599900005],[9.088071,49.689591998000026],[9.150809,49.742850496000074],[9.036080500000025,49.84650399700007],[9.05008,49.866315],[9.10724,49.93085849800008],[9.237795500000061,49.93480049800007],[9.318481499000029,49.835274999000035],[9.422738,49.88113799800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE26A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Main-Spessart","NUTS_NAME":"Main-Spessart"},"geometry":{"type":"Polygon","coordinates":[[[9.623150999000075,50.22903999700003],[9.831141500000058,50.08539099700005],[9.808532,50.04696249700004],[9.964013,50.05471699800006],[10.039553500000068,50.033777],[10.030919,49.95219999900007],[9.648736,49.79147749900005],[9.471497499000066,49.77972650000004],[9.422738,49.88113799800004],[9.480371,49.94413149800005],[9.385584,50.05578349600006],[9.404984500000069,50.08773449700004],[9.519177500000069,50.10585],[9.494769500000075,50.22667199600005],[9.623150999000075,50.22903999700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE26B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schweinfurt, Landkreis","NUTS_NAME":"Schweinfurt, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.450536500000055,50.19670849800008],[10.393697,50.10494250000005],[10.433131,50.07610899600007],[10.320344500000033,50.029543998000065],[10.514161,49.93242649600006],[10.457201500000053,49.89507299700006],[10.50524850000005,49.87668199800004],[10.448965,49.822177],[10.189162,49.892056498000045],[10.030919,49.95219999900007],[10.039553500000068,50.033777],[9.964013,50.05471699800006],[10.179863500000067,50.153876],[10.328379,50.14810399800007],[10.370134,50.22916199900004],[10.450536500000055,50.19670849800008]],[[10.274649,50.05810999800008],[10.221836500000052,50.09134649600003],[10.165328500000044,50.031192],[10.274649,50.05810999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE26C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Würzburg, Landkreis","NUTS_NAME":"Würzburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.189162,49.892056498000045],[10.065310500000066,49.825352],[10.04051,49.74661299600007],[10.127669500000025,49.68680799900005],[10.082171500000072,49.64741549800004],[10.110316,49.62140849900004],[10.083623,49.542955498000026],[9.92656099900006,49.48483550000003],[9.831804500000032,49.70380349800007],[9.632460499000047,49.69318599800005],[9.648736,49.79147749900005],[10.030919,49.95219999900007],[10.189162,49.892056498000045]],[[10.013277500000072,49.804249997000056],[9.872402500000021,49.82224599800003],[9.876827,49.75309249700007],[9.982376,49.71344349800006],[10.013277500000072,49.804249997000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Schwaben","NUTS_NAME":"Schwaben"},"geometry":{"type":"Polygon","coordinates":[[[10.642254500000035,49.016631500000074],[10.680472500000064,48.913612998000076],[10.817673500000069,48.949463499000046],[10.815371,48.90109049800003],[10.936839500000076,48.86327599800006],[11.005941500000063,48.821882997000046],[10.984102,48.69280199700006],[11.022986,48.62017149600007],[11.155116,48.59589999600007],[11.12657450000006,48.55498750000004],[11.295411,48.47457399900003],[11.311675,48.45227299800007],[11.17824950000005,48.41746599900006],[11.198315,48.386679998000034],[11.116768500000035,48.26847799800004],[11.061886,48.27419099800005],[11.03451,48.192992],[10.903411,48.236676500000044],[10.848869,48.147790996000026],[10.786499500000048,48.15013799800005],[10.801532,48.10417049900008],[10.75635950000003,48.04034599800008],[10.826458,47.99526099800005],[10.767090500000052,47.843793498000025],[10.715888500000062,47.705480496000064],[10.954276500000049,47.61652049800006],[10.872826,47.57245899800006],[10.886199,47.53684799800004],[10.454439,47.55579699800006],[10.431323500000076,47.50367749600008],[10.475413,47.431702],[10.178353,47.270113998000056],[10.23393,47.38328599700003],[10.09985049900007,47.35476149700003],[10.09147050000007,47.45907599700007],[9.999526,47.48301699600006],[9.967813500000034,47.54624049600005],[9.765470500000049,47.588856],[9.736566500000038,47.534906498000055],[9.55872050000005,47.54189299800004],[9.692543,47.610768998000026],[9.840336,47.677096498000026],[10.077292500000055,47.63927449700003],[10.139437500000042,47.703273496000065],[10.064681,47.78588749900007],[10.131928500000072,47.82008749600004],[10.110136500000067,47.93714999800005],[10.104207,47.974358496000036],[10.114954500000067,47.97625750000003],[10.132387,48.01539999800008],[10.135728,48.02376149600008],[10.136457,48.10845949700007],[10.095357500000034,48.16401499800003],[9.997606,48.35011799800003],[10.032694500000048,48.45719649800003],[10.133895500000051,48.454871498000045],[10.23077950000004,48.510510999000076],[10.278268,48.51607449800008],[10.314184,48.55628950000005],[10.268706999000074,48.703546998000036],[10.361222500000054,48.65318649900007],[10.487258,48.696662],[10.423689,48.74449599600007],[10.45700050000005,48.917141498000035],[10.409849500000064,48.977435498000034],[10.642254500000035,49.016631500000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE271","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Augsburg, Kreisfreie Stadt","NUTS_NAME":"Augsburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.910751,48.258306996000044],[10.76419,48.300984],[10.856543,48.34584599800007],[10.884084,48.458442998000066],[10.959205500000053,48.37197449900003],[10.910751,48.258306996000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE272","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kaufbeuren, Kreisfreie Stadt","NUTS_NAME":"Kaufbeuren, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.643866,47.91768849700003],[10.656953500000043,47.84116699800006],[10.551547,47.883113998000056],[10.643866,47.91768849700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE273","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kempten (Allgäu), Kreisfreie Stadt","NUTS_NAME":"Kempten (Allgäu), Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.373037,47.713663498000074],[10.285494500000027,47.696604499000046],[10.236971,47.77037750000005],[10.36991850000004,47.762999996000076],[10.373037,47.713663498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG343","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Yambol","NUTS_NAME":"Ямбол"},"geometry":{"type":"Polygon","coordinates":[[[26.71400950000003,42.68456149700006],[26.83868650000005,42.50736949900005],[26.96973,42.43310349800004],[26.862353,42.38767999700008],[26.917073,42.34152199700003],[26.943062500000053,42.144318500000054],[27.059650500000032,42.088336997000056],[26.94922850000006,42.00021349800005],[26.561544500000025,41.92627349800006],[26.505151,41.99440999700005],[26.538056,42.15419699800003],[26.19273350000003,42.208129996000025],[26.175732,42.26146099600004],[26.254705500000057,42.369687500000055],[26.18612,42.36132699700005],[26.232439500000055,42.46978099800003],[26.34071,42.45414499800006],[26.426252500000032,42.57423399800007],[26.55205250000006,42.56821650000006],[26.672502500000064,42.63743199800007],[26.64059050000003,42.69343249800005],[26.71400950000003,42.68456149700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG344","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Stara Zagora","NUTS_NAME":"Стара Загора"},"geometry":{"type":"Polygon","coordinates":[[[25.837491,42.77722749800006],[25.858780500000023,42.62855450100005],[25.91175450000003,42.582457497000064],[25.844794500000035,42.556874496000034],[25.914383,42.47255899800007],[25.874343,42.445862498000054],[25.930477500000052,42.322275999000055],[26.175732,42.26146099600004],[26.19273350000003,42.208129996000025],[26.137642500000027,42.15459199900005],[26.15387050000004,42.041241497000044],[26.057769499000074,42.017656499000054],[25.877450500000066,42.13096949700008],[25.763608500000032,42.052389996000045],[25.609251,42.20095899900008],[25.458139500000073,42.16101249900004],[25.42784850000004,42.08947999700007],[25.348064,42.11437749700008],[25.259848500000032,42.184405500000025],[25.081143,42.183936499000026],[25.083079,42.25491649800006],[25.173759,42.27125149900007],[25.112305500000048,42.39638149700005],[25.194347,42.391169999000056],[25.153932001000044,42.45523550000007],[25.191318500000023,42.49178649800007],[25.03865650000006,42.51863649900008],[25.000570500000038,42.588310001000025],[25.007195500000023,42.73761899800007],[25.01459568300004,42.748554082000055],[25.181868001000055,42.787554499000066],[25.24970950000005,42.74265149900003],[25.61923150000007,42.74802199800007],[25.61496150000005,42.78680849800003],[25.837491,42.77722749800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG4","LEVL_CODE":1,"CNTR_CODE":"BG","NAME_LATN":"YUGOZAPADNA I YUZHNA TSENTRALNA BULGARIA","NUTS_NAME":"ЮГОЗАПАДНА И ЮЖНА ЦЕНТРАЛНА БЪЛГАРИЯ"},"geometry":{"type":"Polygon","coordinates":[[[23.00621,43.19287849700004],[23.143223,43.11142049800003],[23.412767,43.160521997000046],[23.566569,42.993892998000035],[23.62619,43.05803899800003],[23.97150350000004,43.06225149800008],[24.043948501000045,43.05992899800003],[24.062151501000073,42.96479499900005],[24.16529150100007,42.92970149800004],[24.16750150000007,42.816318498000044],[24.127983999000037,42.77494799600004],[24.385927,42.749953497000035],[24.511502,42.79306999900007],[24.754968001000066,42.70852799800008],[25.007195500000023,42.73761899800007],[25.000570500000038,42.588310001000025],[25.03865650000006,42.51863649900008],[25.191318500000023,42.49178649800007],[25.153932001000044,42.45523550000007],[25.194347,42.391169999000056],[25.112305500000048,42.39638149700005],[25.173759,42.27125149900007],[25.083079,42.25491649800006],[25.081143,42.183936499000026],[25.259848500000032,42.184405500000025],[25.348064,42.11437749700008],[25.42784850000004,42.08947999700007],[25.458139500000073,42.16101249900004],[25.609251,42.20095899900008],[25.763608500000032,42.052389996000045],[25.877450500000066,42.13096949700008],[26.057769499000074,42.017656499000054],[26.15387050000004,42.041241497000044],[26.137642500000027,42.15459199900005],[26.19273350000003,42.208129996000025],[26.538056,42.15419699800003],[26.505151,41.99440999700005],[26.561544500000025,41.92627349800006],[26.584593,41.902652998000065],[26.538608500000066,41.823482496000054],[26.369060500000046,41.82068249800005],[26.328071499000032,41.74837899700003],[26.357879,41.711104498000054],[26.16942,41.74869149700004],[26.060393,41.688516],[26.17645550000003,41.554799496000044],[26.15868850000004,41.391182496000056],[25.948626,41.320342],[25.906437500000038,41.307574997000074],[25.224698500000045,41.26463099700004],[25.17937550000005,41.31018799700007],[24.92368350000004,41.402213999000026],[24.783956,41.36018899600003],[24.607909500000062,41.42761799600004],[24.525637,41.56869999700007],[24.059744,41.52211199900006],[24.046779,41.45128599800006],[23.91613249900007,41.48162649900007],[23.62422299900004,41.375727496000025],[23.322212500000035,41.399669999000025],[23.18137150000007,41.318869498000026],[22.927591500000062,41.338539498000046],[22.965055500000062,41.35190749700007],[22.968327500000044,41.51983549800008],[22.948411500000077,41.63179649800003],[23.032124,41.72430099800005],[22.90193350000004,41.87764199900005],[22.867214,42.02219949600004],[22.510412,42.15515849700006],[22.36020650000006,42.311157],[22.520242,42.39757350000008],[22.559255,42.48028399700007],[22.439139500000067,42.56646449900006],[22.461472,42.64849049800006],[22.500916,42.73917749800006],[22.442821499000047,42.82545650000003],[22.51551050000006,42.86853849700003],[22.748399500000062,42.889787496000054],[22.783864,42.98161299900005],[22.98325250000005,43.11146499800003],[23.00621,43.19287849700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG41","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Yugozapaden","NUTS_NAME":"Югозападен"},"geometry":{"type":"Polygon","coordinates":[[[23.00621,43.19287849700004],[23.143223,43.11142049800003],[23.412767,43.160521997000046],[23.566569,42.993892998000035],[23.62619,43.05803899800003],[23.97150350000004,43.06225149800008],[24.043948501000045,43.05992899800003],[24.062151501000073,42.96479499900005],[24.16529150100007,42.92970149800004],[24.16750150000007,42.816318498000044],[24.127983999000037,42.77494799600004],[24.385927,42.749953497000035],[24.357004,42.67730849900005],[24.482858500000077,42.59980749700003],[24.421009500000025,42.55306399600005],[24.24869350000006,42.624356498000054],[23.89048850000006,42.54626950000005],[23.954258,42.46366849800006],[24.06605650000006,42.44992349800003],[23.958568,42.38330149800004],[23.940902,42.33072899800004],[23.994878,42.29411149800006],[23.967058,42.26725049800007],[23.894599,42.27474649800007],[23.77226,42.18118549800005],[23.786217,41.833541998000044],[24.07454750000005,41.67270399600005],[24.10960750000004,41.652153499000065],[24.059744,41.52211199900006],[24.046779,41.45128599800006],[23.91613249900007,41.48162649900007],[23.62422299900004,41.375727496000025],[23.322212500000035,41.399669999000025],[23.18137150000007,41.318869498000026],[22.927591500000062,41.338539498000046],[22.965055500000062,41.35190749700007],[22.968327500000044,41.51983549800008],[22.948411500000077,41.63179649800003],[23.032124,41.72430099800005],[22.90193350000004,41.87764199900005],[22.867214,42.02219949600004],[22.510412,42.15515849700006],[22.36020650000006,42.311157],[22.520242,42.39757350000008],[22.559255,42.48028399700007],[22.439139500000067,42.56646449900006],[22.461472,42.64849049800006],[22.500916,42.73917749800006],[22.442821499000047,42.82545650000003],[22.51551050000006,42.86853849700003],[22.748399500000062,42.889787496000054],[22.783864,42.98161299900005],[22.98325250000005,43.11146499800003],[23.00621,43.19287849700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG411","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Sofia (stolitsa)","NUTS_NAME":"София (столица)"},"geometry":{"type":"Polygon","coordinates":[[[23.290187,42.53270899800003],[23.05651399900006,42.71273399800003],[23.196810500000026,42.728285999000036],[23.247254,42.866676496000025],[23.296743500000048,42.888494997000066],[23.66569750000008,42.83128699900004],[23.496391500000072,42.64963649800006],[23.55950050000007,42.61622749800006],[23.550289500000076,42.533010998000066],[23.630126500000074,42.50035299700005],[23.66236,42.42172849800005],[23.544922,42.413890998000056],[23.509977500000048,42.51221799800004],[23.44768,42.52486599800005],[23.413432500000056,42.463986498000054],[23.290187,42.53270899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG412","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Sofia","NUTS_NAME":"София"},"geometry":{"type":"Polygon","coordinates":[[[23.412767,43.160521997000046],[23.566569,42.993892998000035],[23.62619,43.05803899800003],[23.97150350000004,43.06225149800008],[24.043948501000045,43.05992899800003],[24.062151501000073,42.96479499900005],[24.16529150100007,42.92970149800004],[24.16750150000007,42.816318498000044],[24.127983999000037,42.77494799600004],[24.385927,42.749953497000035],[24.357004,42.67730849900005],[24.482858500000077,42.59980749700003],[24.421009500000025,42.55306399600005],[24.24869350000006,42.624356498000054],[23.89048850000006,42.54626950000005],[23.954258,42.46366849800006],[24.06605650000006,42.44992349800003],[23.958568,42.38330149800004],[23.940902,42.33072899800004],[23.994878,42.29411149800006],[23.967058,42.26725049800007],[23.894599,42.27474649800007],[23.77226,42.18118549800005],[23.495562500000062,42.09755749800007],[23.529472,42.12702649700003],[23.448741,42.182552498000064],[23.320195,42.17514149800007],[23.3864,42.283182999000076],[23.224096,42.37702150000007],[23.221077,42.472789498000054],[23.290187,42.53270899800003],[23.413432500000056,42.463986498000054],[23.44768,42.52486599800005],[23.509977500000048,42.51221799800004],[23.544922,42.413890998000056],[23.66236,42.42172849800005],[23.630126500000074,42.50035299700005],[23.550289500000076,42.533010998000066],[23.55950050000007,42.61622749800006],[23.496391500000072,42.64963649800006],[23.66569750000008,42.83128699900004],[23.296743500000048,42.888494997000066],[23.247254,42.866676496000025],[23.196810500000026,42.728285999000036],[23.05651399900006,42.71273399800003],[22.748399500000062,42.889787496000054],[22.783864,42.98161299900005],[22.98325250000005,43.11146499800003],[23.00621,43.19287849700004],[23.143223,43.11142049800003],[23.412767,43.160521997000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG413","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Blagoevgrad","NUTS_NAME":"Благоевград"},"geometry":{"type":"Polygon","coordinates":[[[24.07454750000005,41.67270399600005],[24.10960750000004,41.652153499000065],[24.059744,41.52211199900006],[24.046779,41.45128599800006],[23.91613249900007,41.48162649900007],[23.62422299900004,41.375727496000025],[23.322212500000035,41.399669999000025],[23.18137150000007,41.318869498000026],[22.927591500000062,41.338539498000046],[22.965055500000062,41.35190749700007],[22.968327500000044,41.51983549800008],[22.948411500000077,41.63179649800003],[23.032124,41.72430099800005],[22.90193350000004,41.87764199900005],[22.867214,42.02219949600004],[22.939498,42.09653099800005],[23.056022,42.041780997000046],[23.199266,42.09875249700008],[23.40919850000006,42.05293549800007],[23.495562500000062,42.09755749800007],[23.77226,42.18118549800005],[23.786217,41.833541998000044],[24.07454750000005,41.67270399600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG414","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Pernik","NUTS_NAME":"Перник"},"geometry":{"type":"Polygon","coordinates":[[[22.748399500000062,42.889787496000054],[23.05651399900006,42.71273399800003],[23.290187,42.53270899800003],[23.221077,42.472789498000054],[23.224096,42.37702150000007],[22.805727,42.376011497000036],[22.582996500000036,42.63865549700006],[22.461472,42.64849049800006],[22.500916,42.73917749800006],[22.442821499000047,42.82545650000003],[22.51551050000006,42.86853849700003],[22.748399500000062,42.889787496000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG415","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Kyustendil","NUTS_NAME":"Кюстендил"},"geometry":{"type":"Polygon","coordinates":[[[23.224096,42.37702150000007],[23.3864,42.283182999000076],[23.320195,42.17514149800007],[23.448741,42.182552498000064],[23.529472,42.12702649700003],[23.495562500000062,42.09755749800007],[23.40919850000006,42.05293549800007],[23.199266,42.09875249700008],[23.056022,42.041780997000046],[22.939498,42.09653099800005],[22.867214,42.02219949600004],[22.510412,42.15515849700006],[22.36020650000006,42.311157],[22.520242,42.39757350000008],[22.559255,42.48028399700007],[22.439139500000067,42.56646449900006],[22.461472,42.64849049800006],[22.582996500000036,42.63865549700006],[22.805727,42.376011497000036],[23.224096,42.37702150000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG42","LEVL_CODE":2,"CNTR_CODE":"BG","NAME_LATN":"Yuzhen tsentralen","NUTS_NAME":"Южен централен"},"geometry":{"type":"Polygon","coordinates":[[[24.511502,42.79306999900007],[24.754968001000066,42.70852799800008],[25.007195500000023,42.73761899800007],[25.000570500000038,42.588310001000025],[25.03865650000006,42.51863649900008],[25.191318500000023,42.49178649800007],[25.153932001000044,42.45523550000007],[25.194347,42.391169999000056],[25.112305500000048,42.39638149700005],[25.173759,42.27125149900007],[25.083079,42.25491649800006],[25.081143,42.183936499000026],[25.259848500000032,42.184405500000025],[25.348064,42.11437749700008],[25.42784850000004,42.08947999700007],[25.458139500000073,42.16101249900004],[25.609251,42.20095899900008],[25.763608500000032,42.052389996000045],[25.877450500000066,42.13096949700008],[26.057769499000074,42.017656499000054],[26.15387050000004,42.041241497000044],[26.137642500000027,42.15459199900005],[26.19273350000003,42.208129996000025],[26.538056,42.15419699800003],[26.505151,41.99440999700005],[26.561544500000025,41.92627349800006],[26.584593,41.902652998000065],[26.538608500000066,41.823482496000054],[26.369060500000046,41.82068249800005],[26.328071499000032,41.74837899700003],[26.357879,41.711104498000054],[26.16942,41.74869149700004],[26.060393,41.688516],[26.17645550000003,41.554799496000044],[26.15868850000004,41.391182496000056],[25.948626,41.320342],[25.906437500000038,41.307574997000074],[25.224698500000045,41.26463099700004],[25.17937550000005,41.31018799700007],[24.92368350000004,41.402213999000026],[24.783956,41.36018899600003],[24.607909500000062,41.42761799600004],[24.525637,41.56869999700007],[24.059744,41.52211199900006],[24.10960750000004,41.652153499000065],[24.07454750000005,41.67270399600005],[23.786217,41.833541998000044],[23.77226,42.18118549800005],[23.894599,42.27474649800007],[23.967058,42.26725049800007],[23.994878,42.29411149800006],[23.940902,42.33072899800004],[23.958568,42.38330149800004],[24.06605650000006,42.44992349800003],[23.954258,42.46366849800006],[23.89048850000006,42.54626950000005],[24.24869350000006,42.624356498000054],[24.421009500000025,42.55306399600005],[24.482858500000077,42.59980749700003],[24.357004,42.67730849900005],[24.385927,42.749953497000035],[24.511502,42.79306999900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG421","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Plovdiv","NUTS_NAME":"Пловдив"},"geometry":{"type":"Polygon","coordinates":[[[25.007195500000023,42.73761899800007],[25.000570500000038,42.588310001000025],[25.03865650000006,42.51863649900008],[25.191318500000023,42.49178649800007],[25.153932001000044,42.45523550000007],[25.194347,42.391169999000056],[25.112305500000048,42.39638149700005],[25.173759,42.27125149900007],[25.083079,42.25491649800006],[25.081143,42.183936499000026],[25.259848500000032,42.184405500000025],[25.348064,42.11437749700008],[25.295173499000043,42.04429699800005],[25.317624,41.96798049900008],[25.218167,41.895590998000046],[25.07422450000007,41.793476498000075],[25.00188,41.83917699800003],[24.831799500000045,41.70185249800005],[24.769521,41.76568099700006],[24.752886500000045,41.88762849600005],[24.549022,41.87176199800007],[24.439807,41.93821849900007],[24.431455,42.124255497000036],[24.51794250000006,42.18094099700005],[24.380928,42.48289649800006],[24.421009500000025,42.55306399600005],[24.482858500000077,42.59980749700003],[24.357004,42.67730849900005],[24.385927,42.749953497000035],[24.511502,42.79306999900007],[24.754968001000066,42.70852799800008],[25.007195500000023,42.73761899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG422","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Haskovo","NUTS_NAME":"Хасково"},"geometry":{"type":"Polygon","coordinates":[[[26.561544500000025,41.92627349800006],[26.584593,41.902652998000065],[26.538608500000066,41.823482496000054],[26.369060500000046,41.82068249800005],[26.328071499000032,41.74837899700003],[26.357879,41.711104498000054],[26.16942,41.74869149700004],[26.060393,41.688516],[26.17645550000003,41.554799496000044],[26.15868850000004,41.391182496000056],[25.948626,41.320342],[25.941516999000044,41.37220849800008],[25.821870500000045,41.41206499800006],[25.864337500000033,41.468386999000074],[25.755163500000037,41.536851496000054],[25.835577,41.64793149800005],[25.601716500000066,41.62337449800003],[25.646185,41.68725199800008],[25.591217500000027,41.761739496000075],[25.284567500000037,41.81544549600005],[25.218167,41.895590998000046],[25.317624,41.96798049900008],[25.295173499000043,42.04429699800005],[25.348064,42.11437749700008],[25.42784850000004,42.08947999700007],[25.458139500000073,42.16101249900004],[25.609251,42.20095899900008],[25.763608500000032,42.052389996000045],[25.877450500000066,42.13096949700008],[26.057769499000074,42.017656499000054],[26.15387050000004,42.041241497000044],[26.137642500000027,42.15459199900005],[26.19273350000003,42.208129996000025],[26.538056,42.15419699800003],[26.505151,41.99440999700005],[26.561544500000025,41.92627349800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG423","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Pazardzhik","NUTS_NAME":"Пазарджик"},"geometry":{"type":"Polygon","coordinates":[[[24.421009500000025,42.55306399600005],[24.380928,42.48289649800006],[24.51794250000006,42.18094099700005],[24.431455,42.124255497000036],[24.439807,41.93821849900007],[24.295767,41.72067949900003],[24.17888,41.73986849700003],[24.07454750000005,41.67270399600005],[23.786217,41.833541998000044],[23.77226,42.18118549800005],[23.894599,42.27474649800007],[23.967058,42.26725049800007],[23.994878,42.29411149800006],[23.940902,42.33072899800004],[23.958568,42.38330149800004],[24.06605650000006,42.44992349800003],[23.954258,42.46366849800006],[23.89048850000006,42.54626950000005],[24.24869350000006,42.624356498000054],[24.421009500000025,42.55306399600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG424","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Smolyan","NUTS_NAME":"Смолян"},"geometry":{"type":"Polygon","coordinates":[[[25.07422450000007,41.793476498000075],[25.049104500000055,41.74618749800004],[25.099273,41.62090049700004],[24.999145,41.567290996000054],[25.061052,41.48962349900006],[25.166985500000067,41.499472498000046],[25.17937550000005,41.31018799700007],[24.92368350000004,41.402213999000026],[24.783956,41.36018899600003],[24.607909500000062,41.42761799600004],[24.525637,41.56869999700007],[24.059744,41.52211199900006],[24.10960750000004,41.652153499000065],[24.07454750000005,41.67270399600005],[24.17888,41.73986849700003],[24.295767,41.72067949900003],[24.439807,41.93821849900007],[24.549022,41.87176199800007],[24.752886500000045,41.88762849600005],[24.769521,41.76568099700006],[24.831799500000045,41.70185249800005],[25.00188,41.83917699800003],[25.07422450000007,41.793476498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"BG425","LEVL_CODE":3,"CNTR_CODE":"BG","NAME_LATN":"Kardzhali","NUTS_NAME":"Кърджали"},"geometry":{"type":"Polygon","coordinates":[[[25.218167,41.895590998000046],[25.284567500000037,41.81544549600005],[25.591217500000027,41.761739496000075],[25.646185,41.68725199800008],[25.601716500000066,41.62337449800003],[25.835577,41.64793149800005],[25.755163500000037,41.536851496000054],[25.864337500000033,41.468386999000074],[25.821870500000045,41.41206499800006],[25.941516999000044,41.37220849800008],[25.948626,41.320342],[25.906437500000038,41.307574997000074],[25.224698500000045,41.26463099700004],[25.17937550000005,41.31018799700007],[25.166985500000067,41.499472498000046],[25.061052,41.48962349900006],[24.999145,41.567290996000054],[25.099273,41.62090049700004],[25.049104500000055,41.74618749800004],[25.07422450000007,41.793476498000075],[25.218167,41.895590998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH","LEVL_CODE":0,"CNTR_CODE":"CH","NAME_LATN":"SCHWEIZ/SUISSE/SVIZZERA","NUTS_NAME":"SCHWEIZ/SUISSE/SVIZZERA"},"geometry":{"type":"Polygon","coordinates":[[[8.727981500000055,47.69268049800007],[8.795708500000046,47.67559549900005],[8.77209750000003,47.718459497000026],[8.843022,47.71226249800003],[8.87499850000006,47.65597349700005],[9.182192,47.655890498000076],[9.495604,47.551454998000054],[9.55872050000005,47.54189299800004],[9.673370500000033,47.38151050000005],[9.530749,47.270581],[9.486198,47.17795649900006],[9.520344,47.10014999800006],[9.47604750000005,47.05179749800004],[9.607078,47.06077449900005],[9.879422500000032,47.01956299600005],[9.876148500000056,46.93463550000007],[10.144974500000046,46.85100949800005],[10.38931799900007,47.00052449900005],[10.489350500000057,46.93779049800003],[10.469651500000055,46.85490899900003],[10.39332150000007,46.68920749600005],[10.402287999000066,46.63681199600006],[10.492064,46.61538649700003],[10.452801,46.53068299800003],[10.295437,46.54989349700003],[10.244874500000037,46.62209199700004],[10.043919500000072,46.53753099800008],[10.042330500000048,46.44236649900006],[10.166681500000038,46.407648],[10.104273500000033,46.33366750000005],[10.176524500000028,46.258276497000054],[10.135604,46.226256499000044],[10.04367,46.22974449700007],[9.996764,46.35093450000005],[9.907831499000054,46.38050599800005],[9.735838500000057,46.35031299800005],[9.714149500000076,46.29270849800008],[9.54957550000006,46.30233799900003],[9.460813500000029,46.375970497000026],[9.464343,46.50778249700005],[9.282739500000048,46.496694999000056],[9.247002500000065,46.44679449800003],[9.300141500000052,46.32645649800003],[9.24853150000007,46.233768],[9.159377500000062,46.16960099900007],[8.98827650000004,45.97228249700004],[9.08880350000004,45.896897],[9.034462,45.82301699900006],[8.912147,45.830444998000075],[8.893755,45.959014996000064],[8.785907,45.989075496000055],[8.852114,46.07564149700005],[8.713936,46.09727199800005],[8.444526500000052,46.24887749800007],[8.452797,46.455350999000075],[8.384717,46.452158498000074],[8.085928,46.26661749700003],[8.15534450000007,46.147631],[8.034763,46.100836498000035],[8.012413,46.01196999800004],[7.908679500000062,45.996936498000025],[7.87713750000006,45.92695499700005],[7.864076500000067,45.916750499000045],[7.578607500000032,45.98601250000007],[7.117904,45.85919249800003],[7.044886,45.92241299800003],[6.891464,46.07387949800005],[6.899467,46.124058],[6.797888999000065,46.136798998000074],[6.864808,46.28059399800003],[6.770442,46.35520549800003],[6.821063999000046,46.42715449800005],[6.519104,46.45636749600004],[6.24136,46.34358249600007],[6.231694,46.329429497000035],[6.219547,46.31187799800006],[6.310211,46.24404549800005],[6.135542,46.141159],[5.956067,46.13209549900006],[5.974414,46.214751996000075],[6.088192,46.246158499000046],[6.125608,46.31722999800007],[6.170135999000024,46.36643599800004],[6.064003,46.41622899600003],[6.138106,46.55766699600008],[6.110961,46.576656496000055],[6.439425,46.76257299900004],[6.460011,46.85155149800005],[6.43275,46.92845549800006],[6.858914,47.16529449800004],[7.061636,47.34373449800006],[6.879806,47.35243999700003],[6.939186,47.433704499000044],[7.00155,47.45374349700006],[6.982914,47.49427049800005],[7.130353,47.503040498000075],[7.170422,47.44345449800005],[7.326466,47.43985349900004],[7.380894,47.43189249800008],[7.421139499000049,47.446387998000034],[7.445019,47.46172349600005],[7.510905500000035,47.50258249700005],[7.555159500000059,47.56456399600006],[7.589039,47.58987799800008],[7.69337,47.600644998000064],[7.634097,47.56111350000003],[7.713784500000031,47.539404999000055],[7.894107500000075,47.58637499800005],[8.01872,47.550453996000044],[8.221265500000072,47.61551499800004],[8.426434500000028,47.567548997000074],[8.52144,47.633524],[8.562841,47.599432497000066],[8.595602,47.605544499000075],[8.606369500000028,47.66900549800005],[8.405565999000032,47.67451349800007],[8.510115,47.77619049900005],[8.61233,47.801461999000026],[8.74010450000003,47.752789],[8.727981500000055,47.69268049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH0","LEVL_CODE":1,"CNTR_CODE":"CH","NAME_LATN":"SCHWEIZ/SUISSE/SVIZZERA","NUTS_NAME":"SCHWEIZ/SUISSE/SVIZZERA"},"geometry":{"type":"Polygon","coordinates":[[[8.727981500000055,47.69268049800007],[8.795708500000046,47.67559549900005],[8.77209750000003,47.718459497000026],[8.843022,47.71226249800003],[8.87499850000006,47.65597349700005],[9.182192,47.655890498000076],[9.495604,47.551454998000054],[9.55872050000005,47.54189299800004],[9.673370500000033,47.38151050000005],[9.530749,47.270581],[9.486198,47.17795649900006],[9.520344,47.10014999800006],[9.47604750000005,47.05179749800004],[9.607078,47.06077449900005],[9.879422500000032,47.01956299600005],[9.876148500000056,46.93463550000007],[10.144974500000046,46.85100949800005],[10.38931799900007,47.00052449900005],[10.489350500000057,46.93779049800003],[10.469651500000055,46.85490899900003],[10.39332150000007,46.68920749600005],[10.402287999000066,46.63681199600006],[10.492064,46.61538649700003],[10.452801,46.53068299800003],[10.295437,46.54989349700003],[10.244874500000037,46.62209199700004],[10.043919500000072,46.53753099800008],[10.042330500000048,46.44236649900006],[10.166681500000038,46.407648],[10.104273500000033,46.33366750000005],[10.176524500000028,46.258276497000054],[10.135604,46.226256499000044],[10.04367,46.22974449700007],[9.996764,46.35093450000005],[9.907831499000054,46.38050599800005],[9.735838500000057,46.35031299800005],[9.714149500000076,46.29270849800008],[9.54957550000006,46.30233799900003],[9.460813500000029,46.375970497000026],[9.464343,46.50778249700005],[9.282739500000048,46.496694999000056],[9.247002500000065,46.44679449800003],[9.300141500000052,46.32645649800003],[9.24853150000007,46.233768],[9.159377500000062,46.16960099900007],[8.98827650000004,45.97228249700004],[9.08880350000004,45.896897],[9.034462,45.82301699900006],[8.912147,45.830444998000075],[8.893755,45.959014996000064],[8.785907,45.989075496000055],[8.852114,46.07564149700005],[8.713936,46.09727199800005],[8.444526500000052,46.24887749800007],[8.452797,46.455350999000075],[8.384717,46.452158498000074],[8.085928,46.26661749700003],[8.15534450000007,46.147631],[8.034763,46.100836498000035],[8.012413,46.01196999800004],[7.908679500000062,45.996936498000025],[7.87713750000006,45.92695499700005],[7.864076500000067,45.916750499000045],[7.578607500000032,45.98601250000007],[7.117904,45.85919249800003],[7.044886,45.92241299800003],[6.891464,46.07387949800005],[6.899467,46.124058],[6.797888999000065,46.136798998000074],[6.864808,46.28059399800003],[6.770442,46.35520549800003],[6.821063999000046,46.42715449800005],[6.519104,46.45636749600004],[6.24136,46.34358249600007],[6.231694,46.329429497000035],[6.219547,46.31187799800006],[6.310211,46.24404549800005],[6.135542,46.141159],[5.956067,46.13209549900006],[5.974414,46.214751996000075],[6.088192,46.246158499000046],[6.125608,46.31722999800007],[6.170135999000024,46.36643599800004],[6.064003,46.41622899600003],[6.138106,46.55766699600008],[6.110961,46.576656496000055],[6.439425,46.76257299900004],[6.460011,46.85155149800005],[6.43275,46.92845549800006],[6.858914,47.16529449800004],[7.061636,47.34373449800006],[6.879806,47.35243999700003],[6.939186,47.433704499000044],[7.00155,47.45374349700006],[6.982914,47.49427049800005],[7.130353,47.503040498000075],[7.170422,47.44345449800005],[7.326466,47.43985349900004],[7.380894,47.43189249800008],[7.421139499000049,47.446387998000034],[7.445019,47.46172349600005],[7.510905500000035,47.50258249700005],[7.555159500000059,47.56456399600006],[7.589039,47.58987799800008],[7.69337,47.600644998000064],[7.634097,47.56111350000003],[7.713784500000031,47.539404999000055],[7.894107500000075,47.58637499800005],[8.01872,47.550453996000044],[8.221265500000072,47.61551499800004],[8.426434500000028,47.567548997000074],[8.52144,47.633524],[8.562841,47.599432497000066],[8.595602,47.605544499000075],[8.606369500000028,47.66900549800005],[8.405565999000032,47.67451349800007],[8.510115,47.77619049900005],[8.61233,47.801461999000026],[8.74010450000003,47.752789],[8.727981500000055,47.69268049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH01","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Région lémanique","NUTS_NAME":"Région lémanique"},"geometry":{"type":"Polygon","coordinates":[[[6.742307,46.827455497000074],[6.852262497000027,46.77425905600006],[6.931451,46.804543498000044],[6.866233500000021,46.90929349800007],[6.896209500000055,46.92532599800006],[6.992414500000052,46.830588496000075],[6.798820500000033,46.64924799800008],[6.795106,46.57521850100005],[6.887994,46.55794899800003],[6.817892500000028,46.516538998000044],[6.962045,46.496879499000045],[6.983955,46.43804699700007],[7.237020500000028,46.553799998000045],[7.221457,46.32921249900005],[7.538435500000048,46.37744350000003],[7.626100500000064,46.44505199900004],[7.712379499000065,46.41389649800004],[8.015749501000073,46.56329649800006],[8.258061500000053,46.52944199800004],[8.410411500000066,46.653020498000046],[8.410029,46.57725749800005],[8.477673500000037,46.52760249800008],[8.384717,46.452158498000074],[8.085928,46.26661749700003],[8.15534450000007,46.147631],[8.034763,46.100836498000035],[8.012413,46.01196999800004],[7.908679500000062,45.996936498000025],[7.87713750000006,45.92695499700005],[7.864076500000067,45.916750499000045],[7.578607500000032,45.98601250000007],[7.117904,45.85919249800003],[7.044886,45.92241299800003],[6.891464,46.07387949800005],[6.899467,46.124058],[6.797888999000065,46.136798998000074],[6.864808,46.28059399800003],[6.770442,46.35520549800003],[6.821063999000046,46.42715449800005],[6.519104,46.45636749600004],[6.24136,46.34358249600007],[6.231694,46.329429497000035],[6.219547,46.31187799800006],[6.310211,46.24404549800005],[6.135542,46.141159],[5.956067,46.13209549900006],[5.974414,46.214751996000075],[6.088192,46.246158499000046],[6.125608,46.31722999800007],[6.170135999000024,46.36643599800004],[6.064003,46.41622899600003],[6.138106,46.55766699600008],[6.110961,46.576656496000055],[6.439425,46.76257299900004],[6.460011,46.85155149800005],[6.720867500000054,46.936142999000026],[6.780046,46.85264299800008],[6.742307,46.827455497000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH011","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Vaud","NUTS_NAME":"Vaud"},"geometry":{"type":"Polygon","coordinates":[[[6.742307,46.827455497000074],[6.852262497000027,46.77425905600006],[6.931451,46.804543498000044],[6.866233500000021,46.90929349800007],[6.896209500000055,46.92532599800006],[6.992414500000052,46.830588496000075],[6.798820500000033,46.64924799800008],[6.795106,46.57521850100005],[6.887994,46.55794899800003],[6.817892500000028,46.516538998000044],[6.962045,46.496879499000045],[6.983955,46.43804699700007],[7.237020500000028,46.553799998000045],[7.221457,46.32921249900005],[7.03316250000006,46.18696549800006],[6.821063999000046,46.42715449800005],[6.519104,46.45636749600004],[6.24136,46.34358249600007],[6.231694,46.329429497000035],[6.219547,46.31187799800006],[6.140711999000075,46.30539650000003],[6.125608,46.31722999800007],[6.170135999000024,46.36643599800004],[6.064003,46.41622899600003],[6.138106,46.55766699600008],[6.110961,46.576656496000055],[6.439425,46.76257299900004],[6.460011,46.85155149800005],[6.720867500000054,46.936142999000026],[6.780046,46.85264299800008],[6.742307,46.827455497000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH012","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Valais","NUTS_NAME":"Valais"},"geometry":{"type":"Polygon","coordinates":[[[8.477673500000037,46.52760249800008],[8.384717,46.452158498000074],[8.085928,46.26661749700003],[8.15534450000007,46.147631],[8.034763,46.100836498000035],[8.012413,46.01196999800004],[7.908679500000062,45.996936498000025],[7.87713750000006,45.92695499700005],[7.864076500000067,45.916750499000045],[7.578607500000032,45.98601250000007],[7.117904,45.85919249800003],[7.044886,45.92241299800003],[6.891464,46.07387949800005],[6.899467,46.124058],[6.797888999000065,46.136798998000074],[6.864808,46.28059399800003],[6.770442,46.35520549800003],[6.821063999000046,46.42715449800005],[7.03316250000006,46.18696549800006],[7.221457,46.32921249900005],[7.538435500000048,46.37744350000003],[7.626100500000064,46.44505199900004],[7.712379499000065,46.41389649800004],[8.015749501000073,46.56329649800006],[8.258061500000053,46.52944199800004],[8.410411500000066,46.653020498000046],[8.410029,46.57725749800005],[8.477673500000037,46.52760249800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH013","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Genève","NUTS_NAME":"Genève"},"geometry":{"type":"Polygon","coordinates":[[[6.140711999000075,46.30539650000003],[6.219547,46.31187799800006],[6.310211,46.24404549800005],[6.135542,46.141159],[5.956067,46.13209549900006],[5.974414,46.214751996000075],[6.088192,46.246158499000046],[6.125608,46.31722999800007],[6.140711999000075,46.30539650000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH02","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Espace Mittelland","NUTS_NAME":"Espace Mittelland"},"geometry":{"type":"Polygon","coordinates":[[[7.375684500000034,47.41407699900003],[7.43689,47.379970498000034],[7.615804,47.43270399900007],[7.606372,47.48947],[7.710048,47.469253500000036],[7.632701500000053,47.41008349800006],[7.644196500000021,47.36726899800004],[7.78798850000004,47.33769249900007],[7.956853,47.45520049600003],[8.026558,47.395621497000036],[8.007361,47.33940299900007],[7.906974500000047,47.341571499000054],[7.824728,47.26563749700006],[7.838686,47.234413998000036],[7.88614700100004,47.12841499900003],[7.868344500000035,47.046654999000054],[7.95617750100007,47.00342149800008],[7.857267001000025,46.87143149700006],[7.973581,46.77826599700006],[8.049088,46.78800799900006],[8.368889499000034,46.78797549900003],[8.395225,46.77153799900003],[8.448959500000058,46.764517],[8.410411500000066,46.653020498000046],[8.258061500000053,46.52944199800004],[8.015749501000073,46.56329649800006],[7.712379499000065,46.41389649800004],[7.626100500000064,46.44505199900004],[7.538435500000048,46.37744350000003],[7.221457,46.32921249900005],[7.237020500000028,46.553799998000045],[6.983955,46.43804699700007],[6.962045,46.496879499000045],[6.817892500000028,46.516538998000044],[6.887994,46.55794899800003],[6.795106,46.57521850100005],[6.798820500000033,46.64924799800008],[6.992414500000052,46.830588496000075],[6.896209500000055,46.92532599800006],[6.866233500000021,46.90929349800007],[6.931451,46.804543498000044],[6.852262497000027,46.77425905600006],[6.742307,46.827455497000074],[6.780046,46.85264299800008],[6.720867500000054,46.936142999000026],[6.460011,46.85155149800005],[6.43275,46.92845549800006],[6.858914,47.16529449800004],[7.061636,47.34373449800006],[6.879806,47.35243999700003],[6.939186,47.433704499000044],[7.00155,47.45374349700006],[6.982914,47.49427049800005],[7.130353,47.503040498000075],[7.170422,47.44345449800005],[7.326466,47.43985349900004],[7.375684500000034,47.41407699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH021","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Bern","NUTS_NAME":"Bern"},"geometry":{"type":"Polygon","coordinates":[[[7.824728,47.26563749700006],[7.838686,47.234413998000036],[7.88614700100004,47.12841499900003],[7.868344500000035,47.046654999000054],[7.95617750100007,47.00342149800008],[7.857267001000025,46.87143149700006],[7.973581,46.77826599700006],[8.049088,46.78800799900006],[8.368889499000034,46.78797549900003],[8.395225,46.77153799900003],[8.448959500000058,46.764517],[8.410411500000066,46.653020498000046],[8.258061500000053,46.52944199800004],[8.015749501000073,46.56329649800006],[7.712379499000065,46.41389649800004],[7.626100500000064,46.44505199900004],[7.538435500000048,46.37744350000003],[7.221457,46.32921249900005],[7.237020500000028,46.553799998000045],[7.380268001000047,46.69018450000004],[7.295976001000042,46.72939249800004],[7.358897,46.86295499800008],[7.187970500000063,46.900243998000064],[7.229703500000028,46.98092549800003],[7.204961501000071,47.00634249600006],[7.051847,46.97711499700006],[7.039971500000036,46.97988099600008],[7.075720501000035,47.09627099900007],[6.866893,47.085119500000076],[6.861454,47.165625496000075],[6.979885,47.174120499000026],[7.163378501000068,47.29307749600008],[7.558402,47.32237250000003],[7.361447500000054,47.219734499000026],[7.38251450000007,47.16141649900004],[7.495571,47.171581],[7.36945200100007,47.11621],[7.44317250000006,47.074495],[7.52299,47.16081549900008],[7.672689501000036,47.16824599700004],[7.58004,47.27634149900007],[7.824728,47.26563749700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH022","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Freiburg","NUTS_NAME":"Freiburg"},"geometry":{"type":"MultiPolygon","coordinates":[[[[7.229703500000028,46.98092549800003],[7.187970500000063,46.900243998000064],[7.358897,46.86295499800008],[7.295976001000042,46.72939249800004],[7.380268001000047,46.69018450000004],[7.237020500000028,46.553799998000045],[6.983955,46.43804699700007],[6.962045,46.496879499000045],[6.817892500000028,46.516538998000044],[6.887994,46.55794899800003],[6.795106,46.57521850100005],[6.798820500000033,46.64924799800008],[6.992414500000052,46.830588496000075],[6.896209500000055,46.92532599800006],[6.928537500000061,46.95271299600006],[7.039971500000036,46.97988099600008],[7.051847,46.97711499700006],[7.204961501000071,47.00634249600006],[7.229703500000028,46.98092549800003]]],[[[6.931451,46.804543498000044],[6.852262497000027,46.77425905600006],[6.742307,46.827455497000074],[6.780046,46.85264299800008],[6.866233500000021,46.90929349800007],[6.931451,46.804543498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH023","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Solothurn","NUTS_NAME":"Solothurn"},"geometry":{"type":"Polygon","coordinates":[[[7.644196500000021,47.36726899800004],[7.78798850000004,47.33769249900007],[7.956853,47.45520049600003],[8.026558,47.395621497000036],[8.007361,47.33940299900007],[7.906974500000047,47.341571499000054],[7.824728,47.26563749700006],[7.58004,47.27634149900007],[7.672689501000036,47.16824599700004],[7.52299,47.16081549900008],[7.44317250000006,47.074495],[7.36945200100007,47.11621],[7.495571,47.171581],[7.38251450000007,47.16141649900004],[7.361447500000054,47.219734499000026],[7.558402,47.32237250000003],[7.43689,47.379970498000034],[7.615804,47.43270399900007],[7.606372,47.48947],[7.710048,47.469253500000036],[7.632701500000053,47.41008349800006],[7.644196500000021,47.36726899800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH024","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Neuchâtel","NUTS_NAME":"Neuchâtel"},"geometry":{"type":"Polygon","coordinates":[[[7.039971500000036,46.97988099600008],[6.928537500000061,46.95271299600006],[6.896209500000055,46.92532599800006],[6.866233500000021,46.90929349800007],[6.780046,46.85264299800008],[6.720867500000054,46.936142999000026],[6.460011,46.85155149800005],[6.43275,46.92845549800006],[6.858914,47.16529449800004],[6.861454,47.165625496000075],[6.866893,47.085119500000076],[7.075720501000035,47.09627099900007],[7.039971500000036,46.97988099600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH025","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Jura","NUTS_NAME":"Jura"},"geometry":{"type":"Polygon","coordinates":[[[7.326466,47.43985349900004],[7.375684500000034,47.41407699900003],[7.43689,47.379970498000034],[7.558402,47.32237250000003],[7.163378501000068,47.29307749600008],[6.979885,47.174120499000026],[6.861454,47.165625496000075],[6.858914,47.16529449800004],[7.061636,47.34373449800006],[6.879806,47.35243999700003],[6.939186,47.433704499000044],[7.00155,47.45374349700006],[6.982914,47.49427049800005],[7.130353,47.503040498000075],[7.170422,47.44345449800005],[7.326466,47.43985349900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH03","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Nordwestschweiz","NUTS_NAME":"Nordwestschweiz"},"geometry":{"type":"Polygon","coordinates":[[[8.426434500000028,47.567548997000074],[8.377023,47.51242449800003],[8.410716,47.24803749700004],[8.412129500000049,47.14073749700003],[8.278714,47.28206899800006],[7.838686,47.234413998000036],[7.824728,47.26563749700006],[7.906974500000047,47.341571499000054],[8.007361,47.33940299900007],[8.026558,47.395621497000036],[7.956853,47.45520049600003],[7.78798850000004,47.33769249900007],[7.644196500000021,47.36726899800004],[7.632701500000053,47.41008349800006],[7.710048,47.469253500000036],[7.606372,47.48947],[7.615804,47.43270399900007],[7.43689,47.379970498000034],[7.375684500000034,47.41407699900003],[7.326466,47.43985349900004],[7.380894,47.43189249800008],[7.421139499000049,47.446387998000034],[7.445019,47.46172349600005],[7.510905500000035,47.50258249700005],[7.555159500000059,47.56456399600006],[7.589039,47.58987799800008],[7.69337,47.600644998000064],[7.634097,47.56111350000003],[7.713784500000031,47.539404999000055],[7.894107500000075,47.58637499800005],[8.01872,47.550453996000044],[8.221265500000072,47.61551499800004],[8.426434500000028,47.567548997000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH031","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Basel-Stadt","NUTS_NAME":"Basel-Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.634097,47.56111350000003],[7.589327,47.519839],[7.555159500000059,47.56456399600006],[7.589039,47.58987799800008],[7.69337,47.600644998000064],[7.634097,47.56111350000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH032","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Basel-Landschaft","NUTS_NAME":"Basel-Landschaft"},"geometry":{"type":"Polygon","coordinates":[[[7.555159500000059,47.56456399600006],[7.589327,47.519839],[7.634097,47.56111350000003],[7.713784500000031,47.539404999000055],[7.852448500000037,47.535262500000044],[7.956853,47.45520049600003],[7.78798850000004,47.33769249900007],[7.644196500000021,47.36726899800004],[7.632701500000053,47.41008349800006],[7.710048,47.469253500000036],[7.606372,47.48947],[7.615804,47.43270399900007],[7.43689,47.379970498000034],[7.375684500000034,47.41407699900003],[7.326466,47.43985349900004],[7.380894,47.43189249800008],[7.421139499000049,47.446387998000034],[7.445019,47.46172349600005],[7.510905500000035,47.50258249700005],[7.555159500000059,47.56456399600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH033","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Aargau","NUTS_NAME":"Aargau"},"geometry":{"type":"Polygon","coordinates":[[[8.426434500000028,47.567548997000074],[8.377023,47.51242449800003],[8.410716,47.24803749700004],[8.412129500000049,47.14073749700003],[8.278714,47.28206899800006],[7.838686,47.234413998000036],[7.824728,47.26563749700006],[7.906974500000047,47.341571499000054],[8.007361,47.33940299900007],[8.026558,47.395621497000036],[7.956853,47.45520049600003],[7.852448500000037,47.535262500000044],[7.713784500000031,47.539404999000055],[7.894107500000075,47.58637499800005],[8.01872,47.550453996000044],[8.221265500000072,47.61551499800004],[8.426434500000028,47.567548997000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH04","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Zürich","NUTS_NAME":"Zürich"},"geometry":{"type":"Polygon","coordinates":[[[8.739470500000039,47.60811649800007],[8.898724499000025,47.52408549900008],[8.883659500000022,47.47018499700005],[8.943635500000028,47.37583749700008],[8.985038,47.31938],[8.942985500000077,47.26742549900007],[8.807784500000025,47.22056],[8.692483499000048,47.163624998000046],[8.410716,47.24803749700004],[8.377023,47.51242449800003],[8.426434500000028,47.567548997000074],[8.52144,47.633524],[8.562841,47.599432497000066],[8.595602,47.605544499000075],[8.606369500000028,47.66900549800005],[8.663354,47.685892498000044],[8.670461,47.68486249800003],[8.821518,47.64873249800007],[8.739470500000039,47.60811649800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH040","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Zürich","NUTS_NAME":"Zürich"},"geometry":{"type":"Polygon","coordinates":[[[8.739470500000039,47.60811649800007],[8.898724499000025,47.52408549900008],[8.883659500000022,47.47018499700005],[8.943635500000028,47.37583749700008],[8.985038,47.31938],[8.942985500000077,47.26742549900007],[8.807784500000025,47.22056],[8.692483499000048,47.163624998000046],[8.410716,47.24803749700004],[8.377023,47.51242449800003],[8.426434500000028,47.567548997000074],[8.52144,47.633524],[8.562841,47.599432497000066],[8.595602,47.605544499000075],[8.606369500000028,47.66900549800005],[8.663354,47.685892498000044],[8.670461,47.68486249800003],[8.821518,47.64873249800007],[8.739470500000039,47.60811649800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH05","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Ostschweiz","NUTS_NAME":"Ostschweiz"},"geometry":{"type":"Polygon","coordinates":[[[8.727981500000055,47.69268049800007],[8.795708500000046,47.67559549900005],[8.77209750000003,47.718459497000026],[8.843022,47.71226249800003],[8.87499850000006,47.65597349700005],[9.182192,47.655890498000076],[9.495604,47.551454998000054],[9.55872050000005,47.54189299800004],[9.673370500000033,47.38151050000005],[9.530749,47.270581],[9.486198,47.17795649900006],[9.520344,47.10014999800006],[9.47604750000005,47.05179749800004],[9.607078,47.06077449900005],[9.879422500000032,47.01956299600005],[9.876148500000056,46.93463550000007],[10.144974500000046,46.85100949800005],[10.38931799900007,47.00052449900005],[10.489350500000057,46.93779049800003],[10.469651500000055,46.85490899900003],[10.39332150000007,46.68920749600005],[10.402287999000066,46.63681199600006],[10.492064,46.61538649700003],[10.452801,46.53068299800003],[10.295437,46.54989349700003],[10.244874500000037,46.62209199700004],[10.043919500000072,46.53753099800008],[10.042330500000048,46.44236649900006],[10.166681500000038,46.407648],[10.104273500000033,46.33366750000005],[10.176524500000028,46.258276497000054],[10.135604,46.226256499000044],[10.04367,46.22974449700007],[9.996764,46.35093450000005],[9.907831499000054,46.38050599800005],[9.735838500000057,46.35031299800005],[9.714149500000076,46.29270849800008],[9.54957550000006,46.30233799900003],[9.460813500000029,46.375970497000026],[9.464343,46.50778249700005],[9.282739500000048,46.496694999000056],[9.247002500000065,46.44679449800003],[9.300141500000052,46.32645649800003],[9.24853150000007,46.233768],[9.159377500000062,46.16960099900007],[9.054617500000063,46.29498049800003],[9.10225950000006,46.41091799800006],[9.021986,46.51253299900003],[9.042803500000048,46.58880049600003],[8.957378,46.63032499600007],[8.843187999000065,46.563368499000035],[8.678729,46.57918949800006],[8.673950500000046,46.69217649800004],[8.877168500000039,46.81312349800004],[8.95601750000003,46.87536199800007],[8.935079,46.92020799800008],[8.962909500000023,46.95468599600008],[8.911338500000056,47.00887299800007],[9.00456550000007,47.17314549900004],[8.807784500000025,47.22056],[8.942985500000077,47.26742549900007],[8.985038,47.31938],[8.943635500000028,47.37583749700008],[8.883659500000022,47.47018499700005],[8.898724499000025,47.52408549900008],[8.739470500000039,47.60811649800007],[8.821518,47.64873249800007],[8.670461,47.68486249800003],[8.663354,47.685892498000044],[8.606369500000028,47.66900549800005],[8.405565999000032,47.67451349800007],[8.510115,47.77619049900005],[8.61233,47.801461999000026],[8.74010450000003,47.752789],[8.727981500000055,47.69268049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH051","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Glarus","NUTS_NAME":"Glarus"},"geometry":{"type":"Polygon","coordinates":[[[9.248880500000041,46.91626099900003],[8.877168500000039,46.81312349800004],[8.95601750000003,46.87536199800007],[8.935079,46.92020799800008],[8.962909500000023,46.95468599600008],[8.911338500000056,47.00887299800007],[9.00456550000007,47.17314549900004],[9.189854500000024,47.12372099600003],[9.174574500000062,47.05292049800005],[9.236411500000031,47.023986499000046],[9.248880500000041,46.91626099900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH052","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Schaffhausen","NUTS_NAME":"Schaffhausen"},"geometry":{"type":"Polygon","coordinates":[[[8.717719500000044,47.69074199800008],[8.670461,47.68486249800003],[8.663354,47.685892498000044],[8.606369500000028,47.66900549800005],[8.405565999000032,47.67451349800007],[8.510115,47.77619049900005],[8.61233,47.801461999000026],[8.74010450000003,47.752789],[8.727981500000055,47.69268049800007],[8.717719500000044,47.69074199800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH053","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Appenzell Ausserrhoden","NUTS_NAME":"Appenzell Ausserrhoden"},"geometry":{"type":"Polygon","coordinates":[[[9.50900850000005,47.39455249800005],[9.502979,47.34737049800003],[9.358357500000068,47.37799149800003],[9.309886500000061,47.32275449800005],[9.342937500000062,47.24933699800005],[9.206248500000072,47.276508496000076],[9.208116499000027,47.36197549800005],[9.232476500000075,47.398071499000025],[9.397470500000054,47.400663499000075],[9.526187,47.45995949700006],[9.617908,47.437167998000064],[9.582882,47.40976899800006],[9.555321,47.39662799800004],[9.50900850000005,47.39455249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH054","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Appenzell Innerrhoden","NUTS_NAME":"Appenzell Innerrhoden"},"geometry":{"type":"Polygon","coordinates":[[[9.46114650000004,47.263994498000045],[9.342937500000062,47.24933699800005],[9.309886500000061,47.32275449800005],[9.358357500000068,47.37799149800003],[9.502979,47.34737049800003],[9.46114650000004,47.263994498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH055","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"St. Gallen","NUTS_NAME":"St. Gallen"},"geometry":{"type":"Polygon","coordinates":[[[9.55872050000005,47.54189299800004],[9.673370500000033,47.38151050000005],[9.530749,47.270581],[9.486198,47.17795649900006],[9.520344,47.10014999800006],[9.47604750000005,47.05179749800004],[9.54571750100007,46.976735],[9.448866,46.87288149900007],[9.248880500000041,46.91626099900003],[9.236411500000031,47.023986499000046],[9.174574500000062,47.05292049800005],[9.189854500000024,47.12372099600003],[9.00456550000007,47.17314549900004],[8.807784500000025,47.22056],[8.942985500000077,47.26742549900007],[8.985038,47.31938],[8.943635500000028,47.37583749700008],[9.012069500000052,47.48947],[9.299058500000058,47.47774499900004],[9.316977001000055,47.50865499900004],[9.268870999000058,47.51171949800005],[9.315444,47.53028949900005],[9.391306500000042,47.47648550000008],[9.495604,47.551454998000054],[9.55872050000005,47.54189299800004]],[[9.342937500000062,47.24933699800005],[9.46114650000004,47.263994498000045],[9.502979,47.34737049800003],[9.50900850000005,47.39455249800005],[9.555321,47.39662799800004],[9.582882,47.40976899800006],[9.617908,47.437167998000064],[9.526187,47.45995949700006],[9.397470500000054,47.400663499000075],[9.232476500000075,47.398071499000025],[9.208116499000027,47.36197549800005],[9.206248500000072,47.276508496000076],[9.342937500000062,47.24933699800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH056","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Graubünden","NUTS_NAME":"Graubünden"},"geometry":{"type":"Polygon","coordinates":[[[10.144974500000046,46.85100949800005],[10.38931799900007,47.00052449900005],[10.489350500000057,46.93779049800003],[10.469651500000055,46.85490899900003],[10.39332150000007,46.68920749600005],[10.402287999000066,46.63681199600006],[10.492064,46.61538649700003],[10.452801,46.53068299800003],[10.295437,46.54989349700003],[10.244874500000037,46.62209199700004],[10.043919500000072,46.53753099800008],[10.042330500000048,46.44236649900006],[10.166681500000038,46.407648],[10.104273500000033,46.33366750000005],[10.176524500000028,46.258276497000054],[10.135604,46.226256499000044],[10.04367,46.22974449700007],[9.996764,46.35093450000005],[9.907831499000054,46.38050599800005],[9.735838500000057,46.35031299800005],[9.714149500000076,46.29270849800008],[9.54957550000006,46.30233799900003],[9.460813500000029,46.375970497000026],[9.464343,46.50778249700005],[9.282739500000048,46.496694999000056],[9.247002500000065,46.44679449800003],[9.300141500000052,46.32645649800003],[9.24853150000007,46.233768],[9.159377500000062,46.16960099900007],[9.054617500000063,46.29498049800003],[9.10225950000006,46.41091799800006],[9.021986,46.51253299900003],[9.042803500000048,46.58880049600003],[8.957378,46.63032499600007],[8.843187999000065,46.563368499000035],[8.678729,46.57918949800006],[8.673950500000046,46.69217649800004],[8.877168500000039,46.81312349800004],[9.248880500000041,46.91626099900003],[9.448866,46.87288149900007],[9.54571750100007,46.976735],[9.47604750000005,47.05179749800004],[9.607078,47.06077449900005],[9.879422500000032,47.01956299600005],[9.876148500000056,46.93463550000007],[10.144974500000046,46.85100949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH057","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Thurgau","NUTS_NAME":"Thurgau"},"geometry":{"type":"Polygon","coordinates":[[[8.843022,47.71226249800003],[8.87499850000006,47.65597349700005],[9.182192,47.655890498000076],[9.495604,47.551454998000054],[9.391306500000042,47.47648550000008],[9.315444,47.53028949900005],[9.268870999000058,47.51171949800005],[9.316977001000055,47.50865499900004],[9.299058500000058,47.47774499900004],[9.012069500000052,47.48947],[8.943635500000028,47.37583749700008],[8.883659500000022,47.47018499700005],[8.898724499000025,47.52408549900008],[8.739470500000039,47.60811649800007],[8.821518,47.64873249800007],[8.670461,47.68486249800003],[8.717719500000044,47.69074199800008],[8.727981500000055,47.69268049800007],[8.795708500000046,47.67559549900005],[8.77209750000003,47.718459497000026],[8.843022,47.71226249800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH06","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Zentralschweiz","NUTS_NAME":"Zentralschweiz"},"geometry":{"type":"Polygon","coordinates":[[[8.278714,47.28206899800006],[8.412129500000049,47.14073749700003],[8.410716,47.24803749700004],[8.692483499000048,47.163624998000046],[8.807784500000025,47.22056],[9.00456550000007,47.17314549900004],[8.911338500000056,47.00887299800007],[8.962909500000023,46.95468599600008],[8.935079,46.92020799800008],[8.95601750000003,46.87536199800007],[8.877168500000039,46.81312349800004],[8.673950500000046,46.69217649800004],[8.678729,46.57918949800006],[8.5392,46.58677799800006],[8.477673500000037,46.52760249800008],[8.410029,46.57725749800005],[8.410411500000066,46.653020498000046],[8.448959500000058,46.764517],[8.395225,46.77153799900003],[8.368889499000034,46.78797549900003],[8.049088,46.78800799900006],[7.973581,46.77826599700006],[7.857267001000025,46.87143149700006],[7.95617750100007,47.00342149800008],[7.868344500000035,47.046654999000054],[7.88614700100004,47.12841499900003],[7.838686,47.234413998000036],[8.278714,47.28206899800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH061","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Luzern","NUTS_NAME":"Luzern"},"geometry":{"type":"Polygon","coordinates":[[[8.412129500000049,47.14073749700003],[8.49297850000005,47.10020099800005],[8.389211500000044,47.07064699700004],[8.51359350000007,47.021925999000075],[8.46815650000002,46.99652449800004],[8.228819,46.973145500000044],[8.069876499000031,46.89576149900006],[8.049088,46.78800799900006],[7.973581,46.77826599700006],[7.857267001000025,46.87143149700006],[7.95617750100007,47.00342149800008],[7.868344500000035,47.046654999000054],[7.88614700100004,47.12841499900003],[7.838686,47.234413998000036],[8.278714,47.28206899800006],[8.412129500000049,47.14073749700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH062","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Uri","NUTS_NAME":"Uri"},"geometry":{"type":"Polygon","coordinates":[[[8.935079,46.92020799800008],[8.95601750000003,46.87536199800007],[8.877168500000039,46.81312349800004],[8.673950500000046,46.69217649800004],[8.678729,46.57918949800006],[8.5392,46.58677799800006],[8.477673500000037,46.52760249800008],[8.410029,46.57725749800005],[8.410411500000066,46.653020498000046],[8.448959500000058,46.764517],[8.491425500000048,46.77305799800007],[8.507028,46.845467496000026],[8.471987500000068,46.85535199900005],[8.458677,46.89895849700008],[8.569149500000037,46.99024799800003],[8.853211,46.885387997000066],[8.935079,46.92020799800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH063","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Schwyz","NUTS_NAME":"Schwyz"},"geometry":{"type":"Polygon","coordinates":[[[9.00456550000007,47.17314549900004],[8.911338500000056,47.00887299800007],[8.962909500000023,46.95468599600008],[8.935079,46.92020799800008],[8.853211,46.885387997000066],[8.569149500000037,46.99024799800003],[8.46815650000002,46.99652449800004],[8.51359350000007,47.021925999000075],[8.389211500000044,47.07064699700004],[8.49297850000005,47.10020099800005],[8.637382999000067,47.09328849800005],[8.692483499000048,47.163624998000046],[8.807784500000025,47.22056],[9.00456550000007,47.17314549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH064","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Obwalden","NUTS_NAME":"Obwalden"},"geometry":{"type":"Polygon","coordinates":[[[8.368889499000034,46.78797549900003],[8.049088,46.78800799900006],[8.069876499000031,46.89576149900006],[8.228819,46.973145500000044],[8.309523500000068,46.968980498000064],[8.343371500000046,46.914896498000076],[8.32194449900004,46.82044549900007],[8.368889499000034,46.78797549900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH065","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Nidwalden","NUTS_NAME":"Nidwalden"},"geometry":{"type":"Polygon","coordinates":[[[8.491425500000048,46.77305799800007],[8.448959500000058,46.764517],[8.395225,46.77153799900003],[8.368889499000034,46.78797549900003],[8.32194449900004,46.82044549900007],[8.343371500000046,46.914896498000076],[8.309523500000068,46.968980498000064],[8.228819,46.973145500000044],[8.46815650000002,46.99652449800004],[8.569149500000037,46.99024799800003],[8.458677,46.89895849700008],[8.471987500000068,46.85535199900005],[8.507028,46.845467496000026],[8.491425500000048,46.77305799800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH066","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Zug","NUTS_NAME":"Zug"},"geometry":{"type":"Polygon","coordinates":[[[8.692483499000048,47.163624998000046],[8.637382999000067,47.09328849800005],[8.49297850000005,47.10020099800005],[8.412129500000049,47.14073749700003],[8.410716,47.24803749700004],[8.692483499000048,47.163624998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH07","LEVL_CODE":2,"CNTR_CODE":"CH","NAME_LATN":"Ticino","NUTS_NAME":"Ticino"},"geometry":{"type":"Polygon","coordinates":[[[9.159377500000062,46.16960099900007],[8.98827650000004,45.97228249700004],[9.08880350000004,45.896897],[9.034462,45.82301699900006],[8.912147,45.830444998000075],[8.893755,45.959014996000064],[8.785907,45.989075496000055],[8.852114,46.07564149700005],[8.713936,46.09727199800005],[8.444526500000052,46.24887749800007],[8.452797,46.455350999000075],[8.384717,46.452158498000074],[8.477673500000037,46.52760249800008],[8.5392,46.58677799800006],[8.678729,46.57918949800006],[8.843187999000065,46.563368499000035],[8.957378,46.63032499600007],[9.042803500000048,46.58880049600003],[9.021986,46.51253299900003],[9.10225950000006,46.41091799800006],[9.054617500000063,46.29498049800003],[9.159377500000062,46.16960099900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CH070","LEVL_CODE":3,"CNTR_CODE":"CH","NAME_LATN":"Ticino","NUTS_NAME":"Ticino"},"geometry":{"type":"Polygon","coordinates":[[[9.159377500000062,46.16960099900007],[8.98827650000004,45.97228249700004],[9.08880350000004,45.896897],[9.034462,45.82301699900006],[8.912147,45.830444998000075],[8.893755,45.959014996000064],[8.785907,45.989075496000055],[8.852114,46.07564149700005],[8.713936,46.09727199800005],[8.444526500000052,46.24887749800007],[8.452797,46.455350999000075],[8.384717,46.452158498000074],[8.477673500000037,46.52760249800008],[8.5392,46.58677799800006],[8.678729,46.57918949800006],[8.843187999000065,46.563368499000035],[8.957378,46.63032499600007],[9.042803500000048,46.58880049600003],[9.021986,46.51253299900003],[9.10225950000006,46.41091799800006],[9.054617500000063,46.29498049800003],[9.159377500000062,46.16960099900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CY","LEVL_CODE":0,"CNTR_CODE":"CY","NAME_LATN":"KÝPROS","NUTS_NAME":"ΚΥΠΡΟΣ"},"geometry":{"type":"Polygon","coordinates":[[[34.43715,35.60157049800006],[33.906834,35.257217999000034],[33.91057550000005,35.17510999700005],[34.07786150000004,34.966491],[33.67788850000005,34.971539998000026],[33.582236,34.82052050000004],[33.006973499000026,34.648288499000046],[33.030378,34.56339849700004],[32.94345150000004,34.570218999000076],[32.874639,34.66677849800004],[32.705401,34.64146499700007],[32.42596450000008,34.74319249700005],[32.315556,34.90226699700003],[32.269781,35.06621399800008],[32.285665,35.105436499000064],[32.42817650000006,35.045067998000036],[32.548685,35.17352499700007],[32.652646,35.19536499900005],[32.881518,35.16221999900006],[32.93490650000007,35.268396997000025],[32.92142450000006,35.40395999700007],[33.371264,35.33413999800007],[33.652597,35.35919099800003],[34.588354,35.695434498000054],[34.43715,35.60157049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CY0","LEVL_CODE":1,"CNTR_CODE":"CY","NAME_LATN":"KÝPROS","NUTS_NAME":"ΚΥΠΡΟΣ"},"geometry":{"type":"Polygon","coordinates":[[[34.43715,35.60157049800006],[33.906834,35.257217999000034],[33.91057550000005,35.17510999700005],[34.07786150000004,34.966491],[33.67788850000005,34.971539998000026],[33.582236,34.82052050000004],[33.006973499000026,34.648288499000046],[33.030378,34.56339849700004],[32.94345150000004,34.570218999000076],[32.874639,34.66677849800004],[32.705401,34.64146499700007],[32.42596450000008,34.74319249700005],[32.315556,34.90226699700003],[32.269781,35.06621399800008],[32.285665,35.105436499000064],[32.42817650000006,35.045067998000036],[32.548685,35.17352499700007],[32.652646,35.19536499900005],[32.881518,35.16221999900006],[32.93490650000007,35.268396997000025],[32.92142450000006,35.40395999700007],[33.371264,35.33413999800007],[33.652597,35.35919099800003],[34.588354,35.695434498000054],[34.43715,35.60157049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CY00","LEVL_CODE":2,"CNTR_CODE":"CY","NAME_LATN":"Kýpros","NUTS_NAME":"Κύπρος"},"geometry":{"type":"Polygon","coordinates":[[[34.43715,35.60157049800006],[33.906834,35.257217999000034],[33.91057550000005,35.17510999700005],[34.07786150000004,34.966491],[33.67788850000005,34.971539998000026],[33.582236,34.82052050000004],[33.006973499000026,34.648288499000046],[33.030378,34.56339849700004],[32.94345150000004,34.570218999000076],[32.874639,34.66677849800004],[32.705401,34.64146499700007],[32.42596450000008,34.74319249700005],[32.315556,34.90226699700003],[32.269781,35.06621399800008],[32.285665,35.105436499000064],[32.42817650000006,35.045067998000036],[32.548685,35.17352499700007],[32.652646,35.19536499900005],[32.881518,35.16221999900006],[32.93490650000007,35.268396997000025],[32.92142450000006,35.40395999700007],[33.371264,35.33413999800007],[33.652597,35.35919099800003],[34.588354,35.695434498000054],[34.43715,35.60157049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CY000","LEVL_CODE":3,"CNTR_CODE":"CY","NAME_LATN":"Kýpros","NUTS_NAME":"Κύπρος"},"geometry":{"type":"Polygon","coordinates":[[[34.43715,35.60157049800006],[33.906834,35.257217999000034],[33.91057550000005,35.17510999700005],[34.07786150000004,34.966491],[33.67788850000005,34.971539998000026],[33.582236,34.82052050000004],[33.006973499000026,34.648288499000046],[33.030378,34.56339849700004],[32.94345150000004,34.570218999000076],[32.874639,34.66677849800004],[32.705401,34.64146499700007],[32.42596450000008,34.74319249700005],[32.315556,34.90226699700003],[32.269781,35.06621399800008],[32.285665,35.105436499000064],[32.42817650000006,35.045067998000036],[32.548685,35.17352499700007],[32.652646,35.19536499900005],[32.881518,35.16221999900006],[32.93490650000007,35.268396997000025],[32.92142450000006,35.40395999700007],[33.371264,35.33413999800007],[33.652597,35.35919099800003],[34.588354,35.695434498000054],[34.43715,35.60157049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ","LEVL_CODE":0,"CNTR_CODE":"CZ","NAME_LATN":"CESKÁ REPUBLIKA","NUTS_NAME":"CESKÁ REPUBLIKA"},"geometry":{"type":"Polygon","coordinates":[[[14.823362,50.87055049600008],[15.00204,50.869024],[15.019613,50.96659899900004],[14.968252,50.990008999000054],[15.032691,51.021315998000034],[15.274085,50.97954299700007],[15.277071,50.891020999000034],[15.370452,50.780894999000054],[15.535267500000032,50.779375998000035],[15.816207,50.755322],[15.860921,50.67491299800008],[15.990918,50.683419],[15.986371,50.61474699900003],[16.024848,50.598625],[16.107318,50.66207299800004],[16.322517,50.66645299700008],[16.443536,50.586257499000055],[16.195729,50.432135],[16.360988500000076,50.37923649600003],[16.548217,50.22978399800007],[16.58029,50.14278799700003],[16.70553,50.09651149800004],[16.86327,50.19812299800003],[17.028323,50.22999699800005],[16.860915,50.40808099700007],[16.907924500000036,50.449454],[17.429605,50.254513],[17.71840399900003,50.32094999900005],[17.758479,50.20656799900007],[17.592736,50.16001399900006],[17.868675,49.972545996000065],[18.035060999000052,50.06577199800006],[18.321821,49.915819999000064],[18.575724,49.91042299900005],[18.627212,49.72232999600004],[18.804583,49.67888899900004],[18.851551,49.517189],[18.564790500000072,49.50693449700003],[18.40359550000005,49.39674549800003],[18.322436,49.31505899900003],[18.14669250000003,49.247281],[18.069837500000062,49.037547999000026],[17.924501,49.02002299700007],[17.884914500000036,48.92775350000005],[17.64693,48.85426599700003],[17.39672550000006,48.813349997000046],[17.201662500000054,48.87802899600007],[16.940278,48.617245498000045],[16.90202350000004,48.71797799800004],[16.648492,48.78520849800003],[16.102884500000073,48.74539599900004],[15.84331,48.873388498000054],[15.753633500000035,48.852178500000036],[15.542449500000032,48.90769699600003],[15.365882999000064,48.98170699900004],[15.160247,48.94168649800008],[15.157331,48.991779500000064],[14.990445,49.009649],[14.979332,48.77235049800004],[14.808667,48.77879949900006],[14.691014,48.58430199700007],[14.470069,48.64843699800008],[14.324520500000062,48.56233949700004],[14.115935,48.58917099800004],[13.839507,48.771605],[13.551578500000062,48.96778749600003],[13.416651500000057,48.98003549800006],[13.397147,49.05100999800004],[13.170907999000065,49.17357950000007],[12.972595500000068,49.33388799800008],[12.77881,49.34807849600003],[12.633763500000043,49.47619499700005],[12.644176,49.52305749900006],[12.593778500000042,49.54219149800008],[12.521638,49.686761999000055],[12.401524,49.75837299700004],[12.550988500000074,49.905088],[12.470510500000046,49.94502550000004],[12.48988950000006,49.98176099900007],[12.260799,50.05815649700003],[12.160679,50.21984899700004],[12.09076,50.25238450000006],[12.13994,50.27771849800007],[12.100900500000023,50.31802799700006],[12.184453,50.32232449800006],[12.335119500000076,50.17200749800003],[12.331084500000031,50.24271399700007],[12.398014999000054,50.32115349800006],[12.58377850000005,50.40707899800003],[12.820126500000072,50.45976249800003],[12.948144500000069,50.40431149800003],[13.059457500000065,50.50104749800005],[13.193359500000042,50.503172498000026],[13.255104,50.59530099800003],[13.501846,50.63364299800003],[13.525244500000042,50.70441099900006],[13.652174,50.73035999700005],[13.855824,50.72732599700004],[13.897461500000077,50.79214599800008],[14.388335,50.90029899700005],[14.258529500000066,50.98798949800005],[14.317873500000076,51.05469899800005],[14.491221,51.04353049800005],[14.599177500000053,50.98711099700006],[14.564038,50.91849749700003],[14.650508500000058,50.929418499000064],[14.6188,50.85780449600003],[14.766383,50.81933250000003],[14.823362,50.87055049600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ0","LEVL_CODE":1,"CNTR_CODE":"CZ","NAME_LATN":"CESKÁ REPUBLIKA","NUTS_NAME":"CESKÁ REPUBLIKA"},"geometry":{"type":"Polygon","coordinates":[[[14.599177500000053,50.98711099700006],[14.564038,50.91849749700003],[14.650508500000058,50.929418499000064],[14.6188,50.85780449600003],[14.766383,50.81933250000003],[14.823362,50.87055049600008],[15.00204,50.869024],[15.019613,50.96659899900004],[14.968252,50.990008999000054],[15.032691,51.021315998000034],[15.274085,50.97954299700007],[15.277071,50.891020999000034],[15.370452,50.780894999000054],[15.535267500000032,50.779375998000035],[15.816207,50.755322],[15.860921,50.67491299800008],[15.990918,50.683419],[15.986371,50.61474699900003],[16.024848,50.598625],[16.107318,50.66207299800004],[16.322517,50.66645299700008],[16.443536,50.586257499000055],[16.195729,50.432135],[16.360988500000076,50.37923649600003],[16.548217,50.22978399800007],[16.58029,50.14278799700003],[16.70553,50.09651149800004],[16.86327,50.19812299800003],[17.028323,50.22999699800005],[16.860915,50.40808099700007],[16.907924500000036,50.449454],[17.429605,50.254513],[17.71840399900003,50.32094999900005],[17.758479,50.20656799900007],[17.592736,50.16001399900006],[17.868675,49.972545996000065],[18.035060999000052,50.06577199800006],[18.321821,49.915819999000064],[18.575724,49.91042299900005],[18.627212,49.72232999600004],[18.804583,49.67888899900004],[18.851551,49.517189],[18.564790500000072,49.50693449700003],[18.40359550000005,49.39674549800003],[18.322436,49.31505899900003],[18.14669250000003,49.247281],[18.069837500000062,49.037547999000026],[17.924501,49.02002299700007],[17.884914500000036,48.92775350000005],[17.64693,48.85426599700003],[17.39672550000006,48.813349997000046],[17.201662500000054,48.87802899600007],[16.940278,48.617245498000045],[16.90202350000004,48.71797799800004],[16.648492,48.78520849800003],[16.102884500000073,48.74539599900004],[15.84331,48.873388498000054],[15.753633500000035,48.852178500000036],[15.542449500000032,48.90769699600003],[15.365882999000064,48.98170699900004],[15.160247,48.94168649800008],[15.157331,48.991779500000064],[14.990445,49.009649],[14.979332,48.77235049800004],[14.808667,48.77879949900006],[14.691014,48.58430199700007],[14.470069,48.64843699800008],[14.324520500000062,48.56233949700004],[14.115935,48.58917099800004],[13.839507,48.771605],[13.551578500000062,48.96778749600003],[13.416651500000057,48.98003549800006],[13.397147,49.05100999800004],[13.170907999000065,49.17357950000007],[12.972595500000068,49.33388799800008],[12.77881,49.34807849600003],[12.633763500000043,49.47619499700005],[12.644176,49.52305749900006],[12.593778500000042,49.54219149800008],[12.521638,49.686761999000055],[12.401524,49.75837299700004],[12.550988500000074,49.905088],[12.470510500000046,49.94502550000004],[12.48988950000006,49.98176099900007],[12.260799,50.05815649700003],[12.160679,50.21984899700004],[12.09076,50.25238450000006],[12.13994,50.27771849800007],[12.100900500000023,50.31802799700006],[12.184453,50.32232449800006],[12.335119500000076,50.17200749800003],[12.331084500000031,50.24271399700007],[12.398014999000054,50.32115349800006],[12.58377850000005,50.40707899800003],[12.820126500000072,50.45976249800003],[12.948144500000069,50.40431149800003],[13.059457500000065,50.50104749800005],[13.193359500000042,50.503172498000026],[13.255104,50.59530099800003],[13.501846,50.63364299800003],[13.525244500000042,50.70441099900006],[13.652174,50.73035999700005],[13.855824,50.72732599700004],[13.897461500000077,50.79214599800008],[14.388335,50.90029899700005],[14.258529500000066,50.98798949800005],[14.317873500000076,51.05469899800005],[14.491221,51.04353049800005],[14.599177500000053,50.98711099700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ01","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Praha","NUTS_NAME":"Praha"},"geometry":{"type":"Polygon","coordinates":[[[14.669353500000057,50.01629299900003],[14.338305500000047,49.94835449900006],[14.247978500000045,50.05828450000007],[14.290380500000026,50.074715498000046],[14.224306,50.102918998000064],[14.50944,50.17417849900005],[14.658959500000037,50.122348997000074],[14.700559,50.07240599600004],[14.640119,50.05686350000008],[14.669353500000057,50.01629299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ010","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Hlavní mesto Praha","NUTS_NAME":"Hlavní mesto Praha"},"geometry":{"type":"Polygon","coordinates":[[[14.669353500000057,50.01629299900003],[14.338305500000047,49.94835449900006],[14.247978500000045,50.05828450000007],[14.290380500000026,50.074715498000046],[14.224306,50.102918998000064],[14.50944,50.17417849900005],[14.658959500000037,50.122348997000074],[14.700559,50.07240599600004],[14.640119,50.05686350000008],[14.669353500000057,50.01629299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ02","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Strední Cechy","NUTS_NAME":"Strední Cechy"},"geometry":{"type":"Polygon","coordinates":[[[15.14677050000006,50.52294249700003],[15.162239,50.42722249800005],[15.104144500000075,50.376095],[15.235668500000031,50.28663149600004],[15.371035,50.28761999900007],[15.398379,50.205730998000035],[15.35796350000004,50.14709699800005],[15.437586500000066,50.10985749800005],[15.362708500000053,50.041348998000046],[15.532581999000058,49.96002149800006],[15.496953,49.86118449800006],[15.22654,49.75578949700008],[15.177543,49.69644049800007],[15.249481,49.64360449800006],[15.229882,49.61490199800005],[14.981185,49.60631650000005],[14.932018500000027,49.54910099800003],[14.779588,49.605034999000054],[14.635810500000048,49.517269499000065],[14.265731,49.567451498000025],[13.76559050000003,49.513990498000055],[13.72752,49.67849049800003],[13.672056,49.69716449800006],[13.816725500000075,49.75607249600006],[13.818361,49.92002499600005],[13.407053500000075,50.089616998000054],[13.535772500000064,50.18898249800003],[14.007528,50.346400997000046],[14.367341500000066,50.35235399800007],[14.359461,50.42840049700004],[14.486485499000025,50.505026996000026],[14.676815,50.486662498000044],[14.909923,50.615496998000026],[15.14677050000006,50.52294249700003]],[[14.669353500000057,50.01629299900003],[14.640119,50.05686350000008],[14.700559,50.07240599600004],[14.658959500000037,50.122348997000074],[14.50944,50.17417849900005],[14.224306,50.102918998000064],[14.290380500000026,50.074715498000046],[14.247978500000045,50.05828450000007],[14.338305500000047,49.94835449900006],[14.669353500000057,50.01629299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ020","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Stredoceský kraj","NUTS_NAME":"Stredoceský kraj"},"geometry":{"type":"Polygon","coordinates":[[[15.14677050000006,50.52294249700003],[15.162239,50.42722249800005],[15.104144500000075,50.376095],[15.235668500000031,50.28663149600004],[15.371035,50.28761999900007],[15.398379,50.205730998000035],[15.35796350000004,50.14709699800005],[15.437586500000066,50.10985749800005],[15.362708500000053,50.041348998000046],[15.532581999000058,49.96002149800006],[15.496953,49.86118449800006],[15.22654,49.75578949700008],[15.177543,49.69644049800007],[15.249481,49.64360449800006],[15.229882,49.61490199800005],[14.981185,49.60631650000005],[14.932018500000027,49.54910099800003],[14.779588,49.605034999000054],[14.635810500000048,49.517269499000065],[14.265731,49.567451498000025],[13.76559050000003,49.513990498000055],[13.72752,49.67849049800003],[13.672056,49.69716449800006],[13.816725500000075,49.75607249600006],[13.818361,49.92002499600005],[13.407053500000075,50.089616998000054],[13.535772500000064,50.18898249800003],[14.007528,50.346400997000046],[14.367341500000066,50.35235399800007],[14.359461,50.42840049700004],[14.486485499000025,50.505026996000026],[14.676815,50.486662498000044],[14.909923,50.615496998000026],[15.14677050000006,50.52294249700003]],[[14.669353500000057,50.01629299900003],[14.640119,50.05686350000008],[14.700559,50.07240599600004],[14.658959500000037,50.122348997000074],[14.50944,50.17417849900005],[14.224306,50.102918998000064],[14.290380500000026,50.074715498000046],[14.247978500000045,50.05828450000007],[14.338305500000047,49.94835449900006],[14.669353500000057,50.01629299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ03","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Jihozápad","NUTS_NAME":"Jihozápad"},"geometry":{"type":"Polygon","coordinates":[[[13.407053500000075,50.089616998000054],[13.818361,49.92002499600005],[13.816725500000075,49.75607249600006],[13.672056,49.69716449800006],[13.72752,49.67849049800003],[13.76559050000003,49.513990498000055],[14.265731,49.567451498000025],[14.635810500000048,49.517269499000065],[14.779588,49.605034999000054],[14.932018500000027,49.54910099800003],[14.903884,49.33314849900006],[15.139395500000035,49.221159998000076],[15.345593500000064,49.21642249800004],[15.350366500000064,49.155695998000056],[15.42224,49.12483449800004],[15.567509,49.12886399900003],[15.597954,49.07659149700004],[15.487282,48.99691499900007],[15.586756500000035,48.94715949700003],[15.542449500000032,48.90769699600003],[15.365882999000064,48.98170699900004],[15.160247,48.94168649800008],[15.157331,48.991779500000064],[14.990445,49.009649],[14.979332,48.77235049800004],[14.808667,48.77879949900006],[14.691014,48.58430199700007],[14.470069,48.64843699800008],[14.324520500000062,48.56233949700004],[14.115935,48.58917099800004],[13.839507,48.771605],[13.551578500000062,48.96778749600003],[13.416651500000057,48.98003549800006],[13.397147,49.05100999800004],[13.170907999000065,49.17357950000007],[12.972595500000068,49.33388799800008],[12.77881,49.34807849600003],[12.633763500000043,49.47619499700005],[12.644176,49.52305749900006],[12.593778500000042,49.54219149800008],[12.521638,49.686761999000055],[12.401524,49.75837299700004],[12.550988500000074,49.905088],[12.928443500000071,49.92093149800007],[13.098285500000031,50.035060498000064],[13.232419,49.99631899800005],[13.301367,50.099643496000056],[13.407053500000075,50.089616998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ031","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Jihoceský kraj","NUTS_NAME":"Jihoceský kraj"},"geometry":{"type":"Polygon","coordinates":[[[14.932018500000027,49.54910099800003],[14.903884,49.33314849900006],[15.139395500000035,49.221159998000076],[15.345593500000064,49.21642249800004],[15.350366500000064,49.155695998000056],[15.42224,49.12483449800004],[15.567509,49.12886399900003],[15.597954,49.07659149700004],[15.487282,48.99691499900007],[15.586756500000035,48.94715949700003],[15.542449500000032,48.90769699600003],[15.365882999000064,48.98170699900004],[15.160247,48.94168649800008],[15.157331,48.991779500000064],[14.990445,49.009649],[14.979332,48.77235049800004],[14.808667,48.77879949900006],[14.691014,48.58430199700007],[14.470069,48.64843699800008],[14.324520500000062,48.56233949700004],[14.115935,48.58917099800004],[13.839507,48.771605],[13.551578500000062,48.96778749600003],[13.543288,49.01807949800008],[13.606967,49.06437249800007],[13.59061550000007,49.147718499000064],[13.692726500000049,49.17021899900004],[13.697608500000058,49.27909049900006],[13.790665,49.36363749900005],[13.76559050000003,49.513990498000055],[14.265731,49.567451498000025],[14.635810500000048,49.517269499000065],[14.779588,49.605034999000054],[14.932018500000027,49.54910099800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ032","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Plzenský kraj","NUTS_NAME":"Plzenský kraj"},"geometry":{"type":"Polygon","coordinates":[[[13.407053500000075,50.089616998000054],[13.818361,49.92002499600005],[13.816725500000075,49.75607249600006],[13.672056,49.69716449800006],[13.72752,49.67849049800003],[13.76559050000003,49.513990498000055],[13.790665,49.36363749900005],[13.697608500000058,49.27909049900006],[13.692726500000049,49.17021899900004],[13.59061550000007,49.147718499000064],[13.606967,49.06437249800007],[13.543288,49.01807949800008],[13.551578500000062,48.96778749600003],[13.416651500000057,48.98003549800006],[13.397147,49.05100999800004],[13.170907999000065,49.17357950000007],[12.972595500000068,49.33388799800008],[12.77881,49.34807849600003],[12.633763500000043,49.47619499700005],[12.644176,49.52305749900006],[12.593778500000042,49.54219149800008],[12.521638,49.686761999000055],[12.401524,49.75837299700004],[12.550988500000074,49.905088],[12.928443500000071,49.92093149800007],[13.098285500000031,50.035060498000064],[13.232419,49.99631899800005],[13.301367,50.099643496000056],[13.407053500000075,50.089616998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ04","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Severozápad","NUTS_NAME":"Severozápad"},"geometry":{"type":"Polygon","coordinates":[[[14.599177500000053,50.98711099700006],[14.564038,50.91849749700003],[14.650508500000058,50.929418499000064],[14.6188,50.85780449600003],[14.442563,50.81133999900004],[14.343548,50.66792399900004],[14.486485499000025,50.505026996000026],[14.359461,50.42840049700004],[14.367341500000066,50.35235399800007],[14.007528,50.346400997000046],[13.535772500000064,50.18898249800003],[13.407053500000075,50.089616998000054],[13.301367,50.099643496000056],[13.232419,49.99631899800005],[13.098285500000031,50.035060498000064],[12.928443500000071,49.92093149800007],[12.550988500000074,49.905088],[12.470510500000046,49.94502550000004],[12.48988950000006,49.98176099900007],[12.260799,50.05815649700003],[12.160679,50.21984899700004],[12.09076,50.25238450000006],[12.13994,50.27771849800007],[12.100900500000023,50.31802799700006],[12.184453,50.32232449800006],[12.335119500000076,50.17200749800003],[12.331084500000031,50.24271399700007],[12.398014999000054,50.32115349800006],[12.58377850000005,50.40707899800003],[12.820126500000072,50.45976249800003],[12.948144500000069,50.40431149800003],[13.059457500000065,50.50104749800005],[13.193359500000042,50.503172498000026],[13.255104,50.59530099800003],[13.501846,50.63364299800003],[13.525244500000042,50.70441099900006],[13.652174,50.73035999700005],[13.855824,50.72732599700004],[13.897461500000077,50.79214599800008],[14.388335,50.90029899700005],[14.258529500000066,50.98798949800005],[14.317873500000076,51.05469899800005],[14.491221,51.04353049800005],[14.599177500000053,50.98711099700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ041","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Karlovarský kraj","NUTS_NAME":"Karlovarský kraj"},"geometry":{"type":"Polygon","coordinates":[[[12.948144500000069,50.40431149800003],[13.221889500000032,50.35225599900008],[13.222257500000069,50.24194149600004],[13.301367,50.099643496000056],[13.232419,49.99631899800005],[13.098285500000031,50.035060498000064],[12.928443500000071,49.92093149800007],[12.550988500000074,49.905088],[12.470510500000046,49.94502550000004],[12.48988950000006,49.98176099900007],[12.260799,50.05815649700003],[12.160679,50.21984899700004],[12.09076,50.25238450000006],[12.13994,50.27771849800007],[12.100900500000023,50.31802799700006],[12.184453,50.32232449800006],[12.335119500000076,50.17200749800003],[12.331084500000031,50.24271399700007],[12.398014999000054,50.32115349800006],[12.58377850000005,50.40707899800003],[12.820126500000072,50.45976249800003],[12.948144500000069,50.40431149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ042","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Ústecký kraj","NUTS_NAME":"Ústecký kraj"},"geometry":{"type":"Polygon","coordinates":[[[14.491221,51.04353049800005],[14.599177500000053,50.98711099700006],[14.564038,50.91849749700003],[14.650508500000058,50.929418499000064],[14.6188,50.85780449600003],[14.442563,50.81133999900004],[14.343548,50.66792399900004],[14.486485499000025,50.505026996000026],[14.359461,50.42840049700004],[14.367341500000066,50.35235399800007],[14.007528,50.346400997000046],[13.535772500000064,50.18898249800003],[13.407053500000075,50.089616998000054],[13.301367,50.099643496000056],[13.222257500000069,50.24194149600004],[13.221889500000032,50.35225599900008],[12.948144500000069,50.40431149800003],[13.059457500000065,50.50104749800005],[13.193359500000042,50.503172498000026],[13.255104,50.59530099800003],[13.501846,50.63364299800003],[13.525244500000042,50.70441099900006],[13.652174,50.73035999700005],[13.855824,50.72732599700004],[13.897461500000077,50.79214599800008],[14.388335,50.90029899700005],[14.258529500000066,50.98798949800005],[14.317873500000076,51.05469899800005],[14.491221,51.04353049800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ05","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Severovýchod","NUTS_NAME":"Severovýchod"},"geometry":{"type":"Polygon","coordinates":[[[15.277071,50.891020999000034],[15.370452,50.780894999000054],[15.535267500000032,50.779375998000035],[15.816207,50.755322],[15.860921,50.67491299800008],[15.990918,50.683419],[15.986371,50.61474699900003],[16.024848,50.598625],[16.107318,50.66207299800004],[16.322517,50.66645299700008],[16.443536,50.586257499000055],[16.195729,50.432135],[16.360988500000076,50.37923649600003],[16.548217,50.22978399800007],[16.58029,50.14278799700003],[16.70553,50.09651149800004],[16.86327,50.19812299800003],[16.81855850000005,50.04447449700007],[16.711997500000052,49.999649496000075],[16.766050500000063,49.87180499800007],[16.73617849900006,49.832898500000056],[16.847163,49.71613449900008],[16.8042,49.598813],[16.393628,49.580610500000034],[16.077788500000054,49.710807498000065],[15.915789,49.68623549700004],[15.496953,49.86118449800006],[15.532581999000058,49.96002149800006],[15.362708500000053,50.041348998000046],[15.437586500000066,50.10985749800005],[15.35796350000004,50.14709699800005],[15.398379,50.205730998000035],[15.371035,50.28761999900007],[15.235668500000031,50.28663149600004],[15.104144500000075,50.376095],[15.162239,50.42722249800005],[15.14677050000006,50.52294249700003],[14.909923,50.615496998000026],[14.676815,50.486662498000044],[14.486485499000025,50.505026996000026],[14.343548,50.66792399900004],[14.442563,50.81133999900004],[14.6188,50.85780449600003],[14.766383,50.81933250000003],[14.823362,50.87055049600008],[15.00204,50.869024],[15.019613,50.96659899900004],[14.968252,50.990008999000054],[15.032691,51.021315998000034],[15.274085,50.97954299700007],[15.277071,50.891020999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ051","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Liberecký kraj","NUTS_NAME":"Liberecký kraj"},"geometry":{"type":"Polygon","coordinates":[[[15.535267500000032,50.779375998000035],[15.57135,50.58116649800007],[15.632218500000022,50.52280449600005],[15.476920500000062,50.546200498000076],[15.401636,50.47991249800003],[15.14677050000006,50.52294249700003],[14.909923,50.615496998000026],[14.676815,50.486662498000044],[14.486485499000025,50.505026996000026],[14.343548,50.66792399900004],[14.442563,50.81133999900004],[14.6188,50.85780449600003],[14.766383,50.81933250000003],[14.823362,50.87055049600008],[15.00204,50.869024],[15.019613,50.96659899900004],[14.968252,50.990008999000054],[15.032691,51.021315998000034],[15.274085,50.97954299700007],[15.277071,50.891020999000034],[15.370452,50.780894999000054],[15.535267500000032,50.779375998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ052","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Královéhradecký kraj","NUTS_NAME":"Královéhradecký kraj"},"geometry":{"type":"Polygon","coordinates":[[[16.107318,50.66207299800004],[16.322517,50.66645299700008],[16.443536,50.586257499000055],[16.195729,50.432135],[16.360988500000076,50.37923649600003],[16.548217,50.22978399800007],[16.58029,50.14278799700003],[16.193794500000024,50.04143449800006],[15.917132999000046,50.170697999000026],[15.437586500000066,50.10985749800005],[15.35796350000004,50.14709699800005],[15.398379,50.205730998000035],[15.371035,50.28761999900007],[15.235668500000031,50.28663149600004],[15.104144500000075,50.376095],[15.162239,50.42722249800005],[15.14677050000006,50.52294249700003],[15.401636,50.47991249800003],[15.476920500000062,50.546200498000076],[15.632218500000022,50.52280449600005],[15.57135,50.58116649800007],[15.535267500000032,50.779375998000035],[15.816207,50.755322],[15.860921,50.67491299800008],[15.990918,50.683419],[15.986371,50.61474699900003],[16.024848,50.598625],[16.107318,50.66207299800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ053","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Pardubický kraj","NUTS_NAME":"Pardubický kraj"},"geometry":{"type":"Polygon","coordinates":[[[16.8042,49.598813],[16.393628,49.580610500000034],[16.077788500000054,49.710807498000065],[15.915789,49.68623549700004],[15.496953,49.86118449800006],[15.532581999000058,49.96002149800006],[15.362708500000053,50.041348998000046],[15.437586500000066,50.10985749800005],[15.917132999000046,50.170697999000026],[16.193794500000024,50.04143449800006],[16.58029,50.14278799700003],[16.70553,50.09651149800004],[16.86327,50.19812299800003],[16.81855850000005,50.04447449700007],[16.711997500000052,49.999649496000075],[16.766050500000063,49.87180499800007],[16.73617849900006,49.832898500000056],[16.847163,49.71613449900008],[16.8042,49.598813]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ06","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Jihovýchod","NUTS_NAME":"Jihovýchod"},"geometry":{"type":"Polygon","coordinates":[[[15.496953,49.86118449800006],[15.915789,49.68623549700004],[16.077788500000054,49.710807498000065],[16.393628,49.580610500000034],[16.8042,49.598813],[16.73839250000003,49.54277299700004],[16.811954,49.531334997000044],[16.805296500000054,49.40573149700003],[16.859725500000025,49.38139399800008],[16.91570949900006,49.428972497000075],[16.875278,49.48803399900004],[16.943,49.493225],[17.159901500000046,49.27505750000006],[17.11208,49.07727099700003],[17.213569500000062,49.09298599600004],[17.64693,48.85426599700003],[17.39672550000006,48.813349997000046],[17.201662500000054,48.87802899600007],[16.940278,48.617245498000045],[16.90202350000004,48.71797799800004],[16.648492,48.78520849800003],[16.102884500000073,48.74539599900004],[15.84331,48.873388498000054],[15.753633500000035,48.852178500000036],[15.542449500000032,48.90769699600003],[15.586756500000035,48.94715949700003],[15.487282,48.99691499900007],[15.597954,49.07659149700004],[15.567509,49.12886399900003],[15.42224,49.12483449800004],[15.350366500000064,49.155695998000056],[15.345593500000064,49.21642249800004],[15.139395500000035,49.221159998000076],[14.903884,49.33314849900006],[14.932018500000027,49.54910099800003],[14.981185,49.60631650000005],[15.229882,49.61490199800005],[15.249481,49.64360449800006],[15.177543,49.69644049800007],[15.22654,49.75578949700008],[15.496953,49.86118449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ063","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Kraj Vysocina","NUTS_NAME":"Kraj Vysocina"},"geometry":{"type":"Polygon","coordinates":[[[16.393628,49.580610500000034],[16.41613250000006,49.530278499000076],[16.378622500000063,49.45187149800006],[16.228185500000052,49.39418199800008],[16.269517500000063,49.31816349800005],[16.246532,49.09964499900008],[15.586756500000035,48.94715949700003],[15.487282,48.99691499900007],[15.597954,49.07659149700004],[15.567509,49.12886399900003],[15.42224,49.12483449800004],[15.350366500000064,49.155695998000056],[15.345593500000064,49.21642249800004],[15.139395500000035,49.221159998000076],[14.903884,49.33314849900006],[14.932018500000027,49.54910099800003],[14.981185,49.60631650000005],[15.229882,49.61490199800005],[15.249481,49.64360449800006],[15.177543,49.69644049800007],[15.22654,49.75578949700008],[15.496953,49.86118449800006],[15.915789,49.68623549700004],[16.077788500000054,49.710807498000065],[16.393628,49.580610500000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ064","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Jihomoravský kraj","NUTS_NAME":"Jihomoravský kraj"},"geometry":{"type":"Polygon","coordinates":[[[16.8042,49.598813],[16.73839250000003,49.54277299700004],[16.811954,49.531334997000044],[16.805296500000054,49.40573149700003],[16.859725500000025,49.38139399800008],[16.91570949900006,49.428972497000075],[16.875278,49.48803399900004],[16.943,49.493225],[17.159901500000046,49.27505750000006],[17.11208,49.07727099700003],[17.213569500000062,49.09298599600004],[17.64693,48.85426599700003],[17.39672550000006,48.813349997000046],[17.201662500000054,48.87802899600007],[16.940278,48.617245498000045],[16.90202350000004,48.71797799800004],[16.648492,48.78520849800003],[16.102884500000073,48.74539599900004],[15.84331,48.873388498000054],[15.753633500000035,48.852178500000036],[15.542449500000032,48.90769699600003],[15.586756500000035,48.94715949700003],[16.246532,49.09964499900008],[16.269517500000063,49.31816349800005],[16.228185500000052,49.39418199800008],[16.378622500000063,49.45187149800006],[16.41613250000006,49.530278499000076],[16.393628,49.580610500000034],[16.8042,49.598813]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ07","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Strední Morava","NUTS_NAME":"Strední Morava"},"geometry":{"type":"Polygon","coordinates":[[[17.147394500000075,49.87799649800007],[17.215568,49.81822649800006],[17.35802250000006,49.862421],[17.387476500000048,49.790929997000035],[17.466901500000063,49.870716498000036],[17.527989,49.77063449700006],[17.663084500000025,49.75743249800007],[17.691041,49.67642999600008],[17.91685650000005,49.53780899800006],[18.163535,49.517101497000056],[18.40359550000005,49.39674549800003],[18.322436,49.31505899900003],[18.14669250000003,49.247281],[18.069837500000062,49.037547999000026],[17.924501,49.02002299700007],[17.884914500000036,48.92775350000005],[17.64693,48.85426599700003],[17.213569500000062,49.09298599600004],[17.11208,49.07727099700003],[17.159901500000046,49.27505750000006],[16.943,49.493225],[16.875278,49.48803399900004],[16.91570949900006,49.428972497000075],[16.859725500000025,49.38139399800008],[16.805296500000054,49.40573149700003],[16.811954,49.531334997000044],[16.73839250000003,49.54277299700004],[16.8042,49.598813],[16.847163,49.71613449900008],[16.73617849900006,49.832898500000056],[16.766050500000063,49.87180499800007],[16.711997500000052,49.999649496000075],[16.81855850000005,50.04447449700007],[16.86327,50.19812299800003],[17.028323,50.22999699800005],[16.860915,50.40808099700007],[16.907924500000036,50.449454],[17.429605,50.254513],[17.258766500000036,50.147419496000055],[17.155207500000074,49.98302849800007],[17.147394500000075,49.87799649800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ071","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Olomoucký kraj","NUTS_NAME":"Olomoucký kraj"},"geometry":{"type":"Polygon","coordinates":[[[17.429605,50.254513],[17.258766500000036,50.147419496000055],[17.155207500000074,49.98302849800007],[17.147394500000075,49.87799649800007],[17.215568,49.81822649800006],[17.35802250000006,49.862421],[17.387476500000048,49.790929997000035],[17.466901500000063,49.870716498000036],[17.527989,49.77063449700006],[17.663084500000025,49.75743249800007],[17.691041,49.67642999600008],[17.91685650000005,49.53780899800006],[17.473811500000068,49.354715500000054],[17.37019950000007,49.412823496000044],[17.333129,49.38292649600004],[17.356750500000032,49.34274849800005],[17.159901500000046,49.27505750000006],[16.943,49.493225],[16.875278,49.48803399900004],[16.91570949900006,49.428972497000075],[16.859725500000025,49.38139399800008],[16.805296500000054,49.40573149700003],[16.811954,49.531334997000044],[16.73839250000003,49.54277299700004],[16.8042,49.598813],[16.847163,49.71613449900008],[16.73617849900006,49.832898500000056],[16.766050500000063,49.87180499800007],[16.711997500000052,49.999649496000075],[16.81855850000005,50.04447449700007],[16.86327,50.19812299800003],[17.028323,50.22999699800005],[16.860915,50.40808099700007],[16.907924500000036,50.449454],[17.429605,50.254513]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ072","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Zlínský kraj","NUTS_NAME":"Zlínský kraj"},"geometry":{"type":"Polygon","coordinates":[[[18.40359550000005,49.39674549800003],[18.322436,49.31505899900003],[18.14669250000003,49.247281],[18.069837500000062,49.037547999000026],[17.924501,49.02002299700007],[17.884914500000036,48.92775350000005],[17.64693,48.85426599700003],[17.213569500000062,49.09298599600004],[17.11208,49.07727099700003],[17.159901500000046,49.27505750000006],[17.356750500000032,49.34274849800005],[17.333129,49.38292649600004],[17.37019950000007,49.412823496000044],[17.473811500000068,49.354715500000054],[17.91685650000005,49.53780899800006],[18.163535,49.517101497000056],[18.40359550000005,49.39674549800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ08","LEVL_CODE":2,"CNTR_CODE":"CZ","NAME_LATN":"Moravskoslezsko","NUTS_NAME":"Moravskoslezsko"},"geometry":{"type":"Polygon","coordinates":[[[18.035060999000052,50.06577199800006],[18.321821,49.915819999000064],[18.575724,49.91042299900005],[18.627212,49.72232999600004],[18.804583,49.67888899900004],[18.851551,49.517189],[18.564790500000072,49.50693449700003],[18.40359550000005,49.39674549800003],[18.163535,49.517101497000056],[17.91685650000005,49.53780899800006],[17.691041,49.67642999600008],[17.663084500000025,49.75743249800007],[17.527989,49.77063449700006],[17.466901500000063,49.870716498000036],[17.387476500000048,49.790929997000035],[17.35802250000006,49.862421],[17.215568,49.81822649800006],[17.147394500000075,49.87799649800007],[17.155207500000074,49.98302849800007],[17.258766500000036,50.147419496000055],[17.429605,50.254513],[17.71840399900003,50.32094999900005],[17.758479,50.20656799900007],[17.592736,50.16001399900006],[17.868675,49.972545996000065],[18.035060999000052,50.06577199800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"CZ080","LEVL_CODE":3,"CNTR_CODE":"CZ","NAME_LATN":"Moravskoslezský kraj","NUTS_NAME":"Moravskoslezský kraj"},"geometry":{"type":"Polygon","coordinates":[[[18.035060999000052,50.06577199800006],[18.321821,49.915819999000064],[18.575724,49.91042299900005],[18.627212,49.72232999600004],[18.804583,49.67888899900004],[18.851551,49.517189],[18.564790500000072,49.50693449700003],[18.40359550000005,49.39674549800003],[18.163535,49.517101497000056],[17.91685650000005,49.53780899800006],[17.691041,49.67642999600008],[17.663084500000025,49.75743249800007],[17.527989,49.77063449700006],[17.466901500000063,49.870716498000036],[17.387476500000048,49.790929997000035],[17.35802250000006,49.862421],[17.215568,49.81822649800006],[17.147394500000075,49.87799649800007],[17.155207500000074,49.98302849800007],[17.258766500000036,50.147419496000055],[17.429605,50.254513],[17.71840399900003,50.32094999900005],[17.758479,50.20656799900007],[17.592736,50.16001399900006],[17.868675,49.972545996000065],[18.035060999000052,50.06577199800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE","LEVL_CODE":0,"CNTR_CODE":"DE","NAME_LATN":"DEUTSCHLAND","NUTS_NAME":"DEUTSCHLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[8.352822,54.96785699600008],[8.372664500000042,54.896856497000044],[8.635925999000051,54.91168299700007],[9.113097,54.873601499000074],[9.420151500000031,54.83195650000005],[9.422934,54.82322299700007],[9.491949500000032,54.82263549800007],[9.584703,54.865198500000076],[9.873658500000033,54.75218399900007],[9.909055,54.79868599800005],[10.03117,54.636575],[9.995853,54.519735499000035],[9.832854,54.47355900100007],[10.198897501000033,54.45499049600005],[10.168517,54.432844997000075],[10.136784,54.315980498000044],[10.174162,54.34574549800004],[10.358537,54.43131899800005],[10.71375049900007,54.30507250000005],[11.125609500000053,54.373613499000044],[11.071833,54.339934999000036],[11.093783,54.199050500000055],[10.755363500000044,54.05405150000007],[10.840886001000058,53.991893997000034],[10.903661501000045,53.956822],[11.181602500000054,54.01479199900007],[11.26820250000003,53.93225999600003],[11.444478,53.91073699600008],[11.56177850000006,54.028088497000056],[11.626486,54.08964549800004],[11.524761500000068,54.069671],[11.68004650000006,54.152386],[11.998484500000075,54.17496999800005],[12.086768500000062,54.18508049600007],[12.088310500000034,54.113248498000075],[12.151354999000034,54.09425949900003],[12.097017,54.15189799900003],[12.201068,54.244695498000056],[12.28553050000005,54.27495199800006],[12.520881067000062,54.474234541000044],[12.92695150000003,54.42796399800005],[12.436138500000027,54.379089],[12.36317450000007,54.267097998000054],[13.055968500000063,54.44981949800007],[13.017391719000045,54.43872077300006],[13.109225,54.27934049700008],[13.319396,54.193505997000045],[13.344701500000042,54.18111149600003],[13.316288500000041,54.16033349700007],[13.489404499000045,54.083846],[13.706645500000036,54.17161499800005],[13.803770500000041,54.09803149900006],[13.742715500000031,54.031386498000074],[13.914190500000075,53.920434999000065],[13.826154,53.87762799800004],[13.80763550000006,53.85568049900007],[13.81915250000003,53.84194499600005],[14.102733,53.73978199800007],[14.257057,53.75519249800004],[14.214806500000066,53.70314299900008],[14.267542,53.697806496000055],[14.412157,53.32963599700008],[14.441223,53.25435999900003],[14.377268,53.20175199800008],[14.357842,53.06766699800005],[14.143658,52.96136849900006],[14.156692,52.895590998000046],[14.127624,52.83166249900006],[14.436438,52.67990049800005],[14.565063,52.62449699900003],[14.639691,52.57157449700003],[14.60426,52.53348949800005],[14.635373,52.49636849900003],[14.534361999000055,52.39500799900003],[14.600891500000046,52.27205199800005],[14.716044,52.235458],[14.682174,52.11468899800008],[14.755227,52.070024997000075],[14.716716,52.001188],[14.693557,51.901287996000065],[14.590146,51.82102599800004],[14.752694,51.66709599700005],[14.729862,51.581776998000066],[14.735769,51.52595199900003],[14.948293,51.47176899800007],[14.974183,51.36394999600003],[15.037271,51.243749998000055],[14.823362,50.87055049600008],[14.766383,50.81933250000003],[14.6188,50.85780449600003],[14.650508500000058,50.929418499000064],[14.564038,50.91849749700003],[14.599177500000053,50.98711099700006],[14.491221,51.04353049800005],[14.317873500000076,51.05469899800005],[14.258529500000066,50.98798949800005],[14.388335,50.90029899700005],[13.897461500000077,50.79214599800008],[13.855824,50.72732599700004],[13.652174,50.73035999700005],[13.525244500000042,50.70441099900006],[13.501846,50.63364299800003],[13.255104,50.59530099800003],[13.193359500000042,50.503172498000026],[13.059457500000065,50.50104749800005],[12.948144500000069,50.40431149800003],[12.820126500000072,50.45976249800003],[12.58377850000005,50.40707899800003],[12.398014999000054,50.32115349800006],[12.331084500000031,50.24271399700007],[12.335119500000076,50.17200749800003],[12.184453,50.32232449800006],[12.100900500000023,50.31802799700006],[12.13994,50.27771849800007],[12.09076,50.25238450000006],[12.160679,50.21984899700004],[12.260799,50.05815649700003],[12.48988950000006,49.98176099900007],[12.470510500000046,49.94502550000004],[12.550988500000074,49.905088],[12.401524,49.75837299700004],[12.521638,49.686761999000055],[12.593778500000042,49.54219149800008],[12.644176,49.52305749900006],[12.633763500000043,49.47619499700005],[12.77881,49.34807849600003],[12.972595500000068,49.33388799800008],[13.170907999000065,49.17357950000007],[13.397147,49.05100999800004],[13.416651500000057,48.98003549800006],[13.551578500000062,48.96778749600003],[13.839507,48.771605],[13.796107,48.71360049800006],[13.825764,48.61861649800005],[13.727090500000031,48.51301950000004],[13.513369500000067,48.59097799700004],[13.438430500000038,48.548895],[13.407838,48.372160500000064],[13.177043,48.29438999600006],[12.94468450000005,48.20669249800005],[12.751555,48.112810497000055],[12.860181500000067,47.996639998000035],[12.875785500000063,47.962608998000064],[13.00389750000005,47.84796899600008],[12.905047,47.72364349900005],[13.047965,47.709118998000065],[13.097948,47.63371699800007],[13.046055500000023,47.52050249800004],[13.003285500000061,47.463374498000064],[12.81025,47.54706299800006],[12.77925650000003,47.579285500000026],[12.807303,47.61022749800003],[12.695795500000031,47.68222299800004],[12.575026500000035,47.63231599900007],[12.338045499000032,47.697087500000066],[12.257177500000068,47.6791],[12.264886500000046,47.73248299800008],[12.183931,47.70037049800004],[12.20731950000004,47.609837998000046],[12.060662,47.61874349800007],[11.632883,47.59244599600004],[11.572426,47.514643998000054],[11.410219,47.495324],[11.38380950000004,47.47220749600007],[11.42142750000005,47.44485199800005],[11.016500500000063,47.396368],[10.991202500000043,47.396130999000036],[10.871043,47.48188199800006],[10.886199,47.53684799800004],[10.454439,47.55579699800006],[10.431323500000076,47.50367749600008],[10.475413,47.431702],[10.178353,47.270113998000056],[10.23393,47.38328599700003],[10.09985049900007,47.35476149700003],[10.09147050000007,47.45907599700007],[9.999526,47.48301699600006],[9.967813500000034,47.54624049600005],[9.765470500000049,47.588856],[9.736566500000038,47.534906498000055],[9.55872050000005,47.54189299800004],[9.495604,47.551454998000054],[9.182192,47.655890498000076],[8.87499850000006,47.65597349700005],[8.843022,47.71226249800003],[8.77209750000003,47.718459497000026],[8.795708500000046,47.67559549900005],[8.727981500000055,47.69268049800007],[8.74010450000003,47.752789],[8.61233,47.801461999000026],[8.510115,47.77619049900005],[8.405565999000032,47.67451349800007],[8.606369500000028,47.66900549800005],[8.595602,47.605544499000075],[8.562841,47.599432497000066],[8.52144,47.633524],[8.426434500000028,47.567548997000074],[8.221265500000072,47.61551499800004],[8.01872,47.550453996000044],[7.894107500000075,47.58637499800005],[7.713784500000031,47.539404999000055],[7.634097,47.56111350000003],[7.69337,47.600644998000064],[7.589039,47.58987799800008],[7.519558,47.66812299900005],[7.545990500000073,47.743573998000045],[7.558898500000055,47.881823498000074],[7.621966500000042,47.97474899800005],[7.568529,48.035183998000036],[7.577291,48.11565499800008],[7.577919500000064,48.121391998000036],[7.680713,48.257266996000055],[7.739841500000068,48.321731500000055],[7.804824,48.59257949800008],[7.95963050000006,48.71858099800005],[8.232632999000032,48.96657150000004],[8.068407500000035,48.99931649700005],[7.910654500000021,49.04516349800008],[7.635651,49.05395049900005],[7.493791,49.16773550000005],[7.36875550000002,49.16145799800006],[7.293177,49.11490749700005],[7.101069,49.15599799900008],[7.057195999000044,49.11193149800005],[7.03497150000004,49.19141499700004],[6.925060500000029,49.22334649800007],[6.837607499000057,49.21293299700005],[6.834567,49.15131099800004],[6.738397500000076,49.16359799900005],[6.723465500000032,49.21882899800005],[6.565664,49.34765149700007],[6.556986,49.419208498000046],[6.367107500000031,49.469507],[6.380052500000033,49.55110499800003],[6.426077500000076,49.663689],[6.507018500000072,49.71264050000008],[6.520996,49.79698549700004],[6.474962500000061,49.821274998000035],[6.235510500000032,49.89918849900005],[6.116835500000036,50.05671099800003],[6.137662500000033,50.129951498000025],[6.176784500000053,50.23586299800007],[6.405028500000071,50.323308498000074],[6.342218500000058,50.38000550000004],[6.374430500000074,50.45226950000006],[6.315556,50.49704249800004],[6.20628750000003,50.521302],[6.192200499000023,50.52105549800007],[6.189312500000028,50.566094],[6.268615500000067,50.625980999000035],[6.020998999000028,50.75429549900008],[6.017985,50.84379649700003],[6.086947500000065,50.91313499800003],[6.018124,50.93459299800003],[6.026435,50.983273997000026],[5.897198,50.974845498000036],[5.877084999000033,51.03210099900008],[6.174812,51.18451349900005],[6.082150999000021,51.17179899900003],[6.072657,51.242587497000045],[6.224405,51.364978998000026],[6.213423,51.50784149800006],[6.091235500000039,51.60590949600004],[6.118209500000034,51.65610299800005],[5.953192,51.74784599700007],[6.063382500000046,51.865443998000046],[6.166407500000048,51.84083449700006],[6.103341500000056,51.89261699800005],[6.167766,51.90080449800007],[6.407779500000061,51.828092],[6.390816,51.873934],[6.72106,51.89648899800005],[6.828513,51.96406649900007],[6.714659,52.03989349600005],[6.760465,52.118569499000046],[7.065685,52.24137299800003],[7.026305,52.28744399700008],[7.072438,52.37311650000004],[6.987941500000034,52.46954099800007],[6.697865500000034,52.48628599700004],[6.70973250000003,52.627823498000055],[7.006229500000074,52.63876299800006],[7.055577,52.643369998000026],[7.092692,52.83820099800005],[7.209295,53.00024449700004],[7.202794500000039,53.11328149800005],[7.208935,53.243064497000034],[7.192496,53.31762750000007],[7.2643,53.325526497000055],[6.999446500000033,53.35988750000007],[7.024107500000071,53.48237149600004],[7.138518,53.53548099900007],[7.090335500000037,53.57965199900008],[7.243681499000047,53.66778199600003],[7.550173,53.67503749900004],[7.809895500000039,53.70766199800005],[8.017161500000043,53.710496498000055],[8.091291,53.638109],[8.164292999000054,53.52261449800005],[8.061327,53.505957],[8.195555,53.409171498000035],[8.315217500000074,53.46179599800007],[8.303364,53.525487],[8.22693,53.52065249800006],[8.329544,53.61402699900003],[8.554898500000036,53.525131],[8.483202500000061,53.600498997000045],[8.520410500000025,53.606205],[8.483350501000075,53.69478399800005],[8.614913999000066,53.88211799900006],[9.029917500000067,53.82556499600008],[8.994673500000033,53.87260649600006],[9.02242,53.87951749900003],[8.832592,54.02392949800003],[8.985603500000025,54.05827499600008],[8.808867500000076,54.17103749900008],[8.844804500000066,54.26632849900005],[8.602134,54.34050799600004],[9.010464,54.49682299800003],[8.806592500000022,54.470951496000055],[8.892347,54.597899],[8.594786,54.881965998000055],[8.340561500000035,54.88057449900003],[8.278555500000039,54.752207998000074],[8.297903,54.91065650000007],[8.416845499000033,55.05838349600003],[8.464195500000073,55.04571499700006],[8.352822,54.96785699600008]]],[[[13.97208450000005,54.06874749700006],[14.226302,53.92865299700003],[14.213077500000054,53.86647949800005],[13.82615050000004,53.849684997000054],[13.938169500000072,53.90960399700003],[13.911077500000033,53.99123049700006],[14.040180500000076,53.94072149900006],[14.047417500000051,54.00135849700007],[13.910431500000072,54.06298749800004],[13.861864500000024,53.99975499900006],[13.879676500000073,54.04025999800007],[13.810579,54.031206498000074],[13.813017,54.099393500000076],[13.75074599900006,54.14996050000008],[13.97208450000005,54.06874749700006]]],[[[13.391083500000036,54.65131699900007],[13.423516,54.578565998000045],[13.67934,54.56278249700006],[13.569855500000074,54.46715049900007],[13.72786,54.35514799900005],[13.72390450000006,54.27340749800004],[13.645403999000052,54.29756049700006],[13.682453500000065,54.34958099900007],[13.507070500000054,54.34420649800006],[13.352598500000056,54.26796899800007],[13.425133,54.23918499800004],[13.394217500000025,54.22250249900003],[13.114586,54.33190799600004],[13.261483500000054,54.38291649900003],[13.149309500000072,54.42391550000008],[13.270131,54.480173498000056],[13.144181,54.54696549800008],[13.243102746000034,54.560483161000036],[13.282402500000046,54.64570449900003],[13.16017050000005,54.56385250000005],[13.26698250000004,54.66663349800007],[13.438944,54.67753299900005],[13.391083500000036,54.65131699900007]],[[13.368071500000042,54.614697498000055],[13.243615336000062,54.56055321700006],[13.370426500000065,54.57788849900004],[13.475655500000073,54.42683399900005],[13.557416500000045,54.440044999000065],[13.368071500000042,54.614697498000055]]],[[[11.22579999900006,54.50503149700006],[11.312892500000032,54.40277],[11.004257,54.44900249900007],[11.066274500000077,54.53572399800004],[11.22579999900006,54.50503149700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE1","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"BADEN-WÜRTTEMBERG","NUTS_NAME":"BADEN-WÜRTTEMBERG"},"geometry":{"type":"Polygon","coordinates":[[[9.648736,49.79147749900005],[9.632460499000047,49.69318599800005],[9.831804500000032,49.70380349800007],[9.92656099900006,49.48483550000003],[10.083623,49.542955498000026],[10.11832750000002,49.47316949800006],[10.094741500000055,49.447898500000065],[10.161698,49.39549399800006],[10.111966500000051,49.38491049800007],[10.124643500000047,49.20082450000007],[10.248023,49.14928099900004],[10.204997499000058,49.098118998000075],[10.256763500000034,49.05949499900004],[10.409849500000064,48.977435498000034],[10.45700050000005,48.917141498000035],[10.423689,48.74449599600007],[10.487258,48.696662],[10.361222500000054,48.65318649900007],[10.268706999000074,48.703546998000036],[10.314184,48.55628950000005],[10.278268,48.51607449800008],[10.23077950000004,48.510510999000076],[10.133895500000051,48.454871498000045],[10.032694500000048,48.45719649800003],[9.997606,48.35011799800003],[10.095357500000034,48.16401499800003],[10.136457,48.10845949700007],[10.135728,48.02376149600008],[10.132387,48.01539999800008],[10.114954500000067,47.97625750000003],[10.104207,47.974358496000036],[10.110136500000067,47.93714999800005],[10.131928500000072,47.82008749600004],[10.064681,47.78588749900007],[10.139437500000042,47.703273496000065],[10.077292500000055,47.63927449700003],[9.840336,47.677096498000026],[9.692543,47.610768998000026],[9.55872050000005,47.54189299800004],[9.495604,47.551454998000054],[9.182192,47.655890498000076],[8.87499850000006,47.65597349700005],[8.843022,47.71226249800003],[8.77209750000003,47.718459497000026],[8.795708500000046,47.67559549900005],[8.727981500000055,47.69268049800007],[8.74010450000003,47.752789],[8.61233,47.801461999000026],[8.510115,47.77619049900005],[8.405565999000032,47.67451349800007],[8.606369500000028,47.66900549800005],[8.595602,47.605544499000075],[8.562841,47.599432497000066],[8.52144,47.633524],[8.426434500000028,47.567548997000074],[8.221265500000072,47.61551499800004],[8.01872,47.550453996000044],[7.894107500000075,47.58637499800005],[7.713784500000031,47.539404999000055],[7.634097,47.56111350000003],[7.69337,47.600644998000064],[7.589039,47.58987799800008],[7.519558,47.66812299900005],[7.545990500000073,47.743573998000045],[7.558898500000055,47.881823498000074],[7.621966500000042,47.97474899800005],[7.568529,48.035183998000036],[7.577291,48.11565499800008],[7.577919500000064,48.121391998000036],[7.680713,48.257266996000055],[7.739841500000068,48.321731500000055],[7.804824,48.59257949800008],[7.95963050000006,48.71858099800005],[8.232632999000032,48.96657150000004],[8.261284,48.980916998000055],[8.277349,48.989939],[8.33997,49.08014999800008],[8.413072500000055,49.24981649800003],[8.466985500000021,49.28297550000008],[8.487268,49.29002649800003],[8.470915500000046,49.34071299800007],[8.497316,49.411346999000045],[8.474739704000058,49.44061602800008],[8.423068,49.54182099900004],[8.422700500000076,49.57419249800006],[8.422439500000053,49.58338549800004],[8.581375,49.519780498000046],[8.62129950000002,49.54500999900006],[8.589315500000055,49.604514996000034],[8.693837,49.619353498000066],[8.70265,49.53532799900006],[8.899572500000033,49.50365549800006],[8.899355,49.48454999900008],[8.801542,49.410945498000046],[8.950348500000075,49.45499299800008],[8.93188,49.47063699600005],[9.08342650000003,49.52608699600006],[9.126459500000067,49.52160699900003],[9.103006,49.57745599900005],[9.41092,49.66350799800006],[9.399960500000077,49.72884049700008],[9.295673,49.740530998000054],[9.471497499000066,49.77972650000004],[9.648736,49.79147749900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE11","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Stuttgart","NUTS_NAME":"Stuttgart"},"geometry":{"type":"Polygon","coordinates":[[[9.648736,49.79147749900005],[9.632460499000047,49.69318599800005],[9.831804500000032,49.70380349800007],[9.92656099900006,49.48483550000003],[10.083623,49.542955498000026],[10.11832750000002,49.47316949800006],[10.094741500000055,49.447898500000065],[10.161698,49.39549399800006],[10.111966500000051,49.38491049800007],[10.124643500000047,49.20082450000007],[10.248023,49.14928099900004],[10.204997499000058,49.098118998000075],[10.256763500000034,49.05949499900004],[10.409849500000064,48.977435498000034],[10.45700050000005,48.917141498000035],[10.423689,48.74449599600007],[10.487258,48.696662],[10.361222500000054,48.65318649900007],[10.268706999000074,48.703546998000036],[10.314184,48.55628950000005],[10.278268,48.51607449800008],[10.23077950000004,48.510510999000076],[9.944071500000064,48.631755498000075],[9.58517450000005,48.536741498000026],[9.582475500000044,48.539146998000035],[9.151229500000056,48.60405549700005],[9.129176500000028,48.60126649800003],[8.768902500000024,48.52184099800007],[8.748469500000056,48.604935],[8.877826500000026,48.723628],[8.804181500000027,48.77777799900008],[8.928800500000023,48.86655699900007],[8.945315,48.96176599900008],[8.876031499000021,49.03516999900006],[8.87779450000005,49.058478500000035],[8.891526500000055,49.111477997000065],[8.826425999000037,49.126592499000026],[8.818233,49.19449699900008],[8.896089,49.18203349800007],[9.049126500000057,49.29286249900008],[9.241855,49.297315498000046],[9.317456,49.384243998000045],[9.44349150000005,49.364336997000066],[9.603823,49.42657799600005],[9.513439500000061,49.44478799800004],[9.49937,49.58738049800007],[9.547451,49.634345997000025],[9.41092,49.66350799800006],[9.399960500000077,49.72884049700008],[9.295673,49.740530998000054],[9.471497499000066,49.77972650000004],[9.648736,49.79147749900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE111","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Stuttgart, Stadtkreis","NUTS_NAME":"Stuttgart, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.24417,48.84700849700005],[9.313649,48.77448349900004],[9.212985,48.72893249800006],[9.245221,48.69462899700005],[9.104720500000042,48.70907949800005],[9.038994,48.74131650000004],[9.064847500000042,48.75557699900003],[9.094126999000025,48.83819099800007],[9.24417,48.84700849700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE112","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Böblingen","NUTS_NAME":"Böblingen"},"geometry":{"type":"Polygon","coordinates":[[[9.064847500000042,48.75557699900003],[9.038994,48.74131650000004],[9.104720500000042,48.70907949800005],[9.174173,48.633544],[9.151229500000056,48.60405549700005],[9.129176500000028,48.60126649800003],[8.768902500000024,48.52184099800007],[8.748469500000056,48.604935],[8.877826500000026,48.723628],[8.804181500000027,48.77777799900008],[8.928800500000023,48.86655699900007],[9.043103,48.821127998000065],[9.064847500000042,48.75557699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE113","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Esslingen","NUTS_NAME":"Esslingen"},"geometry":{"type":"Polygon","coordinates":[[[9.503095500000029,48.753938996000045],[9.491054,48.66892149800003],[9.592995,48.61652649900003],[9.582475500000044,48.539146998000035],[9.151229500000056,48.60405549700005],[9.174173,48.633544],[9.104720500000042,48.70907949800005],[9.245221,48.69462899700005],[9.212985,48.72893249800006],[9.313649,48.77448349900004],[9.503095500000029,48.753938996000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE114","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Göppingen","NUTS_NAME":"Göppingen"},"geometry":{"type":"Polygon","coordinates":[[[9.987770500000067,48.72725349900003],[9.944071500000064,48.631755498000075],[9.58517450000005,48.536741498000026],[9.582475500000044,48.539146998000035],[9.592995,48.61652649900003],[9.491054,48.66892149800003],[9.503095500000029,48.753938996000045],[9.626598,48.774376999000026],[9.987770500000067,48.72725349900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE115","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ludwigsburg","NUTS_NAME":"Ludwigsburg"},"geometry":{"type":"Polygon","coordinates":[[[9.125125500000024,49.063383498000064],[9.399394500000028,49.03817250000003],[9.348001500000066,48.99409049800005],[9.362897499000042,48.91167799900006],[9.24417,48.84700849700005],[9.094126999000025,48.83819099800007],[9.064847500000042,48.75557699900003],[9.043103,48.821127998000065],[8.928800500000023,48.86655699900007],[8.945315,48.96176599900008],[8.876031499000021,49.03516999900006],[9.125125500000024,49.063383498000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE116","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rems-Murr-Kreis","NUTS_NAME":"Rems-Murr-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[9.693309,48.964016996000055],[9.768822,48.833807498000056],[9.626598,48.774376999000026],[9.503095500000029,48.753938996000045],[9.313649,48.77448349900004],[9.24417,48.84700849700005],[9.362897499000042,48.91167799900006],[9.348001500000066,48.99409049800005],[9.399394500000028,49.03817250000003],[9.399536,49.04398449900003],[9.416236,49.04737699800006],[9.500877,49.079000499000074],[9.693309,48.964016996000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE117","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heilbronn, Stadtkreis","NUTS_NAME":"Heilbronn, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.30054,49.11431999800004],[9.161483,49.09313349800004],[9.131608,49.125473],[9.16080450000004,49.14970149800007],[9.044506,49.19737299800005],[9.159719499000062,49.206500498000025],[9.30054,49.11431999800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE118","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heilbronn, Landkreis","NUTS_NAME":"Heilbronn, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.452209,49.22048],[9.343511500000034,49.17050999600008],[9.527490500000056,49.108550996000076],[9.500877,49.079000499000074],[9.416236,49.04737699800006],[9.399536,49.04398449900003],[9.399394500000028,49.03817250000003],[9.125125500000024,49.063383498000064],[8.876031499000021,49.03516999900006],[8.87779450000005,49.058478500000035],[8.891526500000055,49.111477997000065],[8.826425999000037,49.126592499000026],[8.818233,49.19449699900008],[8.896089,49.18203349800007],[9.049126500000057,49.29286249900008],[9.241855,49.297315498000046],[9.317456,49.384243998000045],[9.44349150000005,49.364336997000066],[9.478285,49.318434997000054],[9.422598,49.26166199800008],[9.452209,49.22048]],[[9.044506,49.19737299800005],[9.16080450000004,49.14970149800007],[9.131608,49.125473],[9.161483,49.09313349800004],[9.30054,49.11431999800004],[9.159719499000062,49.206500498000025],[9.044506,49.19737299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE119","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hohenlohekreis","NUTS_NAME":"Hohenlohekreis"},"geometry":{"type":"Polygon","coordinates":[[[9.855564500000071,49.368367498000055],[9.860396500000036,49.29001549600008],[9.652067,49.131007999000076],[9.527490500000056,49.108550996000076],[9.343511500000034,49.17050999600008],[9.452209,49.22048],[9.422598,49.26166199800008],[9.478285,49.318434997000054],[9.44349150000005,49.364336997000066],[9.603823,49.42657799600005],[9.855564500000071,49.368367498000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE11A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwäbisch Hall","NUTS_NAME":"Schwäbisch Hall"},"geometry":{"type":"Polygon","coordinates":[[[10.111966500000051,49.38491049800007],[10.124643500000047,49.20082450000007],[10.248023,49.14928099900004],[10.204997499000058,49.098118998000075],[10.256763500000034,49.05949499900004],[9.975444500000037,49.04087349800005],[10.014082,48.98222499800005],[9.878715500000055,48.91893199800006],[9.693309,48.964016996000055],[9.500877,49.079000499000074],[9.527490500000056,49.108550996000076],[9.652067,49.131007999000076],[9.860396500000036,49.29001549600008],[9.855564500000071,49.368367498000055],[10.111966500000051,49.38491049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE11B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Main-Tauber-Kreis","NUTS_NAME":"Main-Tauber-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[10.083623,49.542955498000026],[10.11832750000002,49.47316949800006],[10.094741500000055,49.447898500000065],[10.161698,49.39549399800006],[10.111966500000051,49.38491049800007],[9.855564500000071,49.368367498000055],[9.603823,49.42657799600005],[9.513439500000061,49.44478799800004],[9.49937,49.58738049800007],[9.547451,49.634345997000025],[9.41092,49.66350799800006],[9.399960500000077,49.72884049700008],[9.295673,49.740530998000054],[9.471497499000066,49.77972650000004],[9.648736,49.79147749900005],[9.632460499000047,49.69318599800005],[9.831804500000032,49.70380349800007],[9.92656099900006,49.48483550000003],[10.083623,49.542955498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE11C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heidenheim","NUTS_NAME":"Heidenheim"},"geometry":{"type":"Polygon","coordinates":[[[10.423689,48.74449599600007],[10.487258,48.696662],[10.361222500000054,48.65318649900007],[10.268706999000074,48.703546998000036],[10.314184,48.55628950000005],[10.278268,48.51607449800008],[10.23077950000004,48.510510999000076],[9.944071500000064,48.631755498000075],[9.987770500000067,48.72725349900003],[10.423689,48.74449599600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE11D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ostalbkreis","NUTS_NAME":"Ostalbkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.409849500000064,48.977435498000034],[10.45700050000005,48.917141498000035],[10.423689,48.74449599600007],[9.987770500000067,48.72725349900003],[9.626598,48.774376999000026],[9.768822,48.833807498000056],[9.693309,48.964016996000055],[9.878715500000055,48.91893199800006],[10.014082,48.98222499800005],[9.975444500000037,49.04087349800005],[10.256763500000034,49.05949499900004],[10.409849500000064,48.977435498000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE12","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Karlsruhe","NUTS_NAME":"Karlsruhe"},"geometry":{"type":"Polygon","coordinates":[[[9.49937,49.58738049800007],[9.513439500000061,49.44478799800004],[9.603823,49.42657799600005],[9.44349150000005,49.364336997000066],[9.317456,49.384243998000045],[9.241855,49.297315498000046],[9.049126500000057,49.29286249900008],[8.896089,49.18203349800007],[8.818233,49.19449699900008],[8.826425999000037,49.126592499000026],[8.891526500000055,49.111477997000065],[8.87779450000005,49.058478500000035],[8.876031499000021,49.03516999900006],[8.945315,48.96176599900008],[8.928800500000023,48.86655699900007],[8.804181500000027,48.77777799900008],[8.877826500000026,48.723628],[8.748469500000056,48.604935],[8.768902500000024,48.52184099800007],[8.75571050000002,48.50414],[8.825225500000045,48.46304199700006],[8.774055,48.416278498000054],[8.737186499000074,48.377976998000065],[8.487894499000049,48.38202349800008],[8.494052,48.314436999000065],[8.345293500000025,48.38446699800005],[8.303992,48.34909449600008],[8.247635500000058,48.395136498000056],[8.275435500000071,48.47785649700006],[8.218828,48.50795449800006],[8.222007,48.60322749800008],[7.95963050000006,48.71858099800005],[8.232632999000032,48.96657150000004],[8.261284,48.980916998000055],[8.277349,48.989939],[8.33997,49.08014999800008],[8.413072500000055,49.24981649800003],[8.466985500000021,49.28297550000008],[8.487268,49.29002649800003],[8.470915500000046,49.34071299800007],[8.497316,49.411346999000045],[8.474739704000058,49.44061602800008],[8.423068,49.54182099900004],[8.422700500000076,49.57419249800006],[8.422439500000053,49.58338549800004],[8.581375,49.519780498000046],[8.62129950000002,49.54500999900006],[8.589315500000055,49.604514996000034],[8.693837,49.619353498000066],[8.70265,49.53532799900006],[8.899572500000033,49.50365549800006],[8.899355,49.48454999900008],[8.801542,49.410945498000046],[8.950348500000075,49.45499299800008],[8.93188,49.47063699600005],[9.08342650000003,49.52608699600006],[9.126459500000067,49.52160699900003],[9.103006,49.57745599900005],[9.41092,49.66350799800006],[9.547451,49.634345997000025],[9.49937,49.58738049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE121","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Baden-Baden, Stadtkreis","NUTS_NAME":"Baden-Baden, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.320420499000022,48.70817099700008],[8.236317502000077,48.66750499800003],[8.108812001000047,48.72753399800007],[8.20220050100005,48.75263649900006],[8.179756,48.843424998000046],[8.287332001000038,48.78949749900005],[8.320420499000022,48.70817099700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE122","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Karlsruhe, Stadtkreis","NUTS_NAME":"Karlsruhe, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.277349,48.989939],[8.33997,49.08014999800008],[8.435602500000073,49.09152899800006],[8.539822,48.953889],[8.277349,48.989939]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE123","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Karlsruhe, Landkreis","NUTS_NAME":"Karlsruhe, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.818233,49.19449699900008],[8.826425999000037,49.126592499000026],[8.891526500000055,49.111477997000065],[8.87779450000005,49.058478500000035],[8.574549500000046,48.988241499000026],[8.467365,48.845113499000036],[8.493572500000028,48.82018199700008],[8.416625,48.83536799700005],[8.305566,48.862692],[8.29534650000005,48.91291349900007],[8.35457450000007,48.934889],[8.261284,48.980916998000055],[8.277349,48.989939],[8.539822,48.953889],[8.435602500000073,49.09152899800006],[8.33997,49.08014999800008],[8.413072500000055,49.24981649800003],[8.466985500000021,49.28297550000008],[8.487268,49.29002649800003],[8.818233,49.19449699900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE124","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rastatt","NUTS_NAME":"Rastatt"},"geometry":{"type":"Polygon","coordinates":[[[8.416625,48.83536799700005],[8.42173450000007,48.743741499000066],[8.48456,48.75670149900003],[8.409324500000025,48.63945749800007],[8.222007,48.60322749800008],[7.95963050000006,48.71858099800005],[8.232632999000032,48.96657150000004],[8.261284,48.980916998000055],[8.35457450000007,48.934889],[8.29534650000005,48.91291349900007],[8.305566,48.862692],[8.416625,48.83536799700005]],[[8.320420499000022,48.70817099700008],[8.287332001000038,48.78949749900005],[8.179756,48.843424998000046],[8.20220050100005,48.75263649900006],[8.108812001000047,48.72753399800007],[8.236317502000077,48.66750499800003],[8.320420499000022,48.70817099700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE125","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heidelberg, Stadtkreis","NUTS_NAME":"Heidelberg, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.573519,49.41584399900006],[8.582887500000027,49.42646949900006],[8.76794250000006,49.45427349600004],[8.790121,49.40251249900007],[8.620378500000072,49.35412299800004],[8.573519,49.41584399900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE126","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mannheim, Stadtkreis","NUTS_NAME":"Mannheim, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.581375,49.519780498000046],[8.546030500000029,49.47743650000007],[8.582887500000027,49.42646949900006],[8.573519,49.41584399900006],[8.497316,49.411346999000045],[8.474739704000058,49.44061602800008],[8.423068,49.54182099900004],[8.422700500000076,49.57419249800006],[8.422439500000053,49.58338549800004],[8.581375,49.519780498000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE127","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neckar-Odenwald-Kreis","NUTS_NAME":"Neckar-Odenwald-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[9.603823,49.42657799600005],[9.44349150000005,49.364336997000066],[9.317456,49.384243998000045],[9.241855,49.297315498000046],[9.049126500000057,49.29286249900008],[8.937966500000073,49.388130998000065],[9.08342650000003,49.52608699600006],[9.126459500000067,49.52160699900003],[9.103006,49.57745599900005],[9.41092,49.66350799800006],[9.547451,49.634345997000025],[9.49937,49.58738049800007],[9.513439500000061,49.44478799800004],[9.603823,49.42657799600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE128","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Neckar-Kreis","NUTS_NAME":"Rhein-Neckar-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.899572500000033,49.50365549800006],[8.899355,49.48454999900008],[8.801542,49.410945498000046],[8.950348500000075,49.45499299800008],[8.93188,49.47063699600005],[9.08342650000003,49.52608699600006],[8.937966500000073,49.388130998000065],[9.049126500000057,49.29286249900008],[8.896089,49.18203349800007],[8.818233,49.19449699900008],[8.487268,49.29002649800003],[8.470915500000046,49.34071299800007],[8.497316,49.411346999000045],[8.573519,49.41584399900006],[8.620378500000072,49.35412299800004],[8.790121,49.40251249900007],[8.76794250000006,49.45427349600004],[8.582887500000027,49.42646949900006],[8.546030500000029,49.47743650000007],[8.581375,49.519780498000046],[8.62129950000002,49.54500999900006],[8.589315500000055,49.604514996000034],[8.693837,49.619353498000066],[8.70265,49.53532799900006],[8.899572500000033,49.50365549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE129","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Pforzheim, Stadtkreis","NUTS_NAME":"Pforzheim, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.71463411700006,48.82434032900005],[8.69102950000007,48.83526949800006],[8.631694500000037,48.91459199900004],[8.755940500000065,48.92464699800007],[8.808609500000046,48.86250349800008],[8.71463411700006,48.82434032900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE12A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Calw","NUTS_NAME":"Calw"},"geometry":{"type":"Polygon","coordinates":[[[8.71463411700006,48.82434032900005],[8.804181500000027,48.77777799900008],[8.877826500000026,48.723628],[8.748469500000056,48.604935],[8.768902500000024,48.52184099800007],[8.75571050000002,48.50414],[8.607631,48.50428749900004],[8.409324500000025,48.63945749800007],[8.48456,48.75670149900003],[8.42173450000007,48.743741499000066],[8.416625,48.83536799700005],[8.493572500000028,48.82018199700008],[8.69102950000007,48.83526949800006],[8.71463411700006,48.82434032900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE12B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Enzkreis","NUTS_NAME":"Enzkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.945315,48.96176599900008],[8.928800500000023,48.86655699900007],[8.804181500000027,48.77777799900008],[8.71463411700006,48.82434032900005],[8.808609500000046,48.86250349800008],[8.755940500000065,48.92464699800007],[8.631694500000037,48.91459199900004],[8.69102950000007,48.83526949800006],[8.493572500000028,48.82018199700008],[8.467365,48.845113499000036],[8.574549500000046,48.988241499000026],[8.87779450000005,49.058478500000035],[8.876031499000021,49.03516999900006],[8.945315,48.96176599900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE12C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Freudenstadt","NUTS_NAME":"Freudenstadt"},"geometry":{"type":"Polygon","coordinates":[[[8.75571050000002,48.50414],[8.825225500000045,48.46304199700006],[8.774055,48.416278498000054],[8.737186499000074,48.377976998000065],[8.487894499000049,48.38202349800008],[8.494052,48.314436999000065],[8.345293500000025,48.38446699800005],[8.303992,48.34909449600008],[8.247635500000058,48.395136498000056],[8.275435500000071,48.47785649700006],[8.218828,48.50795449800006],[8.222007,48.60322749800008],[8.409324500000025,48.63945749800007],[8.607631,48.50428749900004],[8.75571050000002,48.50414]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE13","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Freiburg","NUTS_NAME":"Freiburg"},"geometry":{"type":"Polygon","coordinates":[[[8.247635500000058,48.395136498000056],[8.303992,48.34909449600008],[8.345293500000025,48.38446699800005],[8.494052,48.314436999000065],[8.487894499000049,48.38202349800008],[8.737186499000074,48.377976998000065],[8.683524,48.28078749800005],[8.751237,48.15953999700008],[8.953761,48.10727899800003],[9.012045,47.93692299700007],[9.151105,47.89498299700006],[9.128582,47.846817498000064],[9.030533,47.806520499000044],[9.174700500000029,47.732297998000035],[9.215705,47.66601899600005],[9.182192,47.655890498000076],[8.87499850000006,47.65597349700005],[8.843022,47.71226249800003],[8.77209750000003,47.718459497000026],[8.795708500000046,47.67559549900005],[8.727981500000055,47.69268049800007],[8.74010450000003,47.752789],[8.61233,47.801461999000026],[8.510115,47.77619049900005],[8.405565999000032,47.67451349800007],[8.606369500000028,47.66900549800005],[8.595602,47.605544499000075],[8.562841,47.599432497000066],[8.52144,47.633524],[8.426434500000028,47.567548997000074],[8.221265500000072,47.61551499800004],[8.01872,47.550453996000044],[7.894107500000075,47.58637499800005],[7.713784500000031,47.539404999000055],[7.634097,47.56111350000003],[7.69337,47.600644998000064],[7.589039,47.58987799800008],[7.519558,47.66812299900005],[7.545990500000073,47.743573998000045],[7.558898500000055,47.881823498000074],[7.621966500000042,47.97474899800005],[7.568529,48.035183998000036],[7.577291,48.11565499800008],[7.577919500000064,48.121391998000036],[7.680713,48.257266996000055],[7.739841500000068,48.321731500000055],[7.804824,48.59257949800008],[7.95963050000006,48.71858099800005],[8.222007,48.60322749800008],[8.218828,48.50795449800006],[8.275435500000071,48.47785649700006],[8.247635500000058,48.395136498000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE131","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Freiburg im Breisgau, Stadtkreis","NUTS_NAME":"Freiburg im Breisgau, Stadtkreis"},"geometry":{"type":"Polygon","coordinates":[[[7.850602500000036,48.052035999000054],[7.92648,48.01400799800007],[7.909164,47.91444249800003],[7.836622,47.97263550000008],[7.663138,47.966504],[7.825248500000043,48.069796996000036],[7.850602500000036,48.052035999000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE132","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Breisgau-Hochschwarzwald","NUTS_NAME":"Breisgau-Hochschwarzwald"},"geometry":{"type":"Polygon","coordinates":[[[7.825248500000043,48.069796996000036],[7.663138,47.966504],[7.836622,47.97263550000008],[7.909164,47.91444249800003],[7.92648,48.01400799800007],[7.850602500000036,48.052035999000054],[8.066659500000071,48.062971497000035],[8.136648,48.009147496000026],[8.319668,47.98867249800003],[8.43903350000005,47.853638498000066],[8.267485,47.85203799800007],[8.183383500000048,47.782542500000034],[8.018114500000024,47.84139399900005],[7.889874,47.878971998000054],[7.545990500000073,47.743573998000045],[7.558898500000055,47.881823498000074],[7.621966500000042,47.97474899800005],[7.568529,48.035183998000036],[7.577291,48.11565499800008],[7.825248500000043,48.069796996000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE133","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Emmendingen","NUTS_NAME":"Emmendingen"},"geometry":{"type":"Polygon","coordinates":[[[8.180335,48.18392],[8.125374,48.13169499700007],[8.166075,48.07578999800006],[8.109056500000065,48.063544500000035],[8.136648,48.009147496000026],[8.066659500000071,48.062971497000035],[7.850602500000036,48.052035999000054],[7.825248500000043,48.069796996000036],[7.577291,48.11565499800008],[7.577919500000064,48.121391998000036],[7.680713,48.257266996000055],[8.151861500000052,48.22898949800003],[8.180335,48.18392]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE274","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Memmingen, Kreisfreie Stadt","NUTS_NAME":"Memmingen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.2304,48.04228249700003],[10.180203500000061,47.93029449800008],[10.110136500000067,47.93714999800005],[10.104207,47.974358496000036],[10.114954500000067,47.97625750000003],[10.132387,48.01539999800008],[10.135728,48.02376149600008],[10.2304,48.04228249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE275","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Aichach-Friedberg","NUTS_NAME":"Aichach-Friedberg"},"geometry":{"type":"Polygon","coordinates":[[[11.295411,48.47457399900003],[11.311675,48.45227299800007],[11.17824950000005,48.41746599900006],[11.198315,48.386679998000034],[11.116768500000035,48.26847799800004],[11.061886,48.27419099800005],[11.03451,48.192992],[10.903411,48.236676500000044],[10.910751,48.258306996000044],[10.959205500000053,48.37197449900003],[10.884084,48.458442998000066],[10.871551,48.53121099800006],[10.987044,48.54967349900005],[10.923928500000045,48.586954],[11.022986,48.62017149600007],[11.155116,48.59589999600007],[11.12657450000006,48.55498750000004],[11.295411,48.47457399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE276","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Augsburg, Landkreis","NUTS_NAME":"Augsburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.923928500000045,48.586954],[10.987044,48.54967349900005],[10.871551,48.53121099800006],[10.884084,48.458442998000066],[10.856543,48.34584599800007],[10.76419,48.300984],[10.910751,48.258306996000044],[10.903411,48.236676500000044],[10.848869,48.147790996000026],[10.786499500000048,48.15013799800005],[10.801532,48.10417049900008],[10.701787,48.09787299800007],[10.684728999000072,48.17252449800003],[10.56838699900004,48.16703999600003],[10.573642,48.212120500000026],[10.582048,48.31662599900005],[10.498989,48.330070497000065],[10.548523500000044,48.396637499000065],[10.528355500000032,48.471388998000066],[10.765158500000041,48.497689998000055],[10.779367500000035,48.618930498000054],[10.923928500000045,48.586954]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE277","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dillingen a.d. Donau","NUTS_NAME":"Dillingen a.d. Donau"},"geometry":{"type":"Polygon","coordinates":[[[10.779367500000035,48.618930498000054],[10.765158500000041,48.497689998000055],[10.528355500000032,48.471388998000066],[10.278268,48.51607449800008],[10.314184,48.55628950000005],[10.268706999000074,48.703546998000036],[10.361222500000054,48.65318649900007],[10.487258,48.696662],[10.658442499000046,48.73216999700003],[10.622081,48.68485099800006],[10.779367500000035,48.618930498000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE278","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Günzburg","NUTS_NAME":"Günzburg"},"geometry":{"type":"Polygon","coordinates":[[[10.528355500000032,48.471388998000066],[10.548523500000044,48.396637499000065],[10.498989,48.330070497000065],[10.582048,48.31662599900005],[10.573642,48.212120500000026],[10.295115,48.18451699800005],[10.271433500000057,48.22965399800006],[10.276916500000027,48.35888199900006],[10.184031,48.37338549800006],[10.133895500000051,48.454871498000045],[10.23077950000004,48.510510999000076],[10.278268,48.51607449800008],[10.528355500000032,48.471388998000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE279","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neu-Ulm","NUTS_NAME":"Neu-Ulm"},"geometry":{"type":"Polygon","coordinates":[[[10.133895500000051,48.454871498000045],[10.184031,48.37338549800006],[10.276916500000027,48.35888199900006],[10.271433500000057,48.22965399800006],[10.136457,48.10845949700007],[10.095357500000034,48.16401499800003],[9.997606,48.35011799800003],[10.032694500000048,48.45719649800003],[10.133895500000051,48.454871498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lindau (Bodensee)","NUTS_NAME":"Lindau (Bodensee)"},"geometry":{"type":"Polygon","coordinates":[[[10.077292500000055,47.63927449700003],[9.967813500000034,47.54624049600005],[9.765470500000049,47.588856],[9.736566500000038,47.534906498000055],[9.55872050000005,47.54189299800004],[9.692543,47.610768998000026],[9.840336,47.677096498000026],[10.077292500000055,47.63927449700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ostallgäu","NUTS_NAME":"Ostallgäu"},"geometry":{"type":"Polygon","coordinates":[[[10.767090500000052,47.843793498000025],[10.715888500000062,47.705480496000064],[10.954276500000049,47.61652049800006],[10.872826,47.57245899800006],[10.886199,47.53684799800004],[10.454439,47.55579699800006],[10.511613500000067,47.66640599800007],[10.435189,47.79870599800006],[10.326901,47.85807550000004],[10.528557500000034,47.983941497000046],[10.621906,47.94785249900008],[10.701787,48.09787299800007],[10.801532,48.10417049900008],[10.75635950000003,48.04034599800008],[10.826458,47.99526099800005],[10.767090500000052,47.843793498000025]],[[10.643866,47.91768849700003],[10.551547,47.883113998000056],[10.656953500000043,47.84116699800006],[10.643866,47.91768849700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Unterallgäu","NUTS_NAME":"Unterallgäu"},"geometry":{"type":"Polygon","coordinates":[[[10.295115,48.18451699800005],[10.573642,48.212120500000026],[10.56838699900004,48.16703999600003],[10.684728999000072,48.17252449800003],[10.701787,48.09787299800007],[10.621906,47.94785249900008],[10.528557500000034,47.983941497000046],[10.326901,47.85807550000004],[10.131928500000072,47.82008749600004],[10.110136500000067,47.93714999800005],[10.180203500000061,47.93029449800008],[10.2304,48.04228249700003],[10.135728,48.02376149600008],[10.136457,48.10845949700007],[10.271433500000057,48.22965399800006],[10.295115,48.18451699800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Donau-Ries","NUTS_NAME":"Donau-Ries"},"geometry":{"type":"Polygon","coordinates":[[[10.642254500000035,49.016631500000074],[10.680472500000064,48.913612998000076],[10.817673500000069,48.949463499000046],[10.815371,48.90109049800003],[10.936839500000076,48.86327599800006],[11.005941500000063,48.821882997000046],[10.984102,48.69280199700006],[11.022986,48.62017149600007],[10.923928500000045,48.586954],[10.779367500000035,48.618930498000054],[10.622081,48.68485099800006],[10.658442499000046,48.73216999700003],[10.487258,48.696662],[10.423689,48.74449599600007],[10.45700050000005,48.917141498000035],[10.409849500000064,48.977435498000034],[10.642254500000035,49.016631500000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE27E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oberallgäu","NUTS_NAME":"Oberallgäu"},"geometry":{"type":"Polygon","coordinates":[[[10.326901,47.85807550000004],[10.435189,47.79870599800006],[10.511613500000067,47.66640599800007],[10.454439,47.55579699800006],[10.431323500000076,47.50367749600008],[10.475413,47.431702],[10.178353,47.270113998000056],[10.23393,47.38328599700003],[10.09985049900007,47.35476149700003],[10.09147050000007,47.45907599700007],[9.999526,47.48301699600006],[9.967813500000034,47.54624049600005],[10.077292500000055,47.63927449700003],[10.139437500000042,47.703273496000065],[10.064681,47.78588749900007],[10.131928500000072,47.82008749600004],[10.326901,47.85807550000004]],[[10.373037,47.713663498000074],[10.36991850000004,47.762999996000076],[10.236971,47.77037750000005],[10.285494500000027,47.696604499000046],[10.373037,47.713663498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE3","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"BERLIN","NUTS_NAME":"BERLIN"},"geometry":{"type":"Polygon","coordinates":[[[13.610827500000028,52.54423549900008],[13.66698,52.474166998000044],[13.743118,52.43399849800005],[13.699738,52.37788299700003],[13.42098499900004,52.376247],[13.31212,52.39959949900003],[13.165898,52.39427549700008],[13.088333,52.41961150000003],[13.153695500000026,52.50182150000006],[13.117393,52.51699549700004],[13.164263,52.59890049900008],[13.39854,52.64819449700008],[13.475647500000036,52.66628049900004],[13.610827500000028,52.54423549900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE30","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Berlin","NUTS_NAME":"Berlin"},"geometry":{"type":"Polygon","coordinates":[[[13.610827500000028,52.54423549900008],[13.66698,52.474166998000044],[13.743118,52.43399849800005],[13.699738,52.37788299700003],[13.42098499900004,52.376247],[13.31212,52.39959949900003],[13.165898,52.39427549700008],[13.088333,52.41961150000003],[13.153695500000026,52.50182150000006],[13.117393,52.51699549700004],[13.164263,52.59890049900008],[13.39854,52.64819449700008],[13.475647500000036,52.66628049900004],[13.610827500000028,52.54423549900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE300","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Berlin","NUTS_NAME":"Berlin"},"geometry":{"type":"Polygon","coordinates":[[[13.610827500000028,52.54423549900008],[13.66698,52.474166998000044],[13.743118,52.43399849800005],[13.699738,52.37788299700003],[13.42098499900004,52.376247],[13.31212,52.39959949900003],[13.165898,52.39427549700008],[13.088333,52.41961150000003],[13.153695500000026,52.50182150000006],[13.117393,52.51699549700004],[13.164263,52.59890049900008],[13.39854,52.64819449700008],[13.475647500000036,52.66628049900004],[13.610827500000028,52.54423549900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE4","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"BRANDENBURG","NUTS_NAME":"BRANDENBURG"},"geometry":{"type":"Polygon","coordinates":[[[14.241636500000027,53.42215199700007],[14.098893500000031,53.26195149800003],[14.412157,53.32963599700008],[14.441223,53.25435999900003],[14.377268,53.20175199800008],[14.357842,53.06766699800005],[14.143658,52.96136849900006],[14.156692,52.895590998000046],[14.127624,52.83166249900006],[14.436438,52.67990049800005],[14.565063,52.62449699900003],[14.639691,52.57157449700003],[14.60426,52.53348949800005],[14.635373,52.49636849900003],[14.534361999000055,52.39500799900003],[14.600891500000046,52.27205199800005],[14.716044,52.235458],[14.682174,52.11468899800008],[14.755227,52.070024997000075],[14.716716,52.001188],[14.693557,51.901287996000065],[14.590146,51.82102599800004],[14.752694,51.66709599700005],[14.729862,51.581776998000066],[14.447771,51.54206899600007],[14.163324500000044,51.54104299800008],[13.99631850000003,51.38260749600005],[13.835313,51.37678999700006],[13.691250500000024,51.37401299800007],[13.412191500000063,51.43937449800006],[13.21015,51.404735998000035],[13.207957,51.524353],[13.051025,51.647676999000055],[13.161745,51.690602],[13.12342650000005,51.856899498000075],[13.150913500000058,51.85961049900004],[12.769780500000024,51.97927449900004],[12.376122500000065,52.04511949700003],[12.276724499000068,52.104017999000064],[12.216783,52.170068498000035],[12.297277,52.22754549900003],[12.24446,52.24800199800006],[12.31718,52.45409549800007],[12.171555,52.506336496000074],[12.14324,52.53579749800008],[12.230846,52.62975499800007],[12.201294500000074,52.72900399800005],[12.249203500000021,52.79186199800006],[12.23396150000002,52.85945450000003],[12.12681150000003,52.89019949800007],[11.89485650000006,52.89410449700006],[11.597784500000046,53.03592649800004],[11.338867,53.05509899800006],[11.265732,53.12197799900008],[11.534572,53.125318496000034],[11.553182,53.20889049600004],[11.82889350000005,53.22865499800008],[12.01853,53.299594497000044],[12.045285,53.37098750000007],[12.325101500000073,53.321301497000036],[12.331751500000053,53.31801099900008],[12.984683,53.16498549900007],[13.241403500000047,53.23240099900005],[13.483196,53.29174199900007],[13.551025,53.399436499000046],[13.710065500000042,53.479062499000065],[13.827101500000026,53.498713500000065],[13.778238499000054,53.51343499800004],[13.798463499000036,53.54568949800006],[13.943100500000071,53.42991649900006],[14.241636500000027,53.42215199700007]],[[13.088333,52.41961150000003],[13.165898,52.39427549700008],[13.31212,52.39959949900003],[13.42098499900004,52.376247],[13.699738,52.37788299700003],[13.743118,52.43399849800005],[13.66698,52.474166998000044],[13.610827500000028,52.54423549900008],[13.475647500000036,52.66628049900004],[13.39854,52.64819449700008],[13.164263,52.59890049900008],[13.117393,52.51699549700004],[13.153695500000026,52.50182150000006],[13.088333,52.41961150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Brandenburg","NUTS_NAME":"Brandenburg"},"geometry":{"type":"Polygon","coordinates":[[[14.241636500000027,53.42215199700007],[14.098893500000031,53.26195149800003],[14.412157,53.32963599700008],[14.441223,53.25435999900003],[14.377268,53.20175199800008],[14.357842,53.06766699800005],[14.143658,52.96136849900006],[14.156692,52.895590998000046],[14.127624,52.83166249900006],[14.436438,52.67990049800005],[14.565063,52.62449699900003],[14.639691,52.57157449700003],[14.60426,52.53348949800005],[14.635373,52.49636849900003],[14.534361999000055,52.39500799900003],[14.600891500000046,52.27205199800005],[14.716044,52.235458],[14.682174,52.11468899800008],[14.755227,52.070024997000075],[14.716716,52.001188],[14.693557,51.901287996000065],[14.590146,51.82102599800004],[14.752694,51.66709599700005],[14.729862,51.581776998000066],[14.447771,51.54206899600007],[14.163324500000044,51.54104299800008],[13.99631850000003,51.38260749600005],[13.835313,51.37678999700006],[13.691250500000024,51.37401299800007],[13.412191500000063,51.43937449800006],[13.21015,51.404735998000035],[13.207957,51.524353],[13.051025,51.647676999000055],[13.161745,51.690602],[13.12342650000005,51.856899498000075],[13.150913500000058,51.85961049900004],[12.769780500000024,51.97927449900004],[12.376122500000065,52.04511949700003],[12.276724499000068,52.104017999000064],[12.216783,52.170068498000035],[12.297277,52.22754549900003],[12.24446,52.24800199800006],[12.31718,52.45409549800007],[12.171555,52.506336496000074],[12.14324,52.53579749800008],[12.230846,52.62975499800007],[12.201294500000074,52.72900399800005],[12.249203500000021,52.79186199800006],[12.23396150000002,52.85945450000003],[12.12681150000003,52.89019949800007],[11.89485650000006,52.89410449700006],[11.597784500000046,53.03592649800004],[11.338867,53.05509899800006],[11.265732,53.12197799900008],[11.534572,53.125318496000034],[11.553182,53.20889049600004],[11.82889350000005,53.22865499800008],[12.01853,53.299594497000044],[12.045285,53.37098750000007],[12.325101500000073,53.321301497000036],[12.331751500000053,53.31801099900008],[12.984683,53.16498549900007],[13.241403500000047,53.23240099900005],[13.483196,53.29174199900007],[13.551025,53.399436499000046],[13.710065500000042,53.479062499000065],[13.827101500000026,53.498713500000065],[13.778238499000054,53.51343499800004],[13.798463499000036,53.54568949800006],[13.943100500000071,53.42991649900006],[14.241636500000027,53.42215199700007]],[[13.088333,52.41961150000003],[13.165898,52.39427549700008],[13.31212,52.39959949900003],[13.42098499900004,52.376247],[13.699738,52.37788299700003],[13.743118,52.43399849800005],[13.66698,52.474166998000044],[13.610827500000028,52.54423549900008],[13.475647500000036,52.66628049900004],[13.39854,52.64819449700008],[13.164263,52.59890049900008],[13.117393,52.51699549700004],[13.153695500000026,52.50182150000006],[13.088333,52.41961150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE401","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Brandenburg an der Havel, Kreisfreie Stadt","NUTS_NAME":"Brandenburg an der Havel, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.36118,52.442772],[12.428175,52.45814349800003],[12.692165,52.50557150000003],[12.573125,52.46403349800005],[12.683500500000036,52.41109949600008],[12.505250500000045,52.322464998000044],[12.419125500000064,52.31151199900006],[12.36118,52.442772]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE402","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Cottbus, Kreisfreie Stadt","NUTS_NAME":"Cottbus, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[14.443265,51.70311649900003],[14.302426,51.70206349700004],[14.27399,51.793862499000056],[14.302647,51.86151249900007],[14.501343,51.78060850000003],[14.391831500000023,51.75338549600008],[14.443265,51.70311649900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE403","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Frankfurt (Oder), Kreisfreie Stadt","NUTS_NAME":"Frankfurt (Oder), Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[14.600891500000046,52.27205199800005],[14.396731,52.266980999000054],[14.423764500000061,52.373968500000046],[14.534361999000055,52.39500799900003],[14.600891500000046,52.27205199800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE404","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Potsdam, Kreisfreie Stadt","NUTS_NAME":"Potsdam, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[13.153695500000026,52.50182150000006],[13.088333,52.41961150000003],[13.165898,52.39427549700008],[13.022350500000073,52.34693699800005],[12.893530999000063,52.46076349600003],[13.153695500000026,52.50182150000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE405","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Barnim","NUTS_NAME":"Barnim"},"geometry":{"type":"Polygon","coordinates":[[[14.143658,52.96136849900006],[14.156692,52.895590998000046],[13.731493,52.733783998000035],[13.848850500000026,52.659504],[13.610827500000028,52.54423549900008],[13.475647500000036,52.66628049900004],[13.39854,52.64819449700008],[13.421964,52.804170500000055],[13.553296,52.86154950000008],[13.397831,52.93392],[13.517898,52.988665998000045],[13.885174,53.05230799800006],[13.823976,52.98811050000006],[14.143658,52.96136849900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE406","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dahme-Spreewald","NUTS_NAME":"Dahme-Spreewald"},"geometry":{"type":"Polygon","coordinates":[[[13.699738,52.37788299700003],[13.854037,52.28998899800007],[13.84528799900005,52.12171549800007],[14.445997500000033,52.02635649800004],[14.36967050000004,52.00022049900008],[14.393165,51.93532699900004],[14.088485500000047,51.87830499900008],[13.863734,51.89218849900004],[13.798085,51.737132],[13.507085500000073,51.792032498000026],[13.568933,51.810309],[13.538717,51.865331998000045],[13.581303500000047,51.89154250000007],[13.4121,51.930064],[13.41483850000003,51.976151497000046],[13.652954500000021,52.04814749800005],[13.540823,52.09721599900007],[13.545966,52.193182998000054],[13.581984,52.197266],[13.460408,52.24008649800004],[13.511562,52.267048497000076],[13.499100500000054,52.33666149700008],[13.42098499900004,52.376247],[13.699738,52.37788299700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE407","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Elbe-Elster","NUTS_NAME":"Elbe-Elster"},"geometry":{"type":"Polygon","coordinates":[[[13.507085500000073,51.792032498000026],[13.798085,51.737132],[13.88937450000003,51.61804649800007],[13.65397050000007,51.528902998000035],[13.71988250000004,51.44428349900005],[13.691250500000024,51.37401299800007],[13.412191500000063,51.43937449800006],[13.21015,51.404735998000035],[13.207957,51.524353],[13.051025,51.647676999000055],[13.161745,51.690602],[13.12342650000005,51.856899498000075],[13.150913500000058,51.85961049900004],[13.507085500000073,51.792032498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE408","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Havelland","NUTS_NAME":"Havelland"},"geometry":{"type":"Polygon","coordinates":[[[12.828762500000039,52.69475650000004],[12.963380500000028,52.720312500000034],[13.164263,52.59890049900008],[13.117393,52.51699549700004],[13.153695500000026,52.50182150000006],[12.893530999000063,52.46076349600003],[12.745205,52.45848499700003],[12.716817500000047,52.54637949800008],[12.428372,52.553945998000074],[12.428175,52.45814349800003],[12.36118,52.442772],[12.31718,52.45409549800007],[12.171555,52.506336496000074],[12.14324,52.53579749800008],[12.230846,52.62975499800007],[12.201294500000074,52.72900399800005],[12.249203500000021,52.79186199800006],[12.532588500000031,52.748639997000055],[12.548252,52.81052849800005],[12.828762500000039,52.69475650000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE409","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Märkisch-Oderland","NUTS_NAME":"Märkisch-Oderland"},"geometry":{"type":"Polygon","coordinates":[[[14.156692,52.895590998000046],[14.127624,52.83166249900006],[14.436438,52.67990049800005],[14.565063,52.62449699900003],[14.639691,52.57157449700003],[14.60426,52.53348949800005],[14.635373,52.49636849900003],[14.534361999000055,52.39500799900003],[14.423764500000061,52.373968500000046],[14.31484,52.38220749800007],[14.1885,52.49874649700007],[13.66698,52.474166998000044],[13.610827500000028,52.54423549900008],[13.848850500000026,52.659504],[13.731493,52.733783998000035],[14.156692,52.895590998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oberhavel","NUTS_NAME":"Oberhavel"},"geometry":{"type":"Polygon","coordinates":[[[13.241403500000047,53.23240099900005],[13.406759500000021,53.02526949600008],[13.517898,52.988665998000045],[13.397831,52.93392],[13.553296,52.86154950000008],[13.421964,52.804170500000055],[13.39854,52.64819449700008],[13.164263,52.59890049900008],[12.963380500000028,52.720312500000034],[12.828762500000039,52.69475650000004],[13.046662500000025,52.91758899900003],[13.025242,53.071727],[12.937771500000053,53.104248],[12.984683,53.16498549900007],[13.241403500000047,53.23240099900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oberspreewald-Lausitz","NUTS_NAME":"Oberspreewald-Lausitz"},"geometry":{"type":"Polygon","coordinates":[[[14.088485500000047,51.87830499900008],[14.092074,51.67430699800008],[14.202078500000027,51.61304699600004],[14.163324500000044,51.54104299800008],[13.99631850000003,51.38260749600005],[13.835313,51.37678999700006],[13.691250500000024,51.37401299800007],[13.71988250000004,51.44428349900005],[13.65397050000007,51.528902998000035],[13.88937450000003,51.61804649800007],[13.798085,51.737132],[13.863734,51.89218849900004],[14.088485500000047,51.87830499900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oder-Spree","NUTS_NAME":"Oder-Spree"},"geometry":{"type":"Polygon","coordinates":[[[14.423764500000061,52.373968500000046],[14.396731,52.266980999000054],[14.600891500000046,52.27205199800005],[14.716044,52.235458],[14.682174,52.11468899800008],[14.755227,52.070024997000075],[14.716716,52.001188],[14.445997500000033,52.02635649800004],[13.84528799900005,52.12171549800007],[13.854037,52.28998899800007],[13.699738,52.37788299700003],[13.743118,52.43399849800005],[13.66698,52.474166998000044],[14.1885,52.49874649700007],[14.31484,52.38220749800007],[14.423764500000061,52.373968500000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ostprignitz-Ruppin","NUTS_NAME":"Ostprignitz-Ruppin"},"geometry":{"type":"Polygon","coordinates":[[[12.331751500000053,53.31801099900008],[12.984683,53.16498549900007],[12.937771500000053,53.104248],[13.025242,53.071727],[13.046662500000025,52.91758899900003],[12.828762500000039,52.69475650000004],[12.548252,52.81052849800005],[12.532588500000031,52.748639997000055],[12.249203500000021,52.79186199800006],[12.23396150000002,52.85945450000003],[12.12681150000003,52.89019949800007],[12.416904,53.00474049700006],[12.288967,53.06170049900004],[12.348256,53.23782699800006],[12.325101500000073,53.321301497000036],[12.331751500000053,53.31801099900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Potsdam-Mittelmark","NUTS_NAME":"Potsdam-Mittelmark"},"geometry":{"type":"Polygon","coordinates":[[[12.893530999000063,52.46076349600003],[13.022350500000073,52.34693699800005],[13.165898,52.39427549700008],[13.31212,52.39959949900003],[12.97355,52.145455999000035],[12.957408,52.04231049800006],[12.769780500000024,51.97927449900004],[12.376122500000065,52.04511949700003],[12.276724499000068,52.104017999000064],[12.216783,52.170068498000035],[12.297277,52.22754549900003],[12.24446,52.24800199800006],[12.31718,52.45409549800007],[12.36118,52.442772],[12.419125500000064,52.31151199900006],[12.505250500000045,52.322464998000044],[12.683500500000036,52.41109949600008],[12.573125,52.46403349800005],[12.692165,52.50557150000003],[12.428175,52.45814349800003],[12.428372,52.553945998000074],[12.716817500000047,52.54637949800008],[12.745205,52.45848499700003],[12.893530999000063,52.46076349600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Prignitz","NUTS_NAME":"Prignitz"},"geometry":{"type":"Polygon","coordinates":[[[12.325101500000073,53.321301497000036],[12.348256,53.23782699800006],[12.288967,53.06170049900004],[12.416904,53.00474049700006],[12.12681150000003,52.89019949800007],[11.89485650000006,52.89410449700006],[11.597784500000046,53.03592649800004],[11.338867,53.05509899800006],[11.265732,53.12197799900008],[11.534572,53.125318496000034],[11.553182,53.20889049600004],[11.82889350000005,53.22865499800008],[12.01853,53.299594497000044],[12.045285,53.37098750000007],[12.325101500000073,53.321301497000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40G","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Spree-Neiße","NUTS_NAME":"Spree-Neiße"},"geometry":{"type":"Polygon","coordinates":[[[14.716716,52.001188],[14.693557,51.901287996000065],[14.590146,51.82102599800004],[14.752694,51.66709599700005],[14.729862,51.581776998000066],[14.447771,51.54206899600007],[14.163324500000044,51.54104299800008],[14.202078500000027,51.61304699600004],[14.092074,51.67430699800008],[14.088485500000047,51.87830499900008],[14.393165,51.93532699900004],[14.36967050000004,52.00022049900008],[14.445997500000033,52.02635649800004],[14.716716,52.001188]],[[14.443265,51.70311649900003],[14.391831500000023,51.75338549600008],[14.501343,51.78060850000003],[14.302647,51.86151249900007],[14.27399,51.793862499000056],[14.302426,51.70206349700004],[14.443265,51.70311649900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40H","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Teltow-Fläming","NUTS_NAME":"Teltow-Fläming"},"geometry":{"type":"Polygon","coordinates":[[[13.42098499900004,52.376247],[13.499100500000054,52.33666149700008],[13.511562,52.267048497000076],[13.460408,52.24008649800004],[13.581984,52.197266],[13.545966,52.193182998000054],[13.540823,52.09721599900007],[13.652954500000021,52.04814749800005],[13.41483850000003,51.976151497000046],[13.4121,51.930064],[13.581303500000047,51.89154250000007],[13.538717,51.865331998000045],[13.568933,51.810309],[13.507085500000073,51.792032498000026],[13.150913500000058,51.85961049900004],[12.769780500000024,51.97927449900004],[12.957408,52.04231049800006],[12.97355,52.145455999000035],[13.31212,52.39959949900003],[13.42098499900004,52.376247]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE733","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hersfeld-Rotenburg","NUTS_NAME":"Hersfeld-Rotenburg"},"geometry":{"type":"Polygon","coordinates":[[[10.021558500000026,50.992949499000076],[10.063865,50.94543299600008],[9.949583500000074,50.927478499000074],[10.055980999000042,50.879771497000036],[9.926176500000054,50.767389998000056],[9.77266,50.802213996000035],[9.611464500000068,50.73547649900007],[9.441055,50.79564450000004],[9.508171,50.81873699800008],[9.482093,50.89663549900007],[9.580513500000052,51.02171949900003],[9.777533,51.083158497000056],[10.021558500000026,50.992949499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE734","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kassel, Landkreis","NUTS_NAME":"Kassel, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.685331,51.58201649800003],[9.672377500000039,51.568403998000065],[9.625825001000067,51.580204999000046],[9.647755500000073,51.55251099800006],[9.585751,51.51731749900006],[9.642956001000073,51.41632149900005],[9.557291,51.35137849800003],[9.357435,51.31583449900006],[9.482835,51.26194099700007],[9.568025,51.34000149800005],[9.710295479000024,51.30153774900003],[9.645224002000077,51.27885649700005],[9.759452500000066,51.260649500000056],[9.635957500000075,51.18309799700006],[9.284571001000074,51.25413049800005],[9.189525500000059,51.17041799900005],[9.095078,51.255781497000044],[9.101653500000054,51.364355998000065],[9.174385500000028,51.37140849900004],[9.155410500000073,51.44267499800003],[9.440457,51.65039399800003],[9.685331,51.58201649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE735","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwalm-Eder-Kreis","NUTS_NAME":"Schwalm-Eder-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[9.635957500000075,51.18309799700006],[9.773271,51.14628649900004],[9.777533,51.083158497000056],[9.580513500000052,51.02171949900003],[9.482093,50.89663549900007],[9.508171,50.81873699800008],[9.441055,50.79564450000004],[9.148868500000049,50.83664499900004],[9.109958,50.91897749800006],[8.9746,50.938304],[9.221377,51.12733449800004],[9.189525500000059,51.17041799900005],[9.284571001000074,51.25413049800005],[9.635957500000075,51.18309799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE736","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Waldeck-Frankenberg","NUTS_NAME":"Waldeck-Frankenberg"},"geometry":{"type":"Polygon","coordinates":[[[9.155410500000073,51.44267499800003],[9.174385500000028,51.37140849900004],[9.101653500000054,51.364355998000065],[9.095078,51.255781497000044],[9.189525500000059,51.17041799900005],[9.221377,51.12733449800004],[8.9746,50.938304],[8.477892,50.96904749600003],[8.549084999000058,51.101868],[8.656632500000057,51.094902998000066],[8.758214500000065,51.17718149800004],[8.721997500000043,51.273743996000064],[8.556348,51.277494999000055],[8.68347650000004,51.36217799800005],[8.939502,51.38949899800008],[8.890376500000059,51.48192849900005],[8.970653500000026,51.506773499000076],[9.155410500000073,51.44267499800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE737","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Werra-Meißner-Kreis","NUTS_NAME":"Werra-Meißner-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[9.928339,51.37529899900005],[10.067095,51.22779299800004],[10.206942,51.19064899600005],[10.235999,51.18476749800004],[10.210013,51.144082998000044],[10.118321500000036,51.13957299900005],[10.202181,51.012009],[10.182348500000046,50.99848749900008],[10.021558500000026,50.992949499000076],[9.777533,51.083158497000056],[9.773271,51.14628649900004],[9.635957500000075,51.18309799700006],[9.759452500000066,51.260649500000056],[9.645224002000077,51.27885649700005],[9.710295479000024,51.30153774900003],[9.775202,51.32519599800003],[9.702144,51.36457199800003],[9.803510499000026,51.41040049800006],[9.928339,51.37529899900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE8","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"MECKLENBURG-VORPOMMERN","NUTS_NAME":"MECKLENBURG-VORPOMMERN"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.436138500000027,54.379089],[12.36317450000007,54.267097998000054],[13.055968500000063,54.44981949800007],[13.017391719000045,54.43872077300006],[13.109225,54.27934049700008],[13.319396,54.193505997000045],[13.344701500000042,54.18111149600003],[13.316288500000041,54.16033349700007],[13.489404499000045,54.083846],[13.706645500000036,54.17161499800005],[13.803770500000041,54.09803149900006],[13.742715500000031,54.031386498000074],[13.914190500000075,53.920434999000065],[13.826154,53.87762799800004],[13.80763550000006,53.85568049900007],[13.81915250000003,53.84194499600005],[14.102733,53.73978199800007],[14.257057,53.75519249800004],[14.214806500000066,53.70314299900008],[14.267542,53.697806496000055],[14.412157,53.32963599700008],[14.098893500000031,53.26195149800003],[14.241636500000027,53.42215199700007],[13.943100500000071,53.42991649900006],[13.798463499000036,53.54568949800006],[13.778238499000054,53.51343499800004],[13.827101500000026,53.498713500000065],[13.710065500000042,53.479062499000065],[13.551025,53.399436499000046],[13.483196,53.29174199900007],[13.241403500000047,53.23240099900005],[12.984683,53.16498549900007],[12.331751500000053,53.31801099900008],[12.325101500000073,53.321301497000036],[12.045285,53.37098750000007],[12.01853,53.299594497000044],[11.82889350000005,53.22865499800008],[11.553182,53.20889049600004],[11.534572,53.125318496000034],[11.265732,53.12197799900008],[11.171861500000034,53.15664399700006],[10.988000500000055,53.33342649800005],[10.595047,53.363927500000045],[10.634904500000061,53.45603299900006],[10.911107500000071,53.57257450000003],[10.951918500000033,53.64762249800003],[10.771398,53.74950499700003],[10.762964500000066,53.81115299900006],[10.770835501000022,53.87346249600006],[10.968375500000036,53.90886199800008],[10.903661501000045,53.956822],[11.181602500000054,54.01479199900007],[11.26820250000003,53.93225999600003],[11.444478,53.91073699600008],[11.56177850000006,54.028088497000056],[11.626486,54.08964549800004],[11.524761500000068,54.069671],[11.68004650000006,54.152386],[11.998484500000075,54.17496999800005],[12.086768500000062,54.18508049600007],[12.088310500000034,54.113248498000075],[12.151354999000034,54.09425949900003],[12.097017,54.15189799900003],[12.201068,54.244695498000056],[12.28553050000005,54.27495199800006],[12.520881067000062,54.474234541000044],[12.92695150000003,54.42796399800005],[12.436138500000027,54.379089]]],[[[13.97208450000005,54.06874749700006],[14.226302,53.92865299700003],[14.213077500000054,53.86647949800005],[13.82615050000004,53.849684997000054],[13.938169500000072,53.90960399700003],[13.911077500000033,53.99123049700006],[14.040180500000076,53.94072149900006],[14.047417500000051,54.00135849700007],[13.910431500000072,54.06298749800004],[13.861864500000024,53.99975499900006],[13.879676500000073,54.04025999800007],[13.810579,54.031206498000074],[13.813017,54.099393500000076],[13.75074599900006,54.14996050000008],[13.97208450000005,54.06874749700006]]],[[[13.391083500000036,54.65131699900007],[13.423516,54.578565998000045],[13.67934,54.56278249700006],[13.569855500000074,54.46715049900007],[13.72786,54.35514799900005],[13.72390450000006,54.27340749800004],[13.645403999000052,54.29756049700006],[13.682453500000065,54.34958099900007],[13.507070500000054,54.34420649800006],[13.352598500000056,54.26796899800007],[13.425133,54.23918499800004],[13.394217500000025,54.22250249900003],[13.114586,54.33190799600004],[13.261483500000054,54.38291649900003],[13.149309500000072,54.42391550000008],[13.270131,54.480173498000056],[13.144181,54.54696549800008],[13.243102746000034,54.560483161000036],[13.282402500000046,54.64570449900003],[13.16017050000005,54.56385250000005],[13.26698250000004,54.66663349800007],[13.438944,54.67753299900005],[13.391083500000036,54.65131699900007]],[[13.368071500000042,54.614697498000055],[13.243615336000062,54.56055321700006],[13.370426500000065,54.57788849900004],[13.475655500000073,54.42683399900005],[13.557416500000045,54.440044999000065],[13.368071500000042,54.614697498000055]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Mecklenburg-Vorpommern","NUTS_NAME":"Mecklenburg-Vorpommern"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.436138500000027,54.379089],[12.36317450000007,54.267097998000054],[13.055968500000063,54.44981949800007],[13.017391719000045,54.43872077300006],[13.109225,54.27934049700008],[13.319396,54.193505997000045],[13.344701500000042,54.18111149600003],[13.316288500000041,54.16033349700007],[13.489404499000045,54.083846],[13.706645500000036,54.17161499800005],[13.803770500000041,54.09803149900006],[13.742715500000031,54.031386498000074],[13.914190500000075,53.920434999000065],[13.826154,53.87762799800004],[13.80763550000006,53.85568049900007],[13.81915250000003,53.84194499600005],[14.102733,53.73978199800007],[14.257057,53.75519249800004],[14.214806500000066,53.70314299900008],[14.267542,53.697806496000055],[14.412157,53.32963599700008],[14.098893500000031,53.26195149800003],[14.241636500000027,53.42215199700007],[13.943100500000071,53.42991649900006],[13.798463499000036,53.54568949800006],[13.778238499000054,53.51343499800004],[13.827101500000026,53.498713500000065],[13.710065500000042,53.479062499000065],[13.551025,53.399436499000046],[13.483196,53.29174199900007],[13.241403500000047,53.23240099900005],[12.984683,53.16498549900007],[12.331751500000053,53.31801099900008],[12.325101500000073,53.321301497000036],[12.045285,53.37098750000007],[12.01853,53.299594497000044],[11.82889350000005,53.22865499800008],[11.553182,53.20889049600004],[11.534572,53.125318496000034],[11.265732,53.12197799900008],[11.171861500000034,53.15664399700006],[10.988000500000055,53.33342649800005],[10.595047,53.363927500000045],[10.634904500000061,53.45603299900006],[10.911107500000071,53.57257450000003],[10.951918500000033,53.64762249800003],[10.771398,53.74950499700003],[10.762964500000066,53.81115299900006],[10.770835501000022,53.87346249600006],[10.968375500000036,53.90886199800008],[10.903661501000045,53.956822],[11.181602500000054,54.01479199900007],[11.26820250000003,53.93225999600003],[11.444478,53.91073699600008],[11.56177850000006,54.028088497000056],[11.626486,54.08964549800004],[11.524761500000068,54.069671],[11.68004650000006,54.152386],[11.998484500000075,54.17496999800005],[12.086768500000062,54.18508049600007],[12.088310500000034,54.113248498000075],[12.151354999000034,54.09425949900003],[12.097017,54.15189799900003],[12.201068,54.244695498000056],[12.28553050000005,54.27495199800006],[12.520881067000062,54.474234541000044],[12.92695150000003,54.42796399800005],[12.436138500000027,54.379089]]],[[[13.97208450000005,54.06874749700006],[14.226302,53.92865299700003],[14.213077500000054,53.86647949800005],[13.82615050000004,53.849684997000054],[13.938169500000072,53.90960399700003],[13.911077500000033,53.99123049700006],[14.040180500000076,53.94072149900006],[14.047417500000051,54.00135849700007],[13.910431500000072,54.06298749800004],[13.861864500000024,53.99975499900006],[13.879676500000073,54.04025999800007],[13.810579,54.031206498000074],[13.813017,54.099393500000076],[13.75074599900006,54.14996050000008],[13.97208450000005,54.06874749700006]]],[[[13.391083500000036,54.65131699900007],[13.423516,54.578565998000045],[13.67934,54.56278249700006],[13.569855500000074,54.46715049900007],[13.72786,54.35514799900005],[13.72390450000006,54.27340749800004],[13.645403999000052,54.29756049700006],[13.682453500000065,54.34958099900007],[13.507070500000054,54.34420649800006],[13.352598500000056,54.26796899800007],[13.425133,54.23918499800004],[13.394217500000025,54.22250249900003],[13.114586,54.33190799600004],[13.261483500000054,54.38291649900003],[13.149309500000072,54.42391550000008],[13.270131,54.480173498000056],[13.144181,54.54696549800008],[13.243102746000034,54.560483161000036],[13.282402500000046,54.64570449900003],[13.16017050000005,54.56385250000005],[13.26698250000004,54.66663349800007],[13.438944,54.67753299900005],[13.391083500000036,54.65131699900007]],[[13.368071500000042,54.614697498000055],[13.243615336000062,54.56055321700006],[13.370426500000065,54.57788849900004],[13.475655500000073,54.42683399900005],[13.557416500000045,54.440044999000065],[13.368071500000042,54.614697498000055]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE803","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rostock, Kreisfreie Stadt","NUTS_NAME":"Rostock, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.998484500000075,54.17496999800005],[12.086768500000062,54.18508049600007],[12.088310500000034,54.113248498000075],[12.151354999000034,54.09425949900003],[12.097017,54.15189799900003],[12.201068,54.244695498000056],[12.293673,54.197095998000066],[12.204073,54.182446998000046],[12.191380500000037,54.06928949700006],[12.065573499000038,54.059600498000066],[11.998484500000075,54.17496999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE804","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schwerin, Kreisfreie Stadt","NUTS_NAME":"Schwerin, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.449474500000065,53.68067549600005],[11.504032,53.62190949700005],[11.483396,53.55856049800008],[11.384086500000024,53.544411497000056],[11.301945,53.65294749700007],[11.449474500000065,53.68067549600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80J","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mecklenburgische Seenplatte","NUTS_NAME":"Mecklenburgische Seenplatte"},"geometry":{"type":"Polygon","coordinates":[[[12.986711,54.00396949600008],[13.193213,53.931272996000075],[13.185201500000062,53.83851899900003],[13.233367500000043,53.80807249900005],[13.372178500000075,53.848523998000076],[13.396986499000036,53.830659996000065],[13.344012500000076,53.78798449800007],[13.547447499000043,53.69618549900008],[13.79611,53.68552549800006],[13.698585,53.653872997000065],[13.76031450000005,53.615053498000066],[13.668914500000028,53.51123849800007],[13.710065500000042,53.479062499000065],[13.551025,53.399436499000046],[13.483196,53.29174199900007],[13.241403500000047,53.23240099900005],[12.984683,53.16498549900007],[12.331751500000053,53.31801099900008],[12.275056,53.37897799700005],[12.330679,53.478143999000054],[12.289342500000032,53.51793350000003],[12.305616,53.56769949900007],[12.41067,53.575116500000036],[12.472664500000064,53.698342498000045],[12.653763500000025,53.65569549800006],[12.629207,53.75255549900004],[12.749436,53.82504250000005],[12.808647,54.00493749900005],[12.986711,54.00396949600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80K","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Landkreis Rostock","NUTS_NAME":"Landkreis Rostock"},"geometry":{"type":"Polygon","coordinates":[[[12.808647,54.00493749900005],[12.749436,53.82504250000005],[12.629207,53.75255549900004],[12.653763500000025,53.65569549800006],[12.472664500000064,53.698342498000045],[12.41067,53.575116500000036],[12.305616,53.56769949900007],[11.98378450000007,53.74294],[11.79735,53.77078249700003],[11.756477,53.82558049900007],[11.804275500000074,53.96608599700005],[11.56177850000006,54.028088497000056],[11.626486,54.08964549800004],[11.524761500000068,54.069671],[11.68004650000006,54.152386],[11.998484500000075,54.17496999800005],[12.065573499000038,54.059600498000066],[12.191380500000037,54.06928949700006],[12.204073,54.182446998000046],[12.293673,54.197095998000066],[12.201068,54.244695498000056],[12.28553050000005,54.27495199800006],[12.37242,54.20599649900004],[12.384520500000065,54.11406950000003],[12.808647,54.00493749900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80L","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vorpommern-Rügen","NUTS_NAME":"Vorpommern-Rügen"},"geometry":{"type":"MultiPolygon","coordinates":[[[[13.391083500000036,54.65131699900007],[13.423516,54.578565998000045],[13.67934,54.56278249700006],[13.569855500000074,54.46715049900007],[13.72786,54.35514799900005],[13.72390450000006,54.27340749800004],[13.645403999000052,54.29756049700006],[13.682453500000065,54.34958099900007],[13.507070500000054,54.34420649800006],[13.352598500000056,54.26796899800007],[13.425133,54.23918499800004],[13.394217500000025,54.22250249900003],[13.114586,54.33190799600004],[13.261483500000054,54.38291649900003],[13.149309500000072,54.42391550000008],[13.270131,54.480173498000056],[13.144181,54.54696549800008],[13.243102746000034,54.560483161000036],[13.282402500000046,54.64570449900003],[13.16017050000005,54.56385250000005],[13.26698250000004,54.66663349800007],[13.438944,54.67753299900005],[13.391083500000036,54.65131699900007]],[[13.368071500000042,54.614697498000055],[13.243615336000062,54.56055321700006],[13.370426500000065,54.57788849900004],[13.475655500000073,54.42683399900005],[13.557416500000045,54.440044999000065],[13.368071500000042,54.614697498000055]]],[[[12.436138500000027,54.379089],[12.36317450000007,54.267097998000054],[13.055968500000063,54.44981949800007],[13.017391719000045,54.43872077300006],[13.109225,54.27934049700008],[13.319396,54.193505997000045],[13.216513,54.117889999000056],[13.26178,54.081391996000036],[13.212961,54.04486749800003],[12.986711,54.00396949600008],[12.808647,54.00493749900005],[12.384520500000065,54.11406950000003],[12.37242,54.20599649900004],[12.28553050000005,54.27495199800006],[12.520881067000062,54.474234541000044],[12.92695150000003,54.42796399800005],[12.436138500000027,54.379089]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80M","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nordwestmecklenburg","NUTS_NAME":"Nordwestmecklenburg"},"geometry":{"type":"Polygon","coordinates":[[[11.56177850000006,54.028088497000056],[11.804275500000074,53.96608599700005],[11.756477,53.82558049900007],[11.79735,53.77078249700003],[11.495146,53.771336500000075],[11.449474500000065,53.68067549600005],[11.301945,53.65294749700007],[11.149961,53.566956997000034],[10.951918500000033,53.64762249800003],[10.771398,53.74950499700003],[10.762964500000066,53.81115299900006],[10.770835501000022,53.87346249600006],[10.968375500000036,53.90886199800008],[10.903661501000045,53.956822],[11.181602500000054,54.01479199900007],[11.26820250000003,53.93225999600003],[11.444478,53.91073699600008],[11.56177850000006,54.028088497000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80N","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vorpommern-Greifswald","NUTS_NAME":"Vorpommern-Greifswald"},"geometry":{"type":"MultiPolygon","coordinates":[[[[13.316288500000041,54.16033349700007],[13.489404499000045,54.083846],[13.706645500000036,54.17161499800005],[13.803770500000041,54.09803149900006],[13.742715500000031,54.031386498000074],[13.914190500000075,53.920434999000065],[13.826154,53.87762799800004],[13.80763550000006,53.85568049900007],[13.81915250000003,53.84194499600005],[14.102733,53.73978199800007],[14.257057,53.75519249800004],[14.214806500000066,53.70314299900008],[14.267542,53.697806496000055],[14.412157,53.32963599700008],[14.098893500000031,53.26195149800003],[14.241636500000027,53.42215199700007],[13.943100500000071,53.42991649900006],[13.798463499000036,53.54568949800006],[13.778238499000054,53.51343499800004],[13.827101500000026,53.498713500000065],[13.710065500000042,53.479062499000065],[13.668914500000028,53.51123849800007],[13.76031450000005,53.615053498000066],[13.698585,53.653872997000065],[13.79611,53.68552549800006],[13.547447499000043,53.69618549900008],[13.344012500000076,53.78798449800007],[13.396986499000036,53.830659996000065],[13.372178500000075,53.848523998000076],[13.233367500000043,53.80807249900005],[13.185201500000062,53.83851899900003],[13.193213,53.931272996000075],[12.986711,54.00396949600008],[13.212961,54.04486749800003],[13.26178,54.081391996000036],[13.216513,54.117889999000056],[13.319396,54.193505997000045],[13.344701500000042,54.18111149600003],[13.316288500000041,54.16033349700007]]],[[[13.97208450000005,54.06874749700006],[14.226302,53.92865299700003],[14.213077500000054,53.86647949800005],[13.82615050000004,53.849684997000054],[13.938169500000072,53.90960399700003],[13.911077500000033,53.99123049700006],[14.040180500000076,53.94072149900006],[14.047417500000051,54.00135849700007],[13.910431500000072,54.06298749800004],[13.861864500000024,53.99975499900006],[13.879676500000073,54.04025999800007],[13.810579,54.031206498000074],[13.813017,54.099393500000076],[13.75074599900006,54.14996050000008],[13.97208450000005,54.06874749700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE80O","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ludwigslust-Parchim","NUTS_NAME":"Ludwigslust-Parchim"},"geometry":{"type":"Polygon","coordinates":[[[11.79735,53.77078249700003],[11.98378450000007,53.74294],[12.305616,53.56769949900007],[12.289342500000032,53.51793350000003],[12.330679,53.478143999000054],[12.275056,53.37897799700005],[12.331751500000053,53.31801099900008],[12.325101500000073,53.321301497000036],[12.045285,53.37098750000007],[12.01853,53.299594497000044],[11.82889350000005,53.22865499800008],[11.553182,53.20889049600004],[11.534572,53.125318496000034],[11.265732,53.12197799900008],[11.171861500000034,53.15664399700006],[10.988000500000055,53.33342649800005],[10.595047,53.363927500000045],[10.634904500000061,53.45603299900006],[10.911107500000071,53.57257450000003],[10.951918500000033,53.64762249800003],[11.149961,53.566956997000034],[11.301945,53.65294749700007],[11.384086500000024,53.544411497000056],[11.483396,53.55856049800008],[11.504032,53.62190949700005],[11.449474500000065,53.68067549600005],[11.495146,53.771336500000075],[11.79735,53.77078249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE9","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"NIEDERSACHSEN","NUTS_NAME":"NIEDERSACHSEN"},"geometry":{"type":"Polygon","coordinates":[[[8.614913999000066,53.88211799900006],[9.029917500000067,53.82556499600008],[8.994673500000033,53.87260649600006],[9.02242,53.87951749900003],[9.19975099900006,53.88010449700005],[9.48589,53.70766449700005],[9.550639999000055,53.616034497000044],[9.73010399900005,53.55758099600007],[9.768861,53.50527899700006],[9.916487500000073,53.414067998000064],[10.308312,53.433221497000034],[10.469184,53.38584399900003],[10.595047,53.363927500000045],[10.988000500000055,53.33342649800005],[11.171861500000034,53.15664399700006],[11.265732,53.12197799900008],[11.338867,53.05509899800006],[11.597784500000046,53.03592649800004],[11.51249050000007,53.00744499900003],[11.505027,52.94103249800003],[10.841556,52.85220499900004],[10.759314500000073,52.79583049900003],[10.921412,52.610437],[10.976968,52.62414149700004],[10.937393,52.56789999800003],[11.008781500000055,52.49674749900004],[10.934542500000077,52.471794998000064],[11.0363,52.38520949900004],[11.086244,52.22863399800008],[11.01364650000005,52.19955599900004],[10.964414500000032,52.05664299700004],[10.801428500000043,52.04800049900007],[10.561227,52.004065998000044],[10.648214500000051,51.908245],[10.576971,51.84298749900006],[10.58152,51.78000549700005],[10.701372,51.642187499000045],[10.677283,51.638376],[10.488551,51.57477849700007],[10.365365,51.555891],[9.928339,51.37529899900005],[9.803510499000026,51.41040049800006],[9.702144,51.36457199800003],[9.775202,51.32519599800003],[9.710295479000024,51.30153774900003],[9.568025,51.34000149800005],[9.557291,51.35137849800003],[9.642956001000073,51.41632149900005],[9.585751,51.51731749900006],[9.647755500000073,51.55251099800006],[9.625825001000067,51.580204999000046],[9.672377500000039,51.568403998000065],[9.685331,51.58201649800003],[9.440457,51.65039399800003],[9.417317500000024,51.64726949800007],[9.374706,51.64984049700007],[9.375365,51.74361399900005],[9.459648500000071,51.86279749700003],[9.323343500000021,51.85506099600008],[9.34515250000004,51.89634],[9.308893,51.92271950000003],[9.18955150000005,51.964566998000066],[9.155604499000049,52.097829999000055],[8.985787500000072,52.19457649800006],[9.075462,52.23411699800005],[8.963536500000032,52.27902699900005],[9.125252500000045,52.411993497000026],[9.131973500000072,52.483982498000046],[9.052655999000024,52.500297998000065],[8.937577,52.40210049800004],[8.71523650000006,52.39256899900005],[8.703008999000076,52.50043799700006],[8.297213500000055,52.45649799700004],[8.44432450000005,52.36115349700003],[8.466186500000049,52.26761399900005],[8.44223050000005,52.213789500000075],[8.517049,52.18512399800005],[8.410586500000022,52.115114996000045],[8.096448,52.057145],[7.885159,52.08330249900007],[8.007796,52.11533299700005],[8.016978,52.17205049900008],[7.902993500000036,52.19886949900007],[7.956465,52.27249049900007],[7.931592,52.30146849600004],[7.990608,52.30953950000003],[7.964625500000068,52.32485850000006],[7.608038500000021,52.47401599800003],[7.563964500000054,52.37065199800003],[7.317481,52.280271998000046],[7.099148999000022,52.24305849600006],[7.065685,52.24137299800003],[7.026305,52.28744399700008],[7.072438,52.37311650000004],[6.987941500000034,52.46954099800007],[6.697865500000034,52.48628599700004],[6.70973250000003,52.627823498000055],[7.006229500000074,52.63876299800006],[7.055577,52.643369998000026],[7.092692,52.83820099800005],[7.209295,53.00024449700004],[7.202794500000039,53.11328149800005],[7.208935,53.243064497000034],[7.192496,53.31762750000007],[7.2643,53.325526497000055],[6.999446500000033,53.35988750000007],[7.024107500000071,53.48237149600004],[7.138518,53.53548099900007],[7.090335500000037,53.57965199900008],[7.243681499000047,53.66778199600003],[7.550173,53.67503749900004],[7.809895500000039,53.70766199800005],[8.017161500000043,53.710496498000055],[8.091291,53.638109],[8.164292999000054,53.52261449800005],[8.061327,53.505957],[8.195555,53.409171498000035],[8.315217500000074,53.46179599800007],[8.303364,53.525487],[8.22693,53.52065249800006],[8.329544,53.61402699900003],[8.554898500000036,53.525131],[8.492652,53.47242],[8.652134001000036,53.516016998000055],[8.650632500000029,53.602565],[8.520410500000025,53.606205],[8.483350501000075,53.69478399800005],[8.614913999000066,53.88211799900006]],[[8.711424,53.044629996000026],[8.915827,53.01102149600007],[8.979256,53.04584949900004],[8.947018,53.11610099900008],[8.984185500000024,53.126070998000046],[8.485330500000032,53.22711999900008],[8.654898500000058,53.10886449900005],[8.711424,53.044629996000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE91","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Braunschweig","NUTS_NAME":"Braunschweig"},"geometry":{"type":"Polygon","coordinates":[[[10.759314500000073,52.79583049900003],[10.921412,52.610437],[10.976968,52.62414149700004],[10.937393,52.56789999800003],[11.008781500000055,52.49674749900004],[10.934542500000077,52.471794998000064],[11.0363,52.38520949900004],[11.086244,52.22863399800008],[11.01364650000005,52.19955599900004],[10.964414500000032,52.05664299700004],[10.801428500000043,52.04800049900007],[10.561227,52.004065998000044],[10.648214500000051,51.908245],[10.576971,51.84298749900006],[10.58152,51.78000549700005],[10.701372,51.642187499000045],[10.677283,51.638376],[10.488551,51.57477849700007],[10.365365,51.555891],[9.928339,51.37529899900005],[9.803510499000026,51.41040049800006],[9.702144,51.36457199800003],[9.775202,51.32519599800003],[9.710295479000024,51.30153774900003],[9.568025,51.34000149800005],[9.557291,51.35137849800003],[9.642956001000073,51.41632149900005],[9.585751,51.51731749900006],[9.647755500000073,51.55251099800006],[9.625825001000067,51.580204999000046],[9.672377500000039,51.568403998000065],[9.685331,51.58201649800003],[9.440457,51.65039399800003],[9.417317500000024,51.64726949800007],[9.705215,51.87753199800005],[9.894248,51.90615499900008],[10.065526,51.92735999800004],[10.204688500000032,52.00698999900004],[10.176285500000063,52.14120849600005],[10.234005,52.170195998000054],[10.25707650000004,52.18327799700006],[9.99849850000004,52.23026050000004],[10.034467500000062,52.28376599600006],[10.292147500000056,52.447884498000064],[10.273742500000026,52.510639999000034],[10.410597,52.56412349800007],[10.364143500000068,52.72297549900003],[10.44441,52.81576349900007],[10.759314500000073,52.79583049900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE911","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Braunschweig, Kreisfreie Stadt","NUTS_NAME":"Braunschweig, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.609443500000054,52.34419599900008],[10.642989500000056,52.295423999000036],[10.506921500000033,52.186851999000055],[10.429589500000077,52.22127949900005],[10.424325,52.330227999000044],[10.609443500000054,52.34419599900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE912","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Salzgitter, Kreisfreie Stadt","NUTS_NAME":"Salzgitter, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.506921500000033,52.186851999000055],[10.432407,52.12912549600003],[10.469599,52.045805],[10.286169,52.03382850000003],[10.343103,52.059926500000074],[10.234005,52.170195998000054],[10.25707650000004,52.18327799700006],[10.429589500000077,52.22127949900005],[10.506921500000033,52.186851999000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE913","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wolfsburg, Kreisfreie Stadt","NUTS_NAME":"Wolfsburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.890571,52.46011949700005],[10.887109,52.345375500000046],[10.846469,52.315945999000064],[10.670487500000036,52.37933399900004],[10.646581500000025,52.44375699700004],[10.890571,52.46011949700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE914","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gifhorn","NUTS_NAME":"Gifhorn"},"geometry":{"type":"Polygon","coordinates":[[[10.759314500000073,52.79583049900003],[10.921412,52.610437],[10.976968,52.62414149700004],[10.937393,52.56789999800003],[11.008781500000055,52.49674749900004],[10.934542500000077,52.471794998000064],[10.890571,52.46011949700005],[10.646581500000025,52.44375699700004],[10.670487500000036,52.37933399900004],[10.609443500000054,52.34419599900008],[10.424325,52.330227999000044],[10.292147500000056,52.447884498000064],[10.273742500000026,52.510639999000034],[10.410597,52.56412349800007],[10.364143500000068,52.72297549900003],[10.44441,52.81576349900007],[10.759314500000073,52.79583049900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE915","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Göttingen","NUTS_NAME":"Göttingen"},"geometry":{"type":"Polygon","coordinates":[[[10.160511500000041,51.64639649600008],[10.365365,51.555891],[9.928339,51.37529899900005],[9.803510499000026,51.41040049800006],[9.702144,51.36457199800003],[9.775202,51.32519599800003],[9.710295479000024,51.30153774900003],[9.568025,51.34000149800005],[9.557291,51.35137849800003],[9.642956001000073,51.41632149900005],[9.585751,51.51731749900006],[9.647755500000073,51.55251099800006],[9.625825001000067,51.580204999000046],[9.672377500000039,51.568403998000065],[9.685331,51.58201649800003],[10.160511500000041,51.64639649600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE916","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Goslar","NUTS_NAME":"Goslar"},"geometry":{"type":"Polygon","coordinates":[[[10.561227,52.004065998000044],[10.648214500000051,51.908245],[10.576971,51.84298749900006],[10.58152,51.78000549700005],[10.701372,51.642187499000045],[10.677283,51.638376],[10.504296500000066,51.66318499600004],[10.499275,51.76322049800007],[10.153714500000035,51.793543498000076],[10.065526,51.92735999800004],[10.204688500000032,52.00698999900004],[10.286169,52.03382850000003],[10.469599,52.045805],[10.500743,51.979093496000075],[10.561227,52.004065998000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE917","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Helmstedt","NUTS_NAME":"Helmstedt"},"geometry":{"type":"Polygon","coordinates":[[[10.964414500000032,52.05664299700004],[10.801428500000043,52.04800049900007],[10.885166500000025,52.18007099700003],[10.642989500000056,52.295423999000036],[10.609443500000054,52.34419599900008],[10.670487500000036,52.37933399900004],[10.846469,52.315945999000064],[10.887109,52.345375500000046],[10.890571,52.46011949700005],[10.934542500000077,52.471794998000064],[11.0363,52.38520949900004],[11.086244,52.22863399800008],[11.01364650000005,52.19955599900004],[10.964414500000032,52.05664299700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE918","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Northeim","NUTS_NAME":"Northeim"},"geometry":{"type":"Polygon","coordinates":[[[10.065526,51.92735999800004],[10.153714500000035,51.793543498000076],[10.160511500000041,51.64639649600008],[9.685331,51.58201649800003],[9.440457,51.65039399800003],[9.417317500000024,51.64726949800007],[9.705215,51.87753199800005],[9.894248,51.90615499900008],[10.065526,51.92735999800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE919","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Osterode am Harz","NUTS_NAME":"Osterode am Harz"},"geometry":{"type":"Polygon","coordinates":[[[10.677283,51.638376],[10.488551,51.57477849700007],[10.365365,51.555891],[10.160511500000041,51.64639649600008],[10.153714500000035,51.793543498000076],[10.499275,51.76322049800007],[10.504296500000066,51.66318499600004],[10.677283,51.638376]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE91A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Peine","NUTS_NAME":"Peine"},"geometry":{"type":"Polygon","coordinates":[[[10.424325,52.330227999000044],[10.429589500000077,52.22127949900005],[10.25707650000004,52.18327799700006],[9.99849850000004,52.23026050000004],[10.034467500000062,52.28376599600006],[10.292147500000056,52.447884498000064],[10.424325,52.330227999000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE91B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wolfenbüttel","NUTS_NAME":"Wolfenbüttel"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.561227,52.004065998000044],[10.500743,51.979093496000075],[10.469599,52.045805],[10.432407,52.12912549600003],[10.506921500000033,52.186851999000055],[10.642989500000056,52.295423999000036],[10.885166500000025,52.18007099700003],[10.801428500000043,52.04800049900007],[10.561227,52.004065998000044]]],[[[10.286169,52.03382850000003],[10.204688500000032,52.00698999900004],[10.176285500000063,52.14120849600005],[10.234005,52.170195998000054],[10.343103,52.059926500000074],[10.286169,52.03382850000003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE92","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Hannover","NUTS_NAME":"Hannover"},"geometry":{"type":"Polygon","coordinates":[[[8.915827,53.01102149600007],[8.952130500000067,52.88890849900008],[9.115997,52.896952498000076],[9.325602500000059,52.76771249900003],[9.534658500000035,52.66006649800005],[9.734624,52.63830249800003],[10.273742500000026,52.510639999000034],[10.292147500000056,52.447884498000064],[10.034467500000062,52.28376599600006],[9.99849850000004,52.23026050000004],[10.25707650000004,52.18327799700006],[10.234005,52.170195998000054],[10.176285500000063,52.14120849600005],[10.204688500000032,52.00698999900004],[10.065526,51.92735999800004],[9.894248,51.90615499900008],[9.705215,51.87753199800005],[9.417317500000024,51.64726949800007],[9.374706,51.64984049700007],[9.375365,51.74361399900005],[9.459648500000071,51.86279749700003],[9.323343500000021,51.85506099600008],[9.34515250000004,51.89634],[9.308893,51.92271950000003],[9.18955150000005,51.964566998000066],[9.155604499000049,52.097829999000055],[8.985787500000072,52.19457649800006],[9.075462,52.23411699800005],[8.963536500000032,52.27902699900005],[9.125252500000045,52.411993497000026],[9.131973500000072,52.483982498000046],[9.052655999000024,52.500297998000065],[8.937577,52.40210049800004],[8.71523650000006,52.39256899900005],[8.703008999000076,52.50043799700006],[8.297213500000055,52.45649799700004],[8.308324,52.499111999000036],[8.349646,52.55320599800007],[8.311655500000029,52.687764498000035],[8.472052500000075,52.735567496000044],[8.459277500000042,52.80105599600006],[8.63367,52.85091],[8.694027,52.92014949700007],[8.657161,53.00899449800005],[8.711424,53.044629996000026],[8.915827,53.01102149600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE922","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Diepholz","NUTS_NAME":"Diepholz"},"geometry":{"type":"Polygon","coordinates":[[[8.915827,53.01102149600007],[8.952130500000067,52.88890849900008],[9.115997,52.896952498000076],[9.034512,52.838749],[9.064661,52.76585649800006],[9.01899,52.666056],[8.885044500000049,52.630805499000076],[8.89749850000004,52.55527099600005],[8.703008999000076,52.50043799700006],[8.297213500000055,52.45649799700004],[8.308324,52.499111999000036],[8.349646,52.55320599800007],[8.311655500000029,52.687764498000035],[8.472052500000075,52.735567496000044],[8.459277500000042,52.80105599600006],[8.63367,52.85091],[8.694027,52.92014949700007],[8.657161,53.00899449800005],[8.711424,53.044629996000026],[8.915827,53.01102149600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE923","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hameln-Pyrmont","NUTS_NAME":"Hameln-Pyrmont"},"geometry":{"type":"Polygon","coordinates":[[[9.629650500000025,52.13126349800007],[9.662428,52.08976899900006],[9.647811,51.96479899700006],[9.567893500000025,52.0441],[9.308893,51.92271950000003],[9.18955150000005,51.964566998000066],[9.155604499000049,52.097829999000055],[9.19475649900005,52.14224549700003],[9.157728500000076,52.185339498000076],[9.44830950000005,52.27431999700008],[9.51641150000006,52.212168500000075],[9.49653550000005,52.16629050000006],[9.629650500000025,52.13126349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE925","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hildesheim","NUTS_NAME":"Hildesheim"},"geometry":{"type":"Polygon","coordinates":[[[10.034467500000062,52.28376599600006],[9.99849850000004,52.23026050000004],[10.25707650000004,52.18327799700006],[10.234005,52.170195998000054],[10.176285500000063,52.14120849600005],[10.204688500000032,52.00698999900004],[10.065526,51.92735999800004],[9.894248,51.90615499900008],[9.647811,51.96479899700006],[9.662428,52.08976899900006],[9.629650500000025,52.13126349800007],[9.727796,52.16908699700008],[9.695699500000046,52.197903997000026],[9.783959,52.174311499000055],[9.818020500000046,52.22844199800005],[9.791398,52.25133999800005],[10.034467500000062,52.28376599600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE926","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Holzminden","NUTS_NAME":"Holzminden"},"geometry":{"type":"Polygon","coordinates":[[[9.647811,51.96479899700006],[9.894248,51.90615499900008],[9.705215,51.87753199800005],[9.417317500000024,51.64726949800007],[9.374706,51.64984049700007],[9.375365,51.74361399900005],[9.459648500000071,51.86279749700003],[9.323343500000021,51.85506099600008],[9.34515250000004,51.89634],[9.308893,51.92271950000003],[9.567893500000025,52.0441],[9.647811,51.96479899700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE927","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nienburg (Weser)","NUTS_NAME":"Nienburg (Weser)"},"geometry":{"type":"Polygon","coordinates":[[[9.325602500000059,52.76771249900003],[9.534658500000035,52.66006649800005],[9.238198500000067,52.52969399700004],[9.272908500000028,52.45474099900008],[9.125252500000045,52.411993497000026],[9.131973500000072,52.483982498000046],[9.052655999000024,52.500297998000065],[8.937577,52.40210049800004],[8.71523650000006,52.39256899900005],[8.703008999000076,52.50043799700006],[8.89749850000004,52.55527099600005],[8.885044500000049,52.630805499000076],[9.01899,52.666056],[9.064661,52.76585649800006],[9.034512,52.838749],[9.115997,52.896952498000076],[9.325602500000059,52.76771249900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE928","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schaumburg","NUTS_NAME":"Schaumburg"},"geometry":{"type":"Polygon","coordinates":[[[9.44830950000005,52.27431999700008],[9.157728500000076,52.185339498000076],[9.19475649900005,52.14224549700003],[9.155604499000049,52.097829999000055],[8.985787500000072,52.19457649800006],[9.075462,52.23411699800005],[8.963536500000032,52.27902699900005],[9.125252500000045,52.411993497000026],[9.272908500000028,52.45474099900008],[9.36865350000005,52.435846997000056],[9.324981500000035,52.383283997000035],[9.41102950000004,52.404525],[9.408243500000026,52.29879650000004],[9.44830950000005,52.27431999700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE929","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Region Hannover","NUTS_NAME":"Region Hannover"},"geometry":{"type":"Polygon","coordinates":[[[9.734624,52.63830249800003],[10.273742500000026,52.510639999000034],[10.292147500000056,52.447884498000064],[10.034467500000062,52.28376599600006],[9.791398,52.25133999800005],[9.818020500000046,52.22844199800005],[9.783959,52.174311499000055],[9.695699500000046,52.197903997000026],[9.727796,52.16908699700008],[9.629650500000025,52.13126349800007],[9.49653550000005,52.16629050000006],[9.51641150000006,52.212168500000075],[9.44830950000005,52.27431999700008],[9.408243500000026,52.29879650000004],[9.41102950000004,52.404525],[9.324981500000035,52.383283997000035],[9.36865350000005,52.435846997000056],[9.272908500000028,52.45474099900008],[9.238198500000067,52.52969399700004],[9.534658500000035,52.66006649800005],[9.734624,52.63830249800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE93","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Lüneburg","NUTS_NAME":"Lüneburg"},"geometry":{"type":"Polygon","coordinates":[[[8.614913999000066,53.88211799900006],[9.029917500000067,53.82556499600008],[8.994673500000033,53.87260649600006],[9.02242,53.87951749900003],[9.19975099900006,53.88010449700005],[9.48589,53.70766449700005],[9.550639999000055,53.616034497000044],[9.73010399900005,53.55758099600007],[9.768861,53.50527899700006],[9.916487500000073,53.414067998000064],[10.308312,53.433221497000034],[10.469184,53.38584399900003],[10.595047,53.363927500000045],[10.988000500000055,53.33342649800005],[11.171861500000034,53.15664399700006],[11.265732,53.12197799900008],[11.338867,53.05509899800006],[11.597784500000046,53.03592649800004],[11.51249050000007,53.00744499900003],[11.505027,52.94103249800003],[10.841556,52.85220499900004],[10.759314500000073,52.79583049900003],[10.44441,52.81576349900007],[10.364143500000068,52.72297549900003],[10.410597,52.56412349800007],[10.273742500000026,52.510639999000034],[9.734624,52.63830249800003],[9.534658500000035,52.66006649800005],[9.325602500000059,52.76771249900003],[9.115997,52.896952498000076],[8.952130500000067,52.88890849900008],[8.915827,53.01102149600007],[8.979256,53.04584949900004],[8.947018,53.11610099900008],[8.984185500000024,53.126070998000046],[8.485330500000032,53.22711999900008],[8.499987499000042,53.366847998000026],[8.492652,53.47242],[8.652134001000036,53.516016998000055],[8.650632500000029,53.602565],[8.520410500000025,53.606205],[8.483350501000075,53.69478399800005],[8.614913999000066,53.88211799900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE931","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Celle","NUTS_NAME":"Celle"},"geometry":{"type":"Polygon","coordinates":[[[10.44441,52.81576349900007],[10.364143500000068,52.72297549900003],[10.410597,52.56412349800007],[10.273742500000026,52.510639999000034],[9.734624,52.63830249800003],[9.725827,52.70651799800004],[9.831518,52.746264],[9.864418,52.900866499000074],[9.974777,52.86659899800003],[10.237127,52.93629899800004],[10.44441,52.81576349900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE932","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Cuxhaven","NUTS_NAME":"Cuxhaven"},"geometry":{"type":"Polygon","coordinates":[[[9.029917500000067,53.82556499600008],[9.29858,53.729636],[9.250693500000068,53.706508],[9.278145,53.64351849900004],[9.148606499000039,53.592395500000066],[8.898750501000052,53.51929599700003],[8.963790500000073,53.49452499900008],[8.91071050000005,53.37999999800007],[8.789464999000074,53.38916050000006],[8.771407001000057,53.32143049600006],[8.660872500000039,53.281821498000056],[8.50339150000002,53.293537997000044],[8.499987499000042,53.366847998000026],[8.492652,53.47242],[8.652134001000036,53.516016998000055],[8.650632500000029,53.602565],[8.520410500000025,53.606205],[8.483350501000075,53.69478399800005],[8.614913999000066,53.88211799900006],[9.029917500000067,53.82556499600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE933","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Harburg","NUTS_NAME":"Harburg"},"geometry":{"type":"Polygon","coordinates":[[[10.308312,53.433221497000034],[10.469184,53.38584399900003],[10.225828,53.31360099900007],[10.235776500000043,53.19332449700005],[10.075396500000068,53.12900549900007],[9.6826,53.21948199800005],[9.562704,53.33758799700007],[9.783427,53.44046149800005],[9.768861,53.50527899700006],[9.916487500000073,53.414067998000064],[10.308312,53.433221497000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE934","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lüchow-Dannenberg","NUTS_NAME":"Lüchow-Dannenberg"},"geometry":{"type":"Polygon","coordinates":[[[11.171861500000034,53.15664399700006],[11.265732,53.12197799900008],[11.338867,53.05509899800006],[11.597784500000046,53.03592649800004],[11.51249050000007,53.00744499900003],[11.505027,52.94103249800003],[10.841556,52.85220499900004],[10.804568500000073,52.89740799600003],[10.890487,53.01766749800004],[10.75482450000004,53.12333749900006],[10.907782,53.14917999700003],[10.862362500000074,53.25006699800008],[11.171861500000034,53.15664399700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE935","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lüneburg, Landkreis","NUTS_NAME":"Lüneburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.595047,53.363927500000045],[10.988000500000055,53.33342649800005],[11.171861500000034,53.15664399700006],[10.862362500000074,53.25006699800008],[10.907782,53.14917999700003],[10.75482450000004,53.12333749900006],[10.509027,53.203872499000056],[10.21178750000007,53.06773349900004],[10.042578,53.052297500000066],[10.116814500000032,53.10599499800003],[10.075396500000068,53.12900549900007],[10.235776500000043,53.19332449700005],[10.225828,53.31360099900007],[10.469184,53.38584399900003],[10.595047,53.363927500000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE936","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Osterholz","NUTS_NAME":"Osterholz"},"geometry":{"type":"Polygon","coordinates":[[[8.91071050000005,53.37999999800007],[9.00047,53.31891299600005],[9.071453,53.151297500000055],[8.984185500000024,53.126070998000046],[8.485330500000032,53.22711999900008],[8.499987499000042,53.366847998000026],[8.50339150000002,53.293537997000044],[8.660872500000039,53.281821498000056],[8.771407001000057,53.32143049600006],[8.789464999000074,53.38916050000006],[8.91071050000005,53.37999999800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE937","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rotenburg (Wümme)","NUTS_NAME":"Rotenburg (Wümme)"},"geometry":{"type":"Polygon","coordinates":[[[9.562704,53.33758799700007],[9.6826,53.21948199800005],[9.758328,53.165817497000035],[9.631621500000051,53.12879799900003],[9.607947500000023,53.05447049900005],[9.678386500000045,52.98243099600006],[9.454723,52.91741099600006],[9.43731850000006,53.00281399800008],[9.18778750000007,53.048067496000044],[9.152893500000062,53.103139499000065],[9.18558,53.17840999900005],[9.071453,53.151297500000055],[9.00047,53.31891299600005],[8.91071050000005,53.37999999800007],[8.963790500000073,53.49452499900008],[8.898750501000052,53.51929599700003],[9.148606499000039,53.592395500000066],[9.28285949900004,53.523262497000076],[9.23201750000004,53.47848249900005],[9.359297,53.38732549700006],[9.562704,53.33758799700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE938","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heidekreis","NUTS_NAME":"Heidekreis"},"geometry":{"type":"Polygon","coordinates":[[[10.075396500000068,53.12900549900007],[10.116814500000032,53.10599499800003],[10.042578,53.052297500000066],[10.21178750000007,53.06773349900004],[10.158932,53.013761497000075],[10.237127,52.93629899800004],[9.974777,52.86659899800003],[9.864418,52.900866499000074],[9.831518,52.746264],[9.725827,52.70651799800004],[9.734624,52.63830249800003],[9.534658500000035,52.66006649800005],[9.325602500000059,52.76771249900003],[9.454723,52.91741099600006],[9.678386500000045,52.98243099600006],[9.607947500000023,53.05447049900005],[9.631621500000051,53.12879799900003],[9.758328,53.165817497000035],[9.6826,53.21948199800005],[10.075396500000068,53.12900549900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE939","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Stade","NUTS_NAME":"Stade"},"geometry":{"type":"Polygon","coordinates":[[[9.48589,53.70766449700005],[9.550639999000055,53.616034497000044],[9.73010399900005,53.55758099600007],[9.768861,53.50527899700006],[9.783427,53.44046149800005],[9.562704,53.33758799700007],[9.359297,53.38732549700006],[9.23201750000004,53.47848249900005],[9.28285949900004,53.523262497000076],[9.148606499000039,53.592395500000066],[9.278145,53.64351849900004],[9.250693500000068,53.706508],[9.29858,53.729636],[9.029917500000067,53.82556499600008],[8.994673500000033,53.87260649600006],[9.02242,53.87951749900003],[9.19975099900006,53.88010449700005],[9.48589,53.70766449700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE93A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Uelzen","NUTS_NAME":"Uelzen"},"geometry":{"type":"Polygon","coordinates":[[[10.75482450000004,53.12333749900006],[10.890487,53.01766749800004],[10.804568500000073,52.89740799600003],[10.841556,52.85220499900004],[10.759314500000073,52.79583049900003],[10.44441,52.81576349900007],[10.237127,52.93629899800004],[10.158932,53.013761497000075],[10.21178750000007,53.06773349900004],[10.509027,53.203872499000056],[10.75482450000004,53.12333749900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE93B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Verden","NUTS_NAME":"Verden"},"geometry":{"type":"Polygon","coordinates":[[[9.454723,52.91741099600006],[9.325602500000059,52.76771249900003],[9.115997,52.896952498000076],[8.952130500000067,52.88890849900008],[8.915827,53.01102149600007],[8.979256,53.04584949900004],[8.947018,53.11610099900008],[8.984185500000024,53.126070998000046],[9.071453,53.151297500000055],[9.18558,53.17840999900005],[9.152893500000062,53.103139499000065],[9.18778750000007,53.048067496000044],[9.43731850000006,53.00281399800008],[9.454723,52.91741099600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Weser-Ems","NUTS_NAME":"Weser-Ems"},"geometry":{"type":"Polygon","coordinates":[[[8.061327,53.505957],[8.195555,53.409171498000035],[8.315217500000074,53.46179599800007],[8.303364,53.525487],[8.22693,53.52065249800006],[8.329544,53.61402699900003],[8.554898500000036,53.525131],[8.492652,53.47242],[8.499987499000042,53.366847998000026],[8.485330500000032,53.22711999900008],[8.654898500000058,53.10886449900005],[8.711424,53.044629996000026],[8.657161,53.00899449800005],[8.694027,52.92014949700007],[8.63367,52.85091],[8.459277500000042,52.80105599600006],[8.472052500000075,52.735567496000044],[8.311655500000029,52.687764498000035],[8.349646,52.55320599800007],[8.308324,52.499111999000036],[8.297213500000055,52.45649799700004],[8.44432450000005,52.36115349700003],[8.466186500000049,52.26761399900005],[8.44223050000005,52.213789500000075],[8.517049,52.18512399800005],[8.410586500000022,52.115114996000045],[8.096448,52.057145],[7.885159,52.08330249900007],[8.007796,52.11533299700005],[8.016978,52.17205049900008],[7.902993500000036,52.19886949900007],[7.956465,52.27249049900007],[7.931592,52.30146849600004],[7.990608,52.30953950000003],[7.964625500000068,52.32485850000006],[7.608038500000021,52.47401599800003],[7.563964500000054,52.37065199800003],[7.317481,52.280271998000046],[7.099148999000022,52.24305849600006],[7.065685,52.24137299800003],[7.026305,52.28744399700008],[7.072438,52.37311650000004],[6.987941500000034,52.46954099800007],[6.697865500000034,52.48628599700004],[6.70973250000003,52.627823498000055],[7.006229500000074,52.63876299800006],[7.055577,52.643369998000026],[7.092692,52.83820099800005],[7.209295,53.00024449700004],[7.202794500000039,53.11328149800005],[7.208935,53.243064497000034],[7.192496,53.31762750000007],[7.2643,53.325526497000055],[6.999446500000033,53.35988750000007],[7.024107500000071,53.48237149600004],[7.138518,53.53548099900007],[7.090335500000037,53.57965199900008],[7.243681499000047,53.66778199600003],[7.550173,53.67503749900004],[7.809895500000039,53.70766199800005],[8.017161500000043,53.710496498000055],[8.091291,53.638109],[8.164292999000054,53.52261449800005],[8.061327,53.505957]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE941","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Delmenhorst, Kreisfreie Stadt","NUTS_NAME":"Delmenhorst, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.711424,53.044629996000026],[8.657161,53.00899449800005],[8.578331,53.00014249800006],[8.635673500000053,53.09927549900004],[8.654898500000058,53.10886449900005],[8.711424,53.044629996000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE942","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Emden, Kreisfreie Stadt","NUTS_NAME":"Emden, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.30732,53.35015949900003],[7.2643,53.325526497000055],[6.999446500000033,53.35988750000007],[7.27532750000006,53.406321],[7.30732,53.35015949900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE943","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oldenburg (Oldenburg), Kreisfreie Stadt","NUTS_NAME":"Oldenburg (Oldenburg), Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.303905500000042,53.18908449800006],[8.312264,53.153047498000035],[8.226464500000077,53.082763998000075],[8.128091,53.10783799600006],[8.161447500000065,53.176849],[8.303905500000042,53.18908449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE944","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Osnabrück, Kreisfreie Stadt","NUTS_NAME":"Osnabrück, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.956465,52.27249049900007],[7.931592,52.30146849600004],[7.990608,52.30953950000003],[7.964625500000068,52.32485850000006],[8.180280500000038,52.282305500000064],[7.956465,52.27249049900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE945","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wilhelmshaven, Kreisfreie Stadt","NUTS_NAME":"Wilhelmshaven, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.061327,53.505957],[7.993269,53.60369999800008],[8.091291,53.638109],[8.164292999000054,53.52261449800005],[8.061327,53.505957]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE946","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ammerland","NUTS_NAME":"Ammerland"},"geometry":{"type":"Polygon","coordinates":[[[8.303905500000042,53.18908449800006],[8.161447500000065,53.176849],[8.128091,53.10783799600006],[8.050830500000075,53.059624],[7.719023,53.17888899800005],[7.702119,53.22283950000008],[7.885607500000049,53.35196649900007],[7.970525,53.30469749900004],[8.173923,53.35404099800007],[8.303905500000042,53.18908449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE947","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Aurich","NUTS_NAME":"Aurich"},"geometry":{"type":"Polygon","coordinates":[[[7.421098,53.617601499000045],[7.412109500000042,53.542110999000045],[7.649226,53.561369],[7.677234,53.46293999800008],[7.815484500000025,53.375455500000044],[7.30732,53.35015949900003],[7.27532750000006,53.406321],[6.999446500000033,53.35988750000007],[7.024107500000071,53.48237149600004],[7.138518,53.53548099900007],[7.090335500000037,53.57965199900008],[7.243681499000047,53.66778199600003],[7.550173,53.67503749900004],[7.421098,53.617601499000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE948","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Cloppenburg","NUTS_NAME":"Cloppenburg"},"geometry":{"type":"Polygon","coordinates":[[[7.719023,53.17888899800005],[8.050830500000075,53.059624],[8.124224500000025,52.998951498000054],[8.121675499000048,52.928691997000044],[8.258848500000056,52.874194998000064],[8.200758500000063,52.761566998000035],[8.06117450000005,52.75632899900006],[8.018152500000042,52.683912500000076],[7.689615,52.67892],[7.635240499000076,52.72596899800004],[7.83203450000002,52.920889500000044],[7.61188,53.057707496000035],[7.650209500000074,53.185823500000026],[7.719023,53.17888899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE949","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Emsland","NUTS_NAME":"Emsland"},"geometry":{"type":"Polygon","coordinates":[[[7.61188,53.057707496000035],[7.83203450000002,52.920889500000044],[7.635240499000076,52.72596899800004],[7.689615,52.67892],[7.60749,52.61353349600006],[7.608038500000021,52.47401599800003],[7.563964500000054,52.37065199800003],[7.317481,52.280271998000046],[7.212660500000027,52.44657099900007],[7.284659,52.51772049900006],[7.006229500000074,52.63876299800006],[7.055577,52.643369998000026],[7.092692,52.83820099800005],[7.209295,53.00024449700004],[7.202794500000039,53.11328149800005],[7.61188,53.057707496000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Friesland (DE)","NUTS_NAME":"Friesland (DE)"},"geometry":{"type":"Polygon","coordinates":[[[7.993269,53.60369999800008],[8.061327,53.505957],[8.195555,53.409171498000035],[8.173923,53.35404099800007],[7.970525,53.30469749900004],[7.885607500000049,53.35196649900007],[7.871523,53.37587499700004],[7.975172500000042,53.46109049800003],[7.820136,53.50486199900007],[7.860370500000045,53.61087699600006],[7.809895500000039,53.70766199800005],[8.017161500000043,53.710496498000055],[8.091291,53.638109],[7.993269,53.60369999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Grafschaft Bentheim","NUTS_NAME":"Grafschaft Bentheim"},"geometry":{"type":"Polygon","coordinates":[[[7.006229500000074,52.63876299800006],[7.284659,52.51772049900006],[7.212660500000027,52.44657099900007],[7.317481,52.280271998000046],[7.099148999000022,52.24305849600006],[7.065685,52.24137299800003],[7.026305,52.28744399700008],[7.072438,52.37311650000004],[6.987941500000034,52.46954099800007],[6.697865500000034,52.48628599700004],[6.70973250000003,52.627823498000055],[7.006229500000074,52.63876299800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Leer","NUTS_NAME":"Leer"},"geometry":{"type":"Polygon","coordinates":[[[7.885607500000049,53.35196649900007],[7.702119,53.22283950000008],[7.719023,53.17888899800005],[7.650209500000074,53.185823500000026],[7.61188,53.057707496000035],[7.202794500000039,53.11328149800005],[7.208935,53.243064497000034],[7.192496,53.31762750000007],[7.2643,53.325526497000055],[7.30732,53.35015949900003],[7.815484500000025,53.375455500000044],[7.871523,53.37587499700004],[7.885607500000049,53.35196649900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oldenburg, Landkreis","NUTS_NAME":"Oldenburg, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.635673500000053,53.09927549900004],[8.578331,53.00014249800006],[8.657161,53.00899449800005],[8.694027,52.92014949700007],[8.63367,52.85091],[8.459277500000042,52.80105599600006],[8.258848500000056,52.874194998000064],[8.121675499000048,52.928691997000044],[8.124224500000025,52.998951498000054],[8.050830500000075,53.059624],[8.128091,53.10783799600006],[8.226464500000077,53.082763998000075],[8.312264,53.153047498000035],[8.635673500000053,53.09927549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Osnabrück, Landkreis","NUTS_NAME":"Osnabrück, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.018152500000042,52.683912500000076],[8.080586,52.616925999000046],[8.043814,52.55188099800006],[8.060989,52.443806996000035],[8.308324,52.499111999000036],[8.297213500000055,52.45649799700004],[8.44432450000005,52.36115349700003],[8.466186500000049,52.26761399900005],[8.44223050000005,52.213789500000075],[8.517049,52.18512399800005],[8.410586500000022,52.115114996000045],[8.096448,52.057145],[7.885159,52.08330249900007],[8.007796,52.11533299700005],[8.016978,52.17205049900008],[7.902993500000036,52.19886949900007],[7.956465,52.27249049900007],[8.180280500000038,52.282305500000064],[7.964625500000068,52.32485850000006],[7.608038500000021,52.47401599800003],[7.60749,52.61353349600006],[7.689615,52.67892],[8.018152500000042,52.683912500000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vechta","NUTS_NAME":"Vechta"},"geometry":{"type":"Polygon","coordinates":[[[8.459277500000042,52.80105599600006],[8.472052500000075,52.735567496000044],[8.311655500000029,52.687764498000035],[8.349646,52.55320599800007],[8.308324,52.499111999000036],[8.060989,52.443806996000035],[8.043814,52.55188099800006],[8.080586,52.616925999000046],[8.018152500000042,52.683912500000076],[8.06117450000005,52.75632899900006],[8.200758500000063,52.761566998000035],[8.258848500000056,52.874194998000064],[8.459277500000042,52.80105599600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94G","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wesermarsch","NUTS_NAME":"Wesermarsch"},"geometry":{"type":"Polygon","coordinates":[[[8.554898500000036,53.525131],[8.492652,53.47242],[8.499987499000042,53.366847998000026],[8.485330500000032,53.22711999900008],[8.654898500000058,53.10886449900005],[8.635673500000053,53.09927549900004],[8.312264,53.153047498000035],[8.303905500000042,53.18908449800006],[8.173923,53.35404099800007],[8.195555,53.409171498000035],[8.315217500000074,53.46179599800007],[8.303364,53.525487],[8.22693,53.52065249800006],[8.329544,53.61402699900003],[8.554898500000036,53.525131]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE94H","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wittmund","NUTS_NAME":"Wittmund"},"geometry":{"type":"Polygon","coordinates":[[[7.860370500000045,53.61087699600006],[7.820136,53.50486199900007],[7.975172500000042,53.46109049800003],[7.871523,53.37587499700004],[7.815484500000025,53.375455500000044],[7.677234,53.46293999800008],[7.649226,53.561369],[7.412109500000042,53.542110999000045],[7.421098,53.617601499000045],[7.550173,53.67503749900004],[7.809895500000039,53.70766199800005],[7.860370500000045,53.61087699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"NORDRHEIN-WESTFALEN","NUTS_NAME":"NORDRHEIN-WESTFALEN"},"geometry":{"type":"Polygon","coordinates":[[[8.703008999000076,52.50043799700006],[8.71523650000006,52.39256899900005],[8.937577,52.40210049800004],[9.052655999000024,52.500297998000065],[9.131973500000072,52.483982498000046],[9.125252500000045,52.411993497000026],[8.963536500000032,52.27902699900005],[9.075462,52.23411699800005],[8.985787500000072,52.19457649800006],[9.155604499000049,52.097829999000055],[9.18955150000005,51.964566998000066],[9.308893,51.92271950000003],[9.34515250000004,51.89634],[9.323343500000021,51.85506099600008],[9.459648500000071,51.86279749700003],[9.375365,51.74361399900005],[9.374706,51.64984049700007],[9.417317500000024,51.64726949800007],[9.440457,51.65039399800003],[9.155410500000073,51.44267499800003],[8.970653500000026,51.506773499000076],[8.890376500000059,51.48192849900005],[8.939502,51.38949899800008],[8.68347650000004,51.36217799800005],[8.556348,51.277494999000055],[8.721997500000043,51.273743996000064],[8.758214500000065,51.17718149800004],[8.656632500000057,51.094902998000066],[8.549084999000058,51.101868],[8.477892,50.96904749600003],[8.355495999000027,50.86200099900003],[8.125512,50.78835599800004],[8.166829,50.73571049700007],[8.125776500000029,50.68581399700008],[8.03969,50.69737549800004],[7.963944,50.84603499800005],[7.829263500000025,50.88238649800007],[7.851496,50.92583249900008],[7.785898,50.939913998000065],[7.661003500000049,50.82036450000004],[7.675945,50.77932749800004],[7.440771999000049,50.711437997000075],[7.21239950000006,50.62340499800007],[7.210872,50.64954349800007],[7.195200500000055,50.64272299800007],[6.927901,50.55861849900003],[6.901481500000045,50.46909199900006],[6.759128500000031,50.474589999000045],[6.800711500000034,50.36178149800003],[6.385006,50.384186499000066],[6.425295,50.32301199700004],[6.405028500000071,50.323308498000074],[6.342218500000058,50.38000550000004],[6.374430500000074,50.45226950000006],[6.315556,50.49704249800004],[6.20628750000003,50.521302],[6.192200499000023,50.52105549800007],[6.189312500000028,50.566094],[6.268615500000067,50.625980999000035],[6.020998999000028,50.75429549900008],[6.017985,50.84379649700003],[6.086947500000065,50.91313499800003],[6.018124,50.93459299800003],[6.026435,50.983273997000026],[5.897198,50.974845498000036],[5.877084999000033,51.03210099900008],[6.174812,51.18451349900005],[6.082150999000021,51.17179899900003],[6.072657,51.242587497000045],[6.224405,51.364978998000026],[6.213423,51.50784149800006],[6.091235500000039,51.60590949600004],[6.118209500000034,51.65610299800005],[5.953192,51.74784599700007],[6.063382500000046,51.865443998000046],[6.166407500000048,51.84083449700006],[6.103341500000056,51.89261699800005],[6.167766,51.90080449800007],[6.407779500000061,51.828092],[6.390816,51.873934],[6.72106,51.89648899800005],[6.828513,51.96406649900007],[6.714659,52.03989349600005],[6.760465,52.118569499000046],[7.065685,52.24137299800003],[7.099148999000022,52.24305849600006],[7.317481,52.280271998000046],[7.563964500000054,52.37065199800003],[7.608038500000021,52.47401599800003],[7.964625500000068,52.32485850000006],[7.990608,52.30953950000003],[7.931592,52.30146849600004],[7.956465,52.27249049900007],[7.902993500000036,52.19886949900007],[8.016978,52.17205049900008],[8.007796,52.11533299700005],[7.885159,52.08330249900007],[8.096448,52.057145],[8.410586500000022,52.115114996000045],[8.517049,52.18512399800005],[8.44223050000005,52.213789500000075],[8.466186500000049,52.26761399900005],[8.44432450000005,52.36115349700003],[8.297213500000055,52.45649799700004],[8.703008999000076,52.50043799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Düsseldorf","NUTS_NAME":"Düsseldorf"},"geometry":{"type":"Polygon","coordinates":[[[6.167766,51.90080449800007],[6.407779500000061,51.828092],[6.490212,51.811377],[6.910861500000067,51.74609850000007],[6.940518500000053,51.71546049800003],[6.902276,51.63568450000008],[6.833152,51.57964799700005],[6.928003,51.49776699800003],[6.998652,51.53370049700004],[7.010207,51.53272849800004],[7.104200500000047,51.48126749700003],[7.13603,51.42603799900007],[7.117675,51.38027249800007],[7.167288,51.312902499000074],[7.263617,51.310359497000036],[7.306693500000051,51.23887549800003],[7.295806500000026,51.20539199800004],[7.268179500000031,51.14565749900004],[7.166349999000033,51.15394099900004],[6.997724,51.11797599700003],[6.990593500000045,51.091597998000054],[6.898045,51.064885496000045],[6.853487500000028,51.084256],[6.773632500000076,51.06439],[6.71538350000003,51.015089],[6.480495499000028,51.034178997000026],[6.459210999000049,51.04307299900006],[6.443993500000033,51.08978549700004],[6.291978500000027,51.17545299900007],[6.174812,51.18451349900005],[6.082150999000021,51.17179899900003],[6.072657,51.242587497000045],[6.224405,51.364978998000026],[6.213423,51.50784149800006],[6.091235500000039,51.60590949600004],[6.118209500000034,51.65610299800005],[5.953192,51.74784599700007],[6.063382500000046,51.865443998000046],[6.166407500000048,51.84083449700006],[6.103341500000056,51.89261699800005],[6.167766,51.90080449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA11","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Düsseldorf, Kreisfreie Stadt","NUTS_NAME":"Düsseldorf, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.85604950000004,51.12618849900008],[6.692684500000041,51.22495999700004],[6.714855500000056,51.33319849900005],[6.805203,51.348801998000056],[6.799827,51.27899799800008],[6.939965500000028,51.27284649900008],[6.871618,51.242094499000075],[6.922271500000022,51.193516499000054],[6.85604950000004,51.12618849900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA12","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Duisburg, Kreisfreie Stadt","NUTS_NAME":"Duisburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.78402,51.533232500000054],[6.810042,51.45525549800004],[6.83046,51.352269998000054],[6.805203,51.348801998000056],[6.714855500000056,51.33319849900005],[6.706166,51.33669350000008],[6.635738500000059,51.392964996000046],[6.677939,51.452102],[6.628996,51.50361849800004],[6.78402,51.533232500000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA13","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Essen, Kreisfreie Stadt","NUTS_NAME":"Essen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.010207,51.53272849800004],[7.104200500000047,51.48126749700003],[7.13603,51.42603799900007],[7.117675,51.38027249800007],[6.904033,51.36748249800007],[6.959254500000043,51.437463],[6.899922,51.47128549800004],[6.928003,51.49776699800003],[6.998652,51.53370049700004],[7.010207,51.53272849800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA14","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Krefeld, Kreisfreie Stadt","NUTS_NAME":"Krefeld, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.635738500000059,51.392964996000046],[6.706166,51.33669350000008],[6.585389,51.286496998000075],[6.488729500000034,51.303202498000076],[6.48160450000006,51.39785549900006],[6.537035,51.40630699900004],[6.635738500000059,51.392964996000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA15","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mönchengladbach, Kreisfreie Stadt","NUTS_NAME":"Mönchengladbach, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.522143500000027,51.22572749700004],[6.443993500000033,51.08978549700004],[6.291978500000027,51.17545299900007],[6.522143500000027,51.22572749700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA16","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mülheim an der Ruhr,Kreisfreie Stadt","NUTS_NAME":"Mülheim an der Ruhr,Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.904033,51.36748249800007],[6.83046,51.352269998000054],[6.810042,51.45525549800004],[6.899922,51.47128549800004],[6.959254500000043,51.437463],[6.904033,51.36748249800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA17","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oberhausen, Kreisfreie Stadt","NUTS_NAME":"Oberhausen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.928003,51.49776699800003],[6.899922,51.47128549800004],[6.810042,51.45525549800004],[6.78402,51.533232500000054],[6.833152,51.57964799700005],[6.928003,51.49776699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA18","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Remscheid, Kreisfreie Stadt","NUTS_NAME":"Remscheid, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.295806500000026,51.20539199800004],[7.268179500000031,51.14565749900004],[7.166349999000033,51.15394099900004],[7.137166,51.16551549600007],[7.295806500000026,51.20539199800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA19","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Solingen, Kreisfreie Stadt","NUTS_NAME":"Solingen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.137166,51.16551549600007],[7.166349999000033,51.15394099900004],[6.997724,51.11797599700003],[7.046947,51.207460499000035],[7.137166,51.16551549600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wuppertal, Kreisfreie Stadt","NUTS_NAME":"Wuppertal, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.306693500000051,51.23887549800003],[7.295806500000026,51.20539199800004],[7.137166,51.16551549600007],[7.046947,51.207460499000035],[7.01497,51.24469399800006],[7.167288,51.312902499000074],[7.263617,51.310359497000036],[7.306693500000051,51.23887549800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kleve","NUTS_NAME":"Kleve"},"geometry":{"type":"Polygon","coordinates":[[[6.407779500000061,51.828092],[6.490212,51.811377],[6.489112,51.72040999800004],[6.411264,51.741232998000044],[6.324544500000059,51.636123499000064],[6.498747,51.567061497000054],[6.44835,51.529300498000055],[6.516643500000043,51.42499749900003],[6.224405,51.364978998000026],[6.213423,51.50784149800006],[6.091235500000039,51.60590949600004],[6.118209500000034,51.65610299800005],[5.953192,51.74784599700007],[6.063382500000046,51.865443998000046],[6.166407500000048,51.84083449700006],[6.103341500000056,51.89261699800005],[6.167766,51.90080449800007],[6.407779500000061,51.828092]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mettmann","NUTS_NAME":"Mettmann"},"geometry":{"type":"Polygon","coordinates":[[[7.167288,51.312902499000074],[7.01497,51.24469399800006],[7.046947,51.207460499000035],[6.997724,51.11797599700003],[6.990593500000045,51.091597998000054],[6.898045,51.064885496000045],[6.853487500000028,51.084256],[6.883558500000049,51.10511150000008],[6.85604950000004,51.12618849900008],[6.922271500000022,51.193516499000054],[6.871618,51.242094499000075],[6.939965500000028,51.27284649900008],[6.799827,51.27899799800008],[6.805203,51.348801998000056],[6.83046,51.352269998000054],[6.904033,51.36748249800007],[7.117675,51.38027249800007],[7.167288,51.312902499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Kreis Neuss","NUTS_NAME":"Rhein-Kreis Neuss"},"geometry":{"type":"Polygon","coordinates":[[[6.714855500000056,51.33319849900005],[6.692684500000041,51.22495999700004],[6.85604950000004,51.12618849900008],[6.883558500000049,51.10511150000008],[6.853487500000028,51.084256],[6.773632500000076,51.06439],[6.71538350000003,51.015089],[6.480495499000028,51.034178997000026],[6.459210999000049,51.04307299900006],[6.443993500000033,51.08978549700004],[6.522143500000027,51.22572749700004],[6.625435500000037,51.25075399800005],[6.585389,51.286496998000075],[6.706166,51.33669350000008],[6.714855500000056,51.33319849900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Viersen","NUTS_NAME":"Viersen"},"geometry":{"type":"Polygon","coordinates":[[[6.537035,51.40630699900004],[6.48160450000006,51.39785549900006],[6.488729500000034,51.303202498000076],[6.585389,51.286496998000075],[6.625435500000037,51.25075399800005],[6.522143500000027,51.22572749700004],[6.291978500000027,51.17545299900007],[6.174812,51.18451349900005],[6.082150999000021,51.17179899900003],[6.072657,51.242587497000045],[6.224405,51.364978998000026],[6.516643500000043,51.42499749900003],[6.537035,51.40630699900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA1F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wesel","NUTS_NAME":"Wesel"},"geometry":{"type":"Polygon","coordinates":[[[6.910861500000067,51.74609850000007],[6.940518500000053,51.71546049800003],[6.902276,51.63568450000008],[6.833152,51.57964799700005],[6.78402,51.533232500000054],[6.628996,51.50361849800004],[6.677939,51.452102],[6.635738500000059,51.392964996000046],[6.537035,51.40630699900004],[6.516643500000043,51.42499749900003],[6.44835,51.529300498000055],[6.498747,51.567061497000054],[6.324544500000059,51.636123499000064],[6.411264,51.741232998000044],[6.489112,51.72040999800004],[6.490212,51.811377],[6.910861500000067,51.74609850000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA2","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Köln","NUTS_NAME":"Köln"},"geometry":{"type":"Polygon","coordinates":[[[7.433343,51.21440899700008],[7.497302500000046,51.10619249900003],[7.716120500000045,51.07290849800006],[7.785898,50.939913998000065],[7.661003500000049,50.82036450000004],[7.675945,50.77932749800004],[7.440771999000049,50.711437997000075],[7.21239950000006,50.62340499800007],[7.210872,50.64954349800007],[7.195200500000055,50.64272299800007],[6.927901,50.55861849900003],[6.901481500000045,50.46909199900006],[6.759128500000031,50.474589999000045],[6.800711500000034,50.36178149800003],[6.385006,50.384186499000066],[6.425295,50.32301199700004],[6.405028500000071,50.323308498000074],[6.342218500000058,50.38000550000004],[6.374430500000074,50.45226950000006],[6.315556,50.49704249800004],[6.20628750000003,50.521302],[6.192200499000023,50.52105549800007],[6.189312500000028,50.566094],[6.268615500000067,50.625980999000035],[6.020998999000028,50.75429549900008],[6.017985,50.84379649700003],[6.086947500000065,50.91313499800003],[6.018124,50.93459299800003],[6.026435,50.983273997000026],[5.897198,50.974845498000036],[5.877084999000033,51.03210099900008],[6.174812,51.18451349900005],[6.291978500000027,51.17545299900007],[6.443993500000033,51.08978549700004],[6.459210999000049,51.04307299900006],[6.480495499000028,51.034178997000026],[6.71538350000003,51.015089],[6.773632500000076,51.06439],[6.853487500000028,51.084256],[6.898045,51.064885496000045],[6.990593500000045,51.091597998000054],[6.997724,51.11797599700003],[7.166349999000033,51.15394099900004],[7.268179500000031,51.14565749900004],[7.295806500000026,51.20539199800004],[7.306693500000051,51.23887549800003],[7.433343,51.21440899700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA22","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bonn, Kreisfreie Stadt","NUTS_NAME":"Bonn, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.210872,50.64954349800007],[7.195200500000055,50.64272299800007],[7.026878999000076,50.65495149900005],[7.037018,50.75518599800006],[7.209449500000062,50.73691199700005],[7.168237500000032,50.703489500000046],[7.210872,50.64954349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA23","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Köln, Kreisfreie Stadt","NUTS_NAME":"Köln, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.898045,51.064885496000045],[7.067906,51.01873999900005],[7.161282,50.867294498000035],[6.980936500000041,50.84091599700008],[6.822461500000031,50.927729498000076],[6.84198,51.02465599800007],[6.773632500000076,51.06439],[6.853487500000028,51.084256],[6.898045,51.064885496000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA24","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Leverkusen, Kreisfreie Stadt","NUTS_NAME":"Leverkusen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.067906,51.01873999900005],[6.898045,51.064885496000045],[6.990593500000045,51.091597998000054],[7.116394500000069,51.062832996000054],[7.067906,51.01873999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA26","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Düren","NUTS_NAME":"Düren"},"geometry":{"type":"Polygon","coordinates":[[[6.480495499000028,51.034178997000026],[6.458786,51.00398849800007],[6.519405,50.89580449700003],[6.706284500000038,50.83505399900008],[6.693822500000067,50.75382949700003],[6.590179500000033,50.70861149800004],[6.555215,50.61132949800003],[6.416906499000049,50.60929549800005],[6.284595500000023,50.677666],[6.357402500000035,50.760582999000064],[6.321584499000039,50.868070498000066],[6.230165,50.86345799800006],[6.194494,50.950132996000036],[6.459210999000049,51.04307299900006],[6.480495499000028,51.034178997000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA27","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Erft-Kreis","NUTS_NAME":"Rhein-Erft-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[6.980936500000041,50.84091599700008],[7.030181500000026,50.80066749800005],[6.864216500000055,50.78496049900008],[6.776432,50.718703997000034],[6.693822500000067,50.75382949700003],[6.706284500000038,50.83505399900008],[6.519405,50.89580449700003],[6.458786,51.00398849800007],[6.480495499000028,51.034178997000026],[6.71538350000003,51.015089],[6.773632500000076,51.06439],[6.84198,51.02465599800007],[6.822461500000031,50.927729498000076],[6.980936500000041,50.84091599700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA28","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Euskirchen","NUTS_NAME":"Euskirchen"},"geometry":{"type":"Polygon","coordinates":[[[6.927901,50.55861849900003],[6.901481500000045,50.46909199900006],[6.759128500000031,50.474589999000045],[6.800711500000034,50.36178149800003],[6.385006,50.384186499000066],[6.425295,50.32301199700004],[6.405028500000071,50.323308498000074],[6.342218500000058,50.38000550000004],[6.374430500000074,50.45226950000006],[6.315556,50.49704249800004],[6.416906499000049,50.60929549800005],[6.555215,50.61132949800003],[6.590179500000033,50.70861149800004],[6.693822500000067,50.75382949700003],[6.776432,50.718703997000034],[6.864216500000055,50.78496049900008],[6.915832,50.734131],[6.857046,50.65407399800006],[6.927901,50.55861849900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA29","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Heinsberg","NUTS_NAME":"Heinsberg"},"geometry":{"type":"Polygon","coordinates":[[[6.291978500000027,51.17545299900007],[6.443993500000033,51.08978549700004],[6.459210999000049,51.04307299900006],[6.194494,50.950132996000036],[6.086947500000065,50.91313499800003],[6.018124,50.93459299800003],[6.026435,50.983273997000026],[5.897198,50.974845498000036],[5.877084999000033,51.03210099900008],[6.174812,51.18451349900005],[6.291978500000027,51.17545299900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA2A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Oberbergischer Kreis","NUTS_NAME":"Oberbergischer Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.433343,51.21440899700008],[7.497302500000046,51.10619249900003],[7.716120500000045,51.07290849800006],[7.785898,50.939913998000065],[7.661003500000049,50.82036450000004],[7.398479500000064,50.94975199800007],[7.281371500000034,51.00387599600003],[7.331348499000057,51.061648999000056],[7.268179500000031,51.14565749900004],[7.295806500000026,51.20539199800004],[7.306693500000051,51.23887549800003],[7.433343,51.21440899700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA2B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rheinisch-Bergischer Kreis","NUTS_NAME":"Rheinisch-Bergischer Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.268179500000031,51.14565749900004],[7.331348499000057,51.061648999000056],[7.281371500000034,51.00387599600003],[7.398479500000064,50.94975199800007],[7.161282,50.867294498000035],[7.067906,51.01873999900005],[7.116394500000069,51.062832996000054],[6.990593500000045,51.091597998000054],[6.997724,51.11797599700003],[7.166349999000033,51.15394099900004],[7.268179500000031,51.14565749900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA2C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Sieg-Kreis","NUTS_NAME":"Rhein-Sieg-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.661003500000049,50.82036450000004],[7.675945,50.77932749800004],[7.440771999000049,50.711437997000075],[7.21239950000006,50.62340499800007],[7.210872,50.64954349800007],[7.168237500000032,50.703489500000046],[7.209449500000062,50.73691199700005],[7.037018,50.75518599800006],[7.026878999000076,50.65495149900005],[7.195200500000055,50.64272299800007],[6.927901,50.55861849900003],[6.857046,50.65407399800006],[6.915832,50.734131],[6.864216500000055,50.78496049900008],[7.030181500000026,50.80066749800005],[6.980936500000041,50.84091599700008],[7.161282,50.867294498000035],[7.398479500000064,50.94975199800007],[7.661003500000049,50.82036450000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA2D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Städteregion Aachen","NUTS_NAME":"Städteregion Aachen"},"geometry":{"type":"Polygon","coordinates":[[[6.416906499000049,50.60929549800005],[6.315556,50.49704249800004],[6.20628750000003,50.521302],[6.192200499000023,50.52105549800007],[6.189312500000028,50.566094],[6.268615500000067,50.625980999000035],[6.020998999000028,50.75429549900008],[6.017985,50.84379649700003],[6.086947500000065,50.91313499800003],[6.194494,50.950132996000036],[6.230165,50.86345799800006],[6.321584499000039,50.868070498000066],[6.357402500000035,50.760582999000064],[6.284595500000023,50.677666],[6.416906499000049,50.60929549800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA3","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Münster","NUTS_NAME":"Münster"},"geometry":{"type":"Polygon","coordinates":[[[8.075478,52.01086399800005],[8.320136,51.725705],[8.250911,51.66009249600006],[7.944527500000049,51.700581999000065],[7.735787999000024,51.737370498000075],[7.70188950000005,51.71237899800008],[7.483127500000023,51.729142498000044],[7.40955,51.66458099600004],[7.465229,51.630224998000074],[7.417934,51.599519996000026],[7.314282,51.52242299900007],[7.293587,51.53226249700003],[7.145057,51.55224049800006],[7.13956150000007,51.50615749700006],[7.104200500000047,51.48126749700003],[7.010207,51.53272849800004],[6.998652,51.53370049700004],[6.928003,51.49776699800003],[6.833152,51.57964799700005],[6.902276,51.63568450000008],[6.940518500000053,51.71546049800003],[6.910861500000067,51.74609850000007],[6.490212,51.811377],[6.407779500000061,51.828092],[6.390816,51.873934],[6.72106,51.89648899800005],[6.828513,51.96406649900007],[6.714659,52.03989349600005],[6.760465,52.118569499000046],[7.065685,52.24137299800003],[7.099148999000022,52.24305849600006],[7.317481,52.280271998000046],[7.563964500000054,52.37065199800003],[7.608038500000021,52.47401599800003],[7.964625500000068,52.32485850000006],[7.990608,52.30953950000003],[7.931592,52.30146849600004],[7.956465,52.27249049900007],[7.902993500000036,52.19886949900007],[8.016978,52.17205049900008],[8.007796,52.11533299700005],[7.885159,52.08330249900007],[8.096448,52.057145],[8.075478,52.01086399800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA31","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bottrop, Kreisfreie Stadt","NUTS_NAME":"Bottrop, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.998652,51.53370049700004],[6.928003,51.49776699800003],[6.833152,51.57964799700005],[6.902276,51.63568450000008],[6.979728500000022,51.60576649700005],[6.919661,51.56340049900007],[6.998652,51.53370049700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA32","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gelsenkirchen, Kreisfreie Stadt","NUTS_NAME":"Gelsenkirchen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.145057,51.55224049800006],[7.13956150000007,51.50615749700006],[7.104200500000047,51.48126749700003],[7.010207,51.53272849800004],[6.987663,51.62280949800004],[7.145057,51.55224049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA33","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Münster, Kreisfreie Stadt","NUTS_NAME":"Münster, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.70412250000004,52.03441650000008],[7.773911,51.92028649800005],[7.61322750000005,51.84061349700005],[7.475273500000071,51.93968449600004],[7.51238950000004,51.999670998000056],[7.70412250000004,52.03441650000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA34","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Borken","NUTS_NAME":"Borken"},"geometry":{"type":"Polygon","coordinates":[[[7.267710500000021,52.058392998000045],[7.123316,52.013277997000046],[7.055738,51.88834050000003],[7.172541,51.82149650000008],[6.910861500000067,51.74609850000007],[6.490212,51.811377],[6.407779500000061,51.828092],[6.390816,51.873934],[6.72106,51.89648899800005],[6.828513,51.96406649900007],[6.714659,52.03989349600005],[6.760465,52.118569499000046],[7.065685,52.24137299800003],[7.099148999000022,52.24305849600006],[7.267710500000021,52.058392998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA35","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Coesfeld","NUTS_NAME":"Coesfeld"},"geometry":{"type":"Polygon","coordinates":[[[7.51238950000004,51.999670998000056],[7.475273500000071,51.93968449600004],[7.61322750000005,51.84061349700005],[7.735787999000024,51.737370498000075],[7.70188950000005,51.71237899800008],[7.483127500000023,51.729142498000044],[7.40955,51.66458099600004],[7.172541,51.82149650000008],[7.055738,51.88834050000003],[7.123316,52.013277997000046],[7.267710500000021,52.058392998000045],[7.51238950000004,51.999670998000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA36","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Recklinghausen","NUTS_NAME":"Recklinghausen"},"geometry":{"type":"Polygon","coordinates":[[[7.40955,51.66458099600004],[7.465229,51.630224998000074],[7.417934,51.599519996000026],[7.314282,51.52242299900007],[7.293587,51.53226249700003],[7.145057,51.55224049800006],[6.987663,51.62280949800004],[7.010207,51.53272849800004],[6.998652,51.53370049700004],[6.919661,51.56340049900007],[6.979728500000022,51.60576649700005],[6.902276,51.63568450000008],[6.940518500000053,51.71546049800003],[6.910861500000067,51.74609850000007],[7.172541,51.82149650000008],[7.40955,51.66458099600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA37","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Steinfurt","NUTS_NAME":"Steinfurt"},"geometry":{"type":"Polygon","coordinates":[[[7.964625500000068,52.32485850000006],[7.990608,52.30953950000003],[7.931592,52.30146849600004],[7.956465,52.27249049900007],[7.902993500000036,52.19886949900007],[8.016978,52.17205049900008],[8.007796,52.11533299700005],[7.885159,52.08330249900007],[7.755740500000059,52.098306498000056],[7.70412250000004,52.03441650000008],[7.51238950000004,51.999670998000056],[7.267710500000021,52.058392998000045],[7.099148999000022,52.24305849600006],[7.317481,52.280271998000046],[7.563964500000054,52.37065199800003],[7.608038500000021,52.47401599800003],[7.964625500000068,52.32485850000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA38","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Warendorf","NUTS_NAME":"Warendorf"},"geometry":{"type":"Polygon","coordinates":[[[7.885159,52.08330249900007],[8.096448,52.057145],[8.075478,52.01086399800005],[8.320136,51.725705],[8.250911,51.66009249600006],[7.944527500000049,51.700581999000065],[7.735787999000024,51.737370498000075],[7.61322750000005,51.84061349700005],[7.773911,51.92028649800005],[7.70412250000004,52.03441650000008],[7.755740500000059,52.098306498000056],[7.885159,52.08330249900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA4","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Detmold","NUTS_NAME":"Detmold"},"geometry":{"type":"Polygon","coordinates":[[[8.703008999000076,52.50043799700006],[8.71523650000006,52.39256899900005],[8.937577,52.40210049800004],[9.052655999000024,52.500297998000065],[9.131973500000072,52.483982498000046],[9.125252500000045,52.411993497000026],[8.963536500000032,52.27902699900005],[9.075462,52.23411699800005],[8.985787500000072,52.19457649800006],[9.155604499000049,52.097829999000055],[9.18955150000005,51.964566998000066],[9.308893,51.92271950000003],[9.34515250000004,51.89634],[9.323343500000021,51.85506099600008],[9.459648500000071,51.86279749700003],[9.375365,51.74361399900005],[9.374706,51.64984049700007],[9.417317500000024,51.64726949800007],[9.440457,51.65039399800003],[9.155410500000073,51.44267499800003],[8.970653500000026,51.506773499000076],[8.905395,51.53149649800008],[8.81461,51.539533497000036],[8.834519500000056,51.49470249800004],[8.767671,51.44738599800007],[8.547931,51.46781249900005],[8.472658,51.56338049800007],[8.585998500000073,51.615444497000055],[8.573228500000027,51.66055699900005],[8.403108,51.719355],[8.320136,51.725705],[8.075478,52.01086399800005],[8.096448,52.057145],[8.410586500000022,52.115114996000045],[8.517049,52.18512399800005],[8.44223050000005,52.213789500000075],[8.466186500000049,52.26761399900005],[8.44432450000005,52.36115349700003],[8.297213500000055,52.45649799700004],[8.703008999000076,52.50043799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA41","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bielefeld, Kreisfreie Stadt","NUTS_NAME":"Bielefeld, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.65427950000003,52.060148999000035],[8.60633850000005,51.92737249800007],[8.377677500000061,51.97285199700008],[8.450792500000034,51.988964500000066],[8.471538500000065,52.10074899800003],[8.65427950000003,52.060148999000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA42","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gütersloh","NUTS_NAME":"Gütersloh"},"geometry":{"type":"Polygon","coordinates":[[[8.410586500000022,52.115114996000045],[8.471538500000065,52.10074899800003],[8.450792500000034,51.988964500000066],[8.377677500000061,51.97285199700008],[8.60633850000005,51.92737249800007],[8.748387,51.85867299700004],[8.533772,51.831403],[8.403108,51.719355],[8.320136,51.725705],[8.075478,52.01086399800005],[8.096448,52.057145],[8.410586500000022,52.115114996000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA43","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Herford","NUTS_NAME":"Herford"},"geometry":{"type":"Polygon","coordinates":[[[8.927116,52.184427],[8.891579999000044,52.11188099900005],[8.65427950000003,52.060148999000035],[8.471538500000065,52.10074899800003],[8.410586500000022,52.115114996000045],[8.517049,52.18512399800005],[8.44223050000005,52.213789500000075],[8.466186500000049,52.26761399900005],[8.711542,52.24977549600004],[8.78197350000005,52.162513997000076],[8.927116,52.184427]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA44","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Höxter","NUTS_NAME":"Höxter"},"geometry":{"type":"Polygon","coordinates":[[[9.323343500000021,51.85506099600008],[9.459648500000071,51.86279749700003],[9.375365,51.74361399900005],[9.374706,51.64984049700007],[9.417317500000024,51.64726949800007],[9.440457,51.65039399800003],[9.155410500000073,51.44267499800003],[8.970653500000026,51.506773499000076],[8.905395,51.53149649800008],[9.01716,51.57509299900005],[8.959261500000025,51.78974199700008],[8.952951,51.83657499800006],[9.134004,51.909845],[9.323343500000021,51.85506099600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA45","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lippe","NUTS_NAME":"Lippe"},"geometry":{"type":"Polygon","coordinates":[[[9.155604499000049,52.097829999000055],[9.18955150000005,51.964566998000066],[9.308893,51.92271950000003],[9.34515250000004,51.89634],[9.323343500000021,51.85506099600008],[9.134004,51.909845],[8.952951,51.83657499800006],[8.959261500000025,51.78974199700008],[8.790252,51.79797099900003],[8.748387,51.85867299700004],[8.60633850000005,51.92737249800007],[8.65427950000003,52.060148999000035],[8.891579999000044,52.11188099900005],[8.927116,52.184427],[8.985787500000072,52.19457649800006],[9.155604499000049,52.097829999000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA46","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Minden-Lübbecke","NUTS_NAME":"Minden-Lübbecke"},"geometry":{"type":"Polygon","coordinates":[[[8.703008999000076,52.50043799700006],[8.71523650000006,52.39256899900005],[8.937577,52.40210049800004],[9.052655999000024,52.500297998000065],[9.131973500000072,52.483982498000046],[9.125252500000045,52.411993497000026],[8.963536500000032,52.27902699900005],[9.075462,52.23411699800005],[8.985787500000072,52.19457649800006],[8.927116,52.184427],[8.78197350000005,52.162513997000076],[8.711542,52.24977549600004],[8.466186500000049,52.26761399900005],[8.44432450000005,52.36115349700003],[8.297213500000055,52.45649799700004],[8.703008999000076,52.50043799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA47","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Paderborn","NUTS_NAME":"Paderborn"},"geometry":{"type":"Polygon","coordinates":[[[8.748387,51.85867299700004],[8.790252,51.79797099900003],[8.959261500000025,51.78974199700008],[9.01716,51.57509299900005],[8.905395,51.53149649800008],[8.81461,51.539533497000036],[8.834519500000056,51.49470249800004],[8.767671,51.44738599800007],[8.547931,51.46781249900005],[8.472658,51.56338049800007],[8.585998500000073,51.615444497000055],[8.573228500000027,51.66055699900005],[8.403108,51.719355],[8.533772,51.831403],[8.748387,51.85867299700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA5","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Arnsberg","NUTS_NAME":"Arnsberg"},"geometry":{"type":"Polygon","coordinates":[[[7.735787999000024,51.737370498000075],[7.944527500000049,51.700581999000065],[8.250911,51.66009249600006],[8.320136,51.725705],[8.403108,51.719355],[8.573228500000027,51.66055699900005],[8.585998500000073,51.615444497000055],[8.472658,51.56338049800007],[8.547931,51.46781249900005],[8.767671,51.44738599800007],[8.834519500000056,51.49470249800004],[8.81461,51.539533497000036],[8.905395,51.53149649800008],[8.970653500000026,51.506773499000076],[8.890376500000059,51.48192849900005],[8.939502,51.38949899800008],[8.68347650000004,51.36217799800005],[8.556348,51.277494999000055],[8.721997500000043,51.273743996000064],[8.758214500000065,51.17718149800004],[8.656632500000057,51.094902998000066],[8.549084999000058,51.101868],[8.477892,50.96904749600003],[8.355495999000027,50.86200099900003],[8.125512,50.78835599800004],[8.166829,50.73571049700007],[8.125776500000029,50.68581399700008],[8.03969,50.69737549800004],[7.963944,50.84603499800005],[7.829263500000025,50.88238649800007],[7.851496,50.92583249900008],[7.785898,50.939913998000065],[7.716120500000045,51.07290849800006],[7.497302500000046,51.10619249900003],[7.433343,51.21440899700008],[7.306693500000051,51.23887549800003],[7.263617,51.310359497000036],[7.167288,51.312902499000074],[7.117675,51.38027249800007],[7.13603,51.42603799900007],[7.104200500000047,51.48126749700003],[7.13956150000007,51.50615749700006],[7.145057,51.55224049800006],[7.293587,51.53226249700003],[7.314282,51.52242299900007],[7.417934,51.599519996000026],[7.465229,51.630224998000074],[7.40955,51.66458099600004],[7.483127500000023,51.729142498000044],[7.70188950000005,51.71237899800008],[7.735787999000024,51.737370498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA51","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bochum, Kreisfreie Stadt","NUTS_NAME":"Bochum, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.314282,51.52242299900007],[7.348194,51.48266949600003],[7.252605500000072,51.41333149700006],[7.13603,51.42603799900007],[7.104200500000047,51.48126749700003],[7.13956150000007,51.50615749700006],[7.293587,51.53226249700003],[7.314282,51.52242299900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA52","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dortmund, Kreisfreie Stadt","NUTS_NAME":"Dortmund, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.508453,51.41794099800006],[7.465332500000045,51.416073],[7.348194,51.48266949600003],[7.314282,51.52242299900007],[7.417934,51.599519996000026],[7.633810500000038,51.54553949800004],[7.508453,51.41794099800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA53","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hagen, Kreisfreie Stadt","NUTS_NAME":"Hagen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.58870150000007,51.390975498000046],[7.583192999000062,51.29586499700008],[7.517253,51.264869],[7.376308,51.34341149900007],[7.465332500000045,51.416073],[7.508453,51.41794099800006],[7.58870150000007,51.390975498000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA54","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hamm, Kreisfreie Stadt","NUTS_NAME":"Hamm, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.944527500000049,51.700581999000065],[7.997529,51.674207497000054],[7.82715150000007,51.578628500000036],[7.688047,51.61373750000007],[7.675536499000032,51.66246150000006],[7.728018,51.675288497000054],[7.70188950000005,51.71237899800008],[7.735787999000024,51.737370498000075],[7.944527500000049,51.700581999000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA55","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Herne, Kreisfreie Stadt","NUTS_NAME":"Herne, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.293587,51.53226249700003],[7.13956150000007,51.50615749700006],[7.145057,51.55224049800006],[7.293587,51.53226249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA56","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ennepe-Ruhr-Kreis","NUTS_NAME":"Ennepe-Ruhr-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.465332500000045,51.416073],[7.376308,51.34341149900007],[7.517253,51.264869],[7.433343,51.21440899700008],[7.306693500000051,51.23887549800003],[7.263617,51.310359497000036],[7.167288,51.312902499000074],[7.117675,51.38027249800007],[7.13603,51.42603799900007],[7.252605500000072,51.41333149700006],[7.348194,51.48266949600003],[7.465332500000045,51.416073]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA57","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hochsauerlandkreis","NUTS_NAME":"Hochsauerlandkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.905395,51.53149649800008],[8.970653500000026,51.506773499000076],[8.890376500000059,51.48192849900005],[8.939502,51.38949899800008],[8.68347650000004,51.36217799800005],[8.556348,51.277494999000055],[8.721997500000043,51.273743996000064],[8.758214500000065,51.17718149800004],[8.656632500000057,51.094902998000066],[8.549084999000058,51.101868],[8.238795500000037,51.10433850000004],[8.065788500000053,51.24668199900003],[7.939839,51.23401250000006],[7.863767,51.458848998000065],[8.298522500000047,51.38850599800003],[8.547931,51.46781249900005],[8.767671,51.44738599800007],[8.834519500000056,51.49470249800004],[8.81461,51.539533497000036],[8.905395,51.53149649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA58","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Märkischer Kreis","NUTS_NAME":"Märkischer Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.863767,51.458848998000065],[7.939839,51.23401250000006],[7.939658500000064,51.17833649600004],[7.797824,51.14710849900007],[7.839449500000057,51.12520449800007],[7.817925500000058,51.07498749800004],[7.716120500000045,51.07290849800006],[7.497302500000046,51.10619249900003],[7.433343,51.21440899700008],[7.517253,51.264869],[7.583192999000062,51.29586499700008],[7.58870150000007,51.390975498000046],[7.655632,51.46568699900007],[7.838875500000029,51.478571498000065],[7.863767,51.458848998000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA59","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Olpe","NUTS_NAME":"Olpe"},"geometry":{"type":"Polygon","coordinates":[[[8.238795500000037,51.10433850000004],[8.213782500000036,51.02135749800004],[7.952799,51.02754949700005],[7.924695500000041,50.93545299900006],[7.851496,50.92583249900008],[7.785898,50.939913998000065],[7.716120500000045,51.07290849800006],[7.817925500000058,51.07498749800004],[7.839449500000057,51.12520449800007],[7.797824,51.14710849900007],[7.939658500000064,51.17833649600004],[7.939839,51.23401250000006],[8.065788500000053,51.24668199900003],[8.238795500000037,51.10433850000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA5A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Siegen-Wittgenstein","NUTS_NAME":"Siegen-Wittgenstein"},"geometry":{"type":"Polygon","coordinates":[[[8.549084999000058,51.101868],[8.477892,50.96904749600003],[8.355495999000027,50.86200099900003],[8.125512,50.78835599800004],[8.166829,50.73571049700007],[8.125776500000029,50.68581399700008],[8.03969,50.69737549800004],[7.963944,50.84603499800005],[7.829263500000025,50.88238649800007],[7.851496,50.92583249900008],[7.924695500000041,50.93545299900006],[7.952799,51.02754949700005],[8.213782500000036,51.02135749800004],[8.238795500000037,51.10433850000004],[8.549084999000058,51.101868]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA5B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Soest","NUTS_NAME":"Soest"},"geometry":{"type":"Polygon","coordinates":[[[8.403108,51.719355],[8.573228500000027,51.66055699900005],[8.585998500000073,51.615444497000055],[8.472658,51.56338049800007],[8.547931,51.46781249900005],[8.298522500000047,51.38850599800003],[7.863767,51.458848998000065],[7.838875500000029,51.478571498000065],[7.82715150000007,51.578628500000036],[7.997529,51.674207497000054],[7.944527500000049,51.700581999000065],[8.250911,51.66009249600006],[8.320136,51.725705],[8.403108,51.719355]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEA5C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Unna","NUTS_NAME":"Unna"},"geometry":{"type":"Polygon","coordinates":[[[7.70188950000005,51.71237899800008],[7.728018,51.675288497000054],[7.675536499000032,51.66246150000006],[7.688047,51.61373750000007],[7.82715150000007,51.578628500000036],[7.838875500000029,51.478571498000065],[7.655632,51.46568699900007],[7.58870150000007,51.390975498000046],[7.508453,51.41794099800006],[7.633810500000038,51.54553949800004],[7.417934,51.599519996000026],[7.465229,51.630224998000074],[7.40955,51.66458099600004],[7.483127500000023,51.729142498000044],[7.70188950000005,51.71237899800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"RHEINLAND-PFALZ","NUTS_NAME":"RHEINLAND-PFALZ"},"geometry":{"type":"Polygon","coordinates":[[[8.125776500000029,50.68581399700008],[8.151592,50.59937099800004],[8.11773850000003,50.534509],[7.99285750100006,50.52211150000005],[7.971559500000069,50.40622049800004],[8.121913500000062,50.277224998000065],[7.907826500000056,50.20075650000007],[7.884014,50.16956849800005],[7.925326,50.10884549600007],[7.773997,50.06653999800005],[7.886587500000076,49.971904],[8.175135500000067,50.03425549600007],[8.190038500000071,50.03529599600006],[8.288245,49.995133999000075],[8.343030500000054,49.94050599900004],[8.400055,49.803674999000066],[8.479186,49.764211999000054],[8.448365,49.73366149800006],[8.44637749900005,49.73079949600003],[8.356921,49.69088399800006],[8.414774,49.59505199700004],[8.422439500000053,49.58338549800004],[8.422700500000076,49.57419249800006],[8.423068,49.54182099900004],[8.474739704000058,49.44061602800008],[8.497316,49.411346999000045],[8.470915500000046,49.34071299800007],[8.487268,49.29002649800003],[8.466985500000021,49.28297550000008],[8.413072500000055,49.24981649800003],[8.33997,49.08014999800008],[8.277349,48.989939],[8.261284,48.980916998000055],[8.232632999000032,48.96657150000004],[8.068407500000035,48.99931649700005],[7.910654500000021,49.04516349800008],[7.635651,49.05395049900005],[7.493791,49.16773550000005],[7.36875550000002,49.16145799800006],[7.320341,49.18949899800003],[7.289732,49.23866950000007],[7.394494,49.31635199700003],[7.402075500000024,49.36771850000008],[7.395898500000044,49.372045996000054],[7.292601500000046,49.408222496000064],[7.252589,49.43146550000006],[7.304916500000047,49.474195500000064],[7.27662250000003,49.54862349900003],[7.02798,49.63943849800006],[6.891454,49.613422497000045],[6.606324,49.527990998000064],[6.380052500000033,49.55110499800003],[6.426077500000076,49.663689],[6.507018500000072,49.71264050000008],[6.520996,49.79698549700004],[6.474962500000061,49.821274998000035],[6.235510500000032,49.89918849900005],[6.116835500000036,50.05671099800003],[6.137662500000033,50.129951498000025],[6.176784500000053,50.23586299800007],[6.405028500000071,50.323308498000074],[6.425295,50.32301199700004],[6.385006,50.384186499000066],[6.800711500000034,50.36178149800003],[6.759128500000031,50.474589999000045],[6.901481500000045,50.46909199900006],[6.927901,50.55861849900003],[7.195200500000055,50.64272299800007],[7.210872,50.64954349800007],[7.21239950000006,50.62340499800007],[7.440771999000049,50.711437997000075],[7.675945,50.77932749800004],[7.661003500000049,50.82036450000004],[7.785898,50.939913998000065],[7.851496,50.92583249900008],[7.829263500000025,50.88238649800007],[7.963944,50.84603499800005],[8.03969,50.69737549800004],[8.125776500000029,50.68581399700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB1","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Koblenz","NUTS_NAME":"Koblenz"},"geometry":{"type":"Polygon","coordinates":[[[8.125776500000029,50.68581399700008],[8.151592,50.59937099800004],[8.11773850000003,50.534509],[7.99285750100006,50.52211150000005],[7.971559500000069,50.40622049800004],[8.121913500000062,50.277224998000065],[7.907826500000056,50.20075650000007],[7.884014,50.16956849800005],[7.925326,50.10884549600007],[7.773997,50.06653999800005],[7.764844500000038,50.08259049800006],[7.664822500000071,50.03270099700006],[7.734630500000037,49.998688498000035],[7.963557,49.833101998000075],[7.905437,49.75226349800005],[7.724406500000043,49.72070849700003],[7.70333050000005,49.652514996000036],[7.522752500000024,49.707622498000035],[7.42567850000006,49.653517],[7.439788,49.59398599900004],[7.27662250000003,49.54862349900003],[7.02798,49.63943849800006],[7.044311500000049,49.689301999000065],[7.26378950000003,49.85559449900006],[7.222511,49.88495050000006],[7.254138,49.941236497000034],[7.202684500000032,49.966323],[6.979021,50.09791099800003],[6.958630500000027,50.22632599600007],[7.068778500000064,50.27978549800008],[6.984338499000046,50.34889349800005],[6.917066,50.306880499000044],[6.800711500000034,50.36178149800003],[6.759128500000031,50.474589999000045],[6.901481500000045,50.46909199900006],[6.927901,50.55861849900003],[7.195200500000055,50.64272299800007],[7.210872,50.64954349800007],[7.21239950000006,50.62340499800007],[7.440771999000049,50.711437997000075],[7.675945,50.77932749800004],[7.661003500000049,50.82036450000004],[7.785898,50.939913998000065],[7.851496,50.92583249900008],[7.829263500000025,50.88238649800007],[7.963944,50.84603499800005],[8.03969,50.69737549800004],[8.125776500000029,50.68581399700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB11","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Koblenz, Kreisfreie Stadt","NUTS_NAME":"Koblenz, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.66405450000002,50.38153499800006],[7.688865500000077,50.36012949800005],[7.605484,50.29379399800007],[7.526839,50.288442],[7.486377,50.36328049800005],[7.66405450000002,50.38153499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB12","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ahrweiler","NUTS_NAME":"Ahrweiler"},"geometry":{"type":"Polygon","coordinates":[[[7.21239950000006,50.62340499800007],[7.344739,50.476688498000044],[6.984338499000046,50.34889349800005],[6.917066,50.306880499000044],[6.800711500000034,50.36178149800003],[6.759128500000031,50.474589999000045],[6.901481500000045,50.46909199900006],[6.927901,50.55861849900003],[7.195200500000055,50.64272299800007],[7.210872,50.64954349800007],[7.21239950000006,50.62340499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB13","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Altenkirchen (Westerwald)","NUTS_NAME":"Altenkirchen (Westerwald)"},"geometry":{"type":"Polygon","coordinates":[[[7.851496,50.92583249900008],[7.829263500000025,50.88238649800007],[7.963944,50.84603499800005],[8.03969,50.69737549800004],[7.754625501000021,50.738287497000044],[7.700271001000033,50.69379349800005],[7.700821,50.62291749800005],[7.592425499000058,50.65203549800003],[7.49301,50.57319149800003],[7.440771999000049,50.711437997000075],[7.675945,50.77932749800004],[7.661003500000049,50.82036450000004],[7.785898,50.939913998000065],[7.851496,50.92583249900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB14","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bad Kreuznach","NUTS_NAME":"Bad Kreuznach"},"geometry":{"type":"Polygon","coordinates":[[[7.734630500000037,49.998688498000035],[7.963557,49.833101998000075],[7.905437,49.75226349800005],[7.724406500000043,49.72070849700003],[7.70333050000005,49.652514996000036],[7.522752500000024,49.707622498000035],[7.41005,49.753067498000064],[7.398741500000028,49.856061],[7.734630500000037,49.998688498000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB15","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Birkenfeld","NUTS_NAME":"Birkenfeld"},"geometry":{"type":"Polygon","coordinates":[[[7.398741500000028,49.856061],[7.41005,49.753067498000064],[7.522752500000024,49.707622498000035],[7.42567850000006,49.653517],[7.439788,49.59398599900004],[7.27662250000003,49.54862349900003],[7.02798,49.63943849800006],[7.044311500000049,49.689301999000065],[7.26378950000003,49.85559449900006],[7.222511,49.88495050000006],[7.398741500000028,49.856061]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB16","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Cochem-Zell","NUTS_NAME":"Cochem-Zell"},"geometry":{"type":"Polygon","coordinates":[[[7.381948500000021,50.160750998000026],[7.326468500000033,50.062459998000065],[7.354304,50.01181849800008],[7.202684500000032,49.966323],[6.979021,50.09791099800003],[6.958630500000027,50.22632599600007],[7.068778500000064,50.27978549800008],[7.186137500000029,50.27948149900004],[7.381948500000021,50.160750998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB17","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mayen-Koblenz","NUTS_NAME":"Mayen-Koblenz"},"geometry":{"type":"Polygon","coordinates":[[[7.598346500000048,50.46456449800007],[7.66405450000002,50.38153499800006],[7.486377,50.36328049800005],[7.526839,50.288442],[7.605484,50.29379399800007],[7.635726500000033,50.24560599900008],[7.523485,50.259831],[7.381948500000021,50.160750998000026],[7.186137500000029,50.27948149900004],[7.068778500000064,50.27978549800008],[6.984338499000046,50.34889349800005],[7.344739,50.476688498000044],[7.47946950000005,50.41205999700003],[7.598346500000048,50.46456449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB18","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neuwied","NUTS_NAME":"Neuwied"},"geometry":{"type":"Polygon","coordinates":[[[7.440771999000049,50.711437997000075],[7.49301,50.57319149800003],[7.592425499000058,50.65203549800003],[7.700821,50.62291749800005],[7.66522800100006,50.59458649700008],[7.704229,50.56166399800003],[7.678543001000037,50.496253499000034],[7.598346500000048,50.46456449800007],[7.47946950000005,50.41205999700003],[7.344739,50.476688498000044],[7.21239950000006,50.62340499800007],[7.440771999000049,50.711437997000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB19","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Hunsrück-Kreis","NUTS_NAME":"Rhein-Hunsrück-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.635726500000033,50.24560599900008],[7.578127500000051,50.239937499000064],[7.764844500000038,50.08259049800006],[7.664822500000071,50.03270099700006],[7.734630500000037,49.998688498000035],[7.398741500000028,49.856061],[7.222511,49.88495050000006],[7.254138,49.941236497000034],[7.202684500000032,49.966323],[7.354304,50.01181849800008],[7.326468500000033,50.062459998000065],[7.381948500000021,50.160750998000026],[7.523485,50.259831],[7.635726500000033,50.24560599900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB1A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Lahn-Kreis","NUTS_NAME":"Rhein-Lahn-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.971559500000069,50.40622049800004],[8.121913500000062,50.277224998000065],[7.907826500000056,50.20075650000007],[7.884014,50.16956849800005],[7.925326,50.10884549600007],[7.773997,50.06653999800005],[7.764844500000038,50.08259049800006],[7.578127500000051,50.239937499000064],[7.635726500000033,50.24560599900008],[7.605484,50.29379399800007],[7.688865500000077,50.36012949800005],[7.971559500000069,50.40622049800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB1B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Westerwaldkreis","NUTS_NAME":"Westerwaldkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.03969,50.69737549800004],[8.125776500000029,50.68581399700008],[8.151592,50.59937099800004],[8.11773850000003,50.534509],[7.99285750100006,50.52211150000005],[7.971559500000069,50.40622049800004],[7.688865500000077,50.36012949800005],[7.66405450000002,50.38153499800006],[7.598346500000048,50.46456449800007],[7.678543001000037,50.496253499000034],[7.704229,50.56166399800003],[7.66522800100006,50.59458649700008],[7.700821,50.62291749800005],[7.700271001000033,50.69379349800005],[7.754625501000021,50.738287497000044],[8.03969,50.69737549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB2","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Trier","NUTS_NAME":"Trier"},"geometry":{"type":"Polygon","coordinates":[[[6.800711500000034,50.36178149800003],[6.917066,50.306880499000044],[6.984338499000046,50.34889349800005],[7.068778500000064,50.27978549800008],[6.958630500000027,50.22632599600007],[6.979021,50.09791099800003],[7.202684500000032,49.966323],[7.254138,49.941236497000034],[7.222511,49.88495050000006],[7.26378950000003,49.85559449900006],[7.044311500000049,49.689301999000065],[7.02798,49.63943849800006],[6.891454,49.613422497000045],[6.606324,49.527990998000064],[6.380052500000033,49.55110499800003],[6.426077500000076,49.663689],[6.507018500000072,49.71264050000008],[6.520996,49.79698549700004],[6.474962500000061,49.821274998000035],[6.235510500000032,49.89918849900005],[6.116835500000036,50.05671099800003],[6.137662500000033,50.129951498000025],[6.176784500000053,50.23586299800007],[6.405028500000071,50.323308498000074],[6.425295,50.32301199700004],[6.385006,50.384186499000066],[6.800711500000034,50.36178149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB21","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Trier, Kreisfreie Stadt","NUTS_NAME":"Trier, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[6.712756500000069,49.717308],[6.554235,49.73976449700007],[6.695629,49.85124349800003],[6.746484500000065,49.81346849900007],[6.685564500000055,49.76070049900005],[6.712756500000069,49.717308]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB22","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bernkastel-Wittlich","NUTS_NAME":"Bernkastel-Wittlich"},"geometry":{"type":"Polygon","coordinates":[[[6.979021,50.09791099800003],[7.202684500000032,49.966323],[7.254138,49.941236497000034],[7.222511,49.88495050000006],[7.26378950000003,49.85559449900006],[7.044311500000049,49.689301999000065],[6.978312500000072,49.67763050000008],[6.875374500000021,49.77963499800006],[6.938209500000028,49.826157],[6.711947500000065,49.90481449600003],[6.700651,50.08438549700003],[6.979021,50.09791099800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB23","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Eifelkreis Bitburg-Prüm","NUTS_NAME":"Eifelkreis Bitburg-Prüm"},"geometry":{"type":"Polygon","coordinates":[[[6.425295,50.32301199700004],[6.58690050000007,50.221685999000044],[6.548197,50.14596699700007],[6.584798500000034,50.10921299600005],[6.700651,50.08438549700003],[6.711947500000065,49.90481449600003],[6.474962500000061,49.821274998000035],[6.235510500000032,49.89918849900005],[6.116835500000036,50.05671099800003],[6.137662500000033,50.129951498000025],[6.176784500000053,50.23586299800007],[6.405028500000071,50.323308498000074],[6.425295,50.32301199700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB24","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vulkaneifel","NUTS_NAME":"Vulkaneifel"},"geometry":{"type":"Polygon","coordinates":[[[6.800711500000034,50.36178149800003],[6.917066,50.306880499000044],[6.984338499000046,50.34889349800005],[7.068778500000064,50.27978549800008],[6.958630500000027,50.22632599600007],[6.979021,50.09791099800003],[6.700651,50.08438549700003],[6.584798500000034,50.10921299600005],[6.548197,50.14596699700007],[6.58690050000007,50.221685999000044],[6.425295,50.32301199700004],[6.385006,50.384186499000066],[6.800711500000034,50.36178149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB25","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Trier-Saarburg","NUTS_NAME":"Trier-Saarburg"},"geometry":{"type":"Polygon","coordinates":[[[7.044311500000049,49.689301999000065],[7.02798,49.63943849800006],[6.891454,49.613422497000045],[6.606324,49.527990998000064],[6.380052500000033,49.55110499800003],[6.426077500000076,49.663689],[6.507018500000072,49.71264050000008],[6.520996,49.79698549700004],[6.474962500000061,49.821274998000035],[6.711947500000065,49.90481449600003],[6.938209500000028,49.826157],[6.875374500000021,49.77963499800006],[6.978312500000072,49.67763050000008],[7.044311500000049,49.689301999000065]],[[6.712756500000069,49.717308],[6.685564500000055,49.76070049900005],[6.746484500000065,49.81346849900007],[6.695629,49.85124349800003],[6.554235,49.73976449700007],[6.712756500000069,49.717308]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Rheinhessen-Pfalz","NUTS_NAME":"Rheinhessen-Pfalz"},"geometry":{"type":"Polygon","coordinates":[[[7.773997,50.06653999800005],[7.886587500000076,49.971904],[8.175135500000067,50.03425549600007],[8.190038500000071,50.03529599600006],[8.288245,49.995133999000075],[8.343030500000054,49.94050599900004],[8.400055,49.803674999000066],[8.479186,49.764211999000054],[8.448365,49.73366149800006],[8.44637749900005,49.73079949600003],[8.356921,49.69088399800006],[8.414774,49.59505199700004],[8.422439500000053,49.58338549800004],[8.422700500000076,49.57419249800006],[8.423068,49.54182099900004],[8.474739704000058,49.44061602800008],[8.497316,49.411346999000045],[8.470915500000046,49.34071299800007],[8.487268,49.29002649800003],[8.466985500000021,49.28297550000008],[8.413072500000055,49.24981649800003],[8.33997,49.08014999800008],[8.277349,48.989939],[8.261284,48.980916998000055],[8.232632999000032,48.96657150000004],[8.068407500000035,48.99931649700005],[7.910654500000021,49.04516349800008],[7.635651,49.05395049900005],[7.493791,49.16773550000005],[7.36875550000002,49.16145799800006],[7.320341,49.18949899800003],[7.289732,49.23866950000007],[7.394494,49.31635199700003],[7.402075500000024,49.36771850000008],[7.395898500000044,49.372045996000054],[7.292601500000046,49.408222496000064],[7.252589,49.43146550000006],[7.304916500000047,49.474195500000064],[7.27662250000003,49.54862349900003],[7.439788,49.59398599900004],[7.42567850000006,49.653517],[7.522752500000024,49.707622498000035],[7.70333050000005,49.652514996000036],[7.724406500000043,49.72070849700003],[7.905437,49.75226349800005],[7.963557,49.833101998000075],[7.734630500000037,49.998688498000035],[7.664822500000071,50.03270099700006],[7.764844500000038,50.08259049800006],[7.773997,50.06653999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB31","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Frankenthal (Pfalz), Kreisfreie Stadt","NUTS_NAME":"Frankenthal (Pfalz), Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.423068,49.54182099900004],[8.309558,49.48516149800008],[8.312163500000054,49.549687996000046],[8.422700500000076,49.57419249800006],[8.423068,49.54182099900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB32","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kaiserslautern, Kreisfreie Stadt","NUTS_NAME":"Kaiserslautern, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.848753500000043,49.36220449800004],[7.84051050000005,49.35890499900006],[7.626715999000055,49.433760997000036],[7.719653,49.48947549700006],[7.858871500000021,49.450998999000035],[7.869067,49.40840799800003],[7.815232,49.39570599800004],[7.848753500000043,49.36220449800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB33","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Landau in der Pfalz, Kreisfreie Stadt","NUTS_NAME":"Landau in der Pfalz, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.180823,49.20812749800007],[8.064351,49.17183799900005],[8.05760200100002,49.22185649800008],[8.124960336000072,49.242873545000066],[8.180823,49.20812749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB34","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ludwigshafen am Rhein, Kreisfreie Stadt","NUTS_NAME":"Ludwigshafen am Rhein, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.423068,49.54182099900004],[8.474739704000058,49.44061602800008],[8.309558,49.48516149800008],[8.423068,49.54182099900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB35","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mainz, Kreisfreie Stadt","NUTS_NAME":"Mainz, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.288245,49.995133999000075],[8.343030500000054,49.94050599900004],[8.180697500000065,49.967217],[8.190038500000071,50.03529599600006],[8.288245,49.995133999000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB36","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neustadt an der Weinstraße, Kreisfreie Stadt","NUTS_NAME":"Neustadt an der Weinstraße, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.314431500000069,49.31145149900004],[8.317406,49.308121],[7.999564500000076,49.343793996000045],[8.14813950000007,49.39192699800003],[8.314431500000069,49.31145149900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB37","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Pirmasens, Kreisfreie Stadt","NUTS_NAME":"Pirmasens, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.673350500000026,49.228631998000026],[7.635316500000044,49.16262099700003],[7.514912,49.20042949800006],[7.673350500000026,49.228631998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB38","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Speyer, Kreisfreie Stadt","NUTS_NAME":"Speyer, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.470915500000046,49.34071299800007],[8.487268,49.29002649800003],[8.466985500000021,49.28297550000008],[8.375421500000073,49.341980499000044],[8.470915500000046,49.34071299800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB39","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Worms, Kreisfreie Stadt","NUTS_NAME":"Worms, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.414774,49.59505199700004],[8.281007,49.58676399900003],[8.256195499000057,49.59126050000003],[8.257591,49.68868250000003],[8.44637749900005,49.73079949600003],[8.356921,49.69088399800006],[8.414774,49.59505199700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Zweibrücken, Kreisfreie Stadt","NUTS_NAME":"Zweibrücken, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[7.320341,49.18949899800003],[7.289732,49.23866950000007],[7.394494,49.31635199700003],[7.423524,49.222176999000055],[7.320341,49.18949899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Alzey-Worms","NUTS_NAME":"Alzey-Worms"},"geometry":{"type":"Polygon","coordinates":[[[8.400055,49.803674999000066],[8.479186,49.764211999000054],[8.448365,49.73366149800006],[8.44637749900005,49.73079949600003],[8.257591,49.68868250000003],[8.256195499000057,49.59126050000003],[8.15237650000006,49.62687499800006],[7.905437,49.75226349800005],[7.963557,49.833101998000075],[8.176915500000064,49.90000699800004],[8.23372450000005,49.76846799800006],[8.400055,49.803674999000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bad Dürkheim","NUTS_NAME":"Bad Dürkheim"},"geometry":{"type":"Polygon","coordinates":[[[8.256195499000057,49.59126050000003],[8.281007,49.58676399900003],[8.233288,49.43001449700006],[8.314431500000069,49.31145149900004],[8.14813950000007,49.39192699800003],[7.999564500000076,49.343793996000045],[7.93671850000004,49.30897749800005],[7.86765,49.313322],[7.831963500000029,49.32374999800004],[7.84051050000005,49.35890499900006],[7.848753500000043,49.36220449800004],[7.963398500000039,49.40048199600005],[7.96390849900007,49.48996449900005],[8.15237650000006,49.62687499800006],[8.256195499000057,49.59126050000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Donnersbergkreis","NUTS_NAME":"Donnersbergkreis"},"geometry":{"type":"Polygon","coordinates":[[[7.905437,49.75226349800005],[8.15237650000006,49.62687499800006],[7.96390849900007,49.48996449900005],[7.696266500000036,49.588663498000074],[7.70333050000005,49.652514996000036],[7.724406500000043,49.72070849700003],[7.905437,49.75226349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Germersheim","NUTS_NAME":"Germersheim"},"geometry":{"type":"Polygon","coordinates":[[[8.413072500000055,49.24981649800003],[8.33997,49.08014999800008],[8.277349,48.989939],[8.261284,48.980916998000055],[8.232632999000032,48.96657150000004],[8.068407500000035,48.99931649700005],[8.134272501000055,49.126484496000046],[8.263520500000027,49.12596349800003],[8.216342,49.17106849700008],[8.231770500000039,49.260267498000076],[8.320326,49.29670550000003],[8.413072500000055,49.24981649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kaiserslautern, Landkreis","NUTS_NAME":"Kaiserslautern, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[7.96390849900007,49.48996449900005],[7.963398500000039,49.40048199600005],[7.848753500000043,49.36220449800004],[7.815232,49.39570599800004],[7.869067,49.40840799800003],[7.858871500000021,49.450998999000035],[7.719653,49.48947549700006],[7.626715999000055,49.433760997000036],[7.84051050000005,49.35890499900006],[7.831963500000029,49.32374999800004],[7.402075500000024,49.36771850000008],[7.395898500000044,49.372045996000054],[7.487522,49.49537299800005],[7.696266500000036,49.588663498000074],[7.96390849900007,49.48996449900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3G","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kusel","NUTS_NAME":"Kusel"},"geometry":{"type":"Polygon","coordinates":[[[7.70333050000005,49.652514996000036],[7.696266500000036,49.588663498000074],[7.487522,49.49537299800005],[7.395898500000044,49.372045996000054],[7.292601500000046,49.408222496000064],[7.252589,49.43146550000006],[7.304916500000047,49.474195500000064],[7.27662250000003,49.54862349900003],[7.439788,49.59398599900004],[7.42567850000006,49.653517],[7.522752500000024,49.707622498000035],[7.70333050000005,49.652514996000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3H","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Südliche Weinstraße","NUTS_NAME":"Südliche Weinstraße"},"geometry":{"type":"Polygon","coordinates":[[[7.999564500000076,49.343793996000045],[8.317406,49.308121],[8.320326,49.29670550000003],[8.231770500000039,49.260267498000076],[8.216342,49.17106849700008],[8.263520500000027,49.12596349800003],[8.134272501000055,49.126484496000046],[8.068407500000035,48.99931649700005],[7.910654500000021,49.04516349800008],[7.853279,49.131320998000035],[7.916700001000038,49.17791999800005],[7.849160500000039,49.25741399900005],[7.872069500000066,49.283109997000054],[7.86765,49.313322],[7.93671850000004,49.30897749800005],[7.999564500000076,49.343793996000045]],[[8.05760200100002,49.22185649800008],[8.064351,49.17183799900005],[8.180823,49.20812749800007],[8.124960336000072,49.242873545000066],[8.05760200100002,49.22185649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3I","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rhein-Pfalz-Kreis","NUTS_NAME":"Rhein-Pfalz-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.414774,49.59505199700004],[8.422439500000053,49.58338549800004],[8.422700500000076,49.57419249800006],[8.312163500000054,49.549687996000046],[8.309558,49.48516149800008],[8.474739704000058,49.44061602800008],[8.497316,49.411346999000045],[8.470915500000046,49.34071299800007],[8.375421500000073,49.341980499000044],[8.466985500000021,49.28297550000008],[8.413072500000055,49.24981649800003],[8.320326,49.29670550000003],[8.317406,49.308121],[8.314431500000069,49.31145149900004],[8.233288,49.43001449700006],[8.281007,49.58676399900003],[8.414774,49.59505199700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3J","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mainz-Bingen","NUTS_NAME":"Mainz-Bingen"},"geometry":{"type":"Polygon","coordinates":[[[7.773997,50.06653999800005],[7.886587500000076,49.971904],[8.175135500000067,50.03425549600007],[8.190038500000071,50.03529599600006],[8.180697500000065,49.967217],[8.343030500000054,49.94050599900004],[8.400055,49.803674999000066],[8.23372450000005,49.76846799800006],[8.176915500000064,49.90000699800004],[7.963557,49.833101998000075],[7.734630500000037,49.998688498000035],[7.664822500000071,50.03270099700006],[7.764844500000038,50.08259049800006],[7.773997,50.06653999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEB3K","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Südwestpfalz","NUTS_NAME":"Südwestpfalz"},"geometry":{"type":"Polygon","coordinates":[[[7.831963500000029,49.32374999800004],[7.86765,49.313322],[7.872069500000066,49.283109997000054],[7.849160500000039,49.25741399900005],[7.916700001000038,49.17791999800005],[7.853279,49.131320998000035],[7.910654500000021,49.04516349800008],[7.635651,49.05395049900005],[7.493791,49.16773550000005],[7.36875550000002,49.16145799800006],[7.320341,49.18949899800003],[7.423524,49.222176999000055],[7.394494,49.31635199700003],[7.402075500000024,49.36771850000008],[7.831963500000029,49.32374999800004]],[[7.673350500000026,49.228631998000026],[7.514912,49.20042949800006],[7.635316500000044,49.16262099700003],[7.673350500000026,49.228631998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"SAARLAND","NUTS_NAME":"SAARLAND"},"geometry":{"type":"Polygon","coordinates":[[[7.27662250000003,49.54862349900003],[7.304916500000047,49.474195500000064],[7.252589,49.43146550000006],[7.292601500000046,49.408222496000064],[7.395898500000044,49.372045996000054],[7.402075500000024,49.36771850000008],[7.394494,49.31635199700003],[7.289732,49.23866950000007],[7.320341,49.18949899800003],[7.36875550000002,49.16145799800006],[7.293177,49.11490749700005],[7.101069,49.15599799900008],[7.057195999000044,49.11193149800005],[7.03497150000004,49.19141499700004],[6.925060500000029,49.22334649800007],[6.837607499000057,49.21293299700005],[6.834567,49.15131099800004],[6.738397500000076,49.16359799900005],[6.723465500000032,49.21882899800005],[6.565664,49.34765149700007],[6.556986,49.419208498000046],[6.367107500000031,49.469507],[6.380052500000033,49.55110499800003],[6.606324,49.527990998000064],[6.891454,49.613422497000045],[7.02798,49.63943849800006],[7.27662250000003,49.54862349900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE40I","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Uckermark","NUTS_NAME":"Uckermark"},"geometry":{"type":"Polygon","coordinates":[[[14.412157,53.32963599700008],[14.441223,53.25435999900003],[14.377268,53.20175199800008],[14.357842,53.06766699800005],[14.143658,52.96136849900006],[13.823976,52.98811050000006],[13.885174,53.05230799800006],[13.517898,52.988665998000045],[13.406759500000021,53.02526949600008],[13.241403500000047,53.23240099900005],[13.483196,53.29174199900007],[13.551025,53.399436499000046],[13.710065500000042,53.479062499000065],[13.827101500000026,53.498713500000065],[13.778238499000054,53.51343499800004],[13.798463499000036,53.54568949800006],[13.943100500000071,53.42991649900006],[14.241636500000027,53.42215199700007],[14.098893500000031,53.26195149800003],[14.412157,53.32963599700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE5","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"BREMEN","NUTS_NAME":"BREMEN"},"geometry":{"type":"MultiPolygon","coordinates":[[[[8.979256,53.04584949900004],[8.915827,53.01102149600007],[8.711424,53.044629996000026],[8.654898500000058,53.10886449900005],[8.485330500000032,53.22711999900008],[8.984185500000024,53.126070998000046],[8.947018,53.11610099900008],[8.979256,53.04584949900004]]],[[[8.652134001000036,53.516016998000055],[8.492652,53.47242],[8.554898500000036,53.525131],[8.483202500000061,53.600498997000045],[8.520410500000025,53.606205],[8.650632500000029,53.602565],[8.652134001000036,53.516016998000055]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE50","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Bremen","NUTS_NAME":"Bremen"},"geometry":{"type":"MultiPolygon","coordinates":[[[[8.979256,53.04584949900004],[8.915827,53.01102149600007],[8.711424,53.044629996000026],[8.654898500000058,53.10886449900005],[8.485330500000032,53.22711999900008],[8.984185500000024,53.126070998000046],[8.947018,53.11610099900008],[8.979256,53.04584949900004]]],[[[8.652134001000036,53.516016998000055],[8.492652,53.47242],[8.554898500000036,53.525131],[8.483202500000061,53.600498997000045],[8.520410500000025,53.606205],[8.650632500000029,53.602565],[8.652134001000036,53.516016998000055]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE501","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bremen, Kreisfreie Stadt","NUTS_NAME":"Bremen, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.979256,53.04584949900004],[8.915827,53.01102149600007],[8.711424,53.044629996000026],[8.654898500000058,53.10886449900005],[8.485330500000032,53.22711999900008],[8.984185500000024,53.126070998000046],[8.947018,53.11610099900008],[8.979256,53.04584949900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE502","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bremerhaven, Kreisfreie Stadt","NUTS_NAME":"Bremerhaven, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.652134001000036,53.516016998000055],[8.492652,53.47242],[8.554898500000036,53.525131],[8.483202500000061,53.600498997000045],[8.520410500000025,53.606205],[8.650632500000029,53.602565],[8.652134001000036,53.516016998000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE6","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"HAMBURG","NUTS_NAME":"HAMBURG"},"geometry":{"type":"Polygon","coordinates":[[[10.236678500000039,53.49635449700003],[10.308312,53.433221497000034],[9.916487500000073,53.414067998000064],[9.768861,53.50527899700006],[9.73010399900005,53.55758099600007],[9.770294,53.62746249600008],[9.824958,53.584528],[9.945376,53.65292799700006],[10.072805,53.70963399800007],[10.191279,53.72218649800004],[10.140273,53.68012649800005],[10.22176150000007,53.633722499000044],[10.151152500000023,53.545018],[10.236678500000039,53.49635449700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE60","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Hamburg","NUTS_NAME":"Hamburg"},"geometry":{"type":"Polygon","coordinates":[[[10.236678500000039,53.49635449700003],[10.308312,53.433221497000034],[9.916487500000073,53.414067998000064],[9.768861,53.50527899700006],[9.73010399900005,53.55758099600007],[9.770294,53.62746249600008],[9.824958,53.584528],[9.945376,53.65292799700006],[10.072805,53.70963399800007],[10.191279,53.72218649800004],[10.140273,53.68012649800005],[10.22176150000007,53.633722499000044],[10.151152500000023,53.545018],[10.236678500000039,53.49635449700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE600","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hamburg","NUTS_NAME":"Hamburg"},"geometry":{"type":"Polygon","coordinates":[[[10.236678500000039,53.49635449700003],[10.308312,53.433221497000034],[9.916487500000073,53.414067998000064],[9.768861,53.50527899700006],[9.73010399900005,53.55758099600007],[9.770294,53.62746249600008],[9.824958,53.584528],[9.945376,53.65292799700006],[10.072805,53.70963399800007],[10.191279,53.72218649800004],[10.140273,53.68012649800005],[10.22176150000007,53.633722499000044],[10.151152500000023,53.545018],[10.236678500000039,53.49635449700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE7","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"HESSEN","NUTS_NAME":"HESSEN"},"geometry":{"type":"Polygon","coordinates":[[[9.685331,51.58201649800003],[9.672377500000039,51.568403998000065],[9.625825001000067,51.580204999000046],[9.647755500000073,51.55251099800006],[9.585751,51.51731749900006],[9.642956001000073,51.41632149900005],[9.557291,51.35137849800003],[9.568025,51.34000149800005],[9.710295479000024,51.30153774900003],[9.775202,51.32519599800003],[9.702144,51.36457199800003],[9.803510499000026,51.41040049800006],[9.928339,51.37529899900005],[10.067095,51.22779299800004],[10.206942,51.19064899600005],[10.235999,51.18476749800004],[10.210013,51.144082998000044],[10.118321500000036,51.13957299900005],[10.202181,51.012009],[10.182348500000046,50.99848749900008],[10.021558500000026,50.992949499000076],[10.063865,50.94543299600008],[9.949583500000074,50.927478499000074],[10.055980999000042,50.879771497000036],[9.926176500000054,50.767389998000056],[9.878551,50.63478099800005],[10.077555500000074,50.63762399600006],[10.037268,50.61088399800008],[10.041338500000052,50.51646949800005],[9.935655,50.419606499000054],[9.771871,50.424859498000046],[9.73291450000005,50.356149498000036],[9.745881,50.31040799900006],[9.623150999000075,50.22903999700003],[9.494769500000075,50.22667199600005],[9.519177500000069,50.10585],[9.404984500000069,50.08773449700004],[9.235982500000034,50.146280499000056],[8.990559999000027,50.067119],[9.048820500000033,49.99226549900004],[9.016088500000023,49.991340500000035],[9.05008,49.866315],[9.036080500000025,49.84650399700007],[9.150809,49.742850496000074],[9.088071,49.689591998000026],[9.103006,49.57745599900005],[9.126459500000067,49.52160699900003],[9.08342650000003,49.52608699600006],[8.93188,49.47063699600005],[8.950348500000075,49.45499299800008],[8.801542,49.410945498000046],[8.899355,49.48454999900008],[8.899572500000033,49.50365549800006],[8.70265,49.53532799900006],[8.693837,49.619353498000066],[8.589315500000055,49.604514996000034],[8.62129950000002,49.54500999900006],[8.581375,49.519780498000046],[8.422439500000053,49.58338549800004],[8.414774,49.59505199700004],[8.356921,49.69088399800006],[8.44637749900005,49.73079949600003],[8.448365,49.73366149800006],[8.479186,49.764211999000054],[8.400055,49.803674999000066],[8.343030500000054,49.94050599900004],[8.288245,49.995133999000075],[8.190038500000071,50.03529599600006],[8.175135500000067,50.03425549600007],[7.886587500000076,49.971904],[7.773997,50.06653999800005],[7.925326,50.10884549600007],[7.884014,50.16956849800005],[7.907826500000056,50.20075650000007],[8.121913500000062,50.277224998000065],[7.971559500000069,50.40622049800004],[7.99285750100006,50.52211150000005],[8.11773850000003,50.534509],[8.151592,50.59937099800004],[8.125776500000029,50.68581399700008],[8.166829,50.73571049700007],[8.125512,50.78835599800004],[8.355495999000027,50.86200099900003],[8.477892,50.96904749600003],[8.549084999000058,51.101868],[8.656632500000057,51.094902998000066],[8.758214500000065,51.17718149800004],[8.721997500000043,51.273743996000064],[8.556348,51.277494999000055],[8.68347650000004,51.36217799800005],[8.939502,51.38949899800008],[8.890376500000059,51.48192849900005],[8.970653500000026,51.506773499000076],[9.155410500000073,51.44267499800003],[9.440457,51.65039399800003],[9.685331,51.58201649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Darmstadt","NUTS_NAME":"Darmstadt"},"geometry":{"type":"Polygon","coordinates":[[[9.087218499000073,50.438608],[9.286735,50.437297999000066],[9.335956499000076,50.451862],[9.349258,50.385486497000045],[9.479113,50.44068549600007],[9.73291450000005,50.356149498000036],[9.745881,50.31040799900006],[9.623150999000075,50.22903999700003],[9.494769500000075,50.22667199600005],[9.519177500000069,50.10585],[9.404984500000069,50.08773449700004],[9.235982500000034,50.146280499000056],[8.990559999000027,50.067119],[9.048820500000033,49.99226549900004],[9.016088500000023,49.991340500000035],[9.05008,49.866315],[9.036080500000025,49.84650399700007],[9.150809,49.742850496000074],[9.088071,49.689591998000026],[9.103006,49.57745599900005],[9.126459500000067,49.52160699900003],[9.08342650000003,49.52608699600006],[8.93188,49.47063699600005],[8.950348500000075,49.45499299800008],[8.801542,49.410945498000046],[8.899355,49.48454999900008],[8.899572500000033,49.50365549800006],[8.70265,49.53532799900006],[8.693837,49.619353498000066],[8.589315500000055,49.604514996000034],[8.62129950000002,49.54500999900006],[8.581375,49.519780498000046],[8.422439500000053,49.58338549800004],[8.414774,49.59505199700004],[8.356921,49.69088399800006],[8.44637749900005,49.73079949600003],[8.448365,49.73366149800006],[8.479186,49.764211999000054],[8.400055,49.803674999000066],[8.343030500000054,49.94050599900004],[8.288245,49.995133999000075],[8.190038500000071,50.03529599600006],[8.175135500000067,50.03425549600007],[7.886587500000076,49.971904],[7.773997,50.06653999800005],[7.925326,50.10884549600007],[7.884014,50.16956849800005],[7.907826500000056,50.20075650000007],[8.121913500000062,50.277224998000065],[8.354557,50.29422999800005],[8.31519350000002,50.34037599700008],[8.467215,50.41504849800003],[8.514139500000056,50.399351498000044],[8.577542,50.41501899900004],[8.648858,50.48368249600003],[8.900406500000031,50.42707899700008],[9.042425500000036,50.497989999000026],[9.087218499000073,50.438608]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE711","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Darmstadt, Kreisfreie Stadt","NUTS_NAME":"Darmstadt, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.6745,49.95145249700005],[8.725707500000055,49.95420799800007],[8.749072,49.90167050000008],[8.676616999000032,49.805559996000056],[8.595571,49.837979500000074],[8.627494500000068,49.94216049700003],[8.6745,49.95145249700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE712","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Frankfurt am Main, Kreisfreie Stadt","NUTS_NAME":"Frankfurt am Main, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.789865,50.176048500000036],[8.780646500000046,50.13673449700008],[8.717834,50.06240599900008],[8.591321,50.01769199600005],[8.520716,50.061707],[8.475976500000058,50.10875199700007],[8.590239,50.15954449800006],[8.711376,50.22625749900004],[8.722785500000043,50.16361499700008],[8.789865,50.176048500000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE713","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Offenbach am Main, Kreisfreie Stadt","NUTS_NAME":"Offenbach am Main, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.817965,50.13350949800008],[8.836789999000075,50.07084199800005],[8.717834,50.06240599900008],[8.780646500000046,50.13673449700008],[8.817965,50.13350949800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE714","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wiesbaden, Kreisfreie Stadt","NUTS_NAME":"Wiesbaden, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[8.327649,50.145886500000074],[8.385857,50.08216999800004],[8.33064,50.000539],[8.288245,49.995133999000075],[8.190038500000071,50.03529599600006],[8.175135500000067,50.03425549600007],[8.11233499900004,50.09374849900007],[8.327649,50.145886500000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE715","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bergstraße","NUTS_NAME":"Bergstraße"},"geometry":{"type":"Polygon","coordinates":[[[8.702791,49.753915997000036],[8.797329,49.73829399700003],[8.886963500000036,49.656060497000055],[8.899572500000033,49.50365549800006],[8.70265,49.53532799900006],[8.693837,49.619353498000066],[8.589315500000055,49.604514996000034],[8.62129950000002,49.54500999900006],[8.581375,49.519780498000046],[8.422439500000053,49.58338549800004],[8.414774,49.59505199700004],[8.356921,49.69088399800006],[8.44637749900005,49.73079949600003],[8.448365,49.73366149800006],[8.55927,49.72879199800008],[8.702791,49.753915997000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE716","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Darmstadt-Dieburg","NUTS_NAME":"Darmstadt-Dieburg"},"geometry":{"type":"Polygon","coordinates":[[[9.05008,49.866315],[9.036080500000025,49.84650399700007],[8.797329,49.73829399700003],[8.702791,49.753915997000036],[8.55927,49.72879199800008],[8.521149,49.87114249800004],[8.620113500000059,49.96548399800008],[8.6745,49.95145249700005],[8.627494500000068,49.94216049700003],[8.595571,49.837979500000074],[8.676616999000032,49.805559996000056],[8.749072,49.90167050000008],[8.725707500000055,49.95420799800007],[9.016088500000023,49.991340500000035],[9.05008,49.866315]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE717","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Groß-Gerau","NUTS_NAME":"Groß-Gerau"},"geometry":{"type":"Polygon","coordinates":[[[8.591321,50.01769199600005],[8.620113500000059,49.96548399800008],[8.521149,49.87114249800004],[8.55927,49.72879199800008],[8.448365,49.73366149800006],[8.479186,49.764211999000054],[8.400055,49.803674999000066],[8.343030500000054,49.94050599900004],[8.288245,49.995133999000075],[8.33064,50.000539],[8.520716,50.061707],[8.591321,50.01769199600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE718","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hochtaunuskreis","NUTS_NAME":"Hochtaunuskreis"},"geometry":{"type":"Polygon","coordinates":[[[8.514139500000056,50.399351498000044],[8.609996,50.372659998000074],[8.711376,50.22625749900004],[8.590239,50.15954449800006],[8.353887499000052,50.20100949700003],[8.407287,50.22286249800004],[8.354557,50.29422999800005],[8.31519350000002,50.34037599700008],[8.467215,50.41504849800003],[8.514139500000056,50.399351498000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE719","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Main-Kinzig-Kreis","NUTS_NAME":"Main-Kinzig-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[9.479113,50.44068549600007],[9.73291450000005,50.356149498000036],[9.745881,50.31040799900006],[9.623150999000075,50.22903999700003],[9.494769500000075,50.22667199600005],[9.519177500000069,50.10585],[9.404984500000069,50.08773449700004],[9.235982500000034,50.146280499000056],[8.990559999000027,50.067119],[8.817965,50.13350949800008],[8.780646500000046,50.13673449700008],[8.789865,50.176048500000036],[8.867668,50.28107599600003],[9.149988500000063,50.24766050000005],[9.286735,50.437297999000066],[9.335956499000076,50.451862],[9.349258,50.385486497000045],[9.479113,50.44068549600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Main-Taunus-Kreis","NUTS_NAME":"Main-Taunus-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.590239,50.15954449800006],[8.475976500000058,50.10875199700007],[8.520716,50.061707],[8.33064,50.000539],[8.385857,50.08216999800004],[8.327649,50.145886500000074],[8.353887499000052,50.20100949700003],[8.590239,50.15954449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Odenwaldkreis","NUTS_NAME":"Odenwaldkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.103006,49.57745599900005],[9.126459500000067,49.52160699900003],[9.08342650000003,49.52608699600006],[8.93188,49.47063699600005],[8.950348500000075,49.45499299800008],[8.801542,49.410945498000046],[8.899355,49.48454999900008],[8.899572500000033,49.50365549800006],[8.886963500000036,49.656060497000055],[8.797329,49.73829399700003],[9.036080500000025,49.84650399700007],[9.150809,49.742850496000074],[9.088071,49.689591998000026],[9.103006,49.57745599900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Offenbach, Landkreis","NUTS_NAME":"Offenbach, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[8.990559999000027,50.067119],[9.048820500000033,49.99226549900004],[9.016088500000023,49.991340500000035],[8.725707500000055,49.95420799800007],[8.6745,49.95145249700005],[8.620113500000059,49.96548399800008],[8.591321,50.01769199600005],[8.717834,50.06240599900008],[8.836789999000075,50.07084199800005],[8.817965,50.13350949800008],[8.990559999000027,50.067119]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rheingau-Taunus-Kreis","NUTS_NAME":"Rheingau-Taunus-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.353887499000052,50.20100949700003],[8.327649,50.145886500000074],[8.11233499900004,50.09374849900007],[8.175135500000067,50.03425549600007],[7.886587500000076,49.971904],[7.773997,50.06653999800005],[7.925326,50.10884549600007],[7.884014,50.16956849800005],[7.907826500000056,50.20075650000007],[8.121913500000062,50.277224998000065],[8.354557,50.29422999800005],[8.407287,50.22286249800004],[8.353887499000052,50.20100949700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE71E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wetteraukreis","NUTS_NAME":"Wetteraukreis"},"geometry":{"type":"Polygon","coordinates":[[[9.286735,50.437297999000066],[9.149988500000063,50.24766050000005],[8.867668,50.28107599600003],[8.789865,50.176048500000036],[8.722785500000043,50.16361499700008],[8.711376,50.22625749900004],[8.609996,50.372659998000074],[8.514139500000056,50.399351498000044],[8.577542,50.41501899900004],[8.648858,50.48368249600003],[8.900406500000031,50.42707899700008],[9.042425500000036,50.497989999000026],[9.087218499000073,50.438608],[9.286735,50.437297999000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE72","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Gießen","NUTS_NAME":"Gießen"},"geometry":{"type":"Polygon","coordinates":[[[9.611464500000068,50.73547649900007],[9.641732500000046,50.62778949900007],[9.607138500000076,50.60560249900004],[9.466069,50.66061799900007],[9.482294,50.592301500000076],[9.42753250000004,50.478712],[9.479113,50.44068549600007],[9.349258,50.385486497000045],[9.335956499000076,50.451862],[9.286735,50.437297999000066],[9.087218499000073,50.438608],[9.042425500000036,50.497989999000026],[8.900406500000031,50.42707899700008],[8.648858,50.48368249600003],[8.577542,50.41501899900004],[8.514139500000056,50.399351498000044],[8.467215,50.41504849800003],[8.31519350000002,50.34037599700008],[8.354557,50.29422999800005],[8.121913500000062,50.277224998000065],[7.971559500000069,50.40622049800004],[7.99285750100006,50.52211150000005],[8.11773850000003,50.534509],[8.151592,50.59937099800004],[8.125776500000029,50.68581399700008],[8.166829,50.73571049700007],[8.125512,50.78835599800004],[8.355495999000027,50.86200099900003],[8.477892,50.96904749600003],[8.9746,50.938304],[9.109958,50.91897749800006],[9.148868500000049,50.83664499900004],[9.441055,50.79564450000004],[9.611464500000068,50.73547649900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE721","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gießen, Landkreis","NUTS_NAME":"Gießen, Landkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.042425500000036,50.497989999000026],[8.900406500000031,50.42707899700008],[8.648858,50.48368249600003],[8.577542,50.41501899900004],[8.529493,50.46673449800005],[8.650807,50.51510699800008],[8.55652850000007,50.53579749800008],[8.618906500000037,50.565011997000056],[8.519074,50.63034399700007],[8.573193,50.69925899900005],[8.918371999000044,50.705605999000056],[9.134108,50.556986496000036],[9.042425500000036,50.497989999000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE722","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lahn-Dill-Kreis","NUTS_NAME":"Lahn-Dill-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[8.355495999000027,50.86200099900003],[8.459317500000054,50.785211497000034],[8.455048,50.72202099700007],[8.573193,50.69925899900005],[8.519074,50.63034399700007],[8.618906500000037,50.565011997000056],[8.55652850000007,50.53579749800008],[8.650807,50.51510699800008],[8.529493,50.46673449800005],[8.577542,50.41501899900004],[8.514139500000056,50.399351498000044],[8.467215,50.41504849800003],[8.151592,50.59937099800004],[8.125776500000029,50.68581399700008],[8.166829,50.73571049700007],[8.125512,50.78835599800004],[8.355495999000027,50.86200099900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE723","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Limburg-Weilburg","NUTS_NAME":"Limburg-Weilburg"},"geometry":{"type":"Polygon","coordinates":[[[8.467215,50.41504849800003],[8.31519350000002,50.34037599700008],[8.354557,50.29422999800005],[8.121913500000062,50.277224998000065],[7.971559500000069,50.40622049800004],[7.99285750100006,50.52211150000005],[8.11773850000003,50.534509],[8.151592,50.59937099800004],[8.467215,50.41504849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE724","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Marburg-Biedenkopf","NUTS_NAME":"Marburg-Biedenkopf"},"geometry":{"type":"Polygon","coordinates":[[[8.9746,50.938304],[9.109958,50.91897749800006],[9.148868500000049,50.83664499900004],[8.918371999000044,50.705605999000056],[8.573193,50.69925899900005],[8.455048,50.72202099700007],[8.459317500000054,50.785211497000034],[8.355495999000027,50.86200099900003],[8.477892,50.96904749600003],[8.9746,50.938304]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE725","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vogelsbergkreis","NUTS_NAME":"Vogelsbergkreis"},"geometry":{"type":"Polygon","coordinates":[[[9.441055,50.79564450000004],[9.611464500000068,50.73547649900007],[9.641732500000046,50.62778949900007],[9.607138500000076,50.60560249900004],[9.466069,50.66061799900007],[9.482294,50.592301500000076],[9.42753250000004,50.478712],[9.479113,50.44068549600007],[9.349258,50.385486497000045],[9.335956499000076,50.451862],[9.286735,50.437297999000066],[9.087218499000073,50.438608],[9.042425500000036,50.497989999000026],[9.134108,50.556986496000036],[8.918371999000044,50.705605999000056],[9.148868500000049,50.83664499900004],[9.441055,50.79564450000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE73","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Kassel","NUTS_NAME":"Kassel"},"geometry":{"type":"Polygon","coordinates":[[[9.685331,51.58201649800003],[9.672377500000039,51.568403998000065],[9.625825001000067,51.580204999000046],[9.647755500000073,51.55251099800006],[9.585751,51.51731749900006],[9.642956001000073,51.41632149900005],[9.557291,51.35137849800003],[9.568025,51.34000149800005],[9.710295479000024,51.30153774900003],[9.775202,51.32519599800003],[9.702144,51.36457199800003],[9.803510499000026,51.41040049800006],[9.928339,51.37529899900005],[10.067095,51.22779299800004],[10.206942,51.19064899600005],[10.235999,51.18476749800004],[10.210013,51.144082998000044],[10.118321500000036,51.13957299900005],[10.202181,51.012009],[10.182348500000046,50.99848749900008],[10.021558500000026,50.992949499000076],[10.063865,50.94543299600008],[9.949583500000074,50.927478499000074],[10.055980999000042,50.879771497000036],[9.926176500000054,50.767389998000056],[9.878551,50.63478099800005],[10.077555500000074,50.63762399600006],[10.037268,50.61088399800008],[10.041338500000052,50.51646949800005],[9.935655,50.419606499000054],[9.771871,50.424859498000046],[9.73291450000005,50.356149498000036],[9.479113,50.44068549600007],[9.42753250000004,50.478712],[9.482294,50.592301500000076],[9.466069,50.66061799900007],[9.607138500000076,50.60560249900004],[9.641732500000046,50.62778949900007],[9.611464500000068,50.73547649900007],[9.441055,50.79564450000004],[9.148868500000049,50.83664499900004],[9.109958,50.91897749800006],[8.9746,50.938304],[8.477892,50.96904749600003],[8.549084999000058,51.101868],[8.656632500000057,51.094902998000066],[8.758214500000065,51.17718149800004],[8.721997500000043,51.273743996000064],[8.556348,51.277494999000055],[8.68347650000004,51.36217799800005],[8.939502,51.38949899800008],[8.890376500000059,51.48192849900005],[8.970653500000026,51.506773499000076],[9.155410500000073,51.44267499800003],[9.440457,51.65039399800003],[9.685331,51.58201649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE731","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kassel, Kreisfreie Stadt","NUTS_NAME":"Kassel, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[9.568025,51.34000149800005],[9.482835,51.26194099700007],[9.357435,51.31583449900006],[9.557291,51.35137849800003],[9.568025,51.34000149800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DE732","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Fulda","NUTS_NAME":"Fulda"},"geometry":{"type":"Polygon","coordinates":[[[9.926176500000054,50.767389998000056],[9.878551,50.63478099800005],[10.077555500000074,50.63762399600006],[10.037268,50.61088399800008],[10.041338500000052,50.51646949800005],[9.935655,50.419606499000054],[9.771871,50.424859498000046],[9.73291450000005,50.356149498000036],[9.479113,50.44068549600007],[9.42753250000004,50.478712],[9.482294,50.592301500000076],[9.466069,50.66061799900007],[9.607138500000076,50.60560249900004],[9.641732500000046,50.62778949900007],[9.611464500000068,50.73547649900007],[9.77266,50.802213996000035],[9.926176500000054,50.767389998000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL","LEVL_CODE":0,"CNTR_CODE":"EL","NAME_LATN":"ELLADA","NUTS_NAME":"ΕΛΛΑΔΑ"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.863466500000072,35.92927849700004],[27.769081,35.88635449800006],[27.718461500000046,35.94023049800006],[27.760219,36.08743549700006],[27.703456500000073,36.146589998000024],[27.896705,36.33302949700004],[28.220723500000076,36.45793699700005],[28.24711,36.386773998000024],[28.07672450000007,36.144510997000054],[28.099871,36.104273997000064],[27.95381750000007,36.04525349800008],[27.863466500000072,35.92927849700004]]],[[[26.98143,36.75075549700006],[26.973736,36.672668498000064],[26.917291500000033,36.71847549700004],[27.075595500000077,36.85018549700004],[27.281677500000058,36.916011997000055],[27.354324500000075,36.86148849800003],[26.98143,36.75075549700006]]],[[[27.230274,35.508568],[27.134781,35.39636599900007],[27.086414,35.44509899800005],[27.113149500000077,35.54570749700002],[27.061262,35.59654999800006],[27.210157500000037,35.830795497000054],[27.15044,35.61491749900006],[27.230274,35.508568]]],[[[27.070253500000035,37.717021997000074],[26.85927,37.63627999800008],[26.721821,37.70962149800005],[26.59285550000004,37.67725749700003],[26.566509500000052,37.73064799800005],[26.73162650000006,37.81142049700003],[26.857507500000054,37.79709649700004],[26.986845,37.78441599800004],[27.070253500000035,37.717021997000074]]],[[[27.045025,36.957065499000066],[26.943407,36.926288499000066],[26.92447450000003,36.986069],[26.96731950000003,37.01585749900005],[26.900644498000077,37.07257449900004],[27.03643050000005,37.01351149800007],[27.045025,36.957065499000066]]],[[[26.628431500000033,41.345533498000066],[26.321652,41.250406498000075],[26.31295750000004,41.07548049700006],[26.359039,40.94717649800003],[26.29101550000007,40.93188349800005],[26.12489,40.742109998000046],[26.032758,40.73025699800007],[26.02425,40.794869498000025],[25.945872999000073,40.842150496000045],[25.632429,40.86280449700007],[25.173954500000036,40.94000999800005],[25.139545,40.99068749800006],[25.069244500000025,41.004961998000056],[24.81349,40.85808549800004],[24.627527500000042,40.85849749800008],[24.508724,40.95848099700004],[24.394415,40.93347149700003],[24.307329,40.82701499700005],[24.08863050000008,40.72168399800006],[23.867746500000067,40.77872849800008],[23.765077500000075,40.75789249800005],[23.693813500000033,40.67448449700004],[23.756864500000063,40.634563497000045],[23.917795,40.53438549800006],[23.823702,40.50734349800007],[23.860433500000056,40.41778949700006],[24.010988,40.38943449800007],[24.005043,40.31456749800003],[23.861864,40.370163],[23.694948,40.30393999800003],[23.774591500000042,40.19285599800003],[23.99748850000003,40.10770799800008],[24.02107250000006,40.00879249900004],[23.947227500000054,39.93801899600004],[23.652324500000077,40.224616999000034],[23.401338500000065,40.27941149600008],[23.300182500000062,40.23226949800005],[23.00525649900004,40.35036050000008],[22.814283500000045,40.47844299800005],[22.97918150000004,40.54149599600004],[22.947689500000024,40.62663649700005],[22.858206,40.63497549600004],[22.85528350000004,40.59121349800006],[22.741501,40.55920799800003],[22.735582500000078,40.506091998000045],[22.682419,40.52896499800005],[22.663957500000038,40.49155449600005],[22.585716,40.46374499600006],[22.660866,40.35521299800007],[22.54789550000004,40.145462],[22.594291500000054,40.01237099700006],[22.66243,39.975547998000025],[22.852674500000035,39.800433998000074],[22.915911,39.60293599800008],[22.976824,39.55532849800005],[23.095781500000044,39.499927498000034],[23.351476500000047,39.18777449900006],[23.073730500000067,39.08465199900007],[23.08049950000003,39.147334998000076],[23.11130950000006,39.14481349700003],[23.096821500000033,39.09813299700005],[23.212417500000072,39.14687349600007],[23.163364500000057,39.27289949900006],[22.94211,39.361980499000026],[22.94639,39.299728497000046],[22.821161500000073,39.27750750000007],[22.85026750000003,39.154559998000025],[22.97913,39.09670249800007],[22.997400500000026,39.040111497000055],[22.96126950000007,39.00722899800007],[23.06714450000004,39.038768997000034],[23.01806650000003,39.00265499600005],[22.990511,38.94419099800007],[22.831438,38.889121997000075],[22.617443,38.91184999800004],[22.527525,38.85890949900005],[22.863748499000053,38.754974498000024],[23.034622,38.75635499800006],[23.10184650000008,38.62757849700006],[23.19618999900007,38.680735998000046],[23.321071500000073,38.63822949900003],[23.340513500000043,38.60502249900003],[23.296520500000042,38.56784449700007],[23.37779050000006,38.52575699700003],[23.328289,38.501220499000055],[23.560117,38.502597997000066],[23.65513450000003,38.35249699800005],[23.690418,38.34029749800004],[23.957017500000063,38.28344549900004],[24.079767,38.16191199800005],[23.980698500000074,38.10010349800007],[24.045281,37.90275649700004],[24.00857,37.88817499900006],[24.085396,37.792264498000065],[24.072212500000035,37.682054],[23.944671500000027,37.66884999800004],[23.901515499000027,37.77338399700005],[23.743593,37.85380949700004],[23.672225500000025,37.94181999800003],[23.557282500000042,37.962436498000045],[23.571394500000054,37.993224498000075],[23.596950500000048,38.01517499700003],[23.521479,38.044104997000034],[23.179699500000027,37.95158999700004],[23.133437,37.920439498000064],[23.009341,37.91685749900006],[22.84889350000003,38.02820649900008],[22.94963750000005,38.07875049800003],[23.117532,38.06064599700005],[23.215277,38.09932699800004],[23.224533,38.14897549700004],[23.126179,38.1684],[23.11132250000003,38.20705399800005],[23.066992,38.170379998000044],[22.779249,38.23318499800007],[22.79750450000006,38.27519599800007],[22.691986,38.317390499000055],[22.679586500000028,38.37295549700008],[22.60806850000006,38.35940149800007],[22.560972,38.280079],[22.419674,38.43173599900007],[22.378385500000036,38.33254249700008],[21.941110500000036,38.40624599700004],[21.851717,38.37451949700005],[21.486598730000026,38.301853583000025],[21.486297500000035,38.32768649800005],[21.484744472000045,38.306621567000036],[21.318926,38.49962999600007],[21.335010500000067,38.388656499000035],[21.14287950000005,38.303787499000066],[21.10001550000004,38.33328649900005],[21.146599,38.41347499800003],[21.07562450000006,38.48397799800006],[21.096044500000062,38.53156649700003],[21.02672,38.503150997000034],[20.988678,38.66868599800006],[20.914373500000067,38.67372899800006],[20.871914,38.794937498000024],[20.780818999000076,38.75421149700003],[20.726846500000022,38.814430498000036],[20.76258850000005,38.862751],[20.74679,38.922439498000074],[20.764536,38.95730949800003],[20.810058500000025,38.91819749900003],[20.978819,38.945262498000034],[21.066231,38.87713649800003],[21.164858,38.977615497000045],[21.10860850000006,39.04614650000008],[21.047264,39.00017949700003],[20.803007,39.117034998000065],[20.738746,39.01890949800003],[20.831041500000026,38.95981199800008],[20.758966500000042,38.97145849800006],[20.757292,38.950458500000025],[20.726776,38.949634496000044],[20.705598999000074,39.04753899700006],[20.474756,39.217723996000075],[20.47782150000006,39.27484149800006],[20.464247,39.27310199800007],[20.30067850000006,39.31573499600006],[20.212507500000072,39.446082999000055],[20.274601,39.44560599700003],[20.212242,39.49127899900003],[20.262635999000054,39.51013950000004],[20.136205500000074,39.52841949800006],[20.18475150000006,39.62173099800003],[20.008812500000033,39.691292],[20.226089500000057,39.64782849900007],[20.321796,39.72885249800004],[20.289463500000068,39.80479249900003],[20.391308,39.78848349900005],[20.41706750000003,39.840948998000044],[20.31223650000004,39.991022497000074],[20.393286,39.99561699700007],[20.455286500000057,40.074255497000024],[20.678365,40.096003],[20.709584,40.273352497000076],[20.778501,40.34878899800003],[20.786272500000052,40.42695199900004],[20.843011500000046,40.47952599800004],[20.955839,40.469640998000045],[21.056068500000038,40.61669549900006],[20.959147,40.77049199800007],[20.98020450000007,40.855664999000055],[21.301891500000067,40.865341998000076],[21.417245500000035,40.917696497000065],[21.59323550000005,40.86600249700007],[21.684081,40.94091499800004],[21.787378,40.931125499000075],[21.929438,41.10035049700008],[22.216216,41.17045749700003],[22.332052,41.120272500000056],[22.587711,41.11729599900008],[22.66647,41.18633949800005],[22.732037,41.146391497000025],[22.762478,41.320234],[22.879178500000023,41.34065299700006],[22.927591500000062,41.338539498000046],[23.18137150000007,41.318869498000026],[23.322212500000035,41.399669999000025],[23.62422299900004,41.375727496000025],[23.91613249900007,41.48162649900007],[24.046779,41.45128599800006],[24.059744,41.52211199900006],[24.525637,41.56869999700007],[24.607909500000062,41.42761799600004],[24.783956,41.36018899600003],[24.92368350000004,41.402213999000026],[25.17937550000005,41.31018799700007],[25.224698500000045,41.26463099700004],[25.906437500000038,41.307574997000074],[25.948626,41.320342],[26.15868850000004,41.391182496000056],[26.17645550000003,41.554799496000044],[26.060393,41.688516],[26.16942,41.74869149700004],[26.357879,41.711104498000054],[26.60012050000006,41.60119899700004],[26.628431500000033,41.345533498000066]]],[[[26.380855500000052,39.27061099700006],[26.531208,39.16079349800003],[26.609173,39.01049799700007],[26.542705500000068,39.01456449600005],[26.485323,39.1194],[26.441669500000046,39.102531498000076],[26.531257500000038,39.02105699800006],[26.51775150000003,38.97112999800004],[26.167289500000038,39.01274099800003],[26.103979,39.07213199600005],[26.280182,39.15457149700006],[26.24947350000008,39.20600899900006],[26.168291,39.196602],[26.075947,39.086482998000065],[25.831421,39.18841949800003],[25.90679350000005,39.29335799800003],[26.170395,39.32501599600005],[26.181169500000067,39.37506849700003],[26.424524500000075,39.33404149900008],[26.380855500000052,39.27061099700006]]],[[[26.36516750000004,36.57818199900004],[26.367147500000044,36.509528998000064],[26.300575,36.51485449900008],[26.25991250000004,36.59268949900007],[26.3573,36.57089599900007],[26.38240550200004,36.64425649900005],[26.473434500000053,36.580013499000074],[26.36516750000004,36.57818199900004]]],[[[26.22328,37.571559997000065],[25.974392,37.517813998000065],[26.078008500000067,37.634314998000036],[26.365686500000038,37.69006749700003],[26.33592,37.65026449800007],[26.22328,37.571559997000065]]],[[[23.840978500000062,35.528079999000056],[23.992985,35.51167299800005],[24.078102,35.592281496000055],[24.174324,35.58719999700003],[24.201605,35.538245999000026],[24.166441,35.49475099800003],[24.0597,35.49361799700006],[24.242358,35.475036497000076],[24.26303850000005,35.36258699900003],[24.317411,35.35377499700007],[24.700668500000063,35.427009498000075],[24.928003500000045,35.40690999800006],[25.040821,35.400146498000026],[25.066084,35.339481498000055],[25.490698,35.29833199700005],[25.771959500000037,35.34151849800003],[25.726930500000037,35.30328049700006],[25.726293500000054,35.13115699700006],[25.813135,35.10961549800004],[26.023828500000036,35.22866049700008],[26.162893500000052,35.19927249800003],[26.314392500000054,35.30653],[26.263292500000034,35.265228500000035],[26.281149,35.11158749800006],[26.193916500000057,35.01488149700003],[25.550568,34.99075699700006],[24.770942500000046,34.92988599700004],[24.722517,35.09254449900004],[24.559056,35.10519399900005],[24.380876500000056,35.195171498000036],[24.28752349900003,35.17600649900004],[23.814441500000044,35.24862299800003],[23.589775,35.23132699700005],[23.515789,35.28971499900007],[23.596821,35.62816249800005],[23.61231999900008,35.52050399700005],[23.716612,35.507671499000026],[23.73757149900007,35.69526649900007],[23.773618500000055,35.66851449700005],[23.779693500000064,35.54549399800004],[23.840978500000062,35.528079999000056]]],[[[26.03731349900005,38.19849399800006],[26.01017550000006,38.154545],[25.86412450000006,38.24406799700006],[25.995464500000026,38.342021998000064],[25.94259450000004,38.45983899700008],[25.830856,38.54042799700005],[25.857202500000028,38.580429],[26.159764,38.55309299700008],[26.13371250000006,38.403835498000035],[26.16142099900003,38.329364998000074],[26.099466500000062,38.21035399900006],[26.03731349900005,38.19849399800006]]],[[[25.972538,36.87828049700005],[25.870777,36.790363497000044],[25.74043450000005,36.79272849900008],[25.846332500000074,36.810523999000054],[25.844691999000077,36.85126499800003],[26.006548,36.93881999900003],[26.08844750000003,36.896289996000064],[25.972538,36.87828049700005]]],[[[25.687489,40.421697497000025],[25.560071,40.395632498000055],[25.45186650000005,40.47032149700004],[25.655822500000056,40.49385599800007],[25.687489,40.421697497000025]]],[[[25.5569,36.969550998000045],[25.449783500000024,36.91631299900007],[25.333025,37.076205998000034],[25.537660500000072,37.203227998000045],[25.598738,37.10888299700008],[25.5569,36.969550998000045]]],[[[25.487131,36.40910699900007],[25.443050999000036,36.331870999000046],[25.353496500000062,36.357879499000035],[25.430589500000053,36.38598999900006],[25.41644450000007,36.455508999000074],[25.368629500000054,36.470222499000045],[25.418937500000027,36.474735499000076],[25.487131,36.40910699900007]]],[[[25.459328500000026,37.44529349800007],[25.312966500000073,37.41899549800007],[25.348916,37.49837499900008],[25.459328500000026,37.44529349800007]]],[[[25.443185,39.980241996000075],[25.340635,39.878502499000035],[25.354668,39.78236849700005],[25.249677500000075,39.849107497000034],[25.255276500000036,39.91226149800008],[25.17972750000007,39.84926399700004],[25.226132500000062,39.79654749800005],[25.142272500000047,39.80959349800003],[25.161872500000072,39.84670999700006],[25.046687500000075,39.83291399700005],[25.063945500000045,39.99049649700004],[25.233850500000074,40.00563199800007],[25.292053,39.93993899600008],[25.446156,40.03553850000003],[25.443185,39.980241996000075]]],[[[25.396528,36.72313299800004],[25.385496,36.65177149900006],[25.266128500000036,36.71220049800007],[25.260687,36.76236349800007],[25.309864,36.791236996000066],[25.396528,36.72313299800004]]],[[[25.237235999000063,37.14475650000003],[25.212015,37.12801749800008],[25.289139,37.14229949800006],[25.242174,37.01102849800003],[25.117052,36.992080498000064],[25.107309500000042,37.06349199700003],[25.237235999000063,37.14475650000003]]],[[[25.253063,37.609508497000036],[25.233768500000053,37.53944799800007],[25.114006,37.55059749700007],[24.972765500000037,37.66778949800005],[25.253063,37.609508497000036]]],[[[24.957605500000057,37.89711749800006],[25.000368,37.76972949800006],[24.963575500000047,37.683925498000065],[24.76682450000004,37.87518299800007],[24.709385,37.871387998000046],[24.684284,37.95106499800005],[24.792885,37.99790949800007],[24.83931350000006,37.91334549900006],[24.957605500000057,37.89711749800006]]],[[[24.964029500000038,37.41032399900007],[24.88783250000006,37.369590499000026],[24.855234,37.39535549900006],[24.879831500000023,37.51392349900004],[24.946517999000037,37.48527899900006],[24.964029500000038,37.41032399900007]]],[[[24.774784,40.606056498000044],[24.640228499000045,40.58262999800007],[24.512184,40.63721449600007],[24.557409500000063,40.739089998000054],[24.65240650000004,40.80092999800007],[24.760584,40.745513999000025],[24.774784,40.606056498000044]]],[[[24.682358500000078,38.77332499800008],[24.563664,38.76233099800004],[24.530308500000046,38.79647249800007],[24.56670850000006,38.83881549700004],[24.430897,38.88147899900008],[24.448941,38.94910199800006],[24.573275500000022,38.92756050000003],[24.682358500000078,38.77332499800008]]],[[[23.426233500000023,38.90583049800006],[23.754823500000043,38.706645998000056],[24.155115,38.65118799900006],[24.122717,38.59658799700003],[24.232042500000034,38.52455149900004],[24.181846500000063,38.39474849800007],[24.255039500000066,38.288509497000064],[24.24803550000007,38.22748199800003],[24.350046,38.15526949800005],[24.554556,38.14113999800003],[24.58396350000004,38.02422349900007],[24.510714,37.94909299700004],[24.41704550000003,38.01402649600004],[24.373417,37.96247849700006],[24.30678,38.06962199800006],[24.202724499000055,38.08532349700005],[24.207344,38.179832496000074],[24.153744,38.27940399800008],[24.111498,38.25109099800005],[24.048601,38.395148997000035],[23.648699,38.39869699900004],[23.60085299900004,38.57010649700004],[23.511806500000034,38.58705499700005],[23.196602,38.834670998000036],[22.99010450000003,38.88124449800006],[22.829393500000037,38.825450998000065],[23.12924,39.007640998000056],[23.317873,39.03850549800006],[23.429733,38.95418549800007],[23.426233500000023,38.90583049800006]]],[[[24.54727,36.70661149800003],[24.321747,36.64612599900005],[24.353439500000036,36.75131199900005],[24.401016,36.70126350000004],[24.469988,36.70693599600003],[24.40929250000005,36.763923498000054],[24.537983,36.75758],[24.54727,36.70661149800003]]],[[[24.48255350000005,37.39654149900008],[24.364295998000046,37.30023599900005],[24.399852499000076,37.366565499000046],[24.36558350000007,37.42344299900003],[24.435737500000073,37.46062849900005],[24.48255350000005,37.39654149900008]]],[[[24.35662650000006,37.574202497000044],[24.27572250000003,37.522402499000066],[24.267139499000052,37.60773349800007],[24.350606999000036,37.687102],[24.409574500000076,37.64549499700007],[24.35662650000006,37.574202497000044]]],[[[23.690382475000035,39.087570553000035],[23.661911,39.08702099900006],[23.655265069000052,39.101519418000066],[23.651362178000056,39.110033764000036],[23.636522608000064,39.14240700100004],[23.63581978700006,39.143940239000074],[23.634577274000037,39.14665084100005],[23.605493500000023,39.210098499000026],[23.613432827000054,39.206438615000025],[23.614415847000032,39.20598546100007],[23.777410148000058,39.13084809900005],[23.778539283000043,39.130327589000046],[23.786434,39.126688270000045],[23.790044972000032,39.12502367800005],[23.791225352000026,39.12447954500004],[23.79319950000007,39.12356949900004],[23.789729575000024,39.11905838800004],[23.786377085000026,39.114699949000055],[23.784975072000066,39.11287724700003],[23.78412032600005,39.111766026000055],[23.76924678000006,39.09242952000005],[23.768871813000032,39.09194204100004],[23.767921263000062,39.090706268000076],[23.76664150000005,39.08904249900007],[23.763614216000065,39.08898406600008],[23.757521961000066,39.08886647400004],[23.75650874300004,39.08884691700007],[23.700251877000028,39.087761052000076],[23.698424143000068,39.087725773000045],[23.69460888800006,39.08765213100003],[23.690382475000035,39.087570553000035]]],[[[23.679613,39.972439],[23.750192500000026,39.915984998000056],[23.355198,39.953880498000046],[23.381927500000074,39.99904249800005],[23.308529,40.084636498000066],[23.306672026000058,40.215268835000074],[23.493444500000066,40.03395849800006],[23.679613,39.972439]]],[[[23.565233001000024,37.76540349900006],[23.478239,37.673377999000024],[23.417684500000064,37.761406],[23.488014,37.77302549900003],[23.565233001000024,37.76540349900006]]],[[[21.871086,38.33453749700004],[22.37297250000006,38.14222349700003],[22.639021,38.07839199700004],[22.856842,37.93777849600008],[22.962061,37.94956949800007],[23.017891,37.900067998000054],[23.008983500000056,37.84590299700005],[23.178438,37.803564497000025],[23.12003150000004,37.72998499800008],[23.200378500000056,37.596653],[23.33198,37.53283299800006],[23.367971500000067,37.55260849800004],[23.310201500000062,37.61953749800006],[23.385559,37.64023199800005],[23.413578,37.61856449700008],[23.386825500000043,37.56118399800005],[23.522672500000056,37.518099498000026],[23.434457275000057,37.51798560800006],[23.523903,37.436931498000035],[23.424419500000056,37.411700997000025],[23.268150500000047,37.40636049900007],[23.224903,37.371658498000045],[23.277609,37.34850299800007],[23.193193500000064,37.341045499000074],[23.171631,37.286002997000026],[23.06308,37.39172350000007],[23.14237950000006,37.415977499000064],[23.121271,37.450256498000044],[22.725086,37.57263949700007],[22.774015500000075,37.38739799700005],[22.893175,37.219043499000065],[22.897411500000032,37.11927050000003],[22.95490650000005,37.11929299800005],[22.982481,37.04802699700008],[23.088842500000055,36.81272499900007],[23.086452500000064,36.729018997000026],[23.027857,36.72883549900007],[23.03039350000006,36.63529199800007],[23.196792500000072,36.43466199900007],[22.968313,36.51951199800004],[22.785621500000047,36.796966498000074],[22.63109950000006,36.80678949900005],[22.565794,36.77590199700006],[22.57607450000006,36.73329199800003],[22.476125500000023,36.613116999000056],[22.480207500000063,36.39754099700008],[22.348186500000054,36.51036449800006],[22.386833,36.59114099900006],[22.29641350000003,36.813945997000076],[22.225924,36.893966499000044],[22.126909,36.91332249800007],[22.149515,37.01814649800008],[21.930139500000053,36.98115549900007],[21.957984999000075,36.79297249800004],[21.877396,36.71761649700005],[21.822733,36.80372199900006],[21.704893,36.81129049900005],[21.67955,36.888435498000035],[21.710382500000037,36.94160099900006],[21.576706,37.06180199800008],[21.565725499000052,37.16350949900004],[21.691736,37.293105998000044],[21.680233,37.37743399800007],[21.530187500000068,37.56888199800005],[21.32807150000008,37.66140349700004],[21.28442,37.778659997000034],[21.105238,37.848926498000026],[21.126194,37.93867499600003],[21.255453,37.97354499900007],[21.351074,38.102138498000045],[21.372593,38.218120498000076],[21.605222500000025,38.142891],[21.778976500000056,38.30762499800005],[21.871086,38.33453749700004]]],[[[23.054092,36.21827299800003],[23.047105500000043,36.130561998000076],[22.940229500000044,36.157631],[22.890321500000027,36.32016349700007],[22.951256,36.38523099900004],[23.103981,36.23604199700003],[23.054092,36.21827299800003]]],[[[20.899757,37.77567349800006],[20.988697,37.70522249600003],[20.878824,37.72989299900007],[20.829103,37.644634997000026],[20.625544,37.820702],[20.63263099900007,37.88441899800006],[20.71006,37.922859998000035],[20.899757,37.77567349800006]]],[[[20.628674,38.32527149600003],[20.601241999000024,38.30007949700007],[20.643795500000067,38.251945498000055],[20.680998500000044,38.27783199700008],[20.81529050000006,38.11352899700006],[20.796285500000067,38.065116998000065],[20.514303,38.103931499000055],[20.444082,38.28076949800004],[20.440220500000066,38.161388498000065],[20.34178550000007,38.17613199600004],[20.407196,38.34989549800008],[20.477171,38.309573998000076],[20.547944999000038,38.36228549700007],[20.536981500000024,38.46981049800007],[20.582451,38.46472149900006],[20.628674,38.32527149600003]]],[[[20.726379697000027,38.30626380100006],[20.723535500000025,38.30091499900004],[20.681861397000034,38.364154245000066],[20.681703518000063,38.36439382200007],[20.67858482300005,38.369126351000034],[20.676870332000078,38.37172804100004],[20.67638760500006,38.37246056500004],[20.674318576000076,38.37560025700003],[20.642205783000065,38.424330492000024],[20.640100338000025,38.427525443000036],[20.63224221300004,38.43944992100006],[20.627883214000065,38.446064576000026],[20.621872360000054,38.45518587400005],[20.621309320000023,38.456040270000074],[20.61507050000006,38.46550749900007],[20.627280287000076,38.47754448400008],[20.627569132000076,38.47782924100005],[20.627646765000065,38.47790577500007],[20.647870010000076,38.49784280500006],[20.64853775800003,38.49850110300008],[20.650018871000043,38.49996125400003],[20.650058971000078,38.50000078700003],[20.65047734600006,38.50041324000006],[20.651176500000076,38.50110249900007],[20.672380662000023,38.473754229000065],[20.672496657000067,38.47360462300003],[20.687026349000064,38.45486481300003],[20.687279871000044,38.45453783100004],[20.690643067000053,38.450200116000076],[20.693599595000023,38.446386905000054],[20.70467900400007,38.432097132000024],[20.705893076000052,38.43053126900003],[20.746239073000027,38.378494630000034],[20.746346508000045,38.37835606400006],[20.746497791000024,38.37816094600004],[20.756654337000043,38.36506144300006],[20.757242,38.36430349900007],[20.743891635000068,38.339196779000076],[20.743751848000045,38.33893389600007],[20.726379697000027,38.30626380100006]]],[[[20.721731,38.626224498000056],[20.551553500000068,38.577827497000044],[20.59967050000006,38.779578997000044],[20.693756,38.85181799700007],[20.721731,38.626224498000056]]],[[[20.072651500000063,39.453790498000046],[20.109863500000074,39.36097199900007],[19.868664500000023,39.456530996000026],[19.831421500000033,39.57359249800004],[19.641207,39.749874498000054],[19.673725500000046,39.79635899800007],[19.91553650000003,39.79195249800006],[19.95502350000004,39.766340498000034],[19.845195,39.707595996000066],[19.841421500000024,39.659159498000065],[19.93253,39.62501399800004],[19.93844449900007,39.47206149800007],[20.072651500000063,39.453790498000046]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL3","LEVL_CODE":1,"CNTR_CODE":"EL","NAME_LATN":"ATTIKI","NUTS_NAME":"ATTIKΗ"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.079767,38.16191199800005],[23.980698500000074,38.10010349800007],[24.045281,37.90275649700004],[24.00857,37.88817499900006],[24.085396,37.792264498000065],[24.072212500000035,37.682054],[23.944671500000027,37.66884999800004],[23.901515499000027,37.77338399700005],[23.743593,37.85380949700004],[23.672225500000025,37.94181999800003],[23.557282500000042,37.962436498000045],[23.571394500000054,37.993224498000075],[23.596950500000048,38.01517499700003],[23.521479,38.044104997000034],[23.179699500000027,37.95158999700004],[23.106703,37.99519749700005],[23.117532,38.06064599700005],[23.215277,38.09932699800004],[23.224533,38.14897549700004],[23.126179,38.1684],[23.28664,38.18450949700008],[23.29180350000007,38.23928049700004],[23.393368,38.263484998000024],[23.37436850000006,38.21328349800007],[23.460073500000078,38.20882049900007],[23.515825500000062,38.13584149800005],[23.634155500000077,38.21131149800004],[23.690418,38.34029749800004],[23.957017500000063,38.28344549900004],[24.079767,38.16191199800005]]],[[[23.565233001000024,37.76540349900006],[23.478239,37.673377999000024],[23.417684500000064,37.761406],[23.488014,37.77302549900003],[23.565233001000024,37.76540349900006]]],[[[23.386825500000043,37.56118399800005],[23.522672500000056,37.518099498000026],[23.434457275000057,37.51798560800006],[23.523903,37.436931498000035],[23.424419500000056,37.411700997000025],[23.454609,37.46235649800008],[23.17948749900006,37.52343349800003],[23.200378500000056,37.596653],[23.33198,37.53283299800006],[23.367971500000067,37.55260849800004],[23.310201500000062,37.61953749800006],[23.385559,37.64023199800005],[23.413578,37.61856449700008],[23.386825500000043,37.56118399800005]]],[[[23.054092,36.21827299800003],[23.047105500000043,36.130561998000076],[22.940229500000044,36.157631],[22.890321500000027,36.32016349700007],[22.951256,36.38523099900004],[23.103981,36.23604199700003],[23.054092,36.21827299800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL30","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Attiki","NUTS_NAME":"Aττική"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.079767,38.16191199800005],[23.980698500000074,38.10010349800007],[24.045281,37.90275649700004],[24.00857,37.88817499900006],[24.085396,37.792264498000065],[24.072212500000035,37.682054],[23.944671500000027,37.66884999800004],[23.901515499000027,37.77338399700005],[23.743593,37.85380949700004],[23.672225500000025,37.94181999800003],[23.557282500000042,37.962436498000045],[23.571394500000054,37.993224498000075],[23.596950500000048,38.01517499700003],[23.521479,38.044104997000034],[23.179699500000027,37.95158999700004],[23.106703,37.99519749700005],[23.117532,38.06064599700005],[23.215277,38.09932699800004],[23.224533,38.14897549700004],[23.126179,38.1684],[23.28664,38.18450949700008],[23.29180350000007,38.23928049700004],[23.393368,38.263484998000024],[23.37436850000006,38.21328349800007],[23.460073500000078,38.20882049900007],[23.515825500000062,38.13584149800005],[23.634155500000077,38.21131149800004],[23.690418,38.34029749800004],[23.957017500000063,38.28344549900004],[24.079767,38.16191199800005]]],[[[23.565233001000024,37.76540349900006],[23.478239,37.673377999000024],[23.417684500000064,37.761406],[23.488014,37.77302549900003],[23.565233001000024,37.76540349900006]]],[[[23.386825500000043,37.56118399800005],[23.522672500000056,37.518099498000026],[23.434457275000057,37.51798560800006],[23.523903,37.436931498000035],[23.424419500000056,37.411700997000025],[23.454609,37.46235649800008],[23.17948749900006,37.52343349800003],[23.200378500000056,37.596653],[23.33198,37.53283299800006],[23.367971500000067,37.55260849800004],[23.310201500000062,37.61953749800006],[23.385559,37.64023199800005],[23.413578,37.61856449700008],[23.386825500000043,37.56118399800005]]],[[[23.054092,36.21827299800003],[23.047105500000043,36.130561998000076],[22.940229500000044,36.157631],[22.890321500000027,36.32016349700007],[22.951256,36.38523099900004],[23.103981,36.23604199700003],[23.054092,36.21827299800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL301","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Boreios Tomeas Athinon","NUTS_NAME":"Βόρειος Τομέας Αθηνών"},"geometry":{"type":"Polygon","coordinates":[[[23.822056,37.97382349700007],[23.745092500000055,38.05787299700006],[23.855312500000025,38.11739749600008],[23.96492550000005,38.041740498000024],[23.84950850000007,38.04236999600005],[23.822056,37.97382349700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL302","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ditikos Tomeas Athinon","NUTS_NAME":"Δυτικός Τομέας Αθηνών"},"geometry":{"type":"Polygon","coordinates":[[[23.685508500000026,37.97627599700007],[23.672225500000025,37.94181999800003],[23.557282500000042,37.962436498000045],[23.571394500000054,37.993224498000075],[23.596950500000048,38.01517499700003],[23.720188,38.06255699800005],[23.735294500000066,38.049926998000046],[23.687181500000065,37.97819149800006],[23.685508500000026,37.97627599700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL303","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Kentrikos Tomeas Athinon","NUTS_NAME":"Κεντρικός Τομέας Αθηνών"},"geometry":{"type":"Polygon","coordinates":[[[23.822056,37.97382349700007],[23.806255500000077,37.91736999700004],[23.687181500000065,37.97819149800006],[23.735294500000066,38.049926998000046],[23.745092500000055,38.05787299700006],[23.822056,37.97382349700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL304","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Notios Tomeas Athinon","NUTS_NAME":"Νότιος Τομέας Αθηνών"},"geometry":{"type":"Polygon","coordinates":[[[23.806255500000077,37.91736999700004],[23.743593,37.85380949700004],[23.672225500000025,37.94181999800003],[23.685508500000026,37.97627599700007],[23.687181500000065,37.97819149800006],[23.806255500000077,37.91736999700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL305","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Anatoliki Attiki","NUTS_NAME":"Ανατολική Αττική"},"geometry":{"type":"Polygon","coordinates":[[[23.743593,37.85380949700004],[23.806255500000077,37.91736999700004],[23.822056,37.97382349700007],[23.84950850000007,38.04236999600005],[23.96492550000005,38.041740498000024],[23.855312500000025,38.11739749600008],[23.745092500000055,38.05787299700006],[23.735294500000066,38.049926998000046],[23.720188,38.06255699800005],[23.729721,38.12724349800004],[23.634155500000077,38.21131149800004],[23.690418,38.34029749800004],[23.957017500000063,38.28344549900004],[24.079767,38.16191199800005],[23.980698500000074,38.10010349800007],[24.045281,37.90275649700004],[24.00857,37.88817499900006],[24.085396,37.792264498000065],[24.072212500000035,37.682054],[23.944671500000027,37.66884999800004],[23.901515499000027,37.77338399700005],[23.743593,37.85380949700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL306","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ditiki Attiki","NUTS_NAME":"Δυτική Αττική"},"geometry":{"type":"Polygon","coordinates":[[[23.634155500000077,38.21131149800004],[23.729721,38.12724349800004],[23.720188,38.06255699800005],[23.596950500000048,38.01517499700003],[23.521479,38.044104997000034],[23.179699500000027,37.95158999700004],[23.106703,37.99519749700005],[23.117532,38.06064599700005],[23.215277,38.09932699800004],[23.224533,38.14897549700004],[23.126179,38.1684],[23.28664,38.18450949700008],[23.29180350000007,38.23928049700004],[23.393368,38.263484998000024],[23.37436850000006,38.21328349800007],[23.460073500000078,38.20882049900007],[23.515825500000062,38.13584149800005],[23.634155500000077,38.21131149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL307","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Peiraias, Nisoi","NUTS_NAME":"Πειραιάς, Νήσοι"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.565233001000024,37.76540349900006],[23.478239,37.673377999000024],[23.417684500000064,37.761406],[23.488014,37.77302549900003],[23.565233001000024,37.76540349900006]]],[[[23.386825500000043,37.56118399800005],[23.522672500000056,37.518099498000026],[23.434457275000057,37.51798560800006],[23.523903,37.436931498000035],[23.424419500000056,37.411700997000025],[23.454609,37.46235649800008],[23.17948749900006,37.52343349800003],[23.200378500000056,37.596653],[23.33198,37.53283299800006],[23.367971500000067,37.55260849800004],[23.310201500000062,37.61953749800006],[23.385559,37.64023199800005],[23.413578,37.61856449700008],[23.386825500000043,37.56118399800005]]],[[[23.054092,36.21827299800003],[23.047105500000043,36.130561998000076],[22.940229500000044,36.157631],[22.890321500000027,36.32016349700007],[22.951256,36.38523099900004],[23.103981,36.23604199700003],[23.054092,36.21827299800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL4","LEVL_CODE":1,"CNTR_CODE":"EL","NAME_LATN":"NISIA AIGAIOU, KRITI","NUTS_NAME":"NΗΣΙΑ ΑΙΓΑΙΟΥ, KΡΗΤΗ"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.863466500000072,35.92927849700004],[27.769081,35.88635449800006],[27.718461500000046,35.94023049800006],[27.760219,36.08743549700006],[27.703456500000073,36.146589998000024],[27.896705,36.33302949700004],[28.220723500000076,36.45793699700005],[28.24711,36.386773998000024],[28.07672450000007,36.144510997000054],[28.099871,36.104273997000064],[27.95381750000007,36.04525349800008],[27.863466500000072,35.92927849700004]]],[[[26.98143,36.75075549700006],[26.973736,36.672668498000064],[26.917291500000033,36.71847549700004],[27.075595500000077,36.85018549700004],[27.281677500000058,36.916011997000055],[27.354324500000075,36.86148849800003],[26.98143,36.75075549700006]]],[[[27.230274,35.508568],[27.134781,35.39636599900007],[27.086414,35.44509899800005],[27.113149500000077,35.54570749700002],[27.061262,35.59654999800006],[27.210157500000037,35.830795497000054],[27.15044,35.61491749900006],[27.230274,35.508568]]],[[[27.070253500000035,37.717021997000074],[26.85927,37.63627999800008],[26.721821,37.70962149800005],[26.59285550000004,37.67725749700003],[26.566509500000052,37.73064799800005],[26.73162650000006,37.81142049700003],[26.857507500000054,37.79709649700004],[26.986845,37.78441599800004],[27.070253500000035,37.717021997000074]]],[[[27.045025,36.957065499000066],[26.943407,36.926288499000066],[26.92447450000003,36.986069],[26.96731950000003,37.01585749900005],[26.900644498000077,37.07257449900004],[27.03643050000005,37.01351149800007],[27.045025,36.957065499000066]]],[[[26.380855500000052,39.27061099700006],[26.531208,39.16079349800003],[26.609173,39.01049799700007],[26.542705500000068,39.01456449600005],[26.485323,39.1194],[26.441669500000046,39.102531498000076],[26.531257500000038,39.02105699800006],[26.51775150000003,38.97112999800004],[26.167289500000038,39.01274099800003],[26.103979,39.07213199600005],[26.280182,39.15457149700006],[26.24947350000008,39.20600899900006],[26.168291,39.196602],[26.075947,39.086482998000065],[25.831421,39.18841949800003],[25.90679350000005,39.29335799800003],[26.170395,39.32501599600005],[26.181169500000067,39.37506849700003],[26.424524500000075,39.33404149900008],[26.380855500000052,39.27061099700006]]],[[[26.36516750000004,36.57818199900004],[26.367147500000044,36.509528998000064],[26.300575,36.51485449900008],[26.25991250000004,36.59268949900007],[26.3573,36.57089599900007],[26.38240550200004,36.64425649900005],[26.473434500000053,36.580013499000074],[26.36516750000004,36.57818199900004]]],[[[26.22328,37.571559997000065],[25.974392,37.517813998000065],[26.078008500000067,37.634314998000036],[26.365686500000038,37.69006749700003],[26.33592,37.65026449800007],[26.22328,37.571559997000065]]],[[[23.840978500000062,35.528079999000056],[23.992985,35.51167299800005],[24.078102,35.592281496000055],[24.174324,35.58719999700003],[24.201605,35.538245999000026],[24.166441,35.49475099800003],[24.0597,35.49361799700006],[24.242358,35.475036497000076],[24.26303850000005,35.36258699900003],[24.317411,35.35377499700007],[24.700668500000063,35.427009498000075],[24.928003500000045,35.40690999800006],[25.040821,35.400146498000026],[25.066084,35.339481498000055],[25.490698,35.29833199700005],[25.771959500000037,35.34151849800003],[25.726930500000037,35.30328049700006],[25.726293500000054,35.13115699700006],[25.813135,35.10961549800004],[26.023828500000036,35.22866049700008],[26.162893500000052,35.19927249800003],[26.314392500000054,35.30653],[26.263292500000034,35.265228500000035],[26.281149,35.11158749800006],[26.193916500000057,35.01488149700003],[25.550568,34.99075699700006],[24.770942500000046,34.92988599700004],[24.722517,35.09254449900004],[24.559056,35.10519399900005],[24.380876500000056,35.195171498000036],[24.28752349900003,35.17600649900004],[23.814441500000044,35.24862299800003],[23.589775,35.23132699700005],[23.515789,35.28971499900007],[23.596821,35.62816249800005],[23.61231999900008,35.52050399700005],[23.716612,35.507671499000026],[23.73757149900007,35.69526649900007],[23.773618500000055,35.66851449700005],[23.779693500000064,35.54549399800004],[23.840978500000062,35.528079999000056]]],[[[26.03731349900005,38.19849399800006],[26.01017550000006,38.154545],[25.86412450000006,38.24406799700006],[25.995464500000026,38.342021998000064],[25.94259450000004,38.45983899700008],[25.830856,38.54042799700005],[25.857202500000028,38.580429],[26.159764,38.55309299700008],[26.13371250000006,38.403835498000035],[26.16142099900003,38.329364998000074],[26.099466500000062,38.21035399900006],[26.03731349900005,38.19849399800006]]],[[[25.972538,36.87828049700005],[25.870777,36.790363497000044],[25.74043450000005,36.79272849900008],[25.846332500000074,36.810523999000054],[25.844691999000077,36.85126499800003],[26.006548,36.93881999900003],[26.08844750000003,36.896289996000064],[25.972538,36.87828049700005]]],[[[25.5569,36.969550998000045],[25.449783500000024,36.91631299900007],[25.333025,37.076205998000034],[25.537660500000072,37.203227998000045],[25.598738,37.10888299700008],[25.5569,36.969550998000045]]],[[[25.487131,36.40910699900007],[25.443050999000036,36.331870999000046],[25.353496500000062,36.357879499000035],[25.430589500000053,36.38598999900006],[25.41644450000007,36.455508999000074],[25.368629500000054,36.470222499000045],[25.418937500000027,36.474735499000076],[25.487131,36.40910699900007]]],[[[25.459328500000026,37.44529349800007],[25.312966500000073,37.41899549800007],[25.348916,37.49837499900008],[25.459328500000026,37.44529349800007]]],[[[25.443185,39.980241996000075],[25.340635,39.878502499000035],[25.354668,39.78236849700005],[25.249677500000075,39.849107497000034],[25.255276500000036,39.91226149800008],[25.17972750000007,39.84926399700004],[25.226132500000062,39.79654749800005],[25.142272500000047,39.80959349800003],[25.161872500000072,39.84670999700006],[25.046687500000075,39.83291399700005],[25.063945500000045,39.99049649700004],[25.233850500000074,40.00563199800007],[25.292053,39.93993899600008],[25.446156,40.03553850000003],[25.443185,39.980241996000075]]],[[[25.396528,36.72313299800004],[25.385496,36.65177149900006],[25.266128500000036,36.71220049800007],[25.260687,36.76236349800007],[25.309864,36.791236996000066],[25.396528,36.72313299800004]]],[[[25.237235999000063,37.14475650000003],[25.212015,37.12801749800008],[25.289139,37.14229949800006],[25.242174,37.01102849800003],[25.117052,36.992080498000064],[25.107309500000042,37.06349199700003],[25.237235999000063,37.14475650000003]]],[[[25.253063,37.609508497000036],[25.233768500000053,37.53944799800007],[25.114006,37.55059749700007],[24.972765500000037,37.66778949800005],[25.253063,37.609508497000036]]],[[[24.957605500000057,37.89711749800006],[25.000368,37.76972949800006],[24.963575500000047,37.683925498000065],[24.76682450000004,37.87518299800007],[24.709385,37.871387998000046],[24.684284,37.95106499800005],[24.792885,37.99790949800007],[24.83931350000006,37.91334549900006],[24.957605500000057,37.89711749800006]]],[[[24.964029500000038,37.41032399900007],[24.88783250000006,37.369590499000026],[24.855234,37.39535549900006],[24.879831500000023,37.51392349900004],[24.946517999000037,37.48527899900006],[24.964029500000038,37.41032399900007]]],[[[24.54727,36.70661149800003],[24.321747,36.64612599900005],[24.353439500000036,36.75131199900005],[24.401016,36.70126350000004],[24.469988,36.70693599600003],[24.40929250000005,36.763923498000054],[24.537983,36.75758],[24.54727,36.70661149800003]]],[[[24.48255350000005,37.39654149900008],[24.364295998000046,37.30023599900005],[24.399852499000076,37.366565499000046],[24.36558350000007,37.42344299900003],[24.435737500000073,37.46062849900005],[24.48255350000005,37.39654149900008]]],[[[24.35662650000006,37.574202497000044],[24.27572250000003,37.522402499000066],[24.267139499000052,37.60773349800007],[24.350606999000036,37.687102],[24.409574500000076,37.64549499700007],[24.35662650000006,37.574202497000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL41","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Voreio Aigaio","NUTS_NAME":"Βόρειο Αιγαίο"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.070253500000035,37.717021997000074],[26.85927,37.63627999800008],[26.721821,37.70962149800005],[26.59285550000004,37.67725749700003],[26.566509500000052,37.73064799800005],[26.73162650000006,37.81142049700003],[26.857507500000054,37.79709649700004],[26.986845,37.78441599800004],[27.070253500000035,37.717021997000074]]],[[[26.380855500000052,39.27061099700006],[26.531208,39.16079349800003],[26.609173,39.01049799700007],[26.542705500000068,39.01456449600005],[26.485323,39.1194],[26.441669500000046,39.102531498000076],[26.531257500000038,39.02105699800006],[26.51775150000003,38.97112999800004],[26.167289500000038,39.01274099800003],[26.103979,39.07213199600005],[26.280182,39.15457149700006],[26.24947350000008,39.20600899900006],[26.168291,39.196602],[26.075947,39.086482998000065],[25.831421,39.18841949800003],[25.90679350000005,39.29335799800003],[26.170395,39.32501599600005],[26.181169500000067,39.37506849700003],[26.424524500000075,39.33404149900008],[26.380855500000052,39.27061099700006]]],[[[26.22328,37.571559997000065],[25.974392,37.517813998000065],[26.078008500000067,37.634314998000036],[26.365686500000038,37.69006749700003],[26.33592,37.65026449800007],[26.22328,37.571559997000065]]],[[[26.03731349900005,38.19849399800006],[26.01017550000006,38.154545],[25.86412450000006,38.24406799700006],[25.995464500000026,38.342021998000064],[25.94259450000004,38.45983899700008],[25.830856,38.54042799700005],[25.857202500000028,38.580429],[26.159764,38.55309299700008],[26.13371250000006,38.403835498000035],[26.16142099900003,38.329364998000074],[26.099466500000062,38.21035399900006],[26.03731349900005,38.19849399800006]]],[[[25.443185,39.980241996000075],[25.340635,39.878502499000035],[25.354668,39.78236849700005],[25.249677500000075,39.849107497000034],[25.255276500000036,39.91226149800008],[25.17972750000007,39.84926399700004],[25.226132500000062,39.79654749800005],[25.142272500000047,39.80959349800003],[25.161872500000072,39.84670999700006],[25.046687500000075,39.83291399700005],[25.063945500000045,39.99049649700004],[25.233850500000074,40.00563199800007],[25.292053,39.93993899600008],[25.446156,40.03553850000003],[25.443185,39.980241996000075]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL411","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Lesbos, Limnos","NUTS_NAME":"Λέσβος, Λήμνος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[26.380855500000052,39.27061099700006],[26.531208,39.16079349800003],[26.609173,39.01049799700007],[26.542705500000068,39.01456449600005],[26.485323,39.1194],[26.441669500000046,39.102531498000076],[26.531257500000038,39.02105699800006],[26.51775150000003,38.97112999800004],[26.167289500000038,39.01274099800003],[26.103979,39.07213199600005],[26.280182,39.15457149700006],[26.24947350000008,39.20600899900006],[26.168291,39.196602],[26.075947,39.086482998000065],[25.831421,39.18841949800003],[25.90679350000005,39.29335799800003],[26.170395,39.32501599600005],[26.181169500000067,39.37506849700003],[26.424524500000075,39.33404149900008],[26.380855500000052,39.27061099700006]]],[[[25.443185,39.980241996000075],[25.340635,39.878502499000035],[25.354668,39.78236849700005],[25.249677500000075,39.849107497000034],[25.255276500000036,39.91226149800008],[25.17972750000007,39.84926399700004],[25.226132500000062,39.79654749800005],[25.142272500000047,39.80959349800003],[25.161872500000072,39.84670999700006],[25.046687500000075,39.83291399700005],[25.063945500000045,39.99049649700004],[25.233850500000074,40.00563199800007],[25.292053,39.93993899600008],[25.446156,40.03553850000003],[25.443185,39.980241996000075]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL412","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ikaria, Samos","NUTS_NAME":"Ικαρία, Σάμος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.070253500000035,37.717021997000074],[26.85927,37.63627999800008],[26.721821,37.70962149800005],[26.59285550000004,37.67725749700003],[26.566509500000052,37.73064799800005],[26.73162650000006,37.81142049700003],[26.857507500000054,37.79709649700004],[26.986845,37.78441599800004],[27.070253500000035,37.717021997000074]]],[[[26.22328,37.571559997000065],[25.974392,37.517813998000065],[26.078008500000067,37.634314998000036],[26.365686500000038,37.69006749700003],[26.33592,37.65026449800007],[26.22328,37.571559997000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL413","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Xios","NUTS_NAME":"Χίος"},"geometry":{"type":"Polygon","coordinates":[[[26.03731349900005,38.19849399800006],[26.01017550000006,38.154545],[25.86412450000006,38.24406799700006],[25.995464500000026,38.342021998000064],[25.94259450000004,38.45983899700008],[25.830856,38.54042799700005],[25.857202500000028,38.580429],[26.159764,38.55309299700008],[26.13371250000006,38.403835498000035],[26.16142099900003,38.329364998000074],[26.099466500000062,38.21035399900006],[26.03731349900005,38.19849399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL42","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Notio Aigaio","NUTS_NAME":"Νότιο Αιγαίο"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.863466500000072,35.92927849700004],[27.769081,35.88635449800006],[27.718461500000046,35.94023049800006],[27.760219,36.08743549700006],[27.703456500000073,36.146589998000024],[27.896705,36.33302949700004],[28.220723500000076,36.45793699700005],[28.24711,36.386773998000024],[28.07672450000007,36.144510997000054],[28.099871,36.104273997000064],[27.95381750000007,36.04525349800008],[27.863466500000072,35.92927849700004]]],[[[26.98143,36.75075549700006],[26.973736,36.672668498000064],[26.917291500000033,36.71847549700004],[27.075595500000077,36.85018549700004],[27.281677500000058,36.916011997000055],[27.354324500000075,36.86148849800003],[26.98143,36.75075549700006]]],[[[27.230274,35.508568],[27.134781,35.39636599900007],[27.086414,35.44509899800005],[27.113149500000077,35.54570749700002],[27.061262,35.59654999800006],[27.210157500000037,35.830795497000054],[27.15044,35.61491749900006],[27.230274,35.508568]]],[[[27.045025,36.957065499000066],[26.943407,36.926288499000066],[26.92447450000003,36.986069],[26.96731950000003,37.01585749900005],[26.900644498000077,37.07257449900004],[27.03643050000005,37.01351149800007],[27.045025,36.957065499000066]]],[[[26.36516750000004,36.57818199900004],[26.367147500000044,36.509528998000064],[26.300575,36.51485449900008],[26.25991250000004,36.59268949900007],[26.3573,36.57089599900007],[26.38240550200004,36.64425649900005],[26.473434500000053,36.580013499000074],[26.36516750000004,36.57818199900004]]],[[[25.972538,36.87828049700005],[25.870777,36.790363497000044],[25.74043450000005,36.79272849900008],[25.846332500000074,36.810523999000054],[25.844691999000077,36.85126499800003],[26.006548,36.93881999900003],[26.08844750000003,36.896289996000064],[25.972538,36.87828049700005]]],[[[25.5569,36.969550998000045],[25.449783500000024,36.91631299900007],[25.333025,37.076205998000034],[25.537660500000072,37.203227998000045],[25.598738,37.10888299700008],[25.5569,36.969550998000045]]],[[[25.487131,36.40910699900007],[25.443050999000036,36.331870999000046],[25.353496500000062,36.357879499000035],[25.430589500000053,36.38598999900006],[25.41644450000007,36.455508999000074],[25.368629500000054,36.470222499000045],[25.418937500000027,36.474735499000076],[25.487131,36.40910699900007]]],[[[25.459328500000026,37.44529349800007],[25.312966500000073,37.41899549800007],[25.348916,37.49837499900008],[25.459328500000026,37.44529349800007]]],[[[25.396528,36.72313299800004],[25.385496,36.65177149900006],[25.266128500000036,36.71220049800007],[25.260687,36.76236349800007],[25.309864,36.791236996000066],[25.396528,36.72313299800004]]],[[[25.237235999000063,37.14475650000003],[25.212015,37.12801749800008],[25.289139,37.14229949800006],[25.242174,37.01102849800003],[25.117052,36.992080498000064],[25.107309500000042,37.06349199700003],[25.237235999000063,37.14475650000003]]],[[[25.253063,37.609508497000036],[25.233768500000053,37.53944799800007],[25.114006,37.55059749700007],[24.972765500000037,37.66778949800005],[25.253063,37.609508497000036]]],[[[24.957605500000057,37.89711749800006],[25.000368,37.76972949800006],[24.963575500000047,37.683925498000065],[24.76682450000004,37.87518299800007],[24.709385,37.871387998000046],[24.684284,37.95106499800005],[24.792885,37.99790949800007],[24.83931350000006,37.91334549900006],[24.957605500000057,37.89711749800006]]],[[[24.964029500000038,37.41032399900007],[24.88783250000006,37.369590499000026],[24.855234,37.39535549900006],[24.879831500000023,37.51392349900004],[24.946517999000037,37.48527899900006],[24.964029500000038,37.41032399900007]]],[[[24.54727,36.70661149800003],[24.321747,36.64612599900005],[24.353439500000036,36.75131199900005],[24.401016,36.70126350000004],[24.469988,36.70693599600003],[24.40929250000005,36.763923498000054],[24.537983,36.75758],[24.54727,36.70661149800003]]],[[[24.48255350000005,37.39654149900008],[24.364295998000046,37.30023599900005],[24.399852499000076,37.366565499000046],[24.36558350000007,37.42344299900003],[24.435737500000073,37.46062849900005],[24.48255350000005,37.39654149900008]]],[[[24.35662650000006,37.574202497000044],[24.27572250000003,37.522402499000066],[24.267139499000052,37.60773349800007],[24.350606999000036,37.687102],[24.409574500000076,37.64549499700007],[24.35662650000006,37.574202497000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL421","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Kalimnos, Karpathos, Kos, Rodos","NUTS_NAME":"Κάλυμνος, Κάρπαθος, Κως, Ρόδος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.863466500000072,35.92927849700004],[27.769081,35.88635449800006],[27.718461500000046,35.94023049800006],[27.760219,36.08743549700006],[27.703456500000073,36.146589998000024],[27.896705,36.33302949700004],[28.220723500000076,36.45793699700005],[28.24711,36.386773998000024],[28.07672450000007,36.144510997000054],[28.099871,36.104273997000064],[27.95381750000007,36.04525349800008],[27.863466500000072,35.92927849700004]]],[[[26.98143,36.75075549700006],[26.973736,36.672668498000064],[26.917291500000033,36.71847549700004],[27.075595500000077,36.85018549700004],[27.281677500000058,36.916011997000055],[27.354324500000075,36.86148849800003],[26.98143,36.75075549700006]]],[[[27.230274,35.508568],[27.134781,35.39636599900007],[27.086414,35.44509899800005],[27.113149500000077,35.54570749700002],[27.061262,35.59654999800006],[27.210157500000037,35.830795497000054],[27.15044,35.61491749900006],[27.230274,35.508568]]],[[[27.045025,36.957065499000066],[26.943407,36.926288499000066],[26.92447450000003,36.986069],[26.96731950000003,37.01585749900005],[26.900644498000077,37.07257449900004],[27.03643050000005,37.01351149800007],[27.045025,36.957065499000066]]],[[[26.36516750000004,36.57818199900004],[26.367147500000044,36.509528998000064],[26.300575,36.51485449900008],[26.25991250000004,36.59268949900007],[26.3573,36.57089599900007],[26.38240550200004,36.64425649900005],[26.473434500000053,36.580013499000074],[26.36516750000004,36.57818199900004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL422","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Andros, Thira, Kea, Milos, Mikonos, Naxos, Paros, Siros, Tinos","NUTS_NAME":"Άνδρος, Θήρα, Κέα, Μήλος, Μύκονος, Νάξος, Πάρος, Σύρος, Τήνος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[25.972538,36.87828049700005],[25.870777,36.790363497000044],[25.74043450000005,36.79272849900008],[25.846332500000074,36.810523999000054],[25.844691999000077,36.85126499800003],[26.006548,36.93881999900003],[26.08844750000003,36.896289996000064],[25.972538,36.87828049700005]]],[[[25.5569,36.969550998000045],[25.449783500000024,36.91631299900007],[25.333025,37.076205998000034],[25.537660500000072,37.203227998000045],[25.598738,37.10888299700008],[25.5569,36.969550998000045]]],[[[25.487131,36.40910699900007],[25.443050999000036,36.331870999000046],[25.353496500000062,36.357879499000035],[25.430589500000053,36.38598999900006],[25.41644450000007,36.455508999000074],[25.368629500000054,36.470222499000045],[25.418937500000027,36.474735499000076],[25.487131,36.40910699900007]]],[[[25.459328500000026,37.44529349800007],[25.312966500000073,37.41899549800007],[25.348916,37.49837499900008],[25.459328500000026,37.44529349800007]]],[[[25.396528,36.72313299800004],[25.385496,36.65177149900006],[25.266128500000036,36.71220049800007],[25.260687,36.76236349800007],[25.309864,36.791236996000066],[25.396528,36.72313299800004]]],[[[25.237235999000063,37.14475650000003],[25.212015,37.12801749800008],[25.289139,37.14229949800006],[25.242174,37.01102849800003],[25.117052,36.992080498000064],[25.107309500000042,37.06349199700003],[25.237235999000063,37.14475650000003]]],[[[25.253063,37.609508497000036],[25.233768500000053,37.53944799800007],[25.114006,37.55059749700007],[24.972765500000037,37.66778949800005],[25.253063,37.609508497000036]]],[[[24.957605500000057,37.89711749800006],[25.000368,37.76972949800006],[24.963575500000047,37.683925498000065],[24.76682450000004,37.87518299800007],[24.709385,37.871387998000046],[24.684284,37.95106499800005],[24.792885,37.99790949800007],[24.83931350000006,37.91334549900006],[24.957605500000057,37.89711749800006]]],[[[24.964029500000038,37.41032399900007],[24.88783250000006,37.369590499000026],[24.855234,37.39535549900006],[24.879831500000023,37.51392349900004],[24.946517999000037,37.48527899900006],[24.964029500000038,37.41032399900007]]],[[[24.54727,36.70661149800003],[24.321747,36.64612599900005],[24.353439500000036,36.75131199900005],[24.401016,36.70126350000004],[24.469988,36.70693599600003],[24.40929250000005,36.763923498000054],[24.537983,36.75758],[24.54727,36.70661149800003]]],[[[24.48255350000005,37.39654149900008],[24.364295998000046,37.30023599900005],[24.399852499000076,37.366565499000046],[24.36558350000007,37.42344299900003],[24.435737500000073,37.46062849900005],[24.48255350000005,37.39654149900008]]],[[[24.35662650000006,37.574202497000044],[24.27572250000003,37.522402499000066],[24.267139499000052,37.60773349800007],[24.350606999000036,37.687102],[24.409574500000076,37.64549499700007],[24.35662650000006,37.574202497000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL43","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Kriti","NUTS_NAME":"Κρήτη"},"geometry":{"type":"Polygon","coordinates":[[[23.840978500000062,35.528079999000056],[23.992985,35.51167299800005],[24.078102,35.592281496000055],[24.174324,35.58719999700003],[24.201605,35.538245999000026],[24.166441,35.49475099800003],[24.0597,35.49361799700006],[24.242358,35.475036497000076],[24.26303850000005,35.36258699900003],[24.317411,35.35377499700007],[24.700668500000063,35.427009498000075],[24.928003500000045,35.40690999800006],[25.040821,35.400146498000026],[25.066084,35.339481498000055],[25.490698,35.29833199700005],[25.771959500000037,35.34151849800003],[25.726930500000037,35.30328049700006],[25.726293500000054,35.13115699700006],[25.813135,35.10961549800004],[26.023828500000036,35.22866049700008],[26.162893500000052,35.19927249800003],[26.314392500000054,35.30653],[26.263292500000034,35.265228500000035],[26.281149,35.11158749800006],[26.193916500000057,35.01488149700003],[25.550568,34.99075699700006],[24.770942500000046,34.92988599700004],[24.722517,35.09254449900004],[24.559056,35.10519399900005],[24.380876500000056,35.195171498000036],[24.28752349900003,35.17600649900004],[23.814441500000044,35.24862299800003],[23.589775,35.23132699700005],[23.515789,35.28971499900007],[23.596821,35.62816249800005],[23.61231999900008,35.52050399700005],[23.716612,35.507671499000026],[23.73757149900007,35.69526649900007],[23.773618500000055,35.66851449700005],[23.779693500000064,35.54549399800004],[23.840978500000062,35.528079999000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL431","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Irakleio","NUTS_NAME":"Ηράκλειο"},"geometry":{"type":"Polygon","coordinates":[[[25.040821,35.400146498000026],[25.066084,35.339481498000055],[25.490698,35.29833199700005],[25.512266,35.228957998000055],[25.399017499000024,35.16775899800007],[25.550568,34.99075699700006],[24.770942500000046,34.92988599700004],[24.722517,35.09254449900004],[24.789713,35.12273399800006],[24.79598450000003,35.223655499000074],[24.82736,35.180236997000065],[24.90434250000004,35.19579699700006],[24.925447500000075,35.29428850000005],[24.874618500000054,35.316539499000044],[24.928003500000045,35.40690999800006],[25.040821,35.400146498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL432","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Lasithi","NUTS_NAME":"Λασίθι"},"geometry":{"type":"Polygon","coordinates":[[[25.726293500000054,35.13115699700006],[25.813135,35.10961549800004],[26.023828500000036,35.22866049700008],[26.162893500000052,35.19927249800003],[26.314392500000054,35.30653],[26.263292500000034,35.265228500000035],[26.281149,35.11158749800006],[26.193916500000057,35.01488149700003],[25.550568,34.99075699700006],[25.399017499000024,35.16775899800007],[25.512266,35.228957998000055],[25.490698,35.29833199700005],[25.771959500000037,35.34151849800003],[25.726930500000037,35.30328049700006],[25.726293500000054,35.13115699700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL433","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Rethymni","NUTS_NAME":"Ρεθύμνη"},"geometry":{"type":"Polygon","coordinates":[[[24.928003500000045,35.40690999800006],[24.874618500000054,35.316539499000044],[24.925447500000075,35.29428850000005],[24.90434250000004,35.19579699700006],[24.82736,35.180236997000065],[24.79598450000003,35.223655499000074],[24.789713,35.12273399800006],[24.722517,35.09254449900004],[24.559056,35.10519399900005],[24.380876500000056,35.195171498000036],[24.28752349900003,35.17600649900004],[24.30460350000004,35.23831949700008],[24.270187500000077,35.24919499900005],[24.331455,35.29089749800005],[24.317411,35.35377499700007],[24.700668500000063,35.427009498000075],[24.928003500000045,35.40690999800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL434","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Chania","NUTS_NAME":"Χανιά"},"geometry":{"type":"Polygon","coordinates":[[[23.840978500000062,35.528079999000056],[23.992985,35.51167299800005],[24.078102,35.592281496000055],[24.174324,35.58719999700003],[24.201605,35.538245999000026],[24.166441,35.49475099800003],[24.0597,35.49361799700006],[24.242358,35.475036497000076],[24.26303850000005,35.36258699900003],[24.317411,35.35377499700007],[24.331455,35.29089749800005],[24.270187500000077,35.24919499900005],[24.30460350000004,35.23831949700008],[24.28752349900003,35.17600649900004],[23.814441500000044,35.24862299800003],[23.589775,35.23132699700005],[23.515789,35.28971499900007],[23.596821,35.62816249800005],[23.61231999900008,35.52050399700005],[23.716612,35.507671499000026],[23.73757149900007,35.69526649900007],[23.773618500000055,35.66851449700005],[23.779693500000064,35.54549399800004],[23.840978500000062,35.528079999000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL5","LEVL_CODE":1,"CNTR_CODE":"EL","NAME_LATN":"VOREIA ELLADA","NUTS_NAME":"ΒΟΡΕΙΑ ΕΛΛΑΔΑ"},"geometry":{"type":"MultiPolygon","coordinates":[[[[26.628431500000033,41.345533498000066],[26.321652,41.250406498000075],[26.31295750000004,41.07548049700006],[26.359039,40.94717649800003],[26.29101550000007,40.93188349800005],[26.12489,40.742109998000046],[26.032758,40.73025699800007],[26.02425,40.794869498000025],[25.945872999000073,40.842150496000045],[25.632429,40.86280449700007],[25.173954500000036,40.94000999800005],[25.139545,40.99068749800006],[25.069244500000025,41.004961998000056],[24.81349,40.85808549800004],[24.627527500000042,40.85849749800008],[24.508724,40.95848099700004],[24.394415,40.93347149700003],[24.307329,40.82701499700005],[24.08863050000008,40.72168399800006],[23.867746500000067,40.77872849800008],[23.765077500000075,40.75789249800005],[23.693813500000033,40.67448449700004],[23.756864500000063,40.634563497000045],[23.917795,40.53438549800006],[23.823702,40.50734349800007],[23.860433500000056,40.41778949700006],[24.010988,40.38943449800007],[24.005043,40.31456749800003],[23.861864,40.370163],[23.694948,40.30393999800003],[23.774591500000042,40.19285599800003],[23.99748850000003,40.10770799800008],[24.02107250000006,40.00879249900004],[23.947227500000054,39.93801899600004],[23.652324500000077,40.224616999000034],[23.401338500000065,40.27941149600008],[23.300182500000062,40.23226949800005],[23.00525649900004,40.35036050000008],[22.814283500000045,40.47844299800005],[22.97918150000004,40.54149599600004],[22.947689500000024,40.62663649700005],[22.858206,40.63497549600004],[22.85528350000004,40.59121349800006],[22.741501,40.55920799800003],[22.735582500000078,40.506091998000045],[22.682419,40.52896499800005],[22.663957500000038,40.49155449600005],[22.585716,40.46374499600006],[22.660866,40.35521299800007],[22.54789550000004,40.145462],[22.594291500000054,40.01237099700006],[22.66243,39.975547998000025],[22.508739500000047,39.956676498000036],[22.489490500000045,40.01286299800006],[22.378916,40.03908899800007],[22.370580500000074,40.14012149900003],[22.319086,40.182364998000025],[22.21593850000005,40.10359199800007],[22.20723349900004,40.19213499600005],[22.115207500000054,40.189948997000045],[21.883821500000067,39.955154496000034],[21.918218500000023,39.852611498000044],[21.42811450000005,39.89455049600008],[21.383671,39.83225249600008],[21.296484500000076,39.85777299700004],[21.226316500000053,39.80415349800006],[21.227146,39.708251999000026],[21.120413,39.679416497000034],[21.185011,39.60934849700004],[21.181753,39.501956998000026],[21.27246850000006,39.377406998000026],[21.38977250000005,39.35453799900006],[21.428675,39.295840998000074],[21.349401500000056,39.20803449700003],[21.37354650000003,39.17473599700003],[21.271143,39.117335998000044],[21.127334500000075,39.12431699800004],[21.087236500000074,39.08039099800004],[21.10860850000006,39.04614650000008],[21.047264,39.00017949700003],[20.803007,39.117034998000065],[20.738746,39.01890949800003],[20.831041500000026,38.95981199800008],[20.758966500000042,38.97145849800006],[20.757292,38.950458500000025],[20.726776,38.949634496000044],[20.705598999000074,39.04753899700006],[20.474756,39.217723996000075],[20.47782150000006,39.27484149800006],[20.464247,39.27310199800007],[20.30067850000006,39.31573499600006],[20.212507500000072,39.446082999000055],[20.274601,39.44560599700003],[20.212242,39.49127899900003],[20.262635999000054,39.51013950000004],[20.136205500000074,39.52841949800006],[20.18475150000006,39.62173099800003],[20.008812500000033,39.691292],[20.226089500000057,39.64782849900007],[20.321796,39.72885249800004],[20.289463500000068,39.80479249900003],[20.391308,39.78848349900005],[20.41706750000003,39.840948998000044],[20.31223650000004,39.991022497000074],[20.393286,39.99561699700007],[20.455286500000057,40.074255497000024],[20.678365,40.096003],[20.709584,40.273352497000076],[20.778501,40.34878899800003],[20.786272500000052,40.42695199900004],[20.843011500000046,40.47952599800004],[20.955839,40.469640998000045],[21.056068500000038,40.61669549900006],[20.959147,40.77049199800007],[20.98020450000007,40.855664999000055],[21.301891500000067,40.865341998000076],[21.417245500000035,40.917696497000065],[21.59323550000005,40.86600249700007],[21.684081,40.94091499800004],[21.787378,40.931125499000075],[21.929438,41.10035049700008],[22.216216,41.17045749700003],[22.332052,41.120272500000056],[22.587711,41.11729599900008],[22.66647,41.18633949800005],[22.732037,41.146391497000025],[22.762478,41.320234],[22.879178500000023,41.34065299700006],[22.927591500000062,41.338539498000046],[23.18137150000007,41.318869498000026],[23.322212500000035,41.399669999000025],[23.62422299900004,41.375727496000025],[23.91613249900007,41.48162649900007],[24.046779,41.45128599800006],[24.059744,41.52211199900006],[24.525637,41.56869999700007],[24.607909500000062,41.42761799600004],[24.783956,41.36018899600003],[24.92368350000004,41.402213999000026],[25.17937550000005,41.31018799700007],[25.224698500000045,41.26463099700004],[25.906437500000038,41.307574997000074],[25.948626,41.320342],[26.15868850000004,41.391182496000056],[26.17645550000003,41.554799496000044],[26.060393,41.688516],[26.16942,41.74869149700004],[26.357879,41.711104498000054],[26.60012050000006,41.60119899700004],[26.628431500000033,41.345533498000066]]],[[[25.687489,40.421697497000025],[25.560071,40.395632498000055],[25.45186650000005,40.47032149700004],[25.655822500000056,40.49385599800007],[25.687489,40.421697497000025]]],[[[24.774784,40.606056498000044],[24.640228499000045,40.58262999800007],[24.512184,40.63721449600007],[24.557409500000063,40.739089998000054],[24.65240650000004,40.80092999800007],[24.760584,40.745513999000025],[24.774784,40.606056498000044]]],[[[23.679613,39.972439],[23.750192500000026,39.915984998000056],[23.355198,39.953880498000046],[23.381927500000074,39.99904249800005],[23.308529,40.084636498000066],[23.306672026000058,40.215268835000074],[23.493444500000066,40.03395849800006],[23.679613,39.972439]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL51","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Anatoliki Makedonia, Thraki","NUTS_NAME":"Aνατολική Μακεδονία, Θράκη"},"geometry":{"type":"MultiPolygon","coordinates":[[[[26.628431500000033,41.345533498000066],[26.321652,41.250406498000075],[26.31295750000004,41.07548049700006],[26.359039,40.94717649800003],[26.29101550000007,40.93188349800005],[26.12489,40.742109998000046],[26.032758,40.73025699800007],[26.02425,40.794869498000025],[25.945872999000073,40.842150496000045],[25.632429,40.86280449700007],[25.173954500000036,40.94000999800005],[25.139545,40.99068749800006],[25.069244500000025,41.004961998000056],[24.81349,40.85808549800004],[24.627527500000042,40.85849749800008],[24.508724,40.95848099700004],[24.394415,40.93347149700003],[24.307329,40.82701499700005],[24.08863050000008,40.72168399800006],[23.867746500000067,40.77872849800008],[24.067112,40.89615649800004],[24.117927500000064,41.00608850000003],[23.990661500000044,41.11262899800005],[23.92363750000004,41.08789849800007],[23.721401,41.192653998000026],[23.716823500000032,41.30312349700006],[23.631429500000024,41.32485199900003],[23.62422299900004,41.375727496000025],[23.91613249900007,41.48162649900007],[24.046779,41.45128599800006],[24.059744,41.52211199900006],[24.525637,41.56869999700007],[24.607909500000062,41.42761799600004],[24.783956,41.36018899600003],[24.92368350000004,41.402213999000026],[25.17937550000005,41.31018799700007],[25.224698500000045,41.26463099700004],[25.906437500000038,41.307574997000074],[25.948626,41.320342],[26.15868850000004,41.391182496000056],[26.17645550000003,41.554799496000044],[26.060393,41.688516],[26.16942,41.74869149700004],[26.357879,41.711104498000054],[26.60012050000006,41.60119899700004],[26.628431500000033,41.345533498000066]]],[[[25.687489,40.421697497000025],[25.560071,40.395632498000055],[25.45186650000005,40.47032149700004],[25.655822500000056,40.49385599800007],[25.687489,40.421697497000025]]],[[[24.774784,40.606056498000044],[24.640228499000045,40.58262999800007],[24.512184,40.63721449600007],[24.557409500000063,40.739089998000054],[24.65240650000004,40.80092999800007],[24.760584,40.745513999000025],[24.774784,40.606056498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL511","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Evros","NUTS_NAME":"Έβρος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[26.12489,40.742109998000046],[26.032758,40.73025699800007],[26.02425,40.794869498000025],[25.945872999000073,40.842150496000045],[25.632429,40.86280449700007],[25.62350250000003,40.965648498000064],[25.821775500000058,40.99747849600004],[25.88195,41.10601449600006],[25.964012,41.10881450000005],[25.894016,41.19517899900006],[25.906437500000038,41.307574997000074],[25.948626,41.320342],[26.15868850000004,41.391182496000056],[26.17645550000003,41.554799496000044],[26.060393,41.688516],[26.16942,41.74869149700004],[26.357879,41.711104498000054],[26.60012050000006,41.60119899700004],[26.628431500000033,41.345533498000066],[26.321652,41.250406498000075],[26.31295750000004,41.07548049700006],[26.359039,40.94717649800003],[26.29101550000007,40.93188349800005],[26.12489,40.742109998000046]]],[[[25.687489,40.421697497000025],[25.560071,40.395632498000055],[25.45186650000005,40.47032149700004],[25.655822500000056,40.49385599800007],[25.687489,40.421697497000025]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL512","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Xanthi","NUTS_NAME":"Ξάνθη"},"geometry":{"type":"Polygon","coordinates":[[[25.17937550000005,41.31018799700007],[25.224698500000045,41.26463099700004],[25.04908,41.15163399700003],[25.139545,40.99068749800006],[25.069244500000025,41.004961998000056],[24.81349,40.85808549800004],[24.733339500000056,41.01675049700003],[24.774635499000055,41.08869549700006],[24.69499950000005,41.116164998000045],[24.695331500000066,41.20084749800003],[24.494558,41.198715],[24.460041,41.23265450000008],[24.579567,41.24049],[24.60600450000004,41.29826299600006],[24.783956,41.36018899600003],[24.92368350000004,41.402213999000026],[25.17937550000005,41.31018799700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL513","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Rodopi","NUTS_NAME":"Ροδόπη"},"geometry":{"type":"Polygon","coordinates":[[[25.906437500000038,41.307574997000074],[25.894016,41.19517899900006],[25.964012,41.10881450000005],[25.88195,41.10601449600006],[25.821775500000058,40.99747849600004],[25.62350250000003,40.965648498000064],[25.632429,40.86280449700007],[25.173954500000036,40.94000999800005],[25.139545,40.99068749800006],[25.04908,41.15163399700003],[25.224698500000045,41.26463099700004],[25.906437500000038,41.307574997000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL514","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Drama","NUTS_NAME":"Δράμα"},"geometry":{"type":"Polygon","coordinates":[[[24.783956,41.36018899600003],[24.60600450000004,41.29826299600006],[24.579567,41.24049],[24.460041,41.23265450000008],[24.494558,41.198715],[24.488365,41.13711549800007],[24.227745,40.99107749800004],[24.117927500000064,41.00608850000003],[23.990661500000044,41.11262899800005],[23.92363750000004,41.08789849800007],[23.721401,41.192653998000026],[23.716823500000032,41.30312349700006],[23.631429500000024,41.32485199900003],[23.62422299900004,41.375727496000025],[23.91613249900007,41.48162649900007],[24.046779,41.45128599800006],[24.059744,41.52211199900006],[24.525637,41.56869999700007],[24.607909500000062,41.42761799600004],[24.783956,41.36018899600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL515","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Thasos, Kavala","NUTS_NAME":"Θάσος, Καβάλα"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.733339500000056,41.01675049700003],[24.81349,40.85808549800004],[24.627527500000042,40.85849749800008],[24.508724,40.95848099700004],[24.394415,40.93347149700003],[24.307329,40.82701499700005],[24.08863050000008,40.72168399800006],[23.867746500000067,40.77872849800008],[24.067112,40.89615649800004],[24.117927500000064,41.00608850000003],[24.227745,40.99107749800004],[24.488365,41.13711549800007],[24.494558,41.198715],[24.695331500000066,41.20084749800003],[24.69499950000005,41.116164998000045],[24.774635499000055,41.08869549700006],[24.733339500000056,41.01675049700003]]],[[[24.774784,40.606056498000044],[24.640228499000045,40.58262999800007],[24.512184,40.63721449600007],[24.557409500000063,40.739089998000054],[24.65240650000004,40.80092999800007],[24.760584,40.745513999000025],[24.774784,40.606056498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL52","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Kentriki Makedonia","NUTS_NAME":"Κεντρική Μακεδονία"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.631429500000024,41.32485199900003],[23.716823500000032,41.30312349700006],[23.721401,41.192653998000026],[23.92363750000004,41.08789849800007],[23.990661500000044,41.11262899800005],[24.117927500000064,41.00608850000003],[24.067112,40.89615649800004],[23.867746500000067,40.77872849800008],[23.765077500000075,40.75789249800005],[23.693813500000033,40.67448449700004],[23.756864500000063,40.634563497000045],[23.917795,40.53438549800006],[23.823702,40.50734349800007],[23.860433500000056,40.41778949700006],[24.010988,40.38943449800007],[24.005043,40.31456749800003],[23.861864,40.370163],[23.694948,40.30393999800003],[23.774591500000042,40.19285599800003],[23.99748850000003,40.10770799800008],[24.02107250000006,40.00879249900004],[23.947227500000054,39.93801899600004],[23.652324500000077,40.224616999000034],[23.401338500000065,40.27941149600008],[23.300182500000062,40.23226949800005],[23.00525649900004,40.35036050000008],[22.814283500000045,40.47844299800005],[22.97918150000004,40.54149599600004],[22.947689500000024,40.62663649700005],[22.858206,40.63497549600004],[22.85528350000004,40.59121349800006],[22.741501,40.55920799800003],[22.735582500000078,40.506091998000045],[22.682419,40.52896499800005],[22.663957500000038,40.49155449600005],[22.585716,40.46374499600006],[22.660866,40.35521299800007],[22.54789550000004,40.145462],[22.594291500000054,40.01237099700006],[22.66243,39.975547998000025],[22.508739500000047,39.956676498000036],[22.489490500000045,40.01286299800006],[22.378916,40.03908899800007],[22.370580500000074,40.14012149900003],[22.319086,40.182364998000025],[22.21593850000005,40.10359199800007],[22.20723349900004,40.19213499600005],[22.115207500000054,40.189948997000045],[22.184179500000027,40.301849497000035],[22.092827,40.380370998000046],[22.105428500000073,40.43114849800003],[21.928503,40.48796849900003],[21.88476550000007,40.596347999000045],[21.906351,40.68133599600003],[21.822054,40.69629649800004],[21.704187,40.86348349900004],[21.830457500000023,40.89871999600007],[21.787378,40.931125499000075],[21.929438,41.10035049700008],[22.216216,41.17045749700003],[22.332052,41.120272500000056],[22.587711,41.11729599900008],[22.66647,41.18633949800005],[22.732037,41.146391497000025],[22.762478,41.320234],[22.879178500000023,41.34065299700006],[22.927591500000062,41.338539498000046],[23.18137150000007,41.318869498000026],[23.322212500000035,41.399669999000025],[23.62422299900004,41.375727496000025],[23.631429500000024,41.32485199900003]]],[[[23.679613,39.972439],[23.750192500000026,39.915984998000056],[23.355198,39.953880498000046],[23.381927500000074,39.99904249800005],[23.308529,40.084636498000066],[23.306672026000058,40.215268835000074],[23.493444500000066,40.03395849800006],[23.679613,39.972439]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL521","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Imathia","NUTS_NAME":"Ημαθία"},"geometry":{"type":"Polygon","coordinates":[[[22.491464500000063,40.71102149700005],[22.682419,40.52896499800005],[22.663957500000038,40.49155449600005],[22.591318,40.541973],[22.437425500000074,40.50824749800006],[22.184179500000027,40.301849497000035],[22.092827,40.380370998000046],[22.105428500000073,40.43114849800003],[21.928503,40.48796849900003],[21.88476550000007,40.596347999000045],[21.906351,40.68133599600003],[22.08218,40.733077998000056],[22.311773500000072,40.63983899800007],[22.387333,40.712688498000034],[22.491464500000063,40.71102149700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL522","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Thessaloniki","NUTS_NAME":"Θεσσαλονίκη"},"geometry":{"type":"Polygon","coordinates":[[[23.765077500000075,40.75789249800005],[23.693813500000033,40.67448449700004],[23.756864500000063,40.634563497000045],[23.408943,40.58703599800003],[23.39383250000003,40.53576299700006],[23.309433,40.53631599900007],[23.30523850000003,40.487048998000034],[23.17501050000004,40.48889549800003],[23.123665,40.37775399900005],[23.05888550000003,40.43103799900007],[23.00525649900004,40.35036050000008],[22.814283500000045,40.47844299800005],[22.97918150000004,40.54149599600004],[22.947689500000024,40.62663649700005],[22.858206,40.63497549600004],[22.85528350000004,40.59121349800006],[22.741501,40.55920799800003],[22.735582500000078,40.506091998000045],[22.682419,40.52896499800005],[22.491464500000063,40.71102149700005],[22.56234150000006,40.72547549600006],[22.533951,40.78999349700007],[22.63968650000004,40.840400498000065],[22.685793,40.85967999800005],[22.738985,40.79034799900006],[22.965605,40.82310849800007],[23.00814450000007,40.938274498000055],[23.240366,41.00603449700003],[23.376956500000063,40.853233],[23.765077500000075,40.75789249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL523","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Kilkis","NUTS_NAME":"Κιλκίς"},"geometry":{"type":"Polygon","coordinates":[[[23.240366,41.00603449700003],[23.00814450000007,40.938274498000055],[22.965605,40.82310849800007],[22.738985,40.79034799900006],[22.685793,40.85967999800005],[22.63968650000004,40.840400498000065],[22.26121850000004,40.95791999800008],[22.25058,41.007858498000076],[22.329323,41.045622998000056],[22.332052,41.120272500000056],[22.587711,41.11729599900008],[22.66647,41.18633949800005],[22.732037,41.146391497000025],[22.762478,41.320234],[22.879178500000023,41.34065299700006],[22.880544500000042,41.20267499900007],[22.97800650000005,41.21306599800005],[23.214777,41.10174949700007],[23.184475,41.049079998000025],[23.240366,41.00603449700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL524","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Pella","NUTS_NAME":"Πέλλα"},"geometry":{"type":"Polygon","coordinates":[[[22.332052,41.120272500000056],[22.329323,41.045622998000056],[22.25058,41.007858498000076],[22.26121850000004,40.95791999800008],[22.63968650000004,40.840400498000065],[22.533951,40.78999349700007],[22.56234150000006,40.72547549600006],[22.491464500000063,40.71102149700005],[22.387333,40.712688498000034],[22.311773500000072,40.63983899800007],[22.08218,40.733077998000056],[21.906351,40.68133599600003],[21.822054,40.69629649800004],[21.704187,40.86348349900004],[21.830457500000023,40.89871999600007],[21.787378,40.931125499000075],[21.929438,41.10035049700008],[22.216216,41.17045749700003],[22.332052,41.120272500000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL525","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Pieria","NUTS_NAME":"Πιερία"},"geometry":{"type":"Polygon","coordinates":[[[22.663957500000038,40.49155449600005],[22.585716,40.46374499600006],[22.660866,40.35521299800007],[22.54789550000004,40.145462],[22.594291500000054,40.01237099700006],[22.66243,39.975547998000025],[22.508739500000047,39.956676498000036],[22.489490500000045,40.01286299800006],[22.378916,40.03908899800007],[22.370580500000074,40.14012149900003],[22.319086,40.182364998000025],[22.21593850000005,40.10359199800007],[22.20723349900004,40.19213499600005],[22.115207500000054,40.189948997000045],[22.184179500000027,40.301849497000035],[22.437425500000074,40.50824749800006],[22.591318,40.541973],[22.663957500000038,40.49155449600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL526","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Serres","NUTS_NAME":"Σέρρες"},"geometry":{"type":"Polygon","coordinates":[[[23.62422299900004,41.375727496000025],[23.631429500000024,41.32485199900003],[23.716823500000032,41.30312349700006],[23.721401,41.192653998000026],[23.92363750000004,41.08789849800007],[23.990661500000044,41.11262899800005],[24.117927500000064,41.00608850000003],[24.067112,40.89615649800004],[23.867746500000067,40.77872849800008],[23.765077500000075,40.75789249800005],[23.376956500000063,40.853233],[23.240366,41.00603449700003],[23.184475,41.049079998000025],[23.214777,41.10174949700007],[22.97800650000005,41.21306599800005],[22.880544500000042,41.20267499900007],[22.879178500000023,41.34065299700006],[22.927591500000062,41.338539498000046],[23.18137150000007,41.318869498000026],[23.322212500000035,41.399669999000025],[23.62422299900004,41.375727496000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL527","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Chalkidiki","NUTS_NAME":"Χαλκιδική"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.756864500000063,40.634563497000045],[23.917795,40.53438549800006],[23.823702,40.50734349800007],[23.860433500000056,40.41778949700006],[24.010988,40.38943449800007],[24.005043,40.31456749800003],[23.861864,40.370163],[23.694948,40.30393999800003],[23.774591500000042,40.19285599800003],[23.99748850000003,40.10770799800008],[24.02107250000006,40.00879249900004],[23.947227500000054,39.93801899600004],[23.652324500000077,40.224616999000034],[23.401338500000065,40.27941149600008],[23.300182500000062,40.23226949800005],[23.00525649900004,40.35036050000008],[23.05888550000003,40.43103799900007],[23.123665,40.37775399900005],[23.17501050000004,40.48889549800003],[23.30523850000003,40.487048998000034],[23.309433,40.53631599900007],[23.39383250000003,40.53576299700006],[23.408943,40.58703599800003],[23.756864500000063,40.634563497000045]]],[[[23.679613,39.972439],[23.750192500000026,39.915984998000056],[23.355198,39.953880498000046],[23.381927500000074,39.99904249800005],[23.308529,40.084636498000066],[23.306672026000058,40.215268835000074],[23.493444500000066,40.03395849800006],[23.679613,39.972439]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL53","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Dytiki Makedonia","NUTS_NAME":"Δυτική Μακεδονία"},"geometry":{"type":"Polygon","coordinates":[[[21.830457500000023,40.89871999600007],[21.704187,40.86348349900004],[21.822054,40.69629649800004],[21.906351,40.68133599600003],[21.88476550000007,40.596347999000045],[21.928503,40.48796849900003],[22.105428500000073,40.43114849800003],[22.092827,40.380370998000046],[22.184179500000027,40.301849497000035],[22.115207500000054,40.189948997000045],[21.883821500000067,39.955154496000034],[21.918218500000023,39.852611498000044],[21.42811450000005,39.89455049600008],[21.383671,39.83225249600008],[21.296484500000076,39.85777299700004],[21.097473,39.88035149700005],[21.023021500000027,39.99385849600003],[21.085388,40.054710999000065],[21.01074,40.055736497000055],[20.95655050000005,40.13332349900003],[21.004242,40.15436149800007],[20.868489,40.37674599800005],[20.778501,40.34878899800003],[20.786272500000052,40.42695199900004],[20.843011500000046,40.47952599800004],[20.955839,40.469640998000045],[21.056068500000038,40.61669549900006],[20.959147,40.77049199800007],[20.98020450000007,40.855664999000055],[21.301891500000067,40.865341998000076],[21.417245500000035,40.917696497000065],[21.59323550000005,40.86600249700007],[21.684081,40.94091499800004],[21.787378,40.931125499000075],[21.830457500000023,40.89871999600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL531","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Grebena, Kozani","NUTS_NAME":"Γρεβενά, Κοζάνη"},"geometry":{"type":"Polygon","coordinates":[[[21.906351,40.68133599600003],[21.88476550000007,40.596347999000045],[21.928503,40.48796849900003],[22.105428500000073,40.43114849800003],[22.092827,40.380370998000046],[22.184179500000027,40.301849497000035],[22.115207500000054,40.189948997000045],[21.883821500000067,39.955154496000034],[21.918218500000023,39.852611498000044],[21.42811450000005,39.89455049600008],[21.383671,39.83225249600008],[21.296484500000076,39.85777299700004],[21.097473,39.88035149700005],[21.023021500000027,39.99385849600003],[21.085388,40.054710999000065],[21.01074,40.055736497000055],[20.95655050000005,40.13332349900003],[21.004242,40.15436149800007],[21.080930500000022,40.197483],[21.08828950000003,40.333144996000044],[21.279876500000057,40.361064999000064],[21.32551199900007,40.41678599800008],[21.39970550000004,40.36833649700003],[21.510357,40.52559999700003],[21.776451,40.60776149900005],[21.822054,40.69629649800004],[21.906351,40.68133599600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL532","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Kastoria","NUTS_NAME":"Καστοριά"},"geometry":{"type":"Polygon","coordinates":[[[21.510357,40.52559999700003],[21.39970550000004,40.36833649700003],[21.32551199900007,40.41678599800008],[21.279876500000057,40.361064999000064],[21.08828950000003,40.333144996000044],[21.080930500000022,40.197483],[21.004242,40.15436149800007],[20.868489,40.37674599800005],[20.778501,40.34878899800003],[20.786272500000052,40.42695199900004],[20.843011500000046,40.47952599800004],[20.955839,40.469640998000045],[21.056068500000038,40.61669549900006],[21.192651,40.649406498000076],[21.262428499000066,40.74907299700004],[21.510357,40.52559999700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC0","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Saarland","NUTS_NAME":"Saarland"},"geometry":{"type":"Polygon","coordinates":[[[7.27662250000003,49.54862349900003],[7.304916500000047,49.474195500000064],[7.252589,49.43146550000006],[7.292601500000046,49.408222496000064],[7.395898500000044,49.372045996000054],[7.402075500000024,49.36771850000008],[7.394494,49.31635199700003],[7.289732,49.23866950000007],[7.320341,49.18949899800003],[7.36875550000002,49.16145799800006],[7.293177,49.11490749700005],[7.101069,49.15599799900008],[7.057195999000044,49.11193149800005],[7.03497150000004,49.19141499700004],[6.925060500000029,49.22334649800007],[6.837607499000057,49.21293299700005],[6.834567,49.15131099800004],[6.738397500000076,49.16359799900005],[6.723465500000032,49.21882899800005],[6.565664,49.34765149700007],[6.556986,49.419208498000046],[6.367107500000031,49.469507],[6.380052500000033,49.55110499800003],[6.606324,49.527990998000064],[6.891454,49.613422497000045],[7.02798,49.63943849800006],[7.27662250000003,49.54862349900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC01","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Regionalverband Saarbrücken","NUTS_NAME":"Regionalverband Saarbrücken"},"geometry":{"type":"Polygon","coordinates":[[[7.117021499000032,49.309522498000035],[7.062578500000029,49.27141749800006],[7.12946450000004,49.24032599800006],[7.101069,49.15599799900008],[7.057195999000044,49.11193149800005],[7.03497150000004,49.19141499700004],[6.925060500000029,49.22334649800007],[6.837607499000057,49.21293299700005],[6.834567,49.15131099800004],[6.738397500000076,49.16359799900005],[6.723465500000032,49.21882899800005],[6.944278,49.37766849800005],[7.117021499000032,49.309522498000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC02","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Merzig-Wadern","NUTS_NAME":"Merzig-Wadern"},"geometry":{"type":"Polygon","coordinates":[[[6.910182,49.50170349800004],[6.697266500000069,49.37219399800006],[6.556986,49.419208498000046],[6.367107500000031,49.469507],[6.380052500000033,49.55110499800003],[6.606324,49.527990998000064],[6.891454,49.613422497000045],[6.958445,49.585980997000036],[6.95545450000003,49.51183549800004],[6.910182,49.50170349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC03","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neunkirchen","NUTS_NAME":"Neunkirchen"},"geometry":{"type":"Polygon","coordinates":[[[7.252589,49.43146550000006],[7.292601500000046,49.408222496000064],[7.221017500000073,49.36937699900005],[7.247562500000072,49.307297997000035],[7.117021499000032,49.309522498000035],[6.944278,49.37766849800005],[6.916859,49.414012498000034],[6.989319500000022,49.43675649800008],[7.252589,49.43146550000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC04","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saarlouis","NUTS_NAME":"Saarlouis"},"geometry":{"type":"Polygon","coordinates":[[[6.989319500000022,49.43675649800008],[6.916859,49.414012498000034],[6.944278,49.37766849800005],[6.723465500000032,49.21882899800005],[6.565664,49.34765149700007],[6.556986,49.419208498000046],[6.697266500000069,49.37219399800006],[6.910182,49.50170349800004],[6.989319500000022,49.43675649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC05","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saarpfalz-Kreis","NUTS_NAME":"Saarpfalz-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[7.395898500000044,49.372045996000054],[7.402075500000024,49.36771850000008],[7.394494,49.31635199700003],[7.289732,49.23866950000007],[7.320341,49.18949899800003],[7.36875550000002,49.16145799800006],[7.293177,49.11490749700005],[7.101069,49.15599799900008],[7.12946450000004,49.24032599800006],[7.062578500000029,49.27141749800006],[7.117021499000032,49.309522498000035],[7.247562500000072,49.307297997000035],[7.221017500000073,49.36937699900005],[7.292601500000046,49.408222496000064],[7.395898500000044,49.372045996000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEC06","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"St. Wendel","NUTS_NAME":"St. Wendel"},"geometry":{"type":"Polygon","coordinates":[[[7.27662250000003,49.54862349900003],[7.304916500000047,49.474195500000064],[7.252589,49.43146550000006],[6.989319500000022,49.43675649800008],[6.910182,49.50170349800004],[6.95545450000003,49.51183549800004],[6.958445,49.585980997000036],[6.891454,49.613422497000045],[7.02798,49.63943849800006],[7.27662250000003,49.54862349900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"SACHSEN","NUTS_NAME":"SACHSEN"},"geometry":{"type":"Polygon","coordinates":[[[13.207957,51.524353],[13.21015,51.404735998000035],[13.412191500000063,51.43937449800006],[13.691250500000024,51.37401299800007],[13.835313,51.37678999700006],[13.99631850000003,51.38260749600005],[14.163324500000044,51.54104299800008],[14.447771,51.54206899600007],[14.729862,51.581776998000066],[14.735769,51.52595199900003],[14.948293,51.47176899800007],[14.974183,51.36394999600003],[15.037271,51.243749998000055],[14.823362,50.87055049600008],[14.766383,50.81933250000003],[14.6188,50.85780449600003],[14.650508500000058,50.929418499000064],[14.564038,50.91849749700003],[14.599177500000053,50.98711099700006],[14.491221,51.04353049800005],[14.317873500000076,51.05469899800005],[14.258529500000066,50.98798949800005],[14.388335,50.90029899700005],[13.897461500000077,50.79214599800008],[13.855824,50.72732599700004],[13.652174,50.73035999700005],[13.525244500000042,50.70441099900006],[13.501846,50.63364299800003],[13.255104,50.59530099800003],[13.193359500000042,50.503172498000026],[13.059457500000065,50.50104749800005],[12.948144500000069,50.40431149800003],[12.820126500000072,50.45976249800003],[12.58377850000005,50.40707899800003],[12.398014999000054,50.32115349800006],[12.331084500000031,50.24271399700007],[12.335119500000076,50.17200749800003],[12.184453,50.32232449800006],[12.100900500000023,50.31802799700006],[11.919884500000023,50.42440349800006],[11.96484,50.48485799800005],[11.871462500000064,50.53982249900008],[11.944137500000068,50.59127549800007],[12.010397,50.63546099900003],[12.052068,50.55096099800005],[12.318916500000057,50.67653249800003],[12.226984500000071,50.73821849800004],[12.250825500000076,50.81831849800005],[12.652865500000075,50.923678],[12.617355,50.98079299700004],[12.501522500000021,51.06407599900007],[12.284594500000026,51.09116249600004],[12.196192500000052,51.14190749900007],[12.170904500000063,51.27550849900007],[12.147653500000047,51.31996699900003],[12.19354450000003,51.33251850000005],[12.148583,51.46312499800007],[12.198939,51.53132049700008],[12.580263,51.62606549900005],[13.051025,51.647676999000055],[13.207957,51.524353]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED2","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Dresden","NUTS_NAME":"Dresden"},"geometry":{"type":"Polygon","coordinates":[[[14.948293,51.47176899800007],[14.974183,51.36394999600003],[15.037271,51.243749998000055],[14.823362,50.87055049600008],[14.766383,50.81933250000003],[14.6188,50.85780449600003],[14.650508500000058,50.929418499000064],[14.564038,50.91849749700003],[14.599177500000053,50.98711099700006],[14.491221,51.04353049800005],[14.317873500000076,51.05469899800005],[14.258529500000066,50.98798949800005],[14.388335,50.90029899700005],[13.897461500000077,50.79214599800008],[13.855824,50.72732599700004],[13.652174,50.73035999700005],[13.50345550000003,50.84003250000006],[13.421885499000041,51.03419999800008],[13.276538,51.03103599900004],[13.198708,51.23040549800004],[13.21015,51.404735998000035],[13.412191500000063,51.43937449800006],[13.691250500000024,51.37401299800007],[13.835313,51.37678999700006],[13.99631850000003,51.38260749600005],[14.163324500000044,51.54104299800008],[14.447771,51.54206899600007],[14.729862,51.581776998000066],[14.735769,51.52595199900003],[14.948293,51.47176899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED21","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dresden, Kreisfreie Stadt","NUTS_NAME":"Dresden, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[13.965602,51.06159549800003],[13.817366,50.97518449800003],[13.580317,51.06461950000005],[13.766155500000025,51.175209996000035],[13.965602,51.06159549800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED2C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Bautzen","NUTS_NAME":"Bautzen"},"geometry":{"type":"Polygon","coordinates":[[[14.491221,51.04353049800005],[14.317873500000076,51.05469899800005],[13.965602,51.06159549800003],[13.766155500000025,51.175209996000035],[13.800807,51.30404249900005],[13.856,51.33705949800003],[13.835313,51.37678999700006],[13.99631850000003,51.38260749600005],[14.163324500000044,51.54104299800008],[14.447771,51.54206899600007],[14.522391500000026,51.43231749900008],[14.472599500000058,51.32380149800008],[14.594168500000023,51.31936649900007],[14.698345,51.18034299700008],[14.491221,51.04353049800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED2D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Görlitz","NUTS_NAME":"Görlitz"},"geometry":{"type":"Polygon","coordinates":[[[14.729862,51.581776998000066],[14.735769,51.52595199900003],[14.948293,51.47176899800007],[14.974183,51.36394999600003],[15.037271,51.243749998000055],[14.823362,50.87055049600008],[14.766383,50.81933250000003],[14.6188,50.85780449600003],[14.650508500000058,50.929418499000064],[14.564038,50.91849749700003],[14.599177500000053,50.98711099700006],[14.491221,51.04353049800005],[14.698345,51.18034299700008],[14.594168500000023,51.31936649900007],[14.472599500000058,51.32380149800008],[14.522391500000026,51.43231749900008],[14.447771,51.54206899600007],[14.729862,51.581776998000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED2E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Meißen","NUTS_NAME":"Meißen"},"geometry":{"type":"Polygon","coordinates":[[[13.691250500000024,51.37401299800007],[13.835313,51.37678999700006],[13.856,51.33705949800003],[13.800807,51.30404249900005],[13.766155500000025,51.175209996000035],[13.580317,51.06461950000005],[13.421885499000041,51.03419999800008],[13.276538,51.03103599900004],[13.198708,51.23040549800004],[13.21015,51.404735998000035],[13.412191500000063,51.43937449800006],[13.691250500000024,51.37401299800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED2F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Sächsische Schweiz-Osterzgebirge","NUTS_NAME":"Sächsische Schweiz-Osterzgebirge"},"geometry":{"type":"Polygon","coordinates":[[[14.317873500000076,51.05469899800005],[14.258529500000066,50.98798949800005],[14.388335,50.90029899700005],[13.897461500000077,50.79214599800008],[13.855824,50.72732599700004],[13.652174,50.73035999700005],[13.50345550000003,50.84003250000006],[13.421885499000041,51.03419999800008],[13.580317,51.06461950000005],[13.817366,50.97518449800003],[13.965602,51.06159549800003],[14.317873500000076,51.05469899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED4","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Chemnitz","NUTS_NAME":"Chemnitz"},"geometry":{"type":"Polygon","coordinates":[[[13.198708,51.23040549800004],[13.276538,51.03103599900004],[13.421885499000041,51.03419999800008],[13.50345550000003,50.84003250000006],[13.652174,50.73035999700005],[13.525244500000042,50.70441099900006],[13.501846,50.63364299800003],[13.255104,50.59530099800003],[13.193359500000042,50.503172498000026],[13.059457500000065,50.50104749800005],[12.948144500000069,50.40431149800003],[12.820126500000072,50.45976249800003],[12.58377850000005,50.40707899800003],[12.398014999000054,50.32115349800006],[12.331084500000031,50.24271399700007],[12.335119500000076,50.17200749800003],[12.184453,50.32232449800006],[12.100900500000023,50.31802799700006],[11.919884500000023,50.42440349800006],[11.96484,50.48485799800005],[11.871462500000064,50.53982249900008],[11.944137500000068,50.59127549800007],[12.010397,50.63546099900003],[12.052068,50.55096099800005],[12.318916500000057,50.67653249800003],[12.226984500000071,50.73821849800004],[12.250825500000076,50.81831849800005],[12.652865500000075,50.923678],[12.617355,50.98079299700004],[12.744317500000022,51.00256649900007],[12.716255,51.062260998000056],[12.743633500000044,51.107557],[12.889328999000043,51.09734599900003],[12.849263500000063,51.154375498000036],[12.926016,51.21820749900007],[13.198708,51.23040549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED41","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Chemnitz, Kreisfreie Stadt","NUTS_NAME":"Chemnitz, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[13.048769,50.80656649800005],[12.927583500000026,50.741296498000054],[12.778850500000033,50.78960599700008],[12.804274500000076,50.86691650000006],[12.897387,50.89217399800003],[13.048769,50.80656649800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED42","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Erzgebirgskreis","NUTS_NAME":"Erzgebirgskreis"},"geometry":{"type":"Polygon","coordinates":[[[13.501846,50.63364299800003],[13.255104,50.59530099800003],[13.193359500000042,50.503172498000026],[13.059457500000065,50.50104749800005],[12.948144500000069,50.40431149800003],[12.820126500000072,50.45976249800003],[12.58377850000005,50.40707899800003],[12.457726,50.502451499000074],[12.527747,50.546668999000076],[12.71091,50.642619498000045],[12.649684500000035,50.753735498000026],[12.778850500000033,50.78960599700008],[12.927583500000026,50.741296498000054],[13.048769,50.80656649800005],[13.365762500000073,50.751552],[13.501846,50.63364299800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED43","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mittelsachsen","NUTS_NAME":"Mittelsachsen"},"geometry":{"type":"Polygon","coordinates":[[[13.421885499000041,51.03419999800008],[13.50345550000003,50.84003250000006],[13.652174,50.73035999700005],[13.525244500000042,50.70441099900006],[13.501846,50.63364299800003],[13.365762500000073,50.751552],[13.048769,50.80656649800005],[12.897387,50.89217399800003],[12.804274500000076,50.86691650000006],[12.652865500000075,50.923678],[12.617355,50.98079299700004],[12.744317500000022,51.00256649900007],[12.716255,51.062260998000056],[12.743633500000044,51.107557],[12.889328999000043,51.09734599900003],[12.849263500000063,51.154375498000036],[12.926016,51.21820749900007],[13.198708,51.23040549800004],[13.276538,51.03103599900004],[13.421885499000041,51.03419999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED44","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Vogtlandkreis","NUTS_NAME":"Vogtlandkreis"},"geometry":{"type":"Polygon","coordinates":[[[12.527747,50.546668999000076],[12.457726,50.502451499000074],[12.58377850000005,50.40707899800003],[12.398014999000054,50.32115349800006],[12.331084500000031,50.24271399700007],[12.335119500000076,50.17200749800003],[12.184453,50.32232449800006],[12.100900500000023,50.31802799700006],[11.919884500000023,50.42440349800006],[11.96484,50.48485799800005],[11.871462500000064,50.53982249900008],[11.944137500000068,50.59127549800007],[12.010397,50.63546099900003],[12.052068,50.55096099800005],[12.318916500000057,50.67653249800003],[12.527747,50.546668999000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED45","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Zwickau","NUTS_NAME":"Zwickau"},"geometry":{"type":"Polygon","coordinates":[[[12.804274500000076,50.86691650000006],[12.778850500000033,50.78960599700008],[12.649684500000035,50.753735498000026],[12.71091,50.642619498000045],[12.527747,50.546668999000076],[12.318916500000057,50.67653249800003],[12.226984500000071,50.73821849800004],[12.250825500000076,50.81831849800005],[12.652865500000075,50.923678],[12.804274500000076,50.86691650000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED5","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Leipzig","NUTS_NAME":"Leipzig"},"geometry":{"type":"Polygon","coordinates":[[[12.849263500000063,51.154375498000036],[12.889328999000043,51.09734599900003],[12.743633500000044,51.107557],[12.716255,51.062260998000056],[12.744317500000022,51.00256649900007],[12.617355,50.98079299700004],[12.501522500000021,51.06407599900007],[12.284594500000026,51.09116249600004],[12.196192500000052,51.14190749900007],[12.170904500000063,51.27550849900007],[12.147653500000047,51.31996699900003],[12.19354450000003,51.33251850000005],[12.148583,51.46312499800007],[12.198939,51.53132049700008],[12.580263,51.62606549900005],[13.051025,51.647676999000055],[13.207957,51.524353],[13.21015,51.404735998000035],[13.198708,51.23040549800004],[12.926016,51.21820749900007],[12.849263500000063,51.154375498000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED51","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Leipzig, Kreisfreie Stadt","NUTS_NAME":"Leipzig, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.498738,51.36050449600003],[12.542607,51.32407250000006],[12.485246500000073,51.26136199600006],[12.350118500000065,51.29635999900006],[12.259005500000058,51.23838299600004],[12.243348500000025,51.348401996000064],[12.26825,51.41977299600006],[12.471096500000044,51.442476],[12.498738,51.36050449600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED52","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Leipzig","NUTS_NAME":"Leipzig"},"geometry":{"type":"Polygon","coordinates":[[[12.926016,51.21820749900007],[12.849263500000063,51.154375498000036],[12.889328999000043,51.09734599900003],[12.743633500000044,51.107557],[12.716255,51.062260998000056],[12.744317500000022,51.00256649900007],[12.617355,50.98079299700004],[12.501522500000021,51.06407599900007],[12.284594500000026,51.09116249600004],[12.196192500000052,51.14190749900007],[12.170904500000063,51.27550849900007],[12.147653500000047,51.31996699900003],[12.19354450000003,51.33251850000005],[12.243348500000025,51.348401996000064],[12.259005500000058,51.23838299600004],[12.350118500000065,51.29635999900006],[12.485246500000073,51.26136199600006],[12.542607,51.32407250000006],[12.498738,51.36050449600003],[12.663151500000026,51.446838],[12.942435,51.412829497000075],[12.870107,51.306012500000065],[12.926016,51.21820749900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DED53","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nordsachsen","NUTS_NAME":"Nordsachsen"},"geometry":{"type":"Polygon","coordinates":[[[13.051025,51.647676999000055],[13.207957,51.524353],[13.21015,51.404735998000035],[13.198708,51.23040549800004],[12.926016,51.21820749900007],[12.870107,51.306012500000065],[12.942435,51.412829497000075],[12.663151500000026,51.446838],[12.498738,51.36050449600003],[12.471096500000044,51.442476],[12.26825,51.41977299600006],[12.243348500000025,51.348401996000064],[12.19354450000003,51.33251850000005],[12.148583,51.46312499800007],[12.198939,51.53132049700008],[12.580263,51.62606549900005],[13.051025,51.647676999000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"SACHSEN-ANHALT","NUTS_NAME":"SACHSEN-ANHALT"},"geometry":{"type":"Polygon","coordinates":[[[12.230846,52.62975499800007],[12.14324,52.53579749800008],[12.171555,52.506336496000074],[12.31718,52.45409549800007],[12.24446,52.24800199800006],[12.297277,52.22754549900003],[12.216783,52.170068498000035],[12.276724499000068,52.104017999000064],[12.376122500000065,52.04511949700003],[12.769780500000024,51.97927449900004],[13.150913500000058,51.85961049900004],[13.12342650000005,51.856899498000075],[13.161745,51.690602],[13.051025,51.647676999000055],[12.580263,51.62606549900005],[12.198939,51.53132049700008],[12.148583,51.46312499800007],[12.19354450000003,51.33251850000005],[12.147653500000047,51.31996699900003],[12.170904500000063,51.27550849900007],[12.196192500000052,51.14190749900007],[12.284594500000026,51.09116249600004],[12.250268500000061,51.037873],[12.29299050000003,51.021581499000035],[12.224169,50.942934997000066],[12.163369500000044,50.95882249700003],[12.021018500000025,50.96912249800005],[11.696938500000044,51.08749099700003],[11.484608500000036,51.10543749800007],[11.468567500000063,51.192653998000026],[11.385374500000069,51.24588699800006],[11.473968,51.29505849800006],[11.428868500000021,51.33980999800008],[11.32126,51.41037250000005],[10.978113,51.42688499800005],[10.882051501000035,51.580507999000076],[10.916058999000029,51.616374],[10.701372,51.642187499000045],[10.58152,51.78000549700005],[10.576971,51.84298749900006],[10.648214500000051,51.908245],[10.561227,52.004065998000044],[10.801428500000043,52.04800049900007],[10.964414500000032,52.05664299700004],[11.01364650000005,52.19955599900004],[11.086244,52.22863399800008],[11.0363,52.38520949900004],[10.934542500000077,52.471794998000064],[11.008781500000055,52.49674749900004],[10.937393,52.56789999800003],[10.976968,52.62414149700004],[10.921412,52.610437],[10.759314500000073,52.79583049900003],[10.841556,52.85220499900004],[11.505027,52.94103249800003],[11.51249050000007,53.00744499900003],[11.597784500000046,53.03592649800004],[11.89485650000006,52.89410449700006],[12.12681150000003,52.89019949800007],[12.23396150000002,52.85945450000003],[12.249203500000021,52.79186199800006],[12.201294500000074,52.72900399800005],[12.230846,52.62975499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Sachsen-Anhalt","NUTS_NAME":"Sachsen-Anhalt"},"geometry":{"type":"Polygon","coordinates":[[[12.230846,52.62975499800007],[12.14324,52.53579749800008],[12.171555,52.506336496000074],[12.31718,52.45409549800007],[12.24446,52.24800199800006],[12.297277,52.22754549900003],[12.216783,52.170068498000035],[12.276724499000068,52.104017999000064],[12.376122500000065,52.04511949700003],[12.769780500000024,51.97927449900004],[13.150913500000058,51.85961049900004],[13.12342650000005,51.856899498000075],[13.161745,51.690602],[13.051025,51.647676999000055],[12.580263,51.62606549900005],[12.198939,51.53132049700008],[12.148583,51.46312499800007],[12.19354450000003,51.33251850000005],[12.147653500000047,51.31996699900003],[12.170904500000063,51.27550849900007],[12.196192500000052,51.14190749900007],[12.284594500000026,51.09116249600004],[12.250268500000061,51.037873],[12.29299050000003,51.021581499000035],[12.224169,50.942934997000066],[12.163369500000044,50.95882249700003],[12.021018500000025,50.96912249800005],[11.696938500000044,51.08749099700003],[11.484608500000036,51.10543749800007],[11.468567500000063,51.192653998000026],[11.385374500000069,51.24588699800006],[11.473968,51.29505849800006],[11.428868500000021,51.33980999800008],[11.32126,51.41037250000005],[10.978113,51.42688499800005],[10.882051501000035,51.580507999000076],[10.916058999000029,51.616374],[10.701372,51.642187499000045],[10.58152,51.78000549700005],[10.576971,51.84298749900006],[10.648214500000051,51.908245],[10.561227,52.004065998000044],[10.801428500000043,52.04800049900007],[10.964414500000032,52.05664299700004],[11.01364650000005,52.19955599900004],[11.086244,52.22863399800008],[11.0363,52.38520949900004],[10.934542500000077,52.471794998000064],[11.008781500000055,52.49674749900004],[10.937393,52.56789999800003],[10.976968,52.62414149700004],[10.921412,52.610437],[10.759314500000073,52.79583049900003],[10.841556,52.85220499900004],[11.505027,52.94103249800003],[11.51249050000007,53.00744499900003],[11.597784500000046,53.03592649800004],[11.89485650000006,52.89410449700006],[12.12681150000003,52.89019949800007],[12.23396150000002,52.85945450000003],[12.249203500000021,52.79186199800006],[12.201294500000074,52.72900399800005],[12.230846,52.62975499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE01","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dessau-Roßlau, Kreisfreie Stadt","NUTS_NAME":"Dessau-Roßlau, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.335906500000021,51.73381399900006],[12.282219500000053,51.799711],[12.120971,51.78189649800004],[12.12258550000007,51.89819749700007],[12.281266500000072,51.97029249800005],[12.335906500000021,51.73381399900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE02","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Halle (Saale), Kreisfreie Stadt","NUTS_NAME":"Halle (Saale), Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.003235500000073,51.40827899800007],[11.857053,51.51463399900007],[12.017415500000027,51.54016799800007],[12.052888500000051,51.47595149800003],[12.003235500000073,51.40827899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE03","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Magdeburg, Kreisfreie Stadt","NUTS_NAME":"Magdeburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.704552,52.19947099700005],[11.762853500000062,52.072983],[11.637069,52.030118],[11.521412,52.07233449800003],[11.568795001000069,52.17500749800007],[11.704552,52.19947099700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE04","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Altmarkkreis Salzwedel","NUTS_NAME":"Altmarkkreis Salzwedel"},"geometry":{"type":"Polygon","coordinates":[[[11.587162500000034,52.45869449700007],[11.446087500000033,52.386873999000045],[11.008781500000055,52.49674749900004],[10.937393,52.56789999800003],[10.976968,52.62414149700004],[10.921412,52.610437],[10.759314500000073,52.79583049900003],[10.841556,52.85220499900004],[11.505027,52.94103249800003],[11.632403,52.86079350000006],[11.46881650000006,52.841572496000026],[11.562651500000072,52.766234498000074],[11.490416,52.65167149800004],[11.396673500000077,52.622411999000064],[11.588016,52.61669449800007],[11.587162500000034,52.45869449700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE05","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Anhalt-Bitterfeld","NUTS_NAME":"Anhalt-Bitterfeld"},"geometry":{"type":"Polygon","coordinates":[[[12.376122500000065,52.04511949700003],[12.281266500000072,51.97029249800005],[12.12258550000007,51.89819749700007],[12.120971,51.78189649800004],[12.282219500000053,51.799711],[12.335906500000021,51.73381399900006],[12.580263,51.62606549900005],[12.198939,51.53132049700008],[11.865443500000026,51.67539149800007],[11.857465500000046,51.837535],[12.019559,51.880395501000066],[11.890957,52.00206049600007],[12.276724499000068,52.104017999000064],[12.376122500000065,52.04511949700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE06","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Jerichower Land","NUTS_NAME":"Jerichower Land"},"geometry":{"type":"Polygon","coordinates":[[[12.171555,52.506336496000074],[12.31718,52.45409549800007],[12.24446,52.24800199800006],[12.297277,52.22754549900003],[12.216783,52.170068498000035],[12.276724499000068,52.104017999000064],[11.890957,52.00206049600007],[11.762853500000062,52.072983],[11.704552,52.19947099700005],[11.805296500000054,52.33461149900006],[11.81831,52.334209498000064],[11.851885500000037,52.33157799700007],[11.992091,52.430312498000035],[11.980155,52.53748749600004],[11.998665,52.570951498000056],[11.995299,52.51919449700006],[12.171555,52.506336496000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE07","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Börde","NUTS_NAME":"Börde"},"geometry":{"type":"Polygon","coordinates":[[[11.008781500000055,52.49674749900004],[11.446087500000033,52.386873999000045],[11.587162500000034,52.45869449700007],[11.700076,52.459159999000065],[11.851885500000037,52.33157799700007],[11.81831,52.334209498000064],[11.805296500000054,52.33461149900006],[11.704552,52.19947099700005],[11.568795001000069,52.17500749800007],[11.521412,52.07233449800003],[11.637069,52.030118],[11.597357499000054,51.96597599900008],[11.415964,52.00471649900004],[11.340847500000052,51.901234],[10.964414500000032,52.05664299700004],[11.01364650000005,52.19955599900004],[11.086244,52.22863399800008],[11.0363,52.38520949900004],[10.934542500000077,52.471794998000064],[11.008781500000055,52.49674749900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE08","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Burgenland (DE)","NUTS_NAME":"Burgenland (DE)"},"geometry":{"type":"Polygon","coordinates":[[[12.170904500000063,51.27550849900007],[12.196192500000052,51.14190749900007],[12.284594500000026,51.09116249600004],[12.250268500000061,51.037873],[12.29299050000003,51.021581499000035],[12.224169,50.942934997000066],[12.163369500000044,50.95882249700003],[12.021018500000025,50.96912249800005],[11.696938500000044,51.08749099700003],[11.484608500000036,51.10543749800007],[11.468567500000063,51.192653998000026],[11.385374500000069,51.24588699800006],[11.473968,51.29505849800006],[11.634900500000072,51.31421749800006],[11.848112,51.23590549700003],[12.170904500000063,51.27550849900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE09","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Harz","NUTS_NAME":"Harz"},"geometry":{"type":"Polygon","coordinates":[[[11.340847500000052,51.901234],[11.239632,51.808251],[11.401613500000053,51.72993949800008],[11.130536,51.56637399700003],[10.916058999000029,51.616374],[10.701372,51.642187499000045],[10.58152,51.78000549700005],[10.576971,51.84298749900006],[10.648214500000051,51.908245],[10.561227,52.004065998000044],[10.801428500000043,52.04800049900007],[10.964414500000032,52.05664299700004],[11.340847500000052,51.901234]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Mansfeld-Südharz","NUTS_NAME":"Mansfeld-Südharz"},"geometry":{"type":"Polygon","coordinates":[[[11.751612500000022,51.633283498000026],[11.68566450000003,51.56953299800006],[11.76545999900003,51.44443449800008],[11.544621500000062,51.437812],[11.428868500000021,51.33980999800008],[11.32126,51.41037250000005],[10.978113,51.42688499800005],[10.882051501000035,51.580507999000076],[10.916058999000029,51.616374],[11.130536,51.56637399700003],[11.401613500000053,51.72993949800008],[11.751612500000022,51.633283498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saalekreis","NUTS_NAME":"Saalekreis"},"geometry":{"type":"Polygon","coordinates":[[[12.198939,51.53132049700008],[12.148583,51.46312499800007],[12.19354450000003,51.33251850000005],[12.147653500000047,51.31996699900003],[12.170904500000063,51.27550849900007],[11.848112,51.23590549700003],[11.634900500000072,51.31421749800006],[11.473968,51.29505849800006],[11.428868500000021,51.33980999800008],[11.544621500000062,51.437812],[11.76545999900003,51.44443449800008],[11.68566450000003,51.56953299800006],[11.751612500000022,51.633283498000026],[11.865443500000026,51.67539149800007],[12.198939,51.53132049700008]],[[12.003235500000073,51.40827899800007],[12.052888500000051,51.47595149800003],[12.017415500000027,51.54016799800007],[11.857053,51.51463399900007],[12.003235500000073,51.40827899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Salzlandkreis","NUTS_NAME":"Salzlandkreis"},"geometry":{"type":"Polygon","coordinates":[[[11.890957,52.00206049600007],[12.019559,51.880395501000066],[11.857465500000046,51.837535],[11.865443500000026,51.67539149800007],[11.751612500000022,51.633283498000026],[11.401613500000053,51.72993949800008],[11.239632,51.808251],[11.340847500000052,51.901234],[11.415964,52.00471649900004],[11.597357499000054,51.96597599900008],[11.637069,52.030118],[11.762853500000062,52.072983],[11.890957,52.00206049600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Stendal","NUTS_NAME":"Stendal"},"geometry":{"type":"Polygon","coordinates":[[[12.12681150000003,52.89019949800007],[12.23396150000002,52.85945450000003],[12.249203500000021,52.79186199800006],[12.201294500000074,52.72900399800005],[12.230846,52.62975499800007],[12.14324,52.53579749800008],[12.171555,52.506336496000074],[11.995299,52.51919449700006],[11.998665,52.570951498000056],[11.980155,52.53748749600004],[11.992091,52.430312498000035],[11.851885500000037,52.33157799700007],[11.700076,52.459159999000065],[11.587162500000034,52.45869449700007],[11.588016,52.61669449800007],[11.396673500000077,52.622411999000064],[11.490416,52.65167149800004],[11.562651500000072,52.766234498000074],[11.46881650000006,52.841572496000026],[11.632403,52.86079350000006],[11.505027,52.94103249800003],[11.51249050000007,53.00744499900003],[11.597784500000046,53.03592649800004],[11.89485650000006,52.89410449700006],[12.12681150000003,52.89019949800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEE0E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wittenberg","NUTS_NAME":"Wittenberg"},"geometry":{"type":"Polygon","coordinates":[[[12.769780500000024,51.97927449900004],[13.150913500000058,51.85961049900004],[13.12342650000005,51.856899498000075],[13.161745,51.690602],[13.051025,51.647676999000055],[12.580263,51.62606549900005],[12.335906500000021,51.73381399900006],[12.281266500000072,51.97029249800005],[12.376122500000065,52.04511949700003],[12.769780500000024,51.97927449900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"SCHLESWIG-HOLSTEIN","NUTS_NAME":"SCHLESWIG-HOLSTEIN"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.22579999900006,54.50503149700006],[11.312892500000032,54.40277],[11.004257,54.44900249900007],[11.066274500000077,54.53572399800004],[11.22579999900006,54.50503149700006]]],[[[8.352822,54.96785699600008],[8.372664500000042,54.896856497000044],[8.635925999000051,54.91168299700007],[9.113097,54.873601499000074],[9.420151500000031,54.83195650000005],[9.422934,54.82322299700007],[9.491949500000032,54.82263549800007],[9.584703,54.865198500000076],[9.873658500000033,54.75218399900007],[9.909055,54.79868599800005],[10.03117,54.636575],[9.995853,54.519735499000035],[9.832854,54.47355900100007],[10.198897501000033,54.45499049600005],[10.168517,54.432844997000075],[10.136784,54.315980498000044],[10.174162,54.34574549800004],[10.358537,54.43131899800005],[10.71375049900007,54.30507250000005],[11.125609500000053,54.373613499000044],[11.071833,54.339934999000036],[11.093783,54.199050500000055],[10.755363500000044,54.05405150000007],[10.840886001000058,53.991893997000034],[10.903661501000045,53.956822],[10.968375500000036,53.90886199800008],[10.770835501000022,53.87346249600006],[10.762964500000066,53.81115299900006],[10.771398,53.74950499700003],[10.951918500000033,53.64762249800003],[10.911107500000071,53.57257450000003],[10.634904500000061,53.45603299900006],[10.595047,53.363927500000045],[10.469184,53.38584399900003],[10.308312,53.433221497000034],[10.236678500000039,53.49635449700003],[10.151152500000023,53.545018],[10.22176150000007,53.633722499000044],[10.140273,53.68012649800005],[10.191279,53.72218649800004],[10.072805,53.70963399800007],[9.945376,53.65292799700006],[9.824958,53.584528],[9.770294,53.62746249600008],[9.73010399900005,53.55758099600007],[9.550639999000055,53.616034497000044],[9.48589,53.70766449700005],[9.19975099900006,53.88010449700005],[9.02242,53.87951749900003],[8.832592,54.02392949800003],[8.985603500000025,54.05827499600008],[8.808867500000076,54.17103749900008],[8.844804500000066,54.26632849900005],[8.602134,54.34050799600004],[9.010464,54.49682299800003],[8.806592500000022,54.470951496000055],[8.892347,54.597899],[8.594786,54.881965998000055],[8.340561500000035,54.88057449900003],[8.278555500000039,54.752207998000074],[8.297903,54.91065650000007],[8.416845499000033,55.05838349600003],[8.464195500000073,55.04571499700006],[8.352822,54.96785699600008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Schleswig-Holstein","NUTS_NAME":"Schleswig-Holstein"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.22579999900006,54.50503149700006],[11.312892500000032,54.40277],[11.004257,54.44900249900007],[11.066274500000077,54.53572399800004],[11.22579999900006,54.50503149700006]]],[[[8.352822,54.96785699600008],[8.372664500000042,54.896856497000044],[8.635925999000051,54.91168299700007],[9.113097,54.873601499000074],[9.420151500000031,54.83195650000005],[9.422934,54.82322299700007],[9.491949500000032,54.82263549800007],[9.584703,54.865198500000076],[9.873658500000033,54.75218399900007],[9.909055,54.79868599800005],[10.03117,54.636575],[9.995853,54.519735499000035],[9.832854,54.47355900100007],[10.198897501000033,54.45499049600005],[10.168517,54.432844997000075],[10.136784,54.315980498000044],[10.174162,54.34574549800004],[10.358537,54.43131899800005],[10.71375049900007,54.30507250000005],[11.125609500000053,54.373613499000044],[11.071833,54.339934999000036],[11.093783,54.199050500000055],[10.755363500000044,54.05405150000007],[10.840886001000058,53.991893997000034],[10.903661501000045,53.956822],[10.968375500000036,53.90886199800008],[10.770835501000022,53.87346249600006],[10.762964500000066,53.81115299900006],[10.771398,53.74950499700003],[10.951918500000033,53.64762249800003],[10.911107500000071,53.57257450000003],[10.634904500000061,53.45603299900006],[10.595047,53.363927500000045],[10.469184,53.38584399900003],[10.308312,53.433221497000034],[10.236678500000039,53.49635449700003],[10.151152500000023,53.545018],[10.22176150000007,53.633722499000044],[10.140273,53.68012649800005],[10.191279,53.72218649800004],[10.072805,53.70963399800007],[9.945376,53.65292799700006],[9.824958,53.584528],[9.770294,53.62746249600008],[9.73010399900005,53.55758099600007],[9.550639999000055,53.616034497000044],[9.48589,53.70766449700005],[9.19975099900006,53.88010449700005],[9.02242,53.87951749900003],[8.832592,54.02392949800003],[8.985603500000025,54.05827499600008],[8.808867500000076,54.17103749900008],[8.844804500000066,54.26632849900005],[8.602134,54.34050799600004],[9.010464,54.49682299800003],[8.806592500000022,54.470951496000055],[8.892347,54.597899],[8.594786,54.881965998000055],[8.340561500000035,54.88057449900003],[8.278555500000039,54.752207998000074],[8.297903,54.91065650000007],[8.416845499000033,55.05838349600003],[8.464195500000073,55.04571499700006],[8.352822,54.96785699600008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF01","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Flensburg, Kreisfreie Stadt","NUTS_NAME":"Flensburg, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[9.491949500000032,54.82263549800007],[9.505818,54.77311099800005],[9.452375499000027,54.75204899800008],[9.35722050000004,54.77948199600007],[9.422934,54.82322299700007],[9.491949500000032,54.82263549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF02","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kiel, Kreisfreie Stadt","NUTS_NAME":"Kiel, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.174162,54.34574549800004],[10.217796500000077,54.32940699800008],[10.202694,54.269626499000026],[10.123428,54.25551350000006],[10.033944,54.354512501000045],[10.168517,54.432844997000075],[10.136784,54.315980498000044],[10.174162,54.34574549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF03","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Lübeck, Kreisfreie Stadt","NUTS_NAME":"Lübeck, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.903661501000045,53.956822],[10.968375500000036,53.90886199800008],[10.770835501000022,53.87346249600006],[10.762964500000066,53.81115299900006],[10.582456,53.80468399600005],[10.554771,53.82685299700006],[10.579575500000033,53.89762649700003],[10.811840501000063,53.93385699700008],[10.840886001000058,53.991893997000034],[10.903661501000045,53.956822]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF04","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Neumünster, Kreisfreie Stadt","NUTS_NAME":"Neumünster, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.034851500000059,54.12878499700008],[10.02338,54.05976699900003],[10.055894500000022,54.05712949900004],[9.94435899900003,54.021762497000054],[9.928784500000063,54.10274149600008],[10.034851500000059,54.12878499700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF05","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Dithmarschen","NUTS_NAME":"Dithmarschen"},"geometry":{"type":"Polygon","coordinates":[[[9.175955,54.33493199600008],[9.393511,54.288965],[9.442803,54.17381899800006],[9.311997,54.128045],[9.31108450000005,54.063436996000064],[9.19975099900006,53.88010449700005],[9.02242,53.87951749900003],[8.832592,54.02392949800003],[8.985603500000025,54.05827499600008],[8.808867500000076,54.17103749900008],[8.844804500000066,54.26632849900005],[9.000954500000034,54.375939],[9.175955,54.33493199600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF06","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Herzogtum Lauenburg","NUTS_NAME":"Herzogtum Lauenburg"},"geometry":{"type":"Polygon","coordinates":[[[10.762964500000066,53.81115299900006],[10.771398,53.74950499700003],[10.951918500000033,53.64762249800003],[10.911107500000071,53.57257450000003],[10.634904500000061,53.45603299900006],[10.595047,53.363927500000045],[10.469184,53.38584399900003],[10.308312,53.433221497000034],[10.236678500000039,53.49635449700003],[10.508869,53.60890349700003],[10.410439,53.70238649800007],[10.582456,53.80468399600005],[10.762964500000066,53.81115299900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF07","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nordfriesland","NUTS_NAME":"Nordfriesland"},"geometry":{"type":"Polygon","coordinates":[[[8.372664500000042,54.896856497000044],[8.635925999000051,54.91168299700007],[9.113097,54.873601499000074],[9.120198500000072,54.788125998000055],[9.062382,54.70077150000003],[9.222578500000054,54.64803499700008],[9.310877,54.55957499800007],[9.325775500000077,54.446862],[9.175955,54.33493199600008],[9.000954500000034,54.375939],[8.844804500000066,54.26632849900005],[8.602134,54.34050799600004],[9.010464,54.49682299800003],[8.806592500000022,54.470951496000055],[8.892347,54.597899],[8.594786,54.881965998000055],[8.340561500000035,54.88057449900003],[8.278555500000039,54.752207998000074],[8.297903,54.91065650000007],[8.416845499000033,55.05838349600003],[8.464195500000073,55.04571499700006],[8.352822,54.96785699600008],[8.372664500000042,54.896856497000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF08","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ostholstein","NUTS_NAME":"Ostholstein"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.22579999900006,54.50503149700006],[11.312892500000032,54.40277],[11.004257,54.44900249900007],[11.066274500000077,54.53572399800004],[11.22579999900006,54.50503149700006]]],[[[11.071833,54.339934999000036],[11.093783,54.199050500000055],[10.755363500000044,54.05405150000007],[10.840886001000058,53.991893997000034],[10.811840501000063,53.93385699700008],[10.579575500000033,53.89762649700003],[10.533857500000067,53.91881049600005],[10.448258500000065,53.98624349800008],[10.563627500000052,54.06823049900004],[10.417471001000024,54.083244997000065],[10.538374500000032,54.12266550000004],[10.482442,54.17707349900007],[10.540552500000047,54.227956996000046],[10.716335,54.21439699800004],[10.71375049900007,54.30507250000005],[11.125609500000053,54.373613499000044],[11.071833,54.339934999000036]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF09","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Pinneberg","NUTS_NAME":"Pinneberg"},"geometry":{"type":"Polygon","coordinates":[[[9.945376,53.65292799700006],[9.824958,53.584528],[9.770294,53.62746249600008],[9.73010399900005,53.55758099600007],[9.550639999000055,53.616034497000044],[9.48589,53.70766449700005],[9.648689500000046,53.78160649800003],[9.65566750000005,53.88644299600003],[9.751445,53.89303849800007],[9.940644,53.75199999800003],[9.945376,53.65292799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Plön","NUTS_NAME":"Plön"},"geometry":{"type":"Polygon","coordinates":[[[10.71375049900007,54.30507250000005],[10.716335,54.21439699800004],[10.540552500000047,54.227956996000046],[10.482442,54.17707349900007],[10.538374500000032,54.12266550000004],[10.417471001000024,54.083244997000065],[10.055894500000022,54.05712949900004],[10.02338,54.05976699900003],[10.034851500000059,54.12878499700008],[10.11614650000007,54.12959050100005],[10.123428,54.25551350000006],[10.202694,54.269626499000026],[10.217796500000077,54.32940699800008],[10.174162,54.34574549800004],[10.358537,54.43131899800005],[10.71375049900007,54.30507250000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Rendsburg-Eckernförde","NUTS_NAME":"Rendsburg-Eckernförde"},"geometry":{"type":"Polygon","coordinates":[[[10.03117,54.636575],[9.995853,54.519735499000035],[9.832854,54.47355900100007],[10.198897501000033,54.45499049600005],[10.168517,54.432844997000075],[10.033944,54.354512501000045],[10.123428,54.25551350000006],[10.11614650000007,54.12959050100005],[10.034851500000059,54.12878499700008],[9.928784500000063,54.10274149600008],[9.94435899900003,54.021762497000054],[9.864329500000053,54.01203049800006],[9.523900500000025,54.09559399700004],[9.31108450000005,54.063436996000064],[9.311997,54.128045],[9.442803,54.17381899800006],[9.393511,54.288965],[9.414697,54.345981996000035],[9.556189500000073,54.35786150000007],[9.556244500000048,54.43019399600007],[9.675124500000038,54.496073498000044],[9.945232,54.63672500100006],[10.03117,54.636575]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schleswig-Flensburg","NUTS_NAME":"Schleswig-Flensburg"},"geometry":{"type":"Polygon","coordinates":[[[9.420151500000031,54.83195650000005],[9.422934,54.82322299700007],[9.35722050000004,54.77948199600007],[9.452375499000027,54.75204899800008],[9.505818,54.77311099800005],[9.491949500000032,54.82263549800007],[9.584703,54.865198500000076],[9.873658500000033,54.75218399900007],[9.909055,54.79868599800005],[10.03117,54.636575],[9.945232,54.63672500100006],[9.675124500000038,54.496073498000044],[9.556244500000048,54.43019399600007],[9.556189500000073,54.35786150000007],[9.414697,54.345981996000035],[9.393511,54.288965],[9.175955,54.33493199600008],[9.325775500000077,54.446862],[9.310877,54.55957499800007],[9.222578500000054,54.64803499700008],[9.062382,54.70077150000003],[9.120198500000072,54.788125998000055],[9.113097,54.873601499000074],[9.420151500000031,54.83195650000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Segeberg","NUTS_NAME":"Segeberg"},"geometry":{"type":"Polygon","coordinates":[[[10.417471001000024,54.083244997000065],[10.563627500000052,54.06823049900004],[10.448258500000065,53.98624349800008],[10.533857500000067,53.91881049600005],[10.022741500000052,53.74953099700008],[10.072805,53.70963399800007],[9.945376,53.65292799700006],[9.940644,53.75199999800003],[9.751445,53.89303849800007],[9.77738000100004,53.98052349900007],[9.864329500000053,54.01203049800006],[9.94435899900003,54.021762497000054],[10.055894500000022,54.05712949900004],[10.417471001000024,54.083244997000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Steinburg","NUTS_NAME":"Steinburg"},"geometry":{"type":"Polygon","coordinates":[[[9.864329500000053,54.01203049800006],[9.77738000100004,53.98052349900007],[9.751445,53.89303849800007],[9.65566750000005,53.88644299600003],[9.648689500000046,53.78160649800003],[9.48589,53.70766449700005],[9.19975099900006,53.88010449700005],[9.31108450000005,54.063436996000064],[9.523900500000025,54.09559399700004],[9.864329500000053,54.01203049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEF0F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Stormarn","NUTS_NAME":"Stormarn"},"geometry":{"type":"Polygon","coordinates":[[[10.579575500000033,53.89762649700003],[10.554771,53.82685299700006],[10.582456,53.80468399600005],[10.410439,53.70238649800007],[10.508869,53.60890349700003],[10.236678500000039,53.49635449700003],[10.151152500000023,53.545018],[10.22176150000007,53.633722499000044],[10.140273,53.68012649800005],[10.191279,53.72218649800004],[10.072805,53.70963399800007],[10.022741500000052,53.74953099700008],[10.533857500000067,53.91881049600005],[10.579575500000033,53.89762649700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG","LEVL_CODE":1,"CNTR_CODE":"DE","NAME_LATN":"THÜRINGEN","NUTS_NAME":"THÜRINGEN"},"geometry":{"type":"Polygon","coordinates":[[[12.163369500000044,50.95882249700003],[12.224169,50.942934997000066],[12.29299050000003,51.021581499000035],[12.250268500000061,51.037873],[12.284594500000026,51.09116249600004],[12.501522500000021,51.06407599900007],[12.617355,50.98079299700004],[12.652865500000075,50.923678],[12.250825500000076,50.81831849800005],[12.226984500000071,50.73821849800004],[12.318916500000057,50.67653249800003],[12.052068,50.55096099800005],[12.010397,50.63546099900003],[11.944137500000068,50.59127549800007],[11.871462500000064,50.53982249900008],[11.96484,50.48485799800005],[11.919884500000023,50.42440349800006],[11.603290500000071,50.398766],[11.519853,50.37426849800005],[11.481568,50.43162150000006],[11.420630500000073,50.443366499000035],[11.423893,50.51606449800005],[11.265938,50.47942099900007],[11.259778,50.27494649800008],[11.189943,50.27118550000006],[11.115305,50.36700699600004],[10.945718500000055,50.38646649800006],[10.715266500000041,50.363590997000074],[10.851494500000058,50.26276249800003],[10.729202,50.23000549600005],[10.610115,50.22799499800004],[10.600550500000054,50.33368799800007],[10.450532,50.40185949900007],[10.12726,50.559533],[10.041338500000052,50.51646949800005],[10.037268,50.61088399800008],[10.077555500000074,50.63762399600006],[9.878551,50.63478099800005],[9.926176500000054,50.767389998000056],[10.055980999000042,50.879771497000036],[9.949583500000074,50.927478499000074],[10.063865,50.94543299600008],[10.021558500000026,50.992949499000076],[10.182348500000046,50.99848749900008],[10.202181,51.012009],[10.118321500000036,51.13957299900005],[10.210013,51.144082998000044],[10.235999,51.18476749800004],[10.206942,51.19064899600005],[10.067095,51.22779299800004],[9.928339,51.37529899900005],[10.365365,51.555891],[10.488551,51.57477849700007],[10.677283,51.638376],[10.701372,51.642187499000045],[10.916058999000029,51.616374],[10.882051501000035,51.580507999000076],[10.978113,51.42688499800005],[11.32126,51.41037250000005],[11.428868500000021,51.33980999800008],[11.473968,51.29505849800006],[11.385374500000069,51.24588699800006],[11.468567500000063,51.192653998000026],[11.484608500000036,51.10543749800007],[11.696938500000044,51.08749099700003],[12.021018500000025,50.96912249800005],[12.163369500000044,50.95882249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0","LEVL_CODE":2,"CNTR_CODE":"DE","NAME_LATN":"Thüringen","NUTS_NAME":"Thüringen"},"geometry":{"type":"Polygon","coordinates":[[[12.163369500000044,50.95882249700003],[12.224169,50.942934997000066],[12.29299050000003,51.021581499000035],[12.250268500000061,51.037873],[12.284594500000026,51.09116249600004],[12.501522500000021,51.06407599900007],[12.617355,50.98079299700004],[12.652865500000075,50.923678],[12.250825500000076,50.81831849800005],[12.226984500000071,50.73821849800004],[12.318916500000057,50.67653249800003],[12.052068,50.55096099800005],[12.010397,50.63546099900003],[11.944137500000068,50.59127549800007],[11.871462500000064,50.53982249900008],[11.96484,50.48485799800005],[11.919884500000023,50.42440349800006],[11.603290500000071,50.398766],[11.519853,50.37426849800005],[11.481568,50.43162150000006],[11.420630500000073,50.443366499000035],[11.423893,50.51606449800005],[11.265938,50.47942099900007],[11.259778,50.27494649800008],[11.189943,50.27118550000006],[11.115305,50.36700699600004],[10.945718500000055,50.38646649800006],[10.715266500000041,50.363590997000074],[10.851494500000058,50.26276249800003],[10.729202,50.23000549600005],[10.610115,50.22799499800004],[10.600550500000054,50.33368799800007],[10.450532,50.40185949900007],[10.12726,50.559533],[10.041338500000052,50.51646949800005],[10.037268,50.61088399800008],[10.077555500000074,50.63762399600006],[9.878551,50.63478099800005],[9.926176500000054,50.767389998000056],[10.055980999000042,50.879771497000036],[9.949583500000074,50.927478499000074],[10.063865,50.94543299600008],[10.021558500000026,50.992949499000076],[10.182348500000046,50.99848749900008],[10.202181,51.012009],[10.118321500000036,51.13957299900005],[10.210013,51.144082998000044],[10.235999,51.18476749800004],[10.206942,51.19064899600005],[10.067095,51.22779299800004],[9.928339,51.37529899900005],[10.365365,51.555891],[10.488551,51.57477849700007],[10.677283,51.638376],[10.701372,51.642187499000045],[10.916058999000029,51.616374],[10.882051501000035,51.580507999000076],[10.978113,51.42688499800005],[11.32126,51.41037250000005],[11.428868500000021,51.33980999800008],[11.473968,51.29505849800006],[11.385374500000069,51.24588699800006],[11.468567500000063,51.192653998000026],[11.484608500000036,51.10543749800007],[11.696938500000044,51.08749099700003],[12.021018500000025,50.96912249800005],[12.163369500000044,50.95882249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG01","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Erfurt, Kreisfreie Stadt","NUTS_NAME":"Erfurt, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.170664500000044,51.018729],[11.11025950000004,50.91311249800003],[10.936165,50.894874497000046],[10.959487500000023,50.93419449900006],[10.865766,50.96489349700005],[10.862985500000036,51.02654399800008],[11.066442,51.07941299600003],[11.170664500000044,51.018729]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG02","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gera, Kreisfreie Stadt","NUTS_NAME":"Gera, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[12.163369500000044,50.95882249700003],[12.134889500000043,50.82057649800004],[11.998804500000062,50.84946549800003],[12.021172500000034,50.94795250000004],[12.021018500000025,50.96912249800005],[12.163369500000044,50.95882249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG03","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Jena, Kreisfreie Stadt","NUTS_NAME":"Jena, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.504811,50.90624249900003],[11.547322,50.98905749900007],[11.66578050000004,50.95766099800005],[11.661936,50.88429799800008],[11.504811,50.90624249900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG04","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Suhl, Kreisfreie Stadt","NUTS_NAME":"Suhl, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.826985500000035,50.580954],[10.582464,50.58543599700005],[10.552235500000052,50.61169999800006],[10.733734500000025,50.663490498000044],[10.826985500000035,50.580954]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG05","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Weimar, Kreisfreie Stadt","NUTS_NAME":"Weimar, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[11.402000500000042,50.96405249700007],[11.292821,50.91785099800006],[11.241368500000021,51.02350149800003],[11.373274,51.02006849900005],[11.402000500000042,50.96405249700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG06","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Eichsfeld","NUTS_NAME":"Eichsfeld"},"geometry":{"type":"Polygon","coordinates":[[[10.488551,51.57477849700007],[10.513822,51.49569949700003],[10.480504,51.41674449900006],[10.52317,51.36973549800007],[10.483566,51.343304],[10.391915500000039,51.34349799800003],[10.318011500000068,51.25125649800003],[10.185303500000032,51.218741496000064],[10.206942,51.19064899600005],[10.067095,51.22779299800004],[9.928339,51.37529899900005],[10.365365,51.555891],[10.488551,51.57477849700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG07","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Nordhausen","NUTS_NAME":"Nordhausen"},"geometry":{"type":"Polygon","coordinates":[[[10.916058999000029,51.616374],[10.882051501000035,51.580507999000076],[10.978113,51.42688499800005],[10.52317,51.36973549800007],[10.480504,51.41674449900006],[10.513822,51.49569949700003],[10.488551,51.57477849700007],[10.677283,51.638376],[10.701372,51.642187499000045],[10.916058999000029,51.616374]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG09","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Unstrut-Hainich-Kreis","NUTS_NAME":"Unstrut-Hainich-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[10.483566,51.343304],[10.60940450000004,51.318512500000054],[10.709873,51.21647499800008],[10.922568500000068,51.20532899800003],[10.880309,51.09825249800008],[10.614710500000058,51.03916299800005],[10.210013,51.144082998000044],[10.235999,51.18476749800004],[10.206942,51.19064899600005],[10.185303500000032,51.218741496000064],[10.318011500000068,51.25125649800003],[10.391915500000039,51.34349799800003],[10.483566,51.343304]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0A","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Kyffhäuserkreis","NUTS_NAME":"Kyffhäuserkreis"},"geometry":{"type":"Polygon","coordinates":[[[11.428868500000021,51.33980999800008],[11.473968,51.29505849800006],[11.385374500000069,51.24588699800006],[11.213871,51.24093949600007],[11.06900150000007,51.322618],[10.922568500000068,51.20532899800003],[10.709873,51.21647499800008],[10.60940450000004,51.318512500000054],[10.483566,51.343304],[10.52317,51.36973549800007],[10.978113,51.42688499800005],[11.32126,51.41037250000005],[11.428868500000021,51.33980999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0B","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Schmalkalden-Meiningen","NUTS_NAME":"Schmalkalden-Meiningen"},"geometry":{"type":"Polygon","coordinates":[[[10.752704,50.71847099800004],[10.733734500000025,50.663490498000044],[10.552235500000052,50.61169999800006],[10.582464,50.58543599700005],[10.566062500000044,50.439793999000074],[10.450532,50.40185949900007],[10.12726,50.559533],[10.041338500000052,50.51646949800005],[10.037268,50.61088399800008],[10.077555500000074,50.63762399600006],[10.211366,50.62235549900004],[10.178821500000026,50.67172949800005],[10.202961,50.77663499700003],[10.384705,50.786486499000034],[10.41141,50.850239499000054],[10.537865500000066,50.82703749800004],[10.584063,50.736135499000056],[10.752704,50.71847099800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0C","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Gotha","NUTS_NAME":"Gotha"},"geometry":{"type":"Polygon","coordinates":[[[10.880309,51.09825249800008],[10.862985500000036,51.02654399800008],[10.865766,50.96489349700005],[10.959487500000023,50.93419449900006],[10.936165,50.894874497000046],[10.830877499000053,50.843665497000075],[10.859195500000055,50.83380999800005],[10.752704,50.71847099800004],[10.584063,50.736135499000056],[10.537865500000066,50.82703749800004],[10.41141,50.850239499000054],[10.52086,50.96621299900005],[10.496621,50.992757],[10.614710500000058,51.03916299800005],[10.880309,51.09825249800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0D","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Sömmerda","NUTS_NAME":"Sömmerda"},"geometry":{"type":"Polygon","coordinates":[[[11.385374500000069,51.24588699800006],[11.468567500000063,51.192653998000026],[11.484608500000036,51.10543749800007],[11.19665549900003,51.10131049800003],[11.217299,51.03263149800006],[11.170664500000044,51.018729],[11.066442,51.07941299600003],[10.862985500000036,51.02654399800008],[10.880309,51.09825249800008],[10.922568500000068,51.20532899800003],[11.06900150000007,51.322618],[11.213871,51.24093949600007],[11.385374500000069,51.24588699800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0E","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Hildburghausen","NUTS_NAME":"Hildburghausen"},"geometry":{"type":"Polygon","coordinates":[[[10.826985500000035,50.580954],[10.96865450000007,50.55275249700003],[10.992608,50.52861949800007],[11.039205,50.46241249800005],[10.945718500000055,50.38646649800006],[10.715266500000041,50.363590997000074],[10.851494500000058,50.26276249800003],[10.729202,50.23000549600005],[10.610115,50.22799499800004],[10.600550500000054,50.33368799800007],[10.450532,50.40185949900007],[10.566062500000044,50.439793999000074],[10.582464,50.58543599700005],[10.826985500000035,50.580954]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0F","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Ilm-Kreis","NUTS_NAME":"Ilm-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[11.221885500000042,50.803885999000045],[11.189806,50.73659599800004],[11.052882500000067,50.70811449800004],[11.06651850000003,50.56363299700007],[10.96865450000007,50.55275249700003],[10.826985500000035,50.580954],[10.733734500000025,50.663490498000044],[10.752704,50.71847099800004],[10.859195500000055,50.83380999800005],[10.830877499000053,50.843665497000075],[10.936165,50.894874497000046],[11.11025950000004,50.91311249800003],[11.221885500000042,50.803885999000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0G","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Weimarer Land","NUTS_NAME":"Weimarer Land"},"geometry":{"type":"Polygon","coordinates":[[[11.696938500000044,51.08749099700003],[11.547322,50.98905749900007],[11.504811,50.90624249900003],[11.423872,50.808598998000036],[11.221885500000042,50.803885999000045],[11.11025950000004,50.91311249800003],[11.170664500000044,51.018729],[11.217299,51.03263149800006],[11.19665549900003,51.10131049800003],[11.484608500000036,51.10543749800007],[11.696938500000044,51.08749099700003]],[[11.402000500000042,50.96405249700007],[11.373274,51.02006849900005],[11.241368500000021,51.02350149800003],[11.292821,50.91785099800006],[11.402000500000042,50.96405249700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0H","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Sonneberg","NUTS_NAME":"Sonneberg"},"geometry":{"type":"Polygon","coordinates":[[[11.265938,50.47942099900007],[11.259778,50.27494649800008],[11.189943,50.27118550000006],[11.115305,50.36700699600004],[10.945718500000055,50.38646649800006],[11.039205,50.46241249800005],[10.992608,50.52861949800007],[11.265938,50.47942099900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0I","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saalfeld-Rudolstadt","NUTS_NAME":"Saalfeld-Rudolstadt"},"geometry":{"type":"Polygon","coordinates":[[[11.423872,50.808598998000036],[11.544709500000067,50.75079399900005],[11.462474,50.69529299800007],[11.529032500000028,50.609320997000054],[11.63338,50.592186998000045],[11.487534500000038,50.54282899800006],[11.481568,50.43162150000006],[11.420630500000073,50.443366499000035],[11.423893,50.51606449800005],[11.265938,50.47942099900007],[10.992608,50.52861949800007],[10.96865450000007,50.55275249700003],[11.06651850000003,50.56363299700007],[11.052882500000067,50.70811449800004],[11.189806,50.73659599800004],[11.221885500000042,50.803885999000045],[11.423872,50.808598998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0J","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saale-Holzland-Kreis","NUTS_NAME":"Saale-Holzland-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[12.021018500000025,50.96912249800005],[12.021172500000034,50.94795250000004],[11.872699500000067,50.881792500000074],[11.909426500000052,50.82632299800008],[11.866546,50.78806750000007],[11.544709500000067,50.75079399900005],[11.423872,50.808598998000036],[11.504811,50.90624249900003],[11.661936,50.88429799800008],[11.66578050000004,50.95766099800005],[11.547322,50.98905749900007],[11.696938500000044,51.08749099700003],[12.021018500000025,50.96912249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0K","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Saale-Orla-Kreis","NUTS_NAME":"Saale-Orla-Kreis"},"geometry":{"type":"Polygon","coordinates":[[[11.944137500000068,50.59127549800007],[11.871462500000064,50.53982249900008],[11.96484,50.48485799800005],[11.919884500000023,50.42440349800006],[11.603290500000071,50.398766],[11.519853,50.37426849800005],[11.481568,50.43162150000006],[11.487534500000038,50.54282899800006],[11.63338,50.592186998000045],[11.529032500000028,50.609320997000054],[11.462474,50.69529299800007],[11.544709500000067,50.75079399900005],[11.866546,50.78806750000007],[11.943614,50.76634549800008],[11.866303,50.66597949900006],[11.944137500000068,50.59127549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0L","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Greiz","NUTS_NAME":"Greiz"},"geometry":{"type":"Polygon","coordinates":[[[12.224169,50.942934997000066],[12.207897500000058,50.880472498000074],[12.250825500000076,50.81831849800005],[12.226984500000071,50.73821849800004],[12.318916500000057,50.67653249800003],[12.052068,50.55096099800005],[12.010397,50.63546099900003],[11.944137500000068,50.59127549800007],[11.866303,50.66597949900006],[11.943614,50.76634549800008],[11.866546,50.78806750000007],[11.909426500000052,50.82632299800008],[11.872699500000067,50.881792500000074],[12.021172500000034,50.94795250000004],[11.998804500000062,50.84946549800003],[12.134889500000043,50.82057649800004],[12.163369500000044,50.95882249700003],[12.224169,50.942934997000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0M","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Altenburger Land","NUTS_NAME":"Altenburger Land"},"geometry":{"type":"Polygon","coordinates":[[[12.617355,50.98079299700004],[12.652865500000075,50.923678],[12.250825500000076,50.81831849800005],[12.207897500000058,50.880472498000074],[12.224169,50.942934997000066],[12.29299050000003,51.021581499000035],[12.250268500000061,51.037873],[12.284594500000026,51.09116249600004],[12.501522500000021,51.06407599900007],[12.617355,50.98079299700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0N","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Eisenach, Kreisfreie Stadt","NUTS_NAME":"Eisenach, Kreisfreie Stadt"},"geometry":{"type":"Polygon","coordinates":[[[10.182348500000046,50.99848749900008],[10.202181,51.012009],[10.332562500000051,51.04979749800003],[10.391869,50.99982399800007],[10.320377,50.93071599600006],[10.182348500000046,50.99848749900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DEG0P","LEVL_CODE":3,"CNTR_CODE":"DE","NAME_LATN":"Wartburgkreis","NUTS_NAME":"Wartburgkreis"},"geometry":{"type":"Polygon","coordinates":[[[10.614710500000058,51.03916299800005],[10.496621,50.992757],[10.52086,50.96621299900005],[10.41141,50.850239499000054],[10.384705,50.786486499000034],[10.202961,50.77663499700003],[10.178821500000026,50.67172949800005],[10.211366,50.62235549900004],[10.077555500000074,50.63762399600006],[9.878551,50.63478099800005],[9.926176500000054,50.767389998000056],[10.055980999000042,50.879771497000036],[9.949583500000074,50.927478499000074],[10.063865,50.94543299600008],[10.021558500000026,50.992949499000076],[10.182348500000046,50.99848749900008],[10.320377,50.93071599600006],[10.391869,50.99982399800007],[10.332562500000051,51.04979749800003],[10.202181,51.012009],[10.118321500000036,51.13957299900005],[10.210013,51.144082998000044],[10.614710500000058,51.03916299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK","LEVL_CODE":0,"CNTR_CODE":"DK","NAME_LATN":"DANMARK","NUTS_NAME":"DANMARK"},"geometry":{"type":"MultiPolygon","coordinates":[[[[15.158351,55.083438999000066],[15.082281500000022,54.98780250000004],[14.683027,55.09734649600006],[14.779674,55.29000449800003],[15.158351,55.083438999000066]]],[[[12.564100500000052,55.99403399900007],[12.518339,55.921970499000054],[12.583112,55.807791500000064],[12.584052500000041,55.72177349900005],[12.68128850000005,55.58825699800008],[12.565327500000024,55.55992150000003],[12.504888,55.637469499000076],[12.363498,55.59382749800005],[12.208657500000072,55.50890499700006],[12.221227,55.42562950000007],[12.366144,55.396911497000076],[12.456344500000057,55.29250699800008],[12.028826500000037,55.16483999800005],[12.183695,55.110321],[12.099427999000056,55.089638499000046],[12.171433499000045,55.00372699900004],[12.002428500000065,54.96804599600006],[11.615766500000063,55.08135999800004],[11.828402500000038,55.04231599600007],[11.718178,55.11378499700004],[11.815993500000047,55.13128050000006],[11.748527500000023,55.21917349700004],[11.243200499000068,55.20794299600004],[11.23505,55.286414996000076],[11.129011,55.32628249700008],[11.206539,55.34312799800006],[11.09238350000004,55.34708399800007],[11.215033499000072,55.39254649800006],[11.172648500000037,55.49716450000005],[11.0813,55.51703250000003],[11.161611500000049,55.54041299900007],[11.150483,55.58531549800006],[10.931494499000053,55.66867849800008],[11.092614500000025,55.676707],[10.87105,55.743763],[11.348573,55.75773649900003],[11.524006,55.92478199900006],[11.278078,56.01033799900006],[11.4824,55.961128],[11.777166500000021,55.97676849900006],[11.669935,55.913729998000065],[11.734578,55.82386799600005],[11.610122499000056,55.78175749700006],[11.743550500000026,55.78957349700005],[11.770105500000057,55.74569299800004],[11.65237,55.73053899700005],[11.805708192000054,55.68068123100005],[11.87555,55.73759149800003],[11.864388500000075,55.796055498000044],[11.95999450000005,55.850509497000075],[11.905017,55.93590949900005],[11.963744,55.92930599700003],[12.059644500000047,55.73209],[11.980809,55.730124997000075],[11.951244,55.67552199800008],[12.084782,55.654254],[12.089422,55.78081999900007],[12.010116499000048,55.96801749700006],[11.844483500000024,55.967036998000026],[12.30944450000004,56.128566498000055],[12.625739,56.03853999900008],[12.564100500000052,55.99403399900007]]],[[[12.554239500000051,54.96054849800004],[12.099478,54.904529498000045],[12.284661499000038,54.98041149800008],[12.232811,55.042789500000026],[12.52389450000004,55.007911499000045],[12.554239500000051,54.96054849800004]]],[[[12.166872,54.83466699600007],[11.980852500000026,54.70157199700003],[11.969867,54.55922699700005],[11.866283500000065,54.65716599700005],[11.911294,54.71692649700003],[11.852931975000047,54.76367536300006],[11.860477500000059,54.68697350000008],[11.80862250000007,54.64213949900005],[11.461767500000065,54.61030599900005],[10.956038500000034,54.81500649600008],[11.109383499000046,54.831248996000056],[11.017612500000041,54.88050099600008],[11.067866499000047,54.93843849800004],[11.292961,54.945838999000046],[11.491927,54.83137749800005],[11.591144500000041,54.826515],[11.651233500000046,54.860156998000036],[11.637633,54.90346550000004],[11.795054962000052,54.81123248700004],[11.704489,54.93237699900004],[11.82662750000003,54.96798699800007],[12.166872,54.83466699600007]]],[[[10.272072,56.936893499000064],[10.332744500000047,56.70807649600005],[10.33853350000004,56.70126699900004],[10.359616500000072,56.63785549800008],[10.308140108000032,56.60684268400007],[10.504672,56.51335149800008],[10.837805500000059,56.530620498000076],[10.959905500000048,56.44208149800005],[10.712867,56.14205549700006],[10.609916500000054,56.22810749900003],[10.511949500000071,56.09445549900005],[10.480811,56.139278499000056],[10.528027500000064,56.16802199800003],[10.345911,56.19412599900005],[10.507214500000032,56.261523999000076],[10.432433,56.295199998000044],[10.236166500000024,56.180622999000036],[10.210211,56.13629549900003],[10.281900499000074,56.02040149800007],[10.193888500000071,55.83793999900007],[9.865144500000042,55.84770999700004],[10.066128,55.80712099700003],[10.016188500000055,55.766716],[10.083200500000032,55.74549850000005],[10.009511,55.700878],[9.671952500000032,55.71206999700007],[9.543939500000022,55.70668099800008],[9.860733,55.62514899900003],[9.739651342000059,55.52969921300007],[9.941383500000029,55.51225049900006],[10.312864500000046,55.622698996000054],[10.493877499000064,55.54795049900008],[10.427106,55.42914599800008],[10.624455500000067,55.48692699600008],[10.591878,55.62058949800007],[10.748172,55.47723750000006],[10.663842228000021,55.45011591900004],[10.856239500000072,55.29318950000004],[10.782764,55.30017249900004],[10.812549500000046,55.205943999000056],[10.743033500000024,55.068989],[10.63782468200003,55.05262858400005],[10.716634,55.029849496000054],[10.553716500000064,54.944484498000065],[10.513194,55.006759499000054],[10.612443515000052,55.04956722400004],[10.072239,55.07741149700007],[10.161317,55.133464497000034],[10.107377500000041,55.190322999000045],[9.993244,55.201572500000054],[10.044139,55.13289999600005],[9.979295,55.131988498000055],[9.890983500000061,55.33582299600005],[9.784734,55.365466998000045],[9.825689500000067,55.42327899800006],[9.716944500000068,55.46771549700003],[9.820744500000046,55.44087199900008],[9.679817695000054,55.51198248600008],[9.488094500000045,55.49062699600006],[9.694266500000026,55.46040749900004],[9.596377500000074,55.42455649800007],[9.718288500000028,55.25227749900006],[9.696561,55.19786849900004],[9.498838499000044,55.15687199800004],[9.457965,55.114543997000055],[9.550878,55.075477499000044],[9.422177500000032,55.03345099900008],[9.86746450000004,54.942138499000066],[9.638089,55.046081498000035],[9.746555499000067,55.084243997000044],[9.996468500000049,54.98452350000008],[10.071288500000037,54.87728499600007],[9.781979357000068,54.909590250000065],[9.750627500000064,54.83902350000005],[9.63729950000004,54.86192299900006],[9.598655500000064,54.906142998000064],[9.673917,54.91328399900004],[9.619266500000037,54.934032],[9.420151500000031,54.83195650000005],[9.113097,54.873601499000074],[8.635925999000051,54.91168299700007],[8.632144,55.03979849600006],[8.685461,55.144393997000066],[8.556383,55.15039449800008],[8.569247,55.085104],[8.521362,55.061226],[8.464339500000051,55.108963],[8.514703500000053,55.21203699600005],[8.684189,55.15883999700003],[8.620333500000072,55.42867299800008],[8.29197750000003,55.569064997000055],[8.32998150000003,55.58447349800008],[8.237166500000058,55.566726499000026],[8.298811,55.476150500000074],[8.076632500000073,55.55696499800007],[8.17079,55.81536699900005],[8.1178,55.998211],[8.193086093000034,55.87283399800003],[8.279055500000027,55.87795949800005],[8.245650500000067,55.83682249900005],[8.386469,55.90727849900003],[8.23038850000006,56.09285949700006],[8.126061500000048,56.11844599900007],[8.145366500000023,56.007381499000076],[8.099594,56.036021997000034],[8.117563500000074,56.37189649800007],[8.302711500000044,56.31922899700004],[8.126683,56.430427499000075],[8.124725500000068,56.56368850000007],[8.19683,56.69499599900007],[8.231495,56.60677350000003],[8.554322,56.56212599600008],[8.668789,56.46784949900007],[8.763022500000034,56.55245849700003],[8.676833,56.62150950000006],[8.919017,56.69897349800004],[8.825267,56.70683299800004],[9.002038860000027,56.801205749000076],[8.949772,56.83336649700004],[9.076211,56.825473997000074],[9.021852163000062,56.801562251000064],[9.1771,56.708270999000035],[9.058578500000067,56.62983599700004],[9.050561,56.56131],[9.212072500000033,56.62995549800007],[9.285438500000055,56.62196749800006],[9.247093500000062,56.57242650000006],[9.307318,56.52508749700007],[9.369637,56.53706149800007],[9.287089500000036,56.56544949600004],[9.319437,56.67222699800004],[9.159611,56.67793249700003],[9.24345,56.74848949600005],[9.185678,56.766875999000035],[9.210255500000073,56.86354849800006],[9.164284,56.888828499000056],[9.29455,56.99829499900005],[9.531736500000022,57.00722699900007],[9.583900500000027,56.987621500000046],[9.538178500000072,56.95572149700007],[9.575669499000071,56.91954849900003],[9.687421500000028,57.03817749700005],[10.039944500000047,57.05941],[10.191666500000053,56.987171],[10.30645,56.98275399800008],[10.272072,56.936893499000064]]],[[[10.843383500000073,54.951072500000066],[10.717417,54.72775599900007],[10.636339,54.81753899700004],[10.955073,55.15691749800004],[10.843383500000073,54.951072500000066]]],[[[10.689461,55.89378699800005],[10.621293999000045,55.76484299800006],[10.549744500000031,55.769402],[10.529172,55.85150549600007],[10.601043,55.90681649600003],[10.522061500000063,55.97892749600004],[10.579689,55.947982497000055],[10.689461,55.89378699800005]]],[[[10.422333500000036,57.14775149700006],[10.331549499000062,56.98897949900004],[10.234805,56.987022496000066],[10.017161,57.086907],[9.924750500000073,57.05464949900005],[9.777845,57.10914999600004],[8.668212,56.95006949700007],[8.625678,56.889027498000075],[8.632456,56.863773500000036],[8.47117,56.78329049900003],[8.515605634000053,56.686523847000046],[8.577901844000053,56.69249834800007],[8.541478,56.78775],[8.666719,56.80917549800006],[8.631644,56.838523999000074],[8.634744500000068,56.88579949800004],[8.816511500000047,56.904053],[8.900349500000061,56.952026498000066],[8.906772,56.896442998000055],[8.833179500000028,56.87339499800004],[8.869944500000031,56.79465149600003],[8.77658850000006,56.69454949700008],[8.58326334700007,56.69057483200004],[8.641994500000067,56.66126249800004],[8.544610999000042,56.57684349600004],[8.404045500000052,56.67023699900005],[8.266428,56.775611999000034],[8.23772450000007,56.701515],[8.232896,56.798159498000075],[8.589805500000068,57.11797349600005],[9.399203500000056,57.164098496000065],[9.958295500000077,57.59529899700004],[10.644709500000033,57.742755499000054],[10.426225500000044,57.58919999900007],[10.54697550000003,57.45014049800005],[10.515677499000049,57.38432699700007],[10.550539,57.23988349600006],[10.422333500000036,57.14775149700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK0","LEVL_CODE":1,"CNTR_CODE":"DK","NAME_LATN":"DANMARK","NUTS_NAME":"DANMARK"},"geometry":{"type":"MultiPolygon","coordinates":[[[[15.158351,55.083438999000066],[15.082281500000022,54.98780250000004],[14.683027,55.09734649600006],[14.779674,55.29000449800003],[15.158351,55.083438999000066]]],[[[12.564100500000052,55.99403399900007],[12.518339,55.921970499000054],[12.583112,55.807791500000064],[12.584052500000041,55.72177349900005],[12.68128850000005,55.58825699800008],[12.565327500000024,55.55992150000003],[12.504888,55.637469499000076],[12.363498,55.59382749800005],[12.208657500000072,55.50890499700006],[12.221227,55.42562950000007],[12.366144,55.396911497000076],[12.456344500000057,55.29250699800008],[12.028826500000037,55.16483999800005],[12.183695,55.110321],[12.099427999000056,55.089638499000046],[12.171433499000045,55.00372699900004],[12.002428500000065,54.96804599600006],[11.615766500000063,55.08135999800004],[11.828402500000038,55.04231599600007],[11.718178,55.11378499700004],[11.815993500000047,55.13128050000006],[11.748527500000023,55.21917349700004],[11.243200499000068,55.20794299600004],[11.23505,55.286414996000076],[11.129011,55.32628249700008],[11.206539,55.34312799800006],[11.09238350000004,55.34708399800007],[11.215033499000072,55.39254649800006],[11.172648500000037,55.49716450000005],[11.0813,55.51703250000003],[11.161611500000049,55.54041299900007],[11.150483,55.58531549800006],[10.931494499000053,55.66867849800008],[11.092614500000025,55.676707],[10.87105,55.743763],[11.348573,55.75773649900003],[11.524006,55.92478199900006],[11.278078,56.01033799900006],[11.4824,55.961128],[11.777166500000021,55.97676849900006],[11.669935,55.913729998000065],[11.734578,55.82386799600005],[11.610122499000056,55.78175749700006],[11.743550500000026,55.78957349700005],[11.770105500000057,55.74569299800004],[11.65237,55.73053899700005],[11.805708192000054,55.68068123100005],[11.87555,55.73759149800003],[11.864388500000075,55.796055498000044],[11.95999450000005,55.850509497000075],[11.905017,55.93590949900005],[11.963744,55.92930599700003],[12.059644500000047,55.73209],[11.980809,55.730124997000075],[11.951244,55.67552199800008],[12.084782,55.654254],[12.089422,55.78081999900007],[12.010116499000048,55.96801749700006],[11.844483500000024,55.967036998000026],[12.30944450000004,56.128566498000055],[12.625739,56.03853999900008],[12.564100500000052,55.99403399900007]]],[[[12.554239500000051,54.96054849800004],[12.099478,54.904529498000045],[12.284661499000038,54.98041149800008],[12.232811,55.042789500000026],[12.52389450000004,55.007911499000045],[12.554239500000051,54.96054849800004]]],[[[12.166872,54.83466699600007],[11.980852500000026,54.70157199700003],[11.969867,54.55922699700005],[11.866283500000065,54.65716599700005],[11.911294,54.71692649700003],[11.852931975000047,54.76367536300006],[11.860477500000059,54.68697350000008],[11.80862250000007,54.64213949900005],[11.461767500000065,54.61030599900005],[10.956038500000034,54.81500649600008],[11.109383499000046,54.831248996000056],[11.017612500000041,54.88050099600008],[11.067866499000047,54.93843849800004],[11.292961,54.945838999000046],[11.491927,54.83137749800005],[11.591144500000041,54.826515],[11.651233500000046,54.860156998000036],[11.637633,54.90346550000004],[11.795054962000052,54.81123248700004],[11.704489,54.93237699900004],[11.82662750000003,54.96798699800007],[12.166872,54.83466699600007]]],[[[10.272072,56.936893499000064],[10.332744500000047,56.70807649600005],[10.33853350000004,56.70126699900004],[10.359616500000072,56.63785549800008],[10.308140108000032,56.60684268400007],[10.504672,56.51335149800008],[10.837805500000059,56.530620498000076],[10.959905500000048,56.44208149800005],[10.712867,56.14205549700006],[10.609916500000054,56.22810749900003],[10.511949500000071,56.09445549900005],[10.480811,56.139278499000056],[10.528027500000064,56.16802199800003],[10.345911,56.19412599900005],[10.507214500000032,56.261523999000076],[10.432433,56.295199998000044],[10.236166500000024,56.180622999000036],[10.210211,56.13629549900003],[10.281900499000074,56.02040149800007],[10.193888500000071,55.83793999900007],[9.865144500000042,55.84770999700004],[10.066128,55.80712099700003],[10.016188500000055,55.766716],[10.083200500000032,55.74549850000005],[10.009511,55.700878],[9.671952500000032,55.71206999700007],[9.543939500000022,55.70668099800008],[9.860733,55.62514899900003],[9.739651342000059,55.52969921300007],[9.941383500000029,55.51225049900006],[10.312864500000046,55.622698996000054],[10.493877499000064,55.54795049900008],[10.427106,55.42914599800008],[10.624455500000067,55.48692699600008],[10.591878,55.62058949800007],[10.748172,55.47723750000006],[10.663842228000021,55.45011591900004],[10.856239500000072,55.29318950000004],[10.782764,55.30017249900004],[10.812549500000046,55.205943999000056],[10.743033500000024,55.068989],[10.63782468200003,55.05262858400005],[10.716634,55.029849496000054],[10.553716500000064,54.944484498000065],[10.513194,55.006759499000054],[10.612443515000052,55.04956722400004],[10.072239,55.07741149700007],[10.161317,55.133464497000034],[10.107377500000041,55.190322999000045],[9.993244,55.201572500000054],[10.044139,55.13289999600005],[9.979295,55.131988498000055],[9.890983500000061,55.33582299600005],[9.784734,55.365466998000045],[9.825689500000067,55.42327899800006],[9.716944500000068,55.46771549700003],[9.820744500000046,55.44087199900008],[9.679817695000054,55.51198248600008],[9.488094500000045,55.49062699600006],[9.694266500000026,55.46040749900004],[9.596377500000074,55.42455649800007],[9.718288500000028,55.25227749900006],[9.696561,55.19786849900004],[9.498838499000044,55.15687199800004],[9.457965,55.114543997000055],[9.550878,55.075477499000044],[9.422177500000032,55.03345099900008],[9.86746450000004,54.942138499000066],[9.638089,55.046081498000035],[9.746555499000067,55.084243997000044],[9.996468500000049,54.98452350000008],[10.071288500000037,54.87728499600007],[9.781979357000068,54.909590250000065],[9.750627500000064,54.83902350000005],[9.63729950000004,54.86192299900006],[9.598655500000064,54.906142998000064],[9.673917,54.91328399900004],[9.619266500000037,54.934032],[9.420151500000031,54.83195650000005],[9.113097,54.873601499000074],[8.635925999000051,54.91168299700007],[8.632144,55.03979849600006],[8.685461,55.144393997000066],[8.556383,55.15039449800008],[8.569247,55.085104],[8.521362,55.061226],[8.464339500000051,55.108963],[8.514703500000053,55.21203699600005],[8.684189,55.15883999700003],[8.620333500000072,55.42867299800008],[8.29197750000003,55.569064997000055],[8.32998150000003,55.58447349800008],[8.237166500000058,55.566726499000026],[8.298811,55.476150500000074],[8.076632500000073,55.55696499800007],[8.17079,55.81536699900005],[8.1178,55.998211],[8.193086093000034,55.87283399800003],[8.279055500000027,55.87795949800005],[8.245650500000067,55.83682249900005],[8.386469,55.90727849900003],[8.23038850000006,56.09285949700006],[8.126061500000048,56.11844599900007],[8.145366500000023,56.007381499000076],[8.099594,56.036021997000034],[8.117563500000074,56.37189649800007],[8.302711500000044,56.31922899700004],[8.126683,56.430427499000075],[8.124725500000068,56.56368850000007],[8.19683,56.69499599900007],[8.231495,56.60677350000003],[8.554322,56.56212599600008],[8.668789,56.46784949900007],[8.763022500000034,56.55245849700003],[8.676833,56.62150950000006],[8.919017,56.69897349800004],[8.825267,56.70683299800004],[9.002038860000027,56.801205749000076],[8.949772,56.83336649700004],[9.076211,56.825473997000074],[9.021852163000062,56.801562251000064],[9.1771,56.708270999000035],[9.058578500000067,56.62983599700004],[9.050561,56.56131],[9.212072500000033,56.62995549800007],[9.285438500000055,56.62196749800006],[9.247093500000062,56.57242650000006],[9.307318,56.52508749700007],[9.369637,56.53706149800007],[9.287089500000036,56.56544949600004],[9.319437,56.67222699800004],[9.159611,56.67793249700003],[9.24345,56.74848949600005],[9.185678,56.766875999000035],[9.210255500000073,56.86354849800006],[9.164284,56.888828499000056],[9.29455,56.99829499900005],[9.531736500000022,57.00722699900007],[9.583900500000027,56.987621500000046],[9.538178500000072,56.95572149700007],[9.575669499000071,56.91954849900003],[9.687421500000028,57.03817749700005],[10.039944500000047,57.05941],[10.191666500000053,56.987171],[10.30645,56.98275399800008],[10.272072,56.936893499000064]]],[[[10.843383500000073,54.951072500000066],[10.717417,54.72775599900007],[10.636339,54.81753899700004],[10.955073,55.15691749800004],[10.843383500000073,54.951072500000066]]],[[[10.689461,55.89378699800005],[10.621293999000045,55.76484299800006],[10.549744500000031,55.769402],[10.529172,55.85150549600007],[10.601043,55.90681649600003],[10.522061500000063,55.97892749600004],[10.579689,55.947982497000055],[10.689461,55.89378699800005]]],[[[10.422333500000036,57.14775149700006],[10.331549499000062,56.98897949900004],[10.234805,56.987022496000066],[10.017161,57.086907],[9.924750500000073,57.05464949900005],[9.777845,57.10914999600004],[8.668212,56.95006949700007],[8.625678,56.889027498000075],[8.632456,56.863773500000036],[8.47117,56.78329049900003],[8.515605634000053,56.686523847000046],[8.577901844000053,56.69249834800007],[8.541478,56.78775],[8.666719,56.80917549800006],[8.631644,56.838523999000074],[8.634744500000068,56.88579949800004],[8.816511500000047,56.904053],[8.900349500000061,56.952026498000066],[8.906772,56.896442998000055],[8.833179500000028,56.87339499800004],[8.869944500000031,56.79465149600003],[8.77658850000006,56.69454949700008],[8.58326334700007,56.69057483200004],[8.641994500000067,56.66126249800004],[8.544610999000042,56.57684349600004],[8.404045500000052,56.67023699900005],[8.266428,56.775611999000034],[8.23772450000007,56.701515],[8.232896,56.798159498000075],[8.589805500000068,57.11797349600005],[9.399203500000056,57.164098496000065],[9.958295500000077,57.59529899700004],[10.644709500000033,57.742755499000054],[10.426225500000044,57.58919999900007],[10.54697550000003,57.45014049800005],[10.515677499000049,57.38432699700007],[10.550539,57.23988349600006],[10.422333500000036,57.14775149700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK01","LEVL_CODE":2,"CNTR_CODE":"DK","NAME_LATN":"Hovedstaden","NUTS_NAME":"Hovedstaden"},"geometry":{"type":"MultiPolygon","coordinates":[[[[15.158351,55.083438999000066],[15.082281500000022,54.98780250000004],[14.683027,55.09734649600006],[14.779674,55.29000449800003],[15.158351,55.083438999000066]]],[[[12.564100500000052,55.99403399900007],[12.518339,55.921970499000054],[12.583112,55.807791500000064],[12.584052500000041,55.72177349900005],[12.68128850000005,55.58825699800008],[12.565327500000024,55.55992150000003],[12.504888,55.637469499000076],[12.363498,55.59382749800005],[12.159595,55.61282449600003],[12.150479,55.671020498000075],[12.250782,55.70841099900008],[12.089422,55.78081999900007],[12.010116499000048,55.96801749700006],[11.844483500000024,55.967036998000026],[12.30944450000004,56.128566498000055],[12.625739,56.03853999900008],[12.564100500000052,55.99403399900007]]],[[[12.059644500000047,55.73209],[11.980809,55.730124997000075],[11.87555,55.73759149800003],[11.864388500000075,55.796055498000044],[11.95999450000005,55.850509497000075],[11.905017,55.93590949900005],[11.963744,55.92930599700003],[12.059644500000047,55.73209]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK011","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Byen København","NUTS_NAME":"Byen København"},"geometry":{"type":"Polygon","coordinates":[[[12.68128850000005,55.58825699800008],[12.565327500000024,55.55992150000003],[12.504888,55.637469499000076],[12.45304,55.712244998000074],[12.584052500000041,55.72177349900005],[12.68128850000005,55.58825699800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK012","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Københavns omegn","NUTS_NAME":"Københavns omegn"},"geometry":{"type":"Polygon","coordinates":[[[12.584052500000041,55.72177349900005],[12.45304,55.712244998000074],[12.504888,55.637469499000076],[12.363498,55.59382749800005],[12.159595,55.61282449600003],[12.150479,55.671020498000075],[12.250782,55.70841099900008],[12.583112,55.807791500000064],[12.584052500000041,55.72177349900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK013","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Nordsjælland","NUTS_NAME":"Nordsjælland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.564100500000052,55.99403399900007],[12.518339,55.921970499000054],[12.583112,55.807791500000064],[12.250782,55.70841099900008],[12.089422,55.78081999900007],[12.010116499000048,55.96801749700006],[11.844483500000024,55.967036998000026],[12.30944450000004,56.128566498000055],[12.625739,56.03853999900008],[12.564100500000052,55.99403399900007]]],[[[12.059644500000047,55.73209],[11.980809,55.730124997000075],[11.87555,55.73759149800003],[11.864388500000075,55.796055498000044],[11.95999450000005,55.850509497000075],[11.905017,55.93590949900005],[11.963744,55.92930599700003],[12.059644500000047,55.73209]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK014","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Bornholm","NUTS_NAME":"Bornholm"},"geometry":{"type":"Polygon","coordinates":[[[15.158351,55.083438999000066],[15.082281500000022,54.98780250000004],[14.683027,55.09734649600006],[14.779674,55.29000449800003],[15.158351,55.083438999000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK02","LEVL_CODE":2,"CNTR_CODE":"DK","NAME_LATN":"Sjælland","NUTS_NAME":"Sjælland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.554239500000051,54.96054849800004],[12.099478,54.904529498000045],[12.284661499000038,54.98041149800008],[12.232811,55.042789500000026],[12.52389450000004,55.007911499000045],[12.554239500000051,54.96054849800004]]],[[[11.278078,56.01033799900006],[11.4824,55.961128],[11.777166500000021,55.97676849900006],[11.669935,55.913729998000065],[11.734578,55.82386799600005],[11.610122499000056,55.78175749700006],[11.743550500000026,55.78957349700005],[11.770105500000057,55.74569299800004],[11.65237,55.73053899700005],[11.805708192000054,55.68068123100005],[11.87555,55.73759149800003],[11.980809,55.730124997000075],[11.951244,55.67552199800008],[12.084782,55.654254],[12.089422,55.78081999900007],[12.250782,55.70841099900008],[12.150479,55.671020498000075],[12.159595,55.61282449600003],[12.363498,55.59382749800005],[12.208657500000072,55.50890499700006],[12.221227,55.42562950000007],[12.366144,55.396911497000076],[12.456344500000057,55.29250699800008],[12.028826500000037,55.16483999800005],[12.183695,55.110321],[12.099427999000056,55.089638499000046],[12.171433499000045,55.00372699900004],[12.002428500000065,54.96804599600006],[11.615766500000063,55.08135999800004],[11.828402500000038,55.04231599600007],[11.718178,55.11378499700004],[11.815993500000047,55.13128050000006],[11.748527500000023,55.21917349700004],[11.243200499000068,55.20794299600004],[11.23505,55.286414996000076],[11.129011,55.32628249700008],[11.206539,55.34312799800006],[11.09238350000004,55.34708399800007],[11.215033499000072,55.39254649800006],[11.172648500000037,55.49716450000005],[11.0813,55.51703250000003],[11.161611500000049,55.54041299900007],[11.150483,55.58531549800006],[10.931494499000053,55.66867849800008],[11.092614500000025,55.676707],[10.87105,55.743763],[11.348573,55.75773649900003],[11.524006,55.92478199900006],[11.278078,56.01033799900006]]],[[[12.166872,54.83466699600007],[11.980852500000026,54.70157199700003],[11.969867,54.55922699700005],[11.866283500000065,54.65716599700005],[11.911294,54.71692649700003],[11.852931975000047,54.76367536300006],[11.860477500000059,54.68697350000008],[11.80862250000007,54.64213949900005],[11.461767500000065,54.61030599900005],[10.956038500000034,54.81500649600008],[11.109383499000046,54.831248996000056],[11.017612500000041,54.88050099600008],[11.067866499000047,54.93843849800004],[11.292961,54.945838999000046],[11.491927,54.83137749800005],[11.591144500000041,54.826515],[11.651233500000046,54.860156998000036],[11.637633,54.90346550000004],[11.795054962000052,54.81123248700004],[11.704489,54.93237699900004],[11.82662750000003,54.96798699800007],[12.166872,54.83466699600007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK021","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Østsjælland","NUTS_NAME":"Østsjælland"},"geometry":{"type":"Polygon","coordinates":[[[12.250782,55.70841099900008],[12.150479,55.671020498000075],[12.159595,55.61282449600003],[12.363498,55.59382749800005],[12.208657500000072,55.50890499700006],[12.221227,55.42562950000007],[11.96353350000004,55.40202599700007],[11.779139,55.65905299900004],[11.805708192000054,55.68068123100005],[11.87555,55.73759149800003],[11.980809,55.730124997000075],[11.951244,55.67552199800008],[12.084782,55.654254],[12.089422,55.78081999900007],[12.250782,55.70841099900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK022","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Vest- og Sydsjælland","NUTS_NAME":"Vest- og Sydsjælland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.554239500000051,54.96054849800004],[12.099478,54.904529498000045],[12.284661499000038,54.98041149800008],[12.232811,55.042789500000026],[12.52389450000004,55.007911499000045],[12.554239500000051,54.96054849800004]]],[[[11.278078,56.01033799900006],[11.4824,55.961128],[11.777166500000021,55.97676849900006],[11.669935,55.913729998000065],[11.734578,55.82386799600005],[11.610122499000056,55.78175749700006],[11.743550500000026,55.78957349700005],[11.770105500000057,55.74569299800004],[11.65237,55.73053899700005],[11.805708192000054,55.68068123100005],[11.779139,55.65905299900004],[11.96353350000004,55.40202599700007],[12.221227,55.42562950000007],[12.366144,55.396911497000076],[12.456344500000057,55.29250699800008],[12.028826500000037,55.16483999800005],[12.183695,55.110321],[12.099427999000056,55.089638499000046],[12.171433499000045,55.00372699900004],[12.002428500000065,54.96804599600006],[11.615766500000063,55.08135999800004],[11.828402500000038,55.04231599600007],[11.718178,55.11378499700004],[11.815993500000047,55.13128050000006],[11.748527500000023,55.21917349700004],[11.243200499000068,55.20794299600004],[11.23505,55.286414996000076],[11.129011,55.32628249700008],[11.206539,55.34312799800006],[11.09238350000004,55.34708399800007],[11.215033499000072,55.39254649800006],[11.172648500000037,55.49716450000005],[11.0813,55.51703250000003],[11.161611500000049,55.54041299900007],[11.150483,55.58531549800006],[10.931494499000053,55.66867849800008],[11.092614500000025,55.676707],[10.87105,55.743763],[11.348573,55.75773649900003],[11.524006,55.92478199900006],[11.278078,56.01033799900006]]],[[[12.166872,54.83466699600007],[11.980852500000026,54.70157199700003],[11.969867,54.55922699700005],[11.866283500000065,54.65716599700005],[11.911294,54.71692649700003],[11.852931975000047,54.76367536300006],[11.860477500000059,54.68697350000008],[11.80862250000007,54.64213949900005],[11.461767500000065,54.61030599900005],[10.956038500000034,54.81500649600008],[11.109383499000046,54.831248996000056],[11.017612500000041,54.88050099600008],[11.067866499000047,54.93843849800004],[11.292961,54.945838999000046],[11.491927,54.83137749800005],[11.591144500000041,54.826515],[11.651233500000046,54.860156998000036],[11.637633,54.90346550000004],[11.795054962000052,54.81123248700004],[11.704489,54.93237699900004],[11.82662750000003,54.96798699800007],[12.166872,54.83466699600007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK03","LEVL_CODE":2,"CNTR_CODE":"DK","NAME_LATN":"Syddanmark","NUTS_NAME":"Syddanmark"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.843383500000073,54.951072500000066],[10.717417,54.72775599900007],[10.636339,54.81753899700004],[10.955073,55.15691749800004],[10.843383500000073,54.951072500000066]]],[[[9.447316,55.834682996000026],[9.671952500000032,55.71206999700007],[9.543939500000022,55.70668099800008],[9.860733,55.62514899900003],[9.739651342000059,55.52969921300007],[9.941383500000029,55.51225049900006],[10.312864500000046,55.622698996000054],[10.493877499000064,55.54795049900008],[10.427106,55.42914599800008],[10.624455500000067,55.48692699600008],[10.591878,55.62058949800007],[10.748172,55.47723750000006],[10.663842228000021,55.45011591900004],[10.856239500000072,55.29318950000004],[10.782764,55.30017249900004],[10.812549500000046,55.205943999000056],[10.743033500000024,55.068989],[10.63782468200003,55.05262858400005],[10.716634,55.029849496000054],[10.553716500000064,54.944484498000065],[10.513194,55.006759499000054],[10.612443515000052,55.04956722400004],[10.072239,55.07741149700007],[10.161317,55.133464497000034],[10.107377500000041,55.190322999000045],[9.993244,55.201572500000054],[10.044139,55.13289999600005],[9.979295,55.131988498000055],[9.890983500000061,55.33582299600005],[9.784734,55.365466998000045],[9.825689500000067,55.42327899800006],[9.716944500000068,55.46771549700003],[9.820744500000046,55.44087199900008],[9.679817695000054,55.51198248600008],[9.488094500000045,55.49062699600006],[9.694266500000026,55.46040749900004],[9.596377500000074,55.42455649800007],[9.718288500000028,55.25227749900006],[9.696561,55.19786849900004],[9.498838499000044,55.15687199800004],[9.457965,55.114543997000055],[9.550878,55.075477499000044],[9.422177500000032,55.03345099900008],[9.86746450000004,54.942138499000066],[9.638089,55.046081498000035],[9.746555499000067,55.084243997000044],[9.996468500000049,54.98452350000008],[10.071288500000037,54.87728499600007],[9.781979357000068,54.909590250000065],[9.750627500000064,54.83902350000005],[9.63729950000004,54.86192299900006],[9.598655500000064,54.906142998000064],[9.673917,54.91328399900004],[9.619266500000037,54.934032],[9.420151500000031,54.83195650000005],[9.113097,54.873601499000074],[8.635925999000051,54.91168299700007],[8.632144,55.03979849600006],[8.685461,55.144393997000066],[8.556383,55.15039449800008],[8.569247,55.085104],[8.521362,55.061226],[8.464339500000051,55.108963],[8.514703500000053,55.21203699600005],[8.684189,55.15883999700003],[8.620333500000072,55.42867299800008],[8.29197750000003,55.569064997000055],[8.32998150000003,55.58447349800008],[8.237166500000058,55.566726499000026],[8.298811,55.476150500000074],[8.076632500000073,55.55696499800007],[8.17079,55.81536699900005],[8.468886500000053,55.773105498000064],[8.56659650000006,55.849891],[8.744571999000073,55.80639749900007],[8.881475500000022,55.89574749900004],[9.00953049900005,55.83066149700005],[9.219408500000043,55.95471299600007],[9.387007500000038,55.92414649800003],[9.447316,55.834682996000026]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK031","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Fyn","NUTS_NAME":"Fyn"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.843383500000073,54.951072500000066],[10.717417,54.72775599900007],[10.636339,54.81753899700004],[10.955073,55.15691749800004],[10.843383500000073,54.951072500000066]]],[[[10.493877499000064,55.54795049900008],[10.427106,55.42914599800008],[10.624455500000067,55.48692699600008],[10.591878,55.62058949800007],[10.748172,55.47723750000006],[10.663842228000021,55.45011591900004],[10.856239500000072,55.29318950000004],[10.782764,55.30017249900004],[10.812549500000046,55.205943999000056],[10.743033500000024,55.068989],[10.63782468200003,55.05262858400005],[10.716634,55.029849496000054],[10.553716500000064,54.944484498000065],[10.513194,55.006759499000054],[10.612443515000052,55.04956722400004],[10.072239,55.07741149700007],[10.161317,55.133464497000034],[10.107377500000041,55.190322999000045],[9.993244,55.201572500000054],[10.044139,55.13289999600005],[9.979295,55.131988498000055],[9.890983500000061,55.33582299600005],[9.784734,55.365466998000045],[9.825689500000067,55.42327899800006],[9.716944500000068,55.46771549700003],[9.820744500000046,55.44087199900008],[9.679817695000054,55.51198248600008],[9.739651342000059,55.52969921300007],[9.941383500000029,55.51225049900006],[10.312864500000046,55.622698996000054],[10.493877499000064,55.54795049900008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK032","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Sydjylland","NUTS_NAME":"Sydjylland"},"geometry":{"type":"Polygon","coordinates":[[[9.447316,55.834682996000026],[9.671952500000032,55.71206999700007],[9.543939500000022,55.70668099800008],[9.860733,55.62514899900003],[9.739651342000059,55.52969921300007],[9.679817695000054,55.51198248600008],[9.488094500000045,55.49062699600006],[9.694266500000026,55.46040749900004],[9.596377500000074,55.42455649800007],[9.718288500000028,55.25227749900006],[9.696561,55.19786849900004],[9.498838499000044,55.15687199800004],[9.457965,55.114543997000055],[9.550878,55.075477499000044],[9.422177500000032,55.03345099900008],[9.86746450000004,54.942138499000066],[9.638089,55.046081498000035],[9.746555499000067,55.084243997000044],[9.996468500000049,54.98452350000008],[10.071288500000037,54.87728499600007],[9.781979357000068,54.909590250000065],[9.750627500000064,54.83902350000005],[9.63729950000004,54.86192299900006],[9.598655500000064,54.906142998000064],[9.673917,54.91328399900004],[9.619266500000037,54.934032],[9.420151500000031,54.83195650000005],[9.113097,54.873601499000074],[8.635925999000051,54.91168299700007],[8.632144,55.03979849600006],[8.685461,55.144393997000066],[8.556383,55.15039449800008],[8.569247,55.085104],[8.521362,55.061226],[8.464339500000051,55.108963],[8.514703500000053,55.21203699600005],[8.684189,55.15883999700003],[8.620333500000072,55.42867299800008],[8.29197750000003,55.569064997000055],[8.32998150000003,55.58447349800008],[8.237166500000058,55.566726499000026],[8.298811,55.476150500000074],[8.076632500000073,55.55696499800007],[8.17079,55.81536699900005],[8.468886500000053,55.773105498000064],[8.56659650000006,55.849891],[8.744571999000073,55.80639749900007],[8.881475500000022,55.89574749900004],[9.00953049900005,55.83066149700005],[9.219408500000043,55.95471299600007],[9.387007500000038,55.92414649800003],[9.447316,55.834682996000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK04","LEVL_CODE":2,"CNTR_CODE":"DK","NAME_LATN":"Midtjylland","NUTS_NAME":"Midtjylland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[9.058578500000067,56.62983599700004],[9.050561,56.56131],[9.212072500000033,56.62995549800007],[9.285438500000055,56.62196749800006],[9.247093500000062,56.57242650000006],[9.307318,56.52508749700007],[9.369637,56.53706149800007],[9.287089500000036,56.56544949600004],[9.319437,56.67222699800004],[9.525292,56.680663],[9.79582350000004,56.56161399900003],[10.194114500000069,56.68469350000004],[10.33853350000004,56.70126699900004],[10.359616500000072,56.63785549800008],[10.308140108000032,56.60684268400007],[10.504672,56.51335149800008],[10.837805500000059,56.530620498000076],[10.959905500000048,56.44208149800005],[10.712867,56.14205549700006],[10.609916500000054,56.22810749900003],[10.511949500000071,56.09445549900005],[10.480811,56.139278499000056],[10.528027500000064,56.16802199800003],[10.345911,56.19412599900005],[10.507214500000032,56.261523999000076],[10.432433,56.295199998000044],[10.236166500000024,56.180622999000036],[10.210211,56.13629549900003],[10.281900499000074,56.02040149800007],[10.193888500000071,55.83793999900007],[9.865144500000042,55.84770999700004],[10.066128,55.80712099700003],[10.016188500000055,55.766716],[10.083200500000032,55.74549850000005],[10.009511,55.700878],[9.671952500000032,55.71206999700007],[9.447316,55.834682996000026],[9.387007500000038,55.92414649800003],[9.219408500000043,55.95471299600007],[9.00953049900005,55.83066149700005],[8.881475500000022,55.89574749900004],[8.744571999000073,55.80639749900007],[8.56659650000006,55.849891],[8.468886500000053,55.773105498000064],[8.17079,55.81536699900005],[8.1178,55.998211],[8.193086093000034,55.87283399800003],[8.279055500000027,55.87795949800005],[8.245650500000067,55.83682249900005],[8.386469,55.90727849900003],[8.23038850000006,56.09285949700006],[8.126061500000048,56.11844599900007],[8.145366500000023,56.007381499000076],[8.099594,56.036021997000034],[8.117563500000074,56.37189649800007],[8.302711500000044,56.31922899700004],[8.126683,56.430427499000075],[8.124725500000068,56.56368850000007],[8.19683,56.69499599900007],[8.231495,56.60677350000003],[8.554322,56.56212599600008],[8.668789,56.46784949900007],[8.763022500000034,56.55245849700003],[8.676833,56.62150950000006],[8.919017,56.69897349800004],[8.825267,56.70683299800004],[9.002038860000027,56.801205749000076],[8.949772,56.83336649700004],[9.076211,56.825473997000074],[9.021852163000062,56.801562251000064],[9.1771,56.708270999000035],[9.058578500000067,56.62983599700004]]],[[[10.689461,55.89378699800005],[10.621293999000045,55.76484299800006],[10.549744500000031,55.769402],[10.529172,55.85150549600007],[10.601043,55.90681649600003],[10.522061500000063,55.97892749600004],[10.579689,55.947982497000055],[10.689461,55.89378699800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK041","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Vestjylland","NUTS_NAME":"Vestjylland"},"geometry":{"type":"Polygon","coordinates":[[[9.058578500000067,56.62983599700004],[9.050561,56.56131],[9.212072500000033,56.62995549800007],[9.285438500000055,56.62196749800006],[9.247093500000062,56.57242650000006],[9.307318,56.52508749700007],[9.369637,56.53706149800007],[9.287089500000036,56.56544949600004],[9.319437,56.67222699800004],[9.525292,56.680663],[9.79582350000004,56.56161399900003],[9.721719,56.33024599700008],[9.508164,56.308493998000074],[9.43832750000007,56.369549997000036],[9.230953500000055,56.281963],[9.222243500000047,56.233911496000076],[9.366234,56.193317499000045],[9.418871500000023,56.094847497000046],[9.317764,56.100027999000076],[9.345228,56.02710449800003],[9.559031,55.915911496000035],[9.387007500000038,55.92414649800003],[9.219408500000043,55.95471299600007],[9.00953049900005,55.83066149700005],[8.881475500000022,55.89574749900004],[8.744571999000073,55.80639749900007],[8.56659650000006,55.849891],[8.468886500000053,55.773105498000064],[8.17079,55.81536699900005],[8.1178,55.998211],[8.193086093000034,55.87283399800003],[8.279055500000027,55.87795949800005],[8.245650500000067,55.83682249900005],[8.386469,55.90727849900003],[8.23038850000006,56.09285949700006],[8.126061500000048,56.11844599900007],[8.145366500000023,56.007381499000076],[8.099594,56.036021997000034],[8.117563500000074,56.37189649800007],[8.302711500000044,56.31922899700004],[8.126683,56.430427499000075],[8.124725500000068,56.56368850000007],[8.19683,56.69499599900007],[8.231495,56.60677350000003],[8.554322,56.56212599600008],[8.668789,56.46784949900007],[8.763022500000034,56.55245849700003],[8.676833,56.62150950000006],[8.919017,56.69897349800004],[8.825267,56.70683299800004],[9.002038860000027,56.801205749000076],[8.949772,56.83336649700004],[9.076211,56.825473997000074],[9.021852163000062,56.801562251000064],[9.1771,56.708270999000035],[9.058578500000067,56.62983599700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK042","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Østjylland","NUTS_NAME":"Østjylland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.308140108000032,56.60684268400007],[10.504672,56.51335149800008],[10.837805500000059,56.530620498000076],[10.959905500000048,56.44208149800005],[10.712867,56.14205549700006],[10.609916500000054,56.22810749900003],[10.511949500000071,56.09445549900005],[10.480811,56.139278499000056],[10.528027500000064,56.16802199800003],[10.345911,56.19412599900005],[10.507214500000032,56.261523999000076],[10.432433,56.295199998000044],[10.236166500000024,56.180622999000036],[10.210211,56.13629549900003],[10.281900499000074,56.02040149800007],[10.193888500000071,55.83793999900007],[9.865144500000042,55.84770999700004],[10.066128,55.80712099700003],[10.016188500000055,55.766716],[10.083200500000032,55.74549850000005],[10.009511,55.700878],[9.671952500000032,55.71206999700007],[9.447316,55.834682996000026],[9.387007500000038,55.92414649800003],[9.559031,55.915911496000035],[9.345228,56.02710449800003],[9.317764,56.100027999000076],[9.418871500000023,56.094847497000046],[9.366234,56.193317499000045],[9.222243500000047,56.233911496000076],[9.230953500000055,56.281963],[9.43832750000007,56.369549997000036],[9.508164,56.308493998000074],[9.721719,56.33024599700008],[9.79582350000004,56.56161399900003],[10.194114500000069,56.68469350000004],[10.33853350000004,56.70126699900004],[10.359616500000072,56.63785549800008],[10.308140108000032,56.60684268400007]]],[[[10.689461,55.89378699800005],[10.621293999000045,55.76484299800006],[10.549744500000031,55.769402],[10.529172,55.85150549600007],[10.601043,55.90681649600003],[10.522061500000063,55.97892749600004],[10.579689,55.947982497000055],[10.689461,55.89378699800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK05","LEVL_CODE":2,"CNTR_CODE":"DK","NAME_LATN":"Nordjylland","NUTS_NAME":"Nordjylland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.422333500000036,57.14775149700006],[10.331549499000062,56.98897949900004],[10.234805,56.987022496000066],[10.017161,57.086907],[9.924750500000073,57.05464949900005],[9.777845,57.10914999600004],[8.668212,56.95006949700007],[8.625678,56.889027498000075],[8.632456,56.863773500000036],[8.47117,56.78329049900003],[8.515605634000053,56.686523847000046],[8.577901844000053,56.69249834800007],[8.541478,56.78775],[8.666719,56.80917549800006],[8.631644,56.838523999000074],[8.634744500000068,56.88579949800004],[8.816511500000047,56.904053],[8.900349500000061,56.952026498000066],[8.906772,56.896442998000055],[8.833179500000028,56.87339499800004],[8.869944500000031,56.79465149600003],[8.77658850000006,56.69454949700008],[8.58326334700007,56.69057483200004],[8.641994500000067,56.66126249800004],[8.544610999000042,56.57684349600004],[8.404045500000052,56.67023699900005],[8.266428,56.775611999000034],[8.23772450000007,56.701515],[8.232896,56.798159498000075],[8.589805500000068,57.11797349600005],[9.399203500000056,57.164098496000065],[9.958295500000077,57.59529899700004],[10.644709500000033,57.742755499000054],[10.426225500000044,57.58919999900007],[10.54697550000003,57.45014049800005],[10.515677499000049,57.38432699700007],[10.550539,57.23988349600006],[10.422333500000036,57.14775149700006]]],[[[10.272072,56.936893499000064],[10.332744500000047,56.70807649600005],[10.33853350000004,56.70126699900004],[10.194114500000069,56.68469350000004],[9.79582350000004,56.56161399900003],[9.525292,56.680663],[9.319437,56.67222699800004],[9.159611,56.67793249700003],[9.24345,56.74848949600005],[9.185678,56.766875999000035],[9.210255500000073,56.86354849800006],[9.164284,56.888828499000056],[9.29455,56.99829499900005],[9.531736500000022,57.00722699900007],[9.583900500000027,56.987621500000046],[9.538178500000072,56.95572149700007],[9.575669499000071,56.91954849900003],[9.687421500000028,57.03817749700005],[10.039944500000047,57.05941],[10.191666500000053,56.987171],[10.30645,56.98275399800008],[10.272072,56.936893499000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"DK050","LEVL_CODE":3,"CNTR_CODE":"DK","NAME_LATN":"Nordjylland","NUTS_NAME":"Nordjylland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.422333500000036,57.14775149700006],[10.331549499000062,56.98897949900004],[10.234805,56.987022496000066],[10.017161,57.086907],[9.924750500000073,57.05464949900005],[9.777845,57.10914999600004],[8.668212,56.95006949700007],[8.625678,56.889027498000075],[8.632456,56.863773500000036],[8.47117,56.78329049900003],[8.515605634000053,56.686523847000046],[8.577901844000053,56.69249834800007],[8.541478,56.78775],[8.666719,56.80917549800006],[8.631644,56.838523999000074],[8.634744500000068,56.88579949800004],[8.816511500000047,56.904053],[8.900349500000061,56.952026498000066],[8.906772,56.896442998000055],[8.833179500000028,56.87339499800004],[8.869944500000031,56.79465149600003],[8.77658850000006,56.69454949700008],[8.58326334700007,56.69057483200004],[8.641994500000067,56.66126249800004],[8.544610999000042,56.57684349600004],[8.404045500000052,56.67023699900005],[8.266428,56.775611999000034],[8.23772450000007,56.701515],[8.232896,56.798159498000075],[8.589805500000068,57.11797349600005],[9.399203500000056,57.164098496000065],[9.958295500000077,57.59529899700004],[10.644709500000033,57.742755499000054],[10.426225500000044,57.58919999900007],[10.54697550000003,57.45014049800005],[10.515677499000049,57.38432699700007],[10.550539,57.23988349600006],[10.422333500000036,57.14775149700006]]],[[[10.272072,56.936893499000064],[10.332744500000047,56.70807649600005],[10.33853350000004,56.70126699900004],[10.194114500000069,56.68469350000004],[9.79582350000004,56.56161399900003],[9.525292,56.680663],[9.319437,56.67222699800004],[9.159611,56.67793249700003],[9.24345,56.74848949600005],[9.185678,56.766875999000035],[9.210255500000073,56.86354849800006],[9.164284,56.888828499000056],[9.29455,56.99829499900005],[9.531736500000022,57.00722699900007],[9.583900500000027,56.987621500000046],[9.538178500000072,56.95572149700007],[9.575669499000071,56.91954849900003],[9.687421500000028,57.03817749700005],[10.039944500000047,57.05941],[10.191666500000053,56.987171],[10.30645,56.98275399800008],[10.272072,56.936893499000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE","LEVL_CODE":0,"CNTR_CODE":"EE","NAME_LATN":"EESTI","NUTS_NAME":"EESTI"},"geometry":{"type":"MultiPolygon","coordinates":[[[[25.696893,59.66648649900003],[25.830159499000047,59.56406499900004],[25.97139050000004,59.58334649800008],[25.963809,59.635560500000054],[26.759436,59.499518497000054],[27.81278150000003,59.395083999000065],[28.041862500000036,59.47010249900006],[28.209798001000024,59.371387998000046],[27.90429050000006,59.241254997000055],[27.745650500000067,58.98457399600005],[27.357049499000027,58.787144],[27.55321250000003,58.36598149500003],[27.49074599900007,58.225123],[27.621053500000073,58.005111499000066],[27.694286500000032,57.99368749900003],[27.656367500000044,57.95533599700008],[27.678896,57.92370499900005],[27.81819850000005,57.88889349800007],[27.549077500000067,57.83191849900004],[27.545464500000037,57.73333599800003],[27.377525500000047,57.67137749900007],[27.351579,57.51823699800008],[26.92589950000007,57.633293500000036],[26.524905500000045,57.51624749700005],[26.024105500000076,57.76862199900006],[26.056459,57.848431496000046],[25.67896950000005,57.90439199600007],[25.295673,58.08144949800004],[25.264865,58.06008599900008],[25.302209,58.00057849900003],[25.212184500000035,58.080787497000074],[25.046307,58.040146],[24.834082500000022,57.97277949900007],[24.35281750000007,57.87655650000005],[24.478264500000023,58.06639],[24.456178,58.20262599900008],[24.58152,58.32541549800004],[24.30554950000004,58.37915799800004],[24.26787150000007,58.278897],[24.111982500000067,58.24046150000004],[24.010079499000028,58.31985250000008],[23.751265,58.337618],[23.65194899900007,58.51961049800008],[23.521601,58.56487949900003],[23.495821,58.68935350000004],[23.52346650000004,58.736095499000044],[23.803831500000058,58.74600799800004],[23.454556500000024,58.769563497000036],[23.534799500000076,58.814358499000036],[23.43050050000005,58.93722299800004],[23.664107500000057,58.97277850000006],[23.404937,59.02580499900006],[23.541659,59.090069],[23.487214,59.203182499000036],[23.73073850000003,59.23713250000003],[24.09245050000004,59.269714],[24.11595650000004,59.31162799900005],[24.036608500000057,59.38873549700003],[24.241418,59.344277],[24.375312500000064,59.474935997000046],[24.83234550000003,59.46616149700003],[24.796578,59.571803500000044],[24.96559749900007,59.48995199700005],[25.440579,59.492673],[25.553802,59.53765549700006],[25.49711750000006,59.64766050000003],[25.668137,59.563720499000055],[25.70753,59.57967050000008],[25.670612,59.654334],[25.696893,59.66648649900003]]],[[[23.406006,58.55095750000004],[23.127107,58.537262498000075],[23.311371,58.43477499900007],[23.159414,58.47724549800006],[23.088114500000074,58.37832399900003],[22.699216999000043,58.270439],[22.746683,58.22932250000008],[22.719874,58.212165],[22.309404,58.20763799900004],[22.255681,58.17297999900006],[22.20074150000005,57.98174350000005],[22.056729500000074,57.91449599800006],[21.981501,57.97695699900004],[22.224689500000068,58.13846349900007],[21.828614,58.30703299700008],[22.02261,58.36599099600005],[21.895906,58.478811],[21.917137,58.51694699900003],[22.101566500000047,58.41636499800006],[22.20327750000007,58.54603049700006],[22.287071,58.485732],[22.31590250000005,58.56527699700007],[22.543011500000034,58.64205149500003],[23.138064,58.55026399900004],[23.165065500000026,58.56943399800008],[23.06999,58.59950249900004],[23.19201950000007,58.68430049700004],[23.34560750000003,58.61022999700003],[23.406006,58.55095750000004]]],[[[22.770002500000032,58.819814499000074],[22.569418,58.68699399900004],[22.465819,58.715576500000054],[22.503350500000067,58.73316649700007],[22.39301099900007,58.891237997000076],[22.043935500000032,58.929377499000054],[22.397777500000075,58.93771550000008],[22.592449,59.08807299600005],[22.931714500000055,58.971051],[23.069148,58.838146],[22.770002500000032,58.819814499000074]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE0","LEVL_CODE":1,"CNTR_CODE":"EE","NAME_LATN":"EESTI","NUTS_NAME":"EESTI"},"geometry":{"type":"MultiPolygon","coordinates":[[[[25.696893,59.66648649900003],[25.830159499000047,59.56406499900004],[25.97139050000004,59.58334649800008],[25.963809,59.635560500000054],[26.759436,59.499518497000054],[27.81278150000003,59.395083999000065],[28.041862500000036,59.47010249900006],[28.209798001000024,59.371387998000046],[27.90429050000006,59.241254997000055],[27.745650500000067,58.98457399600005],[27.357049499000027,58.787144],[27.55321250000003,58.36598149500003],[27.49074599900007,58.225123],[27.621053500000073,58.005111499000066],[27.694286500000032,57.99368749900003],[27.656367500000044,57.95533599700008],[27.678896,57.92370499900005],[27.81819850000005,57.88889349800007],[27.549077500000067,57.83191849900004],[27.545464500000037,57.73333599800003],[27.377525500000047,57.67137749900007],[27.351579,57.51823699800008],[26.92589950000007,57.633293500000036],[26.524905500000045,57.51624749700005],[26.024105500000076,57.76862199900006],[26.056459,57.848431496000046],[25.67896950000005,57.90439199600007],[25.295673,58.08144949800004],[25.264865,58.06008599900008],[25.302209,58.00057849900003],[25.212184500000035,58.080787497000074],[25.046307,58.040146],[24.834082500000022,57.97277949900007],[24.35281750000007,57.87655650000005],[24.478264500000023,58.06639],[24.456178,58.20262599900008],[24.58152,58.32541549800004],[24.30554950000004,58.37915799800004],[24.26787150000007,58.278897],[24.111982500000067,58.24046150000004],[24.010079499000028,58.31985250000008],[23.751265,58.337618],[23.65194899900007,58.51961049800008],[23.521601,58.56487949900003],[23.495821,58.68935350000004],[23.52346650000004,58.736095499000044],[23.803831500000058,58.74600799800004],[23.454556500000024,58.769563497000036],[23.534799500000076,58.814358499000036],[23.43050050000005,58.93722299800004],[23.664107500000057,58.97277850000006],[23.404937,59.02580499900006],[23.541659,59.090069],[23.487214,59.203182499000036],[23.73073850000003,59.23713250000003],[24.09245050000004,59.269714],[24.11595650000004,59.31162799900005],[24.036608500000057,59.38873549700003],[24.241418,59.344277],[24.375312500000064,59.474935997000046],[24.83234550000003,59.46616149700003],[24.796578,59.571803500000044],[24.96559749900007,59.48995199700005],[25.440579,59.492673],[25.553802,59.53765549700006],[25.49711750000006,59.64766050000003],[25.668137,59.563720499000055],[25.70753,59.57967050000008],[25.670612,59.654334],[25.696893,59.66648649900003]]],[[[23.406006,58.55095750000004],[23.127107,58.537262498000075],[23.311371,58.43477499900007],[23.159414,58.47724549800006],[23.088114500000074,58.37832399900003],[22.699216999000043,58.270439],[22.746683,58.22932250000008],[22.719874,58.212165],[22.309404,58.20763799900004],[22.255681,58.17297999900006],[22.20074150000005,57.98174350000005],[22.056729500000074,57.91449599800006],[21.981501,57.97695699900004],[22.224689500000068,58.13846349900007],[21.828614,58.30703299700008],[22.02261,58.36599099600005],[21.895906,58.478811],[21.917137,58.51694699900003],[22.101566500000047,58.41636499800006],[22.20327750000007,58.54603049700006],[22.287071,58.485732],[22.31590250000005,58.56527699700007],[22.543011500000034,58.64205149500003],[23.138064,58.55026399900004],[23.165065500000026,58.56943399800008],[23.06999,58.59950249900004],[23.19201950000007,58.68430049700004],[23.34560750000003,58.61022999700003],[23.406006,58.55095750000004]]],[[[22.770002500000032,58.819814499000074],[22.569418,58.68699399900004],[22.465819,58.715576500000054],[22.503350500000067,58.73316649700007],[22.39301099900007,58.891237997000076],[22.043935500000032,58.929377499000054],[22.397777500000075,58.93771550000008],[22.592449,59.08807299600005],[22.931714500000055,58.971051],[23.069148,58.838146],[22.770002500000032,58.819814499000074]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE00","LEVL_CODE":2,"CNTR_CODE":"EE","NAME_LATN":"Eesti","NUTS_NAME":"Eesti"},"geometry":{"type":"MultiPolygon","coordinates":[[[[25.696893,59.66648649900003],[25.830159499000047,59.56406499900004],[25.97139050000004,59.58334649800008],[25.963809,59.635560500000054],[26.759436,59.499518497000054],[27.81278150000003,59.395083999000065],[28.041862500000036,59.47010249900006],[28.209798001000024,59.371387998000046],[27.90429050000006,59.241254997000055],[27.745650500000067,58.98457399600005],[27.357049499000027,58.787144],[27.55321250000003,58.36598149500003],[27.49074599900007,58.225123],[27.621053500000073,58.005111499000066],[27.694286500000032,57.99368749900003],[27.656367500000044,57.95533599700008],[27.678896,57.92370499900005],[27.81819850000005,57.88889349800007],[27.549077500000067,57.83191849900004],[27.545464500000037,57.73333599800003],[27.377525500000047,57.67137749900007],[27.351579,57.51823699800008],[26.92589950000007,57.633293500000036],[26.524905500000045,57.51624749700005],[26.024105500000076,57.76862199900006],[26.056459,57.848431496000046],[25.67896950000005,57.90439199600007],[25.295673,58.08144949800004],[25.264865,58.06008599900008],[25.302209,58.00057849900003],[25.212184500000035,58.080787497000074],[25.046307,58.040146],[24.834082500000022,57.97277949900007],[24.35281750000007,57.87655650000005],[24.478264500000023,58.06639],[24.456178,58.20262599900008],[24.58152,58.32541549800004],[24.30554950000004,58.37915799800004],[24.26787150000007,58.278897],[24.111982500000067,58.24046150000004],[24.010079499000028,58.31985250000008],[23.751265,58.337618],[23.65194899900007,58.51961049800008],[23.521601,58.56487949900003],[23.495821,58.68935350000004],[23.52346650000004,58.736095499000044],[23.803831500000058,58.74600799800004],[23.454556500000024,58.769563497000036],[23.534799500000076,58.814358499000036],[23.43050050000005,58.93722299800004],[23.664107500000057,58.97277850000006],[23.404937,59.02580499900006],[23.541659,59.090069],[23.487214,59.203182499000036],[23.73073850000003,59.23713250000003],[24.09245050000004,59.269714],[24.11595650000004,59.31162799900005],[24.036608500000057,59.38873549700003],[24.241418,59.344277],[24.375312500000064,59.474935997000046],[24.83234550000003,59.46616149700003],[24.796578,59.571803500000044],[24.96559749900007,59.48995199700005],[25.440579,59.492673],[25.553802,59.53765549700006],[25.49711750000006,59.64766050000003],[25.668137,59.563720499000055],[25.70753,59.57967050000008],[25.670612,59.654334],[25.696893,59.66648649900003]]],[[[23.406006,58.55095750000004],[23.127107,58.537262498000075],[23.311371,58.43477499900007],[23.159414,58.47724549800006],[23.088114500000074,58.37832399900003],[22.699216999000043,58.270439],[22.746683,58.22932250000008],[22.719874,58.212165],[22.309404,58.20763799900004],[22.255681,58.17297999900006],[22.20074150000005,57.98174350000005],[22.056729500000074,57.91449599800006],[21.981501,57.97695699900004],[22.224689500000068,58.13846349900007],[21.828614,58.30703299700008],[22.02261,58.36599099600005],[21.895906,58.478811],[21.917137,58.51694699900003],[22.101566500000047,58.41636499800006],[22.20327750000007,58.54603049700006],[22.287071,58.485732],[22.31590250000005,58.56527699700007],[22.543011500000034,58.64205149500003],[23.138064,58.55026399900004],[23.165065500000026,58.56943399800008],[23.06999,58.59950249900004],[23.19201950000007,58.68430049700004],[23.34560750000003,58.61022999700003],[23.406006,58.55095750000004]]],[[[22.770002500000032,58.819814499000074],[22.569418,58.68699399900004],[22.465819,58.715576500000054],[22.503350500000067,58.73316649700007],[22.39301099900007,58.891237997000076],[22.043935500000032,58.929377499000054],[22.397777500000075,58.93771550000008],[22.592449,59.08807299600005],[22.931714500000055,58.971051],[23.069148,58.838146],[22.770002500000032,58.819814499000074]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE001","LEVL_CODE":3,"CNTR_CODE":"EE","NAME_LATN":"Põhja-Eesti","NUTS_NAME":"Põhja-Eesti"},"geometry":{"type":"Polygon","coordinates":[[[25.830159499000047,59.56406499900004],[25.84495350000003,59.44398999600003],[25.955085500000052,59.27823599900006],[25.536391500000036,59.26303899700008],[25.375405500000056,59.00704999900006],[24.86776,59.227984],[24.645683500000075,59.21911249900006],[24.568171500000062,59.19154249700006],[24.539940500000057,59.13529550000004],[24.57138850000007,59.09421399900003],[24.09364850000003,58.98935699700007],[24.086451,59.01352699800003],[24.166703,59.054853998000056],[23.73073850000003,59.23713250000003],[24.09245050000004,59.269714],[24.11595650000004,59.31162799900005],[24.036608500000057,59.38873549700003],[24.241418,59.344277],[24.375312500000064,59.474935997000046],[24.83234550000003,59.46616149700003],[24.796578,59.571803500000044],[24.96559749900007,59.48995199700005],[25.440579,59.492673],[25.553802,59.53765549700006],[25.49711750000006,59.64766050000003],[25.668137,59.563720499000055],[25.70753,59.57967050000008],[25.670612,59.654334],[25.696893,59.66648649900003],[25.830159499000047,59.56406499900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE004","LEVL_CODE":3,"CNTR_CODE":"EE","NAME_LATN":"Lääne-Eesti","NUTS_NAME":"Lääne-Eesti"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.07598499900007,58.75238],[24.185920500000066,58.675142],[24.86251050000004,58.75701399700006],[24.931347500000072,58.661320499000055],[25.30718,58.706151498000054],[25.24708,58.628175998000074],[25.01085850000004,58.48441199800004],[24.98016750000005,58.322910500000035],[25.265903500000036,58.256492499000046],[25.014598500000034,58.07356299700007],[25.046307,58.040146],[24.834082500000022,57.97277949900007],[24.35281750000007,57.87655650000005],[24.478264500000023,58.06639],[24.456178,58.20262599900008],[24.58152,58.32541549800004],[24.30554950000004,58.37915799800004],[24.26787150000007,58.278897],[24.111982500000067,58.24046150000004],[24.010079499000028,58.31985250000008],[23.751265,58.337618],[23.65194899900007,58.51961049800008],[23.521601,58.56487949900003],[23.495821,58.68935350000004],[23.52346650000004,58.736095499000044],[23.803831500000058,58.74600799800004],[23.454556500000024,58.769563497000036],[23.534799500000076,58.814358499000036],[23.43050050000005,58.93722299800004],[23.664107500000057,58.97277850000006],[23.404937,59.02580499900006],[23.541659,59.090069],[23.487214,59.203182499000036],[23.73073850000003,59.23713250000003],[24.166703,59.054853998000056],[24.086451,59.01352699800003],[24.09364850000003,58.98935699700007],[24.192442,58.84598549700007],[24.07598499900007,58.75238]]],[[[23.406006,58.55095750000004],[23.127107,58.537262498000075],[23.311371,58.43477499900007],[23.159414,58.47724549800006],[23.088114500000074,58.37832399900003],[22.699216999000043,58.270439],[22.746683,58.22932250000008],[22.719874,58.212165],[22.309404,58.20763799900004],[22.255681,58.17297999900006],[22.20074150000005,57.98174350000005],[22.056729500000074,57.91449599800006],[21.981501,57.97695699900004],[22.224689500000068,58.13846349900007],[21.828614,58.30703299700008],[22.02261,58.36599099600005],[21.895906,58.478811],[21.917137,58.51694699900003],[22.101566500000047,58.41636499800006],[22.20327750000007,58.54603049700006],[22.287071,58.485732],[22.31590250000005,58.56527699700007],[22.543011500000034,58.64205149500003],[23.138064,58.55026399900004],[23.165065500000026,58.56943399800008],[23.06999,58.59950249900004],[23.19201950000007,58.68430049700004],[23.34560750000003,58.61022999700003],[23.406006,58.55095750000004]]],[[[22.770002500000032,58.819814499000074],[22.569418,58.68699399900004],[22.465819,58.715576500000054],[22.503350500000067,58.73316649700007],[22.39301099900007,58.891237997000076],[22.043935500000032,58.929377499000054],[22.397777500000075,58.93771550000008],[22.592449,59.08807299600005],[22.931714500000055,58.971051],[23.069148,58.838146],[22.770002500000032,58.819814499000074]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE006","LEVL_CODE":3,"CNTR_CODE":"EE","NAME_LATN":"Kesk-Eesti","NUTS_NAME":"Kesk-Eesti"},"geometry":{"type":"Polygon","coordinates":[[[26.759436,59.499518497000054],[26.795385,59.407083],[26.744019500000036,59.35255399700003],[26.79112150000003,59.337451498000064],[26.741608,59.26590099800006],[26.95278550000006,59.16470699700005],[26.816972500000077,59.12064850000007],[26.799976,58.98893449900004],[26.70862,58.96264199600006],[26.15756250000004,58.88936049900008],[25.586204500000065,58.634947],[25.24708,58.628175998000074],[25.30718,58.706151498000054],[24.931347500000072,58.661320499000055],[24.86251050000004,58.75701399700006],[24.185920500000066,58.675142],[24.07598499900007,58.75238],[24.192442,58.84598549700007],[24.09364850000003,58.98935699700007],[24.57138850000007,59.09421399900003],[24.539940500000057,59.13529550000004],[24.568171500000062,59.19154249700006],[24.645683500000075,59.21911249900006],[24.86776,59.227984],[25.375405500000056,59.00704999900006],[25.536391500000036,59.26303899700008],[25.955085500000052,59.27823599900006],[25.84495350000003,59.44398999600003],[25.830159499000047,59.56406499900004],[25.97139050000004,59.58334649800008],[25.963809,59.635560500000054],[26.759436,59.499518497000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE007","LEVL_CODE":3,"CNTR_CODE":"EE","NAME_LATN":"Kirde-Eesti","NUTS_NAME":"Kirde-Eesti"},"geometry":{"type":"Polygon","coordinates":[[[28.041862500000036,59.47010249900006],[28.209798001000024,59.371387998000046],[27.90429050000006,59.241254997000055],[27.745650500000067,58.98457399600005],[27.357049499000027,58.787144],[26.755372,58.88014550000008],[26.70862,58.96264199600006],[26.799976,58.98893449900004],[26.816972500000077,59.12064850000007],[26.95278550000006,59.16470699700005],[26.741608,59.26590099800006],[26.79112150000003,59.337451498000064],[26.744019500000036,59.35255399700003],[26.795385,59.407083],[26.759436,59.499518497000054],[27.81278150000003,59.395083999000065],[28.041862500000036,59.47010249900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EE008","LEVL_CODE":3,"CNTR_CODE":"EE","NAME_LATN":"Lõuna-Eesti","NUTS_NAME":"Lõuna-Eesti"},"geometry":{"type":"Polygon","coordinates":[[[26.70862,58.96264199600006],[26.755372,58.88014550000008],[27.357049499000027,58.787144],[27.55321250000003,58.36598149500003],[27.49074599900007,58.225123],[27.621053500000073,58.005111499000066],[27.694286500000032,57.99368749900003],[27.656367500000044,57.95533599700008],[27.678896,57.92370499900005],[27.81819850000005,57.88889349800007],[27.549077500000067,57.83191849900004],[27.545464500000037,57.73333599800003],[27.377525500000047,57.67137749900007],[27.351579,57.51823699800008],[26.92589950000007,57.633293500000036],[26.524905500000045,57.51624749700005],[26.024105500000076,57.76862199900006],[26.056459,57.848431496000046],[25.67896950000005,57.90439199600007],[25.295673,58.08144949800004],[25.264865,58.06008599900008],[25.302209,58.00057849900003],[25.212184500000035,58.080787497000074],[25.046307,58.040146],[25.014598500000034,58.07356299700007],[25.265903500000036,58.256492499000046],[24.98016750000005,58.322910500000035],[25.01085850000004,58.48441199800004],[25.24708,58.628175998000074],[25.586204500000065,58.634947],[26.15756250000004,58.88936049900008],[26.70862,58.96264199600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL533","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Florina","NUTS_NAME":"Φλώρινα"},"geometry":{"type":"Polygon","coordinates":[[[21.787378,40.931125499000075],[21.830457500000023,40.89871999600007],[21.704187,40.86348349900004],[21.822054,40.69629649800004],[21.776451,40.60776149900005],[21.510357,40.52559999700003],[21.262428499000066,40.74907299700004],[21.192651,40.649406498000076],[21.056068500000038,40.61669549900006],[20.959147,40.77049199800007],[20.98020450000007,40.855664999000055],[21.301891500000067,40.865341998000076],[21.417245500000035,40.917696497000065],[21.59323550000005,40.86600249700007],[21.684081,40.94091499800004],[21.787378,40.931125499000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL54","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Ipeiros","NUTS_NAME":"Ήπειρος"},"geometry":{"type":"Polygon","coordinates":[[[21.226316500000053,39.80415349800006],[21.227146,39.708251999000026],[21.120413,39.679416497000034],[21.185011,39.60934849700004],[21.181753,39.501956998000026],[21.27246850000006,39.377406998000026],[21.38977250000005,39.35453799900006],[21.428675,39.295840998000074],[21.349401500000056,39.20803449700003],[21.37354650000003,39.17473599700003],[21.271143,39.117335998000044],[21.127334500000075,39.12431699800004],[21.087236500000074,39.08039099800004],[21.10860850000006,39.04614650000008],[21.047264,39.00017949700003],[20.803007,39.117034998000065],[20.738746,39.01890949800003],[20.831041500000026,38.95981199800008],[20.758966500000042,38.97145849800006],[20.757292,38.950458500000025],[20.726776,38.949634496000044],[20.705598999000074,39.04753899700006],[20.474756,39.217723996000075],[20.47782150000006,39.27484149800006],[20.464247,39.27310199800007],[20.30067850000006,39.31573499600006],[20.212507500000072,39.446082999000055],[20.274601,39.44560599700003],[20.212242,39.49127899900003],[20.262635999000054,39.51013950000004],[20.136205500000074,39.52841949800006],[20.18475150000006,39.62173099800003],[20.008812500000033,39.691292],[20.226089500000057,39.64782849900007],[20.321796,39.72885249800004],[20.289463500000068,39.80479249900003],[20.391308,39.78848349900005],[20.41706750000003,39.840948998000044],[20.31223650000004,39.991022497000074],[20.393286,39.99561699700007],[20.455286500000057,40.074255497000024],[20.678365,40.096003],[20.709584,40.273352497000076],[20.778501,40.34878899800003],[20.868489,40.37674599800005],[21.004242,40.15436149800007],[20.95655050000005,40.13332349900003],[21.01074,40.055736497000055],[21.085388,40.054710999000065],[21.023021500000027,39.99385849600003],[21.097473,39.88035149700005],[21.296484500000076,39.85777299700004],[21.226316500000053,39.80415349800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL541","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Arta, Prebeza","NUTS_NAME":"Άρτα, Πρέβεζα"},"geometry":{"type":"Polygon","coordinates":[[[21.181753,39.501956998000026],[21.27246850000006,39.377406998000026],[21.38977250000005,39.35453799900006],[21.428675,39.295840998000074],[21.349401500000056,39.20803449700003],[21.37354650000003,39.17473599700003],[21.271143,39.117335998000044],[21.127334500000075,39.12431699800004],[21.087236500000074,39.08039099800004],[21.10860850000006,39.04614650000008],[21.047264,39.00017949700003],[20.803007,39.117034998000065],[20.738746,39.01890949800003],[20.831041500000026,38.95981199800008],[20.758966500000042,38.97145849800006],[20.757292,38.950458500000025],[20.726776,38.949634496000044],[20.705598999000074,39.04753899700006],[20.474756,39.217723996000075],[20.47782150000006,39.27484149800006],[20.640453,39.333347497000034],[20.680445,39.28834149800008],[20.742061500000034,39.381122497000035],[20.815332500000068,39.34717199800008],[20.992342,39.39329549800004],[21.059881,39.50614949900006],[21.147371500000077,39.53987099700004],[21.181753,39.501956998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL542","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Thesprotia","NUTS_NAME":"Θεσπρωτία"},"geometry":{"type":"Polygon","coordinates":[[[20.529415,39.665161],[20.51057050000003,39.587218998000026],[20.614967500000034,39.49525399600003],[20.680445,39.28834149800008],[20.640453,39.333347497000034],[20.47782150000006,39.27484149800006],[20.464247,39.27310199800007],[20.30067850000006,39.31573499600006],[20.212507500000072,39.446082999000055],[20.274601,39.44560599700003],[20.212242,39.49127899900003],[20.262635999000054,39.51013950000004],[20.136205500000074,39.52841949800006],[20.18475150000006,39.62173099800003],[20.008812500000033,39.691292],[20.226089500000057,39.64782849900007],[20.321796,39.72885249800004],[20.289463500000068,39.80479249900003],[20.391308,39.78848349900005],[20.491705,39.75796899900007],[20.529415,39.665161]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL543","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ioannina","NUTS_NAME":"Ιωάννινα"},"geometry":{"type":"Polygon","coordinates":[[[21.004242,40.15436149800007],[20.95655050000005,40.13332349900003],[21.01074,40.055736497000055],[21.085388,40.054710999000065],[21.023021500000027,39.99385849600003],[21.097473,39.88035149700005],[21.296484500000076,39.85777299700004],[21.226316500000053,39.80415349800006],[21.227146,39.708251999000026],[21.120413,39.679416497000034],[21.185011,39.60934849700004],[21.181753,39.501956998000026],[21.147371500000077,39.53987099700004],[21.059881,39.50614949900006],[20.992342,39.39329549800004],[20.815332500000068,39.34717199800008],[20.742061500000034,39.381122497000035],[20.680445,39.28834149800008],[20.614967500000034,39.49525399600003],[20.51057050000003,39.587218998000026],[20.529415,39.665161],[20.491705,39.75796899900007],[20.391308,39.78848349900005],[20.41706750000003,39.840948998000044],[20.31223650000004,39.991022497000074],[20.393286,39.99561699700007],[20.455286500000057,40.074255497000024],[20.678365,40.096003],[20.709584,40.273352497000076],[20.778501,40.34878899800003],[20.868489,40.37674599800005],[21.004242,40.15436149800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL6","LEVL_CODE":1,"CNTR_CODE":"EL","NAME_LATN":"KENTRIKI ELLADA","NUTS_NAME":"ΚΕΝΤΡΙΚΗ ΕΛΛΑΔΑ"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.682358500000078,38.77332499800008],[24.563664,38.76233099800004],[24.530308500000046,38.79647249800007],[24.56670850000006,38.83881549700004],[24.430897,38.88147899900008],[24.448941,38.94910199800006],[24.573275500000022,38.92756050000003],[24.682358500000078,38.77332499800008]]],[[[23.426233500000023,38.90583049800006],[23.754823500000043,38.706645998000056],[24.155115,38.65118799900006],[24.122717,38.59658799700003],[24.232042500000034,38.52455149900004],[24.181846500000063,38.39474849800007],[24.255039500000066,38.288509497000064],[24.24803550000007,38.22748199800003],[24.350046,38.15526949800005],[24.554556,38.14113999800003],[24.58396350000004,38.02422349900007],[24.510714,37.94909299700004],[24.41704550000003,38.01402649600004],[24.373417,37.96247849700006],[24.30678,38.06962199800006],[24.202724499000055,38.08532349700005],[24.207344,38.179832496000074],[24.153744,38.27940399800008],[24.111498,38.25109099800005],[24.048601,38.395148997000035],[23.648699,38.39869699900004],[23.60085299900004,38.57010649700004],[23.511806500000034,38.58705499700005],[23.196602,38.834670998000036],[22.99010450000003,38.88124449800006],[22.829393500000037,38.825450998000065],[23.12924,39.007640998000056],[23.317873,39.03850549800006],[23.429733,38.95418549800007],[23.426233500000023,38.90583049800006]]],[[[23.690382475000035,39.087570553000035],[23.661911,39.08702099900006],[23.655265069000052,39.101519418000066],[23.651362178000056,39.110033764000036],[23.636522608000064,39.14240700100004],[23.63581978700006,39.143940239000074],[23.634577274000037,39.14665084100005],[23.605493500000023,39.210098499000026],[23.613432827000054,39.206438615000025],[23.614415847000032,39.20598546100007],[23.777410148000058,39.13084809900005],[23.778539283000043,39.130327589000046],[23.786434,39.126688270000045],[23.790044972000032,39.12502367800005],[23.791225352000026,39.12447954500004],[23.79319950000007,39.12356949900004],[23.789729575000024,39.11905838800004],[23.786377085000026,39.114699949000055],[23.784975072000066,39.11287724700003],[23.78412032600005,39.111766026000055],[23.76924678000006,39.09242952000005],[23.768871813000032,39.09194204100004],[23.767921263000062,39.090706268000076],[23.76664150000005,39.08904249900007],[23.763614216000065,39.08898406600008],[23.757521961000066,39.08886647400004],[23.75650874300004,39.08884691700007],[23.700251877000028,39.087761052000076],[23.698424143000068,39.087725773000045],[23.69460888800006,39.08765213100003],[23.690382475000035,39.087570553000035]]],[[[22.20723349900004,40.19213499600005],[22.21593850000005,40.10359199800007],[22.319086,40.182364998000025],[22.370580500000074,40.14012149900003],[22.378916,40.03908899800007],[22.489490500000045,40.01286299800006],[22.508739500000047,39.956676498000036],[22.66243,39.975547998000025],[22.852674500000035,39.800433998000074],[22.915911,39.60293599800008],[22.976824,39.55532849800005],[23.095781500000044,39.499927498000034],[23.351476500000047,39.18777449900006],[23.073730500000067,39.08465199900007],[23.08049950000003,39.147334998000076],[23.11130950000006,39.14481349700003],[23.096821500000033,39.09813299700005],[23.212417500000072,39.14687349600007],[23.163364500000057,39.27289949900006],[22.94211,39.361980499000026],[22.94639,39.299728497000046],[22.821161500000073,39.27750750000007],[22.85026750000003,39.154559998000025],[22.97913,39.09670249800007],[22.997400500000026,39.040111497000055],[22.96126950000007,39.00722899800007],[23.06714450000004,39.038768997000034],[23.01806650000003,39.00265499600005],[22.990511,38.94419099800007],[22.831438,38.889121997000075],[22.617443,38.91184999800004],[22.527525,38.85890949900005],[22.863748499000053,38.754974498000024],[23.034622,38.75635499800006],[23.10184650000008,38.62757849700006],[23.19618999900007,38.680735998000046],[23.321071500000073,38.63822949900003],[23.340513500000043,38.60502249900003],[23.296520500000042,38.56784449700007],[23.37779050000006,38.52575699700003],[23.328289,38.501220499000055],[23.560117,38.502597997000066],[23.65513450000003,38.35249699800005],[23.690418,38.34029749800004],[23.634155500000077,38.21131149800004],[23.515825500000062,38.13584149800005],[23.460073500000078,38.20882049900007],[23.37436850000006,38.21328349800007],[23.393368,38.263484998000024],[23.29180350000007,38.23928049700004],[23.28664,38.18450949700008],[23.126179,38.1684],[23.11132250000003,38.20705399800005],[23.066992,38.170379998000044],[22.779249,38.23318499800007],[22.79750450000006,38.27519599800007],[22.691986,38.317390499000055],[22.679586500000028,38.37295549700008],[22.60806850000006,38.35940149800007],[22.560972,38.280079],[22.419674,38.43173599900007],[22.378385500000036,38.33254249700008],[21.941110500000036,38.40624599700004],[21.851717,38.37451949700005],[21.486598730000026,38.301853583000025],[21.486297500000035,38.32768649800005],[21.484744472000045,38.306621567000036],[21.318926,38.49962999600007],[21.335010500000067,38.388656499000035],[21.14287950000005,38.303787499000066],[21.10001550000004,38.33328649900005],[21.146599,38.41347499800003],[21.07562450000006,38.48397799800006],[21.096044500000062,38.53156649700003],[21.02672,38.503150997000034],[20.988678,38.66868599800006],[20.914373500000067,38.67372899800006],[20.871914,38.794937498000024],[20.780818999000076,38.75421149700003],[20.726846500000022,38.814430498000036],[20.76258850000005,38.862751],[20.74679,38.922439498000074],[20.764536,38.95730949800003],[20.810058500000025,38.91819749900003],[20.978819,38.945262498000034],[21.066231,38.87713649800003],[21.164858,38.977615497000045],[21.10860850000006,39.04614650000008],[21.087236500000074,39.08039099800004],[21.127334500000075,39.12431699800004],[21.271143,39.117335998000044],[21.37354650000003,39.17473599700003],[21.349401500000056,39.20803449700003],[21.428675,39.295840998000074],[21.38977250000005,39.35453799900006],[21.27246850000006,39.377406998000026],[21.181753,39.501956998000026],[21.185011,39.60934849700004],[21.120413,39.679416497000034],[21.227146,39.708251999000026],[21.226316500000053,39.80415349800006],[21.296484500000076,39.85777299700004],[21.383671,39.83225249600008],[21.42811450000005,39.89455049600008],[21.918218500000023,39.852611498000044],[21.883821500000067,39.955154496000034],[22.115207500000054,40.189948997000045],[22.20723349900004,40.19213499600005]]],[[[21.871086,38.33453749700004],[22.37297250000006,38.14222349700003],[22.639021,38.07839199700004],[22.856842,37.93777849600008],[22.962061,37.94956949800007],[23.017891,37.900067998000054],[23.008983500000056,37.84590299700005],[23.178438,37.803564497000025],[23.12003150000004,37.72998499800008],[23.200378500000056,37.596653],[23.17948749900006,37.52343349800003],[23.454609,37.46235649800008],[23.424419500000056,37.411700997000025],[23.268150500000047,37.40636049900007],[23.224903,37.371658498000045],[23.277609,37.34850299800007],[23.193193500000064,37.341045499000074],[23.171631,37.286002997000026],[23.06308,37.39172350000007],[23.14237950000006,37.415977499000064],[23.121271,37.450256498000044],[22.725086,37.57263949700007],[22.774015500000075,37.38739799700005],[22.893175,37.219043499000065],[22.897411500000032,37.11927050000003],[22.95490650000005,37.11929299800005],[22.982481,37.04802699700008],[23.088842500000055,36.81272499900007],[23.086452500000064,36.729018997000026],[23.027857,36.72883549900007],[23.03039350000006,36.63529199800007],[23.196792500000072,36.43466199900007],[22.968313,36.51951199800004],[22.785621500000047,36.796966498000074],[22.63109950000006,36.80678949900005],[22.565794,36.77590199700006],[22.57607450000006,36.73329199800003],[22.476125500000023,36.613116999000056],[22.480207500000063,36.39754099700008],[22.348186500000054,36.51036449800006],[22.386833,36.59114099900006],[22.29641350000003,36.813945997000076],[22.225924,36.893966499000044],[22.126909,36.91332249800007],[22.149515,37.01814649800008],[21.930139500000053,36.98115549900007],[21.957984999000075,36.79297249800004],[21.877396,36.71761649700005],[21.822733,36.80372199900006],[21.704893,36.81129049900005],[21.67955,36.888435498000035],[21.710382500000037,36.94160099900006],[21.576706,37.06180199800008],[21.565725499000052,37.16350949900004],[21.691736,37.293105998000044],[21.680233,37.37743399800007],[21.530187500000068,37.56888199800005],[21.32807150000008,37.66140349700004],[21.28442,37.778659997000034],[21.105238,37.848926498000026],[21.126194,37.93867499600003],[21.255453,37.97354499900007],[21.351074,38.102138498000045],[21.372593,38.218120498000076],[21.605222500000025,38.142891],[21.778976500000056,38.30762499800005],[21.871086,38.33453749700004]]],[[[23.133437,37.920439498000064],[23.009341,37.91685749900006],[22.84889350000003,38.02820649900008],[22.94963750000005,38.07875049800003],[23.117532,38.06064599700005],[23.106703,37.99519749700005],[23.179699500000027,37.95158999700004],[23.133437,37.920439498000064]]],[[[20.899757,37.77567349800006],[20.988697,37.70522249600003],[20.878824,37.72989299900007],[20.829103,37.644634997000026],[20.625544,37.820702],[20.63263099900007,37.88441899800006],[20.71006,37.922859998000035],[20.899757,37.77567349800006]]],[[[20.628674,38.32527149600003],[20.601241999000024,38.30007949700007],[20.643795500000067,38.251945498000055],[20.680998500000044,38.27783199700008],[20.81529050000006,38.11352899700006],[20.796285500000067,38.065116998000065],[20.514303,38.103931499000055],[20.444082,38.28076949800004],[20.440220500000066,38.161388498000065],[20.34178550000007,38.17613199600004],[20.407196,38.34989549800008],[20.477171,38.309573998000076],[20.547944999000038,38.36228549700007],[20.536981500000024,38.46981049800007],[20.582451,38.46472149900006],[20.628674,38.32527149600003]]],[[[20.726379697000027,38.30626380100006],[20.723535500000025,38.30091499900004],[20.681861397000034,38.364154245000066],[20.681703518000063,38.36439382200007],[20.67858482300005,38.369126351000034],[20.676870332000078,38.37172804100004],[20.67638760500006,38.37246056500004],[20.674318576000076,38.37560025700003],[20.642205783000065,38.424330492000024],[20.640100338000025,38.427525443000036],[20.63224221300004,38.43944992100006],[20.627883214000065,38.446064576000026],[20.621872360000054,38.45518587400005],[20.621309320000023,38.456040270000074],[20.61507050000006,38.46550749900007],[20.627280287000076,38.47754448400008],[20.627569132000076,38.47782924100005],[20.627646765000065,38.47790577500007],[20.647870010000076,38.49784280500006],[20.64853775800003,38.49850110300008],[20.650018871000043,38.49996125400003],[20.650058971000078,38.50000078700003],[20.65047734600006,38.50041324000006],[20.651176500000076,38.50110249900007],[20.672380662000023,38.473754229000065],[20.672496657000067,38.47360462300003],[20.687026349000064,38.45486481300003],[20.687279871000044,38.45453783100004],[20.690643067000053,38.450200116000076],[20.693599595000023,38.446386905000054],[20.70467900400007,38.432097132000024],[20.705893076000052,38.43053126900003],[20.746239073000027,38.378494630000034],[20.746346508000045,38.37835606400006],[20.746497791000024,38.37816094600004],[20.756654337000043,38.36506144300006],[20.757242,38.36430349900007],[20.743891635000068,38.339196779000076],[20.743751848000045,38.33893389600007],[20.726379697000027,38.30626380100006]]],[[[20.721731,38.626224498000056],[20.551553500000068,38.577827497000044],[20.59967050000006,38.779578997000044],[20.693756,38.85181799700007],[20.721731,38.626224498000056]]],[[[20.072651500000063,39.453790498000046],[20.109863500000074,39.36097199900007],[19.868664500000023,39.456530996000026],[19.831421500000033,39.57359249800004],[19.641207,39.749874498000054],[19.673725500000046,39.79635899800007],[19.91553650000003,39.79195249800006],[19.95502350000004,39.766340498000034],[19.845195,39.707595996000066],[19.841421500000024,39.659159498000065],[19.93253,39.62501399800004],[19.93844449900007,39.47206149800007],[20.072651500000063,39.453790498000046]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL61","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Thessalia","NUTS_NAME":"Θεσσαλία"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.690382475000035,39.087570553000035],[23.661911,39.08702099900006],[23.655265069000052,39.101519418000066],[23.651362178000056,39.110033764000036],[23.636522608000064,39.14240700100004],[23.63581978700006,39.143940239000074],[23.634577274000037,39.14665084100005],[23.605493500000023,39.210098499000026],[23.613432827000054,39.206438615000025],[23.614415847000032,39.20598546100007],[23.777410148000058,39.13084809900005],[23.778539283000043,39.130327589000046],[23.786434,39.126688270000045],[23.790044972000032,39.12502367800005],[23.791225352000026,39.12447954500004],[23.79319950000007,39.12356949900004],[23.789729575000024,39.11905838800004],[23.786377085000026,39.114699949000055],[23.784975072000066,39.11287724700003],[23.78412032600005,39.111766026000055],[23.76924678000006,39.09242952000005],[23.768871813000032,39.09194204100004],[23.767921263000062,39.090706268000076],[23.76664150000005,39.08904249900007],[23.763614216000065,39.08898406600008],[23.757521961000066,39.08886647400004],[23.75650874300004,39.08884691700007],[23.700251877000028,39.087761052000076],[23.698424143000068,39.087725773000045],[23.69460888800006,39.08765213100003],[23.690382475000035,39.087570553000035]]],[[[22.20723349900004,40.19213499600005],[22.21593850000005,40.10359199800007],[22.319086,40.182364998000025],[22.370580500000074,40.14012149900003],[22.378916,40.03908899800007],[22.489490500000045,40.01286299800006],[22.508739500000047,39.956676498000036],[22.66243,39.975547998000025],[22.852674500000035,39.800433998000074],[22.915911,39.60293599800008],[22.976824,39.55532849800005],[23.095781500000044,39.499927498000034],[23.351476500000047,39.18777449900006],[23.073730500000067,39.08465199900007],[23.08049950000003,39.147334998000076],[23.11130950000006,39.14481349700003],[23.096821500000033,39.09813299700005],[23.212417500000072,39.14687349600007],[23.163364500000057,39.27289949900006],[22.94211,39.361980499000026],[22.94639,39.299728497000046],[22.821161500000073,39.27750750000007],[22.85026750000003,39.154559998000025],[22.97913,39.09670249800007],[22.997400500000026,39.040111497000055],[22.96126950000007,39.00722899800007],[23.06714450000004,39.038768997000034],[23.01806650000003,39.00265499600005],[22.930694500000072,38.969772498000054],[22.514013500000033,39.04854949800006],[22.505670500000065,39.13119899900005],[22.258116,39.272308499000076],[22.167362,39.12552649700007],[21.952619500000026,39.02455149800005],[21.939703,39.065337998000075],[21.858555,39.15161499800007],[21.728924,39.11552049800008],[21.639002,39.25025149800007],[21.396904,39.16447849600007],[21.37354650000003,39.17473599700003],[21.349401500000056,39.20803449700003],[21.428675,39.295840998000074],[21.38977250000005,39.35453799900006],[21.27246850000006,39.377406998000026],[21.181753,39.501956998000026],[21.185011,39.60934849700004],[21.120413,39.679416497000034],[21.227146,39.708251999000026],[21.226316500000053,39.80415349800006],[21.296484500000076,39.85777299700004],[21.383671,39.83225249600008],[21.42811450000005,39.89455049600008],[21.918218500000023,39.852611498000044],[21.883821500000067,39.955154496000034],[22.115207500000054,40.189948997000045],[22.20723349900004,40.19213499600005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL611","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Karditsa, Trikala","NUTS_NAME":"Καρδίτσα, Τρίκαλα"},"geometry":{"type":"Polygon","coordinates":[[[21.918218500000023,39.852611498000044],[22.027919500000053,39.829005996000035],[21.921249500000044,39.77846149900006],[21.961603,39.709797],[22.028099,39.73291799800006],[22.07974450000006,39.65111949900006],[22.217598,39.621909998000035],[22.17589950000007,39.53241749800003],[22.236686500000076,39.51890949700004],[22.221634,39.42489999800006],[22.263551500000062,39.37611399700006],[22.212953500000026,39.310298998000064],[22.258116,39.272308499000076],[22.167362,39.12552649700007],[21.952619500000026,39.02455149800005],[21.939703,39.065337998000075],[21.858555,39.15161499800007],[21.728924,39.11552049800008],[21.639002,39.25025149800007],[21.396904,39.16447849600007],[21.37354650000003,39.17473599700003],[21.349401500000056,39.20803449700003],[21.428675,39.295840998000074],[21.38977250000005,39.35453799900006],[21.27246850000006,39.377406998000026],[21.181753,39.501956998000026],[21.185011,39.60934849700004],[21.120413,39.679416497000034],[21.227146,39.708251999000026],[21.226316500000053,39.80415349800006],[21.296484500000076,39.85777299700004],[21.383671,39.83225249600008],[21.42811450000005,39.89455049600008],[21.918218500000023,39.852611498000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL612","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Larisa","NUTS_NAME":"Λάρισα"},"geometry":{"type":"Polygon","coordinates":[[[22.66243,39.975547998000025],[22.852674500000035,39.800433998000074],[22.915911,39.60293599800008],[22.757719,39.52566549800008],[22.700319500000035,39.447505998000054],[22.721868500000028,39.42140150000006],[22.60813150000007,39.400554498000076],[22.669069500000035,39.35472499800005],[22.655466,39.30368049700007],[22.531555500000024,39.25735449800004],[22.555687,39.16275849800007],[22.505670500000065,39.13119899900005],[22.258116,39.272308499000076],[22.212953500000026,39.310298998000064],[22.263551500000062,39.37611399700006],[22.221634,39.42489999800006],[22.236686500000076,39.51890949700004],[22.17589950000007,39.53241749800003],[22.217598,39.621909998000035],[22.07974450000006,39.65111949900006],[22.028099,39.73291799800006],[21.961603,39.709797],[21.921249500000044,39.77846149900006],[22.027919500000053,39.829005996000035],[21.918218500000023,39.852611498000044],[21.883821500000067,39.955154496000034],[22.115207500000054,40.189948997000045],[22.20723349900004,40.19213499600005],[22.21593850000005,40.10359199800007],[22.319086,40.182364998000025],[22.370580500000074,40.14012149900003],[22.378916,40.03908899800007],[22.489490500000045,40.01286299800006],[22.508739500000047,39.956676498000036],[22.66243,39.975547998000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL613","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Magnisia","NUTS_NAME":"Μαγνησία"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.690382475000035,39.087570553000035],[23.661911,39.08702099900006],[23.655265069000052,39.101519418000066],[23.651362178000056,39.110033764000036],[23.636522608000064,39.14240700100004],[23.63581978700006,39.143940239000074],[23.634577274000037,39.14665084100005],[23.605493500000023,39.210098499000026],[23.613432827000054,39.206438615000025],[23.614415847000032,39.20598546100007],[23.777410148000058,39.13084809900005],[23.778539283000043,39.130327589000046],[23.786434,39.126688270000045],[23.790044972000032,39.12502367800005],[23.791225352000026,39.12447954500004],[23.79319950000007,39.12356949900004],[23.789729575000024,39.11905838800004],[23.786377085000026,39.114699949000055],[23.784975072000066,39.11287724700003],[23.78412032600005,39.111766026000055],[23.76924678000006,39.09242952000005],[23.768871813000032,39.09194204100004],[23.767921263000062,39.090706268000076],[23.76664150000005,39.08904249900007],[23.763614216000065,39.08898406600008],[23.757521961000066,39.08886647400004],[23.75650874300004,39.08884691700007],[23.700251877000028,39.087761052000076],[23.698424143000068,39.087725773000045],[23.69460888800006,39.08765213100003],[23.690382475000035,39.087570553000035]]],[[[23.095781500000044,39.499927498000034],[23.351476500000047,39.18777449900006],[23.073730500000067,39.08465199900007],[23.08049950000003,39.147334998000076],[23.11130950000006,39.14481349700003],[23.096821500000033,39.09813299700005],[23.212417500000072,39.14687349600007],[23.163364500000057,39.27289949900006],[22.94211,39.361980499000026],[22.94639,39.299728497000046],[22.821161500000073,39.27750750000007],[22.85026750000003,39.154559998000025],[22.97913,39.09670249800007],[22.997400500000026,39.040111497000055],[22.96126950000007,39.00722899800007],[23.06714450000004,39.038768997000034],[23.01806650000003,39.00265499600005],[22.930694500000072,38.969772498000054],[22.514013500000033,39.04854949800006],[22.505670500000065,39.13119899900005],[22.555687,39.16275849800007],[22.531555500000024,39.25735449800004],[22.655466,39.30368049700007],[22.669069500000035,39.35472499800005],[22.60813150000007,39.400554498000076],[22.721868500000028,39.42140150000006],[22.700319500000035,39.447505998000054],[22.757719,39.52566549800008],[22.915911,39.60293599800008],[22.976824,39.55532849800005],[23.095781500000044,39.499927498000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL62","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Ionia Nisia","NUTS_NAME":"Ιόνια Νησιά"},"geometry":{"type":"MultiPolygon","coordinates":[[[[20.899757,37.77567349800006],[20.988697,37.70522249600003],[20.878824,37.72989299900007],[20.829103,37.644634997000026],[20.625544,37.820702],[20.63263099900007,37.88441899800006],[20.71006,37.922859998000035],[20.899757,37.77567349800006]]],[[[20.628674,38.32527149600003],[20.601241999000024,38.30007949700007],[20.643795500000067,38.251945498000055],[20.680998500000044,38.27783199700008],[20.81529050000006,38.11352899700006],[20.796285500000067,38.065116998000065],[20.514303,38.103931499000055],[20.444082,38.28076949800004],[20.440220500000066,38.161388498000065],[20.34178550000007,38.17613199600004],[20.407196,38.34989549800008],[20.477171,38.309573998000076],[20.547944999000038,38.36228549700007],[20.536981500000024,38.46981049800007],[20.582451,38.46472149900006],[20.628674,38.32527149600003]]],[[[20.726379697000027,38.30626380100006],[20.723535500000025,38.30091499900004],[20.681861397000034,38.364154245000066],[20.681703518000063,38.36439382200007],[20.67858482300005,38.369126351000034],[20.676870332000078,38.37172804100004],[20.67638760500006,38.37246056500004],[20.674318576000076,38.37560025700003],[20.642205783000065,38.424330492000024],[20.640100338000025,38.427525443000036],[20.63224221300004,38.43944992100006],[20.627883214000065,38.446064576000026],[20.621872360000054,38.45518587400005],[20.621309320000023,38.456040270000074],[20.61507050000006,38.46550749900007],[20.627280287000076,38.47754448400008],[20.627569132000076,38.47782924100005],[20.627646765000065,38.47790577500007],[20.647870010000076,38.49784280500006],[20.64853775800003,38.49850110300008],[20.650018871000043,38.49996125400003],[20.650058971000078,38.50000078700003],[20.65047734600006,38.50041324000006],[20.651176500000076,38.50110249900007],[20.672380662000023,38.473754229000065],[20.672496657000067,38.47360462300003],[20.687026349000064,38.45486481300003],[20.687279871000044,38.45453783100004],[20.690643067000053,38.450200116000076],[20.693599595000023,38.446386905000054],[20.70467900400007,38.432097132000024],[20.705893076000052,38.43053126900003],[20.746239073000027,38.378494630000034],[20.746346508000045,38.37835606400006],[20.746497791000024,38.37816094600004],[20.756654337000043,38.36506144300006],[20.757242,38.36430349900007],[20.743891635000068,38.339196779000076],[20.743751848000045,38.33893389600007],[20.726379697000027,38.30626380100006]]],[[[20.721731,38.626224498000056],[20.551553500000068,38.577827497000044],[20.59967050000006,38.779578997000044],[20.693756,38.85181799700007],[20.721731,38.626224498000056]]],[[[20.072651500000063,39.453790498000046],[20.109863500000074,39.36097199900007],[19.868664500000023,39.456530996000026],[19.831421500000033,39.57359249800004],[19.641207,39.749874498000054],[19.673725500000046,39.79635899800007],[19.91553650000003,39.79195249800006],[19.95502350000004,39.766340498000034],[19.845195,39.707595996000066],[19.841421500000024,39.659159498000065],[19.93253,39.62501399800004],[19.93844449900007,39.47206149800007],[20.072651500000063,39.453790498000046]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL621","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Zakynthos","NUTS_NAME":"Ζάκυνθος"},"geometry":{"type":"Polygon","coordinates":[[[20.899757,37.77567349800006],[20.988697,37.70522249600003],[20.878824,37.72989299900007],[20.829103,37.644634997000026],[20.625544,37.820702],[20.63263099900007,37.88441899800006],[20.71006,37.922859998000035],[20.899757,37.77567349800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL622","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Kerkyra","NUTS_NAME":"Κέρκυρα"},"geometry":{"type":"Polygon","coordinates":[[[19.91553650000003,39.79195249800006],[19.95502350000004,39.766340498000034],[19.845195,39.707595996000066],[19.841421500000024,39.659159498000065],[19.93253,39.62501399800004],[19.93844449900007,39.47206149800007],[20.072651500000063,39.453790498000046],[20.109863500000074,39.36097199900007],[19.868664500000023,39.456530996000026],[19.831421500000033,39.57359249800004],[19.641207,39.749874498000054],[19.673725500000046,39.79635899800007],[19.91553650000003,39.79195249800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES","LEVL_CODE":0,"CNTR_CODE":"ES","NAME_LATN":"ESPAÑA","NUTS_NAME":"ESPAÑA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.31011,39.87821599800003],[4.28757,39.814689498000064],[4.02217550000006,39.92149749600003],[3.823853500000041,39.921970498000064],[3.791304,40.016177998000046],[4.094098,40.05835349800003],[4.264508500000034,39.980529998000065],[4.31011,39.87821599800003]]],[[[3.079701,39.901542498000026],[3.109828500000049,39.85826099900004],[3.189126500000043,39.88608549700007],[3.119941999000048,39.83151649700005],[3.144903500000055,39.77384950000004],[3.254287,39.729602999000065],[3.348478500000056,39.788887],[3.43512,39.74422449800005],[3.455952500000024,39.65670399700008],[3.301008,39.50396349700003],[3.229418500000065,39.35907349800004],[3.073082500000055,39.268691998000065],[2.956790500000068,39.362800496000034],[2.787235,39.36309449600003],[2.721845500000029,39.47583799700004],[2.751244500000041,39.503772998000045],[2.689343999000073,39.552272997000046],[2.538242500000024,39.52297599900004],[2.522538500000053,39.45799999600007],[2.344436,39.58722699700007],[2.77852050000007,39.85567899800003],[3.213000500000021,39.95537599600004],[3.079701,39.901542498000026]]],[[[-7.491478499999971,43.72754299700006],[-7.245640499999979,43.56518199800007],[-7.031837,43.54447149600003],[-6.107349,43.55629349600008],[-5.84171,43.65595649800008],[-5.694942,43.54321299800006],[-5.407633,43.55561049800008],[-5.214872,43.47370850000004],[-4.512300998999933,43.393204],[-4.174620499999946,43.40098950000004],[-3.785064998999928,43.49179049900005],[-3.834026,43.44116599900008],[-3.814483998999947,43.40204999800005],[-3.587068,43.51339750100004],[-3.435443,43.464725498000064],[-3.42664100099995,43.413513],[-3.153338,43.35322199800004],[-3.017594499999973,43.32303249800003],[-3.037911,43.37359999600005],[-2.94542649999994,43.435573496000075],[-2.746815499999968,43.452121498000054],[-2.508807499999932,43.37654499800004],[-2.412847,43.321082997000076],[-2.178597499999967,43.287845497000035],[-1.790047499999957,43.395621997000035],[-1.785978,43.35047899600005],[-1.728903,43.29608899600004],[-1.625259,43.30670549800004],[-1.608788499999946,43.251976],[-1.534212,43.29453649800007],[-1.384407500999941,43.25365449800006],[-1.414168,43.12828849700003],[-1.473489499999971,43.08662049800006],[-1.44129049999998,43.04634099800006],[-1.35373150099997,43.02798099800003],[-1.3322,43.10771199800007],[-1.270122,43.118247996000036],[-1.309055499999943,43.07115950000008],[-0.945180499999935,42.95343799800003],[-0.751951499999961,42.96712099900003],[-0.724501,42.92015849900008],[-0.551047,42.777637499000036],[-0.313342,42.84936499600008],[-0.160069499999963,42.79801949800003],[-0.055795,42.69225699800006],[0.175701,42.73704149800005],[0.295051500000056,42.673358998000026],[0.360133500000074,42.724353998000026],[0.477755500000058,42.70001999600004],[0.660127,42.69095249800006],[0.66215950000003,42.84124349700005],[0.858215,42.82574099800007],[1.078708,42.788341496000044],[1.17077,42.70814149800003],[1.357304,42.71942199800003],[1.442566,42.603668],[1.413773,42.53564449700008],[1.468273,42.51002099800007],[1.455091,42.43606199800007],[1.725801,42.50440199800005],[1.731011,42.49240099800005],[1.932950500000061,42.45493699800005],[1.964552,42.38215699800003],[2.012462,42.35175299800005],[2.306832499000052,42.42904299600008],[2.526214500000037,42.33339299700003],[2.675338500000066,42.34233849800006],[2.673950500000046,42.40464399900003],[2.839501,42.45927099800008],[3.174803999000062,42.43537499900003],[3.163047,42.36050799800006],[3.322315500000059,42.31938149700005],[3.266109500000027,42.23871599700004],[3.161765,42.26394249900005],[3.111661500000025,42.20384949700008],[3.124232,42.12801749800008],[3.213827,42.061301997000044],[3.21998050000002,41.92232149800003],[2.778513,41.648807497000064],[2.270422,41.46080049700004],[2.119573,41.29206099800007],[1.645323500000075,41.19562149600006],[0.966513,41.02842349800005],[0.699735499000042,40.803840499000046],[0.876583500000038,40.70131649700005],[0.721663985000021,40.60513691300008],[0.746041500000047,40.636180997000054],[0.602681,40.62206249800005],[0.515238500000066,40.522918497000035],[-0.188467,39.72195449700007],[-0.334561,39.42353449800004],[-0.195017499999949,39.05113249800007],[0.012785310000027,38.86459284600005],[0.197371,38.804336498000055],[0.205479500000024,38.731944998000074],[-0.027940499999943,38.61920149700006],[-0.097914499999945,38.523395497000024],[-0.301367,38.481929498000056],[-0.409126,38.35868849800005],[-0.50490949999994,38.333378],[-0.50988,38.20200349700008],[-0.612482499999942,38.175410998000075],[-0.647937499999955,37.99972899800008],[-0.762135499999943,37.84700799700005],[-0.751674499999979,37.794719497000074],[-0.784334,37.81915649900003],[-0.859979499999952,37.721446997000044],[-0.733145,37.63270199700003],[-0.738589693999927,37.69975169400004],[-0.720821,37.60587299700006],[-0.920817499999941,37.55564099800006],[-1.074336,37.58328599600003],[-1.111013,37.53418349800006],[-1.325323499999968,37.56244249800005],[-1.630033,37.37518299800007],[-1.806549,37.214004497000076],[-1.903591,36.937160498000026],[-2.12434,36.731842],[-2.192171000999963,36.72158499900007],[-2.368091499999935,36.84145749800007],[-2.562035,36.81660099700008],[-2.700591,36.68272399700004],[-3.128691,36.75088799800005],[-3.420361,36.694095498000024],[-3.777457499999969,36.737926499000025],[-4.418207,36.717219998000076],[-4.638161499999967,36.50709149800008],[-4.936012499999947,36.499389499000074],[-5.176345499999968,36.414661498000044],[-5.252405,36.31127549800004],[-5.339225,36.152034997000044],[-5.35152249999993,36.15256899800005],[-5.424657499999967,36.17650199800005],[-5.430406,36.071304499000064],[-5.60843,36.007053496000026],[-5.797297499999956,36.07829699800004],[-5.914955499999962,36.18479549800003],[-6.040427499999964,36.19237899800004],[-6.308641499999965,36.532573497000044],[-6.229004,36.462669497000036],[-6.173302000999968,36.521442498000056],[-6.245686,36.50899499800005],[-6.232875,36.584945497000035],[-6.387676,36.62863149800006],[-6.440116,36.72086349700004],[-6.345561,36.79876699700003],[-6.514539499999955,36.97180549800004],[-6.893693499999927,37.16146499900003],[-7.051858,37.20842349800006],[-7.40191649999997,37.174827497000024],[-7.463588499999958,37.485454498000024],[-7.51269149999996,37.526256498000066],[-7.418441,37.74499499800004],[-7.279641,37.876762496000026],[-7.263284499999941,37.979907999000034],[-7.108363,38.040984998000056],[-7.002483499999926,38.02271649900007],[-6.931738499999938,38.20837799700007],[-7.107952499999953,38.18812149900003],[-7.316636,38.43987649800005],[-7.255615,38.72397599800007],[-7.203135,38.75101749700008],[-7.039417499999956,38.86484549700003],[-7.052114,38.90723399700005],[-6.951391499999943,39.02407049800007],[-7.02995,39.11653149600005],[-7.143882,39.10932149800004],[-7.136354499999925,39.17224899800004],[-7.24126,39.20853049800007],[-7.231467,39.27843099900008],[-7.310608,39.34102249800003],[-7.294676,39.455993499000044],[-7.534902449999947,39.66198689000004],[-7.015405,39.670856498000035],[-6.986185499999976,39.81074499700003],[-6.903161499999953,39.87268849800006],[-6.864203,40.011867497000026],[-7.01196049999993,40.12693399900007],[-7.012223,40.22654349700008],[-6.95129849999995,40.25744599800004],[-6.865144,40.27069449800007],[-6.781047499999943,40.36473099700004],[-6.848108,40.45544049700004],[-6.795071,40.51197799800008],[-6.844862,40.56593699700005],[-6.793930499999931,40.66174349600004],[-6.831229499999949,40.75269699800003],[-6.801935,40.861045997000076],[-6.929903499999966,41.02946649800003],[-6.808375,41.036526],[-6.689786,41.20524149700003],[-6.479713,41.29437999800007],[-6.391473,41.38374699800005],[-6.318309499999941,41.38614249600005],[-6.189352,41.57504649800006],[-6.302936,41.66290249800005],[-6.548242499999958,41.68563849800006],[-6.568986499999937,41.743514997000034],[-6.51576149999994,41.870739],[-6.571177499999976,41.88363249900004],[-6.549125499999946,41.944511496000075],[-6.588461499999937,41.967761997000025],[-6.983513499999958,41.97290399800005],[-7.17188,41.97989049600005],[-7.200464499999953,41.879749497000034],[-7.427273499999956,41.80749949800003],[-7.452781,41.865161998000076],[-7.613568,41.82929599800008],[-7.588074,41.87964999600007],[-7.697163499999931,41.907523998000045],[-7.876458499999956,41.84949499600003],[-7.905759499999931,41.92737949700006],[-8.05186249999997,41.820613997000066],[-8.165075499999944,41.81830199900003],[-8.216984,41.91365499600005],[-8.083840000999942,42.03905099800005],[-8.181348499999956,42.06683349700006],[-8.199000499999954,42.15441899800004],[-8.331857499999956,42.084220997000045],[-8.630127,42.050002997000036],[-8.863186,41.87206649800004],[-8.898017,42.11191949900007],[-8.614507499999945,42.297950498000034],[-8.62887749999993,42.35115449600005],[-8.65887349999997,42.29143499800006],[-8.870927,42.25394799600008],[-8.855232499999943,42.30938350000008],[-8.815974,42.27797699700005],[-8.656309,42.425891998000054],[-8.838805,42.384059996000076],[-8.911087000999942,42.462329999000076],[-8.833692499999927,42.47013449800005],[-8.726749499999926,42.68825149600008],[-8.902564,42.64233799800007],[-9.04231649999997,42.51715499800008],[-9.065002499999935,42.599761998000076],[-9.03095449999995,42.697562996000045],[-8.88834,42.816467497000076],[-9.09112,42.74672699600006],[-9.145302,42.80194849800006],[-9.105557499999975,42.82309349800005],[-9.125363499999935,42.905425996000076],[-9.221031,42.94280249900004],[-9.28191849999996,42.89287950000005],[-9.277219,43.04410949900006],[-9.213974,43.16154499900006],[-8.93966949999998,43.22681799900005],[-8.99068449999993,43.27455499800004],[-8.910672499999976,43.32125449800003],[-8.556806499999936,43.31056249800008],[-8.400093,43.389320498000075],[-8.360344,43.34165199800003],[-8.30548,43.40443799800005],[-8.209778,43.32762899800008],[-8.243831499999942,43.41756049700007],[-8.316718,43.443595998000035],[-8.151337499999954,43.51648699700007],[-8.347553499999947,43.46301649800006],[-8.304874,43.524406498000076],[-7.904372499999965,43.76910399600007],[-7.859451,43.76281749800006],[-7.883540499999981,43.662837996000064],[-7.661467499999958,43.776561498000035],[-7.699736499999972,43.735115],[-7.491478499999971,43.72754299700006]]],[[[1.423349,38.90504449800005],[1.371127500000057,38.83033349700003],[1.21306450000003,38.90135949900008],[1.310512500000073,39.04295349800003],[1.603850500000021,39.09101849700005],[1.605787500000019,39.02961349800006],[1.423349,38.90504449800005]]],[[[1.451187499000071,38.730296999000075],[1.527470550000032,38.67332874900006],[1.569296050000048,38.691528249000044],[1.584409,38.66284149900008],[1.523969550000061,38.65303774900008],[1.447129500000074,38.68503149900005],[1.385188,38.64133849900003],[1.386598500000048,38.72178249900003],[1.433151550000048,38.77172474900004],[1.451187499000071,38.730296999000075]]],[[[-2.950587,35.31841799800003],[-2.92736,35.27426399900003],[-2.970093499999962,35.289367499000036],[-2.950587,35.31841799800003]]],[[[-5.350729000999934,35.90900299700007],[-5.342521175999934,35.87303803900005],[-5.382034499999975,35.91260649900005],[-5.350729000999934,35.90900299700007]]],[[[-13.734242499999937,28.90740599700007],[-13.78629,28.83903299700006],[-13.854198499999939,28.86207399700004],[-13.829453499999943,29.008733499000073],[-13.75620749999996,29.07760949900006],[-13.541315,29.122923498000034],[-13.470518,29.238542497000026],[-13.419878,29.18417149700008],[-13.464197,29.129623498000058],[-13.482365499999958,28.99933249700007],[-13.734242499999937,28.90740599700007]]],[[[-14.22429649999998,28.162038997000025],[-14.331749,28.044091998000056],[-14.492716,28.083705998000028],[-14.226619,28.21153449700006],[-14.203373,28.328212497000038],[-14.033473,28.593721497000047],[-14.016622499999926,28.71499399700008],[-13.87708349999997,28.75204099800004],[-13.830917499999941,28.693815999000037],[-13.825466,28.553829],[-13.922949,28.24746299700007],[-14.22429649999998,28.162038997000025]]],[[[-15.403546499999948,28.17169199800003],[-15.419742499999927,28.061523497000053],[-15.369922499999973,27.99678049700003],[-15.424459499999955,27.807855497000048],[-15.600111,27.734939497000028],[-15.788058499999977,27.835252497000056],[-15.832990499999937,27.909723498000062],[-15.820978,28.014224997000042],[-15.707619499999964,28.08881199700005],[-15.709276000999978,28.165738997000062],[-15.46804149999997,28.12534149700008],[-15.403546499999948,28.17169199800003]]],[[[-16.492656499999953,28.078862998000034],[-16.640173,28.004987997000057],[-16.707897499999945,28.010785998000074],[-16.924217,28.350005999000075],[-16.511211499999945,28.420334498000045],[-16.38525950099995,28.54562549900004],[-16.134222,28.581258499000057],[-16.143300499999953,28.520728999000028],[-16.36059249999994,28.379459498000074],[-16.361009499999966,28.306573997000044],[-16.492656499999953,28.078862998000034]]],[[[-17.09916050099997,28.094314499000063],[-17.19942649999996,28.02373149700003],[-17.34921849999995,28.099218497000038],[-17.318047,28.204323998000064],[-17.11809349999993,28.149373998000044],[-17.09916050099997,28.094314499000063]]],[[[-17.76021949999995,28.569320998000023],[-17.84223549999996,28.452651498000023],[-18.007618499999978,28.75788349800007],[-17.948734499999944,28.840764996000075],[-17.777729,28.839669997000044],[-17.724292499999933,28.743291998000075],[-17.770277,28.676038997000035],[-17.76021949999995,28.569320998000023]]],[[[-17.883365499999968,27.804454496000062],[-17.97904000099993,27.64051999700007],[-18.15835949999996,27.712901499000054],[-18.131494,27.771581498000046],[-18.036621000999958,27.76222799900006],[-17.958796,27.841645999000036],[-17.883365499999968,27.804454496000062]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES1","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"NOROESTE","NUTS_NAME":"NOROESTE"},"geometry":{"type":"Polygon","coordinates":[[[-7.491478499999971,43.72754299700006],[-7.245640499999979,43.56518199800007],[-7.031837,43.54447149600003],[-6.107349,43.55629349600008],[-5.84171,43.65595649800008],[-5.694942,43.54321299800006],[-5.407633,43.55561049800008],[-5.214872,43.47370850000004],[-4.512300998999933,43.393204],[-4.174620499999946,43.40098950000004],[-3.785064998999928,43.49179049900005],[-3.834026,43.44116599900008],[-3.814483998999947,43.40204999800005],[-3.587068,43.51339750100004],[-3.435443,43.464725498000064],[-3.42664100099995,43.413513],[-3.153338,43.35322199800004],[-3.157362,43.30125799800004],[-3.450147,43.236207998000054],[-3.417681499999958,43.13336949700005],[-3.64878700099996,43.18002699800007],[-3.945488,43.00571849600004],[-3.991000000999975,42.93103449600005],[-3.83956,42.93972750100005],[-3.914852,42.86003099700008],[-3.836812,42.87872699600007],[-3.815811,42.81256099900003],[-3.99967,42.76893249600005],[-4.002319,42.83086749700004],[-4.04593,42.76656699600005],[-4.081359,42.76141749800007],[-4.182724499999949,42.810298998000064],[-4.144280499999979,42.86856049800008],[-4.219518000999926,42.850891001000036],[-4.238510998999971,42.95431499700004],[-4.431889499999954,43.045623496000076],[-4.737022998999976,43.02092349800006],[-4.851440998999976,43.12587749800008],[-4.841038499999968,43.180709499000045],[-4.912568,43.23680849700003],[-5.078465,43.16039599700008],[-5.101744499999938,43.10177999900003],[-5.369141,43.09450899800004],[-5.493756499999961,43.02128999800004],[-5.688321,43.056812498000056],[-5.804008499999952,42.96255499800003],[-6.081869499999925,43.075653],[-6.226611499999933,43.00883199700007],[-6.358922,43.05839549700005],[-6.433087499999942,42.939705497000034],[-6.546554,42.91210199900007],[-6.824167,42.91501249800007],[-6.871256499999959,42.87379449800005],[-6.841323,42.83230949800003],[-6.867409,42.78713249800006],[-7.04516750099998,42.69473250000004],[-7.012612499999932,42.657767996000075],[-7.076834,42.50812399800003],[-6.822765499999946,42.490833],[-6.843498499999953,42.40582649700008],[-6.73394,42.35909649600006],[-6.784308,42.25360849900005],[-7.033233500999927,42.07457749800005],[-6.974782499999947,42.05515649700004],[-6.983513499999958,41.97290399800005],[-7.17188,41.97989049600005],[-7.200464499999953,41.879749497000034],[-7.427273499999956,41.80749949800003],[-7.452781,41.865161998000076],[-7.613568,41.82929599800008],[-7.588074,41.87964999600007],[-7.697163499999931,41.907523998000045],[-7.876458499999956,41.84949499600003],[-7.905759499999931,41.92737949700006],[-8.05186249999997,41.820613997000066],[-8.165075499999944,41.81830199900003],[-8.216984,41.91365499600005],[-8.083840000999942,42.03905099800005],[-8.181348499999956,42.06683349700006],[-8.199000499999954,42.15441899800004],[-8.331857499999956,42.084220997000045],[-8.630127,42.050002997000036],[-8.863186,41.87206649800004],[-8.898017,42.11191949900007],[-8.614507499999945,42.297950498000034],[-8.62887749999993,42.35115449600005],[-8.65887349999997,42.29143499800006],[-8.870927,42.25394799600008],[-8.855232499999943,42.30938350000008],[-8.815974,42.27797699700005],[-8.656309,42.425891998000054],[-8.838805,42.384059996000076],[-8.911087000999942,42.462329999000076],[-8.833692499999927,42.47013449800005],[-8.726749499999926,42.68825149600008],[-8.902564,42.64233799800007],[-9.04231649999997,42.51715499800008],[-9.065002499999935,42.599761998000076],[-9.03095449999995,42.697562996000045],[-8.88834,42.816467497000076],[-9.09112,42.74672699600006],[-9.145302,42.80194849800006],[-9.105557499999975,42.82309349800005],[-9.125363499999935,42.905425996000076],[-9.221031,42.94280249900004],[-9.28191849999996,42.89287950000005],[-9.277219,43.04410949900006],[-9.213974,43.16154499900006],[-8.93966949999998,43.22681799900005],[-8.99068449999993,43.27455499800004],[-8.910672499999976,43.32125449800003],[-8.556806499999936,43.31056249800008],[-8.400093,43.389320498000075],[-8.360344,43.34165199800003],[-8.30548,43.40443799800005],[-8.209778,43.32762899800008],[-8.243831499999942,43.41756049700007],[-8.316718,43.443595998000035],[-8.151337499999954,43.51648699700007],[-8.347553499999947,43.46301649800006],[-8.304874,43.524406498000076],[-7.904372499999965,43.76910399600007],[-7.859451,43.76281749800006],[-7.883540499999981,43.662837996000064],[-7.661467499999958,43.776561498000035],[-7.699736499999972,43.735115],[-7.491478499999971,43.72754299700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES11","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Galicia","NUTS_NAME":"Galicia"},"geometry":{"type":"Polygon","coordinates":[[[-7.491478499999971,43.72754299700006],[-7.245640499999979,43.56518199800007],[-7.031837,43.54447149600003],[-7.04760700099996,43.47781349600007],[-7.182441,43.392383496000036],[-6.970987,43.20418149900007],[-6.957335,43.13541399800005],[-6.868033499999967,43.18938449800004],[-6.823265499999934,43.143635],[-6.997143,43.01942849900007],[-6.873130000999936,42.98846049700006],[-6.824167,42.91501249800007],[-6.871256499999959,42.87379449800005],[-6.841323,42.83230949800003],[-6.867409,42.78713249800006],[-7.04516750099998,42.69473250000004],[-7.012612499999932,42.657767996000075],[-7.076834,42.50812399800003],[-6.822765499999946,42.490833],[-6.843498499999953,42.40582649700008],[-6.73394,42.35909649600006],[-6.784308,42.25360849900005],[-7.033233500999927,42.07457749800005],[-6.974782499999947,42.05515649700004],[-6.983513499999958,41.97290399800005],[-7.17188,41.97989049600005],[-7.200464499999953,41.879749497000034],[-7.427273499999956,41.80749949800003],[-7.452781,41.865161998000076],[-7.613568,41.82929599800008],[-7.588074,41.87964999600007],[-7.697163499999931,41.907523998000045],[-7.876458499999956,41.84949499600003],[-7.905759499999931,41.92737949700006],[-8.05186249999997,41.820613997000066],[-8.165075499999944,41.81830199900003],[-8.216984,41.91365499600005],[-8.083840000999942,42.03905099800005],[-8.181348499999956,42.06683349700006],[-8.199000499999954,42.15441899800004],[-8.331857499999956,42.084220997000045],[-8.630127,42.050002997000036],[-8.863186,41.87206649800004],[-8.898017,42.11191949900007],[-8.614507499999945,42.297950498000034],[-8.62887749999993,42.35115449600005],[-8.65887349999997,42.29143499800006],[-8.870927,42.25394799600008],[-8.855232499999943,42.30938350000008],[-8.815974,42.27797699700005],[-8.656309,42.425891998000054],[-8.838805,42.384059996000076],[-8.911087000999942,42.462329999000076],[-8.833692499999927,42.47013449800005],[-8.726749499999926,42.68825149600008],[-8.902564,42.64233799800007],[-9.04231649999997,42.51715499800008],[-9.065002499999935,42.599761998000076],[-9.03095449999995,42.697562996000045],[-8.88834,42.816467497000076],[-9.09112,42.74672699600006],[-9.145302,42.80194849800006],[-9.105557499999975,42.82309349800005],[-9.125363499999935,42.905425996000076],[-9.221031,42.94280249900004],[-9.28191849999996,42.89287950000005],[-9.277219,43.04410949900006],[-9.213974,43.16154499900006],[-8.93966949999998,43.22681799900005],[-8.99068449999993,43.27455499800004],[-8.910672499999976,43.32125449800003],[-8.556806499999936,43.31056249800008],[-8.400093,43.389320498000075],[-8.360344,43.34165199800003],[-8.30548,43.40443799800005],[-8.209778,43.32762899800008],[-8.243831499999942,43.41756049700007],[-8.316718,43.443595998000035],[-8.151337499999954,43.51648699700007],[-8.347553499999947,43.46301649800006],[-8.304874,43.524406498000076],[-7.904372499999965,43.76910399600007],[-7.859451,43.76281749800006],[-7.883540499999981,43.662837996000064],[-7.661467499999958,43.776561498000035],[-7.699736499999972,43.735115],[-7.491478499999971,43.72754299700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES111","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"A Coruña","NUTS_NAME":"A Coruña"},"geometry":{"type":"Polygon","coordinates":[[[-7.699736499999972,43.735115],[-7.690936499999964,43.65396099800006],[-7.780061,43.44880299700003],[-7.881873,43.40745549800005],[-7.946494499999972,43.28580099700008],[-7.908123,42.951213997000025],[-7.999684499999944,42.848857999000074],[-8.330392,42.82728199700006],[-8.406952,42.75418099800004],[-8.726749499999926,42.68825149600008],[-8.902564,42.64233799800007],[-9.04231649999997,42.51715499800008],[-9.065002499999935,42.599761998000076],[-9.03095449999995,42.697562996000045],[-8.88834,42.816467497000076],[-9.09112,42.74672699600006],[-9.145302,42.80194849800006],[-9.105557499999975,42.82309349800005],[-9.125363499999935,42.905425996000076],[-9.221031,42.94280249900004],[-9.28191849999996,42.89287950000005],[-9.277219,43.04410949900006],[-9.213974,43.16154499900006],[-8.93966949999998,43.22681799900005],[-8.99068449999993,43.27455499800004],[-8.910672499999976,43.32125449800003],[-8.556806499999936,43.31056249800008],[-8.400093,43.389320498000075],[-8.360344,43.34165199800003],[-8.30548,43.40443799800005],[-8.209778,43.32762899800008],[-8.243831499999942,43.41756049700007],[-8.316718,43.443595998000035],[-8.151337499999954,43.51648699700007],[-8.347553499999947,43.46301649800006],[-8.304874,43.524406498000076],[-7.904372499999965,43.76910399600007],[-7.859451,43.76281749800006],[-7.883540499999981,43.662837996000064],[-7.661467499999958,43.776561498000035],[-7.699736499999972,43.735115]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES112","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Lugo","NUTS_NAME":"Lugo"},"geometry":{"type":"Polygon","coordinates":[[[-7.031837,43.54447149600003],[-7.04760700099996,43.47781349600007],[-7.182441,43.392383496000036],[-6.970987,43.20418149900007],[-6.957335,43.13541399800005],[-6.868033499999967,43.18938449800004],[-6.823265499999934,43.143635],[-6.997143,43.01942849900007],[-6.873130000999936,42.98846049700006],[-6.824167,42.91501249800007],[-6.871256499999959,42.87379449800005],[-6.841323,42.83230949800003],[-6.867409,42.78713249800006],[-7.04516750099998,42.69473250000004],[-7.012612499999932,42.657767996000075],[-7.076834,42.50812399800003],[-7.202651,42.325687498000036],[-7.304465499999935,42.428855998000074],[-7.541938000999949,42.38442599900003],[-7.697842,42.42952749800003],[-7.913142,42.508365498000046],[-7.930711,42.57857449800008],[-7.861387499999978,42.70616899800007],[-7.978367,42.77867149700006],[-7.999684499999944,42.848857999000074],[-7.908123,42.951213997000025],[-7.946494499999972,43.28580099700008],[-7.881873,43.40745549800005],[-7.780061,43.44880299700003],[-7.690936499999964,43.65396099800006],[-7.699736499999972,43.735115],[-7.491478499999971,43.72754299700006],[-7.245640499999979,43.56518199800007],[-7.031837,43.54447149600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES113","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Ourense","NUTS_NAME":"Ourense"},"geometry":{"type":"Polygon","coordinates":[[[-7.076834,42.50812399800003],[-6.822765499999946,42.490833],[-6.843498499999953,42.40582649700008],[-6.73394,42.35909649600006],[-6.784308,42.25360849900005],[-7.033233500999927,42.07457749800005],[-6.974782499999947,42.05515649700004],[-6.983513499999958,41.97290399800005],[-7.17188,41.97989049600005],[-7.200464499999953,41.879749497000034],[-7.427273499999956,41.80749949800003],[-7.452781,41.865161998000076],[-7.613568,41.82929599800008],[-7.588074,41.87964999600007],[-7.697163499999931,41.907523998000045],[-7.876458499999956,41.84949499600003],[-7.905759499999931,41.92737949700006],[-8.05186249999997,41.820613997000066],[-8.165075499999944,41.81830199900003],[-8.216984,41.91365499600005],[-8.083840000999942,42.03905099800005],[-8.181348499999956,42.06683349700006],[-8.199000499999954,42.15441899800004],[-8.161808,42.244113998000046],[-8.264513,42.25283499600005],[-8.35765349999997,42.479702],[-8.173817499999927,42.57195649800008],[-7.930711,42.57857449800008],[-7.913142,42.508365498000046],[-7.697842,42.42952749800003],[-7.541938000999949,42.38442599900003],[-7.304465499999935,42.428855998000074],[-7.202651,42.325687498000036],[-7.076834,42.50812399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES114","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Pontevedra","NUTS_NAME":"Pontevedra"},"geometry":{"type":"Polygon","coordinates":[[[-7.930711,42.57857449800008],[-8.173817499999927,42.57195649800008],[-8.35765349999997,42.479702],[-8.264513,42.25283499600005],[-8.161808,42.244113998000046],[-8.199000499999954,42.15441899800004],[-8.331857499999956,42.084220997000045],[-8.630127,42.050002997000036],[-8.863186,41.87206649800004],[-8.898017,42.11191949900007],[-8.614507499999945,42.297950498000034],[-8.62887749999993,42.35115449600005],[-8.65887349999997,42.29143499800006],[-8.870927,42.25394799600008],[-8.855232499999943,42.30938350000008],[-8.815974,42.27797699700005],[-8.656309,42.425891998000054],[-8.838805,42.384059996000076],[-8.911087000999942,42.462329999000076],[-8.833692499999927,42.47013449800005],[-8.726749499999926,42.68825149600008],[-8.406952,42.75418099800004],[-8.330392,42.82728199700006],[-7.999684499999944,42.848857999000074],[-7.978367,42.77867149700006],[-7.861387499999978,42.70616899800007],[-7.930711,42.57857449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES12","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Principado de Asturias","NUTS_NAME":"Principado de Asturias"},"geometry":{"type":"Polygon","coordinates":[[[-4.512300998999933,43.393204],[-4.537742500999968,43.269249],[-4.606570499999975,43.300475997000035],[-4.730961498999932,43.25725950000003],[-4.734174998999947,43.18980399800006],[-4.841038499999968,43.180709499000045],[-4.912568,43.23680849700003],[-5.078465,43.16039599700008],[-5.101744499999938,43.10177999900003],[-5.369141,43.09450899800004],[-5.493756499999961,43.02128999800004],[-5.688321,43.056812498000056],[-5.804008499999952,42.96255499800003],[-6.081869499999925,43.075653],[-6.226611499999933,43.00883199700007],[-6.358922,43.05839549700005],[-6.433087499999942,42.939705497000034],[-6.546554,42.91210199900007],[-6.824167,42.91501249800007],[-6.873130000999936,42.98846049700006],[-6.997143,43.01942849900007],[-6.823265499999934,43.143635],[-6.868033499999967,43.18938449800004],[-6.957335,43.13541399800005],[-6.970987,43.20418149900007],[-7.182441,43.392383496000036],[-7.04760700099996,43.47781349600007],[-7.031837,43.54447149600003],[-6.107349,43.55629349600008],[-5.84171,43.65595649800008],[-5.694942,43.54321299800006],[-5.407633,43.55561049800008],[-5.214872,43.47370850000004],[-4.512300998999933,43.393204]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES120","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Asturias","NUTS_NAME":"Asturias"},"geometry":{"type":"Polygon","coordinates":[[[-4.512300998999933,43.393204],[-4.537742500999968,43.269249],[-4.606570499999975,43.300475997000035],[-4.730961498999932,43.25725950000003],[-4.734174998999947,43.18980399800006],[-4.841038499999968,43.180709499000045],[-4.912568,43.23680849700003],[-5.078465,43.16039599700008],[-5.101744499999938,43.10177999900003],[-5.369141,43.09450899800004],[-5.493756499999961,43.02128999800004],[-5.688321,43.056812498000056],[-5.804008499999952,42.96255499800003],[-6.081869499999925,43.075653],[-6.226611499999933,43.00883199700007],[-6.358922,43.05839549700005],[-6.433087499999942,42.939705497000034],[-6.546554,42.91210199900007],[-6.824167,42.91501249800007],[-6.873130000999936,42.98846049700006],[-6.997143,43.01942849900007],[-6.823265499999934,43.143635],[-6.868033499999967,43.18938449800004],[-6.957335,43.13541399800005],[-6.970987,43.20418149900007],[-7.182441,43.392383496000036],[-7.04760700099996,43.47781349600007],[-7.031837,43.54447149600003],[-6.107349,43.55629349600008],[-5.84171,43.65595649800008],[-5.694942,43.54321299800006],[-5.407633,43.55561049800008],[-5.214872,43.47370850000004],[-4.512300998999933,43.393204]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES13","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Cantabria","NUTS_NAME":"Cantabria"},"geometry":{"type":"Polygon","coordinates":[[[-3.42664100099995,43.413513],[-3.153338,43.35322199800004],[-3.157362,43.30125799800004],[-3.450147,43.236207998000054],[-3.417681499999958,43.13336949700005],[-3.64878700099996,43.18002699800007],[-3.945488,43.00571849600004],[-3.991000000999975,42.93103449600005],[-3.83956,42.93972750100005],[-3.914852,42.86003099700008],[-3.836812,42.87872699600007],[-3.815811,42.81256099900003],[-3.99967,42.76893249600005],[-4.002319,42.83086749700004],[-4.04593,42.76656699600005],[-4.081359,42.76141749800007],[-4.182724499999949,42.810298998000064],[-4.144280499999979,42.86856049800008],[-4.219518000999926,42.850891001000036],[-4.238510998999971,42.95431499700004],[-4.431889499999954,43.045623496000076],[-4.737022998999976,43.02092349800006],[-4.851440998999976,43.12587749800008],[-4.841038499999968,43.180709499000045],[-4.734174998999947,43.18980399800006],[-4.730961498999932,43.25725950000003],[-4.606570499999975,43.300475997000035],[-4.537742500999968,43.269249],[-4.512300998999933,43.393204],[-4.174620499999946,43.40098950000004],[-3.785064998999928,43.49179049900005],[-3.834026,43.44116599900008],[-3.814483998999947,43.40204999800005],[-3.587068,43.51339750100004],[-3.435443,43.464725498000064],[-3.42664100099995,43.413513]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES130","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Cantabria","NUTS_NAME":"Cantabria"},"geometry":{"type":"Polygon","coordinates":[[[-3.42664100099995,43.413513],[-3.153338,43.35322199800004],[-3.157362,43.30125799800004],[-3.450147,43.236207998000054],[-3.417681499999958,43.13336949700005],[-3.64878700099996,43.18002699800007],[-3.945488,43.00571849600004],[-3.991000000999975,42.93103449600005],[-3.83956,42.93972750100005],[-3.914852,42.86003099700008],[-3.836812,42.87872699600007],[-3.815811,42.81256099900003],[-3.99967,42.76893249600005],[-4.002319,42.83086749700004],[-4.04593,42.76656699600005],[-4.081359,42.76141749800007],[-4.182724499999949,42.810298998000064],[-4.144280499999979,42.86856049800008],[-4.219518000999926,42.850891001000036],[-4.238510998999971,42.95431499700004],[-4.431889499999954,43.045623496000076],[-4.737022998999976,43.02092349800006],[-4.851440998999976,43.12587749800008],[-4.841038499999968,43.180709499000045],[-4.734174998999947,43.18980399800006],[-4.730961498999932,43.25725950000003],[-4.606570499999975,43.300475997000035],[-4.537742500999968,43.269249],[-4.512300998999933,43.393204],[-4.174620499999946,43.40098950000004],[-3.785064998999928,43.49179049900005],[-3.834026,43.44116599900008],[-3.814483998999947,43.40204999800005],[-3.587068,43.51339750100004],[-3.435443,43.464725498000064],[-3.42664100099995,43.413513]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES2","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"NORESTE","NUTS_NAME":"NORESTE"},"geometry":{"type":"Polygon","coordinates":[[[-2.508807499999932,43.37654499800004],[-2.412847,43.321082997000076],[-2.178597499999967,43.287845497000035],[-1.790047499999957,43.395621997000035],[-1.785978,43.35047899600005],[-1.728903,43.29608899600004],[-1.625259,43.30670549800004],[-1.608788499999946,43.251976],[-1.534212,43.29453649800007],[-1.384407500999941,43.25365449800006],[-1.414168,43.12828849700003],[-1.473489499999971,43.08662049800006],[-1.44129049999998,43.04634099800006],[-1.35373150099997,43.02798099800003],[-1.3322,43.10771199800007],[-1.270122,43.118247996000036],[-1.309055499999943,43.07115950000008],[-0.945180499999935,42.95343799800003],[-0.751951499999961,42.96712099900003],[-0.724501,42.92015849900008],[-0.551047,42.777637499000036],[-0.313342,42.84936499600008],[-0.160069499999963,42.79801949800003],[-0.055795,42.69225699800006],[0.175701,42.73704149800005],[0.295051500000056,42.673358998000026],[0.360133500000074,42.724353998000026],[0.477755500000058,42.70001999600004],[0.660127,42.69095249800006],[0.767896,42.60979449600006],[0.690539500000057,42.48760199900005],[0.767070500000045,42.34881599700003],[0.685666500000025,42.09275049800004],[0.561936500000058,41.93820199800007],[0.607546,41.91929999800004],[0.580113500000039,41.86344199800004],[0.327072,41.678157999000064],[0.349668500000064,41.59945699700006],[0.428549,41.602367498000035],[0.446238,41.542907497000044],[0.341453499000067,41.480896],[0.335948500000029,41.40743249800005],[0.385724,41.278839497000035],[0.303379500000062,41.16437549800003],[0.200918,41.125358497000036],[0.220409500000073,41.07142999900003],[0.293564500000059,40.970562],[0.238971,40.88470449700003],[0.278288500000031,40.821033499000066],[0.170789500000069,40.73283699900003],[0.042237,40.69100549800004],[-0.197124499999973,40.78445799800005],[-0.235937499999977,40.69111249800005],[-0.381465499999933,40.66158699700003],[-0.370432499999936,40.613597997000056],[-0.293232499999931,40.611045999000055],[-0.271374500999968,40.47850049800007],[-0.347301,40.44394349700008],[-0.279997499999979,40.36949949800004],[-0.388490499999932,40.261615998000025],[-0.544136,40.251666998000076],[-0.624734,40.07609949700003],[-0.837749500999962,39.97681799600008],[-0.797643,39.88107649800003],[-0.912739,39.87310750000006],[-0.920412,39.96441649600007],[-1.142361499999936,39.971855498000025],[-1.165154,40.01010899700003],[-1.072093499999937,40.060496998000076],[-1.252389000999926,40.14331449800005],[-1.300741,40.20640199700006],[-1.357073,40.12890249900005],[-1.448831499999926,40.14535849600003],[-1.439184499999953,40.197028997000075],[-1.540461499999935,40.190822498000045],[-1.690325,40.309676997000054],[-1.719256,40.27712249600006],[-1.697659,40.321307998000066],[-1.806344,40.39824699800005],[-1.692495,40.48683899800005],[-1.67808249999996,40.59096899800005],[-1.545494499999961,40.59521099800003],[-1.539224499999932,40.792651996000075],[-1.620291499999951,40.879493498000045],[-1.617433499999947,40.94374099600003],[-1.954023,41.16977299900003],[-2.05169,41.14685799800003],[-2.14624,41.184402497000065],[-2.170485499999927,41.31883649800005],[-2.101518499999941,41.44578949800007],[-2.029267499999946,41.38416299600004],[-1.937873499999966,41.40739449800003],[-1.993325499999969,41.60013199800005],[-1.806217499999946,41.66425650000008],[-1.775364499999966,41.72478099800003],[-1.825316,41.77836599800003],[-1.85653949999994,41.966388497000025],[-1.982089,41.919250497000064],[-2.10996149999994,41.95785549800007],[-2.161917,42.066718998000056],[-2.127329499999973,42.097152497000025],[-2.269596,42.088539],[-2.282167499999957,42.13168699800008],[-2.441275,42.13647049600007],[-2.57992,41.99624999900004],[-2.746179,42.008662998000034],[-2.706087,42.10249349700007],[-2.738445,42.12480099600003],[-2.913637499999936,42.022838499000045],[-2.93428549999993,42.088363498000035],[-3.03439,42.08594499700007],[-3.129416,42.19943599900006],[-3.088611499999956,42.24570449600003],[-3.109279,42.351859999000055],[-3.06372749999997,42.356059998000035],[-3.046025499999928,42.44512549800004],[-3.134294499999953,42.54202249700006],[-3.059854,42.58915699800008],[-3.084177499999953,42.64065199800007],[-2.858117499999935,42.63816849800003],[-3.136669,42.76544199600005],[-3.159259499999962,42.80454649700005],[-3.118991499999936,42.891223998000044],[-3.217000499999926,42.82646950000003],[-3.286797499999977,42.88509749600007],[-3.223018,42.94879149800005],[-3.089547,42.89894849800004],[-2.981502,42.938754997000046],[-3.036989499999947,42.982185497000046],[-3.089023499999939,43.00158299800006],[-3.177834,43.021934497000075],[-3.142332,43.02869049900005],[-3.133973499999968,43.09717949900005],[-3.181819,43.12082299600007],[-3.141396,43.16148349800005],[-3.278148499999929,43.19437049800007],[-3.417681499999958,43.13336949700005],[-3.450147,43.236207998000054],[-3.157362,43.30125799800004],[-3.153338,43.35322199800004],[-3.017594499999973,43.32303249800003],[-3.037911,43.37359999600005],[-2.94542649999994,43.435573496000075],[-2.746815499999968,43.452121498000054],[-2.508807499999932,43.37654499800004]],[[-2.86849,42.73987599600008],[-2.517415,42.646297497000035],[-2.609000499999979,42.695922996000036],[-2.558325499999967,42.76269549800003],[-2.833424,42.79524249900004],[-2.86849,42.73987599600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES21","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"País Vasco","NUTS_NAME":"País Vasco"},"geometry":{"type":"Polygon","coordinates":[[[-2.508807499999932,43.37654499800004],[-2.412847,43.321082997000076],[-2.178597499999967,43.287845497000035],[-1.790047499999957,43.395621997000035],[-1.785978,43.35047899600005],[-1.728903,43.29608899600004],[-1.908253,43.216773998000065],[-1.902236,43.135459998000044],[-2.022847,43.065150998000036],[-2.038360499999953,42.98095699700008],[-2.250807,42.89569099800008],[-2.232710499999939,42.84378049900005],[-2.312871500999961,42.70865249900004],[-2.288498499999946,42.655036997000025],[-2.500101499999971,42.616443497000034],[-2.453259499999945,42.56707749800006],[-2.396772,42.59906749800007],[-2.420719,42.489269498000056],[-2.594668,42.47234349800004],[-2.691447499999924,42.54273599800007],[-2.680563499999948,42.59448649700005],[-2.763742499999978,42.622802499000045],[-2.825015499999949,42.55491249800008],[-2.858117499999935,42.63816849800003],[-3.136669,42.76544199600005],[-3.159259499999962,42.80454649700005],[-3.118991499999936,42.891223998000044],[-3.217000499999926,42.82646950000003],[-3.286797499999977,42.88509749600007],[-3.223018,42.94879149800005],[-3.089547,42.89894849800004],[-2.981502,42.938754997000046],[-3.036989499999947,42.982185497000046],[-3.089023499999939,43.00158299800006],[-3.177834,43.021934497000075],[-3.142332,43.02869049900005],[-3.133973499999968,43.09717949900005],[-3.181819,43.12082299600007],[-3.141396,43.16148349800005],[-3.278148499999929,43.19437049800007],[-3.417681499999958,43.13336949700005],[-3.450147,43.236207998000054],[-3.157362,43.30125799800004],[-3.153338,43.35322199800004],[-3.017594499999973,43.32303249800003],[-3.037911,43.37359999600005],[-2.94542649999994,43.435573496000075],[-2.746815499999968,43.452121498000054],[-2.508807499999932,43.37654499800004]],[[-2.86849,42.73987599600008],[-2.517415,42.646297497000035],[-2.609000499999979,42.695922996000036],[-2.558325499999967,42.76269549800003],[-2.833424,42.79524249900004],[-2.86849,42.73987599600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES211","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Araba/Álava","NUTS_NAME":"Araba/Álava"},"geometry":{"type":"Polygon","coordinates":[[[-2.94642749999997,43.170814998000026],[-2.968796,43.095291],[-2.890317499999981,43.045794999000066],[-2.633655,43.01985950000005],[-2.660806,43.07246799700005],[-2.54667749999993,43.088130498000055],[-2.524742,43.045266498000046],[-2.59656,43.00920099800004],[-2.581052000999932,42.964214500000026],[-2.250807,42.89569099800008],[-2.232710499999939,42.84378049900005],[-2.312871500999961,42.70865249900004],[-2.288498499999946,42.655036997000025],[-2.500101499999971,42.616443497000034],[-2.453259499999945,42.56707749800006],[-2.396772,42.59906749800007],[-2.420719,42.489269498000056],[-2.594668,42.47234349800004],[-2.691447499999924,42.54273599800007],[-2.680563499999948,42.59448649700005],[-2.763742499999978,42.622802499000045],[-2.825015499999949,42.55491249800008],[-2.858117499999935,42.63816849800003],[-3.136669,42.76544199600005],[-3.159259499999962,42.80454649700005],[-3.118991499999936,42.891223998000044],[-3.217000499999926,42.82646950000003],[-3.286797499999977,42.88509749600007],[-3.223018,42.94879149800005],[-3.089547,42.89894849800004],[-2.981502,42.938754997000046],[-3.036989499999947,42.982185497000046],[-3.089023499999939,43.00158299800006],[-3.177834,43.021934497000075],[-3.142332,43.02869049900005],[-3.133973499999968,43.09717949900005],[-3.181819,43.12082299600007],[-3.141396,43.16148349800005],[-3.026935,43.16131599700003],[-3.025271499999974,43.21477099600003],[-2.94642749999997,43.170814998000026]],[[-2.86849,42.73987599600008],[-2.517415,42.646297497000035],[-2.609000499999979,42.695922996000036],[-2.558325499999967,42.76269549800003],[-2.833424,42.79524249900004],[-2.86849,42.73987599600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES212","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Gipuzkoa","NUTS_NAME":"Gipuzkoa"},"geometry":{"type":"Polygon","coordinates":[[[-1.785978,43.35047899600005],[-1.728903,43.29608899600004],[-1.908253,43.216773998000065],[-1.902236,43.135459998000044],[-2.022847,43.065150998000036],[-2.038360499999953,42.98095699700008],[-2.250807,42.89569099800008],[-2.581052000999932,42.964214500000026],[-2.59656,43.00920099800004],[-2.524742,43.045266498000046],[-2.54667749999993,43.088130498000055],[-2.502389499999936,43.09532199800003],[-2.509749,43.161392499000044],[-2.412847,43.321082997000076],[-2.178597499999967,43.287845497000035],[-1.790047499999957,43.395621997000035],[-1.785978,43.35047899600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES213","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Bizkaia","NUTS_NAME":"Bizkaia"},"geometry":{"type":"Polygon","coordinates":[[[-2.508807499999932,43.37654499800004],[-2.412847,43.321082997000076],[-2.509749,43.161392499000044],[-2.502389499999936,43.09532199800003],[-2.54667749999993,43.088130498000055],[-2.660806,43.07246799700005],[-2.633655,43.01985950000005],[-2.890317499999981,43.045794999000066],[-2.968796,43.095291],[-2.94642749999997,43.170814998000026],[-3.025271499999974,43.21477099600003],[-3.026935,43.16131599700003],[-3.141396,43.16148349800005],[-3.278148499999929,43.19437049800007],[-3.417681499999958,43.13336949700005],[-3.450147,43.236207998000054],[-3.157362,43.30125799800004],[-3.153338,43.35322199800004],[-3.017594499999973,43.32303249800003],[-3.037911,43.37359999600005],[-2.94542649999994,43.435573496000075],[-2.746815499999968,43.452121498000054],[-2.508807499999932,43.37654499800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES22","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Comunidad Foral de Navarra","NUTS_NAME":"Comunidad Foral de Navarra"},"geometry":{"type":"Polygon","coordinates":[[[-1.608788499999946,43.251976],[-1.534212,43.29453649800007],[-1.384407500999941,43.25365449800006],[-1.414168,43.12828849700003],[-1.473489499999971,43.08662049800006],[-1.44129049999998,43.04634099800006],[-1.35373150099997,43.02798099800003],[-1.3322,43.10771199800007],[-1.270122,43.118247996000036],[-1.309055499999943,43.07115950000008],[-0.945180499999935,42.95343799800003],[-0.751951499999961,42.96712099900003],[-0.724501,42.92015849900008],[-0.819091499999956,42.90063499900003],[-0.901080499999978,42.74269099800006],[-1.027979,42.700031496000065],[-1.038592499999936,42.64829649700005],[-1.152407499999924,42.64753349700004],[-1.202788,42.54963299800005],[-1.288137499999948,42.52276249800008],[-1.272777499999961,42.48187249800003],[-1.341505,42.42391599800004],[-1.329078,42.355361998000035],[-1.417525,42.212349],[-1.398874499999977,42.126201500000036],[-1.304814499999964,42.04293049800003],[-1.410749499999952,41.91894149800004],[-1.847183499999971,42.00798799900008],[-1.901578,42.02966699800004],[-1.897332,42.07953649600006],[-1.821622,42.153552998000066],[-1.679659498999968,42.15167249800004],[-1.681452,42.192401999000026],[-1.996641,42.36388399800006],[-2.089972,42.346950496000034],[-2.111264,42.41952899900008],[-2.420719,42.489269498000056],[-2.396772,42.59906749800007],[-2.453259499999945,42.56707749800006],[-2.500101499999971,42.616443497000034],[-2.288498499999946,42.655036997000025],[-2.312871500999961,42.70865249900004],[-2.232710499999939,42.84378049900005],[-2.250807,42.89569099800008],[-2.038360499999953,42.98095699700008],[-2.022847,43.065150998000036],[-1.902236,43.135459998000044],[-1.908253,43.216773998000065],[-1.728903,43.29608899600004],[-1.625259,43.30670549800004],[-1.608788499999946,43.251976]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES220","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Navarra","NUTS_NAME":"Navarra"},"geometry":{"type":"Polygon","coordinates":[[[-1.608788499999946,43.251976],[-1.534212,43.29453649800007],[-1.384407500999941,43.25365449800006],[-1.414168,43.12828849700003],[-1.473489499999971,43.08662049800006],[-1.44129049999998,43.04634099800006],[-1.35373150099997,43.02798099800003],[-1.3322,43.10771199800007],[-1.270122,43.118247996000036],[-1.309055499999943,43.07115950000008],[-0.945180499999935,42.95343799800003],[-0.751951499999961,42.96712099900003],[-0.724501,42.92015849900008],[-0.819091499999956,42.90063499900003],[-0.901080499999978,42.74269099800006],[-1.027979,42.700031496000065],[-1.038592499999936,42.64829649700005],[-1.152407499999924,42.64753349700004],[-1.202788,42.54963299800005],[-1.288137499999948,42.52276249800008],[-1.272777499999961,42.48187249800003],[-1.341505,42.42391599800004],[-1.329078,42.355361998000035],[-1.417525,42.212349],[-1.398874499999977,42.126201500000036],[-1.304814499999964,42.04293049800003],[-1.410749499999952,41.91894149800004],[-1.847183499999971,42.00798799900008],[-1.901578,42.02966699800004],[-1.897332,42.07953649600006],[-1.821622,42.153552998000066],[-1.679659498999968,42.15167249800004],[-1.681452,42.192401999000026],[-1.996641,42.36388399800006],[-2.089972,42.346950496000034],[-2.111264,42.41952899900008],[-2.420719,42.489269498000056],[-2.396772,42.59906749800007],[-2.453259499999945,42.56707749800006],[-2.500101499999971,42.616443497000034],[-2.288498499999946,42.655036997000025],[-2.312871500999961,42.70865249900004],[-2.232710499999939,42.84378049900005],[-2.250807,42.89569099800008],[-2.038360499999953,42.98095699700008],[-2.022847,43.065150998000036],[-1.902236,43.135459998000044],[-1.908253,43.216773998000065],[-1.728903,43.29608899600004],[-1.625259,43.30670549800004],[-1.608788499999946,43.251976]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES23","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"La Rioja","NUTS_NAME":"La Rioja"},"geometry":{"type":"Polygon","coordinates":[[[-2.858117499999935,42.63816849800003],[-2.825015499999949,42.55491249800008],[-2.763742499999978,42.622802499000045],[-2.680563499999948,42.59448649700005],[-2.691447499999924,42.54273599800007],[-2.594668,42.47234349800004],[-2.420719,42.489269498000056],[-2.111264,42.41952899900008],[-2.089972,42.346950496000034],[-1.996641,42.36388399800006],[-1.681452,42.192401999000026],[-1.679659498999968,42.15167249800004],[-1.821622,42.153552998000066],[-1.897332,42.07953649600006],[-1.901578,42.02966699800004],[-1.847183499999971,42.00798799900008],[-1.85653949999994,41.966388497000025],[-1.982089,41.919250497000064],[-2.10996149999994,41.95785549800007],[-2.161917,42.066718998000056],[-2.127329499999973,42.097152497000025],[-2.269596,42.088539],[-2.282167499999957,42.13168699800008],[-2.441275,42.13647049600007],[-2.57992,41.99624999900004],[-2.746179,42.008662998000034],[-2.706087,42.10249349700007],[-2.738445,42.12480099600003],[-2.913637499999936,42.022838499000045],[-2.93428549999993,42.088363498000035],[-3.03439,42.08594499700007],[-3.129416,42.19943599900006],[-3.088611499999956,42.24570449600003],[-3.109279,42.351859999000055],[-3.06372749999997,42.356059998000035],[-3.046025499999928,42.44512549800004],[-3.134294499999953,42.54202249700006],[-3.059854,42.58915699800008],[-3.084177499999953,42.64065199800007],[-2.858117499999935,42.63816849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES230","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"La Rioja","NUTS_NAME":"La Rioja"},"geometry":{"type":"Polygon","coordinates":[[[-2.858117499999935,42.63816849800003],[-2.825015499999949,42.55491249800008],[-2.763742499999978,42.622802499000045],[-2.680563499999948,42.59448649700005],[-2.691447499999924,42.54273599800007],[-2.594668,42.47234349800004],[-2.420719,42.489269498000056],[-2.111264,42.41952899900008],[-2.089972,42.346950496000034],[-1.996641,42.36388399800006],[-1.681452,42.192401999000026],[-1.679659498999968,42.15167249800004],[-1.821622,42.153552998000066],[-1.897332,42.07953649600006],[-1.901578,42.02966699800004],[-1.847183499999971,42.00798799900008],[-1.85653949999994,41.966388497000025],[-1.982089,41.919250497000064],[-2.10996149999994,41.95785549800007],[-2.161917,42.066718998000056],[-2.127329499999973,42.097152497000025],[-2.269596,42.088539],[-2.282167499999957,42.13168699800008],[-2.441275,42.13647049600007],[-2.57992,41.99624999900004],[-2.746179,42.008662998000034],[-2.706087,42.10249349700007],[-2.738445,42.12480099600003],[-2.913637499999936,42.022838499000045],[-2.93428549999993,42.088363498000035],[-3.03439,42.08594499700007],[-3.129416,42.19943599900006],[-3.088611499999956,42.24570449600003],[-3.109279,42.351859999000055],[-3.06372749999997,42.356059998000035],[-3.046025499999928,42.44512549800004],[-3.134294499999953,42.54202249700006],[-3.059854,42.58915699800008],[-3.084177499999953,42.64065199800007],[-2.858117499999935,42.63816849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES24","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Aragón","NUTS_NAME":"Aragón"},"geometry":{"type":"Polygon","coordinates":[[[-0.724501,42.92015849900008],[-0.551047,42.777637499000036],[-0.313342,42.84936499600008],[-0.160069499999963,42.79801949800003],[-0.055795,42.69225699800006],[0.175701,42.73704149800005],[0.295051500000056,42.673358998000026],[0.360133500000074,42.724353998000026],[0.477755500000058,42.70001999600004],[0.660127,42.69095249800006],[0.767896,42.60979449600006],[0.690539500000057,42.48760199900005],[0.767070500000045,42.34881599700003],[0.685666500000025,42.09275049800004],[0.561936500000058,41.93820199800007],[0.607546,41.91929999800004],[0.580113500000039,41.86344199800004],[0.327072,41.678157999000064],[0.349668500000064,41.59945699700006],[0.428549,41.602367498000035],[0.446238,41.542907497000044],[0.341453499000067,41.480896],[0.335948500000029,41.40743249800005],[0.385724,41.278839497000035],[0.303379500000062,41.16437549800003],[0.200918,41.125358497000036],[0.220409500000073,41.07142999900003],[0.293564500000059,40.970562],[0.238971,40.88470449700003],[0.278288500000031,40.821033499000066],[0.170789500000069,40.73283699900003],[0.042237,40.69100549800004],[-0.197124499999973,40.78445799800005],[-0.235937499999977,40.69111249800005],[-0.381465499999933,40.66158699700003],[-0.370432499999936,40.613597997000056],[-0.293232499999931,40.611045999000055],[-0.271374500999968,40.47850049800007],[-0.347301,40.44394349700008],[-0.279997499999979,40.36949949800004],[-0.388490499999932,40.261615998000025],[-0.544136,40.251666998000076],[-0.624734,40.07609949700003],[-0.837749500999962,39.97681799600008],[-0.797643,39.88107649800003],[-0.912739,39.87310750000006],[-0.920412,39.96441649600007],[-1.142361499999936,39.971855498000025],[-1.165154,40.01010899700003],[-1.072093499999937,40.060496998000076],[-1.252389000999926,40.14331449800005],[-1.300741,40.20640199700006],[-1.357073,40.12890249900005],[-1.448831499999926,40.14535849600003],[-1.439184499999953,40.197028997000075],[-1.540461499999935,40.190822498000045],[-1.690325,40.309676997000054],[-1.719256,40.27712249600006],[-1.697659,40.321307998000066],[-1.806344,40.39824699800005],[-1.692495,40.48683899800005],[-1.67808249999996,40.59096899800005],[-1.545494499999961,40.59521099800003],[-1.539224499999932,40.792651996000075],[-1.620291499999951,40.879493498000045],[-1.617433499999947,40.94374099600003],[-1.954023,41.16977299900003],[-2.05169,41.14685799800003],[-2.14624,41.184402497000065],[-2.170485499999927,41.31883649800005],[-2.101518499999941,41.44578949800007],[-2.029267499999946,41.38416299600004],[-1.937873499999966,41.40739449800003],[-1.993325499999969,41.60013199800005],[-1.806217499999946,41.66425650000008],[-1.775364499999966,41.72478099800003],[-1.825316,41.77836599800003],[-1.85653949999994,41.966388497000025],[-1.847183499999971,42.00798799900008],[-1.410749499999952,41.91894149800004],[-1.304814499999964,42.04293049800003],[-1.398874499999977,42.126201500000036],[-1.417525,42.212349],[-1.329078,42.355361998000035],[-1.341505,42.42391599800004],[-1.272777499999961,42.48187249800003],[-1.288137499999948,42.52276249800008],[-1.202788,42.54963299800005],[-1.152407499999924,42.64753349700004],[-1.038592499999936,42.64829649700005],[-1.027979,42.700031496000065],[-0.901080499999978,42.74269099800006],[-0.819091499999956,42.90063499900003],[-0.724501,42.92015849900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES241","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Huesca","NUTS_NAME":"Huesca"},"geometry":{"type":"Polygon","coordinates":[[[-0.313342,42.84936499600008],[-0.160069499999963,42.79801949800003],[-0.055795,42.69225699800006],[0.175701,42.73704149800005],[0.295051500000056,42.673358998000026],[0.360133500000074,42.724353998000026],[0.477755500000058,42.70001999600004],[0.660127,42.69095249800006],[0.767896,42.60979449600006],[0.690539500000057,42.48760199900005],[0.767070500000045,42.34881599700003],[0.685666500000025,42.09275049800004],[0.561936500000058,41.93820199800007],[0.607546,41.91929999800004],[0.580113500000039,41.86344199800004],[0.327072,41.678157999000064],[0.349668500000064,41.59945699700006],[0.428549,41.602367498000035],[0.446238,41.542907497000044],[0.341453499000067,41.480896],[0.335948500000029,41.40743249800005],[-0.049394,41.360454497000035],[-0.156404,41.589965996000046],[-0.245929499999932,41.57373049800003],[-0.362273499999958,41.63781349800007],[-0.657802499999946,41.95748149800005],[-0.884915500999966,42.00356299800006],[-0.804766499999971,42.09642799800008],[-0.763122000999942,42.06746299800005],[-0.728917499999966,42.29771449900005],[-0.736129499999947,42.36928949800006],[-0.780679499999962,42.38605499800008],[-0.805021499999953,42.27202999800005],[-0.917997499999956,42.25547799800006],[-0.815332499999954,42.39493949700005],[-0.905622,42.436432498000045],[-0.877708,42.46162049800006],[-0.924782,42.58794049800008],[-0.901080499999978,42.74269099800006],[-0.819091499999956,42.90063499900003],[-0.724501,42.92015849900008],[-0.551047,42.777637499000036],[-0.313342,42.84936499600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES242","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Teruel","NUTS_NAME":"Teruel"},"geometry":{"type":"Polygon","coordinates":[[[0.220409500000073,41.07142999900003],[0.293564500000059,40.970562],[0.238971,40.88470449700003],[0.278288500000031,40.821033499000066],[0.170789500000069,40.73283699900003],[0.042237,40.69100549800004],[-0.197124499999973,40.78445799800005],[-0.235937499999977,40.69111249800005],[-0.381465499999933,40.66158699700003],[-0.370432499999936,40.613597997000056],[-0.293232499999931,40.611045999000055],[-0.271374500999968,40.47850049800007],[-0.347301,40.44394349700008],[-0.279997499999979,40.36949949800004],[-0.388490499999932,40.261615998000025],[-0.544136,40.251666998000076],[-0.624734,40.07609949700003],[-0.837749500999962,39.97681799600008],[-0.797643,39.88107649800003],[-0.912739,39.87310750000006],[-0.920412,39.96441649600007],[-1.142361499999936,39.971855498000025],[-1.165154,40.01010899700003],[-1.072093499999937,40.060496998000076],[-1.252389000999926,40.14331449800005],[-1.300741,40.20640199700006],[-1.357073,40.12890249900005],[-1.448831499999926,40.14535849600003],[-1.439184499999953,40.197028997000075],[-1.540461499999935,40.190822498000045],[-1.690325,40.309676997000054],[-1.719256,40.27712249600006],[-1.697659,40.321307998000066],[-1.806344,40.39824699800005],[-1.692495,40.48683899800005],[-1.67808249999996,40.59096899800005],[-1.545494499999961,40.59521099800003],[-1.539224499999932,40.792651996000075],[-1.620291499999951,40.879493498000045],[-1.617433499999947,40.94374099600003],[-1.472974,40.94880649800007],[-1.405203499999971,41.06631450000003],[-1.287409499999967,41.04886649700006],[-1.254678,41.122577498000055],[-1.108607,41.164683998000044],[-0.957231,41.07635099600003],[-0.788477499999942,41.107909998000025],[-0.761383499999965,41.169658500000025],[-0.693936,41.112292998000044],[-0.609864499999958,41.19167699600007],[-0.629684499999939,41.31175599800008],[-0.531537,41.227805997000075],[-0.508116499999971,41.255813500000045],[-0.581749499999944,41.34811799800008],[-0.479892,41.345931998000026],[-0.012176,41.12024699800003],[0.036972,41.04656999600007],[0.220409500000073,41.07142999900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES243","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Zaragoza","NUTS_NAME":"Zaragoza"},"geometry":{"type":"Polygon","coordinates":[[[-0.815332499999954,42.39493949700005],[-0.917997499999956,42.25547799800006],[-0.805021499999953,42.27202999800005],[-0.780679499999962,42.38605499800008],[-0.736129499999947,42.36928949800006],[-0.728917499999966,42.29771449900005],[-0.763122000999942,42.06746299800005],[-0.804766499999971,42.09642799800008],[-0.884915500999966,42.00356299800006],[-0.657802499999946,41.95748149800005],[-0.362273499999958,41.63781349800007],[-0.245929499999932,41.57373049800003],[-0.156404,41.589965996000046],[-0.049394,41.360454497000035],[0.335948500000029,41.40743249800005],[0.385724,41.278839497000035],[0.303379500000062,41.16437549800003],[0.200918,41.125358497000036],[0.220409500000073,41.07142999900003],[0.036972,41.04656999600007],[-0.012176,41.12024699800003],[-0.479892,41.345931998000026],[-0.581749499999944,41.34811799800008],[-0.508116499999971,41.255813500000045],[-0.531537,41.227805997000075],[-0.629684499999939,41.31175599800008],[-0.609864499999958,41.19167699600007],[-0.693936,41.112292998000044],[-0.761383499999965,41.169658500000025],[-0.788477499999942,41.107909998000025],[-0.957231,41.07635099600003],[-1.108607,41.164683998000044],[-1.254678,41.122577498000055],[-1.287409499999967,41.04886649700006],[-1.405203499999971,41.06631450000003],[-1.472974,40.94880649800007],[-1.617433499999947,40.94374099600003],[-1.954023,41.16977299900003],[-2.05169,41.14685799800003],[-2.14624,41.184402497000065],[-2.170485499999927,41.31883649800005],[-2.101518499999941,41.44578949800007],[-2.029267499999946,41.38416299600004],[-1.937873499999966,41.40739449800003],[-1.993325499999969,41.60013199800005],[-1.806217499999946,41.66425650000008],[-1.775364499999966,41.72478099800003],[-1.825316,41.77836599800003],[-1.85653949999994,41.966388497000025],[-1.847183499999971,42.00798799900008],[-1.410749499999952,41.91894149800004],[-1.304814499999964,42.04293049800003],[-1.398874499999977,42.126201500000036],[-1.417525,42.212349],[-1.329078,42.355361998000035],[-1.341505,42.42391599800004],[-1.272777499999961,42.48187249800003],[-1.288137499999948,42.52276249800008],[-1.202788,42.54963299800005],[-1.152407499999924,42.64753349700004],[-1.038592499999936,42.64829649700005],[-1.027979,42.700031496000065],[-0.901080499999978,42.74269099800006],[-0.924782,42.58794049800008],[-0.877708,42.46162049800006],[-0.905622,42.436432498000045],[-0.815332499999954,42.39493949700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES3","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"COMUNIDAD DE MADRID","NUTS_NAME":"COMUNIDAD DE MADRID"},"geometry":{"type":"Polygon","coordinates":[[[-3.130407,40.40515399800006],[-3.194294499999955,40.24800599800005],[-3.10929,40.285162996000054],[-3.067689,40.15788499800004],[-3.096598000999961,40.142359498000076],[-3.060548499999925,40.090534999000056],[-3.161419499999965,40.06489699700006],[-3.504916500999968,40.047123499000065],[-3.814671499999974,39.885770999000044],[-3.875525,39.92769999800004],[-3.623443,40.053848499000026],[-3.607468,40.10893199800006],[-4.080290499999933,40.26690149900003],[-4.143561500999965,40.24479199900003],[-4.190704,40.297315997000055],[-4.295019499999967,40.21884399700008],[-4.37994,40.31830099700005],[-4.437711499999978,40.23789549800006],[-4.578901,40.21715749800006],[-4.548248499999943,40.34058949800004],[-4.455591,40.31931049600007],[-4.421466499999951,40.40624199800004],[-4.320921,40.41026799700006],[-4.323894499999938,40.54771199600003],[-4.161019499999952,40.62342149700004],[-4.160293499999966,40.68985349900004],[-4.069931,40.793870500000025],[-3.9798,40.799341998000045],[-3.890160499999979,40.968803998000055],[-3.539573,41.16499349800006],[-3.394300499999929,41.000234497000065],[-3.491514499999937,40.798693497000045],[-3.43421,40.73189099800004],[-3.468398,40.68989649800005],[-3.322232499999927,40.64377550100005],[-3.33019,40.597330998000075],[-3.130407,40.40515399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES30","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Comunidad de Madrid","NUTS_NAME":"Comunidad de Madrid"},"geometry":{"type":"Polygon","coordinates":[[[-3.130407,40.40515399800006],[-3.194294499999955,40.24800599800005],[-3.10929,40.285162996000054],[-3.067689,40.15788499800004],[-3.096598000999961,40.142359498000076],[-3.060548499999925,40.090534999000056],[-3.161419499999965,40.06489699700006],[-3.504916500999968,40.047123499000065],[-3.814671499999974,39.885770999000044],[-3.875525,39.92769999800004],[-3.623443,40.053848499000026],[-3.607468,40.10893199800006],[-4.080290499999933,40.26690149900003],[-4.143561500999965,40.24479199900003],[-4.190704,40.297315997000055],[-4.295019499999967,40.21884399700008],[-4.37994,40.31830099700005],[-4.437711499999978,40.23789549800006],[-4.578901,40.21715749800006],[-4.548248499999943,40.34058949800004],[-4.455591,40.31931049600007],[-4.421466499999951,40.40624199800004],[-4.320921,40.41026799700006],[-4.323894499999938,40.54771199600003],[-4.161019499999952,40.62342149700004],[-4.160293499999966,40.68985349900004],[-4.069931,40.793870500000025],[-3.9798,40.799341998000045],[-3.890160499999979,40.968803998000055],[-3.539573,41.16499349800006],[-3.394300499999929,41.000234497000065],[-3.491514499999937,40.798693497000045],[-3.43421,40.73189099800004],[-3.468398,40.68989649800005],[-3.322232499999927,40.64377550100005],[-3.33019,40.597330998000075],[-3.130407,40.40515399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES300","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Madrid","NUTS_NAME":"Madrid"},"geometry":{"type":"Polygon","coordinates":[[[-3.130407,40.40515399800006],[-3.194294499999955,40.24800599800005],[-3.10929,40.285162996000054],[-3.067689,40.15788499800004],[-3.096598000999961,40.142359498000076],[-3.060548499999925,40.090534999000056],[-3.161419499999965,40.06489699700006],[-3.504916500999968,40.047123499000065],[-3.814671499999974,39.885770999000044],[-3.875525,39.92769999800004],[-3.623443,40.053848499000026],[-3.607468,40.10893199800006],[-4.080290499999933,40.26690149900003],[-4.143561500999965,40.24479199900003],[-4.190704,40.297315997000055],[-4.295019499999967,40.21884399700008],[-4.37994,40.31830099700005],[-4.437711499999978,40.23789549800006],[-4.578901,40.21715749800006],[-4.548248499999943,40.34058949800004],[-4.455591,40.31931049600007],[-4.421466499999951,40.40624199800004],[-4.320921,40.41026799700006],[-4.323894499999938,40.54771199600003],[-4.161019499999952,40.62342149700004],[-4.160293499999966,40.68985349900004],[-4.069931,40.793870500000025],[-3.9798,40.799341998000045],[-3.890160499999979,40.968803998000055],[-3.539573,41.16499349800006],[-3.394300499999929,41.000234497000065],[-3.491514499999937,40.798693497000045],[-3.43421,40.73189099800004],[-3.468398,40.68989649800005],[-3.322232499999927,40.64377550100005],[-3.33019,40.597330998000075],[-3.130407,40.40515399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES4","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"CENTRO (ES)","NUTS_NAME":"CENTRO (ES)"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-4.851440998999976,43.12587749800008],[-4.737022998999976,43.02092349800006],[-4.431889499999954,43.045623496000076],[-4.238510998999971,42.95431499700004],[-4.219518000999926,42.850891001000036],[-4.144280499999979,42.86856049800008],[-4.182724499999949,42.810298998000064],[-4.081359,42.76141749800007],[-4.04593,42.76656699600005],[-4.002319,42.83086749700004],[-3.99967,42.76893249600005],[-3.815811,42.81256099900003],[-3.836812,42.87872699600007],[-3.914852,42.86003099700008],[-3.83956,42.93972750100005],[-3.991000000999975,42.93103449600005],[-3.945488,43.00571849600004],[-3.64878700099996,43.18002699800007],[-3.417681499999958,43.13336949700005],[-3.278148499999929,43.19437049800007],[-3.141396,43.16148349800005],[-3.181819,43.12082299600007],[-3.133973499999968,43.09717949900005],[-3.142332,43.02869049900005],[-3.177834,43.021934497000075],[-3.089023499999939,43.00158299800006],[-3.036989499999947,42.982185497000046],[-2.981502,42.938754997000046],[-3.089547,42.89894849800004],[-3.223018,42.94879149800005],[-3.286797499999977,42.88509749600007],[-3.217000499999926,42.82646950000003],[-3.118991499999936,42.891223998000044],[-3.159259499999962,42.80454649700005],[-3.136669,42.76544199600005],[-2.858117499999935,42.63816849800003],[-3.084177499999953,42.64065199800007],[-3.059854,42.58915699800008],[-3.134294499999953,42.54202249700006],[-3.046025499999928,42.44512549800004],[-3.06372749999997,42.356059998000035],[-3.109279,42.351859999000055],[-3.088611499999956,42.24570449600003],[-3.129416,42.19943599900006],[-3.03439,42.08594499700007],[-2.93428549999993,42.088363498000035],[-2.913637499999936,42.022838499000045],[-2.738445,42.12480099600003],[-2.706087,42.10249349700007],[-2.746179,42.008662998000034],[-2.57992,41.99624999900004],[-2.441275,42.13647049600007],[-2.282167499999957,42.13168699800008],[-2.269596,42.088539],[-2.127329499999973,42.097152497000025],[-2.161917,42.066718998000056],[-2.10996149999994,41.95785549800007],[-1.982089,41.919250497000064],[-1.85653949999994,41.966388497000025],[-1.825316,41.77836599800003],[-1.775364499999966,41.72478099800003],[-1.806217499999946,41.66425650000008],[-1.993325499999969,41.60013199800005],[-1.937873499999966,41.40739449800003],[-2.029267499999946,41.38416299600004],[-2.101518499999941,41.44578949800007],[-2.170485499999927,41.31883649800005],[-2.14624,41.184402497000065],[-2.05169,41.14685799800003],[-1.954023,41.16977299900003],[-1.617433499999947,40.94374099600003],[-1.620291499999951,40.879493498000045],[-1.539224499999932,40.792651996000075],[-1.545494499999961,40.59521099800003],[-1.67808249999996,40.59096899800005],[-1.692495,40.48683899800005],[-1.806344,40.39824699800005],[-1.697659,40.321307998000066],[-1.719256,40.27712249600006],[-1.690325,40.309676997000054],[-1.540461499999935,40.190822498000045],[-1.439184499999953,40.197028997000075],[-1.448831499999926,40.14535849600003],[-1.376832,40.01651349800005],[-1.165154,40.01010899700003],[-1.142361499999936,39.971855498000025],[-1.20815349999998,39.943820998000035],[-1.200163,39.858135496000045],[-1.279547499999978,39.67790199600006],[-1.368502499999977,39.68963249600006],[-1.504335,39.56391899700003],[-1.505146499999967,39.41801449700006],[-1.161849,39.305431498000075],[-1.266710499999931,39.05103249900003],[-1.146609,38.929362998000045],[-0.959359,38.94458749700004],[-0.928884499999924,38.783839998000076],[-0.961375499999974,38.77479549700007],[-0.91573649999998,38.695968497000024],[-1.02687,38.655509497000025],[-1.119623499999932,38.737724497000045],[-1.402179499999932,38.690803499000026],[-1.498696,38.53206249900006],[-1.478507,38.37638849900003],[-1.589503,38.31000149700003],[-1.741687499999955,38.38049699700008],[-1.907508,38.297969497000054],[-2.048963499999957,38.30579399900006],[-2.21349349999997,38.201820498000075],[-2.341602,38.02601999800004],[-2.551274000999967,38.08411799900006],[-2.446249499999965,38.18521099700007],[-2.481576,38.39310099900007],[-2.571873,38.41415049600005],[-2.566616,38.490489997000054],[-2.604288,38.51290899800006],[-2.762069,38.532779498000025],[-2.992628,38.450976997000055],[-3.002507499999979,38.40828699700006],[-3.062778,38.47640249800003],[-3.130193,38.438796997000054],[-3.33534949999995,38.480529999000055],[-3.453166499999952,38.400233996000054],[-3.583084,38.45150349800008],[-3.620483,38.39401999700004],[-4.253038499999946,38.40123],[-4.268895,38.34721299700004],[-4.453019,38.40091699900006],[-4.705425499999933,38.57075099600007],[-4.860213499999929,38.61402499800005],[-4.877013499999975,38.68614199800004],[-5.008071499999971,38.71527499800004],[-5.046996000999968,38.72913349700008],[-5.166655,38.71171549600007],[-5.185018,38.65946599700004],[-5.56897749999996,38.432639997000024],[-5.521041499999967,38.20727149800007],[-5.584845499999972,38.13175199900007],[-5.734962499999938,38.086303497000074],[-5.740212,38.13013049800003],[-5.684302499999944,38.15711199800006],[-5.723288499999967,38.19248949800004],[-5.874359,38.158622997000066],[-5.953754,37.99539549900004],[-6.180307499999969,37.941077997000036],[-6.36732,38.04896899700003],[-6.495118,38.01918399700003],[-6.622533,38.09721399700004],[-6.80790449999995,38.11116399700006],[-6.795174499999973,38.17859649800005],[-6.931738499999938,38.20837799700007],[-7.107952499999953,38.18812149900003],[-7.316636,38.43987649800005],[-7.255615,38.72397599800007],[-7.203135,38.75101749700008],[-7.039417499999956,38.86484549700003],[-7.052114,38.90723399700005],[-6.951391499999943,39.02407049800007],[-7.02995,39.11653149600005],[-7.143882,39.10932149800004],[-7.136354499999925,39.17224899800004],[-7.24126,39.20853049800007],[-7.231467,39.27843099900008],[-7.310608,39.34102249800003],[-7.294676,39.455993499000044],[-7.534902449999947,39.66198689000004],[-7.015405,39.670856498000035],[-6.986185499999976,39.81074499700003],[-6.903161499999953,39.87268849800006],[-6.864203,40.011867497000026],[-7.01196049999993,40.12693399900007],[-7.012223,40.22654349700008],[-6.95129849999995,40.25744599800004],[-6.865144,40.27069449800007],[-6.781047499999943,40.36473099700004],[-6.848108,40.45544049700004],[-6.795071,40.51197799800008],[-6.844862,40.56593699700005],[-6.793930499999931,40.66174349600004],[-6.831229499999949,40.75269699800003],[-6.801935,40.861045997000076],[-6.929903499999966,41.02946649800003],[-6.808375,41.036526],[-6.689786,41.20524149700003],[-6.479713,41.29437999800007],[-6.391473,41.38374699800005],[-6.318309499999941,41.38614249600005],[-6.189352,41.57504649800006],[-6.302936,41.66290249800005],[-6.548242499999958,41.68563849800006],[-6.568986499999937,41.743514997000034],[-6.51576149999994,41.870739],[-6.571177499999976,41.88363249900004],[-6.549125499999946,41.944511496000075],[-6.588461499999937,41.967761997000025],[-6.983513499999958,41.97290399800005],[-6.974782499999947,42.05515649700004],[-7.033233500999927,42.07457749800005],[-6.784308,42.25360849900005],[-6.73394,42.35909649600006],[-6.843498499999953,42.40582649700008],[-6.822765499999946,42.490833],[-7.076834,42.50812399800003],[-7.012612499999932,42.657767996000075],[-7.04516750099998,42.69473250000004],[-6.867409,42.78713249800006],[-6.841323,42.83230949800003],[-6.871256499999959,42.87379449800005],[-6.824167,42.91501249800007],[-6.546554,42.91210199900007],[-6.433087499999942,42.939705497000034],[-6.358922,43.05839549700005],[-6.226611499999933,43.00883199700007],[-6.081869499999925,43.075653],[-5.804008499999952,42.96255499800003],[-5.688321,43.056812498000056],[-5.493756499999961,43.02128999800004],[-5.369141,43.09450899800004],[-5.101744499999938,43.10177999900003],[-5.078465,43.16039599700008],[-4.912568,43.23680849700003],[-4.841038499999968,43.180709499000045],[-4.851440998999976,43.12587749800008]],[[-4.421466499999951,40.40624199800004],[-4.455591,40.31931049600007],[-4.548248499999943,40.34058949800004],[-4.578901,40.21715749800006],[-4.437711499999978,40.23789549800006],[-4.37994,40.31830099700005],[-4.295019499999967,40.21884399700008],[-4.190704,40.297315997000055],[-4.143561500999965,40.24479199900003],[-4.080290499999933,40.26690149900003],[-3.607468,40.10893199800006],[-3.623443,40.053848499000026],[-3.875525,39.92769999800004],[-3.814671499999974,39.885770999000044],[-3.504916500999968,40.047123499000065],[-3.161419499999965,40.06489699700006],[-3.060548499999925,40.090534999000056],[-3.096598000999961,40.142359498000076],[-3.067689,40.15788499800004],[-3.10929,40.285162996000054],[-3.194294499999955,40.24800599800005],[-3.130407,40.40515399800006],[-3.33019,40.597330998000075],[-3.322232499999927,40.64377550100005],[-3.468398,40.68989649800005],[-3.43421,40.73189099800004],[-3.491514499999937,40.798693497000045],[-3.394300499999929,41.000234497000065],[-3.539573,41.16499349800006],[-3.890160499999979,40.968803998000055],[-3.9798,40.799341998000045],[-4.069931,40.793870500000025],[-4.160293499999966,40.68985349900004],[-4.161019499999952,40.62342149700004],[-4.323894499999938,40.54771199600003],[-4.320921,40.41026799700006],[-4.421466499999951,40.40624199800004]]],[[[-2.609000499999979,42.695922996000036],[-2.517415,42.646297497000035],[-2.86849,42.73987599600008],[-2.833424,42.79524249900004],[-2.558325499999967,42.76269549800003],[-2.609000499999979,42.695922996000036]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES41","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Castilla y León","NUTS_NAME":"Castilla y León"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-4.851440998999976,43.12587749800008],[-4.737022998999976,43.02092349800006],[-4.431889499999954,43.045623496000076],[-4.238510998999971,42.95431499700004],[-4.219518000999926,42.850891001000036],[-4.144280499999979,42.86856049800008],[-4.182724499999949,42.810298998000064],[-4.081359,42.76141749800007],[-4.04593,42.76656699600005],[-4.002319,42.83086749700004],[-3.99967,42.76893249600005],[-3.815811,42.81256099900003],[-3.836812,42.87872699600007],[-3.914852,42.86003099700008],[-3.83956,42.93972750100005],[-3.991000000999975,42.93103449600005],[-3.945488,43.00571849600004],[-3.64878700099996,43.18002699800007],[-3.417681499999958,43.13336949700005],[-3.278148499999929,43.19437049800007],[-3.141396,43.16148349800005],[-3.181819,43.12082299600007],[-3.133973499999968,43.09717949900005],[-3.142332,43.02869049900005],[-3.177834,43.021934497000075],[-3.089023499999939,43.00158299800006],[-3.036989499999947,42.982185497000046],[-2.981502,42.938754997000046],[-3.089547,42.89894849800004],[-3.223018,42.94879149800005],[-3.286797499999977,42.88509749600007],[-3.217000499999926,42.82646950000003],[-3.118991499999936,42.891223998000044],[-3.159259499999962,42.80454649700005],[-3.136669,42.76544199600005],[-2.858117499999935,42.63816849800003],[-3.084177499999953,42.64065199800007],[-3.059854,42.58915699800008],[-3.134294499999953,42.54202249700006],[-3.046025499999928,42.44512549800004],[-3.06372749999997,42.356059998000035],[-3.109279,42.351859999000055],[-3.088611499999956,42.24570449600003],[-3.129416,42.19943599900006],[-3.03439,42.08594499700007],[-2.93428549999993,42.088363498000035],[-2.913637499999936,42.022838499000045],[-2.738445,42.12480099600003],[-2.706087,42.10249349700007],[-2.746179,42.008662998000034],[-2.57992,41.99624999900004],[-2.441275,42.13647049600007],[-2.282167499999957,42.13168699800008],[-2.269596,42.088539],[-2.127329499999973,42.097152497000025],[-2.161917,42.066718998000056],[-2.10996149999994,41.95785549800007],[-1.982089,41.919250497000064],[-1.85653949999994,41.966388497000025],[-1.825316,41.77836599800003],[-1.775364499999966,41.72478099800003],[-1.806217499999946,41.66425650000008],[-1.993325499999969,41.60013199800005],[-1.937873499999966,41.40739449800003],[-2.029267499999946,41.38416299600004],[-2.101518499999941,41.44578949800007],[-2.170485499999927,41.31883649800005],[-2.14624,41.184402497000065],[-2.05169,41.14685799800003],[-2.056429499999979,41.06969849800004],[-2.130192499999964,41.117515497000056],[-2.418909000999975,41.05771649800005],[-2.53651549999995,41.162536498000065],[-2.596128,41.152690997000036],[-2.589493499999946,41.22454449700007],[-2.71678049999997,41.27568049800004],[-2.819604,41.25735099800005],[-2.888434,41.325248498000065],[-3.061916499999938,41.27343749800008],[-3.207373,41.30424499700007],[-3.539573,41.16499349800006],[-3.890160499999979,40.968803998000055],[-3.9798,40.799341998000045],[-4.069931,40.793870500000025],[-4.160293499999966,40.68985349900004],[-4.161019499999952,40.62342149700004],[-4.323894499999938,40.54771199600003],[-4.320921,40.41026799700006],[-4.421466499999951,40.40624199800004],[-4.455591,40.31931049600007],[-4.548248499999943,40.34058949800004],[-4.578901,40.21715749800006],[-4.654064,40.19802599800005],[-4.698169,40.282226499000046],[-4.804282499999943,40.275127499000064],[-4.954579,40.121162500000025],[-5.013376499999936,40.111095497000065],[-5.018576499999938,40.159092],[-5.202154,40.08288599900004],[-5.336016499999971,40.11585599800003],[-5.36803649999996,40.164146498000036],[-5.345126499999935,40.26498399800005],[-5.531008499999928,40.19475949800005],[-5.73760249999998,40.29416299800005],[-5.793185,40.28674699900006],[-5.797554499999933,40.353286497000056],[-5.920884,40.28094499600007],[-5.992191499999933,40.299053499000024],[-6.105761,40.35683850000004],[-6.065001499999937,40.396358],[-6.239977499999952,40.485900998000034],[-6.536334499999953,40.347293998000055],[-6.587075,40.270774997000046],[-6.865144,40.27069449800007],[-6.781047499999943,40.36473099700004],[-6.848108,40.45544049700004],[-6.795071,40.51197799800008],[-6.844862,40.56593699700005],[-6.793930499999931,40.66174349600004],[-6.831229499999949,40.75269699800003],[-6.801935,40.861045997000076],[-6.929903499999966,41.02946649800003],[-6.808375,41.036526],[-6.689786,41.20524149700003],[-6.479713,41.29437999800007],[-6.391473,41.38374699800005],[-6.318309499999941,41.38614249600005],[-6.189352,41.57504649800006],[-6.302936,41.66290249800005],[-6.548242499999958,41.68563849800006],[-6.568986499999937,41.743514997000034],[-6.51576149999994,41.870739],[-6.571177499999976,41.88363249900004],[-6.549125499999946,41.944511496000075],[-6.588461499999937,41.967761997000025],[-6.983513499999958,41.97290399800005],[-6.974782499999947,42.05515649700004],[-7.033233500999927,42.07457749800005],[-6.784308,42.25360849900005],[-6.73394,42.35909649600006],[-6.843498499999953,42.40582649700008],[-6.822765499999946,42.490833],[-7.076834,42.50812399800003],[-7.012612499999932,42.657767996000075],[-7.04516750099998,42.69473250000004],[-6.867409,42.78713249800006],[-6.841323,42.83230949800003],[-6.871256499999959,42.87379449800005],[-6.824167,42.91501249800007],[-6.546554,42.91210199900007],[-6.433087499999942,42.939705497000034],[-6.358922,43.05839549700005],[-6.226611499999933,43.00883199700007],[-6.081869499999925,43.075653],[-5.804008499999952,42.96255499800003],[-5.688321,43.056812498000056],[-5.493756499999961,43.02128999800004],[-5.369141,43.09450899800004],[-5.101744499999938,43.10177999900003],[-5.078465,43.16039599700008],[-4.912568,43.23680849700003],[-4.841038499999968,43.180709499000045],[-4.851440998999976,43.12587749800008]]],[[[-2.609000499999979,42.695922996000036],[-2.517415,42.646297497000035],[-2.86849,42.73987599600008],[-2.833424,42.79524249900004],[-2.558325499999967,42.76269549800003],[-2.609000499999979,42.695922996000036]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES411","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Ávila","NUTS_NAME":"Ávila"},"geometry":{"type":"Polygon","coordinates":[[[-4.722127499999942,41.14932999800004],[-4.56638,40.956218499000045],[-4.558946499999934,40.83961849800005],[-4.493801,40.833541998000044],[-4.431154,40.74240849800003],[-4.419649499999935,40.63345349600007],[-4.319344,40.647599498000034],[-4.255874,40.65141699700007],[-4.274165,40.67708949900003],[-4.160293499999966,40.68985349900004],[-4.161019499999952,40.62342149700004],[-4.323894499999938,40.54771199600003],[-4.320921,40.41026799700006],[-4.421466499999951,40.40624199800004],[-4.455591,40.31931049600007],[-4.548248499999943,40.34058949800004],[-4.578901,40.21715749800006],[-4.654064,40.19802599800005],[-4.698169,40.282226499000046],[-4.804282499999943,40.275127499000064],[-4.954579,40.121162500000025],[-5.013376499999936,40.111095497000065],[-5.018576499999938,40.159092],[-5.202154,40.08288599900004],[-5.336016499999971,40.11585599800003],[-5.36803649999996,40.164146498000036],[-5.345126499999935,40.26498399800005],[-5.531008499999928,40.19475949800005],[-5.73760249999998,40.29416299800005],[-5.634213,40.46048349800003],[-5.560963,40.477134997000064],[-5.574642500999971,40.444289998000045],[-5.514902499999948,40.42342749800008],[-5.465586,40.46594999900003],[-5.425593,40.556675],[-5.524729000999969,40.54077149700004],[-5.510113,40.590404498000055],[-5.382989499999951,40.62079649800006],[-5.377733,40.67786399800008],[-5.161537000999942,40.81402999900007],[-5.089870499999961,40.996646998000074],[-5.153020499999968,41.063762498000074],[-5.124957,41.13205699900004],[-4.96764,41.15231699700007],[-4.863706,41.09403199800005],[-4.722127499999942,41.14932999800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES412","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Burgos","NUTS_NAME":"Burgos"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-2.609000499999979,42.695922996000036],[-2.517415,42.646297497000035],[-2.86849,42.73987599600008],[-2.833424,42.79524249900004],[-2.558325499999967,42.76269549800003],[-2.609000499999979,42.695922996000036]]],[[[-3.141396,43.16148349800005],[-3.181819,43.12082299600007],[-3.133973499999968,43.09717949900005],[-3.142332,43.02869049900005],[-3.177834,43.021934497000075],[-3.089023499999939,43.00158299800006],[-3.036989499999947,42.982185497000046],[-2.981502,42.938754997000046],[-3.089547,42.89894849800004],[-3.223018,42.94879149800005],[-3.286797499999977,42.88509749600007],[-3.217000499999926,42.82646950000003],[-3.118991499999936,42.891223998000044],[-3.159259499999962,42.80454649700005],[-3.136669,42.76544199600005],[-2.858117499999935,42.63816849800003],[-3.084177499999953,42.64065199800007],[-3.059854,42.58915699800008],[-3.134294499999953,42.54202249700006],[-3.046025499999928,42.44512549800004],[-3.06372749999997,42.356059998000035],[-3.109279,42.351859999000055],[-3.088611499999956,42.24570449600003],[-3.129416,42.19943599900006],[-3.03439,42.08594499700007],[-2.93428549999993,42.088363498000035],[-2.913637499999936,42.022838499000045],[-3.171373499999959,41.79301449700006],[-3.28377949999998,41.883804498000075],[-3.275476,41.768672998000056],[-3.354409,41.73725900100004],[-3.459186,41.58963399600003],[-3.541596,41.57760250000007],[-3.671751499999971,41.56042499800003],[-3.739509,41.45078249800008],[-3.980689499999926,41.58336649800003],[-4.076383499999963,41.72607799800005],[-4.040900499999964,41.768344497000044],[-4.088854,41.87064349800005],[-3.891499499999952,41.99663149600008],[-4.03300749999994,41.954071],[-3.982134000999963,42.05125799800004],[-4.079914499999973,42.03676599600004],[-4.103062499999965,42.08702449700007],[-4.056233499999962,42.100895],[-4.205443998999954,42.13458999800008],[-4.335365499999966,42.43058399600005],[-4.242615499999943,42.456210998000074],[-4.303028,42.55492399800005],[-4.266213499999935,42.69440449600006],[-4.081359,42.76141749800007],[-4.04593,42.76656699600005],[-4.002319,42.83086749700004],[-3.99967,42.76893249600005],[-3.815811,42.81256099900003],[-3.836812,42.87872699600007],[-3.914852,42.86003099700008],[-3.83956,42.93972750100005],[-3.991000000999975,42.93103449600005],[-3.945488,43.00571849600004],[-3.64878700099996,43.18002699800007],[-3.417681499999958,43.13336949700005],[-3.278148499999929,43.19437049800007],[-3.141396,43.16148349800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES413","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"León","NUTS_NAME":"León"},"geometry":{"type":"Polygon","coordinates":[[[-4.851440998999976,43.12587749800008],[-4.737022998999976,43.02092349800006],[-4.90095249999996,42.858157996000045],[-4.936373499999945,42.744422996000026],[-4.890591,42.69278699800003],[-4.900012998999955,42.450648999000066],[-4.984217499999943,42.337844999000026],[-4.94250749899993,42.34651949900007],[-4.953461498999957,42.29713049700007],[-5.002154499999961,42.28910449800003],[-5.054823499999941,42.257499996000035],[-5.093766,42.30926899700006],[-5.24469849999997,42.263847498000075],[-5.241496499999926,42.21449649600004],[-5.388113,42.22142799800008],[-5.376311499999929,42.041458],[-5.421605499999941,42.03066649800007],[-5.520905499999969,42.097129997000025],[-5.560710499999971,42.05595799800005],[-5.665348499999936,42.14113999700004],[-6.212393499999962,42.148067498000046],[-6.784308,42.25360849900005],[-6.73394,42.35909649600006],[-6.843498499999953,42.40582649700008],[-6.822765499999946,42.490833],[-7.076834,42.50812399800003],[-7.012612499999932,42.657767996000075],[-7.04516750099998,42.69473250000004],[-6.867409,42.78713249800006],[-6.841323,42.83230949800003],[-6.871256499999959,42.87379449800005],[-6.824167,42.91501249800007],[-6.546554,42.91210199900007],[-6.433087499999942,42.939705497000034],[-6.358922,43.05839549700005],[-6.226611499999933,43.00883199700007],[-6.081869499999925,43.075653],[-5.804008499999952,42.96255499800003],[-5.688321,43.056812498000056],[-5.493756499999961,43.02128999800004],[-5.369141,43.09450899800004],[-5.101744499999938,43.10177999900003],[-5.078465,43.16039599700008],[-4.912568,43.23680849700003],[-4.841038499999968,43.180709499000045],[-4.851440998999976,43.12587749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES414","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Palencia","NUTS_NAME":"Palencia"},"geometry":{"type":"Polygon","coordinates":[[[-4.238510998999971,42.95431499700004],[-4.219518000999926,42.850891001000036],[-4.144280499999979,42.86856049800008],[-4.182724499999949,42.810298998000064],[-4.081359,42.76141749800007],[-4.266213499999935,42.69440449600006],[-4.303028,42.55492399800005],[-4.242615499999943,42.456210998000074],[-4.335365499999966,42.43058399600005],[-4.205443998999954,42.13458999800008],[-4.056233499999962,42.100895],[-4.103062499999965,42.08702449700007],[-4.079914499999973,42.03676599600004],[-3.982134000999963,42.05125799800004],[-4.03300749999994,41.954071],[-3.891499499999952,41.99663149600008],[-4.088854,41.87064349800005],[-4.040900499999964,41.768344497000044],[-4.256262499999934,41.80636599800005],[-4.450098499999967,41.758358],[-4.664797500999953,41.899459500000034],[-4.789274499999976,41.81801599800008],[-4.873985,41.933032998000044],[-4.983705499999928,41.910014998000065],[-5.000166499999978,41.95104949800003],[-4.917844498999955,42.090778500000056],[-4.927515,42.15019599800007],[-5.026982,42.153892496000026],[-5.002154499999961,42.28910449800003],[-4.953461498999957,42.29713049700007],[-4.94250749899993,42.34651949900007],[-4.984217499999943,42.337844999000026],[-4.900012998999955,42.450648999000066],[-4.890591,42.69278699800003],[-4.936373499999945,42.744422996000026],[-4.90095249999996,42.858157996000045],[-4.737022998999976,43.02092349800006],[-4.431889499999954,43.045623496000076],[-4.238510998999971,42.95431499700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES415","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Salamanca","NUTS_NAME":"Salamanca"},"geometry":{"type":"Polygon","coordinates":[[[-5.288771,41.18200699700003],[-5.124957,41.13205699900004],[-5.153020499999968,41.063762498000074],[-5.089870499999961,40.996646998000074],[-5.161537000999942,40.81402999900007],[-5.377733,40.67786399800008],[-5.382989499999951,40.62079649800006],[-5.510113,40.590404498000055],[-5.524729000999969,40.54077149700004],[-5.425593,40.556675],[-5.465586,40.46594999900003],[-5.514902499999948,40.42342749800008],[-5.574642500999971,40.444289998000045],[-5.560963,40.477134997000064],[-5.634213,40.46048349800003],[-5.73760249999998,40.29416299800005],[-5.793185,40.28674699900006],[-5.797554499999933,40.353286497000056],[-5.920884,40.28094499600007],[-5.992191499999933,40.299053499000024],[-6.105761,40.35683850000004],[-6.065001499999937,40.396358],[-6.239977499999952,40.485900998000034],[-6.536334499999953,40.347293998000055],[-6.587075,40.270774997000046],[-6.865144,40.27069449800007],[-6.781047499999943,40.36473099700004],[-6.848108,40.45544049700004],[-6.795071,40.51197799800008],[-6.844862,40.56593699700005],[-6.793930499999931,40.66174349600004],[-6.831229499999949,40.75269699800003],[-6.801935,40.861045997000076],[-6.929903499999966,41.02946649800003],[-6.808375,41.036526],[-6.689786,41.20524149700003],[-6.479713,41.29437999800007],[-6.008857499999976,41.20051950000004],[-6.023912,41.15739499800003],[-5.951337499999966,41.14028949800007],[-5.897697500999925,41.23861299600003],[-5.656983499999967,41.24556349800008],[-5.407621499999948,41.18560799900007],[-5.328978,41.11718749800008],[-5.288771,41.18200699700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES416","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Segovia","NUTS_NAME":"Segovia"},"geometry":{"type":"Polygon","coordinates":[[[-3.980689499999926,41.58336649800003],[-3.739509,41.45078249800008],[-3.671751499999971,41.56042499800003],[-3.541596,41.57760250000007],[-3.207373,41.30424499700007],[-3.539573,41.16499349800006],[-3.890160499999979,40.968803998000055],[-3.9798,40.799341998000045],[-4.069931,40.793870500000025],[-4.160293499999966,40.68985349900004],[-4.274165,40.67708949900003],[-4.255874,40.65141699700007],[-4.319344,40.647599498000034],[-4.419649499999935,40.63345349600007],[-4.431154,40.74240849800003],[-4.493801,40.833541998000044],[-4.558946499999934,40.83961849800005],[-4.56638,40.956218499000045],[-4.722127499999942,41.14932999800004],[-4.559565499999962,41.30217749800005],[-4.467808499999933,41.29589850000008],[-4.514645,41.40962599800008],[-4.065298,41.49473949800006],[-3.980689499999926,41.58336649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES417","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Soria","NUTS_NAME":"Soria"},"geometry":{"type":"Polygon","coordinates":[[[-1.85653949999994,41.966388497000025],[-1.825316,41.77836599800003],[-1.775364499999966,41.72478099800003],[-1.806217499999946,41.66425650000008],[-1.993325499999969,41.60013199800005],[-1.937873499999966,41.40739449800003],[-2.029267499999946,41.38416299600004],[-2.101518499999941,41.44578949800007],[-2.170485499999927,41.31883649800005],[-2.14624,41.184402497000065],[-2.05169,41.14685799800003],[-2.056429499999979,41.06969849800004],[-2.130192499999964,41.117515497000056],[-2.418909000999975,41.05771649800005],[-2.53651549999995,41.162536498000065],[-2.596128,41.152690997000036],[-2.589493499999946,41.22454449700007],[-2.71678049999997,41.27568049800004],[-2.819604,41.25735099800005],[-2.888434,41.325248498000065],[-3.061916499999938,41.27343749800008],[-3.207373,41.30424499700007],[-3.541596,41.57760250000007],[-3.459186,41.58963399600003],[-3.354409,41.73725900100004],[-3.275476,41.768672998000056],[-3.28377949999998,41.883804498000075],[-3.171373499999959,41.79301449700006],[-2.913637499999936,42.022838499000045],[-2.738445,42.12480099600003],[-2.706087,42.10249349700007],[-2.746179,42.008662998000034],[-2.57992,41.99624999900004],[-2.441275,42.13647049600007],[-2.282167499999957,42.13168699800008],[-2.269596,42.088539],[-2.127329499999973,42.097152497000025],[-2.161917,42.066718998000056],[-2.10996149999994,41.95785549800007],[-1.982089,41.919250497000064],[-1.85653949999994,41.966388497000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES418","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Valladolid","NUTS_NAME":"Valladolid"},"geometry":{"type":"Polygon","coordinates":[[[-5.093766,42.30926899700006],[-5.054823499999941,42.257499996000035],[-5.002154499999961,42.28910449800003],[-5.026982,42.153892496000026],[-4.927515,42.15019599800007],[-4.917844498999955,42.090778500000056],[-5.000166499999978,41.95104949800003],[-4.983705499999928,41.910014998000065],[-4.873985,41.933032998000044],[-4.789274499999976,41.81801599800008],[-4.664797500999953,41.899459500000034],[-4.450098499999967,41.758358],[-4.256262499999934,41.80636599800005],[-4.040900499999964,41.768344497000044],[-4.076383499999963,41.72607799800005],[-3.980689499999926,41.58336649800003],[-4.065298,41.49473949800006],[-4.514645,41.40962599800008],[-4.467808499999933,41.29589850000008],[-4.559565499999962,41.30217749800005],[-4.722127499999942,41.14932999800004],[-4.863706,41.09403199800005],[-4.96764,41.15231699700007],[-5.124957,41.13205699900004],[-5.288771,41.18200699700003],[-5.340324000999942,41.19412249700008],[-5.294991499999981,41.24062349900004],[-5.332519499999933,41.47566249600004],[-5.229624,41.52805299800008],[-5.391403,41.76050949800003],[-5.276837,41.822887499000046],[-5.320896,41.93750749700007],[-5.280669499999931,41.978789996000046],[-5.376311499999929,42.041458],[-5.388113,42.22142799800008],[-5.241496499999926,42.21449649600004],[-5.24469849999997,42.263847498000075],[-5.093766,42.30926899700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES419","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Zamora","NUTS_NAME":"Zamora"},"geometry":{"type":"Polygon","coordinates":[[[-5.665348499999936,42.14113999700004],[-5.560710499999971,42.05595799800005],[-5.520905499999969,42.097129997000025],[-5.421605499999941,42.03066649800007],[-5.376311499999929,42.041458],[-5.280669499999931,41.978789996000046],[-5.320896,41.93750749700007],[-5.276837,41.822887499000046],[-5.391403,41.76050949800003],[-5.229624,41.52805299800008],[-5.332519499999933,41.47566249600004],[-5.294991499999981,41.24062349900004],[-5.340324000999942,41.19412249700008],[-5.288771,41.18200699700003],[-5.328978,41.11718749800008],[-5.407621499999948,41.18560799900007],[-5.656983499999967,41.24556349800008],[-5.897697500999925,41.23861299600003],[-5.951337499999966,41.14028949800007],[-6.023912,41.15739499800003],[-6.008857499999976,41.20051950000004],[-6.479713,41.29437999800007],[-6.391473,41.38374699800005],[-6.318309499999941,41.38614249600005],[-6.189352,41.57504649800006],[-6.302936,41.66290249800005],[-6.548242499999958,41.68563849800006],[-6.568986499999937,41.743514997000034],[-6.51576149999994,41.870739],[-6.571177499999976,41.88363249900004],[-6.549125499999946,41.944511496000075],[-6.588461499999937,41.967761997000025],[-6.983513499999958,41.97290399800005],[-6.974782499999947,42.05515649700004],[-7.033233500999927,42.07457749800005],[-6.784308,42.25360849900005],[-6.212393499999962,42.148067498000046],[-5.665348499999936,42.14113999700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES42","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Castilla-La Mancha","NUTS_NAME":"Castilla-La Mancha"},"geometry":{"type":"Polygon","coordinates":[[[-2.888434,41.325248498000065],[-2.819604,41.25735099800005],[-2.71678049999997,41.27568049800004],[-2.589493499999946,41.22454449700007],[-2.596128,41.152690997000036],[-2.53651549999995,41.162536498000065],[-2.418909000999975,41.05771649800005],[-2.130192499999964,41.117515497000056],[-2.056429499999979,41.06969849800004],[-2.05169,41.14685799800003],[-1.954023,41.16977299900003],[-1.617433499999947,40.94374099600003],[-1.620291499999951,40.879493498000045],[-1.539224499999932,40.792651996000075],[-1.545494499999961,40.59521099800003],[-1.67808249999996,40.59096899800005],[-1.692495,40.48683899800005],[-1.806344,40.39824699800005],[-1.697659,40.321307998000066],[-1.719256,40.27712249600006],[-1.690325,40.309676997000054],[-1.540461499999935,40.190822498000045],[-1.439184499999953,40.197028997000075],[-1.448831499999926,40.14535849600003],[-1.376832,40.01651349800005],[-1.165154,40.01010899700003],[-1.142361499999936,39.971855498000025],[-1.20815349999998,39.943820998000035],[-1.200163,39.858135496000045],[-1.279547499999978,39.67790199600006],[-1.368502499999977,39.68963249600006],[-1.504335,39.56391899700003],[-1.505146499999967,39.41801449700006],[-1.161849,39.305431498000075],[-1.266710499999931,39.05103249900003],[-1.146609,38.929362998000045],[-0.959359,38.94458749700004],[-0.928884499999924,38.783839998000076],[-0.961375499999974,38.77479549700007],[-0.91573649999998,38.695968497000024],[-1.02687,38.655509497000025],[-1.119623499999932,38.737724497000045],[-1.402179499999932,38.690803499000026],[-1.498696,38.53206249900006],[-1.478507,38.37638849900003],[-1.589503,38.31000149700003],[-1.741687499999955,38.38049699700008],[-1.907508,38.297969497000054],[-2.048963499999957,38.30579399900006],[-2.21349349999997,38.201820498000075],[-2.341602,38.02601999800004],[-2.551274000999967,38.08411799900006],[-2.446249499999965,38.18521099700007],[-2.481576,38.39310099900007],[-2.571873,38.41415049600005],[-2.566616,38.490489997000054],[-2.604288,38.51290899800006],[-2.762069,38.532779498000025],[-2.992628,38.450976997000055],[-3.002507499999979,38.40828699700006],[-3.062778,38.47640249800003],[-3.130193,38.438796997000054],[-3.33534949999995,38.480529999000055],[-3.453166499999952,38.400233996000054],[-3.583084,38.45150349800008],[-3.620483,38.39401999700004],[-4.253038499999946,38.40123],[-4.268895,38.34721299700004],[-4.453019,38.40091699900006],[-4.705425499999933,38.57075099600007],[-4.860213499999929,38.61402499800005],[-4.877013499999975,38.68614199800004],[-5.008071499999971,38.71527499800004],[-5.046996000999968,38.72913349700008],[-4.990040499999964,38.73980349900006],[-4.943528499999957,38.854427497000074],[-4.847362499999974,38.88244999800003],[-4.831171000999973,38.93566149900005],[-4.918264,38.95709599800006],[-4.962548,39.05877699700005],[-4.832323,39.05154799600007],[-4.874463499999933,39.099064],[-4.810313,39.19946299700007],[-4.649006,39.16494749800006],[-4.761050499999953,39.32083499700008],[-4.710084499999937,39.34579449700004],[-4.687733,39.45033649800007],[-4.755411,39.415759997000066],[-4.810473499999944,39.39661399800008],[-4.940783499999952,39.39516450000008],[-4.952554499999962,39.395038498000076],[-5.206670499999973,39.59727449800005],[-5.137835499999937,39.71504199900005],[-5.166938499999958,39.79992649700006],[-5.308869,39.760432998000056],[-5.284090499999934,39.86391849800003],[-5.406153,39.87777299800007],[-5.336016499999971,40.11585599800003],[-5.202154,40.08288599900004],[-5.018576499999938,40.159092],[-5.013376499999936,40.111095497000065],[-4.954579,40.121162500000025],[-4.804282499999943,40.275127499000064],[-4.698169,40.282226499000046],[-4.654064,40.19802599800005],[-4.578901,40.21715749800006],[-4.437711499999978,40.23789549800006],[-4.37994,40.31830099700005],[-4.295019499999967,40.21884399700008],[-4.190704,40.297315997000055],[-4.143561500999965,40.24479199900003],[-4.080290499999933,40.26690149900003],[-3.607468,40.10893199800006],[-3.623443,40.053848499000026],[-3.875525,39.92769999800004],[-3.814671499999974,39.885770999000044],[-3.504916500999968,40.047123499000065],[-3.161419499999965,40.06489699700006],[-3.060548499999925,40.090534999000056],[-3.096598000999961,40.142359498000076],[-3.067689,40.15788499800004],[-3.10929,40.285162996000054],[-3.194294499999955,40.24800599800005],[-3.130407,40.40515399800006],[-3.33019,40.597330998000075],[-3.322232499999927,40.64377550100005],[-3.468398,40.68989649800005],[-3.43421,40.73189099800004],[-3.491514499999937,40.798693497000045],[-3.394300499999929,41.000234497000065],[-3.539573,41.16499349800006],[-3.207373,41.30424499700007],[-3.061916499999938,41.27343749800008],[-2.888434,41.325248498000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES421","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Albacete","NUTS_NAME":"Albacete"},"geometry":{"type":"Polygon","coordinates":[[[-0.928884499999924,38.783839998000076],[-0.961375499999974,38.77479549700007],[-0.91573649999998,38.695968497000024],[-1.02687,38.655509497000025],[-1.119623499999932,38.737724497000045],[-1.402179499999932,38.690803499000026],[-1.498696,38.53206249900006],[-1.478507,38.37638849900003],[-1.589503,38.31000149700003],[-1.741687499999955,38.38049699700008],[-1.907508,38.297969497000054],[-2.048963499999957,38.30579399900006],[-2.21349349999997,38.201820498000075],[-2.341602,38.02601999800004],[-2.551274000999967,38.08411799900006],[-2.446249499999965,38.18521099700007],[-2.481576,38.39310099900007],[-2.571873,38.41415049600005],[-2.566616,38.490489997000054],[-2.604288,38.51290899800006],[-2.762069,38.532779498000025],[-2.750089499999945,38.61559299600003],[-2.66885149999996,38.64924999800007],[-2.637884000999975,38.73465349800006],[-2.732238499999937,38.78187549800003],[-2.762306,38.880798500000026],[-2.88289,38.938522498000054],[-2.809932,39.02386099800003],[-2.84265849999997,39.13452499700003],[-2.722453,39.26636499700004],[-2.74224849999996,39.31818399700006],[-2.511038,39.34218199700007],[-2.374067,39.23280699800006],[-2.321182,39.28376399600006],[-2.288666499999977,39.226684499000044],[-2.050948499999947,39.266699999000025],[-2.096247,39.35893999700005],[-2.020318499999973,39.35184849800004],[-1.989782,39.29370099800008],[-1.769832,39.28052899800008],[-1.505146499999967,39.41801449700006],[-1.161849,39.305431498000075],[-1.266710499999931,39.05103249900003],[-1.146609,38.929362998000045],[-0.959359,38.94458749700004],[-0.928884499999924,38.783839998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES422","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Ciudad Real","NUTS_NAME":"Ciudad Real"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-4.030101499999944,39.52201449800003],[-4.118536499999948,39.45616149800003],[-4.148096499999951,39.36280449800006],[-3.972221,39.35221499800008],[-3.965489499999933,39.298216997000054],[-3.85737249999994,39.25854099800006],[-3.532597,39.30419549700008],[-3.410286,39.40462849800008],[-3.30914,39.399188999000046],[-3.294205,39.47088249800004],[-2.931376499999942,39.47151949800008],[-2.882377499999961,39.350914],[-2.776028499999939,39.39479449700008],[-2.74224849999996,39.31818399700006],[-2.722453,39.26636499700004],[-2.84265849999997,39.13452499700003],[-2.809932,39.02386099800003],[-2.88289,38.938522498000054],[-2.762306,38.880798500000026],[-2.732238499999937,38.78187549800003],[-2.637884000999975,38.73465349800006],[-2.66885149999996,38.64924999800007],[-2.750089499999945,38.61559299600003],[-2.762069,38.532779498000025],[-2.992628,38.450976997000055],[-3.002507499999979,38.40828699700006],[-3.062778,38.47640249800003],[-3.130193,38.438796997000054],[-3.33534949999995,38.480529999000055],[-3.453166499999952,38.400233996000054],[-3.583084,38.45150349800008],[-3.620483,38.39401999700004],[-4.253038499999946,38.40123],[-4.268895,38.34721299700004],[-4.453019,38.40091699900006],[-4.705425499999933,38.57075099600007],[-4.860213499999929,38.61402499800005],[-4.877013499999975,38.68614199800004],[-5.008071499999971,38.71527499800004],[-5.046996000999968,38.72913349700008],[-4.990040499999964,38.73980349900006],[-4.943528499999957,38.854427497000074],[-4.847362499999974,38.88244999800003],[-4.831171000999973,38.93566149900005],[-4.918264,38.95709599800006],[-4.962548,39.05877699700005],[-4.832323,39.05154799600007],[-4.874463499999933,39.099064],[-4.810313,39.19946299700007],[-4.649006,39.16494749800006],[-4.761050499999953,39.32083499700008],[-4.710084499999937,39.34579449700004],[-4.687733,39.45033649800007],[-4.618168,39.45444099900004],[-4.525398,39.561535],[-4.461619500999973,39.478587996000044],[-4.237303,39.479172498000025],[-4.158755499999927,39.576801499000055],[-4.041801,39.57539349600006],[-4.030101499999944,39.52201449800003]]],[[[-4.810473499999944,39.39661399800008],[-4.940783499999952,39.39516450000008],[-4.86316,39.556190497000046],[-4.76251,39.51408],[-4.755411,39.415759997000066],[-4.810473499999944,39.39661399800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES423","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Cuenca","NUTS_NAME":"Cuenca"},"geometry":{"type":"Polygon","coordinates":[[[-1.806344,40.39824699800005],[-1.697659,40.321307998000066],[-1.719256,40.27712249600006],[-1.690325,40.309676997000054],[-1.540461499999935,40.190822498000045],[-1.439184499999953,40.197028997000075],[-1.448831499999926,40.14535849600003],[-1.376832,40.01651349800005],[-1.165154,40.01010899700003],[-1.142361499999936,39.971855498000025],[-1.20815349999998,39.943820998000035],[-1.200163,39.858135496000045],[-1.279547499999978,39.67790199600006],[-1.368502499999977,39.68963249600006],[-1.504335,39.56391899700003],[-1.505146499999967,39.41801449700006],[-1.769832,39.28052899800008],[-1.989782,39.29370099800008],[-2.020318499999973,39.35184849800004],[-2.096247,39.35893999700005],[-2.050948499999947,39.266699999000025],[-2.288666499999977,39.226684499000044],[-2.321182,39.28376399600006],[-2.374067,39.23280699800006],[-2.511038,39.34218199700007],[-2.74224849999996,39.31818399700006],[-2.776028499999939,39.39479449700008],[-2.882377499999961,39.350914],[-2.931376499999942,39.47151949800008],[-2.908235499999932,39.64236449800006],[-3.096273,39.869140499000025],[-3.096601499999963,39.986968997000076],[-3.142571,39.984993],[-3.161419499999965,40.06489699700006],[-3.060548499999925,40.090534999000056],[-3.096598000999961,40.142359498000076],[-3.067689,40.15788499800004],[-3.016523,40.152484997000045],[-3.005689,40.20889649800006],[-2.904853499999945,40.15792449600008],[-2.825984,40.20226649700004],[-2.77685,40.44664749900005],[-2.722903499999973,40.397140498000056],[-2.64157849999998,40.44887149900006],[-2.519810500999938,40.42344699600005],[-2.538715499999967,40.53404599600003],[-2.380002,40.54360949800008],[-2.398729,40.61378099700005],[-2.292190499999947,40.57902549700003],[-2.274596499999973,40.64477149600003],[-1.926436499999966,40.59408199600006],[-1.806344,40.39824699800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES424","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Guadalajara","NUTS_NAME":"Guadalajara"},"geometry":{"type":"Polygon","coordinates":[[[-2.05169,41.14685799800003],[-1.954023,41.16977299900003],[-1.617433499999947,40.94374099600003],[-1.620291499999951,40.879493498000045],[-1.539224499999932,40.792651996000075],[-1.545494499999961,40.59521099800003],[-1.67808249999996,40.59096899800005],[-1.692495,40.48683899800005],[-1.806344,40.39824699800005],[-1.926436499999966,40.59408199600006],[-2.274596499999973,40.64477149600003],[-2.292190499999947,40.57902549700003],[-2.398729,40.61378099700005],[-2.380002,40.54360949800008],[-2.538715499999967,40.53404599600003],[-2.519810500999938,40.42344699600005],[-2.64157849999998,40.44887149900006],[-2.722903499999973,40.397140498000056],[-2.77685,40.44664749900005],[-2.825984,40.20226649700004],[-2.904853499999945,40.15792449600008],[-3.005689,40.20889649800006],[-3.016523,40.152484997000045],[-3.067689,40.15788499800004],[-3.10929,40.285162996000054],[-3.194294499999955,40.24800599800005],[-3.130407,40.40515399800006],[-3.33019,40.597330998000075],[-3.322232499999927,40.64377550100005],[-3.468398,40.68989649800005],[-3.43421,40.73189099800004],[-3.491514499999937,40.798693497000045],[-3.394300499999929,41.000234497000065],[-3.539573,41.16499349800006],[-3.207373,41.30424499700007],[-3.061916499999938,41.27343749800008],[-2.888434,41.325248498000065],[-2.819604,41.25735099800005],[-2.71678049999997,41.27568049800004],[-2.589493499999946,41.22454449700007],[-2.596128,41.152690997000036],[-2.53651549999995,41.162536498000065],[-2.418909000999975,41.05771649800005],[-2.130192499999964,41.117515497000056],[-2.056429499999979,41.06969849800004],[-2.05169,41.14685799800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES425","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Toledo","NUTS_NAME":"Toledo"},"geometry":{"type":"Polygon","coordinates":[[[-3.161419499999965,40.06489699700006],[-3.142571,39.984993],[-3.096601499999963,39.986968997000076],[-3.096273,39.869140499000025],[-2.908235499999932,39.64236449800006],[-2.931376499999942,39.47151949800008],[-3.294205,39.47088249800004],[-3.30914,39.399188999000046],[-3.410286,39.40462849800008],[-3.532597,39.30419549700008],[-3.85737249999994,39.25854099800006],[-3.965489499999933,39.298216997000054],[-3.972221,39.35221499800008],[-4.148096499999951,39.36280449800006],[-4.118536499999948,39.45616149800003],[-4.030101499999944,39.52201449800003],[-4.041801,39.57539349600006],[-4.158755499999927,39.576801499000055],[-4.237303,39.479172498000025],[-4.461619500999973,39.478587996000044],[-4.525398,39.561535],[-4.618168,39.45444099900004],[-4.687733,39.45033649800007],[-4.755411,39.415759997000066],[-4.76251,39.51408],[-4.86316,39.556190497000046],[-4.940783499999952,39.39516450000008],[-4.952554499999962,39.395038498000076],[-5.206670499999973,39.59727449800005],[-5.137835499999937,39.71504199900005],[-5.166938499999958,39.79992649700006],[-5.308869,39.760432998000056],[-5.284090499999934,39.86391849800003],[-5.406153,39.87777299800007],[-5.336016499999971,40.11585599800003],[-5.202154,40.08288599900004],[-5.018576499999938,40.159092],[-5.013376499999936,40.111095497000065],[-4.954579,40.121162500000025],[-4.804282499999943,40.275127499000064],[-4.698169,40.282226499000046],[-4.654064,40.19802599800005],[-4.578901,40.21715749800006],[-4.437711499999978,40.23789549800006],[-4.37994,40.31830099700005],[-4.295019499999967,40.21884399700008],[-4.190704,40.297315997000055],[-4.143561500999965,40.24479199900003],[-4.080290499999933,40.26690149900003],[-3.607468,40.10893199800006],[-3.623443,40.053848499000026],[-3.875525,39.92769999800004],[-3.814671499999974,39.885770999000044],[-3.504916500999968,40.047123499000065],[-3.161419499999965,40.06489699700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES43","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Extremadura","NUTS_NAME":"Extremadura"},"geometry":{"type":"Polygon","coordinates":[[[-5.992191499999933,40.299053499000024],[-5.920884,40.28094499600007],[-5.797554499999933,40.353286497000056],[-5.793185,40.28674699900006],[-5.73760249999998,40.29416299800005],[-5.531008499999928,40.19475949800005],[-5.345126499999935,40.26498399800005],[-5.36803649999996,40.164146498000036],[-5.336016499999971,40.11585599800003],[-5.406153,39.87777299800007],[-5.284090499999934,39.86391849800003],[-5.308869,39.760432998000056],[-5.166938499999958,39.79992649700006],[-5.137835499999937,39.71504199900005],[-5.206670499999973,39.59727449800005],[-4.952554499999962,39.395038498000076],[-4.940783499999952,39.39516450000008],[-4.810473499999944,39.39661399800008],[-4.755411,39.415759997000066],[-4.687733,39.45033649800007],[-4.710084499999937,39.34579449700004],[-4.761050499999953,39.32083499700008],[-4.649006,39.16494749800006],[-4.810313,39.19946299700007],[-4.874463499999933,39.099064],[-4.832323,39.05154799600007],[-4.962548,39.05877699700005],[-4.918264,38.95709599800006],[-4.831171000999973,38.93566149900005],[-4.847362499999974,38.88244999800003],[-4.943528499999957,38.854427497000074],[-4.990040499999964,38.73980349900006],[-5.046996000999968,38.72913349700008],[-5.166655,38.71171549600007],[-5.185018,38.65946599700004],[-5.56897749999996,38.432639997000024],[-5.521041499999967,38.20727149800007],[-5.584845499999972,38.13175199900007],[-5.734962499999938,38.086303497000074],[-5.740212,38.13013049800003],[-5.684302499999944,38.15711199800006],[-5.723288499999967,38.19248949800004],[-5.874359,38.158622997000066],[-5.953754,37.99539549900004],[-6.180307499999969,37.941077997000036],[-6.36732,38.04896899700003],[-6.495118,38.01918399700003],[-6.622533,38.09721399700004],[-6.80790449999995,38.11116399700006],[-6.795174499999973,38.17859649800005],[-6.931738499999938,38.20837799700007],[-7.107952499999953,38.18812149900003],[-7.316636,38.43987649800005],[-7.255615,38.72397599800007],[-7.203135,38.75101749700008],[-7.039417499999956,38.86484549700003],[-7.052114,38.90723399700005],[-6.951391499999943,39.02407049800007],[-7.02995,39.11653149600005],[-7.143882,39.10932149800004],[-7.136354499999925,39.17224899800004],[-7.24126,39.20853049800007],[-7.231467,39.27843099900008],[-7.310608,39.34102249800003],[-7.294676,39.455993499000044],[-7.534902449999947,39.66198689000004],[-7.015405,39.670856498000035],[-6.986185499999976,39.81074499700003],[-6.903161499999953,39.87268849800006],[-6.864203,40.011867497000026],[-7.01196049999993,40.12693399900007],[-7.012223,40.22654349700008],[-6.95129849999995,40.25744599800004],[-6.865144,40.27069449800007],[-6.587075,40.270774997000046],[-6.536334499999953,40.347293998000055],[-6.239977499999952,40.485900998000034],[-6.065001499999937,40.396358],[-6.105761,40.35683850000004],[-5.992191499999933,40.299053499000024]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES431","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Badajoz","NUTS_NAME":"Badajoz"},"geometry":{"type":"Polygon","coordinates":[[[-5.046996000999968,38.72913349700008],[-5.166655,38.71171549600007],[-5.185018,38.65946599700004],[-5.56897749999996,38.432639997000024],[-5.521041499999967,38.20727149800007],[-5.584845499999972,38.13175199900007],[-5.734962499999938,38.086303497000074],[-5.740212,38.13013049800003],[-5.684302499999944,38.15711199800006],[-5.723288499999967,38.19248949800004],[-5.874359,38.158622997000066],[-5.953754,37.99539549900004],[-6.180307499999969,37.941077997000036],[-6.36732,38.04896899700003],[-6.495118,38.01918399700003],[-6.622533,38.09721399700004],[-6.80790449999995,38.11116399700006],[-6.795174499999973,38.17859649800005],[-6.931738499999938,38.20837799700007],[-7.107952499999953,38.18812149900003],[-7.316636,38.43987649800005],[-7.255615,38.72397599800007],[-7.203135,38.75101749700008],[-7.039417499999956,38.86484549700003],[-7.052114,38.90723399700005],[-6.951391499999943,39.02407049800007],[-7.02995,39.11653149600005],[-7.143882,39.10932149800004],[-7.136354499999925,39.17224899800004],[-7.24126,39.20853049800007],[-7.231467,39.27843099900008],[-7.118947499999933,39.38372049700007],[-7.07395,39.381282998000074],[-7.081431,39.313300996000066],[-7.044203499999981,39.32698449800006],[-7.00934,39.38854599800004],[-7.049458499999957,39.436732997000036],[-6.804478,39.390453498000056],[-6.746818,39.32181199900003],[-6.800934,39.23947949800004],[-6.767576499999961,39.18843449800005],[-6.324184,39.18648899800007],[-6.312005,39.11220149800005],[-6.142163499999981,39.03163899900005],[-6.01487949999995,39.12935249800006],[-6.009812499999953,39.05264649800006],[-5.810424,39.14463399700003],[-5.691497500999958,39.083729497000036],[-5.603874,39.10064699800006],[-5.576616,39.15095899900007],[-5.613298499999928,39.188266498000075],[-5.556734000999938,39.208732498000074],[-5.413859,39.15670399900006],[-5.347203499999978,39.23921199600005],[-5.340058,39.33335849800005],[-5.247668499999975,39.30187250000006],[-5.122153,39.337714998000024],[-5.113629499999945,39.38751199700005],[-4.952554499999962,39.395038498000076],[-4.940783499999952,39.39516450000008],[-4.810473499999944,39.39661399800008],[-4.755411,39.415759997000066],[-4.687733,39.45033649800007],[-4.710084499999937,39.34579449700004],[-4.761050499999953,39.32083499700008],[-4.649006,39.16494749800006],[-4.810313,39.19946299700007],[-4.874463499999933,39.099064],[-4.832323,39.05154799600007],[-4.962548,39.05877699700005],[-4.918264,38.95709599800006],[-4.831171000999973,38.93566149900005],[-4.847362499999974,38.88244999800003],[-4.943528499999957,38.854427497000074],[-4.990040499999964,38.73980349900006],[-5.046996000999968,38.72913349700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES432","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Cáceres","NUTS_NAME":"Cáceres"},"geometry":{"type":"Polygon","coordinates":[[[-7.534902449999947,39.66198689000004],[-7.015405,39.670856498000035],[-6.986185499999976,39.81074499700003],[-6.903161499999953,39.87268849800006],[-6.864203,40.011867497000026],[-7.01196049999993,40.12693399900007],[-7.012223,40.22654349700008],[-6.95129849999995,40.25744599800004],[-6.865144,40.27069449800007],[-6.587075,40.270774997000046],[-6.536334499999953,40.347293998000055],[-6.239977499999952,40.485900998000034],[-6.065001499999937,40.396358],[-6.105761,40.35683850000004],[-5.992191499999933,40.299053499000024],[-5.920884,40.28094499600007],[-5.797554499999933,40.353286497000056],[-5.793185,40.28674699900006],[-5.73760249999998,40.29416299800005],[-5.531008499999928,40.19475949800005],[-5.345126499999935,40.26498399800005],[-5.36803649999996,40.164146498000036],[-5.336016499999971,40.11585599800003],[-5.406153,39.87777299800007],[-5.284090499999934,39.86391849800003],[-5.308869,39.760432998000056],[-5.166938499999958,39.79992649700006],[-5.137835499999937,39.71504199900005],[-5.206670499999973,39.59727449800005],[-4.952554499999962,39.395038498000076],[-5.113629499999945,39.38751199700005],[-5.122153,39.337714998000024],[-5.247668499999975,39.30187250000006],[-5.340058,39.33335849800005],[-5.347203499999978,39.23921199600005],[-5.413859,39.15670399900006],[-5.556734000999938,39.208732498000074],[-5.613298499999928,39.188266498000075],[-5.576616,39.15095899900007],[-5.603874,39.10064699800006],[-5.691497500999958,39.083729497000036],[-5.810424,39.14463399700003],[-6.009812499999953,39.05264649800006],[-6.01487949999995,39.12935249800006],[-6.142163499999981,39.03163899900005],[-6.312005,39.11220149800005],[-6.324184,39.18648899800007],[-6.767576499999961,39.18843449800005],[-6.800934,39.23947949800004],[-6.746818,39.32181199900003],[-6.804478,39.390453498000056],[-7.049458499999957,39.436732997000036],[-7.00934,39.38854599800004],[-7.044203499999981,39.32698449800006],[-7.081431,39.313300996000066],[-7.07395,39.381282998000074],[-7.118947499999933,39.38372049700007],[-7.231467,39.27843099900008],[-7.310608,39.34102249800003],[-7.294676,39.455993499000044],[-7.534902449999947,39.66198689000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES5","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"ESTE","NUTS_NAME":"ESTE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.31011,39.87821599800003],[4.28757,39.814689498000064],[4.02217550000006,39.92149749600003],[3.823853500000041,39.921970498000064],[3.791304,40.016177998000046],[4.094098,40.05835349800003],[4.264508500000034,39.980529998000065],[4.31011,39.87821599800003]]],[[[3.079701,39.901542498000026],[3.109828500000049,39.85826099900004],[3.189126500000043,39.88608549700007],[3.119941999000048,39.83151649700005],[3.144903500000055,39.77384950000004],[3.254287,39.729602999000065],[3.348478500000056,39.788887],[3.43512,39.74422449800005],[3.455952500000024,39.65670399700008],[3.301008,39.50396349700003],[3.229418500000065,39.35907349800004],[3.073082500000055,39.268691998000065],[2.956790500000068,39.362800496000034],[2.787235,39.36309449600003],[2.721845500000029,39.47583799700004],[2.751244500000041,39.503772998000045],[2.689343999000073,39.552272997000046],[2.538242500000024,39.52297599900004],[2.522538500000053,39.45799999600007],[2.344436,39.58722699700007],[2.77852050000007,39.85567899800003],[3.213000500000021,39.95537599600004],[3.079701,39.901542498000026]]],[[[1.078708,42.788341496000044],[1.17077,42.70814149800003],[1.357304,42.71942199800003],[1.442566,42.603668],[1.413773,42.53564449700008],[1.468273,42.51002099800007],[1.455091,42.43606199800007],[1.725801,42.50440199800005],[1.731011,42.49240099800005],[1.932950500000061,42.45493699800005],[1.964552,42.38215699800003],[2.012462,42.35175299800005],[2.306832499000052,42.42904299600008],[2.526214500000037,42.33339299700003],[2.675338500000066,42.34233849800006],[2.673950500000046,42.40464399900003],[2.839501,42.45927099800008],[3.174803999000062,42.43537499900003],[3.163047,42.36050799800006],[3.322315500000059,42.31938149700005],[3.266109500000027,42.23871599700004],[3.161765,42.26394249900005],[3.111661500000025,42.20384949700008],[3.124232,42.12801749800008],[3.213827,42.061301997000044],[3.21998050000002,41.92232149800003],[2.778513,41.648807497000064],[2.270422,41.46080049700004],[2.119573,41.29206099800007],[1.645323500000075,41.19562149600006],[0.966513,41.02842349800005],[0.699735499000042,40.803840499000046],[0.876583500000038,40.70131649700005],[0.721663985000021,40.60513691300008],[0.746041500000047,40.636180997000054],[0.602681,40.62206249800005],[0.515238500000066,40.522918497000035],[-0.188467,39.72195449700007],[-0.334561,39.42353449800004],[-0.195017499999949,39.05113249800007],[0.012785310000027,38.86459284600005],[0.197371,38.804336498000055],[0.205479500000024,38.731944998000074],[-0.027940499999943,38.61920149700006],[-0.097914499999945,38.523395497000024],[-0.301367,38.481929498000056],[-0.409126,38.35868849800005],[-0.50490949999994,38.333378],[-0.50988,38.20200349700008],[-0.612482499999942,38.175410998000075],[-0.647937499999955,37.99972899800008],[-0.762135499999943,37.84700799700005],[-0.900112,37.923125998000046],[-1.030276499999957,38.097076497000046],[-0.96761749999996,38.27564249900007],[-1.084330499999965,38.346217996000064],[-1.094109499999945,38.43793099800007],[-1.013821499999949,38.49706249800005],[-1.02687,38.655509497000025],[-0.91573649999998,38.695968497000024],[-0.961375499999974,38.77479549700007],[-0.928884499999924,38.783839998000076],[-0.959359,38.94458749700004],[-1.146609,38.929362998000045],[-1.266710499999931,39.05103249900003],[-1.161849,39.305431498000075],[-1.505146499999967,39.41801449700006],[-1.504335,39.56391899700003],[-1.368502499999977,39.68963249600006],[-1.279547499999978,39.67790199600006],[-1.200163,39.858135496000045],[-1.20815349999998,39.943820998000035],[-1.142361499999936,39.971855498000025],[-0.920412,39.96441649600007],[-0.912739,39.87310750000006],[-0.797643,39.88107649800003],[-0.837749500999962,39.97681799600008],[-0.624734,40.07609949700003],[-0.544136,40.251666998000076],[-0.388490499999932,40.261615998000025],[-0.279997499999979,40.36949949800004],[-0.347301,40.44394349700008],[-0.271374500999968,40.47850049800007],[-0.293232499999931,40.611045999000055],[-0.370432499999936,40.613597997000056],[-0.381465499999933,40.66158699700003],[-0.235937499999977,40.69111249800005],[-0.197124499999973,40.78445799800005],[0.042237,40.69100549800004],[0.170789500000069,40.73283699900003],[0.278288500000031,40.821033499000066],[0.238971,40.88470449700003],[0.293564500000059,40.970562],[0.220409500000073,41.07142999900003],[0.200918,41.125358497000036],[0.303379500000062,41.16437549800003],[0.385724,41.278839497000035],[0.335948500000029,41.40743249800005],[0.341453499000067,41.480896],[0.446238,41.542907497000044],[0.428549,41.602367498000035],[0.349668500000064,41.59945699700006],[0.327072,41.678157999000064],[0.580113500000039,41.86344199800004],[0.607546,41.91929999800004],[0.561936500000058,41.93820199800007],[0.685666500000025,42.09275049800004],[0.767070500000045,42.34881599700003],[0.690539500000057,42.48760199900005],[0.767896,42.60979449600006],[0.660127,42.69095249800006],[0.66215950000003,42.84124349700005],[0.858215,42.82574099800007],[1.078708,42.788341496000044]]],[[[1.423349,38.90504449800005],[1.371127500000057,38.83033349700003],[1.21306450000003,38.90135949900008],[1.310512500000073,39.04295349800003],[1.603850500000021,39.09101849700005],[1.605787500000019,39.02961349800006],[1.423349,38.90504449800005]]],[[[1.451187499000071,38.730296999000075],[1.527470550000032,38.67332874900006],[1.569296050000048,38.691528249000044],[1.584409,38.66284149900008],[1.523969550000061,38.65303774900008],[1.447129500000074,38.68503149900005],[1.385188,38.64133849900003],[1.386598500000048,38.72178249900003],[1.433151550000048,38.77172474900004],[1.451187499000071,38.730296999000075]]],[[[-1.252389000999926,40.14331449800005],[-1.072093499999937,40.060496998000076],[-1.165154,40.01010899700003],[-1.376832,40.01651349800005],[-1.448831499999926,40.14535849600003],[-1.357073,40.12890249900005],[-1.300741,40.20640199700006],[-1.252389000999926,40.14331449800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES51","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Cataluña","NUTS_NAME":"Cataluña"},"geometry":{"type":"Polygon","coordinates":[[[1.078708,42.788341496000044],[1.17077,42.70814149800003],[1.357304,42.71942199800003],[1.442566,42.603668],[1.413773,42.53564449700008],[1.468273,42.51002099800007],[1.455091,42.43606199800007],[1.725801,42.50440199800005],[1.731011,42.49240099800005],[1.932950500000061,42.45493699800005],[1.964552,42.38215699800003],[2.012462,42.35175299800005],[2.306832499000052,42.42904299600008],[2.526214500000037,42.33339299700003],[2.675338500000066,42.34233849800006],[2.673950500000046,42.40464399900003],[2.839501,42.45927099800008],[3.174803999000062,42.43537499900003],[3.163047,42.36050799800006],[3.322315500000059,42.31938149700005],[3.266109500000027,42.23871599700004],[3.161765,42.26394249900005],[3.111661500000025,42.20384949700008],[3.124232,42.12801749800008],[3.213827,42.061301997000044],[3.21998050000002,41.92232149800003],[2.778513,41.648807497000064],[2.270422,41.46080049700004],[2.119573,41.29206099800007],[1.645323500000075,41.19562149600006],[0.966513,41.02842349800005],[0.699735499000042,40.803840499000046],[0.876583500000038,40.70131649700005],[0.721663985000021,40.60513691300008],[0.746041500000047,40.636180997000054],[0.602681,40.62206249800005],[0.515238500000066,40.522918497000035],[0.170789500000069,40.73283699900003],[0.278288500000031,40.821033499000066],[0.238971,40.88470449700003],[0.293564500000059,40.970562],[0.220409500000073,41.07142999900003],[0.200918,41.125358497000036],[0.303379500000062,41.16437549800003],[0.385724,41.278839497000035],[0.335948500000029,41.40743249800005],[0.341453499000067,41.480896],[0.446238,41.542907497000044],[0.428549,41.602367498000035],[0.349668500000064,41.59945699700006],[0.327072,41.678157999000064],[0.580113500000039,41.86344199800004],[0.607546,41.91929999800004],[0.561936500000058,41.93820199800007],[0.685666500000025,42.09275049800004],[0.767070500000045,42.34881599700003],[0.690539500000057,42.48760199900005],[0.767896,42.60979449600006],[0.660127,42.69095249800006],[0.66215950000003,42.84124349700005],[0.858215,42.82574099800007],[1.078708,42.788341496000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES511","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Barcelona","NUTS_NAME":"Barcelona"},"geometry":{"type":"Polygon","coordinates":[[[2.778513,41.648807497000064],[2.270422,41.46080049700004],[2.119573,41.29206099800007],[1.645323500000075,41.19562149600006],[1.578886500000067,41.35633850000005],[1.39644,41.51288199700008],[1.454576,41.56917199900005],[1.392940500000066,41.57508099700004],[1.359712,41.61352150000005],[1.450236,41.64654949700008],[1.382012,41.705737999000064],[1.417096500000071,41.79461299700006],[1.530819,41.75254799700008],[1.631552500000055,41.84469599600004],[1.586972,41.88854199800005],[1.682840499000065,41.96736149900005],[1.647949,42.00013349900007],[1.734415,42.15934],[1.680501500000048,42.20631399600006],[1.690231500000039,42.27708049800003],[1.842707,42.306377498000074],[2.069994999000073,42.29156899700007],[2.016809500000022,42.232684998000025],[2.060614500000042,42.17786849700008],[1.965309,42.199062500000025],[2.008132499000055,42.14973849800003],[2.270128500000055,42.14518749700005],[2.490463500000033,42.06729900100004],[2.480389500000058,41.92285549800005],[2.33764,41.889141],[2.344198500000061,41.82081599700007],[2.547438,41.719047498000066],[2.77699750000005,41.73546999700005],[2.778513,41.648807497000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES512","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Girona","NUTS_NAME":"Girona"},"geometry":{"type":"Polygon","coordinates":[[[1.964552,42.38215699800003],[2.012462,42.35175299800005],[2.306832499000052,42.42904299600008],[2.526214500000037,42.33339299700003],[2.675338500000066,42.34233849800006],[2.673950500000046,42.40464399900003],[2.839501,42.45927099800008],[3.174803999000062,42.43537499900003],[3.163047,42.36050799800006],[3.322315500000059,42.31938149700005],[3.266109500000027,42.23871599700004],[3.161765,42.26394249900005],[3.111661500000025,42.20384949700008],[3.124232,42.12801749800008],[3.213827,42.061301997000044],[3.21998050000002,41.92232149800003],[2.778513,41.648807497000064],[2.77699750000005,41.73546999700005],[2.547438,41.719047498000066],[2.344198500000061,41.82081599700007],[2.33764,41.889141],[2.480389500000058,41.92285549800005],[2.490463500000033,42.06729900100004],[2.270128500000055,42.14518749700005],[2.008132499000055,42.14973849800003],[1.965309,42.199062500000025],[2.060614500000042,42.17786849700008],[2.016809500000022,42.232684998000025],[2.069994999000073,42.29156899700007],[1.842707,42.306377498000074],[1.731011,42.49240099800005],[1.932950500000061,42.45493699800005],[1.964552,42.38215699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES513","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Lleida","NUTS_NAME":"Lleida"},"geometry":{"type":"Polygon","coordinates":[[[0.858215,42.82574099800007],[1.078708,42.788341496000044],[1.17077,42.70814149800003],[1.357304,42.71942199800003],[1.442566,42.603668],[1.413773,42.53564449700008],[1.468273,42.51002099800007],[1.455091,42.43606199800007],[1.725801,42.50440199800005],[1.731011,42.49240099800005],[1.842707,42.306377498000074],[1.690231500000039,42.27708049800003],[1.680501500000048,42.20631399600006],[1.734415,42.15934],[1.647949,42.00013349900007],[1.682840499000065,41.96736149900005],[1.586972,41.88854199800005],[1.631552500000055,41.84469599600004],[1.530819,41.75254799700008],[1.417096500000071,41.79461299700006],[1.382012,41.705737999000064],[1.450236,41.64654949700008],[1.359712,41.61352150000005],[1.392940500000066,41.57508099700004],[1.219950500000039,41.56956849800008],[1.168002,41.54045899800008],[1.181035,41.48810199800005],[1.108228500000052,41.466812497000035],[1.03733,41.498222499000065],[1.075205500000038,41.459896],[0.951892,41.35180649800003],[0.618808,41.29093549900006],[0.461325,41.3349],[0.385724,41.278839497000035],[0.335948500000029,41.40743249800005],[0.341453499000067,41.480896],[0.446238,41.542907497000044],[0.428549,41.602367498000035],[0.349668500000064,41.59945699700006],[0.327072,41.678157999000064],[0.580113500000039,41.86344199800004],[0.607546,41.91929999800004],[0.561936500000058,41.93820199800007],[0.685666500000025,42.09275049800004],[0.767070500000045,42.34881599700003],[0.690539500000057,42.48760199900005],[0.767896,42.60979449600006],[0.660127,42.69095249800006],[0.66215950000003,42.84124349700005],[0.858215,42.82574099800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES514","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Tarragona","NUTS_NAME":"Tarragona"},"geometry":{"type":"Polygon","coordinates":[[[1.454576,41.56917199900005],[1.39644,41.51288199700008],[1.578886500000067,41.35633850000005],[1.645323500000075,41.19562149600006],[0.966513,41.02842349800005],[0.699735499000042,40.803840499000046],[0.876583500000038,40.70131649700005],[0.721663985000021,40.60513691300008],[0.746041500000047,40.636180997000054],[0.602681,40.62206249800005],[0.515238500000066,40.522918497000035],[0.170789500000069,40.73283699900003],[0.278288500000031,40.821033499000066],[0.238971,40.88470449700003],[0.293564500000059,40.970562],[0.220409500000073,41.07142999900003],[0.200918,41.125358497000036],[0.303379500000062,41.16437549800003],[0.385724,41.278839497000035],[0.461325,41.3349],[0.618808,41.29093549900006],[0.951892,41.35180649800003],[1.075205500000038,41.459896],[1.03733,41.498222499000065],[1.108228500000052,41.466812497000035],[1.181035,41.48810199800005],[1.168002,41.54045899800008],[1.219950500000039,41.56956849800008],[1.392940500000066,41.57508099700004],[1.454576,41.56917199900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES52","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Comunidad Valenciana","NUTS_NAME":"Comunidad Valenciana"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.197124499999973,40.78445799800005],[0.042237,40.69100549800004],[0.170789500000069,40.73283699900003],[0.515238500000066,40.522918497000035],[-0.188467,39.72195449700007],[-0.334561,39.42353449800004],[-0.195017499999949,39.05113249800007],[0.012785310000027,38.86459284600005],[0.197371,38.804336498000055],[0.205479500000024,38.731944998000074],[-0.027940499999943,38.61920149700006],[-0.097914499999945,38.523395497000024],[-0.301367,38.481929498000056],[-0.409126,38.35868849800005],[-0.50490949999994,38.333378],[-0.50988,38.20200349700008],[-0.612482499999942,38.175410998000075],[-0.647937499999955,37.99972899800008],[-0.762135499999943,37.84700799700005],[-0.900112,37.923125998000046],[-1.030276499999957,38.097076497000046],[-0.96761749999996,38.27564249900007],[-1.084330499999965,38.346217996000064],[-1.094109499999945,38.43793099800007],[-1.013821499999949,38.49706249800005],[-1.02687,38.655509497000025],[-0.91573649999998,38.695968497000024],[-0.961375499999974,38.77479549700007],[-0.928884499999924,38.783839998000076],[-0.959359,38.94458749700004],[-1.146609,38.929362998000045],[-1.266710499999931,39.05103249900003],[-1.161849,39.305431498000075],[-1.505146499999967,39.41801449700006],[-1.504335,39.56391899700003],[-1.368502499999977,39.68963249600006],[-1.279547499999978,39.67790199600006],[-1.200163,39.858135496000045],[-1.20815349999998,39.943820998000035],[-1.142361499999936,39.971855498000025],[-0.920412,39.96441649600007],[-0.912739,39.87310750000006],[-0.797643,39.88107649800003],[-0.837749500999962,39.97681799600008],[-0.624734,40.07609949700003],[-0.544136,40.251666998000076],[-0.388490499999932,40.261615998000025],[-0.279997499999979,40.36949949800004],[-0.347301,40.44394349700008],[-0.271374500999968,40.47850049800007],[-0.293232499999931,40.611045999000055],[-0.370432499999936,40.613597997000056],[-0.381465499999933,40.66158699700003],[-0.235937499999977,40.69111249800005],[-0.197124499999973,40.78445799800005]]],[[[-1.252389000999926,40.14331449800005],[-1.072093499999937,40.060496998000076],[-1.165154,40.01010899700003],[-1.376832,40.01651349800005],[-1.448831499999926,40.14535849600003],[-1.357073,40.12890249900005],[-1.300741,40.20640199700006],[-1.252389000999926,40.14331449800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES521","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Alicante / Alacant","NUTS_NAME":"Alicante / Alacant"},"geometry":{"type":"Polygon","coordinates":[[[0.012785310000027,38.86459284600005],[0.197371,38.804336498000055],[0.205479500000024,38.731944998000074],[-0.027940499999943,38.61920149700006],[-0.097914499999945,38.523395497000024],[-0.301367,38.481929498000056],[-0.409126,38.35868849800005],[-0.50490949999994,38.333378],[-0.50988,38.20200349700008],[-0.612482499999942,38.175410998000075],[-0.647937499999955,37.99972899800008],[-0.762135499999943,37.84700799700005],[-0.900112,37.923125998000046],[-1.030276499999957,38.097076497000046],[-0.96761749999996,38.27564249900007],[-1.084330499999965,38.346217996000064],[-1.094109499999945,38.43793099800007],[-1.013821499999949,38.49706249800005],[-1.02687,38.655509497000025],[-0.91573649999998,38.695968497000024],[-0.961375499999974,38.77479549700007],[-0.928884499999924,38.783839998000076],[-0.61798749999997,38.68665299700007],[-0.507049499999937,38.742973498000026],[-0.595154,38.795100998000066],[-0.310736,38.881511500000045],[0.012785310000027,38.86459284600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES522","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Castellón / Castelló","NUTS_NAME":"Castellón / Castelló"},"geometry":{"type":"Polygon","coordinates":[[[0.170789500000069,40.73283699900003],[0.515238500000066,40.522918497000035],[-0.188467,39.72195449700007],[-0.328195,39.80148699800003],[-0.46094549999998,39.71535849800006],[-0.530530499999941,39.79598249900005],[-0.630954,39.74634099900004],[-0.656098499999928,39.83596799800006],[-0.730404000999954,39.81483449600006],[-0.797643,39.88107649800003],[-0.837749500999962,39.97681799600008],[-0.624734,40.07609949700003],[-0.544136,40.251666998000076],[-0.388490499999932,40.261615998000025],[-0.279997499999979,40.36949949800004],[-0.347301,40.44394349700008],[-0.271374500999968,40.47850049800007],[-0.293232499999931,40.611045999000055],[-0.370432499999936,40.613597997000056],[-0.381465499999933,40.66158699700003],[-0.235937499999977,40.69111249800005],[-0.197124499999973,40.78445799800005],[0.042237,40.69100549800004],[0.170789500000069,40.73283699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES523","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Valencia / València","NUTS_NAME":"Valencia / València"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.920412,39.96441649600007],[-0.912739,39.87310750000006],[-0.797643,39.88107649800003],[-0.730404000999954,39.81483449600006],[-0.656098499999928,39.83596799800006],[-0.630954,39.74634099900004],[-0.530530499999941,39.79598249900005],[-0.46094549999998,39.71535849800006],[-0.328195,39.80148699800003],[-0.188467,39.72195449700007],[-0.334561,39.42353449800004],[-0.195017499999949,39.05113249800007],[0.012785310000027,38.86459284600005],[-0.310736,38.881511500000045],[-0.595154,38.795100998000066],[-0.507049499999937,38.742973498000026],[-0.61798749999997,38.68665299700007],[-0.928884499999924,38.783839998000076],[-0.959359,38.94458749700004],[-1.146609,38.929362998000045],[-1.266710499999931,39.05103249900003],[-1.161849,39.305431498000075],[-1.505146499999967,39.41801449700006],[-1.504335,39.56391899700003],[-1.368502499999977,39.68963249600006],[-1.279547499999978,39.67790199600006],[-1.200163,39.858135496000045],[-1.20815349999998,39.943820998000035],[-1.142361499999936,39.971855498000025],[-0.920412,39.96441649600007]]],[[[-1.252389000999926,40.14331449800005],[-1.072093499999937,40.060496998000076],[-1.165154,40.01010899700003],[-1.376832,40.01651349800005],[-1.448831499999926,40.14535849600003],[-1.357073,40.12890249900005],[-1.300741,40.20640199700006],[-1.252389000999926,40.14331449800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES53","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Illes Balears","NUTS_NAME":"Illes Balears"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.31011,39.87821599800003],[4.28757,39.814689498000064],[4.02217550000006,39.92149749600003],[3.823853500000041,39.921970498000064],[3.791304,40.016177998000046],[4.094098,40.05835349800003],[4.264508500000034,39.980529998000065],[4.31011,39.87821599800003]]],[[[3.079701,39.901542498000026],[3.109828500000049,39.85826099900004],[3.189126500000043,39.88608549700007],[3.119941999000048,39.83151649700005],[3.144903500000055,39.77384950000004],[3.254287,39.729602999000065],[3.348478500000056,39.788887],[3.43512,39.74422449800005],[3.455952500000024,39.65670399700008],[3.301008,39.50396349700003],[3.229418500000065,39.35907349800004],[3.073082500000055,39.268691998000065],[2.956790500000068,39.362800496000034],[2.787235,39.36309449600003],[2.721845500000029,39.47583799700004],[2.751244500000041,39.503772998000045],[2.689343999000073,39.552272997000046],[2.538242500000024,39.52297599900004],[2.522538500000053,39.45799999600007],[2.344436,39.58722699700007],[2.77852050000007,39.85567899800003],[3.213000500000021,39.95537599600004],[3.079701,39.901542498000026]]],[[[1.423349,38.90504449800005],[1.371127500000057,38.83033349700003],[1.21306450000003,38.90135949900008],[1.310512500000073,39.04295349800003],[1.603850500000021,39.09101849700005],[1.605787500000019,39.02961349800006],[1.423349,38.90504449800005]]],[[[1.451187499000071,38.730296999000075],[1.527470550000032,38.67332874900006],[1.569296050000048,38.691528249000044],[1.584409,38.66284149900008],[1.523969550000061,38.65303774900008],[1.447129500000074,38.68503149900005],[1.385188,38.64133849900003],[1.386598500000048,38.72178249900003],[1.433151550000048,38.77172474900004],[1.451187499000071,38.730296999000075]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES531","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Eivissa y Formentera","NUTS_NAME":"Eivissa y Formentera"},"geometry":{"type":"MultiPolygon","coordinates":[[[[1.423349,38.90504449800005],[1.371127500000057,38.83033349700003],[1.21306450000003,38.90135949900008],[1.310512500000073,39.04295349800003],[1.603850500000021,39.09101849700005],[1.605787500000019,39.02961349800006],[1.423349,38.90504449800005]]],[[[1.451187499000071,38.730296999000075],[1.527470550000032,38.67332874900006],[1.569296050000048,38.691528249000044],[1.584409,38.66284149900008],[1.523969550000061,38.65303774900008],[1.447129500000074,38.68503149900005],[1.385188,38.64133849900003],[1.386598500000048,38.72178249900003],[1.433151550000048,38.77172474900004],[1.451187499000071,38.730296999000075]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES532","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Mallorca","NUTS_NAME":"Mallorca"},"geometry":{"type":"Polygon","coordinates":[[[3.079701,39.901542498000026],[3.109828500000049,39.85826099900004],[3.189126500000043,39.88608549700007],[3.119941999000048,39.83151649700005],[3.144903500000055,39.77384950000004],[3.254287,39.729602999000065],[3.348478500000056,39.788887],[3.43512,39.74422449800005],[3.455952500000024,39.65670399700008],[3.301008,39.50396349700003],[3.229418500000065,39.35907349800004],[3.073082500000055,39.268691998000065],[2.956790500000068,39.362800496000034],[2.787235,39.36309449600003],[2.721845500000029,39.47583799700004],[2.751244500000041,39.503772998000045],[2.689343999000073,39.552272997000046],[2.538242500000024,39.52297599900004],[2.522538500000053,39.45799999600007],[2.344436,39.58722699700007],[2.77852050000007,39.85567899800003],[3.213000500000021,39.95537599600004],[3.079701,39.901542498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES533","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Menorca","NUTS_NAME":"Menorca"},"geometry":{"type":"Polygon","coordinates":[[[4.094098,40.05835349800003],[4.264508500000034,39.980529998000065],[4.31011,39.87821599800003],[4.28757,39.814689498000064],[4.02217550000006,39.92149749600003],[3.823853500000041,39.921970498000064],[3.791304,40.016177998000046],[4.094098,40.05835349800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES6","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"SUR","NUTS_NAME":"SUR"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-1.084330499999965,38.346217996000064],[-0.96761749999996,38.27564249900007],[-1.030276499999957,38.097076497000046],[-0.900112,37.923125998000046],[-0.762135499999943,37.84700799700005],[-0.751674499999979,37.794719497000074],[-0.784334,37.81915649900003],[-0.859979499999952,37.721446997000044],[-0.733145,37.63270199700003],[-0.738589693999927,37.69975169400004],[-0.720821,37.60587299700006],[-0.920817499999941,37.55564099800006],[-1.074336,37.58328599600003],[-1.111013,37.53418349800006],[-1.325323499999968,37.56244249800005],[-1.630033,37.37518299800007],[-1.806549,37.214004497000076],[-1.903591,36.937160498000026],[-2.12434,36.731842],[-2.192171000999963,36.72158499900007],[-2.368091499999935,36.84145749800007],[-2.562035,36.81660099700008],[-2.700591,36.68272399700004],[-3.128691,36.75088799800005],[-3.420361,36.694095498000024],[-3.777457499999969,36.737926499000025],[-4.418207,36.717219998000076],[-4.638161499999967,36.50709149800008],[-4.936012499999947,36.499389499000074],[-5.176345499999968,36.414661498000044],[-5.252405,36.31127549800004],[-5.339225,36.152034997000044],[-5.35152249999993,36.15256899800005],[-5.424657499999967,36.17650199800005],[-5.430406,36.071304499000064],[-5.60843,36.007053496000026],[-5.797297499999956,36.07829699800004],[-5.914955499999962,36.18479549800003],[-6.040427499999964,36.19237899800004],[-6.308641499999965,36.532573497000044],[-6.229004,36.462669497000036],[-6.173302000999968,36.521442498000056],[-6.245686,36.50899499800005],[-6.232875,36.584945497000035],[-6.387676,36.62863149800006],[-6.440116,36.72086349700004],[-6.345561,36.79876699700003],[-6.514539499999955,36.97180549800004],[-6.893693499999927,37.16146499900003],[-7.051858,37.20842349800006],[-7.40191649999997,37.174827497000024],[-7.463588499999958,37.485454498000024],[-7.51269149999996,37.526256498000066],[-7.418441,37.74499499800004],[-7.279641,37.876762496000026],[-7.263284499999941,37.979907999000034],[-7.108363,38.040984998000056],[-7.002483499999926,38.02271649900007],[-6.931738499999938,38.20837799700007],[-6.795174499999973,38.17859649800005],[-6.80790449999995,38.11116399700006],[-6.622533,38.09721399700004],[-6.495118,38.01918399700003],[-6.36732,38.04896899700003],[-6.180307499999969,37.941077997000036],[-5.953754,37.99539549900004],[-5.874359,38.158622997000066],[-5.723288499999967,38.19248949800004],[-5.684302499999944,38.15711199800006],[-5.740212,38.13013049800003],[-5.734962499999938,38.086303497000074],[-5.584845499999972,38.13175199900007],[-5.521041499999967,38.20727149800007],[-5.56897749999996,38.432639997000024],[-5.185018,38.65946599700004],[-5.166655,38.71171549600007],[-5.046996000999968,38.72913349700008],[-5.008071499999971,38.71527499800004],[-4.877013499999975,38.68614199800004],[-4.860213499999929,38.61402499800005],[-4.705425499999933,38.57075099600007],[-4.453019,38.40091699900006],[-4.268895,38.34721299700004],[-4.253038499999946,38.40123],[-3.620483,38.39401999700004],[-3.583084,38.45150349800008],[-3.453166499999952,38.400233996000054],[-3.33534949999995,38.480529999000055],[-3.130193,38.438796997000054],[-3.062778,38.47640249800003],[-3.002507499999979,38.40828699700006],[-2.992628,38.450976997000055],[-2.762069,38.532779498000025],[-2.604288,38.51290899800006],[-2.566616,38.490489997000054],[-2.571873,38.41415049600005],[-2.481576,38.39310099900007],[-2.446249499999965,38.18521099700007],[-2.551274000999967,38.08411799900006],[-2.341602,38.02601999800004],[-2.21349349999997,38.201820498000075],[-2.048963499999957,38.30579399900006],[-1.907508,38.297969497000054],[-1.741687499999955,38.38049699700008],[-1.589503,38.31000149700003],[-1.478507,38.37638849900003],[-1.498696,38.53206249900006],[-1.402179499999932,38.690803499000026],[-1.119623499999932,38.737724497000045],[-1.02687,38.655509497000025],[-1.013821499999949,38.49706249800005],[-1.094109499999945,38.43793099800007],[-1.084330499999965,38.346217996000064]]],[[[-2.950587,35.31841799800003],[-2.92736,35.27426399900003],[-2.970093499999962,35.289367499000036],[-2.950587,35.31841799800003]]],[[[-5.350729000999934,35.90900299700007],[-5.342521175999934,35.87303803900005],[-5.382034499999975,35.91260649900005],[-5.350729000999934,35.90900299700007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL623","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ithaki, Kefallinia","NUTS_NAME":"Ιθάκη, Κεφαλληνία"},"geometry":{"type":"MultiPolygon","coordinates":[[[[20.628674,38.32527149600003],[20.601241999000024,38.30007949700007],[20.643795500000067,38.251945498000055],[20.680998500000044,38.27783199700008],[20.81529050000006,38.11352899700006],[20.796285500000067,38.065116998000065],[20.514303,38.103931499000055],[20.444082,38.28076949800004],[20.440220500000066,38.161388498000065],[20.34178550000007,38.17613199600004],[20.407196,38.34989549800008],[20.477171,38.309573998000076],[20.547944999000038,38.36228549700007],[20.536981500000024,38.46981049800007],[20.582451,38.46472149900006],[20.628674,38.32527149600003]]],[[[20.726379697000027,38.30626380100006],[20.723535500000025,38.30091499900004],[20.681861397000034,38.364154245000066],[20.681703518000063,38.36439382200007],[20.67858482300005,38.369126351000034],[20.676870332000078,38.37172804100004],[20.67638760500006,38.37246056500004],[20.674318576000076,38.37560025700003],[20.642205783000065,38.424330492000024],[20.640100338000025,38.427525443000036],[20.63224221300004,38.43944992100006],[20.627883214000065,38.446064576000026],[20.621872360000054,38.45518587400005],[20.621309320000023,38.456040270000074],[20.61507050000006,38.46550749900007],[20.627280287000076,38.47754448400008],[20.627569132000076,38.47782924100005],[20.627646765000065,38.47790577500007],[20.647870010000076,38.49784280500006],[20.64853775800003,38.49850110300008],[20.650018871000043,38.49996125400003],[20.650058971000078,38.50000078700003],[20.65047734600006,38.50041324000006],[20.651176500000076,38.50110249900007],[20.672380662000023,38.473754229000065],[20.672496657000067,38.47360462300003],[20.687026349000064,38.45486481300003],[20.687279871000044,38.45453783100004],[20.690643067000053,38.450200116000076],[20.693599595000023,38.446386905000054],[20.70467900400007,38.432097132000024],[20.705893076000052,38.43053126900003],[20.746239073000027,38.378494630000034],[20.746346508000045,38.37835606400006],[20.746497791000024,38.37816094600004],[20.756654337000043,38.36506144300006],[20.757242,38.36430349900007],[20.743891635000068,38.339196779000076],[20.743751848000045,38.33893389600007],[20.726379697000027,38.30626380100006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL624","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Lefkada","NUTS_NAME":"Λευκάδα"},"geometry":{"type":"Polygon","coordinates":[[[20.721731,38.626224498000056],[20.551553500000068,38.577827497000044],[20.59967050000006,38.779578997000044],[20.693756,38.85181799700007],[20.721731,38.626224498000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL63","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Dytiki Ellada","NUTS_NAME":"Δυτική Ελλάδα"},"geometry":{"type":"MultiPolygon","coordinates":[[[[21.871086,38.33453749700004],[22.37297250000006,38.14222349700003],[22.362901500000078,38.022987998000076],[22.221796,37.88794699700003],[22.26432,37.83939049800006],[22.024662,37.788459998000064],[21.89015549900006,37.86580299800005],[21.794531,37.79080199800006],[21.792816,37.602542999000036],[21.87857450000007,37.60453399900007],[21.99938750000007,37.49891299800004],[21.96572850000007,37.48326099700006],[21.88610650000004,37.454856997000036],[21.89699950000005,37.40159599800006],[21.680233,37.37743399800007],[21.530187500000068,37.56888199800005],[21.32807150000008,37.66140349700004],[21.28442,37.778659997000034],[21.105238,37.848926498000026],[21.126194,37.93867499600003],[21.255453,37.97354499900007],[21.351074,38.102138498000045],[21.372593,38.218120498000076],[21.605222500000025,38.142891],[21.778976500000056,38.30762499800005],[21.871086,38.33453749700004]]],[[[21.396904,39.16447849600007],[21.436165,39.10585399800004],[21.374632500000075,39.01817299900006],[21.490255500000046,38.97023],[21.507583500000067,38.889121997000075],[21.637462,38.870811497000034],[21.57425499900006,38.848701498000025],[21.571245,38.73678599800007],[21.776687500000037,38.67531949800008],[21.830959500000063,38.737224498000046],[21.944411500000058,38.74248849800006],[21.946253,38.78343199700004],[21.999378,38.76886749600004],[21.999883500000067,38.508353998000075],[21.879766500000073,38.46499249800007],[21.851717,38.37451949700005],[21.486598730000026,38.301853583000025],[21.486297500000035,38.32768649800005],[21.484744472000045,38.306621567000036],[21.318926,38.49962999600007],[21.335010500000067,38.388656499000035],[21.14287950000005,38.303787499000066],[21.10001550000004,38.33328649900005],[21.146599,38.41347499800003],[21.07562450000006,38.48397799800006],[21.096044500000062,38.53156649700003],[21.02672,38.503150997000034],[20.988678,38.66868599800006],[20.914373500000067,38.67372899800006],[20.871914,38.794937498000024],[20.780818999000076,38.75421149700003],[20.726846500000022,38.814430498000036],[20.76258850000005,38.862751],[20.74679,38.922439498000074],[20.764536,38.95730949800003],[20.810058500000025,38.91819749900003],[20.978819,38.945262498000034],[21.066231,38.87713649800003],[21.164858,38.977615497000045],[21.10860850000006,39.04614650000008],[21.087236500000074,39.08039099800004],[21.127334500000075,39.12431699800004],[21.271143,39.117335998000044],[21.37354650000003,39.17473599700003],[21.396904,39.16447849600007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL631","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Aitoloakarnania","NUTS_NAME":"Αιτωλοακαρνανία"},"geometry":{"type":"Polygon","coordinates":[[[21.486598730000026,38.301853583000025],[21.486297500000035,38.32768649800005],[21.484744472000045,38.306621567000036],[21.318926,38.49962999600007],[21.335010500000067,38.388656499000035],[21.14287950000005,38.303787499000066],[21.10001550000004,38.33328649900005],[21.146599,38.41347499800003],[21.07562450000006,38.48397799800006],[21.096044500000062,38.53156649700003],[21.02672,38.503150997000034],[20.988678,38.66868599800006],[20.914373500000067,38.67372899800006],[20.871914,38.794937498000024],[20.780818999000076,38.75421149700003],[20.726846500000022,38.814430498000036],[20.76258850000005,38.862751],[20.74679,38.922439498000074],[20.764536,38.95730949800003],[20.810058500000025,38.91819749900003],[20.978819,38.945262498000034],[21.066231,38.87713649800003],[21.164858,38.977615497000045],[21.10860850000006,39.04614650000008],[21.087236500000074,39.08039099800004],[21.127334500000075,39.12431699800004],[21.271143,39.117335998000044],[21.37354650000003,39.17473599700003],[21.396904,39.16447849600007],[21.436165,39.10585399800004],[21.374632500000075,39.01817299900006],[21.490255500000046,38.97023],[21.507583500000067,38.889121997000075],[21.637462,38.870811497000034],[21.57425499900006,38.848701498000025],[21.571245,38.73678599800007],[21.776687500000037,38.67531949800008],[21.830959500000063,38.737224498000046],[21.944411500000058,38.74248849800006],[21.946253,38.78343199700004],[21.999378,38.76886749600004],[21.999883500000067,38.508353998000075],[21.879766500000073,38.46499249800007],[21.851717,38.37451949700005],[21.486598730000026,38.301853583000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL632","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Achaia","NUTS_NAME":"Αχαΐα"},"geometry":{"type":"Polygon","coordinates":[[[22.37297250000006,38.14222349700003],[22.362901500000078,38.022987998000076],[22.221796,37.88794699700003],[22.26432,37.83939049800006],[22.024662,37.788459998000064],[21.89015549900006,37.86580299800005],[21.81131,37.95663849700003],[21.672683500000062,37.839206500000046],[21.63952050000006,37.89208599700004],[21.496893,37.919956],[21.351074,38.102138498000045],[21.372593,38.218120498000076],[21.605222500000025,38.142891],[21.778976500000056,38.30762499800005],[21.871086,38.33453749700004],[22.37297250000006,38.14222349700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL633","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Ileia","NUTS_NAME":"Ηλεία"},"geometry":{"type":"Polygon","coordinates":[[[21.89015549900006,37.86580299800005],[21.794531,37.79080199800006],[21.792816,37.602542999000036],[21.87857450000007,37.60453399900007],[21.99938750000007,37.49891299800004],[21.96572850000007,37.48326099700006],[21.88610650000004,37.454856997000036],[21.89699950000005,37.40159599800006],[21.680233,37.37743399800007],[21.530187500000068,37.56888199800005],[21.32807150000008,37.66140349700004],[21.28442,37.778659997000034],[21.105238,37.848926498000026],[21.126194,37.93867499600003],[21.255453,37.97354499900007],[21.351074,38.102138498000045],[21.496893,37.919956],[21.63952050000006,37.89208599700004],[21.672683500000062,37.839206500000046],[21.81131,37.95663849700003],[21.89015549900006,37.86580299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL64","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Sterea Ellada","NUTS_NAME":"Στερεά Ελλάδα"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.682358500000078,38.77332499800008],[24.563664,38.76233099800004],[24.530308500000046,38.79647249800007],[24.56670850000006,38.83881549700004],[24.430897,38.88147899900008],[24.448941,38.94910199800006],[24.573275500000022,38.92756050000003],[24.682358500000078,38.77332499800008]]],[[[23.426233500000023,38.90583049800006],[23.754823500000043,38.706645998000056],[24.155115,38.65118799900006],[24.122717,38.59658799700003],[24.232042500000034,38.52455149900004],[24.181846500000063,38.39474849800007],[24.255039500000066,38.288509497000064],[24.24803550000007,38.22748199800003],[24.350046,38.15526949800005],[24.554556,38.14113999800003],[24.58396350000004,38.02422349900007],[24.510714,37.94909299700004],[24.41704550000003,38.01402649600004],[24.373417,37.96247849700006],[24.30678,38.06962199800006],[24.202724499000055,38.08532349700005],[24.207344,38.179832496000074],[24.153744,38.27940399800008],[24.111498,38.25109099800005],[24.048601,38.395148997000035],[23.648699,38.39869699900004],[23.60085299900004,38.57010649700004],[23.511806500000034,38.58705499700005],[23.196602,38.834670998000036],[22.99010450000003,38.88124449800006],[22.829393500000037,38.825450998000065],[23.12924,39.007640998000056],[23.317873,39.03850549800006],[23.429733,38.95418549800007],[23.426233500000023,38.90583049800006]]],[[[22.514013500000033,39.04854949800006],[22.930694500000072,38.969772498000054],[23.01806650000003,39.00265499600005],[22.990511,38.94419099800007],[22.831438,38.889121997000075],[22.617443,38.91184999800004],[22.527525,38.85890949900005],[22.863748499000053,38.754974498000024],[23.034622,38.75635499800006],[23.10184650000008,38.62757849700006],[23.19618999900007,38.680735998000046],[23.321071500000073,38.63822949900003],[23.340513500000043,38.60502249900003],[23.296520500000042,38.56784449700007],[23.37779050000006,38.52575699700003],[23.328289,38.501220499000055],[23.560117,38.502597997000066],[23.65513450000003,38.35249699800005],[23.690418,38.34029749800004],[23.634155500000077,38.21131149800004],[23.515825500000062,38.13584149800005],[23.460073500000078,38.20882049900007],[23.37436850000006,38.21328349800007],[23.393368,38.263484998000024],[23.29180350000007,38.23928049700004],[23.28664,38.18450949700008],[23.126179,38.1684],[23.11132250000003,38.20705399800005],[23.066992,38.170379998000044],[22.779249,38.23318499800007],[22.79750450000006,38.27519599800007],[22.691986,38.317390499000055],[22.679586500000028,38.37295549700008],[22.60806850000006,38.35940149800007],[22.560972,38.280079],[22.419674,38.43173599900007],[22.378385500000036,38.33254249700008],[21.941110500000036,38.40624599700004],[21.851717,38.37451949700005],[21.879766500000073,38.46499249800007],[21.999883500000067,38.508353998000075],[21.999378,38.76886749600004],[21.946253,38.78343199700004],[21.944411500000058,38.74248849800006],[21.830959500000063,38.737224498000046],[21.776687500000037,38.67531949800008],[21.571245,38.73678599800007],[21.57425499900006,38.848701498000025],[21.637462,38.870811497000034],[21.507583500000067,38.889121997000075],[21.490255500000046,38.97023],[21.374632500000075,39.01817299900006],[21.436165,39.10585399800004],[21.396904,39.16447849600007],[21.639002,39.25025149800007],[21.728924,39.11552049800008],[21.858555,39.15161499800007],[21.939703,39.065337998000075],[21.952619500000026,39.02455149800005],[22.167362,39.12552649700007],[22.258116,39.272308499000076],[22.505670500000065,39.13119899900005],[22.514013500000033,39.04854949800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL641","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Voiotia","NUTS_NAME":"Βοιωτία"},"geometry":{"type":"Polygon","coordinates":[[[23.37779050000006,38.52575699700003],[23.328289,38.501220499000055],[23.270763500000044,38.442722497000034],[23.288496,38.41170899800005],[23.397024,38.47082899700007],[23.496658500000024,38.38064550000007],[23.65513450000003,38.35249699800005],[23.690418,38.34029749800004],[23.634155500000077,38.21131149800004],[23.515825500000062,38.13584149800005],[23.460073500000078,38.20882049900007],[23.37436850000006,38.21328349800007],[23.393368,38.263484998000024],[23.29180350000007,38.23928049700004],[23.28664,38.18450949700008],[23.126179,38.1684],[23.11132250000003,38.20705399800005],[23.066992,38.170379998000044],[22.779249,38.23318499800007],[22.79750450000006,38.27519599800007],[22.691986,38.317390499000055],[22.679586500000028,38.37295549700008],[22.60806850000006,38.35940149800007],[22.60329450000006,38.43514249900005],[22.538177500000074,38.45589849800007],[22.487697500000024,38.56152349700005],[22.581539,38.55955499700008],[22.64093,38.51171099700008],[22.862062500000036,38.58849699800004],[23.037508,38.53683099600005],[23.11247650000007,38.579612498000074],[23.23097050000007,38.501273998000045],[23.37779050000006,38.52575699700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL642","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Evvoia","NUTS_NAME":"Εύβοια"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.682358500000078,38.77332499800008],[24.563664,38.76233099800004],[24.530308500000046,38.79647249800007],[24.56670850000006,38.83881549700004],[24.430897,38.88147899900008],[24.448941,38.94910199800006],[24.573275500000022,38.92756050000003],[24.682358500000078,38.77332499800008]]],[[[23.426233500000023,38.90583049800006],[23.754823500000043,38.706645998000056],[24.155115,38.65118799900006],[24.122717,38.59658799700003],[24.232042500000034,38.52455149900004],[24.181846500000063,38.39474849800007],[24.255039500000066,38.288509497000064],[24.24803550000007,38.22748199800003],[24.350046,38.15526949800005],[24.554556,38.14113999800003],[24.58396350000004,38.02422349900007],[24.510714,37.94909299700004],[24.41704550000003,38.01402649600004],[24.373417,37.96247849700006],[24.30678,38.06962199800006],[24.202724499000055,38.08532349700005],[24.207344,38.179832496000074],[24.153744,38.27940399800008],[24.111498,38.25109099800005],[24.048601,38.395148997000035],[23.648699,38.39869699900004],[23.60085299900004,38.57010649700004],[23.511806500000034,38.58705499700005],[23.196602,38.834670998000036],[22.99010450000003,38.88124449800006],[22.829393500000037,38.825450998000065],[23.12924,39.007640998000056],[23.317873,39.03850549800006],[23.429733,38.95418549800007],[23.426233500000023,38.90583049800006]]],[[[23.560117,38.502597997000066],[23.65513450000003,38.35249699800005],[23.496658500000024,38.38064550000007],[23.397024,38.47082899700007],[23.288496,38.41170899800005],[23.270763500000044,38.442722497000034],[23.328289,38.501220499000055],[23.560117,38.502597997000066]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL643","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Evrytania","NUTS_NAME":"Ευρυτανία"},"geometry":{"type":"Polygon","coordinates":[[[21.939703,39.065337998000075],[21.825994500000036,38.94756299900007],[21.946253,38.78343199700004],[21.944411500000058,38.74248849800006],[21.830959500000063,38.737224498000046],[21.776687500000037,38.67531949800008],[21.571245,38.73678599800007],[21.57425499900006,38.848701498000025],[21.637462,38.870811497000034],[21.507583500000067,38.889121997000075],[21.490255500000046,38.97023],[21.374632500000075,39.01817299900006],[21.436165,39.10585399800004],[21.396904,39.16447849600007],[21.639002,39.25025149800007],[21.728924,39.11552049800008],[21.858555,39.15161499800007],[21.939703,39.065337998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL644","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Fthiotida","NUTS_NAME":"Φθιώτιδα"},"geometry":{"type":"Polygon","coordinates":[[[22.505670500000065,39.13119899900005],[22.514013500000033,39.04854949800006],[22.930694500000072,38.969772498000054],[23.01806650000003,39.00265499600005],[22.990511,38.94419099800007],[22.831438,38.889121997000075],[22.617443,38.91184999800004],[22.527525,38.85890949900005],[22.863748499000053,38.754974498000024],[23.034622,38.75635499800006],[23.10184650000008,38.62757849700006],[23.19618999900007,38.680735998000046],[23.321071500000073,38.63822949900003],[23.340513500000043,38.60502249900003],[23.296520500000042,38.56784449700007],[23.37779050000006,38.52575699700003],[23.23097050000007,38.501273998000045],[23.11247650000007,38.579612498000074],[23.037508,38.53683099600005],[22.862062500000036,38.58849699800004],[22.64093,38.51171099700008],[22.581539,38.55955499700008],[22.545942500000024,38.66563399800003],[22.427171500000043,38.73295199700004],[22.307194,38.717033498000035],[22.31624050000005,38.788936497000066],[22.13529,38.71247099800007],[21.999378,38.76886749600004],[21.946253,38.78343199700004],[21.825994500000036,38.94756299900007],[21.939703,39.065337998000075],[21.952619500000026,39.02455149800005],[22.167362,39.12552649700007],[22.258116,39.272308499000076],[22.505670500000065,39.13119899900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL645","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Fokida","NUTS_NAME":"Φωκίδα"},"geometry":{"type":"Polygon","coordinates":[[[22.581539,38.55955499700008],[22.487697500000024,38.56152349700005],[22.538177500000074,38.45589849800007],[22.60329450000006,38.43514249900005],[22.60806850000006,38.35940149800007],[22.560972,38.280079],[22.419674,38.43173599900007],[22.378385500000036,38.33254249700008],[21.941110500000036,38.40624599700004],[21.851717,38.37451949700005],[21.879766500000073,38.46499249800007],[21.999883500000067,38.508353998000075],[21.999378,38.76886749600004],[22.13529,38.71247099800007],[22.31624050000005,38.788936497000066],[22.307194,38.717033498000035],[22.427171500000043,38.73295199700004],[22.545942500000024,38.66563399800003],[22.581539,38.55955499700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL65","LEVL_CODE":2,"CNTR_CODE":"EL","NAME_LATN":"Peloponnisos","NUTS_NAME":"Πελοπόννησος"},"geometry":{"type":"MultiPolygon","coordinates":[[[[22.639021,38.07839199700004],[22.856842,37.93777849600008],[22.962061,37.94956949800007],[23.017891,37.900067998000054],[23.008983500000056,37.84590299700005],[23.178438,37.803564497000025],[23.12003150000004,37.72998499800008],[23.200378500000056,37.596653],[23.17948749900006,37.52343349800003],[23.454609,37.46235649800008],[23.424419500000056,37.411700997000025],[23.268150500000047,37.40636049900007],[23.224903,37.371658498000045],[23.277609,37.34850299800007],[23.193193500000064,37.341045499000074],[23.171631,37.286002997000026],[23.06308,37.39172350000007],[23.14237950000006,37.415977499000064],[23.121271,37.450256498000044],[22.725086,37.57263949700007],[22.774015500000075,37.38739799700005],[22.893175,37.219043499000065],[22.897411500000032,37.11927050000003],[22.95490650000005,37.11929299800005],[22.982481,37.04802699700008],[23.088842500000055,36.81272499900007],[23.086452500000064,36.729018997000026],[23.027857,36.72883549900007],[23.03039350000006,36.63529199800007],[23.196792500000072,36.43466199900007],[22.968313,36.51951199800004],[22.785621500000047,36.796966498000074],[22.63109950000006,36.80678949900005],[22.565794,36.77590199700006],[22.57607450000006,36.73329199800003],[22.476125500000023,36.613116999000056],[22.480207500000063,36.39754099700008],[22.348186500000054,36.51036449800006],[22.386833,36.59114099900006],[22.29641350000003,36.813945997000076],[22.225924,36.893966499000044],[22.126909,36.91332249800007],[22.149515,37.01814649800008],[21.930139500000053,36.98115549900007],[21.957984999000075,36.79297249800004],[21.877396,36.71761649700005],[21.822733,36.80372199900006],[21.704893,36.81129049900005],[21.67955,36.888435498000035],[21.710382500000037,36.94160099900006],[21.576706,37.06180199800008],[21.565725499000052,37.16350949900004],[21.691736,37.293105998000044],[21.680233,37.37743399800007],[21.89699950000005,37.40159599800006],[21.88610650000004,37.454856997000036],[21.96572850000007,37.48326099700006],[21.99938750000007,37.49891299800004],[21.87857450000007,37.60453399900007],[21.792816,37.602542999000036],[21.794531,37.79080199800006],[21.89015549900006,37.86580299800005],[22.024662,37.788459998000064],[22.26432,37.83939049800006],[22.221796,37.88794699700003],[22.362901500000078,38.022987998000076],[22.37297250000006,38.14222349700003],[22.639021,38.07839199700004]]],[[[23.133437,37.920439498000064],[23.009341,37.91685749900006],[22.84889350000003,38.02820649900008],[22.94963750000005,38.07875049800003],[23.117532,38.06064599700005],[23.106703,37.99519749700005],[23.179699500000027,37.95158999700004],[23.133437,37.920439498000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL651","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Argolida, Arkadia","NUTS_NAME":"Αργολίδα, Αρκαδία"},"geometry":{"type":"Polygon","coordinates":[[[22.26432,37.83939049800006],[22.411724,37.80005249800007],[22.501453500000025,37.86078249800005],[22.53453650000006,37.80837999800008],[22.78488150000004,37.73433699900005],[23.12003150000004,37.72998499800008],[23.200378500000056,37.596653],[23.17948749900006,37.52343349800003],[23.454609,37.46235649800008],[23.424419500000056,37.411700997000025],[23.268150500000047,37.40636049900007],[23.224903,37.371658498000045],[23.277609,37.34850299800007],[23.193193500000064,37.341045499000074],[23.171631,37.286002997000026],[23.06308,37.39172350000007],[23.14237950000006,37.415977499000064],[23.121271,37.450256498000044],[22.725086,37.57263949700007],[22.774015500000075,37.38739799700005],[22.893175,37.219043499000065],[22.897411500000032,37.11927050000003],[22.95490650000005,37.11929299800005],[22.982481,37.04802699700008],[22.871532500000058,36.998977498000045],[22.713613500000065,37.033027496000045],[22.604665500000067,37.29693599900003],[22.526424500000076,37.318133997000075],[22.401909,37.32465349800003],[22.42176450000005,37.21580499800007],[22.224659,37.27553949700007],[22.25198350000005,37.16933449700008],[22.18392350000005,37.137683999000046],[22.105213,37.22858050000008],[21.955913500000065,37.373496998000064],[21.993688500000076,37.416202499000065],[21.96572850000007,37.48326099700006],[21.99938750000007,37.49891299800004],[21.87857450000007,37.60453399900007],[21.792816,37.602542999000036],[21.794531,37.79080199800006],[21.89015549900006,37.86580299800005],[22.024662,37.788459998000064],[22.26432,37.83939049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL652","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Korinthia","NUTS_NAME":"Κορινθία"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.133437,37.920439498000064],[23.009341,37.91685749900006],[22.84889350000003,38.02820649900008],[22.94963750000005,38.07875049800003],[23.117532,38.06064599700005],[23.106703,37.99519749700005],[23.179699500000027,37.95158999700004],[23.133437,37.920439498000064]]],[[[22.639021,38.07839199700004],[22.856842,37.93777849600008],[22.962061,37.94956949800007],[23.017891,37.900067998000054],[23.008983500000056,37.84590299700005],[23.178438,37.803564497000025],[23.12003150000004,37.72998499800008],[22.78488150000004,37.73433699900005],[22.53453650000006,37.80837999800008],[22.501453500000025,37.86078249800005],[22.411724,37.80005249800007],[22.26432,37.83939049800006],[22.221796,37.88794699700003],[22.362901500000078,38.022987998000076],[22.37297250000006,38.14222349700003],[22.639021,38.07839199700004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"EL653","LEVL_CODE":3,"CNTR_CODE":"EL","NAME_LATN":"Lakonia, Messinia","NUTS_NAME":"Λακωνία, Μεσσηνία"},"geometry":{"type":"Polygon","coordinates":[[[22.105213,37.22858050000008],[22.18392350000005,37.137683999000046],[22.25198350000005,37.16933449700008],[22.224659,37.27553949700007],[22.42176450000005,37.21580499800007],[22.401909,37.32465349800003],[22.526424500000076,37.318133997000075],[22.604665500000067,37.29693599900003],[22.713613500000065,37.033027496000045],[22.871532500000058,36.998977498000045],[22.982481,37.04802699700008],[23.088842500000055,36.81272499900007],[23.086452500000064,36.729018997000026],[23.027857,36.72883549900007],[23.03039350000006,36.63529199800007],[23.196792500000072,36.43466199900007],[22.968313,36.51951199800004],[22.785621500000047,36.796966498000074],[22.63109950000006,36.80678949900005],[22.565794,36.77590199700006],[22.57607450000006,36.73329199800003],[22.476125500000023,36.613116999000056],[22.480207500000063,36.39754099700008],[22.348186500000054,36.51036449800006],[22.386833,36.59114099900006],[22.29641350000003,36.813945997000076],[22.225924,36.893966499000044],[22.126909,36.91332249800007],[22.149515,37.01814649800008],[21.930139500000053,36.98115549900007],[21.957984999000075,36.79297249800004],[21.877396,36.71761649700005],[21.822733,36.80372199900006],[21.704893,36.81129049900005],[21.67955,36.888435498000035],[21.710382500000037,36.94160099900006],[21.576706,37.06180199800008],[21.565725499000052,37.16350949900004],[21.691736,37.293105998000044],[21.680233,37.37743399800007],[21.89699950000005,37.40159599800006],[21.88610650000004,37.454856997000036],[21.96572850000007,37.48326099700006],[21.993688500000076,37.416202499000065],[21.955913500000065,37.373496998000064],[22.105213,37.22858050000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR21","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Champagne-Ardenne","NUTS_NAME":"Champagne-Ardenne"},"geometry":{"type":"Polygon","coordinates":[[[4.793194,49.981999],[4.889914,49.90920999800005],[4.851578,49.793255],[4.969431,49.80182599800003],[5.153738500000031,49.717925999000045],[5.393511,49.61711099900003],[5.275101,49.54189549700004],[5.107566500000075,49.584689498000046],[5.059498500000075,49.50056649800007],[5.101156,49.37815049800008],[4.950990500000046,49.23686749600006],[4.993376,49.20962349700005],[4.939760500000034,49.18507099800007],[5.03953,48.97408949700008],[4.918824,48.905575498000076],[4.888582,48.80061349600004],[5.009218500000031,48.74138899900004],[4.98843050000005,48.684422],[4.999322,48.61483449900004],[5.470055,48.42092649800003],[5.409781500000065,48.39261349800006],[5.416975500000035,48.34269549600003],[5.526527,48.34685349700004],[5.730055,48.19831849700006],[5.629007,48.08441199600003],[5.884723500000064,47.92604649800006],[5.681527,47.80911149700006],[5.691057500000056,47.684624996000025],[5.419852,47.677725999000074],[5.37408,47.60453799800007],[5.25292150000007,47.57695049700004],[5.211408500000061,47.641887498000074],[4.968567500000063,47.68732399900006],[4.958992,47.761870498000064],[4.917962,47.777093999000044],[4.99412650000005,47.819746],[4.789416500000073,48.00786649600008],[4.704233,48.020234998000035],[4.293421500000022,47.92567349600006],[4.203697499000043,47.97342199800005],[3.894036500000027,47.92929099600008],[3.914676,47.97502299900003],[3.801662,48.10672499800006],[3.667867,48.139210999000056],[3.587964,48.30080649700005],[3.414788999000052,48.39026849800007],[3.405396500000052,48.52801499800006],[3.557419164000066,48.61671378700004],[3.451096,48.634649997000054],[3.444068500000071,48.736630999000056],[3.395836,48.75924699700005],[3.485183500000062,48.85191049800005],[3.678962499000022,49.01829399800005],[3.585201501000029,49.03886699700007],[3.621108001000039,49.14976999900006],[3.748671001000048,49.158524501000045],[3.665503500000057,49.22285549800006],[3.643940500000042,49.31271349600007],[3.934719,49.398405499000035],[4.035498001000065,49.359906],[4.047974500000066,49.405644],[4.076765500000022,49.570718500000055],[4.02461850100002,49.62125549800004],[4.250094501000035,49.75709899700007],[4.233133,49.95782849700004],[4.432494,49.94161599600005],[4.694556,49.99650949900007],[4.680489,50.06813850000003],[4.796697,50.14867799800004],[4.896794,50.13742049800004],[4.793194,49.981999]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR211","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Ardennes","NUTS_NAME":"Ardennes"},"geometry":{"type":"Polygon","coordinates":[[[4.969431,49.80182599800003],[5.153738500000031,49.717925999000045],[5.393511,49.61711099900003],[5.275101,49.54189549700004],[5.107566500000075,49.584689498000046],[5.059498500000075,49.50056649800007],[5.101156,49.37815049800008],[4.950990500000046,49.23686749600006],[4.634265,49.23180249600006],[4.601459499000043,49.28999550000003],[4.422579,49.28064399700003],[4.248453,49.38225749800006],[4.047974500000066,49.405644],[4.076765500000022,49.570718500000055],[4.02461850100002,49.62125549800004],[4.250094501000035,49.75709899700007],[4.233133,49.95782849700004],[4.432494,49.94161599600005],[4.694556,49.99650949900007],[4.680489,50.06813850000003],[4.796697,50.14867799800004],[4.896794,50.13742049800004],[4.793194,49.981999],[4.889914,49.90920999800005],[4.851578,49.793255],[4.969431,49.80182599800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR212","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Aube","NUTS_NAME":"Aube"},"geometry":{"type":"Polygon","coordinates":[[[4.670186999000066,48.53188449700008],[4.677378,48.43226999900003],[4.841477,48.33946499800004],[4.861619,48.266358996000065],[4.834381,48.10516199800003],[4.731872500000065,48.119444],[4.704233,48.020234998000035],[4.293421500000022,47.92567349600006],[4.203697499000043,47.97342199800005],[3.894036500000027,47.92929099600008],[3.914676,47.97502299900003],[3.801662,48.10672499800006],[3.667867,48.139210999000056],[3.587964,48.30080649700005],[3.414788999000052,48.39026849800007],[3.405396500000052,48.52801499800006],[3.557419164000066,48.61671378700004],[3.643634,48.53625949900004],[3.825897500000053,48.515169498000034],[4.079784999000026,48.70112099800008],[4.326947,48.69741199900005],[4.314915,48.61622149600004],[4.397416999000029,48.56392799900004],[4.670186999000066,48.53188449700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR213","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Marne","NUTS_NAME":"Marne"},"geometry":{"type":"Polygon","coordinates":[[[4.950990500000046,49.23686749600006],[4.993376,49.20962349700005],[4.939760500000034,49.18507099800007],[5.03953,48.97408949700008],[4.918824,48.905575498000076],[4.888582,48.80061349600004],[5.009218500000031,48.74138899900004],[4.98843050000005,48.684422],[4.781669,48.67020099900003],[4.85364450000003,48.63480050000004],[4.769928,48.58695799800006],[4.79918,48.52978249900008],[4.670186999000066,48.53188449700008],[4.397416999000029,48.56392799900004],[4.314915,48.61622149600004],[4.326947,48.69741199900005],[4.079784999000026,48.70112099800008],[3.825897500000053,48.515169498000034],[3.643634,48.53625949900004],[3.557419164000066,48.61671378700004],[3.451096,48.634649997000054],[3.444068500000071,48.736630999000056],[3.395836,48.75924699700005],[3.485183500000062,48.85191049800005],[3.678962499000022,49.01829399800005],[3.585201501000029,49.03886699700007],[3.621108001000039,49.14976999900006],[3.748671001000048,49.158524501000045],[3.665503500000057,49.22285549800006],[3.643940500000042,49.31271349600007],[3.934719,49.398405499000035],[4.035498001000065,49.359906],[4.047974500000066,49.405644],[4.248453,49.38225749800006],[4.422579,49.28064399700003],[4.601459499000043,49.28999550000003],[4.634265,49.23180249600006],[4.950990500000046,49.23686749600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR214","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Marne","NUTS_NAME":"Haute-Marne"},"geometry":{"type":"Polygon","coordinates":[[[5.470055,48.42092649800003],[5.409781500000065,48.39261349800006],[5.416975500000035,48.34269549600003],[5.526527,48.34685349700004],[5.730055,48.19831849700006],[5.629007,48.08441199600003],[5.884723500000064,47.92604649800006],[5.681527,47.80911149700006],[5.691057500000056,47.684624996000025],[5.419852,47.677725999000074],[5.37408,47.60453799800007],[5.25292150000007,47.57695049700004],[5.211408500000061,47.641887498000074],[4.968567500000063,47.68732399900006],[4.958992,47.761870498000064],[4.917962,47.777093999000044],[4.99412650000005,47.819746],[4.789416500000073,48.00786649600008],[4.704233,48.020234998000035],[4.731872500000065,48.119444],[4.834381,48.10516199800003],[4.861619,48.266358996000065],[4.841477,48.33946499800004],[4.677378,48.43226999900003],[4.670186999000066,48.53188449700008],[4.79918,48.52978249900008],[4.769928,48.58695799800006],[4.85364450000003,48.63480050000004],[4.781669,48.67020099900003],[4.98843050000005,48.684422],[4.999322,48.61483449900004],[5.470055,48.42092649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR22","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Picardie","NUTS_NAME":"Picardie"},"geometry":{"type":"Polygon","coordinates":[[[2.495685,50.194559998000045],[2.403272,50.16480299800003],[2.375452500000051,50.10925499900003],[2.418616500000041,50.08765550000004],[2.514943500000072,50.14081499900004],[2.776931499000057,50.115931497000076],[2.763325,50.03833149800005],[3.090252,50.05374049800008],[3.172704500000066,50.01199649700004],[3.745749,50.053570498000056],[4.140853,49.978759996000065],[4.233133,49.95782849700004],[4.250094501000035,49.75709899700007],[4.02461850100002,49.62125549800004],[4.076765500000022,49.570718500000055],[4.047974500000066,49.405644],[4.035498001000065,49.359906],[3.934719,49.398405499000035],[3.643940500000042,49.31271349600007],[3.665503500000057,49.22285549800006],[3.748671001000048,49.158524501000045],[3.621108001000039,49.14976999900006],[3.585201501000029,49.03886699700007],[3.678962499000022,49.01829399800005],[3.485183500000062,48.85191049800005],[3.265159500000038,48.939061],[3.165230500000064,49.099653501000034],[3.07188,49.11755350000004],[2.590528500000062,49.079653998000026],[2.321773,49.18443],[1.882564500000058,49.16249099800007],[1.704359,49.23219699600003],[1.802659,49.27343049600006],[1.71393050000006,49.40922499800007],[1.787548999000023,49.50622899800004],[1.693719999000052,49.60106449800003],[1.72312450000004,49.672537],[1.688618500000075,49.69370399700006],[1.783834,49.758309498000074],[1.712209,49.886757997000075],[1.379698,50.06500999700006],[1.545791500000064,50.214481498000055],[1.683794500000033,50.18302549800006],[1.539681500000029,50.27449399900007],[1.55583,50.361316500000044],[1.641539500000022,50.35214999600004],[1.923473,50.320425999000065],[2.085712500000056,50.20100999600004],[2.495685,50.194559998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR221","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Aisne","NUTS_NAME":"Aisne"},"geometry":{"type":"Polygon","coordinates":[[[4.140853,49.978759996000065],[4.233133,49.95782849700004],[4.250094501000035,49.75709899700007],[4.02461850100002,49.62125549800004],[4.076765500000022,49.570718500000055],[4.047974500000066,49.405644],[4.035498001000065,49.359906],[3.934719,49.398405499000035],[3.643940500000042,49.31271349600007],[3.665503500000057,49.22285549800006],[3.748671001000048,49.158524501000045],[3.621108001000039,49.14976999900006],[3.585201501000029,49.03886699700007],[3.678962499000022,49.01829399800005],[3.485183500000062,48.85191049800005],[3.265159500000038,48.939061],[3.165230500000064,49.099653501000034],[3.07188,49.11755350000004],[3.148086,49.160749501000055],[2.971809501000052,49.18753],[2.958581,49.226135501000044],[3.023463502000027,49.21646399800005],[3.037354500000049,49.28670699800006],[2.964255500000036,49.32138599800004],[3.165124500000047,49.44084749600006],[3.10552350100005,49.47444400100005],[3.11864950000006,49.705831498000066],[3.061327,49.78138049800003],[3.172704500000066,50.01199649700004],[3.745749,50.053570498000056],[4.140853,49.978759996000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR222","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Oise","NUTS_NAME":"Oise"},"geometry":{"type":"Polygon","coordinates":[[[3.11864950000006,49.705831498000066],[3.10552350100005,49.47444400100005],[3.165124500000047,49.44084749600006],[2.964255500000036,49.32138599800004],[3.037354500000049,49.28670699800006],[3.023463502000027,49.21646399800005],[2.958581,49.226135501000044],[2.971809501000052,49.18753],[3.148086,49.160749501000055],[3.07188,49.11755350000004],[2.590528500000062,49.079653998000026],[2.321773,49.18443],[1.882564500000058,49.16249099800007],[1.704359,49.23219699600003],[1.802659,49.27343049600006],[1.71393050000006,49.40922499800007],[1.787548999000023,49.50622899800004],[1.693719999000052,49.60106449800003],[1.72312450000004,49.672537],[1.688618500000075,49.69370399700006],[1.783834,49.758309498000074],[1.848982,49.70163199800004],[2.315514500000063,49.68980049600003],[2.649634,49.571762499000044],[2.885069,49.709546997000075],[3.11864950000006,49.705831498000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR223","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Somme","NUTS_NAME":"Somme"},"geometry":{"type":"Polygon","coordinates":[[[1.641539500000022,50.35214999600004],[1.923473,50.320425999000065],[2.085712500000056,50.20100999600004],[2.495685,50.194559998000045],[2.403272,50.16480299800003],[2.375452500000051,50.10925499900003],[2.418616500000041,50.08765550000004],[2.514943500000072,50.14081499900004],[2.776931499000057,50.115931497000076],[2.763325,50.03833149800005],[3.090252,50.05374049800008],[3.172704500000066,50.01199649700004],[3.061327,49.78138049800003],[3.11864950000006,49.705831498000066],[2.885069,49.709546997000075],[2.649634,49.571762499000044],[2.315514500000063,49.68980049600003],[1.848982,49.70163199800004],[1.783834,49.758309498000074],[1.712209,49.886757997000075],[1.379698,50.06500999700006],[1.545791500000064,50.214481498000055],[1.683794500000033,50.18302549800006],[1.539681500000029,50.27449399900007],[1.55583,50.361316500000044],[1.641539500000022,50.35214999600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR23","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Haute-Normandie","NUTS_NAME":"Haute-Normandie"},"geometry":{"type":"Polygon","coordinates":[[[1.447285,49.05350949800004],[1.470892,48.974814500000036],[1.518385500000022,48.978357997000046],[1.501526500000068,48.941051997000045],[1.329809500000067,48.760958996000056],[1.121396,48.78919549900007],[0.814824500000043,48.67016349800008],[0.595637500000066,48.88802949800004],[0.421085,48.886761998000054],[0.386109,48.91054749800003],[0.412810500000035,48.95062649800008],[0.446057500000052,49.019007498000065],[0.366966,49.05235399900005],[0.408278500000051,49.128573998000036],[0.388406,49.21193750000003],[0.319827,49.250473498000076],[0.38356,49.285759496000026],[0.297224500000027,49.429863],[0.338978500000053,49.44092549900006],[0.065609,49.51257699800004],[0.192298,49.70696249800005],[0.562341500000059,49.84552749800008],[1.07273,49.927711497000075],[1.379698,50.06500999700006],[1.712209,49.886757997000075],[1.783834,49.758309498000074],[1.688618500000075,49.69370399700006],[1.72312450000004,49.672537],[1.693719999000052,49.60106449800003],[1.787548999000023,49.50622899800004],[1.71393050000006,49.40922499800007],[1.802659,49.27343049600006],[1.704359,49.23219699600003],[1.608799,49.077893999000025],[1.447285,49.05350949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR231","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Eure","NUTS_NAME":"Eure"},"geometry":{"type":"Polygon","coordinates":[[[1.71393050000006,49.40922499800007],[1.802659,49.27343049600006],[1.704359,49.23219699600003],[1.608799,49.077893999000025],[1.447285,49.05350949800004],[1.470892,48.974814500000036],[1.518385500000022,48.978357997000046],[1.501526500000068,48.941051997000045],[1.329809500000067,48.760958996000056],[1.121396,48.78919549900007],[0.814824500000043,48.67016349800008],[0.595637500000066,48.88802949800004],[0.421085,48.886761998000054],[0.386109,48.91054749800003],[0.412810500000035,48.95062649800008],[0.446057500000052,49.019007498000065],[0.366966,49.05235399900005],[0.408278500000051,49.128573998000036],[0.388406,49.21193750000003],[0.319827,49.250473498000076],[0.38356,49.285759496000026],[0.297224500000027,49.429863],[0.338978500000053,49.44092549900006],[0.523098,49.47909799800004],[0.649831500000062,49.40887949800003],[0.802835500000072,49.42732049800003],[0.932235500000047,49.34489],[0.841715500000021,49.329356],[0.999108500000034,49.25196949900004],[1.272137,49.346284498000045],[1.383247,49.45804999900008],[1.71393050000006,49.40922499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR232","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Seine-Maritime","NUTS_NAME":"Seine-Maritime"},"geometry":{"type":"Polygon","coordinates":[[[1.783834,49.758309498000074],[1.688618500000075,49.69370399700006],[1.72312450000004,49.672537],[1.693719999000052,49.60106449800003],[1.787548999000023,49.50622899800004],[1.71393050000006,49.40922499800007],[1.383247,49.45804999900008],[1.272137,49.346284498000045],[0.999108500000034,49.25196949900004],[0.841715500000021,49.329356],[0.932235500000047,49.34489],[0.802835500000072,49.42732049800003],[0.649831500000062,49.40887949800003],[0.523098,49.47909799800004],[0.338978500000053,49.44092549900006],[0.065609,49.51257699800004],[0.192298,49.70696249800005],[0.562341500000059,49.84552749800008],[1.07273,49.927711497000075],[1.379698,50.06500999700006],[1.712209,49.886757997000075],[1.783834,49.758309498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR24","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Centre","NUTS_NAME":"Centre"},"geometry":{"type":"Polygon","coordinates":[[[1.501526500000068,48.941051997000045],[1.626540500000033,48.74804799900005],[1.602704,48.66309299800008],[1.757405500000061,48.57495449900006],[1.801452500000039,48.46604099600006],[1.922146500000053,48.457599498000036],[1.99409,48.28658399900007],[2.402663,48.32071749900007],[2.522821,48.197726497000076],[2.476016500000071,48.12963649900007],[2.936316,48.163391499000056],[3.029468,48.133204498000055],[3.12857,47.983377],[3.009697,47.90552949800008],[3.02379650000006,47.786369999000044],[2.851990500000056,47.757862999000054],[2.976535500000068,47.56942449700006],[2.845187,47.544932997000046],[2.874630500000023,47.52042499800007],[2.931569,47.44085499800008],[2.876565,47.32186850000005],[2.983081,47.26080299600005],[3.074774499000057,47.029995],[3.032063,46.794909500000074],[2.71502,46.744980998000074],[2.566998,46.652483],[2.614960500000052,46.55327649900005],[2.351579,46.51213949600003],[2.281043500000067,46.42040349600006],[2.167784500000039,46.42406899800005],[1.747591500000055,46.45001799600004],[1.72784,46.38895750000006],[1.52535,46.42664899600004],[1.415185500000064,46.347215],[1.356024500000046,46.40011949800004],[1.177279,46.383947997000064],[1.213061,46.43307699700006],[1.149148500000024,46.502205496000045],[0.915865500000052,46.59662799800003],[0.927891,46.69264899800004],[0.867469,46.74821649800003],[0.704727499000057,46.902806998000074],[0.692567,46.97430449800004],[0.567741,47.00628399800007],[0.599612500000035,46.95628899800005],[0.32483650000006,46.93065199800003],[0.305386500000054,47.052699500000074],[0.175696,47.06072749700007],[0.05383,47.163733499000045],[0.230000500000074,47.608397498000045],[0.378485,47.568527998000036],[0.382790500000056,47.64309749800003],[0.614432500000021,47.69421549600003],[0.580504500000075,47.71282549800003],[0.848432500000058,47.942154498000036],[0.793996500000048,48.06944],[0.841217499000038,48.10305950000003],[0.91665050000006,48.13893649800008],[0.797658499000022,48.194455],[0.830483500000071,48.21070849800003],[0.755662,48.299999498000034],[0.940575500000023,48.39835499800006],[0.96774,48.52387949800004],[0.850635500000067,48.58262549800003],[0.814824500000043,48.67016349800008],[1.121396,48.78919549900007],[1.329809500000067,48.760958996000056],[1.501526500000068,48.941051997000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR241","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Cher","NUTS_NAME":"Cher"},"geometry":{"type":"Polygon","coordinates":[[[2.874630500000023,47.52042499800007],[2.931569,47.44085499800008],[2.876565,47.32186850000005],[2.983081,47.26080299600005],[3.074774499000057,47.029995],[3.032063,46.794909500000074],[2.71502,46.744980998000074],[2.566998,46.652483],[2.614960500000052,46.55327649900005],[2.351579,46.51213949600003],[2.281043500000067,46.42040349600006],[2.167784500000039,46.42406899800005],[2.151407500000062,46.45769249600005],[2.204327,46.48421749900007],[2.158696500000076,46.55734449800008],[2.189441500000044,46.642138497000076],[2.08709650000003,46.71245049800007],[2.067215500000032,46.751720498000054],[2.116644500000064,46.77971649700004],[2.042716,46.840141998000036],[2.154964,46.911672999000075],[2.070702500000039,46.93399849800005],[2.108850500000074,46.99718949600003],[2.028817500000059,47.046931],[2.053602,47.101510500000074],[1.774454,47.13075649600006],[1.839080500000023,47.219303498000045],[1.903812,47.215246998000055],[1.942334500000072,47.291038498000034],[2.158081,47.298636499000054],[2.097082,47.36680050000007],[2.114994,47.42148149800005],[2.230393500000048,47.40752349700006],[2.24774350000007,47.491730500000074],[2.119572500000061,47.58294699700008],[2.239213500000062,47.620975498000064],[2.563206500000035,47.57213649600004],[2.669387500000028,47.484676498000056],[2.731819,47.53808949900008],[2.874630500000023,47.52042499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR242","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Eure-et-Loir","NUTS_NAME":"Eure-et-Loir"},"geometry":{"type":"Polygon","coordinates":[[[1.922146500000053,48.457599498000036],[1.99409,48.28658399900007],[1.865674,48.08385799600006],[1.54530950000003,48.04458949800005],[1.520124,47.98227499600006],[1.268243,47.961539998000035],[1.026004,48.08840749900003],[1.038972,48.13140499600007],[0.841217499000038,48.10305950000003],[0.91665050000006,48.13893649800008],[0.797658499000022,48.194455],[0.830483500000071,48.21070849800003],[0.755662,48.299999498000034],[0.940575500000023,48.39835499800006],[0.96774,48.52387949800004],[0.850635500000067,48.58262549800003],[0.814824500000043,48.67016349800008],[1.121396,48.78919549900007],[1.329809500000067,48.760958996000056],[1.501526500000068,48.941051997000045],[1.626540500000033,48.74804799900005],[1.602704,48.66309299800008],[1.757405500000061,48.57495449900006],[1.801452500000039,48.46604099600006],[1.922146500000053,48.457599498000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR243","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Indre","NUTS_NAME":"Indre"},"geometry":{"type":"Polygon","coordinates":[[[1.839080500000023,47.219303498000045],[1.774454,47.13075649600006],[2.053602,47.101510500000074],[2.028817500000059,47.046931],[2.108850500000074,46.99718949600003],[2.070702500000039,46.93399849800005],[2.154964,46.911672999000075],[2.042716,46.840141998000036],[2.116644500000064,46.77971649700004],[2.067215500000032,46.751720498000054],[2.08709650000003,46.71245049800007],[2.189441500000044,46.642138497000076],[2.158696500000076,46.55734449800008],[2.204327,46.48421749900007],[2.151407500000062,46.45769249600005],[2.167784500000039,46.42406899800005],[1.747591500000055,46.45001799600004],[1.72784,46.38895750000006],[1.52535,46.42664899600004],[1.415185500000064,46.347215],[1.356024500000046,46.40011949800004],[1.177279,46.383947997000064],[1.213061,46.43307699700006],[1.149148500000024,46.502205496000045],[0.915865500000052,46.59662799800003],[0.927891,46.69264899800004],[0.867469,46.74821649800003],[1.01168750000005,46.75780749900008],[1.056371,46.99643799800003],[1.253116,47.021765498000036],[1.365884,47.12254849900006],[1.326659,47.186221499000055],[1.592427,47.27667549800003],[1.839080500000023,47.219303498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR244","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Indre-et-Loire","NUTS_NAME":"Indre-et-Loire"},"geometry":{"type":"Polygon","coordinates":[[[1.326659,47.186221499000055],[1.365884,47.12254849900006],[1.253116,47.021765498000036],[1.056371,46.99643799800003],[1.01168750000005,46.75780749900008],[0.867469,46.74821649800003],[0.704727499000057,46.902806998000074],[0.692567,46.97430449800004],[0.567741,47.00628399800007],[0.599612500000035,46.95628899800005],[0.32483650000006,46.93065199800003],[0.305386500000054,47.052699500000074],[0.175696,47.06072749700007],[0.05383,47.163733499000045],[0.230000500000074,47.608397498000045],[0.378485,47.568527998000036],[0.382790500000056,47.64309749800003],[0.614432500000021,47.69421549600003],[0.834174500000074,47.67692999800005],[0.863892,47.59963049900006],[1.033351500000038,47.60700699800003],[1.133364500000027,47.44895099800004],[1.108121500000038,47.29840149900008],[1.241373500000066,47.288002498000026],[1.326659,47.186221499000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR245","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Loir-et-Cher","NUTS_NAME":"Loir-et-Cher"},"geometry":{"type":"Polygon","coordinates":[[[1.520124,47.98227499600006],[1.56482,47.98977199800004],[1.590502500000071,47.87891499600005],[1.547816,47.769537498000034],[1.706474,47.73322299900008],[1.739943500000038,47.66153399800004],[2.20368250000007,47.67854850000003],[2.239213500000062,47.620975498000064],[2.119572500000061,47.58294699700008],[2.24774350000007,47.491730500000074],[2.230393500000048,47.40752349700006],[2.114994,47.42148149800005],[2.097082,47.36680050000007],[2.158081,47.298636499000054],[1.942334500000072,47.291038498000034],[1.903812,47.215246998000055],[1.839080500000023,47.219303498000045],[1.592427,47.27667549800003],[1.326659,47.186221499000055],[1.241373500000066,47.288002498000026],[1.108121500000038,47.29840149900008],[1.133364500000027,47.44895099800004],[1.033351500000038,47.60700699800003],[0.863892,47.59963049900006],[0.834174500000074,47.67692999800005],[0.614432500000021,47.69421549600003],[0.580504500000075,47.71282549800003],[0.848432500000058,47.942154498000036],[0.793996500000048,48.06944],[0.841217499000038,48.10305950000003],[1.038972,48.13140499600007],[1.026004,48.08840749900003],[1.268243,47.961539998000035],[1.520124,47.98227499600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR246","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Loiret","NUTS_NAME":"Loiret"},"geometry":{"type":"Polygon","coordinates":[[[2.402663,48.32071749900007],[2.522821,48.197726497000076],[2.476016500000071,48.12963649900007],[2.936316,48.163391499000056],[3.029468,48.133204498000055],[3.12857,47.983377],[3.009697,47.90552949800008],[3.02379650000006,47.786369999000044],[2.851990500000056,47.757862999000054],[2.976535500000068,47.56942449700006],[2.845187,47.544932997000046],[2.874630500000023,47.52042499800007],[2.731819,47.53808949900008],[2.669387500000028,47.484676498000056],[2.563206500000035,47.57213649600004],[2.239213500000062,47.620975498000064],[2.20368250000007,47.67854850000003],[1.739943500000038,47.66153399800004],[1.706474,47.73322299900008],[1.547816,47.769537498000034],[1.590502500000071,47.87891499600005],[1.56482,47.98977199800004],[1.520124,47.98227499600006],[1.54530950000003,48.04458949800005],[1.865674,48.08385799600006],[1.99409,48.28658399900007],[2.402663,48.32071749900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR25","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Basse-Normandie","NUTS_NAME":"Basse-Normandie"},"geometry":{"type":"Polygon","coordinates":[[[-1.942384,49.725959998000064],[-1.630755499999964,49.65202349600003],[-1.26649249999997,49.69559849600006],[-1.228839499999935,49.60570549800008],[-1.307770499999947,49.54571899900003],[-1.11962,49.355568],[-0.961539499999958,49.396434998000075],[-0.225695,49.281821999000044],[0.297224500000027,49.429863],[0.38356,49.285759496000026],[0.319827,49.250473498000076],[0.388406,49.21193750000003],[0.408278500000051,49.128573998000036],[0.366966,49.05235399900005],[0.446057500000052,49.019007498000065],[0.412810500000035,48.95062649800008],[0.386109,48.91054749800003],[0.421085,48.886761998000054],[0.595637500000066,48.88802949800004],[0.814824500000043,48.67016349800008],[0.850635500000067,48.58262549800003],[0.96774,48.52387949800004],[0.940575500000023,48.39835499800006],[0.755662,48.299999498000034],[0.830483500000071,48.21070849800003],[0.797658499000022,48.194455],[0.395541,48.32046199800004],[0.35289,48.45968799900004],[-0.054527,48.38200449900006],[-0.051349,48.452927],[-0.148717499999975,48.458068997000055],[-0.144602499999962,48.527754498000036],[-0.234124,48.561847500000056],[-0.654219,48.44454249900008],[-0.86036,48.501458498000034],[-1.070164499999976,48.50849199900006],[-1.249492500999963,48.54369199800004],[-1.42794,48.461915],[-1.571089499999971,48.626441999000065],[-1.391047,48.644653496000046],[-1.572866499999975,48.74789450000003],[-1.556390499999964,49.02472699800006],[-1.504676,49.02013399800006],[-1.600102499999934,49.04160299700004],[-1.605357,49.205581497000026],[-1.567265,49.22087849900004],[-1.625514,49.21006399800007],[-1.809161499999959,49.372093],[-1.868679499999928,49.51237099600007],[-1.855007499999942,49.64282250000008],[-1.947120499999926,49.674759],[-1.942384,49.725959998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR251","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Calvados","NUTS_NAME":"Calvados"},"geometry":{"type":"Polygon","coordinates":[[[0.412810500000035,48.95062649800008],[0.127605,48.95221999600005],[-0.126172,48.816565996000065],[-0.692715,48.84729949800004],[-0.840941,48.752223498000035],[-1.15957149999997,48.82724050000007],[-1.021838,48.90451199700004],[-1.075511499999948,48.92137349800004],[-1.056536499999936,48.958623498000065],[-0.860870499999976,49.03082599700008],[-0.92591,49.16404799800006],[-0.900875499999927,49.202972499000055],[-1.078168,49.23388999800005],[-1.140311499999939,49.298509497000055],[-1.11962,49.355568],[-0.961539499999958,49.396434998000075],[-0.225695,49.281821999000044],[0.297224500000027,49.429863],[0.38356,49.285759496000026],[0.319827,49.250473498000076],[0.388406,49.21193750000003],[0.408278500000051,49.128573998000036],[0.366966,49.05235399900005],[0.446057500000052,49.019007498000065],[0.412810500000035,48.95062649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR252","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Manche","NUTS_NAME":"Manche"},"geometry":{"type":"Polygon","coordinates":[[[-1.11962,49.355568],[-1.140311499999939,49.298509497000055],[-1.078168,49.23388999800005],[-0.900875499999927,49.202972499000055],[-0.92591,49.16404799800006],[-0.860870499999976,49.03082599700008],[-1.056536499999936,48.958623498000065],[-1.075511499999948,48.92137349800004],[-1.021838,48.90451199700004],[-1.15957149999997,48.82724050000007],[-0.840941,48.752223498000035],[-0.734739499999932,48.685519499000065],[-0.773750499999949,48.56340749800006],[-0.86036,48.501458498000034],[-1.070164499999976,48.50849199900006],[-1.249492500999963,48.54369199800004],[-1.42794,48.461915],[-1.571089499999971,48.626441999000065],[-1.391047,48.644653496000046],[-1.572866499999975,48.74789450000003],[-1.556390499999964,49.02472699800006],[-1.504676,49.02013399800006],[-1.600102499999934,49.04160299700004],[-1.605357,49.205581497000026],[-1.567265,49.22087849900004],[-1.625514,49.21006399800007],[-1.809161499999959,49.372093],[-1.868679499999928,49.51237099600007],[-1.855007499999942,49.64282250000008],[-1.947120499999926,49.674759],[-1.942384,49.725959998000064],[-1.630755499999964,49.65202349600003],[-1.26649249999997,49.69559849600006],[-1.228839499999935,49.60570549800008],[-1.307770499999947,49.54571899900003],[-1.11962,49.355568]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR253","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Orne","NUTS_NAME":"Orne"},"geometry":{"type":"Polygon","coordinates":[[[0.412810500000035,48.95062649800008],[0.386109,48.91054749800003],[0.421085,48.886761998000054],[0.595637500000066,48.88802949800004],[0.814824500000043,48.67016349800008],[0.850635500000067,48.58262549800003],[0.96774,48.52387949800004],[0.940575500000023,48.39835499800006],[0.755662,48.299999498000034],[0.830483500000071,48.21070849800003],[0.797658499000022,48.194455],[0.395541,48.32046199800004],[0.35289,48.45968799900004],[-0.054527,48.38200449900006],[-0.051349,48.452927],[-0.148717499999975,48.458068997000055],[-0.144602499999962,48.527754498000036],[-0.234124,48.561847500000056],[-0.654219,48.44454249900008],[-0.86036,48.501458498000034],[-0.773750499999949,48.56340749800006],[-0.734739499999932,48.685519499000065],[-0.840941,48.752223498000035],[-0.692715,48.84729949800004],[-0.126172,48.816565996000065],[0.127605,48.95221999600005],[0.412810500000035,48.95062649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR26","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Bourgogne","NUTS_NAME":"Bourgogne"},"geometry":{"type":"Polygon","coordinates":[[[3.914676,47.97502299900003],[3.894036500000027,47.92929099600008],[4.203697499000043,47.97342199800005],[4.293421500000022,47.92567349600006],[4.704233,48.020234998000035],[4.789416500000073,48.00786649600008],[4.99412650000005,47.819746],[4.917962,47.777093999000044],[4.958992,47.761870498000064],[4.968567500000063,47.68732399900006],[5.211408500000061,47.641887498000074],[5.25292150000007,47.57695049700004],[5.37408,47.60453799800007],[5.477542,47.60871849800003],[5.486405,47.52696849600005],[5.375406999000063,47.46016749800003],[5.496923,47.388544996000064],[5.472388,47.318409],[5.518537500000036,47.30418399800004],[5.438506,47.14152950000005],[5.255232500000034,46.97988749800004],[5.328820500000063,46.887597999000036],[5.465138500000023,46.840725500000076],[5.32739250000003,46.815101998000046],[5.440604,46.63791199900004],[5.362118500000065,46.57726649800003],[5.415741,46.47314249800007],[5.310563499000068,46.446769998000036],[5.173731,46.51270099800007],[4.953853999000046,46.51342299800007],[4.780208500000072,46.176675998000064],[4.727254500000072,46.17911299900004],[4.69344,46.30232599800007],[4.405813500000022,46.29605799600006],[4.388074500000073,46.219790498000066],[4.282425500000045,46.15681299800008],[3.988531,46.16985450000004],[3.890131,46.21448699800004],[3.899538499000073,46.27590799800004],[3.984741,46.329729996000026],[3.998868500000071,46.46486699800005],[3.74101150000007,46.56021499600007],[3.629422500000032,46.74945649700004],[3.455304500000068,46.65239349800004],[3.43421,46.71222249700003],[3.165739,46.69172199800005],[3.032063,46.794909500000074],[3.074774499000057,47.029995],[2.983081,47.26080299600005],[2.876565,47.32186850000005],[2.931569,47.44085499800008],[2.874630500000023,47.52042499800007],[2.845187,47.544932997000046],[2.976535500000068,47.56942449700006],[2.851990500000056,47.757862999000054],[3.02379650000006,47.786369999000044],[3.009697,47.90552949800008],[3.12857,47.983377],[3.029468,48.133204498000055],[2.936316,48.163391499000056],[3.031462,48.248878998000066],[3.049454,48.36002749800008],[3.414788999000052,48.39026849800007],[3.587964,48.30080649700005],[3.667867,48.139210999000056],[3.801662,48.10672499800006],[3.914676,47.97502299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR261","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Côte-d'Or","NUTS_NAME":"Côte-d'Or"},"geometry":{"type":"Polygon","coordinates":[[[4.958992,47.761870498000064],[4.968567500000063,47.68732399900006],[5.211408500000061,47.641887498000074],[5.25292150000007,47.57695049700004],[5.37408,47.60453799800007],[5.477542,47.60871849800003],[5.486405,47.52696849600005],[5.375406999000063,47.46016749800003],[5.496923,47.388544996000064],[5.472388,47.318409],[5.518537500000036,47.30418399800004],[5.438506,47.14152950000005],[5.255232500000034,46.97988749800004],[4.685225,46.90095299600006],[4.554814,47.02005199700005],[4.209831500000064,47.155410998000036],[4.221394500000031,47.219785],[4.135147500000073,47.23735749900004],[4.106080500000076,47.33925499800006],[4.065197,47.40726299700003],[4.226147,47.670704998000076],[4.284060999000076,47.68265949800008],[4.239462,47.72442249800008],[4.332469,47.756512996000026],[4.324864500000047,47.84719249800003],[4.246378,47.87387149800003],[4.293421500000022,47.92567349600006],[4.704233,48.020234998000035],[4.789416500000073,48.00786649600008],[4.99412650000005,47.819746],[4.917962,47.777093999000044],[4.958992,47.761870498000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR262","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Nièvre","NUTS_NAME":"Nièvre"},"geometry":{"type":"Polygon","coordinates":[[[3.124024,47.575624],[3.235026,47.48963299700006],[3.51350750000006,47.52766699800003],[3.726289999000073,47.39824249800006],[3.816575500000056,47.37863550000003],[3.867055500000049,47.43461349800003],[3.871413,47.367169500000045],[3.948778,47.394412998000064],[4.002077500000041,47.31061399600003],[4.106080500000076,47.33925499800006],[4.135147500000073,47.23735749900004],[4.221394500000031,47.219785],[4.209831500000064,47.155410998000036],[4.181895,47.150507997000034],[4.148591,47.11346850000007],[4.115965500000073,47.12333599800007],[4.047943,47.10893549600007],[4.061668,47.023382998000045],[3.993651999000065,46.978988498000035],[4.100883,46.85910349800008],[4.058502,46.78078499900005],[3.826555,46.703821496000046],[3.629422500000032,46.74945649700004],[3.455304500000068,46.65239349800004],[3.43421,46.71222249700003],[3.165739,46.69172199800005],[3.032063,46.794909500000074],[3.074774499000057,47.029995],[2.983081,47.26080299600005],[2.876565,47.32186850000005],[2.931569,47.44085499800008],[2.874630500000023,47.52042499800007],[2.845187,47.544932997000046],[2.976535500000068,47.56942449700006],[3.124024,47.575624]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR263","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Saône-et-Loire","NUTS_NAME":"Saône-et-Loire"},"geometry":{"type":"Polygon","coordinates":[[[5.255232500000034,46.97988749800004],[5.328820500000063,46.887597999000036],[5.465138500000023,46.840725500000076],[5.32739250000003,46.815101998000046],[5.440604,46.63791199900004],[5.362118500000065,46.57726649800003],[5.415741,46.47314249800007],[5.310563499000068,46.446769998000036],[5.173731,46.51270099800007],[4.953853999000046,46.51342299800007],[4.780208500000072,46.176675998000064],[4.727254500000072,46.17911299900004],[4.69344,46.30232599800007],[4.405813500000022,46.29605799600006],[4.388074500000073,46.219790498000066],[4.282425500000045,46.15681299800008],[3.988531,46.16985450000004],[3.890131,46.21448699800004],[3.899538499000073,46.27590799800004],[3.984741,46.329729996000026],[3.998868500000071,46.46486699800005],[3.74101150000007,46.56021499600007],[3.629422500000032,46.74945649700004],[3.826555,46.703821496000046],[4.058502,46.78078499900005],[4.100883,46.85910349800008],[3.993651999000065,46.978988498000035],[4.061668,47.023382998000045],[4.047943,47.10893549600007],[4.115965500000073,47.12333599800007],[4.148591,47.11346850000007],[4.181895,47.150507997000034],[4.209831500000064,47.155410998000036],[4.554814,47.02005199700005],[4.685225,46.90095299600006],[5.255232500000034,46.97988749800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR264","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Yonne","NUTS_NAME":"Yonne"},"geometry":{"type":"Polygon","coordinates":[[[4.293421500000022,47.92567349600006],[4.246378,47.87387149800003],[4.324864500000047,47.84719249800003],[4.332469,47.756512996000026],[4.239462,47.72442249800008],[4.284060999000076,47.68265949800008],[4.226147,47.670704998000076],[4.065197,47.40726299700003],[4.106080500000076,47.33925499800006],[4.002077500000041,47.31061399600003],[3.948778,47.394412998000064],[3.871413,47.367169500000045],[3.867055500000049,47.43461349800003],[3.816575500000056,47.37863550000003],[3.726289999000073,47.39824249800006],[3.51350750000006,47.52766699800003],[3.235026,47.48963299700006],[3.124024,47.575624],[2.976535500000068,47.56942449700006],[2.851990500000056,47.757862999000054],[3.02379650000006,47.786369999000044],[3.009697,47.90552949800008],[3.12857,47.983377],[3.029468,48.133204498000055],[2.936316,48.163391499000056],[3.031462,48.248878998000066],[3.049454,48.36002749800008],[3.414788999000052,48.39026849800007],[3.587964,48.30080649700005],[3.667867,48.139210999000056],[3.801662,48.10672499800006],[3.914676,47.97502299900003],[3.894036500000027,47.92929099600008],[4.203697499000043,47.97342199800005],[4.293421500000022,47.92567349600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR3","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"NORD - PAS-DE-CALAIS","NUTS_NAME":"NORD - PAS-DE-CALAIS"},"geometry":{"type":"Polygon","coordinates":[[[2.63495,50.81274399900008],[2.863276,50.708343496000055],[3.018708500000059,50.773532998000064],[3.098481,50.77901899900007],[3.176996,50.75616449700004],[3.245294,50.71300949700003],[3.286492,50.52756899700006],[3.615081500000031,50.490399],[3.65551,50.46173549800005],[3.710389,50.303165499000045],[4.027774500000021,50.35833049800004],[4.221769,50.255832500000054],[4.126875,50.13492199800004],[4.231028,50.071983498000066],[4.140853,49.978759996000065],[3.745749,50.053570498000056],[3.172704500000066,50.01199649700004],[3.090252,50.05374049800008],[2.763325,50.03833149800005],[2.776931499000057,50.115931497000076],[2.514943500000072,50.14081499900004],[2.418616500000041,50.08765550000004],[2.375452500000051,50.10925499900003],[2.403272,50.16480299800003],[2.495685,50.194559998000045],[2.085712500000056,50.20100999600004],[1.923473,50.320425999000065],[1.641539500000022,50.35214999600004],[1.555429500000059,50.399936498000045],[1.560707,50.69967249800004],[1.606003,50.791911999000035],[1.580953,50.869552499000065],[2.067705,51.00649999600006],[2.546011,51.08938199700003],[2.632617,50.94594549900006],[2.607036,50.91268949800008],[2.63495,50.81274399900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR30","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Nord - Pas-de-Calais","NUTS_NAME":"Nord - Pas-de-Calais"},"geometry":{"type":"Polygon","coordinates":[[[2.63495,50.81274399900008],[2.863276,50.708343496000055],[3.018708500000059,50.773532998000064],[3.098481,50.77901899900007],[3.176996,50.75616449700004],[3.245294,50.71300949700003],[3.286492,50.52756899700006],[3.615081500000031,50.490399],[3.65551,50.46173549800005],[3.710389,50.303165499000045],[4.027774500000021,50.35833049800004],[4.221769,50.255832500000054],[4.126875,50.13492199800004],[4.231028,50.071983498000066],[4.140853,49.978759996000065],[3.745749,50.053570498000056],[3.172704500000066,50.01199649700004],[3.090252,50.05374049800008],[2.763325,50.03833149800005],[2.776931499000057,50.115931497000076],[2.514943500000072,50.14081499900004],[2.418616500000041,50.08765550000004],[2.375452500000051,50.10925499900003],[2.403272,50.16480299800003],[2.495685,50.194559998000045],[2.085712500000056,50.20100999600004],[1.923473,50.320425999000065],[1.641539500000022,50.35214999600004],[1.555429500000059,50.399936498000045],[1.560707,50.69967249800004],[1.606003,50.791911999000035],[1.580953,50.869552499000065],[2.067705,51.00649999600006],[2.546011,51.08938199700003],[2.632617,50.94594549900006],[2.607036,50.91268949800008],[2.63495,50.81274399900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR301","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Nord","NUTS_NAME":"Nord"},"geometry":{"type":"Polygon","coordinates":[[[2.607036,50.91268949800008],[2.63495,50.81274399900008],[2.863276,50.708343496000055],[3.018708500000059,50.773532998000064],[3.098481,50.77901899900007],[3.176996,50.75616449700004],[3.245294,50.71300949700003],[3.286492,50.52756899700006],[3.615081500000031,50.490399],[3.65551,50.46173549800005],[3.710389,50.303165499000045],[4.027774500000021,50.35833049800004],[4.221769,50.255832500000054],[4.126875,50.13492199800004],[4.231028,50.071983498000066],[4.140853,49.978759996000065],[3.745749,50.053570498000056],[3.172704500000066,50.01199649700004],[3.090252,50.05374049800008],[3.186852,50.23492999800004],[3.048575500000027,50.27769649800007],[3.084305500000028,50.311187999000026],[2.990406499000073,50.395989996000026],[3.069372,50.45589749800007],[2.799390500000072,50.52856699800003],[2.789077,50.58306149800006],[2.869731,50.631091498000046],[2.809193500000049,50.66937],[2.745912,50.604615],[2.374131500000033,50.67219249900006],[2.33805650000005,50.73623699600006],[2.411479,50.76673449700007],[2.213433500000065,50.81054749700007],[2.067705,51.00649999600006],[2.546011,51.08938199700003],[2.632617,50.94594549900006],[2.607036,50.91268949800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR302","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Pas-de-Calais","NUTS_NAME":"Pas-de-Calais"},"geometry":{"type":"Polygon","coordinates":[[[2.374131500000033,50.67219249900006],[2.745912,50.604615],[2.809193500000049,50.66937],[2.869731,50.631091498000046],[2.789077,50.58306149800006],[2.799390500000072,50.52856699800003],[3.069372,50.45589749800007],[2.990406499000073,50.395989996000026],[3.084305500000028,50.311187999000026],[3.048575500000027,50.27769649800007],[3.186852,50.23492999800004],[3.090252,50.05374049800008],[2.763325,50.03833149800005],[2.776931499000057,50.115931497000076],[2.514943500000072,50.14081499900004],[2.418616500000041,50.08765550000004],[2.375452500000051,50.10925499900003],[2.403272,50.16480299800003],[2.495685,50.194559998000045],[2.085712500000056,50.20100999600004],[1.923473,50.320425999000065],[1.641539500000022,50.35214999600004],[1.555429500000059,50.399936498000045],[1.560707,50.69967249800004],[1.606003,50.791911999000035],[1.580953,50.869552499000065],[2.067705,51.00649999600006],[2.213433500000065,50.81054749700007],[2.411479,50.76673449700007],[2.33805650000005,50.73623699600006],[2.374131500000033,50.67219249900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR4","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"EST","NUTS_NAME":"EST"},"geometry":{"type":"Polygon","coordinates":[[[5.393511,49.61711099900003],[5.470882999000025,49.49723799800006],[5.734555999000065,49.545690500000035],[5.818117,49.54631049900007],[5.893386,49.49694449700007],[6.367107500000031,49.469507],[6.556986,49.419208498000046],[6.565664,49.34765149700007],[6.723465500000032,49.21882899800005],[6.738397500000076,49.16359799900005],[6.834567,49.15131099800004],[6.837607499000057,49.21293299700005],[6.925060500000029,49.22334649800007],[7.03497150000004,49.19141499700004],[7.057195999000044,49.11193149800005],[7.101069,49.15599799900008],[7.293177,49.11490749700005],[7.36875550000002,49.16145799800006],[7.493791,49.16773550000005],[7.635651,49.05395049900005],[7.910654500000021,49.04516349800008],[8.068407500000035,48.99931649700005],[8.232632999000032,48.96657150000004],[7.95963050000006,48.71858099800005],[7.804824,48.59257949800008],[7.739841500000068,48.321731500000055],[7.680713,48.257266996000055],[7.577919500000064,48.121391998000036],[7.577291,48.11565499800008],[7.568529,48.035183998000036],[7.621966500000042,47.97474899800005],[7.558898500000055,47.881823498000074],[7.545990500000073,47.743573998000045],[7.519558,47.66812299900005],[7.589039,47.58987799800008],[7.555159500000059,47.56456399600006],[7.510905500000035,47.50258249700005],[7.445019,47.46172349600005],[7.421139499000049,47.446387998000034],[7.380894,47.43189249800008],[7.326466,47.43985349900004],[7.170422,47.44345449800005],[7.130353,47.503040498000075],[6.982914,47.49427049800005],[7.00155,47.45374349700006],[6.939186,47.433704499000044],[6.879806,47.35243999700003],[7.061636,47.34373449800006],[6.858914,47.16529449800004],[6.43275,46.92845549800006],[6.460011,46.85155149800005],[6.439425,46.76257299900004],[6.110961,46.576656496000055],[6.138106,46.55766699600008],[6.064003,46.41622899600003],[5.869188500000064,46.26497799800006],[5.725178500000027,46.26068699700005],[5.649801500000024,46.334190498000055],[5.473656500000061,46.264284],[5.310563499000068,46.446769998000036],[5.415741,46.47314249800007],[5.362118500000065,46.57726649800003],[5.440604,46.63791199900004],[5.32739250000003,46.815101998000046],[5.465138500000023,46.840725500000076],[5.328820500000063,46.887597999000036],[5.255232500000034,46.97988749800004],[5.438506,47.14152950000005],[5.518537500000036,47.30418399800004],[5.472388,47.318409],[5.496923,47.388544996000064],[5.375406999000063,47.46016749800003],[5.486405,47.52696849600005],[5.477542,47.60871849800003],[5.37408,47.60453799800007],[5.419852,47.677725999000074],[5.691057500000056,47.684624996000025],[5.681527,47.80911149700006],[5.884723500000064,47.92604649800006],[5.629007,48.08441199600003],[5.730055,48.19831849700006],[5.526527,48.34685349700004],[5.416975500000035,48.34269549600003],[5.409781500000065,48.39261349800006],[5.470055,48.42092649800003],[4.999322,48.61483449900004],[4.98843050000005,48.684422],[5.009218500000031,48.74138899900004],[4.888582,48.80061349600004],[4.918824,48.905575498000076],[5.03953,48.97408949700008],[4.939760500000034,49.18507099800007],[4.993376,49.20962349700005],[4.950990500000046,49.23686749600006],[5.101156,49.37815049800008],[5.059498500000075,49.50056649800007],[5.107566500000075,49.584689498000046],[5.275101,49.54189549700004],[5.393511,49.61711099900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR41","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Lorraine","NUTS_NAME":"Lorraine"},"geometry":{"type":"Polygon","coordinates":[[[5.393511,49.61711099900003],[5.470882999000025,49.49723799800006],[5.734555999000065,49.545690500000035],[5.818117,49.54631049900007],[5.893386,49.49694449700007],[6.367107500000031,49.469507],[6.556986,49.419208498000046],[6.565664,49.34765149700007],[6.723465500000032,49.21882899800005],[6.738397500000076,49.16359799900005],[6.834567,49.15131099800004],[6.837607499000057,49.21293299700005],[6.925060500000029,49.22334649800007],[7.03497150000004,49.19141499700004],[7.057195999000044,49.11193149800005],[7.101069,49.15599799900008],[7.293177,49.11490749700005],[7.36875550000002,49.16145799800006],[7.493791,49.16773550000005],[7.635651,49.05395049900005],[7.536287500000071,48.93241299600004],[7.326848,48.943375497000034],[7.067510500000026,49.06636650000007],[6.954672,48.89389049700003],[7.094149500000071,48.86288299900008],[7.052069,48.84688349900006],[7.07924950000006,48.78628749600006],[7.186039500000049,48.833916499000054],[7.30645,48.76911249700004],[7.242264,48.692105996000066],[7.304349500000058,48.66020399700005],[7.220277500000066,48.55467599900004],[7.079356,48.536418499000035],[7.123163500000032,48.51358799800005],[7.07596250000006,48.35257849900006],[7.198287,48.31047149800003],[7.060182500000053,48.14464249800005],[7.076707500000055,48.108482998000056],[6.944532500000037,47.99993949700007],[6.896905500000059,47.89082899700003],[6.920041,47.85028599700007],[6.846175500000072,47.82294249600005],[6.823533500000053,47.81305099900004],[6.601246500000059,47.94437449800006],[6.478264500000023,47.885443499000075],[6.390344,47.959216998000045],[6.205224500000043,47.93204499800004],[6.147018500000058,48.01552049800006],[5.884723500000064,47.92604649800006],[5.629007,48.08441199600003],[5.730055,48.19831849700006],[5.526527,48.34685349700004],[5.416975500000035,48.34269549600003],[5.409781500000065,48.39261349800006],[5.470055,48.42092649800003],[4.999322,48.61483449900004],[4.98843050000005,48.684422],[5.009218500000031,48.74138899900004],[4.888582,48.80061349600004],[4.918824,48.905575498000076],[5.03953,48.97408949700008],[4.939760500000034,49.18507099800007],[4.993376,49.20962349700005],[4.950990500000046,49.23686749600006],[5.101156,49.37815049800008],[5.059498500000075,49.50056649800007],[5.107566500000075,49.584689498000046],[5.275101,49.54189549700004],[5.393511,49.61711099900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR411","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Meurthe-et-Moselle","NUTS_NAME":"Meurthe-et-Moselle"},"geometry":{"type":"Polygon","coordinates":[[[5.734555999000065,49.545690500000035],[5.818117,49.54631049900007],[5.893386,49.49694449700007],[5.940096,49.477607498000054],[5.911838500000044,49.40753649800007],[5.943703499000037,49.33613699800003],[6.032436,49.25085949800007],[5.978547500000047,49.19499449700004],[6.004980500000045,49.11389499900008],[5.924798,49.09189649900003],[5.957570500000031,49.047821499000065],[6.12585050000007,48.93816799800004],[6.311186,48.921065998000074],[6.370878,48.78470199800006],[7.008736,48.611838499000044],[7.079356,48.536418499000035],[7.123163500000032,48.51358799800005],[6.815033,48.39506849800006],[6.622816,48.47304349700005],[5.980402,48.34896599800004],[5.855381500000021,48.40937299800004],[5.905899,48.494977497000036],[5.765150500000061,48.496489],[5.715484500000059,48.56230049800007],[5.7646,48.70107749700003],[5.714038500000072,48.74462249800007],[5.778495,48.79989949800006],[5.747021500000074,48.91525999600003],[5.847263,48.951786500000026],[5.801011500000072,48.98619700100005],[5.853481499000054,49.04045],[5.746933,49.12460299900005],[5.720934500000055,49.247702],[5.768284,49.29977449800003],[5.692120500000044,49.41611000100005],[5.473180500000069,49.41074949800003],[5.470882999000025,49.49723799800006],[5.734555999000065,49.545690500000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR412","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Meuse","NUTS_NAME":"Meuse"},"geometry":{"type":"Polygon","coordinates":[[[5.470882999000025,49.49723799800006],[5.473180500000069,49.41074949800003],[5.692120500000044,49.41611000100005],[5.768284,49.29977449800003],[5.720934500000055,49.247702],[5.746933,49.12460299900005],[5.853481499000054,49.04045],[5.801011500000072,48.98619700100005],[5.847263,48.951786500000026],[5.747021500000074,48.91525999600003],[5.778495,48.79989949800006],[5.714038500000072,48.74462249800007],[5.7646,48.70107749700003],[5.715484500000059,48.56230049800007],[5.765150500000061,48.496489],[5.470055,48.42092649800003],[4.999322,48.61483449900004],[4.98843050000005,48.684422],[5.009218500000031,48.74138899900004],[4.888582,48.80061349600004],[4.918824,48.905575498000076],[5.03953,48.97408949700008],[4.939760500000034,49.18507099800007],[4.993376,49.20962349700005],[4.950990500000046,49.23686749600006],[5.101156,49.37815049800008],[5.059498500000075,49.50056649800007],[5.107566500000075,49.584689498000046],[5.275101,49.54189549700004],[5.393511,49.61711099900003],[5.470882999000025,49.49723799800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR413","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Moselle","NUTS_NAME":"Moselle"},"geometry":{"type":"Polygon","coordinates":[[[6.367107500000031,49.469507],[6.556986,49.419208498000046],[6.565664,49.34765149700007],[6.723465500000032,49.21882899800005],[6.738397500000076,49.16359799900005],[6.834567,49.15131099800004],[6.837607499000057,49.21293299700005],[6.925060500000029,49.22334649800007],[7.03497150000004,49.19141499700004],[7.057195999000044,49.11193149800005],[7.101069,49.15599799900008],[7.293177,49.11490749700005],[7.36875550000002,49.16145799800006],[7.493791,49.16773550000005],[7.635651,49.05395049900005],[7.536287500000071,48.93241299600004],[7.326848,48.943375497000034],[7.067510500000026,49.06636650000007],[6.954672,48.89389049700003],[7.094149500000071,48.86288299900008],[7.052069,48.84688349900006],[7.07924950000006,48.78628749600006],[7.186039500000049,48.833916499000054],[7.30645,48.76911249700004],[7.242264,48.692105996000066],[7.304349500000058,48.66020399700005],[7.220277500000066,48.55467599900004],[7.079356,48.536418499000035],[7.008736,48.611838499000044],[6.370878,48.78470199800006],[6.311186,48.921065998000074],[6.12585050000007,48.93816799800004],[5.957570500000031,49.047821499000065],[5.924798,49.09189649900003],[6.004980500000045,49.11389499900008],[5.978547500000047,49.19499449700004],[6.032436,49.25085949800007],[5.943703499000037,49.33613699800003],[5.911838500000044,49.40753649800007],[5.940096,49.477607498000054],[5.893386,49.49694449700007],[6.367107500000031,49.469507]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR414","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Vosges","NUTS_NAME":"Vosges"},"geometry":{"type":"Polygon","coordinates":[[[7.198287,48.31047149800003],[7.060182500000053,48.14464249800005],[7.076707500000055,48.108482998000056],[6.944532500000037,47.99993949700007],[6.896905500000059,47.89082899700003],[6.920041,47.85028599700007],[6.846175500000072,47.82294249600005],[6.823533500000053,47.81305099900004],[6.601246500000059,47.94437449800006],[6.478264500000023,47.885443499000075],[6.390344,47.959216998000045],[6.205224500000043,47.93204499800004],[6.147018500000058,48.01552049800006],[5.884723500000064,47.92604649800006],[5.629007,48.08441199600003],[5.730055,48.19831849700006],[5.526527,48.34685349700004],[5.416975500000035,48.34269549600003],[5.409781500000065,48.39261349800006],[5.470055,48.42092649800003],[5.765150500000061,48.496489],[5.905899,48.494977497000036],[5.855381500000021,48.40937299800004],[5.980402,48.34896599800004],[6.622816,48.47304349700005],[6.815033,48.39506849800006],[7.123163500000032,48.51358799800005],[7.07596250000006,48.35257849900006],[7.198287,48.31047149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR42","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Alsace","NUTS_NAME":"Alsace"},"geometry":{"type":"Polygon","coordinates":[[[7.326848,48.943375497000034],[7.536287500000071,48.93241299600004],[7.635651,49.05395049900005],[7.910654500000021,49.04516349800008],[8.068407500000035,48.99931649700005],[8.232632999000032,48.96657150000004],[7.95963050000006,48.71858099800005],[7.804824,48.59257949800008],[7.739841500000068,48.321731500000055],[7.680713,48.257266996000055],[7.577919500000064,48.121391998000036],[7.577291,48.11565499800008],[7.568529,48.035183998000036],[7.621966500000042,47.97474899800005],[7.558898500000055,47.881823498000074],[7.545990500000073,47.743573998000045],[7.519558,47.66812299900005],[7.589039,47.58987799800008],[7.555159500000059,47.56456399600006],[7.510905500000035,47.50258249700005],[7.445019,47.46172349600005],[7.421139499000049,47.446387998000034],[7.380894,47.43189249800008],[7.326466,47.43985349900004],[7.170422,47.44345449800005],[7.130353,47.503040498000075],[7.085383,47.59305199600004],[7.012172,47.60043899800007],[7.037414500000068,47.72155249800005],[6.846175500000072,47.82294249600005],[6.920041,47.85028599700007],[6.896905500000059,47.89082899700003],[6.944532500000037,47.99993949700007],[7.076707500000055,48.108482998000056],[7.060182500000053,48.14464249800005],[7.198287,48.31047149800003],[7.07596250000006,48.35257849900006],[7.123163500000032,48.51358799800005],[7.079356,48.536418499000035],[7.220277500000066,48.55467599900004],[7.304349500000058,48.66020399700005],[7.242264,48.692105996000066],[7.30645,48.76911249700004],[7.186039500000049,48.833916499000054],[7.07924950000006,48.78628749600006],[7.052069,48.84688349900006],[7.094149500000071,48.86288299900008],[6.954672,48.89389049700003],[7.067510500000026,49.06636650000007],[7.326848,48.943375497000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR421","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Bas-Rhin","NUTS_NAME":"Bas-Rhin"},"geometry":{"type":"Polygon","coordinates":[[[7.635651,49.05395049900005],[7.910654500000021,49.04516349800008],[8.068407500000035,48.99931649700005],[8.232632999000032,48.96657150000004],[7.95963050000006,48.71858099800005],[7.804824,48.59257949800008],[7.739841500000068,48.321731500000055],[7.680713,48.257266996000055],[7.577919500000064,48.121391998000036],[7.198287,48.31047149800003],[7.07596250000006,48.35257849900006],[7.123163500000032,48.51358799800005],[7.079356,48.536418499000035],[7.220277500000066,48.55467599900004],[7.304349500000058,48.66020399700005],[7.242264,48.692105996000066],[7.30645,48.76911249700004],[7.186039500000049,48.833916499000054],[7.07924950000006,48.78628749600006],[7.052069,48.84688349900006],[7.094149500000071,48.86288299900008],[6.954672,48.89389049700003],[7.067510500000026,49.06636650000007],[7.326848,48.943375497000034],[7.536287500000071,48.93241299600004],[7.635651,49.05395049900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR422","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haut-Rhin","NUTS_NAME":"Haut-Rhin"},"geometry":{"type":"Polygon","coordinates":[[[7.577919500000064,48.121391998000036],[7.577291,48.11565499800008],[7.568529,48.035183998000036],[7.621966500000042,47.97474899800005],[7.558898500000055,47.881823498000074],[7.545990500000073,47.743573998000045],[7.519558,47.66812299900005],[7.589039,47.58987799800008],[7.555159500000059,47.56456399600006],[7.510905500000035,47.50258249700005],[7.445019,47.46172349600005],[7.421139499000049,47.446387998000034],[7.380894,47.43189249800008],[7.326466,47.43985349900004],[7.170422,47.44345449800005],[7.130353,47.503040498000075],[7.085383,47.59305199600004],[7.012172,47.60043899800007],[7.037414500000068,47.72155249800005],[6.846175500000072,47.82294249600005],[6.920041,47.85028599700007],[6.896905500000059,47.89082899700003],[6.944532500000037,47.99993949700007],[7.076707500000055,48.108482998000056],[7.060182500000053,48.14464249800005],[7.198287,48.31047149800003],[7.577919500000064,48.121391998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR43","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Franche-Comté","NUTS_NAME":"Franche-Comté"},"geometry":{"type":"Polygon","coordinates":[[[6.147018500000058,48.01552049800006],[6.205224500000043,47.93204499800004],[6.390344,47.959216998000045],[6.478264500000023,47.885443499000075],[6.601246500000059,47.94437449800006],[6.823533500000053,47.81305099900004],[6.846175500000072,47.82294249600005],[7.037414500000068,47.72155249800005],[7.012172,47.60043899800007],[7.085383,47.59305199600004],[7.130353,47.503040498000075],[6.982914,47.49427049800005],[7.00155,47.45374349700006],[6.939186,47.433704499000044],[6.879806,47.35243999700003],[7.061636,47.34373449800006],[6.858914,47.16529449800004],[6.43275,46.92845549800006],[6.460011,46.85155149800005],[6.439425,46.76257299900004],[6.110961,46.576656496000055],[6.138106,46.55766699600008],[6.064003,46.41622899600003],[5.869188500000064,46.26497799800006],[5.725178500000027,46.26068699700005],[5.649801500000024,46.334190498000055],[5.473656500000061,46.264284],[5.310563499000068,46.446769998000036],[5.415741,46.47314249800007],[5.362118500000065,46.57726649800003],[5.440604,46.63791199900004],[5.32739250000003,46.815101998000046],[5.465138500000023,46.840725500000076],[5.328820500000063,46.887597999000036],[5.255232500000034,46.97988749800004],[5.438506,47.14152950000005],[5.518537500000036,47.30418399800004],[5.472388,47.318409],[5.496923,47.388544996000064],[5.375406999000063,47.46016749800003],[5.486405,47.52696849600005],[5.477542,47.60871849800003],[5.37408,47.60453799800007],[5.419852,47.677725999000074],[5.691057500000056,47.684624996000025],[5.681527,47.80911149700006],[5.884723500000064,47.92604649800006],[6.147018500000058,48.01552049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR431","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Doubs","NUTS_NAME":"Doubs"},"geometry":{"type":"Polygon","coordinates":[[[6.80699850000002,47.56279999600008],[6.945001500000046,47.51334049900004],[6.939186,47.433704499000044],[6.879806,47.35243999700003],[7.061636,47.34373449800006],[6.858914,47.16529449800004],[6.43275,46.92845549800006],[6.460011,46.85155149800005],[6.439425,46.76257299900004],[6.110961,46.576656496000055],[6.138106,46.55766699600008],[6.048568,46.60771899700006],[6.105520500000068,46.65047999900003],[6.069826,46.689527498000075],[6.20333,46.772681497000065],[6.022699,46.85259049700005],[5.968911,46.976833998000075],[5.744114,47.02240149800008],[5.811472500000036,47.169050998000046],[5.698727,47.26500050000004],[6.241565500000036,47.41074049800005],[6.33156850000006,47.50345199800006],[6.80699850000002,47.56279999600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR432","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Jura","NUTS_NAME":"Jura"},"geometry":{"type":"Polygon","coordinates":[[[5.698727,47.26500050000004],[5.811472500000036,47.169050998000046],[5.744114,47.02240149800008],[5.968911,46.976833998000075],[6.022699,46.85259049700005],[6.20333,46.772681497000065],[6.069826,46.689527498000075],[6.105520500000068,46.65047999900003],[6.048568,46.60771899700006],[6.138106,46.55766699600008],[6.064003,46.41622899600003],[5.869188500000064,46.26497799800006],[5.725178500000027,46.26068699700005],[5.649801500000024,46.334190498000055],[5.473656500000061,46.264284],[5.310563499000068,46.446769998000036],[5.415741,46.47314249800007],[5.362118500000065,46.57726649800003],[5.440604,46.63791199900004],[5.32739250000003,46.815101998000046],[5.465138500000023,46.840725500000076],[5.328820500000063,46.887597999000036],[5.255232500000034,46.97988749800004],[5.438506,47.14152950000005],[5.518537500000036,47.30418399800004],[5.698727,47.26500050000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR433","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Saône","NUTS_NAME":"Haute-Saône"},"geometry":{"type":"Polygon","coordinates":[[[6.823533500000053,47.81305099900004],[6.757641,47.747875],[6.80699850000002,47.56279999600008],[6.33156850000006,47.50345199800006],[6.241565500000036,47.41074049800005],[5.698727,47.26500050000004],[5.518537500000036,47.30418399800004],[5.472388,47.318409],[5.496923,47.388544996000064],[5.375406999000063,47.46016749800003],[5.486405,47.52696849600005],[5.477542,47.60871849800003],[5.37408,47.60453799800007],[5.419852,47.677725999000074],[5.691057500000056,47.684624996000025],[5.681527,47.80911149700006],[5.884723500000064,47.92604649800006],[6.147018500000058,48.01552049800006],[6.205224500000043,47.93204499800004],[6.390344,47.959216998000045],[6.478264500000023,47.885443499000075],[6.601246500000059,47.94437449800006],[6.823533500000053,47.81305099900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR434","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Territoire de Belfort","NUTS_NAME":"Territoire de Belfort"},"geometry":{"type":"Polygon","coordinates":[[[7.130353,47.503040498000075],[6.982914,47.49427049800005],[7.00155,47.45374349700006],[6.939186,47.433704499000044],[6.945001500000046,47.51334049900004],[6.80699850000002,47.56279999600008],[6.757641,47.747875],[6.823533500000053,47.81305099900004],[6.846175500000072,47.82294249600005],[7.037414500000068,47.72155249800005],[7.012172,47.60043899800007],[7.085383,47.59305199600004],[7.130353,47.503040498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR5","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"OUEST","NUTS_NAME":"OUEST"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-3.084001,48.865695998000035],[-2.686398499999939,48.49315249800003],[-2.3278,48.67327899800006],[-2.210832,48.57585899800006],[-2.123706499999969,48.604404498000065],[-2.148958,48.63373199600005],[-2.045604,48.63681399600006],[-2.006897,48.56610849700007],[-1.982181074999971,48.554643027000054],[-2.028575,48.65031049800007],[-1.850131,48.69939399900005],[-1.872124,48.645713998000076],[-1.571089499999971,48.626441999000065],[-1.42794,48.461915],[-1.249492500999963,48.54369199800004],[-1.070164499999976,48.50849199900006],[-0.86036,48.501458498000034],[-0.654219,48.44454249900008],[-0.234124,48.561847500000056],[-0.144602499999962,48.527754498000036],[-0.148717499999975,48.458068997000055],[-0.051349,48.452927],[-0.054527,48.38200449900006],[0.35289,48.45968799900004],[0.395541,48.32046199800004],[0.797658499000022,48.194455],[0.91665050000006,48.13893649800008],[0.841217499000038,48.10305950000003],[0.793996500000048,48.06944],[0.848432500000058,47.942154498000036],[0.580504500000075,47.71282549800003],[0.614432500000021,47.69421549600003],[0.382790500000056,47.64309749800003],[0.378485,47.568527998000036],[0.230000500000074,47.608397498000045],[0.05383,47.163733499000045],[0.175696,47.06072749700007],[0.305386500000054,47.052699500000074],[0.32483650000006,46.93065199800003],[0.599612500000035,46.95628899800005],[0.567741,47.00628399800007],[0.692567,46.97430449800004],[0.704727499000057,46.902806998000074],[0.867469,46.74821649800003],[0.927891,46.69264899800004],[0.915865500000052,46.59662799800003],[1.149148500000024,46.502205496000045],[1.213061,46.43307699700006],[1.177279,46.383947997000064],[0.810388500000045,46.22950949800003],[0.823432500000024,46.128584498000066],[0.817911,46.04788149600006],[0.942597,45.957437998000046],[0.835689500000058,45.92272399800004],[0.783778500000039,45.79347549700003],[0.710518500000035,45.802201498000045],[0.629741,45.71456999600008],[0.486204500000042,45.538295498000025],[0.302616999000065,45.45888249600006],[0.247881500000062,45.36927249800004],[0.266603,45.29774599900003],[0.004336,45.19162799900005],[-0.040197499999977,45.10237999800006],[-0.258733,45.114627496000026],[-0.379539499999964,45.155939998000065],[-0.425493,45.27863649600005],[-0.708231,45.32748049800006],[-0.819719499999962,45.49202749800003],[-1.230697499999962,45.68057249700007],[-1.241852,45.785945998000045],[-1.13209,45.80492799900003],[-1.155564,45.841975999000056],[-1.053074,46.00382999900006],[-1.123773,46.11335349800004],[-1.242043499999966,46.15744399700003],[-1.11135,46.261157998000044],[-1.129406,46.310271998000076],[-1.202044,46.31608599800006],[-1.204990499999951,46.26650599800007],[-1.812344499999938,46.49341949700005],[-1.857028,46.60985199600003],[-2.141985,46.81897349700006],[-2.149297718999946,46.89421903900006],[-2.307551,47.024879498000075],[-2.218334,47.00781999800006],[-2.148764873999937,46.89464346800003],[-1.980413,47.02890799800008],[-2.247857,47.13375099800004],[-2.166129,47.16842249900003],[-2.169989499999929,47.268470998000055],[-2.010275000999968,47.31473899700006],[-2.297556499999928,47.23431399900005],[-2.54710800099997,47.292373498000075],[-2.500607499999944,47.323757],[-2.558945,47.37456899800003],[-2.43403,47.413044],[-2.458493,47.44811999800004],[-2.501003,47.49271749900004],[-2.420451499999956,47.49520099800003],[-2.849214,47.49888249600008],[-2.919619,47.547725498000034],[-2.730782,47.54229749900003],[-2.721059,47.60310749700005],[-2.898194,47.58569699700007],[-2.98203,47.65580749800006],[-2.971034,47.55858999600008],[-3.128996499999971,47.59746149600005],[-3.089519499999938,47.47268299900003],[-3.130722499999933,47.47313299800004],[-3.158280499999933,47.60671599700004],[-3.338828,47.695411498000055],[-3.300046499999951,47.774661998000056],[-3.453312500999971,47.69553749600004],[-3.528605312999957,47.77829169600005],[-3.850509,47.79721049800003],[-3.958664499999941,47.896976496000036],[-4.099355499999945,47.86249150000003],[-4.144733499999973,47.909335998000074],[-4.115504,47.976466998000035],[-4.151007499999935,47.910702],[-4.113312,47.86203749800006],[-4.176410499999974,47.80505749800005],[-4.374185,47.79797],[-4.347687,47.849003],[-4.42255849999998,47.962432998000054],[-4.734697499999925,48.038242497000056],[-4.269574,48.13277050000005],[-4.303489,48.195732],[-4.465324499999952,48.238849499000025],[-4.557571,48.16983049800007],[-4.545083,48.25028599800004],[-4.625327500999958,48.259071498000026],[-4.554775,48.338588498000036],[-4.535044,48.28407649800005],[-4.225793499999952,48.28749099600003],[-4.332875499999943,48.31668099900003],[-4.261421499999926,48.360431498000025],[-4.45806,48.32748799800004],[-4.273602500999971,48.44343549900003],[-4.772754,48.32923499900005],[-4.769921499999953,48.52113699800003],[-4.349134,48.67647549900005],[-4.281066499999952,48.642818498000054],[-3.96533,48.72175999800004],[-3.950495,48.65286999700004],[-3.859493,48.61433799800005],[-3.831078499999933,48.712436498000045],[-3.643659499999956,48.69629649800004],[-3.659150499999953,48.65920999800005],[-3.573228,48.674724498000046],[-3.549404,48.74568549900005],[-3.584439499999974,48.79017999800004],[-3.504829499999971,48.83930599800004],[-3.43019,48.797252498000034],[-3.084001,48.865695998000035]]],[[[-1.195956,45.828918498000064],[-1.225263,45.804081],[-1.387045499999942,45.95379250000008],[-1.413434499999937,46.04686349800005],[-1.245950000999926,45.98994049800007],[-1.195956,45.828918498000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR51","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Pays de la Loire","NUTS_NAME":"Pays de la Loire"},"geometry":{"type":"Polygon","coordinates":[[[-0.051349,48.452927],[-0.054527,48.38200449900006],[0.35289,48.45968799900004],[0.395541,48.32046199800004],[0.797658499000022,48.194455],[0.91665050000006,48.13893649800008],[0.841217499000038,48.10305950000003],[0.793996500000048,48.06944],[0.848432500000058,47.942154498000036],[0.580504500000075,47.71282549800003],[0.614432500000021,47.69421549600003],[0.382790500000056,47.64309749800003],[0.378485,47.568527998000036],[0.230000500000074,47.608397498000045],[0.05383,47.163733499000045],[-0.102116,47.06479999700008],[-0.493149,47.08338599700005],[-0.595493499999975,46.997910500000046],[-0.891964,46.975820498000076],[-0.656161499999939,46.70030299800004],[-0.613968,46.62012449900004],[-0.610179499999958,46.41373299800006],[-0.537795000999949,46.38646399800007],[-0.750471499999946,46.304254498000034],[-0.963101,46.32262099900004],[-0.936756499999944,46.36638649900004],[-1.129406,46.310271998000076],[-1.202044,46.31608599800006],[-1.204990499999951,46.26650599800007],[-1.812344499999938,46.49341949700005],[-1.857028,46.60985199600003],[-2.141985,46.81897349700006],[-2.149297718999946,46.89421903900006],[-2.307551,47.024879498000075],[-2.218334,47.00781999800006],[-2.148764873999937,46.89464346800003],[-1.980413,47.02890799800008],[-2.247857,47.13375099800004],[-2.166129,47.16842249900003],[-2.169989499999929,47.268470998000055],[-2.010275000999968,47.31473899700006],[-2.297556499999928,47.23431399900005],[-2.54710800099997,47.292373498000075],[-2.500607499999944,47.323757],[-2.558945,47.37456899800003],[-2.43403,47.413044],[-2.458493,47.44811999800004],[-2.098850499999969,47.53345099900008],[-2.097034,47.63135649800006],[-1.973398,47.69415349600007],[-1.657515,47.709435],[-1.424917,47.83284149900004],[-1.245885,47.77671749800004],[-1.238248,47.80999249600006],[-1.156485,47.96429099800008],[-1.015823500999943,48.00328599800008],[-1.101965,48.26250599800005],[-1.045019499999967,48.327728498000056],[-1.070164499999976,48.50849199900006],[-0.86036,48.501458498000034],[-0.654219,48.44454249900008],[-0.234124,48.561847500000056],[-0.144602499999962,48.527754498000036],[-0.148717499999975,48.458068997000055],[-0.051349,48.452927]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR511","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Loire-Atlantique","NUTS_NAME":"Loire-Atlantique"},"geometry":{"type":"Polygon","coordinates":[[[-1.245885,47.77671749800004],[-1.137899499999946,47.61892699800006],[-1.006839500999945,47.588742497000055],[-1.172936000999925,47.57281149800008],[-1.157446499999935,47.51020899800005],[-0.965485499999943,47.491797500000075],[-0.923462499999971,47.39877299800003],[-1.354199,47.30415399800006],[-1.181111,47.240895497000054],[-1.161776499999974,47.18583449800008],[-1.238224,47.13490399600005],[-1.115534499999967,47.062693499000034],[-1.148567,47.02954899900004],[-1.200686,47.04002299800004],[-1.268303499999945,47.08475049600003],[-1.366064,47.041701999000054],[-1.364036499999941,46.957828498000026],[-1.458267499999977,46.92583849700003],[-1.472179,47.03134349700008],[-1.554803499999935,47.01627949700003],[-1.500797499999976,46.88339049800004],[-1.553118,46.86298449900005],[-1.722465499999942,46.883816],[-1.980413,47.02890799800008],[-2.247857,47.13375099800004],[-2.166129,47.16842249900003],[-2.169989499999929,47.268470998000055],[-2.010275000999968,47.31473899700006],[-2.297556499999928,47.23431399900005],[-2.54710800099997,47.292373498000075],[-2.500607499999944,47.323757],[-2.558945,47.37456899800003],[-2.43403,47.413044],[-2.458493,47.44811999800004],[-2.098850499999969,47.53345099900008],[-2.097034,47.63135649800006],[-1.973398,47.69415349600007],[-1.657515,47.709435],[-1.424917,47.83284149900004],[-1.245885,47.77671749800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR512","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Maine-et-Loire","NUTS_NAME":"Maine-et-Loire"},"geometry":{"type":"Polygon","coordinates":[[[-0.381701000999954,47.76056650000004],[-0.134504499999935,47.632488497000054],[0.230000500000074,47.608397498000045],[0.05383,47.163733499000045],[-0.102116,47.06479999700008],[-0.493149,47.08338599700005],[-0.595493499999975,46.997910500000046],[-0.891964,46.975820498000076],[-1.148567,47.02954899900004],[-1.115534499999967,47.062693499000034],[-1.238224,47.13490399600005],[-1.161776499999974,47.18583449800008],[-1.181111,47.240895497000054],[-1.354199,47.30415399800006],[-0.923462499999971,47.39877299800003],[-0.965485499999943,47.491797500000075],[-1.157446499999935,47.51020899800005],[-1.172936000999925,47.57281149800008],[-1.006839500999945,47.588742497000055],[-1.137899499999946,47.61892699800006],[-1.245885,47.77671749800004],[-1.238248,47.80999249600006],[-0.653782499999977,47.73337949800003],[-0.381701000999954,47.76056650000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR513","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Mayenne","NUTS_NAME":"Mayenne"},"geometry":{"type":"Polygon","coordinates":[[[-0.054527,48.38200449900006],[-0.157094,48.340877497000065],[-0.146834499999954,48.205006998000044],[-0.252555,48.15738249800006],[-0.230941,48.05898099800004],[-0.341665,48.043097500000044],[-0.276567499999942,47.95742450000006],[-0.406543,47.91688],[-0.370745499999941,47.859583996000026],[-0.447918,47.83203199700006],[-0.381701000999954,47.76056650000004],[-0.653782499999977,47.73337949800003],[-1.238248,47.80999249600006],[-1.156485,47.96429099800008],[-1.015823500999943,48.00328599800008],[-1.101965,48.26250599800005],[-1.045019499999967,48.327728498000056],[-1.070164499999976,48.50849199900006],[-0.86036,48.501458498000034],[-0.654219,48.44454249900008],[-0.234124,48.561847500000056],[-0.144602499999962,48.527754498000036],[-0.148717499999975,48.458068997000055],[-0.051349,48.452927],[-0.054527,48.38200449900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR514","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Sarthe","NUTS_NAME":"Sarthe"},"geometry":{"type":"Polygon","coordinates":[[[0.797658499000022,48.194455],[0.91665050000006,48.13893649800008],[0.841217499000038,48.10305950000003],[0.793996500000048,48.06944],[0.848432500000058,47.942154498000036],[0.580504500000075,47.71282549800003],[0.614432500000021,47.69421549600003],[0.382790500000056,47.64309749800003],[0.378485,47.568527998000036],[0.230000500000074,47.608397498000045],[-0.134504499999935,47.632488497000054],[-0.381701000999954,47.76056650000004],[-0.447918,47.83203199700006],[-0.370745499999941,47.859583996000026],[-0.406543,47.91688],[-0.276567499999942,47.95742450000006],[-0.341665,48.043097500000044],[-0.230941,48.05898099800004],[-0.252555,48.15738249800006],[-0.146834499999954,48.205006998000044],[-0.157094,48.340877497000065],[-0.054527,48.38200449900006],[0.35289,48.45968799900004],[0.395541,48.32046199800004],[0.797658499000022,48.194455]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR515","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Vendée","NUTS_NAME":"Vendée"},"geometry":{"type":"Polygon","coordinates":[[[-1.200686,47.04002299800004],[-1.148567,47.02954899900004],[-0.891964,46.975820498000076],[-0.656161499999939,46.70030299800004],[-0.613968,46.62012449900004],[-0.610179499999958,46.41373299800006],[-0.537795000999949,46.38646399800007],[-0.750471499999946,46.304254498000034],[-0.963101,46.32262099900004],[-0.936756499999944,46.36638649900004],[-1.129406,46.310271998000076],[-1.202044,46.31608599800006],[-1.204990499999951,46.26650599800007],[-1.812344499999938,46.49341949700005],[-1.857028,46.60985199600003],[-2.141985,46.81897349700006],[-2.149297718999946,46.89421903900006],[-2.307551,47.024879498000075],[-2.218334,47.00781999800006],[-2.148764873999937,46.89464346800003],[-1.980413,47.02890799800008],[-1.722465499999942,46.883816],[-1.553118,46.86298449900005],[-1.500797499999976,46.88339049800004],[-1.554803499999935,47.01627949700003],[-1.472179,47.03134349700008],[-1.458267499999977,46.92583849700003],[-1.364036499999941,46.957828498000026],[-1.366064,47.041701999000054],[-1.268303499999945,47.08475049600003],[-1.200686,47.04002299800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR52","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Bretagne","NUTS_NAME":"Bretagne"},"geometry":{"type":"Polygon","coordinates":[[[-3.084001,48.865695998000035],[-2.686398499999939,48.49315249800003],[-2.3278,48.67327899800006],[-2.210832,48.57585899800006],[-2.123706499999969,48.604404498000065],[-2.148958,48.63373199600005],[-2.045604,48.63681399600006],[-2.006897,48.56610849700007],[-1.982181074999971,48.554643027000054],[-2.028575,48.65031049800007],[-1.850131,48.69939399900005],[-1.872124,48.645713998000076],[-1.571089499999971,48.626441999000065],[-1.42794,48.461915],[-1.249492500999963,48.54369199800004],[-1.070164499999976,48.50849199900006],[-1.045019499999967,48.327728498000056],[-1.101965,48.26250599800005],[-1.015823500999943,48.00328599800008],[-1.156485,47.96429099800008],[-1.238248,47.80999249600006],[-1.245885,47.77671749800004],[-1.424917,47.83284149900004],[-1.657515,47.709435],[-1.973398,47.69415349600007],[-2.097034,47.63135649800006],[-2.098850499999969,47.53345099900008],[-2.458493,47.44811999800004],[-2.501003,47.49271749900004],[-2.420451499999956,47.49520099800003],[-2.849214,47.49888249600008],[-2.919619,47.547725498000034],[-2.730782,47.54229749900003],[-2.721059,47.60310749700005],[-2.898194,47.58569699700007],[-2.98203,47.65580749800006],[-2.971034,47.55858999600008],[-3.128996499999971,47.59746149600005],[-3.089519499999938,47.47268299900003],[-3.130722499999933,47.47313299800004],[-3.158280499999933,47.60671599700004],[-3.338828,47.695411498000055],[-3.300046499999951,47.774661998000056],[-3.453312500999971,47.69553749600004],[-3.528605312999957,47.77829169600005],[-3.850509,47.79721049800003],[-3.958664499999941,47.896976496000036],[-4.099355499999945,47.86249150000003],[-4.144733499999973,47.909335998000074],[-4.115504,47.976466998000035],[-4.151007499999935,47.910702],[-4.113312,47.86203749800006],[-4.176410499999974,47.80505749800005],[-4.374185,47.79797],[-4.347687,47.849003],[-4.42255849999998,47.962432998000054],[-4.734697499999925,48.038242497000056],[-4.269574,48.13277050000005],[-4.303489,48.195732],[-4.465324499999952,48.238849499000025],[-4.557571,48.16983049800007],[-4.545083,48.25028599800004],[-4.625327500999958,48.259071498000026],[-4.554775,48.338588498000036],[-4.535044,48.28407649800005],[-4.225793499999952,48.28749099600003],[-4.332875499999943,48.31668099900003],[-4.261421499999926,48.360431498000025],[-4.45806,48.32748799800004],[-4.273602500999971,48.44343549900003],[-4.772754,48.32923499900005],[-4.769921499999953,48.52113699800003],[-4.349134,48.67647549900005],[-4.281066499999952,48.642818498000054],[-3.96533,48.72175999800004],[-3.950495,48.65286999700004],[-3.859493,48.61433799800005],[-3.831078499999933,48.712436498000045],[-3.643659499999956,48.69629649800004],[-3.659150499999953,48.65920999800005],[-3.573228,48.674724498000046],[-3.549404,48.74568549900005],[-3.584439499999974,48.79017999800004],[-3.504829499999971,48.83930599800004],[-3.43019,48.797252498000034],[-3.084001,48.865695998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR521","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Côtes-d'Armor","NUTS_NAME":"Côtes-d'Armor"},"geometry":{"type":"Polygon","coordinates":[[[-3.084001,48.865695998000035],[-2.686398499999939,48.49315249800003],[-2.3278,48.67327899800006],[-2.210832,48.57585899800006],[-2.123706499999969,48.604404498000065],[-2.148958,48.63373199600005],[-2.045604,48.63681399600006],[-2.006897,48.56610849700007],[-1.982181074999971,48.554643027000054],[-1.908992,48.48155399800004],[-1.958049499999959,48.293119500000046],[-2.186751499999957,48.24778649800004],[-2.287274,48.133744],[-2.53434,48.15693949900003],[-2.548678499999937,48.07647699600005],[-2.655765499999973,48.03254099800006],[-2.660977000999935,48.13062149800004],[-3.011761499999977,48.17214850000005],[-3.564844499999936,48.18574749700008],[-3.52486,48.28117099600007],[-3.554302,48.378858500000035],[-3.607688,48.38832149900003],[-3.604373499999951,48.49620199600008],[-3.539283,48.53825099800008],[-3.659150499999953,48.65920999800005],[-3.573228,48.674724498000046],[-3.549404,48.74568549900005],[-3.584439499999974,48.79017999800004],[-3.504829499999971,48.83930599800004],[-3.43019,48.797252498000034],[-3.084001,48.865695998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES61","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Andalucía","NUTS_NAME":"Andalucía"},"geometry":{"type":"Polygon","coordinates":[[[-4.453019,38.40091699900006],[-4.268895,38.34721299700004],[-4.253038499999946,38.40123],[-3.620483,38.39401999700004],[-3.583084,38.45150349800008],[-3.453166499999952,38.400233996000054],[-3.33534949999995,38.480529999000055],[-3.130193,38.438796997000054],[-3.062778,38.47640249800003],[-3.002507499999979,38.40828699700006],[-2.992628,38.450976997000055],[-2.762069,38.532779498000025],[-2.604288,38.51290899800006],[-2.566616,38.490489997000054],[-2.571873,38.41415049600005],[-2.481576,38.39310099900007],[-2.446249499999965,38.18521099700007],[-2.551274000999967,38.08411799900006],[-2.341602,38.02601999800004],[-2.207631499999934,37.91658799800007],[-1.974468,37.86848049900004],[-2.011892499999931,37.67254649700004],[-1.976776499999971,37.61873249900003],[-1.845223499999975,37.454826497000056],[-1.630033,37.37518299800007],[-1.806549,37.214004497000076],[-1.903591,36.937160498000026],[-2.12434,36.731842],[-2.192171000999963,36.72158499900007],[-2.368091499999935,36.84145749800007],[-2.562035,36.81660099700008],[-2.700591,36.68272399700004],[-3.128691,36.75088799800005],[-3.420361,36.694095498000024],[-3.777457499999969,36.737926499000025],[-4.418207,36.717219998000076],[-4.638161499999967,36.50709149800008],[-4.936012499999947,36.499389499000074],[-5.176345499999968,36.414661498000044],[-5.252405,36.31127549800004],[-5.339225,36.152034997000044],[-5.35152249999993,36.15256899800005],[-5.424657499999967,36.17650199800005],[-5.430406,36.071304499000064],[-5.60843,36.007053496000026],[-5.797297499999956,36.07829699800004],[-5.914955499999962,36.18479549800003],[-6.040427499999964,36.19237899800004],[-6.308641499999965,36.532573497000044],[-6.229004,36.462669497000036],[-6.173302000999968,36.521442498000056],[-6.245686,36.50899499800005],[-6.232875,36.584945497000035],[-6.387676,36.62863149800006],[-6.440116,36.72086349700004],[-6.345561,36.79876699700003],[-6.514539499999955,36.97180549800004],[-6.893693499999927,37.16146499900003],[-7.051858,37.20842349800006],[-7.40191649999997,37.174827497000024],[-7.463588499999958,37.485454498000024],[-7.51269149999996,37.526256498000066],[-7.418441,37.74499499800004],[-7.279641,37.876762496000026],[-7.263284499999941,37.979907999000034],[-7.108363,38.040984998000056],[-7.002483499999926,38.02271649900007],[-6.931738499999938,38.20837799700007],[-6.795174499999973,38.17859649800005],[-6.80790449999995,38.11116399700006],[-6.622533,38.09721399700004],[-6.495118,38.01918399700003],[-6.36732,38.04896899700003],[-6.180307499999969,37.941077997000036],[-5.953754,37.99539549900004],[-5.874359,38.158622997000066],[-5.723288499999967,38.19248949800004],[-5.684302499999944,38.15711199800006],[-5.740212,38.13013049800003],[-5.734962499999938,38.086303497000074],[-5.584845499999972,38.13175199900007],[-5.521041499999967,38.20727149800007],[-5.56897749999996,38.432639997000024],[-5.185018,38.65946599700004],[-5.166655,38.71171549600007],[-5.046996000999968,38.72913349700008],[-5.008071499999971,38.71527499800004],[-4.877013499999975,38.68614199800004],[-4.860213499999929,38.61402499800005],[-4.705425499999933,38.57075099600007],[-4.453019,38.40091699900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES611","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Almería","NUTS_NAME":"Almería"},"geometry":{"type":"Polygon","coordinates":[[[-1.630033,37.37518299800007],[-1.806549,37.214004497000076],[-1.903591,36.937160498000026],[-2.12434,36.731842],[-2.192171000999963,36.72158499900007],[-2.368091499999935,36.84145749800007],[-2.562035,36.81660099700008],[-2.700591,36.68272399700004],[-3.128691,36.75088799800005],[-3.140185499999973,36.78778449600003],[-3.028999,36.84443650000003],[-3.057178499999964,36.91154099800008],[-2.976204499999938,36.97812649600007],[-3.021972499999947,37.098922498000036],[-2.948956500999941,37.096557500000074],[-2.844565,37.27943049700008],[-2.660100499999942,37.21200199800006],[-2.635610499999927,37.394221498000036],[-2.358488499999964,37.52230449800004],[-2.37595749999997,37.612269998000045],[-2.319931,37.648101498000074],[-2.27675249999993,37.77622999700003],[-2.294914,37.87633149800007],[-2.207631499999934,37.91658799800007],[-1.974468,37.86848049900004],[-2.011892499999931,37.67254649700004],[-1.976776499999971,37.61873249900003],[-1.845223499999975,37.454826497000056],[-1.630033,37.37518299800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES612","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Cádiz","NUTS_NAME":"Cádiz"},"geometry":{"type":"Polygon","coordinates":[[[-5.142423,37.00389099900008],[-5.086809,36.90511699800004],[-5.134113499999955,36.838592497000036],[-5.301629000999981,36.86967099700007],[-5.34076,36.81944299700007],[-5.29098,36.77519599900006],[-5.343394,36.659950500000036],[-5.522210499999971,36.53181049700004],[-5.611827499999947,36.54162999700003],[-5.524297,36.497738],[-5.429368499999953,36.533180497000046],[-5.320127500999945,36.32508499800008],[-5.252405,36.31127549800004],[-5.339225,36.152034997000044],[-5.35152249999993,36.15256899800005],[-5.424657499999967,36.17650199800005],[-5.430406,36.071304499000064],[-5.60843,36.007053496000026],[-5.797297499999956,36.07829699800004],[-5.914955499999962,36.18479549800003],[-6.040427499999964,36.19237899800004],[-6.308641499999965,36.532573497000044],[-6.229004,36.462669497000036],[-6.173302000999968,36.521442498000056],[-6.245686,36.50899499800005],[-6.232875,36.584945497000035],[-6.387676,36.62863149800006],[-6.440116,36.72086349700004],[-6.345561,36.79876699700003],[-6.331072499999948,36.898837997000044],[-5.893642499999942,36.85161599700007],[-5.845565499999964,36.915245],[-5.717068500999972,36.91954799700005],[-5.666174499999954,36.975772498000026],[-5.533873,36.89348199800003],[-5.550906,36.923152997000045],[-5.469782499999951,36.983978498000056],[-5.511037499999929,37.033634498000026],[-5.431367000999956,37.05042599700005],[-5.438999,36.93531799700003],[-5.302151499999979,37.037395497000034],[-5.235064000999955,36.94121149700004],[-5.142423,37.00389099900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES613","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Córdoba","NUTS_NAME":"Córdoba"},"geometry":{"type":"Polygon","coordinates":[[[-5.008071499999971,38.71527499800004],[-4.877013499999975,38.68614199800004],[-4.860213499999929,38.61402499800005],[-4.705425499999933,38.57075099600007],[-4.453019,38.40091699900006],[-4.268895,38.34721299700004],[-4.21024,38.30149099700003],[-4.163972,38.14466099800006],[-4.282274499999971,38.02284999800003],[-4.286759,37.796577497000044],[-4.206599499999925,37.77876649800004],[-4.238501499999927,37.719833497000025],[-4.16438949999997,37.68153399700003],[-4.220159499999966,37.620269998000026],[-4.000835,37.40211499800006],[-4.23643949999996,37.33588049700006],[-4.255331,37.22958749700007],[-4.327640499999973,37.18435649800006],[-4.378304499999956,37.27245349900005],[-4.539496499999927,37.21133399800004],[-4.653374499999927,37.25585549700003],[-4.671938499999953,37.33501799700008],[-4.782256,37.32853699700007],[-4.88477649999993,37.43801899600004],[-4.999864,37.70663049900003],[-5.358010499999978,37.584884498000065],[-5.411963000999947,37.68354799800005],[-5.309434,37.69810099800003],[-5.339817499999981,37.83357250000006],[-5.408625,37.86917099800007],[-5.415665,37.94372199800006],[-5.584845499999972,38.13175199900007],[-5.521041499999967,38.20727149800007],[-5.56897749999996,38.432639997000024],[-5.185018,38.65946599700004],[-5.166655,38.71171549600007],[-5.046996000999968,38.72913349700008],[-5.008071499999971,38.71527499800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES614","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Granada","NUTS_NAME":"Granada"},"geometry":{"type":"Polygon","coordinates":[[[-2.341602,38.02601999800004],[-2.207631499999934,37.91658799800007],[-2.294914,37.87633149800007],[-2.27675249999993,37.77622999700003],[-2.319931,37.648101498000074],[-2.37595749999997,37.612269998000045],[-2.358488499999964,37.52230449800004],[-2.635610499999927,37.394221498000036],[-2.660100499999942,37.21200199800006],[-2.844565,37.27943049700008],[-2.948956500999941,37.096557500000074],[-3.021972499999947,37.098922498000036],[-2.976204499999938,36.97812649600007],[-3.057178499999964,36.91154099800008],[-3.028999,36.84443650000003],[-3.140185499999973,36.78778449600003],[-3.128691,36.75088799800005],[-3.420361,36.694095498000024],[-3.777457499999969,36.737926499000025],[-3.768160499999965,36.80196349900007],[-3.827184000999978,36.85554099900003],[-4.270058,37.027057497000044],[-4.327640499999973,37.18435649800006],[-4.255331,37.22958749700007],[-4.23643949999996,37.33588049700006],[-4.000835,37.40211499800006],[-3.84225749999996,37.38577249700006],[-3.791726499999925,37.46620549800008],[-3.506552499999941,37.61724099800006],[-3.399311499999953,37.56406799700005],[-3.215751499999953,37.64420349900007],[-2.97818,37.60276799600007],[-2.884035499999925,37.68133549800007],[-2.903782499999977,37.75118999800003],[-2.784602,37.91767099700007],[-2.619673,37.988567498000066],[-2.551274000999967,38.08411799900006],[-2.341602,38.02601999800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES615","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Huelva","NUTS_NAME":"Huelva"},"geometry":{"type":"Polygon","coordinates":[[[-6.180307499999969,37.941077997000036],[-6.125164,37.903388998000025],[-6.134084499999972,37.78788349800004],[-6.390103499999952,37.74774149800004],[-6.536165499999981,37.65102349800003],[-6.395199,37.611061],[-6.306702499999972,37.43511949800006],[-6.384643499999981,37.33753599700003],[-6.333069499999965,37.29914849700003],[-6.385999,37.072017997000046],[-6.354009499999961,36.98852949900004],[-6.378956499999958,36.93474199800005],[-6.331072499999948,36.898837997000044],[-6.345561,36.79876699700003],[-6.514539499999955,36.97180549800004],[-6.893693499999927,37.16146499900003],[-7.051858,37.20842349800006],[-7.40191649999997,37.174827497000024],[-7.463588499999958,37.485454498000024],[-7.51269149999996,37.526256498000066],[-7.418441,37.74499499800004],[-7.279641,37.876762496000026],[-7.263284499999941,37.979907999000034],[-7.108363,38.040984998000056],[-7.002483499999926,38.02271649900007],[-6.931738499999938,38.20837799700007],[-6.795174499999973,38.17859649800005],[-6.80790449999995,38.11116399700006],[-6.622533,38.09721399700004],[-6.495118,38.01918399700003],[-6.36732,38.04896899700003],[-6.180307499999969,37.941077997000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES616","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Jaén","NUTS_NAME":"Jaén"},"geometry":{"type":"Polygon","coordinates":[[[-2.551274000999967,38.08411799900006],[-2.619673,37.988567498000066],[-2.784602,37.91767099700007],[-2.903782499999977,37.75118999800003],[-2.884035499999925,37.68133549800007],[-2.97818,37.60276799600007],[-3.215751499999953,37.64420349900007],[-3.399311499999953,37.56406799700005],[-3.506552499999941,37.61724099800006],[-3.791726499999925,37.46620549800008],[-3.84225749999996,37.38577249700006],[-4.000835,37.40211499800006],[-4.220159499999966,37.620269998000026],[-4.16438949999997,37.68153399700003],[-4.238501499999927,37.719833497000025],[-4.206599499999925,37.77876649800004],[-4.286759,37.796577497000044],[-4.282274499999971,38.02284999800003],[-4.163972,38.14466099800006],[-4.21024,38.30149099700003],[-4.268895,38.34721299700004],[-4.253038499999946,38.40123],[-3.620483,38.39401999700004],[-3.583084,38.45150349800008],[-3.453166499999952,38.400233996000054],[-3.33534949999995,38.480529999000055],[-3.130193,38.438796997000054],[-3.062778,38.47640249800003],[-3.002507499999979,38.40828699700006],[-2.992628,38.450976997000055],[-2.762069,38.532779498000025],[-2.604288,38.51290899800006],[-2.566616,38.490489997000054],[-2.571873,38.41415049600005],[-2.481576,38.39310099900007],[-2.446249499999965,38.18521099700007],[-2.551274000999967,38.08411799900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES617","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Málaga","NUTS_NAME":"Málaga"},"geometry":{"type":"Polygon","coordinates":[[[-4.327640499999973,37.18435649800006],[-4.270058,37.027057497000044],[-3.827184000999978,36.85554099900003],[-3.768160499999965,36.80196349900007],[-3.777457499999969,36.737926499000025],[-4.418207,36.717219998000076],[-4.638161499999967,36.50709149800008],[-4.936012499999947,36.499389499000074],[-5.176345499999968,36.414661498000044],[-5.252405,36.31127549800004],[-5.320127500999945,36.32508499800008],[-5.429368499999953,36.533180497000046],[-5.524297,36.497738],[-5.611827499999947,36.54162999700003],[-5.522210499999971,36.53181049700004],[-5.343394,36.659950500000036],[-5.29098,36.77519599900006],[-5.34076,36.81944299700007],[-5.301629000999981,36.86967099700007],[-5.134113499999955,36.838592497000036],[-5.086809,36.90511699800004],[-5.142423,37.00389099900008],[-4.908404,37.11467749800005],[-4.939640499999939,37.175728],[-4.791943,37.15435799900007],[-4.708635,37.19772399800007],[-4.733459499999981,37.25006099800004],[-4.653374499999927,37.25585549700003],[-4.539496499999927,37.21133399800004],[-4.378304499999956,37.27245349900005],[-4.327640499999973,37.18435649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES618","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Sevilla","NUTS_NAME":"Sevilla"},"geometry":{"type":"Polygon","coordinates":[[[-5.740212,38.13013049800003],[-5.734962499999938,38.086303497000074],[-5.584845499999972,38.13175199900007],[-5.415665,37.94372199800006],[-5.408625,37.86917099800007],[-5.339817499999981,37.83357250000006],[-5.309434,37.69810099800003],[-5.411963000999947,37.68354799800005],[-5.358010499999978,37.584884498000065],[-4.999864,37.70663049900003],[-4.88477649999993,37.43801899600004],[-4.782256,37.32853699700007],[-4.671938499999953,37.33501799700008],[-4.653374499999927,37.25585549700003],[-4.733459499999981,37.25006099800004],[-4.708635,37.19772399800007],[-4.791943,37.15435799900007],[-4.939640499999939,37.175728],[-4.908404,37.11467749800005],[-5.142423,37.00389099900008],[-5.235064000999955,36.94121149700004],[-5.302151499999979,37.037395497000034],[-5.438999,36.93531799700003],[-5.431367000999956,37.05042599700005],[-5.511037499999929,37.033634498000026],[-5.469782499999951,36.983978498000056],[-5.550906,36.923152997000045],[-5.533873,36.89348199800003],[-5.666174499999954,36.975772498000026],[-5.717068500999972,36.91954799700005],[-5.845565499999964,36.915245],[-5.893642499999942,36.85161599700007],[-6.331072499999948,36.898837997000044],[-6.378956499999958,36.93474199800005],[-6.354009499999961,36.98852949900004],[-6.385999,37.072017997000046],[-6.333069499999965,37.29914849700003],[-6.384643499999981,37.33753599700003],[-6.306702499999972,37.43511949800006],[-6.395199,37.611061],[-6.536165499999981,37.65102349800003],[-6.390103499999952,37.74774149800004],[-6.134084499999972,37.78788349800004],[-6.125164,37.903388998000025],[-6.180307499999969,37.941077997000036],[-5.953754,37.99539549900004],[-5.874359,38.158622997000066],[-5.723288499999967,38.19248949800004],[-5.684302499999944,38.15711199800006],[-5.740212,38.13013049800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES62","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Región de Murcia","NUTS_NAME":"Región de Murcia"},"geometry":{"type":"Polygon","coordinates":[[[-1.02687,38.655509497000025],[-1.013821499999949,38.49706249800005],[-1.094109499999945,38.43793099800007],[-1.084330499999965,38.346217996000064],[-0.96761749999996,38.27564249900007],[-1.030276499999957,38.097076497000046],[-0.900112,37.923125998000046],[-0.762135499999943,37.84700799700005],[-0.751674499999979,37.794719497000074],[-0.784334,37.81915649900003],[-0.859979499999952,37.721446997000044],[-0.733145,37.63270199700003],[-0.738589693999927,37.69975169400004],[-0.720821,37.60587299700006],[-0.920817499999941,37.55564099800006],[-1.074336,37.58328599600003],[-1.111013,37.53418349800006],[-1.325323499999968,37.56244249800005],[-1.630033,37.37518299800007],[-1.845223499999975,37.454826497000056],[-1.976776499999971,37.61873249900003],[-2.011892499999931,37.67254649700004],[-1.974468,37.86848049900004],[-2.207631499999934,37.91658799800007],[-2.341602,38.02601999800004],[-2.21349349999997,38.201820498000075],[-2.048963499999957,38.30579399900006],[-1.907508,38.297969497000054],[-1.741687499999955,38.38049699700008],[-1.589503,38.31000149700003],[-1.478507,38.37638849900003],[-1.498696,38.53206249900006],[-1.402179499999932,38.690803499000026],[-1.119623499999932,38.737724497000045],[-1.02687,38.655509497000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES620","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Murcia","NUTS_NAME":"Murcia"},"geometry":{"type":"Polygon","coordinates":[[[-1.02687,38.655509497000025],[-1.013821499999949,38.49706249800005],[-1.094109499999945,38.43793099800007],[-1.084330499999965,38.346217996000064],[-0.96761749999996,38.27564249900007],[-1.030276499999957,38.097076497000046],[-0.900112,37.923125998000046],[-0.762135499999943,37.84700799700005],[-0.751674499999979,37.794719497000074],[-0.784334,37.81915649900003],[-0.859979499999952,37.721446997000044],[-0.733145,37.63270199700003],[-0.738589693999927,37.69975169400004],[-0.720821,37.60587299700006],[-0.920817499999941,37.55564099800006],[-1.074336,37.58328599600003],[-1.111013,37.53418349800006],[-1.325323499999968,37.56244249800005],[-1.630033,37.37518299800007],[-1.845223499999975,37.454826497000056],[-1.976776499999971,37.61873249900003],[-2.011892499999931,37.67254649700004],[-1.974468,37.86848049900004],[-2.207631499999934,37.91658799800007],[-2.341602,38.02601999800004],[-2.21349349999997,38.201820498000075],[-2.048963499999957,38.30579399900006],[-1.907508,38.297969497000054],[-1.741687499999955,38.38049699700008],[-1.589503,38.31000149700003],[-1.478507,38.37638849900003],[-1.498696,38.53206249900006],[-1.402179499999932,38.690803499000026],[-1.119623499999932,38.737724497000045],[-1.02687,38.655509497000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES63","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Ciudad Autónoma de Ceuta","NUTS_NAME":"Ciudad Autónoma de Ceuta"},"geometry":{"type":"Polygon","coordinates":[[[-5.342521175999934,35.87303803900005],[-5.382034499999975,35.91260649900005],[-5.350729000999934,35.90900299700007],[-5.342521175999934,35.87303803900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES630","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Ceuta","NUTS_NAME":"Ceuta"},"geometry":{"type":"Polygon","coordinates":[[[-5.342521175999934,35.87303803900005],[-5.382034499999975,35.91260649900005],[-5.350729000999934,35.90900299700007],[-5.342521175999934,35.87303803900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES64","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Ciudad Autónoma de Melilla","NUTS_NAME":"Ciudad Autónoma de Melilla"},"geometry":{"type":"Polygon","coordinates":[[[-2.92736,35.27426399900003],[-2.970093499999962,35.289367499000036],[-2.950587,35.31841799800003],[-2.92736,35.27426399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES640","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Melilla","NUTS_NAME":"Melilla"},"geometry":{"type":"Polygon","coordinates":[[[-2.92736,35.27426399900003],[-2.970093499999962,35.289367499000036],[-2.950587,35.31841799800003],[-2.92736,35.27426399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES7","LEVL_CODE":1,"CNTR_CODE":"ES","NAME_LATN":"CANARIAS","NUTS_NAME":"CANARIAS"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-13.734242499999937,28.90740599700007],[-13.78629,28.83903299700006],[-13.854198499999939,28.86207399700004],[-13.829453499999943,29.008733499000073],[-13.75620749999996,29.07760949900006],[-13.541315,29.122923498000034],[-13.470518,29.238542497000026],[-13.419878,29.18417149700008],[-13.464197,29.129623498000058],[-13.482365499999958,28.99933249700007],[-13.734242499999937,28.90740599700007]]],[[[-14.22429649999998,28.162038997000025],[-14.331749,28.044091998000056],[-14.492716,28.083705998000028],[-14.226619,28.21153449700006],[-14.203373,28.328212497000038],[-14.033473,28.593721497000047],[-14.016622499999926,28.71499399700008],[-13.87708349999997,28.75204099800004],[-13.830917499999941,28.693815999000037],[-13.825466,28.553829],[-13.922949,28.24746299700007],[-14.22429649999998,28.162038997000025]]],[[[-15.403546499999948,28.17169199800003],[-15.419742499999927,28.061523497000053],[-15.369922499999973,27.99678049700003],[-15.424459499999955,27.807855497000048],[-15.600111,27.734939497000028],[-15.788058499999977,27.835252497000056],[-15.832990499999937,27.909723498000062],[-15.820978,28.014224997000042],[-15.707619499999964,28.08881199700005],[-15.709276000999978,28.165738997000062],[-15.46804149999997,28.12534149700008],[-15.403546499999948,28.17169199800003]]],[[[-16.492656499999953,28.078862998000034],[-16.640173,28.004987997000057],[-16.707897499999945,28.010785998000074],[-16.924217,28.350005999000075],[-16.511211499999945,28.420334498000045],[-16.38525950099995,28.54562549900004],[-16.134222,28.581258499000057],[-16.143300499999953,28.520728999000028],[-16.36059249999994,28.379459498000074],[-16.361009499999966,28.306573997000044],[-16.492656499999953,28.078862998000034]]],[[[-17.09916050099997,28.094314499000063],[-17.19942649999996,28.02373149700003],[-17.34921849999995,28.099218497000038],[-17.318047,28.204323998000064],[-17.11809349999993,28.149373998000044],[-17.09916050099997,28.094314499000063]]],[[[-17.76021949999995,28.569320998000023],[-17.84223549999996,28.452651498000023],[-18.007618499999978,28.75788349800007],[-17.948734499999944,28.840764996000075],[-17.777729,28.839669997000044],[-17.724292499999933,28.743291998000075],[-17.770277,28.676038997000035],[-17.76021949999995,28.569320998000023]]],[[[-17.883365499999968,27.804454496000062],[-17.97904000099993,27.64051999700007],[-18.15835949999996,27.712901499000054],[-18.131494,27.771581498000046],[-18.036621000999958,27.76222799900006],[-17.958796,27.841645999000036],[-17.883365499999968,27.804454496000062]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES70","LEVL_CODE":2,"CNTR_CODE":"ES","NAME_LATN":"Canarias","NUTS_NAME":"Canarias"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-13.734242499999937,28.90740599700007],[-13.78629,28.83903299700006],[-13.854198499999939,28.86207399700004],[-13.829453499999943,29.008733499000073],[-13.75620749999996,29.07760949900006],[-13.541315,29.122923498000034],[-13.470518,29.238542497000026],[-13.419878,29.18417149700008],[-13.464197,29.129623498000058],[-13.482365499999958,28.99933249700007],[-13.734242499999937,28.90740599700007]]],[[[-14.22429649999998,28.162038997000025],[-14.331749,28.044091998000056],[-14.492716,28.083705998000028],[-14.226619,28.21153449700006],[-14.203373,28.328212497000038],[-14.033473,28.593721497000047],[-14.016622499999926,28.71499399700008],[-13.87708349999997,28.75204099800004],[-13.830917499999941,28.693815999000037],[-13.825466,28.553829],[-13.922949,28.24746299700007],[-14.22429649999998,28.162038997000025]]],[[[-15.403546499999948,28.17169199800003],[-15.419742499999927,28.061523497000053],[-15.369922499999973,27.99678049700003],[-15.424459499999955,27.807855497000048],[-15.600111,27.734939497000028],[-15.788058499999977,27.835252497000056],[-15.832990499999937,27.909723498000062],[-15.820978,28.014224997000042],[-15.707619499999964,28.08881199700005],[-15.709276000999978,28.165738997000062],[-15.46804149999997,28.12534149700008],[-15.403546499999948,28.17169199800003]]],[[[-16.492656499999953,28.078862998000034],[-16.640173,28.004987997000057],[-16.707897499999945,28.010785998000074],[-16.924217,28.350005999000075],[-16.511211499999945,28.420334498000045],[-16.38525950099995,28.54562549900004],[-16.134222,28.581258499000057],[-16.143300499999953,28.520728999000028],[-16.36059249999994,28.379459498000074],[-16.361009499999966,28.306573997000044],[-16.492656499999953,28.078862998000034]]],[[[-17.09916050099997,28.094314499000063],[-17.19942649999996,28.02373149700003],[-17.34921849999995,28.099218497000038],[-17.318047,28.204323998000064],[-17.11809349999993,28.149373998000044],[-17.09916050099997,28.094314499000063]]],[[[-17.76021949999995,28.569320998000023],[-17.84223549999996,28.452651498000023],[-18.007618499999978,28.75788349800007],[-17.948734499999944,28.840764996000075],[-17.777729,28.839669997000044],[-17.724292499999933,28.743291998000075],[-17.770277,28.676038997000035],[-17.76021949999995,28.569320998000023]]],[[[-17.883365499999968,27.804454496000062],[-17.97904000099993,27.64051999700007],[-18.15835949999996,27.712901499000054],[-18.131494,27.771581498000046],[-18.036621000999958,27.76222799900006],[-17.958796,27.841645999000036],[-17.883365499999968,27.804454496000062]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES703","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"El Hierro","NUTS_NAME":"El Hierro"},"geometry":{"type":"Polygon","coordinates":[[[-17.883365499999968,27.804454496000062],[-17.97904000099993,27.64051999700007],[-18.15835949999996,27.712901499000054],[-18.131494,27.771581498000046],[-18.036621000999958,27.76222799900006],[-17.958796,27.841645999000036],[-17.883365499999968,27.804454496000062]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES704","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Fuerteventura","NUTS_NAME":"Fuerteventura"},"geometry":{"type":"Polygon","coordinates":[[[-14.22429649999998,28.162038997000025],[-14.331749,28.044091998000056],[-14.492716,28.083705998000028],[-14.226619,28.21153449700006],[-14.203373,28.328212497000038],[-14.033473,28.593721497000047],[-14.016622499999926,28.71499399700008],[-13.87708349999997,28.75204099800004],[-13.830917499999941,28.693815999000037],[-13.825466,28.553829],[-13.922949,28.24746299700007],[-14.22429649999998,28.162038997000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES705","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Gran Canaria","NUTS_NAME":"Gran Canaria"},"geometry":{"type":"Polygon","coordinates":[[[-15.403546499999948,28.17169199800003],[-15.419742499999927,28.061523497000053],[-15.369922499999973,27.99678049700003],[-15.424459499999955,27.807855497000048],[-15.600111,27.734939497000028],[-15.788058499999977,27.835252497000056],[-15.832990499999937,27.909723498000062],[-15.820978,28.014224997000042],[-15.707619499999964,28.08881199700005],[-15.709276000999978,28.165738997000062],[-15.46804149999997,28.12534149700008],[-15.403546499999948,28.17169199800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES706","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"La Gomera","NUTS_NAME":"La Gomera"},"geometry":{"type":"Polygon","coordinates":[[[-17.19942649999996,28.02373149700003],[-17.34921849999995,28.099218497000038],[-17.318047,28.204323998000064],[-17.11809349999993,28.149373998000044],[-17.09916050099997,28.094314499000063],[-17.19942649999996,28.02373149700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES707","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"La Palma","NUTS_NAME":"La Palma"},"geometry":{"type":"Polygon","coordinates":[[[-17.76021949999995,28.569320998000023],[-17.84223549999996,28.452651498000023],[-18.007618499999978,28.75788349800007],[-17.948734499999944,28.840764996000075],[-17.777729,28.839669997000044],[-17.724292499999933,28.743291998000075],[-17.770277,28.676038997000035],[-17.76021949999995,28.569320998000023]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES708","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Lanzarote","NUTS_NAME":"Lanzarote"},"geometry":{"type":"Polygon","coordinates":[[[-13.464197,29.129623498000058],[-13.482365499999958,28.99933249700007],[-13.734242499999937,28.90740599700007],[-13.78629,28.83903299700006],[-13.854198499999939,28.86207399700004],[-13.829453499999943,29.008733499000073],[-13.75620749999996,29.07760949900006],[-13.541315,29.122923498000034],[-13.470518,29.238542497000026],[-13.419878,29.18417149700008],[-13.464197,29.129623498000058]]]}},{"type":"Feature","properties":{"NUTS_ID":"ES709","LEVL_CODE":3,"CNTR_CODE":"ES","NAME_LATN":"Tenerife","NUTS_NAME":"Tenerife"},"geometry":{"type":"Polygon","coordinates":[[[-16.36059249999994,28.379459498000074],[-16.361009499999966,28.306573997000044],[-16.492656499999953,28.078862998000034],[-16.640173,28.004987997000057],[-16.707897499999945,28.010785998000074],[-16.924217,28.350005999000075],[-16.511211499999945,28.420334498000045],[-16.38525950099995,28.54562549900004],[-16.134222,28.581258499000057],[-16.143300499999953,28.520728999000028],[-16.36059249999994,28.379459498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI","LEVL_CODE":0,"CNTR_CODE":"FI","NAME_LATN":"SUOMI / FINLAND","NUTS_NAME":"SUOMI / FINLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.984522,70.01396549800006],[28.404522,69.81841799900008],[29.13434250000006,69.69532949900008],[29.336974,69.47831049900003],[28.831907,69.22435249800003],[28.805792500000052,69.11114699900008],[28.92968,69.05190499900004],[28.415766500000075,68.91545250000007],[28.800767500000063,68.869286],[28.433917500000064,68.53967199900006],[28.646142500000053,68.19630450000005],[29.327081500000077,68.07454449900007],[29.65940050000006,67.80296249900005],[30.017040500000064,67.67355649900003],[29.93022250000007,67.52252749900003],[29.033642,66.942135],[29.572938500000078,66.43285599900008],[29.92330050000004,66.12714649900005],[30.13848,65.668682],[29.721886,65.637089],[29.863920500000063,65.56044799900008],[29.754731500000048,65.49736949700008],[29.7466,65.34740549900005],[29.601807,65.25993599900005],[29.885679500000037,65.20629949900007],[29.819355,65.14428249800005],[29.896880500000066,65.10514399900006],[29.626998,65.06059999800004],[29.599355,64.99505849600007],[29.64493,64.86649549900005],[30.086534500000027,64.77399049800005],[30.04142150000007,64.74119549900007],[30.13013550000005,64.63473],[29.989796500000068,64.58714649900003],[30.044892,64.40202999700006],[30.482463,64.26232699900004],[30.509623,64.22168750000003],[30.466391,64.20443149900007],[30.553427999000064,64.13224799800008],[30.260437,63.82200949800006],[29.971917,63.75716549900005],[30.48403,63.466702],[31.239880500000027,63.217652497000074],[31.272150500000066,63.107623999000054],[31.586729,62.908702],[31.162545,62.446056499000065],[30.143966500000033,61.85223799800008],[29.496255500000075,61.49189249600005],[29.24483250000003,61.27068449600006],[27.991279500000076,60.668978999000046],[27.797273500000074,60.545343],[27.425736,60.465340999000034],[27.161235,60.571331],[27.182419500000037,60.52854099700005],[26.885828140000058,60.46577349100005],[26.928463500000078,60.429250496000066],[26.570195349000073,60.44022449100004],[26.660412,60.41773950000004],[26.543459,60.37468249700004],[26.508428,60.40369799700005],[26.552183126000045,60.43506634700003],[26.453932,60.487652499000035],[26.426291,60.40769849800006],[26.509094,60.36822599900006],[26.316229500000077,60.37331599600003],[26.235871,60.459724995000045],[26.26649750000007,60.39011749500003],[26.18525150000005,60.36500599900006],[25.91208750000004,60.486244499000065],[26.11142650000005,60.33493549500008],[26.009723687000076,60.31647918700003],[26.010455,60.321356999000045],[25.993759220000072,60.33733616200004],[25.912287500000048,60.36600449700006],[25.863847,60.32347849700005],[25.936341,60.26061999900003],[25.786976500000037,60.23728599600008],[25.627897500000074,60.326477999000076],[25.720239500000048,60.339234497000064],[25.669688500000063,60.383327500000064],[25.432495500000073,60.22252099900004],[24.575224,60.178468500000065],[24.64701450000007,60.110719],[24.393869,59.96564299700003],[24.478533,60.04183949700007],[24.34782050000007,60.02487650000006],[24.370151,60.083031997000035],[24.33219950000006,60.09312150000005],[24.264736,60.039963999000065],[24.161796500000037,60.049338497000065],[24.202054,60.02742099800008],[23.839481500000034,60.018403],[23.956796,60.00431099600007],[23.692564563000076,59.97204467600005],[23.727499,59.943315995000034],[23.355432500000063,59.91736499800004],[23.580104,59.990419],[23.428231500000038,59.97311],[23.547022500000026,60.097934999000074],[23.186451500000032,59.827804],[22.932191500000044,59.82402199900008],[23.223772,59.960071],[23.09581950000006,59.94754749800006],[23.118828808000046,60.000895125000056],[22.941069,59.90828399600008],[22.809486,60.01684249700003],[23.031015,59.987883],[22.98478650000004,60.033119499000065],[23.04588801600005,60.04808119000006],[22.983959500000026,60.08516549800004],[23.028183,60.10148649900003],[23.023937500000045,60.134961],[22.878123,60.155944],[22.87021,60.19823749900007],[23.079409,60.375626999000076],[22.789934,60.25689299600003],[22.518889,60.21428999900007],[22.55508050000003,60.23669849700008],[22.55207450000006,60.26696699900003],[22.45401399900004,60.265965],[22.60413650000004,60.37388149900005],[22.470867,60.391068],[22.50068,60.418537497000045],[22.295877999000027,60.384289497000054],[22.253048428000056,60.41175714800005],[22.247403,60.419651497000075],[22.241909278000037,60.41890473300003],[22.198754999000073,60.44661],[22.016979,60.463471],[21.963584500000024,60.536594998000055],[21.897526,60.55281050000008],[21.693564500000036,60.46756799900004],[21.824349,60.53964949600004],[21.846205,60.63480049800006],[21.585519499000043,60.491007],[21.572621,60.500252498000066],[21.592718,60.51764350000008],[21.559374500000047,60.559245],[21.462105,60.581443500000034],[21.32788050000005,60.69758699700003],[21.472907,60.70669749900003],[21.394421500000078,60.822362500000054],[21.17464650000005,60.87034399600003],[21.320113500000048,60.927714999000045],[21.240474499000072,60.98223249800003],[21.294469500000048,61.00893449600005],[21.438336500000048,60.91122949800007],[21.29621750000007,61.080258996000055],[21.419929500000023,61.047442997000076],[21.441026,61.194390500000054],[21.507654500000058,61.199961],[21.419794,61.243645],[21.61136350000004,61.22957849800008],[21.476622500000076,61.276801498000054],[21.563164500000028,61.38372699900003],[21.447379999000077,61.429939997000076],[21.596482,61.44696749600007],[21.450419,61.50930499800006],[21.633457500000077,61.514035],[21.497934,61.589301996000074],[21.703777500000058,61.55769450000008],[21.506391500000063,61.723968],[21.607499,61.71708650000005],[21.421109500000057,61.806871],[21.493800500000077,61.80678],[21.439728500000058,61.925559],[21.27957,61.99326299900008],[21.321595,62.06678649600008],[21.305312,62.142479997000066],[21.39997850000003,62.20194699900003],[21.328509500000052,62.308114999000054],[21.366348,62.370906999000056],[21.11144,62.48973949900005],[21.103148,62.553908998000054],[21.229786,62.607195999000055],[21.08101,62.63231749900007],[21.17998,62.75026599700004],[21.121829500000047,62.799090499000044],[21.206829,62.80357650000008],[21.26096350000006,62.86910799800006],[21.249345,62.89127599800008],[21.403198,62.87959050000006],[21.483848500000022,62.97023950000005],[21.455967718000068,63.01229484900006],[21.33738050000005,63.040954499000065],[21.42188,63.085517],[21.678992,63.051163],[21.501572500000066,63.17212499900006],[21.539687,63.26745049900006],[21.585874500000045,63.28661999700006],[21.573153,63.27353349600003],[21.598209,63.23116649900004],[21.670264,63.21060399900006],[22.137469558000078,63.26130588700005],[22.159774,63.25531699900006],[22.22091,63.27866849600008],[22.22476671900006,63.29133424100007],[22.39302850000007,63.333281500000055],[22.203152500000044,63.473991],[22.403968500000076,63.43821349600006],[22.289125999000078,63.522546498000054],[22.32430850000003,63.58182749900004],[22.415758,63.476043],[22.518773500000066,63.59385349900003],[22.42851999900006,63.60385099900003],[22.75393450000007,63.71667049900003],[22.671665500000074,63.75909299600005],[22.763338,63.799496],[22.66741250000007,63.81300549900004],[22.871356500000047,63.83397849900007],[22.910391,63.85553499900004],[22.819994499000074,63.89851899700005],[22.96786,63.86471850000004],[22.927715464000073,63.82609472100006],[22.989051,63.78802199900008],[23.02727150000004,63.78876750000006],[23.00474550000007,63.85302299800003],[23.406509,63.940069],[23.39561400100007,64.09290799900003],[23.617333001000077,64.04877199700007],[23.581215,64.11728299900005],[23.720776500000056,64.18612999900006],[23.68772050000007,64.21953549800003],[24.3712,64.54583399800003],[24.406876500000067,64.66637349700005],[24.565660500000035,64.73118299900005],[24.539910500000076,64.81454849600004],[25.104935,64.921151],[25.316963500000043,64.82914199900006],[25.393918,64.89429649900006],[25.197500500000046,65.008802],[25.487671,64.96143649900006],[25.403687,64.99759549700008],[25.384283,65.096682],[25.229462999000077,65.12699449700006],[25.303327,65.198627],[25.255772500000035,65.21988150000004],[25.317455,65.28966599700004],[25.268597500000055,65.31379299700006],[25.35660499900007,65.34515899800004],[25.262014,65.408999],[25.366445,65.42749949800003],[25.08460749900007,65.58894949600005],[24.887724500000047,65.66578199900005],[24.657702,65.64094649900005],[24.677564500000074,65.65918699900004],[24.641739,65.70509749800004],[24.312017500000024,65.80646299600005],[24.159964,65.74979899900006],[24.155129,65.81602699900003],[23.918210499000054,66.16141],[23.64559950000006,66.30140849900005],[23.650217501000043,66.45498399900004],[23.856645,66.55690399900004],[23.907815500000027,66.72142649900007],[23.879574500000047,66.76324399900005],[23.995160501000043,66.81970899800007],[23.55127200100003,67.17146199900003],[23.560987999000076,67.26085349800007],[23.722462,67.28252899800003],[23.764582500000074,67.428214],[23.39402350000006,67.48582099800007],[23.55897800100007,67.61996149800007],[23.477203,67.70166250000005],[23.494326001000047,67.86402599900003],[23.652348999000026,67.958793],[23.15471750100005,68.136291],[23.143492500000036,68.24682799900006],[22.844611,68.38334150000003],[21.88894300100003,68.58438349900007],[20.548636500000043,69.05996850000008],[21.057545,69.03628949800003],[21.108676,69.10392949900006],[20.987576500000046,69.191927],[21.27882150000005,69.31188399800004],[21.627087500000073,69.27658799800008],[21.983611500000052,69.07289349900003],[22.340785,68.82722549900006],[22.374526,68.71666699900004],[23.167591,68.62851849800006],[23.871476,68.83651899800003],[24.903199500000028,68.55459099900008],[25.116428,68.63958749900007],[25.157167,68.79989199900007],[25.777502500000026,69.018279],[25.702101,69.25366150000008],[25.976791500000047,69.61067350000008],[25.89156,69.66550199800008],[25.950607,69.69655049900007],[26.490588,69.94501949900007],[27.037634500000024,69.91038799900008],[27.959615,70.09209449800005],[27.984522,70.01396549800006]]],[[[24.879357,65.06499149900003],[25.069460500000048,65.03647999900005],[24.795605,65.05142949900005],[24.878009999000028,64.96759499900003],[24.792422500000043,64.941664],[24.573687,65.021007],[24.879357,65.06499149900003]]],[[[22.78140650000006,60.01763999800005],[22.392005,60.00003749800004],[22.363324,60.061845997000034],[22.48949250000004,60.15767099700008],[22.414017,60.160186500000066],[22.461433500000055,60.20432499700007],[22.845721999000034,60.237640999000064],[22.92399,60.310044996000045],[22.983654500000057,60.30009099700004],[22.862508,60.20971950000006],[22.837934500000074,60.093759498000054],[22.769107500000075,60.07104499900004],[22.78140650000006,60.01763999800005]]],[[[21.458234,63.210639996000054],[21.26949599900007,63.157128999000065],[21.08398550000004,63.28635849900007],[21.25427650000006,63.29772050000008],[21.200068,63.253674498000066],[21.458234,63.210639996000054]]],[[[19.916036500000075,60.42270650100005],[20.275351500000056,60.285269999000036],[20.16831250000007,60.17514399700008],[20.017924277000077,60.23618142000004],[20.043062,60.102843],[19.98094150000003,60.10417199900007],[19.957714500000066,60.060943500000064],[19.749325500000054,60.094862499000044],[19.675316817000066,60.21566829400007],[19.513948,60.17833699900007],[19.610745,60.29433850000004],[19.77291450000007,60.29507699900006],[19.737465,60.33260749700008],[19.809224499000038,60.319052499000065],[19.76569150000006,60.264571499000056],[19.804114,60.199625999000034],[19.896939501000077,60.28221550000006],[19.761749,60.40846249800006],[19.916036500000075,60.42270650100005]],[[20.03254050000004,60.31737399900004],[20.022423531000072,60.26115329100003],[20.057007500000054,60.30628099900008],[20.03254050000004,60.31737399900004]]],[[[20.213452500000074,59.999726499000076],[20.17940550000003,59.96864349700007],[19.974522500000035,60.07411649900007],[20.074453,60.07716049900006],[20.08269350000006,60.10944549800007],[20.225252500000067,60.078295499000035],[20.213452500000074,59.999726499000076]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1","LEVL_CODE":1,"CNTR_CODE":"FI","NAME_LATN":"MANNER-SUOMI","NUTS_NAME":"MANNER-SUOMI"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.984522,70.01396549800006],[28.404522,69.81841799900008],[29.13434250000006,69.69532949900008],[29.336974,69.47831049900003],[28.831907,69.22435249800003],[28.805792500000052,69.11114699900008],[28.92968,69.05190499900004],[28.415766500000075,68.91545250000007],[28.800767500000063,68.869286],[28.433917500000064,68.53967199900006],[28.646142500000053,68.19630450000005],[29.327081500000077,68.07454449900007],[29.65940050000006,67.80296249900005],[30.017040500000064,67.67355649900003],[29.93022250000007,67.52252749900003],[29.033642,66.942135],[29.572938500000078,66.43285599900008],[29.92330050000004,66.12714649900005],[30.13848,65.668682],[29.721886,65.637089],[29.863920500000063,65.56044799900008],[29.754731500000048,65.49736949700008],[29.7466,65.34740549900005],[29.601807,65.25993599900005],[29.885679500000037,65.20629949900007],[29.819355,65.14428249800005],[29.896880500000066,65.10514399900006],[29.626998,65.06059999800004],[29.599355,64.99505849600007],[29.64493,64.86649549900005],[30.086534500000027,64.77399049800005],[30.04142150000007,64.74119549900007],[30.13013550000005,64.63473],[29.989796500000068,64.58714649900003],[30.044892,64.40202999700006],[30.482463,64.26232699900004],[30.509623,64.22168750000003],[30.466391,64.20443149900007],[30.553427999000064,64.13224799800008],[30.260437,63.82200949800006],[29.971917,63.75716549900005],[30.48403,63.466702],[31.239880500000027,63.217652497000074],[31.272150500000066,63.107623999000054],[31.586729,62.908702],[31.162545,62.446056499000065],[30.143966500000033,61.85223799800008],[29.496255500000075,61.49189249600005],[29.24483250000003,61.27068449600006],[27.991279500000076,60.668978999000046],[27.797273500000074,60.545343],[27.425736,60.465340999000034],[27.161235,60.571331],[27.182419500000037,60.52854099700005],[26.885828140000058,60.46577349100005],[26.928463500000078,60.429250496000066],[26.570195349000073,60.44022449100004],[26.660412,60.41773950000004],[26.543459,60.37468249700004],[26.508428,60.40369799700005],[26.552183126000045,60.43506634700003],[26.453932,60.487652499000035],[26.426291,60.40769849800006],[26.509094,60.36822599900006],[26.316229500000077,60.37331599600003],[26.235871,60.459724995000045],[26.26649750000007,60.39011749500003],[26.18525150000005,60.36500599900006],[25.91208750000004,60.486244499000065],[26.11142650000005,60.33493549500008],[26.009723687000076,60.31647918700003],[26.010455,60.321356999000045],[25.993759220000072,60.33733616200004],[25.912287500000048,60.36600449700006],[25.863847,60.32347849700005],[25.936341,60.26061999900003],[25.786976500000037,60.23728599600008],[25.627897500000074,60.326477999000076],[25.720239500000048,60.339234497000064],[25.669688500000063,60.383327500000064],[25.432495500000073,60.22252099900004],[24.575224,60.178468500000065],[24.64701450000007,60.110719],[24.393869,59.96564299700003],[24.478533,60.04183949700007],[24.34782050000007,60.02487650000006],[24.370151,60.083031997000035],[24.33219950000006,60.09312150000005],[24.264736,60.039963999000065],[24.161796500000037,60.049338497000065],[24.202054,60.02742099800008],[23.839481500000034,60.018403],[23.956796,60.00431099600007],[23.692564563000076,59.97204467600005],[23.727499,59.943315995000034],[23.355432500000063,59.91736499800004],[23.580104,59.990419],[23.428231500000038,59.97311],[23.547022500000026,60.097934999000074],[23.186451500000032,59.827804],[22.932191500000044,59.82402199900008],[23.223772,59.960071],[23.09581950000006,59.94754749800006],[23.118828808000046,60.000895125000056],[22.941069,59.90828399600008],[22.809486,60.01684249700003],[23.031015,59.987883],[22.98478650000004,60.033119499000065],[23.04588801600005,60.04808119000006],[22.983959500000026,60.08516549800004],[23.028183,60.10148649900003],[23.023937500000045,60.134961],[22.878123,60.155944],[22.87021,60.19823749900007],[23.079409,60.375626999000076],[22.789934,60.25689299600003],[22.518889,60.21428999900007],[22.55508050000003,60.23669849700008],[22.55207450000006,60.26696699900003],[22.45401399900004,60.265965],[22.60413650000004,60.37388149900005],[22.470867,60.391068],[22.50068,60.418537497000045],[22.295877999000027,60.384289497000054],[22.253048428000056,60.41175714800005],[22.247403,60.419651497000075],[22.241909278000037,60.41890473300003],[22.198754999000073,60.44661],[22.016979,60.463471],[21.963584500000024,60.536594998000055],[21.897526,60.55281050000008],[21.693564500000036,60.46756799900004],[21.824349,60.53964949600004],[21.846205,60.63480049800006],[21.585519499000043,60.491007],[21.572621,60.500252498000066],[21.592718,60.51764350000008],[21.559374500000047,60.559245],[21.462105,60.581443500000034],[21.32788050000005,60.69758699700003],[21.472907,60.70669749900003],[21.394421500000078,60.822362500000054],[21.17464650000005,60.87034399600003],[21.320113500000048,60.927714999000045],[21.240474499000072,60.98223249800003],[21.294469500000048,61.00893449600005],[21.438336500000048,60.91122949800007],[21.29621750000007,61.080258996000055],[21.419929500000023,61.047442997000076],[21.441026,61.194390500000054],[21.507654500000058,61.199961],[21.419794,61.243645],[21.61136350000004,61.22957849800008],[21.476622500000076,61.276801498000054],[21.563164500000028,61.38372699900003],[21.447379999000077,61.429939997000076],[21.596482,61.44696749600007],[21.450419,61.50930499800006],[21.633457500000077,61.514035],[21.497934,61.589301996000074],[21.703777500000058,61.55769450000008],[21.506391500000063,61.723968],[21.607499,61.71708650000005],[21.421109500000057,61.806871],[21.493800500000077,61.80678],[21.439728500000058,61.925559],[21.27957,61.99326299900008],[21.321595,62.06678649600008],[21.305312,62.142479997000066],[21.39997850000003,62.20194699900003],[21.328509500000052,62.308114999000054],[21.366348,62.370906999000056],[21.11144,62.48973949900005],[21.103148,62.553908998000054],[21.229786,62.607195999000055],[21.08101,62.63231749900007],[21.17998,62.75026599700004],[21.121829500000047,62.799090499000044],[21.206829,62.80357650000008],[21.26096350000006,62.86910799800006],[21.249345,62.89127599800008],[21.403198,62.87959050000006],[21.483848500000022,62.97023950000005],[21.455967718000068,63.01229484900006],[21.33738050000005,63.040954499000065],[21.42188,63.085517],[21.678992,63.051163],[21.501572500000066,63.17212499900006],[21.539687,63.26745049900006],[21.585874500000045,63.28661999700006],[21.573153,63.27353349600003],[21.598209,63.23116649900004],[21.670264,63.21060399900006],[22.137469558000078,63.26130588700005],[22.159774,63.25531699900006],[22.22091,63.27866849600008],[22.22476671900006,63.29133424100007],[22.39302850000007,63.333281500000055],[22.203152500000044,63.473991],[22.403968500000076,63.43821349600006],[22.289125999000078,63.522546498000054],[22.32430850000003,63.58182749900004],[22.415758,63.476043],[22.518773500000066,63.59385349900003],[22.42851999900006,63.60385099900003],[22.75393450000007,63.71667049900003],[22.671665500000074,63.75909299600005],[22.763338,63.799496],[22.66741250000007,63.81300549900004],[22.871356500000047,63.83397849900007],[22.910391,63.85553499900004],[22.819994499000074,63.89851899700005],[22.96786,63.86471850000004],[22.927715464000073,63.82609472100006],[22.989051,63.78802199900008],[23.02727150000004,63.78876750000006],[23.00474550000007,63.85302299800003],[23.406509,63.940069],[23.39561400100007,64.09290799900003],[23.617333001000077,64.04877199700007],[23.581215,64.11728299900005],[23.720776500000056,64.18612999900006],[23.68772050000007,64.21953549800003],[24.3712,64.54583399800003],[24.406876500000067,64.66637349700005],[24.565660500000035,64.73118299900005],[24.539910500000076,64.81454849600004],[25.104935,64.921151],[25.316963500000043,64.82914199900006],[25.393918,64.89429649900006],[25.197500500000046,65.008802],[25.487671,64.96143649900006],[25.403687,64.99759549700008],[25.384283,65.096682],[25.229462999000077,65.12699449700006],[25.303327,65.198627],[25.255772500000035,65.21988150000004],[25.317455,65.28966599700004],[25.268597500000055,65.31379299700006],[25.35660499900007,65.34515899800004],[25.262014,65.408999],[25.366445,65.42749949800003],[25.08460749900007,65.58894949600005],[24.887724500000047,65.66578199900005],[24.657702,65.64094649900005],[24.677564500000074,65.65918699900004],[24.641739,65.70509749800004],[24.312017500000024,65.80646299600005],[24.159964,65.74979899900006],[24.155129,65.81602699900003],[23.918210499000054,66.16141],[23.64559950000006,66.30140849900005],[23.650217501000043,66.45498399900004],[23.856645,66.55690399900004],[23.907815500000027,66.72142649900007],[23.879574500000047,66.76324399900005],[23.995160501000043,66.81970899800007],[23.55127200100003,67.17146199900003],[23.560987999000076,67.26085349800007],[23.722462,67.28252899800003],[23.764582500000074,67.428214],[23.39402350000006,67.48582099800007],[23.55897800100007,67.61996149800007],[23.477203,67.70166250000005],[23.494326001000047,67.86402599900003],[23.652348999000026,67.958793],[23.15471750100005,68.136291],[23.143492500000036,68.24682799900006],[22.844611,68.38334150000003],[21.88894300100003,68.58438349900007],[20.548636500000043,69.05996850000008],[21.057545,69.03628949800003],[21.108676,69.10392949900006],[20.987576500000046,69.191927],[21.27882150000005,69.31188399800004],[21.627087500000073,69.27658799800008],[21.983611500000052,69.07289349900003],[22.340785,68.82722549900006],[22.374526,68.71666699900004],[23.167591,68.62851849800006],[23.871476,68.83651899800003],[24.903199500000028,68.55459099900008],[25.116428,68.63958749900007],[25.157167,68.79989199900007],[25.777502500000026,69.018279],[25.702101,69.25366150000008],[25.976791500000047,69.61067350000008],[25.89156,69.66550199800008],[25.950607,69.69655049900007],[26.490588,69.94501949900007],[27.037634500000024,69.91038799900008],[27.959615,70.09209449800005],[27.984522,70.01396549800006]]],[[[24.879357,65.06499149900003],[25.069460500000048,65.03647999900005],[24.795605,65.05142949900005],[24.878009999000028,64.96759499900003],[24.792422500000043,64.941664],[24.573687,65.021007],[24.879357,65.06499149900003]]],[[[22.78140650000006,60.01763999800005],[22.392005,60.00003749800004],[22.363324,60.061845997000034],[22.48949250000004,60.15767099700008],[22.414017,60.160186500000066],[22.461433500000055,60.20432499700007],[22.845721999000034,60.237640999000064],[22.92399,60.310044996000045],[22.983654500000057,60.30009099700004],[22.862508,60.20971950000006],[22.837934500000074,60.093759498000054],[22.769107500000075,60.07104499900004],[22.78140650000006,60.01763999800005]]],[[[21.458234,63.210639996000054],[21.26949599900007,63.157128999000065],[21.08398550000004,63.28635849900007],[21.25427650000006,63.29772050000008],[21.200068,63.253674498000066],[21.458234,63.210639996000054]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI19","LEVL_CODE":2,"CNTR_CODE":"FI","NAME_LATN":"Länsi-Suomi","NUTS_NAME":"Länsi-Suomi"},"geometry":{"type":"MultiPolygon","coordinates":[[[[22.927715464000073,63.82609472100006],[22.989051,63.78802199900008],[23.02727150000004,63.78876750000006],[23.235590500000058,63.71710799500005],[23.477644501000043,63.77913750000005],[23.74408500100003,63.675444],[23.42240950000007,63.61602099900006],[23.651032001000033,63.53094149800006],[23.69698950000003,63.38032699900003],[24.359008,63.11643450000008],[24.77288550000003,63.13336849600006],[24.731613001000028,63.14704],[24.75417150000004,63.26592349900005],[24.803607001000046,63.275935497000035],[24.77292650000004,63.358689499000036],[25.03632250000004,63.458078],[25.251103999000065,63.612298],[25.892824,63.43965549900008],[26.138652,63.457591999000044],[26.176724,63.43886549800004],[26.070552,63.354783499000064],[26.175867,63.24138299800006],[26.163523500000053,63.147045],[26.334283500000026,63.035092500000076],[26.222582,63.02587699900005],[26.33256,62.98357199700007],[26.039845,62.93611350000003],[26.356167,62.747412],[26.446567,62.78917499900007],[26.598391500000048,62.610774],[26.470699500000023,62.51892199900004],[26.709129500000074,62.45294899700008],[26.680536500000073,62.40382399900005],[26.776128,62.354111],[26.61078150000003,62.29497999700004],[26.652633500000036,62.21692849900006],[26.544035500000064,62.26562749800007],[26.220722,62.11856249600004],[26.414589500000034,61.972885500000075],[26.324290500000075,61.880033999000034],[26.53375,61.69132699600004],[26.315036,61.60844849700004],[26.137583500000062,61.619534],[25.921634,61.78734749900008],[25.771173001000022,61.68506699900007],[25.501333500000044,61.67685849800006],[25.437427,61.486905999000044],[24.97047149900004,61.45070549700006],[24.855481500000053,61.46942749800007],[24.912393501000054,61.415839],[24.91393350000004,61.32488299900007],[24.57366650000006,61.30393950000007],[23.756743500000027,61.05735799900003],[23.8785,61.02200849900004],[23.839356500000065,60.93970849700008],[23.712323499000036,61.00903649700007],[23.195350500000075,61.00122299900005],[22.968488500000035,61.04325799900005],[22.823893500000054,61.10143899900004],[22.81591650000007,60.993411999000045],[22.008124,60.89744250000007],[21.825185,61.03417949900006],[21.419929500000023,61.047442997000076],[21.441026,61.194390500000054],[21.507654500000058,61.199961],[21.419794,61.243645],[21.61136350000004,61.22957849800008],[21.476622500000076,61.276801498000054],[21.563164500000028,61.38372699900003],[21.447379999000077,61.429939997000076],[21.596482,61.44696749600007],[21.450419,61.50930499800006],[21.633457500000077,61.514035],[21.497934,61.589301996000074],[21.703777500000058,61.55769450000008],[21.506391500000063,61.723968],[21.607499,61.71708650000005],[21.421109500000057,61.806871],[21.493800500000077,61.80678],[21.439728500000058,61.925559],[21.27957,61.99326299900008],[21.321595,62.06678649600008],[21.305312,62.142479997000066],[21.39997850000003,62.20194699900003],[21.328509500000052,62.308114999000054],[21.366348,62.370906999000056],[21.11144,62.48973949900005],[21.103148,62.553908998000054],[21.229786,62.607195999000055],[21.08101,62.63231749900007],[21.17998,62.75026599700004],[21.121829500000047,62.799090499000044],[21.206829,62.80357650000008],[21.26096350000006,62.86910799800006],[21.249345,62.89127599800008],[21.403198,62.87959050000006],[21.483848500000022,62.97023950000005],[21.455967718000068,63.01229484900006],[21.33738050000005,63.040954499000065],[21.42188,63.085517],[21.678992,63.051163],[21.501572500000066,63.17212499900006],[21.539687,63.26745049900006],[21.585874500000045,63.28661999700006],[21.573153,63.27353349600003],[21.598209,63.23116649900004],[21.670264,63.21060399900006],[22.137469558000078,63.26130588700005],[22.159774,63.25531699900006],[22.22091,63.27866849600008],[22.22476671900006,63.29133424100007],[22.39302850000007,63.333281500000055],[22.203152500000044,63.473991],[22.403968500000076,63.43821349600006],[22.289125999000078,63.522546498000054],[22.32430850000003,63.58182749900004],[22.415758,63.476043],[22.518773500000066,63.59385349900003],[22.42851999900006,63.60385099900003],[22.75393450000007,63.71667049900003],[22.671665500000074,63.75909299600005],[22.763338,63.799496],[22.66741250000007,63.81300549900004],[22.871356500000047,63.83397849900007],[22.910391,63.85553499900004],[22.819994499000074,63.89851899700005],[22.96786,63.86471850000004],[22.927715464000073,63.82609472100006]]],[[[21.458234,63.210639996000054],[21.26949599900007,63.157128999000065],[21.08398550000004,63.28635849900007],[21.25427650000006,63.29772050000008],[21.200068,63.253674498000066],[21.458234,63.210639996000054]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI193","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Keski-Suomi","NUTS_NAME":"Keski-Suomi"},"geometry":{"type":"Polygon","coordinates":[[[26.138652,63.457591999000044],[26.176724,63.43886549800004],[26.070552,63.354783499000064],[26.175867,63.24138299800006],[26.163523500000053,63.147045],[26.334283500000026,63.035092500000076],[26.222582,63.02587699900005],[26.33256,62.98357199700007],[26.039845,62.93611350000003],[26.356167,62.747412],[26.446567,62.78917499900007],[26.598391500000048,62.610774],[26.470699500000023,62.51892199900004],[26.709129500000074,62.45294899700008],[26.680536500000073,62.40382399900005],[26.776128,62.354111],[26.61078150000003,62.29497999700004],[26.652633500000036,62.21692849900006],[26.544035500000064,62.26562749800007],[26.220722,62.11856249600004],[26.414589500000034,61.972885500000075],[26.324290500000075,61.880033999000034],[26.53375,61.69132699600004],[26.315036,61.60844849700004],[26.137583500000062,61.619534],[25.921634,61.78734749900008],[25.771173001000022,61.68506699900007],[25.501333500000044,61.67685849800006],[25.437427,61.486905999000044],[24.97047149900004,61.45070549700006],[24.916930500000035,61.58947849900005],[24.78388250000006,61.62216699900006],[24.85497350000003,61.673207999000056],[24.735943500000076,61.69376450000004],[24.721573500000034,61.76341749900007],[24.82070550000003,61.77013550000004],[24.55195450000008,61.91227699900003],[24.730907500000058,62.02319399900006],[24.673700499000063,62.115459],[24.498859,62.204566],[24.335942,62.16804249900008],[24.012998,62.36333049700005],[24.20128150000005,62.482662],[24.513591500000075,62.50315099800008],[24.584304,62.61171949700008],[24.503637,62.62126049900007],[24.582167,62.664791],[24.36228250000005,62.85285850000008],[24.44679,62.89602849900007],[24.215557,63.030186497000045],[24.359008,63.11643450000008],[24.77288550000003,63.13336849600006],[24.731613001000028,63.14704],[24.75417150000004,63.26592349900005],[24.803607001000046,63.275935497000035],[24.77292650000004,63.358689499000036],[25.03632250000004,63.458078],[25.251103999000065,63.612298],[25.892824,63.43965549900008],[26.138652,63.457591999000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI194","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Etelä-Pohjanmaa","NUTS_NAME":"Etelä-Pohjanmaa"},"geometry":{"type":"Polygon","coordinates":[[[23.69698950000003,63.38032699900003],[24.359008,63.11643450000008],[24.215557,63.030186497000045],[24.44679,62.89602849900007],[24.36228250000005,62.85285850000008],[24.582167,62.664791],[24.503637,62.62126049900007],[24.584304,62.61171949700008],[24.513591500000075,62.50315099800008],[24.20128150000005,62.482662],[24.012998,62.36333049700005],[23.835225500000035,62.494341],[22.831244,62.27088999700004],[22.591739,62.311133],[22.524819,62.210199497000076],[21.97467350000005,61.97976650000004],[21.647834500000044,62.019592500000044],[21.641687500000046,62.14945099900007],[21.767498,62.222690998000076],[21.672848,62.26012049900004],[21.68656050000004,62.35522149700006],[21.558781,62.357789997000054],[21.554119,62.42430699800008],[21.614795,62.55431299900005],[21.59176,62.605447],[21.790959500000042,62.69423349700003],[21.729202500000042,62.757742499000074],[21.772054,62.86357299600007],[22.20067,62.75158249800006],[22.658199,63.133147500000064],[22.55939950000004,63.265711500000066],[22.717527,63.354803996000044],[23.112965,63.34731],[23.419658500000025,63.48166],[23.69698950000003,63.38032699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI195","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Pohjanmaa","NUTS_NAME":"Pohjanmaa"},"geometry":{"type":"MultiPolygon","coordinates":[[[[22.927715464000073,63.82609472100006],[22.989051,63.78802199900008],[23.02727150000004,63.78876750000006],[23.235590500000058,63.71710799500005],[23.477644501000043,63.77913750000005],[23.74408500100003,63.675444],[23.42240950000007,63.61602099900006],[23.651032001000033,63.53094149800006],[23.69698950000003,63.38032699900003],[23.419658500000025,63.48166],[23.112965,63.34731],[22.717527,63.354803996000044],[22.55939950000004,63.265711500000066],[22.658199,63.133147500000064],[22.20067,62.75158249800006],[21.772054,62.86357299600007],[21.729202500000042,62.757742499000074],[21.790959500000042,62.69423349700003],[21.59176,62.605447],[21.614795,62.55431299900005],[21.554119,62.42430699800008],[21.558781,62.357789997000054],[21.68656050000004,62.35522149700006],[21.672848,62.26012049900004],[21.767498,62.222690998000076],[21.641687500000046,62.14945099900007],[21.647834500000044,62.019592500000044],[21.27957,61.99326299900008],[21.321595,62.06678649600008],[21.305312,62.142479997000066],[21.39997850000003,62.20194699900003],[21.328509500000052,62.308114999000054],[21.366348,62.370906999000056],[21.11144,62.48973949900005],[21.103148,62.553908998000054],[21.229786,62.607195999000055],[21.08101,62.63231749900007],[21.17998,62.75026599700004],[21.121829500000047,62.799090499000044],[21.206829,62.80357650000008],[21.26096350000006,62.86910799800006],[21.249345,62.89127599800008],[21.403198,62.87959050000006],[21.483848500000022,62.97023950000005],[21.455967718000068,63.01229484900006],[21.33738050000005,63.040954499000065],[21.42188,63.085517],[21.678992,63.051163],[21.501572500000066,63.17212499900006],[21.539687,63.26745049900006],[21.585874500000045,63.28661999700006],[21.573153,63.27353349600003],[21.598209,63.23116649900004],[21.670264,63.21060399900006],[22.137469558000078,63.26130588700005],[22.159774,63.25531699900006],[22.22091,63.27866849600008],[22.22476671900006,63.29133424100007],[22.39302850000007,63.333281500000055],[22.203152500000044,63.473991],[22.403968500000076,63.43821349600006],[22.289125999000078,63.522546498000054],[22.32430850000003,63.58182749900004],[22.415758,63.476043],[22.518773500000066,63.59385349900003],[22.42851999900006,63.60385099900003],[22.75393450000007,63.71667049900003],[22.671665500000074,63.75909299600005],[22.763338,63.799496],[22.66741250000007,63.81300549900004],[22.871356500000047,63.83397849900007],[22.910391,63.85553499900004],[22.819994499000074,63.89851899700005],[22.96786,63.86471850000004],[22.927715464000073,63.82609472100006]]],[[[21.458234,63.210639996000054],[21.26949599900007,63.157128999000065],[21.08398550000004,63.28635849900007],[21.25427650000006,63.29772050000008],[21.200068,63.253674498000066],[21.458234,63.210639996000054]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI196","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Satakunta","NUTS_NAME":"Satakunta"},"geometry":{"type":"Polygon","coordinates":[[[21.647834500000044,62.019592500000044],[21.97467350000005,61.97976650000004],[22.524819,62.210199497000076],[22.591739,62.311133],[22.831244,62.27088999700004],[22.861617500000023,62.17256199800005],[22.770884500000022,62.15440049700004],[22.780230500000073,62.06072949800006],[22.520742,62.001516],[22.8235,61.800021],[22.64716150000004,61.69650649700003],[22.72491750000006,61.64496599600005],[22.696039,61.59976899900005],[22.434146500000054,61.48892749600003],[22.561397500000055,61.41956850000008],[22.645419,61.24662449900006],[22.934203500000024,61.204980498000054],[22.915667,61.08654150000007],[22.968488500000035,61.04325799900005],[22.823893500000054,61.10143899900004],[22.81591650000007,60.993411999000045],[22.008124,60.89744250000007],[21.825185,61.03417949900006],[21.419929500000023,61.047442997000076],[21.441026,61.194390500000054],[21.507654500000058,61.199961],[21.419794,61.243645],[21.61136350000004,61.22957849800008],[21.476622500000076,61.276801498000054],[21.563164500000028,61.38372699900003],[21.447379999000077,61.429939997000076],[21.596482,61.44696749600007],[21.450419,61.50930499800006],[21.633457500000077,61.514035],[21.497934,61.589301996000074],[21.703777500000058,61.55769450000008],[21.506391500000063,61.723968],[21.607499,61.71708650000005],[21.421109500000057,61.806871],[21.493800500000077,61.80678],[21.439728500000058,61.925559],[21.27957,61.99326299900008],[21.647834500000044,62.019592500000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI197","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Pirkanmaa","NUTS_NAME":"Pirkanmaa"},"geometry":{"type":"Polygon","coordinates":[[[24.012998,62.36333049700005],[24.335942,62.16804249900008],[24.498859,62.204566],[24.673700499000063,62.115459],[24.730907500000058,62.02319399900006],[24.55195450000008,61.91227699900003],[24.82070550000003,61.77013550000004],[24.721573500000034,61.76341749900007],[24.735943500000076,61.69376450000004],[24.85497350000003,61.673207999000056],[24.78388250000006,61.62216699900006],[24.916930500000035,61.58947849900005],[24.97047149900004,61.45070549700006],[24.855481500000053,61.46942749800007],[24.912393501000054,61.415839],[24.91393350000004,61.32488299900007],[24.57366650000006,61.30393950000007],[23.756743500000027,61.05735799900003],[23.8785,61.02200849900004],[23.839356500000065,60.93970849700008],[23.712323499000036,61.00903649700007],[23.195350500000075,61.00122299900005],[22.968488500000035,61.04325799900005],[22.915667,61.08654150000007],[22.934203500000024,61.204980498000054],[22.645419,61.24662449900006],[22.561397500000055,61.41956850000008],[22.434146500000054,61.48892749600003],[22.696039,61.59976899900005],[22.72491750000006,61.64496599600005],[22.64716150000004,61.69650649700003],[22.8235,61.800021],[22.520742,62.001516],[22.780230500000073,62.06072949800006],[22.770884500000022,62.15440049700004],[22.861617500000023,62.17256199800005],[22.831244,62.27088999700004],[23.835225500000035,62.494341],[24.012998,62.36333049700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1B","LEVL_CODE":2,"CNTR_CODE":"FI","NAME_LATN":"Helsinki-Uusimaa","NUTS_NAME":"Helsinki-Uusimaa"},"geometry":{"type":"Polygon","coordinates":[[[26.009723687000076,60.31647918700003],[26.010455,60.321356999000045],[25.993759220000072,60.33733616200004],[25.912287500000048,60.36600449700006],[25.863847,60.32347849700005],[25.936341,60.26061999900003],[25.786976500000037,60.23728599600008],[25.627897500000074,60.326477999000076],[25.720239500000048,60.339234497000064],[25.669688500000063,60.383327500000064],[25.432495500000073,60.22252099900004],[24.575224,60.178468500000065],[24.64701450000007,60.110719],[24.393869,59.96564299700003],[24.478533,60.04183949700007],[24.34782050000007,60.02487650000006],[24.370151,60.083031997000035],[24.33219950000006,60.09312150000005],[24.264736,60.039963999000065],[24.161796500000037,60.049338497000065],[24.202054,60.02742099800008],[23.839481500000034,60.018403],[23.956796,60.00431099600007],[23.692564563000076,59.97204467600005],[23.727499,59.943315995000034],[23.355432500000063,59.91736499800004],[23.580104,59.990419],[23.428231500000038,59.97311],[23.547022500000026,60.097934999000074],[23.186451500000032,59.827804],[22.932191500000044,59.82402199900008],[23.223772,59.960071],[23.09581950000006,59.94754749800006],[23.118828808000046,60.000895125000056],[22.941069,59.90828399600008],[22.809486,60.01684249700003],[23.031015,59.987883],[22.98478650000004,60.033119499000065],[23.04588801600005,60.04808119000006],[22.983959500000026,60.08516549800004],[23.028183,60.10148649900003],[23.64895,60.20156399700005],[23.586307500000032,60.258839496000064],[23.727779,60.30773399600008],[23.793594,60.42883050000006],[23.692216,60.44990149600005],[23.871836,60.50793499800005],[23.90049250000004,60.64975849900003],[24.42366750000008,60.55249199900004],[24.54894,60.559840500000064],[24.6159,60.667569997000044],[25.192922,60.72473849800008],[25.05826950000005,60.724986999000066],[25.152437500000076,60.76836549700005],[25.11096599900003,60.81858450000004],[25.47451650000005,60.792689],[25.610861,60.67147599700007],[26.14181,60.69049449900007],[26.20835950000003,60.75799249900007],[26.539152999000066,60.59493749900008],[26.453932,60.487652499000035],[26.426291,60.40769849800006],[26.509094,60.36822599900006],[26.316229500000077,60.37331599600003],[26.235871,60.459724995000045],[26.26649750000007,60.39011749500003],[26.18525150000005,60.36500599900006],[25.91208750000004,60.486244499000065],[26.11142650000005,60.33493549500008],[26.009723687000076,60.31647918700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1B1","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Helsinki-Uusimaa","NUTS_NAME":"Helsinki-Uusimaa"},"geometry":{"type":"Polygon","coordinates":[[[26.009723687000076,60.31647918700003],[26.010455,60.321356999000045],[25.993759220000072,60.33733616200004],[25.912287500000048,60.36600449700006],[25.863847,60.32347849700005],[25.936341,60.26061999900003],[25.786976500000037,60.23728599600008],[25.627897500000074,60.326477999000076],[25.720239500000048,60.339234497000064],[25.669688500000063,60.383327500000064],[25.432495500000073,60.22252099900004],[24.575224,60.178468500000065],[24.64701450000007,60.110719],[24.393869,59.96564299700003],[24.478533,60.04183949700007],[24.34782050000007,60.02487650000006],[24.370151,60.083031997000035],[24.33219950000006,60.09312150000005],[24.264736,60.039963999000065],[24.161796500000037,60.049338497000065],[24.202054,60.02742099800008],[23.839481500000034,60.018403],[23.956796,60.00431099600007],[23.692564563000076,59.97204467600005],[23.727499,59.943315995000034],[23.355432500000063,59.91736499800004],[23.580104,59.990419],[23.428231500000038,59.97311],[23.547022500000026,60.097934999000074],[23.186451500000032,59.827804],[22.932191500000044,59.82402199900008],[23.223772,59.960071],[23.09581950000006,59.94754749800006],[23.118828808000046,60.000895125000056],[22.941069,59.90828399600008],[22.809486,60.01684249700003],[23.031015,59.987883],[22.98478650000004,60.033119499000065],[23.04588801600005,60.04808119000006],[22.983959500000026,60.08516549800004],[23.028183,60.10148649900003],[23.64895,60.20156399700005],[23.586307500000032,60.258839496000064],[23.727779,60.30773399600008],[23.793594,60.42883050000006],[23.692216,60.44990149600005],[23.871836,60.50793499800005],[23.90049250000004,60.64975849900003],[24.42366750000008,60.55249199900004],[24.54894,60.559840500000064],[24.6159,60.667569997000044],[25.192922,60.72473849800008],[25.05826950000005,60.724986999000066],[25.152437500000076,60.76836549700005],[25.11096599900003,60.81858450000004],[25.47451650000005,60.792689],[25.610861,60.67147599700007],[26.14181,60.69049449900007],[26.20835950000003,60.75799249900007],[26.539152999000066,60.59493749900008],[26.453932,60.487652499000035],[26.426291,60.40769849800006],[26.509094,60.36822599900006],[26.316229500000077,60.37331599600003],[26.235871,60.459724995000045],[26.26649750000007,60.39011749500003],[26.18525150000005,60.36500599900006],[25.91208750000004,60.486244499000065],[26.11142650000005,60.33493549500008],[26.009723687000076,60.31647918700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C","LEVL_CODE":2,"CNTR_CODE":"FI","NAME_LATN":"Etelä-Suomi","NUTS_NAME":"Etelä-Suomi"},"geometry":{"type":"MultiPolygon","coordinates":[[[[29.496255500000075,61.49189249600005],[29.24483250000003,61.27068449600006],[27.991279500000076,60.668978999000046],[27.797273500000074,60.545343],[27.425736,60.465340999000034],[27.161235,60.571331],[27.182419500000037,60.52854099700005],[26.885828140000058,60.46577349100005],[26.928463500000078,60.429250496000066],[26.570195349000073,60.44022449100004],[26.660412,60.41773950000004],[26.543459,60.37468249700004],[26.508428,60.40369799700005],[26.552183126000045,60.43506634700003],[26.453932,60.487652499000035],[26.539152999000066,60.59493749900008],[26.20835950000003,60.75799249900007],[26.14181,60.69049449900007],[25.610861,60.67147599700007],[25.47451650000005,60.792689],[25.11096599900003,60.81858450000004],[25.152437500000076,60.76836549700005],[25.05826950000005,60.724986999000066],[25.192922,60.72473849800008],[24.6159,60.667569997000044],[24.54894,60.559840500000064],[24.42366750000008,60.55249199900004],[23.90049250000004,60.64975849900003],[23.871836,60.50793499800005],[23.692216,60.44990149600005],[23.793594,60.42883050000006],[23.727779,60.30773399600008],[23.586307500000032,60.258839496000064],[23.64895,60.20156399700005],[23.028183,60.10148649900003],[23.023937500000045,60.134961],[22.878123,60.155944],[22.87021,60.19823749900007],[23.079409,60.375626999000076],[22.789934,60.25689299600003],[22.518889,60.21428999900007],[22.55508050000003,60.23669849700008],[22.55207450000006,60.26696699900003],[22.45401399900004,60.265965],[22.60413650000004,60.37388149900005],[22.470867,60.391068],[22.50068,60.418537497000045],[22.295877999000027,60.384289497000054],[22.253048428000056,60.41175714800005],[22.247403,60.419651497000075],[22.241909278000037,60.41890473300003],[22.198754999000073,60.44661],[22.016979,60.463471],[21.963584500000024,60.536594998000055],[21.897526,60.55281050000008],[21.693564500000036,60.46756799900004],[21.824349,60.53964949600004],[21.846205,60.63480049800006],[21.585519499000043,60.491007],[21.572621,60.500252498000066],[21.592718,60.51764350000008],[21.559374500000047,60.559245],[21.462105,60.581443500000034],[21.32788050000005,60.69758699700003],[21.472907,60.70669749900003],[21.394421500000078,60.822362500000054],[21.17464650000005,60.87034399600003],[21.320113500000048,60.927714999000045],[21.240474499000072,60.98223249800003],[21.294469500000048,61.00893449600005],[21.438336500000048,60.91122949800007],[21.29621750000007,61.080258996000055],[21.419929500000023,61.047442997000076],[21.825185,61.03417949900006],[22.008124,60.89744250000007],[22.81591650000007,60.993411999000045],[22.823893500000054,61.10143899900004],[22.968488500000035,61.04325799900005],[23.195350500000075,61.00122299900005],[23.712323499000036,61.00903649700007],[23.839356500000065,60.93970849700008],[23.8785,61.02200849900004],[23.756743500000027,61.05735799900003],[24.57366650000006,61.30393950000007],[24.91393350000004,61.32488299900007],[24.912393501000054,61.415839],[24.855481500000053,61.46942749800007],[24.97047149900004,61.45070549700006],[25.437427,61.486905999000044],[25.501333500000044,61.67685849800006],[25.771173001000022,61.68506699900007],[25.921634,61.78734749900008],[26.137583500000062,61.619534],[26.315036,61.60844849700004],[26.34807900100003,61.504390500000056],[26.243660001000023,61.44645],[26.483068,61.32283449800008],[26.513659500000074,61.274324],[26.641758500000037,61.282538997000074],[26.661315,61.20687649900003],[26.881218,61.28520149900004],[26.929526,61.21003450000006],[27.205819500000075,61.162468],[27.577049500000044,61.26319750000005],[27.595544500000074,61.34316549700003],[28.273757500000045,61.384163999000066],[28.43936350000007,61.51073249900003],[28.648602500000038,61.464921496000045],[29.261573,61.557764500000076],[29.559279,61.722068],[30.143966500000033,61.85223799800008],[29.496255500000075,61.49189249600005]]],[[[22.78140650000006,60.01763999800005],[22.392005,60.00003749800004],[22.363324,60.061845997000034],[22.48949250000004,60.15767099700008],[22.414017,60.160186500000066],[22.461433500000055,60.20432499700007],[22.845721999000034,60.237640999000064],[22.92399,60.310044996000045],[22.983654500000057,60.30009099700004],[22.862508,60.20971950000006],[22.837934500000074,60.093759498000054],[22.769107500000075,60.07104499900004],[22.78140650000006,60.01763999800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C1","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Varsinais-Suomi","NUTS_NAME":"Varsinais-Suomi"},"geometry":{"type":"MultiPolygon","coordinates":[[[[23.295712,60.949920500000076],[23.135622500000068,60.88352099900004],[23.230054,60.82540899900005],[23.154599,60.77417049600007],[23.181413,60.75825649700005],[23.90049250000004,60.64975849900003],[23.871836,60.50793499800005],[23.692216,60.44990149600005],[23.793594,60.42883050000006],[23.727779,60.30773399600008],[23.586307500000032,60.258839496000064],[23.64895,60.20156399700005],[23.028183,60.10148649900003],[23.023937500000045,60.134961],[22.878123,60.155944],[22.87021,60.19823749900007],[23.079409,60.375626999000076],[22.789934,60.25689299600003],[22.518889,60.21428999900007],[22.55508050000003,60.23669849700008],[22.55207450000006,60.26696699900003],[22.45401399900004,60.265965],[22.60413650000004,60.37388149900005],[22.470867,60.391068],[22.50068,60.418537497000045],[22.295877999000027,60.384289497000054],[22.253048428000056,60.41175714800005],[22.247403,60.419651497000075],[22.241909278000037,60.41890473300003],[22.198754999000073,60.44661],[22.016979,60.463471],[21.963584500000024,60.536594998000055],[21.897526,60.55281050000008],[21.693564500000036,60.46756799900004],[21.824349,60.53964949600004],[21.846205,60.63480049800006],[21.585519499000043,60.491007],[21.572621,60.500252498000066],[21.592718,60.51764350000008],[21.559374500000047,60.559245],[21.462105,60.581443500000034],[21.32788050000005,60.69758699700003],[21.472907,60.70669749900003],[21.394421500000078,60.822362500000054],[21.17464650000005,60.87034399600003],[21.320113500000048,60.927714999000045],[21.240474499000072,60.98223249800003],[21.294469500000048,61.00893449600005],[21.438336500000048,60.91122949800007],[21.29621750000007,61.080258996000055],[21.419929500000023,61.047442997000076],[21.825185,61.03417949900006],[22.008124,60.89744250000007],[22.81591650000007,60.993411999000045],[22.823893500000054,61.10143899900004],[22.968488500000035,61.04325799900005],[23.195350500000075,61.00122299900005],[23.295712,60.949920500000076]]],[[[22.78140650000006,60.01763999800005],[22.392005,60.00003749800004],[22.363324,60.061845997000034],[22.48949250000004,60.15767099700008],[22.414017,60.160186500000066],[22.461433500000055,60.20432499700007],[22.845721999000034,60.237640999000064],[22.92399,60.310044996000045],[22.983654500000057,60.30009099700004],[22.862508,60.20971950000006],[22.837934500000074,60.093759498000054],[22.769107500000075,60.07104499900004],[22.78140650000006,60.01763999800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C2","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Kanta-Häme","NUTS_NAME":"Kanta-Häme"},"geometry":{"type":"Polygon","coordinates":[[[25.11096599900003,60.81858450000004],[25.152437500000076,60.76836549700005],[25.05826950000005,60.724986999000066],[25.192922,60.72473849800008],[24.6159,60.667569997000044],[24.54894,60.559840500000064],[24.42366750000008,60.55249199900004],[23.90049250000004,60.64975849900003],[23.181413,60.75825649700005],[23.154599,60.77417049600007],[23.230054,60.82540899900005],[23.135622500000068,60.88352099900004],[23.295712,60.949920500000076],[23.195350500000075,61.00122299900005],[23.712323499000036,61.00903649700007],[23.839356500000065,60.93970849700008],[23.8785,61.02200849900004],[23.756743500000027,61.05735799900003],[24.57366650000006,61.30393950000007],[24.91393350000004,61.32488299900007],[25.259859499000072,61.18589099900004],[25.198075,61.076246997000055],[25.028688,61.03094849900003],[25.007527,60.927932498000075],[25.093762,60.918998],[25.07856349900004,60.839236999000036],[25.11096599900003,60.81858450000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C3","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Päijät-Häme","NUTS_NAME":"Päijät-Häme"},"geometry":{"type":"Polygon","coordinates":[[[26.483068,61.32283449800008],[26.513659500000074,61.274324],[26.03981500100008,61.104347500000074],[26.132901,61.02774549900005],[26.060193500000025,60.93608149600004],[26.168979001000025,60.929372500000056],[26.077825501000063,60.835475],[26.20835950000003,60.75799249900007],[26.14181,60.69049449900007],[25.610861,60.67147599700007],[25.47451650000005,60.792689],[25.11096599900003,60.81858450000004],[25.07856349900004,60.839236999000036],[25.093762,60.918998],[25.007527,60.927932498000075],[25.028688,61.03094849900003],[25.198075,61.076246997000055],[25.259859499000072,61.18589099900004],[24.91393350000004,61.32488299900007],[24.912393501000054,61.415839],[24.855481500000053,61.46942749800007],[24.97047149900004,61.45070549700006],[25.437427,61.486905999000044],[25.501333500000044,61.67685849800006],[25.771173001000022,61.68506699900007],[25.921634,61.78734749900008],[26.137583500000062,61.619534],[26.315036,61.60844849700004],[26.34807900100003,61.504390500000056],[26.243660001000023,61.44645],[26.483068,61.32283449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C4","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Kymenlaakso","NUTS_NAME":"Kymenlaakso"},"geometry":{"type":"Polygon","coordinates":[[[26.929526,61.21003450000006],[27.205819500000075,61.162468],[27.141151500000035,61.01054399900005],[27.24832250000003,60.97030949800006],[27.121067,60.96401549600006],[27.991279500000076,60.668978999000046],[27.797273500000074,60.545343],[27.425736,60.465340999000034],[27.161235,60.571331],[27.182419500000037,60.52854099700005],[26.885828140000058,60.46577349100005],[26.928463500000078,60.429250496000066],[26.570195349000073,60.44022449100004],[26.660412,60.41773950000004],[26.543459,60.37468249700004],[26.508428,60.40369799700005],[26.552183126000045,60.43506634700003],[26.453932,60.487652499000035],[26.539152999000066,60.59493749900008],[26.20835950000003,60.75799249900007],[26.077825501000063,60.835475],[26.168979001000025,60.929372500000056],[26.060193500000025,60.93608149600004],[26.132901,61.02774549900005],[26.03981500100008,61.104347500000074],[26.513659500000074,61.274324],[26.641758500000037,61.282538997000074],[26.661315,61.20687649900003],[26.881218,61.28520149900004],[26.929526,61.21003450000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1C5","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Etelä-Karjala","NUTS_NAME":"Etelä-Karjala"},"geometry":{"type":"Polygon","coordinates":[[[27.991279500000076,60.668978999000046],[27.121067,60.96401549600006],[27.24832250000003,60.97030949800006],[27.141151500000035,61.01054399900005],[27.205819500000075,61.162468],[27.577049500000044,61.26319750000005],[27.595544500000074,61.34316549700003],[28.273757500000045,61.384163999000066],[28.43936350000007,61.51073249900003],[28.648602500000038,61.464921496000045],[29.261573,61.557764500000076],[29.559279,61.722068],[30.143966500000033,61.85223799800008],[29.496255500000075,61.49189249600005],[29.24483250000003,61.27068449600006],[27.991279500000076,60.668978999000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D","LEVL_CODE":2,"CNTR_CODE":"FI","NAME_LATN":"Pohjois- ja Itä-Suomi","NUTS_NAME":"Pohjois- ja Itä-Suomi"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.984522,70.01396549800006],[28.404522,69.81841799900008],[29.13434250000006,69.69532949900008],[29.336974,69.47831049900003],[28.831907,69.22435249800003],[28.805792500000052,69.11114699900008],[28.92968,69.05190499900004],[28.415766500000075,68.91545250000007],[28.800767500000063,68.869286],[28.433917500000064,68.53967199900006],[28.646142500000053,68.19630450000005],[29.327081500000077,68.07454449900007],[29.65940050000006,67.80296249900005],[30.017040500000064,67.67355649900003],[29.93022250000007,67.52252749900003],[29.033642,66.942135],[29.572938500000078,66.43285599900008],[29.92330050000004,66.12714649900005],[30.13848,65.668682],[29.721886,65.637089],[29.863920500000063,65.56044799900008],[29.754731500000048,65.49736949700008],[29.7466,65.34740549900005],[29.601807,65.25993599900005],[29.885679500000037,65.20629949900007],[29.819355,65.14428249800005],[29.896880500000066,65.10514399900006],[29.626998,65.06059999800004],[29.599355,64.99505849600007],[29.64493,64.86649549900005],[30.086534500000027,64.77399049800005],[30.04142150000007,64.74119549900007],[30.13013550000005,64.63473],[29.989796500000068,64.58714649900003],[30.044892,64.40202999700006],[30.482463,64.26232699900004],[30.509623,64.22168750000003],[30.466391,64.20443149900007],[30.553427999000064,64.13224799800008],[30.260437,63.82200949800006],[29.971917,63.75716549900005],[30.48403,63.466702],[31.239880500000027,63.217652497000074],[31.272150500000066,63.107623999000054],[31.586729,62.908702],[31.162545,62.446056499000065],[30.143966500000033,61.85223799800008],[29.559279,61.722068],[29.261573,61.557764500000076],[28.648602500000038,61.464921496000045],[28.43936350000007,61.51073249900003],[28.273757500000045,61.384163999000066],[27.595544500000074,61.34316549700003],[27.577049500000044,61.26319750000005],[27.205819500000075,61.162468],[26.929526,61.21003450000006],[26.881218,61.28520149900004],[26.661315,61.20687649900003],[26.641758500000037,61.282538997000074],[26.513659500000074,61.274324],[26.483068,61.32283449800008],[26.243660001000023,61.44645],[26.34807900100003,61.504390500000056],[26.315036,61.60844849700004],[26.53375,61.69132699600004],[26.324290500000075,61.880033999000034],[26.414589500000034,61.972885500000075],[26.220722,62.11856249600004],[26.544035500000064,62.26562749800007],[26.652633500000036,62.21692849900006],[26.61078150000003,62.29497999700004],[26.776128,62.354111],[26.680536500000073,62.40382399900005],[26.709129500000074,62.45294899700008],[26.470699500000023,62.51892199900004],[26.598391500000048,62.610774],[26.446567,62.78917499900007],[26.356167,62.747412],[26.039845,62.93611350000003],[26.33256,62.98357199700007],[26.222582,63.02587699900005],[26.334283500000026,63.035092500000076],[26.163523500000053,63.147045],[26.175867,63.24138299800006],[26.070552,63.354783499000064],[26.176724,63.43886549800004],[26.138652,63.457591999000044],[25.892824,63.43965549900008],[25.251103999000065,63.612298],[25.03632250000004,63.458078],[24.77292650000004,63.358689499000036],[24.803607001000046,63.275935497000035],[24.75417150000004,63.26592349900005],[24.731613001000028,63.14704],[24.77288550000003,63.13336849600006],[24.359008,63.11643450000008],[23.69698950000003,63.38032699900003],[23.651032001000033,63.53094149800006],[23.42240950000007,63.61602099900006],[23.74408500100003,63.675444],[23.477644501000043,63.77913750000005],[23.235590500000058,63.71710799500005],[23.02727150000004,63.78876750000006],[23.00474550000007,63.85302299800003],[23.406509,63.940069],[23.39561400100007,64.09290799900003],[23.617333001000077,64.04877199700007],[23.581215,64.11728299900005],[23.720776500000056,64.18612999900006],[23.68772050000007,64.21953549800003],[24.3712,64.54583399800003],[24.406876500000067,64.66637349700005],[24.565660500000035,64.73118299900005],[24.539910500000076,64.81454849600004],[25.104935,64.921151],[25.316963500000043,64.82914199900006],[25.393918,64.89429649900006],[25.197500500000046,65.008802],[25.487671,64.96143649900006],[25.403687,64.99759549700008],[25.384283,65.096682],[25.229462999000077,65.12699449700006],[25.303327,65.198627],[25.255772500000035,65.21988150000004],[25.317455,65.28966599700004],[25.268597500000055,65.31379299700006],[25.35660499900007,65.34515899800004],[25.262014,65.408999],[25.366445,65.42749949800003],[25.08460749900007,65.58894949600005],[24.887724500000047,65.66578199900005],[24.657702,65.64094649900005],[24.677564500000074,65.65918699900004],[24.641739,65.70509749800004],[24.312017500000024,65.80646299600005],[24.159964,65.74979899900006],[24.155129,65.81602699900003],[23.918210499000054,66.16141],[23.64559950000006,66.30140849900005],[23.650217501000043,66.45498399900004],[23.856645,66.55690399900004],[23.907815500000027,66.72142649900007],[23.879574500000047,66.76324399900005],[23.995160501000043,66.81970899800007],[23.55127200100003,67.17146199900003],[23.560987999000076,67.26085349800007],[23.722462,67.28252899800003],[23.764582500000074,67.428214],[23.39402350000006,67.48582099800007],[23.55897800100007,67.61996149800007],[23.477203,67.70166250000005],[23.494326001000047,67.86402599900003],[23.652348999000026,67.958793],[23.15471750100005,68.136291],[23.143492500000036,68.24682799900006],[22.844611,68.38334150000003],[21.88894300100003,68.58438349900007],[20.548636500000043,69.05996850000008],[21.057545,69.03628949800003],[21.108676,69.10392949900006],[20.987576500000046,69.191927],[21.27882150000005,69.31188399800004],[21.627087500000073,69.27658799800008],[21.983611500000052,69.07289349900003],[22.340785,68.82722549900006],[22.374526,68.71666699900004],[23.167591,68.62851849800006],[23.871476,68.83651899800003],[24.903199500000028,68.55459099900008],[25.116428,68.63958749900007],[25.157167,68.79989199900007],[25.777502500000026,69.018279],[25.702101,69.25366150000008],[25.976791500000047,69.61067350000008],[25.89156,69.66550199800008],[25.950607,69.69655049900007],[26.490588,69.94501949900007],[27.037634500000024,69.91038799900008],[27.959615,70.09209449800005],[27.984522,70.01396549800006]]],[[[24.879357,65.06499149900003],[25.069460500000048,65.03647999900005],[24.795605,65.05142949900005],[24.878009999000028,64.96759499900003],[24.792422500000043,64.941664],[24.573687,65.021007],[24.879357,65.06499149900003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D1","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Etelä-Savo","NUTS_NAME":"Etelä-Savo"},"geometry":{"type":"Polygon","coordinates":[[[28.65516,62.657019999000056],[29.031536,62.53053650000004],[29.104931500000077,62.47959449900003],[29.101953500000036,62.39707949900003],[29.29596550000008,62.38672799900007],[29.430081999000038,62.23652899700005],[29.407268,62.20178499900004],[29.482342,62.056682496000064],[29.66037250000005,61.97689949800008],[29.61486350100006,61.91332899900004],[29.690667500000075,61.82584049700006],[29.559279,61.722068],[29.261573,61.557764500000076],[28.648602500000038,61.464921496000045],[28.43936350000007,61.51073249900003],[28.273757500000045,61.384163999000066],[27.595544500000074,61.34316549700003],[27.577049500000044,61.26319750000005],[27.205819500000075,61.162468],[26.929526,61.21003450000006],[26.881218,61.28520149900004],[26.661315,61.20687649900003],[26.641758500000037,61.282538997000074],[26.513659500000074,61.274324],[26.483068,61.32283449800008],[26.243660001000023,61.44645],[26.34807900100003,61.504390500000056],[26.315036,61.60844849700004],[26.53375,61.69132699600004],[26.324290500000075,61.880033999000034],[26.414589500000034,61.972885500000075],[26.220722,62.11856249600004],[26.544035500000064,62.26562749800007],[26.652633500000036,62.21692849900006],[26.61078150000003,62.29497999700004],[26.776128,62.354111],[26.680536500000073,62.40382399900005],[26.709129500000074,62.45294899700008],[27.3843,62.51428650000008],[27.833064500000035,62.27819599700007],[28.61618550000003,62.141008001000046],[28.652808,62.23132299900004],[28.244780500000047,62.42404449700007],[28.381875,62.46163449900007],[28.40108250000003,62.575045499000055],[28.52078349900006,62.54845499600003],[28.466983,62.583161],[28.65516,62.657019999000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D2","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Pohjois-Savo","NUTS_NAME":"Pohjois-Savo"},"geometry":{"type":"Polygon","coordinates":[[[28.359609500000033,63.75475349900006],[28.606482999000036,63.67466049900008],[28.603476,63.52745599800005],[28.759542499000077,63.387808997000036],[28.610819,63.20383499600007],[29.038022500000068,63.01085149800008],[29.06755,62.98418550000008],[28.930923,62.936619],[29.097615,62.84558799900003],[28.942882999000062,62.77489499600006],[28.79010549900005,62.83379499900008],[28.642575500000078,62.76078449600004],[28.702890500000024,62.694259497000075],[28.65516,62.657019999000056],[28.466983,62.583161],[28.52078349900006,62.54845499600003],[28.40108250000003,62.575045499000055],[28.381875,62.46163449900007],[28.244780500000047,62.42404449700007],[28.652808,62.23132299900004],[28.61618550000003,62.141008001000046],[27.833064500000035,62.27819599700007],[27.3843,62.51428650000008],[26.709129500000074,62.45294899700008],[26.470699500000023,62.51892199900004],[26.598391500000048,62.610774],[26.446567,62.78917499900007],[26.356167,62.747412],[26.039845,62.93611350000003],[26.33256,62.98357199700007],[26.222582,63.02587699900005],[26.334283500000026,63.035092500000076],[26.163523500000053,63.147045],[26.175867,63.24138299800006],[26.070552,63.354783499000064],[26.176724,63.43886549800004],[26.138652,63.457591999000044],[26.291817,63.58995149900005],[26.21814499900006,63.831633500000066],[26.870419,64.03697749900005],[27.74505,63.90837799700006],[28.273933,63.66881249700003],[28.359609500000033,63.75475349900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D3","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Pohjois-Karjala","NUTS_NAME":"Pohjois-Karjala"},"geometry":{"type":"Polygon","coordinates":[[[29.971917,63.75716549900005],[30.48403,63.466702],[31.239880500000027,63.217652497000074],[31.272150500000066,63.107623999000054],[31.586729,62.908702],[31.162545,62.446056499000065],[30.143966500000033,61.85223799800008],[29.559279,61.722068],[29.690667500000075,61.82584049700006],[29.61486350100006,61.91332899900004],[29.66037250000005,61.97689949800008],[29.482342,62.056682496000064],[29.407268,62.20178499900004],[29.430081999000038,62.23652899700005],[29.29596550000008,62.38672799900007],[29.101953500000036,62.39707949900003],[29.104931500000077,62.47959449900003],[29.031536,62.53053650000004],[28.65516,62.657019999000056],[28.702890500000024,62.694259497000075],[28.642575500000078,62.76078449600004],[28.79010549900005,62.83379499900008],[28.942882999000062,62.77489499600006],[29.097615,62.84558799900003],[28.930923,62.936619],[29.06755,62.98418550000008],[29.038022500000068,63.01085149800008],[28.610819,63.20383499600007],[28.759542499000077,63.387808997000036],[28.603476,63.52745599800005],[28.606482999000036,63.67466049900008],[28.359609500000033,63.75475349900006],[28.459982500000024,63.87111849900003],[29.68254950000005,63.878943],[29.775711,63.70335649800006],[29.971917,63.75716549900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D4","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Kainuu","NUTS_NAME":"Kainuu"},"geometry":{"type":"Polygon","coordinates":[[[29.754731500000048,65.49736949700008],[29.7466,65.34740549900005],[29.601807,65.25993599900005],[29.885679500000037,65.20629949900007],[29.819355,65.14428249800005],[29.896880500000066,65.10514399900006],[29.626998,65.06059999800004],[29.599355,64.99505849600007],[29.64493,64.86649549900005],[30.086534500000027,64.77399049800005],[30.04142150000007,64.74119549900007],[30.13013550000005,64.63473],[29.989796500000068,64.58714649900003],[30.044892,64.40202999700006],[30.482463,64.26232699900004],[30.509623,64.22168750000003],[30.466391,64.20443149900007],[30.553427999000064,64.13224799800008],[30.260437,63.82200949800006],[29.971917,63.75716549900005],[29.775711,63.70335649800006],[29.68254950000005,63.878943],[28.459982500000024,63.87111849900003],[28.359609500000033,63.75475349900006],[28.273933,63.66881249700003],[27.74505,63.90837799700006],[26.870419,64.03697749900005],[26.78112550000003,64.05710399700007],[26.767518,64.26516899900008],[26.291123,64.47906849700007],[26.840549500000066,64.66188199800007],[26.90830150000005,64.75909249800003],[27.40266,64.85524849900008],[27.385771500000033,64.959017],[27.262523500000043,65.02734249900004],[27.329903499000068,65.09066249700004],[27.894294,65.09147149600005],[28.033897500000023,65.23856399600004],[28.608589,65.28668149900005],[28.646934500000043,65.38934099900007],[29.213235,65.51848149900007],[29.754731500000048,65.49736949700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D5","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Keski-Pohjanmaa","NUTS_NAME":"Keski-Pohjanmaa"},"geometry":{"type":"Polygon","coordinates":[[[23.711572500000045,63.977640499000074],[23.910618,63.96498699700004],[23.879619,64.01241499900004],[24.08708800100004,64.07115199700007],[24.247460500000045,63.87695599700004],[25.03632250000004,63.458078],[24.77292650000004,63.358689499000036],[24.803607001000046,63.275935497000035],[24.75417150000004,63.26592349900005],[24.731613001000028,63.14704],[24.77288550000003,63.13336849600006],[24.359008,63.11643450000008],[23.69698950000003,63.38032699900003],[23.651032001000033,63.53094149800006],[23.42240950000007,63.61602099900006],[23.74408500100003,63.675444],[23.477644501000043,63.77913750000005],[23.235590500000058,63.71710799500005],[23.02727150000004,63.78876750000006],[23.00474550000007,63.85302299800003],[23.406509,63.940069],[23.39561400100007,64.09290799900003],[23.617333001000077,64.04877199700007],[23.711572500000045,63.977640499000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D6","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Pohjois-Pohjanmaa","NUTS_NAME":"Pohjois-Pohjanmaa"},"geometry":{"type":"MultiPolygon","coordinates":[[[[29.92330050000004,66.12714649900005],[30.13848,65.668682],[29.721886,65.637089],[29.863920500000063,65.56044799900008],[29.754731500000048,65.49736949700008],[29.213235,65.51848149900007],[28.646934500000043,65.38934099900007],[28.608589,65.28668149900005],[28.033897500000023,65.23856399600004],[27.894294,65.09147149600005],[27.329903499000068,65.09066249700004],[27.262523500000043,65.02734249900004],[27.385771500000033,64.959017],[27.40266,64.85524849900008],[26.90830150000005,64.75909249800003],[26.840549500000066,64.66188199800007],[26.291123,64.47906849700007],[26.767518,64.26516899900008],[26.78112550000003,64.05710399700007],[26.870419,64.03697749900005],[26.21814499900006,63.831633500000066],[26.291817,63.58995149900005],[26.138652,63.457591999000044],[25.892824,63.43965549900008],[25.251103999000065,63.612298],[25.03632250000004,63.458078],[24.247460500000045,63.87695599700004],[24.08708800100004,64.07115199700007],[23.879619,64.01241499900004],[23.910618,63.96498699700004],[23.711572500000045,63.977640499000074],[23.617333001000077,64.04877199700007],[23.581215,64.11728299900005],[23.720776500000056,64.18612999900006],[23.68772050000007,64.21953549800003],[24.3712,64.54583399800003],[24.406876500000067,64.66637349700005],[24.565660500000035,64.73118299900005],[24.539910500000076,64.81454849600004],[25.104935,64.921151],[25.316963500000043,64.82914199900006],[25.393918,64.89429649900006],[25.197500500000046,65.008802],[25.487671,64.96143649900006],[25.403687,64.99759549700008],[25.384283,65.096682],[25.229462999000077,65.12699449700006],[25.303327,65.198627],[25.255772500000035,65.21988150000004],[25.317455,65.28966599700004],[25.268597500000055,65.31379299700006],[25.35660499900007,65.34515899800004],[25.262014,65.408999],[25.366445,65.42749949800003],[25.08460749900007,65.58894949600005],[25.424521,65.742569],[26.160127,65.82233599900007],[26.158250500000065,65.72555649900005],[27.071996,65.72977249900003],[27.406743,65.938286],[27.654889,65.96036149900004],[27.761115500000074,65.82732],[28.513211,65.84621049900005],[28.370830500000068,65.99845850000008],[28.764253500000052,66.04334799900005],[28.60417650000005,66.12816549800004],[28.715774500000066,66.13591099900003],[28.768438,66.18455599900005],[28.749671500000034,66.25448599800006],[28.538099499000054,66.33261299900005],[28.695322499000042,66.48585599900008],[29.572938500000078,66.43285599900008],[29.92330050000004,66.12714649900005]]],[[[24.879357,65.06499149900003],[25.069460500000048,65.03647999900005],[24.795605,65.05142949900005],[24.878009999000028,64.96759499900003],[24.792422500000043,64.941664],[24.573687,65.021007],[24.879357,65.06499149900003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI1D7","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Lappi","NUTS_NAME":"Lappi"},"geometry":{"type":"Polygon","coordinates":[[[27.984522,70.01396549800006],[28.404522,69.81841799900008],[29.13434250000006,69.69532949900008],[29.336974,69.47831049900003],[28.831907,69.22435249800003],[28.805792500000052,69.11114699900008],[28.92968,69.05190499900004],[28.415766500000075,68.91545250000007],[28.800767500000063,68.869286],[28.433917500000064,68.53967199900006],[28.646142500000053,68.19630450000005],[29.327081500000077,68.07454449900007],[29.65940050000006,67.80296249900005],[30.017040500000064,67.67355649900003],[29.93022250000007,67.52252749900003],[29.033642,66.942135],[29.572938500000078,66.43285599900008],[28.695322499000042,66.48585599900008],[28.538099499000054,66.33261299900005],[28.749671500000034,66.25448599800006],[28.768438,66.18455599900005],[28.715774500000066,66.13591099900003],[28.60417650000005,66.12816549800004],[28.764253500000052,66.04334799900005],[28.370830500000068,65.99845850000008],[28.513211,65.84621049900005],[27.761115500000074,65.82732],[27.654889,65.96036149900004],[27.406743,65.938286],[27.071996,65.72977249900003],[26.158250500000065,65.72555649900005],[26.160127,65.82233599900007],[25.424521,65.742569],[25.08460749900007,65.58894949600005],[24.887724500000047,65.66578199900005],[24.657702,65.64094649900005],[24.677564500000074,65.65918699900004],[24.641739,65.70509749800004],[24.312017500000024,65.80646299600005],[24.159964,65.74979899900006],[24.155129,65.81602699900003],[23.918210499000054,66.16141],[23.64559950000006,66.30140849900005],[23.650217501000043,66.45498399900004],[23.856645,66.55690399900004],[23.907815500000027,66.72142649900007],[23.879574500000047,66.76324399900005],[23.995160501000043,66.81970899800007],[23.55127200100003,67.17146199900003],[23.560987999000076,67.26085349800007],[23.722462,67.28252899800003],[23.764582500000074,67.428214],[23.39402350000006,67.48582099800007],[23.55897800100007,67.61996149800007],[23.477203,67.70166250000005],[23.494326001000047,67.86402599900003],[23.652348999000026,67.958793],[23.15471750100005,68.136291],[23.143492500000036,68.24682799900006],[22.844611,68.38334150000003],[21.88894300100003,68.58438349900007],[20.548636500000043,69.05996850000008],[21.057545,69.03628949800003],[21.108676,69.10392949900006],[20.987576500000046,69.191927],[21.27882150000005,69.31188399800004],[21.627087500000073,69.27658799800008],[21.983611500000052,69.07289349900003],[22.340785,68.82722549900006],[22.374526,68.71666699900004],[23.167591,68.62851849800006],[23.871476,68.83651899800003],[24.903199500000028,68.55459099900008],[25.116428,68.63958749900007],[25.157167,68.79989199900007],[25.777502500000026,69.018279],[25.702101,69.25366150000008],[25.976791500000047,69.61067350000008],[25.89156,69.66550199800008],[25.950607,69.69655049900007],[26.490588,69.94501949900007],[27.037634500000024,69.91038799900008],[27.959615,70.09209449800005],[27.984522,70.01396549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI2","LEVL_CODE":1,"CNTR_CODE":"FI","NAME_LATN":"ÅLAND","NUTS_NAME":"ÅLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.916036500000075,60.42270650100005],[20.275351500000056,60.285269999000036],[20.16831250000007,60.17514399700008],[20.017924277000077,60.23618142000004],[20.043062,60.102843],[19.98094150000003,60.10417199900007],[19.957714500000066,60.060943500000064],[19.749325500000054,60.094862499000044],[19.675316817000066,60.21566829400007],[19.513948,60.17833699900007],[19.610745,60.29433850000004],[19.77291450000007,60.29507699900006],[19.737465,60.33260749700008],[19.809224499000038,60.319052499000065],[19.76569150000006,60.264571499000056],[19.804114,60.199625999000034],[19.896939501000077,60.28221550000006],[19.761749,60.40846249800006],[19.916036500000075,60.42270650100005]],[[20.03254050000004,60.31737399900004],[20.022423531000072,60.26115329100003],[20.057007500000054,60.30628099900008],[20.03254050000004,60.31737399900004]]],[[[20.213452500000074,59.999726499000076],[20.17940550000003,59.96864349700007],[19.974522500000035,60.07411649900007],[20.074453,60.07716049900006],[20.08269350000006,60.10944549800007],[20.225252500000067,60.078295499000035],[20.213452500000074,59.999726499000076]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI20","LEVL_CODE":2,"CNTR_CODE":"FI","NAME_LATN":"Åland","NUTS_NAME":"Åland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.916036500000075,60.42270650100005],[20.275351500000056,60.285269999000036],[20.16831250000007,60.17514399700008],[20.017924277000077,60.23618142000004],[20.043062,60.102843],[19.98094150000003,60.10417199900007],[19.957714500000066,60.060943500000064],[19.749325500000054,60.094862499000044],[19.675316817000066,60.21566829400007],[19.513948,60.17833699900007],[19.610745,60.29433850000004],[19.77291450000007,60.29507699900006],[19.737465,60.33260749700008],[19.809224499000038,60.319052499000065],[19.76569150000006,60.264571499000056],[19.804114,60.199625999000034],[19.896939501000077,60.28221550000006],[19.761749,60.40846249800006],[19.916036500000075,60.42270650100005]],[[20.03254050000004,60.31737399900004],[20.022423531000072,60.26115329100003],[20.057007500000054,60.30628099900008],[20.03254050000004,60.31737399900004]]],[[[20.213452500000074,59.999726499000076],[20.17940550000003,59.96864349700007],[19.974522500000035,60.07411649900007],[20.074453,60.07716049900006],[20.08269350000006,60.10944549800007],[20.225252500000067,60.078295499000035],[20.213452500000074,59.999726499000076]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FI200","LEVL_CODE":3,"CNTR_CODE":"FI","NAME_LATN":"Åland","NUTS_NAME":"Åland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.916036500000075,60.42270650100005],[20.275351500000056,60.285269999000036],[20.16831250000007,60.17514399700008],[20.017924277000077,60.23618142000004],[20.043062,60.102843],[19.98094150000003,60.10417199900007],[19.957714500000066,60.060943500000064],[19.749325500000054,60.094862499000044],[19.675316817000066,60.21566829400007],[19.513948,60.17833699900007],[19.610745,60.29433850000004],[19.77291450000007,60.29507699900006],[19.737465,60.33260749700008],[19.809224499000038,60.319052499000065],[19.76569150000006,60.264571499000056],[19.804114,60.199625999000034],[19.896939501000077,60.28221550000006],[19.761749,60.40846249800006],[19.916036500000075,60.42270650100005]],[[20.03254050000004,60.31737399900004],[20.022423531000072,60.26115329100003],[20.057007500000054,60.30628099900008],[20.03254050000004,60.31737399900004]]],[[[20.213452500000074,59.999726499000076],[20.17940550000003,59.96864349700007],[19.974522500000035,60.07411649900007],[20.074453,60.07716049900006],[20.08269350000006,60.10944549800007],[20.225252500000067,60.078295499000035],[20.213452500000074,59.999726499000076]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR1","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"ÎLE DE FRANCE","NUTS_NAME":"ÎLE DE FRANCE"},"geometry":{"type":"Polygon","coordinates":[[[1.704359,49.23219699600003],[1.882564500000058,49.16249099800007],[2.321773,49.18443],[2.590528500000062,49.079653998000026],[3.07188,49.11755350000004],[3.165230500000064,49.099653501000034],[3.265159500000038,48.939061],[3.485183500000062,48.85191049800005],[3.395836,48.75924699700005],[3.444068500000071,48.736630999000056],[3.451096,48.634649997000054],[3.557419164000066,48.61671378700004],[3.405396500000052,48.52801499800006],[3.414788999000052,48.39026849800007],[3.049454,48.36002749800008],[3.031462,48.248878998000066],[2.936316,48.163391499000056],[2.476016500000071,48.12963649900007],[2.522821,48.197726497000076],[2.402663,48.32071749900007],[1.99409,48.28658399900007],[1.922146500000053,48.457599498000036],[1.801452500000039,48.46604099600006],[1.757405500000061,48.57495449900006],[1.602704,48.66309299800008],[1.626540500000033,48.74804799900005],[1.501526500000068,48.941051997000045],[1.518385500000022,48.978357997000046],[1.470892,48.974814500000036],[1.447285,49.05350949800004],[1.608799,49.077893999000025],[1.704359,49.23219699600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR10","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Île de France","NUTS_NAME":"Île de France"},"geometry":{"type":"Polygon","coordinates":[[[1.704359,49.23219699600003],[1.882564500000058,49.16249099800007],[2.321773,49.18443],[2.590528500000062,49.079653998000026],[3.07188,49.11755350000004],[3.165230500000064,49.099653501000034],[3.265159500000038,48.939061],[3.485183500000062,48.85191049800005],[3.395836,48.75924699700005],[3.444068500000071,48.736630999000056],[3.451096,48.634649997000054],[3.557419164000066,48.61671378700004],[3.405396500000052,48.52801499800006],[3.414788999000052,48.39026849800007],[3.049454,48.36002749800008],[3.031462,48.248878998000066],[2.936316,48.163391499000056],[2.476016500000071,48.12963649900007],[2.522821,48.197726497000076],[2.402663,48.32071749900007],[1.99409,48.28658399900007],[1.922146500000053,48.457599498000036],[1.801452500000039,48.46604099600006],[1.757405500000061,48.57495449900006],[1.602704,48.66309299800008],[1.626540500000033,48.74804799900005],[1.501526500000068,48.941051997000045],[1.518385500000022,48.978357997000046],[1.470892,48.974814500000036],[1.447285,49.05350949800004],[1.608799,49.077893999000025],[1.704359,49.23219699600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR101","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Paris","NUTS_NAME":"Paris"},"geometry":{"type":"Polygon","coordinates":[[[2.416345,48.849239],[2.469741500000055,48.835805],[2.331903500000067,48.81701349800005],[2.224219,48.85351699900008],[2.319887,48.90046],[2.416345,48.849239]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR102","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Seine-et-Marne","NUTS_NAME":"Seine-et-Marne"},"geometry":{"type":"Polygon","coordinates":[[[3.485183500000062,48.85191049800005],[3.395836,48.75924699700005],[3.444068500000071,48.736630999000056],[3.451096,48.634649997000054],[3.557419164000066,48.61671378700004],[3.405396500000052,48.52801499800006],[3.414788999000052,48.39026849800007],[3.049454,48.36002749800008],[3.031462,48.248878998000066],[2.936316,48.163391499000056],[2.476016500000071,48.12963649900007],[2.522821,48.197726497000076],[2.402663,48.32071749900007],[2.50288050000006,48.39291299800004],[2.502907500000049,48.54600499800006],[2.571653500000025,48.69201699800004],[2.592273500000033,48.80743749800007],[2.559406500000023,48.885336496000036],[2.6026,48.92935649700007],[2.553056499000036,49.00981599700003],[2.590528500000062,49.079653998000026],[3.07188,49.11755350000004],[3.165230500000064,49.099653501000034],[3.265159500000038,48.939061],[3.485183500000062,48.85191049800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR103","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Yvelines","NUTS_NAME":"Yvelines"},"geometry":{"type":"Polygon","coordinates":[[[2.200591,48.908679498000026],[2.145867,48.83689499700006],[2.226554,48.77610199800006],[2.011351,48.659910496000066],[2.01757,48.557593997000026],[1.93789,48.56292649800008],[1.966741,48.533469998000044],[1.922146500000053,48.457599498000036],[1.801452500000039,48.46604099600006],[1.757405500000061,48.57495449900006],[1.602704,48.66309299800008],[1.626540500000033,48.74804799900005],[1.501526500000068,48.941051997000045],[1.518385500000022,48.978357997000046],[1.470892,48.974814500000036],[1.447285,49.05350949800004],[1.608799,49.077893999000025],[2.110076500000048,49.008717998000066],[2.200591,48.908679498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR","LEVL_CODE":0,"CNTR_CODE":"FR","NAME_LATN":"FRANCE","NUTS_NAME":"FRANCE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[55.797395,-21.350129496999955],[55.609923500000036,-21.387309496999933],[55.292974,-21.229784996999967],[55.217144,-21.036134996999976],[55.275858,-21.001988496999957],[55.283669,-20.926209497999935],[55.395047,-20.881834998999977],[55.64773750000006,-20.915604496999947],[55.82579650000008,-21.143524497999977],[55.797395,-21.350129496999955]]],[[[45.21328150000005,-12.87307549999997],[45.14830050000006,-13.000090498999953],[45.071559,-12.902030998999976],[45.154967,-12.923567997999953],[45.04258,-12.74402349899998],[45.08842350000003,-12.669118998999977],[45.125466999000025,-12.727415998999959],[45.236856,-12.753384997999945],[45.18605050000008,-12.839496],[45.21328150000005,-12.87307549999997]]],[[[9.492015,42.804485498000076],[9.446189,42.673576498000045],[9.53385350000002,42.542114498000046],[9.55911750000007,42.19669699600007],[9.549904,42.10403799800008],[9.41378,41.9557],[9.402272,41.85870349800007],[9.388422,41.655113],[9.219715,41.36760399800005],[8.788534,41.55707549600004],[8.79128,41.629024498000035],[8.91601,41.687378],[8.684517,41.74700149700004],[8.802622,41.896465496000076],[8.609244500000045,41.90718099700007],[8.591134,41.96215449600004],[8.747044500000072,42.04953],[8.585460500000067,42.151729498000066],[8.57300649900003,42.23857099700007],[8.691523500000073,42.266464],[8.60024450000003,42.35300049600005],[8.553934,42.33502199800006],[8.573408,42.38140499900004],[8.655066500000032,42.415080998000064],[8.666671,42.51559849900008],[8.727043,42.56160349800007],[9.059321500000067,42.660064498000054],[9.122977500000047,42.731223999000065],[9.301698500000043,42.678638500000034],[9.345141500000068,42.73666399600006],[9.309376500000042,42.83339299800008],[9.36147,42.924758997000026],[9.343248500000072,42.99974049900004],[9.463280500000053,42.98673649800003],[9.492015,42.804485498000076]]],[[[2.63495,50.81274399900008],[2.863276,50.708343496000055],[3.018708500000059,50.773532998000064],[3.098481,50.77901899900007],[3.176996,50.75616449700004],[3.245294,50.71300949700003],[3.286492,50.52756899700006],[3.615081500000031,50.490399],[3.65551,50.46173549800005],[3.710389,50.303165499000045],[4.027774500000021,50.35833049800004],[4.221769,50.255832500000054],[4.126875,50.13492199800004],[4.231028,50.071983498000066],[4.140853,49.978759996000065],[4.233133,49.95782849700004],[4.432494,49.94161599600005],[4.694556,49.99650949900007],[4.680489,50.06813850000003],[4.796697,50.14867799800004],[4.896794,50.13742049800004],[4.793194,49.981999],[4.889914,49.90920999800005],[4.851578,49.793255],[4.969431,49.80182599800003],[5.153738500000031,49.717925999000045],[5.393511,49.61711099900003],[5.470882999000025,49.49723799800006],[5.734555999000065,49.545690500000035],[5.818117,49.54631049900007],[5.893386,49.49694449700007],[6.367107500000031,49.469507],[6.556986,49.419208498000046],[6.565664,49.34765149700007],[6.723465500000032,49.21882899800005],[6.738397500000076,49.16359799900005],[6.834567,49.15131099800004],[6.837607499000057,49.21293299700005],[6.925060500000029,49.22334649800007],[7.03497150000004,49.19141499700004],[7.057195999000044,49.11193149800005],[7.101069,49.15599799900008],[7.293177,49.11490749700005],[7.36875550000002,49.16145799800006],[7.493791,49.16773550000005],[7.635651,49.05395049900005],[7.910654500000021,49.04516349800008],[8.068407500000035,48.99931649700005],[8.232632999000032,48.96657150000004],[7.95963050000006,48.71858099800005],[7.804824,48.59257949800008],[7.739841500000068,48.321731500000055],[7.680713,48.257266996000055],[7.577919500000064,48.121391998000036],[7.577291,48.11565499800008],[7.568529,48.035183998000036],[7.621966500000042,47.97474899800005],[7.558898500000055,47.881823498000074],[7.545990500000073,47.743573998000045],[7.519558,47.66812299900005],[7.589039,47.58987799800008],[7.555159500000059,47.56456399600006],[7.510905500000035,47.50258249700005],[7.445019,47.46172349600005],[7.421139499000049,47.446387998000034],[7.380894,47.43189249800008],[7.326466,47.43985349900004],[7.170422,47.44345449800005],[7.130353,47.503040498000075],[6.982914,47.49427049800005],[7.00155,47.45374349700006],[6.939186,47.433704499000044],[6.879806,47.35243999700003],[7.061636,47.34373449800006],[6.858914,47.16529449800004],[6.43275,46.92845549800006],[6.460011,46.85155149800005],[6.439425,46.76257299900004],[6.110961,46.576656496000055],[6.138106,46.55766699600008],[6.064003,46.41622899600003],[6.170135999000024,46.36643599800004],[6.125608,46.31722999800007],[6.088192,46.246158499000046],[5.974414,46.214751996000075],[5.956067,46.13209549900006],[6.135542,46.141159],[6.310211,46.24404549800005],[6.219547,46.31187799800006],[6.231694,46.329429497000035],[6.24136,46.34358249600007],[6.519104,46.45636749600004],[6.821063999000046,46.42715449800005],[6.770442,46.35520549800003],[6.864808,46.28059399800003],[6.797888999000065,46.136798998000074],[6.899467,46.124058],[6.891464,46.07387949800005],[7.044886,45.92241299800003],[6.818154500000048,45.83595249800004],[6.802368500000057,45.77856199900003],[6.845962,45.690503997000064],[7.000765,45.64000749600007],[7.00001750000007,45.50466699700007],[7.104723,45.46845449800003],[7.185578500000076,45.40326749800005],[7.109843500000068,45.32694349600007],[7.125157,45.243994497000074],[6.893854500000032,45.13691449700008],[6.630051,45.109856499000045],[6.6733,45.01962749900008],[6.744224,45.01574049800007],[6.751577,44.90558199700007],[7.006606,44.839803998000036],[7.065755,44.71346449600003],[6.948443,44.65474199800008],[6.932863,44.57307149900004],[6.853195,44.52856949800008],[6.912351500000057,44.450991499000054],[6.887428,44.36128699900007],[7.007760500000074,44.23670049800006],[7.357702500000073,44.11620099800007],[7.684466,44.173948996000036],[7.714236500000027,44.06151349800007],[7.501208,43.87497399800003],[7.529827,43.78400799600007],[7.462001301000043,43.760283908000076],[7.438017,43.75141549800003],[7.413383,43.734576996000044],[7.415625,43.725730998000074],[7.158075,43.65397649800008],[6.933721,43.48006449600007],[6.855953500000055,43.41176199600005],[6.748416,43.41979999800003],[6.716054,43.34670249800007],[6.585208500000022,43.27951449700004],[6.696511,43.26633449800005],[6.621062,43.15778350000005],[6.56200750000005,43.18908699800005],[6.358751500000039,43.10664749800003],[6.200337,43.11518099800003],[6.148761,43.02589799900005],[6.091259500000035,43.03535449700007],[6.125455500000044,43.07826999800005],[5.923431,43.12327950000008],[5.880380500000058,43.10545350000007],[5.911596,43.065773],[5.829222,43.04917549800007],[5.807260500000041,43.115710996000075],[5.671879,43.17926799800006],[5.337685500000021,43.21367649800004],[5.376047,43.25515749900006],[5.326588500000071,43.35578899600006],[5.053915,43.32427199800003],[4.96771,43.42610149600006],[4.824366500000053,43.42425899800003],[4.878544,43.355830998000044],[4.837622,43.32720549700008],[4.571876499000041,43.365741498000034],[4.597235500000068,43.405902998000045],[4.547286,43.44861999600005],[4.230281,43.46018599900003],[4.132095500000048,43.48568349800007],[4.101040500000067,43.55437099800008],[3.907951,43.51719299800004],[3.515212500000075,43.274783497000044],[3.400148500000057,43.28725049800005],[3.240562500000067,43.21280299800003],[3.084473,43.056163998000045],[3.050878,42.87390899900004],[3.009882,42.92759799800007],[2.988900500000057,42.864599998000074],[2.94807,42.83443249800007],[2.98537,42.79993299800003],[3.035430500000075,42.84206949900005],[3.050192,42.87063599900006],[3.04350450000004,42.83814999900005],[3.051833500000043,42.54395299600003],[3.174803999000062,42.43537499900003],[2.839501,42.45927099800008],[2.673950500000046,42.40464399900003],[2.675338500000066,42.34233849800006],[2.526214500000037,42.33339299700003],[2.306832499000052,42.42904299600008],[2.012462,42.35175299800005],[1.964552,42.38215699800003],[1.932950500000061,42.45493699800005],[1.731011,42.49240099800005],[1.725801,42.50440199800005],[1.786098500000037,42.573657999000034],[1.574720500000069,42.64779299800006],[1.442566,42.603668],[1.357304,42.71942199800003],[1.17077,42.70814149800003],[1.078708,42.788341496000044],[0.858215,42.82574099800007],[0.66215950000003,42.84124349700005],[0.660127,42.69095249800006],[0.477755500000058,42.70001999600004],[0.360133500000074,42.724353998000026],[0.295051500000056,42.673358998000026],[0.175701,42.73704149800005],[-0.055795,42.69225699800006],[-0.160069499999963,42.79801949800003],[-0.313342,42.84936499600008],[-0.551047,42.777637499000036],[-0.724501,42.92015849900008],[-0.751951499999961,42.96712099900003],[-0.945180499999935,42.95343799800003],[-1.309055499999943,43.07115950000008],[-1.270122,43.118247996000036],[-1.3322,43.10771199800007],[-1.35373150099997,43.02798099800003],[-1.44129049999998,43.04634099800006],[-1.473489499999971,43.08662049800006],[-1.414168,43.12828849700003],[-1.384407500999941,43.25365449800006],[-1.534212,43.29453649800007],[-1.608788499999946,43.251976],[-1.625259,43.30670549800004],[-1.728903,43.29608899600004],[-1.785978,43.35047899600005],[-1.628819,43.41350149800007],[-1.524870499999963,43.52970099800007],[-1.448207499999967,43.64227299800007],[-1.253891,44.46760549800007],[-1.193711499999949,44.65898899700005],[-1.004679,44.64818549800003],[-1.161932499999978,44.774902498000074],[-1.261136499999964,44.62758249600006],[-1.156919,45.472530498000026],[-1.064105499999926,45.554893496000034],[-1.065570499999978,45.51051699800007],[-0.809971499999961,45.35167699800007],[-0.746965499999931,45.22681799600008],[-0.690119,45.23636249800006],[-0.708231,45.32748049800006],[-0.819719499999962,45.49202749800003],[-1.230697499999962,45.68057249700007],[-1.241852,45.785945998000045],[-1.13209,45.80492799900003],[-1.155564,45.841975999000056],[-1.053074,46.00382999900006],[-1.123773,46.11335349800004],[-1.242043499999966,46.15744399700003],[-1.11135,46.261157998000044],[-1.129406,46.310271998000076],[-1.202044,46.31608599800006],[-1.204990499999951,46.26650599800007],[-1.812344499999938,46.49341949700005],[-1.857028,46.60985199600003],[-2.141985,46.81897349700006],[-2.149297718999946,46.89421903900006],[-2.307551,47.024879498000075],[-2.218334,47.00781999800006],[-2.148764873999937,46.89464346800003],[-1.980413,47.02890799800008],[-2.247857,47.13375099800004],[-2.166129,47.16842249900003],[-2.169989499999929,47.268470998000055],[-2.010275000999968,47.31473899700006],[-2.297556499999928,47.23431399900005],[-2.54710800099997,47.292373498000075],[-2.500607499999944,47.323757],[-2.558945,47.37456899800003],[-2.43403,47.413044],[-2.458493,47.44811999800004],[-2.501003,47.49271749900004],[-2.420451499999956,47.49520099800003],[-2.849214,47.49888249600008],[-2.919619,47.547725498000034],[-2.730782,47.54229749900003],[-2.721059,47.60310749700005],[-2.898194,47.58569699700007],[-2.98203,47.65580749800006],[-2.971034,47.55858999600008],[-3.128996499999971,47.59746149600005],[-3.089519499999938,47.47268299900003],[-3.130722499999933,47.47313299800004],[-3.158280499999933,47.60671599700004],[-3.338828,47.695411498000055],[-3.300046499999951,47.774661998000056],[-3.453312500999971,47.69553749600004],[-3.528605312999957,47.77829169600005],[-3.850509,47.79721049800003],[-3.958664499999941,47.896976496000036],[-4.099355499999945,47.86249150000003],[-4.144733499999973,47.909335998000074],[-4.115504,47.976466998000035],[-4.151007499999935,47.910702],[-4.113312,47.86203749800006],[-4.176410499999974,47.80505749800005],[-4.374185,47.79797],[-4.347687,47.849003],[-4.42255849999998,47.962432998000054],[-4.734697499999925,48.038242497000056],[-4.269574,48.13277050000005],[-4.303489,48.195732],[-4.465324499999952,48.238849499000025],[-4.557571,48.16983049800007],[-4.545083,48.25028599800004],[-4.625327500999958,48.259071498000026],[-4.554775,48.338588498000036],[-4.535044,48.28407649800005],[-4.225793499999952,48.28749099600003],[-4.332875499999943,48.31668099900003],[-4.261421499999926,48.360431498000025],[-4.45806,48.32748799800004],[-4.273602500999971,48.44343549900003],[-4.772754,48.32923499900005],[-4.769921499999953,48.52113699800003],[-4.349134,48.67647549900005],[-4.281066499999952,48.642818498000054],[-3.96533,48.72175999800004],[-3.950495,48.65286999700004],[-3.859493,48.61433799800005],[-3.831078499999933,48.712436498000045],[-3.643659499999956,48.69629649800004],[-3.659150499999953,48.65920999800005],[-3.573228,48.674724498000046],[-3.549404,48.74568549900005],[-3.584439499999974,48.79017999800004],[-3.504829499999971,48.83930599800004],[-3.43019,48.797252498000034],[-3.084001,48.865695998000035],[-2.686398499999939,48.49315249800003],[-2.3278,48.67327899800006],[-2.210832,48.57585899800006],[-2.123706499999969,48.604404498000065],[-2.148958,48.63373199600005],[-2.045604,48.63681399600006],[-2.006897,48.56610849700007],[-1.982181074999971,48.554643027000054],[-2.028575,48.65031049800007],[-1.850131,48.69939399900005],[-1.872124,48.645713998000076],[-1.571089499999971,48.626441999000065],[-1.391047,48.644653496000046],[-1.572866499999975,48.74789450000003],[-1.556390499999964,49.02472699800006],[-1.504676,49.02013399800006],[-1.600102499999934,49.04160299700004],[-1.605357,49.205581497000026],[-1.567265,49.22087849900004],[-1.625514,49.21006399800007],[-1.809161499999959,49.372093],[-1.868679499999928,49.51237099600007],[-1.855007499999942,49.64282250000008],[-1.947120499999926,49.674759],[-1.942384,49.725959998000064],[-1.630755499999964,49.65202349600003],[-1.26649249999997,49.69559849600006],[-1.228839499999935,49.60570549800008],[-1.307770499999947,49.54571899900003],[-1.11962,49.355568],[-0.961539499999958,49.396434998000075],[-0.225695,49.281821999000044],[0.297224500000027,49.429863],[0.338978500000053,49.44092549900006],[0.065609,49.51257699800004],[0.192298,49.70696249800005],[0.562341500000059,49.84552749800008],[1.07273,49.927711497000075],[1.379698,50.06500999700006],[1.545791500000064,50.214481498000055],[1.683794500000033,50.18302549800006],[1.539681500000029,50.27449399900007],[1.55583,50.361316500000044],[1.641539500000022,50.35214999600004],[1.555429500000059,50.399936498000045],[1.560707,50.69967249800004],[1.606003,50.791911999000035],[1.580953,50.869552499000065],[2.067705,51.00649999600006],[2.546011,51.08938199700003],[2.632617,50.94594549900006],[2.607036,50.91268949800008],[2.63495,50.81274399900008]]],[[[-1.195956,45.828918498000064],[-1.225263,45.804081],[-1.387045499999942,45.95379250000008],[-1.413434499999937,46.04686349800005],[-1.245950000999926,45.98994049800007],[-1.195956,45.828918498000064]]],[[[-52.94350749999995,5.45198750000003],[-52.364701,4.920734500000037],[-52.300845,4.950978500000076],[-52.145722,4.790893],[-51.813244,4.634343500000057],[-51.629835499999956,4.189126499000054],[-51.65604,4.05388750000003],[-51.77759949999995,3.974536],[-51.798616,3.886936001000038],[-51.972580499999935,3.705449],[-52.23357700099996,3.240668],[-52.292488,3.226403001000051],[-52.348031499999934,3.136390999000071],[-52.32591949999994,3.077791],[-52.55521049999999,2.640293],[-52.52731749999998,2.58393],[-52.565187,2.516254],[-52.660556,2.373956001000067],[-52.84180649999996,2.291378],[-52.904169,2.190938],[-52.980611,2.167639499000074],[-53.133876500999975,2.225365001000057],[-53.28104949999994,2.18784800100002],[-53.226680499999986,2.263608],[-53.34006549999998,2.351948500000049],[-53.462361,2.252782001000071],[-53.534661,2.251549001000058],[-53.723031,2.31207],[-53.769087499999955,2.375374501000067],[-53.94430649999998,2.215348500000061],[-54.066706,2.189358500000026],[-54.106466499999954,2.121296500000028],[-54.152372499999956,2.119626501000027],[-54.19038949999998,2.171099],[-54.270671499999935,2.144490500000074],[-54.360693,2.174917500000049],[-54.362251500999946,2.211756500000035],[-54.460852,2.207485001000066],[-54.54886749999997,2.270018],[-54.513794,2.279992],[-54.53909649999997,2.324723],[-54.58785949999998,2.322023501000047],[-54.590468,2.351944001000049],[-54.508611,2.335833500000035],[-54.47613949999993,2.435383500000057],[-54.41146,2.445936500000073],[-54.208187,2.777737500000057],[-54.172856,2.93860550100004],[-54.189067499999965,3.206341500000065],[-54.069237499999986,3.297167001000048],[-53.979807,3.606575],[-54.082868,3.674424999000053],[-54.121917500999984,3.792688501000043],[-54.197729,3.805496500000061],[-54.191877,3.847389001000067],[-54.35672649999998,4.047393501000045],[-54.323058,4.144421],[-54.39576249999993,4.202166501000022],[-54.386382,4.352911500000062],[-54.441826499999934,4.369863],[-54.415114,4.60441],[-54.468998,4.890381],[-54.415525,5.084697],[-54.018837,5.521456500000056],[-54.00809450099996,5.623116500000037],[-53.96836549999995,5.745176500000071],[-53.79925949999995,5.723899501000062],[-53.469359,5.56524550000006],[-53.261408,5.548479500000042],[-52.94350749999995,5.45198750000003]]],[[[-61.003888,14.808371500000021],[-60.95242149999996,14.748496500000044],[-60.872843,14.772254498000052],[-60.946177499999976,14.72632],[-60.810356499999955,14.480542],[-60.85767549999997,14.398175],[-60.893263,14.420223500000077],[-60.86607649999996,14.470458],[-61.081969,14.470402499000045],[-61.087707,14.534112500000049],[-60.98930949999993,14.548964500000068],[-61.018952,14.611703500000033],[-61.090783499999986,14.60018],[-61.150699,14.648276],[-61.229269,14.82211249900007],[-61.180351,14.87490199900003],[-61.003888,14.808371500000021]]],[[[-61.19499800099999,15.930456500000048],[-61.242508499999985,15.874458999000069],[-61.33183749999995,15.900605],[-61.291233000999966,16.003174999000066],[-61.19499800099999,15.930456500000048]]],[[[-61.25487149999998,16.25613899900003],[-61.465641,16.196540499000037],[-61.52729849999997,16.219122998000046],[-61.55065349999995,16.286972499000058],[-61.50920149999996,16.38560099700004],[-61.54121049999998,16.438072999000042],[-61.46405749999997,16.513064498000062],[-61.375652499999944,16.340646999000057],[-61.25848049999996,16.30302449900006],[-61.25487149999998,16.25613899900003]]],[[[-61.67261249999996,16.324603499000034],[-61.611944499999936,16.267183999000054],[-61.555114499999945,16.284743999000057],[-61.59044249999994,16.19234399900006],[-61.555346499999985,16.053240498000036],[-61.69680049999994,15.946588999000028],[-61.769683,16.05896149700004],[-61.80593,16.257474497000032],[-61.764951,16.353860999000062],[-61.67261249999996,16.324603499000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR104","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Essonne","NUTS_NAME":"Essonne"},"geometry":{"type":"Polygon","coordinates":[[[2.32072050000005,48.74875799700004],[2.571653500000025,48.69201699800004],[2.502907500000049,48.54600499800006],[2.50288050000006,48.39291299800004],[2.402663,48.32071749900007],[1.99409,48.28658399900007],[1.922146500000053,48.457599498000036],[1.966741,48.533469998000044],[1.93789,48.56292649800008],[2.01757,48.557593997000026],[2.011351,48.659910496000066],[2.226554,48.77610199800006],[2.32072050000005,48.74875799700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR105","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Hauts-de-Seine","NUTS_NAME":"Hauts-de-Seine"},"geometry":{"type":"Polygon","coordinates":[[[2.319887,48.90046],[2.224219,48.85351699900008],[2.331903500000067,48.81701349800005],[2.32072050000005,48.74875799700004],[2.226554,48.77610199800006],[2.145867,48.83689499700006],[2.200591,48.908679498000026],[2.290969500000074,48.95096599900006],[2.335568,48.940679997000075],[2.319887,48.90046]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR106","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Seine-Saint-Denis","NUTS_NAME":"Seine-Saint-Denis"},"geometry":{"type":"Polygon","coordinates":[[[2.592273500000033,48.80743749800007],[2.416345,48.849239],[2.319887,48.90046],[2.335568,48.940679997000075],[2.290969500000074,48.95096599900006],[2.553056499000036,49.00981599700003],[2.6026,48.92935649700007],[2.559406500000023,48.885336496000036],[2.592273500000033,48.80743749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR107","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Val-de-Marne","NUTS_NAME":"Val-de-Marne"},"geometry":{"type":"Polygon","coordinates":[[[2.592273500000033,48.80743749800007],[2.571653500000025,48.69201699800004],[2.32072050000005,48.74875799700004],[2.331903500000067,48.81701349800005],[2.469741500000055,48.835805],[2.416345,48.849239],[2.592273500000033,48.80743749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR108","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Val-d'Oise","NUTS_NAME":"Val-d'Oise"},"geometry":{"type":"Polygon","coordinates":[[[2.590528500000062,49.079653998000026],[2.553056499000036,49.00981599700003],[2.290969500000074,48.95096599900006],[2.200591,48.908679498000026],[2.110076500000048,49.008717998000066],[1.608799,49.077893999000025],[1.704359,49.23219699600003],[1.882564500000058,49.16249099800007],[2.321773,49.18443],[2.590528500000062,49.079653998000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR2","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"BASSIN PARISIEN","NUTS_NAME":"BASSIN PARISIEN"},"geometry":{"type":"Polygon","coordinates":[[[2.495685,50.194559998000045],[2.403272,50.16480299800003],[2.375452500000051,50.10925499900003],[2.418616500000041,50.08765550000004],[2.514943500000072,50.14081499900004],[2.776931499000057,50.115931497000076],[2.763325,50.03833149800005],[3.090252,50.05374049800008],[3.172704500000066,50.01199649700004],[3.745749,50.053570498000056],[4.140853,49.978759996000065],[4.233133,49.95782849700004],[4.432494,49.94161599600005],[4.694556,49.99650949900007],[4.680489,50.06813850000003],[4.796697,50.14867799800004],[4.896794,50.13742049800004],[4.793194,49.981999],[4.889914,49.90920999800005],[4.851578,49.793255],[4.969431,49.80182599800003],[5.153738500000031,49.717925999000045],[5.393511,49.61711099900003],[5.275101,49.54189549700004],[5.107566500000075,49.584689498000046],[5.059498500000075,49.50056649800007],[5.101156,49.37815049800008],[4.950990500000046,49.23686749600006],[4.993376,49.20962349700005],[4.939760500000034,49.18507099800007],[5.03953,48.97408949700008],[4.918824,48.905575498000076],[4.888582,48.80061349600004],[5.009218500000031,48.74138899900004],[4.98843050000005,48.684422],[4.999322,48.61483449900004],[5.470055,48.42092649800003],[5.409781500000065,48.39261349800006],[5.416975500000035,48.34269549600003],[5.526527,48.34685349700004],[5.730055,48.19831849700006],[5.629007,48.08441199600003],[5.884723500000064,47.92604649800006],[5.681527,47.80911149700006],[5.691057500000056,47.684624996000025],[5.419852,47.677725999000074],[5.37408,47.60453799800007],[5.477542,47.60871849800003],[5.486405,47.52696849600005],[5.375406999000063,47.46016749800003],[5.496923,47.388544996000064],[5.472388,47.318409],[5.518537500000036,47.30418399800004],[5.438506,47.14152950000005],[5.255232500000034,46.97988749800004],[5.328820500000063,46.887597999000036],[5.465138500000023,46.840725500000076],[5.32739250000003,46.815101998000046],[5.440604,46.63791199900004],[5.362118500000065,46.57726649800003],[5.415741,46.47314249800007],[5.310563499000068,46.446769998000036],[5.173731,46.51270099800007],[4.953853999000046,46.51342299800007],[4.780208500000072,46.176675998000064],[4.727254500000072,46.17911299900004],[4.69344,46.30232599800007],[4.405813500000022,46.29605799600006],[4.388074500000073,46.219790498000066],[4.282425500000045,46.15681299800008],[3.988531,46.16985450000004],[3.890131,46.21448699800004],[3.899538499000073,46.27590799800004],[3.984741,46.329729996000026],[3.998868500000071,46.46486699800005],[3.74101150000007,46.56021499600007],[3.629422500000032,46.74945649700004],[3.455304500000068,46.65239349800004],[3.43421,46.71222249700003],[3.165739,46.69172199800005],[3.032063,46.794909500000074],[2.71502,46.744980998000074],[2.566998,46.652483],[2.614960500000052,46.55327649900005],[2.351579,46.51213949600003],[2.281043500000067,46.42040349600006],[2.167784500000039,46.42406899800005],[1.747591500000055,46.45001799600004],[1.72784,46.38895750000006],[1.52535,46.42664899600004],[1.415185500000064,46.347215],[1.356024500000046,46.40011949800004],[1.177279,46.383947997000064],[1.213061,46.43307699700006],[1.149148500000024,46.502205496000045],[0.915865500000052,46.59662799800003],[0.927891,46.69264899800004],[0.867469,46.74821649800003],[0.704727499000057,46.902806998000074],[0.692567,46.97430449800004],[0.567741,47.00628399800007],[0.599612500000035,46.95628899800005],[0.32483650000006,46.93065199800003],[0.305386500000054,47.052699500000074],[0.175696,47.06072749700007],[0.05383,47.163733499000045],[0.230000500000074,47.608397498000045],[0.378485,47.568527998000036],[0.382790500000056,47.64309749800003],[0.614432500000021,47.69421549600003],[0.580504500000075,47.71282549800003],[0.848432500000058,47.942154498000036],[0.793996500000048,48.06944],[0.841217499000038,48.10305950000003],[0.91665050000006,48.13893649800008],[0.797658499000022,48.194455],[0.395541,48.32046199800004],[0.35289,48.45968799900004],[-0.054527,48.38200449900006],[-0.051349,48.452927],[-0.148717499999975,48.458068997000055],[-0.144602499999962,48.527754498000036],[-0.234124,48.561847500000056],[-0.654219,48.44454249900008],[-0.86036,48.501458498000034],[-1.070164499999976,48.50849199900006],[-1.249492500999963,48.54369199800004],[-1.42794,48.461915],[-1.571089499999971,48.626441999000065],[-1.391047,48.644653496000046],[-1.572866499999975,48.74789450000003],[-1.556390499999964,49.02472699800006],[-1.504676,49.02013399800006],[-1.600102499999934,49.04160299700004],[-1.605357,49.205581497000026],[-1.567265,49.22087849900004],[-1.625514,49.21006399800007],[-1.809161499999959,49.372093],[-1.868679499999928,49.51237099600007],[-1.855007499999942,49.64282250000008],[-1.947120499999926,49.674759],[-1.942384,49.725959998000064],[-1.630755499999964,49.65202349600003],[-1.26649249999997,49.69559849600006],[-1.228839499999935,49.60570549800008],[-1.307770499999947,49.54571899900003],[-1.11962,49.355568],[-0.961539499999958,49.396434998000075],[-0.225695,49.281821999000044],[0.297224500000027,49.429863],[0.338978500000053,49.44092549900006],[0.065609,49.51257699800004],[0.192298,49.70696249800005],[0.562341500000059,49.84552749800008],[1.07273,49.927711497000075],[1.379698,50.06500999700006],[1.545791500000064,50.214481498000055],[1.683794500000033,50.18302549800006],[1.539681500000029,50.27449399900007],[1.55583,50.361316500000044],[1.641539500000022,50.35214999600004],[1.923473,50.320425999000065],[2.085712500000056,50.20100999600004],[2.495685,50.194559998000045]],[[1.704359,49.23219699600003],[1.608799,49.077893999000025],[1.447285,49.05350949800004],[1.470892,48.974814500000036],[1.518385500000022,48.978357997000046],[1.501526500000068,48.941051997000045],[1.626540500000033,48.74804799900005],[1.602704,48.66309299800008],[1.757405500000061,48.57495449900006],[1.801452500000039,48.46604099600006],[1.922146500000053,48.457599498000036],[1.99409,48.28658399900007],[2.402663,48.32071749900007],[2.522821,48.197726497000076],[2.476016500000071,48.12963649900007],[2.936316,48.163391499000056],[3.031462,48.248878998000066],[3.049454,48.36002749800008],[3.414788999000052,48.39026849800007],[3.405396500000052,48.52801499800006],[3.557419164000066,48.61671378700004],[3.451096,48.634649997000054],[3.444068500000071,48.736630999000056],[3.395836,48.75924699700005],[3.485183500000062,48.85191049800005],[3.265159500000038,48.939061],[3.165230500000064,49.099653501000034],[3.07188,49.11755350000004],[2.590528500000062,49.079653998000026],[2.321773,49.18443],[1.882564500000058,49.16249099800007],[1.704359,49.23219699600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR522","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Finistère","NUTS_NAME":"Finistère"},"geometry":{"type":"Polygon","coordinates":[[[-3.950495,48.65286999700004],[-3.859493,48.61433799800005],[-3.831078499999933,48.712436498000045],[-3.643659499999956,48.69629649800004],[-3.659150499999953,48.65920999800005],[-3.539283,48.53825099800008],[-3.604373499999951,48.49620199600008],[-3.607688,48.38832149900003],[-3.554302,48.378858500000035],[-3.52486,48.28117099600007],[-3.564844499999936,48.18574749700008],[-3.734452499999975,48.112607499000035],[-3.641038499999979,47.985788497000044],[-3.39199150099995,47.95727999800005],[-3.457660499999974,47.83663249800003],[-3.526130499999965,47.842902999000046],[-3.528605312999957,47.77829169600005],[-3.850509,47.79721049800003],[-3.958664499999941,47.896976496000036],[-4.099355499999945,47.86249150000003],[-4.144733499999973,47.909335998000074],[-4.115504,47.976466998000035],[-4.151007499999935,47.910702],[-4.113312,47.86203749800006],[-4.176410499999974,47.80505749800005],[-4.374185,47.79797],[-4.347687,47.849003],[-4.42255849999998,47.962432998000054],[-4.734697499999925,48.038242497000056],[-4.269574,48.13277050000005],[-4.303489,48.195732],[-4.465324499999952,48.238849499000025],[-4.557571,48.16983049800007],[-4.545083,48.25028599800004],[-4.625327500999958,48.259071498000026],[-4.554775,48.338588498000036],[-4.535044,48.28407649800005],[-4.225793499999952,48.28749099600003],[-4.332875499999943,48.31668099900003],[-4.261421499999926,48.360431498000025],[-4.45806,48.32748799800004],[-4.273602500999971,48.44343549900003],[-4.772754,48.32923499900005],[-4.769921499999953,48.52113699800003],[-4.349134,48.67647549900005],[-4.281066499999952,48.642818498000054],[-3.96533,48.72175999800004],[-3.950495,48.65286999700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR523","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Ille-et-Vilaine","NUTS_NAME":"Ille-et-Vilaine"},"geometry":{"type":"Polygon","coordinates":[[[-1.872124,48.645713998000076],[-1.571089499999971,48.626441999000065],[-1.42794,48.461915],[-1.249492500999963,48.54369199800004],[-1.070164499999976,48.50849199900006],[-1.045019499999967,48.327728498000056],[-1.101965,48.26250599800005],[-1.015823500999943,48.00328599800008],[-1.156485,47.96429099800008],[-1.238248,47.80999249600006],[-1.245885,47.77671749800004],[-1.424917,47.83284149900004],[-1.657515,47.709435],[-1.973398,47.69415349600007],[-2.097034,47.63135649800006],[-2.131471,47.72224049700003],[-2.036051499999928,47.83346050000006],[-2.10945049999998,47.848835499000074],[-2.078953499999955,47.919485999000074],[-2.144933499999979,47.98403399600005],[-2.28754349999997,47.99149949800005],[-2.168483499999979,48.07620550000007],[-2.287274,48.133744],[-2.186751499999957,48.24778649800004],[-1.958049499999959,48.293119500000046],[-1.908992,48.48155399800004],[-1.982181074999971,48.554643027000054],[-2.028575,48.65031049800007],[-1.850131,48.69939399900005],[-1.872124,48.645713998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR524","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Morbihan","NUTS_NAME":"Morbihan"},"geometry":{"type":"Polygon","coordinates":[[[-3.011761499999977,48.17214850000005],[-2.660977000999935,48.13062149800004],[-2.655765499999973,48.03254099800006],[-2.548678499999937,48.07647699600005],[-2.53434,48.15693949900003],[-2.287274,48.133744],[-2.168483499999979,48.07620550000007],[-2.28754349999997,47.99149949800005],[-2.144933499999979,47.98403399600005],[-2.078953499999955,47.919485999000074],[-2.10945049999998,47.848835499000074],[-2.036051499999928,47.83346050000006],[-2.131471,47.72224049700003],[-2.097034,47.63135649800006],[-2.098850499999969,47.53345099900008],[-2.458493,47.44811999800004],[-2.501003,47.49271749900004],[-2.420451499999956,47.49520099800003],[-2.849214,47.49888249600008],[-2.919619,47.547725498000034],[-2.730782,47.54229749900003],[-2.721059,47.60310749700005],[-2.898194,47.58569699700007],[-2.98203,47.65580749800006],[-2.971034,47.55858999600008],[-3.128996499999971,47.59746149600005],[-3.089519499999938,47.47268299900003],[-3.130722499999933,47.47313299800004],[-3.158280499999933,47.60671599700004],[-3.338828,47.695411498000055],[-3.300046499999951,47.774661998000056],[-3.453312500999971,47.69553749600004],[-3.528605312999957,47.77829169600005],[-3.526130499999965,47.842902999000046],[-3.457660499999974,47.83663249800003],[-3.39199150099995,47.95727999800005],[-3.641038499999979,47.985788497000044],[-3.734452499999975,48.112607499000035],[-3.564844499999936,48.18574749700008],[-3.011761499999977,48.17214850000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR53","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Poitou-Charentes","NUTS_NAME":"Poitou-Charentes"},"geometry":{"type":"MultiPolygon","coordinates":[[[[0.05383,47.163733499000045],[0.175696,47.06072749700007],[0.305386500000054,47.052699500000074],[0.32483650000006,46.93065199800003],[0.599612500000035,46.95628899800005],[0.567741,47.00628399800007],[0.692567,46.97430449800004],[0.704727499000057,46.902806998000074],[0.867469,46.74821649800003],[0.927891,46.69264899800004],[0.915865500000052,46.59662799800003],[1.149148500000024,46.502205496000045],[1.213061,46.43307699700006],[1.177279,46.383947997000064],[0.810388500000045,46.22950949800003],[0.823432500000024,46.128584498000066],[0.817911,46.04788149600006],[0.942597,45.957437998000046],[0.835689500000058,45.92272399800004],[0.783778500000039,45.79347549700003],[0.710518500000035,45.802201498000045],[0.629741,45.71456999600008],[0.486204500000042,45.538295498000025],[0.302616999000065,45.45888249600006],[0.247881500000062,45.36927249800004],[0.266603,45.29774599900003],[0.004336,45.19162799900005],[-0.040197499999977,45.10237999800006],[-0.258733,45.114627496000026],[-0.379539499999964,45.155939998000065],[-0.425493,45.27863649600005],[-0.708231,45.32748049800006],[-0.819719499999962,45.49202749800003],[-1.230697499999962,45.68057249700007],[-1.241852,45.785945998000045],[-1.13209,45.80492799900003],[-1.155564,45.841975999000056],[-1.053074,46.00382999900006],[-1.123773,46.11335349800004],[-1.242043499999966,46.15744399700003],[-1.11135,46.261157998000044],[-1.129406,46.310271998000076],[-0.936756499999944,46.36638649900004],[-0.963101,46.32262099900004],[-0.750471499999946,46.304254498000034],[-0.537795000999949,46.38646399800007],[-0.610179499999958,46.41373299800006],[-0.613968,46.62012449900004],[-0.656161499999939,46.70030299800004],[-0.891964,46.975820498000076],[-0.595493499999975,46.997910500000046],[-0.493149,47.08338599700005],[-0.102116,47.06479999700008],[0.05383,47.163733499000045]]],[[[-1.195956,45.828918498000064],[-1.225263,45.804081],[-1.387045499999942,45.95379250000008],[-1.413434499999937,46.04686349800005],[-1.245950000999926,45.98994049800007],[-1.195956,45.828918498000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR531","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Charente","NUTS_NAME":"Charente"},"geometry":{"type":"Polygon","coordinates":[[[0.823432500000024,46.128584498000066],[0.817911,46.04788149600006],[0.942597,45.957437998000046],[0.835689500000058,45.92272399800004],[0.783778500000039,45.79347549700003],[0.710518500000035,45.802201498000045],[0.629741,45.71456999600008],[0.486204500000042,45.538295498000025],[0.302616999000065,45.45888249600006],[0.247881500000062,45.36927249800004],[0.266603,45.29774599900003],[0.004336,45.19162799900005],[-0.312193,45.37694599900004],[-0.253369,45.41766249900007],[-0.246194499999945,45.49667899900004],[-0.431920499999933,45.62446999800005],[-0.418701,45.72701450000005],[-0.463152499999978,45.75184599800008],[-0.156055499999979,45.77951549800008],[-0.102938499999937,45.96966149800005],[-0.038156,45.99242349800005],[-0.039498499999979,46.05310749800003],[0.197354499000028,46.09555299800007],[0.413283,46.04908949800006],[0.469146,46.06154749600006],[0.443261500000062,46.10156949800006],[0.492229,46.136082499000054],[0.595675,46.072659497000075],[0.823432500000024,46.128584498000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR532","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Charente-Maritime","NUTS_NAME":"Charente-Maritime"},"geometry":{"type":"MultiPolygon","coordinates":[[[[0.004336,45.19162799900005],[-0.040197499999977,45.10237999800006],[-0.258733,45.114627496000026],[-0.379539499999964,45.155939998000065],[-0.425493,45.27863649600005],[-0.708231,45.32748049800006],[-0.819719499999962,45.49202749800003],[-1.230697499999962,45.68057249700007],[-1.241852,45.785945998000045],[-1.13209,45.80492799900003],[-1.155564,45.841975999000056],[-1.053074,46.00382999900006],[-1.123773,46.11335349800004],[-1.242043499999966,46.15744399700003],[-1.11135,46.261157998000044],[-1.129406,46.310271998000076],[-0.936756499999944,46.36638649900004],[-0.963101,46.32262099900004],[-0.750471499999946,46.304254498000034],[-0.751466,46.24507099900006],[-0.616334,46.13658449600007],[-0.303649499999949,46.07757250000003],[-0.102938499999937,45.96966149800005],[-0.156055499999979,45.77951549800008],[-0.463152499999978,45.75184599800008],[-0.418701,45.72701450000005],[-0.431920499999933,45.62446999800005],[-0.246194499999945,45.49667899900004],[-0.253369,45.41766249900007],[-0.312193,45.37694599900004],[0.004336,45.19162799900005]]],[[[-1.195956,45.828918498000064],[-1.225263,45.804081],[-1.387045499999942,45.95379250000008],[-1.413434499999937,46.04686349800005],[-1.245950000999926,45.98994049800007],[-1.195956,45.828918498000064]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR533","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Deux-Sèvres","NUTS_NAME":"Deux-Sèvres"},"geometry":{"type":"Polygon","coordinates":[[[-0.102116,47.06479999700008],[-0.033119499999941,46.980244998000046],[-0.035641,46.874931497000034],[0.033762500000023,46.85407199800005],[-0.049093,46.82303399800003],[0.036505,46.728515499000025],[-0.031846499999972,46.661693996000054],[0.022895500000061,46.595267],[-0.043262,46.47243599900003],[0.015100500000074,46.32625250000007],[0.177366500000062,46.32810950000004],[0.107698500000026,46.18599049900007],[0.197354499000028,46.09555299800007],[-0.039498499999979,46.05310749800003],[-0.038156,45.99242349800005],[-0.102938499999937,45.96966149800005],[-0.303649499999949,46.07757250000003],[-0.616334,46.13658449600007],[-0.751466,46.24507099900006],[-0.750471499999946,46.304254498000034],[-0.537795000999949,46.38646399800007],[-0.610179499999958,46.41373299800006],[-0.613968,46.62012449900004],[-0.656161499999939,46.70030299800004],[-0.891964,46.975820498000076],[-0.595493499999975,46.997910500000046],[-0.493149,47.08338599700005],[-0.102116,47.06479999700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR534","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Vienne","NUTS_NAME":"Vienne"},"geometry":{"type":"Polygon","coordinates":[[[0.867469,46.74821649800003],[0.927891,46.69264899800004],[0.915865500000052,46.59662799800003],[1.149148500000024,46.502205496000045],[1.213061,46.43307699700006],[1.177279,46.383947997000064],[0.810388500000045,46.22950949800003],[0.823432500000024,46.128584498000066],[0.595675,46.072659497000075],[0.492229,46.136082499000054],[0.443261500000062,46.10156949800006],[0.469146,46.06154749600006],[0.413283,46.04908949800006],[0.197354499000028,46.09555299800007],[0.107698500000026,46.18599049900007],[0.177366500000062,46.32810950000004],[0.015100500000074,46.32625250000007],[-0.043262,46.47243599900003],[0.022895500000061,46.595267],[-0.031846499999972,46.661693996000054],[0.036505,46.728515499000025],[-0.049093,46.82303399800003],[0.033762500000023,46.85407199800005],[-0.035641,46.874931497000034],[-0.033119499999941,46.980244998000046],[-0.102116,47.06479999700008],[0.05383,47.163733499000045],[0.175696,47.06072749700007],[0.305386500000054,47.052699500000074],[0.32483650000006,46.93065199800003],[0.599612500000035,46.95628899800005],[0.567741,47.00628399800007],[0.692567,46.97430449800004],[0.704727499000057,46.902806998000074],[0.867469,46.74821649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR6","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"SUD-OUEST","NUTS_NAME":"SUD-OUEST"},"geometry":{"type":"Polygon","coordinates":[[[2.281043500000067,46.42040349600006],[2.318604500000049,46.33440549900007],[2.480760500000031,46.28100849600003],[2.565372500000024,46.14303599900006],[2.550394500000039,46.08606949800003],[2.60902149900005,45.96664399800005],[2.388014,45.827372996000065],[2.492129500000033,45.73766999600008],[2.528279,45.658533498000054],[2.461168,45.59614650000003],[2.515881,45.55387949800007],[2.50841250000002,45.47850149800007],[2.487533,45.41816699900005],[2.528832,45.392374999000026],[2.369421500000044,45.38727149700003],[2.191158,45.22305049800008],[2.230417500000044,45.16191649800004],[2.095157,45.05603349800003],[2.133296500000029,44.98574199900003],[2.062908,44.976504498000054],[2.167976500000066,44.81050199800006],[2.149042500000064,44.72213849800005],[2.207473,44.61552899800006],[2.285289500000033,44.666758498000036],[2.478947500000061,44.64800999800008],[2.716769500000055,44.92882799600005],[2.933775500000024,44.780645496000034],[2.981675500000051,44.64467299800003],[3.14346050000006,44.43548199800006],[3.120173500000021,44.26183849600005],[3.373648,44.17075949800005],[3.263114,44.09242549800007],[3.448355,44.01910349800005],[3.358362,43.91382949800004],[3.262606500000061,43.89100250000007],[3.207685999000034,43.812486998000054],[3.064905,43.83556550000003],[3.060680500000046,43.692804498000044],[2.935457,43.694664998000064],[2.75452,43.61414049900003],[2.645487500000058,43.66066949800006],[2.615601,43.56663599800004],[2.67602450000004,43.51345550000008],[2.565782500000068,43.42295799900006],[2.265415,43.45291349800004],[2.215026,43.38261099600004],[2.029134,43.436895496000034],[1.856448500000056,43.443452999000044],[1.80295,43.392548496000074],[1.813485,43.34149799700003],[1.722903500000029,43.341531],[1.688423500000056,43.27355449600003],[1.713483,43.18561949800005],[1.949341,43.12097299900006],[1.985759500000029,43.02995649700006],[1.984954999000024,42.96726849700008],[1.933727500000032,42.93914049800003],[1.986922,42.87100699700005],[1.859382,42.82080499800003],[1.950635500000033,42.73686649600006],[2.084955999000044,42.75165799800004],[2.166049,42.663917498000046],[1.786098500000037,42.573657999000034],[1.574720500000069,42.64779299800006],[1.442566,42.603668],[1.357304,42.71942199800003],[1.17077,42.70814149800003],[1.078708,42.788341496000044],[0.858215,42.82574099800007],[0.66215950000003,42.84124349700005],[0.660127,42.69095249800006],[0.477755500000058,42.70001999600004],[0.360133500000074,42.724353998000026],[0.295051500000056,42.673358998000026],[0.175701,42.73704149800005],[-0.055795,42.69225699800006],[-0.160069499999963,42.79801949800003],[-0.313342,42.84936499600008],[-0.551047,42.777637499000036],[-0.724501,42.92015849900008],[-0.751951499999961,42.96712099900003],[-0.945180499999935,42.95343799800003],[-1.309055499999943,43.07115950000008],[-1.270122,43.118247996000036],[-1.3322,43.10771199800007],[-1.35373150099997,43.02798099800003],[-1.44129049999998,43.04634099800006],[-1.473489499999971,43.08662049800006],[-1.414168,43.12828849700003],[-1.384407500999941,43.25365449800006],[-1.534212,43.29453649800007],[-1.608788499999946,43.251976],[-1.625259,43.30670549800004],[-1.728903,43.29608899600004],[-1.785978,43.35047899600005],[-1.628819,43.41350149800007],[-1.524870499999963,43.52970099800007],[-1.448207499999967,43.64227299800007],[-1.253891,44.46760549800007],[-1.193711499999949,44.65898899700005],[-1.004679,44.64818549800003],[-1.161932499999978,44.774902498000074],[-1.261136499999964,44.62758249600006],[-1.156919,45.472530498000026],[-1.064105499999926,45.554893496000034],[-1.065570499999978,45.51051699800007],[-0.809971499999961,45.35167699800007],[-0.746965499999931,45.22681799600008],[-0.690119,45.23636249800006],[-0.708231,45.32748049800006],[-0.425493,45.27863649600005],[-0.379539499999964,45.155939998000065],[-0.258733,45.114627496000026],[-0.040197499999977,45.10237999800006],[0.004336,45.19162799900005],[0.266603,45.29774599900003],[0.247881500000062,45.36927249800004],[0.302616999000065,45.45888249600006],[0.486204500000042,45.538295498000025],[0.629741,45.71456999600008],[0.710518500000035,45.802201498000045],[0.783778500000039,45.79347549700003],[0.835689500000058,45.92272399800004],[0.942597,45.957437998000046],[0.817911,46.04788149600006],[0.823432500000024,46.128584498000066],[0.810388500000045,46.22950949800003],[1.177279,46.383947997000064],[1.356024500000046,46.40011949800004],[1.415185500000064,46.347215],[1.52535,46.42664899600004],[1.72784,46.38895750000006],[1.747591500000055,46.45001799600004],[2.167784500000039,46.42406899800005],[2.281043500000067,46.42040349600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR61","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Aquitaine","NUTS_NAME":"Aquitaine"},"geometry":{"type":"Polygon","coordinates":[[[0.752317,45.61765599700004],[0.811495,45.575871498000026],[1.023243,45.60996049800008],[1.167285500000048,45.52482299800005],[1.119942,45.47469699800007],[1.253154500000051,45.44422],[1.325074500000028,45.37993749900005],[1.235802,45.32204199800003],[1.227747500000021,45.20576049700003],[1.413087499000028,45.124907499000074],[1.378250499000046,45.10608949900006],[1.44826,45.01931399800003],[1.409125500000073,45.006723],[1.442178,44.87834299600007],[1.075142,44.57732549800005],[0.981513500000062,44.54444099800003],[1.064081500000043,44.37850849800003],[0.896382,44.38183649900003],[0.869440500000053,44.309249496000064],[0.950791,44.27506550000004],[0.853023,44.18391149800004],[0.89206850000005,44.13733099900003],[0.787737500000048,44.14276099800003],[0.741885,44.06519899800003],[0.076043500000026,43.98313849800007],[0.059019,43.896810498000036],[-0.036165,43.98373599800004],[-0.240259,43.89794799800006],[-0.185553,43.863012498000046],[-0.227139499999964,43.80827099700008],[-0.194138,43.737014997000074],[-0.282116,43.643073496000056],[-0.242833,43.584978996000075],[-0.096788,43.58240499900006],[-0.018912,43.47005599800008],[-0.065757499999961,43.46348349600004],[-0.067268,43.41041249800003],[0.009650499000031,43.42219699800006],[-0.046122499999967,43.300858498000025],[-0.017172,43.26995099900006],[-0.287666499999943,43.00555349800004],[-0.313342,42.84936499600008],[-0.551047,42.777637499000036],[-0.724501,42.92015849900008],[-0.751951499999961,42.96712099900003],[-0.945180499999935,42.95343799800003],[-1.309055499999943,43.07115950000008],[-1.270122,43.118247996000036],[-1.3322,43.10771199800007],[-1.35373150099997,43.02798099800003],[-1.44129049999998,43.04634099800006],[-1.473489499999971,43.08662049800006],[-1.414168,43.12828849700003],[-1.384407500999941,43.25365449800006],[-1.534212,43.29453649800007],[-1.608788499999946,43.251976],[-1.625259,43.30670549800004],[-1.728903,43.29608899600004],[-1.785978,43.35047899600005],[-1.628819,43.41350149800007],[-1.524870499999963,43.52970099800007],[-1.448207499999967,43.64227299800007],[-1.253891,44.46760549800007],[-1.193711499999949,44.65898899700005],[-1.004679,44.64818549800003],[-1.161932499999978,44.774902498000074],[-1.261136499999964,44.62758249600006],[-1.156919,45.472530498000026],[-1.064105499999926,45.554893496000034],[-1.065570499999978,45.51051699800007],[-0.809971499999961,45.35167699800007],[-0.746965499999931,45.22681799600008],[-0.690119,45.23636249800006],[-0.708231,45.32748049800006],[-0.425493,45.27863649600005],[-0.379539499999964,45.155939998000065],[-0.258733,45.114627496000026],[-0.040197499999977,45.10237999800006],[0.004336,45.19162799900005],[0.266603,45.29774599900003],[0.247881500000062,45.36927249800004],[0.302616999000065,45.45888249600006],[0.486204500000042,45.538295498000025],[0.629741,45.71456999600008],[0.775646,45.66774499700006],[0.752317,45.61765599700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR611","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Dordogne","NUTS_NAME":"Dordogne"},"geometry":{"type":"Polygon","coordinates":[[[1.253154500000051,45.44422],[1.325074500000028,45.37993749900005],[1.235802,45.32204199800003],[1.227747500000021,45.20576049700003],[1.413087499000028,45.124907499000074],[1.378250499000046,45.10608949900006],[1.44826,45.01931399800003],[1.409125500000073,45.006723],[1.442178,44.87834299600007],[1.075142,44.57732549800005],[0.979463,44.64388149600006],[0.833656500000075,44.59971699600004],[0.842298500000027,44.66622049700004],[0.797721500000023,44.70176799800004],[0.355528,44.65496399800003],[0.297326500000054,44.762287496000056],[0.263721,44.82641449800008],[0.315049500000043,44.84537049800008],[0.277076500000021,44.86747649900008],[0.192842,44.82080349700004],[-0.010517499999935,44.857897999000045],[0.073291500000039,45.07012449800004],[-0.040197499999977,45.10237999800006],[0.004336,45.19162799900005],[0.266603,45.29774599900003],[0.247881500000062,45.36927249800004],[0.302616999000065,45.45888249600006],[0.486204500000042,45.538295498000025],[0.629741,45.71456999600008],[0.775646,45.66774499700006],[0.752317,45.61765599700004],[0.811495,45.575871498000026],[1.023243,45.60996049800008],[1.167285500000048,45.52482299800005],[1.119942,45.47469699800007],[1.253154500000051,45.44422]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR612","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Gironde","NUTS_NAME":"Gironde"},"geometry":{"type":"Polygon","coordinates":[[[-0.708231,45.32748049800006],[-0.425493,45.27863649600005],[-0.379539499999964,45.155939998000065],[-0.258733,45.114627496000026],[-0.040197499999977,45.10237999800006],[0.073291500000039,45.07012449800004],[-0.010517499999935,44.857897999000045],[0.192842,44.82080349700004],[0.277076500000021,44.86747649900008],[0.315049500000043,44.84537049800008],[0.263721,44.82641449800008],[0.297326500000054,44.762287496000056],[0.105009,44.70755449600006],[0.182752500000049,44.66136299800007],[-0.015565,44.50478749600006],[0.023592500000063,44.377462999000045],[-0.078824499999939,44.353695996000056],[-0.032985499999938,44.271724998000025],[-0.14069,44.226407497000025],[-0.193590499999971,44.27175199800007],[-0.223447,44.20589449700003],[-0.389615,44.20948399800005],[-0.431844499999954,44.32257249600008],[-0.627805,44.442721498000026],[-1.029161,44.42269799800005],[-0.987734499999931,44.50830149800004],[-1.08516549999996,44.53219549700003],[-1.253891,44.46760549800007],[-1.193711499999949,44.65898899700005],[-1.004679,44.64818549800003],[-1.161932499999978,44.774902498000074],[-1.261136499999964,44.62758249600006],[-1.156919,45.472530498000026],[-1.064105499999926,45.554893496000034],[-1.065570499999978,45.51051699800007],[-0.809971499999961,45.35167699800007],[-0.746965499999931,45.22681799600008],[-0.690119,45.23636249800006],[-0.708231,45.32748049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR613","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Landes","NUTS_NAME":"Landes"},"geometry":{"type":"Polygon","coordinates":[[[-0.14069,44.226407497000025],[-0.129152,44.15240749800006],[0.135791500000039,44.124189],[0.060218,44.02487449600005],[0.076043500000026,43.98313849800007],[0.059019,43.896810498000036],[-0.036165,43.98373599800004],[-0.240259,43.89794799800006],[-0.185553,43.863012498000046],[-0.227139499999964,43.80827099700008],[-0.194138,43.737014997000074],[-0.282116,43.643073496000056],[-0.242833,43.584978996000075],[-0.449716499999965,43.54986699800003],[-0.449211,43.59630499700006],[-0.614372,43.53771350000005],[-0.770303,43.57916649800006],[-0.992383,43.50402249600006],[-1.524870499999963,43.52970099800007],[-1.448207499999967,43.64227299800007],[-1.253891,44.46760549800007],[-1.08516549999996,44.53219549700003],[-0.987734499999931,44.50830149800004],[-1.029161,44.42269799800005],[-0.627805,44.442721498000026],[-0.431844499999954,44.32257249600008],[-0.389615,44.20948399800005],[-0.223447,44.20589449700003],[-0.193590499999971,44.27175199800007],[-0.14069,44.226407497000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR614","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Lot-et-Garonne","NUTS_NAME":"Lot-et-Garonne"},"geometry":{"type":"Polygon","coordinates":[[[0.297326500000054,44.762287496000056],[0.355528,44.65496399800003],[0.797721500000023,44.70176799800004],[0.842298500000027,44.66622049700004],[0.833656500000075,44.59971699600004],[0.979463,44.64388149600006],[1.075142,44.57732549800005],[0.981513500000062,44.54444099800003],[1.064081500000043,44.37850849800003],[0.896382,44.38183649900003],[0.869440500000053,44.309249496000064],[0.950791,44.27506550000004],[0.853023,44.18391149800004],[0.89206850000005,44.13733099900003],[0.787737500000048,44.14276099800003],[0.741885,44.06519899800003],[0.076043500000026,43.98313849800007],[0.060218,44.02487449600005],[0.135791500000039,44.124189],[-0.129152,44.15240749800006],[-0.14069,44.226407497000025],[-0.032985499999938,44.271724998000025],[-0.078824499999939,44.353695996000056],[0.023592500000063,44.377462999000045],[-0.015565,44.50478749600006],[0.182752500000049,44.66136299800007],[0.105009,44.70755449600006],[0.297326500000054,44.762287496000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR615","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Pyrénées-Atlantiques","NUTS_NAME":"Pyrénées-Atlantiques"},"geometry":{"type":"Polygon","coordinates":[[[-0.449211,43.59630499700006],[-0.449716499999965,43.54986699800003],[-0.242833,43.584978996000075],[-0.096788,43.58240499900006],[-0.018912,43.47005599800008],[-0.065757499999961,43.46348349600004],[-0.067268,43.41041249800003],[0.009650499000031,43.42219699800006],[-0.046122499999967,43.300858498000025],[-0.017172,43.26995099900006],[-0.287666499999943,43.00555349800004],[-0.313342,42.84936499600008],[-0.551047,42.777637499000036],[-0.724501,42.92015849900008],[-0.751951499999961,42.96712099900003],[-0.945180499999935,42.95343799800003],[-1.309055499999943,43.07115950000008],[-1.270122,43.118247996000036],[-1.3322,43.10771199800007],[-1.35373150099997,43.02798099800003],[-1.44129049999998,43.04634099800006],[-1.473489499999971,43.08662049800006],[-1.414168,43.12828849700003],[-1.384407500999941,43.25365449800006],[-1.534212,43.29453649800007],[-1.608788499999946,43.251976],[-1.625259,43.30670549800004],[-1.728903,43.29608899600004],[-1.785978,43.35047899600005],[-1.628819,43.41350149800007],[-1.524870499999963,43.52970099800007],[-0.992383,43.50402249600006],[-0.770303,43.57916649800006],[-0.614372,43.53771350000005],[-0.449211,43.59630499700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR713","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Drôme","NUTS_NAME":"Drôme"},"geometry":{"type":"Polygon","coordinates":[[[5.80147,44.706777500000044],[5.830441,44.690548997000064],[5.790623500000038,44.653293],[5.641717500000027,44.65107749800006],[5.597252500000025,44.543273498000076],[5.664504,44.50189499700008],[5.458414,44.499388999000075],[5.418397500000026,44.42476849800005],[5.492943,44.337132498000074],[5.630529500000023,44.33331449900004],[5.676035999000021,44.19142849800005],[5.498786500000051,44.11571649700005],[5.355353,44.21456549800007],[5.154939,44.23049049600007],[5.149949500000048,44.30193449800004],[4.825547500000027,44.22856849800007],[4.804566,44.30389699700004],[4.650611,44.32980299600007],[4.692773500000044,44.558538498000075],[4.779066499000066,44.65469599800008],[4.761766500000022,44.77359449800008],[4.886594500000058,44.93665249600008],[4.803822500000024,45.12271399900004],[4.80049,45.29835699600005],[5.009649500000023,45.34222999800005],[5.201718500000027,45.217400498000075],[5.181984500000056,45.08454449900006],[5.385827,45.034621996000055],[5.494190500000059,45.07166599900006],[5.46416,44.79242499800006],[5.80147,44.706777500000044]],[[5.070316,44.37644749800006],[4.959273,44.41978299800007],[4.869529500000056,44.345077],[4.987017500000036,44.29271349900006],[5.070316,44.37644749800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR714","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Isère","NUTS_NAME":"Isère"},"geometry":{"type":"Polygon","coordinates":[[[5.623743,45.61326849900007],[5.739983500000051,45.43757799900004],[5.890229,45.40101549900004],[5.966172,45.49230799900005],[6.129954,45.43589799800003],[6.19476049900004,45.35224450000004],[6.127633999000068,45.23096399800005],[6.26057,45.12684399600005],[6.20392350000003,45.012471],[6.323686,44.99870849800004],[6.355365,44.85482049800004],[6.015539500000045,44.83527399800005],[5.80147,44.706777500000044],[5.46416,44.79242499800006],[5.494190500000059,45.07166599900006],[5.385827,45.034621996000055],[5.181984500000056,45.08454449900006],[5.201718500000027,45.217400498000075],[5.009649500000023,45.34222999800005],[4.80049,45.29835699600005],[4.75599950000003,45.36567499800003],[4.756931500000064,45.45570399700006],[4.872527500000047,45.532077498000035],[4.777129,45.58739550000007],[5.153992500000072,45.699719998000035],[5.059609,45.78207299800005],[5.101069,45.813378999000065],[5.223596500000042,45.76865999700004],[5.340918999000053,45.88050949600006],[5.404375,45.856872499000076],[5.623743,45.61326849900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR715","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Loire","NUTS_NAME":"Loire"},"geometry":{"type":"Polygon","coordinates":[[[4.388074500000073,46.219790498000066],[4.431914500000062,46.15603250000004],[4.322114500000055,46.129978496000035],[4.284753500000022,46.05323099800006],[4.286666,45.974495998000066],[4.346022,45.92993949800007],[4.323217999000065,45.903884996000045],[4.394473,45.86516549800007],[4.36592150000007,45.671155],[4.504776,45.576543498000035],[4.683512500000063,45.56541099800006],[4.655258,45.48607749700005],[4.756931500000064,45.45570399700006],[4.75599950000003,45.36567499800003],[4.614825,45.31153599600003],[4.606401,45.254056999000056],[4.483132,45.23644549800008],[4.349434500000029,45.27420949800006],[4.331653500000073,45.318478998000046],[4.368010500000025,45.33710749800008],[4.329783500000076,45.369281998000076],[3.89740850000004,45.35707999700003],[3.98468899900007,45.495232498000064],[3.777585999000053,45.69056249800008],[3.712625500000058,45.80128799900007],[3.754369500000053,45.88602799900008],[3.694015,45.930728],[3.832069500000046,45.99966349600004],[3.768207500000074,46.23881449600003],[3.899538499000073,46.27590799800004],[3.890131,46.21448699800004],[3.988531,46.16985450000004],[4.282425500000045,46.15681299800008],[4.388074500000073,46.219790498000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR716","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Rhône","NUTS_NAME":"Rhône"},"geometry":{"type":"Polygon","coordinates":[[[4.780208500000072,46.176675998000064],[4.731369,45.94132049900003],[4.872074,45.903455499000074],[4.923797500000035,45.803994998000064],[5.101069,45.813378999000065],[5.059609,45.78207299800005],[5.153992500000072,45.699719998000035],[4.777129,45.58739550000007],[4.872527500000047,45.532077498000035],[4.756931500000064,45.45570399700006],[4.655258,45.48607749700005],[4.683512500000063,45.56541099800006],[4.504776,45.576543498000035],[4.36592150000007,45.671155],[4.394473,45.86516549800007],[4.323217999000065,45.903884996000045],[4.346022,45.92993949800007],[4.286666,45.974495998000066],[4.284753500000022,46.05323099800006],[4.322114500000055,46.129978496000035],[4.431914500000062,46.15603250000004],[4.388074500000073,46.219790498000066],[4.405813500000022,46.29605799600006],[4.69344,46.30232599800007],[4.727254500000072,46.17911299900004],[4.780208500000072,46.176675998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR717","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Savoie","NUTS_NAME":"Savoie"},"geometry":{"type":"Polygon","coordinates":[[[6.802368500000057,45.77856199900003],[6.845962,45.690503997000064],[7.000765,45.64000749600007],[7.00001750000007,45.50466699700007],[7.104723,45.46845449800003],[7.185578500000076,45.40326749800005],[7.109843500000068,45.32694349600007],[7.125157,45.243994497000074],[6.893854500000032,45.13691449700008],[6.630051,45.109856499000045],[6.451804500000037,45.051720498000066],[6.26057,45.12684399600005],[6.127633999000068,45.23096399800005],[6.19476049900004,45.35224450000004],[6.129954,45.43589799800003],[5.966172,45.49230799900005],[5.890229,45.40101549900004],[5.739983500000051,45.43757799900004],[5.623743,45.61326849900007],[5.776236,45.72638399700003],[5.83123,45.93845749700006],[5.913946,45.80406049900006],[6.042249,45.73925599800003],[6.166239,45.75571799700003],[6.237939500000039,45.685374496000065],[6.328635,45.69280999700004],[6.511057500000049,45.909055497000054],[6.689002,45.72804799700003],[6.802368500000057,45.77856199900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR718","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Savoie","NUTS_NAME":"Haute-Savoie"},"geometry":{"type":"Polygon","coordinates":[[[6.821063999000046,46.42715449800005],[6.770442,46.35520549800003],[6.864808,46.28059399800003],[6.797888999000065,46.136798998000074],[6.899467,46.124058],[6.891464,46.07387949800005],[7.044886,45.92241299800003],[6.818154500000048,45.83595249800004],[6.802368500000057,45.77856199900003],[6.689002,45.72804799700003],[6.511057500000049,45.909055497000054],[6.328635,45.69280999700004],[6.237939500000039,45.685374496000065],[6.166239,45.75571799700003],[6.042249,45.73925599800003],[5.913946,45.80406049900006],[5.83123,45.93845749700006],[5.819607,46.103823498000054],[5.956067,46.13209549900006],[6.135542,46.141159],[6.310211,46.24404549800005],[6.219547,46.31187799800006],[6.231694,46.329429497000035],[6.24136,46.34358249600007],[6.519104,46.45636749600004],[6.821063999000046,46.42715449800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR72","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Auvergne","NUTS_NAME":"Auvergne"},"geometry":{"type":"Polygon","coordinates":[[[3.032063,46.794909500000074],[3.165739,46.69172199800005],[3.43421,46.71222249700003],[3.455304500000068,46.65239349800004],[3.629422500000032,46.74945649700004],[3.74101150000007,46.56021499600007],[3.998868500000071,46.46486699800005],[3.984741,46.329729996000026],[3.899538499000073,46.27590799800004],[3.768207500000074,46.23881449600003],[3.832069500000046,45.99966349600004],[3.694015,45.930728],[3.754369500000053,45.88602799900008],[3.712625500000058,45.80128799900007],[3.777585999000053,45.69056249800008],[3.98468899900007,45.495232498000064],[3.89740850000004,45.35707999700003],[4.329783500000076,45.369281998000076],[4.368010500000025,45.33710749800008],[4.331653500000073,45.318478998000046],[4.349434500000029,45.27420949800006],[4.483132,45.23644549800008],[4.422980499000062,45.137264498000036],[4.37394050000006,45.143959496000036],[4.346376500000076,45.07504099800008],[4.379061500000034,45.03617449800004],[4.30212,45.02648049800007],[4.31464,44.973134498000036],[3.862531,44.74386599600007],[3.630213,44.880930498000055],[3.478246,44.80967799700005],[3.361347500000022,44.971407999000064],[3.188543,44.86144799900006],[3.103125,44.88463249700004],[2.981675500000051,44.64467299800003],[2.933775500000024,44.780645496000034],[2.716769500000055,44.92882799600005],[2.478947500000061,44.64800999800008],[2.285289500000033,44.666758498000036],[2.207473,44.61552899800006],[2.149042500000064,44.72213849800005],[2.167976500000066,44.81050199800006],[2.062908,44.976504498000054],[2.133296500000029,44.98574199900003],[2.095157,45.05603349800003],[2.230417500000044,45.16191649800004],[2.191158,45.22305049800008],[2.369421500000044,45.38727149700003],[2.528832,45.392374999000026],[2.487533,45.41816699900005],[2.50841250000002,45.47850149800007],[2.515881,45.55387949800007],[2.461168,45.59614650000003],[2.528279,45.658533498000054],[2.492129500000033,45.73766999600008],[2.388014,45.827372996000065],[2.60902149900005,45.96664399800005],[2.550394500000039,46.08606949800003],[2.565372500000024,46.14303599900006],[2.480760500000031,46.28100849600003],[2.318604500000049,46.33440549900007],[2.281043500000067,46.42040349600006],[2.351579,46.51213949600003],[2.614960500000052,46.55327649900005],[2.566998,46.652483],[2.71502,46.744980998000074],[3.032063,46.794909500000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR721","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Allier","NUTS_NAME":"Allier"},"geometry":{"type":"Polygon","coordinates":[[[3.032063,46.794909500000074],[3.165739,46.69172199800005],[3.43421,46.71222249700003],[3.455304500000068,46.65239349800004],[3.629422500000032,46.74945649700004],[3.74101150000007,46.56021499600007],[3.998868500000071,46.46486699800005],[3.984741,46.329729996000026],[3.899538499000073,46.27590799800004],[3.768207500000074,46.23881449600003],[3.832069500000046,45.99966349600004],[3.694015,45.930728],[3.588343500000065,46.02418099600004],[3.464141,46.00816899600005],[3.419738,46.07437699900004],[3.01428,46.10290949800003],[2.923603,46.17471999800006],[2.938071,46.24194349800007],[2.7326,46.22341249700003],[2.648279,46.12097499600003],[2.565372500000024,46.14303599900006],[2.480760500000031,46.28100849600003],[2.318604500000049,46.33440549900007],[2.281043500000067,46.42040349600006],[2.351579,46.51213949600003],[2.614960500000052,46.55327649900005],[2.566998,46.652483],[2.71502,46.744980998000074],[3.032063,46.794909500000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR722","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Cantal","NUTS_NAME":"Cantal"},"geometry":{"type":"Polygon","coordinates":[[[3.103500500000052,45.354371996000054],[3.082253500000036,45.28968849900008],[3.211256,45.281408998000074],[3.288028,45.12042099800004],[3.361451500000044,45.10418599600007],[3.282284,45.103187999000056],[3.361347500000022,44.971407999000064],[3.188543,44.86144799900006],[3.103125,44.88463249700004],[2.981675500000051,44.64467299800003],[2.933775500000024,44.780645496000034],[2.716769500000055,44.92882799600005],[2.478947500000061,44.64800999800008],[2.285289500000033,44.666758498000036],[2.207473,44.61552899800006],[2.149042500000064,44.72213849800005],[2.167976500000066,44.81050199800006],[2.062908,44.976504498000054],[2.133296500000029,44.98574199900003],[2.095157,45.05603349800003],[2.230417500000044,45.16191649800004],[2.191158,45.22305049800008],[2.369421500000044,45.38727149700003],[2.528832,45.392374999000026],[2.487533,45.41816699900005],[2.50841250000002,45.47850149800007],[2.714530500000023,45.38146599800007],[2.922724,45.36810799600005],[2.999547,45.29045449800003],[3.103500500000052,45.354371996000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR723","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Loire","NUTS_NAME":"Haute-Loire"},"geometry":{"type":"Polygon","coordinates":[[[3.89740850000004,45.35707999700003],[4.329783500000076,45.369281998000076],[4.368010500000025,45.33710749800008],[4.331653500000073,45.318478998000046],[4.349434500000029,45.27420949800006],[4.483132,45.23644549800008],[4.422980499000062,45.137264498000036],[4.37394050000006,45.143959496000036],[4.346376500000076,45.07504099800008],[4.379061500000034,45.03617449800004],[4.30212,45.02648049800007],[4.31464,44.973134498000036],[3.862531,44.74386599600007],[3.630213,44.880930498000055],[3.478246,44.80967799700005],[3.361347500000022,44.971407999000064],[3.282284,45.103187999000056],[3.361451500000044,45.10418599600007],[3.288028,45.12042099800004],[3.211256,45.281408998000074],[3.082253500000036,45.28968849900008],[3.103500500000052,45.354371996000054],[3.340851500000042,45.421358499000064],[3.50646850000004,45.411243497000044],[3.618619500000023,45.33770949800004],[3.804940500000043,45.39210449800004],[3.89740850000004,45.35707999700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR724","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Puy-de-Dôme","NUTS_NAME":"Puy-de-Dôme"},"geometry":{"type":"Polygon","coordinates":[[[3.694015,45.930728],[3.754369500000053,45.88602799900008],[3.712625500000058,45.80128799900007],[3.777585999000053,45.69056249800008],[3.98468899900007,45.495232498000064],[3.89740850000004,45.35707999700003],[3.804940500000043,45.39210449800004],[3.618619500000023,45.33770949800004],[3.50646850000004,45.411243497000044],[3.340851500000042,45.421358499000064],[3.103500500000052,45.354371996000054],[2.999547,45.29045449800003],[2.922724,45.36810799600005],[2.714530500000023,45.38146599800007],[2.50841250000002,45.47850149800007],[2.515881,45.55387949800007],[2.461168,45.59614650000003],[2.528279,45.658533498000054],[2.492129500000033,45.73766999600008],[2.388014,45.827372996000065],[2.60902149900005,45.96664399800005],[2.550394500000039,46.08606949800003],[2.565372500000024,46.14303599900006],[2.648279,46.12097499600003],[2.7326,46.22341249700003],[2.938071,46.24194349800007],[2.923603,46.17471999800006],[3.01428,46.10290949800003],[3.419738,46.07437699900004],[3.464141,46.00816899600005],[3.588343500000065,46.02418099600004],[3.694015,45.930728]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR8","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"MÉDITERRANÉE","NUTS_NAME":"MÉDITERRANÉE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[9.492015,42.804485498000076],[9.446189,42.673576498000045],[9.53385350000002,42.542114498000046],[9.55911750000007,42.19669699600007],[9.549904,42.10403799800008],[9.41378,41.9557],[9.402272,41.85870349800007],[9.388422,41.655113],[9.219715,41.36760399800005],[8.788534,41.55707549600004],[8.79128,41.629024498000035],[8.91601,41.687378],[8.684517,41.74700149700004],[8.802622,41.896465496000076],[8.609244500000045,41.90718099700007],[8.591134,41.96215449600004],[8.747044500000072,42.04953],[8.585460500000067,42.151729498000066],[8.57300649900003,42.23857099700007],[8.691523500000073,42.266464],[8.60024450000003,42.35300049600005],[8.553934,42.33502199800006],[8.573408,42.38140499900004],[8.655066500000032,42.415080998000064],[8.666671,42.51559849900008],[8.727043,42.56160349800007],[9.059321500000067,42.660064498000054],[9.122977500000047,42.731223999000065],[9.301698500000043,42.678638500000034],[9.345141500000068,42.73666399600006],[9.309376500000042,42.83339299800008],[9.36147,42.924758997000026],[9.343248500000072,42.99974049900004],[9.463280500000053,42.98673649800003],[9.492015,42.804485498000076]]],[[[6.26057,45.12684399600005],[6.451804500000037,45.051720498000066],[6.630051,45.109856499000045],[6.6733,45.01962749900008],[6.744224,45.01574049800007],[6.751577,44.90558199700007],[7.006606,44.839803998000036],[7.065755,44.71346449600003],[6.948443,44.65474199800008],[6.932863,44.57307149900004],[6.853195,44.52856949800008],[6.912351500000057,44.450991499000054],[6.887428,44.36128699900007],[7.007760500000074,44.23670049800006],[7.357702500000073,44.11620099800007],[7.684466,44.173948996000036],[7.714236500000027,44.06151349800007],[7.501208,43.87497399800003],[7.529827,43.78400799600007],[7.462001301000043,43.760283908000076],[7.438017,43.75141549800003],[7.413383,43.734576996000044],[7.415625,43.725730998000074],[7.158075,43.65397649800008],[6.933721,43.48006449600007],[6.855953500000055,43.41176199600005],[6.748416,43.41979999800003],[6.716054,43.34670249800007],[6.585208500000022,43.27951449700004],[6.696511,43.26633449800005],[6.621062,43.15778350000005],[6.56200750000005,43.18908699800005],[6.358751500000039,43.10664749800003],[6.200337,43.11518099800003],[6.148761,43.02589799900005],[6.091259500000035,43.03535449700007],[6.125455500000044,43.07826999800005],[5.923431,43.12327950000008],[5.880380500000058,43.10545350000007],[5.911596,43.065773],[5.829222,43.04917549800007],[5.807260500000041,43.115710996000075],[5.671879,43.17926799800006],[5.337685500000021,43.21367649800004],[5.376047,43.25515749900006],[5.326588500000071,43.35578899600006],[5.053915,43.32427199800003],[4.96771,43.42610149600006],[4.824366500000053,43.42425899800003],[4.878544,43.355830998000044],[4.837622,43.32720549700008],[4.571876499000041,43.365741498000034],[4.597235500000068,43.405902998000045],[4.547286,43.44861999600005],[4.230281,43.46018599900003],[4.132095500000048,43.48568349800007],[4.101040500000067,43.55437099800008],[3.907951,43.51719299800004],[3.515212500000075,43.274783497000044],[3.400148500000057,43.28725049800005],[3.240562500000067,43.21280299800003],[3.084473,43.056163998000045],[3.050878,42.87390899900004],[3.009882,42.92759799800007],[2.988900500000057,42.864599998000074],[2.94807,42.83443249800007],[2.98537,42.79993299800003],[3.035430500000075,42.84206949900005],[3.050192,42.87063599900006],[3.04350450000004,42.83814999900005],[3.051833500000043,42.54395299600003],[3.174803999000062,42.43537499900003],[2.839501,42.45927099800008],[2.673950500000046,42.40464399900003],[2.675338500000066,42.34233849800006],[2.526214500000037,42.33339299700003],[2.306832499000052,42.42904299600008],[2.012462,42.35175299800005],[1.964552,42.38215699800003],[1.932950500000061,42.45493699800005],[1.731011,42.49240099800005],[1.725801,42.50440199800005],[1.786098500000037,42.573657999000034],[2.166049,42.663917498000046],[2.084955999000044,42.75165799800004],[1.950635500000033,42.73686649600006],[1.859382,42.82080499800003],[1.986922,42.87100699700005],[1.933727500000032,42.93914049800003],[1.984954999000024,42.96726849700008],[1.985759500000029,43.02995649700006],[1.949341,43.12097299900006],[1.713483,43.18561949800005],[1.688423500000056,43.27355449600003],[1.722903500000029,43.341531],[1.813485,43.34149799700003],[1.80295,43.392548496000074],[1.856448500000056,43.443452999000044],[2.029134,43.436895496000034],[2.215026,43.38261099600004],[2.265415,43.45291349800004],[2.565782500000068,43.42295799900006],[2.67602450000004,43.51345550000008],[2.615601,43.56663599800004],[2.645487500000058,43.66066949800006],[2.75452,43.61414049900003],[2.935457,43.694664998000064],[3.060680500000046,43.692804498000044],[3.064905,43.83556550000003],[3.207685999000034,43.812486998000054],[3.262606500000061,43.89100250000007],[3.358362,43.91382949800004],[3.448355,44.01910349800005],[3.263114,44.09242549800007],[3.373648,44.17075949800005],[3.120173500000021,44.26183849600005],[3.14346050000006,44.43548199800006],[2.981675500000051,44.64467299800003],[3.103125,44.88463249700004],[3.188543,44.86144799900006],[3.361347500000022,44.971407999000064],[3.478246,44.80967799700005],[3.630213,44.880930498000055],[3.862531,44.74386599600007],[3.998161500000037,44.459798497000065],[4.06844250000006,44.405057498000076],[4.053996500000039,44.337807498000075],[4.258899500000041,44.26442299800004],[4.395846500000061,44.34406449800008],[4.400397,44.28753349600004],[4.474941500000057,44.34202699600007],[4.649227500000052,44.27035999900005],[4.650611,44.32980299600007],[4.804566,44.30389699700004],[4.825547500000027,44.22856849800007],[5.149949500000048,44.30193449800004],[5.154939,44.23049049600007],[5.355353,44.21456549800007],[5.498786500000051,44.11571649700005],[5.676035999000021,44.19142849800005],[5.630529500000023,44.33331449900004],[5.492943,44.337132498000074],[5.418397500000026,44.42476849800005],[5.458414,44.499388999000075],[5.664504,44.50189499700008],[5.597252500000025,44.543273498000076],[5.641717500000027,44.65107749800006],[5.790623500000038,44.653293],[5.830441,44.690548997000064],[5.80147,44.706777500000044],[6.015539500000045,44.83527399800005],[6.355365,44.85482049800004],[6.323686,44.99870849800004],[6.20392350000003,45.012471],[6.26057,45.12684399600005]]],[[[5.070316,44.37644749800006],[4.987017500000036,44.29271349900006],[4.869529500000056,44.345077],[4.959273,44.41978299800007],[5.070316,44.37644749800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR81","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Languedoc-Roussillon","NUTS_NAME":"Languedoc-Roussillon"},"geometry":{"type":"Polygon","coordinates":[[[3.361347500000022,44.971407999000064],[3.478246,44.80967799700005],[3.630213,44.880930498000055],[3.862531,44.74386599600007],[3.998161500000037,44.459798497000065],[4.06844250000006,44.405057498000076],[4.053996500000039,44.337807498000075],[4.258899500000041,44.26442299800004],[4.395846500000061,44.34406449800008],[4.400397,44.28753349600004],[4.474941500000057,44.34202699600007],[4.649227500000052,44.27035999900005],[4.722071,44.18742049600007],[4.72183,44.08032099900004],[4.843332500000031,44.00985799800003],[4.739059500000053,43.92406199800007],[4.641914499000052,43.86747649800003],[4.612276,43.72475449700005],[4.486414500000024,43.698852997000074],[4.426171,43.62447649600006],[4.460681500000021,43.589260499000034],[4.230281,43.46018599900003],[4.132095500000048,43.48568349800007],[4.101040500000067,43.55437099800008],[3.907951,43.51719299800004],[3.515212500000075,43.274783497000044],[3.400148500000057,43.28725049800005],[3.240562500000067,43.21280299800003],[3.084473,43.056163998000045],[3.050878,42.87390899900004],[3.009882,42.92759799800007],[2.988900500000057,42.864599998000074],[2.94807,42.83443249800007],[2.98537,42.79993299800003],[3.035430500000075,42.84206949900005],[3.050192,42.87063599900006],[3.04350450000004,42.83814999900005],[3.051833500000043,42.54395299600003],[3.174803999000062,42.43537499900003],[2.839501,42.45927099800008],[2.673950500000046,42.40464399900003],[2.675338500000066,42.34233849800006],[2.526214500000037,42.33339299700003],[2.306832499000052,42.42904299600008],[2.012462,42.35175299800005],[1.964552,42.38215699800003],[1.932950500000061,42.45493699800005],[1.731011,42.49240099800005],[1.725801,42.50440199800005],[1.786098500000037,42.573657999000034],[2.166049,42.663917498000046],[2.084955999000044,42.75165799800004],[1.950635500000033,42.73686649600006],[1.859382,42.82080499800003],[1.986922,42.87100699700005],[1.933727500000032,42.93914049800003],[1.984954999000024,42.96726849700008],[1.985759500000029,43.02995649700006],[1.949341,43.12097299900006],[1.713483,43.18561949800005],[1.688423500000056,43.27355449600003],[1.722903500000029,43.341531],[1.813485,43.34149799700003],[1.80295,43.392548496000074],[1.856448500000056,43.443452999000044],[2.029134,43.436895496000034],[2.215026,43.38261099600004],[2.265415,43.45291349800004],[2.565782500000068,43.42295799900006],[2.67602450000004,43.51345550000008],[2.615601,43.56663599800004],[2.645487500000058,43.66066949800006],[2.75452,43.61414049900003],[2.935457,43.694664998000064],[3.060680500000046,43.692804498000044],[3.064905,43.83556550000003],[3.207685999000034,43.812486998000054],[3.262606500000061,43.89100250000007],[3.358362,43.91382949800004],[3.448355,44.01910349800005],[3.263114,44.09242549800007],[3.373648,44.17075949800005],[3.120173500000021,44.26183849600005],[3.14346050000006,44.43548199800006],[2.981675500000051,44.64467299800003],[3.103125,44.88463249700004],[3.188543,44.86144799900006],[3.361347500000022,44.971407999000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR811","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Aude","NUTS_NAME":"Aude"},"geometry":{"type":"Polygon","coordinates":[[[2.265415,43.45291349800004],[2.565782500000068,43.42295799900006],[2.592748500000027,43.40390099600006],[2.539922500000046,43.34562550000004],[2.617078500000048,43.28427099700008],[2.752523,43.254595496000036],[2.868318500000044,43.329586],[3.004353,43.320130998000025],[3.240562500000067,43.21280299800003],[3.084473,43.056163998000045],[3.050878,42.87390899900004],[3.009882,42.92759799800007],[2.988900500000057,42.864599998000074],[2.865274,42.91833950000006],[2.737364,42.83560999900004],[2.335494500000038,42.839923998000074],[2.355390500000055,42.72700849800003],[2.166049,42.663917498000046],[2.084955999000044,42.75165799800004],[1.950635500000033,42.73686649600006],[1.859382,42.82080499800003],[1.986922,42.87100699700005],[1.933727500000032,42.93914049800003],[1.984954999000024,42.96726849700008],[1.985759500000029,43.02995649700006],[1.949341,43.12097299900006],[1.713483,43.18561949800005],[1.688423500000056,43.27355449600003],[1.722903500000029,43.341531],[1.813485,43.34149799700003],[1.80295,43.392548496000074],[1.856448500000056,43.443452999000044],[2.029134,43.436895496000034],[2.215026,43.38261099600004],[2.265415,43.45291349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR812","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Gard","NUTS_NAME":"Gard"},"geometry":{"type":"Polygon","coordinates":[[[4.649227500000052,44.27035999900005],[4.722071,44.18742049600007],[4.72183,44.08032099900004],[4.843332500000031,44.00985799800003],[4.739059500000053,43.92406199800007],[4.641914499000052,43.86747649800003],[4.612276,43.72475449700005],[4.486414500000024,43.698852997000074],[4.426171,43.62447649600006],[4.460681500000021,43.589260499000034],[4.230281,43.46018599900003],[4.132095500000048,43.48568349800007],[4.101040500000067,43.55437099800008],[4.193765,43.64207349900005],[4.141428,43.738216],[3.902433500000029,43.87896249800008],[3.803986,43.87901399800006],[3.827000500000054,43.93619199600005],[3.787331,43.96686849800005],[3.623586,43.91697399800006],[3.578441,43.84366199800007],[3.358362,43.91382949800004],[3.448355,44.01910349800005],[3.263114,44.09242549800007],[3.373648,44.17075949800005],[3.873875500000054,44.13043049800007],[3.957271500000047,44.193786998000064],[3.951628500000027,44.33276299700003],[3.885812,44.38797099900006],[3.998161500000037,44.459798497000065],[4.06844250000006,44.405057498000076],[4.053996500000039,44.337807498000075],[4.258899500000041,44.26442299800004],[4.395846500000061,44.34406449800008],[4.400397,44.28753349600004],[4.474941500000057,44.34202699600007],[4.649227500000052,44.27035999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR813","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Hérault","NUTS_NAME":"Hérault"},"geometry":{"type":"Polygon","coordinates":[[[4.101040500000067,43.55437099800008],[3.907951,43.51719299800004],[3.515212500000075,43.274783497000044],[3.400148500000057,43.28725049800005],[3.240562500000067,43.21280299800003],[3.004353,43.320130998000025],[2.868318500000044,43.329586],[2.752523,43.254595496000036],[2.617078500000048,43.28427099700008],[2.539922500000046,43.34562550000004],[2.592748500000027,43.40390099600006],[2.565782500000068,43.42295799900006],[2.67602450000004,43.51345550000008],[2.615601,43.56663599800004],[2.645487500000058,43.66066949800006],[2.75452,43.61414049900003],[2.935457,43.694664998000064],[3.060680500000046,43.692804498000044],[3.064905,43.83556550000003],[3.207685999000034,43.812486998000054],[3.262606500000061,43.89100250000007],[3.358362,43.91382949800004],[3.578441,43.84366199800007],[3.623586,43.91697399800006],[3.787331,43.96686849800005],[3.827000500000054,43.93619199600005],[3.803986,43.87901399800006],[3.902433500000029,43.87896249800008],[4.141428,43.738216],[4.193765,43.64207349900005],[4.101040500000067,43.55437099800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR814","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Lozère","NUTS_NAME":"Lozère"},"geometry":{"type":"Polygon","coordinates":[[[3.862531,44.74386599600007],[3.998161500000037,44.459798497000065],[3.885812,44.38797099900006],[3.951628500000027,44.33276299700003],[3.957271500000047,44.193786998000064],[3.873875500000054,44.13043049800007],[3.373648,44.17075949800005],[3.120173500000021,44.26183849600005],[3.14346050000006,44.43548199800006],[2.981675500000051,44.64467299800003],[3.103125,44.88463249700004],[3.188543,44.86144799900006],[3.361347500000022,44.971407999000064],[3.478246,44.80967799700005],[3.630213,44.880930498000055],[3.862531,44.74386599600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR815","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Pyrénées-Orientales","NUTS_NAME":"Pyrénées-Orientales"},"geometry":{"type":"Polygon","coordinates":[[[2.988900500000057,42.864599998000074],[2.94807,42.83443249800007],[2.98537,42.79993299800003],[3.035430500000075,42.84206949900005],[3.050192,42.87063599900006],[3.04350450000004,42.83814999900005],[3.051833500000043,42.54395299600003],[3.174803999000062,42.43537499900003],[2.839501,42.45927099800008],[2.673950500000046,42.40464399900003],[2.675338500000066,42.34233849800006],[2.526214500000037,42.33339299700003],[2.306832499000052,42.42904299600008],[2.012462,42.35175299800005],[1.964552,42.38215699800003],[1.932950500000061,42.45493699800005],[1.731011,42.49240099800005],[1.725801,42.50440199800005],[1.786098500000037,42.573657999000034],[2.166049,42.663917498000046],[2.355390500000055,42.72700849800003],[2.335494500000038,42.839923998000074],[2.737364,42.83560999900004],[2.865274,42.91833950000006],[2.988900500000057,42.864599998000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR82","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Provence-Alpes-Côte d'Azur","NUTS_NAME":"Provence-Alpes-Côte d'Azur"},"geometry":{"type":"MultiPolygon","coordinates":[[[[6.26057,45.12684399600005],[6.451804500000037,45.051720498000066],[6.630051,45.109856499000045],[6.6733,45.01962749900008],[6.744224,45.01574049800007],[6.751577,44.90558199700007],[7.006606,44.839803998000036],[7.065755,44.71346449600003],[6.948443,44.65474199800008],[6.932863,44.57307149900004],[6.853195,44.52856949800008],[6.912351500000057,44.450991499000054],[6.887428,44.36128699900007],[7.007760500000074,44.23670049800006],[7.357702500000073,44.11620099800007],[7.684466,44.173948996000036],[7.714236500000027,44.06151349800007],[7.501208,43.87497399800003],[7.529827,43.78400799600007],[7.462001301000043,43.760283908000076],[7.438017,43.75141549800003],[7.413383,43.734576996000044],[7.415625,43.725730998000074],[7.158075,43.65397649800008],[6.933721,43.48006449600007],[6.855953500000055,43.41176199600005],[6.748416,43.41979999800003],[6.716054,43.34670249800007],[6.585208500000022,43.27951449700004],[6.696511,43.26633449800005],[6.621062,43.15778350000005],[6.56200750000005,43.18908699800005],[6.358751500000039,43.10664749800003],[6.200337,43.11518099800003],[6.148761,43.02589799900005],[6.091259500000035,43.03535449700007],[6.125455500000044,43.07826999800005],[5.923431,43.12327950000008],[5.880380500000058,43.10545350000007],[5.911596,43.065773],[5.829222,43.04917549800007],[5.807260500000041,43.115710996000075],[5.671879,43.17926799800006],[5.337685500000021,43.21367649800004],[5.376047,43.25515749900006],[5.326588500000071,43.35578899600006],[5.053915,43.32427199800003],[4.96771,43.42610149600006],[4.824366500000053,43.42425899800003],[4.878544,43.355830998000044],[4.837622,43.32720549700008],[4.571876499000041,43.365741498000034],[4.597235500000068,43.405902998000045],[4.547286,43.44861999600005],[4.230281,43.46018599900003],[4.460681500000021,43.589260499000034],[4.426171,43.62447649600006],[4.486414500000024,43.698852997000074],[4.612276,43.72475449700005],[4.641914499000052,43.86747649800003],[4.739059500000053,43.92406199800007],[4.843332500000031,44.00985799800003],[4.72183,44.08032099900004],[4.722071,44.18742049600007],[4.649227500000052,44.27035999900005],[4.650611,44.32980299600007],[4.804566,44.30389699700004],[4.825547500000027,44.22856849800007],[5.149949500000048,44.30193449800004],[5.154939,44.23049049600007],[5.355353,44.21456549800007],[5.498786500000051,44.11571649700005],[5.676035999000021,44.19142849800005],[5.630529500000023,44.33331449900004],[5.492943,44.337132498000074],[5.418397500000026,44.42476849800005],[5.458414,44.499388999000075],[5.664504,44.50189499700008],[5.597252500000025,44.543273498000076],[5.641717500000027,44.65107749800006],[5.790623500000038,44.653293],[5.830441,44.690548997000064],[5.80147,44.706777500000044],[6.015539500000045,44.83527399800005],[6.355365,44.85482049800004],[6.323686,44.99870849800004],[6.20392350000003,45.012471],[6.26057,45.12684399600005]]],[[[5.070316,44.37644749800006],[4.987017500000036,44.29271349900006],[4.869529500000056,44.345077],[4.959273,44.41978299800007],[5.070316,44.37644749800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR821","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Alpes-de-Haute-Provence","NUTS_NAME":"Alpes-de-Haute-Provence"},"geometry":{"type":"Polygon","coordinates":[[[6.887428,44.36128699900007],[6.686704500000076,44.168123998000056],[6.745692,44.04282699600003],[6.943949500000031,43.89949649600004],[6.670719,43.887744498000075],[6.709827500000074,43.81065999600003],[6.636390500000061,43.788946999000075],[6.41442550000005,43.79158999800006],[6.383545500000025,43.73443099800005],[6.209453,43.79691799700004],[6.021736500000031,43.668281998000055],[5.905655500000023,43.73942999900004],[5.757337500000062,43.729404498000065],[5.654449,43.82511199800007],[5.544779500000061,43.817611999000064],[5.60823,43.91477099800005],[5.512673500000062,43.945397498000034],[5.544593,44.069889498000066],[5.502642,44.06344199800003],[5.498786500000051,44.11571649700005],[5.676035999000021,44.19142849800005],[5.909304,44.190461998000046],[5.822718,44.27551699600008],[5.921823999000026,44.24840549800007],[5.953541,44.39501299800003],[6.082682,44.47740149800006],[6.227249,44.38173699600003],[6.262488500000075,44.41225249800004],[6.233518500000059,44.46374399800004],[6.327105,44.46345849800008],[6.357312,44.52264149900003],[6.468562,44.45129399800004],[6.632299500000045,44.44691049900007],[6.767231,44.58738699700007],[6.948443,44.65474199800008],[6.932863,44.57307149900004],[6.853195,44.52856949800008],[6.912351500000057,44.450991499000054],[6.887428,44.36128699900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR822","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Hautes-Alpes","NUTS_NAME":"Hautes-Alpes"},"geometry":{"type":"Polygon","coordinates":[[[6.630051,45.109856499000045],[6.6733,45.01962749900008],[6.744224,45.01574049800007],[6.751577,44.90558199700007],[7.006606,44.839803998000036],[7.065755,44.71346449600003],[6.948443,44.65474199800008],[6.767231,44.58738699700007],[6.632299500000045,44.44691049900007],[6.468562,44.45129399800004],[6.357312,44.52264149900003],[6.327105,44.46345849800008],[6.233518500000059,44.46374399800004],[6.262488500000075,44.41225249800004],[6.227249,44.38173699600003],[6.082682,44.47740149800006],[5.953541,44.39501299800003],[5.921823999000026,44.24840549800007],[5.822718,44.27551699600008],[5.909304,44.190461998000046],[5.676035999000021,44.19142849800005],[5.630529500000023,44.33331449900004],[5.492943,44.337132498000074],[5.418397500000026,44.42476849800005],[5.458414,44.499388999000075],[5.664504,44.50189499700008],[5.597252500000025,44.543273498000076],[5.641717500000027,44.65107749800006],[5.790623500000038,44.653293],[5.830441,44.690548997000064],[5.80147,44.706777500000044],[6.015539500000045,44.83527399800005],[6.355365,44.85482049800004],[6.323686,44.99870849800004],[6.20392350000003,45.012471],[6.26057,45.12684399600005],[6.451804500000037,45.051720498000066],[6.630051,45.109856499000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR823","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Alpes-Maritimes","NUTS_NAME":"Alpes-Maritimes"},"geometry":{"type":"Polygon","coordinates":[[[7.007760500000074,44.23670049800006],[7.357702500000073,44.11620099800007],[7.684466,44.173948996000036],[7.714236500000027,44.06151349800007],[7.501208,43.87497399800003],[7.529827,43.78400799600007],[7.462001301000043,43.760283908000076],[7.438017,43.75141549800003],[7.413383,43.734576996000044],[7.415625,43.725730998000074],[7.158075,43.65397649800008],[6.933721,43.48006449600007],[6.884229,43.50253599900003],[6.902640500000075,43.602668496000035],[6.761138,43.66588999800007],[6.752780500000028,43.74098999800003],[6.636390500000061,43.788946999000075],[6.709827500000074,43.81065999600003],[6.670719,43.887744498000075],[6.943949500000031,43.89949649600004],[6.745692,44.04282699600003],[6.686704500000076,44.168123998000056],[6.887428,44.36128699900007],[7.007760500000074,44.23670049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR824","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Bouches-du-Rhône","NUTS_NAME":"Bouches-du-Rhône"},"geometry":{"type":"Polygon","coordinates":[[[5.753644499000075,43.72461949700005],[5.79920850000002,43.65992999600007],[5.697808,43.642541996000034],[5.656025500000055,43.57399649800004],[5.725071,43.55126749800007],[5.727839500000073,43.46690849600003],[5.788349,43.42026049800006],[5.682792500000062,43.399159498000074],[5.68533350000007,43.32273199800005],[5.762797499000044,43.28235349800008],[5.671879,43.17926799800006],[5.337685500000021,43.21367649800004],[5.376047,43.25515749900006],[5.326588500000071,43.35578899600006],[5.053915,43.32427199800003],[4.96771,43.42610149600006],[4.824366500000053,43.42425899800003],[4.878544,43.355830998000044],[4.837622,43.32720549700008],[4.571876499000041,43.365741498000034],[4.597235500000068,43.405902998000045],[4.547286,43.44861999600005],[4.230281,43.46018599900003],[4.460681500000021,43.589260499000034],[4.426171,43.62447649600006],[4.486414500000024,43.698852997000074],[4.612276,43.72475449700005],[4.641914499000052,43.86747649800003],[4.739059500000053,43.92406199800007],[5.48808450000007,43.667643],[5.753644499000075,43.72461949700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR825","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Var","NUTS_NAME":"Var"},"geometry":{"type":"Polygon","coordinates":[[[6.752780500000028,43.74098999800003],[6.761138,43.66588999800007],[6.902640500000075,43.602668496000035],[6.884229,43.50253599900003],[6.933721,43.48006449600007],[6.855953500000055,43.41176199600005],[6.748416,43.41979999800003],[6.716054,43.34670249800007],[6.585208500000022,43.27951449700004],[6.696511,43.26633449800005],[6.621062,43.15778350000005],[6.56200750000005,43.18908699800005],[6.358751500000039,43.10664749800003],[6.200337,43.11518099800003],[6.148761,43.02589799900005],[6.091259500000035,43.03535449700007],[6.125455500000044,43.07826999800005],[5.923431,43.12327950000008],[5.880380500000058,43.10545350000007],[5.911596,43.065773],[5.829222,43.04917549800007],[5.807260500000041,43.115710996000075],[5.671879,43.17926799800006],[5.762797499000044,43.28235349800008],[5.68533350000007,43.32273199800005],[5.682792500000062,43.399159498000074],[5.788349,43.42026049800006],[5.727839500000073,43.46690849600003],[5.725071,43.55126749800007],[5.656025500000055,43.57399649800004],[5.697808,43.642541996000034],[5.79920850000002,43.65992999600007],[5.753644499000075,43.72461949700005],[5.757337500000062,43.729404498000065],[5.905655500000023,43.73942999900004],[6.021736500000031,43.668281998000055],[6.209453,43.79691799700004],[6.383545500000025,43.73443099800005],[6.41442550000005,43.79158999800006],[6.636390500000061,43.788946999000075],[6.752780500000028,43.74098999800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR826","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Vaucluse","NUTS_NAME":"Vaucluse"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.804566,44.30389699700004],[4.825547500000027,44.22856849800007],[5.149949500000048,44.30193449800004],[5.154939,44.23049049600007],[5.355353,44.21456549800007],[5.498786500000051,44.11571649700005],[5.502642,44.06344199800003],[5.544593,44.069889498000066],[5.512673500000062,43.945397498000034],[5.60823,43.91477099800005],[5.544779500000061,43.817611999000064],[5.654449,43.82511199800007],[5.757337500000062,43.729404498000065],[5.753644499000075,43.72461949700005],[5.48808450000007,43.667643],[4.739059500000053,43.92406199800007],[4.843332500000031,44.00985799800003],[4.72183,44.08032099900004],[4.722071,44.18742049600007],[4.649227500000052,44.27035999900005],[4.650611,44.32980299600007],[4.804566,44.30389699700004]]],[[[5.070316,44.37644749800006],[4.987017500000036,44.29271349900006],[4.869529500000056,44.345077],[4.959273,44.41978299800007],[5.070316,44.37644749800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR83","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Corse","NUTS_NAME":"Corse"},"geometry":{"type":"Polygon","coordinates":[[[9.492015,42.804485498000076],[9.446189,42.673576498000045],[9.53385350000002,42.542114498000046],[9.55911750000007,42.19669699600007],[9.549904,42.10403799800008],[9.41378,41.9557],[9.402272,41.85870349800007],[9.388422,41.655113],[9.219715,41.36760399800005],[8.788534,41.55707549600004],[8.79128,41.629024498000035],[8.91601,41.687378],[8.684517,41.74700149700004],[8.802622,41.896465496000076],[8.609244500000045,41.90718099700007],[8.591134,41.96215449600004],[8.747044500000072,42.04953],[8.585460500000067,42.151729498000066],[8.57300649900003,42.23857099700007],[8.691523500000073,42.266464],[8.60024450000003,42.35300049600005],[8.553934,42.33502199800006],[8.573408,42.38140499900004],[8.655066500000032,42.415080998000064],[8.666671,42.51559849900008],[8.727043,42.56160349800007],[9.059321500000067,42.660064498000054],[9.122977500000047,42.731223999000065],[9.301698500000043,42.678638500000034],[9.345141500000068,42.73666399600006],[9.309376500000042,42.83339299800008],[9.36147,42.924758997000026],[9.343248500000072,42.99974049900004],[9.463280500000053,42.98673649800003],[9.492015,42.804485498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR831","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Corse-du-Sud","NUTS_NAME":"Corse-du-Sud"},"geometry":{"type":"Polygon","coordinates":[[[9.402272,41.85870349800007],[9.388422,41.655113],[9.219715,41.36760399800005],[8.788534,41.55707549600004],[8.79128,41.629024498000035],[8.91601,41.687378],[8.684517,41.74700149700004],[8.802622,41.896465496000076],[8.609244500000045,41.90718099700007],[8.591134,41.96215449600004],[8.747044500000072,42.04953],[8.585460500000067,42.151729498000066],[8.57300649900003,42.23857099700007],[8.691523500000073,42.266464],[8.60024450000003,42.35300049600005],[8.553934,42.33502199800006],[8.573408,42.38140499900004],[9.045743500000071,42.20772599800006],[9.16153,42.024932996000075],[9.221089,42.02760049800003],[9.236727,41.846577498000045],[9.402272,41.85870349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR832","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Corse","NUTS_NAME":"Haute-Corse"},"geometry":{"type":"Polygon","coordinates":[[[9.402272,41.85870349800007],[9.236727,41.846577498000045],[9.221089,42.02760049800003],[9.16153,42.024932996000075],[9.045743500000071,42.20772599800006],[8.573408,42.38140499900004],[8.655066500000032,42.415080998000064],[8.666671,42.51559849900008],[8.727043,42.56160349800007],[9.059321500000067,42.660064498000054],[9.122977500000047,42.731223999000065],[9.301698500000043,42.678638500000034],[9.345141500000068,42.73666399600006],[9.309376500000042,42.83339299800008],[9.36147,42.924758997000026],[9.343248500000072,42.99974049900004],[9.463280500000053,42.98673649800003],[9.492015,42.804485498000076],[9.446189,42.673576498000045],[9.53385350000002,42.542114498000046],[9.55911750000007,42.19669699600007],[9.549904,42.10403799800008],[9.41378,41.9557],[9.402272,41.85870349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"DÉPARTEMENTS D'OUTRE-MER","NUTS_NAME":"DÉPARTEMENTS D'OUTRE-MER"},"geometry":{"type":"MultiPolygon","coordinates":[[[[55.797395,-21.350129496999955],[55.609923500000036,-21.387309496999933],[55.292974,-21.229784996999967],[55.217144,-21.036134996999976],[55.275858,-21.001988496999957],[55.283669,-20.926209497999935],[55.395047,-20.881834998999977],[55.64773750000006,-20.915604496999947],[55.82579650000008,-21.143524497999977],[55.797395,-21.350129496999955]]],[[[45.21328150000005,-12.87307549999997],[45.14830050000006,-13.000090498999953],[45.071559,-12.902030998999976],[45.154967,-12.923567997999953],[45.04258,-12.74402349899998],[45.08842350000003,-12.669118998999977],[45.125466999000025,-12.727415998999959],[45.236856,-12.753384997999945],[45.18605050000008,-12.839496],[45.21328150000005,-12.87307549999997]]],[[[-52.94350749999995,5.45198750000003],[-52.364701,4.920734500000037],[-52.300845,4.950978500000076],[-52.145722,4.790893],[-51.813244,4.634343500000057],[-51.629835499999956,4.189126499000054],[-51.65604,4.05388750000003],[-51.77759949999995,3.974536],[-51.798616,3.886936001000038],[-51.972580499999935,3.705449],[-52.23357700099996,3.240668],[-52.292488,3.226403001000051],[-52.348031499999934,3.136390999000071],[-52.32591949999994,3.077791],[-52.55521049999999,2.640293],[-52.52731749999998,2.58393],[-52.565187,2.516254],[-52.660556,2.373956001000067],[-52.84180649999996,2.291378],[-52.904169,2.190938],[-52.980611,2.167639499000074],[-53.133876500999975,2.225365001000057],[-53.28104949999994,2.18784800100002],[-53.226680499999986,2.263608],[-53.34006549999998,2.351948500000049],[-53.462361,2.252782001000071],[-53.534661,2.251549001000058],[-53.723031,2.31207],[-53.769087499999955,2.375374501000067],[-53.94430649999998,2.215348500000061],[-54.066706,2.189358500000026],[-54.106466499999954,2.121296500000028],[-54.152372499999956,2.119626501000027],[-54.19038949999998,2.171099],[-54.270671499999935,2.144490500000074],[-54.360693,2.174917500000049],[-54.362251500999946,2.211756500000035],[-54.460852,2.207485001000066],[-54.54886749999997,2.270018],[-54.513794,2.279992],[-54.53909649999997,2.324723],[-54.58785949999998,2.322023501000047],[-54.590468,2.351944001000049],[-54.508611,2.335833500000035],[-54.47613949999993,2.435383500000057],[-54.41146,2.445936500000073],[-54.208187,2.777737500000057],[-54.172856,2.93860550100004],[-54.189067499999965,3.206341500000065],[-54.069237499999986,3.297167001000048],[-53.979807,3.606575],[-54.082868,3.674424999000053],[-54.121917500999984,3.792688501000043],[-54.197729,3.805496500000061],[-54.191877,3.847389001000067],[-54.35672649999998,4.047393501000045],[-54.323058,4.144421],[-54.39576249999993,4.202166501000022],[-54.386382,4.352911500000062],[-54.441826499999934,4.369863],[-54.415114,4.60441],[-54.468998,4.890381],[-54.415525,5.084697],[-54.018837,5.521456500000056],[-54.00809450099996,5.623116500000037],[-53.96836549999995,5.745176500000071],[-53.79925949999995,5.723899501000062],[-53.469359,5.56524550000006],[-53.261408,5.548479500000042],[-52.94350749999995,5.45198750000003]]],[[[-61.003888,14.808371500000021],[-60.95242149999996,14.748496500000044],[-60.872843,14.772254498000052],[-60.946177499999976,14.72632],[-60.810356499999955,14.480542],[-60.85767549999997,14.398175],[-60.893263,14.420223500000077],[-60.86607649999996,14.470458],[-61.081969,14.470402499000045],[-61.087707,14.534112500000049],[-60.98930949999993,14.548964500000068],[-61.018952,14.611703500000033],[-61.090783499999986,14.60018],[-61.150699,14.648276],[-61.229269,14.82211249900007],[-61.180351,14.87490199900003],[-61.003888,14.808371500000021]]],[[[-61.19499800099999,15.930456500000048],[-61.242508499999985,15.874458999000069],[-61.33183749999995,15.900605],[-61.291233000999966,16.003174999000066],[-61.19499800099999,15.930456500000048]]],[[[-61.25487149999998,16.25613899900003],[-61.465641,16.196540499000037],[-61.52729849999997,16.219122998000046],[-61.55065349999995,16.286972499000058],[-61.50920149999996,16.38560099700004],[-61.54121049999998,16.438072999000042],[-61.46405749999997,16.513064498000062],[-61.375652499999944,16.340646999000057],[-61.25848049999996,16.30302449900006],[-61.25487149999998,16.25613899900003]]],[[[-61.67261249999996,16.324603499000034],[-61.611944499999936,16.267183999000054],[-61.555114499999945,16.284743999000057],[-61.59044249999994,16.19234399900006],[-61.555346499999985,16.053240498000036],[-61.69680049999994,15.946588999000028],[-61.769683,16.05896149700004],[-61.80593,16.257474497000032],[-61.764951,16.353860999000062],[-61.67261249999996,16.324603499000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA1","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Guadeloupe","NUTS_NAME":"Guadeloupe"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-61.19499800099999,15.930456500000048],[-61.242508499999985,15.874458999000069],[-61.33183749999995,15.900605],[-61.291233000999966,16.003174999000066],[-61.19499800099999,15.930456500000048]]],[[[-61.25487149999998,16.25613899900003],[-61.465641,16.196540499000037],[-61.52729849999997,16.219122998000046],[-61.55065349999995,16.286972499000058],[-61.50920149999996,16.38560099700004],[-61.54121049999998,16.438072999000042],[-61.46405749999997,16.513064498000062],[-61.375652499999944,16.340646999000057],[-61.25848049999996,16.30302449900006],[-61.25487149999998,16.25613899900003]]],[[[-61.67261249999996,16.324603499000034],[-61.611944499999936,16.267183999000054],[-61.555114499999945,16.284743999000057],[-61.59044249999994,16.19234399900006],[-61.555346499999985,16.053240498000036],[-61.69680049999994,15.946588999000028],[-61.769683,16.05896149700004],[-61.80593,16.257474497000032],[-61.764951,16.353860999000062],[-61.67261249999996,16.324603499000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA10","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Guadeloupe","NUTS_NAME":"Guadeloupe"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-61.19499800099999,15.930456500000048],[-61.242508499999985,15.874458999000069],[-61.33183749999995,15.900605],[-61.291233000999966,16.003174999000066],[-61.19499800099999,15.930456500000048]]],[[[-61.25487149999998,16.25613899900003],[-61.465641,16.196540499000037],[-61.52729849999997,16.219122998000046],[-61.55065349999995,16.286972499000058],[-61.50920149999996,16.38560099700004],[-61.54121049999998,16.438072999000042],[-61.46405749999997,16.513064498000062],[-61.375652499999944,16.340646999000057],[-61.25848049999996,16.30302449900006],[-61.25487149999998,16.25613899900003]]],[[[-61.67261249999996,16.324603499000034],[-61.611944499999936,16.267183999000054],[-61.555114499999945,16.284743999000057],[-61.59044249999994,16.19234399900006],[-61.555346499999985,16.053240498000036],[-61.69680049999994,15.946588999000028],[-61.769683,16.05896149700004],[-61.80593,16.257474497000032],[-61.764951,16.353860999000062],[-61.67261249999996,16.324603499000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA2","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Martinique","NUTS_NAME":"Martinique"},"geometry":{"type":"Polygon","coordinates":[[[-61.003888,14.808371500000021],[-60.95242149999996,14.748496500000044],[-60.872843,14.772254498000052],[-60.946177499999976,14.72632],[-60.810356499999955,14.480542],[-60.85767549999997,14.398175],[-60.893263,14.420223500000077],[-60.86607649999996,14.470458],[-61.081969,14.470402499000045],[-61.087707,14.534112500000049],[-60.98930949999993,14.548964500000068],[-61.018952,14.611703500000033],[-61.090783499999986,14.60018],[-61.150699,14.648276],[-61.229269,14.82211249900007],[-61.180351,14.87490199900003],[-61.003888,14.808371500000021]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA20","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Martinique","NUTS_NAME":"Martinique"},"geometry":{"type":"Polygon","coordinates":[[[-61.003888,14.808371500000021],[-60.95242149999996,14.748496500000044],[-60.872843,14.772254498000052],[-60.946177499999976,14.72632],[-60.810356499999955,14.480542],[-60.85767549999997,14.398175],[-60.893263,14.420223500000077],[-60.86607649999996,14.470458],[-61.081969,14.470402499000045],[-61.087707,14.534112500000049],[-60.98930949999993,14.548964500000068],[-61.018952,14.611703500000033],[-61.090783499999986,14.60018],[-61.150699,14.648276],[-61.229269,14.82211249900007],[-61.180351,14.87490199900003],[-61.003888,14.808371500000021]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA3","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Guyane","NUTS_NAME":"Guyane"},"geometry":{"type":"Polygon","coordinates":[[[-51.629835499999956,4.189126499000054],[-51.65604,4.05388750000003],[-51.77759949999995,3.974536],[-51.798616,3.886936001000038],[-51.972580499999935,3.705449],[-52.23357700099996,3.240668],[-52.292488,3.226403001000051],[-52.348031499999934,3.136390999000071],[-52.32591949999994,3.077791],[-52.55521049999999,2.640293],[-52.52731749999998,2.58393],[-52.565187,2.516254],[-52.660556,2.373956001000067],[-52.84180649999996,2.291378],[-52.904169,2.190938],[-52.980611,2.167639499000074],[-53.133876500999975,2.225365001000057],[-53.28104949999994,2.18784800100002],[-53.226680499999986,2.263608],[-53.34006549999998,2.351948500000049],[-53.462361,2.252782001000071],[-53.534661,2.251549001000058],[-53.723031,2.31207],[-53.769087499999955,2.375374501000067],[-53.94430649999998,2.215348500000061],[-54.066706,2.189358500000026],[-54.106466499999954,2.121296500000028],[-54.152372499999956,2.119626501000027],[-54.19038949999998,2.171099],[-54.270671499999935,2.144490500000074],[-54.360693,2.174917500000049],[-54.362251500999946,2.211756500000035],[-54.460852,2.207485001000066],[-54.54886749999997,2.270018],[-54.513794,2.279992],[-54.53909649999997,2.324723],[-54.58785949999998,2.322023501000047],[-54.590468,2.351944001000049],[-54.508611,2.335833500000035],[-54.47613949999993,2.435383500000057],[-54.41146,2.445936500000073],[-54.208187,2.777737500000057],[-54.172856,2.93860550100004],[-54.189067499999965,3.206341500000065],[-54.069237499999986,3.297167001000048],[-53.979807,3.606575],[-54.082868,3.674424999000053],[-54.121917500999984,3.792688501000043],[-54.197729,3.805496500000061],[-54.191877,3.847389001000067],[-54.35672649999998,4.047393501000045],[-54.323058,4.144421],[-54.39576249999993,4.202166501000022],[-54.386382,4.352911500000062],[-54.441826499999934,4.369863],[-54.415114,4.60441],[-54.468998,4.890381],[-54.415525,5.084697],[-54.018837,5.521456500000056],[-54.00809450099996,5.623116500000037],[-53.96836549999995,5.745176500000071],[-53.79925949999995,5.723899501000062],[-53.469359,5.56524550000006],[-53.261408,5.548479500000042],[-52.94350749999995,5.45198750000003],[-52.364701,4.920734500000037],[-52.300845,4.950978500000076],[-52.145722,4.790893],[-51.813244,4.634343500000057],[-51.629835499999956,4.189126499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA30","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Guyane","NUTS_NAME":"Guyane"},"geometry":{"type":"Polygon","coordinates":[[[-51.629835499999956,4.189126499000054],[-51.65604,4.05388750000003],[-51.77759949999995,3.974536],[-51.798616,3.886936001000038],[-51.972580499999935,3.705449],[-52.23357700099996,3.240668],[-52.292488,3.226403001000051],[-52.348031499999934,3.136390999000071],[-52.32591949999994,3.077791],[-52.55521049999999,2.640293],[-52.52731749999998,2.58393],[-52.565187,2.516254],[-52.660556,2.373956001000067],[-52.84180649999996,2.291378],[-52.904169,2.190938],[-52.980611,2.167639499000074],[-53.133876500999975,2.225365001000057],[-53.28104949999994,2.18784800100002],[-53.226680499999986,2.263608],[-53.34006549999998,2.351948500000049],[-53.462361,2.252782001000071],[-53.534661,2.251549001000058],[-53.723031,2.31207],[-53.769087499999955,2.375374501000067],[-53.94430649999998,2.215348500000061],[-54.066706,2.189358500000026],[-54.106466499999954,2.121296500000028],[-54.152372499999956,2.119626501000027],[-54.19038949999998,2.171099],[-54.270671499999935,2.144490500000074],[-54.360693,2.174917500000049],[-54.362251500999946,2.211756500000035],[-54.460852,2.207485001000066],[-54.54886749999997,2.270018],[-54.513794,2.279992],[-54.53909649999997,2.324723],[-54.58785949999998,2.322023501000047],[-54.590468,2.351944001000049],[-54.508611,2.335833500000035],[-54.47613949999993,2.435383500000057],[-54.41146,2.445936500000073],[-54.208187,2.777737500000057],[-54.172856,2.93860550100004],[-54.189067499999965,3.206341500000065],[-54.069237499999986,3.297167001000048],[-53.979807,3.606575],[-54.082868,3.674424999000053],[-54.121917500999984,3.792688501000043],[-54.197729,3.805496500000061],[-54.191877,3.847389001000067],[-54.35672649999998,4.047393501000045],[-54.323058,4.144421],[-54.39576249999993,4.202166501000022],[-54.386382,4.352911500000062],[-54.441826499999934,4.369863],[-54.415114,4.60441],[-54.468998,4.890381],[-54.415525,5.084697],[-54.018837,5.521456500000056],[-54.00809450099996,5.623116500000037],[-53.96836549999995,5.745176500000071],[-53.79925949999995,5.723899501000062],[-53.469359,5.56524550000006],[-53.261408,5.548479500000042],[-52.94350749999995,5.45198750000003],[-52.364701,4.920734500000037],[-52.300845,4.950978500000076],[-52.145722,4.790893],[-51.813244,4.634343500000057],[-51.629835499999956,4.189126499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA4","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"La Réunion","NUTS_NAME":"La Réunion"},"geometry":{"type":"Polygon","coordinates":[[[55.64773750000006,-20.915604496999947],[55.82579650000008,-21.143524497999977],[55.797395,-21.350129496999955],[55.609923500000036,-21.387309496999933],[55.292974,-21.229784996999967],[55.217144,-21.036134996999976],[55.275858,-21.001988496999957],[55.283669,-20.926209497999935],[55.395047,-20.881834998999977],[55.64773750000006,-20.915604496999947]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA40","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"La Réunion","NUTS_NAME":"La Réunion"},"geometry":{"type":"Polygon","coordinates":[[[55.64773750000006,-20.915604496999947],[55.82579650000008,-21.143524497999977],[55.797395,-21.350129496999955],[55.609923500000036,-21.387309496999933],[55.292974,-21.229784996999967],[55.217144,-21.036134996999976],[55.275858,-21.001988496999957],[55.283669,-20.926209497999935],[55.395047,-20.881834998999977],[55.64773750000006,-20.915604496999947]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA5","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Mayotte","NUTS_NAME":"Mayotte"},"geometry":{"type":"Polygon","coordinates":[[[45.21328150000005,-12.87307549999997],[45.14830050000006,-13.000090498999953],[45.071559,-12.902030998999976],[45.154967,-12.923567997999953],[45.04258,-12.74402349899998],[45.08842350000003,-12.669118998999977],[45.125466999000025,-12.727415998999959],[45.236856,-12.753384997999945],[45.18605050000008,-12.839496],[45.21328150000005,-12.87307549999997]]]}},{"type":"Feature","properties":{"NUTS_ID":"FRA50","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Mayotte","NUTS_NAME":"Mayotte"},"geometry":{"type":"Polygon","coordinates":[[[45.21328150000005,-12.87307549999997],[45.14830050000006,-13.000090498999953],[45.071559,-12.902030998999976],[45.154967,-12.923567997999953],[45.04258,-12.74402349899998],[45.08842350000003,-12.669118998999977],[45.125466999000025,-12.727415998999959],[45.236856,-12.753384997999945],[45.18605050000008,-12.839496],[45.21328150000005,-12.87307549999997]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR","LEVL_CODE":0,"CNTR_CODE":"HR","NAME_LATN":"HRVATSKA","NUTS_NAME":"HRVATSKA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.585924309000063,45.238482153000064],[14.332406,45.355255],[14.226582500000063,45.15383849800003],[14.165939499000046,45.07278049800004],[14.169086500000049,44.979018996000036],[14.076002908000078,44.97715276000008],[13.979018,44.84097649800003],[14.003381,44.805259498000055],[13.906588,44.76863499800004],[13.763872,44.98577499600003],[13.606798,45.11697399700006],[13.60086749900006,45.24365249700003],[13.489867,45.486835496000026],[13.583067,45.47740999600006],[13.887097500000039,45.425872998000045],[13.984909500000072,45.45602199800004],[13.962175500000058,45.50601349600004],[14.011006500000065,45.51988049900007],[14.109059,45.48245849600005],[14.118196500000067,45.48123899600006],[14.436836499000037,45.51059299900004],[14.570012500000075,45.67294449600007],[14.823365500000023,45.46070999800003],[14.932349,45.529082],[15.226437500000031,45.42708799700006],[15.385532,45.48658049800008],[15.27705050000003,45.604461498000035],[15.406294,45.647968],[15.357104,45.71397099700005],[15.319616,45.674262498000076],[15.256749,45.718848999000045],[15.331290500000023,45.76285499800008],[15.404425,45.79271699600008],[15.708988500000032,45.84641299900005],[15.67752,45.86888799900004],[15.706404500000076,45.97534299800003],[15.732483,46.04452549900003],[15.627262,46.08595349800004],[15.59612450000003,46.14635599800005],[15.647185500000035,46.21717399700003],[15.775107500000047,46.211418998000056],[15.791475500000047,46.259327496000026],[15.876868500000057,46.28005549800008],[16.088382999000032,46.388544499000034],[16.301549,46.37828799600004],[16.24214850000004,46.49007549600003],[16.38205350000004,46.551329999000075],[16.596805,46.47590249800004],[16.854754999000022,46.35044099800007],[16.876043500000037,46.32060249600005],[16.97316550000005,46.22472749800005],[17.158216500000037,46.17029199900003],[17.294325,45.988544500000046],[17.560871,45.93783949900006],[17.651499,45.84783549900004],[17.911642,45.79095099600005],[18.446884,45.737048997000045],[18.623518,45.83937849800003],[18.659298,45.91699599800006],[18.821306,45.91438299600003],[18.889734,45.921180496000034],[18.84653650000007,45.815754998000045],[18.919455499000037,45.79011399600006],[18.854031500000076,45.767097498000055],[18.978496,45.739960997000026],[18.975786,45.65629249700004],[18.897893999000075,45.57232799900004],[19.099119,45.516377498000054],[19.033942,45.48682149800004],[18.994097,45.48548099800007],[19.004717,45.435031999000046],[19.029667500000073,45.40994349700003],[18.974392,45.37359549900003],[19.260654,45.242654998000035],[19.420648500000027,45.23342549800003],[19.425167500000043,45.167692498000065],[19.176552,45.202589498000066],[19.14172650000006,45.129974],[19.086456500000054,45.14539099800004],[19.102553500000056,45.01075999700004],[19.052091500000074,44.97677699900004],[19.156347,44.95319249700003],[18.998224,44.91545299900008],[19.022069500000043,44.855353998000055],[18.849769500000036,44.85454549600007],[18.764814500000057,44.904578996000055],[18.795906,44.99245049700005],[18.531997500000045,45.090629497000066],[18.256834,45.13940799900007],[18.120003,45.08023049800005],[18.009428,45.15173699800005],[17.849125,45.044250497000064],[17.669786500000043,45.133567998000046],[17.143591499000024,45.162589499000035],[16.919224,45.272827],[16.815445,45.184578499000054],[16.529669,45.22657399900004],[16.394388,45.112418999000056],[16.354521,45.003344998000046],[16.288761,44.99568949600007],[16.089756,45.10438149600003],[16.01497850000004,45.21757499800003],[15.836288500000023,45.22247699800005],[15.764636,45.16401650000006],[15.791166500000031,45.11510449800005],[15.741518,45.06027999600008],[15.783576,44.97471249800003],[15.736722,44.93563099800008],[15.793308500000023,44.846645496000065],[15.72873099900005,44.82531049800008],[15.781581,44.749927498000034],[15.89406,44.74935149800007],[16.031725,44.65331649900003],[16.058061500000065,44.60846699800004],[16.01869,44.56270199700003],[16.120580500000074,44.51544549700003],[16.13193699900006,44.45468150000005],[16.172439500000053,44.402801500000066],[16.131405,44.378067498000064],[16.218702500000063,44.34813699800003],[16.187046,44.29577999900005],[16.208290376000036,44.216682898000045],[16.544348,43.974178497000025],[16.811584500000038,43.75549299700003],[17.007986500000072,43.57624049900005],[17.28249950000003,43.46776199800007],[17.250774500000034,43.402832],[17.310936,43.29849949800007],[17.45084550000007,43.17686849900008],[17.475122500000055,43.16145699600003],[17.641218,43.08916450000004],[17.712072500000033,42.973003498000026],[17.581335,42.93842299800008],[17.355711,43.084621500000026],[16.886247500000025,43.40092849800004],[16.38828650000005,43.50912849800005],[16.475813,43.53484350000008],[16.347627500000044,43.55099849800007],[16.11359999900003,43.47080599600008],[16.013214,43.50250599900005],[15.951715500000034,43.50550449600007],[15.91692050000006,43.63210299800005],[15.975348500000052,43.63030649700005],[15.85398850000007,43.721671996000055],[15.906212499000048,43.72008899700006],[15.823885500000074,43.78556799900008],[15.87976750000007,43.72956399600008],[15.700071500000035,43.768661500000064],[15.558911500000022,43.871951996000064],[15.307637,44.03755199600005],[15.111249,44.26010149700005],[15.189659,44.23639699900008],[15.21089150000006,44.29221749800007],[15.291243500000064,44.24532699700006],[15.276153500000078,44.33257649700005],[15.598931351000033,44.172556788000065],[15.525706999000022,44.26912999800004],[15.289471500000047,44.36322799800007],[14.988486500000022,44.57393250000007],[14.896002,44.69689949600007],[14.882382500000062,44.84449450000005],[14.927815500000065,44.95193149600004],[14.881313500000033,45.03440499800007],[14.589105421000056,45.236414151000076],[14.58782,45.23892999900005],[14.585924309000063,45.238482153000064]],[[14.037782500000048,45.000358498000026],[14.074614229000076,44.97919341000005],[14.039097,45.03140999800007],[14.037782500000048,45.000358498000026]]],[[[14.585924309000063,45.238482153000064],[14.589105421000056,45.236414151000076],[14.663294,45.09139649600007],[14.820514500000058,44.97092799700005],[14.706944500000077,44.94189849900005],[14.617293500000073,44.97933949800006],[14.626157,45.03903199800004],[14.43812949900007,45.06486449800008],[14.525705500000072,45.12473699800006],[14.527404999000055,45.22465899800005],[14.585924309000063,45.238482153000064]]],[[[17.289999,43.00550099800006],[17.46978399900007,42.914893996000046],[17.74478350000004,42.834583],[17.64915650000006,42.88892349900004],[17.844461500000023,42.90235499800008],[17.891266,42.80977249600005],[18.244060500000046,42.60269949700006],[18.366398,42.616142498000045],[18.438104,42.555705],[18.43380049900003,42.48400849800004],[18.525213,42.420462996000026],[17.846662500000036,42.788524497000026],[17.696689500000048,42.83482749800004],[17.761528,42.761123498000075],[17.000639,43.046752998000045],[17.289999,43.00550099800006]]],[[[16.716209500000048,42.98745349700005],[17.047741,42.98026249700007],[17.202293500000053,42.91736999900007],[16.63859,42.92608249700004],[16.62772749900006,42.98522549900008],[16.716209500000048,42.98745349700005]]],[[[16.72083850000007,43.15782549900007],[17.19667050000004,43.125133496000046],[16.658453,43.121654498000055],[16.366495,43.19478999800003],[16.650303,43.21850599800007],[16.72083850000007,43.15782549900007]]],[[[16.440695,43.39253199600006],[16.899605,43.31423949900005],[16.85471350000006,43.271259497000074],[16.408132500000022,43.319767],[16.440695,43.39253199600006]]],[[[14.7325,44.70423899800005],[14.808002500000043,44.63653949900004],[15.080239500000062,44.47634499800006],[14.915559500000029,44.538463498000056],[14.976783500000067,44.503123998000035],[15.259307,44.32829649800004],[15.170937500000036,44.360900998000034],[15.23628350000007,44.31817649800007],[15.131373500000052,44.32906349700005],[15.158192499000052,44.29406350000005],[14.896632,44.52800349800003],[14.7325,44.70423899800005]]],[[[14.85711150000003,44.177299498000025],[15.051659500000028,44.00758749700003],[15.225797500000056,43.89610649800005],[15.129062500000032,43.92160799900006],[15.200639500000023,43.87714399600003],[15.161318,43.88573850000006],[14.81858550000004,44.15513999800004],[14.85711150000003,44.177299498000025]]],[[[14.360228500000062,45.08841299800008],[14.474727500000029,44.95611549800003],[14.463760500000035,44.72549799800004],[14.545010500000046,44.63052349800006],[14.51701550000007,44.60891699800004],[14.391382,44.69341299700005],[14.302028500000063,44.93503199600008],[14.390509500000064,44.90182899800004],[14.39368150000007,44.959632998000075],[14.272551500000077,45.120250498000075],[14.365512999000032,45.15925999900003],[14.360228500000062,45.08841299800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR0","LEVL_CODE":1,"CNTR_CODE":"HR","NAME_LATN":"HRVATSKA","NUTS_NAME":"HRVATSKA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.585924309000063,45.238482153000064],[14.332406,45.355255],[14.226582500000063,45.15383849800003],[14.165939499000046,45.07278049800004],[14.169086500000049,44.979018996000036],[14.076002908000078,44.97715276000008],[13.979018,44.84097649800003],[14.003381,44.805259498000055],[13.906588,44.76863499800004],[13.763872,44.98577499600003],[13.606798,45.11697399700006],[13.60086749900006,45.24365249700003],[13.489867,45.486835496000026],[13.583067,45.47740999600006],[13.887097500000039,45.425872998000045],[13.984909500000072,45.45602199800004],[13.962175500000058,45.50601349600004],[14.011006500000065,45.51988049900007],[14.109059,45.48245849600005],[14.118196500000067,45.48123899600006],[14.436836499000037,45.51059299900004],[14.570012500000075,45.67294449600007],[14.823365500000023,45.46070999800003],[14.932349,45.529082],[15.226437500000031,45.42708799700006],[15.385532,45.48658049800008],[15.27705050000003,45.604461498000035],[15.406294,45.647968],[15.357104,45.71397099700005],[15.319616,45.674262498000076],[15.256749,45.718848999000045],[15.331290500000023,45.76285499800008],[15.404425,45.79271699600008],[15.708988500000032,45.84641299900005],[15.67752,45.86888799900004],[15.706404500000076,45.97534299800003],[15.732483,46.04452549900003],[15.627262,46.08595349800004],[15.59612450000003,46.14635599800005],[15.647185500000035,46.21717399700003],[15.775107500000047,46.211418998000056],[15.791475500000047,46.259327496000026],[15.876868500000057,46.28005549800008],[16.088382999000032,46.388544499000034],[16.301549,46.37828799600004],[16.24214850000004,46.49007549600003],[16.38205350000004,46.551329999000075],[16.596805,46.47590249800004],[16.854754999000022,46.35044099800007],[16.876043500000037,46.32060249600005],[16.97316550000005,46.22472749800005],[17.158216500000037,46.17029199900003],[17.294325,45.988544500000046],[17.560871,45.93783949900006],[17.651499,45.84783549900004],[17.911642,45.79095099600005],[18.446884,45.737048997000045],[18.623518,45.83937849800003],[18.659298,45.91699599800006],[18.821306,45.91438299600003],[18.889734,45.921180496000034],[18.84653650000007,45.815754998000045],[18.919455499000037,45.79011399600006],[18.854031500000076,45.767097498000055],[18.978496,45.739960997000026],[18.975786,45.65629249700004],[18.897893999000075,45.57232799900004],[19.099119,45.516377498000054],[19.033942,45.48682149800004],[18.994097,45.48548099800007],[19.004717,45.435031999000046],[19.029667500000073,45.40994349700003],[18.974392,45.37359549900003],[19.260654,45.242654998000035],[19.420648500000027,45.23342549800003],[19.425167500000043,45.167692498000065],[19.176552,45.202589498000066],[19.14172650000006,45.129974],[19.086456500000054,45.14539099800004],[19.102553500000056,45.01075999700004],[19.052091500000074,44.97677699900004],[19.156347,44.95319249700003],[18.998224,44.91545299900008],[19.022069500000043,44.855353998000055],[18.849769500000036,44.85454549600007],[18.764814500000057,44.904578996000055],[18.795906,44.99245049700005],[18.531997500000045,45.090629497000066],[18.256834,45.13940799900007],[18.120003,45.08023049800005],[18.009428,45.15173699800005],[17.849125,45.044250497000064],[17.669786500000043,45.133567998000046],[17.143591499000024,45.162589499000035],[16.919224,45.272827],[16.815445,45.184578499000054],[16.529669,45.22657399900004],[16.394388,45.112418999000056],[16.354521,45.003344998000046],[16.288761,44.99568949600007],[16.089756,45.10438149600003],[16.01497850000004,45.21757499800003],[15.836288500000023,45.22247699800005],[15.764636,45.16401650000006],[15.791166500000031,45.11510449800005],[15.741518,45.06027999600008],[15.783576,44.97471249800003],[15.736722,44.93563099800008],[15.793308500000023,44.846645496000065],[15.72873099900005,44.82531049800008],[15.781581,44.749927498000034],[15.89406,44.74935149800007],[16.031725,44.65331649900003],[16.058061500000065,44.60846699800004],[16.01869,44.56270199700003],[16.120580500000074,44.51544549700003],[16.13193699900006,44.45468150000005],[16.172439500000053,44.402801500000066],[16.131405,44.378067498000064],[16.218702500000063,44.34813699800003],[16.187046,44.29577999900005],[16.208290376000036,44.216682898000045],[16.544348,43.974178497000025],[16.811584500000038,43.75549299700003],[17.007986500000072,43.57624049900005],[17.28249950000003,43.46776199800007],[17.250774500000034,43.402832],[17.310936,43.29849949800007],[17.45084550000007,43.17686849900008],[17.475122500000055,43.16145699600003],[17.641218,43.08916450000004],[17.712072500000033,42.973003498000026],[17.581335,42.93842299800008],[17.355711,43.084621500000026],[16.886247500000025,43.40092849800004],[16.38828650000005,43.50912849800005],[16.475813,43.53484350000008],[16.347627500000044,43.55099849800007],[16.11359999900003,43.47080599600008],[16.013214,43.50250599900005],[15.951715500000034,43.50550449600007],[15.91692050000006,43.63210299800005],[15.975348500000052,43.63030649700005],[15.85398850000007,43.721671996000055],[15.906212499000048,43.72008899700006],[15.823885500000074,43.78556799900008],[15.87976750000007,43.72956399600008],[15.700071500000035,43.768661500000064],[15.558911500000022,43.871951996000064],[15.307637,44.03755199600005],[15.111249,44.26010149700005],[15.189659,44.23639699900008],[15.21089150000006,44.29221749800007],[15.291243500000064,44.24532699700006],[15.276153500000078,44.33257649700005],[15.598931351000033,44.172556788000065],[15.525706999000022,44.26912999800004],[15.289471500000047,44.36322799800007],[14.988486500000022,44.57393250000007],[14.896002,44.69689949600007],[14.882382500000062,44.84449450000005],[14.927815500000065,44.95193149600004],[14.881313500000033,45.03440499800007],[14.589105421000056,45.236414151000076],[14.58782,45.23892999900005],[14.585924309000063,45.238482153000064]],[[14.037782500000048,45.000358498000026],[14.074614229000076,44.97919341000005],[14.039097,45.03140999800007],[14.037782500000048,45.000358498000026]]],[[[14.585924309000063,45.238482153000064],[14.589105421000056,45.236414151000076],[14.663294,45.09139649600007],[14.820514500000058,44.97092799700005],[14.706944500000077,44.94189849900005],[14.617293500000073,44.97933949800006],[14.626157,45.03903199800004],[14.43812949900007,45.06486449800008],[14.525705500000072,45.12473699800006],[14.527404999000055,45.22465899800005],[14.585924309000063,45.238482153000064]]],[[[17.289999,43.00550099800006],[17.46978399900007,42.914893996000046],[17.74478350000004,42.834583],[17.64915650000006,42.88892349900004],[17.844461500000023,42.90235499800008],[17.891266,42.80977249600005],[18.244060500000046,42.60269949700006],[18.366398,42.616142498000045],[18.438104,42.555705],[18.43380049900003,42.48400849800004],[18.525213,42.420462996000026],[17.846662500000036,42.788524497000026],[17.696689500000048,42.83482749800004],[17.761528,42.761123498000075],[17.000639,43.046752998000045],[17.289999,43.00550099800006]]],[[[16.716209500000048,42.98745349700005],[17.047741,42.98026249700007],[17.202293500000053,42.91736999900007],[16.63859,42.92608249700004],[16.62772749900006,42.98522549900008],[16.716209500000048,42.98745349700005]]],[[[16.72083850000007,43.15782549900007],[17.19667050000004,43.125133496000046],[16.658453,43.121654498000055],[16.366495,43.19478999800003],[16.650303,43.21850599800007],[16.72083850000007,43.15782549900007]]],[[[16.440695,43.39253199600006],[16.899605,43.31423949900005],[16.85471350000006,43.271259497000074],[16.408132500000022,43.319767],[16.440695,43.39253199600006]]],[[[14.7325,44.70423899800005],[14.808002500000043,44.63653949900004],[15.080239500000062,44.47634499800006],[14.915559500000029,44.538463498000056],[14.976783500000067,44.503123998000035],[15.259307,44.32829649800004],[15.170937500000036,44.360900998000034],[15.23628350000007,44.31817649800007],[15.131373500000052,44.32906349700005],[15.158192499000052,44.29406350000005],[14.896632,44.52800349800003],[14.7325,44.70423899800005]]],[[[14.85711150000003,44.177299498000025],[15.051659500000028,44.00758749700003],[15.225797500000056,43.89610649800005],[15.129062500000032,43.92160799900006],[15.200639500000023,43.87714399600003],[15.161318,43.88573850000006],[14.81858550000004,44.15513999800004],[14.85711150000003,44.177299498000025]]],[[[14.360228500000062,45.08841299800008],[14.474727500000029,44.95611549800003],[14.463760500000035,44.72549799800004],[14.545010500000046,44.63052349800006],[14.51701550000007,44.60891699800004],[14.391382,44.69341299700005],[14.302028500000063,44.93503199600008],[14.390509500000064,44.90182899800004],[14.39368150000007,44.959632998000075],[14.272551500000077,45.120250498000075],[14.365512999000032,45.15925999900003],[14.360228500000062,45.08841299800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR03","LEVL_CODE":2,"CNTR_CODE":"HR","NAME_LATN":"Jadranska Hrvatska","NUTS_NAME":"Jadranska Hrvatska"},"geometry":{"type":"MultiPolygon","coordinates":[[[[17.289999,43.00550099800006],[17.46978399900007,42.914893996000046],[17.74478350000004,42.834583],[17.64915650000006,42.88892349900004],[17.844461500000023,42.90235499800008],[17.891266,42.80977249600005],[18.244060500000046,42.60269949700006],[18.366398,42.616142498000045],[18.438104,42.555705],[18.43380049900003,42.48400849800004],[18.525213,42.420462996000026],[17.846662500000036,42.788524497000026],[17.696689500000048,42.83482749800004],[17.761528,42.761123498000075],[17.000639,43.046752998000045],[17.289999,43.00550099800006]]],[[[14.585924309000063,45.238482153000064],[14.332406,45.355255],[14.226582500000063,45.15383849800003],[14.165939499000046,45.07278049800004],[14.169086500000049,44.979018996000036],[14.076002908000078,44.97715276000008],[13.979018,44.84097649800003],[14.003381,44.805259498000055],[13.906588,44.76863499800004],[13.763872,44.98577499600003],[13.606798,45.11697399700006],[13.60086749900006,45.24365249700003],[13.489867,45.486835496000026],[13.583067,45.47740999600006],[13.887097500000039,45.425872998000045],[13.984909500000072,45.45602199800004],[13.962175500000058,45.50601349600004],[14.011006500000065,45.51988049900007],[14.109059,45.48245849600005],[14.118196500000067,45.48123899600006],[14.436836499000037,45.51059299900004],[14.570012500000075,45.67294449600007],[14.823365500000023,45.46070999800003],[14.932349,45.529082],[15.226437500000031,45.42708799700006],[15.177821,45.31204999900007],[15.10687450000006,45.278392996000036],[15.021124,45.32352449800004],[14.961522,45.273318998000036],[15.017143,45.12289049900005],[15.0589,45.111690500000066],[15.202997500000038,45.10612099800005],[15.560695499000076,44.88255299800005],[15.571555,44.92574699600004],[15.736722,44.93563099800008],[15.793308500000023,44.846645496000065],[15.72873099900005,44.82531049800008],[15.781581,44.749927498000034],[15.89406,44.74935149800007],[16.031725,44.65331649900003],[16.058061500000065,44.60846699800004],[16.01869,44.56270199700003],[16.120580500000074,44.51544549700003],[16.13193699900006,44.45468150000005],[16.172439500000053,44.402801500000066],[16.131405,44.378067498000064],[16.218702500000063,44.34813699800003],[16.187046,44.29577999900005],[16.208290376000036,44.216682898000045],[16.544348,43.974178497000025],[16.811584500000038,43.75549299700003],[17.007986500000072,43.57624049900005],[17.28249950000003,43.46776199800007],[17.250774500000034,43.402832],[17.310936,43.29849949800007],[17.45084550000007,43.17686849900008],[17.475122500000055,43.16145699600003],[17.641218,43.08916450000004],[17.712072500000033,42.973003498000026],[17.581335,42.93842299800008],[17.355711,43.084621500000026],[16.886247500000025,43.40092849800004],[16.38828650000005,43.50912849800005],[16.475813,43.53484350000008],[16.347627500000044,43.55099849800007],[16.11359999900003,43.47080599600008],[16.013214,43.50250599900005],[15.951715500000034,43.50550449600007],[15.91692050000006,43.63210299800005],[15.975348500000052,43.63030649700005],[15.85398850000007,43.721671996000055],[15.906212499000048,43.72008899700006],[15.823885500000074,43.78556799900008],[15.87976750000007,43.72956399600008],[15.700071500000035,43.768661500000064],[15.558911500000022,43.871951996000064],[15.307637,44.03755199600005],[15.111249,44.26010149700005],[15.189659,44.23639699900008],[15.21089150000006,44.29221749800007],[15.291243500000064,44.24532699700006],[15.276153500000078,44.33257649700005],[15.598931351000033,44.172556788000065],[15.525706999000022,44.26912999800004],[15.289471500000047,44.36322799800007],[14.988486500000022,44.57393250000007],[14.896002,44.69689949600007],[14.882382500000062,44.84449450000005],[14.927815500000065,44.95193149600004],[14.881313500000033,45.03440499800007],[14.589105421000056,45.236414151000076],[14.58782,45.23892999900005],[14.585924309000063,45.238482153000064]],[[14.037782500000048,45.000358498000026],[14.074614229000076,44.97919341000005],[14.039097,45.03140999800007],[14.037782500000048,45.000358498000026]]],[[[14.585924309000063,45.238482153000064],[14.589105421000056,45.236414151000076],[14.663294,45.09139649600007],[14.820514500000058,44.97092799700005],[14.706944500000077,44.94189849900005],[14.617293500000073,44.97933949800006],[14.626157,45.03903199800004],[14.43812949900007,45.06486449800008],[14.525705500000072,45.12473699800006],[14.527404999000055,45.22465899800005],[14.585924309000063,45.238482153000064]]],[[[16.716209500000048,42.98745349700005],[17.047741,42.98026249700007],[17.202293500000053,42.91736999900007],[16.63859,42.92608249700004],[16.62772749900006,42.98522549900008],[16.716209500000048,42.98745349700005]]],[[[16.72083850000007,43.15782549900007],[17.19667050000004,43.125133496000046],[16.658453,43.121654498000055],[16.366495,43.19478999800003],[16.650303,43.21850599800007],[16.72083850000007,43.15782549900007]]],[[[16.440695,43.39253199600006],[16.899605,43.31423949900005],[16.85471350000006,43.271259497000074],[16.408132500000022,43.319767],[16.440695,43.39253199600006]]],[[[14.7325,44.70423899800005],[14.808002500000043,44.63653949900004],[15.080239500000062,44.47634499800006],[14.915559500000029,44.538463498000056],[14.976783500000067,44.503123998000035],[15.259307,44.32829649800004],[15.170937500000036,44.360900998000034],[15.23628350000007,44.31817649800007],[15.131373500000052,44.32906349700005],[15.158192499000052,44.29406350000005],[14.896632,44.52800349800003],[14.7325,44.70423899800005]]],[[[14.85711150000003,44.177299498000025],[15.051659500000028,44.00758749700003],[15.225797500000056,43.89610649800005],[15.129062500000032,43.92160799900006],[15.200639500000023,43.87714399600003],[15.161318,43.88573850000006],[14.81858550000004,44.15513999800004],[14.85711150000003,44.177299498000025]]],[[[14.360228500000062,45.08841299800008],[14.474727500000029,44.95611549800003],[14.463760500000035,44.72549799800004],[14.545010500000046,44.63052349800006],[14.51701550000007,44.60891699800004],[14.391382,44.69341299700005],[14.302028500000063,44.93503199600008],[14.390509500000064,44.90182899800004],[14.39368150000007,44.959632998000075],[14.272551500000077,45.120250498000075],[14.365512999000032,45.15925999900003],[14.360228500000062,45.08841299800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR031","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Primorsko-goranska županija","NUTS_NAME":"Primorsko-goranska županija"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.585924309000063,45.238482153000064],[14.332406,45.355255],[14.226582500000063,45.15383849800003],[14.185978,45.30371099800004],[14.205359500000043,45.404300498000055],[14.109059,45.48245849600005],[14.118196500000067,45.48123899600006],[14.436836499000037,45.51059299900004],[14.570012500000075,45.67294449600007],[14.823365500000023,45.46070999800003],[14.932349,45.529082],[15.226437500000031,45.42708799700006],[15.177821,45.31204999900007],[15.10687450000006,45.278392996000036],[15.021124,45.32352449800004],[14.961522,45.273318998000036],[15.017143,45.12289049900005],[15.016236500000048,45.06866049800004],[14.881313500000033,45.03440499800007],[14.589105421000056,45.236414151000076],[14.58782,45.23892999900005],[14.585924309000063,45.238482153000064]]],[[[14.585924309000063,45.238482153000064],[14.589105421000056,45.236414151000076],[14.663294,45.09139649600007],[14.820514500000058,44.97092799700005],[14.706944500000077,44.94189849900005],[14.617293500000073,44.97933949800006],[14.626157,45.03903199800004],[14.43812949900007,45.06486449800008],[14.525705500000072,45.12473699800006],[14.527404999000055,45.22465899800005],[14.585924309000063,45.238482153000064]]],[[[14.360228500000062,45.08841299800008],[14.474727500000029,44.95611549800003],[14.463760500000035,44.72549799800004],[14.545010500000046,44.63052349800006],[14.51701550000007,44.60891699800004],[14.391382,44.69341299700005],[14.302028500000063,44.93503199600008],[14.390509500000064,44.90182899800004],[14.39368150000007,44.959632998000075],[14.272551500000077,45.120250498000075],[14.365512999000032,45.15925999900003],[14.360228500000062,45.08841299800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR032","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Licko-senjska županija","NUTS_NAME":"Licko-senjska županija"},"geometry":{"type":"Polygon","coordinates":[[[15.0589,45.111690500000066],[15.202997500000038,45.10612099800005],[15.560695499000076,44.88255299800005],[15.571555,44.92574699600004],[15.736722,44.93563099800008],[15.793308500000023,44.846645496000065],[15.72873099900005,44.82531049800008],[15.781581,44.749927498000034],[15.89406,44.74935149800007],[16.031725,44.65331649900003],[16.058061500000065,44.60846699800004],[16.01869,44.56270199700003],[16.120580500000074,44.51544549700003],[16.13193699900006,44.45468150000005],[15.932315,44.50256349800003],[15.781257500000038,44.428397999000026],[15.771088500000076,44.36004649600005],[15.828376,44.31777549900005],[15.782112,44.271228998000026],[15.320396500000072,44.41809849800006],[15.289471500000047,44.36322799800007],[14.988486500000022,44.57393250000007],[14.896002,44.69689949600007],[14.882382500000062,44.84449450000005],[14.927815500000065,44.95193149600004],[14.881313500000033,45.03440499800007],[15.016236500000048,45.06866049800004],[15.017143,45.12289049900005],[15.0589,45.111690500000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR033","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Zadarska županija","NUTS_NAME":"Zadarska županija"},"geometry":{"type":"MultiPolygon","coordinates":[[[[16.187046,44.29577999900005],[16.208290376000036,44.216682898000045],[16.13873650000005,44.21114749800006],[16.089935500000024,44.116866998000035],[15.963009,44.18878949900005],[15.834395500000028,44.07808699700007],[15.847681,44.00221249800006],[15.740998500000046,43.85746749700007],[15.558911500000022,43.871951996000064],[15.307637,44.03755199600005],[15.111249,44.26010149700005],[15.189659,44.23639699900008],[15.21089150000006,44.29221749800007],[15.291243500000064,44.24532699700006],[15.276153500000078,44.33257649700005],[15.598931351000033,44.172556788000065],[15.525706999000022,44.26912999800004],[15.289471500000047,44.36322799800007],[15.320396500000072,44.41809849800006],[15.782112,44.271228998000026],[15.828376,44.31777549900005],[15.771088500000076,44.36004649600005],[15.781257500000038,44.428397999000026],[15.932315,44.50256349800003],[16.13193699900006,44.45468150000005],[16.172439500000053,44.402801500000066],[16.131405,44.378067498000064],[16.218702500000063,44.34813699800003],[16.187046,44.29577999900005]]],[[[14.808002500000043,44.63653949900004],[15.080239500000062,44.47634499800006],[14.915559500000029,44.538463498000056],[14.976783500000067,44.503123998000035],[15.259307,44.32829649800004],[15.170937500000036,44.360900998000034],[15.23628350000007,44.31817649800007],[15.131373500000052,44.32906349700005],[15.158192499000052,44.29406350000005],[14.896632,44.52800349800003],[14.7325,44.70423899800005],[14.808002500000043,44.63653949900004]]],[[[15.051659500000028,44.00758749700003],[15.225797500000056,43.89610649800005],[15.129062500000032,43.92160799900006],[15.200639500000023,43.87714399600003],[15.161318,43.88573850000006],[14.81858550000004,44.15513999800004],[14.85711150000003,44.177299498000025],[15.051659500000028,44.00758749700003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR034","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Šibensko-kninska županija","NUTS_NAME":"Šibensko-kninska županija"},"geometry":{"type":"Polygon","coordinates":[[[16.208290376000036,44.216682898000045],[16.544348,43.974178497000025],[16.355537,43.92513299800004],[16.344303,43.878597496000054],[16.439642,43.813949498000056],[16.344028500000036,43.76786049900005],[16.33266450000002,43.68960549800005],[16.152869999000075,43.69390099700007],[16.046131,43.593909997000026],[16.06724950000006,43.512068999000064],[16.013214,43.50250599900005],[15.951715500000034,43.50550449600007],[15.91692050000006,43.63210299800005],[15.975348500000052,43.63030649700005],[15.85398850000007,43.721671996000055],[15.906212499000048,43.72008899700006],[15.823885500000074,43.78556799900008],[15.87976750000007,43.72956399600008],[15.700071500000035,43.768661500000064],[15.558911500000022,43.871951996000064],[15.740998500000046,43.85746749700007],[15.847681,44.00221249800006],[15.834395500000028,44.07808699700007],[15.963009,44.18878949900005],[16.089935500000024,44.116866998000035],[16.13873650000005,44.21114749800006],[16.208290376000036,44.216682898000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR035","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Splitsko-dalmatinska županija","NUTS_NAME":"Splitsko-dalmatinska županija"},"geometry":{"type":"MultiPolygon","coordinates":[[[[16.811584500000038,43.75549299700003],[17.007986500000072,43.57624049900005],[17.28249950000003,43.46776199800007],[17.250774500000034,43.402832],[17.310936,43.29849949800007],[17.45084550000007,43.17686849900008],[17.345957,43.15895099800008],[17.407087500000046,43.106509998000035],[17.355711,43.084621500000026],[16.886247500000025,43.40092849800004],[16.38828650000005,43.50912849800005],[16.475813,43.53484350000008],[16.347627500000044,43.55099849800007],[16.11359999900003,43.47080599600008],[16.013214,43.50250599900005],[16.06724950000006,43.512068999000064],[16.046131,43.593909997000026],[16.152869999000075,43.69390099700007],[16.33266450000002,43.68960549800005],[16.344028500000036,43.76786049900005],[16.439642,43.813949498000056],[16.344303,43.878597496000054],[16.355537,43.92513299800004],[16.544348,43.974178497000025],[16.811584500000038,43.75549299700003]]],[[[17.19667050000004,43.125133496000046],[16.658453,43.121654498000055],[16.366495,43.19478999800003],[16.650303,43.21850599800007],[16.72083850000007,43.15782549900007],[17.19667050000004,43.125133496000046]]],[[[16.899605,43.31423949900005],[16.85471350000006,43.271259497000074],[16.408132500000022,43.319767],[16.440695,43.39253199600006],[16.899605,43.31423949900005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR036","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Istarska županija","NUTS_NAME":"Istarska županija"},"geometry":{"type":"Polygon","coordinates":[[[14.109059,45.48245849600005],[14.205359500000043,45.404300498000055],[14.185978,45.30371099800004],[14.226582500000063,45.15383849800003],[14.165939499000046,45.07278049800004],[14.169086500000049,44.979018996000036],[14.076002908000078,44.97715276000008],[13.979018,44.84097649800003],[14.003381,44.805259498000055],[13.906588,44.76863499800004],[13.763872,44.98577499600003],[13.606798,45.11697399700006],[13.60086749900006,45.24365249700003],[13.489867,45.486835496000026],[13.583067,45.47740999600006],[13.887097500000039,45.425872998000045],[13.984909500000072,45.45602199800004],[13.962175500000058,45.50601349600004],[14.011006500000065,45.51988049900007],[14.109059,45.48245849600005]],[[14.037782500000048,45.000358498000026],[14.074614229000076,44.97919341000005],[14.039097,45.03140999800007],[14.037782500000048,45.000358498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR037","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Dubrovacko-neretvanska županija","NUTS_NAME":"Dubrovacko-neretvanska županija"},"geometry":{"type":"MultiPolygon","coordinates":[[[[17.46978399900007,42.914893996000046],[17.74478350000004,42.834583],[17.64915650000006,42.88892349900004],[17.844461500000023,42.90235499800008],[17.891266,42.80977249600005],[18.244060500000046,42.60269949700006],[18.366398,42.616142498000045],[18.438104,42.555705],[18.43380049900003,42.48400849800004],[18.525213,42.420462996000026],[17.846662500000036,42.788524497000026],[17.696689500000048,42.83482749800004],[17.761528,42.761123498000075],[17.000639,43.046752998000045],[17.289999,43.00550099800006],[17.46978399900007,42.914893996000046]]],[[[17.712072500000033,42.973003498000026],[17.581335,42.93842299800008],[17.355711,43.084621500000026],[17.407087500000046,43.106509998000035],[17.345957,43.15895099800008],[17.45084550000007,43.17686849900008],[17.475122500000055,43.16145699600003],[17.641218,43.08916450000004],[17.712072500000033,42.973003498000026]]],[[[17.047741,42.98026249700007],[17.202293500000053,42.91736999900007],[16.63859,42.92608249700004],[16.62772749900006,42.98522549900008],[16.716209500000048,42.98745349700005],[17.047741,42.98026249700007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04","LEVL_CODE":2,"CNTR_CODE":"HR","NAME_LATN":"Kontinentalna Hrvatska","NUTS_NAME":"Kontinentalna Hrvatska"},"geometry":{"type":"Polygon","coordinates":[[[16.596805,46.47590249800004],[16.854754999000022,46.35044099800007],[16.876043500000037,46.32060249600005],[16.97316550000005,46.22472749800005],[17.158216500000037,46.17029199900003],[17.294325,45.988544500000046],[17.560871,45.93783949900006],[17.651499,45.84783549900004],[17.911642,45.79095099600005],[18.446884,45.737048997000045],[18.623518,45.83937849800003],[18.659298,45.91699599800006],[18.821306,45.91438299600003],[18.889734,45.921180496000034],[18.84653650000007,45.815754998000045],[18.919455499000037,45.79011399600006],[18.854031500000076,45.767097498000055],[18.978496,45.739960997000026],[18.975786,45.65629249700004],[18.897893999000075,45.57232799900004],[19.099119,45.516377498000054],[19.033942,45.48682149800004],[18.994097,45.48548099800007],[19.004717,45.435031999000046],[19.029667500000073,45.40994349700003],[18.974392,45.37359549900003],[19.260654,45.242654998000035],[19.420648500000027,45.23342549800003],[19.425167500000043,45.167692498000065],[19.176552,45.202589498000066],[19.14172650000006,45.129974],[19.086456500000054,45.14539099800004],[19.102553500000056,45.01075999700004],[19.052091500000074,44.97677699900004],[19.156347,44.95319249700003],[18.998224,44.91545299900008],[19.022069500000043,44.855353998000055],[18.849769500000036,44.85454549600007],[18.764814500000057,44.904578996000055],[18.795906,44.99245049700005],[18.531997500000045,45.090629497000066],[18.256834,45.13940799900007],[18.120003,45.08023049800005],[18.009428,45.15173699800005],[17.849125,45.044250497000064],[17.669786500000043,45.133567998000046],[17.143591499000024,45.162589499000035],[16.919224,45.272827],[16.815445,45.184578499000054],[16.529669,45.22657399900004],[16.394388,45.112418999000056],[16.354521,45.003344998000046],[16.288761,44.99568949600007],[16.089756,45.10438149600003],[16.01497850000004,45.21757499800003],[15.836288500000023,45.22247699800005],[15.764636,45.16401650000006],[15.791166500000031,45.11510449800005],[15.741518,45.06027999600008],[15.783576,44.97471249800003],[15.736722,44.93563099800008],[15.571555,44.92574699600004],[15.560695499000076,44.88255299800005],[15.202997500000038,45.10612099800005],[15.0589,45.111690500000066],[15.017143,45.12289049900005],[14.961522,45.273318998000036],[15.021124,45.32352449800004],[15.10687450000006,45.278392996000036],[15.177821,45.31204999900007],[15.226437500000031,45.42708799700006],[15.385532,45.48658049800008],[15.27705050000003,45.604461498000035],[15.406294,45.647968],[15.357104,45.71397099700005],[15.319616,45.674262498000076],[15.256749,45.718848999000045],[15.331290500000023,45.76285499800008],[15.404425,45.79271699600008],[15.708988500000032,45.84641299900005],[15.67752,45.86888799900004],[15.706404500000076,45.97534299800003],[15.732483,46.04452549900003],[15.627262,46.08595349800004],[15.59612450000003,46.14635599800005],[15.647185500000035,46.21717399700003],[15.775107500000047,46.211418998000056],[15.791475500000047,46.259327496000026],[15.876868500000057,46.28005549800008],[16.088382999000032,46.388544499000034],[16.301549,46.37828799600004],[16.24214850000004,46.49007549600003],[16.38205350000004,46.551329999000075],[16.596805,46.47590249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR041","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Grad Zagreb","NUTS_NAME":"Grad Zagreb"},"geometry":{"type":"Polygon","coordinates":[[[15.957638,45.90189749600006],[16.155048500000078,45.96892949900007],[16.222990500000037,45.84868599600003],[16.200516,45.77932749800004],[16.015934,45.743488497000044],[15.91854950000004,45.61451699600008],[15.777105500000062,45.68995649900006],[15.885401500000057,45.75075149800006],[15.822393500000032,45.839157],[15.957638,45.90189749600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR042","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Zagrebacka županija","NUTS_NAME":"Zagrebacka županija"},"geometry":{"type":"Polygon","coordinates":[[[16.34169950000006,46.02716049800006],[16.701584,45.92182149900003],[16.680895,45.822075],[16.495064,45.774017496000056],[16.609901500000035,45.669875998000066],[16.467722,45.57859799900007],[16.341448,45.646495998000034],[16.24377450000003,45.59643949800005],[16.15187650000007,45.636108499000045],[16.066921500000035,45.56907649800007],[16.100429500000075,45.511123499000064],[15.922427,45.478004498000075],[15.694810500000074,45.60323649900005],[15.568013,45.58911899900005],[15.331290500000023,45.76285499800008],[15.404425,45.79271699600008],[15.708988500000032,45.84641299900005],[15.67752,45.86888799900004],[15.706404500000076,45.97534299800003],[15.821916500000043,45.988593998000056],[15.957638,45.90189749600006],[15.822393500000032,45.839157],[15.885401500000057,45.75075149800006],[15.777105500000062,45.68995649900006],[15.91854950000004,45.61451699600008],[16.015934,45.743488497000044],[16.200516,45.77932749800004],[16.222990500000037,45.84868599600003],[16.155048500000078,45.96892949900007],[16.251993,46.069504],[16.34169950000006,46.02716049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR043","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Krapinsko-zagorska županija","NUTS_NAME":"Krapinsko-zagorska županija"},"geometry":{"type":"Polygon","coordinates":[[[16.251993,46.069504],[16.155048500000078,45.96892949900007],[15.957638,45.90189749600006],[15.821916500000043,45.988593998000056],[15.706404500000076,45.97534299800003],[15.732483,46.04452549900003],[15.627262,46.08595349800004],[15.59612450000003,46.14635599800005],[15.647185500000035,46.21717399700003],[15.775107500000047,46.211418998000056],[15.791475500000047,46.259327496000026],[15.876868500000057,46.28005549800008],[15.995996500000047,46.18605049800004],[16.247850499000037,46.19055549800004],[16.251993,46.069504]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR044","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Varaždinska županija","NUTS_NAME":"Varaždinska županija"},"geometry":{"type":"Polygon","coordinates":[[[16.301549,46.37828799600004],[16.380291,46.31344599700003],[16.773952500000064,46.303722496000034],[16.675796500000047,46.202312498000026],[16.375261500000022,46.12712849800005],[16.34169950000006,46.02716049800006],[16.251993,46.069504],[16.247850499000037,46.19055549800004],[15.995996500000047,46.18605049800004],[15.876868500000057,46.28005549800008],[16.088382999000032,46.388544499000034],[16.301549,46.37828799600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR045","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Koprivnicko-križevacka županija","NUTS_NAME":"Koprivnicko-križevacka županija"},"geometry":{"type":"Polygon","coordinates":[[[16.876043500000037,46.32060249600005],[16.97316550000005,46.22472749800005],[17.158216500000037,46.17029199900003],[17.294325,45.988544500000046],[17.190119,45.977813498000046],[17.12116049900004,45.88227099800008],[16.986143,45.932105998000054],[16.867464,46.08370199800004],[16.722269,46.040127496000025],[16.701584,45.92182149900003],[16.34169950000006,46.02716049800006],[16.375261500000022,46.12712849800005],[16.675796500000047,46.202312498000026],[16.773952500000064,46.303722496000034],[16.854754999000022,46.35044099800007],[16.876043500000037,46.32060249600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR046","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Medimurska županija","NUTS_NAME":"Medimurska županija"},"geometry":{"type":"Polygon","coordinates":[[[16.596805,46.47590249800004],[16.854754999000022,46.35044099800007],[16.773952500000064,46.303722496000034],[16.380291,46.31344599700003],[16.301549,46.37828799600004],[16.24214850000004,46.49007549600003],[16.38205350000004,46.551329999000075],[16.596805,46.47590249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR047","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Bjelovarsko-bilogorska županija","NUTS_NAME":"Bjelovarsko-bilogorska županija"},"geometry":{"type":"Polygon","coordinates":[[[17.12116049900004,45.88227099800008],[17.385662,45.701205998000034],[17.487211500000058,45.704150997000056],[17.440700500000048,45.57588599800005],[17.414487500000064,45.50844199800008],[16.952934500000026,45.54412849900007],[16.930092,45.48154850000003],[16.793823,45.62419149800007],[16.609901500000035,45.669875998000066],[16.495064,45.774017496000056],[16.680895,45.822075],[16.701584,45.92182149900003],[16.722269,46.040127496000025],[16.867464,46.08370199800004],[16.986143,45.932105998000054],[17.12116049900004,45.88227099800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR048","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Viroviticko-podravska županija","NUTS_NAME":"Viroviticko-podravska županija"},"geometry":{"type":"Polygon","coordinates":[[[17.651499,45.84783549900004],[17.911642,45.79095099600005],[18.04769150000004,45.634982999000044],[17.88551350000006,45.47422849900005],[17.440700500000048,45.57588599800005],[17.487211500000058,45.704150997000056],[17.385662,45.701205998000034],[17.12116049900004,45.88227099800008],[17.190119,45.977813498000046],[17.294325,45.988544500000046],[17.560871,45.93783949900006],[17.651499,45.84783549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR049","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Požeško-slavonska županija","NUTS_NAME":"Požeško-slavonska županija"},"geometry":{"type":"Polygon","coordinates":[[[17.88551350000006,45.47422849900005],[18.091774,45.402255998000044],[18.114149,45.305732498000054],[18.071609500000022,45.26601399600003],[17.893511,45.28570549600005],[17.759294500000067,45.182330998000054],[17.699537500000076,45.20259099900005],[17.694099,45.27048099600006],[17.567526,45.28169999700003],[17.404405500000053,45.39172349800003],[17.17596049900004,45.34859849800006],[16.930092,45.48154850000003],[16.952934500000026,45.54412849900007],[17.414487500000064,45.50844199800008],[17.440700500000048,45.57588599800005],[17.88551350000006,45.47422849900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04A","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Brodsko-posavska županija","NUTS_NAME":"Brodsko-posavska županija"},"geometry":{"type":"Polygon","coordinates":[[[18.071609500000022,45.26601399600003],[18.53625495800003,45.22948123600003],[18.574276,45.22213749700006],[18.494813500000078,45.12485499700006],[18.531997500000045,45.090629497000066],[18.256834,45.13940799900007],[18.120003,45.08023049800005],[18.009428,45.15173699800005],[17.849125,45.044250497000064],[17.669786500000043,45.133567998000046],[17.143591499000024,45.162589499000035],[17.07399,45.217086996000035],[17.17596049900004,45.34859849800006],[17.404405500000053,45.39172349800003],[17.567526,45.28169999700003],[17.694099,45.27048099600006],[17.699537500000076,45.20259099900005],[17.759294500000067,45.182330998000054],[17.893511,45.28570549600005],[18.071609500000022,45.26601399600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04B","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Osjecko-baranjska županija","NUTS_NAME":"Osjecko-baranjska županija"},"geometry":{"type":"Polygon","coordinates":[[[19.033942,45.48682149800004],[18.994097,45.48548099800007],[19.004717,45.435031999000046],[18.84808349900004,45.48161699700006],[18.643877,45.39300899600005],[18.63088250000004,45.31471649800005],[18.510113,45.29018799800008],[18.53625495800003,45.22948123600003],[18.071609500000022,45.26601399600003],[18.114149,45.305732498000054],[18.091774,45.402255998000044],[17.88551350000006,45.47422849900005],[18.04769150000004,45.634982999000044],[17.911642,45.79095099600005],[18.446884,45.737048997000045],[18.623518,45.83937849800003],[18.659298,45.91699599800006],[18.821306,45.91438299600003],[18.889734,45.921180496000034],[18.84653650000007,45.815754998000045],[18.919455499000037,45.79011399600006],[18.854031500000076,45.767097498000055],[18.978496,45.739960997000026],[18.975786,45.65629249700004],[18.897893999000075,45.57232799900004],[19.099119,45.516377498000054],[19.033942,45.48682149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04C","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Vukovarsko-srijemska županija","NUTS_NAME":"Vukovarsko-srijemska županija"},"geometry":{"type":"Polygon","coordinates":[[[19.004717,45.435031999000046],[19.029667500000073,45.40994349700003],[18.974392,45.37359549900003],[19.260654,45.242654998000035],[19.420648500000027,45.23342549800003],[19.425167500000043,45.167692498000065],[19.176552,45.202589498000066],[19.14172650000006,45.129974],[19.086456500000054,45.14539099800004],[19.102553500000056,45.01075999700004],[19.052091500000074,44.97677699900004],[19.156347,44.95319249700003],[18.998224,44.91545299900008],[19.022069500000043,44.855353998000055],[18.849769500000036,44.85454549600007],[18.764814500000057,44.904578996000055],[18.795906,44.99245049700005],[18.531997500000045,45.090629497000066],[18.494813500000078,45.12485499700006],[18.574276,45.22213749700006],[18.53625495800003,45.22948123600003],[18.510113,45.29018799800008],[18.63088250000004,45.31471649800005],[18.643877,45.39300899600005],[18.84808349900004,45.48161699700006],[19.004717,45.435031999000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04D","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Karlovacka županija","NUTS_NAME":"Karlovacka županija"},"geometry":{"type":"Polygon","coordinates":[[[15.922427,45.478004498000075],[15.831122500000049,45.49429249800005],[15.84312650000004,45.42507549900006],[15.792476499000031,45.37987899800004],[15.784621,45.26990899700007],[15.836288500000023,45.22247699800005],[15.764636,45.16401650000006],[15.791166500000031,45.11510449800005],[15.741518,45.06027999600008],[15.783576,44.97471249800003],[15.736722,44.93563099800008],[15.571555,44.92574699600004],[15.560695499000076,44.88255299800005],[15.202997500000038,45.10612099800005],[15.0589,45.111690500000066],[15.017143,45.12289049900005],[14.961522,45.273318998000036],[15.021124,45.32352449800004],[15.10687450000006,45.278392996000036],[15.177821,45.31204999900007],[15.226437500000031,45.42708799700006],[15.385532,45.48658049800008],[15.27705050000003,45.604461498000035],[15.406294,45.647968],[15.357104,45.71397099700005],[15.319616,45.674262498000076],[15.256749,45.718848999000045],[15.331290500000023,45.76285499800008],[15.568013,45.58911899900005],[15.694810500000074,45.60323649900005],[15.922427,45.478004498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"HR04E","LEVL_CODE":3,"CNTR_CODE":"HR","NAME_LATN":"Sisacko-moslavacka županija","NUTS_NAME":"Sisacko-moslavacka županija"},"geometry":{"type":"Polygon","coordinates":[[[16.930092,45.48154850000003],[17.17596049900004,45.34859849800006],[17.07399,45.217086996000035],[17.143591499000024,45.162589499000035],[16.919224,45.272827],[16.815445,45.184578499000054],[16.529669,45.22657399900004],[16.394388,45.112418999000056],[16.354521,45.003344998000046],[16.288761,44.99568949600007],[16.089756,45.10438149600003],[16.01497850000004,45.21757499800003],[15.836288500000023,45.22247699800005],[15.784621,45.26990899700007],[15.792476499000031,45.37987899800004],[15.84312650000004,45.42507549900006],[15.831122500000049,45.49429249800005],[15.922427,45.478004498000075],[16.100429500000075,45.511123499000064],[16.066921500000035,45.56907649800007],[16.15187650000007,45.636108499000045],[16.24377450000003,45.59643949800005],[16.341448,45.646495998000034],[16.467722,45.57859799900007],[16.609901500000035,45.669875998000066],[16.793823,45.62419149800007],[16.930092,45.48154850000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU","LEVL_CODE":0,"CNTR_CODE":"HU","NAME_LATN":"MAGYARORSZÁG","NUTS_NAME":"MAGYARORSZÁG"},"geometry":{"type":"Polygon","coordinates":[[[22.155306,48.40339649800006],[22.26507,48.40991949800008],[22.240338999000073,48.386390498000026],[22.384609,48.23410049600005],[22.515094500000032,48.238057997000055],[22.590280500000063,48.107631498000046],[22.830129500000055,48.11207949700008],[22.881429500000024,48.05487049800007],[22.836399,47.989948499000036],[22.89627050000007,47.954120500000045],[22.680243999000027,47.78774099800006],[22.31726,47.76592199600003],[22.180837500000052,47.60009449800003],[22.12832,47.59808949600006],[22.012631,47.50135449800007],[22.011991,47.37579649800006],[21.938521500000036,47.37301099800004],[21.792838500000073,47.10724299900005],[21.658955,47.02213149700003],[21.60309650000005,46.86840799600003],[21.483475500000054,46.76463549700003],[21.52933950000005,46.72074199900004],[21.43006950000006,46.693812998000055],[21.441398,46.651466999000036],[21.314121999000065,46.617556],[21.260401500000057,46.501945996000075],[21.317476,46.45065599600008],[21.296291,46.40669299800004],[21.206304,46.40323099600005],[21.103170500000033,46.26259049600003],[20.7756,46.27590999800003],[20.70530350000007,46.16093749800007],[20.6406,46.12720699800008],[20.501525,46.18993599800007],[20.264296,46.12637349900007],[19.86539,46.15033449800006],[19.698099,46.18793099800007],[19.567619500000035,46.17871849800008],[19.30270950000005,45.99155049900003],[19.133789,46.03707099700006],[19.008840500000076,45.92330949800004],[18.889734,45.921180496000034],[18.821306,45.91438299600003],[18.659298,45.91699599800006],[18.623518,45.83937849800003],[18.446884,45.737048997000045],[17.911642,45.79095099600005],[17.651499,45.84783549900004],[17.560871,45.93783949900006],[17.294325,45.988544500000046],[17.158216500000037,46.17029199900003],[16.97316550000005,46.22472749800005],[16.876043500000037,46.32060249600005],[16.854754999000022,46.35044099800007],[16.596805,46.47590249800004],[16.39033150000006,46.63820750000008],[16.42654550000003,46.69493699800006],[16.370793500000048,46.72224349800007],[16.31216150000006,46.778002498000035],[16.347822500000063,46.84051750000003],[16.291385500000047,46.87283350000007],[16.113849,46.86906799800005],[16.302262,46.99842649800007],[16.508267500000045,47.00125599800003],[16.438882,47.02970299800006],[16.520565500000032,47.05618149800006],[16.46332650000005,47.094956498000045],[16.529331,47.12723199800007],[16.419486500000062,47.19491549800006],[16.483743,47.28759999600004],[16.43376150000006,47.352918499000054],[16.445455,47.40720949700005],[16.64622,47.446596999000064],[16.71450550000003,47.539630998000064],[16.652076,47.622903499000074],[16.421846,47.66470449800005],[16.61228950000003,47.76022649600003],[16.749031,47.68141649700004],[17.093074,47.70823599900007],[17.07445350000006,47.80825550000003],[17.00994950000006,47.858343998000066],[17.085187500000075,47.87443749800008],[17.09466150000003,47.970871498000065],[17.160797500000058,48.00665650000008],[17.24742750000007,48.012008998000056],[17.705436500000076,47.758992498000055],[17.893923,47.73945699800004],[18.65544150000005,47.76008150000007],[18.848478,47.818228],[18.768924,47.868569498000056],[18.754815500000063,47.97508249800006],[18.928392,48.05683249800006],[19.01432250000005,48.077736498000036],[19.451139,48.094441998000036],[19.52604,48.203133997000066],[19.644519,48.239175499000055],[19.899319,48.124797998000076],[20.051879,48.16770399600006],[20.133181500000035,48.253985998000076],[20.325701500000037,48.27279849800004],[20.463937,48.463966999000036],[20.506496,48.53441350000003],[20.850424,48.581631499000025],[21.11743750000005,48.49105349700005],[21.440056,48.58523299800004],[21.61392150000006,48.50941299900006],[21.721957,48.35105049800006],[22.12107750000007,48.37831149800007],[22.155306,48.40339649800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU1","LEVL_CODE":1,"CNTR_CODE":"HU","NAME_LATN":"KÖZÉP-MAGYARORSZÁG","NUTS_NAME":"KÖZÉP-MAGYARORSZÁG"},"geometry":{"type":"Polygon","coordinates":[[[19.42492050000004,47.81662999800005],[19.477327,47.687782997000056],[19.570937,47.7349],[19.666329,47.58855],[19.652673,47.54783799900008],[19.789935,47.53137099700007],[19.74696350000005,47.488931498000056],[19.990462500000035,47.346628],[20.017585,47.303893998000035],[19.988269500000058,47.25306149800008],[20.098951500000055,47.18146399900007],[20.047241,47.12519599900003],[20.094639,47.007426499000076],[19.779767500000048,46.94870449800004],[19.587165500000026,47.120925499000066],[19.319190500000047,47.012678],[19.29528650000003,47.03592250000003],[19.347792500000025,47.09322199700006],[19.23677,47.13080099700005],[18.96590750000007,47.02896499900004],[18.873017,47.14370549900008],[18.919185,47.27414199900005],[18.757157,47.40332599800007],[18.785584,47.456860498000026],[18.68843350000003,47.577067498000076],[18.77766450000007,47.64124849800004],[18.747727,47.66114299800006],[18.864824499000065,47.651052496000034],[18.854260500000066,47.71470099800007],[18.94147650000002,47.72467349800007],[18.848478,47.818228],[18.768924,47.868569498000056],[18.754815500000063,47.97508249800006],[18.928392,48.05683249800006],[19.006969,48.010337496000034],[18.942689,47.94399499600007],[19.08613450000007,47.83817049800007],[19.42492050000004,47.81662999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU10","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Közép-Magyarország","NUTS_NAME":"Közép-Magyarország"},"geometry":{"type":"Polygon","coordinates":[[[19.42492050000004,47.81662999800005],[19.477327,47.687782997000056],[19.570937,47.7349],[19.666329,47.58855],[19.652673,47.54783799900008],[19.789935,47.53137099700007],[19.74696350000005,47.488931498000056],[19.990462500000035,47.346628],[20.017585,47.303893998000035],[19.988269500000058,47.25306149800008],[20.098951500000055,47.18146399900007],[20.047241,47.12519599900003],[20.094639,47.007426499000076],[19.779767500000048,46.94870449800004],[19.587165500000026,47.120925499000066],[19.319190500000047,47.012678],[19.29528650000003,47.03592250000003],[19.347792500000025,47.09322199700006],[19.23677,47.13080099700005],[18.96590750000007,47.02896499900004],[18.873017,47.14370549900008],[18.919185,47.27414199900005],[18.757157,47.40332599800007],[18.785584,47.456860498000026],[18.68843350000003,47.577067498000076],[18.77766450000007,47.64124849800004],[18.747727,47.66114299800006],[18.864824499000065,47.651052496000034],[18.854260500000066,47.71470099800007],[18.94147650000002,47.72467349800007],[18.848478,47.818228],[18.768924,47.868569498000056],[18.754815500000063,47.97508249800006],[18.928392,48.05683249800006],[19.006969,48.010337496000034],[18.942689,47.94399499600007],[19.08613450000007,47.83817049800007],[19.42492050000004,47.81662999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU101","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Budapest","NUTS_NAME":"Budapest"},"geometry":{"type":"Polygon","coordinates":[[[19.10880399900003,47.610265999000035],[19.334497500000055,47.46318799800008],[19.140451,47.349436997000055],[19.08353550000004,47.40484399800005],[18.976957500000026,47.38262450000008],[18.929383,47.57180249600003],[19.10880399900003,47.610265999000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU102","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Pest","NUTS_NAME":"Pest"},"geometry":{"type":"Polygon","coordinates":[[[19.570937,47.7349],[19.666329,47.58855],[19.652673,47.54783799900008],[19.789935,47.53137099700007],[19.74696350000005,47.488931498000056],[19.990462500000035,47.346628],[20.017585,47.303893998000035],[19.988269500000058,47.25306149800008],[20.098951500000055,47.18146399900007],[20.047241,47.12519599900003],[20.094639,47.007426499000076],[19.779767500000048,46.94870449800004],[19.587165500000026,47.120925499000066],[19.319190500000047,47.012678],[19.29528650000003,47.03592250000003],[19.347792500000025,47.09322199700006],[19.23677,47.13080099700005],[18.96590750000007,47.02896499900004],[18.873017,47.14370549900008],[18.919185,47.27414199900005],[18.757157,47.40332599800007],[18.785584,47.456860498000026],[18.68843350000003,47.577067498000076],[18.77766450000007,47.64124849800004],[18.747727,47.66114299800006],[18.864824499000065,47.651052496000034],[18.854260500000066,47.71470099800007],[18.94147650000002,47.72467349800007],[18.848478,47.818228],[18.768924,47.868569498000056],[18.754815500000063,47.97508249800006],[18.928392,48.05683249800006],[19.006969,48.010337496000034],[18.942689,47.94399499600007],[19.08613450000007,47.83817049800007],[19.42492050000004,47.81662999800005],[19.477327,47.687782997000056],[19.570937,47.7349]],[[19.10880399900003,47.610265999000035],[18.929383,47.57180249600003],[18.976957500000026,47.38262450000008],[19.08353550000004,47.40484399800005],[19.140451,47.349436997000055],[19.334497500000055,47.46318799800008],[19.10880399900003,47.610265999000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU2","LEVL_CODE":1,"CNTR_CODE":"HU","NAME_LATN":"DUNÁNTÚL","NUTS_NAME":"DUNÁNTÚL"},"geometry":{"type":"Polygon","coordinates":[[[17.705436500000076,47.758992498000055],[17.893923,47.73945699800004],[18.65544150000005,47.76008150000007],[18.848478,47.818228],[18.94147650000002,47.72467349800007],[18.854260500000066,47.71470099800007],[18.864824499000065,47.651052496000034],[18.747727,47.66114299800006],[18.77766450000007,47.64124849800004],[18.68843350000003,47.577067498000076],[18.785584,47.456860498000026],[18.757157,47.40332599800007],[18.919185,47.27414199900005],[18.873017,47.14370549900008],[18.96590750000007,47.02896499900004],[18.925097,46.85716850000006],[19.00624350000004,46.70492349600005],[18.862301,46.61581350000006],[18.923924500000055,46.463970998000036],[18.885012,46.39557699900007],[18.921786,46.30806449700003],[18.82207550000004,46.205366998000045],[18.85447,46.19261],[18.802291,46.10876499800003],[18.73278650000003,46.04514199800008],[18.868295,46.02080049800003],[18.821306,45.91438299600003],[18.659298,45.91699599800006],[18.623518,45.83937849800003],[18.446884,45.737048997000045],[17.911642,45.79095099600005],[17.651499,45.84783549900004],[17.560871,45.93783949900006],[17.294325,45.988544500000046],[17.158216500000037,46.17029199900003],[16.97316550000005,46.22472749800005],[16.876043500000037,46.32060249600005],[16.854754999000022,46.35044099800007],[16.596805,46.47590249800004],[16.39033150000006,46.63820750000008],[16.42654550000003,46.69493699800006],[16.370793500000048,46.72224349800007],[16.31216150000006,46.778002498000035],[16.347822500000063,46.84051750000003],[16.291385500000047,46.87283350000007],[16.113849,46.86906799800005],[16.302262,46.99842649800007],[16.508267500000045,47.00125599800003],[16.438882,47.02970299800006],[16.520565500000032,47.05618149800006],[16.46332650000005,47.094956498000045],[16.529331,47.12723199800007],[16.419486500000062,47.19491549800006],[16.483743,47.28759999600004],[16.43376150000006,47.352918499000054],[16.445455,47.40720949700005],[16.64622,47.446596999000064],[16.71450550000003,47.539630998000064],[16.652076,47.622903499000074],[16.421846,47.66470449800005],[16.61228950000003,47.76022649600003],[16.749031,47.68141649700004],[17.093074,47.70823599900007],[17.07445350000006,47.80825550000003],[17.00994950000006,47.858343998000066],[17.085187500000075,47.87443749800008],[17.09466150000003,47.970871498000065],[17.160797500000058,48.00665650000008],[17.24742750000007,48.012008998000056],[17.705436500000076,47.758992498000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU21","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Közép-Dunántúl","NUTS_NAME":"Közép-Dunántúl"},"geometry":{"type":"Polygon","coordinates":[[[18.854260500000066,47.71470099800007],[18.864824499000065,47.651052496000034],[18.747727,47.66114299800006],[18.77766450000007,47.64124849800004],[18.68843350000003,47.577067498000076],[18.785584,47.456860498000026],[18.757157,47.40332599800007],[18.919185,47.27414199900005],[18.873017,47.14370549900008],[18.96590750000007,47.02896499900004],[18.925097,46.85716850000006],[18.636275,46.68721749800005],[18.55658550000004,46.78446449900008],[18.462384,46.73966549800008],[18.408736500000032,46.803397999000026],[18.210897500000044,46.77816149900008],[18.174225500000034,46.946213999000065],[18.20524850000004,46.971633],[17.41906750000004,46.75065649600003],[17.325419500000066,46.828989499000045],[17.321602,46.893327998000075],[17.23051950000007,46.919913499000074],[17.220904,46.99529849800007],[17.074528500000042,47.04871349600006],[17.216165,47.083856498000046],[17.198132500000042,47.26408749800004],[17.258762500000046,47.343398498000056],[17.17239150000006,47.43248399700008],[17.737528,47.47290749600006],[17.787135,47.40506799800005],[17.71753250000006,47.34090199800005],[17.765227500000037,47.27287850000005],[17.883888,47.38958699700004],[17.884379500000023,47.54962599900006],[17.933129,47.61619699800008],[17.88704050000007,47.63763449800007],[17.893923,47.73945699800004],[18.65544150000005,47.76008150000007],[18.848478,47.818228],[18.94147650000002,47.72467349800007],[18.854260500000066,47.71470099800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU211","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Fejér","NUTS_NAME":"Fejér"},"geometry":{"type":"Polygon","coordinates":[[[18.96590750000007,47.02896499900004],[18.925097,46.85716850000006],[18.636275,46.68721749800005],[18.55658550000004,46.78446449900008],[18.462384,46.73966549800008],[18.408736500000032,46.803397999000026],[18.210897500000044,46.77816149900008],[18.174225500000034,46.946213999000065],[18.20524850000004,46.971633],[18.224334999000064,47.213434998000025],[18.05424750000003,47.278282499000056],[18.035719,47.33434199700008],[18.194863,47.46747049900006],[18.311649500000044,47.39905999700005],[18.506083500000045,47.49796499800004],[18.488542,47.55465699700005],[18.68843350000003,47.577067498000076],[18.785584,47.456860498000026],[18.757157,47.40332599800007],[18.919185,47.27414199900005],[18.873017,47.14370549900008],[18.96590750000007,47.02896499900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU212","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Komárom-Esztergom","NUTS_NAME":"Komárom-Esztergom"},"geometry":{"type":"Polygon","coordinates":[[[18.848478,47.818228],[18.94147650000002,47.72467349800007],[18.854260500000066,47.71470099800007],[18.864824499000065,47.651052496000034],[18.747727,47.66114299800006],[18.77766450000007,47.64124849800004],[18.68843350000003,47.577067498000076],[18.488542,47.55465699700005],[18.506083500000045,47.49796499800004],[18.311649500000044,47.39905999700005],[18.194863,47.46747049900006],[18.035719,47.33434199700008],[17.883888,47.38958699700004],[17.884379500000023,47.54962599900006],[17.933129,47.61619699800008],[17.88704050000007,47.63763449800007],[17.893923,47.73945699800004],[18.65544150000005,47.76008150000007],[18.848478,47.818228]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU213","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Veszprém","NUTS_NAME":"Veszprém"},"geometry":{"type":"Polygon","coordinates":[[[17.883888,47.38958699700004],[18.035719,47.33434199700008],[18.05424750000003,47.278282499000056],[18.224334999000064,47.213434998000025],[18.20524850000004,46.971633],[17.41906750000004,46.75065649600003],[17.325419500000066,46.828989499000045],[17.321602,46.893327998000075],[17.23051950000007,46.919913499000074],[17.220904,46.99529849800007],[17.074528500000042,47.04871349600006],[17.216165,47.083856498000046],[17.198132500000042,47.26408749800004],[17.258762500000046,47.343398498000056],[17.17239150000006,47.43248399700008],[17.737528,47.47290749600006],[17.787135,47.40506799800005],[17.71753250000006,47.34090199800005],[17.765227500000037,47.27287850000005],[17.883888,47.38958699700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU22","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Nyugat-Dunántúl","NUTS_NAME":"Nyugat-Dunántúl"},"geometry":{"type":"Polygon","coordinates":[[[17.893923,47.73945699800004],[17.88704050000007,47.63763449800007],[17.933129,47.61619699800008],[17.884379500000023,47.54962599900006],[17.883888,47.38958699700004],[17.765227500000037,47.27287850000005],[17.71753250000006,47.34090199800005],[17.787135,47.40506799800005],[17.737528,47.47290749600006],[17.17239150000006,47.43248399700008],[17.258762500000046,47.343398498000056],[17.198132500000042,47.26408749800004],[17.216165,47.083856498000046],[17.074528500000042,47.04871349600006],[17.220904,46.99529849800007],[17.23051950000007,46.919913499000074],[17.321602,46.893327998000075],[17.325419500000066,46.828989499000045],[17.41906750000004,46.75065649600003],[17.269003,46.73385449800003],[17.221494500000063,46.67169749800007],[17.187541,46.42907199700005],[17.036631,46.41587049800006],[17.057865,46.33972749800006],[17.019986500000073,46.307627998000044],[16.876043500000037,46.32060249600005],[16.854754999000022,46.35044099800007],[16.596805,46.47590249800004],[16.39033150000006,46.63820750000008],[16.42654550000003,46.69493699800006],[16.370793500000048,46.72224349800007],[16.31216150000006,46.778002498000035],[16.347822500000063,46.84051750000003],[16.291385500000047,46.87283350000007],[16.113849,46.86906799800005],[16.302262,46.99842649800007],[16.508267500000045,47.00125599800003],[16.438882,47.02970299800006],[16.520565500000032,47.05618149800006],[16.46332650000005,47.094956498000045],[16.529331,47.12723199800007],[16.419486500000062,47.19491549800006],[16.483743,47.28759999600004],[16.43376150000006,47.352918499000054],[16.445455,47.40720949700005],[16.64622,47.446596999000064],[16.71450550000003,47.539630998000064],[16.652076,47.622903499000074],[16.421846,47.66470449800005],[16.61228950000003,47.76022649600003],[16.749031,47.68141649700004],[17.093074,47.70823599900007],[17.07445350000006,47.80825550000003],[17.00994950000006,47.858343998000066],[17.085187500000075,47.87443749800008],[17.09466150000003,47.970871498000065],[17.160797500000058,48.00665650000008],[17.24742750000007,48.012008998000056],[17.705436500000076,47.758992498000055],[17.893923,47.73945699800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU221","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Gyor-Moson-Sopron","NUTS_NAME":"Gyor-Moson-Sopron"},"geometry":{"type":"Polygon","coordinates":[[[17.705436500000076,47.758992498000055],[17.893923,47.73945699800004],[17.88704050000007,47.63763449800007],[17.933129,47.61619699800008],[17.884379500000023,47.54962599900006],[17.883888,47.38958699700004],[17.765227500000037,47.27287850000005],[17.71753250000006,47.34090199800005],[17.787135,47.40506799800005],[17.737528,47.47290749600006],[17.17239150000006,47.43248399700008],[17.04958350000004,47.39499199900007],[17.054891500000053,47.46056650000003],[16.947482500000035,47.40069499600003],[16.64622,47.446596999000064],[16.71450550000003,47.539630998000064],[16.652076,47.622903499000074],[16.421846,47.66470449800005],[16.61228950000003,47.76022649600003],[16.749031,47.68141649700004],[17.093074,47.70823599900007],[17.07445350000006,47.80825550000003],[17.00994950000006,47.858343998000066],[17.085187500000075,47.87443749800008],[17.09466150000003,47.970871498000065],[17.160797500000058,48.00665650000008],[17.24742750000007,48.012008998000056],[17.705436500000076,47.758992498000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU222","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Vas","NUTS_NAME":"Vas"},"geometry":{"type":"Polygon","coordinates":[[[17.17239150000006,47.43248399700008],[17.258762500000046,47.343398498000056],[17.198132500000042,47.26408749800004],[17.216165,47.083856498000046],[17.074528500000042,47.04871349600006],[17.000848500000075,47.03310349600008],[16.991198,46.96307899800007],[16.630324,46.92538899700003],[16.503228500000034,46.804632998000045],[16.361848,46.78020849700005],[16.405800500000055,46.771772998000074],[16.370793500000048,46.72224349800007],[16.31216150000006,46.778002498000035],[16.347822500000063,46.84051750000003],[16.291385500000047,46.87283350000007],[16.113849,46.86906799800005],[16.302262,46.99842649800007],[16.508267500000045,47.00125599800003],[16.438882,47.02970299800006],[16.520565500000032,47.05618149800006],[16.46332650000005,47.094956498000045],[16.529331,47.12723199800007],[16.419486500000062,47.19491549800006],[16.483743,47.28759999600004],[16.43376150000006,47.352918499000054],[16.445455,47.40720949700005],[16.64622,47.446596999000064],[16.947482500000035,47.40069499600003],[17.054891500000053,47.46056650000003],[17.04958350000004,47.39499199900007],[17.17239150000006,47.43248399700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU223","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Zala","NUTS_NAME":"Zala"},"geometry":{"type":"Polygon","coordinates":[[[17.41906750000004,46.75065649600003],[17.269003,46.73385449800003],[17.221494500000063,46.67169749800007],[17.187541,46.42907199700005],[17.036631,46.41587049800006],[17.057865,46.33972749800006],[17.019986500000073,46.307627998000044],[16.876043500000037,46.32060249600005],[16.854754999000022,46.35044099800007],[16.596805,46.47590249800004],[16.39033150000006,46.63820750000008],[16.42654550000003,46.69493699800006],[16.370793500000048,46.72224349800007],[16.405800500000055,46.771772998000074],[16.361848,46.78020849700005],[16.503228500000034,46.804632998000045],[16.630324,46.92538899700003],[16.991198,46.96307899800007],[17.000848500000075,47.03310349600008],[17.074528500000042,47.04871349600006],[17.220904,46.99529849800007],[17.23051950000007,46.919913499000074],[17.321602,46.893327998000075],[17.325419500000066,46.828989499000045],[17.41906750000004,46.75065649600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU23","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Dél-Dunántúl","NUTS_NAME":"Dél-Dunántúl"},"geometry":{"type":"Polygon","coordinates":[[[18.174225500000034,46.946213999000065],[18.210897500000044,46.77816149900008],[18.408736500000032,46.803397999000026],[18.462384,46.73966549800008],[18.55658550000004,46.78446449900008],[18.636275,46.68721749800005],[18.925097,46.85716850000006],[19.00624350000004,46.70492349600005],[18.862301,46.61581350000006],[18.923924500000055,46.463970998000036],[18.885012,46.39557699900007],[18.921786,46.30806449700003],[18.82207550000004,46.205366998000045],[18.85447,46.19261],[18.802291,46.10876499800003],[18.73278650000003,46.04514199800008],[18.868295,46.02080049800003],[18.821306,45.91438299600003],[18.659298,45.91699599800006],[18.623518,45.83937849800003],[18.446884,45.737048997000045],[17.911642,45.79095099600005],[17.651499,45.84783549900004],[17.560871,45.93783949900006],[17.294325,45.988544500000046],[17.158216500000037,46.17029199900003],[16.97316550000005,46.22472749800005],[16.876043500000037,46.32060249600005],[17.019986500000073,46.307627998000044],[17.057865,46.33972749800006],[17.036631,46.41587049800006],[17.187541,46.42907199700005],[17.221494500000063,46.67169749800007],[17.269003,46.73385449800003],[17.41906750000004,46.75065649600003],[18.20524850000004,46.971633],[18.174225500000034,46.946213999000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU231","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Baranya","NUTS_NAME":"Baranya"},"geometry":{"type":"Polygon","coordinates":[[[18.802291,46.10876499800003],[18.73278650000003,46.04514199800008],[18.868295,46.02080049800003],[18.821306,45.91438299600003],[18.659298,45.91699599800006],[18.623518,45.83937849800003],[18.446884,45.737048997000045],[17.911642,45.79095099600005],[17.651499,45.84783549900004],[17.734571,45.878112498000064],[17.62475850000004,45.987859498000034],[17.716544,46.197235997000064],[18.05289650000003,46.29581499900007],[18.199117999000066,46.33181150000007],[18.18547250000006,46.397516],[18.337715,46.40871799800004],[18.421707,46.27691449800005],[18.364611500000024,46.21898549900004],[18.508446500000048,46.29084149800008],[18.592920500000048,46.18137999900006],[18.802291,46.10876499800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU232","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Somogy","NUTS_NAME":"Somogy"},"geometry":{"type":"Polygon","coordinates":[[[18.20524850000004,46.971633],[18.174225500000034,46.946213999000065],[18.210897500000044,46.77816149900008],[18.11379850000003,46.73019149800007],[18.08445050000006,46.58287149600005],[18.032376500000055,46.53691050000003],[18.087437500000078,46.32918699700008],[18.05289650000003,46.29581499900007],[17.716544,46.197235997000064],[17.62475850000004,45.987859498000034],[17.734571,45.878112498000064],[17.651499,45.84783549900004],[17.560871,45.93783949900006],[17.294325,45.988544500000046],[17.158216500000037,46.17029199900003],[16.97316550000005,46.22472749800005],[16.876043500000037,46.32060249600005],[17.019986500000073,46.307627998000044],[17.057865,46.33972749800006],[17.036631,46.41587049800006],[17.187541,46.42907199700005],[17.221494500000063,46.67169749800007],[17.269003,46.73385449800003],[17.41906750000004,46.75065649600003],[18.20524850000004,46.971633]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU233","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Tolna","NUTS_NAME":"Tolna"},"geometry":{"type":"Polygon","coordinates":[[[18.802291,46.10876499800003],[18.592920500000048,46.18137999900006],[18.508446500000048,46.29084149800008],[18.364611500000024,46.21898549900004],[18.421707,46.27691449800005],[18.337715,46.40871799800004],[18.18547250000006,46.397516],[18.199117999000066,46.33181150000007],[18.05289650000003,46.29581499900007],[18.087437500000078,46.32918699700008],[18.032376500000055,46.53691050000003],[18.08445050000006,46.58287149600005],[18.11379850000003,46.73019149800007],[18.210897500000044,46.77816149900008],[18.408736500000032,46.803397999000026],[18.462384,46.73966549800008],[18.55658550000004,46.78446449900008],[18.636275,46.68721749800005],[18.925097,46.85716850000006],[19.00624350000004,46.70492349600005],[18.862301,46.61581350000006],[18.923924500000055,46.463970998000036],[18.885012,46.39557699900007],[18.921786,46.30806449700003],[18.82207550000004,46.205366998000045],[18.85447,46.19261],[18.802291,46.10876499800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU3","LEVL_CODE":1,"CNTR_CODE":"HU","NAME_LATN":"ALFÖLD ÉS ÉSZAK","NUTS_NAME":"ALFÖLD ÉS ÉSZAK"},"geometry":{"type":"Polygon","coordinates":[[[21.61392150000006,48.50941299900006],[21.721957,48.35105049800006],[22.12107750000007,48.37831149800007],[22.155306,48.40339649800006],[22.26507,48.40991949800008],[22.240338999000073,48.386390498000026],[22.384609,48.23410049600005],[22.515094500000032,48.238057997000055],[22.590280500000063,48.107631498000046],[22.830129500000055,48.11207949700008],[22.881429500000024,48.05487049800007],[22.836399,47.989948499000036],[22.89627050000007,47.954120500000045],[22.680243999000027,47.78774099800006],[22.31726,47.76592199600003],[22.180837500000052,47.60009449800003],[22.12832,47.59808949600006],[22.012631,47.50135449800007],[22.011991,47.37579649800006],[21.938521500000036,47.37301099800004],[21.792838500000073,47.10724299900005],[21.658955,47.02213149700003],[21.60309650000005,46.86840799600003],[21.483475500000054,46.76463549700003],[21.52933950000005,46.72074199900004],[21.43006950000006,46.693812998000055],[21.441398,46.651466999000036],[21.314121999000065,46.617556],[21.260401500000057,46.501945996000075],[21.317476,46.45065599600008],[21.296291,46.40669299800004],[21.206304,46.40323099600005],[21.103170500000033,46.26259049600003],[20.7756,46.27590999800003],[20.70530350000007,46.16093749800007],[20.6406,46.12720699800008],[20.501525,46.18993599800007],[20.264296,46.12637349900007],[19.86539,46.15033449800006],[19.698099,46.18793099800007],[19.567619500000035,46.17871849800008],[19.30270950000005,45.99155049900003],[19.133789,46.03707099700006],[19.008840500000076,45.92330949800004],[18.889734,45.921180496000034],[18.821306,45.91438299600003],[18.868295,46.02080049800003],[18.73278650000003,46.04514199800008],[18.802291,46.10876499800003],[18.85447,46.19261],[18.82207550000004,46.205366998000045],[18.921786,46.30806449700003],[18.885012,46.39557699900007],[18.923924500000055,46.463970998000036],[18.862301,46.61581350000006],[19.00624350000004,46.70492349600005],[18.925097,46.85716850000006],[18.96590750000007,47.02896499900004],[19.23677,47.13080099700005],[19.347792500000025,47.09322199700006],[19.29528650000003,47.03592250000003],[19.319190500000047,47.012678],[19.587165500000026,47.120925499000066],[19.779767500000048,46.94870449800004],[20.094639,47.007426499000076],[20.047241,47.12519599900003],[20.098951500000055,47.18146399900007],[19.988269500000058,47.25306149800008],[20.017585,47.303893998000035],[19.990462500000035,47.346628],[19.74696350000005,47.488931498000056],[19.789935,47.53137099700007],[19.652673,47.54783799900008],[19.666329,47.58855],[19.570937,47.7349],[19.477327,47.687782997000056],[19.42492050000004,47.81662999800005],[19.08613450000007,47.83817049800007],[18.942689,47.94399499600007],[19.006969,48.010337496000034],[18.928392,48.05683249800006],[19.01432250000005,48.077736498000036],[19.451139,48.094441998000036],[19.52604,48.203133997000066],[19.644519,48.239175499000055],[19.899319,48.124797998000076],[20.051879,48.16770399600006],[20.133181500000035,48.253985998000076],[20.325701500000037,48.27279849800004],[20.463937,48.463966999000036],[20.506496,48.53441350000003],[20.850424,48.581631499000025],[21.11743750000005,48.49105349700005],[21.440056,48.58523299800004],[21.61392150000006,48.50941299900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR62","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Midi-Pyrénées","NUTS_NAME":"Midi-Pyrénées"},"geometry":{"type":"Polygon","coordinates":[[[1.651891500000033,45.02528549800007],[1.774613500000044,44.92353999800008],[2.062908,44.976504498000054],[2.167976500000066,44.81050199800006],[2.149042500000064,44.72213849800005],[2.207473,44.61552899800006],[2.285289500000033,44.666758498000036],[2.478947500000061,44.64800999800008],[2.716769500000055,44.92882799600005],[2.933775500000024,44.780645496000034],[2.981675500000051,44.64467299800003],[3.14346050000006,44.43548199800006],[3.120173500000021,44.26183849600005],[3.373648,44.17075949800005],[3.263114,44.09242549800007],[3.448355,44.01910349800005],[3.358362,43.91382949800004],[3.262606500000061,43.89100250000007],[3.207685999000034,43.812486998000054],[3.064905,43.83556550000003],[3.060680500000046,43.692804498000044],[2.935457,43.694664998000064],[2.75452,43.61414049900003],[2.645487500000058,43.66066949800006],[2.615601,43.56663599800004],[2.67602450000004,43.51345550000008],[2.565782500000068,43.42295799900006],[2.265415,43.45291349800004],[2.215026,43.38261099600004],[2.029134,43.436895496000034],[1.856448500000056,43.443452999000044],[1.80295,43.392548496000074],[1.813485,43.34149799700003],[1.722903500000029,43.341531],[1.688423500000056,43.27355449600003],[1.713483,43.18561949800005],[1.949341,43.12097299900006],[1.985759500000029,43.02995649700006],[1.984954999000024,42.96726849700008],[1.933727500000032,42.93914049800003],[1.986922,42.87100699700005],[1.859382,42.82080499800003],[1.950635500000033,42.73686649600006],[2.084955999000044,42.75165799800004],[2.166049,42.663917498000046],[1.786098500000037,42.573657999000034],[1.574720500000069,42.64779299800006],[1.442566,42.603668],[1.357304,42.71942199800003],[1.17077,42.70814149800003],[1.078708,42.788341496000044],[0.858215,42.82574099800007],[0.66215950000003,42.84124349700005],[0.660127,42.69095249800006],[0.477755500000058,42.70001999600004],[0.360133500000074,42.724353998000026],[0.295051500000056,42.673358998000026],[0.175701,42.73704149800005],[-0.055795,42.69225699800006],[-0.160069499999963,42.79801949800003],[-0.313342,42.84936499600008],[-0.287666499999943,43.00555349800004],[-0.017172,43.26995099900006],[-0.046122499999967,43.300858498000025],[0.009650499000031,43.42219699800006],[-0.067268,43.41041249800003],[-0.065757499999961,43.46348349600004],[-0.018912,43.47005599800008],[-0.096788,43.58240499900006],[-0.242833,43.584978996000075],[-0.282116,43.643073496000056],[-0.194138,43.737014997000074],[-0.227139499999964,43.80827099700008],[-0.185553,43.863012498000046],[-0.240259,43.89794799800006],[-0.036165,43.98373599800004],[0.059019,43.896810498000036],[0.076043500000026,43.98313849800007],[0.741885,44.06519899800003],[0.787737500000048,44.14276099800003],[0.89206850000005,44.13733099900003],[0.853023,44.18391149800004],[0.950791,44.27506550000004],[0.869440500000053,44.309249496000064],[0.896382,44.38183649900003],[1.064081500000043,44.37850849800003],[0.981513500000062,44.54444099800003],[1.075142,44.57732549800005],[1.442178,44.87834299600007],[1.409125500000073,45.006723],[1.44826,45.01931399800003],[1.651891500000033,45.02528549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR621","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Ariège","NUTS_NAME":"Ariège"},"geometry":{"type":"Polygon","coordinates":[[[1.688423500000056,43.27355449600003],[1.713483,43.18561949800005],[1.949341,43.12097299900006],[1.985759500000029,43.02995649700006],[1.984954999000024,42.96726849700008],[1.933727500000032,42.93914049800003],[1.986922,42.87100699700005],[1.859382,42.82080499800003],[1.950635500000033,42.73686649600006],[2.084955999000044,42.75165799800004],[2.166049,42.663917498000046],[1.786098500000037,42.573657999000034],[1.574720500000069,42.64779299800006],[1.442566,42.603668],[1.357304,42.71942199800003],[1.17077,42.70814149800003],[1.078708,42.788341496000044],[0.858215,42.82574099800007],[0.874915,42.957776498000044],[0.979095,42.974432497000066],[1.007047500000056,43.007586497000034],[0.978956999000047,43.07257999700005],[1.053547500000036,43.14545599600007],[1.223567500000058,43.08592450000003],[1.302712,43.12259449900006],[1.213984,43.162433],[1.368483500000025,43.209234999000046],[1.293933500000037,43.26780849900007],[1.33170350000006,43.31455599800006],[1.467612,43.208834496000065],[1.503993500000036,43.220946996000066],[1.496222,43.29276449900004],[1.688423500000056,43.27355449600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR622","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Aveyron","NUTS_NAME":"Aveyron"},"geometry":{"type":"Polygon","coordinates":[[[2.981675500000051,44.64467299800003],[3.14346050000006,44.43548199800006],[3.120173500000021,44.26183849600005],[3.373648,44.17075949800005],[3.263114,44.09242549800007],[3.448355,44.01910349800005],[3.358362,43.91382949800004],[3.262606500000061,43.89100250000007],[3.207685999000034,43.812486998000054],[3.064905,43.83556550000003],[3.060680500000046,43.692804498000044],[2.935457,43.694664998000064],[2.824041500000021,43.76198349800006],[2.670037,43.74953099800007],[2.564157500000022,43.84133199900003],[2.460865500000068,44.050320498000076],[2.284884,44.145331999000064],[2.181526,44.13857999600003],[2.153219500000034,44.19677349600005],[1.990172,44.14944999700003],[1.904345,44.20951199800004],[1.970903,44.272744498000066],[1.868781500000068,44.29203850000005],[1.882078,44.34006899600007],[1.841127500000027,44.48115949800007],[2.207473,44.61552899800006],[2.285289500000033,44.666758498000036],[2.478947500000061,44.64800999800008],[2.716769500000055,44.92882799600005],[2.933775500000024,44.780645496000034],[2.981675500000051,44.64467299800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR623","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Garonne","NUTS_NAME":"Haute-Garonne"},"geometry":{"type":"Polygon","coordinates":[[[2.029134,43.436895496000034],[1.856448500000056,43.443452999000044],[1.80295,43.392548496000074],[1.813485,43.34149799700003],[1.722903500000029,43.341531],[1.688423500000056,43.27355449600003],[1.496222,43.29276449900004],[1.503993500000036,43.220946996000066],[1.467612,43.208834496000065],[1.33170350000006,43.31455599800006],[1.293933500000037,43.26780849900007],[1.368483500000025,43.209234999000046],[1.213984,43.162433],[1.302712,43.12259449900006],[1.223567500000058,43.08592450000003],[1.053547500000036,43.14545599600007],[0.978956999000047,43.07257999700005],[1.007047500000056,43.007586497000034],[0.979095,42.974432497000066],[0.874915,42.957776498000044],[0.858215,42.82574099800007],[0.66215950000003,42.84124349700005],[0.660127,42.69095249800006],[0.477755500000058,42.70001999600004],[0.471866500000033,42.869590498000036],[0.562344500000052,42.86075599600008],[0.643533,42.96374449800004],[0.441708500000061,43.13096649600004],[0.634733500000038,43.29352449800007],[0.606807500000059,43.310883998000065],[0.770418,43.41680799800008],[0.993664500000023,43.366532998000025],[1.059199500000034,43.50538649600003],[1.199346500000047,43.56918150000007],[0.953983,43.78737],[1.144601500000022,43.822294498000076],[1.214563500000054,43.768012499000065],[1.360026499000071,43.81720249600005],[1.293825500000025,43.845596998000076],[1.365183,43.88957499600008],[1.555622,43.91831699800008],[1.588585,43.817176998000036],[1.720701999000028,43.68834749800004],[1.687776,43.630702996000025],[2.029134,43.436895496000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR624","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Gers","NUTS_NAME":"Gers"},"geometry":{"type":"Polygon","coordinates":[[[0.741885,44.06519899800003],[0.864661,44.03869499600006],[0.76169950000002,43.93246750000003],[0.88399,43.890841],[0.895985,43.79147049800008],[0.953983,43.78737],[1.199346500000047,43.56918150000007],[1.059199500000034,43.50538649600003],[0.993664500000023,43.366532998000025],[0.770418,43.41680799800008],[0.606807500000059,43.310883998000065],[0.18203,43.37118449800005],[0.112250500000073,43.51737150000008],[-0.001665499999945,43.56532149800006],[-0.015012500999944,43.605526],[-0.096788,43.58240499900006],[-0.242833,43.584978996000075],[-0.282116,43.643073496000056],[-0.194138,43.737014997000074],[-0.227139499999964,43.80827099700008],[-0.185553,43.863012498000046],[-0.240259,43.89794799800006],[-0.036165,43.98373599800004],[0.059019,43.896810498000036],[0.076043500000026,43.98313849800007],[0.741885,44.06519899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR625","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Lot","NUTS_NAME":"Lot"},"geometry":{"type":"Polygon","coordinates":[[[2.062908,44.976504498000054],[2.167976500000066,44.81050199800006],[2.149042500000064,44.72213849800005],[2.207473,44.61552899800006],[1.841127500000027,44.48115949800007],[1.882078,44.34006899600007],[1.633981499000072,44.268560499000046],[1.570723,44.30150599600006],[1.587249,44.24900649800003],[1.540951,44.22753849800006],[1.471674500000063,44.28680249700005],[1.376239499000064,44.222311998000066],[1.281321500000047,44.23533849700004],[1.304009500000063,44.29418849800004],[1.213900500000022,44.26822799800004],[1.081221500000026,44.35438849800005],[1.132472,44.39386999800007],[1.064081500000043,44.37850849800003],[0.981513500000062,44.54444099800003],[1.075142,44.57732549800005],[1.442178,44.87834299600007],[1.409125500000073,45.006723],[1.44826,45.01931399800003],[1.651891500000033,45.02528549800007],[1.774613500000044,44.92353999800008],[2.062908,44.976504498000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR626","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Hautes-Pyrénées","NUTS_NAME":"Hautes-Pyrénées"},"geometry":{"type":"Polygon","coordinates":[[[-0.001665499999945,43.56532149800006],[0.112250500000073,43.51737150000008],[0.18203,43.37118449800005],[0.606807500000059,43.310883998000065],[0.634733500000038,43.29352449800007],[0.441708500000061,43.13096649600004],[0.643533,42.96374449800004],[0.562344500000052,42.86075599600008],[0.471866500000033,42.869590498000036],[0.477755500000058,42.70001999600004],[0.360133500000074,42.724353998000026],[0.295051500000056,42.673358998000026],[0.175701,42.73704149800005],[-0.055795,42.69225699800006],[-0.160069499999963,42.79801949800003],[-0.313342,42.84936499600008],[-0.287666499999943,43.00555349800004],[-0.017172,43.26995099900006],[-0.046122499999967,43.300858498000025],[0.009650499000031,43.42219699800006],[-0.067268,43.41041249800003],[-0.065757499999961,43.46348349600004],[-0.018912,43.47005599800008],[-0.096788,43.58240499900006],[-0.015012500999944,43.605526],[-0.001665499999945,43.56532149800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR627","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Tarn","NUTS_NAME":"Tarn"},"geometry":{"type":"Polygon","coordinates":[[[2.935457,43.694664998000064],[2.75452,43.61414049900003],[2.645487500000058,43.66066949800006],[2.615601,43.56663599800004],[2.67602450000004,43.51345550000008],[2.565782500000068,43.42295799900006],[2.265415,43.45291349800004],[2.215026,43.38261099600004],[2.029134,43.436895496000034],[1.687776,43.630702996000025],[1.720701999000028,43.68834749800004],[1.588585,43.817176998000036],[1.555622,43.91831699800008],[1.535262500000044,43.95966899800004],[1.622724500000061,43.96070199800005],[1.704213,44.04435299800008],[1.658709500000043,44.11663750000008],[1.990172,44.14944999700003],[2.153219500000034,44.19677349600005],[2.181526,44.13857999600003],[2.284884,44.145331999000064],[2.460865500000068,44.050320498000076],[2.564157500000022,43.84133199900003],[2.670037,43.74953099800007],[2.824041500000021,43.76198349800006],[2.935457,43.694664998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR628","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Tarn-et-Garonne","NUTS_NAME":"Tarn-et-Garonne"},"geometry":{"type":"Polygon","coordinates":[[[1.882078,44.34006899600007],[1.868781500000068,44.29203850000005],[1.970903,44.272744498000066],[1.904345,44.20951199800004],[1.990172,44.14944999700003],[1.658709500000043,44.11663750000008],[1.704213,44.04435299800008],[1.622724500000061,43.96070199800005],[1.535262500000044,43.95966899800004],[1.555622,43.91831699800008],[1.365183,43.88957499600008],[1.293825500000025,43.845596998000076],[1.360026499000071,43.81720249600005],[1.214563500000054,43.768012499000065],[1.144601500000022,43.822294498000076],[0.953983,43.78737],[0.895985,43.79147049800008],[0.88399,43.890841],[0.76169950000002,43.93246750000003],[0.864661,44.03869499600006],[0.741885,44.06519899800003],[0.787737500000048,44.14276099800003],[0.89206850000005,44.13733099900003],[0.853023,44.18391149800004],[0.950791,44.27506550000004],[0.869440500000053,44.309249496000064],[0.896382,44.38183649900003],[1.064081500000043,44.37850849800003],[1.132472,44.39386999800007],[1.081221500000026,44.35438849800005],[1.213900500000022,44.26822799800004],[1.304009500000063,44.29418849800004],[1.281321500000047,44.23533849700004],[1.376239499000064,44.222311998000066],[1.471674500000063,44.28680249700005],[1.540951,44.22753849800006],[1.587249,44.24900649800003],[1.570723,44.30150599600006],[1.633981499000072,44.268560499000046],[1.882078,44.34006899600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR63","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Limousin","NUTS_NAME":"Limousin"},"geometry":{"type":"Polygon","coordinates":[[[2.281043500000067,46.42040349600006],[2.318604500000049,46.33440549900007],[2.480760500000031,46.28100849600003],[2.565372500000024,46.14303599900006],[2.550394500000039,46.08606949800003],[2.60902149900005,45.96664399800005],[2.388014,45.827372996000065],[2.492129500000033,45.73766999600008],[2.528279,45.658533498000054],[2.461168,45.59614650000003],[2.515881,45.55387949800007],[2.50841250000002,45.47850149800007],[2.487533,45.41816699900005],[2.528832,45.392374999000026],[2.369421500000044,45.38727149700003],[2.191158,45.22305049800008],[2.230417500000044,45.16191649800004],[2.095157,45.05603349800003],[2.133296500000029,44.98574199900003],[2.062908,44.976504498000054],[1.774613500000044,44.92353999800008],[1.651891500000033,45.02528549800007],[1.44826,45.01931399800003],[1.378250499000046,45.10608949900006],[1.413087499000028,45.124907499000074],[1.227747500000021,45.20576049700003],[1.235802,45.32204199800003],[1.325074500000028,45.37993749900005],[1.253154500000051,45.44422],[1.119942,45.47469699800007],[1.167285500000048,45.52482299800005],[1.023243,45.60996049800008],[0.811495,45.575871498000026],[0.752317,45.61765599700004],[0.775646,45.66774499700006],[0.629741,45.71456999600008],[0.710518500000035,45.802201498000045],[0.783778500000039,45.79347549700003],[0.835689500000058,45.92272399800004],[0.942597,45.957437998000046],[0.817911,46.04788149600006],[0.823432500000024,46.128584498000066],[0.810388500000045,46.22950949800003],[1.177279,46.383947997000064],[1.356024500000046,46.40011949800004],[1.415185500000064,46.347215],[1.52535,46.42664899600004],[1.72784,46.38895750000006],[1.747591500000055,46.45001799600004],[2.167784500000039,46.42406899800005],[2.281043500000067,46.42040349600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR631","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Corrèze","NUTS_NAME":"Corrèze"},"geometry":{"type":"Polygon","coordinates":[[[2.492129500000033,45.73766999600008],[2.528279,45.658533498000054],[2.461168,45.59614650000003],[2.515881,45.55387949800007],[2.50841250000002,45.47850149800007],[2.487533,45.41816699900005],[2.528832,45.392374999000026],[2.369421500000044,45.38727149700003],[2.191158,45.22305049800008],[2.230417500000044,45.16191649800004],[2.095157,45.05603349800003],[2.133296500000029,44.98574199900003],[2.062908,44.976504498000054],[1.774613500000044,44.92353999800008],[1.651891500000033,45.02528549800007],[1.44826,45.01931399800003],[1.378250499000046,45.10608949900006],[1.413087499000028,45.124907499000074],[1.227747500000021,45.20576049700003],[1.235802,45.32204199800003],[1.325074500000028,45.37993749900005],[1.253154500000051,45.44422],[1.898732,45.69827649600006],[2.058420500000068,45.75447749800003],[2.268858,45.66335249900004],[2.492129500000033,45.73766999600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR632","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Creuse","NUTS_NAME":"Creuse"},"geometry":{"type":"Polygon","coordinates":[[[2.167784500000039,46.42406899800005],[2.281043500000067,46.42040349600006],[2.318604500000049,46.33440549900007],[2.480760500000031,46.28100849600003],[2.565372500000024,46.14303599900006],[2.550394500000039,46.08606949800003],[2.60902149900005,45.96664399800005],[2.388014,45.827372996000065],[2.492129500000033,45.73766999600008],[2.268858,45.66335249900004],[2.058420500000068,45.75447749800003],[1.898732,45.69827649600006],[1.902147500000069,45.76954549800007],[1.834272,45.825447498000074],[1.602394500000059,45.85746049800008],[1.637126,45.92597150000006],[1.508342500000026,45.93769350000008],[1.573693,45.98620050000005],[1.503528,46.131417998000074],[1.372554,46.21603349600008],[1.445412,46.33422949800007],[1.415185500000064,46.347215],[1.52535,46.42664899600004],[1.72784,46.38895750000006],[1.747591500000055,46.45001799600004],[2.167784500000039,46.42406899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR633","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Haute-Vienne","NUTS_NAME":"Haute-Vienne"},"geometry":{"type":"Polygon","coordinates":[[[1.415185500000064,46.347215],[1.445412,46.33422949800007],[1.372554,46.21603349600008],[1.503528,46.131417998000074],[1.573693,45.98620050000005],[1.508342500000026,45.93769350000008],[1.637126,45.92597150000006],[1.602394500000059,45.85746049800008],[1.834272,45.825447498000074],[1.902147500000069,45.76954549800007],[1.898732,45.69827649600006],[1.253154500000051,45.44422],[1.119942,45.47469699800007],[1.167285500000048,45.52482299800005],[1.023243,45.60996049800008],[0.811495,45.575871498000026],[0.752317,45.61765599700004],[0.775646,45.66774499700006],[0.629741,45.71456999600008],[0.710518500000035,45.802201498000045],[0.783778500000039,45.79347549700003],[0.835689500000058,45.92272399800004],[0.942597,45.957437998000046],[0.817911,46.04788149600006],[0.823432500000024,46.128584498000066],[0.810388500000045,46.22950949800003],[1.177279,46.383947997000064],[1.356024500000046,46.40011949800004],[1.415185500000064,46.347215]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR7","LEVL_CODE":1,"CNTR_CODE":"FR","NAME_LATN":"CENTRE-EST","NUTS_NAME":"CENTRE-EST"},"geometry":{"type":"Polygon","coordinates":[[[3.032063,46.794909500000074],[3.165739,46.69172199800005],[3.43421,46.71222249700003],[3.455304500000068,46.65239349800004],[3.629422500000032,46.74945649700004],[3.74101150000007,46.56021499600007],[3.998868500000071,46.46486699800005],[3.984741,46.329729996000026],[3.899538499000073,46.27590799800004],[3.890131,46.21448699800004],[3.988531,46.16985450000004],[4.282425500000045,46.15681299800008],[4.388074500000073,46.219790498000066],[4.405813500000022,46.29605799600006],[4.69344,46.30232599800007],[4.727254500000072,46.17911299900004],[4.780208500000072,46.176675998000064],[4.953853999000046,46.51342299800007],[5.173731,46.51270099800007],[5.310563499000068,46.446769998000036],[5.473656500000061,46.264284],[5.649801500000024,46.334190498000055],[5.725178500000027,46.26068699700005],[5.869188500000064,46.26497799800006],[6.064003,46.41622899600003],[6.170135999000024,46.36643599800004],[6.125608,46.31722999800007],[6.088192,46.246158499000046],[5.974414,46.214751996000075],[5.956067,46.13209549900006],[6.135542,46.141159],[6.310211,46.24404549800005],[6.219547,46.31187799800006],[6.231694,46.329429497000035],[6.24136,46.34358249600007],[6.519104,46.45636749600004],[6.821063999000046,46.42715449800005],[6.770442,46.35520549800003],[6.864808,46.28059399800003],[6.797888999000065,46.136798998000074],[6.899467,46.124058],[6.891464,46.07387949800005],[7.044886,45.92241299800003],[6.818154500000048,45.83595249800004],[6.802368500000057,45.77856199900003],[6.845962,45.690503997000064],[7.000765,45.64000749600007],[7.00001750000007,45.50466699700007],[7.104723,45.46845449800003],[7.185578500000076,45.40326749800005],[7.109843500000068,45.32694349600007],[7.125157,45.243994497000074],[6.893854500000032,45.13691449700008],[6.630051,45.109856499000045],[6.451804500000037,45.051720498000066],[6.26057,45.12684399600005],[6.20392350000003,45.012471],[6.323686,44.99870849800004],[6.355365,44.85482049800004],[6.015539500000045,44.83527399800005],[5.80147,44.706777500000044],[5.830441,44.690548997000064],[5.790623500000038,44.653293],[5.641717500000027,44.65107749800006],[5.597252500000025,44.543273498000076],[5.664504,44.50189499700008],[5.458414,44.499388999000075],[5.418397500000026,44.42476849800005],[5.492943,44.337132498000074],[5.630529500000023,44.33331449900004],[5.676035999000021,44.19142849800005],[5.498786500000051,44.11571649700005],[5.355353,44.21456549800007],[5.154939,44.23049049600007],[5.149949500000048,44.30193449800004],[4.825547500000027,44.22856849800007],[4.804566,44.30389699700004],[4.650611,44.32980299600007],[4.649227500000052,44.27035999900005],[4.474941500000057,44.34202699600007],[4.400397,44.28753349600004],[4.395846500000061,44.34406449800008],[4.258899500000041,44.26442299800004],[4.053996500000039,44.337807498000075],[4.06844250000006,44.405057498000076],[3.998161500000037,44.459798497000065],[3.862531,44.74386599600007],[3.630213,44.880930498000055],[3.478246,44.80967799700005],[3.361347500000022,44.971407999000064],[3.188543,44.86144799900006],[3.103125,44.88463249700004],[2.981675500000051,44.64467299800003],[2.933775500000024,44.780645496000034],[2.716769500000055,44.92882799600005],[2.478947500000061,44.64800999800008],[2.285289500000033,44.666758498000036],[2.207473,44.61552899800006],[2.149042500000064,44.72213849800005],[2.167976500000066,44.81050199800006],[2.062908,44.976504498000054],[2.133296500000029,44.98574199900003],[2.095157,45.05603349800003],[2.230417500000044,45.16191649800004],[2.191158,45.22305049800008],[2.369421500000044,45.38727149700003],[2.528832,45.392374999000026],[2.487533,45.41816699900005],[2.50841250000002,45.47850149800007],[2.515881,45.55387949800007],[2.461168,45.59614650000003],[2.528279,45.658533498000054],[2.492129500000033,45.73766999600008],[2.388014,45.827372996000065],[2.60902149900005,45.96664399800005],[2.550394500000039,46.08606949800003],[2.565372500000024,46.14303599900006],[2.480760500000031,46.28100849600003],[2.318604500000049,46.33440549900007],[2.281043500000067,46.42040349600006],[2.351579,46.51213949600003],[2.614960500000052,46.55327649900005],[2.566998,46.652483],[2.71502,46.744980998000074],[3.032063,46.794909500000074]],[[4.869529500000056,44.345077],[4.987017500000036,44.29271349900006],[5.070316,44.37644749800006],[4.959273,44.41978299800007],[4.869529500000056,44.345077]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR71","LEVL_CODE":2,"CNTR_CODE":"FR","NAME_LATN":"Rhône-Alpes","NUTS_NAME":"Rhône-Alpes"},"geometry":{"type":"Polygon","coordinates":[[[5.310563499000068,46.446769998000036],[5.473656500000061,46.264284],[5.649801500000024,46.334190498000055],[5.725178500000027,46.26068699700005],[5.869188500000064,46.26497799800006],[6.064003,46.41622899600003],[6.170135999000024,46.36643599800004],[6.125608,46.31722999800007],[6.088192,46.246158499000046],[5.974414,46.214751996000075],[5.956067,46.13209549900006],[6.135542,46.141159],[6.310211,46.24404549800005],[6.219547,46.31187799800006],[6.231694,46.329429497000035],[6.24136,46.34358249600007],[6.519104,46.45636749600004],[6.821063999000046,46.42715449800005],[6.770442,46.35520549800003],[6.864808,46.28059399800003],[6.797888999000065,46.136798998000074],[6.899467,46.124058],[6.891464,46.07387949800005],[7.044886,45.92241299800003],[6.818154500000048,45.83595249800004],[6.802368500000057,45.77856199900003],[6.845962,45.690503997000064],[7.000765,45.64000749600007],[7.00001750000007,45.50466699700007],[7.104723,45.46845449800003],[7.185578500000076,45.40326749800005],[7.109843500000068,45.32694349600007],[7.125157,45.243994497000074],[6.893854500000032,45.13691449700008],[6.630051,45.109856499000045],[6.451804500000037,45.051720498000066],[6.26057,45.12684399600005],[6.20392350000003,45.012471],[6.323686,44.99870849800004],[6.355365,44.85482049800004],[6.015539500000045,44.83527399800005],[5.80147,44.706777500000044],[5.830441,44.690548997000064],[5.790623500000038,44.653293],[5.641717500000027,44.65107749800006],[5.597252500000025,44.543273498000076],[5.664504,44.50189499700008],[5.458414,44.499388999000075],[5.418397500000026,44.42476849800005],[5.492943,44.337132498000074],[5.630529500000023,44.33331449900004],[5.676035999000021,44.19142849800005],[5.498786500000051,44.11571649700005],[5.355353,44.21456549800007],[5.154939,44.23049049600007],[5.149949500000048,44.30193449800004],[4.825547500000027,44.22856849800007],[4.804566,44.30389699700004],[4.650611,44.32980299600007],[4.649227500000052,44.27035999900005],[4.474941500000057,44.34202699600007],[4.400397,44.28753349600004],[4.395846500000061,44.34406449800008],[4.258899500000041,44.26442299800004],[4.053996500000039,44.337807498000075],[4.06844250000006,44.405057498000076],[3.998161500000037,44.459798497000065],[3.862531,44.74386599600007],[4.31464,44.973134498000036],[4.30212,45.02648049800007],[4.379061500000034,45.03617449800004],[4.346376500000076,45.07504099800008],[4.37394050000006,45.143959496000036],[4.422980499000062,45.137264498000036],[4.483132,45.23644549800008],[4.349434500000029,45.27420949800006],[4.331653500000073,45.318478998000046],[4.368010500000025,45.33710749800008],[4.329783500000076,45.369281998000076],[3.89740850000004,45.35707999700003],[3.98468899900007,45.495232498000064],[3.777585999000053,45.69056249800008],[3.712625500000058,45.80128799900007],[3.754369500000053,45.88602799900008],[3.694015,45.930728],[3.832069500000046,45.99966349600004],[3.768207500000074,46.23881449600003],[3.899538499000073,46.27590799800004],[3.890131,46.21448699800004],[3.988531,46.16985450000004],[4.282425500000045,46.15681299800008],[4.388074500000073,46.219790498000066],[4.405813500000022,46.29605799600006],[4.69344,46.30232599800007],[4.727254500000072,46.17911299900004],[4.780208500000072,46.176675998000064],[4.953853999000046,46.51342299800007],[5.173731,46.51270099800007],[5.310563499000068,46.446769998000036]],[[4.869529500000056,44.345077],[4.987017500000036,44.29271349900006],[5.070316,44.37644749800006],[4.959273,44.41978299800007],[4.869529500000056,44.345077]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR711","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Ain","NUTS_NAME":"Ain"},"geometry":{"type":"Polygon","coordinates":[[[5.310563499000068,46.446769998000036],[5.473656500000061,46.264284],[5.649801500000024,46.334190498000055],[5.725178500000027,46.26068699700005],[5.869188500000064,46.26497799800006],[6.064003,46.41622899600003],[6.170135999000024,46.36643599800004],[6.125608,46.31722999800007],[6.088192,46.246158499000046],[5.974414,46.214751996000075],[5.956067,46.13209549900006],[5.819607,46.103823498000054],[5.83123,45.93845749700006],[5.776236,45.72638399700003],[5.623743,45.61326849900007],[5.404375,45.856872499000076],[5.340918999000053,45.88050949600006],[5.223596500000042,45.76865999700004],[5.101069,45.813378999000065],[4.923797500000035,45.803994998000064],[4.872074,45.903455499000074],[4.731369,45.94132049900003],[4.780208500000072,46.176675998000064],[4.953853999000046,46.51342299800007],[5.173731,46.51270099800007],[5.310563499000068,46.446769998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"FR712","LEVL_CODE":3,"CNTR_CODE":"FR","NAME_LATN":"Ardèche","NUTS_NAME":"Ardèche"},"geometry":{"type":"Polygon","coordinates":[[[4.80049,45.29835699600005],[4.803822500000024,45.12271399900004],[4.886594500000058,44.93665249600008],[4.761766500000022,44.77359449800008],[4.779066499000066,44.65469599800008],[4.692773500000044,44.558538498000075],[4.650611,44.32980299600007],[4.649227500000052,44.27035999900005],[4.474941500000057,44.34202699600007],[4.400397,44.28753349600004],[4.395846500000061,44.34406449800008],[4.258899500000041,44.26442299800004],[4.053996500000039,44.337807498000075],[4.06844250000006,44.405057498000076],[3.998161500000037,44.459798497000065],[3.862531,44.74386599600007],[4.31464,44.973134498000036],[4.30212,45.02648049800007],[4.379061500000034,45.03617449800004],[4.346376500000076,45.07504099800008],[4.37394050000006,45.143959496000036],[4.422980499000062,45.137264498000036],[4.483132,45.23644549800008],[4.606401,45.254056999000056],[4.614825,45.31153599600003],[4.75599950000003,45.36567499800003],[4.80049,45.29835699600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF","LEVL_CODE":1,"CNTR_CODE":"IT","NAME_LATN":"SUD","NUTS_NAME":"SUD"},"geometry":{"type":"Polygon","coordinates":[[[14.710411,42.17232149600005],[14.718282,42.10572499800003],[14.779646999000022,42.070021498000074],[15.138178500000038,41.92700749800008],[16.059203,41.94750049800007],[16.176531500000067,41.88478949900008],[16.197651500000063,41.80332799800004],[16.141325,41.74129799700006],[15.89714,41.60323199800007],[15.923958500000026,41.49635299800008],[16.02473850000007,41.425590498000076],[16.542302499000073,41.229440999000076],[17.054502,41.07813399500003],[17.388981,40.89186349800008],[18.014299,40.64179799800007],[18.097453,40.515377998000076],[18.423196500000074,40.29373849700005],[18.517056,40.135147998000036],[18.407547500000078,39.974780500000065],[18.369068,39.793742],[18.047947500000078,39.92847849800006],[17.993416500000023,40.00004949700008],[18.010002,40.10898749800003],[17.903836500000068,40.25683649700005],[17.763504499000078,40.295506999000054],[17.510776500000077,40.294624999000064],[17.224865,40.408818997000026],[17.248176,40.46044399900006],[17.127098,40.51762049900003],[16.993565,40.493809498000076],[16.867264,40.39784549800004],[16.644317,40.119095998000034],[16.59911950000003,40.040231997000035],[16.62626,39.95547799800005],[16.490359500000068,39.80335299700005],[16.527955500000076,39.66865950000005],[16.770733499000073,39.62110549700003],[17.025064500000042,39.48357849800004],[17.13565,39.37396599900006],[17.10419150000007,39.263840998000035],[17.148327,39.210276],[17.11019250000004,39.09960299800008],[17.188881,39.02047199700007],[17.101278500000035,38.90490999600007],[16.890732,38.92712849800006],[16.566985500000044,38.76548199800004],[16.53582350000005,38.70611849800008],[16.582084500000065,38.46979049600003],[16.488372,38.35253349900006],[16.326207999000076,38.29452150000003],[16.169725500000027,38.14043699800004],[16.063809500000048,37.92441899600004],[15.678212,37.95411949700008],[15.634096500000055,38.01831599900004],[15.636063,38.231473497000025],[15.808353500000067,38.29168949900003],[15.918956,38.50676149800006],[15.928529,38.55029899800007],[15.828991,38.61772449700004],[15.848117500000058,38.658864497000025],[15.983204,38.72185349600005],[16.12969,38.71597199900003],[16.213753500000053,38.81043999800005],[16.215453500000024,38.92621249800004],[16.093153,39.04874299700003],[16.036746,39.349211996000065],[15.873549,39.55545449700003],[15.755958500000077,39.92349599900007],[15.644945499000073,40.042790498000045],[15.532799,40.07377949600004],[15.356917,39.99909099700005],[15.132452500000056,40.17125949800004],[14.908925,40.24347149700003],[15.001006,40.379208997000035],[14.787251500000025,40.66618049700003],[14.747420500000032,40.67764649800006],[14.69241450000004,40.63406599800004],[14.633576,40.64854399700005],[14.528490500000032,40.60715249700007],[14.46862,40.62020149800003],[14.324673,40.56908499800005],[14.479247500000042,40.69592299800007],[14.460677500000031,40.74293050000006],[14.274676,40.84441749700005],[14.04498650000005,40.79068099700004],[14.03213050000005,40.89883899700004],[13.760795500000029,41.22316799600003],[13.889761500000077,41.297446499000046],[13.873718500000052,41.338294],[13.861228,41.41653499800003],[13.977928,41.46245399900005],[14.026036,41.555293998000025],[13.941038499000058,41.68794399600006],[13.729585,41.798873497000045],[13.573855500000036,41.75432049900007],[13.379543500000068,41.817429496000045],[13.383656,41.901868998000054],[13.296299500000032,41.94858849900004],[13.054555500000049,42.01634599700003],[13.030574500000057,42.11535299900004],[13.087079500000073,42.17784499800007],[13.248800500000073,42.127403497000046],[13.379911500000048,42.18042199800004],[13.153368,42.356970499000056],[13.194177,42.38916949800006],[13.116188500000021,42.44388449600007],[13.178439,42.47931599800006],[13.191380500000037,42.58749099900007],[13.39403149900005,42.591233998000064],[13.409763,42.63902249800003],[13.357774500000062,42.69406999700004],[13.48879,42.73365799700008],[13.534506,42.81658549800005],[13.91573150000005,42.89457699900004],[14.005647500000066,42.69359099900004],[14.146906500000057,42.53059649800008],[14.253960499000073,42.44479849800007],[14.509372,42.25165949800004],[14.710411,42.17232149600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF1","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Abruzzo","NUTS_NAME":"Abruzzo"},"geometry":{"type":"Polygon","coordinates":[[[14.710411,42.17232149600005],[14.718282,42.10572499800003],[14.779646999000022,42.070021498000074],[14.485359,41.759622998000054],[14.442357,41.841226998000025],[14.229757500000062,41.87705649600008],[14.147189500000025,41.82726699800003],[14.194203500000071,41.74811499900005],[13.941038499000058,41.68794399600006],[13.729585,41.798873497000045],[13.573855500000036,41.75432049900007],[13.379543500000068,41.817429496000045],[13.383656,41.901868998000054],[13.296299500000032,41.94858849900004],[13.054555500000049,42.01634599700003],[13.030574500000057,42.11535299900004],[13.087079500000073,42.17784499800007],[13.248800500000073,42.127403497000046],[13.379911500000048,42.18042199800004],[13.153368,42.356970499000056],[13.194177,42.38916949800006],[13.116188500000021,42.44388449600007],[13.178439,42.47931599800006],[13.191380500000037,42.58749099900007],[13.39403149900005,42.591233998000064],[13.409763,42.63902249800003],[13.357774500000062,42.69406999700004],[13.48879,42.73365799700008],[13.534506,42.81658549800005],[13.91573150000005,42.89457699900004],[14.005647500000066,42.69359099900004],[14.146906500000057,42.53059649800008],[14.253960499000073,42.44479849800007],[14.509372,42.25165949800004],[14.710411,42.17232149600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF11","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"L'Aquila","NUTS_NAME":"L'Aquila"},"geometry":{"type":"Polygon","coordinates":[[[13.765287500000056,42.420117496000046],[13.844353,42.281053996000026],[13.78924950000004,42.153905999000074],[14.085253,42.086697497000046],[14.099871,41.90428699800003],[14.229757500000062,41.87705649600008],[14.147189500000025,41.82726699800003],[14.194203500000071,41.74811499900005],[13.941038499000058,41.68794399600006],[13.729585,41.798873497000045],[13.573855500000036,41.75432049900007],[13.379543500000068,41.817429496000045],[13.383656,41.901868998000054],[13.296299500000032,41.94858849900004],[13.054555500000049,42.01634599700003],[13.030574500000057,42.11535299900004],[13.087079500000073,42.17784499800007],[13.248800500000073,42.127403497000046],[13.379911500000048,42.18042199800004],[13.153368,42.356970499000056],[13.194177,42.38916949800006],[13.116188500000021,42.44388449600007],[13.178439,42.47931599800006],[13.191380500000037,42.58749099900007],[13.39403149900005,42.591233998000064],[13.526071,42.449633997000035],[13.765287500000056,42.420117496000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF12","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Teramo","NUTS_NAME":"Teramo"},"geometry":{"type":"Polygon","coordinates":[[[14.146906500000057,42.53059649800008],[13.810672500000067,42.49284449600003],[13.765287500000056,42.420117496000046],[13.526071,42.449633997000035],[13.39403149900005,42.591233998000064],[13.409763,42.63902249800003],[13.357774500000062,42.69406999700004],[13.48879,42.73365799700008],[13.534506,42.81658549800005],[13.91573150000005,42.89457699900004],[14.005647500000066,42.69359099900004],[14.146906500000057,42.53059649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF13","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pescara","NUTS_NAME":"Pescara"},"geometry":{"type":"Polygon","coordinates":[[[14.146906500000057,42.53059649800008],[14.253960499000073,42.44479849800007],[14.171526,42.43661799700004],[14.093579,42.334749499000054],[14.133118500000023,42.22427849600007],[14.085253,42.086697497000046],[13.78924950000004,42.153905999000074],[13.844353,42.281053996000026],[13.765287500000056,42.420117496000046],[13.810672500000067,42.49284449600003],[14.146906500000057,42.53059649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF14","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Chieti","NUTS_NAME":"Chieti"},"geometry":{"type":"Polygon","coordinates":[[[14.779646999000022,42.070021498000074],[14.485359,41.759622998000054],[14.442357,41.841226998000025],[14.229757500000062,41.87705649600008],[14.099871,41.90428699800003],[14.085253,42.086697497000046],[14.133118500000023,42.22427849600007],[14.093579,42.334749499000054],[14.171526,42.43661799700004],[14.253960499000073,42.44479849800007],[14.509372,42.25165949800004],[14.710411,42.17232149600005],[14.718282,42.10572499800003],[14.779646999000022,42.070021498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF2","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Molise","NUTS_NAME":"Molise"},"geometry":{"type":"Polygon","coordinates":[[[15.097028500000022,41.76780949700003],[15.124263,41.69609649800003],[14.934101,41.619428998000046],[14.937399500000026,41.527060998000024],[15.007655500000055,41.486356998000076],[14.603068500000063,41.36392599800007],[14.5051,41.382461497000065],[14.382519,41.44321649800003],[14.119531,41.50648449800008],[14.081558,41.46494399900007],[14.11146350000007,41.38639899900005],[13.977928,41.46245399900005],[14.026036,41.555293998000025],[13.941038499000058,41.68794399600006],[14.194203500000071,41.74811499900005],[14.147189500000025,41.82726699800003],[14.229757500000062,41.87705649600008],[14.442357,41.841226998000025],[14.485359,41.759622998000054],[14.779646999000022,42.070021498000074],[15.138178500000038,41.92700749800008],[15.097028500000022,41.76780949700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF21","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Isernia","NUTS_NAME":"Isernia"},"geometry":{"type":"Polygon","coordinates":[[[14.485359,41.759622998000054],[14.520794500000022,41.69888199800005],[14.424154,41.64386049800004],[14.512163,41.59642199800004],[14.39207250000004,41.50775050000004],[14.382519,41.44321649800003],[14.119531,41.50648449800008],[14.081558,41.46494399900007],[14.11146350000007,41.38639899900005],[13.977928,41.46245399900005],[14.026036,41.555293998000025],[13.941038499000058,41.68794399600006],[14.194203500000071,41.74811499900005],[14.147189500000025,41.82726699800003],[14.229757500000062,41.87705649600008],[14.442357,41.841226998000025],[14.485359,41.759622998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF22","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Campobasso","NUTS_NAME":"Campobasso"},"geometry":{"type":"Polygon","coordinates":[[[15.138178500000038,41.92700749800008],[15.097028500000022,41.76780949700003],[15.124263,41.69609649800003],[14.934101,41.619428998000046],[14.937399500000026,41.527060998000024],[15.007655500000055,41.486356998000076],[14.603068500000063,41.36392599800007],[14.5051,41.382461497000065],[14.382519,41.44321649800003],[14.39207250000004,41.50775050000004],[14.512163,41.59642199800004],[14.424154,41.64386049800004],[14.520794500000022,41.69888199800005],[14.485359,41.759622998000054],[14.779646999000022,42.070021498000074],[15.138178500000038,41.92700749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF3","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Campania","NUTS_NAME":"Campania"},"geometry":{"type":"Polygon","coordinates":[[[14.382519,41.44321649800003],[14.5051,41.382461497000065],[14.603068500000063,41.36392599800007],[15.007655500000055,41.486356998000076],[15.046827,41.44331049800007],[15.091753500000038,41.41789249800007],[15.07223,41.332430499000054],[15.149196500000073,41.28041249700004],[15.27988750000003,41.24076249800004],[15.20939050000004,41.16765349900004],[15.267235,41.10660699700003],[15.542921,41.05582350000003],[15.572237,40.99836799700006],[15.527298500000029,40.907801998000025],[15.354225500000041,40.882618498000056],[15.377602500000023,40.836103998000056],[15.335024500000031,40.834843997000064],[15.388417,40.794122998000034],[15.381197,40.724950998000054],[15.503555499000072,40.66380599900003],[15.449538500000074,40.609062497000025],[15.53941,40.568025498000054],[15.538927500000057,40.48801149800005],[15.806531500000062,40.27222249700003],[15.644945499000073,40.042790498000045],[15.532799,40.07377949600004],[15.356917,39.99909099700005],[15.132452500000056,40.17125949800004],[14.908925,40.24347149700003],[15.001006,40.379208997000035],[14.787251500000025,40.66618049700003],[14.747420500000032,40.67764649800006],[14.69241450000004,40.63406599800004],[14.633576,40.64854399700005],[14.528490500000032,40.60715249700007],[14.46862,40.62020149800003],[14.324673,40.56908499800005],[14.479247500000042,40.69592299800007],[14.460677500000031,40.74293050000006],[14.274676,40.84441749700005],[14.04498650000005,40.79068099700004],[14.03213050000005,40.89883899700004],[13.760795500000029,41.22316799600003],[13.889761500000077,41.297446499000046],[13.873718500000052,41.338294],[13.861228,41.41653499800003],[13.977928,41.46245399900005],[14.11146350000007,41.38639899900005],[14.081558,41.46494399900007],[14.119531,41.50648449800008],[14.382519,41.44321649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF31","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Caserta","NUTS_NAME":"Caserta"},"geometry":{"type":"Polygon","coordinates":[[[14.382519,41.44321649800003],[14.5051,41.382461497000065],[14.439518500000077,41.34421599800004],[14.489129,41.30060999800003],[14.418203500000061,41.22223099800004],[14.463661500000057,41.156825998000045],[14.35205,41.15204749800006],[14.434199499000044,41.10775299800008],[14.413019500000075,41.05715549800004],[14.5312,41.01330649800008],[14.29764350000005,41.004706498000075],[14.252277,40.94603999800006],[14.112237,40.97894249600006],[14.018023,40.94786649900004],[14.03213050000005,40.89883899700004],[13.760795500000029,41.22316799600003],[13.889761500000077,41.297446499000046],[13.873718500000052,41.338294],[13.861228,41.41653499800003],[13.977928,41.46245399900005],[14.11146350000007,41.38639899900005],[14.081558,41.46494399900007],[14.119531,41.50648449800008],[14.382519,41.44321649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF32","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Benevento","NUTS_NAME":"Benevento"},"geometry":{"type":"Polygon","coordinates":[[[15.046827,41.44331049800007],[15.091753500000038,41.41789249800007],[15.07223,41.332430499000054],[15.149196500000073,41.28041249700004],[14.986165500000027,41.27111249600006],[15.004946,41.20813899700005],[14.956966500000021,41.147354998000026],[15.010900500000048,41.11806149800003],[14.974531499000022,41.09318499700004],[14.760381500000051,41.00933949700004],[14.596678500000053,41.05878449800008],[14.571968,41.00946649700006],[14.5312,41.01330649800008],[14.413019500000075,41.05715549800004],[14.434199499000044,41.10775299800008],[14.35205,41.15204749800006],[14.463661500000057,41.156825998000045],[14.418203500000061,41.22223099800004],[14.489129,41.30060999800003],[14.439518500000077,41.34421599800004],[14.5051,41.382461497000065],[14.603068500000063,41.36392599800007],[15.007655500000055,41.486356998000076],[15.046827,41.44331049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF33","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Napoli","NUTS_NAME":"Napoli"},"geometry":{"type":"Polygon","coordinates":[[[14.581142500000055,40.99403449600004],[14.605582,40.847567498000046],[14.507053,40.77292149800007],[14.590655,40.693966998000064],[14.577195,40.62865499900005],[14.46862,40.62020149800003],[14.324673,40.56908499800005],[14.479247500000042,40.69592299800007],[14.460677500000031,40.74293050000006],[14.274676,40.84441749700005],[14.04498650000005,40.79068099700004],[14.03213050000005,40.89883899700004],[14.018023,40.94786649900004],[14.112237,40.97894249600006],[14.252277,40.94603999800006],[14.29764350000005,41.004706498000075],[14.5312,41.01330649800008],[14.571968,41.00946649700006],[14.581142500000055,40.99403449600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF34","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Avellino","NUTS_NAME":"Avellino"},"geometry":{"type":"Polygon","coordinates":[[[15.542921,41.05582350000003],[15.572237,40.99836799700006],[15.527298500000029,40.907801998000025],[15.354225500000041,40.882618498000056],[15.377602500000023,40.836103998000056],[15.335024500000031,40.834843997000064],[15.239176500000042,40.77386599700003],[15.247136,40.71764199800003],[15.169034,40.70660449700006],[15.132343,40.761145498000076],[14.605582,40.847567498000046],[14.581142500000055,40.99403449600004],[14.571968,41.00946649700006],[14.596678500000053,41.05878449800008],[14.760381500000051,41.00933949700004],[14.974531499000022,41.09318499700004],[15.010900500000048,41.11806149800003],[14.956966500000021,41.147354998000026],[15.004946,41.20813899700005],[14.986165500000027,41.27111249600006],[15.149196500000073,41.28041249700004],[15.27988750000003,41.24076249800004],[15.20939050000004,41.16765349900004],[15.267235,41.10660699700003],[15.542921,41.05582350000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF35","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Salerno","NUTS_NAME":"Salerno"},"geometry":{"type":"Polygon","coordinates":[[[15.335024500000031,40.834843997000064],[15.388417,40.794122998000034],[15.381197,40.724950998000054],[15.503555499000072,40.66380599900003],[15.449538500000074,40.609062497000025],[15.53941,40.568025498000054],[15.538927500000057,40.48801149800005],[15.806531500000062,40.27222249700003],[15.644945499000073,40.042790498000045],[15.532799,40.07377949600004],[15.356917,39.99909099700005],[15.132452500000056,40.17125949800004],[14.908925,40.24347149700003],[15.001006,40.379208997000035],[14.787251500000025,40.66618049700003],[14.747420500000032,40.67764649800006],[14.69241450000004,40.63406599800004],[14.633576,40.64854399700005],[14.528490500000032,40.60715249700007],[14.46862,40.62020149800003],[14.577195,40.62865499900005],[14.590655,40.693966998000064],[14.507053,40.77292149800007],[14.605582,40.847567498000046],[15.132343,40.761145498000076],[15.169034,40.70660449700006],[15.247136,40.71764199800003],[15.239176500000042,40.77386599700003],[15.335024500000031,40.834843997000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF4","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Puglia","NUTS_NAME":"Puglia"},"geometry":{"type":"Polygon","coordinates":[[[16.542302499000073,41.229440999000076],[17.054502,41.07813399500003],[17.388981,40.89186349800008],[18.014299,40.64179799800007],[18.097453,40.515377998000076],[18.423196500000074,40.29373849700005],[18.517056,40.135147998000036],[18.407547500000078,39.974780500000065],[18.369068,39.793742],[18.047947500000078,39.92847849800006],[17.993416500000023,40.00004949700008],[18.010002,40.10898749800003],[17.903836500000068,40.25683649700005],[17.763504499000078,40.295506999000054],[17.510776500000077,40.294624999000064],[17.224865,40.408818997000026],[17.248176,40.46044399900006],[17.127098,40.51762049900003],[16.993565,40.493809498000076],[16.867264,40.39784549800004],[16.719678499000054,40.481393499000035],[16.72499,40.71381449800003],[16.501127,40.76227099700003],[16.413857,40.70184350000005],[16.244098500000064,40.83800849800008],[16.202532,40.91751099900006],[15.976693500000067,40.95843049800004],[16.04068650000005,41.036222999000074],[15.870311,41.13989999800003],[15.542921,41.05582350000003],[15.267235,41.10660699700003],[15.20939050000004,41.16765349900004],[15.27988750000003,41.24076249800004],[15.149196500000073,41.28041249700004],[15.07223,41.332430499000054],[15.091753500000038,41.41789249800007],[15.046827,41.44331049800007],[15.007655500000055,41.486356998000076],[14.937399500000026,41.527060998000024],[14.934101,41.619428998000046],[15.124263,41.69609649800003],[15.097028500000022,41.76780949700003],[15.138178500000038,41.92700749800008],[16.059203,41.94750049800007],[16.176531500000067,41.88478949900008],[16.197651500000063,41.80332799800004],[16.141325,41.74129799700006],[15.89714,41.60323199800007],[15.923958500000026,41.49635299800008],[16.02473850000007,41.425590498000076],[16.542302499000073,41.229440999000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF43","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Taranto","NUTS_NAME":"Taranto"},"geometry":{"type":"Polygon","coordinates":[[[17.353692500000022,40.74857549700005],[17.47351050000003,40.61054499800008],[17.448863500000073,40.551879996000025],[17.487073500000065,40.48214149800003],[17.79841349900005,40.379635998000026],[17.763504499000078,40.295506999000054],[17.510776500000077,40.294624999000064],[17.224865,40.408818997000026],[17.248176,40.46044399900006],[17.127098,40.51762049900003],[16.993565,40.493809498000076],[16.867264,40.39784549800004],[16.719678499000054,40.481393499000035],[16.72499,40.71381449800003],[17.14860150000004,40.709911498000054],[17.245182,40.80028149800006],[17.353692500000022,40.74857549700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF44","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Brindisi","NUTS_NAME":"Brindisi"},"geometry":{"type":"Polygon","coordinates":[[[18.097453,40.515377998000076],[17.79841349900005,40.379635998000026],[17.487073500000065,40.48214149800003],[17.448863500000073,40.551879996000025],[17.47351050000003,40.61054499800008],[17.353692500000022,40.74857549700005],[17.398670500000037,40.78566099600005],[17.29534650000005,40.817173998000044],[17.388981,40.89186349800008],[18.014299,40.64179799800007],[18.097453,40.515377998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF45","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Lecce","NUTS_NAME":"Lecce"},"geometry":{"type":"Polygon","coordinates":[[[17.763504499000078,40.295506999000054],[17.79841349900005,40.379635998000026],[18.097453,40.515377998000076],[18.423196500000074,40.29373849700005],[18.517056,40.135147998000036],[18.407547500000078,39.974780500000065],[18.369068,39.793742],[18.047947500000078,39.92847849800006],[17.993416500000023,40.00004949700008],[18.010002,40.10898749800003],[17.903836500000068,40.25683649700005],[17.763504499000078,40.295506999000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF46","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Foggia","NUTS_NAME":"Foggia"},"geometry":{"type":"Polygon","coordinates":[[[16.02473850000007,41.425590498000076],[15.937005500000055,41.425793998000074],[16.03856550000006,41.382622499000036],[15.98197650000003,41.360767999000075],[16.040297500000065,41.26988649800006],[15.870311,41.13989999800003],[15.542921,41.05582350000003],[15.267235,41.10660699700003],[15.20939050000004,41.16765349900004],[15.27988750000003,41.24076249800004],[15.149196500000073,41.28041249700004],[15.07223,41.332430499000054],[15.091753500000038,41.41789249800007],[15.046827,41.44331049800007],[15.007655500000055,41.486356998000076],[14.937399500000026,41.527060998000024],[14.934101,41.619428998000046],[15.124263,41.69609649800003],[15.097028500000022,41.76780949700003],[15.138178500000038,41.92700749800008],[16.059203,41.94750049800007],[16.176531500000067,41.88478949900008],[16.197651500000063,41.80332799800004],[16.141325,41.74129799700006],[15.89714,41.60323199800007],[15.923958500000026,41.49635299800008],[16.02473850000007,41.425590498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF47","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Bari","NUTS_NAME":"Bari"},"geometry":{"type":"Polygon","coordinates":[[[17.388981,40.89186349800008],[17.29534650000005,40.817173998000044],[17.398670500000037,40.78566099600005],[17.353692500000022,40.74857549700005],[17.245182,40.80028149800006],[17.14860150000004,40.709911498000054],[16.72499,40.71381449800003],[16.501127,40.76227099700003],[16.413857,40.70184350000005],[16.244098500000064,40.83800849800008],[16.202532,40.91751099900006],[16.391652500000077,41.196112999000036],[16.503909,41.151320498000075],[16.542302499000073,41.229440999000076],[17.054502,41.07813399500003],[17.388981,40.89186349800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF48","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Barletta-Andria-Trani","NUTS_NAME":"Barletta-Andria-Trani"},"geometry":{"type":"Polygon","coordinates":[[[16.02473850000007,41.425590498000076],[16.542302499000073,41.229440999000076],[16.503909,41.151320498000075],[16.391652500000077,41.196112999000036],[16.202532,40.91751099900006],[15.976693500000067,40.95843049800004],[16.04068650000005,41.036222999000074],[15.870311,41.13989999800003],[16.040297500000065,41.26988649800006],[15.98197650000003,41.360767999000075],[16.03856550000006,41.382622499000036],[15.937005500000055,41.425793998000074],[16.02473850000007,41.425590498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF5","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Basilicata","NUTS_NAME":"Basilicata"},"geometry":{"type":"Polygon","coordinates":[[[16.244098500000064,40.83800849800008],[16.413857,40.70184350000005],[16.501127,40.76227099700003],[16.72499,40.71381449800003],[16.719678499000054,40.481393499000035],[16.867264,40.39784549800004],[16.644317,40.119095998000034],[16.42964950000004,40.13643899900006],[16.398881,40.05539999800004],[16.33783550000004,39.93580699700004],[16.04910299900007,39.89473899800004],[15.982234,39.98406149700003],[15.848727500000052,40.00678749900004],[15.755958500000077,39.92349599900007],[15.644945499000073,40.042790498000045],[15.806531500000062,40.27222249700003],[15.538927500000057,40.48801149800005],[15.53941,40.568025498000054],[15.449538500000074,40.609062497000025],[15.503555499000072,40.66380599900003],[15.381197,40.724950998000054],[15.388417,40.794122998000034],[15.335024500000031,40.834843997000064],[15.377602500000023,40.836103998000056],[15.354225500000041,40.882618498000056],[15.527298500000029,40.907801998000025],[15.572237,40.99836799700006],[15.542921,41.05582350000003],[15.870311,41.13989999800003],[16.04068650000005,41.036222999000074],[15.976693500000067,40.95843049800004],[16.202532,40.91751099900006],[16.244098500000064,40.83800849800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF51","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Potenza","NUTS_NAME":"Potenza"},"geometry":{"type":"Polygon","coordinates":[[[16.048302,40.63554750000003],[16.195569499000044,40.26537149600006],[16.375725,40.31039949900003],[16.34870250000006,40.205377998000074],[16.398881,40.05539999800004],[16.33783550000004,39.93580699700004],[16.04910299900007,39.89473899800004],[15.982234,39.98406149700003],[15.848727500000052,40.00678749900004],[15.755958500000077,39.92349599900007],[15.644945499000073,40.042790498000045],[15.806531500000062,40.27222249700003],[15.538927500000057,40.48801149800005],[15.53941,40.568025498000054],[15.449538500000074,40.609062497000025],[15.503555499000072,40.66380599900003],[15.381197,40.724950998000054],[15.388417,40.794122998000034],[15.335024500000031,40.834843997000064],[15.377602500000023,40.836103998000056],[15.354225500000041,40.882618498000056],[15.527298500000029,40.907801998000025],[15.572237,40.99836799700006],[15.542921,41.05582350000003],[15.870311,41.13989999800003],[16.04068650000005,41.036222999000074],[15.976693500000067,40.95843049800004],[16.202532,40.91751099900006],[16.244098500000064,40.83800849800008],[16.122208,40.796970498000064],[16.145934500000067,40.69568299800005],[16.048302,40.63554750000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF52","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Matera","NUTS_NAME":"Matera"},"geometry":{"type":"Polygon","coordinates":[[[16.244098500000064,40.83800849800008],[16.413857,40.70184350000005],[16.501127,40.76227099700003],[16.72499,40.71381449800003],[16.719678499000054,40.481393499000035],[16.867264,40.39784549800004],[16.644317,40.119095998000034],[16.42964950000004,40.13643899900006],[16.398881,40.05539999800004],[16.34870250000006,40.205377998000074],[16.375725,40.31039949900003],[16.195569499000044,40.26537149600006],[16.048302,40.63554750000003],[16.145934500000067,40.69568299800005],[16.122208,40.796970498000064],[16.244098500000064,40.83800849800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF6","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Calabria","NUTS_NAME":"Calabria"},"geometry":{"type":"Polygon","coordinates":[[[16.62626,39.95547799800005],[16.490359500000068,39.80335299700005],[16.527955500000076,39.66865950000005],[16.770733499000073,39.62110549700003],[17.025064500000042,39.48357849800004],[17.13565,39.37396599900006],[17.10419150000007,39.263840998000035],[17.148327,39.210276],[17.11019250000004,39.09960299800008],[17.188881,39.02047199700007],[17.101278500000035,38.90490999600007],[16.890732,38.92712849800006],[16.566985500000044,38.76548199800004],[16.53582350000005,38.70611849800008],[16.582084500000065,38.46979049600003],[16.488372,38.35253349900006],[16.326207999000076,38.29452150000003],[16.169725500000027,38.14043699800004],[16.063809500000048,37.92441899600004],[15.678212,37.95411949700008],[15.634096500000055,38.01831599900004],[15.636063,38.231473497000025],[15.808353500000067,38.29168949900003],[15.918956,38.50676149800006],[15.928529,38.55029899800007],[15.828991,38.61772449700004],[15.848117500000058,38.658864497000025],[15.983204,38.72185349600005],[16.12969,38.71597199900003],[16.213753500000053,38.81043999800005],[16.215453500000024,38.92621249800004],[16.093153,39.04874299700003],[16.036746,39.349211996000065],[15.873549,39.55545449700003],[15.755958500000077,39.92349599900007],[15.848727500000052,40.00678749900004],[15.982234,39.98406149700003],[16.04910299900007,39.89473899800004],[16.33783550000004,39.93580699700004],[16.398881,40.05539999800004],[16.42964950000004,40.13643899900006],[16.644317,40.119095998000034],[16.59911950000003,40.040231997000035],[16.62626,39.95547799800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF61","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Cosenza","NUTS_NAME":"Cosenza"},"geometry":{"type":"Polygon","coordinates":[[[16.644317,40.119095998000034],[16.59911950000003,40.040231997000035],[16.62626,39.95547799800005],[16.490359500000068,39.80335299700005],[16.527955500000076,39.66865950000005],[16.770733499000073,39.62110549700003],[17.025064500000042,39.48357849800004],[16.868257,39.356808998000076],[16.704999500000042,39.354602998000075],[16.778255,39.195902997000076],[16.613149,39.194210498000075],[16.50351150000006,39.160382998000046],[16.470642500000054,39.05128049800004],[16.32634050000007,39.109420998000076],[16.093153,39.04874299700003],[16.036746,39.349211996000065],[15.873549,39.55545449700003],[15.755958500000077,39.92349599900007],[15.848727500000052,40.00678749900004],[15.982234,39.98406149700003],[16.04910299900007,39.89473899800004],[16.33783550000004,39.93580699700004],[16.398881,40.05539999800004],[16.42964950000004,40.13643899900006],[16.644317,40.119095998000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF62","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Crotone","NUTS_NAME":"Crotone"},"geometry":{"type":"Polygon","coordinates":[[[16.890732,38.92712849800006],[16.916013,38.955655996000075],[16.885375500000066,39.00490099800004],[16.679447500000038,39.09221949700003],[16.613149,39.194210498000075],[16.778255,39.195902997000076],[16.704999500000042,39.354602998000075],[16.868257,39.356808998000076],[17.025064500000042,39.48357849800004],[17.13565,39.37396599900006],[17.10419150000007,39.263840998000035],[17.148327,39.210276],[17.11019250000004,39.09960299800008],[17.188881,39.02047199700007],[17.101278500000035,38.90490999600007],[16.890732,38.92712849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF63","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Catanzaro","NUTS_NAME":"Catanzaro"},"geometry":{"type":"Polygon","coordinates":[[[16.890732,38.92712849800006],[16.566985500000044,38.76548199800004],[16.53582350000005,38.70611849800008],[16.582084500000065,38.46979049600003],[16.407205,38.51401599800005],[16.419985,38.55395199800006],[16.328951,38.64221249800005],[16.352737,38.78895149900006],[16.213753500000053,38.81043999800005],[16.215453500000024,38.92621249800004],[16.093153,39.04874299700003],[16.32634050000007,39.109420998000076],[16.470642500000054,39.05128049800004],[16.50351150000006,39.160382998000046],[16.613149,39.194210498000075],[16.679447500000038,39.09221949700003],[16.885375500000066,39.00490099800004],[16.916013,38.955655996000075],[16.890732,38.92712849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF64","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Vibo Valentia","NUTS_NAME":"Vibo Valentia"},"geometry":{"type":"Polygon","coordinates":[[[16.419985,38.55395199800006],[16.349424,38.53733549800006],[16.392939,38.447877498000025],[16.350220500000034,38.42885149700004],[16.08015,38.56206049700006],[15.918956,38.50676149800006],[15.928529,38.55029899800007],[15.828991,38.61772449700004],[15.848117500000058,38.658864497000025],[15.983204,38.72185349600005],[16.12969,38.71597199900003],[16.213753500000053,38.81043999800005],[16.352737,38.78895149900006],[16.328951,38.64221249800005],[16.419985,38.55395199800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITF65","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Reggio di Calabria","NUTS_NAME":"Reggio di Calabria"},"geometry":{"type":"Polygon","coordinates":[[[16.419985,38.55395199800006],[16.407205,38.51401599800005],[16.582084500000065,38.46979049600003],[16.488372,38.35253349900006],[16.326207999000076,38.29452150000003],[16.169725500000027,38.14043699800004],[16.063809500000048,37.92441899600004],[15.678212,37.95411949700008],[15.634096500000055,38.01831599900004],[15.636063,38.231473497000025],[15.808353500000067,38.29168949900003],[15.918956,38.50676149800006],[16.08015,38.56206049700006],[16.350220500000034,38.42885149700004],[16.392939,38.447877498000025],[16.349424,38.53733549800006],[16.419985,38.55395199800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG","LEVL_CODE":1,"CNTR_CODE":"IT","NAME_LATN":"ISOLE","NUTS_NAME":"ISOLE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[15.572551,38.23420499800005],[15.493701500000043,38.074642],[15.258082500000057,37.80722399700005],[15.176152500000057,37.57575649800003],[15.091007500000046,37.50002999800006],[15.091499,37.35770849900007],[15.255533,37.24181699800005],[15.189416,37.23612499800004],[15.186361,37.19083699600003],[15.296380500000055,37.105878],[15.315962500000069,37.035012497000025],[15.111754500000075,36.85025599800008],[15.116582,36.67495599800003],[15.000358,36.70284749700005],[14.780548500000066,36.70395149700005],[14.493577,36.78673349700006],[14.337594500000023,37.001979998000024],[14.036207,37.10602799800006],[13.750427999000067,37.149429498000075],[13.550732500000038,37.286950500000046],[13.272413,37.38909699900006],[13.17173,37.49062699700005],[13.020311,37.49505049800007],[12.896496,37.57711849700007],[12.672787,37.560080998000046],[12.471194,37.703940497000076],[12.442929500000048,37.81105199800004],[12.561744500000032,38.06556199700003],[12.711114,38.109605997000074],[12.731687,38.18029749800007],[12.811005,38.081515497000055],[12.878819500000077,38.02912599700005],[12.977262500000052,38.04020599800003],[13.078245500000037,38.08820049900004],[13.052464,38.13869699900005],[13.105645,38.19080999900007],[13.191043,38.16928849900006],[13.318283500000064,38.22422299800007],[13.365693500000077,38.182289498000046],[13.383294500000034,38.10926849700007],[13.537639500000068,38.11271549700007],[13.566937500000051,38.03840849900007],[13.74382950000006,37.97024649700006],[14.02708750000005,38.04060899700005],[14.183486,38.01939649800005],[14.526039,38.042452998000044],[14.74790899900006,38.165756997000074],[14.915518500000076,38.192245497000044],[15.113349500000027,38.125076498000055],[15.24130450000007,38.216853997000044],[15.541000500000052,38.30162849900006],[15.652927,38.26710399600006],[15.572551,38.23420499800005]]],[[[9.42346199900004,41.176741],[9.443369,41.09925399900004],[9.566906499000027,41.116017498000076],[9.503958500000067,41.010275498000055],[9.664734500000066,40.996196498000074],[9.498756,40.917446],[9.644384,40.91990549800005],[9.748871999000073,40.66040699700005],[9.82702050000006,40.51211249800008],[9.62652699900002,40.22487699700008],[9.735204500000066,40.075661998000044],[9.687999499000057,39.98521649800006],[9.651118500000052,39.549323997000045],[9.555865,39.13396699600003],[9.109986500000048,39.214018500000066],[9.008827,39.12130449800003],[9.045840500000054,39.05619349800003],[9.011614500000064,38.98710349800007],[8.859653,38.87746949800004],[8.721109,38.93861149900005],[8.646450500000071,38.89431099800004],[8.612392,38.957787997000025],[8.567906,39.042935999000065],[8.469251500000041,39.05392449800007],[8.487763,39.08878149800006],[8.368104499000026,39.21381049900003],[8.436887500000068,39.29004899700004],[8.373012500000073,39.37460649800005],[8.393,39.446529498000075],[8.470979,39.60082999800005],[8.444049,39.758194497000034],[8.50225450000005,39.713006998000026],[8.552179,39.863522998000064],[8.39866150000006,39.904071498000064],[8.413568500000054,40.019162996000034],[8.376406,40.03584549800007],[8.492870500000038,40.08687150000003],[8.457160500000043,40.17520899800007],[8.482863500000065,40.28547049800005],[8.382633500000054,40.338828996000075],[8.399809,40.407568498000046],[8.306376,40.58615149800005],[8.198148,40.57122499700006],[8.193946,40.618244498000024],[8.161289500000066,40.56088349800007],[8.146809500000074,40.62602949800004],[8.204835,40.680237498000054],[8.135010500000021,40.73634899800004],[8.21902,40.86889049800004],[8.17597,40.934798999000066],[8.201709,40.97256649700006],[8.313342500000033,40.844939],[8.41712750000005,40.83831349800005],[8.52743,40.82418799700008],[8.802688500000045,40.931047997000064],[9.01318599900003,41.12518049800008],[9.15260550000005,41.155430496000065],[9.163662,41.235624],[9.42346199900004,41.176741]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG1","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Sicilia","NUTS_NAME":"Sicilia"},"geometry":{"type":"Polygon","coordinates":[[[15.572551,38.23420499800005],[15.493701500000043,38.074642],[15.258082500000057,37.80722399700005],[15.176152500000057,37.57575649800003],[15.091007500000046,37.50002999800006],[15.091499,37.35770849900007],[15.255533,37.24181699800005],[15.189416,37.23612499800004],[15.186361,37.19083699600003],[15.296380500000055,37.105878],[15.315962500000069,37.035012497000025],[15.111754500000075,36.85025599800008],[15.116582,36.67495599800003],[15.000358,36.70284749700005],[14.780548500000066,36.70395149700005],[14.493577,36.78673349700006],[14.337594500000023,37.001979998000024],[14.036207,37.10602799800006],[13.750427999000067,37.149429498000075],[13.550732500000038,37.286950500000046],[13.272413,37.38909699900006],[13.17173,37.49062699700005],[13.020311,37.49505049800007],[12.896496,37.57711849700007],[12.672787,37.560080998000046],[12.471194,37.703940497000076],[12.442929500000048,37.81105199800004],[12.561744500000032,38.06556199700003],[12.711114,38.109605997000074],[12.731687,38.18029749800007],[12.811005,38.081515497000055],[12.878819500000077,38.02912599700005],[12.977262500000052,38.04020599800003],[13.078245500000037,38.08820049900004],[13.052464,38.13869699900005],[13.105645,38.19080999900007],[13.191043,38.16928849900006],[13.318283500000064,38.22422299800007],[13.365693500000077,38.182289498000046],[13.383294500000034,38.10926849700007],[13.537639500000068,38.11271549700007],[13.566937500000051,38.03840849900007],[13.74382950000006,37.97024649700006],[14.02708750000005,38.04060899700005],[14.183486,38.01939649800005],[14.526039,38.042452998000044],[14.74790899900006,38.165756997000074],[14.915518500000076,38.192245497000044],[15.113349500000027,38.125076498000055],[15.24130450000007,38.216853997000044],[15.541000500000052,38.30162849900006],[15.652927,38.26710399600006],[15.572551,38.23420499800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG11","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Trapani","NUTS_NAME":"Trapani"},"geometry":{"type":"Polygon","coordinates":[[[12.811005,38.081515497000055],[12.878819500000077,38.02912599700005],[12.977262500000052,38.04020599800003],[13.053573,37.958657499000026],[12.95412250000004,37.897085497000035],[12.94587,37.83883499800004],[13.100284500000043,37.77107799800007],[13.012097500000039,37.74267149900004],[12.894702,37.660761998000055],[12.896496,37.57711849700007],[12.672787,37.560080998000046],[12.471194,37.703940497000076],[12.442929500000048,37.81105199800004],[12.561744500000032,38.06556199700003],[12.711114,38.109605997000074],[12.731687,38.18029749800007],[12.811005,38.081515497000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG12","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Palermo","NUTS_NAME":"Palermo"},"geometry":{"type":"Polygon","coordinates":[[[13.365693500000077,38.182289498000046],[13.383294500000034,38.10926849700007],[13.537639500000068,38.11271549700007],[13.566937500000051,38.03840849900007],[13.74382950000006,37.97024649700006],[14.02708750000005,38.04060899700005],[14.183486,38.01939649800005],[14.27997,37.84009149800005],[14.268567,37.70454299900007],[14.135052500000029,37.616705498000044],[13.89929950000004,37.61814399700006],[13.808151,37.745115499000065],[13.73252750000006,37.68296599800004],[13.569134,37.635940498000025],[13.394994500000053,37.659568497000066],[13.40384750000004,37.56947249800004],[13.356429,37.539224998000066],[13.348253500000055,37.65700399900004],[13.252279,37.61178399800008],[13.187789500000065,37.69162599800006],[13.012097500000039,37.74267149900004],[13.100284500000043,37.77107799800007],[12.94587,37.83883499800004],[12.95412250000004,37.897085497000035],[13.053573,37.958657499000026],[12.977262500000052,38.04020599800003],[13.078245500000037,38.08820049900004],[13.052464,38.13869699900005],[13.105645,38.19080999900007],[13.191043,38.16928849900006],[13.318283500000064,38.22422299800007],[13.365693500000077,38.182289498000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG13","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Messina","NUTS_NAME":"Messina"},"geometry":{"type":"Polygon","coordinates":[[[15.572551,38.23420499800005],[15.493701500000043,38.074642],[15.258082500000057,37.80722399700005],[15.153630500000077,37.89504799700006],[15.002247,37.89782499700004],[14.941245,37.950788998000064],[14.764789,37.940228998000066],[14.7387,37.84746299600005],[14.802672500000028,37.81566399700006],[14.742269,37.81813199800007],[14.617697,37.80491449700003],[14.549224500000037,37.87525399800006],[14.474725500000034,37.810779998000044],[14.426285500000063,37.85310599700006],[14.27997,37.84009149800005],[14.183486,38.01939649800005],[14.526039,38.042452998000044],[14.74790899900006,38.165756997000074],[14.915518500000076,38.192245497000044],[15.113349500000027,38.125076498000055],[15.24130450000007,38.216853997000044],[15.541000500000052,38.30162849900006],[15.652927,38.26710399600006],[15.572551,38.23420499800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG14","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Agrigento","NUTS_NAME":"Agrigento"},"geometry":{"type":"Polygon","coordinates":[[[13.187789500000065,37.69162599800006],[13.252279,37.61178399800008],[13.348253500000055,37.65700399900004],[13.356429,37.539224998000066],[13.40384750000004,37.56947249800004],[13.394994500000053,37.659568497000066],[13.569134,37.635940498000025],[13.73252750000006,37.68296599800004],[13.823527500000068,37.67506099700006],[13.830910500000073,37.61859699900003],[13.703936500000054,37.58283899700007],[13.650620500000059,37.461170999000046],[13.844521,37.42642849900005],[13.910206500000072,37.386753999000064],[13.881666,37.359842997000044],[14.03153050000003,37.292187496000054],[13.982346,37.19306799800006],[14.036207,37.10602799800006],[13.750427999000067,37.149429498000075],[13.550732500000038,37.286950500000046],[13.272413,37.38909699900006],[13.17173,37.49062699700005],[13.020311,37.49505049800007],[12.896496,37.57711849700007],[12.894702,37.660761998000055],[13.012097500000039,37.74267149900004],[13.187789500000065,37.69162599800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG15","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Caltanissetta","NUTS_NAME":"Caltanissetta"},"geometry":{"type":"Polygon","coordinates":[[[13.89929950000004,37.61814399700006],[14.135052500000029,37.616705498000044],[14.164418,37.612206498000035],[14.13678150000004,37.47302549800003],[14.056007500000021,37.40688149600004],[14.089263,37.343747499000074],[14.352670500000045,37.30613999800005],[14.381465,37.19219749800004],[14.476427500000057,37.13448049700003],[14.444122500000049,37.052950500000065],[14.337594500000023,37.001979998000024],[14.036207,37.10602799800006],[13.982346,37.19306799800006],[14.03153050000003,37.292187496000054],[13.881666,37.359842997000044],[13.910206500000072,37.386753999000064],[13.844521,37.42642849900005],[13.650620500000059,37.461170999000046],[13.703936500000054,37.58283899700007],[13.830910500000073,37.61859699900003],[13.823527500000068,37.67506099700006],[13.73252750000006,37.68296599800004],[13.808151,37.745115499000065],[13.89929950000004,37.61814399700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG16","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Enna","NUTS_NAME":"Enna"},"geometry":{"type":"Polygon","coordinates":[[[14.742269,37.81813199800007],[14.724423,37.68134899700004],[14.797874,37.71272749900004],[14.825735,37.63066249600007],[14.792905500000074,37.510496500000045],[14.544221,37.54287050000005],[14.527960500000063,37.45892199800005],[14.633579500000053,37.42464750000005],[14.502693,37.30913349700006],[14.44677,37.34754249700006],[14.454231500000049,37.303157999000064],[14.352670500000045,37.30613999800005],[14.089263,37.343747499000074],[14.056007500000021,37.40688149600004],[14.13678150000004,37.47302549800003],[14.164418,37.612206498000035],[14.135052500000029,37.616705498000044],[14.268567,37.70454299900007],[14.27997,37.84009149800005],[14.426285500000063,37.85310599700006],[14.474725500000034,37.810779998000044],[14.549224500000037,37.87525399800006],[14.617697,37.80491449700003],[14.742269,37.81813199800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG17","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Catania","NUTS_NAME":"Catania"},"geometry":{"type":"Polygon","coordinates":[[[15.258082500000057,37.80722399700005],[15.176152500000057,37.57575649800003],[15.091007500000046,37.50002999800006],[15.091499,37.35770849900007],[14.887514500000066,37.407754999000076],[14.778590500000064,37.33047399700007],[14.892172500000072,37.311026],[14.774491500000067,37.228677498000025],[14.871487500000057,37.20413999900006],[14.843895499000041,37.13027499800006],[14.793879,37.13205299700007],[14.444122500000049,37.052950500000065],[14.476427500000057,37.13448049700003],[14.381465,37.19219749800004],[14.352670500000045,37.30613999800005],[14.454231500000049,37.303157999000064],[14.44677,37.34754249700006],[14.502693,37.30913349700006],[14.633579500000053,37.42464750000005],[14.527960500000063,37.45892199800005],[14.544221,37.54287050000005],[14.792905500000074,37.510496500000045],[14.825735,37.63066249600007],[14.797874,37.71272749900004],[14.724423,37.68134899700004],[14.742269,37.81813199800007],[14.802672500000028,37.81566399700006],[14.7387,37.84746299600005],[14.764789,37.940228998000066],[14.941245,37.950788998000064],[15.002247,37.89782499700004],[15.153630500000077,37.89504799700006],[15.258082500000057,37.80722399700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG18","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ragusa","NUTS_NAME":"Ragusa"},"geometry":{"type":"Polygon","coordinates":[[[14.793879,37.13205299700007],[14.937812,36.90333299800005],[14.861816500000032,36.81831449900005],[14.988681500000041,36.77654149800003],[15.000358,36.70284749700005],[14.780548500000066,36.70395149700005],[14.493577,36.78673349700006],[14.337594500000023,37.001979998000024],[14.444122500000049,37.052950500000065],[14.793879,37.13205299700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG19","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Siracusa","NUTS_NAME":"Siracusa"},"geometry":{"type":"Polygon","coordinates":[[[15.091499,37.35770849900007],[15.255533,37.24181699800005],[15.189416,37.23612499800004],[15.186361,37.19083699600003],[15.296380500000055,37.105878],[15.315962500000069,37.035012497000025],[15.111754500000075,36.85025599800008],[15.116582,36.67495599800003],[15.000358,36.70284749700005],[14.988681500000041,36.77654149800003],[14.861816500000032,36.81831449900005],[14.937812,36.90333299800005],[14.793879,37.13205299700007],[14.843895499000041,37.13027499800006],[14.871487500000057,37.20413999900006],[14.774491500000067,37.228677498000025],[14.892172500000072,37.311026],[14.778590500000064,37.33047399700007],[14.887514500000066,37.407754999000076],[15.091499,37.35770849900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG2","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Sardegna","NUTS_NAME":"Sardegna"},"geometry":{"type":"Polygon","coordinates":[[[9.42346199900004,41.176741],[9.443369,41.09925399900004],[9.566906499000027,41.116017498000076],[9.503958500000067,41.010275498000055],[9.664734500000066,40.996196498000074],[9.498756,40.917446],[9.644384,40.91990549800005],[9.748871999000073,40.66040699700005],[9.82702050000006,40.51211249800008],[9.62652699900002,40.22487699700008],[9.735204500000066,40.075661998000044],[9.687999499000057,39.98521649800006],[9.651118500000052,39.549323997000045],[9.555865,39.13396699600003],[9.109986500000048,39.214018500000066],[9.008827,39.12130449800003],[9.045840500000054,39.05619349800003],[9.011614500000064,38.98710349800007],[8.859653,38.87746949800004],[8.721109,38.93861149900005],[8.646450500000071,38.89431099800004],[8.612392,38.957787997000025],[8.567906,39.042935999000065],[8.469251500000041,39.05392449800007],[8.487763,39.08878149800006],[8.368104499000026,39.21381049900003],[8.436887500000068,39.29004899700004],[8.373012500000073,39.37460649800005],[8.393,39.446529498000075],[8.470979,39.60082999800005],[8.444049,39.758194497000034],[8.50225450000005,39.713006998000026],[8.552179,39.863522998000064],[8.39866150000006,39.904071498000064],[8.413568500000054,40.019162996000034],[8.376406,40.03584549800007],[8.492870500000038,40.08687150000003],[8.457160500000043,40.17520899800007],[8.482863500000065,40.28547049800005],[8.382633500000054,40.338828996000075],[8.399809,40.407568498000046],[8.306376,40.58615149800005],[8.198148,40.57122499700006],[8.193946,40.618244498000024],[8.161289500000066,40.56088349800007],[8.146809500000074,40.62602949800004],[8.204835,40.680237498000054],[8.135010500000021,40.73634899800004],[8.21902,40.86889049800004],[8.17597,40.934798999000066],[8.201709,40.97256649700006],[8.313342500000033,40.844939],[8.41712750000005,40.83831349800005],[8.52743,40.82418799700008],[8.802688500000045,40.931047997000064],[9.01318599900003,41.12518049800008],[9.15260550000005,41.155430496000065],[9.163662,41.235624],[9.42346199900004,41.176741]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG25","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Sassari","NUTS_NAME":"Sassari"},"geometry":{"type":"Polygon","coordinates":[[[8.41712750000005,40.83831349800005],[8.52743,40.82418799700008],[8.802688500000045,40.931047997000064],[8.971572500000036,40.95497799700007],[8.981091499000058,40.90313899800003],[8.896273429000075,40.871503751000034],[9.027442,40.795912497000074],[9.011948500000074,40.680744498000024],[9.175360500000068,40.66176299800003],[9.189468,40.53682749800004],[9.282608,40.47876350000007],[9.028019,40.29659949900008],[8.832827500000064,40.412393500000064],[8.782712,40.34978049800003],[8.620306500000027,40.31897850000007],[8.514485,40.421467],[8.399809,40.407568498000046],[8.306376,40.58615149800005],[8.198148,40.57122499700006],[8.193946,40.618244498000024],[8.161289500000066,40.56088349800007],[8.146809500000074,40.62602949800004],[8.204835,40.680237498000054],[8.135010500000021,40.73634899800004],[8.21902,40.86889049800004],[8.17597,40.934798999000066],[8.201709,40.97256649700006],[8.313342500000033,40.844939],[8.41712750000005,40.83831349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG26","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Nuoro","NUTS_NAME":"Nuoro"},"geometry":{"type":"Polygon","coordinates":[[[9.748871999000073,40.66040699700005],[9.82702050000006,40.51211249800008],[9.62652699900002,40.22487699700008],[9.525955,40.21865999700003],[9.429505500000062,40.10205749800008],[9.336921,40.072381998000026],[9.280398500000047,39.91739849800007],[9.213035,39.914713998000025],[9.200221,39.82631599600006],[9.165613,39.83256049800008],[9.128848,39.88852249800004],[8.988095500000043,39.914795499000036],[8.993460500000026,39.986953999000036],[8.945665,40.01320499900004],[9.041103,40.05507649700007],[9.010634500000037,40.20000749800005],[8.672723500000075,40.20437949700005],[8.620306500000027,40.31897850000007],[8.782712,40.34978049800003],[8.832827500000064,40.412393500000064],[9.028019,40.29659949900008],[9.282608,40.47876350000007],[9.189468,40.53682749800004],[9.286468500000069,40.525873997000076],[9.42019,40.65826399700006],[9.583139500000073,40.70603649800006],[9.748871999000073,40.66040699700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG27","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Cagliari","NUTS_NAME":"Cagliari"},"geometry":{"type":"Polygon","coordinates":[[[9.651118500000052,39.549323997000045],[9.555865,39.13396699600003],[9.109986500000048,39.214018500000066],[9.008827,39.12130449800003],[9.045840500000054,39.05619349800003],[9.011614500000064,38.98710349800007],[8.859653,38.87746949800004],[8.721109,38.93861149900005],[8.646450500000071,38.89431099800004],[8.612392,38.957787997000025],[8.659946,39.039539],[8.853132,39.08453899800003],[8.758236500000066,39.20096999900005],[8.699288,39.37497649800008],[8.97827,39.42208999800005],[9.07368150000002,39.75027749600008],[9.003260500000067,39.76061149700007],[9.106292,39.84701049800003],[9.165613,39.83256049800008],[9.200221,39.82631599600006],[9.213035,39.914713998000025],[9.280398500000047,39.91739849800007],[9.292432500000075,39.813523],[9.397308,39.73654649800005],[9.419351,39.62687149800007],[9.651118500000052,39.549323997000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG28","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Oristano","NUTS_NAME":"Oristano"},"geometry":{"type":"Polygon","coordinates":[[[8.620306500000027,40.31897850000007],[8.672723500000075,40.20437949700005],[9.010634500000037,40.20000749800005],[9.041103,40.05507649700007],[8.945665,40.01320499900004],[8.993460500000026,39.986953999000036],[8.988095500000043,39.914795499000036],[9.128848,39.88852249800004],[9.165613,39.83256049800008],[9.106292,39.84701049800003],[9.003260500000067,39.76061149700007],[8.939079500000048,39.78298649800007],[8.861800500000072,39.66822549600005],[8.671323,39.63121149800003],[8.50225450000005,39.713006998000026],[8.552179,39.863522998000064],[8.39866150000006,39.904071498000064],[8.413568500000054,40.019162996000034],[8.376406,40.03584549800007],[8.492870500000038,40.08687150000003],[8.457160500000043,40.17520899800007],[8.482863500000065,40.28547049800005],[8.382633500000054,40.338828996000075],[8.399809,40.407568498000046],[8.514485,40.421467],[8.620306500000027,40.31897850000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG29","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Olbia-Tempio","NUTS_NAME":"Olbia-Tempio"},"geometry":{"type":"Polygon","coordinates":[[[9.42346199900004,41.176741],[9.443369,41.09925399900004],[9.566906499000027,41.116017498000076],[9.503958500000067,41.010275498000055],[9.664734500000066,40.996196498000074],[9.498756,40.917446],[9.644384,40.91990549800005],[9.748871999000073,40.66040699700005],[9.583139500000073,40.70603649800006],[9.42019,40.65826399700006],[9.286468500000069,40.525873997000076],[9.189468,40.53682749800004],[9.175360500000068,40.66176299800003],[9.011948500000074,40.680744498000024],[9.027442,40.795912497000074],[8.896273429000075,40.871503751000034],[8.981091499000058,40.90313899800003],[8.971572500000036,40.95497799700007],[8.802688500000045,40.931047997000064],[9.01318599900003,41.12518049800008],[9.15260550000005,41.155430496000065],[9.163662,41.235624],[9.42346199900004,41.176741]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG2A","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ogliastra","NUTS_NAME":"Ogliastra"},"geometry":{"type":"Polygon","coordinates":[[[9.651118500000052,39.549323997000045],[9.419351,39.62687149800007],[9.397308,39.73654649800005],[9.292432500000075,39.813523],[9.280398500000047,39.91739849800007],[9.336921,40.072381998000026],[9.429505500000062,40.10205749800008],[9.525955,40.21865999700003],[9.62652699900002,40.22487699700008],[9.735204500000066,40.075661998000044],[9.687999499000057,39.98521649800006],[9.651118500000052,39.549323997000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG2B","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Medio Campidano","NUTS_NAME":"Medio Campidano"},"geometry":{"type":"Polygon","coordinates":[[[9.003260500000067,39.76061149700007],[9.07368150000002,39.75027749600008],[8.97827,39.42208999800005],[8.699288,39.37497649800008],[8.495952500000044,39.483071497000026],[8.393,39.446529498000075],[8.470979,39.60082999800005],[8.444049,39.758194497000034],[8.50225450000005,39.713006998000026],[8.671323,39.63121149800003],[8.861800500000072,39.66822549600005],[8.939079500000048,39.78298649800007],[9.003260500000067,39.76061149700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITG2C","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Carbonia-Iglesias","NUTS_NAME":"Carbonia-Iglesias"},"geometry":{"type":"Polygon","coordinates":[[[8.659946,39.039539],[8.612392,38.957787997000025],[8.567906,39.042935999000065],[8.469251500000041,39.05392449800007],[8.487763,39.08878149800006],[8.368104499000026,39.21381049900003],[8.436887500000068,39.29004899700004],[8.373012500000073,39.37460649800005],[8.393,39.446529498000075],[8.495952500000044,39.483071497000026],[8.699288,39.37497649800008],[8.758236500000066,39.20096999900005],[8.853132,39.08453899800003],[8.659946,39.039539]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH","LEVL_CODE":1,"CNTR_CODE":"IT","NAME_LATN":"NORD-EST","NUTS_NAME":"NORD-EST"},"geometry":{"type":"Polygon","coordinates":[[[12.121343500000023,47.00700199800008],[12.143811,46.91377999700006],[12.273703,46.88438799800008],[12.312809,46.78386599600003],[12.477924,46.679835497000056],[12.690635,46.656972],[12.731392999000036,46.63428799900004],[13.504249500000071,46.566303997000034],[13.714184999000054,46.522703499000045],[13.684032,46.43747250000007],[13.375492500000064,46.29823249800006],[13.423357,46.206053997000026],[13.664347500000076,46.177549998000075],[13.496938999000065,46.05133499800007],[13.476164999000048,46.00608149800007],[13.526538,45.97231849800005],[13.643036,45.98270199800004],[13.575802500000066,45.847736498000074],[13.597145,45.81952250000006],[13.596243,45.80793750000004],[13.795683500000052,45.74320449900006],[13.918656500000054,45.63351749800006],[13.850761500000033,45.58493249600008],[13.722823500000061,45.59472549600008],[13.799811,45.60977199600006],[13.753360500000042,45.680596998000055],[13.579783,45.786951498000064],[13.520863500000075,45.762534498000036],[13.53142,45.72383149800004],[13.373012,45.68000649800007],[13.459073076000038,45.70803148200008],[13.406493500000067,45.725175],[13.130853,45.77184449900005],[13.067232500000046,45.710487999000065],[13.098785501000066,45.644532996000066],[12.43416250000007,45.424485996000044],[12.486428499000056,45.49952449700004],[12.631836,45.53433849800007],[12.413683999000057,45.54431050000005],[12.164675500000044,45.39072949900003],[12.132562,45.30037049900005],[12.200494,45.257336999000074],[12.204566500000055,45.19767599700003],[12.298646500000075,45.21860999900008],[12.330571500000076,45.16055049900007],[12.328777,45.151860998000075],[12.332355500000062,45.09098350000005],[12.302155606000042,45.13788754600006],[12.279706043000033,45.123266827000066],[12.31234899900005,44.96700849800004],[12.409864,45.031746499000064],[12.494037,44.97136599700008],[12.485925,44.895982497000034],[12.389819500000044,44.87723599900005],[12.399056500000029,44.79261550000007],[12.265868500000067,44.82332599800003],[12.239688,44.69413849700004],[12.269892,44.630007496000076],[12.384282,44.22472649900004],[12.450349,44.16218949600005],[12.75078,43.97060150000004],[12.733553500000028,43.869933497000034],[12.669299,43.82322749800005],[12.493956,43.91554650000006],[12.513435500000071,43.99126949600003],[12.404607500000054,43.95567850000003],[12.4177,43.89902999600008],[12.283797500000048,43.764898997000046],[12.107464,43.75375349600006],[11.986528500000077,43.76191399700008],[11.710189,43.877430497000034],[11.646041500000024,43.99065699600004],[11.752958,44.12051850000006],[11.715906500000074,44.122540498000035],[11.524959500000023,44.157639998000036],[11.421995,44.23987949800005],[11.195749999000043,44.15139849800005],[11.263648500000045,44.10422449600003],[11.202439500000025,44.10072099900003],[11.049449,44.09022999800004],[11.004782,44.12143099600007],[10.917739,44.06276249700005],[10.814787,44.11617449800008],[10.653418,44.15778149800008],[10.624078,44.12035150000003],[10.470149,44.226041],[10.253875,44.26856699800004],[10.142054500000029,44.353852998000036],[9.996692,44.39711249800007],[9.96213,44.467052996000064],[9.81996,44.46640549800003],[9.686725500000023,44.36592199800003],[9.577126500000077,44.43523699900004],[9.479059500000062,44.40924049900008],[9.439189,44.42905349800003],[9.49682850000005,44.48271250000005],[9.493363,44.555858998000076],[9.202995500000043,44.61347649800007],[9.200074,44.686099496000054],[9.32466050000005,44.689999999000065],[9.286998,44.772481999000036],[9.363122,44.82239099800006],[9.27743350000003,44.895598],[9.293407,44.93250949800006],[9.37082950000007,45.048136998000075],[9.532049500000028,45.07842749800005],[9.548684500000036,45.13264849800004],[9.713752,45.058392998000045],[9.895781,45.07964199600008],[9.891101500000048,45.13089799900007],[9.988977499000043,45.13210999800003],[10.083503,45.04395850000003],[10.464030500000035,44.937171500000034],[10.504344,44.92241799800007],[10.730587,44.991889498000035],[10.887909500000035,44.914265998000076],[11.246231500000022,44.951427996000064],[11.426765500000045,44.950075999000035],[11.175304,45.06006149800004],[11.205502500000023,45.10948749800008],[11.028722500000072,45.123041],[11.050501,45.14956749800007],[10.936780500000054,45.232916497000076],[10.782865,45.315409499000054],[10.728336,45.291306500000076],[10.714573500000029,45.41758599800005],[10.654655500000047,45.41582699800006],[10.631074,45.60951250000005],[10.840176500000041,45.83275850000007],[10.561458500000072,45.78391799900004],[10.502388,45.830372999000076],[10.508597500000064,45.92356549800007],[10.452963,45.989621998000075],[10.585728,46.24794349800004],[10.515751500000022,46.34322449600006],[10.632285500000023,46.40139],[10.622145500000045,46.44810199800003],[10.452801,46.53068299800003],[10.492064,46.61538649700003],[10.402287999000066,46.63681199600006],[10.39332150000007,46.68920749600005],[10.469651500000055,46.85490899900003],[10.667368500000066,46.87549049800003],[10.757589500000051,46.78518599900008],[11.02225,46.76541049800005],[11.164281500000072,46.96572249900004],[11.627199500000074,47.01329899900003],[11.747028500000056,46.96887249800005],[12.136014,47.08066749900007],[12.24074550000006,47.06916849800007],[12.121343500000023,47.00700199800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH1","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Provincia Autonoma di Bolzano/Bozen","NUTS_NAME":"Provincia Autonoma di Bolzano/Bozen"},"geometry":{"type":"Polygon","coordinates":[[[12.24074550000006,47.06916849800007],[12.121343500000023,47.00700199800008],[12.143811,46.91377999700006],[12.273703,46.88438799800008],[12.312809,46.78386599600003],[12.477924,46.679835497000056],[12.192270500000063,46.59456849800006],[12.068302500000073,46.675115498000025],[11.998413,46.532861998000044],[11.828336500000034,46.50891449800008],[11.641985500000033,46.492654498000036],[11.550103500000034,46.34461249800006],[11.476937500000076,46.36392299900007],[11.20649,46.21977949600006],[11.13886350000007,46.28332849800006],[11.203599,46.342358500000046],[11.220452500000022,46.46205699800004],[11.186909,46.508897997000076],[11.06138899900003,46.48896549800003],[11.074952500000052,46.44237349800005],[10.978699500000062,46.48395399900005],[10.622145500000045,46.44810199800003],[10.452801,46.53068299800003],[10.492064,46.61538649700003],[10.402287999000066,46.63681199600006],[10.39332150000007,46.68920749600005],[10.469651500000055,46.85490899900003],[10.667368500000066,46.87549049800003],[10.757589500000051,46.78518599900008],[11.02225,46.76541049800005],[11.164281500000072,46.96572249900004],[11.627199500000074,47.01329899900003],[11.747028500000056,46.96887249800005],[12.136014,47.08066749900007],[12.24074550000006,47.06916849800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH10","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Bolzano-Bozen","NUTS_NAME":"Bolzano-Bozen"},"geometry":{"type":"Polygon","coordinates":[[[12.24074550000006,47.06916849800007],[12.121343500000023,47.00700199800008],[12.143811,46.91377999700006],[12.273703,46.88438799800008],[12.312809,46.78386599600003],[12.477924,46.679835497000056],[12.192270500000063,46.59456849800006],[12.068302500000073,46.675115498000025],[11.998413,46.532861998000044],[11.828336500000034,46.50891449800008],[11.641985500000033,46.492654498000036],[11.550103500000034,46.34461249800006],[11.476937500000076,46.36392299900007],[11.20649,46.21977949600006],[11.13886350000007,46.28332849800006],[11.203599,46.342358500000046],[11.220452500000022,46.46205699800004],[11.186909,46.508897997000076],[11.06138899900003,46.48896549800003],[11.074952500000052,46.44237349800005],[10.978699500000062,46.48395399900005],[10.622145500000045,46.44810199800003],[10.452801,46.53068299800003],[10.492064,46.61538649700003],[10.402287999000066,46.63681199600006],[10.39332150000007,46.68920749600005],[10.469651500000055,46.85490899900003],[10.667368500000066,46.87549049800003],[10.757589500000051,46.78518599900008],[11.02225,46.76541049800005],[11.164281500000072,46.96572249900004],[11.627199500000074,47.01329899900003],[11.747028500000056,46.96887249800005],[12.136014,47.08066749900007],[12.24074550000006,47.06916849800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH2","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Provincia Autonoma di Trento","NUTS_NAME":"Provincia Autonoma di Trento"},"geometry":{"type":"Polygon","coordinates":[[[11.828336500000034,46.50891449800008],[11.888829,46.442007998000065],[11.774402500000065,46.35823799600007],[11.962298500000031,46.18786099700003],[11.884214,46.11700750000006],[11.68532250000004,46.093570499000066],[11.684318500000074,45.98407799800003],[11.373413500000026,45.983213996000075],[11.349281499000028,45.909543496000026],[11.258357500000045,45.920678499000076],[11.138337500000034,45.69708849800003],[10.843867500000044,45.718593998000074],[10.887604,45.811162499000034],[10.840176500000041,45.83275850000007],[10.561458500000072,45.78391799900004],[10.502388,45.830372999000076],[10.508597500000064,45.92356549800007],[10.452963,45.989621998000075],[10.585728,46.24794349800004],[10.515751500000022,46.34322449600006],[10.632285500000023,46.40139],[10.622145500000045,46.44810199800003],[10.978699500000062,46.48395399900005],[11.074952500000052,46.44237349800005],[11.06138899900003,46.48896549800003],[11.186909,46.508897997000076],[11.220452500000022,46.46205699800004],[11.203599,46.342358500000046],[11.13886350000007,46.28332849800006],[11.20649,46.21977949600006],[11.476937500000076,46.36392299900007],[11.550103500000034,46.34461249800006],[11.641985500000033,46.492654498000036],[11.828336500000034,46.50891449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH20","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Trento","NUTS_NAME":"Trento"},"geometry":{"type":"Polygon","coordinates":[[[11.828336500000034,46.50891449800008],[11.888829,46.442007998000065],[11.774402500000065,46.35823799600007],[11.962298500000031,46.18786099700003],[11.884214,46.11700750000006],[11.68532250000004,46.093570499000066],[11.684318500000074,45.98407799800003],[11.373413500000026,45.983213996000075],[11.349281499000028,45.909543496000026],[11.258357500000045,45.920678499000076],[11.138337500000034,45.69708849800003],[10.843867500000044,45.718593998000074],[10.887604,45.811162499000034],[10.840176500000041,45.83275850000007],[10.561458500000072,45.78391799900004],[10.502388,45.830372999000076],[10.508597500000064,45.92356549800007],[10.452963,45.989621998000075],[10.585728,46.24794349800004],[10.515751500000022,46.34322449600006],[10.632285500000023,46.40139],[10.622145500000045,46.44810199800003],[10.978699500000062,46.48395399900005],[11.074952500000052,46.44237349800005],[11.06138899900003,46.48896549800003],[11.186909,46.508897997000076],[11.220452500000022,46.46205699800004],[11.203599,46.342358500000046],[11.13886350000007,46.28332849800006],[11.20649,46.21977949600006],[11.476937500000076,46.36392299900007],[11.550103500000034,46.34461249800006],[11.641985500000033,46.492654498000036],[11.828336500000034,46.50891449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH3","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Veneto","NUTS_NAME":"Veneto"},"geometry":{"type":"Polygon","coordinates":[[[12.731392999000036,46.63428799900004],[12.749661501000048,46.54399550000005],[12.498703,46.41222250000004],[12.320955499000036,46.26891449900006],[12.495829,46.15256049800007],[12.400534500000049,46.04199650000004],[12.425841500000047,45.94739499800005],[12.661673,45.792440998000075],[12.779838,45.854448500000046],[12.979058,45.834048996000035],[13.098785501000066,45.644532996000066],[12.43416250000007,45.424485996000044],[12.486428499000056,45.49952449700004],[12.631836,45.53433849800007],[12.413683999000057,45.54431050000005],[12.164675500000044,45.39072949900003],[12.132562,45.30037049900005],[12.200494,45.257336999000074],[12.204566500000055,45.19767599700003],[12.298646500000075,45.21860999900008],[12.330571500000076,45.16055049900007],[12.328777,45.151860998000075],[12.332355500000062,45.09098350000005],[12.302155606000042,45.13788754600006],[12.279706043000033,45.123266827000066],[12.31234899900005,44.96700849800004],[12.409864,45.031746499000064],[12.494037,44.97136599700008],[12.485925,44.895982497000034],[12.389819500000044,44.87723599900005],[12.399056500000029,44.79261550000007],[12.295169500000043,44.859279496000056],[12.280897500000037,44.941755998000076],[12.098028500000055,44.971468998000034],[11.804749,44.97694949800007],[11.624835,44.889211500000044],[11.426765500000045,44.950075999000035],[11.175304,45.06006149800004],[11.205502500000023,45.10948749800008],[11.028722500000072,45.123041],[11.050501,45.14956749800007],[10.936780500000054,45.232916497000076],[10.782865,45.315409499000054],[10.728336,45.291306500000076],[10.714573500000029,45.41758599800005],[10.654655500000047,45.41582699800006],[10.631074,45.60951250000005],[10.840176500000041,45.83275850000007],[10.887604,45.811162499000034],[10.843867500000044,45.718593998000074],[11.138337500000034,45.69708849800003],[11.258357500000045,45.920678499000076],[11.349281499000028,45.909543496000026],[11.373413500000026,45.983213996000075],[11.684318500000074,45.98407799800003],[11.68532250000004,46.093570499000066],[11.884214,46.11700750000006],[11.962298500000031,46.18786099700003],[11.774402500000065,46.35823799600007],[11.888829,46.442007998000065],[11.828336500000034,46.50891449800008],[11.998413,46.532861998000044],[12.068302500000073,46.675115498000025],[12.192270500000063,46.59456849800006],[12.477924,46.679835497000056],[12.690635,46.656972],[12.731392999000036,46.63428799900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH31","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Verona","NUTS_NAME":"Verona"},"geometry":{"type":"Polygon","coordinates":[[[11.138337500000034,45.69708849800003],[11.32383,45.49399449800006],[11.352456500000073,45.34789049800003],[11.487054500000056,45.26595199800005],[11.390903500000036,45.25262349800005],[11.416932,45.12932899800006],[11.445834499000057,45.08066349600006],[11.40244,45.05299499900008],[11.205502500000023,45.10948749800008],[11.028722500000072,45.123041],[11.050501,45.14956749800007],[10.936780500000054,45.232916497000076],[10.782865,45.315409499000054],[10.728336,45.291306500000076],[10.714573500000029,45.41758599800005],[10.654655500000047,45.41582699800006],[10.631074,45.60951250000005],[10.840176500000041,45.83275850000007],[10.887604,45.811162499000034],[10.843867500000044,45.718593998000074],[11.138337500000034,45.69708849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH32","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Vicenza","NUTS_NAME":"Vicenza"},"geometry":{"type":"Polygon","coordinates":[[[11.684318500000074,45.98407799800003],[11.799595500000066,45.88055799800003],[11.735801461000051,45.83604295200007],[11.820648500000061,45.77882499800006],[11.821666500000049,45.687179496000056],[11.646586,45.63419799600007],[11.657772500000021,45.54017299600008],[11.758991,45.52041649700004],[11.615733500000033,45.38802149600008],[11.580114500000036,45.28257749900007],[11.617731500000048,45.258523498000045],[11.487054500000056,45.26595199800005],[11.352456500000073,45.34789049800003],[11.32383,45.49399449800006],[11.138337500000034,45.69708849800003],[11.258357500000045,45.920678499000076],[11.349281499000028,45.909543496000026],[11.373413500000026,45.983213996000075],[11.684318500000074,45.98407799800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH33","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Belluno","NUTS_NAME":"Belluno"},"geometry":{"type":"Polygon","coordinates":[[[12.690635,46.656972],[12.731392999000036,46.63428799900004],[12.749661501000048,46.54399550000005],[12.498703,46.41222250000004],[12.320955499000036,46.26891449900006],[12.495829,46.15256049800007],[12.400534500000049,46.04199650000004],[12.30280349900005,46.07635449600008],[11.961599,45.95467899800008],[11.950563,45.895793500000025],[11.799595500000066,45.88055799800003],[11.684318500000074,45.98407799800003],[11.68532250000004,46.093570499000066],[11.884214,46.11700750000006],[11.962298500000031,46.18786099700003],[11.774402500000065,46.35823799600007],[11.888829,46.442007998000065],[11.828336500000034,46.50891449800008],[11.998413,46.532861998000044],[12.068302500000073,46.675115498000025],[12.192270500000063,46.59456849800006],[12.477924,46.679835497000056],[12.690635,46.656972]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH34","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Treviso","NUTS_NAME":"Treviso"},"geometry":{"type":"Polygon","coordinates":[[[12.400534500000049,46.04199650000004],[12.425841500000047,45.94739499800005],[12.661673,45.792440998000075],[12.685051,45.71299250000004],[12.503972500000032,45.68757399900005],[12.424055499000076,45.62081199800008],[12.439573500000051,45.559267498000054],[12.354873500000053,45.588891498000066],[12.246226,45.53050649800008],[12.092476500000032,45.598335],[11.821666500000049,45.687179496000056],[11.820648500000061,45.77882499800006],[11.735801461000051,45.83604295200007],[11.799595500000066,45.88055799800003],[11.950563,45.895793500000025],[11.961599,45.95467899800008],[12.30280349900005,46.07635449600008],[12.400534500000049,46.04199650000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH35","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Venezia","NUTS_NAME":"Venezia"},"geometry":{"type":"MultiPolygon","coordinates":[[[[13.098785501000066,45.644532996000066],[12.43416250000007,45.424485996000044],[12.486428499000056,45.49952449700004],[12.631836,45.53433849800007],[12.413683999000057,45.54431050000005],[12.164675500000044,45.39072949900003],[12.132562,45.30037049900005],[12.028681,45.317453500000056],[11.975058,45.39660199800005],[12.007612,45.53194950000005],[12.092476500000032,45.598335],[12.246226,45.53050649800008],[12.354873500000053,45.588891498000066],[12.439573500000051,45.559267498000054],[12.424055499000076,45.62081199800008],[12.503972500000032,45.68757399900005],[12.685051,45.71299250000004],[12.661673,45.792440998000075],[12.779838,45.854448500000046],[12.979058,45.834048996000035],[13.098785501000066,45.644532996000066]]],[[[12.200494,45.257336999000074],[12.204566500000055,45.19767599700003],[12.298646500000075,45.21860999900008],[12.330571500000076,45.16055049900007],[12.328777,45.151860998000075],[12.332355500000062,45.09098350000005],[12.302155606000042,45.13788754600006],[12.279706043000033,45.123266827000066],[12.136543,45.05625599900003],[12.025934,45.13865899800004],[11.968785,45.13398299800008],[11.977841,45.18713949800008],[12.200494,45.257336999000074]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH36","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Padova","NUTS_NAME":"Padova"},"geometry":{"type":"Polygon","coordinates":[[[12.092476500000032,45.598335],[12.007612,45.53194950000005],[11.975058,45.39660199800005],[12.028681,45.317453500000056],[12.132562,45.30037049900005],[12.200494,45.257336999000074],[11.977841,45.18713949800008],[11.968785,45.13398299800008],[11.416932,45.12932899800006],[11.390903500000036,45.25262349800005],[11.487054500000056,45.26595199800005],[11.617731500000048,45.258523498000045],[11.580114500000036,45.28257749900007],[11.615733500000033,45.38802149600008],[11.758991,45.52041649700004],[11.657772500000021,45.54017299600008],[11.646586,45.63419799600007],[11.821666500000049,45.687179496000056],[12.092476500000032,45.598335]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH37","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Rovigo","NUTS_NAME":"Rovigo"},"geometry":{"type":"Polygon","coordinates":[[[12.025934,45.13865899800004],[12.136543,45.05625599900003],[12.279706043000033,45.123266827000066],[12.31234899900005,44.96700849800004],[12.409864,45.031746499000064],[12.494037,44.97136599700008],[12.485925,44.895982497000034],[12.389819500000044,44.87723599900005],[12.399056500000029,44.79261550000007],[12.295169500000043,44.859279496000056],[12.280897500000037,44.941755998000076],[12.098028500000055,44.971468998000034],[11.804749,44.97694949800007],[11.624835,44.889211500000044],[11.426765500000045,44.950075999000035],[11.175304,45.06006149800004],[11.205502500000023,45.10948749800008],[11.40244,45.05299499900008],[11.445834499000057,45.08066349600006],[11.416932,45.12932899800006],[11.968785,45.13398299800008],[12.025934,45.13865899800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH4","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Friuli-Venezia Giulia","NUTS_NAME":"Friuli-Venezia Giulia"},"geometry":{"type":"Polygon","coordinates":[[[13.476164999000048,46.00608149800007],[13.526538,45.97231849800005],[13.643036,45.98270199800004],[13.575802500000066,45.847736498000074],[13.597145,45.81952250000006],[13.596243,45.80793750000004],[13.795683500000052,45.74320449900006],[13.918656500000054,45.63351749800006],[13.850761500000033,45.58493249600008],[13.722823500000061,45.59472549600008],[13.799811,45.60977199600006],[13.753360500000042,45.680596998000055],[13.579783,45.786951498000064],[13.520863500000075,45.762534498000036],[13.53142,45.72383149800004],[13.373012,45.68000649800007],[13.459073076000038,45.70803148200008],[13.406493500000067,45.725175],[13.130853,45.77184449900005],[13.067232500000046,45.710487999000065],[13.098785501000066,45.644532996000066],[12.979058,45.834048996000035],[12.779838,45.854448500000046],[12.661673,45.792440998000075],[12.425841500000047,45.94739499800005],[12.400534500000049,46.04199650000004],[12.495829,46.15256049800007],[12.320955499000036,46.26891449900006],[12.498703,46.41222250000004],[12.749661501000048,46.54399550000005],[12.731392999000036,46.63428799900004],[13.504249500000071,46.566303997000034],[13.714184999000054,46.522703499000045],[13.684032,46.43747250000007],[13.375492500000064,46.29823249800006],[13.423357,46.206053997000026],[13.664347500000076,46.177549998000075],[13.496938999000065,46.05133499800007],[13.476164999000048,46.00608149800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH41","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pordenone","NUTS_NAME":"Pordenone"},"geometry":{"type":"Polygon","coordinates":[[[12.979058,45.834048996000035],[12.779838,45.854448500000046],[12.661673,45.792440998000075],[12.425841500000047,45.94739499800005],[12.400534500000049,46.04199650000004],[12.495829,46.15256049800007],[12.320955499000036,46.26891449900006],[12.498703,46.41222250000004],[12.708022999000036,46.32529199900006],[12.973565001000054,46.328256],[12.903873500000032,45.96327849900007],[12.979058,45.834048996000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH42","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Udine","NUTS_NAME":"Udine"},"geometry":{"type":"Polygon","coordinates":[[[13.504249500000071,46.566303997000034],[13.714184999000054,46.522703499000045],[13.684032,46.43747250000007],[13.375492500000064,46.29823249800006],[13.423357,46.206053997000026],[13.664347500000076,46.177549998000075],[13.496938999000065,46.05133499800007],[13.405132001000027,45.91707899700003],[13.435100500000033,45.76052149800006],[13.406493500000067,45.725175],[13.130853,45.77184449900005],[13.067232500000046,45.710487999000065],[13.098785501000066,45.644532996000066],[12.979058,45.834048996000035],[12.903873500000032,45.96327849900007],[12.973565001000054,46.328256],[12.708022999000036,46.32529199900006],[12.498703,46.41222250000004],[12.749661501000048,46.54399550000005],[12.731392999000036,46.63428799900004],[13.504249500000071,46.566303997000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH43","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Gorizia","NUTS_NAME":"Gorizia"},"geometry":{"type":"Polygon","coordinates":[[[13.597145,45.81952250000006],[13.596243,45.80793750000004],[13.579783,45.786951498000064],[13.520863500000075,45.762534498000036],[13.53142,45.72383149800004],[13.373012,45.68000649800007],[13.459073076000038,45.70803148200008],[13.406493500000067,45.725175],[13.435100500000033,45.76052149800006],[13.405132001000027,45.91707899700003],[13.496938999000065,46.05133499800007],[13.476164999000048,46.00608149800007],[13.526538,45.97231849800005],[13.643036,45.98270199800004],[13.575802500000066,45.847736498000074],[13.597145,45.81952250000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH44","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Trieste","NUTS_NAME":"Trieste"},"geometry":{"type":"Polygon","coordinates":[[[13.722823500000061,45.59472549600008],[13.799811,45.60977199600006],[13.753360500000042,45.680596998000055],[13.579783,45.786951498000064],[13.596243,45.80793750000004],[13.795683500000052,45.74320449900006],[13.918656500000054,45.63351749800006],[13.850761500000033,45.58493249600008],[13.722823500000061,45.59472549600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH5","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Emilia-Romagna","NUTS_NAME":"Emilia-Romagna"},"geometry":{"type":"Polygon","coordinates":[[[9.548684500000036,45.13264849800004],[9.713752,45.058392998000045],[9.895781,45.07964199600008],[9.891101500000048,45.13089799900007],[9.988977499000043,45.13210999800003],[10.083503,45.04395850000003],[10.464030500000035,44.937171500000034],[10.504344,44.92241799800007],[10.730587,44.991889498000035],[10.887909500000035,44.914265998000076],[11.246231500000022,44.951427996000064],[11.426765500000045,44.950075999000035],[11.624835,44.889211500000044],[11.804749,44.97694949800007],[12.098028500000055,44.971468998000034],[12.280897500000037,44.941755998000076],[12.295169500000043,44.859279496000056],[12.399056500000029,44.79261550000007],[12.265868500000067,44.82332599800003],[12.239688,44.69413849700004],[12.269892,44.630007496000076],[12.384282,44.22472649900004],[12.450349,44.16218949600005],[12.75078,43.97060150000004],[12.733553500000028,43.869933497000034],[12.669299,43.82322749800005],[12.493956,43.91554650000006],[12.513435500000071,43.99126949600003],[12.404607500000054,43.95567850000003],[12.4177,43.89902999600008],[12.283797500000048,43.764898997000046],[12.107464,43.75375349600006],[11.986528500000077,43.76191399700008],[11.710189,43.877430497000034],[11.646041500000024,43.99065699600004],[11.752958,44.12051850000006],[11.715906500000074,44.122540498000035],[11.524959500000023,44.157639998000036],[11.421995,44.23987949800005],[11.195749999000043,44.15139849800005],[11.263648500000045,44.10422449600003],[11.202439500000025,44.10072099900003],[11.049449,44.09022999800004],[11.004782,44.12143099600007],[10.917739,44.06276249700005],[10.814787,44.11617449800008],[10.653418,44.15778149800008],[10.624078,44.12035150000003],[10.470149,44.226041],[10.253875,44.26856699800004],[10.142054500000029,44.353852998000036],[9.996692,44.39711249800007],[9.96213,44.467052996000064],[9.81996,44.46640549800003],[9.686725500000023,44.36592199800003],[9.577126500000077,44.43523699900004],[9.479059500000062,44.40924049900008],[9.439189,44.42905349800003],[9.49682850000005,44.48271250000005],[9.493363,44.555858998000076],[9.202995500000043,44.61347649800007],[9.200074,44.686099496000054],[9.32466050000005,44.689999999000065],[9.286998,44.772481999000036],[9.363122,44.82239099800006],[9.27743350000003,44.895598],[9.293407,44.93250949800006],[9.37082950000007,45.048136998000075],[9.532049500000028,45.07842749800005],[9.548684500000036,45.13264849800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU31","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Észak-Magyarország","NUTS_NAME":"Észak-Magyarország"},"geometry":{"type":"Polygon","coordinates":[[[21.61392150000006,48.50941299900006],[21.721957,48.35105049800006],[22.12107750000007,48.37831149800007],[21.928462,48.25123349800003],[21.453454,48.17945999900007],[21.414271,48.11987250000004],[21.46211150000005,48.07135999600007],[21.412975,48.03392699900007],[21.127827500000024,48.04572950000005],[21.079003,48.00216849900005],[21.12218050000007,47.96182749800005],[21.035947500000077,47.74632849900007],[20.827971500000046,47.659018496000044],[20.771698,47.654476500000044],[20.394156500000065,47.42046349700007],[20.208908500000064,47.60727999800008],[20.019603,47.57997449800007],[19.993662500000028,47.672739500000034],[19.666329,47.58855],[19.570937,47.7349],[19.477327,47.687782997000056],[19.42492050000004,47.81662999800005],[19.08613450000007,47.83817049800007],[18.942689,47.94399499600007],[19.006969,48.010337496000034],[18.928392,48.05683249800006],[19.01432250000005,48.077736498000036],[19.451139,48.094441998000036],[19.52604,48.203133997000066],[19.644519,48.239175499000055],[19.899319,48.124797998000076],[20.051879,48.16770399600006],[20.133181500000035,48.253985998000076],[20.325701500000037,48.27279849800004],[20.463937,48.463966999000036],[20.506496,48.53441350000003],[20.850424,48.581631499000025],[21.11743750000005,48.49105349700005],[21.440056,48.58523299800004],[21.61392150000006,48.50941299900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU311","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Borsod-Abaúj-Zemplén","NUTS_NAME":"Borsod-Abaúj-Zemplén"},"geometry":{"type":"Polygon","coordinates":[[[22.12107750000007,48.37831149800007],[21.928462,48.25123349800003],[21.453454,48.17945999900007],[21.414271,48.11987250000004],[21.46211150000005,48.07135999600007],[21.412975,48.03392699900007],[21.127827500000024,48.04572950000005],[21.079003,48.00216849900005],[21.12218050000007,47.96182749800005],[21.035947500000077,47.74632849900007],[20.827971500000046,47.659018496000044],[20.771698,47.654476500000044],[20.568908,47.71783249900005],[20.595962,47.74010149900005],[20.449700500000063,47.88988649800007],[20.498533,47.941151],[20.454475,47.967277497000055],[20.50092950000004,48.068493998000065],[20.468356500000027,48.13107099800004],[20.248875,48.088982498000064],[20.093064,48.14322099800006],[20.051879,48.16770399600006],[20.133181500000035,48.253985998000076],[20.325701500000037,48.27279849800004],[20.463937,48.463966999000036],[20.506496,48.53441350000003],[20.850424,48.581631499000025],[21.11743750000005,48.49105349700005],[21.440056,48.58523299800004],[21.61392150000006,48.50941299900006],[21.721957,48.35105049800006],[22.12107750000007,48.37831149800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU312","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Heves","NUTS_NAME":"Heves"},"geometry":{"type":"Polygon","coordinates":[[[20.771698,47.654476500000044],[20.394156500000065,47.42046349700007],[20.208908500000064,47.60727999800008],[20.019603,47.57997449800007],[19.993662500000028,47.672739500000034],[19.666329,47.58855],[19.570937,47.7349],[19.765762,47.88864349800008],[19.854467,47.890565],[19.809143,47.92279499700004],[19.964881,47.948234],[20.031603500000074,48.00738749800007],[19.968403500000022,48.08910399800004],[20.093064,48.14322099800006],[20.248875,48.088982498000064],[20.468356500000027,48.13107099800004],[20.50092950000004,48.068493998000065],[20.454475,47.967277497000055],[20.498533,47.941151],[20.449700500000063,47.88988649800007],[20.595962,47.74010149900005],[20.568908,47.71783249900005],[20.771698,47.654476500000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU313","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Nógrád","NUTS_NAME":"Nógrád"},"geometry":{"type":"Polygon","coordinates":[[[20.051879,48.16770399600006],[20.093064,48.14322099800006],[19.968403500000022,48.08910399800004],[20.031603500000074,48.00738749800007],[19.964881,47.948234],[19.809143,47.92279499700004],[19.854467,47.890565],[19.765762,47.88864349800008],[19.570937,47.7349],[19.477327,47.687782997000056],[19.42492050000004,47.81662999800005],[19.08613450000007,47.83817049800007],[18.942689,47.94399499600007],[19.006969,48.010337496000034],[18.928392,48.05683249800006],[19.01432250000005,48.077736498000036],[19.451139,48.094441998000036],[19.52604,48.203133997000066],[19.644519,48.239175499000055],[19.899319,48.124797998000076],[20.051879,48.16770399600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU32","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Észak-Alföld","NUTS_NAME":"Észak-Alföld"},"geometry":{"type":"Polygon","coordinates":[[[22.240338999000073,48.386390498000026],[22.384609,48.23410049600005],[22.515094500000032,48.238057997000055],[22.590280500000063,48.107631498000046],[22.830129500000055,48.11207949700008],[22.881429500000024,48.05487049800007],[22.836399,47.989948499000036],[22.89627050000007,47.954120500000045],[22.680243999000027,47.78774099800006],[22.31726,47.76592199600003],[22.180837500000052,47.60009449800003],[22.12832,47.59808949600006],[22.012631,47.50135449800007],[22.011991,47.37579649800006],[21.938521500000036,47.37301099800004],[21.792838500000073,47.10724299900005],[21.658955,47.02213149700003],[21.497074,46.95177750000005],[21.247840500000052,47.006699998000045],[21.232018500000038,47.05936],[21.295557,47.086577498000054],[21.30089350000003,47.150679],[20.992097500000057,47.23228799800006],[20.847731500000066,47.14824899800004],[20.811284,47.037716499000055],[20.70784099900004,47.035010998000075],[20.727598500000056,46.94553099800004],[20.69322,46.91066699700008],[20.459983500000078,46.90281999800004],[20.407567500000027,46.85922399800006],[20.424265500000047,46.80347099700003],[20.175200500000074,46.754480998000076],[20.061163,46.80639099600006],[20.000983,46.81740449600005],[20.114705,46.84264299800003],[20.08559450000007,46.874453999000025],[20.159643500000072,46.91904399900005],[20.094639,47.007426499000076],[20.047241,47.12519599900003],[20.098951500000055,47.18146399900007],[19.988269500000058,47.25306149800008],[20.017585,47.303893998000035],[19.990462500000035,47.346628],[19.74696350000005,47.488931498000056],[19.789935,47.53137099700007],[19.652673,47.54783799900008],[19.666329,47.58855],[19.993662500000028,47.672739500000034],[20.019603,47.57997449800007],[20.208908500000064,47.60727999800008],[20.394156500000065,47.42046349700007],[20.771698,47.654476500000044],[20.827971500000046,47.659018496000044],[21.035947500000077,47.74632849900007],[21.12218050000007,47.96182749800005],[21.079003,48.00216849900005],[21.127827500000024,48.04572950000005],[21.412975,48.03392699900007],[21.46211150000005,48.07135999600007],[21.414271,48.11987250000004],[21.453454,48.17945999900007],[21.928462,48.25123349800003],[22.12107750000007,48.37831149800007],[22.155306,48.40339649800006],[22.26507,48.40991949800008],[22.240338999000073,48.386390498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU321","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Hajdú-Bihar","NUTS_NAME":"Hajdú-Bihar"},"geometry":{"type":"Polygon","coordinates":[[[22.12832,47.59808949600006],[22.012631,47.50135449800007],[22.011991,47.37579649800006],[21.938521500000036,47.37301099800004],[21.792838500000073,47.10724299900005],[21.658955,47.02213149700003],[21.497074,46.95177750000005],[21.247840500000052,47.006699998000045],[21.232018500000038,47.05936],[21.295557,47.086577498000054],[21.30089350000003,47.150679],[20.992097500000057,47.23228799800006],[21.030259500000057,47.356073],[20.994493,47.567060498000046],[20.84243850000007,47.59655149800005],[20.827971500000046,47.659018496000044],[21.035947500000077,47.74632849900007],[21.12218050000007,47.96182749800005],[21.2708,47.90484299800005],[21.565877,47.92967999900003],[21.54159,47.79025749900006],[21.636362,47.729265498000075],[21.83332,47.668893997000055],[21.924725,47.733086498000034],[22.01845650000007,47.62748649700006],[22.104404500000044,47.65834699800007],[22.12832,47.59808949600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU322","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Jász-Nagykun-Szolnok","NUTS_NAME":"Jász-Nagykun-Szolnok"},"geometry":{"type":"Polygon","coordinates":[[[20.827971500000046,47.659018496000044],[20.84243850000007,47.59655149800005],[20.994493,47.567060498000046],[21.030259500000057,47.356073],[20.992097500000057,47.23228799800006],[20.847731500000066,47.14824899800004],[20.811284,47.037716499000055],[20.70784099900004,47.035010998000075],[20.727598500000056,46.94553099800004],[20.69322,46.91066699700008],[20.459983500000078,46.90281999800004],[20.407567500000027,46.85922399800006],[20.424265500000047,46.80347099700003],[20.175200500000074,46.754480998000076],[20.061163,46.80639099600006],[20.000983,46.81740449600005],[20.114705,46.84264299800003],[20.08559450000007,46.874453999000025],[20.159643500000072,46.91904399900005],[20.094639,47.007426499000076],[20.047241,47.12519599900003],[20.098951500000055,47.18146399900007],[19.988269500000058,47.25306149800008],[20.017585,47.303893998000035],[19.990462500000035,47.346628],[19.74696350000005,47.488931498000056],[19.789935,47.53137099700007],[19.652673,47.54783799900008],[19.666329,47.58855],[19.993662500000028,47.672739500000034],[20.019603,47.57997449800007],[20.208908500000064,47.60727999800008],[20.394156500000065,47.42046349700007],[20.771698,47.654476500000044],[20.827971500000046,47.659018496000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU323","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Szabolcs-Szatmár-Bereg","NUTS_NAME":"Szabolcs-Szatmár-Bereg"},"geometry":{"type":"Polygon","coordinates":[[[22.89627050000007,47.954120500000045],[22.680243999000027,47.78774099800006],[22.31726,47.76592199600003],[22.180837500000052,47.60009449800003],[22.12832,47.59808949600006],[22.104404500000044,47.65834699800007],[22.01845650000007,47.62748649700006],[21.924725,47.733086498000034],[21.83332,47.668893997000055],[21.636362,47.729265498000075],[21.54159,47.79025749900006],[21.565877,47.92967999900003],[21.2708,47.90484299800005],[21.12218050000007,47.96182749800005],[21.079003,48.00216849900005],[21.127827500000024,48.04572950000005],[21.412975,48.03392699900007],[21.46211150000005,48.07135999600007],[21.414271,48.11987250000004],[21.453454,48.17945999900007],[21.928462,48.25123349800003],[22.12107750000007,48.37831149800007],[22.155306,48.40339649800006],[22.26507,48.40991949800008],[22.240338999000073,48.386390498000026],[22.384609,48.23410049600005],[22.515094500000032,48.238057997000055],[22.590280500000063,48.107631498000046],[22.830129500000055,48.11207949700008],[22.881429500000024,48.05487049800007],[22.836399,47.989948499000036],[22.89627050000007,47.954120500000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU33","LEVL_CODE":2,"CNTR_CODE":"HU","NAME_LATN":"Dél-Alföld","NUTS_NAME":"Dél-Alföld"},"geometry":{"type":"Polygon","coordinates":[[[21.247840500000052,47.006699998000045],[21.497074,46.95177750000005],[21.658955,47.02213149700003],[21.60309650000005,46.86840799600003],[21.483475500000054,46.76463549700003],[21.52933950000005,46.72074199900004],[21.43006950000006,46.693812998000055],[21.441398,46.651466999000036],[21.314121999000065,46.617556],[21.260401500000057,46.501945996000075],[21.317476,46.45065599600008],[21.296291,46.40669299800004],[21.206304,46.40323099600005],[21.103170500000033,46.26259049600003],[20.7756,46.27590999800003],[20.70530350000007,46.16093749800007],[20.6406,46.12720699800008],[20.501525,46.18993599800007],[20.264296,46.12637349900007],[19.86539,46.15033449800006],[19.698099,46.18793099800007],[19.567619500000035,46.17871849800008],[19.30270950000005,45.99155049900003],[19.133789,46.03707099700006],[19.008840500000076,45.92330949800004],[18.889734,45.921180496000034],[18.821306,45.91438299600003],[18.868295,46.02080049800003],[18.73278650000003,46.04514199800008],[18.802291,46.10876499800003],[18.85447,46.19261],[18.82207550000004,46.205366998000045],[18.921786,46.30806449700003],[18.885012,46.39557699900007],[18.923924500000055,46.463970998000036],[18.862301,46.61581350000006],[19.00624350000004,46.70492349600005],[18.925097,46.85716850000006],[18.96590750000007,47.02896499900004],[19.23677,47.13080099700005],[19.347792500000025,47.09322199700006],[19.29528650000003,47.03592250000003],[19.319190500000047,47.012678],[19.587165500000026,47.120925499000066],[19.779767500000048,46.94870449800004],[20.094639,47.007426499000076],[20.159643500000072,46.91904399900005],[20.08559450000007,46.874453999000025],[20.114705,46.84264299800003],[20.000983,46.81740449600005],[20.061163,46.80639099600006],[20.175200500000074,46.754480998000076],[20.424265500000047,46.80347099700003],[20.407567500000027,46.85922399800006],[20.459983500000078,46.90281999800004],[20.69322,46.91066699700008],[20.727598500000056,46.94553099800004],[20.70784099900004,47.035010998000075],[20.811284,47.037716499000055],[20.847731500000066,47.14824899800004],[20.992097500000057,47.23228799800006],[21.30089350000003,47.150679],[21.295557,47.086577498000054],[21.232018500000038,47.05936],[21.247840500000052,47.006699998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU331","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Bács-Kiskun","NUTS_NAME":"Bács-Kiskun"},"geometry":{"type":"Polygon","coordinates":[[[20.094639,47.007426499000076],[20.159643500000072,46.91904399900005],[20.08559450000007,46.874453999000025],[20.114705,46.84264299800003],[20.000983,46.81740449600005],[20.061163,46.80639099600006],[19.969213,46.726733996000064],[20.011152999000046,46.69494599600006],[19.994492500000035,46.59148449800006],[19.779974,46.531852498000035],[19.895200500000044,46.48026249900005],[19.881061500000044,46.38604049800006],[19.65595950000005,46.346945498000025],[19.689029,46.30572549800007],[19.63817,46.26409749700008],[19.698099,46.18793099800007],[19.567619500000035,46.17871849800008],[19.30270950000005,45.99155049900003],[19.133789,46.03707099700006],[19.008840500000076,45.92330949800004],[18.889734,45.921180496000034],[18.821306,45.91438299600003],[18.868295,46.02080049800003],[18.73278650000003,46.04514199800008],[18.802291,46.10876499800003],[18.85447,46.19261],[18.82207550000004,46.205366998000045],[18.921786,46.30806449700003],[18.885012,46.39557699900007],[18.923924500000055,46.463970998000036],[18.862301,46.61581350000006],[19.00624350000004,46.70492349600005],[18.925097,46.85716850000006],[18.96590750000007,47.02896499900004],[19.23677,47.13080099700005],[19.347792500000025,47.09322199700006],[19.29528650000003,47.03592250000003],[19.319190500000047,47.012678],[19.587165500000026,47.120925499000066],[19.779767500000048,46.94870449800004],[20.094639,47.007426499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU332","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Békés","NUTS_NAME":"Békés"},"geometry":{"type":"Polygon","coordinates":[[[21.658955,47.02213149700003],[21.60309650000005,46.86840799600003],[21.483475500000054,46.76463549700003],[21.52933950000005,46.72074199900004],[21.43006950000006,46.693812998000055],[21.441398,46.651466999000036],[21.314121999000065,46.617556],[21.260401500000057,46.501945996000075],[21.317476,46.45065599600008],[21.296291,46.40669299800004],[21.206304,46.40323099600005],[21.103170500000033,46.26259049600003],[20.7756,46.27590999800003],[20.738757,46.384123498000065],[20.54891150000003,46.391294997000045],[20.614569,46.52636049800003],[20.569139500000063,46.63282099600008],[20.595882500000073,46.75956799800008],[20.424265500000047,46.80347099700003],[20.407567500000027,46.85922399800006],[20.459983500000078,46.90281999800004],[20.69322,46.91066699700008],[20.727598500000056,46.94553099800004],[20.70784099900004,47.035010998000075],[20.811284,47.037716499000055],[20.847731500000066,47.14824899800004],[20.992097500000057,47.23228799800006],[21.30089350000003,47.150679],[21.295557,47.086577498000054],[21.232018500000038,47.05936],[21.247840500000052,47.006699998000045],[21.497074,46.95177750000005],[21.658955,47.02213149700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"HU333","LEVL_CODE":3,"CNTR_CODE":"HU","NAME_LATN":"Csongrád","NUTS_NAME":"Csongrád"},"geometry":{"type":"Polygon","coordinates":[[[20.424265500000047,46.80347099700003],[20.595882500000073,46.75956799800008],[20.569139500000063,46.63282099600008],[20.614569,46.52636049800003],[20.54891150000003,46.391294997000045],[20.738757,46.384123498000065],[20.7756,46.27590999800003],[20.70530350000007,46.16093749800007],[20.6406,46.12720699800008],[20.501525,46.18993599800007],[20.264296,46.12637349900007],[19.86539,46.15033449800006],[19.698099,46.18793099800007],[19.63817,46.26409749700008],[19.689029,46.30572549800007],[19.65595950000005,46.346945498000025],[19.881061500000044,46.38604049800006],[19.895200500000044,46.48026249900005],[19.779974,46.531852498000035],[19.994492500000035,46.59148449800006],[20.011152999000046,46.69494599600006],[19.969213,46.726733996000064],[20.061163,46.80639099600006],[20.175200500000074,46.754480998000076],[20.424265500000047,46.80347099700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE","LEVL_CODE":0,"CNTR_CODE":"IE","NAME_LATN":"IRELAND","NUTS_NAME":"IRELAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-10.189941,53.556088500000044],[-9.800738259999946,53.60891226000007],[-9.911659,53.65312749800006],[-9.900522,53.764440996000076],[-9.543502,53.80220649800003],[-9.626968499999975,53.82289099700006],[-9.546066,53.88420849800008],[-9.937296,53.87381399700007],[-9.915183,53.94451049700007],[-9.790024,53.941201],[-9.991458499999965,54.10296949900004],[-9.896853499999963,54.11763049700005],[-9.988074,54.21403799900003],[-10.093689,54.156438499000046],[-10.060135,54.09908249700004],[-10.131331,54.10298399900006],[-10.055089499999951,54.21873850000003],[-10.070641499999965,54.27627250000006],[-9.996987499999932,54.305758999000034],[-9.887919,54.268800498000076],[-9.986373499999956,54.22454649700006],[-9.874725,54.21973449800004],[-9.857578,54.32446949800004],[-9.786128,54.34158549800003],[-9.261366,54.308216498000036],[-9.132258499999978,54.162360497000066],[-9.055131,54.29366599900004],[-8.508256,54.21698549700005],[-8.618876,54.25972899800007],[-8.50034949999997,54.32146849800006],[-8.678318499999932,54.34851650000007],[-8.287109,54.47895],[-8.134833,54.60335350000008],[-8.113292499999943,54.652660998000044],[-8.462826,54.568445997000026],[-8.376357499999926,54.61381649700007],[-8.679424,54.61958349900004],[-8.788656499999945,54.68891599600005],[-8.646348498999942,54.772875499000065],[-8.423189499999978,54.762527996000074],[-8.565171499999963,54.828002996000066],[-8.304506,54.88823649700004],[-8.458770499999957,54.94166799800007],[-8.363363,54.94810499700003],[-8.45208649999995,55.00039250000003],[-8.319419,55.06810249900008],[-8.28179849999998,55.158847997000066],[-8.138147,55.13951599900008],[-7.982110000999967,55.22650699900004],[-7.853429499999947,55.16610499800004],[-7.894535499999961,55.13504599700008],[-7.8255,55.17477349700005],[-7.866177,55.22953499700003],[-7.630938499999957,55.276289],[-7.522851,55.136603500000035],[-7.633058000999938,55.058061],[-7.55847,55.03574749900008],[-7.659480499999972,54.96910349800004],[-7.625162,54.962477497000066],[-7.513202,55.040163997000036],[-7.533077499999933,55.07207699800006],[-7.516833499999962,55.08345849800003],[-7.454660499999932,55.06328099600006],[-7.554363499999965,55.19592699800006],[-7.522001,55.286463],[-7.252384499999948,55.278456],[-7.399569499999927,55.37629099800006],[-7.339527,55.37495799900006],[-6.924876,55.23369449900008],[-7.256068499999969,55.067034997000064],[-7.396395499999926,55.016722997000045],[-7.44311,54.87147149800006],[-7.548892,54.789589498000055],[-7.534506,54.747139],[-7.921372,54.69654449800004],[-7.703411500999948,54.60828799800004],[-8.177718,54.46497350000004],[-7.872462499999926,54.29415499800007],[-7.855390499999942,54.22252249600007],[-7.565956,54.12651449800006],[-7.290923499999963,54.17190949600007],[-7.279176499999949,54.12441650000005],[-7.248977,54.20340750000008],[-7.145823499999949,54.22494499900006],[-7.211874499999965,54.299679],[-7.02863549999995,54.421306497000046],[-6.910962499999926,54.37472549900008],[-6.79513849999995,54.20911399800008],[-6.643909,54.180019497000046],[-6.668772499999932,54.07254799900005],[-6.623778499999958,54.03654849600008],[-6.268015499999933,54.102336999000045],[-6.103183499999943,53.99999950000006],[-6.38110249999994,54.01270399900005],[-6.369552,53.90455349700005],[-6.237978,53.86060149700006],[-6.247098,53.72245449900004],[-6.214899499999945,53.633406998000055],[-6.076308499999925,53.51958699800008],[-6.134826499999974,53.45252749900004],[-6.04398649999996,53.37856249600003],[-6.212808,53.33487549800003],[-6.100643499999933,53.28238999800004],[-6.102773,53.21077299900003],[-6.041955,53.11962099900006],[-6.046814,52.991224],[-5.999142,52.96649599800003],[-6.144516500999941,52.73769649800005],[-6.225527,52.64820149700006],[-6.204002499999945,52.544962998000074],[-6.380481499999973,52.35391150000004],[-6.510815499999978,52.351184498000066],[-6.313716,52.23998499900006],[-6.360401499999966,52.17429799800004],[-6.808146499999964,52.21769649700008],[-6.762292,52.25508049900003],[-6.899396,52.15835150000004],[-6.965714,52.24050599700007],[-7.023463499999934,52.133342998000046],[-7.586105112999974,52.10111825300004],[-7.645049,52.06281849900006],[-7.588845499999934,52.05158099700003],[-7.586489,51.991399999000066],[-7.842156,51.954218498000046],[-7.872479,51.87750499900005],[-8.06342449999994,51.80898099800004],[-8.31868849999995,51.842880498000056],[-8.273785499999974,51.79800749900005],[-8.339985,51.716754],[-8.531225499999948,51.68981899800008],[-8.534105,51.60396149800005],[-8.700837499999977,51.65090999900008],[-8.678507,51.60910049600005],[-8.71486,51.56947649800003],[-8.866604,51.591910496000025],[-9.344147,51.46970149900005],[-9.430925499999944,51.51073099700005],[-9.394508516999963,51.55019734000007],[-9.820953499999973,51.44911749800008],[-9.83894349999997,51.481587500000046],[-9.523876,51.61919749800006],[-9.850705000999938,51.54110399900003],[-9.460062499999935,51.68021249600008],[-9.439506499999936,51.723395],[-10.143066499999975,51.59705299800004],[-9.945926499999928,51.71385349900004],[-10.006702,51.71578849900004],[-9.980664,51.733126999000035],[-9.581580500999962,51.870546499000056],[-10.129688000999977,51.737415498000075],[-10.225089500999957,51.781539499000075],[-10.171304,51.805255498000065],[-10.206829,51.843787],[-10.343244,51.78360349800005],[-10.335582,51.84606599800003],[-10.398117499999955,51.844182500000045],[-10.359855687999982,51.89811573800006],[-10.292081,51.91340949900007],[-10.305404416999977,51.91783544400005],[-10.179232499999955,51.963561998000046],[-10.321406,51.954053498000064],[-9.792653,52.15557449800008],[-10.45567,52.09744099900007],[-10.472409,52.18078450000007],[-10.176341499999978,52.29132850000008],[-10.14654749999994,52.26203399700006],[-10.182858499999952,52.231039],[-10.033870499999978,52.283115999000074],[-9.76673200099998,52.248066496000035],[-9.850593,52.28042149700008],[-9.837091499999929,52.389020498000036],[-9.942014,52.422852996000074],[-9.696583500999964,52.47951599900006],[-9.617206499999952,52.58082599800008],[-9.365192,52.57212449900004],[-8.728047813999979,52.66343504100007],[-8.961225499999955,52.68304449900006],[-8.952241,52.78927249800006],[-9.162081499999942,52.61912150000006],[-9.546329499999956,52.64086549700005],[-9.938699,52.559491500000036],[-9.498436499999968,52.75007849800005],[-9.433057499999961,52.88299899800006],[-9.347913499999947,52.928738500000065],[-9.469911499999967,52.94786499800006],[-9.282228499999974,53.144655999000065],[-9.009308499999975,53.14081749800005],[-9.05427,53.164757499000075],[-8.92340749999994,53.20873799900005],[-8.999464,53.22568449800008],[-8.933212,53.270670996000035],[-9.52627549999994,53.22205849800008],[-9.63143794299998,53.296942787000035],[-9.630072499999926,53.29734849600004],[-9.631445286999963,53.29734542800003],[-9.633099,53.388104500000054],[-9.774115000999927,53.29205949800007],[-9.906928,53.32518749800005],[-9.783823499999926,53.389879998000026],[-9.85217849999998,53.434093],[-9.934913,53.379014],[-10.179155499999979,53.40698899800003],[-10.081167499999935,53.48861949800005],[-10.017894499999954,53.46405999900003],[-10.189941,53.556088500000044]],[[-9.800738259999946,53.60891226000007],[-9.786883,53.60339249700007],[-9.684343499999954,53.60861199800007],[-9.800738259999946,53.60891226000007]],[[-7.717386499999975,55.09681550000005],[-7.664568,55.15841349900006],[-7.731828569999948,55.177358222000066],[-7.690315,55.14115649800004],[-7.717386499999975,55.09681550000005]],[[-8.68468695699994,52.65991484500006],[-8.651927499999942,52.65725549600006],[-8.614523,52.68985749700005],[-8.68468695699994,52.65991484500006]]],[[[-9.96094549999998,54.018669500000044],[-9.914447,53.96378799900003],[-9.96222,53.87436949800008],[-10.072347,53.97315799900008],[-10.237484,53.979537],[-9.96094549999998,54.018669500000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE0","LEVL_CODE":1,"CNTR_CODE":"IE","NAME_LATN":"IRELAND","NUTS_NAME":"IRELAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-10.189941,53.556088500000044],[-9.800738259999946,53.60891226000007],[-9.911659,53.65312749800006],[-9.900522,53.764440996000076],[-9.543502,53.80220649800003],[-9.626968499999975,53.82289099700006],[-9.546066,53.88420849800008],[-9.937296,53.87381399700007],[-9.915183,53.94451049700007],[-9.790024,53.941201],[-9.991458499999965,54.10296949900004],[-9.896853499999963,54.11763049700005],[-9.988074,54.21403799900003],[-10.093689,54.156438499000046],[-10.060135,54.09908249700004],[-10.131331,54.10298399900006],[-10.055089499999951,54.21873850000003],[-10.070641499999965,54.27627250000006],[-9.996987499999932,54.305758999000034],[-9.887919,54.268800498000076],[-9.986373499999956,54.22454649700006],[-9.874725,54.21973449800004],[-9.857578,54.32446949800004],[-9.786128,54.34158549800003],[-9.261366,54.308216498000036],[-9.132258499999978,54.162360497000066],[-9.055131,54.29366599900004],[-8.508256,54.21698549700005],[-8.618876,54.25972899800007],[-8.50034949999997,54.32146849800006],[-8.678318499999932,54.34851650000007],[-8.287109,54.47895],[-8.134833,54.60335350000008],[-8.113292499999943,54.652660998000044],[-8.462826,54.568445997000026],[-8.376357499999926,54.61381649700007],[-8.679424,54.61958349900004],[-8.788656499999945,54.68891599600005],[-8.646348498999942,54.772875499000065],[-8.423189499999978,54.762527996000074],[-8.565171499999963,54.828002996000066],[-8.304506,54.88823649700004],[-8.458770499999957,54.94166799800007],[-8.363363,54.94810499700003],[-8.45208649999995,55.00039250000003],[-8.319419,55.06810249900008],[-8.28179849999998,55.158847997000066],[-8.138147,55.13951599900008],[-7.982110000999967,55.22650699900004],[-7.853429499999947,55.16610499800004],[-7.894535499999961,55.13504599700008],[-7.8255,55.17477349700005],[-7.866177,55.22953499700003],[-7.630938499999957,55.276289],[-7.522851,55.136603500000035],[-7.633058000999938,55.058061],[-7.55847,55.03574749900008],[-7.659480499999972,54.96910349800004],[-7.625162,54.962477497000066],[-7.513202,55.040163997000036],[-7.533077499999933,55.07207699800006],[-7.516833499999962,55.08345849800003],[-7.454660499999932,55.06328099600006],[-7.554363499999965,55.19592699800006],[-7.522001,55.286463],[-7.252384499999948,55.278456],[-7.399569499999927,55.37629099800006],[-7.339527,55.37495799900006],[-6.924876,55.23369449900008],[-7.256068499999969,55.067034997000064],[-7.396395499999926,55.016722997000045],[-7.44311,54.87147149800006],[-7.548892,54.789589498000055],[-7.534506,54.747139],[-7.921372,54.69654449800004],[-7.703411500999948,54.60828799800004],[-8.177718,54.46497350000004],[-7.872462499999926,54.29415499800007],[-7.855390499999942,54.22252249600007],[-7.565956,54.12651449800006],[-7.290923499999963,54.17190949600007],[-7.279176499999949,54.12441650000005],[-7.248977,54.20340750000008],[-7.145823499999949,54.22494499900006],[-7.211874499999965,54.299679],[-7.02863549999995,54.421306497000046],[-6.910962499999926,54.37472549900008],[-6.79513849999995,54.20911399800008],[-6.643909,54.180019497000046],[-6.668772499999932,54.07254799900005],[-6.623778499999958,54.03654849600008],[-6.268015499999933,54.102336999000045],[-6.103183499999943,53.99999950000006],[-6.38110249999994,54.01270399900005],[-6.369552,53.90455349700005],[-6.237978,53.86060149700006],[-6.247098,53.72245449900004],[-6.214899499999945,53.633406998000055],[-6.076308499999925,53.51958699800008],[-6.134826499999974,53.45252749900004],[-6.04398649999996,53.37856249600003],[-6.212808,53.33487549800003],[-6.100643499999933,53.28238999800004],[-6.102773,53.21077299900003],[-6.041955,53.11962099900006],[-6.046814,52.991224],[-5.999142,52.96649599800003],[-6.144516500999941,52.73769649800005],[-6.225527,52.64820149700006],[-6.204002499999945,52.544962998000074],[-6.380481499999973,52.35391150000004],[-6.510815499999978,52.351184498000066],[-6.313716,52.23998499900006],[-6.360401499999966,52.17429799800004],[-6.808146499999964,52.21769649700008],[-6.762292,52.25508049900003],[-6.899396,52.15835150000004],[-6.965714,52.24050599700007],[-7.023463499999934,52.133342998000046],[-7.586105112999974,52.10111825300004],[-7.645049,52.06281849900006],[-7.588845499999934,52.05158099700003],[-7.586489,51.991399999000066],[-7.842156,51.954218498000046],[-7.872479,51.87750499900005],[-8.06342449999994,51.80898099800004],[-8.31868849999995,51.842880498000056],[-8.273785499999974,51.79800749900005],[-8.339985,51.716754],[-8.531225499999948,51.68981899800008],[-8.534105,51.60396149800005],[-8.700837499999977,51.65090999900008],[-8.678507,51.60910049600005],[-8.71486,51.56947649800003],[-8.866604,51.591910496000025],[-9.344147,51.46970149900005],[-9.430925499999944,51.51073099700005],[-9.394508516999963,51.55019734000007],[-9.820953499999973,51.44911749800008],[-9.83894349999997,51.481587500000046],[-9.523876,51.61919749800006],[-9.850705000999938,51.54110399900003],[-9.460062499999935,51.68021249600008],[-9.439506499999936,51.723395],[-10.143066499999975,51.59705299800004],[-9.945926499999928,51.71385349900004],[-10.006702,51.71578849900004],[-9.980664,51.733126999000035],[-9.581580500999962,51.870546499000056],[-10.129688000999977,51.737415498000075],[-10.225089500999957,51.781539499000075],[-10.171304,51.805255498000065],[-10.206829,51.843787],[-10.343244,51.78360349800005],[-10.335582,51.84606599800003],[-10.398117499999955,51.844182500000045],[-10.359855687999982,51.89811573800006],[-10.292081,51.91340949900007],[-10.305404416999977,51.91783544400005],[-10.179232499999955,51.963561998000046],[-10.321406,51.954053498000064],[-9.792653,52.15557449800008],[-10.45567,52.09744099900007],[-10.472409,52.18078450000007],[-10.176341499999978,52.29132850000008],[-10.14654749999994,52.26203399700006],[-10.182858499999952,52.231039],[-10.033870499999978,52.283115999000074],[-9.76673200099998,52.248066496000035],[-9.850593,52.28042149700008],[-9.837091499999929,52.389020498000036],[-9.942014,52.422852996000074],[-9.696583500999964,52.47951599900006],[-9.617206499999952,52.58082599800008],[-9.365192,52.57212449900004],[-8.728047813999979,52.66343504100007],[-8.961225499999955,52.68304449900006],[-8.952241,52.78927249800006],[-9.162081499999942,52.61912150000006],[-9.546329499999956,52.64086549700005],[-9.938699,52.559491500000036],[-9.498436499999968,52.75007849800005],[-9.433057499999961,52.88299899800006],[-9.347913499999947,52.928738500000065],[-9.469911499999967,52.94786499800006],[-9.282228499999974,53.144655999000065],[-9.009308499999975,53.14081749800005],[-9.05427,53.164757499000075],[-8.92340749999994,53.20873799900005],[-8.999464,53.22568449800008],[-8.933212,53.270670996000035],[-9.52627549999994,53.22205849800008],[-9.63143794299998,53.296942787000035],[-9.630072499999926,53.29734849600004],[-9.631445286999963,53.29734542800003],[-9.633099,53.388104500000054],[-9.774115000999927,53.29205949800007],[-9.906928,53.32518749800005],[-9.783823499999926,53.389879998000026],[-9.85217849999998,53.434093],[-9.934913,53.379014],[-10.179155499999979,53.40698899800003],[-10.081167499999935,53.48861949800005],[-10.017894499999954,53.46405999900003],[-10.189941,53.556088500000044]],[[-9.800738259999946,53.60891226000007],[-9.786883,53.60339249700007],[-9.684343499999954,53.60861199800007],[-9.800738259999946,53.60891226000007]],[[-7.717386499999975,55.09681550000005],[-7.664568,55.15841349900006],[-7.731828569999948,55.177358222000066],[-7.690315,55.14115649800004],[-7.717386499999975,55.09681550000005]],[[-8.68468695699994,52.65991484500006],[-8.651927499999942,52.65725549600006],[-8.614523,52.68985749700005],[-8.68468695699994,52.65991484500006]]],[[[-9.96094549999998,54.018669500000044],[-9.914447,53.96378799900003],[-9.96222,53.87436949800008],[-10.072347,53.97315799900008],[-10.237484,53.979537],[-9.96094549999998,54.018669500000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE01","LEVL_CODE":2,"CNTR_CODE":"IE","NAME_LATN":"Border, Midland and Western","NUTS_NAME":"Border, Midland and Western"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-10.189941,53.556088500000044],[-9.800738259999946,53.60891226000007],[-9.911659,53.65312749800006],[-9.900522,53.764440996000076],[-9.543502,53.80220649800003],[-9.626968499999975,53.82289099700006],[-9.546066,53.88420849800008],[-9.937296,53.87381399700007],[-9.915183,53.94451049700007],[-9.790024,53.941201],[-9.991458499999965,54.10296949900004],[-9.896853499999963,54.11763049700005],[-9.988074,54.21403799900003],[-10.093689,54.156438499000046],[-10.060135,54.09908249700004],[-10.131331,54.10298399900006],[-10.055089499999951,54.21873850000003],[-10.070641499999965,54.27627250000006],[-9.996987499999932,54.305758999000034],[-9.887919,54.268800498000076],[-9.986373499999956,54.22454649700006],[-9.874725,54.21973449800004],[-9.857578,54.32446949800004],[-9.786128,54.34158549800003],[-9.261366,54.308216498000036],[-9.132258499999978,54.162360497000066],[-9.055131,54.29366599900004],[-8.508256,54.21698549700005],[-8.618876,54.25972899800007],[-8.50034949999997,54.32146849800006],[-8.678318499999932,54.34851650000007],[-8.287109,54.47895],[-8.134833,54.60335350000008],[-8.113292499999943,54.652660998000044],[-8.462826,54.568445997000026],[-8.376357499999926,54.61381649700007],[-8.679424,54.61958349900004],[-8.788656499999945,54.68891599600005],[-8.646348498999942,54.772875499000065],[-8.423189499999978,54.762527996000074],[-8.565171499999963,54.828002996000066],[-8.304506,54.88823649700004],[-8.458770499999957,54.94166799800007],[-8.363363,54.94810499700003],[-8.45208649999995,55.00039250000003],[-8.319419,55.06810249900008],[-8.28179849999998,55.158847997000066],[-8.138147,55.13951599900008],[-7.982110000999967,55.22650699900004],[-7.853429499999947,55.16610499800004],[-7.894535499999961,55.13504599700008],[-7.8255,55.17477349700005],[-7.866177,55.22953499700003],[-7.630938499999957,55.276289],[-7.522851,55.136603500000035],[-7.633058000999938,55.058061],[-7.55847,55.03574749900008],[-7.659480499999972,54.96910349800004],[-7.625162,54.962477497000066],[-7.513202,55.040163997000036],[-7.533077499999933,55.07207699800006],[-7.516833499999962,55.08345849800003],[-7.454660499999932,55.06328099600006],[-7.554363499999965,55.19592699800006],[-7.522001,55.286463],[-7.252384499999948,55.278456],[-7.399569499999927,55.37629099800006],[-7.339527,55.37495799900006],[-6.924876,55.23369449900008],[-7.256068499999969,55.067034997000064],[-7.396395499999926,55.016722997000045],[-7.44311,54.87147149800006],[-7.548892,54.789589498000055],[-7.534506,54.747139],[-7.921372,54.69654449800004],[-7.703411500999948,54.60828799800004],[-8.177718,54.46497350000004],[-7.872462499999926,54.29415499800007],[-7.855390499999942,54.22252249600007],[-7.565956,54.12651449800006],[-7.290923499999963,54.17190949600007],[-7.279176499999949,54.12441650000005],[-7.248977,54.20340750000008],[-7.145823499999949,54.22494499900006],[-7.211874499999965,54.299679],[-7.02863549999995,54.421306497000046],[-6.910962499999926,54.37472549900008],[-6.79513849999995,54.20911399800008],[-6.643909,54.180019497000046],[-6.668772499999932,54.07254799900005],[-6.623778499999958,54.03654849600008],[-6.268015499999933,54.102336999000045],[-6.103183499999943,53.99999950000006],[-6.38110249999994,54.01270399900005],[-6.369552,53.90455349700005],[-6.237978,53.86060149700006],[-6.247098,53.72245449900004],[-6.461498,53.71293599900008],[-6.450152499999945,53.76225999700006],[-6.749452499999961,53.91374299700004],[-6.974685,53.870847],[-6.91360549999996,53.80773349800006],[-7.002189,53.765574998000034],[-7.343828499999972,53.79906249700008],[-7.225776,53.70332299900008],[-6.958578,53.653257],[-7.063349,53.468781500000034],[-7.194891499999926,53.434817500000065],[-6.980562500999952,53.31429399700005],[-7.036046,53.190018500000065],[-7.168954,53.18488599900007],[-7.06613650099996,53.169343498000046],[-7.04514949999998,53.07339949700008],[-7.090516,53.00391699900007],[-6.979713,52.977801498000076],[-6.939413,52.88005449700006],[-6.955549499999961,52.81362799800007],[-7.119209,52.82373699900006],[-7.187502,52.893327500000055],[-7.399012499999969,52.79568949800006],[-7.674081,52.78129399900007],[-7.734649499999932,52.858362498000076],[-7.702182499999935,52.934068498000045],[-7.79797949999994,52.980953499000066],[-7.962705,52.85075499900006],[-8.039676,52.88934799900005],[-7.920399,53.088658500000065],[-8.081456499999945,53.16688449800006],[-8.359736,52.971351499000036],[-8.634478,53.04178499800008],[-8.85870449999993,52.96821],[-8.89154,53.06064199800005],[-9.009308499999975,53.14081749800005],[-9.05427,53.164757499000075],[-8.92340749999994,53.20873799900005],[-8.999464,53.22568449800008],[-8.933212,53.270670996000035],[-9.52627549999994,53.22205849800008],[-9.63143794299998,53.296942787000035],[-9.630072499999926,53.29734849600004],[-9.631445286999963,53.29734542800003],[-9.633099,53.388104500000054],[-9.774115000999927,53.29205949800007],[-9.906928,53.32518749800005],[-9.783823499999926,53.389879998000026],[-9.85217849999998,53.434093],[-9.934913,53.379014],[-10.179155499999979,53.40698899800003],[-10.081167499999935,53.48861949800005],[-10.017894499999954,53.46405999900003],[-10.189941,53.556088500000044]],[[-9.800738259999946,53.60891226000007],[-9.786883,53.60339249700007],[-9.684343499999954,53.60861199800007],[-9.800738259999946,53.60891226000007]],[[-7.717386499999975,55.09681550000005],[-7.664568,55.15841349900006],[-7.731828569999948,55.177358222000066],[-7.690315,55.14115649800004],[-7.717386499999975,55.09681550000005]]],[[[-9.96094549999998,54.018669500000044],[-9.914447,53.96378799900003],[-9.96222,53.87436949800008],[-10.072347,53.97315799900008],[-10.237484,53.979537],[-9.96094549999998,54.018669500000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE011","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"Border","NUTS_NAME":"Border"},"geometry":{"type":"Polygon","coordinates":[[[-7.339527,55.37495799900006],[-6.924876,55.23369449900008],[-7.256068499999969,55.067034997000064],[-7.396395499999926,55.016722997000045],[-7.44311,54.87147149800006],[-7.548892,54.789589498000055],[-7.534506,54.747139],[-7.921372,54.69654449800004],[-7.703411500999948,54.60828799800004],[-8.177718,54.46497350000004],[-7.872462499999926,54.29415499800007],[-7.855390499999942,54.22252249600007],[-7.565956,54.12651449800006],[-7.290923499999963,54.17190949600007],[-7.279176499999949,54.12441650000005],[-7.248977,54.20340750000008],[-7.145823499999949,54.22494499900006],[-7.211874499999965,54.299679],[-7.02863549999995,54.421306497000046],[-6.910962499999926,54.37472549900008],[-6.79513849999995,54.20911399800008],[-6.643909,54.180019497000046],[-6.668772499999932,54.07254799900005],[-6.623778499999958,54.03654849600008],[-6.268015499999933,54.102336999000045],[-6.103183499999943,53.99999950000006],[-6.38110249999994,54.01270399900005],[-6.369552,53.90455349700005],[-6.237978,53.86060149700006],[-6.247098,53.72245449900004],[-6.461498,53.71293599900008],[-6.450152499999945,53.76225999700006],[-6.749452499999961,53.91374299700004],[-6.974685,53.870847],[-6.91360549999996,53.80773349800006],[-7.002189,53.765574998000034],[-7.343828499999972,53.79906249700008],[-7.462332499999945,53.80145999900003],[-7.634527499999933,53.93941399900007],[-7.78818949999993,53.816362998000045],[-7.918767000999935,53.82148599800007],[-8.11553749999996,53.953230500000075],[-8.053147499999966,54.104686498000035],[-8.148062,54.12523950000008],[-8.425305499999979,53.96431899600003],[-8.33288049999993,53.92772999700003],[-8.477339,53.92091699900004],[-8.647964,54.034305],[-8.795409499999948,53.96560149700008],[-9.01851450099997,54.025168],[-8.916954499999974,54.11804299900007],[-9.132258499999978,54.162360497000066],[-9.055131,54.29366599900004],[-8.508256,54.21698549700005],[-8.618876,54.25972899800007],[-8.50034949999997,54.32146849800006],[-8.678318499999932,54.34851650000007],[-8.287109,54.47895],[-8.134833,54.60335350000008],[-8.113292499999943,54.652660998000044],[-8.462826,54.568445997000026],[-8.376357499999926,54.61381649700007],[-8.679424,54.61958349900004],[-8.788656499999945,54.68891599600005],[-8.646348498999942,54.772875499000065],[-8.423189499999978,54.762527996000074],[-8.565171499999963,54.828002996000066],[-8.304506,54.88823649700004],[-8.458770499999957,54.94166799800007],[-8.363363,54.94810499700003],[-8.45208649999995,55.00039250000003],[-8.319419,55.06810249900008],[-8.28179849999998,55.158847997000066],[-8.138147,55.13951599900008],[-7.982110000999967,55.22650699900004],[-7.853429499999947,55.16610499800004],[-7.894535499999961,55.13504599700008],[-7.8255,55.17477349700005],[-7.866177,55.22953499700003],[-7.630938499999957,55.276289],[-7.522851,55.136603500000035],[-7.633058000999938,55.058061],[-7.55847,55.03574749900008],[-7.659480499999972,54.96910349800004],[-7.625162,54.962477497000066],[-7.513202,55.040163997000036],[-7.533077499999933,55.07207699800006],[-7.516833499999962,55.08345849800003],[-7.454660499999932,55.06328099600006],[-7.554363499999965,55.19592699800006],[-7.522001,55.286463],[-7.252384499999948,55.278456],[-7.399569499999927,55.37629099800006],[-7.339527,55.37495799900006]],[[-7.664568,55.15841349900006],[-7.731828569999948,55.177358222000066],[-7.690315,55.14115649800004],[-7.717386499999975,55.09681550000005],[-7.664568,55.15841349900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE012","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"Midland","NUTS_NAME":"Midland"},"geometry":{"type":"Polygon","coordinates":[[[-7.343828499999972,53.79906249700008],[-7.225776,53.70332299900008],[-6.958578,53.653257],[-7.063349,53.468781500000034],[-7.194891499999926,53.434817500000065],[-6.980562500999952,53.31429399700005],[-7.036046,53.190018500000065],[-7.168954,53.18488599900007],[-7.06613650099996,53.169343498000046],[-7.04514949999998,53.07339949700008],[-7.090516,53.00391699900007],[-6.979713,52.977801498000076],[-6.939413,52.88005449700006],[-6.955549499999961,52.81362799800007],[-7.119209,52.82373699900006],[-7.187502,52.893327500000055],[-7.399012499999969,52.79568949800006],[-7.674081,52.78129399900007],[-7.734649499999932,52.858362498000076],[-7.702182499999935,52.934068498000045],[-7.79797949999994,52.980953499000066],[-7.962705,52.85075499900006],[-8.039676,52.88934799900005],[-7.920399,53.088658500000065],[-8.081456499999945,53.16688449800006],[-7.968462,53.218818499000065],[-8.051905499999975,53.27382799600008],[-8.027095,53.32122099600008],[-7.910323499999947,53.371518500000036],[-8.035976499999947,53.60768649800008],[-7.875693,53.77736349600008],[-7.918767000999935,53.82148599800007],[-7.78818949999993,53.816362998000045],[-7.634527499999933,53.93941399900007],[-7.462332499999945,53.80145999900003],[-7.343828499999972,53.79906249700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE013","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"West","NUTS_NAME":"West"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-10.189941,53.556088500000044],[-9.800738259999946,53.60891226000007],[-9.911659,53.65312749800006],[-9.900522,53.764440996000076],[-9.543502,53.80220649800003],[-9.626968499999975,53.82289099700006],[-9.546066,53.88420849800008],[-9.937296,53.87381399700007],[-9.915183,53.94451049700007],[-9.790024,53.941201],[-9.991458499999965,54.10296949900004],[-9.896853499999963,54.11763049700005],[-9.988074,54.21403799900003],[-10.093689,54.156438499000046],[-10.060135,54.09908249700004],[-10.131331,54.10298399900006],[-10.055089499999951,54.21873850000003],[-10.070641499999965,54.27627250000006],[-9.996987499999932,54.305758999000034],[-9.887919,54.268800498000076],[-9.986373499999956,54.22454649700006],[-9.874725,54.21973449800004],[-9.857578,54.32446949800004],[-9.786128,54.34158549800003],[-9.261366,54.308216498000036],[-9.132258499999978,54.162360497000066],[-8.916954499999974,54.11804299900007],[-9.01851450099997,54.025168],[-8.795409499999948,53.96560149700008],[-8.647964,54.034305],[-8.477339,53.92091699900004],[-8.33288049999993,53.92772999700003],[-8.425305499999979,53.96431899600003],[-8.148062,54.12523950000008],[-8.053147499999966,54.104686498000035],[-8.11553749999996,53.953230500000075],[-7.918767000999935,53.82148599800007],[-7.875693,53.77736349600008],[-8.035976499999947,53.60768649800008],[-7.910323499999947,53.371518500000036],[-8.027095,53.32122099600008],[-8.051905499999975,53.27382799600008],[-7.968462,53.218818499000065],[-8.081456499999945,53.16688449800006],[-8.359736,52.971351499000036],[-8.634478,53.04178499800008],[-8.85870449999993,52.96821],[-8.89154,53.06064199800005],[-9.009308499999975,53.14081749800005],[-9.05427,53.164757499000075],[-8.92340749999994,53.20873799900005],[-8.999464,53.22568449800008],[-8.933212,53.270670996000035],[-9.52627549999994,53.22205849800008],[-9.63143794299998,53.296942787000035],[-9.630072499999926,53.29734849600004],[-9.631445286999963,53.29734542800003],[-9.633099,53.388104500000054],[-9.774115000999927,53.29205949800007],[-9.906928,53.32518749800005],[-9.783823499999926,53.389879998000026],[-9.85217849999998,53.434093],[-9.934913,53.379014],[-10.179155499999979,53.40698899800003],[-10.081167499999935,53.48861949800005],[-10.017894499999954,53.46405999900003],[-10.189941,53.556088500000044]],[[-9.800738259999946,53.60891226000007],[-9.786883,53.60339249700007],[-9.684343499999954,53.60861199800007],[-9.800738259999946,53.60891226000007]]],[[[-9.96094549999998,54.018669500000044],[-9.914447,53.96378799900003],[-9.96222,53.87436949800008],[-10.072347,53.97315799900008],[-10.237484,53.979537],[-9.96094549999998,54.018669500000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE02","LEVL_CODE":2,"CNTR_CODE":"IE","NAME_LATN":"Southern and Eastern","NUTS_NAME":"Southern and Eastern"},"geometry":{"type":"Polygon","coordinates":[[[-10.359855687999982,51.89811573800006],[-10.292081,51.91340949900007],[-10.305404416999977,51.91783544400005],[-10.179232499999955,51.963561998000046],[-10.321406,51.954053498000064],[-9.792653,52.15557449800008],[-10.45567,52.09744099900007],[-10.472409,52.18078450000007],[-10.176341499999978,52.29132850000008],[-10.14654749999994,52.26203399700006],[-10.182858499999952,52.231039],[-10.033870499999978,52.283115999000074],[-9.76673200099998,52.248066496000035],[-9.850593,52.28042149700008],[-9.837091499999929,52.389020498000036],[-9.942014,52.422852996000074],[-9.696583500999964,52.47951599900006],[-9.617206499999952,52.58082599800008],[-9.365192,52.57212449900004],[-8.728047813999979,52.66343504100007],[-8.961225499999955,52.68304449900006],[-8.952241,52.78927249800006],[-9.162081499999942,52.61912150000006],[-9.546329499999956,52.64086549700005],[-9.938699,52.559491500000036],[-9.498436499999968,52.75007849800005],[-9.433057499999961,52.88299899800006],[-9.347913499999947,52.928738500000065],[-9.469911499999967,52.94786499800006],[-9.282228499999974,53.144655999000065],[-9.009308499999975,53.14081749800005],[-8.89154,53.06064199800005],[-8.85870449999993,52.96821],[-8.634478,53.04178499800008],[-8.359736,52.971351499000036],[-8.081456499999945,53.16688449800006],[-7.920399,53.088658500000065],[-8.039676,52.88934799900005],[-7.962705,52.85075499900006],[-7.79797949999994,52.980953499000066],[-7.702182499999935,52.934068498000045],[-7.734649499999932,52.858362498000076],[-7.674081,52.78129399900007],[-7.399012499999969,52.79568949800006],[-7.187502,52.893327500000055],[-7.119209,52.82373699900006],[-6.955549499999961,52.81362799800007],[-6.939413,52.88005449700006],[-6.979713,52.977801498000076],[-7.090516,53.00391699900007],[-7.04514949999998,53.07339949700008],[-7.06613650099996,53.169343498000046],[-7.168954,53.18488599900007],[-7.036046,53.190018500000065],[-6.980562500999952,53.31429399700005],[-7.194891499999926,53.434817500000065],[-7.063349,53.468781500000034],[-6.958578,53.653257],[-7.225776,53.70332299900008],[-7.343828499999972,53.79906249700008],[-7.002189,53.765574998000034],[-6.91360549999996,53.80773349800006],[-6.974685,53.870847],[-6.749452499999961,53.91374299700004],[-6.450152499999945,53.76225999700006],[-6.461498,53.71293599900008],[-6.247098,53.72245449900004],[-6.214899499999945,53.633406998000055],[-6.076308499999925,53.51958699800008],[-6.134826499999974,53.45252749900004],[-6.04398649999996,53.37856249600003],[-6.212808,53.33487549800003],[-6.100643499999933,53.28238999800004],[-6.102773,53.21077299900003],[-6.041955,53.11962099900006],[-6.046814,52.991224],[-5.999142,52.96649599800003],[-6.144516500999941,52.73769649800005],[-6.225527,52.64820149700006],[-6.204002499999945,52.544962998000074],[-6.380481499999973,52.35391150000004],[-6.510815499999978,52.351184498000066],[-6.313716,52.23998499900006],[-6.360401499999966,52.17429799800004],[-6.808146499999964,52.21769649700008],[-6.762292,52.25508049900003],[-6.899396,52.15835150000004],[-6.965714,52.24050599700007],[-7.023463499999934,52.133342998000046],[-7.586105112999974,52.10111825300004],[-7.645049,52.06281849900006],[-7.588845499999934,52.05158099700003],[-7.586489,51.991399999000066],[-7.842156,51.954218498000046],[-7.872479,51.87750499900005],[-8.06342449999994,51.80898099800004],[-8.31868849999995,51.842880498000056],[-8.273785499999974,51.79800749900005],[-8.339985,51.716754],[-8.531225499999948,51.68981899800008],[-8.534105,51.60396149800005],[-8.700837499999977,51.65090999900008],[-8.678507,51.60910049600005],[-8.71486,51.56947649800003],[-8.866604,51.591910496000025],[-9.344147,51.46970149900005],[-9.430925499999944,51.51073099700005],[-9.394508516999963,51.55019734000007],[-9.820953499999973,51.44911749800008],[-9.83894349999997,51.481587500000046],[-9.523876,51.61919749800006],[-9.850705000999938,51.54110399900003],[-9.460062499999935,51.68021249600008],[-9.439506499999936,51.723395],[-10.143066499999975,51.59705299800004],[-9.945926499999928,51.71385349900004],[-10.006702,51.71578849900004],[-9.980664,51.733126999000035],[-9.581580500999962,51.870546499000056],[-10.129688000999977,51.737415498000075],[-10.225089500999957,51.781539499000075],[-10.171304,51.805255498000065],[-10.206829,51.843787],[-10.343244,51.78360349800005],[-10.335582,51.84606599800003],[-10.398117499999955,51.844182500000045],[-10.359855687999982,51.89811573800006]],[[-8.651927499999942,52.65725549600006],[-8.614523,52.68985749700005],[-8.68468695699994,52.65991484500006],[-8.651927499999942,52.65725549600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE021","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"Dublin","NUTS_NAME":"Dublin"},"geometry":{"type":"Polygon","coordinates":[[[-6.102773,53.21077299900003],[-6.340626499999928,53.17974499600007],[-6.54688,53.279420997000045],[-6.329222,53.492143500000054],[-6.41914,53.569993999000076],[-6.214899499999945,53.633406998000055],[-6.076308499999925,53.51958699800008],[-6.134826499999974,53.45252749900004],[-6.04398649999996,53.37856249600003],[-6.212808,53.33487549800003],[-6.100643499999933,53.28238999800004],[-6.102773,53.21077299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE022","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"Mid-East","NUTS_NAME":"Mid-East"},"geometry":{"type":"Polygon","coordinates":[[[-6.247098,53.72245449900004],[-6.214899499999945,53.633406998000055],[-6.41914,53.569993999000076],[-6.329222,53.492143500000054],[-6.54688,53.279420997000045],[-6.340626499999928,53.17974499600007],[-6.102773,53.21077299900003],[-6.041955,53.11962099900006],[-6.046814,52.991224],[-5.999142,52.96649599800003],[-6.144516500999941,52.73769649800005],[-6.368553499999962,52.795315498000036],[-6.565406,52.681951996000066],[-6.711146499999927,52.78175999900003],[-6.664682,52.83624750000007],[-6.518704,52.81976449800004],[-6.505131499999948,52.87769049800005],[-6.939413,52.88005449700006],[-6.979713,52.977801498000076],[-7.090516,53.00391699900007],[-7.04514949999998,53.07339949700008],[-7.06613650099996,53.169343498000046],[-7.168954,53.18488599900007],[-7.036046,53.190018500000065],[-6.980562500999952,53.31429399700005],[-7.194891499999926,53.434817500000065],[-7.063349,53.468781500000034],[-6.958578,53.653257],[-7.225776,53.70332299900008],[-7.343828499999972,53.79906249700008],[-7.002189,53.765574998000034],[-6.91360549999996,53.80773349800006],[-6.974685,53.870847],[-6.749452499999961,53.91374299700004],[-6.450152499999945,53.76225999700006],[-6.461498,53.71293599900008],[-6.247098,53.72245449900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE023","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"Mid-West","NUTS_NAME":"Mid-West"},"geometry":{"type":"Polygon","coordinates":[[[-7.674081,52.78129399900007],[-7.630603,52.67695049800005],[-7.722461500999941,52.60165299800008],[-8.175573,52.67705549800007],[-8.231604500999936,52.51539149700005],[-8.404372499999965,52.46393799800006],[-8.167976499999952,52.367853997000054],[-8.183208499999978,52.28767549600008],[-8.341029499999934,52.329276496000034],[-8.51683400099995,52.28051750000003],[-8.80131549999993,52.387470998000026],[-9.217751,52.298225],[-9.347903,52.361465],[-9.268899,52.40772449900004],[-9.365192,52.57212449900004],[-8.728047813999979,52.66343504100007],[-8.961225499999955,52.68304449900006],[-8.952241,52.78927249800006],[-9.162081499999942,52.61912150000006],[-9.546329499999956,52.64086549700005],[-9.938699,52.559491500000036],[-9.498436499999968,52.75007849800005],[-9.433057499999961,52.88299899800006],[-9.347913499999947,52.928738500000065],[-9.469911499999967,52.94786499800006],[-9.282228499999974,53.144655999000065],[-9.009308499999975,53.14081749800005],[-8.89154,53.06064199800005],[-8.85870449999993,52.96821],[-8.634478,53.04178499800008],[-8.359736,52.971351499000036],[-8.081456499999945,53.16688449800006],[-7.920399,53.088658500000065],[-8.039676,52.88934799900005],[-7.962705,52.85075499900006],[-7.79797949999994,52.980953499000066],[-7.702182499999935,52.934068498000045],[-7.734649499999932,52.858362498000076],[-7.674081,52.78129399900007]],[[-8.614523,52.68985749700005],[-8.68468695699994,52.65991484500006],[-8.651927499999942,52.65725549600006],[-8.614523,52.68985749700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE024","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"South-East (IE)","NUTS_NAME":"South-East (IE)"},"geometry":{"type":"Polygon","coordinates":[[[-6.144516500999941,52.73769649800005],[-6.225527,52.64820149700006],[-6.204002499999945,52.544962998000074],[-6.380481499999973,52.35391150000004],[-6.510815499999978,52.351184498000066],[-6.313716,52.23998499900006],[-6.360401499999966,52.17429799800004],[-6.808146499999964,52.21769649700008],[-6.762292,52.25508049900003],[-6.899396,52.15835150000004],[-6.965714,52.24050599700007],[-7.023463499999934,52.133342998000046],[-7.586105112999974,52.10111825300004],[-7.645049,52.06281849900006],[-7.588845499999934,52.05158099700003],[-7.586489,51.991399999000066],[-7.842156,51.954218498000046],[-8.137958499999968,52.16380049900005],[-8.103399,52.23118249600003],[-8.183208499999978,52.28767549600008],[-8.167976499999952,52.367853997000054],[-8.404372499999965,52.46393799800006],[-8.231604500999936,52.51539149700005],[-8.175573,52.67705549800007],[-7.722461500999941,52.60165299800008],[-7.630603,52.67695049800005],[-7.674081,52.78129399900007],[-7.399012499999969,52.79568949800006],[-7.187502,52.893327500000055],[-7.119209,52.82373699900006],[-6.955549499999961,52.81362799800007],[-6.939413,52.88005449700006],[-6.505131499999948,52.87769049800005],[-6.518704,52.81976449800004],[-6.664682,52.83624750000007],[-6.711146499999927,52.78175999900003],[-6.565406,52.681951996000066],[-6.368553499999962,52.795315498000036],[-6.144516500999941,52.73769649800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"IE025","LEVL_CODE":3,"CNTR_CODE":"IE","NAME_LATN":"South-West (IE)","NUTS_NAME":"South-West (IE)"},"geometry":{"type":"Polygon","coordinates":[[[-10.359855687999982,51.89811573800006],[-10.292081,51.91340949900007],[-10.305404416999977,51.91783544400005],[-10.179232499999955,51.963561998000046],[-10.321406,51.954053498000064],[-9.792653,52.15557449800008],[-10.45567,52.09744099900007],[-10.472409,52.18078450000007],[-10.176341499999978,52.29132850000008],[-10.14654749999994,52.26203399700006],[-10.182858499999952,52.231039],[-10.033870499999978,52.283115999000074],[-9.76673200099998,52.248066496000035],[-9.850593,52.28042149700008],[-9.837091499999929,52.389020498000036],[-9.942014,52.422852996000074],[-9.696583500999964,52.47951599900006],[-9.617206499999952,52.58082599800008],[-9.365192,52.57212449900004],[-9.268899,52.40772449900004],[-9.347903,52.361465],[-9.217751,52.298225],[-8.80131549999993,52.387470998000026],[-8.51683400099995,52.28051750000003],[-8.341029499999934,52.329276496000034],[-8.183208499999978,52.28767549600008],[-8.103399,52.23118249600003],[-8.137958499999968,52.16380049900005],[-7.842156,51.954218498000046],[-7.872479,51.87750499900005],[-8.06342449999994,51.80898099800004],[-8.31868849999995,51.842880498000056],[-8.273785499999974,51.79800749900005],[-8.339985,51.716754],[-8.531225499999948,51.68981899800008],[-8.534105,51.60396149800005],[-8.700837499999977,51.65090999900008],[-8.678507,51.60910049600005],[-8.71486,51.56947649800003],[-8.866604,51.591910496000025],[-9.344147,51.46970149900005],[-9.430925499999944,51.51073099700005],[-9.394508516999963,51.55019734000007],[-9.820953499999973,51.44911749800008],[-9.83894349999997,51.481587500000046],[-9.523876,51.61919749800006],[-9.850705000999938,51.54110399900003],[-9.460062499999935,51.68021249600008],[-9.439506499999936,51.723395],[-10.143066499999975,51.59705299800004],[-9.945926499999928,51.71385349900004],[-10.006702,51.71578849900004],[-9.980664,51.733126999000035],[-9.581580500999962,51.870546499000056],[-10.129688000999977,51.737415498000075],[-10.225089500999957,51.781539499000075],[-10.171304,51.805255498000065],[-10.206829,51.843787],[-10.343244,51.78360349800005],[-10.335582,51.84606599800003],[-10.398117499999955,51.844182500000045],[-10.359855687999982,51.89811573800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IS","LEVL_CODE":0,"CNTR_CODE":"IS","NAME_LATN":"ÍSLAND","NUTS_NAME":"ÍSLAND"},"geometry":{"type":"Polygon","coordinates":[[[-15.760810499999934,66.28006149900006],[-15.430173,66.16163499800007],[-14.93162,66.36978149900006],[-14.530650500999968,66.37767699900007],[-14.967716499999938,66.28248349900008],[-14.924508499999945,66.21469499900007],[-15.18742949999995,66.10784799800007],[-14.675396499999977,66.04599499800008],[-14.604863499999965,65.95930899900003],[-14.898231,65.75496949800004],[-14.765248,65.78918149900005],[-14.841373499999975,65.725313],[-14.332446,65.789426],[-14.387281499999972,65.72221250000007],[-13.606742500999928,65.51016799900003],[-13.650465500999928,65.38695499900007],[-13.846573,65.35524799600006],[-13.722466,65.33099599800005],[-14.010868,65.26640199900004],[-13.57414149999994,65.26908749700004],[-14.017767,65.19021149600007],[-13.625271,65.18894849800006],[-13.758167,65.131403],[-13.636847,65.08534949800003],[-13.513874499999929,65.16649749900006],[-13.495021,65.07634249600005],[-13.66023249999995,64.98576699800003],[-14.231701,65.03179049900007],[-13.683768,64.91270399900003],[-14.040764,64.93530899900003],[-13.753369499999962,64.865692],[-14.068476499999974,64.79378849700004],[-13.98338149999995,64.73325599900005],[-14.083025499999962,64.70016650000008],[-14.501916499999936,64.79170899900004],[-14.22605249999998,64.64503099800004],[-14.495304,64.65391699800006],[-14.405240499999934,64.60145449900006],[-14.584016499999962,64.54241699900007],[-14.340790499999969,64.59750749900007],[-14.540632000999949,64.40184899800005],[-14.760699,64.37360499900007],[-14.551639499999965,64.40693499900004],[-14.711258499999929,64.42394],[-14.962440499999957,64.23983099800006],[-15.366780499999948,64.29066550000005],[-15.422439,64.26269499700004],[-15.186311,64.23033899700005],[-15.95667549999996,64.13536899900004],[-16.639990499999953,63.79922399800006],[-17.660508499999935,63.72017949800005],[-18.177745,63.45859599700003],[-19.127906499999938,63.40079949900007],[-20.18721549999998,63.53600799900005],[-21.210137500999963,63.87885799900005],[-22.007397,63.835987500000044],[-22.130868499999963,63.836598],[-22.703829,63.80051649900008],[-22.749017,63.970331497000075],[-22.683565,64.08148949900004],[-22.540135499999963,63.97506199900005],[-22.125507,64.04059899900005],[-21.926079,64.09629649900006],[-22.010318499999983,64.16258049700008],[-21.705319499999973,64.17748450000005],[-21.907457,64.23266349900007],[-21.73173849999995,64.34287350000005],[-21.37004749999994,64.38032599800005],[-21.355564,64.38318849800004],[-21.64215,64.40292250000005],[-22.03162149999997,64.30302399800007],[-22.06708,64.31809799700005],[-21.861896,64.38705549900004],[-22.040082499999983,64.40472799900004],[-21.964579,64.49829199900006],[-21.76251949999994,64.58152550000005],[-22.225971499999957,64.47706899900004],[-22.231269,64.56085399800003],[-22.361436499999968,64.58370499600005],[-22.352363499999967,64.76963849900005],[-22.415979499999935,64.81220099800004],[-23.364219,64.82716450000004],[-23.802157499999964,64.727832],[-24.04781,64.878827],[-23.221026,64.924046],[-23.228978,64.994626],[-23.176275499999974,65.01104699900003],[-23.084976,65.00375149900003],[-23.161752,64.94629449900003],[-23.087116,64.91319049900005],[-23.10227249999997,64.96799450000003],[-22.723567499999945,65.080719],[-22.682073500999934,65.05730150000005],[-22.746932,65.01998949800003],[-22.614797,64.99050199800007],[-21.80732249999994,65.02585],[-21.832671,65.070086],[-21.70998949999995,65.15945799700006],[-22.560879,65.16934599800004],[-21.681208,65.45147499900008],[-22.391134499999964,65.484707],[-22.093993499999954,65.603457],[-22.46992549999993,65.50547550000005],[-22.541347499999972,65.64385849900003],[-22.532732499999952,65.55431199800006],[-22.63515949999993,65.52119650000003],[-22.71101749999997,65.58741850000007],[-22.665180499999963,65.63647499800004],[-22.739881,65.62688650000007],[-22.71576,65.49839149900004],[-22.867612,65.53537499900006],[-22.7729,65.60254949900008],[-22.943063,65.61397099800007],[-23.219362,65.48717750000003],[-23.881348,65.40733249800007],[-24.532042499999932,65.50307899900008],[-24.313444499999946,65.64204799800007],[-23.785013499999934,65.53109949900005],[-24.07938950099998,65.64357949900005],[-23.82273749999996,65.614083],[-24.060913,65.69788549900005],[-24.099384,65.80526849800003],[-23.602928,65.69699649900008],[-23.551351,65.604823],[-23.258767,65.67726349800006],[-23.532288,65.72041699900007],[-23.17193649999996,65.77495599700006],[-23.708342,65.77925049700008],[-23.868307,65.88655649900005],[-23.182204499999955,65.83778449700003],[-23.819687,66.03491199900003],[-23.363424,65.98342899800008],[-23.66479949999996,66.113966],[-23.36292549999996,66.08325749700003],[-23.603688,66.14894650000008],[-23.478353,66.19469349900004],[-22.98477,66.04112799900008],[-23.084244,65.96560499600008],[-22.862088,66.03487249800008],[-22.987218499999983,65.90939849800003],[-22.813339,65.98768449900007],[-22.841733,65.89115949900008],[-22.75713049999996,66.05561399800007],[-22.56932949999998,65.97747749900003],[-22.65887000099997,65.82466699600008],[-22.52354349999996,65.97990799900003],[-22.415798,65.908334],[-22.582402499999944,65.78310949600007],[-22.360858,65.92552599800007],[-22.465116,66.05865799900005],[-22.976494,66.22213249900005],[-22.362860499999954,66.26985549900007],[-22.707935,66.30012150000005],[-22.606511,66.366348],[-23.195897499999944,66.35003399600004],[-22.93171049999995,66.46936049900006],[-22.419978499999957,66.45904349900007],[-22.235895,66.35675999900008],[-22.227941,66.26369349900006],[-21.741101,66.18164149900008],[-21.765164,66.12080899800003],[-21.73148,66.06771799700005],[-21.62859150099996,66.04753799900004],[-21.667259,66.01549899900004],[-21.545334000999958,66.08792249800007],[-21.327746499999932,66.006199],[-21.608357,65.95821899800006],[-21.28961449999997,65.92644449900007],[-21.28864049999993,65.83212450000008],[-21.469181499999934,65.76982349900004],[-21.342016499999943,65.73476750000003],[-21.782532499999945,65.76451349700005],[-21.636721,65.644187],[-21.39565349999998,65.62315199700004],[-21.480286499999977,65.559568],[-21.29994449999998,65.59562499800006],[-21.474996,65.44616749900007],[-21.282908499999962,65.48345599900006],[-21.094104,65.21000549600006],[-21.094844499999965,65.44756299800008],[-20.91352,65.35300799800007],[-20.990073,65.47367799900007],[-20.932713499999977,65.58107750000005],[-20.671690499999954,65.68711399900008],[-20.624893,65.67363199900007],[-20.631715500999974,65.59743149900004],[-20.378496000999974,65.609833],[-20.264036,65.72534549800008],[-20.421178,66.08405149900005],[-20.113411,66.11330849800004],[-19.647483,65.75102649900003],[-19.473689,65.73660650000005],[-19.361081499999955,65.82932399900005],[-19.508402,65.964345],[-19.40412649999996,65.99444499600008],[-19.43621449999995,66.04766449900006],[-18.77751,66.191427],[-18.544888,66.08678650000007],[-18.507263,66.02431449900007],[-18.529367499999978,65.97252549600006],[-18.19074250099993,65.851921],[-18.160408,65.73242499900005],[-18.052459,65.657922],[-18.092563499999926,65.750939],[-18.05872,65.83999699800006],[-18.32334100099996,66.12036299600004],[-17.933026,66.14752599800005],[-17.639081499999975,65.98938699700005],[-17.412598499999945,65.99224649600006],[-17.145071499999972,66.20707699900004],[-16.879783499999974,66.12259949900005],[-16.49728349999998,66.19722049900008],[-16.416826499999956,66.277237],[-16.56935650099996,66.48356099800003],[-16.19401249999993,66.53742],[-15.958868499999937,66.51431250000007],[-15.935112,66.41800699800007],[-15.684705,66.32908099900004],[-15.760810499999934,66.28006149900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IS0","LEVL_CODE":1,"CNTR_CODE":"IS","NAME_LATN":"ÍSLAND","NUTS_NAME":"ÍSLAND"},"geometry":{"type":"Polygon","coordinates":[[[-15.760810499999934,66.28006149900006],[-15.430173,66.16163499800007],[-14.93162,66.36978149900006],[-14.530650500999968,66.37767699900007],[-14.967716499999938,66.28248349900008],[-14.924508499999945,66.21469499900007],[-15.18742949999995,66.10784799800007],[-14.675396499999977,66.04599499800008],[-14.604863499999965,65.95930899900003],[-14.898231,65.75496949800004],[-14.765248,65.78918149900005],[-14.841373499999975,65.725313],[-14.332446,65.789426],[-14.387281499999972,65.72221250000007],[-13.606742500999928,65.51016799900003],[-13.650465500999928,65.38695499900007],[-13.846573,65.35524799600006],[-13.722466,65.33099599800005],[-14.010868,65.26640199900004],[-13.57414149999994,65.26908749700004],[-14.017767,65.19021149600007],[-13.625271,65.18894849800006],[-13.758167,65.131403],[-13.636847,65.08534949800003],[-13.513874499999929,65.16649749900006],[-13.495021,65.07634249600005],[-13.66023249999995,64.98576699800003],[-14.231701,65.03179049900007],[-13.683768,64.91270399900003],[-14.040764,64.93530899900003],[-13.753369499999962,64.865692],[-14.068476499999974,64.79378849700004],[-13.98338149999995,64.73325599900005],[-14.083025499999962,64.70016650000008],[-14.501916499999936,64.79170899900004],[-14.22605249999998,64.64503099800004],[-14.495304,64.65391699800006],[-14.405240499999934,64.60145449900006],[-14.584016499999962,64.54241699900007],[-14.340790499999969,64.59750749900007],[-14.540632000999949,64.40184899800005],[-14.760699,64.37360499900007],[-14.551639499999965,64.40693499900004],[-14.711258499999929,64.42394],[-14.962440499999957,64.23983099800006],[-15.366780499999948,64.29066550000005],[-15.422439,64.26269499700004],[-15.186311,64.23033899700005],[-15.95667549999996,64.13536899900004],[-16.639990499999953,63.79922399800006],[-17.660508499999935,63.72017949800005],[-18.177745,63.45859599700003],[-19.127906499999938,63.40079949900007],[-20.18721549999998,63.53600799900005],[-21.210137500999963,63.87885799900005],[-22.007397,63.835987500000044],[-22.130868499999963,63.836598],[-22.703829,63.80051649900008],[-22.749017,63.970331497000075],[-22.683565,64.08148949900004],[-22.540135499999963,63.97506199900005],[-22.125507,64.04059899900005],[-21.926079,64.09629649900006],[-22.010318499999983,64.16258049700008],[-21.705319499999973,64.17748450000005],[-21.907457,64.23266349900007],[-21.73173849999995,64.34287350000005],[-21.37004749999994,64.38032599800005],[-21.355564,64.38318849800004],[-21.64215,64.40292250000005],[-22.03162149999997,64.30302399800007],[-22.06708,64.31809799700005],[-21.861896,64.38705549900004],[-22.040082499999983,64.40472799900004],[-21.964579,64.49829199900006],[-21.76251949999994,64.58152550000005],[-22.225971499999957,64.47706899900004],[-22.231269,64.56085399800003],[-22.361436499999968,64.58370499600005],[-22.352363499999967,64.76963849900005],[-22.415979499999935,64.81220099800004],[-23.364219,64.82716450000004],[-23.802157499999964,64.727832],[-24.04781,64.878827],[-23.221026,64.924046],[-23.228978,64.994626],[-23.176275499999974,65.01104699900003],[-23.084976,65.00375149900003],[-23.161752,64.94629449900003],[-23.087116,64.91319049900005],[-23.10227249999997,64.96799450000003],[-22.723567499999945,65.080719],[-22.682073500999934,65.05730150000005],[-22.746932,65.01998949800003],[-22.614797,64.99050199800007],[-21.80732249999994,65.02585],[-21.832671,65.070086],[-21.70998949999995,65.15945799700006],[-22.560879,65.16934599800004],[-21.681208,65.45147499900008],[-22.391134499999964,65.484707],[-22.093993499999954,65.603457],[-22.46992549999993,65.50547550000005],[-22.541347499999972,65.64385849900003],[-22.532732499999952,65.55431199800006],[-22.63515949999993,65.52119650000003],[-22.71101749999997,65.58741850000007],[-22.665180499999963,65.63647499800004],[-22.739881,65.62688650000007],[-22.71576,65.49839149900004],[-22.867612,65.53537499900006],[-22.7729,65.60254949900008],[-22.943063,65.61397099800007],[-23.219362,65.48717750000003],[-23.881348,65.40733249800007],[-24.532042499999932,65.50307899900008],[-24.313444499999946,65.64204799800007],[-23.785013499999934,65.53109949900005],[-24.07938950099998,65.64357949900005],[-23.82273749999996,65.614083],[-24.060913,65.69788549900005],[-24.099384,65.80526849800003],[-23.602928,65.69699649900008],[-23.551351,65.604823],[-23.258767,65.67726349800006],[-23.532288,65.72041699900007],[-23.17193649999996,65.77495599700006],[-23.708342,65.77925049700008],[-23.868307,65.88655649900005],[-23.182204499999955,65.83778449700003],[-23.819687,66.03491199900003],[-23.363424,65.98342899800008],[-23.66479949999996,66.113966],[-23.36292549999996,66.08325749700003],[-23.603688,66.14894650000008],[-23.478353,66.19469349900004],[-22.98477,66.04112799900008],[-23.084244,65.96560499600008],[-22.862088,66.03487249800008],[-22.987218499999983,65.90939849800003],[-22.813339,65.98768449900007],[-22.841733,65.89115949900008],[-22.75713049999996,66.05561399800007],[-22.56932949999998,65.97747749900003],[-22.65887000099997,65.82466699600008],[-22.52354349999996,65.97990799900003],[-22.415798,65.908334],[-22.582402499999944,65.78310949600007],[-22.360858,65.92552599800007],[-22.465116,66.05865799900005],[-22.976494,66.22213249900005],[-22.362860499999954,66.26985549900007],[-22.707935,66.30012150000005],[-22.606511,66.366348],[-23.195897499999944,66.35003399600004],[-22.93171049999995,66.46936049900006],[-22.419978499999957,66.45904349900007],[-22.235895,66.35675999900008],[-22.227941,66.26369349900006],[-21.741101,66.18164149900008],[-21.765164,66.12080899800003],[-21.73148,66.06771799700005],[-21.62859150099996,66.04753799900004],[-21.667259,66.01549899900004],[-21.545334000999958,66.08792249800007],[-21.327746499999932,66.006199],[-21.608357,65.95821899800006],[-21.28961449999997,65.92644449900007],[-21.28864049999993,65.83212450000008],[-21.469181499999934,65.76982349900004],[-21.342016499999943,65.73476750000003],[-21.782532499999945,65.76451349700005],[-21.636721,65.644187],[-21.39565349999998,65.62315199700004],[-21.480286499999977,65.559568],[-21.29994449999998,65.59562499800006],[-21.474996,65.44616749900007],[-21.282908499999962,65.48345599900006],[-21.094104,65.21000549600006],[-21.094844499999965,65.44756299800008],[-20.91352,65.35300799800007],[-20.990073,65.47367799900007],[-20.932713499999977,65.58107750000005],[-20.671690499999954,65.68711399900008],[-20.624893,65.67363199900007],[-20.631715500999974,65.59743149900004],[-20.378496000999974,65.609833],[-20.264036,65.72534549800008],[-20.421178,66.08405149900005],[-20.113411,66.11330849800004],[-19.647483,65.75102649900003],[-19.473689,65.73660650000005],[-19.361081499999955,65.82932399900005],[-19.508402,65.964345],[-19.40412649999996,65.99444499600008],[-19.43621449999995,66.04766449900006],[-18.77751,66.191427],[-18.544888,66.08678650000007],[-18.507263,66.02431449900007],[-18.529367499999978,65.97252549600006],[-18.19074250099993,65.851921],[-18.160408,65.73242499900005],[-18.052459,65.657922],[-18.092563499999926,65.750939],[-18.05872,65.83999699800006],[-18.32334100099996,66.12036299600004],[-17.933026,66.14752599800005],[-17.639081499999975,65.98938699700005],[-17.412598499999945,65.99224649600006],[-17.145071499999972,66.20707699900004],[-16.879783499999974,66.12259949900005],[-16.49728349999998,66.19722049900008],[-16.416826499999956,66.277237],[-16.56935650099996,66.48356099800003],[-16.19401249999993,66.53742],[-15.958868499999937,66.51431250000007],[-15.935112,66.41800699800007],[-15.684705,66.32908099900004],[-15.760810499999934,66.28006149900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IS00","LEVL_CODE":2,"CNTR_CODE":"IS","NAME_LATN":"Ísland","NUTS_NAME":"Ísland"},"geometry":{"type":"Polygon","coordinates":[[[-15.760810499999934,66.28006149900006],[-15.430173,66.16163499800007],[-14.93162,66.36978149900006],[-14.530650500999968,66.37767699900007],[-14.967716499999938,66.28248349900008],[-14.924508499999945,66.21469499900007],[-15.18742949999995,66.10784799800007],[-14.675396499999977,66.04599499800008],[-14.604863499999965,65.95930899900003],[-14.898231,65.75496949800004],[-14.765248,65.78918149900005],[-14.841373499999975,65.725313],[-14.332446,65.789426],[-14.387281499999972,65.72221250000007],[-13.606742500999928,65.51016799900003],[-13.650465500999928,65.38695499900007],[-13.846573,65.35524799600006],[-13.722466,65.33099599800005],[-14.010868,65.26640199900004],[-13.57414149999994,65.26908749700004],[-14.017767,65.19021149600007],[-13.625271,65.18894849800006],[-13.758167,65.131403],[-13.636847,65.08534949800003],[-13.513874499999929,65.16649749900006],[-13.495021,65.07634249600005],[-13.66023249999995,64.98576699800003],[-14.231701,65.03179049900007],[-13.683768,64.91270399900003],[-14.040764,64.93530899900003],[-13.753369499999962,64.865692],[-14.068476499999974,64.79378849700004],[-13.98338149999995,64.73325599900005],[-14.083025499999962,64.70016650000008],[-14.501916499999936,64.79170899900004],[-14.22605249999998,64.64503099800004],[-14.495304,64.65391699800006],[-14.405240499999934,64.60145449900006],[-14.584016499999962,64.54241699900007],[-14.340790499999969,64.59750749900007],[-14.540632000999949,64.40184899800005],[-14.760699,64.37360499900007],[-14.551639499999965,64.40693499900004],[-14.711258499999929,64.42394],[-14.962440499999957,64.23983099800006],[-15.366780499999948,64.29066550000005],[-15.422439,64.26269499700004],[-15.186311,64.23033899700005],[-15.95667549999996,64.13536899900004],[-16.639990499999953,63.79922399800006],[-17.660508499999935,63.72017949800005],[-18.177745,63.45859599700003],[-19.127906499999938,63.40079949900007],[-20.18721549999998,63.53600799900005],[-21.210137500999963,63.87885799900005],[-22.007397,63.835987500000044],[-22.130868499999963,63.836598],[-22.703829,63.80051649900008],[-22.749017,63.970331497000075],[-22.683565,64.08148949900004],[-22.540135499999963,63.97506199900005],[-22.125507,64.04059899900005],[-21.926079,64.09629649900006],[-22.010318499999983,64.16258049700008],[-21.705319499999973,64.17748450000005],[-21.907457,64.23266349900007],[-21.73173849999995,64.34287350000005],[-21.37004749999994,64.38032599800005],[-21.355564,64.38318849800004],[-21.64215,64.40292250000005],[-22.03162149999997,64.30302399800007],[-22.06708,64.31809799700005],[-21.861896,64.38705549900004],[-22.040082499999983,64.40472799900004],[-21.964579,64.49829199900006],[-21.76251949999994,64.58152550000005],[-22.225971499999957,64.47706899900004],[-22.231269,64.56085399800003],[-22.361436499999968,64.58370499600005],[-22.352363499999967,64.76963849900005],[-22.415979499999935,64.81220099800004],[-23.364219,64.82716450000004],[-23.802157499999964,64.727832],[-24.04781,64.878827],[-23.221026,64.924046],[-23.228978,64.994626],[-23.176275499999974,65.01104699900003],[-23.084976,65.00375149900003],[-23.161752,64.94629449900003],[-23.087116,64.91319049900005],[-23.10227249999997,64.96799450000003],[-22.723567499999945,65.080719],[-22.682073500999934,65.05730150000005],[-22.746932,65.01998949800003],[-22.614797,64.99050199800007],[-21.80732249999994,65.02585],[-21.832671,65.070086],[-21.70998949999995,65.15945799700006],[-22.560879,65.16934599800004],[-21.681208,65.45147499900008],[-22.391134499999964,65.484707],[-22.093993499999954,65.603457],[-22.46992549999993,65.50547550000005],[-22.541347499999972,65.64385849900003],[-22.532732499999952,65.55431199800006],[-22.63515949999993,65.52119650000003],[-22.71101749999997,65.58741850000007],[-22.665180499999963,65.63647499800004],[-22.739881,65.62688650000007],[-22.71576,65.49839149900004],[-22.867612,65.53537499900006],[-22.7729,65.60254949900008],[-22.943063,65.61397099800007],[-23.219362,65.48717750000003],[-23.881348,65.40733249800007],[-24.532042499999932,65.50307899900008],[-24.313444499999946,65.64204799800007],[-23.785013499999934,65.53109949900005],[-24.07938950099998,65.64357949900005],[-23.82273749999996,65.614083],[-24.060913,65.69788549900005],[-24.099384,65.80526849800003],[-23.602928,65.69699649900008],[-23.551351,65.604823],[-23.258767,65.67726349800006],[-23.532288,65.72041699900007],[-23.17193649999996,65.77495599700006],[-23.708342,65.77925049700008],[-23.868307,65.88655649900005],[-23.182204499999955,65.83778449700003],[-23.819687,66.03491199900003],[-23.363424,65.98342899800008],[-23.66479949999996,66.113966],[-23.36292549999996,66.08325749700003],[-23.603688,66.14894650000008],[-23.478353,66.19469349900004],[-22.98477,66.04112799900008],[-23.084244,65.96560499600008],[-22.862088,66.03487249800008],[-22.987218499999983,65.90939849800003],[-22.813339,65.98768449900007],[-22.841733,65.89115949900008],[-22.75713049999996,66.05561399800007],[-22.56932949999998,65.97747749900003],[-22.65887000099997,65.82466699600008],[-22.52354349999996,65.97990799900003],[-22.415798,65.908334],[-22.582402499999944,65.78310949600007],[-22.360858,65.92552599800007],[-22.465116,66.05865799900005],[-22.976494,66.22213249900005],[-22.362860499999954,66.26985549900007],[-22.707935,66.30012150000005],[-22.606511,66.366348],[-23.195897499999944,66.35003399600004],[-22.93171049999995,66.46936049900006],[-22.419978499999957,66.45904349900007],[-22.235895,66.35675999900008],[-22.227941,66.26369349900006],[-21.741101,66.18164149900008],[-21.765164,66.12080899800003],[-21.73148,66.06771799700005],[-21.62859150099996,66.04753799900004],[-21.667259,66.01549899900004],[-21.545334000999958,66.08792249800007],[-21.327746499999932,66.006199],[-21.608357,65.95821899800006],[-21.28961449999997,65.92644449900007],[-21.28864049999993,65.83212450000008],[-21.469181499999934,65.76982349900004],[-21.342016499999943,65.73476750000003],[-21.782532499999945,65.76451349700005],[-21.636721,65.644187],[-21.39565349999998,65.62315199700004],[-21.480286499999977,65.559568],[-21.29994449999998,65.59562499800006],[-21.474996,65.44616749900007],[-21.282908499999962,65.48345599900006],[-21.094104,65.21000549600006],[-21.094844499999965,65.44756299800008],[-20.91352,65.35300799800007],[-20.990073,65.47367799900007],[-20.932713499999977,65.58107750000005],[-20.671690499999954,65.68711399900008],[-20.624893,65.67363199900007],[-20.631715500999974,65.59743149900004],[-20.378496000999974,65.609833],[-20.264036,65.72534549800008],[-20.421178,66.08405149900005],[-20.113411,66.11330849800004],[-19.647483,65.75102649900003],[-19.473689,65.73660650000005],[-19.361081499999955,65.82932399900005],[-19.508402,65.964345],[-19.40412649999996,65.99444499600008],[-19.43621449999995,66.04766449900006],[-18.77751,66.191427],[-18.544888,66.08678650000007],[-18.507263,66.02431449900007],[-18.529367499999978,65.97252549600006],[-18.19074250099993,65.851921],[-18.160408,65.73242499900005],[-18.052459,65.657922],[-18.092563499999926,65.750939],[-18.05872,65.83999699800006],[-18.32334100099996,66.12036299600004],[-17.933026,66.14752599800005],[-17.639081499999975,65.98938699700005],[-17.412598499999945,65.99224649600006],[-17.145071499999972,66.20707699900004],[-16.879783499999974,66.12259949900005],[-16.49728349999998,66.19722049900008],[-16.416826499999956,66.277237],[-16.56935650099996,66.48356099800003],[-16.19401249999993,66.53742],[-15.958868499999937,66.51431250000007],[-15.935112,66.41800699800007],[-15.684705,66.32908099900004],[-15.760810499999934,66.28006149900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IS001","LEVL_CODE":3,"CNTR_CODE":"IS","NAME_LATN":"Höfuðborgarsvæði","NUTS_NAME":"Höfuðborgarsvæði"},"geometry":{"type":"Polygon","coordinates":[[[-21.372127499999976,64.19385349800007],[-21.64816449999995,63.96963849800005],[-22.125507,64.04059899900005],[-21.926079,64.09629649900006],[-22.010318499999983,64.16258049700008],[-21.705319499999973,64.17748450000005],[-21.907457,64.23266349900007],[-21.73173849999995,64.34287350000005],[-21.37004749999994,64.38032599800005],[-21.162504,64.35121850000007],[-21.407942,64.23437949700008],[-21.372127499999976,64.19385349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"IS002","LEVL_CODE":3,"CNTR_CODE":"IS","NAME_LATN":"Landsbyggð","NUTS_NAME":"Landsbyggð"},"geometry":{"type":"Polygon","coordinates":[[[-15.760810499999934,66.28006149900006],[-15.430173,66.16163499800007],[-14.93162,66.36978149900006],[-14.530650500999968,66.37767699900007],[-14.967716499999938,66.28248349900008],[-14.924508499999945,66.21469499900007],[-15.18742949999995,66.10784799800007],[-14.675396499999977,66.04599499800008],[-14.604863499999965,65.95930899900003],[-14.898231,65.75496949800004],[-14.765248,65.78918149900005],[-14.841373499999975,65.725313],[-14.332446,65.789426],[-14.387281499999972,65.72221250000007],[-13.606742500999928,65.51016799900003],[-13.650465500999928,65.38695499900007],[-13.846573,65.35524799600006],[-13.722466,65.33099599800005],[-14.010868,65.26640199900004],[-13.57414149999994,65.26908749700004],[-14.017767,65.19021149600007],[-13.625271,65.18894849800006],[-13.758167,65.131403],[-13.636847,65.08534949800003],[-13.513874499999929,65.16649749900006],[-13.495021,65.07634249600005],[-13.66023249999995,64.98576699800003],[-14.231701,65.03179049900007],[-13.683768,64.91270399900003],[-14.040764,64.93530899900003],[-13.753369499999962,64.865692],[-14.068476499999974,64.79378849700004],[-13.98338149999995,64.73325599900005],[-14.083025499999962,64.70016650000008],[-14.501916499999936,64.79170899900004],[-14.22605249999998,64.64503099800004],[-14.495304,64.65391699800006],[-14.405240499999934,64.60145449900006],[-14.584016499999962,64.54241699900007],[-14.340790499999969,64.59750749900007],[-14.540632000999949,64.40184899800005],[-14.760699,64.37360499900007],[-14.551639499999965,64.40693499900004],[-14.711258499999929,64.42394],[-14.962440499999957,64.23983099800006],[-15.366780499999948,64.29066550000005],[-15.422439,64.26269499700004],[-15.186311,64.23033899700005],[-15.95667549999996,64.13536899900004],[-16.639990499999953,63.79922399800006],[-17.660508499999935,63.72017949800005],[-18.177745,63.45859599700003],[-19.127906499999938,63.40079949900007],[-20.18721549999998,63.53600799900005],[-21.210137500999963,63.87885799900005],[-22.007397,63.835987500000044],[-22.130868499999963,63.836598],[-22.703829,63.80051649900008],[-22.749017,63.970331497000075],[-22.683565,64.08148949900004],[-22.540135499999963,63.97506199900005],[-22.125507,64.04059899900005],[-21.64816449999995,63.96963849800005],[-21.372127499999976,64.19385349800007],[-21.407942,64.23437949700008],[-21.162504,64.35121850000007],[-21.37004749999994,64.38032599800005],[-21.355564,64.38318849800004],[-21.64215,64.40292250000005],[-22.03162149999997,64.30302399800007],[-22.06708,64.31809799700005],[-21.861896,64.38705549900004],[-22.040082499999983,64.40472799900004],[-21.964579,64.49829199900006],[-21.76251949999994,64.58152550000005],[-22.225971499999957,64.47706899900004],[-22.231269,64.56085399800003],[-22.361436499999968,64.58370499600005],[-22.352363499999967,64.76963849900005],[-22.415979499999935,64.81220099800004],[-23.364219,64.82716450000004],[-23.802157499999964,64.727832],[-24.04781,64.878827],[-23.221026,64.924046],[-23.228978,64.994626],[-23.176275499999974,65.01104699900003],[-23.084976,65.00375149900003],[-23.161752,64.94629449900003],[-23.087116,64.91319049900005],[-23.10227249999997,64.96799450000003],[-22.723567499999945,65.080719],[-22.682073500999934,65.05730150000005],[-22.746932,65.01998949800003],[-22.614797,64.99050199800007],[-21.80732249999994,65.02585],[-21.832671,65.070086],[-21.70998949999995,65.15945799700006],[-22.560879,65.16934599800004],[-21.681208,65.45147499900008],[-22.391134499999964,65.484707],[-22.093993499999954,65.603457],[-22.46992549999993,65.50547550000005],[-22.541347499999972,65.64385849900003],[-22.532732499999952,65.55431199800006],[-22.63515949999993,65.52119650000003],[-22.71101749999997,65.58741850000007],[-22.665180499999963,65.63647499800004],[-22.739881,65.62688650000007],[-22.71576,65.49839149900004],[-22.867612,65.53537499900006],[-22.7729,65.60254949900008],[-22.943063,65.61397099800007],[-23.219362,65.48717750000003],[-23.881348,65.40733249800007],[-24.532042499999932,65.50307899900008],[-24.313444499999946,65.64204799800007],[-23.785013499999934,65.53109949900005],[-24.07938950099998,65.64357949900005],[-23.82273749999996,65.614083],[-24.060913,65.69788549900005],[-24.099384,65.80526849800003],[-23.602928,65.69699649900008],[-23.551351,65.604823],[-23.258767,65.67726349800006],[-23.532288,65.72041699900007],[-23.17193649999996,65.77495599700006],[-23.708342,65.77925049700008],[-23.868307,65.88655649900005],[-23.182204499999955,65.83778449700003],[-23.819687,66.03491199900003],[-23.363424,65.98342899800008],[-23.66479949999996,66.113966],[-23.36292549999996,66.08325749700003],[-23.603688,66.14894650000008],[-23.478353,66.19469349900004],[-22.98477,66.04112799900008],[-23.084244,65.96560499600008],[-22.862088,66.03487249800008],[-22.987218499999983,65.90939849800003],[-22.813339,65.98768449900007],[-22.841733,65.89115949900008],[-22.75713049999996,66.05561399800007],[-22.56932949999998,65.97747749900003],[-22.65887000099997,65.82466699600008],[-22.52354349999996,65.97990799900003],[-22.415798,65.908334],[-22.582402499999944,65.78310949600007],[-22.360858,65.92552599800007],[-22.465116,66.05865799900005],[-22.976494,66.22213249900005],[-22.362860499999954,66.26985549900007],[-22.707935,66.30012150000005],[-22.606511,66.366348],[-23.195897499999944,66.35003399600004],[-22.93171049999995,66.46936049900006],[-22.419978499999957,66.45904349900007],[-22.235895,66.35675999900008],[-22.227941,66.26369349900006],[-21.741101,66.18164149900008],[-21.765164,66.12080899800003],[-21.73148,66.06771799700005],[-21.62859150099996,66.04753799900004],[-21.667259,66.01549899900004],[-21.545334000999958,66.08792249800007],[-21.327746499999932,66.006199],[-21.608357,65.95821899800006],[-21.28961449999997,65.92644449900007],[-21.28864049999993,65.83212450000008],[-21.469181499999934,65.76982349900004],[-21.342016499999943,65.73476750000003],[-21.782532499999945,65.76451349700005],[-21.636721,65.644187],[-21.39565349999998,65.62315199700004],[-21.480286499999977,65.559568],[-21.29994449999998,65.59562499800006],[-21.474996,65.44616749900007],[-21.282908499999962,65.48345599900006],[-21.094104,65.21000549600006],[-21.094844499999965,65.44756299800008],[-20.91352,65.35300799800007],[-20.990073,65.47367799900007],[-20.932713499999977,65.58107750000005],[-20.671690499999954,65.68711399900008],[-20.624893,65.67363199900007],[-20.631715500999974,65.59743149900004],[-20.378496000999974,65.609833],[-20.264036,65.72534549800008],[-20.421178,66.08405149900005],[-20.113411,66.11330849800004],[-19.647483,65.75102649900003],[-19.473689,65.73660650000005],[-19.361081499999955,65.82932399900005],[-19.508402,65.964345],[-19.40412649999996,65.99444499600008],[-19.43621449999995,66.04766449900006],[-18.77751,66.191427],[-18.544888,66.08678650000007],[-18.507263,66.02431449900007],[-18.529367499999978,65.97252549600006],[-18.19074250099993,65.851921],[-18.160408,65.73242499900005],[-18.052459,65.657922],[-18.092563499999926,65.750939],[-18.05872,65.83999699800006],[-18.32334100099996,66.12036299600004],[-17.933026,66.14752599800005],[-17.639081499999975,65.98938699700005],[-17.412598499999945,65.99224649600006],[-17.145071499999972,66.20707699900004],[-16.879783499999974,66.12259949900005],[-16.49728349999998,66.19722049900008],[-16.416826499999956,66.277237],[-16.56935650099996,66.48356099800003],[-16.19401249999993,66.53742],[-15.958868499999937,66.51431250000007],[-15.935112,66.41800699800007],[-15.684705,66.32908099900004],[-15.760810499999934,66.28006149900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"IT","LEVL_CODE":0,"CNTR_CODE":"IT","NAME_LATN":"ITALIA","NUTS_NAME":"ITALIA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.121343500000023,47.00700199800008],[12.143811,46.91377999700006],[12.273703,46.88438799800008],[12.312809,46.78386599600003],[12.477924,46.679835497000056],[12.690635,46.656972],[12.731392999000036,46.63428799900004],[13.504249500000071,46.566303997000034],[13.714184999000054,46.522703499000045],[13.684032,46.43747250000007],[13.375492500000064,46.29823249800006],[13.423357,46.206053997000026],[13.664347500000076,46.177549998000075],[13.496938999000065,46.05133499800007],[13.476164999000048,46.00608149800007],[13.526538,45.97231849800005],[13.643036,45.98270199800004],[13.575802500000066,45.847736498000074],[13.597145,45.81952250000006],[13.596243,45.80793750000004],[13.795683500000052,45.74320449900006],[13.918656500000054,45.63351749800006],[13.850761500000033,45.58493249600008],[13.722823500000061,45.59472549600008],[13.799811,45.60977199600006],[13.753360500000042,45.680596998000055],[13.579783,45.786951498000064],[13.520863500000075,45.762534498000036],[13.53142,45.72383149800004],[13.373012,45.68000649800007],[13.459073076000038,45.70803148200008],[13.406493500000067,45.725175],[13.130853,45.77184449900005],[13.067232500000046,45.710487999000065],[13.098785501000066,45.644532996000066],[12.43416250000007,45.424485996000044],[12.486428499000056,45.49952449700004],[12.631836,45.53433849800007],[12.413683999000057,45.54431050000005],[12.164675500000044,45.39072949900003],[12.132562,45.30037049900005],[12.200494,45.257336999000074],[12.204566500000055,45.19767599700003],[12.298646500000075,45.21860999900008],[12.330571500000076,45.16055049900007],[12.328777,45.151860998000075],[12.332355500000062,45.09098350000005],[12.302155606000042,45.13788754600006],[12.279706043000033,45.123266827000066],[12.31234899900005,44.96700849800004],[12.409864,45.031746499000064],[12.494037,44.97136599700008],[12.485925,44.895982497000034],[12.389819500000044,44.87723599900005],[12.399056500000029,44.79261550000007],[12.265868500000067,44.82332599800003],[12.239688,44.69413849700004],[12.269892,44.630007496000076],[12.384282,44.22472649900004],[12.450349,44.16218949600005],[12.75078,43.97060150000004],[13.172615,43.75034599800006],[13.530893500000047,43.619984498000065],[13.642096500000036,43.47414249800005],[13.742970499000023,43.29414699800003],[13.849453,43.066659497000046],[13.91573150000005,42.89457699900004],[14.005647500000066,42.69359099900004],[14.146906500000057,42.53059649800008],[14.253960499000073,42.44479849800007],[14.509372,42.25165949800004],[14.710411,42.17232149600005],[14.718282,42.10572499800003],[14.779646999000022,42.070021498000074],[15.138178500000038,41.92700749800008],[16.059203,41.94750049800007],[16.176531500000067,41.88478949900008],[16.197651500000063,41.80332799800004],[16.141325,41.74129799700006],[15.89714,41.60323199800007],[15.923958500000026,41.49635299800008],[16.02473850000007,41.425590498000076],[16.542302499000073,41.229440999000076],[17.054502,41.07813399500003],[17.388981,40.89186349800008],[18.014299,40.64179799800007],[18.097453,40.515377998000076],[18.423196500000074,40.29373849700005],[18.517056,40.135147998000036],[18.407547500000078,39.974780500000065],[18.369068,39.793742],[18.047947500000078,39.92847849800006],[17.993416500000023,40.00004949700008],[18.010002,40.10898749800003],[17.903836500000068,40.25683649700005],[17.763504499000078,40.295506999000054],[17.510776500000077,40.294624999000064],[17.224865,40.408818997000026],[17.248176,40.46044399900006],[17.127098,40.51762049900003],[16.993565,40.493809498000076],[16.867264,40.39784549800004],[16.644317,40.119095998000034],[16.59911950000003,40.040231997000035],[16.62626,39.95547799800005],[16.490359500000068,39.80335299700005],[16.527955500000076,39.66865950000005],[16.770733499000073,39.62110549700003],[17.025064500000042,39.48357849800004],[17.13565,39.37396599900006],[17.10419150000007,39.263840998000035],[17.148327,39.210276],[17.11019250000004,39.09960299800008],[17.188881,39.02047199700007],[17.101278500000035,38.90490999600007],[16.890732,38.92712849800006],[16.566985500000044,38.76548199800004],[16.53582350000005,38.70611849800008],[16.582084500000065,38.46979049600003],[16.488372,38.35253349900006],[16.326207999000076,38.29452150000003],[16.169725500000027,38.14043699800004],[16.063809500000048,37.92441899600004],[15.678212,37.95411949700008],[15.634096500000055,38.01831599900004],[15.636063,38.231473497000025],[15.808353500000067,38.29168949900003],[15.918956,38.50676149800006],[15.928529,38.55029899800007],[15.828991,38.61772449700004],[15.848117500000058,38.658864497000025],[15.983204,38.72185349600005],[16.12969,38.71597199900003],[16.213753500000053,38.81043999800005],[16.215453500000024,38.92621249800004],[16.093153,39.04874299700003],[16.036746,39.349211996000065],[15.873549,39.55545449700003],[15.755958500000077,39.92349599900007],[15.644945499000073,40.042790498000045],[15.532799,40.07377949600004],[15.356917,39.99909099700005],[15.132452500000056,40.17125949800004],[14.908925,40.24347149700003],[15.001006,40.379208997000035],[14.787251500000025,40.66618049700003],[14.747420500000032,40.67764649800006],[14.69241450000004,40.63406599800004],[14.633576,40.64854399700005],[14.528490500000032,40.60715249700007],[14.46862,40.62020149800003],[14.324673,40.56908499800005],[14.479247500000042,40.69592299800007],[14.460677500000031,40.74293050000006],[14.274676,40.84441749700005],[14.04498650000005,40.79068099700004],[14.03213050000005,40.89883899700004],[13.760795500000029,41.22316799600003],[13.622573,41.26081199600003],[13.549584500000037,41.20663449600005],[13.28150850000003,41.29756449800004],[13.067982,41.221978999000044],[12.952066,41.360478998000076],[12.773321500000065,41.41626399800003],[12.61795,41.45189],[12.416086500000063,41.65572399600006],[12.231206,41.741052996000064],[12.14400850000004,41.913842],[11.915473500000076,42.039472],[11.835364,42.029520998000066],[11.733844,42.15805899800006],[11.632385,42.29534649800007],[11.44993850000003,42.37767050000008],[11.09798150000006,42.39314949800007],[11.104887500000075,42.44757350000003],[11.183892,42.463179998000044],[11.17657950000006,42.541543999000055],[10.949364499000069,42.738098998000055],[10.73105350000003,42.80122749800006],[10.786732500000028,42.888363998000045],[10.705672500000048,42.941864499000076],[10.499046,42.93527050000006],[10.528247499000031,43.231603998000026],[10.330268500000045,43.47383049800004],[10.299847,43.58192899800008],[10.258118,43.815146998000046],[10.143476,43.97542049800006],[10.018769500000076,44.044535997000025],[9.848135,44.109999996000056],[9.833157500000027,44.04951299700008],[9.511344,44.21667549700004],[9.23027250000007,44.34827149800003],[9.218968,44.298583499000074],[8.83545650000002,44.423954999000046],[8.63301,44.37980349700007],[8.449540500000069,44.286779498000044],[8.403300500000057,44.18100949800004],[8.269205,44.13914099700003],[8.135350999000025,43.93894049700003],[7.672410500000069,43.77635699900003],[7.529827,43.78400799600007],[7.501208,43.87497399800003],[7.714236500000027,44.06151349800007],[7.684466,44.173948996000036],[7.357702500000073,44.11620099800007],[7.007760500000074,44.23670049800006],[6.887428,44.36128699900007],[6.912351500000057,44.450991499000054],[6.853195,44.52856949800008],[6.932863,44.57307149900004],[6.948443,44.65474199800008],[7.065755,44.71346449600003],[7.006606,44.839803998000036],[6.751577,44.90558199700007],[6.744224,45.01574049800007],[6.6733,45.01962749900008],[6.630051,45.109856499000045],[6.893854500000032,45.13691449700008],[7.125157,45.243994497000074],[7.109843500000068,45.32694349600007],[7.185578500000076,45.40326749800005],[7.104723,45.46845449800003],[7.00001750000007,45.50466699700007],[7.000765,45.64000749600007],[6.845962,45.690503997000064],[6.802368500000057,45.77856199900003],[6.818154500000048,45.83595249800004],[7.044886,45.92241299800003],[7.117904,45.85919249800003],[7.578607500000032,45.98601250000007],[7.864076500000067,45.916750499000045],[7.87713750000006,45.92695499700005],[7.908679500000062,45.996936498000025],[8.012413,46.01196999800004],[8.034763,46.100836498000035],[8.15534450000007,46.147631],[8.085928,46.26661749700003],[8.384717,46.452158498000074],[8.452797,46.455350999000075],[8.444526500000052,46.24887749800007],[8.713936,46.09727199800005],[8.852114,46.07564149700005],[8.785907,45.989075496000055],[8.893755,45.959014996000064],[8.912147,45.830444998000075],[9.034462,45.82301699900006],[9.08880350000004,45.896897],[8.98827650000004,45.97228249700004],[9.159377500000062,46.16960099900007],[9.24853150000007,46.233768],[9.300141500000052,46.32645649800003],[9.247002500000065,46.44679449800003],[9.282739500000048,46.496694999000056],[9.464343,46.50778249700005],[9.460813500000029,46.375970497000026],[9.54957550000006,46.30233799900003],[9.714149500000076,46.29270849800008],[9.735838500000057,46.35031299800005],[9.907831499000054,46.38050599800005],[9.996764,46.35093450000005],[10.04367,46.22974449700007],[10.135604,46.226256499000044],[10.176524500000028,46.258276497000054],[10.104273500000033,46.33366750000005],[10.166681500000038,46.407648],[10.042330500000048,46.44236649900006],[10.043919500000072,46.53753099800008],[10.244874500000037,46.62209199700004],[10.295437,46.54989349700003],[10.452801,46.53068299800003],[10.492064,46.61538649700003],[10.402287999000066,46.63681199600006],[10.39332150000007,46.68920749600005],[10.469651500000055,46.85490899900003],[10.667368500000066,46.87549049800003],[10.757589500000051,46.78518599900008],[11.02225,46.76541049800005],[11.164281500000072,46.96572249900004],[11.627199500000074,47.01329899900003],[11.747028500000056,46.96887249800005],[12.136014,47.08066749900007],[12.24074550000006,47.06916849800007],[12.121343500000023,47.00700199800008]],[[12.404607500000054,43.95567850000003],[12.4177,43.89902999600008],[12.493956,43.91554650000006],[12.513435500000071,43.99126949600003],[12.404607500000054,43.95567850000003]],[[12.44668850000005,41.90174499700004],[12.458003,41.901485498000056],[12.45582,41.90719749800007],[12.44668850000005,41.90174499700004]]],[[[15.572551,38.23420499800005],[15.493701500000043,38.074642],[15.258082500000057,37.80722399700005],[15.176152500000057,37.57575649800003],[15.091007500000046,37.50002999800006],[15.091499,37.35770849900007],[15.255533,37.24181699800005],[15.189416,37.23612499800004],[15.186361,37.19083699600003],[15.296380500000055,37.105878],[15.315962500000069,37.035012497000025],[15.111754500000075,36.85025599800008],[15.116582,36.67495599800003],[15.000358,36.70284749700005],[14.780548500000066,36.70395149700005],[14.493577,36.78673349700006],[14.337594500000023,37.001979998000024],[14.036207,37.10602799800006],[13.750427999000067,37.149429498000075],[13.550732500000038,37.286950500000046],[13.272413,37.38909699900006],[13.17173,37.49062699700005],[13.020311,37.49505049800007],[12.896496,37.57711849700007],[12.672787,37.560080998000046],[12.471194,37.703940497000076],[12.442929500000048,37.81105199800004],[12.561744500000032,38.06556199700003],[12.711114,38.109605997000074],[12.731687,38.18029749800007],[12.811005,38.081515497000055],[12.878819500000077,38.02912599700005],[12.977262500000052,38.04020599800003],[13.078245500000037,38.08820049900004],[13.052464,38.13869699900005],[13.105645,38.19080999900007],[13.191043,38.16928849900006],[13.318283500000064,38.22422299800007],[13.365693500000077,38.182289498000046],[13.383294500000034,38.10926849700007],[13.537639500000068,38.11271549700007],[13.566937500000051,38.03840849900007],[13.74382950000006,37.97024649700006],[14.02708750000005,38.04060899700005],[14.183486,38.01939649800005],[14.526039,38.042452998000044],[14.74790899900006,38.165756997000074],[14.915518500000076,38.192245497000044],[15.113349500000027,38.125076498000055],[15.24130450000007,38.216853997000044],[15.541000500000052,38.30162849900006],[15.652927,38.26710399600006],[15.572551,38.23420499800005]]],[[[10.443561499000054,42.846501998000065],[10.42203950000004,42.70751299600005],[10.342138500000033,42.76413499800003],[10.155029,42.72833549600006],[10.103713500000026,42.76818049700006],[10.443561499000054,42.846501998000065]]],[[[9.42346199900004,41.176741],[9.443369,41.09925399900004],[9.566906499000027,41.116017498000076],[9.503958500000067,41.010275498000055],[9.664734500000066,40.996196498000074],[9.498756,40.917446],[9.644384,40.91990549800005],[9.748871999000073,40.66040699700005],[9.82702050000006,40.51211249800008],[9.62652699900002,40.22487699700008],[9.735204500000066,40.075661998000044],[9.687999499000057,39.98521649800006],[9.651118500000052,39.549323997000045],[9.555865,39.13396699600003],[9.109986500000048,39.214018500000066],[9.008827,39.12130449800003],[9.045840500000054,39.05619349800003],[9.011614500000064,38.98710349800007],[8.859653,38.87746949800004],[8.721109,38.93861149900005],[8.646450500000071,38.89431099800004],[8.612392,38.957787997000025],[8.567906,39.042935999000065],[8.469251500000041,39.05392449800007],[8.487763,39.08878149800006],[8.368104499000026,39.21381049900003],[8.436887500000068,39.29004899700004],[8.373012500000073,39.37460649800005],[8.393,39.446529498000075],[8.470979,39.60082999800005],[8.444049,39.758194497000034],[8.50225450000005,39.713006998000026],[8.552179,39.863522998000064],[8.39866150000006,39.904071498000064],[8.413568500000054,40.019162996000034],[8.376406,40.03584549800007],[8.492870500000038,40.08687150000003],[8.457160500000043,40.17520899800007],[8.482863500000065,40.28547049800005],[8.382633500000054,40.338828996000075],[8.399809,40.407568498000046],[8.306376,40.58615149800005],[8.198148,40.57122499700006],[8.193946,40.618244498000024],[8.161289500000066,40.56088349800007],[8.146809500000074,40.62602949800004],[8.204835,40.680237498000054],[8.135010500000021,40.73634899800004],[8.21902,40.86889049800004],[8.17597,40.934798999000066],[8.201709,40.97256649700006],[8.313342500000033,40.844939],[8.41712750000005,40.83831349800005],[8.52743,40.82418799700008],[8.802688500000045,40.931047997000064],[9.01318599900003,41.12518049800008],[9.15260550000005,41.155430496000065],[9.163662,41.235624],[9.42346199900004,41.176741]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC","LEVL_CODE":1,"CNTR_CODE":"IT","NAME_LATN":"NORD-OVEST","NUTS_NAME":"NORD-OVEST"},"geometry":{"type":"Polygon","coordinates":[[[10.585728,46.24794349800004],[10.452963,45.989621998000075],[10.508597500000064,45.92356549800007],[10.502388,45.830372999000076],[10.561458500000072,45.78391799900004],[10.840176500000041,45.83275850000007],[10.631074,45.60951250000005],[10.654655500000047,45.41582699800006],[10.714573500000029,45.41758599800005],[10.728336,45.291306500000076],[10.782865,45.315409499000054],[10.936780500000054,45.232916497000076],[11.050501,45.14956749800007],[11.028722500000072,45.123041],[11.205502500000023,45.10948749800008],[11.175304,45.06006149800004],[11.426765500000045,44.950075999000035],[11.246231500000022,44.951427996000064],[10.887909500000035,44.914265998000076],[10.730587,44.991889498000035],[10.504344,44.92241799800007],[10.464030500000035,44.937171500000034],[10.083503,45.04395850000003],[9.988977499000043,45.13210999800003],[9.891101500000048,45.13089799900007],[9.895781,45.07964199600008],[9.713752,45.058392998000045],[9.548684500000036,45.13264849800004],[9.532049500000028,45.07842749800005],[9.37082950000007,45.048136998000075],[9.293407,44.93250949800006],[9.27743350000003,44.895598],[9.363122,44.82239099800006],[9.286998,44.772481999000036],[9.32466050000005,44.689999999000065],[9.200074,44.686099496000054],[9.202995500000043,44.61347649800007],[9.493363,44.555858998000076],[9.49682850000005,44.48271250000005],[9.439189,44.42905349800003],[9.479059500000062,44.40924049900008],[9.577126500000077,44.43523699900004],[9.686725500000023,44.36592199800003],[10.069286500000032,44.097985497000025],[10.018769500000076,44.044535997000025],[9.848135,44.109999996000056],[9.833157500000027,44.04951299700008],[9.511344,44.21667549700004],[9.23027250000007,44.34827149800003],[9.218968,44.298583499000074],[8.83545650000002,44.423954999000046],[8.63301,44.37980349700007],[8.449540500000069,44.286779498000044],[8.403300500000057,44.18100949800004],[8.269205,44.13914099700003],[8.135350999000025,43.93894049700003],[7.672410500000069,43.77635699900003],[7.529827,43.78400799600007],[7.501208,43.87497399800003],[7.714236500000027,44.06151349800007],[7.684466,44.173948996000036],[7.357702500000073,44.11620099800007],[7.007760500000074,44.23670049800006],[6.887428,44.36128699900007],[6.912351500000057,44.450991499000054],[6.853195,44.52856949800008],[6.932863,44.57307149900004],[6.948443,44.65474199800008],[7.065755,44.71346449600003],[7.006606,44.839803998000036],[6.751577,44.90558199700007],[6.744224,45.01574049800007],[6.6733,45.01962749900008],[6.630051,45.109856499000045],[6.893854500000032,45.13691449700008],[7.125157,45.243994497000074],[7.109843500000068,45.32694349600007],[7.185578500000076,45.40326749800005],[7.104723,45.46845449800003],[7.00001750000007,45.50466699700007],[7.000765,45.64000749600007],[6.845962,45.690503997000064],[6.802368500000057,45.77856199900003],[6.818154500000048,45.83595249800004],[7.044886,45.92241299800003],[7.117904,45.85919249800003],[7.578607500000032,45.98601250000007],[7.864076500000067,45.916750499000045],[7.87713750000006,45.92695499700005],[7.908679500000062,45.996936498000025],[8.012413,46.01196999800004],[8.034763,46.100836498000035],[8.15534450000007,46.147631],[8.085928,46.26661749700003],[8.384717,46.452158498000074],[8.452797,46.455350999000075],[8.444526500000052,46.24887749800007],[8.713936,46.09727199800005],[8.852114,46.07564149700005],[8.785907,45.989075496000055],[8.893755,45.959014996000064],[8.912147,45.830444998000075],[9.034462,45.82301699900006],[9.08880350000004,45.896897],[8.98827650000004,45.97228249700004],[9.159377500000062,46.16960099900007],[9.24853150000007,46.233768],[9.300141500000052,46.32645649800003],[9.247002500000065,46.44679449800003],[9.282739500000048,46.496694999000056],[9.464343,46.50778249700005],[9.460813500000029,46.375970497000026],[9.54957550000006,46.30233799900003],[9.714149500000076,46.29270849800008],[9.735838500000057,46.35031299800005],[9.907831499000054,46.38050599800005],[9.996764,46.35093450000005],[10.04367,46.22974449700007],[10.135604,46.226256499000044],[10.176524500000028,46.258276497000054],[10.104273500000033,46.33366750000005],[10.166681500000038,46.407648],[10.042330500000048,46.44236649900006],[10.043919500000072,46.53753099800008],[10.244874500000037,46.62209199700004],[10.295437,46.54989349700003],[10.452801,46.53068299800003],[10.622145500000045,46.44810199800003],[10.632285500000023,46.40139],[10.515751500000022,46.34322449600006],[10.585728,46.24794349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC1","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Piemonte","NUTS_NAME":"Piemonte"},"geometry":{"type":"Polygon","coordinates":[[[8.730787500000076,45.375541500000054],[8.690663500000028,45.29252649800003],[8.613533500000074,45.354502498000045],[8.51356,45.31327850000008],[8.547797500000058,45.168147496000074],[8.657043,45.028422497000065],[8.887709,45.05914899800007],[9.049136,44.85029099900004],[9.213572999000064,44.75233349800004],[9.200074,44.686099496000054],[9.202995500000043,44.61347649800007],[9.137694500000066,44.573915998000075],[8.921793500000035,44.67230899600003],[8.91701,44.56058649800008],[8.773596500000053,44.503908498000044],[8.718786,44.58072399800005],[8.648572999000066,44.58334699900007],[8.576182,44.509206],[8.261596,44.519424496000056],[8.252819499000054,44.528740500000026],[8.196678,44.47839299800006],[8.20795750000002,44.404814498000064],[8.06244749900003,44.30286949900005],[8.097202,44.17868499900004],[7.97896350000002,44.13351749800006],[8.015629500000045,44.11067649800003],[7.727960500000052,44.12729349800003],[7.714236500000027,44.06151349800007],[7.684466,44.173948996000036],[7.357702500000073,44.11620099800007],[7.007760500000074,44.23670049800006],[6.887428,44.36128699900007],[6.912351500000057,44.450991499000054],[6.853195,44.52856949800008],[6.932863,44.57307149900004],[6.948443,44.65474199800008],[7.065755,44.71346449600003],[7.006606,44.839803998000036],[6.751577,44.90558199700007],[6.744224,45.01574049800007],[6.6733,45.01962749900008],[6.630051,45.109856499000045],[6.893854500000032,45.13691449700008],[7.125157,45.243994497000074],[7.109843500000068,45.32694349600007],[7.185578500000076,45.40326749800005],[7.104723,45.46845449800003],[7.489346,45.58310099700003],[7.895811,45.59002799900003],[7.936649,45.72434049900005],[7.862662500000056,45.792161],[7.864076500000067,45.916750499000045],[7.87713750000006,45.92695499700005],[7.908679500000062,45.996936498000025],[8.012413,46.01196999800004],[8.034763,46.100836498000035],[8.15534450000007,46.147631],[8.085928,46.26661749700003],[8.384717,46.452158498000074],[8.452797,46.455350999000075],[8.444526500000052,46.24887749800007],[8.713936,46.09727199800005],[8.726449500000058,46.01796599800008],[8.571646500000043,45.90133499900003],[8.593811,45.82822499800005],[8.555684,45.77499],[8.654234,45.71204799600008],[8.706896500000028,45.55831599900006],[8.84291550000006,45.393840999000076],[8.730787500000076,45.375541500000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC11","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Torino","NUTS_NAME":"Torino"},"geometry":{"type":"Polygon","coordinates":[[[7.895811,45.59002799900003],[7.882509500000026,45.533905497000035],[8.038455500000055,45.44139899900006],[8.027012,45.403092496000056],[7.977598,45.31739749800005],[7.99034,45.23162549800003],[8.151933500000041,45.16855849800004],[8.128986499000064,45.12299599800008],[7.896150999000042,45.09498449600005],[7.921432501000027,45.02345549800003],[7.883448500000043,44.93623499800003],[7.943386500000031,44.84630549900004],[7.598907500000053,44.837102499000025],[7.471091499000067,44.75609749900008],[7.296082500000068,44.78208949800006],[7.065755,44.71346449600003],[7.006606,44.839803998000036],[6.751577,44.90558199700007],[6.744224,45.01574049800007],[6.6733,45.01962749900008],[6.630051,45.109856499000045],[6.893854500000032,45.13691449700008],[7.125157,45.243994497000074],[7.109843500000068,45.32694349600007],[7.185578500000076,45.40326749800005],[7.104723,45.46845449800003],[7.489346,45.58310099700003],[7.895811,45.59002799900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC12","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Vercelli","NUTS_NAME":"Vercelli"},"geometry":{"type":"Polygon","coordinates":[[[8.349897,45.772373996000056],[8.392018,45.70347749800004],[8.310590500000046,45.697208999000054],[8.391455,45.61390799800006],[8.4061,45.41921149800004],[8.51356,45.31327850000008],[8.547797500000058,45.168147496000074],[8.151933500000041,45.16855849800004],[7.99034,45.23162549800003],[7.977598,45.31739749800005],[8.027012,45.403092496000056],[8.122848,45.376085998000065],[8.292550500000061,45.54158949600003],[8.255178500000056,45.60868099800007],[8.32793,45.64694499900003],[8.181237,45.752092496000046],[7.936649,45.72434049900005],[7.862662500000056,45.792161],[7.864076500000067,45.916750499000045],[7.87713750000006,45.92695499700005],[8.209025500000052,45.945044996000036],[8.303001,45.886856],[8.335927,45.84448450000008],[8.318237,45.766932498000074],[8.349897,45.772373996000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC13","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Biella","NUTS_NAME":"Biella"},"geometry":{"type":"Polygon","coordinates":[[[8.027012,45.403092496000056],[8.038455500000055,45.44139899900006],[7.882509500000026,45.533905497000035],[7.895811,45.59002799900003],[7.936649,45.72434049900005],[8.181237,45.752092496000046],[8.32793,45.64694499900003],[8.255178500000056,45.60868099800007],[8.292550500000061,45.54158949600003],[8.122848,45.376085998000065],[8.027012,45.403092496000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC14","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Verbano-Cusio-Ossola","NUTS_NAME":"Verbano-Cusio-Ossola"},"geometry":{"type":"Polygon","coordinates":[[[8.713936,46.09727199800005],[8.726449500000058,46.01796599800008],[8.571646500000043,45.90133499900003],[8.593811,45.82822499800005],[8.473713499000041,45.86643799700005],[8.349897,45.772373996000056],[8.318237,45.766932498000074],[8.335927,45.84448450000008],[8.303001,45.886856],[8.209025500000052,45.945044996000036],[7.87713750000006,45.92695499700005],[7.908679500000062,45.996936498000025],[8.012413,46.01196999800004],[8.034763,46.100836498000035],[8.15534450000007,46.147631],[8.085928,46.26661749700003],[8.384717,46.452158498000074],[8.452797,46.455350999000075],[8.444526500000052,46.24887749800007],[8.713936,46.09727199800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC15","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Novara","NUTS_NAME":"Novara"},"geometry":{"type":"Polygon","coordinates":[[[8.593811,45.82822499800005],[8.555684,45.77499],[8.654234,45.71204799600008],[8.706896500000028,45.55831599900006],[8.84291550000006,45.393840999000076],[8.730787500000076,45.375541500000054],[8.690663500000028,45.29252649800003],[8.613533500000074,45.354502498000045],[8.51356,45.31327850000008],[8.4061,45.41921149800004],[8.391455,45.61390799800006],[8.310590500000046,45.697208999000054],[8.392018,45.70347749800004],[8.349897,45.772373996000056],[8.473713499000041,45.86643799700005],[8.593811,45.82822499800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC16","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Cuneo","NUTS_NAME":"Cuneo"},"geometry":{"type":"Polygon","coordinates":[[[7.943386500000031,44.84630549900004],[8.130588,44.810117],[8.095101500000055,44.758488998000075],[8.152246500000047,44.71313699800004],[8.256835,44.73225449800003],[8.217624500000056,44.61515849800003],[8.252819499000054,44.528740500000026],[8.196678,44.47839299800006],[8.20795750000002,44.404814498000064],[8.06244749900003,44.30286949900005],[8.097202,44.17868499900004],[7.97896350000002,44.13351749800006],[8.015629500000045,44.11067649800003],[7.727960500000052,44.12729349800003],[7.714236500000027,44.06151349800007],[7.684466,44.173948996000036],[7.357702500000073,44.11620099800007],[7.007760500000074,44.23670049800006],[6.887428,44.36128699900007],[6.912351500000057,44.450991499000054],[6.853195,44.52856949800008],[6.932863,44.57307149900004],[6.948443,44.65474199800008],[7.065755,44.71346449600003],[7.296082500000068,44.78208949800006],[7.471091499000067,44.75609749900008],[7.598907500000053,44.837102499000025],[7.943386500000031,44.84630549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC17","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Asti","NUTS_NAME":"Asti"},"geometry":{"type":"Polygon","coordinates":[[[8.128986499000064,45.12299599800008],[8.158971500000064,45.02747849800005],[8.311515500000041,45.07065099600004],[8.387373,44.968930500000056],[8.376081,44.848818496000035],[8.502041500000075,44.785736996000026],[8.420566,44.692289998000035],[8.348377500000026,44.69372849700005],[8.367931,44.57451049800005],[8.261596,44.519424496000056],[8.252819499000054,44.528740500000026],[8.217624500000056,44.61515849800003],[8.256835,44.73225449800003],[8.152246500000047,44.71313699800004],[8.095101500000055,44.758488998000075],[8.130588,44.810117],[7.943386500000031,44.84630549900004],[7.883448500000043,44.93623499800003],[7.921432501000027,45.02345549800003],[7.896150999000042,45.09498449600005],[8.128986499000064,45.12299599800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC18","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Alessandria","NUTS_NAME":"Alessandria"},"geometry":{"type":"Polygon","coordinates":[[[8.547797500000058,45.168147496000074],[8.657043,45.028422497000065],[8.887709,45.05914899800007],[9.049136,44.85029099900004],[9.213572999000064,44.75233349800004],[9.200074,44.686099496000054],[9.202995500000043,44.61347649800007],[9.137694500000066,44.573915998000075],[8.921793500000035,44.67230899600003],[8.91701,44.56058649800008],[8.773596500000053,44.503908498000044],[8.718786,44.58072399800005],[8.648572999000066,44.58334699900007],[8.576182,44.509206],[8.261596,44.519424496000056],[8.367931,44.57451049800005],[8.348377500000026,44.69372849700005],[8.420566,44.692289998000035],[8.502041500000075,44.785736996000026],[8.376081,44.848818496000035],[8.387373,44.968930500000056],[8.311515500000041,45.07065099600004],[8.158971500000064,45.02747849800005],[8.128986499000064,45.12299599800008],[8.151933500000041,45.16855849800004],[8.547797500000058,45.168147496000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC2","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Valle d'Aosta/Vallée d'Aoste","NUTS_NAME":"Valle d'Aosta/Vallée d'Aoste"},"geometry":{"type":"Polygon","coordinates":[[[7.864076500000067,45.916750499000045],[7.862662500000056,45.792161],[7.936649,45.72434049900005],[7.895811,45.59002799900003],[7.489346,45.58310099700003],[7.104723,45.46845449800003],[7.00001750000007,45.50466699700007],[7.000765,45.64000749600007],[6.845962,45.690503997000064],[6.802368500000057,45.77856199900003],[6.818154500000048,45.83595249800004],[7.044886,45.92241299800003],[7.117904,45.85919249800003],[7.578607500000032,45.98601250000007],[7.864076500000067,45.916750499000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC20","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Valle d'Aosta/Vallée d'Aoste","NUTS_NAME":"Valle d'Aosta/Vallée d'Aoste"},"geometry":{"type":"Polygon","coordinates":[[[7.864076500000067,45.916750499000045],[7.862662500000056,45.792161],[7.936649,45.72434049900005],[7.895811,45.59002799900003],[7.489346,45.58310099700003],[7.104723,45.46845449800003],[7.00001750000007,45.50466699700007],[7.000765,45.64000749600007],[6.845962,45.690503997000064],[6.802368500000057,45.77856199900003],[6.818154500000048,45.83595249800004],[7.044886,45.92241299800003],[7.117904,45.85919249800003],[7.578607500000032,45.98601250000007],[7.864076500000067,45.916750499000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC3","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Liguria","NUTS_NAME":"Liguria"},"geometry":{"type":"Polygon","coordinates":[[[8.921793500000035,44.67230899600003],[9.137694500000066,44.573915998000075],[9.202995500000043,44.61347649800007],[9.493363,44.555858998000076],[9.49682850000005,44.48271250000005],[9.439189,44.42905349800003],[9.479059500000062,44.40924049900008],[9.577126500000077,44.43523699900004],[9.686725500000023,44.36592199800003],[10.069286500000032,44.097985497000025],[10.018769500000076,44.044535997000025],[9.848135,44.109999996000056],[9.833157500000027,44.04951299700008],[9.511344,44.21667549700004],[9.23027250000007,44.34827149800003],[9.218968,44.298583499000074],[8.83545650000002,44.423954999000046],[8.63301,44.37980349700007],[8.449540500000069,44.286779498000044],[8.403300500000057,44.18100949800004],[8.269205,44.13914099700003],[8.135350999000025,43.93894049700003],[7.672410500000069,43.77635699900003],[7.529827,43.78400799600007],[7.501208,43.87497399800003],[7.714236500000027,44.06151349800007],[7.727960500000052,44.12729349800003],[8.015629500000045,44.11067649800003],[7.97896350000002,44.13351749800006],[8.097202,44.17868499900004],[8.06244749900003,44.30286949900005],[8.20795750000002,44.404814498000064],[8.196678,44.47839299800006],[8.252819499000054,44.528740500000026],[8.261596,44.519424496000056],[8.576182,44.509206],[8.648572999000066,44.58334699900007],[8.718786,44.58072399800005],[8.773596500000053,44.503908498000044],[8.91701,44.56058649800008],[8.921793500000035,44.67230899600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC31","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Imperia","NUTS_NAME":"Imperia"},"geometry":{"type":"Polygon","coordinates":[[[8.015629500000045,44.11067649800003],[8.03668649900004,44.05840549900006],[7.980374,44.012809496000045],[8.135350999000025,43.93894049700003],[7.672410500000069,43.77635699900003],[7.529827,43.78400799600007],[7.501208,43.87497399800003],[7.714236500000027,44.06151349800007],[7.727960500000052,44.12729349800003],[8.015629500000045,44.11067649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC32","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Savona","NUTS_NAME":"Savona"},"geometry":{"type":"Polygon","coordinates":[[[8.261596,44.519424496000056],[8.576182,44.509206],[8.662724,44.48571849700005],[8.588732,44.426353998000025],[8.63301,44.37980349700007],[8.449540500000069,44.286779498000044],[8.403300500000057,44.18100949800004],[8.269205,44.13914099700003],[8.135350999000025,43.93894049700003],[7.980374,44.012809496000045],[8.03668649900004,44.05840549900006],[8.015629500000045,44.11067649800003],[7.97896350000002,44.13351749800006],[8.097202,44.17868499900004],[8.06244749900003,44.30286949900005],[8.20795750000002,44.404814498000064],[8.196678,44.47839299800006],[8.252819499000054,44.528740500000026],[8.261596,44.519424496000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC33","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Genova","NUTS_NAME":"Genova"},"geometry":{"type":"Polygon","coordinates":[[[9.202995500000043,44.61347649800007],[9.493363,44.555858998000076],[9.49682850000005,44.48271250000005],[9.439189,44.42905349800003],[9.479059500000062,44.40924049900008],[9.573356500000045,44.27073349800003],[9.511344,44.21667549700004],[9.23027250000007,44.34827149800003],[9.218968,44.298583499000074],[8.83545650000002,44.423954999000046],[8.63301,44.37980349700007],[8.588732,44.426353998000025],[8.662724,44.48571849700005],[8.576182,44.509206],[8.648572999000066,44.58334699900007],[8.718786,44.58072399800005],[8.773596500000053,44.503908498000044],[8.91701,44.56058649800008],[8.921793500000035,44.67230899600003],[9.137694500000066,44.573915998000075],[9.202995500000043,44.61347649800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC34","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"La Spezia","NUTS_NAME":"La Spezia"},"geometry":{"type":"Polygon","coordinates":[[[9.686725500000023,44.36592199800003],[10.069286500000032,44.097985497000025],[10.018769500000076,44.044535997000025],[9.848135,44.109999996000056],[9.833157500000027,44.04951299700008],[9.511344,44.21667549700004],[9.573356500000045,44.27073349800003],[9.479059500000062,44.40924049900008],[9.577126500000077,44.43523699900004],[9.686725500000023,44.36592199800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC4","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Lombardia","NUTS_NAME":"Lombardia"},"geometry":{"type":"Polygon","coordinates":[[[10.585728,46.24794349800004],[10.452963,45.989621998000075],[10.508597500000064,45.92356549800007],[10.502388,45.830372999000076],[10.561458500000072,45.78391799900004],[10.840176500000041,45.83275850000007],[10.631074,45.60951250000005],[10.654655500000047,45.41582699800006],[10.714573500000029,45.41758599800005],[10.728336,45.291306500000076],[10.782865,45.315409499000054],[10.936780500000054,45.232916497000076],[11.050501,45.14956749800007],[11.028722500000072,45.123041],[11.205502500000023,45.10948749800008],[11.175304,45.06006149800004],[11.426765500000045,44.950075999000035],[11.246231500000022,44.951427996000064],[10.887909500000035,44.914265998000076],[10.730587,44.991889498000035],[10.504344,44.92241799800007],[10.464030500000035,44.937171500000034],[10.083503,45.04395850000003],[9.988977499000043,45.13210999800003],[9.891101500000048,45.13089799900007],[9.895781,45.07964199600008],[9.713752,45.058392998000045],[9.548684500000036,45.13264849800004],[9.532049500000028,45.07842749800005],[9.37082950000007,45.048136998000075],[9.293407,44.93250949800006],[9.27743350000003,44.895598],[9.363122,44.82239099800006],[9.286998,44.772481999000036],[9.32466050000005,44.689999999000065],[9.200074,44.686099496000054],[9.213572999000064,44.75233349800004],[9.049136,44.85029099900004],[8.887709,45.05914899800007],[8.657043,45.028422497000065],[8.547797500000058,45.168147496000074],[8.51356,45.31327850000008],[8.613533500000074,45.354502498000045],[8.690663500000028,45.29252649800003],[8.730787500000076,45.375541500000054],[8.84291550000006,45.393840999000076],[8.706896500000028,45.55831599900006],[8.654234,45.71204799600008],[8.555684,45.77499],[8.593811,45.82822499800005],[8.571646500000043,45.90133499900003],[8.726449500000058,46.01796599800008],[8.713936,46.09727199800005],[8.852114,46.07564149700005],[8.785907,45.989075496000055],[8.893755,45.959014996000064],[8.912147,45.830444998000075],[9.034462,45.82301699900006],[9.08880350000004,45.896897],[8.98827650000004,45.97228249700004],[9.159377500000062,46.16960099900007],[9.24853150000007,46.233768],[9.300141500000052,46.32645649800003],[9.247002500000065,46.44679449800003],[9.282739500000048,46.496694999000056],[9.464343,46.50778249700005],[9.460813500000029,46.375970497000026],[9.54957550000006,46.30233799900003],[9.714149500000076,46.29270849800008],[9.735838500000057,46.35031299800005],[9.907831499000054,46.38050599800005],[9.996764,46.35093450000005],[10.04367,46.22974449700007],[10.135604,46.226256499000044],[10.176524500000028,46.258276497000054],[10.104273500000033,46.33366750000005],[10.166681500000038,46.407648],[10.042330500000048,46.44236649900006],[10.043919500000072,46.53753099800008],[10.244874500000037,46.62209199700004],[10.295437,46.54989349700003],[10.452801,46.53068299800003],[10.622145500000045,46.44810199800003],[10.632285500000023,46.40139],[10.515751500000022,46.34322449600006],[10.585728,46.24794349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC41","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Varese","NUTS_NAME":"Varese"},"geometry":{"type":"Polygon","coordinates":[[[8.912147,45.830444998000075],[8.950393,45.719876],[8.898668,45.67671899900006],[9.055754999000044,45.64650399900006],[9.052902,45.623193497000045],[9.064448500000026,45.58201249800004],[8.967275,45.608280999000044],[8.706896500000028,45.55831599900006],[8.654234,45.71204799600008],[8.555684,45.77499],[8.593811,45.82822499800005],[8.571646500000043,45.90133499900003],[8.726449500000058,46.01796599800008],[8.713936,46.09727199800005],[8.852114,46.07564149700005],[8.785907,45.989075496000055],[8.893755,45.959014996000064],[8.912147,45.830444998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC42","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Como","NUTS_NAME":"Como"},"geometry":{"type":"Polygon","coordinates":[[[9.414123500000073,46.13686949800007],[9.29235649900005,46.11574250000007],[9.256142,45.947572],[9.332798,45.866596498000035],[9.247139,45.74265199800004],[9.194449,45.66768999900006],[9.055754999000044,45.64650399900006],[8.898668,45.67671899900006],[8.950393,45.719876],[8.912147,45.830444998000075],[9.034462,45.82301699900006],[9.08880350000004,45.896897],[8.98827650000004,45.97228249700004],[9.159377500000062,46.16960099900007],[9.24853150000007,46.233768],[9.415229500000066,46.21527599600006],[9.414123500000073,46.13686949800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC43","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Lecco","NUTS_NAME":"Lecco"},"geometry":{"type":"Polygon","coordinates":[[[9.414123500000073,46.13686949800007],[9.526920999000026,46.011509999000054],[9.486118500000032,46.00300549700006],[9.54029,45.949211],[9.524037500000077,45.870348499000045],[9.466562,45.84944999700008],[9.518698,45.78403549800004],[9.445865,45.76051849800007],[9.476588,45.66924849800006],[9.31407850000005,45.660702997000044],[9.247139,45.74265199800004],[9.332798,45.866596498000035],[9.256142,45.947572],[9.29235649900005,46.11574250000007],[9.414123500000073,46.13686949800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC44","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Sondrio","NUTS_NAME":"Sondrio"},"geometry":{"type":"Polygon","coordinates":[[[10.452801,46.53068299800003],[10.622145500000045,46.44810199800003],[10.632285500000023,46.40139],[10.515751500000022,46.34322449600006],[10.179631500000028,46.17757099700003],[10.16959,46.05705349700003],[9.643481500000064,46.05997349800003],[9.526920999000026,46.011509999000054],[9.414123500000073,46.13686949800007],[9.415229500000066,46.21527599600006],[9.24853150000007,46.233768],[9.300141500000052,46.32645649800003],[9.247002500000065,46.44679449800003],[9.282739500000048,46.496694999000056],[9.464343,46.50778249700005],[9.460813500000029,46.375970497000026],[9.54957550000006,46.30233799900003],[9.714149500000076,46.29270849800008],[9.735838500000057,46.35031299800005],[9.907831499000054,46.38050599800005],[9.996764,46.35093450000005],[10.04367,46.22974449700007],[10.135604,46.226256499000044],[10.176524500000028,46.258276497000054],[10.104273500000033,46.33366750000005],[10.166681500000038,46.407648],[10.042330500000048,46.44236649900006],[10.043919500000072,46.53753099800008],[10.244874500000037,46.62209199700004],[10.295437,46.54989349700003],[10.452801,46.53068299800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC46","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Bergamo","NUTS_NAME":"Bergamo"},"geometry":{"type":"Polygon","coordinates":[[[10.16959,46.05705349700003],[10.262241500000073,46.02076449900005],[10.090674500000034,45.92295499800008],[10.149827,45.863099496000075],[10.061131500000045,45.78384099800007],[10.065532,45.68927199800004],[9.850573,45.60315799700004],[9.889638499000057,45.42733499800005],[9.519207,45.504481998000074],[9.550321500000052,45.52419449900003],[9.495871,45.63970399800007],[9.476588,45.66924849800006],[9.445865,45.76051849800007],[9.518698,45.78403549800004],[9.466562,45.84944999700008],[9.524037500000077,45.870348499000045],[9.54029,45.949211],[9.486118500000032,46.00300549700006],[9.526920999000026,46.011509999000054],[9.643481500000064,46.05997349800003],[10.16959,46.05705349700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC47","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Brescia","NUTS_NAME":"Brescia"},"geometry":{"type":"Polygon","coordinates":[[[10.515751500000022,46.34322449600006],[10.585728,46.24794349800004],[10.452963,45.989621998000075],[10.508597500000064,45.92356549800007],[10.502388,45.830372999000076],[10.561458500000072,45.78391799900004],[10.840176500000041,45.83275850000007],[10.631074,45.60951250000005],[10.654655500000047,45.41582699800006],[10.456251500000064,45.41548949600008],[10.447208500000045,45.32709349600003],[10.317723,45.20471599800004],[9.898013999000057,45.33848399800007],[9.889638499000057,45.42733499800005],[9.850573,45.60315799700004],[10.065532,45.68927199800004],[10.061131500000045,45.78384099800007],[10.149827,45.863099496000075],[10.090674500000034,45.92295499800008],[10.262241500000073,46.02076449900005],[10.16959,46.05705349700003],[10.179631500000028,46.17757099700003],[10.515751500000022,46.34322449600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC48","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pavia","NUTS_NAME":"Pavia"},"geometry":{"type":"Polygon","coordinates":[[[9.338846500000045,45.31296649800004],[9.438572,45.18763399800008],[9.531601500000022,45.16535199600003],[9.548684500000036,45.13264849800004],[9.532049500000028,45.07842749800005],[9.37082950000007,45.048136998000075],[9.293407,44.93250949800006],[9.27743350000003,44.895598],[9.363122,44.82239099800006],[9.286998,44.772481999000036],[9.32466050000005,44.689999999000065],[9.200074,44.686099496000054],[9.213572999000064,44.75233349800004],[9.049136,44.85029099900004],[8.887709,45.05914899800007],[8.657043,45.028422497000065],[8.547797500000058,45.168147496000074],[8.51356,45.31327850000008],[8.613533500000074,45.354502498000045],[8.690663500000028,45.29252649800003],[8.730787500000076,45.375541500000054],[8.84291550000006,45.393840999000076],[8.982958,45.26499450100005],[9.201773500000058,45.334816500000045],[9.338846500000045,45.31296649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC49","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Lodi","NUTS_NAME":"Lodi"},"geometry":{"type":"Polygon","coordinates":[[[9.895781,45.07964199600008],[9.713752,45.058392998000045],[9.548684500000036,45.13264849800004],[9.531601500000022,45.16535199600003],[9.438572,45.18763399800008],[9.338846500000045,45.31296649800004],[9.417553500000054,45.45153499800006],[9.48393,45.44869149800007],[9.467488500000059,45.37460499800005],[9.583025500000076,45.36287599800005],[9.784733,45.165756498000064],[9.891101500000048,45.13089799900007],[9.895781,45.07964199600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC4A","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Cremona","NUTS_NAME":"Cremona"},"geometry":{"type":"Polygon","coordinates":[[[9.889638499000057,45.42733499800005],[9.898013999000057,45.33848399800007],[10.317723,45.20471599800004],[10.379216500000041,45.13846549800007],[10.492495,45.13241499800006],[10.416303,45.04131350000006],[10.540125500000045,45.06316050000004],[10.45572450000003,44.98234699600005],[10.596879500000057,45.00395499900003],[10.464030500000035,44.937171500000034],[10.083503,45.04395850000003],[9.988977499000043,45.13210999800003],[9.891101500000048,45.13089799900007],[9.784733,45.165756498000064],[9.583025500000076,45.36287599800005],[9.467488500000059,45.37460499800005],[9.48393,45.44869149800007],[9.519207,45.504481998000074],[9.889638499000057,45.42733499800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC4B","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Mantova","NUTS_NAME":"Mantova"},"geometry":{"type":"Polygon","coordinates":[[[11.205502500000023,45.10948749800008],[11.175304,45.06006149800004],[11.426765500000045,44.950075999000035],[11.246231500000022,44.951427996000064],[10.887909500000035,44.914265998000076],[10.730587,44.991889498000035],[10.504344,44.92241799800007],[10.464030500000035,44.937171500000034],[10.596879500000057,45.00395499900003],[10.45572450000003,44.98234699600005],[10.540125500000045,45.06316050000004],[10.416303,45.04131350000006],[10.492495,45.13241499800006],[10.379216500000041,45.13846549800007],[10.317723,45.20471599800004],[10.447208500000045,45.32709349600003],[10.456251500000064,45.41548949600008],[10.654655500000047,45.41582699800006],[10.714573500000029,45.41758599800005],[10.728336,45.291306500000076],[10.782865,45.315409499000054],[10.936780500000054,45.232916497000076],[11.050501,45.14956749800007],[11.028722500000072,45.123041],[11.205502500000023,45.10948749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC4C","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Milano","NUTS_NAME":"Milano"},"geometry":{"type":"Polygon","coordinates":[[[8.967275,45.608280999000044],[9.064448500000026,45.58201249800004],[9.052902,45.623193497000045],[9.305092999000067,45.53665299900007],[9.495871,45.63970399800007],[9.550321500000052,45.52419449900003],[9.519207,45.504481998000074],[9.48393,45.44869149800007],[9.417553500000054,45.45153499800006],[9.338846500000045,45.31296649800004],[9.201773500000058,45.334816500000045],[8.982958,45.26499450100005],[8.84291550000006,45.393840999000076],[8.706896500000028,45.55831599900006],[8.967275,45.608280999000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITC4D","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Monza e della Brianza","NUTS_NAME":"Monza e della Brianza"},"geometry":{"type":"Polygon","coordinates":[[[9.476588,45.66924849800006],[9.495871,45.63970399800007],[9.305092999000067,45.53665299900007],[9.052902,45.623193497000045],[9.055754999000044,45.64650399900006],[9.194449,45.66768999900006],[9.247139,45.74265199800004],[9.31407850000005,45.660702997000044],[9.476588,45.66924849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH51","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Piacenza","NUTS_NAME":"Piacenza"},"geometry":{"type":"Polygon","coordinates":[[[10.083503,45.04395850000003],[9.994189500000061,44.95519749700003],[9.986707500000023,44.84555199600004],[9.493363,44.555858998000076],[9.202995500000043,44.61347649800007],[9.200074,44.686099496000054],[9.32466050000005,44.689999999000065],[9.286998,44.772481999000036],[9.363122,44.82239099800006],[9.27743350000003,44.895598],[9.293407,44.93250949800006],[9.37082950000007,45.048136998000075],[9.532049500000028,45.07842749800005],[9.548684500000036,45.13264849800004],[9.713752,45.058392998000045],[9.895781,45.07964199600008],[9.891101500000048,45.13089799900007],[9.988977499000043,45.13210999800003],[10.083503,45.04395850000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH52","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Parma","NUTS_NAME":"Parma"},"geometry":{"type":"Polygon","coordinates":[[[10.464030500000035,44.937171500000034],[10.504344,44.92241799800007],[10.423537,44.79819149600007],[10.447076500000037,44.71765699900004],[10.399894,44.56033099800004],[10.142054500000029,44.353852998000036],[9.996692,44.39711249800007],[9.96213,44.467052996000064],[9.81996,44.46640549800003],[9.686725500000023,44.36592199800003],[9.577126500000077,44.43523699900004],[9.479059500000062,44.40924049900008],[9.439189,44.42905349800003],[9.49682850000005,44.48271250000005],[9.493363,44.555858998000076],[9.986707500000023,44.84555199600004],[9.994189500000061,44.95519749700003],[10.083503,45.04395850000003],[10.464030500000035,44.937171500000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH53","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Reggio nell'Emilia","NUTS_NAME":"Reggio nell'Emilia"},"geometry":{"type":"Polygon","coordinates":[[[10.887909500000035,44.914265998000076],[10.815014500000075,44.807213997000076],[10.842064500000049,44.79552599700003],[10.764983,44.540407997000045],[10.61426350000005,44.37473950000003],[10.528048500000068,44.35087099800006],[10.470149,44.226041],[10.253875,44.26856699800004],[10.142054500000029,44.353852998000036],[10.399894,44.56033099800004],[10.447076500000037,44.71765699900004],[10.423537,44.79819149600007],[10.504344,44.92241799800007],[10.730587,44.991889498000035],[10.887909500000035,44.914265998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH54","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Modena","NUTS_NAME":"Modena"},"geometry":{"type":"Polygon","coordinates":[[[11.246231500000022,44.951427996000064],[11.234766500000035,44.905968],[11.368615,44.842669999000066],[11.293370500000037,44.802797498000075],[11.129913,44.78290799600006],[11.078112500000032,44.64701549800003],[11.152475499000047,44.58660549800004],[10.990250500000059,44.440522497000075],[11.048163,44.416117],[11.04016,44.33103599800006],[10.949927500000058,44.29314449700007],[10.957128500000067,44.22605049900005],[10.850155,44.22744049900007],[10.814787,44.11617449800008],[10.653418,44.15778149800008],[10.624078,44.12035150000003],[10.470149,44.226041],[10.528048500000068,44.35087099800006],[10.61426350000005,44.37473950000003],[10.764983,44.540407997000045],[10.842064500000049,44.79552599700003],[10.815014500000075,44.807213997000076],[10.887909500000035,44.914265998000076],[11.246231500000022,44.951427996000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH55","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Bologna","NUTS_NAME":"Bologna"},"geometry":{"type":"Polygon","coordinates":[[[11.789198500000055,44.55433449700007],[11.814468500000032,44.39348649800007],[11.524959500000023,44.157639998000036],[11.421995,44.23987949800005],[11.195749999000043,44.15139849800005],[11.263648500000045,44.10422449600003],[11.202439500000025,44.10072099900003],[11.049449,44.09022999800004],[11.004782,44.12143099600007],[10.917739,44.06276249700005],[10.814787,44.11617449800008],[10.850155,44.22744049900007],[10.957128500000067,44.22605049900005],[10.949927500000058,44.29314449700007],[11.04016,44.33103599800006],[11.048163,44.416117],[10.990250500000059,44.440522497000075],[11.152475499000047,44.58660549800004],[11.078112500000032,44.64701549800003],[11.129913,44.78290799600006],[11.293370500000037,44.802797498000075],[11.253176999000061,44.71630999800004],[11.45280450000007,44.76049599700008],[11.674120500000072,44.63228249600007],[11.800312500000075,44.634510498000054],[11.789198500000055,44.55433449700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH56","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ferrara","NUTS_NAME":"Ferrara"},"geometry":{"type":"Polygon","coordinates":[[[12.399056500000029,44.79261550000007],[12.265868500000067,44.82332599800003],[12.239688,44.69413849700004],[12.269892,44.630007496000076],[12.153804500000035,44.54843549700007],[12.002220499000032,44.600186499000074],[11.789198500000055,44.55433449700007],[11.800312500000075,44.634510498000054],[11.674120500000072,44.63228249600007],[11.45280450000007,44.76049599700008],[11.253176999000061,44.71630999800004],[11.293370500000037,44.802797498000075],[11.368615,44.842669999000066],[11.234766500000035,44.905968],[11.246231500000022,44.951427996000064],[11.426765500000045,44.950075999000035],[11.624835,44.889211500000044],[11.804749,44.97694949800007],[12.098028500000055,44.971468998000034],[12.280897500000037,44.941755998000076],[12.295169500000043,44.859279496000056],[12.399056500000029,44.79261550000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH57","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ravenna","NUTS_NAME":"Ravenna"},"geometry":{"type":"Polygon","coordinates":[[[12.384282,44.22472649900004],[12.213164,44.22081549600006],[12.049321500000076,44.32511249800007],[11.888630999000043,44.17063899600004],[11.756718,44.194878997000046],[11.715906500000074,44.122540498000035],[11.524959500000023,44.157639998000036],[11.814468500000032,44.39348649800007],[11.789198500000055,44.55433449700007],[12.002220499000032,44.600186499000074],[12.153804500000035,44.54843549700007],[12.269892,44.630007496000076],[12.384282,44.22472649900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH58","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Forlì-Cesena","NUTS_NAME":"Forlì-Cesena"},"geometry":{"type":"Polygon","coordinates":[[[12.384282,44.22472649900004],[12.450349,44.16218949600005],[12.452092,44.087181498000064],[12.312544500000058,43.940214998000044],[12.158803,43.920320996000044],[12.107464,43.75375349600006],[11.986528500000077,43.76191399700008],[11.710189,43.877430497000034],[11.646041500000024,43.99065699600004],[11.752958,44.12051850000006],[11.715906500000074,44.122540498000035],[11.756718,44.194878997000046],[11.888630999000043,44.17063899600004],[12.049321500000076,44.32511249800007],[12.213164,44.22081549600006],[12.384282,44.22472649900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITH59","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Rimini","NUTS_NAME":"Rimini"},"geometry":{"type":"Polygon","coordinates":[[[12.450349,44.16218949600005],[12.75078,43.97060150000004],[12.733553500000028,43.869933497000034],[12.669299,43.82322749800005],[12.493956,43.91554650000006],[12.513435500000071,43.99126949600003],[12.404607500000054,43.95567850000003],[12.4177,43.89902999600008],[12.283797500000048,43.764898997000046],[12.107464,43.75375349600006],[12.158803,43.920320996000044],[12.312544500000058,43.940214998000044],[12.452092,44.087181498000064],[12.450349,44.16218949600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI","LEVL_CODE":1,"CNTR_CODE":"IT","NAME_LATN":"CENTRO (IT)","NUTS_NAME":"CENTRO (IT)"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.470149,44.226041],[10.624078,44.12035150000003],[10.653418,44.15778149800008],[10.814787,44.11617449800008],[10.917739,44.06276249700005],[11.004782,44.12143099600007],[11.049449,44.09022999800004],[11.202439500000025,44.10072099900003],[11.263648500000045,44.10422449600003],[11.195749999000043,44.15139849800005],[11.421995,44.23987949800005],[11.524959500000023,44.157639998000036],[11.715906500000074,44.122540498000035],[11.752958,44.12051850000006],[11.646041500000024,43.99065699600004],[11.710189,43.877430497000034],[11.986528500000077,43.76191399700008],[12.107464,43.75375349600006],[12.283797500000048,43.764898997000046],[12.4177,43.89902999600008],[12.493956,43.91554650000006],[12.669299,43.82322749800005],[12.733553500000028,43.869933497000034],[12.75078,43.97060150000004],[13.172615,43.75034599800006],[13.530893500000047,43.619984498000065],[13.642096500000036,43.47414249800005],[13.742970499000023,43.29414699800003],[13.849453,43.066659497000046],[13.91573150000005,42.89457699900004],[13.534506,42.81658549800005],[13.48879,42.73365799700008],[13.357774500000062,42.69406999700004],[13.409763,42.63902249800003],[13.39403149900005,42.591233998000064],[13.191380500000037,42.58749099900007],[13.178439,42.47931599800006],[13.116188500000021,42.44388449600007],[13.194177,42.38916949800006],[13.153368,42.356970499000056],[13.379911500000048,42.18042199800004],[13.248800500000073,42.127403497000046],[13.087079500000073,42.17784499800007],[13.030574500000057,42.11535299900004],[13.054555500000049,42.01634599700003],[13.296299500000032,41.94858849900004],[13.383656,41.901868998000054],[13.379543500000068,41.817429496000045],[13.573855500000036,41.75432049900007],[13.729585,41.798873497000045],[13.941038499000058,41.68794399600006],[14.026036,41.555293998000025],[13.977928,41.46245399900005],[13.861228,41.41653499800003],[13.873718500000052,41.338294],[13.889761500000077,41.297446499000046],[13.760795500000029,41.22316799600003],[13.622573,41.26081199600003],[13.549584500000037,41.20663449600005],[13.28150850000003,41.29756449800004],[13.067982,41.221978999000044],[12.952066,41.360478998000076],[12.773321500000065,41.41626399800003],[12.61795,41.45189],[12.416086500000063,41.65572399600006],[12.231206,41.741052996000064],[12.14400850000004,41.913842],[11.915473500000076,42.039472],[11.835364,42.029520998000066],[11.733844,42.15805899800006],[11.632385,42.29534649800007],[11.44993850000003,42.37767050000008],[11.09798150000006,42.39314949800007],[11.104887500000075,42.44757350000003],[11.183892,42.463179998000044],[11.17657950000006,42.541543999000055],[10.949364499000069,42.738098998000055],[10.73105350000003,42.80122749800006],[10.786732500000028,42.888363998000045],[10.705672500000048,42.941864499000076],[10.499046,42.93527050000006],[10.528247499000031,43.231603998000026],[10.330268500000045,43.47383049800004],[10.299847,43.58192899800008],[10.258118,43.815146998000046],[10.143476,43.97542049800006],[10.018769500000076,44.044535997000025],[10.069286500000032,44.097985497000025],[9.686725500000023,44.36592199800003],[9.81996,44.46640549800003],[9.96213,44.467052996000064],[9.996692,44.39711249800007],[10.142054500000029,44.353852998000036],[10.253875,44.26856699800004],[10.470149,44.226041]],[[12.44668850000005,41.90174499700004],[12.458003,41.901485498000056],[12.45582,41.90719749800007],[12.44668850000005,41.90174499700004]]],[[[10.443561499000054,42.846501998000065],[10.42203950000004,42.70751299600005],[10.342138500000033,42.76413499800003],[10.155029,42.72833549600006],[10.103713500000026,42.76818049700006],[10.443561499000054,42.846501998000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI1","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Toscana","NUTS_NAME":"Toscana"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.470149,44.226041],[10.624078,44.12035150000003],[10.653418,44.15778149800008],[10.814787,44.11617449800008],[10.917739,44.06276249700005],[11.004782,44.12143099600007],[11.049449,44.09022999800004],[11.202439500000025,44.10072099900003],[11.263648500000045,44.10422449600003],[11.195749999000043,44.15139849800005],[11.421995,44.23987949800005],[11.524959500000023,44.157639998000036],[11.715906500000074,44.122540498000035],[11.752958,44.12051850000006],[11.646041500000024,43.99065699600004],[11.710189,43.877430497000034],[11.986528500000077,43.76191399700008],[12.107464,43.75375349600006],[12.283797500000048,43.764898997000046],[12.371335500000043,43.70900450000005],[12.18552550000004,43.64181099700005],[12.213856,43.610824499000046],[12.095412,43.522593],[12.141248500000074,43.46891149800007],[12.025227,43.415556998000056],[12.11878450000006,43.364341498000044],[12.131859,43.29395849800005],[12.223722500000065,43.29624549700003],[12.033978,43.246685],[12.027506,43.19159999900006],[11.96126350000003,43.16689049800004],[11.920441500000038,43.13165049700007],[11.936083500000052,43.055908498000065],[11.982233500000063,43.04885999700008],[11.952102500000024,42.90097049600007],[11.958612500000072,42.87430749600003],[11.933317,42.86932999800007],[11.89498750000007,42.83465349800008],[11.746037,42.78574249800005],[11.818677499000046,42.74575549700006],[11.804794500000071,42.64398299700008],[11.578577,42.566193498000075],[11.561255,42.51727549700007],[11.617744500000072,42.43606149800007],[11.488907,42.43935399900005],[11.44993850000003,42.37767050000008],[11.09798150000006,42.39314949800007],[11.104887500000075,42.44757350000003],[11.183892,42.463179998000044],[11.17657950000006,42.541543999000055],[10.949364499000069,42.738098998000055],[10.73105350000003,42.80122749800006],[10.786732500000028,42.888363998000045],[10.705672500000048,42.941864499000076],[10.499046,42.93527050000006],[10.528247499000031,43.231603998000026],[10.330268500000045,43.47383049800004],[10.299847,43.58192899800008],[10.258118,43.815146998000046],[10.143476,43.97542049800006],[10.018769500000076,44.044535997000025],[10.069286500000032,44.097985497000025],[9.686725500000023,44.36592199800003],[9.81996,44.46640549800003],[9.96213,44.467052996000064],[9.996692,44.39711249800007],[10.142054500000029,44.353852998000036],[10.253875,44.26856699800004],[10.470149,44.226041]]],[[[10.443561499000054,42.846501998000065],[10.42203950000004,42.70751299600005],[10.342138500000033,42.76413499800003],[10.155029,42.72833549600006],[10.103713500000026,42.76818049700006],[10.443561499000054,42.846501998000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI11","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Massa-Carrara","NUTS_NAME":"Massa-Carrara"},"geometry":{"type":"Polygon","coordinates":[[[10.142054500000029,44.353852998000036],[10.253875,44.26856699800004],[10.185177500000066,44.205460498000036],[10.18648950000005,44.123726496000074],[10.239129500000047,44.07161549800003],[10.143476,43.97542049800006],[10.018769500000076,44.044535997000025],[10.069286500000032,44.097985497000025],[9.686725500000023,44.36592199800003],[9.81996,44.46640549800003],[9.96213,44.467052996000064],[9.996692,44.39711249800007],[10.142054500000029,44.353852998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI12","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Lucca","NUTS_NAME":"Lucca"},"geometry":{"type":"Polygon","coordinates":[[[10.470149,44.226041],[10.624078,44.12035150000003],[10.722345,44.08295950000007],[10.737820999000064,44.02492399800008],[10.64813950000007,43.911130996000054],[10.727781500000049,43.81415199700007],[10.711115999000071,43.78633399800003],[10.50088450000004,43.75327899800004],[10.427754,43.82459849700007],[10.258118,43.815146998000046],[10.143476,43.97542049800006],[10.239129500000047,44.07161549800003],[10.18648950000005,44.123726496000074],[10.185177500000066,44.205460498000036],[10.253875,44.26856699800004],[10.470149,44.226041]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI13","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pistoia","NUTS_NAME":"Pistoia"},"geometry":{"type":"Polygon","coordinates":[[[10.814787,44.11617449800008],[10.917739,44.06276249700005],[11.004782,44.12143099600007],[11.049449,44.09022999800004],[11.07067550000005,43.982918],[11.020522500000027,43.922503498000026],[11.024563500000056,43.84456649800006],[10.96412,43.81248099900006],[10.727781500000049,43.81415199700007],[10.64813950000007,43.911130996000054],[10.737820999000064,44.02492399800008],[10.722345,44.08295950000007],[10.624078,44.12035150000003],[10.653418,44.15778149800008],[10.814787,44.11617449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI14","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Firenze","NUTS_NAME":"Firenze"},"geometry":{"type":"Polygon","coordinates":[[[11.524959500000023,44.157639998000036],[11.715906500000074,44.122540498000035],[11.752958,44.12051850000006],[11.646041500000024,43.99065699600004],[11.710189,43.877430497000034],[11.629579,43.84594549700006],[11.596508500000027,43.68594550000006],[11.483942,43.626108],[11.521549500000049,43.58106599900003],[11.397507500000074,43.54822899900006],[11.229665,43.48967049700008],[11.034008,43.54665849700007],[10.952091,43.45229649800007],[10.824755500000038,43.54232399600005],[10.825868,43.620613],[10.933905500000037,43.657497497000065],[10.711115999000071,43.78633399800003],[10.727781500000049,43.81415199700007],[10.96412,43.81248099900006],[11.056753499000024,43.758748999000034],[11.175131500000077,43.95166149800008],[11.202439500000025,44.10072099900003],[11.263648500000045,44.10422449600003],[11.195749999000043,44.15139849800005],[11.421995,44.23987949800005],[11.524959500000023,44.157639998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI15","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Prato","NUTS_NAME":"Prato"},"geometry":{"type":"Polygon","coordinates":[[[10.96412,43.81248099900006],[11.024563500000056,43.84456649800006],[11.020522500000027,43.922503498000026],[11.07067550000005,43.982918],[11.049449,44.09022999800004],[11.202439500000025,44.10072099900003],[11.175131500000077,43.95166149800008],[11.056753499000024,43.758748999000034],[10.96412,43.81248099900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI16","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Livorno","NUTS_NAME":"Livorno"},"geometry":{"type":"MultiPolygon","coordinates":[[[[10.705672500000048,42.941864499000076],[10.499046,42.93527050000006],[10.528247499000031,43.231603998000026],[10.330268500000045,43.47383049800004],[10.299847,43.58192899800008],[10.502454,43.62859949800003],[10.464676,43.54180449800003],[10.50904950000006,43.36321449800005],[10.667189,43.26532149800005],[10.649284500000022,43.17157549800004],[10.717287500000054,43.10850799800005],[10.796260500000074,43.01764499800004],[10.705672500000048,42.941864499000076]]],[[[10.443561499000054,42.846501998000065],[10.42203950000004,42.70751299600005],[10.342138500000033,42.76413499800003],[10.155029,42.72833549600006],[10.103713500000026,42.76818049700006],[10.443561499000054,42.846501998000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI17","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pisa","NUTS_NAME":"Pisa"},"geometry":{"type":"Polygon","coordinates":[[[10.711115999000071,43.78633399800003],[10.933905500000037,43.657497497000065],[10.825868,43.620613],[10.824755500000038,43.54232399600005],[10.952091,43.45229649800007],[11.012687500000027,43.37656549800005],[10.952097500000036,43.30203249700003],[10.99658850000003,43.24013599700004],[10.911575500000026,43.16261049800005],[10.717287500000054,43.10850799800005],[10.649284500000022,43.17157549800004],[10.667189,43.26532149800005],[10.50904950000006,43.36321449800005],[10.464676,43.54180449800003],[10.502454,43.62859949800003],[10.299847,43.58192899800008],[10.258118,43.815146998000046],[10.427754,43.82459849700007],[10.50088450000004,43.75327899800004],[10.711115999000071,43.78633399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI18","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Arezzo","NUTS_NAME":"Arezzo"},"geometry":{"type":"Polygon","coordinates":[[[11.986528500000077,43.76191399700008],[12.107464,43.75375349600006],[12.283797500000048,43.764898997000046],[12.371335500000043,43.70900450000005],[12.18552550000004,43.64181099700005],[12.213856,43.610824499000046],[12.095412,43.522593],[12.141248500000074,43.46891149800007],[12.025227,43.415556998000056],[12.11878450000006,43.364341498000044],[12.131859,43.29395849800005],[12.223722500000065,43.29624549700003],[12.033978,43.246685],[12.027506,43.19159999900006],[11.96126350000003,43.16689049800004],[11.720545,43.25479899800007],[11.397507500000074,43.54822899900006],[11.521549500000049,43.58106599900003],[11.483942,43.626108],[11.596508500000027,43.68594550000006],[11.629579,43.84594549700006],[11.710189,43.877430497000034],[11.986528500000077,43.76191399700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI19","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Siena","NUTS_NAME":"Siena"},"geometry":{"type":"Polygon","coordinates":[[[11.96126350000003,43.16689049800004],[11.920441500000038,43.13165049700007],[11.936083500000052,43.055908498000065],[11.982233500000063,43.04885999700008],[11.952102500000024,42.90097049600007],[11.958612500000072,42.87430749600003],[11.933317,42.86932999800007],[11.89498750000007,42.83465349800008],[11.746037,42.78574249800005],[11.63178750000003,42.80729350000007],[11.609541500000034,42.95156849700004],[11.350415500000054,42.97199649600003],[11.361079,43.08321499700003],[11.08243999900003,43.084028],[11.056917500000054,43.17510899800004],[10.911575500000026,43.16261049800005],[10.99658850000003,43.24013599700004],[10.952097500000036,43.30203249700003],[11.012687500000027,43.37656549800005],[10.952091,43.45229649800007],[11.034008,43.54665849700007],[11.229665,43.48967049700008],[11.397507500000074,43.54822899900006],[11.720545,43.25479899800007],[11.96126350000003,43.16689049800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI1A","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Grosseto","NUTS_NAME":"Grosseto"},"geometry":{"type":"Polygon","coordinates":[[[11.350415500000054,42.97199649600003],[11.609541500000034,42.95156849700004],[11.63178750000003,42.80729350000007],[11.746037,42.78574249800005],[11.818677499000046,42.74575549700006],[11.804794500000071,42.64398299700008],[11.578577,42.566193498000075],[11.561255,42.51727549700007],[11.617744500000072,42.43606149800007],[11.488907,42.43935399900005],[11.44993850000003,42.37767050000008],[11.09798150000006,42.39314949800007],[11.104887500000075,42.44757350000003],[11.183892,42.463179998000044],[11.17657950000006,42.541543999000055],[10.949364499000069,42.738098998000055],[10.73105350000003,42.80122749800006],[10.786732500000028,42.888363998000045],[10.705672500000048,42.941864499000076],[10.796260500000074,43.01764499800004],[10.717287500000054,43.10850799800005],[10.911575500000026,43.16261049800005],[11.056917500000054,43.17510899800004],[11.08243999900003,43.084028],[11.361079,43.08321499700003],[11.350415500000054,42.97199649600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI2","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Umbria","NUTS_NAME":"Umbria"},"geometry":{"type":"Polygon","coordinates":[[[12.480548,43.52743699600006],[12.616341500000033,43.42969049700008],[12.767457,43.45982999600005],[12.749178,43.37848949900007],[12.862358,43.21123249900006],[12.895834500000035,42.964570998000056],[13.160879500000021,42.83187449700006],[13.235287500000027,42.86766299800007],[13.264171,42.808744999000055],[13.189226,42.733565498000075],[13.144793,42.64605749800006],[12.896352,42.61641799800003],[12.892744,42.56347599600008],[12.570011499000032,42.382892500000025],[12.444709,42.39879949800007],[12.467339999000046,42.42037899800005],[12.413013,42.42688299800005],[12.412598,42.478194],[12.275523,42.515978500000074],[12.204354500000022,42.66044299600003],[11.941077,42.683648997000034],[11.979501,42.76456799700003],[11.89498750000007,42.83465349800008],[11.933317,42.86932999800007],[11.958612500000072,42.87430749600003],[11.952102500000024,42.90097049600007],[11.982233500000063,43.04885999700008],[11.936083500000052,43.055908498000065],[11.920441500000038,43.13165049700007],[11.96126350000003,43.16689049800004],[12.027506,43.19159999900006],[12.033978,43.246685],[12.223722500000065,43.29624549700003],[12.131859,43.29395849800005],[12.11878450000006,43.364341498000044],[12.025227,43.415556998000056],[12.141248500000074,43.46891149800007],[12.095412,43.522593],[12.213856,43.610824499000046],[12.371290500000043,43.611140498000054],[12.31281,43.54335999800003],[12.480548,43.52743699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI21","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Perugia","NUTS_NAME":"Perugia"},"geometry":{"type":"Polygon","coordinates":[[[12.480548,43.52743699600006],[12.616341500000033,43.42969049700008],[12.767457,43.45982999600005],[12.749178,43.37848949900007],[12.862358,43.21123249900006],[12.895834500000035,42.964570998000056],[13.160879500000021,42.83187449700006],[13.235287500000027,42.86766299800007],[13.264171,42.808744999000055],[13.189226,42.733565498000075],[13.144793,42.64605749800006],[12.896352,42.61641799800003],[12.826491,42.67003450000004],[12.712109499000064,42.622338497000044],[12.584599,42.749106496000024],[12.385974500000032,42.673279498000056],[12.299247500000035,42.795006501000046],[12.330569,42.88961899600008],[12.12805,42.94307300100007],[12.032135,42.92824699600004],[12.009011500000042,42.89074750000003],[12.03011250000003,42.88130300100005],[11.933317,42.86932999800007],[11.958612500000072,42.87430749600003],[11.952102500000024,42.90097049600007],[11.982233500000063,43.04885999700008],[11.936083500000052,43.055908498000065],[11.920441500000038,43.13165049700007],[11.96126350000003,43.16689049800004],[12.027506,43.19159999900006],[12.033978,43.246685],[12.223722500000065,43.29624549700003],[12.131859,43.29395849800005],[12.11878450000006,43.364341498000044],[12.025227,43.415556998000056],[12.141248500000074,43.46891149800007],[12.095412,43.522593],[12.213856,43.610824499000046],[12.371290500000043,43.611140498000054],[12.31281,43.54335999800003],[12.480548,43.52743699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI22","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Terni","NUTS_NAME":"Terni"},"geometry":{"type":"Polygon","coordinates":[[[12.299247500000035,42.795006501000046],[12.385974500000032,42.673279498000056],[12.584599,42.749106496000024],[12.712109499000064,42.622338497000044],[12.826491,42.67003450000004],[12.896352,42.61641799800003],[12.892744,42.56347599600008],[12.570011499000032,42.382892500000025],[12.444709,42.39879949800007],[12.467339999000046,42.42037899800005],[12.413013,42.42688299800005],[12.412598,42.478194],[12.275523,42.515978500000074],[12.204354500000022,42.66044299600003],[11.941077,42.683648997000034],[11.979501,42.76456799700003],[11.89498750000007,42.83465349800008],[11.933317,42.86932999800007],[12.03011250000003,42.88130300100005],[12.009011500000042,42.89074750000003],[12.032135,42.92824699600004],[12.12805,42.94307300100007],[12.330569,42.88961899600008],[12.299247500000035,42.795006501000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI3","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Marche","NUTS_NAME":"Marche"},"geometry":{"type":"Polygon","coordinates":[[[12.75078,43.97060150000004],[13.172615,43.75034599800006],[13.530893500000047,43.619984498000065],[13.642096500000036,43.47414249800005],[13.742970499000023,43.29414699800003],[13.849453,43.066659497000046],[13.91573150000005,42.89457699900004],[13.534506,42.81658549800005],[13.48879,42.73365799700008],[13.357774500000062,42.69406999700004],[13.189226,42.733565498000075],[13.264171,42.808744999000055],[13.235287500000027,42.86766299800007],[13.160879500000021,42.83187449700006],[12.895834500000035,42.964570998000056],[12.862358,43.21123249900006],[12.749178,43.37848949900007],[12.767457,43.45982999600005],[12.616341500000033,43.42969049700008],[12.480548,43.52743699600006],[12.31281,43.54335999800003],[12.371290500000043,43.611140498000054],[12.213856,43.610824499000046],[12.18552550000004,43.64181099700005],[12.371335500000043,43.70900450000005],[12.283797500000048,43.764898997000046],[12.4177,43.89902999600008],[12.493956,43.91554650000006],[12.669299,43.82322749800005],[12.733553500000028,43.869933497000034],[12.75078,43.97060150000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT005","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Panevėžio apskritis","NUTS_NAME":"Panevėžio apskritis"},"geometry":{"type":"Polygon","coordinates":[[[25.821386500000074,56.052178999000034],[26.046154500000057,55.944105998000055],[25.68623,55.71492950000004],[24.722873,55.72524549700006],[24.726243,55.60910499700003],[24.655061500000045,55.51768749800004],[24.650563,55.46156499800003],[24.42978850000003,55.39088649800004],[24.038382,55.44357749900007],[23.879444,55.55816249600008],[24.067673,55.69586299700006],[23.95852750000006,55.831165499000065],[24.08519,55.87642099700008],[24.095391,55.94271949900008],[24.013254500000073,55.961159999000074],[24.179826,56.12818999700005],[24.123573500000077,56.236111996000034],[24.1518,56.25334850000007],[24.579554,56.288315],[24.702389,56.40388899800007],[24.91819,56.44086499900004],[25.09203750000006,56.186041997000075],[25.672947,56.14948199600008],[25.687702,56.086562997000044],[25.821386500000074,56.052178999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT006","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Šiaulių apskritis","NUTS_NAME":"Šiaulių apskritis"},"geometry":{"type":"Polygon","coordinates":[[[24.1518,56.25334850000007],[24.123573500000077,56.236111996000034],[24.179826,56.12818999700005],[24.013254500000073,55.961159999000074],[24.095391,55.94271949900008],[24.08519,55.87642099700008],[23.95852750000006,55.831165499000065],[24.067673,55.69586299700006],[23.879444,55.55816249600008],[23.571644500000048,55.47304199700005],[23.097397,55.564629],[23.080698,55.503066999000055],[22.675759,55.471065499000076],[22.466301500000043,55.67294849800004],[22.52960550000006,55.699158999000076],[22.48619,55.788684997000075],[22.68907050000007,55.89285649900006],[22.584328,55.979530999000076],[22.621349,56.042632999000034],[22.738152,56.06932749900005],[22.53031,56.15307949600003],[22.646966,56.22238549800005],[22.567716499000028,56.36153399800003],[22.635563500000046,56.368167999000036],[22.920517,56.39914299800006],[23.045231,56.32003399700005],[23.765474,56.373081],[24.1518,56.25334850000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT007","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Tauragės apskritis","NUTS_NAME":"Tauragės apskritis"},"geometry":{"type":"Polygon","coordinates":[[[22.675759,55.471065499000076],[22.81870250000003,55.32182899700007],[22.98098550000003,55.31412149600004],[22.996637500000077,55.25330549700004],[23.504275,55.193331999000065],[23.41193750000008,55.073508996000044],[22.588968,55.07025149700007],[22.066402912000058,55.027111999000056],[21.882659155000056,55.09474270700008],[21.65107099900007,55.17998350000005],[21.86641050000003,55.20315049800007],[21.854010500000072,55.277349997000044],[22.020298,55.38490349800003],[21.871759,55.42450899900007],[21.808149,55.54883549700003],[21.879833500000075,55.59701799900006],[22.466301500000043,55.67294849800004],[22.675759,55.471065499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT008","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Telšių apskritis","NUTS_NAME":"Telšių apskritis"},"geometry":{"type":"Polygon","coordinates":[[[22.635563500000046,56.368167999000036],[22.567716499000028,56.36153399800003],[22.646966,56.22238549800005],[22.53031,56.15307949600003],[22.738152,56.06932749900005],[22.621349,56.042632999000034],[22.584328,55.979530999000076],[22.68907050000007,55.89285649900006],[22.48619,55.788684997000075],[22.52960550000006,55.699158999000076],[22.466301500000043,55.67294849800004],[21.879833500000075,55.59701799900006],[21.729068,55.74981799700004],[21.512083500000074,55.80882749700004],[21.70398,56.07721],[22.01329450000003,56.19609199800004],[21.932097500000054,56.318853500000046],[21.978209,56.38514299800005],[22.635563500000046,56.368167999000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT009","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Utenos apskritis","NUTS_NAME":"Utenos apskritis"},"geometry":{"type":"Polygon","coordinates":[[[26.630365,55.68066699800005],[26.623579500000062,55.565641],[26.524342,55.51005750000007],[26.553886,55.388916],[26.43989349900005,55.34859449900006],[26.835622500000056,55.28564849700007],[26.743454,55.254102997000075],[26.341755,55.21877049700004],[25.765000499000053,55.31585749700008],[25.648718,55.19654249800004],[25.794025500000032,55.12037699800004],[25.760759,55.03965349900005],[25.22202850000008,55.04150499700006],[25.147675500000048,55.095690500000046],[25.24168,55.19730149800006],[24.967869500000063,55.35659049800006],[24.788612,55.37246599900004],[24.819741,55.43822249800007],[24.655061500000045,55.51768749800004],[24.726243,55.60910499700003],[24.722873,55.72524549700006],[25.68623,55.71492950000004],[26.046154500000057,55.944105998000055],[26.385134,55.70467849600004],[26.630365,55.68066699800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT00A","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Vilniaus apskritis","NUTS_NAME":"Vilniaus apskritis"},"geometry":{"type":"Polygon","coordinates":[[[26.743454,55.254102997000075],[26.67651550000005,55.158772999000064],[26.323387500000024,55.14494149600006],[26.229339,55.10559249800008],[26.271291500000075,55.07756399600004],[26.208658,54.99742649800004],[25.779577500000073,54.85483950000008],[25.733722500000056,54.80223499700003],[25.762614,54.57689449900005],[25.531922500000064,54.34203149800004],[25.685688500000026,54.318358997000075],[25.774569,54.23725899900006],[25.779838,54.160037],[25.530006,54.15841049800008],[25.59022250000004,54.23255849800006],[25.49710950000008,54.309776498000076],[25.214800500000024,54.25603099600005],[25.07084750000007,54.13387849800006],[24.835949500000027,54.149027498000066],[24.756055,54.22396049900004],[24.933917500000064,54.294920998000066],[24.91258650000003,54.361139],[25.01897450000007,54.441532],[24.438120500000025,54.42777749900006],[24.414311,54.53326899800004],[24.503864500000077,54.729119998000044],[24.619247,54.829968],[24.761334500000032,54.82327699900003],[24.811022500000036,54.90341749700008],[24.55363150000005,54.98992449800005],[24.646419,55.04964999800006],[24.391269500000078,55.29687849900006],[24.42978850000003,55.39088649800004],[24.650563,55.46156499800003],[24.655061500000045,55.51768749800004],[24.819741,55.43822249800007],[24.788612,55.37246599900004],[24.967869500000063,55.35659049800006],[25.24168,55.19730149800006],[25.147675500000048,55.095690500000046],[25.22202850000008,55.04150499700006],[25.760759,55.03965349900005],[25.794025500000032,55.12037699800004],[25.648718,55.19654249800004],[25.765000499000053,55.31585749700008],[26.341755,55.21877049700004],[26.743454,55.254102997000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"LU","LEVL_CODE":0,"CNTR_CODE":"LU","NAME_LATN":"LUXEMBOURG","NUTS_NAME":"LUXEMBOURG"},"geometry":{"type":"Polygon","coordinates":[[[6.137662500000033,50.129951498000025],[6.116835500000036,50.05671099800003],[6.235510500000032,49.89918849900005],[6.474962500000061,49.821274998000035],[6.520996,49.79698549700004],[6.507018500000072,49.71264050000008],[6.426077500000076,49.663689],[6.380052500000033,49.55110499800003],[6.367107500000031,49.469507],[5.893386,49.49694449700007],[5.818117,49.54631049900007],[5.910688,49.662388],[5.746319,49.85359499900005],[5.895269,50.112216498000066],[6.02489950000006,50.18277949700007],[6.137662500000033,50.129951498000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"LU0","LEVL_CODE":1,"CNTR_CODE":"LU","NAME_LATN":"LUXEMBOURG","NUTS_NAME":"LUXEMBOURG"},"geometry":{"type":"Polygon","coordinates":[[[6.137662500000033,50.129951498000025],[6.116835500000036,50.05671099800003],[6.235510500000032,49.89918849900005],[6.474962500000061,49.821274998000035],[6.520996,49.79698549700004],[6.507018500000072,49.71264050000008],[6.426077500000076,49.663689],[6.380052500000033,49.55110499800003],[6.367107500000031,49.469507],[5.893386,49.49694449700007],[5.818117,49.54631049900007],[5.910688,49.662388],[5.746319,49.85359499900005],[5.895269,50.112216498000066],[6.02489950000006,50.18277949700007],[6.137662500000033,50.129951498000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"LU00","LEVL_CODE":2,"CNTR_CODE":"LU","NAME_LATN":"Luxembourg","NUTS_NAME":"Luxembourg"},"geometry":{"type":"Polygon","coordinates":[[[6.137662500000033,50.129951498000025],[6.116835500000036,50.05671099800003],[6.235510500000032,49.89918849900005],[6.474962500000061,49.821274998000035],[6.520996,49.79698549700004],[6.507018500000072,49.71264050000008],[6.426077500000076,49.663689],[6.380052500000033,49.55110499800003],[6.367107500000031,49.469507],[5.893386,49.49694449700007],[5.818117,49.54631049900007],[5.910688,49.662388],[5.746319,49.85359499900005],[5.895269,50.112216498000066],[6.02489950000006,50.18277949700007],[6.137662500000033,50.129951498000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"LU000","LEVL_CODE":3,"CNTR_CODE":"LU","NAME_LATN":"Luxembourg","NUTS_NAME":"Luxembourg"},"geometry":{"type":"Polygon","coordinates":[[[6.137662500000033,50.129951498000025],[6.116835500000036,50.05671099800003],[6.235510500000032,49.89918849900005],[6.474962500000061,49.821274998000035],[6.520996,49.79698549700004],[6.507018500000072,49.71264050000008],[6.426077500000076,49.663689],[6.380052500000033,49.55110499800003],[6.367107500000031,49.469507],[5.893386,49.49694449700007],[5.818117,49.54631049900007],[5.910688,49.662388],[5.746319,49.85359499900005],[5.895269,50.112216498000066],[6.02489950000006,50.18277949700007],[6.137662500000033,50.129951498000025]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV","LEVL_CODE":0,"CNTR_CODE":"LV","NAME_LATN":"LATVIJA","NUTS_NAME":"LATVIJA"},"geometry":{"type":"Polygon","coordinates":[[[27.351579,57.51823699800008],[27.556349500000067,57.53777450000007],[27.517335,57.42696099900007],[27.690777,57.37056099800003],[27.870156500000064,57.28979499800005],[27.818273500000032,57.143264498000065],[27.715167500000064,57.09765049600003],[27.714762500000063,57.00744099700006],[27.76852,57.00250499600003],[27.659444500000063,56.83436549900006],[27.847114,56.880228997000074],[27.975965,56.824560500000075],[27.898484499000062,56.75237499900004],[28.043213500000036,56.590376],[28.141912,56.57305549900008],[28.094682,56.516884996000044],[28.192765,56.44856599800005],[28.163174,56.37959449900006],[28.241536,56.27906799900006],[28.154555500000072,56.16984350000007],[27.65308950000008,55.929588497000054],[27.61550550000004,55.78707150000008],[27.13303650000006,55.85159550000003],[26.783514500000024,55.67811499900006],[26.630365,55.68066699800005],[26.385134,55.70467849600004],[26.046154500000057,55.944105998000055],[25.821386500000074,56.052178999000034],[25.687702,56.086562997000044],[25.672947,56.14948199600008],[25.09203750000006,56.186041997000075],[24.91819,56.44086499900004],[24.702389,56.40388899800007],[24.579554,56.288315],[24.1518,56.25334850000007],[23.765474,56.373081],[23.045231,56.32003399700005],[22.920517,56.39914299800006],[22.635563500000046,56.368167999000036],[21.978209,56.38514299800005],[21.579046,56.31965999700003],[21.230745,56.16227299700006],[21.199444,56.07678999700005],[21.064238,56.06913699800003],[20.984757500000057,56.21285799800006],[20.970678500000076,56.35258449600008],[21.05211,56.82362099800008],[21.374895500000036,57.00309699700006],[21.417394,57.080737499000065],[21.419337,57.29095249800008],[21.70315050000005,57.568530499000076],[22.605003500000066,57.75857399800003],[22.577911,57.71556499800005],[22.61870250000004,57.599504499000034],[23.14689050000004,57.31625799900007],[23.304514,57.06428949900004],[23.624208500000066,56.96450149900005],[23.934532500000046,57.006370496000045],[24.11920250000003,57.08629],[24.410199500000033,57.26602550000007],[24.35281750000007,57.87655650000005],[24.834082500000022,57.97277949900007],[25.046307,58.040146],[25.212184500000035,58.080787497000074],[25.302209,58.00057849900003],[25.264865,58.06008599900008],[25.295673,58.08144949800004],[25.67896950000005,57.90439199600007],[26.056459,57.848431496000046],[26.024105500000076,57.76862199900006],[26.524905500000045,57.51624749700005],[26.92589950000007,57.633293500000036],[27.351579,57.51823699800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV0","LEVL_CODE":1,"CNTR_CODE":"LV","NAME_LATN":"LATVIJA","NUTS_NAME":"LATVIJA"},"geometry":{"type":"Polygon","coordinates":[[[25.295673,58.08144949800004],[25.67896950000005,57.90439199600007],[26.056459,57.848431496000046],[26.024105500000076,57.76862199900006],[26.524905500000045,57.51624749700005],[26.92589950000007,57.633293500000036],[27.351579,57.51823699800008],[27.556349500000067,57.53777450000007],[27.517335,57.42696099900007],[27.690777,57.37056099800003],[27.870156500000064,57.28979499800005],[27.818273500000032,57.143264498000065],[27.715167500000064,57.09765049600003],[27.714762500000063,57.00744099700006],[27.76852,57.00250499600003],[27.659444500000063,56.83436549900006],[27.847114,56.880228997000074],[27.975965,56.824560500000075],[27.898484499000062,56.75237499900004],[28.043213500000036,56.590376],[28.141912,56.57305549900008],[28.094682,56.516884996000044],[28.192765,56.44856599800005],[28.163174,56.37959449900006],[28.241536,56.27906799900006],[28.154555500000072,56.16984350000007],[27.65308950000008,55.929588497000054],[27.61550550000004,55.78707150000008],[27.13303650000006,55.85159550000003],[26.783514500000024,55.67811499900006],[26.630365,55.68066699800005],[26.385134,55.70467849600004],[26.046154500000057,55.944105998000055],[25.821386500000074,56.052178999000034],[25.687702,56.086562997000044],[25.672947,56.14948199600008],[25.09203750000006,56.186041997000075],[24.91819,56.44086499900004],[24.702389,56.40388899800007],[24.579554,56.288315],[24.1518,56.25334850000007],[23.765474,56.373081],[23.045231,56.32003399700005],[22.920517,56.39914299800006],[22.635563500000046,56.368167999000036],[21.978209,56.38514299800005],[21.579046,56.31965999700003],[21.230745,56.16227299700006],[21.199444,56.07678999700005],[21.064238,56.06913699800003],[20.984757500000057,56.21285799800006],[20.970678500000076,56.35258449600008],[21.05211,56.82362099800008],[21.374895500000036,57.00309699700006],[21.417394,57.080737499000065],[21.419337,57.29095249800008],[21.70315050000005,57.568530499000076],[22.605003500000066,57.75857399800003],[22.577911,57.71556499800005],[22.61870250000004,57.599504499000034],[23.14689050000004,57.31625799900007],[23.304514,57.06428949900004],[23.624208500000066,56.96450149900005],[23.934532500000046,57.006370496000045],[24.11920250000003,57.08629],[24.410199500000033,57.26602550000007],[24.35281750000007,57.87655650000005],[24.834082500000022,57.97277949900007],[25.046307,58.040146],[25.212184500000035,58.080787497000074],[25.302209,58.00057849900003],[25.264865,58.06008599900008],[25.295673,58.08144949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV00","LEVL_CODE":2,"CNTR_CODE":"LV","NAME_LATN":"Latvija","NUTS_NAME":"Latvija"},"geometry":{"type":"Polygon","coordinates":[[[25.295673,58.08144949800004],[25.67896950000005,57.90439199600007],[26.056459,57.848431496000046],[26.024105500000076,57.76862199900006],[26.524905500000045,57.51624749700005],[26.92589950000007,57.633293500000036],[27.351579,57.51823699800008],[27.556349500000067,57.53777450000007],[27.517335,57.42696099900007],[27.690777,57.37056099800003],[27.870156500000064,57.28979499800005],[27.818273500000032,57.143264498000065],[27.715167500000064,57.09765049600003],[27.714762500000063,57.00744099700006],[27.76852,57.00250499600003],[27.659444500000063,56.83436549900006],[27.847114,56.880228997000074],[27.975965,56.824560500000075],[27.898484499000062,56.75237499900004],[28.043213500000036,56.590376],[28.141912,56.57305549900008],[28.094682,56.516884996000044],[28.192765,56.44856599800005],[28.163174,56.37959449900006],[28.241536,56.27906799900006],[28.154555500000072,56.16984350000007],[27.65308950000008,55.929588497000054],[27.61550550000004,55.78707150000008],[27.13303650000006,55.85159550000003],[26.783514500000024,55.67811499900006],[26.630365,55.68066699800005],[26.385134,55.70467849600004],[26.046154500000057,55.944105998000055],[25.821386500000074,56.052178999000034],[25.687702,56.086562997000044],[25.672947,56.14948199600008],[25.09203750000006,56.186041997000075],[24.91819,56.44086499900004],[24.702389,56.40388899800007],[24.579554,56.288315],[24.1518,56.25334850000007],[23.765474,56.373081],[23.045231,56.32003399700005],[22.920517,56.39914299800006],[22.635563500000046,56.368167999000036],[21.978209,56.38514299800005],[21.579046,56.31965999700003],[21.230745,56.16227299700006],[21.199444,56.07678999700005],[21.064238,56.06913699800003],[20.984757500000057,56.21285799800006],[20.970678500000076,56.35258449600008],[21.05211,56.82362099800008],[21.374895500000036,57.00309699700006],[21.417394,57.080737499000065],[21.419337,57.29095249800008],[21.70315050000005,57.568530499000076],[22.605003500000066,57.75857399800003],[22.577911,57.71556499800005],[22.61870250000004,57.599504499000034],[23.14689050000004,57.31625799900007],[23.304514,57.06428949900004],[23.624208500000066,56.96450149900005],[23.934532500000046,57.006370496000045],[24.11920250000003,57.08629],[24.410199500000033,57.26602550000007],[24.35281750000007,57.87655650000005],[24.834082500000022,57.97277949900007],[25.046307,58.040146],[25.212184500000035,58.080787497000074],[25.302209,58.00057849900003],[25.264865,58.06008599900008],[25.295673,58.08144949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV003","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Kurzeme","NUTS_NAME":"Kurzeme"},"geometry":{"type":"Polygon","coordinates":[[[23.14689050000004,57.31625799900007],[22.928004499000053,57.130407999000056],[22.639120500000047,57.096998],[22.642755,57.016615],[22.504019,57.01854349800004],[22.443349500000068,56.93881049700008],[22.538387,56.91627199900006],[22.494176,56.85434249600007],[22.528519,56.84001699800007],[22.78411650000004,56.799257999000076],[22.82558,56.77188749900006],[22.820532,56.693411498000046],[22.788968,56.68217899900003],[22.845142500000065,56.60933150000005],[22.736828,56.566719997000064],[22.729448,56.423599996000064],[22.920517,56.39914299800006],[22.635563500000046,56.368167999000036],[21.978209,56.38514299800005],[21.579046,56.31965999700003],[21.230745,56.16227299700006],[21.199444,56.07678999700005],[21.064238,56.06913699800003],[20.984757500000057,56.21285799800006],[20.970678500000076,56.35258449600008],[21.05211,56.82362099800008],[21.374895500000036,57.00309699700006],[21.417394,57.080737499000065],[21.419337,57.29095249800008],[21.70315050000005,57.568530499000076],[22.605003500000066,57.75857399800003],[22.577911,57.71556499800005],[22.61870250000004,57.599504499000034],[23.14689050000004,57.31625799900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV005","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Latgale","NUTS_NAME":"Latgale"},"geometry":{"type":"Polygon","coordinates":[[[28.154555500000072,56.16984350000007],[27.65308950000008,55.929588497000054],[27.61550550000004,55.78707150000008],[27.13303650000006,55.85159550000003],[26.783514500000024,55.67811499900006],[26.630365,55.68066699800005],[26.385134,55.70467849600004],[26.046154500000057,55.944105998000055],[25.821386500000074,56.052178999000034],[25.878224,56.08232499700006],[25.837029,56.11950050000007],[26.04630650000007,56.06678199700008],[26.214871,56.17227799900007],[26.333445,56.096426500000064],[26.082442,56.408561500000076],[26.263796,56.52175149900006],[26.365101500000037,56.451981],[26.624916,56.540223999000034],[26.713823500000046,56.545844],[26.777853,56.713013997000076],[26.991629500000045,56.86741599800007],[26.866536,56.97398549800005],[26.87408,57.02927599700007],[27.163065,57.227108498000064],[27.54735850000003,57.26502350000004],[27.690777,57.37056099800003],[27.870156500000064,57.28979499800005],[27.818273500000032,57.143264498000065],[27.715167500000064,57.09765049600003],[27.714762500000063,57.00744099700006],[27.76852,57.00250499600003],[27.659444500000063,56.83436549900006],[27.847114,56.880228997000074],[27.975965,56.824560500000075],[27.898484499000062,56.75237499900004],[28.043213500000036,56.590376],[28.141912,56.57305549900008],[28.094682,56.516884996000044],[28.192765,56.44856599800005],[28.163174,56.37959449900006],[28.241536,56.27906799900006],[28.154555500000072,56.16984350000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV006","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Riga","NUTS_NAME":"Riga"},"geometry":{"type":"Polygon","coordinates":[[[23.934532500000046,57.006370496000045],[24.11920250000003,57.08629],[24.324504,56.96837449800006],[24.227986,56.89658249900003],[24.310566,56.859312497000076],[23.945543500000042,56.94407749800007],[23.934532500000046,57.006370496000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV007","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Pieriga","NUTS_NAME":"Pieriga"},"geometry":{"type":"Polygon","coordinates":[[[25.50603250000006,56.81515599900007],[25.245489,56.73184199800005],[25.2823,56.695318],[25.029814500000043,56.726446999000075],[25.048745,56.59221899900007],[24.665825,56.54079049600006],[24.700243500000056,56.58094250000005],[24.57903650000003,56.67302699800007],[24.634435500000052,56.714098498000055],[24.153795500000058,56.66502599900008],[23.747771,56.740853497000046],[23.773385,56.80166349600006],[23.597190500000067,56.89371449600003],[23.467407500000036,56.884927497000035],[23.489361,56.813796497000055],[23.27533950000003,56.70726399800003],[22.820532,56.693411498000046],[22.82558,56.77188749900006],[22.78411650000004,56.799257999000076],[22.528519,56.84001699800007],[22.494176,56.85434249600007],[22.538387,56.91627199900006],[22.443349500000068,56.93881049700008],[22.504019,57.01854349800004],[22.642755,57.016615],[22.639120500000047,57.096998],[22.928004499000053,57.130407999000056],[23.14689050000004,57.31625799900007],[23.304514,57.06428949900004],[23.624208500000066,56.96450149900005],[23.934532500000046,57.006370496000045],[23.945543500000042,56.94407749800007],[24.310566,56.859312497000076],[24.227986,56.89658249900003],[24.324504,56.96837449800006],[24.11920250000003,57.08629],[24.410199500000033,57.26602550000007],[24.35281750000007,57.87655650000005],[24.834082500000022,57.97277949900007],[24.899050500000044,57.82610149900006],[25.073248,57.774223997000036],[25.099370500000077,57.68741699800006],[25.088715,57.62881199700007],[24.89455950100006,57.57284999900003],[25.090708500000062,57.50843699900008],[24.78870150000006,57.41232399800003],[24.952239,57.27826549900004],[24.924553,57.23661249600008],[24.96920750000004,57.14395999800007],[25.120859,57.095082999000056],[25.148207,56.94734599700007],[25.508479,56.98380999600005],[25.539148,56.931095999000036],[25.50603250000006,56.81515599900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV008","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Vidzeme","NUTS_NAME":"Vidzeme"},"geometry":{"type":"Polygon","coordinates":[[[27.351579,57.51823699800008],[27.556349500000067,57.53777450000007],[27.517335,57.42696099900007],[27.690777,57.37056099800003],[27.54735850000003,57.26502350000004],[27.163065,57.227108498000064],[26.87408,57.02927599700007],[26.866536,56.97398549800005],[26.991629500000045,56.86741599800007],[26.777853,56.713013997000076],[26.713823500000046,56.545844],[26.624916,56.540223999000034],[26.390899500000046,56.640967999000054],[26.2832,56.58526099800008],[26.135739499000067,56.65732999900007],[26.026318,56.618349],[25.836148,56.70954749800006],[25.850295,56.77060499700008],[25.766068500000074,56.82904049800004],[25.69398,56.769340498000076],[25.50603250000006,56.81515599900007],[25.539148,56.931095999000036],[25.508479,56.98380999600005],[25.148207,56.94734599700007],[25.120859,57.095082999000056],[24.96920750000004,57.14395999800007],[24.924553,57.23661249600008],[24.952239,57.27826549900004],[24.78870150000006,57.41232399800003],[25.090708500000062,57.50843699900008],[24.89455950100006,57.57284999900003],[25.088715,57.62881199700007],[25.099370500000077,57.68741699800006],[25.073248,57.774223997000036],[24.899050500000044,57.82610149900006],[24.834082500000022,57.97277949900007],[25.046307,58.040146],[25.212184500000035,58.080787497000074],[25.302209,58.00057849900003],[25.264865,58.06008599900008],[25.295673,58.08144949800004],[25.67896950000005,57.90439199600007],[26.056459,57.848431496000046],[26.024105500000076,57.76862199900006],[26.524905500000045,57.51624749700005],[26.92589950000007,57.633293500000036],[27.351579,57.51823699800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"LV009","LEVL_CODE":3,"CNTR_CODE":"LV","NAME_LATN":"Zemgale","NUTS_NAME":"Zemgale"},"geometry":{"type":"Polygon","coordinates":[[[25.50603250000006,56.81515599900007],[25.69398,56.769340498000076],[25.766068500000074,56.82904049800004],[25.850295,56.77060499700008],[25.836148,56.70954749800006],[26.026318,56.618349],[26.135739499000067,56.65732999900007],[26.2832,56.58526099800008],[26.390899500000046,56.640967999000054],[26.624916,56.540223999000034],[26.365101500000037,56.451981],[26.263796,56.52175149900006],[26.082442,56.408561500000076],[26.333445,56.096426500000064],[26.214871,56.17227799900007],[26.04630650000007,56.06678199700008],[25.837029,56.11950050000007],[25.878224,56.08232499700006],[25.821386500000074,56.052178999000034],[25.687702,56.086562997000044],[25.672947,56.14948199600008],[25.09203750000006,56.186041997000075],[24.91819,56.44086499900004],[24.702389,56.40388899800007],[24.579554,56.288315],[24.1518,56.25334850000007],[23.765474,56.373081],[23.045231,56.32003399700005],[22.920517,56.39914299800006],[22.729448,56.423599996000064],[22.736828,56.566719997000064],[22.845142500000065,56.60933150000005],[22.788968,56.68217899900003],[22.820532,56.693411498000046],[23.27533950000003,56.70726399800003],[23.489361,56.813796497000055],[23.467407500000036,56.884927497000035],[23.597190500000067,56.89371449600003],[23.773385,56.80166349600006],[23.747771,56.740853497000046],[24.153795500000058,56.66502599900008],[24.634435500000052,56.714098498000055],[24.57903650000003,56.67302699800007],[24.700243500000056,56.58094250000005],[24.665825,56.54079049600006],[25.048745,56.59221899900007],[25.029814500000043,56.726446999000075],[25.2823,56.695318],[25.245489,56.73184199800005],[25.50603250000006,56.81515599900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ME","LEVL_CODE":0,"CNTR_CODE":"ME","NAME_LATN":"CRNA GORA","NUTS_NAME":"ЦРНА ГОРА"},"geometry":{"type":"Polygon","coordinates":[[[19.224069,43.52754099900005],[19.227058,43.47903949700003],[19.448891,43.387774498000056],[19.544844500000067,43.24756099800004],[20.063936500000068,43.00682399800007],[20.34477350000003,42.91084999800006],[20.352928500000075,42.83338149800005],[20.254951,42.814374497000074],[20.23432249900003,42.75274699900007],[20.024653,42.765138498000056],[20.03323050000006,42.697433498000066],[20.111944,42.65540249800006],[20.0763,42.55582349800005],[19.82060050000007,42.47109599800007],[19.744089,42.543232001000035],[19.757534,42.63716499600008],[19.704206500000055,42.65668849700006],[19.621872,42.58974449700003],[19.282484,42.18001550100007],[19.402660500000025,42.103740498000036],[19.342348,41.906965],[19.372067,41.85032049700004],[19.150603500000045,41.94886199800004],[19.082872500000065,42.11298749800005],[18.88044350000007,42.278823997000075],[18.77419450000008,42.26862349600003],[18.683983,42.38303399800003],[18.546882500000038,42.41530599600003],[18.707367,42.392784],[18.689521633000027,42.464353483000025],[18.504755,42.45133949700005],[18.525213,42.420462996000026],[18.43380049900003,42.48400849800004],[18.438104,42.555705],[18.56724950000006,42.65140549700004],[18.559433,42.72252649600006],[18.468349500000045,42.79198449800003],[18.49192799900004,42.98396299700005],[18.665016,43.029556498000034],[18.651333,43.15224849800006],[18.708839500000067,43.21791850000005],[18.683405,43.24548699800005],[18.912216,43.36102299600003],[18.998855500000047,43.257667499000036],[19.076616500000057,43.307907],[19.017517,43.430240498000046],[18.909717499000067,43.49679549800004],[18.99369450000006,43.55749899600005],[19.049223,43.501789],[19.224069,43.52754099900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ME0","LEVL_CODE":1,"CNTR_CODE":"ME","NAME_LATN":"CRNA GORA","NUTS_NAME":"ЦРНА ГОРА"},"geometry":{"type":"Polygon","coordinates":[[[19.224069,43.52754099900005],[19.227058,43.47903949700003],[19.448891,43.387774498000056],[19.544844500000067,43.24756099800004],[20.063936500000068,43.00682399800007],[20.34477350000003,42.91084999800006],[20.352928500000075,42.83338149800005],[20.254951,42.814374497000074],[20.23432249900003,42.75274699900007],[20.024653,42.765138498000056],[20.03323050000006,42.697433498000066],[20.111944,42.65540249800006],[20.0763,42.55582349800005],[19.82060050000007,42.47109599800007],[19.744089,42.543232001000035],[19.757534,42.63716499600008],[19.704206500000055,42.65668849700006],[19.621872,42.58974449700003],[19.282484,42.18001550100007],[19.402660500000025,42.103740498000036],[19.342348,41.906965],[19.372067,41.85032049700004],[19.150603500000045,41.94886199800004],[19.082872500000065,42.11298749800005],[18.88044350000007,42.278823997000075],[18.77419450000008,42.26862349600003],[18.683983,42.38303399800003],[18.546882500000038,42.41530599600003],[18.707367,42.392784],[18.689521633000027,42.464353483000025],[18.504755,42.45133949700005],[18.525213,42.420462996000026],[18.43380049900003,42.48400849800004],[18.438104,42.555705],[18.56724950000006,42.65140549700004],[18.559433,42.72252649600006],[18.468349500000045,42.79198449800003],[18.49192799900004,42.98396299700005],[18.665016,43.029556498000034],[18.651333,43.15224849800006],[18.708839500000067,43.21791850000005],[18.683405,43.24548699800005],[18.912216,43.36102299600003],[18.998855500000047,43.257667499000036],[19.076616500000057,43.307907],[19.017517,43.430240498000046],[18.909717499000067,43.49679549800004],[18.99369450000006,43.55749899600005],[19.049223,43.501789],[19.224069,43.52754099900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ME00","LEVL_CODE":2,"CNTR_CODE":"ME","NAME_LATN":"Crna Gora","NUTS_NAME":"Црна Гора"},"geometry":{"type":"Polygon","coordinates":[[[19.224069,43.52754099900005],[19.227058,43.47903949700003],[19.448891,43.387774498000056],[19.544844500000067,43.24756099800004],[20.063936500000068,43.00682399800007],[20.34477350000003,42.91084999800006],[20.352928500000075,42.83338149800005],[20.254951,42.814374497000074],[20.23432249900003,42.75274699900007],[20.024653,42.765138498000056],[20.03323050000006,42.697433498000066],[20.111944,42.65540249800006],[20.0763,42.55582349800005],[19.82060050000007,42.47109599800007],[19.744089,42.543232001000035],[19.757534,42.63716499600008],[19.704206500000055,42.65668849700006],[19.621872,42.58974449700003],[19.282484,42.18001550100007],[19.402660500000025,42.103740498000036],[19.342348,41.906965],[19.372067,41.85032049700004],[19.150603500000045,41.94886199800004],[19.082872500000065,42.11298749800005],[18.88044350000007,42.278823997000075],[18.77419450000008,42.26862349600003],[18.683983,42.38303399800003],[18.546882500000038,42.41530599600003],[18.707367,42.392784],[18.689521633000027,42.464353483000025],[18.504755,42.45133949700005],[18.525213,42.420462996000026],[18.43380049900003,42.48400849800004],[18.438104,42.555705],[18.56724950000006,42.65140549700004],[18.559433,42.72252649600006],[18.468349500000045,42.79198449800003],[18.49192799900004,42.98396299700005],[18.665016,43.029556498000034],[18.651333,43.15224849800006],[18.708839500000067,43.21791850000005],[18.683405,43.24548699800005],[18.912216,43.36102299600003],[18.998855500000047,43.257667499000036],[19.076616500000057,43.307907],[19.017517,43.430240498000046],[18.909717499000067,43.49679549800004],[18.99369450000006,43.55749899600005],[19.049223,43.501789],[19.224069,43.52754099900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ME000","LEVL_CODE":3,"CNTR_CODE":"ME","NAME_LATN":"Crna Gora","NUTS_NAME":"Црна Гора"},"geometry":{"type":"Polygon","coordinates":[[[19.224069,43.52754099900005],[19.227058,43.47903949700003],[19.448891,43.387774498000056],[19.544844500000067,43.24756099800004],[20.063936500000068,43.00682399800007],[20.34477350000003,42.91084999800006],[20.352928500000075,42.83338149800005],[20.254951,42.814374497000074],[20.23432249900003,42.75274699900007],[20.024653,42.765138498000056],[20.03323050000006,42.697433498000066],[20.111944,42.65540249800006],[20.0763,42.55582349800005],[19.82060050000007,42.47109599800007],[19.744089,42.543232001000035],[19.757534,42.63716499600008],[19.704206500000055,42.65668849700006],[19.621872,42.58974449700003],[19.282484,42.18001550100007],[19.402660500000025,42.103740498000036],[19.342348,41.906965],[19.372067,41.85032049700004],[19.150603500000045,41.94886199800004],[19.082872500000065,42.11298749800005],[18.88044350000007,42.278823997000075],[18.77419450000008,42.26862349600003],[18.683983,42.38303399800003],[18.546882500000038,42.41530599600003],[18.707367,42.392784],[18.689521633000027,42.464353483000025],[18.504755,42.45133949700005],[18.525213,42.420462996000026],[18.43380049900003,42.48400849800004],[18.438104,42.555705],[18.56724950000006,42.65140549700004],[18.559433,42.72252649600006],[18.468349500000045,42.79198449800003],[18.49192799900004,42.98396299700005],[18.665016,43.029556498000034],[18.651333,43.15224849800006],[18.708839500000067,43.21791850000005],[18.683405,43.24548699800005],[18.912216,43.36102299600003],[18.998855500000047,43.257667499000036],[19.076616500000057,43.307907],[19.017517,43.430240498000046],[18.909717499000067,43.49679549800004],[18.99369450000006,43.55749899600005],[19.049223,43.501789],[19.224069,43.52754099900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK","LEVL_CODE":0,"CNTR_CODE":"MK","NAME_LATN":"PORANEŠNA JUGOSLOVENSKA REPUBLIKA MAKEDONIJA","NUTS_NAME":"ПОРАНЕШНА ЈУГОСЛОВЕНСКА РЕПУБЛИКА МАКЕДОНИЈА"},"geometry":{"type":"Polygon","coordinates":[[[22.36020650000006,42.311157],[22.510412,42.15515849700006],[22.867214,42.02219949600004],[22.90193350000004,41.87764199900005],[23.032124,41.72430099800005],[22.948411500000077,41.63179649800003],[22.968327500000044,41.51983549800008],[22.965055500000062,41.35190749700007],[22.927591500000062,41.338539498000046],[22.879178500000023,41.34065299700006],[22.762478,41.320234],[22.732037,41.146391497000025],[22.66647,41.18633949800005],[22.587711,41.11729599900008],[22.332052,41.120272500000056],[22.216216,41.17045749700003],[21.929438,41.10035049700008],[21.787378,40.931125499000075],[21.684081,40.94091499800004],[21.59323550000005,40.86600249700007],[21.417245500000035,40.917696497000065],[21.301891500000067,40.865341998000076],[20.98020450000007,40.855664999000055],[20.968201500000077,40.91114599700006],[20.837823,40.92768299800008],[20.732268500000032,40.91173249800005],[20.670626,41.08735149800003],[20.595609500000023,41.092411998000046],[20.515577,41.23095550000005],[20.560363,41.40845850100004],[20.456284500000038,41.554024],[20.557774,41.581870501000026],[20.514314,41.73048299700008],[20.594286,41.87732749700007],[20.727972500000078,41.86427699800004],[20.776524,41.91848549700006],[20.748154,42.02570499600006],[20.794234500000073,42.08364549600003],[21.10894,42.20617099700007],[21.212697500000047,42.110701],[21.32123450000006,42.10798899800005],[21.30238950000006,42.14130799900005],[21.385102500000073,42.24647949900003],[21.444318500000065,42.23493049800004],[21.44485750000007,42.27789799800007],[21.586946500000067,42.26281749700007],[22.29804450000006,42.36744499900004],[22.36020650000006,42.311157]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK0","LEVL_CODE":1,"CNTR_CODE":"MK","NAME_LATN":"PORANEŠNA JUGOSLOVENSKA REPUBLIKA MAKEDONIJA","NUTS_NAME":"ПОРАНЕШНА ЈУГОСЛОВЕНСКА РЕПУБЛИКА МАКЕДОНИЈА"},"geometry":{"type":"Polygon","coordinates":[[[22.36020650000006,42.311157],[22.510412,42.15515849700006],[22.867214,42.02219949600004],[22.90193350000004,41.87764199900005],[23.032124,41.72430099800005],[22.948411500000077,41.63179649800003],[22.968327500000044,41.51983549800008],[22.965055500000062,41.35190749700007],[22.927591500000062,41.338539498000046],[22.879178500000023,41.34065299700006],[22.762478,41.320234],[22.732037,41.146391497000025],[22.66647,41.18633949800005],[22.587711,41.11729599900008],[22.332052,41.120272500000056],[22.216216,41.17045749700003],[21.929438,41.10035049700008],[21.787378,40.931125499000075],[21.684081,40.94091499800004],[21.59323550000005,40.86600249700007],[21.417245500000035,40.917696497000065],[21.301891500000067,40.865341998000076],[20.98020450000007,40.855664999000055],[20.968201500000077,40.91114599700006],[20.837823,40.92768299800008],[20.732268500000032,40.91173249800005],[20.670626,41.08735149800003],[20.595609500000023,41.092411998000046],[20.515577,41.23095550000005],[20.560363,41.40845850100004],[20.456284500000038,41.554024],[20.557774,41.581870501000026],[20.514314,41.73048299700008],[20.594286,41.87732749700007],[20.727972500000078,41.86427699800004],[20.776524,41.91848549700006],[20.748154,42.02570499600006],[20.794234500000073,42.08364549600003],[21.10894,42.20617099700007],[21.212697500000047,42.110701],[21.32123450000006,42.10798899800005],[21.30238950000006,42.14130799900005],[21.385102500000073,42.24647949900003],[21.444318500000065,42.23493049800004],[21.44485750000007,42.27789799800007],[21.586946500000067,42.26281749700007],[22.29804450000006,42.36744499900004],[22.36020650000006,42.311157]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK00","LEVL_CODE":2,"CNTR_CODE":"MK","NAME_LATN":"Poranešna jugoslovenska Republika Makedonija","NUTS_NAME":"Поранешна југословенска Република Македонија"},"geometry":{"type":"Polygon","coordinates":[[[22.36020650000006,42.311157],[22.510412,42.15515849700006],[22.867214,42.02219949600004],[22.90193350000004,41.87764199900005],[23.032124,41.72430099800005],[22.948411500000077,41.63179649800003],[22.968327500000044,41.51983549800008],[22.965055500000062,41.35190749700007],[22.927591500000062,41.338539498000046],[22.879178500000023,41.34065299700006],[22.762478,41.320234],[22.732037,41.146391497000025],[22.66647,41.18633949800005],[22.587711,41.11729599900008],[22.332052,41.120272500000056],[22.216216,41.17045749700003],[21.929438,41.10035049700008],[21.787378,40.931125499000075],[21.684081,40.94091499800004],[21.59323550000005,40.86600249700007],[21.417245500000035,40.917696497000065],[21.301891500000067,40.865341998000076],[20.98020450000007,40.855664999000055],[20.968201500000077,40.91114599700006],[20.837823,40.92768299800008],[20.732268500000032,40.91173249800005],[20.670626,41.08735149800003],[20.595609500000023,41.092411998000046],[20.515577,41.23095550000005],[20.560363,41.40845850100004],[20.456284500000038,41.554024],[20.557774,41.581870501000026],[20.514314,41.73048299700008],[20.594286,41.87732749700007],[20.727972500000078,41.86427699800004],[20.776524,41.91848549700006],[20.748154,42.02570499600006],[20.794234500000073,42.08364549600003],[21.10894,42.20617099700007],[21.212697500000047,42.110701],[21.32123450000006,42.10798899800005],[21.30238950000006,42.14130799900005],[21.385102500000073,42.24647949900003],[21.444318500000065,42.23493049800004],[21.44485750000007,42.27789799800007],[21.586946500000067,42.26281749700007],[22.29804450000006,42.36744499900004],[22.36020650000006,42.311157]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK001","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Vardarski","NUTS_NAME":"Вардарски"},"geometry":{"type":"Polygon","coordinates":[[[22.03418350000004,42.015945499000054],[22.112967,41.83356949800003],[21.956399500000032,41.69400599800008],[22.26637949900004,41.55637199700004],[22.370469499000023,41.378327],[22.295784,41.32994649800003],[22.299713500000053,41.26493899800005],[22.216216,41.17045749700003],[21.929438,41.10035049700008],[21.881946,41.26977399900005],[21.816538500000036,41.306898998000065],[21.824118500000054,41.433879997000076],[21.47827650000005,41.50472449800003],[21.37388050000004,41.60552099600005],[21.393491,41.71796599800007],[21.611381,41.759243499000036],[21.63598250000007,41.835702498000046],[21.728494499000078,41.81767499600005],[21.744069500000023,41.89941349900005],[21.821308500000043,41.96056649800005],[22.03418350000004,42.015945499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK002","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Istočen","NUTS_NAME":"Источен"},"geometry":{"type":"Polygon","coordinates":[[[22.867214,42.02219949600004],[22.90193350000004,41.87764199900005],[23.032124,41.72430099800005],[22.948411500000077,41.63179649800003],[22.968327500000044,41.51983549800008],[22.813104,41.527968498000064],[22.706113,41.587848500000064],[22.626353,41.74977199800003],[22.465575500000057,41.78729399900004],[22.394415500000036,41.68940499800004],[22.317597500000034,41.685580498000036],[22.341551500000037,41.60763499800004],[22.26637949900004,41.55637199700004],[21.956399500000032,41.69400599800008],[22.112967,41.83356949800003],[22.03418350000004,42.015945499000054],[22.262596,42.02019449900007],[22.266975,42.06993049700003],[22.356659500000035,42.05100699600007],[22.510412,42.15515849700006],[22.867214,42.02219949600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK003","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Jugozapaden","NUTS_NAME":"Југозападен"},"geometry":{"type":"Polygon","coordinates":[[[21.23788599900007,41.830002],[21.393491,41.71796599800007],[21.37388050000004,41.60552099600005],[21.333837500000072,41.49875699800003],[20.954759500000023,41.32389599800007],[21.039126500000066,41.21500599700005],[20.903026,41.12486099800003],[20.837823,40.92768299800008],[20.732268500000032,40.91173249800005],[20.670626,41.08735149800003],[20.595609500000023,41.092411998000046],[20.515577,41.23095550000005],[20.560363,41.40845850100004],[20.456284500000038,41.554024],[20.557774,41.581870501000026],[20.782889,41.49006749800003],[20.829152,41.583192997000026],[20.778336,41.62894349600003],[20.800562,41.652038498000024],[21.052090500000077,41.66056399900003],[21.016109,41.786526998000056],[21.23788599900007,41.830002]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK004","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Jugoistočen","NUTS_NAME":"Југоисточен"},"geometry":{"type":"Polygon","coordinates":[[[22.968327500000044,41.51983549800008],[22.965055500000062,41.35190749700007],[22.927591500000062,41.338539498000046],[22.879178500000023,41.34065299700006],[22.762478,41.320234],[22.732037,41.146391497000025],[22.66647,41.18633949800005],[22.587711,41.11729599900008],[22.332052,41.120272500000056],[22.216216,41.17045749700003],[22.299713500000053,41.26493899800005],[22.295784,41.32994649800003],[22.370469499000023,41.378327],[22.26637949900004,41.55637199700004],[22.341551500000037,41.60763499800004],[22.317597500000034,41.685580498000036],[22.394415500000036,41.68940499800004],[22.465575500000057,41.78729399900004],[22.626353,41.74977199800003],[22.706113,41.587848500000064],[22.813104,41.527968498000064],[22.968327500000044,41.51983549800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK005","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Пелагониски","NUTS_NAME":"Пелагониски"},"geometry":{"type":"Polygon","coordinates":[[[21.929438,41.10035049700008],[21.787378,40.931125499000075],[21.684081,40.94091499800004],[21.59323550000005,40.86600249700007],[21.417245500000035,40.917696497000065],[21.301891500000067,40.865341998000076],[20.98020450000007,40.855664999000055],[20.968201500000077,40.91114599700006],[20.837823,40.92768299800008],[20.903026,41.12486099800003],[21.039126500000066,41.21500599700005],[20.954759500000023,41.32389599800007],[21.333837500000072,41.49875699800003],[21.37388050000004,41.60552099600005],[21.47827650000005,41.50472449800003],[21.824118500000054,41.433879997000076],[21.816538500000036,41.306898998000065],[21.881946,41.26977399900005],[21.929438,41.10035049700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK006","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Pološki","NUTS_NAME":"Полошки"},"geometry":{"type":"Polygon","coordinates":[[[21.212697500000047,42.110701],[21.14987350000007,42.01281549600003],[21.23788599900007,41.830002],[21.016109,41.786526998000056],[21.052090500000077,41.66056399900003],[20.800562,41.652038498000024],[20.778336,41.62894349600003],[20.829152,41.583192997000026],[20.782889,41.49006749800003],[20.557774,41.581870501000026],[20.514314,41.73048299700008],[20.594286,41.87732749700007],[20.727972500000078,41.86427699800004],[20.776524,41.91848549700006],[20.748154,42.02570499600006],[20.794234500000073,42.08364549600003],[21.10894,42.20617099700007],[21.212697500000047,42.110701]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK007","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Severoistočen","NUTS_NAME":"Североисточен"},"geometry":{"type":"Polygon","coordinates":[[[22.36020650000006,42.311157],[22.510412,42.15515849700006],[22.356659500000035,42.05100699600007],[22.266975,42.06993049700003],[22.262596,42.02019449900007],[22.03418350000004,42.015945499000054],[21.821308500000043,41.96056649800005],[21.559101,42.067177998000034],[21.489154,42.156716997000046],[21.51858050000004,42.20637599600008],[21.444318500000065,42.23493049800004],[21.44485750000007,42.27789799800007],[21.586946500000067,42.26281749700007],[22.29804450000006,42.36744499900004],[22.36020650000006,42.311157]]]}},{"type":"Feature","properties":{"NUTS_ID":"MK008","LEVL_CODE":3,"CNTR_CODE":"MK","NAME_LATN":"Skopski","NUTS_NAME":"Скопски"},"geometry":{"type":"Polygon","coordinates":[[[21.444318500000065,42.23493049800004],[21.51858050000004,42.20637599600008],[21.489154,42.156716997000046],[21.559101,42.067177998000034],[21.821308500000043,41.96056649800005],[21.744069500000023,41.89941349900005],[21.728494499000078,41.81767499600005],[21.63598250000007,41.835702498000046],[21.611381,41.759243499000036],[21.393491,41.71796599800007],[21.23788599900007,41.830002],[21.14987350000007,42.01281549600003],[21.212697500000047,42.110701],[21.32123450000006,42.10798899800005],[21.30238950000006,42.14130799900005],[21.385102500000073,42.24647949900003],[21.444318500000065,42.23493049800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"MT","LEVL_CODE":0,"CNTR_CODE":"MT","NAME_LATN":"MALTA","NUTS_NAME":"MALTA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.572043500000063,35.84975449700005],[14.535398,35.80750299700003],[14.416679500000043,35.82818949800003],[14.342219999000065,35.873474497000075],[14.319842,35.970367999000075],[14.37609150000003,35.987453998000035],[14.350481,35.969952498000055],[14.572043500000063,35.84975449700005]]],[[[14.336181,36.03225849800003],[14.218846,36.020974497000054],[14.186361,36.036271998000075],[14.184842,36.07407849700007],[14.30005650000004,36.05719499800006],[14.336181,36.03225849800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"MT0","LEVL_CODE":1,"CNTR_CODE":"MT","NAME_LATN":"MALTA","NUTS_NAME":"MALTA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.572043500000063,35.84975449700005],[14.535398,35.80750299700003],[14.416679500000043,35.82818949800003],[14.342219999000065,35.873474497000075],[14.319842,35.970367999000075],[14.37609150000003,35.987453998000035],[14.350481,35.969952498000055],[14.572043500000063,35.84975449700005]]],[[[14.336181,36.03225849800003],[14.218846,36.020974497000054],[14.186361,36.036271998000075],[14.184842,36.07407849700007],[14.30005650000004,36.05719499800006],[14.336181,36.03225849800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"MT00","LEVL_CODE":2,"CNTR_CODE":"MT","NAME_LATN":"Malta","NUTS_NAME":"Malta"},"geometry":{"type":"MultiPolygon","coordinates":[[[[14.572043500000063,35.84975449700005],[14.535398,35.80750299700003],[14.416679500000043,35.82818949800003],[14.342219999000065,35.873474497000075],[14.319842,35.970367999000075],[14.37609150000003,35.987453998000035],[14.350481,35.969952498000055],[14.572043500000063,35.84975449700005]]],[[[14.336181,36.03225849800003],[14.218846,36.020974497000054],[14.186361,36.036271998000075],[14.184842,36.07407849700007],[14.30005650000004,36.05719499800006],[14.336181,36.03225849800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"MT001","LEVL_CODE":3,"CNTR_CODE":"MT","NAME_LATN":"Malta","NUTS_NAME":"Malta"},"geometry":{"type":"Polygon","coordinates":[[[14.572043500000063,35.84975449700005],[14.535398,35.80750299700003],[14.416679500000043,35.82818949800003],[14.342219999000065,35.873474497000075],[14.319842,35.970367999000075],[14.37609150000003,35.987453998000035],[14.350481,35.969952498000055],[14.572043500000063,35.84975449700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"MT002","LEVL_CODE":3,"CNTR_CODE":"MT","NAME_LATN":"Gozo and Comino / Ghawdex u Kemmuna","NUTS_NAME":"Gozo and Comino / Ghawdex u Kemmuna"},"geometry":{"type":"Polygon","coordinates":[[[14.336181,36.03225849800003],[14.218846,36.020974497000054],[14.186361,36.036271998000075],[14.184842,36.07407849700007],[14.30005650000004,36.05719499800006],[14.336181,36.03225849800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL","LEVL_CODE":0,"CNTR_CODE":"NL","NAME_LATN":"NEDERLAND","NUTS_NAME":"NEDERLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[7.209295,53.00024449700004],[7.092692,52.83820099800005],[7.055577,52.643369998000026],[7.006229500000074,52.63876299800006],[6.70973250000003,52.627823498000055],[6.697865500000034,52.48628599700004],[6.987941500000034,52.46954099800007],[7.072438,52.37311650000004],[7.026305,52.28744399700008],[7.065685,52.24137299800003],[6.760465,52.118569499000046],[6.714659,52.03989349600005],[6.828513,51.96406649900007],[6.72106,51.89648899800005],[6.390816,51.873934],[6.407779500000061,51.828092],[6.167766,51.90080449800007],[6.103341500000056,51.89261699800005],[6.166407500000048,51.84083449700006],[6.063382500000046,51.865443998000046],[5.953192,51.74784599700007],[6.118209500000034,51.65610299800005],[6.091235500000039,51.60590949600004],[6.213423,51.50784149800006],[6.224405,51.364978998000026],[6.072657,51.242587497000045],[6.082150999000021,51.17179899900003],[6.174812,51.18451349900005],[5.877084999000033,51.03210099900008],[5.897198,50.974845498000036],[6.026435,50.983273997000026],[6.018124,50.93459299800003],[6.086947500000065,50.91313499800003],[6.017985,50.84379649700003],[6.020998999000028,50.75429549900008],[5.892073,50.75523749800004],[5.682000499000026,50.757446497000046],[5.687622,50.811923998000054],[5.644038500000022,50.87138349800006],[5.758272499000043,50.95479499900006],[5.766149,51.009235499000056],[5.798274,51.05985349800005],[5.855658500000061,51.14459949900004],[5.566283500000054,51.22083649700005],[5.515765,51.29532449800007],[5.237716500000033,51.261600498000064],[5.071045,51.39342099800007],[5.10218,51.42900499600006],[5.037678500000027,51.47936999800004],[4.927639501000044,51.39542449600003],[4.773122500000056,51.413783496000065],[4.841390500000045,51.48071299900005],[4.759926,51.50246449700006],[4.669544,51.42638399800006],[4.535450500000024,51.422889499000064],[4.548236,51.473228497000036],[4.379466500000035,51.44664],[4.431206,51.363845998000045],[4.279565,51.37601749600003],[4.243669500000067,51.37472949800008],[3.9754,51.46165099900003],[3.815721999000061,51.38460149900004],[3.677822,51.44882949700008],[3.71355,51.470973997000044],[3.574728,51.43867849700007],[3.4342,51.52615749800003],[3.839144500000032,51.605071999000074],[3.899378,51.56350749600006],[3.869229,51.54168799800004],[4.223578,51.43865599600008],[4.164783500000055,51.52251049700004],[3.982944500000031,51.58132999900005],[4.193678,51.635616998000046],[4.161857001000044,51.66683157600005],[4.135041090000072,51.67330369700005],[3.968077500000049,51.61403999600003],[3.678739,51.695324],[3.839083,51.75829699900004],[3.866527500000075,51.81832449800004],[4.067505500000038,51.84529999700004],[4.051014500000065,51.919221496000034],[3.980849,51.925444],[3.983133,51.98452499800004],[4.127788500000065,52.00054249600004],[4.198018500000046,52.05414449600005],[4.374232500000062,52.187089996000054],[4.493847500000072,52.32826],[4.560596500000031,52.437426],[4.609676500000035,52.573401],[4.649071,52.756177499000046],[4.730989,52.962188498000046],[4.874539,52.88948449700007],[5.164383500000042,53.00091050000003],[5.41122850000005,53.15172449700003],[5.586983,53.30121249800004],[6.191301500000066,53.410937998000065],[6.882578,53.44065499800007],[6.874905,53.40801299700007],[7.092712500000061,53.25701749600006],[7.208935,53.243064497000034],[7.202794500000039,53.11328149800005],[7.209295,53.00024449700004]]],[[[4.897455500000035,53.07828899800006],[4.718368500000054,52.985955999000055],[4.72006650000003,53.06102750000008],[4.863693,53.18050149700008],[4.897455500000035,53.07828899800006]]],[[[4.234817500000077,51.348253999000065],[3.977666500000055,51.22513199800005],[3.856339500000047,51.21105599900005],[3.621433500000023,51.292781998000066],[3.380661,51.274299496000026],[3.36721650000004,51.368134498000074],[3.547133500000029,51.404273998000065],[3.818089,51.33777249600007],[3.978855500000066,51.40676499800003],[4.234817500000077,51.348253999000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL1","LEVL_CODE":1,"CNTR_CODE":"NL","NAME_LATN":"NOORD-NEDERLAND","NUTS_NAME":"NOORD-NEDERLAND"},"geometry":{"type":"Polygon","coordinates":[[[7.209295,53.00024449700004],[7.092692,52.83820099800005],[7.055577,52.643369998000026],[7.006229500000074,52.63876299800006],[6.70973250000003,52.627823498000055],[6.629429500000072,52.669658499000036],[6.518520999000032,52.61412799700008],[6.163003500000059,52.68006149800004],[6.120073,52.751021998000056],[6.201702500000067,52.79367049700005],[6.119814,52.854267],[5.819826500000033,52.81731750000006],[5.795148500000039,52.80649949800005],[5.377261500000031,52.764804999000035],[5.167425500000036,52.99879849800004],[5.164383500000042,53.00091050000003],[5.41122850000005,53.15172449700003],[5.586983,53.30121249800004],[6.191301500000066,53.410937998000065],[6.882578,53.44065499800007],[6.874905,53.40801299700007],[7.092712500000061,53.25701749600006],[7.208935,53.243064497000034],[7.202794500000039,53.11328149800005],[7.209295,53.00024449700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL11","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Groningen","NUTS_NAME":"Groningen"},"geometry":{"type":"Polygon","coordinates":[[[7.208935,53.243064497000034],[7.202794500000039,53.11328149800005],[7.209295,53.00024449700004],[7.092692,52.83820099800005],[6.935702,52.99336249800007],[6.81381,53.07104849700005],[6.530556,53.195545],[6.315237,53.094051498000056],[6.176817,53.15945049800007],[6.286981500000024,53.341380997000044],[6.175716500000021,53.36752699800007],[6.191301500000066,53.410937998000065],[6.882578,53.44065499800007],[6.874905,53.40801299700007],[7.092712500000061,53.25701749600006],[7.208935,53.243064497000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL111","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Oost-Groningen","NUTS_NAME":"Oost-Groningen"},"geometry":{"type":"Polygon","coordinates":[[[7.092712500000061,53.25701749600006],[7.208935,53.243064497000034],[7.202794500000039,53.11328149800005],[7.209295,53.00024449700004],[7.092692,52.83820099800005],[6.935702,52.99336249800007],[6.81381,53.07104849700005],[6.81423249900007,53.193595997000045],[6.911510500000077,53.24296949600006],[7.092712500000061,53.25701749600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL112","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Delfzijl en omgeving","NUTS_NAME":"Delfzijl en omgeving"},"geometry":{"type":"Polygon","coordinates":[[[6.874905,53.40801299700007],[7.092712500000061,53.25701749600006],[6.911510500000077,53.24296949600006],[6.887363500000049,53.298068998000076],[6.595750500000065,53.34709199900004],[6.874905,53.40801299700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL113","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Overig Groningen","NUTS_NAME":"Overig Groningen"},"geometry":{"type":"Polygon","coordinates":[[[6.882578,53.44065499800007],[6.874905,53.40801299700007],[6.595750500000065,53.34709199900004],[6.887363500000049,53.298068998000076],[6.911510500000077,53.24296949600006],[6.81423249900007,53.193595997000045],[6.81381,53.07104849700005],[6.530556,53.195545],[6.315237,53.094051498000056],[6.176817,53.15945049800007],[6.286981500000024,53.341380997000044],[6.175716500000021,53.36752699800007],[6.191301500000066,53.410937998000065],[6.882578,53.44065499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL12","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Friesland (NL)","NUTS_NAME":"Friesland (NL)"},"geometry":{"type":"Polygon","coordinates":[[[5.795148500000039,52.80649949800005],[5.377261500000031,52.764804999000035],[5.167425500000036,52.99879849800004],[5.164383500000042,53.00091050000003],[5.41122850000005,53.15172449700003],[5.586983,53.30121249800004],[6.191301500000066,53.410937998000065],[6.175716500000021,53.36752699800007],[6.286981500000024,53.341380997000044],[6.176817,53.15945049800007],[6.315237,53.094051498000056],[6.42764150000005,52.97179399700008],[6.369053,52.921973499000046],[6.119814,52.854267],[5.819826500000033,52.81731750000006],[5.795148500000039,52.80649949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL121","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Noord-Friesland","NUTS_NAME":"Noord-Friesland"},"geometry":{"type":"Polygon","coordinates":[[[6.012965,53.15446099600007],[5.882135500000061,53.106895500000064],[5.957521,53.05361950000008],[5.90208,53.020587997000064],[5.751414500000067,53.04701999700006],[5.41122850000005,53.15172449700003],[5.586983,53.30121249800004],[6.191301500000066,53.410937998000065],[6.175716500000021,53.36752699800007],[6.286981500000024,53.341380997000044],[6.176817,53.15945049800007],[6.012965,53.15446099600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL122","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidwest-Friesland","NUTS_NAME":"Zuidwest-Friesland"},"geometry":{"type":"Polygon","coordinates":[[[5.751414500000067,53.04701999700006],[5.639991,52.921921],[5.883161500000028,52.88901899800004],[5.819826500000033,52.81731750000006],[5.795148500000039,52.80649949800005],[5.377261500000031,52.764804999000035],[5.167425500000036,52.99879849800004],[5.164383500000042,53.00091050000003],[5.41122850000005,53.15172449700003],[5.751414500000067,53.04701999700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL123","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidoost-Friesland","NUTS_NAME":"Zuidoost-Friesland"},"geometry":{"type":"Polygon","coordinates":[[[6.315237,53.094051498000056],[6.42764150000005,52.97179399700008],[6.369053,52.921973499000046],[6.119814,52.854267],[5.819826500000033,52.81731750000006],[5.883161500000028,52.88901899800004],[5.639991,52.921921],[5.751414500000067,53.04701999700006],[5.90208,53.020587997000064],[5.957521,53.05361950000008],[5.882135500000061,53.106895500000064],[6.012965,53.15446099600007],[6.176817,53.15945049800007],[6.315237,53.094051498000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL13","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Drenthe","NUTS_NAME":"Drenthe"},"geometry":{"type":"Polygon","coordinates":[[[7.092692,52.83820099800005],[7.055577,52.643369998000026],[7.006229500000074,52.63876299800006],[6.70973250000003,52.627823498000055],[6.629429500000072,52.669658499000036],[6.518520999000032,52.61412799700008],[6.163003500000059,52.68006149800004],[6.120073,52.751021998000056],[6.201702500000067,52.79367049700005],[6.119814,52.854267],[6.369053,52.921973499000046],[6.42764150000005,52.97179399700008],[6.315237,53.094051498000056],[6.530556,53.195545],[6.81381,53.07104849700005],[6.935702,52.99336249800007],[7.092692,52.83820099800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL131","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Noord-Drenthe","NUTS_NAME":"Noord-Drenthe"},"geometry":{"type":"Polygon","coordinates":[[[6.81381,53.07104849700005],[6.935702,52.99336249800007],[6.727706500000068,52.95471950000007],[6.728629,52.853966],[6.587468,52.735691],[6.369053,52.921973499000046],[6.42764150000005,52.97179399700008],[6.315237,53.094051498000056],[6.530556,53.195545],[6.81381,53.07104849700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL132","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidoost-Drenthe","NUTS_NAME":"Zuidoost-Drenthe"},"geometry":{"type":"Polygon","coordinates":[[[7.092692,52.83820099800005],[7.055577,52.643369998000026],[7.006229500000074,52.63876299800006],[6.70973250000003,52.627823498000055],[6.629429500000072,52.669658499000036],[6.587468,52.735691],[6.728629,52.853966],[6.727706500000068,52.95471950000007],[6.935702,52.99336249800007],[7.092692,52.83820099800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL133","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidwest-Drenthe","NUTS_NAME":"Zuidwest-Drenthe"},"geometry":{"type":"Polygon","coordinates":[[[6.369053,52.921973499000046],[6.587468,52.735691],[6.629429500000072,52.669658499000036],[6.518520999000032,52.61412799700008],[6.163003500000059,52.68006149800004],[6.120073,52.751021998000056],[6.201702500000067,52.79367049700005],[6.119814,52.854267],[6.369053,52.921973499000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL2","LEVL_CODE":1,"CNTR_CODE":"NL","NAME_LATN":"OOST-NEDERLAND","NUTS_NAME":"OOST-NEDERLAND"},"geometry":{"type":"Polygon","coordinates":[[[6.163003500000059,52.68006149800004],[6.518520999000032,52.61412799700008],[6.629429500000072,52.669658499000036],[6.70973250000003,52.627823498000055],[6.697865500000034,52.48628599700004],[6.987941500000034,52.46954099800007],[7.072438,52.37311650000004],[7.026305,52.28744399700008],[7.065685,52.24137299800003],[6.760465,52.118569499000046],[6.714659,52.03989349600005],[6.828513,51.96406649900007],[6.72106,51.89648899800005],[6.390816,51.873934],[6.407779500000061,51.828092],[6.167766,51.90080449800007],[6.103341500000056,51.89261699800005],[6.166407500000048,51.84083449700006],[6.063382500000046,51.865443998000046],[5.953192,51.74784599700007],[5.865158500000064,51.757407998000076],[5.597879500000033,51.82808299800007],[5.128057500000068,51.73760999900003],[5.000534,51.82093799800003],[5.149456,51.933452498000065],[5.606011500000022,51.94324849700007],[5.626232,51.97391149900005],[5.537160500000027,52.08866550000005],[5.459184,52.08028399700004],[5.514091,52.135890998000036],[5.404633,52.249629998000046],[5.335462,52.29021849800006],[5.079161,52.388652999000044],[5.185487500000022,52.41564549900005],[5.060426500000062,52.57893749800007],[5.361148,52.67572799800007],[5.377261500000031,52.764804999000035],[5.795148500000039,52.80649949800005],[5.819826500000033,52.81731750000006],[6.119814,52.854267],[6.201702500000067,52.79367049700005],[6.120073,52.751021998000056],[6.163003500000059,52.68006149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL21","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Overijssel","NUTS_NAME":"Overijssel"},"geometry":{"type":"Polygon","coordinates":[[[6.163003500000059,52.68006149800004],[6.518520999000032,52.61412799700008],[6.629429500000072,52.669658499000036],[6.70973250000003,52.627823498000055],[6.697865500000034,52.48628599700004],[6.987941500000034,52.46954099800007],[7.072438,52.37311650000004],[7.026305,52.28744399700008],[7.065685,52.24137299800003],[6.760465,52.118569499000046],[6.381997500000068,52.246086],[6.166411500000038,52.23104499900006],[6.071355999000048,52.322582],[6.126104,52.38441850000004],[6.109792500000026,52.44057450000008],[5.864311,52.51816949600004],[5.777970500000038,52.60753249800007],[6.017299500000036,52.64323049800004],[5.795148500000039,52.80649949800005],[5.819826500000033,52.81731750000006],[6.119814,52.854267],[6.201702500000067,52.79367049700005],[6.120073,52.751021998000056],[6.163003500000059,52.68006149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL211","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Noord-Overijssel","NUTS_NAME":"Noord-Overijssel"},"geometry":{"type":"Polygon","coordinates":[[[6.629429500000072,52.669658499000036],[6.70973250000003,52.627823498000055],[6.697865500000034,52.48628599700004],[6.390622500000063,52.42805499900004],[6.109792500000026,52.44057450000008],[5.864311,52.51816949600004],[5.777970500000038,52.60753249800007],[6.017299500000036,52.64323049800004],[5.795148500000039,52.80649949800005],[5.819826500000033,52.81731750000006],[6.119814,52.854267],[6.201702500000067,52.79367049700005],[6.120073,52.751021998000056],[6.163003500000059,52.68006149800004],[6.518520999000032,52.61412799700008],[6.629429500000072,52.669658499000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL212","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidwest-Overijssel","NUTS_NAME":"Zuidwest-Overijssel"},"geometry":{"type":"Polygon","coordinates":[[[6.390622500000063,52.42805499900004],[6.326621500000044,52.30167749600008],[6.381997500000068,52.246086],[6.166411500000038,52.23104499900006],[6.071355999000048,52.322582],[6.126104,52.38441850000004],[6.109792500000026,52.44057450000008],[6.390622500000063,52.42805499900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL213","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Twente","NUTS_NAME":"Twente"},"geometry":{"type":"Polygon","coordinates":[[[6.697865500000034,52.48628599700004],[6.987941500000034,52.46954099800007],[7.072438,52.37311650000004],[7.026305,52.28744399700008],[7.065685,52.24137299800003],[6.760465,52.118569499000046],[6.381997500000068,52.246086],[6.326621500000044,52.30167749600008],[6.390622500000063,52.42805499900004],[6.697865500000034,52.48628599700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL22","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Gelderland","NUTS_NAME":"Gelderland"},"geometry":{"type":"Polygon","coordinates":[[[6.071355999000048,52.322582],[6.166411500000038,52.23104499900006],[6.381997500000068,52.246086],[6.760465,52.118569499000046],[6.714659,52.03989349600005],[6.828513,51.96406649900007],[6.72106,51.89648899800005],[6.390816,51.873934],[6.407779500000061,51.828092],[6.167766,51.90080449800007],[6.103341500000056,51.89261699800005],[6.166407500000048,51.84083449700006],[6.063382500000046,51.865443998000046],[5.953192,51.74784599700007],[5.865158500000064,51.757407998000076],[5.597879500000033,51.82808299800007],[5.128057500000068,51.73760999900003],[5.000534,51.82093799800003],[5.149456,51.933452498000065],[5.606011500000022,51.94324849700007],[5.626232,51.97391149900005],[5.537160500000027,52.08866550000005],[5.459184,52.08028399700004],[5.514091,52.135890998000036],[5.404633,52.249629998000046],[5.533282500000041,52.27274349800007],[5.587707500000022,52.36145399800006],[5.810012500000028,52.43572599700008],[5.864311,52.51816949600004],[6.109792500000026,52.44057450000008],[6.126104,52.38441850000004],[6.071355999000048,52.322582]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL221","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Veluwe","NUTS_NAME":"Veluwe"},"geometry":{"type":"Polygon","coordinates":[[[6.109792500000026,52.44057450000008],[6.126104,52.38441850000004],[6.071355999000048,52.322582],[6.166411500000038,52.23104499900006],[6.18487,52.16143049900006],[6.008872500000052,52.07390999900008],[5.665807,51.95278549900007],[5.606011500000022,51.94324849700007],[5.626232,51.97391149900005],[5.537160500000027,52.08866550000005],[5.459184,52.08028399700004],[5.514091,52.135890998000036],[5.404633,52.249629998000046],[5.533282500000041,52.27274349800007],[5.587707500000022,52.36145399800006],[5.810012500000028,52.43572599700008],[5.864311,52.51816949600004],[6.109792500000026,52.44057450000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL224","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidwest-Gelderland","NUTS_NAME":"Zuidwest-Gelderland"},"geometry":{"type":"Polygon","coordinates":[[[5.606011500000022,51.94324849700007],[5.665807,51.95278549900007],[5.711011,51.89683149800004],[5.53111750000005,51.89479049900007],[5.597879500000033,51.82808299800007],[5.128057500000068,51.73760999900003],[5.000534,51.82093799800003],[5.149456,51.933452498000065],[5.606011500000022,51.94324849700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL225","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Achterhoek","NUTS_NAME":"Achterhoek"},"geometry":{"type":"Polygon","coordinates":[[[6.760465,52.118569499000046],[6.714659,52.03989349600005],[6.828513,51.96406649900007],[6.72106,51.89648899800005],[6.390816,51.873934],[6.407779500000061,51.828092],[6.167766,51.90080449800007],[6.090462,51.96783049700008],[6.177322500000059,52.020385496000074],[6.008872500000052,52.07390999900008],[6.18487,52.16143049900006],[6.166411500000038,52.23104499900006],[6.381997500000068,52.246086],[6.760465,52.118569499000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL226","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Arnhem/Nijmegen","NUTS_NAME":"Arnhem/Nijmegen"},"geometry":{"type":"Polygon","coordinates":[[[6.167766,51.90080449800007],[6.103341500000056,51.89261699800005],[6.166407500000048,51.84083449700006],[6.063382500000046,51.865443998000046],[5.953192,51.74784599700007],[5.865158500000064,51.757407998000076],[5.597879500000033,51.82808299800007],[5.53111750000005,51.89479049900007],[5.711011,51.89683149800004],[5.665807,51.95278549900007],[6.008872500000052,52.07390999900008],[6.177322500000059,52.020385496000074],[6.090462,51.96783049700008],[6.167766,51.90080449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL23","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Flevoland","NUTS_NAME":"Flevoland"},"geometry":{"type":"Polygon","coordinates":[[[5.795148500000039,52.80649949800005],[6.017299500000036,52.64323049800004],[5.777970500000038,52.60753249800007],[5.864311,52.51816949600004],[5.810012500000028,52.43572599700008],[5.587707500000022,52.36145399800006],[5.533282500000041,52.27274349800007],[5.404633,52.249629998000046],[5.335462,52.29021849800006],[5.079161,52.388652999000044],[5.185487500000022,52.41564549900005],[5.060426500000062,52.57893749800007],[5.361148,52.67572799800007],[5.377261500000031,52.764804999000035],[5.795148500000039,52.80649949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL230","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Flevoland","NUTS_NAME":"Flevoland"},"geometry":{"type":"Polygon","coordinates":[[[5.795148500000039,52.80649949800005],[6.017299500000036,52.64323049800004],[5.777970500000038,52.60753249800007],[5.864311,52.51816949600004],[5.810012500000028,52.43572599700008],[5.587707500000022,52.36145399800006],[5.533282500000041,52.27274349800007],[5.404633,52.249629998000046],[5.335462,52.29021849800006],[5.079161,52.388652999000044],[5.185487500000022,52.41564549900005],[5.060426500000062,52.57893749800007],[5.361148,52.67572799800007],[5.377261500000031,52.764804999000035],[5.795148500000039,52.80649949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL3","LEVL_CODE":1,"CNTR_CODE":"NL","NAME_LATN":"WEST-NEDERLAND","NUTS_NAME":"WEST-NEDERLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.167425500000036,52.99879849800004],[5.377261500000031,52.764804999000035],[5.361148,52.67572799800007],[5.060426500000062,52.57893749800007],[5.185487500000022,52.41564549900005],[5.079161,52.388652999000044],[5.335462,52.29021849800006],[5.404633,52.249629998000046],[5.514091,52.135890998000036],[5.459184,52.08028399700004],[5.537160500000027,52.08866550000005],[5.626232,51.97391149900005],[5.606011500000022,51.94324849700007],[5.149456,51.933452498000065],[5.000534,51.82093799800003],[4.67629450000004,51.72491849900007],[4.620420500000023,51.71412649900003],[4.249017,51.64580149900007],[4.190816999000049,51.59738549800005],[4.279565,51.37601749600003],[4.243669500000067,51.37472949800008],[3.9754,51.46165099900003],[3.815721999000061,51.38460149900004],[3.677822,51.44882949700008],[3.71355,51.470973997000044],[3.574728,51.43867849700007],[3.4342,51.52615749800003],[3.839144500000032,51.605071999000074],[3.899378,51.56350749600006],[3.869229,51.54168799800004],[4.223578,51.43865599600008],[4.164783500000055,51.52251049700004],[3.982944500000031,51.58132999900005],[4.193678,51.635616998000046],[4.161857001000044,51.66683157600005],[4.135041090000072,51.67330369700005],[3.968077500000049,51.61403999600003],[3.678739,51.695324],[3.839083,51.75829699900004],[3.866527500000075,51.81832449800004],[4.067505500000038,51.84529999700004],[4.051014500000065,51.919221496000034],[3.980849,51.925444],[3.983133,51.98452499800004],[4.127788500000065,52.00054249600004],[4.198018500000046,52.05414449600005],[4.374232500000062,52.187089996000054],[4.493847500000072,52.32826],[4.560596500000031,52.437426],[4.609676500000035,52.573401],[4.649071,52.756177499000046],[4.730989,52.962188498000046],[4.874539,52.88948449700007],[5.164383500000042,53.00091050000003],[5.167425500000036,52.99879849800004]]],[[[4.897455500000035,53.07828899800006],[4.718368500000054,52.985955999000055],[4.72006650000003,53.06102750000008],[4.863693,53.18050149700008],[4.897455500000035,53.07828899800006]]],[[[4.234817500000077,51.348253999000065],[3.977666500000055,51.22513199800005],[3.856339500000047,51.21105599900005],[3.621433500000023,51.292781998000066],[3.380661,51.274299496000026],[3.36721650000004,51.368134498000074],[3.547133500000029,51.404273998000065],[3.818089,51.33777249600007],[3.978855500000066,51.40676499800003],[4.234817500000077,51.348253999000065]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL31","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Utrecht","NUTS_NAME":"Utrecht"},"geometry":{"type":"Polygon","coordinates":[[[5.335462,52.29021849800006],[5.404633,52.249629998000046],[5.514091,52.135890998000036],[5.459184,52.08028399700004],[5.537160500000027,52.08866550000005],[5.626232,51.97391149900005],[5.606011500000022,51.94324849700007],[5.149456,51.933452498000065],[4.877899500000069,51.93783549700004],[4.798712500000022,52.022167],[4.829823,52.06685249600008],[4.792076,52.122359998000036],[4.892281,52.16181199600004],[4.794524,52.22672649800006],[5.021537500000022,52.302498498000034],[5.065306500000077,52.28515649900004],[5.022848,52.272613499000045],[5.04613850000004,52.165805997000064],[5.192641,52.17784099900007],[5.335462,52.29021849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL310","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Utrecht","NUTS_NAME":"Utrecht"},"geometry":{"type":"Polygon","coordinates":[[[5.335462,52.29021849800006],[5.404633,52.249629998000046],[5.514091,52.135890998000036],[5.459184,52.08028399700004],[5.537160500000027,52.08866550000005],[5.626232,51.97391149900005],[5.606011500000022,51.94324849700007],[5.149456,51.933452498000065],[4.877899500000069,51.93783549700004],[4.798712500000022,52.022167],[4.829823,52.06685249600008],[4.792076,52.122359998000036],[4.892281,52.16181199600004],[4.794524,52.22672649800006],[5.021537500000022,52.302498498000034],[5.065306500000077,52.28515649900004],[5.022848,52.272613499000045],[5.04613850000004,52.165805997000064],[5.192641,52.17784099900007],[5.335462,52.29021849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL32","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Noord-Holland","NUTS_NAME":"Noord-Holland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.167425500000036,52.99879849800004],[5.377261500000031,52.764804999000035],[5.361148,52.67572799800007],[5.060426500000062,52.57893749800007],[5.185487500000022,52.41564549900005],[5.079161,52.388652999000044],[5.335462,52.29021849800006],[5.192641,52.17784099900007],[5.04613850000004,52.165805997000064],[5.022848,52.272613499000045],[5.065306500000077,52.28515649900004],[5.021537500000022,52.302498498000034],[4.794524,52.22672649800006],[4.728766,52.20981949800006],[4.550627,52.22700499700005],[4.611684,52.31358699800006],[4.493847500000072,52.32826],[4.560596500000031,52.437426],[4.609676500000035,52.573401],[4.649071,52.756177499000046],[4.730989,52.962188498000046],[4.874539,52.88948449700007],[5.164383500000042,53.00091050000003],[5.167425500000036,52.99879849800004]]],[[[4.897455500000035,53.07828899800006],[4.718368500000054,52.985955999000055],[4.72006650000003,53.06102750000008],[4.863693,53.18050149700008],[4.897455500000035,53.07828899800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL321","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Kop van Noord-Holland","NUTS_NAME":"Kop van Noord-Holland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.167425500000036,52.99879849800004],[5.377261500000031,52.764804999000035],[5.361148,52.67572799800007],[5.060426500000062,52.57893749800007],[4.926488,52.60705199700004],[4.864557,52.65003950000005],[4.899045500000057,52.70682899900004],[4.728919500000075,52.69138349800005],[4.649071,52.756177499000046],[4.730989,52.962188498000046],[4.874539,52.88948449700007],[5.164383500000042,53.00091050000003],[5.167425500000036,52.99879849800004]]],[[[4.897455500000035,53.07828899800006],[4.718368500000054,52.985955999000055],[4.72006650000003,53.06102750000008],[4.863693,53.18050149700008],[4.897455500000035,53.07828899800006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL322","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Alkmaar en omgeving","NUTS_NAME":"Alkmaar en omgeving"},"geometry":{"type":"Polygon","coordinates":[[[4.926488,52.60705199700004],[4.781546500000047,52.557121496000036],[4.609676500000035,52.573401],[4.649071,52.756177499000046],[4.728919500000075,52.69138349800005],[4.899045500000057,52.70682899900004],[4.864557,52.65003950000005],[4.926488,52.60705199700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL323","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"IJmond","NUTS_NAME":"IJmond"},"geometry":{"type":"Polygon","coordinates":[[[4.781546500000047,52.557121496000036],[4.782528,52.51953499700005],[4.674783500000046,52.45544449700003],[4.722983499000065,52.43333399900007],[4.560596500000031,52.437426],[4.609676500000035,52.573401],[4.781546500000047,52.557121496000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL324","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Agglomeratie Haarlem","NUTS_NAME":"Agglomeratie Haarlem"},"geometry":{"type":"Polygon","coordinates":[[[4.722983499000065,52.43333399900007],[4.739188500000068,52.43122099700008],[4.611684,52.31358699800006],[4.493847500000072,52.32826],[4.560596500000031,52.437426],[4.722983499000065,52.43333399900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL325","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zaanstreek","NUTS_NAME":"Zaanstreek"},"geometry":{"type":"Polygon","coordinates":[[[4.739188500000068,52.43122099700008],[4.722983499000065,52.43333399900007],[4.674783500000046,52.45544449700003],[4.782528,52.51953499700005],[4.934380500000032,52.511611998000035],[4.852457500000071,52.47035999900004],[4.855963,52.41653050000008],[4.739188500000068,52.43122099700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL326","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Groot-Amsterdam","NUTS_NAME":"Groot-Amsterdam"},"geometry":{"type":"Polygon","coordinates":[[[5.060426500000062,52.57893749800007],[5.185487500000022,52.41564549900005],[5.079161,52.388652999000044],[5.021537500000022,52.302498498000034],[4.794524,52.22672649800006],[4.728766,52.20981949800006],[4.550627,52.22700499700005],[4.611684,52.31358699800006],[4.739188500000068,52.43122099700008],[4.855963,52.41653050000008],[4.852457500000071,52.47035999900004],[4.934380500000032,52.511611998000035],[4.782528,52.51953499700005],[4.781546500000047,52.557121496000036],[4.926488,52.60705199700004],[5.060426500000062,52.57893749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL327","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Het Gooi en Vechtstreek","NUTS_NAME":"Het Gooi en Vechtstreek"},"geometry":{"type":"Polygon","coordinates":[[[5.335462,52.29021849800006],[5.192641,52.17784099900007],[5.04613850000004,52.165805997000064],[5.022848,52.272613499000045],[5.065306500000077,52.28515649900004],[5.021537500000022,52.302498498000034],[5.079161,52.388652999000044],[5.335462,52.29021849800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL33","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Zuid-Holland","NUTS_NAME":"Zuid-Holland"},"geometry":{"type":"Polygon","coordinates":[[[4.550627,52.22700499700005],[4.728766,52.20981949800006],[4.794524,52.22672649800006],[4.892281,52.16181199600004],[4.792076,52.122359998000036],[4.829823,52.06685249600008],[4.798712500000022,52.022167],[4.877899500000069,51.93783549700004],[5.149456,51.933452498000065],[5.000534,51.82093799800003],[4.67629450000004,51.72491849900007],[4.620420500000023,51.71412649900003],[4.249017,51.64580149900007],[4.161857001000044,51.66683157600005],[4.135041090000072,51.67330369700005],[4.047814500000072,51.69436249800003],[4.013337,51.759010500000045],[3.839083,51.75829699900004],[3.866527500000075,51.81832449800004],[4.067505500000038,51.84529999700004],[4.051014500000065,51.919221496000034],[3.980849,51.925444],[3.983133,51.98452499800004],[4.127788500000065,52.00054249600004],[4.198018500000046,52.05414449600005],[4.374232500000062,52.187089996000054],[4.493847500000072,52.32826],[4.611684,52.31358699800006],[4.550627,52.22700499700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL332","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Agglomeratie 's-Gravenhage","NUTS_NAME":"Agglomeratie 's-Gravenhage"},"geometry":{"type":"Polygon","coordinates":[[[4.53061150000002,52.08048249800004],[4.557198500000027,52.06123750000006],[4.407886500000075,51.968177997000055],[4.373484500000075,52.03261949600005],[4.198018500000046,52.05414449600005],[4.374232500000062,52.187089996000054],[4.442916,52.145301998000036],[4.401231,52.11191199800004],[4.53061150000002,52.08048249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL333","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Delft en Westland","NUTS_NAME":"Delft en Westland"},"geometry":{"type":"Polygon","coordinates":[[[4.407886500000075,51.968177997000055],[4.292683500000066,51.921279999000035],[4.127788500000065,52.00054249600004],[4.198018500000046,52.05414449600005],[4.373484500000075,52.03261949600005],[4.407886500000075,51.968177997000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL337","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Agglomeratie Leiden en Bollenstreek","NUTS_NAME":"Agglomeratie Leiden en Bollenstreek"},"geometry":{"type":"Polygon","coordinates":[[[4.611684,52.31358699800006],[4.550627,52.22700499700005],[4.728766,52.20981949800006],[4.53061150000002,52.08048249800004],[4.401231,52.11191199800004],[4.442916,52.145301998000036],[4.374232500000062,52.187089996000054],[4.493847500000072,52.32826],[4.611684,52.31358699800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL338","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Oost-Zuid-Holland","NUTS_NAME":"Oost-Zuid-Holland"},"geometry":{"type":"Polygon","coordinates":[[[4.877899500000069,51.93783549700004],[4.743652,51.90557849800007],[4.557198500000027,52.06123750000006],[4.53061150000002,52.08048249800004],[4.728766,52.20981949800006],[4.794524,52.22672649800006],[4.892281,52.16181199600004],[4.792076,52.122359998000036],[4.829823,52.06685249600008],[4.798712500000022,52.022167],[4.877899500000069,51.93783549700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL339","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Groot-Rijnmond","NUTS_NAME":"Groot-Rijnmond"},"geometry":{"type":"Polygon","coordinates":[[[4.743652,51.90557849800007],[4.55341899900003,51.82999049700004],[4.621221,51.801089998000066],[4.620420500000023,51.71412649900003],[4.249017,51.64580149900007],[4.161857001000044,51.66683157600005],[4.135041090000072,51.67330369700005],[4.047814500000072,51.69436249800003],[4.013337,51.759010500000045],[3.839083,51.75829699900004],[3.866527500000075,51.81832449800004],[4.067505500000038,51.84529999700004],[4.051014500000065,51.919221496000034],[3.980849,51.925444],[3.983133,51.98452499800004],[4.127788500000065,52.00054249600004],[4.292683500000066,51.921279999000035],[4.407886500000075,51.968177997000055],[4.557198500000027,52.06123750000006],[4.743652,51.90557849800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL33A","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidoost-Zuid-Holland","NUTS_NAME":"Zuidoost-Zuid-Holland"},"geometry":{"type":"Polygon","coordinates":[[[5.149456,51.933452498000065],[5.000534,51.82093799800003],[4.67629450000004,51.72491849900007],[4.620420500000023,51.71412649900003],[4.621221,51.801089998000066],[4.55341899900003,51.82999049700004],[4.743652,51.90557849800007],[4.877899500000069,51.93783549700004],[5.149456,51.933452498000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL34","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Zeeland","NUTS_NAME":"Zeeland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.190816999000049,51.59738549800005],[4.279565,51.37601749600003],[4.243669500000067,51.37472949800008],[3.9754,51.46165099900003],[3.815721999000061,51.38460149900004],[3.677822,51.44882949700008],[3.71355,51.470973997000044],[3.574728,51.43867849700007],[3.4342,51.52615749800003],[3.839144500000032,51.605071999000074],[3.899378,51.56350749600006],[3.869229,51.54168799800004],[4.223578,51.43865599600008],[4.164783500000055,51.52251049700004],[3.982944500000031,51.58132999900005],[4.193678,51.635616998000046],[4.161857001000044,51.66683157600005],[4.249017,51.64580149900007],[4.190816999000049,51.59738549800005]]],[[[4.234817500000077,51.348253999000065],[3.977666500000055,51.22513199800005],[3.856339500000047,51.21105599900005],[3.621433500000023,51.292781998000066],[3.380661,51.274299496000026],[3.36721650000004,51.368134498000074],[3.547133500000029,51.404273998000065],[3.818089,51.33777249600007],[3.978855500000066,51.40676499800003],[4.234817500000077,51.348253999000065]]],[[[4.135041090000072,51.67330369700005],[3.968077500000049,51.61403999600003],[3.678739,51.695324],[3.839083,51.75829699900004],[4.013337,51.759010500000045],[4.047814500000072,51.69436249800003],[4.135041090000072,51.67330369700005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL341","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zeeuwsch-Vlaanderen","NUTS_NAME":"Zeeuwsch-Vlaanderen"},"geometry":{"type":"Polygon","coordinates":[[[4.234817500000077,51.348253999000065],[3.977666500000055,51.22513199800005],[3.856339500000047,51.21105599900005],[3.621433500000023,51.292781998000066],[3.380661,51.274299496000026],[3.36721650000004,51.368134498000074],[3.547133500000029,51.404273998000065],[3.818089,51.33777249600007],[3.978855500000066,51.40676499800003],[4.234817500000077,51.348253999000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL342","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Overig Zeeland","NUTS_NAME":"Overig Zeeland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[4.190816999000049,51.59738549800005],[4.279565,51.37601749600003],[4.243669500000067,51.37472949800008],[3.9754,51.46165099900003],[3.815721999000061,51.38460149900004],[3.677822,51.44882949700008],[3.71355,51.470973997000044],[3.574728,51.43867849700007],[3.4342,51.52615749800003],[3.839144500000032,51.605071999000074],[3.899378,51.56350749600006],[3.869229,51.54168799800004],[4.223578,51.43865599600008],[4.164783500000055,51.52251049700004],[3.982944500000031,51.58132999900005],[4.193678,51.635616998000046],[4.161857001000044,51.66683157600005],[4.249017,51.64580149900007],[4.190816999000049,51.59738549800005]]],[[[4.135041090000072,51.67330369700005],[3.968077500000049,51.61403999600003],[3.678739,51.695324],[3.839083,51.75829699900004],[4.013337,51.759010500000045],[4.047814500000072,51.69436249800003],[4.135041090000072,51.67330369700005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL4","LEVL_CODE":1,"CNTR_CODE":"NL","NAME_LATN":"ZUID-NEDERLAND","NUTS_NAME":"ZUID-NEDERLAND"},"geometry":{"type":"Polygon","coordinates":[[[5.953192,51.74784599700007],[6.118209500000034,51.65610299800005],[6.091235500000039,51.60590949600004],[6.213423,51.50784149800006],[6.224405,51.364978998000026],[6.072657,51.242587497000045],[6.082150999000021,51.17179899900003],[6.174812,51.18451349900005],[5.877084999000033,51.03210099900008],[5.897198,50.974845498000036],[6.026435,50.983273997000026],[6.018124,50.93459299800003],[6.086947500000065,50.91313499800003],[6.017985,50.84379649700003],[6.020998999000028,50.75429549900008],[5.892073,50.75523749800004],[5.682000499000026,50.757446497000046],[5.687622,50.811923998000054],[5.644038500000022,50.87138349800006],[5.758272499000043,50.95479499900006],[5.766149,51.009235499000056],[5.798274,51.05985349800005],[5.855658500000061,51.14459949900004],[5.566283500000054,51.22083649700005],[5.515765,51.29532449800007],[5.237716500000033,51.261600498000064],[5.071045,51.39342099800007],[5.10218,51.42900499600006],[5.037678500000027,51.47936999800004],[4.927639501000044,51.39542449600003],[4.773122500000056,51.413783496000065],[4.841390500000045,51.48071299900005],[4.759926,51.50246449700006],[4.669544,51.42638399800006],[4.535450500000024,51.422889499000064],[4.548236,51.473228497000036],[4.379466500000035,51.44664],[4.431206,51.363845998000045],[4.279565,51.37601749600003],[4.190816999000049,51.59738549800005],[4.249017,51.64580149900007],[4.620420500000023,51.71412649900003],[4.67629450000004,51.72491849900007],[5.000534,51.82093799800003],[5.128057500000068,51.73760999900003],[5.597879500000033,51.82808299800007],[5.865158500000064,51.757407998000076],[5.953192,51.74784599700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL41","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Noord-Brabant","NUTS_NAME":"Noord-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[5.865158500000064,51.757407998000076],[6.047766500000023,51.558505997000054],[5.838077,51.566467498000065],[5.931195500000058,51.38460499800004],[5.840245500000037,51.346924],[5.672037500000044,51.31501399800004],[5.566283500000054,51.22083649700005],[5.515765,51.29532449800007],[5.237716500000033,51.261600498000064],[5.071045,51.39342099800007],[5.10218,51.42900499600006],[5.037678500000027,51.47936999800004],[4.927639501000044,51.39542449600003],[4.773122500000056,51.413783496000065],[4.841390500000045,51.48071299900005],[4.759926,51.50246449700006],[4.669544,51.42638399800006],[4.535450500000024,51.422889499000064],[4.548236,51.473228497000036],[4.379466500000035,51.44664],[4.431206,51.363845998000045],[4.279565,51.37601749600003],[4.190816999000049,51.59738549800005],[4.249017,51.64580149900007],[4.620420500000023,51.71412649900003],[4.67629450000004,51.72491849900007],[5.000534,51.82093799800003],[5.128057500000068,51.73760999900003],[5.597879500000033,51.82808299800007],[5.865158500000064,51.757407998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL411","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"West-Noord-Brabant","NUTS_NAME":"West-Noord-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[4.759926,51.50246449700006],[4.669544,51.42638399800006],[4.535450500000024,51.422889499000064],[4.548236,51.473228497000036],[4.379466500000035,51.44664],[4.431206,51.363845998000045],[4.279565,51.37601749600003],[4.190816999000049,51.59738549800005],[4.249017,51.64580149900007],[4.620420500000023,51.71412649900003],[4.67629450000004,51.72491849900007],[4.917944500000033,51.719939999000076],[4.91857,51.60147499700008],[4.759926,51.50246449700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL412","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Midden-Noord-Brabant","NUTS_NAME":"Midden-Noord-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[5.128057500000068,51.73760999900003],[5.081756001000031,51.72229],[5.106194500000072,51.63610849600008],[5.273257,51.55282950000003],[5.191001999000036,51.430213998000056],[5.10218,51.42900499600006],[5.037678500000027,51.47936999800004],[4.927639501000044,51.39542449600003],[4.773122500000056,51.413783496000065],[4.841390500000045,51.48071299900005],[4.759926,51.50246449700006],[4.91857,51.60147499700008],[4.917944500000033,51.719939999000076],[4.67629450000004,51.72491849900007],[5.000534,51.82093799800003],[5.128057500000068,51.73760999900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL413","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Noordoost-Noord-Brabant","NUTS_NAME":"Noordoost-Noord-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[5.865158500000064,51.757407998000076],[6.047766500000023,51.558505997000054],[5.838077,51.566467498000065],[5.273257,51.55282950000003],[5.106194500000072,51.63610849600008],[5.081756001000031,51.72229],[5.128057500000068,51.73760999900003],[5.597879500000033,51.82808299800007],[5.865158500000064,51.757407998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL414","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuidoost-Noord-Brabant","NUTS_NAME":"Zuidoost-Noord-Brabant"},"geometry":{"type":"Polygon","coordinates":[[[5.838077,51.566467498000065],[5.931195500000058,51.38460499800004],[5.840245500000037,51.346924],[5.672037500000044,51.31501399800004],[5.566283500000054,51.22083649700005],[5.515765,51.29532449800007],[5.237716500000033,51.261600498000064],[5.071045,51.39342099800007],[5.10218,51.42900499600006],[5.191001999000036,51.430213998000056],[5.273257,51.55282950000003],[5.838077,51.566467498000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL42","LEVL_CODE":2,"CNTR_CODE":"NL","NAME_LATN":"Limburg (NL)","NUTS_NAME":"Limburg (NL)"},"geometry":{"type":"Polygon","coordinates":[[[6.091235500000039,51.60590949600004],[6.213423,51.50784149800006],[6.224405,51.364978998000026],[6.072657,51.242587497000045],[6.082150999000021,51.17179899900003],[6.174812,51.18451349900005],[5.877084999000033,51.03210099900008],[5.897198,50.974845498000036],[6.026435,50.983273997000026],[6.018124,50.93459299800003],[6.086947500000065,50.91313499800003],[6.017985,50.84379649700003],[6.020998999000028,50.75429549900008],[5.892073,50.75523749800004],[5.682000499000026,50.757446497000046],[5.687622,50.811923998000054],[5.644038500000022,50.87138349800006],[5.758272499000043,50.95479499900006],[5.766149,51.009235499000056],[5.798274,51.05985349800005],[5.855658500000061,51.14459949900004],[5.566283500000054,51.22083649700005],[5.672037500000044,51.31501399800004],[5.840245500000037,51.346924],[5.931195500000058,51.38460499800004],[5.838077,51.566467498000065],[6.047766500000023,51.558505997000054],[5.865158500000064,51.757407998000076],[5.953192,51.74784599700007],[6.118209500000034,51.65610299800005],[6.091235500000039,51.60590949600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL421","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Noord-Limburg","NUTS_NAME":"Noord-Limburg"},"geometry":{"type":"Polygon","coordinates":[[[5.953192,51.74784599700007],[6.118209500000034,51.65610299800005],[6.091235500000039,51.60590949600004],[6.213423,51.50784149800006],[6.224405,51.364978998000026],[6.072657,51.242587497000045],[5.840245500000037,51.346924],[5.931195500000058,51.38460499800004],[5.838077,51.566467498000065],[6.047766500000023,51.558505997000054],[5.865158500000064,51.757407998000076],[5.953192,51.74784599700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL422","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Midden-Limburg","NUTS_NAME":"Midden-Limburg"},"geometry":{"type":"Polygon","coordinates":[[[6.072657,51.242587497000045],[6.082150999000021,51.17179899900003],[6.174812,51.18451349900005],[5.877084999000033,51.03210099900008],[5.798274,51.05985349800005],[5.855658500000061,51.14459949900004],[5.566283500000054,51.22083649700005],[5.672037500000044,51.31501399800004],[5.840245500000037,51.346924],[6.072657,51.242587497000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"NL423","LEVL_CODE":3,"CNTR_CODE":"NL","NAME_LATN":"Zuid-Limburg","NUTS_NAME":"Zuid-Limburg"},"geometry":{"type":"Polygon","coordinates":[[[5.798274,51.05985349800005],[5.877084999000033,51.03210099900008],[5.897198,50.974845498000036],[6.026435,50.983273997000026],[6.018124,50.93459299800003],[6.086947500000065,50.91313499800003],[6.017985,50.84379649700003],[6.020998999000028,50.75429549900008],[5.892073,50.75523749800004],[5.682000499000026,50.757446497000046],[5.687622,50.811923998000054],[5.644038500000022,50.87138349800006],[5.758272499000043,50.95479499900006],[5.766149,51.009235499000056],[5.798274,51.05985349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI31","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Pesaro e Urbino","NUTS_NAME":"Pesaro e Urbino"},"geometry":{"type":"Polygon","coordinates":[[[13.172615,43.75034599800006],[12.767457,43.45982999600005],[12.616341500000033,43.42969049700008],[12.480548,43.52743699600006],[12.31281,43.54335999800003],[12.371290500000043,43.611140498000054],[12.213856,43.610824499000046],[12.18552550000004,43.64181099700005],[12.371335500000043,43.70900450000005],[12.283797500000048,43.764898997000046],[12.4177,43.89902999600008],[12.493956,43.91554650000006],[12.669299,43.82322749800005],[12.733553500000028,43.869933497000034],[12.75078,43.97060150000004],[13.172615,43.75034599800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI32","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ancona","NUTS_NAME":"Ancona"},"geometry":{"type":"Polygon","coordinates":[[[13.642096500000036,43.47414249800005],[13.601663,43.459482998000055],[13.655629,43.44727799900005],[13.631521,43.42079149800003],[13.447286500000075,43.444685496000034],[13.335890500000062,43.39145999800007],[13.280791500000078,43.46116549900006],[13.091834500000061,43.43584049800006],[13.01592,43.310744498000076],[12.883348500000068,43.268245],[12.862358,43.21123249900006],[12.749178,43.37848949900007],[12.767457,43.45982999600005],[13.172615,43.75034599800006],[13.530893500000047,43.619984498000065],[13.642096500000036,43.47414249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI33","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Macerata","NUTS_NAME":"Macerata"},"geometry":{"type":"Polygon","coordinates":[[[13.742970499000023,43.29414699800003],[13.600999500000057,43.267770496000026],[13.442206500000054,43.14831349700006],[13.459740500000066,43.02039749700003],[13.338925999000026,43.02873549800006],[13.216139500000054,42.95109399800003],[13.241772,42.891976498000076],[13.235287500000027,42.86766299800007],[13.160879500000021,42.83187449700006],[12.895834500000035,42.964570998000056],[12.862358,43.21123249900006],[12.883348500000068,43.268245],[13.01592,43.310744498000076],[13.091834500000061,43.43584049800006],[13.280791500000078,43.46116549900006],[13.335890500000062,43.39145999800007],[13.447286500000075,43.444685496000034],[13.631521,43.42079149800003],[13.655629,43.44727799900005],[13.601663,43.459482998000055],[13.642096500000036,43.47414249800005],[13.742970499000023,43.29414699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI34","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Ascoli Piceno","NUTS_NAME":"Ascoli Piceno"},"geometry":{"type":"Polygon","coordinates":[[[13.849453,43.066659497000046],[13.91573150000005,42.89457699900004],[13.534506,42.81658549800005],[13.48879,42.73365799700008],[13.357774500000062,42.69406999700004],[13.189226,42.733565498000075],[13.264171,42.808744999000055],[13.235287500000027,42.86766299800007],[13.241772,42.891976498000076],[13.375335499000073,42.89021349600006],[13.401655,42.97296849800006],[13.849453,43.066659497000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI35","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Fermo","NUTS_NAME":"Fermo"},"geometry":{"type":"Polygon","coordinates":[[[13.849453,43.066659497000046],[13.401655,42.97296849800006],[13.375335499000073,42.89021349600006],[13.241772,42.891976498000076],[13.216139500000054,42.95109399800003],[13.338925999000026,43.02873549800006],[13.459740500000066,43.02039749700003],[13.442206500000054,43.14831349700006],[13.600999500000057,43.267770496000026],[13.742970499000023,43.29414699800003],[13.849453,43.066659497000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI4","LEVL_CODE":2,"CNTR_CODE":"IT","NAME_LATN":"Lazio","NUTS_NAME":"Lazio"},"geometry":{"type":"Polygon","coordinates":[[[12.412598,42.478194],[12.413013,42.42688299800005],[12.467339999000046,42.42037899800005],[12.444709,42.39879949800007],[12.570011499000032,42.382892500000025],[12.892744,42.56347599600008],[12.896352,42.61641799800003],[13.144793,42.64605749800006],[13.189226,42.733565498000075],[13.357774500000062,42.69406999700004],[13.409763,42.63902249800003],[13.39403149900005,42.591233998000064],[13.191380500000037,42.58749099900007],[13.178439,42.47931599800006],[13.116188500000021,42.44388449600007],[13.194177,42.38916949800006],[13.153368,42.356970499000056],[13.379911500000048,42.18042199800004],[13.248800500000073,42.127403497000046],[13.087079500000073,42.17784499800007],[13.030574500000057,42.11535299900004],[13.054555500000049,42.01634599700003],[13.296299500000032,41.94858849900004],[13.383656,41.901868998000054],[13.379543500000068,41.817429496000045],[13.573855500000036,41.75432049900007],[13.729585,41.798873497000045],[13.941038499000058,41.68794399600006],[14.026036,41.555293998000025],[13.977928,41.46245399900005],[13.861228,41.41653499800003],[13.873718500000052,41.338294],[13.889761500000077,41.297446499000046],[13.760795500000029,41.22316799600003],[13.622573,41.26081199600003],[13.549584500000037,41.20663449600005],[13.28150850000003,41.29756449800004],[13.067982,41.221978999000044],[12.952066,41.360478998000076],[12.773321500000065,41.41626399800003],[12.61795,41.45189],[12.416086500000063,41.65572399600006],[12.231206,41.741052996000064],[12.14400850000004,41.913842],[11.915473500000076,42.039472],[11.835364,42.029520998000066],[11.733844,42.15805899800006],[11.632385,42.29534649800007],[11.44993850000003,42.37767050000008],[11.488907,42.43935399900005],[11.617744500000072,42.43606149800007],[11.561255,42.51727549700007],[11.578577,42.566193498000075],[11.804794500000071,42.64398299700008],[11.818677499000046,42.74575549700006],[11.746037,42.78574249800005],[11.89498750000007,42.83465349800008],[11.979501,42.76456799700003],[11.941077,42.683648997000034],[12.204354500000022,42.66044299600003],[12.275523,42.515978500000074],[12.412598,42.478194]],[[12.44668850000005,41.90174499700004],[12.458003,41.901485498000056],[12.45582,41.90719749800007],[12.44668850000005,41.90174499700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI41","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Viterbo","NUTS_NAME":"Viterbo"},"geometry":{"type":"Polygon","coordinates":[[[12.444709,42.39879949800007],[12.51963550000005,42.294725],[12.429426,42.18385649700008],[12.412264,42.25742299800004],[12.330476500000032,42.145752998000034],[12.215765500000032,42.18670799800003],[12.002283,42.15844599600007],[11.911396500000023,42.241111997000075],[11.822823500000027,42.14646799800005],[11.733844,42.15805899800006],[11.632385,42.29534649800007],[11.44993850000003,42.37767050000008],[11.488907,42.43935399900005],[11.617744500000072,42.43606149800007],[11.561255,42.51727549700007],[11.578577,42.566193498000075],[11.804794500000071,42.64398299700008],[11.818677499000046,42.74575549700006],[11.746037,42.78574249800005],[11.89498750000007,42.83465349800008],[11.979501,42.76456799700003],[11.941077,42.683648997000034],[12.204354500000022,42.66044299600003],[12.275523,42.515978500000074],[12.412598,42.478194],[12.413013,42.42688299800005],[12.467339999000046,42.42037899800005],[12.444709,42.39879949800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI42","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Rieti","NUTS_NAME":"Rieti"},"geometry":{"type":"Polygon","coordinates":[[[13.357774500000062,42.69406999700004],[13.409763,42.63902249800003],[13.39403149900005,42.591233998000064],[13.191380500000037,42.58749099900007],[13.178439,42.47931599800006],[13.116188500000021,42.44388449600007],[13.194177,42.38916949800006],[13.153368,42.356970499000056],[13.379911500000048,42.18042199800004],[13.248800500000073,42.127403497000046],[13.087079500000073,42.17784499800007],[13.030574500000057,42.11535299900004],[12.645279500000072,42.164390996000066],[12.608330500000022,42.18155750000005],[12.639451999000073,42.23702699800003],[12.605068,42.274298498000064],[12.51963550000005,42.294725],[12.444709,42.39879949800007],[12.570011499000032,42.382892500000025],[12.892744,42.56347599600008],[12.896352,42.61641799800003],[13.144793,42.64605749800006],[13.189226,42.733565498000075],[13.357774500000062,42.69406999700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI43","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Roma","NUTS_NAME":"Roma"},"geometry":{"type":"Polygon","coordinates":[[[13.030574500000057,42.11535299900004],[13.054555500000049,42.01634599700003],[13.296299500000032,41.94858849900004],[13.184175500000038,41.84600349800007],[13.009848,41.82549050000006],[12.99241150000006,41.76858449900004],[13.149572499000044,41.65926649900007],[13.17509,41.583537998000054],[13.103706,41.545005],[12.939094,41.713351],[12.850943,41.71480549800003],[12.755877500000054,41.57270699800006],[12.66180349900003,41.598156],[12.633927500000027,41.67190849800005],[12.530179,41.598249500000065],[12.72118049900007,41.505104998000036],[12.773321500000065,41.41626399800003],[12.61795,41.45189],[12.416086500000063,41.65572399600006],[12.231206,41.741052996000064],[12.14400850000004,41.913842],[11.915473500000076,42.039472],[11.835364,42.029520998000066],[11.733844,42.15805899800006],[11.822823500000027,42.14646799800005],[11.911396500000023,42.241111997000075],[12.002283,42.15844599600007],[12.215765500000032,42.18670799800003],[12.330476500000032,42.145752998000034],[12.412264,42.25742299800004],[12.429426,42.18385649700008],[12.51963550000005,42.294725],[12.605068,42.274298498000064],[12.639451999000073,42.23702699800003],[12.608330500000022,42.18155750000005],[12.645279500000072,42.164390996000066],[13.030574500000057,42.11535299900004]],[[12.45582,41.90719749800007],[12.44668850000005,41.90174499700004],[12.458003,41.901485498000056],[12.45582,41.90719749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI44","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Latina","NUTS_NAME":"Latina"},"geometry":{"type":"Polygon","coordinates":[[[13.17509,41.583537998000054],[13.270973,41.523874498000055],[13.298859500000049,41.47039199900007],[13.254728500000056,41.43252899700008],[13.306523,41.40252299700006],[13.427312,41.397253499000044],[13.45567150000005,41.44933899800003],[13.548216,41.41954849700005],[13.625198500000067,41.31598350000007],[13.701719500000024,41.359260498000026],[13.763046,41.30131449700008],[13.873718500000052,41.338294],[13.889761500000077,41.297446499000046],[13.760795500000029,41.22316799600003],[13.622573,41.26081199600003],[13.549584500000037,41.20663449600005],[13.28150850000003,41.29756449800004],[13.067982,41.221978999000044],[12.952066,41.360478998000076],[12.773321500000065,41.41626399800003],[12.72118049900007,41.505104998000036],[12.530179,41.598249500000065],[12.633927500000027,41.67190849800005],[12.66180349900003,41.598156],[12.755877500000054,41.57270699800006],[12.850943,41.71480549800003],[12.939094,41.713351],[13.103706,41.545005],[13.17509,41.583537998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"ITI45","LEVL_CODE":3,"CNTR_CODE":"IT","NAME_LATN":"Frosinone","NUTS_NAME":"Frosinone"},"geometry":{"type":"Polygon","coordinates":[[[13.941038499000058,41.68794399600006],[14.026036,41.555293998000025],[13.977928,41.46245399900005],[13.861228,41.41653499800003],[13.873718500000052,41.338294],[13.763046,41.30131449700008],[13.701719500000024,41.359260498000026],[13.625198500000067,41.31598350000007],[13.548216,41.41954849700005],[13.45567150000005,41.44933899800003],[13.427312,41.397253499000044],[13.306523,41.40252299700006],[13.254728500000056,41.43252899700008],[13.298859500000049,41.47039199900007],[13.270973,41.523874498000055],[13.17509,41.583537998000054],[13.149572499000044,41.65926649900007],[12.99241150000006,41.76858449900004],[13.009848,41.82549050000006],[13.184175500000038,41.84600349800007],[13.296299500000032,41.94858849900004],[13.383656,41.901868998000054],[13.379543500000068,41.817429496000045],[13.573855500000036,41.75432049900007],[13.729585,41.798873497000045],[13.941038499000058,41.68794399600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"LI","LEVL_CODE":0,"CNTR_CODE":"LI","NAME_LATN":"LIECHTENSTEIN","NUTS_NAME":"LIECHTENSTEIN"},"geometry":{"type":"Polygon","coordinates":[[[9.607078,47.06077449900005],[9.47604750000005,47.05179749800004],[9.520344,47.10014999800006],[9.486198,47.17795649900006],[9.530749,47.270581],[9.620580500000074,47.15164549900004],[9.607078,47.06077449900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"LI0","LEVL_CODE":1,"CNTR_CODE":"LI","NAME_LATN":"LIECHTENSTEIN","NUTS_NAME":"LIECHTENSTEIN"},"geometry":{"type":"Polygon","coordinates":[[[9.620580500000074,47.15164549900004],[9.607078,47.06077449900005],[9.47604750000005,47.05179749800004],[9.520344,47.10014999800006],[9.486198,47.17795649900006],[9.530749,47.270581],[9.620580500000074,47.15164549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LI00","LEVL_CODE":2,"CNTR_CODE":"LI","NAME_LATN":"Liechtenstein","NUTS_NAME":"Liechtenstein"},"geometry":{"type":"Polygon","coordinates":[[[9.620580500000074,47.15164549900004],[9.607078,47.06077449900005],[9.47604750000005,47.05179749800004],[9.520344,47.10014999800006],[9.486198,47.17795649900006],[9.530749,47.270581],[9.620580500000074,47.15164549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LI000","LEVL_CODE":3,"CNTR_CODE":"LI","NAME_LATN":"Liechtenstein","NUTS_NAME":"Liechtenstein"},"geometry":{"type":"Polygon","coordinates":[[[9.620580500000074,47.15164549900004],[9.607078,47.06077449900005],[9.47604750000005,47.05179749800004],[9.520344,47.10014999800006],[9.486198,47.17795649900006],[9.530749,47.270581],[9.620580500000074,47.15164549900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT","LEVL_CODE":0,"CNTR_CODE":"LT","NAME_LATN":"LIETUVA","NUTS_NAME":"LIETUVA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.91819,56.44086499900004],[25.09203750000006,56.186041997000075],[25.672947,56.14948199600008],[25.687702,56.086562997000044],[25.821386500000074,56.052178999000034],[26.046154500000057,55.944105998000055],[26.385134,55.70467849600004],[26.630365,55.68066699800005],[26.623579500000062,55.565641],[26.524342,55.51005750000007],[26.553886,55.388916],[26.43989349900005,55.34859449900006],[26.835622500000056,55.28564849700007],[26.743454,55.254102997000075],[26.67651550000005,55.158772999000064],[26.323387500000024,55.14494149600006],[26.229339,55.10559249800008],[26.271291500000075,55.07756399600004],[26.208658,54.99742649800004],[25.779577500000073,54.85483950000008],[25.733722500000056,54.80223499700003],[25.762614,54.57689449900005],[25.531922500000064,54.34203149800004],[25.685688500000026,54.318358997000075],[25.774569,54.23725899900006],[25.779838,54.160037],[25.530006,54.15841049800008],[25.59022250000004,54.23255849800006],[25.49710950000008,54.309776498000076],[25.214800500000024,54.25603099600005],[25.07084750000007,54.13387849800006],[24.835949500000027,54.149027498000066],[24.793882500000052,54.11539049700008],[24.822516500000063,54.04345699900006],[24.73895550000003,53.96586449700004],[24.62589550000007,54.01048899800003],[24.435035500000026,53.90100299900007],[24.201624500000037,53.969946496000034],[23.51465,53.956559997000056],[23.482689,53.99711999900006],[23.52863750000006,54.06579799800005],[23.485877500000072,54.15324399700006],[23.321498500000075,54.25332599700005],[23.092209,54.298474499000065],[23.010273,54.38280599900003],[22.792095500000073,54.363358996000045],[22.704324,54.467884498000046],[22.683843500000023,54.58593649900007],[22.749024,54.631435999000075],[22.74368437800007,54.72925697200003],[22.74355550000007,54.73161799800005],[22.876243809000073,54.82263057800003],[22.879939500000035,54.825165499000036],[22.846403,54.89729850000003],[22.74843463600007,54.93279726900005],[22.642446500000062,54.97120199700004],[22.588968,55.07025149700007],[22.066402912000058,55.027111999000056],[21.882659155000056,55.09474270700008],[21.65107099900007,55.17998350000005],[21.65061459700007,55.18012854600005],[21.64969377700004,55.18042118500006],[21.432753630000036,55.24936541000005],[21.432520500000066,55.24943949900006],[21.273739482000053,55.245753008000065],[21.264349500000037,55.24553499700005],[21.297724,55.29712149900007],[21.18927750000006,55.34100249900007],[21.246628500000043,55.49004350000007],[21.129503500000055,55.70281349700008],[21.08229,55.72983749800005],[21.064238,56.06913699800003],[21.199444,56.07678999700005],[21.230745,56.16227299700006],[21.579046,56.31965999700003],[21.978209,56.38514299800005],[22.635563500000046,56.368167999000036],[22.920517,56.39914299800006],[23.045231,56.32003399700005],[23.765474,56.373081],[24.1518,56.25334850000007],[24.579554,56.288315],[24.702389,56.40388899800007],[24.91819,56.44086499900004]]],[[[20.986621,55.283702997000034],[20.982104,55.27790699700006],[20.954067,55.28084249900007],[21.089427,55.502812997000035],[21.106483500000024,55.614936998000076],[21.094303,55.72240349600003],[21.13515,55.678510998000036],[21.11057150000005,55.42052799900006],[20.986621,55.283702997000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT0","LEVL_CODE":1,"CNTR_CODE":"LT","NAME_LATN":"LIETUVA","NUTS_NAME":"LIETUVA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.91819,56.44086499900004],[25.09203750000006,56.186041997000075],[25.672947,56.14948199600008],[25.687702,56.086562997000044],[25.821386500000074,56.052178999000034],[26.046154500000057,55.944105998000055],[26.385134,55.70467849600004],[26.630365,55.68066699800005],[26.623579500000062,55.565641],[26.524342,55.51005750000007],[26.553886,55.388916],[26.43989349900005,55.34859449900006],[26.835622500000056,55.28564849700007],[26.743454,55.254102997000075],[26.67651550000005,55.158772999000064],[26.323387500000024,55.14494149600006],[26.229339,55.10559249800008],[26.271291500000075,55.07756399600004],[26.208658,54.99742649800004],[25.779577500000073,54.85483950000008],[25.733722500000056,54.80223499700003],[25.762614,54.57689449900005],[25.531922500000064,54.34203149800004],[25.685688500000026,54.318358997000075],[25.774569,54.23725899900006],[25.779838,54.160037],[25.530006,54.15841049800008],[25.59022250000004,54.23255849800006],[25.49710950000008,54.309776498000076],[25.214800500000024,54.25603099600005],[25.07084750000007,54.13387849800006],[24.835949500000027,54.149027498000066],[24.793882500000052,54.11539049700008],[24.822516500000063,54.04345699900006],[24.73895550000003,53.96586449700004],[24.62589550000007,54.01048899800003],[24.435035500000026,53.90100299900007],[24.201624500000037,53.969946496000034],[23.51465,53.956559997000056],[23.482689,53.99711999900006],[23.52863750000006,54.06579799800005],[23.485877500000072,54.15324399700006],[23.321498500000075,54.25332599700005],[23.092209,54.298474499000065],[23.010273,54.38280599900003],[22.792095500000073,54.363358996000045],[22.704324,54.467884498000046],[22.683843500000023,54.58593649900007],[22.749024,54.631435999000075],[22.74368437800007,54.72925697200003],[22.74355550000007,54.73161799800005],[22.876243809000073,54.82263057800003],[22.879939500000035,54.825165499000036],[22.846403,54.89729850000003],[22.74843463600007,54.93279726900005],[22.642446500000062,54.97120199700004],[22.588968,55.07025149700007],[22.066402912000058,55.027111999000056],[21.882659155000056,55.09474270700008],[21.65107099900007,55.17998350000005],[21.65061459700007,55.18012854600005],[21.64969377700004,55.18042118500006],[21.432753630000036,55.24936541000005],[21.432520500000066,55.24943949900006],[21.273739482000053,55.245753008000065],[21.264349500000037,55.24553499700005],[21.297724,55.29712149900007],[21.18927750000006,55.34100249900007],[21.246628500000043,55.49004350000007],[21.129503500000055,55.70281349700008],[21.08229,55.72983749800005],[21.064238,56.06913699800003],[21.199444,56.07678999700005],[21.230745,56.16227299700006],[21.579046,56.31965999700003],[21.978209,56.38514299800005],[22.635563500000046,56.368167999000036],[22.920517,56.39914299800006],[23.045231,56.32003399700005],[23.765474,56.373081],[24.1518,56.25334850000007],[24.579554,56.288315],[24.702389,56.40388899800007],[24.91819,56.44086499900004]]],[[[20.986621,55.283702997000034],[20.982104,55.27790699700006],[20.954067,55.28084249900007],[21.089427,55.502812997000035],[21.106483500000024,55.614936998000076],[21.094303,55.72240349600003],[21.13515,55.678510998000036],[21.11057150000005,55.42052799900006],[20.986621,55.283702997000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT00","LEVL_CODE":2,"CNTR_CODE":"LT","NAME_LATN":"Lietuva","NUTS_NAME":"Lietuva"},"geometry":{"type":"MultiPolygon","coordinates":[[[[24.91819,56.44086499900004],[25.09203750000006,56.186041997000075],[25.672947,56.14948199600008],[25.687702,56.086562997000044],[25.821386500000074,56.052178999000034],[26.046154500000057,55.944105998000055],[26.385134,55.70467849600004],[26.630365,55.68066699800005],[26.623579500000062,55.565641],[26.524342,55.51005750000007],[26.553886,55.388916],[26.43989349900005,55.34859449900006],[26.835622500000056,55.28564849700007],[26.743454,55.254102997000075],[26.67651550000005,55.158772999000064],[26.323387500000024,55.14494149600006],[26.229339,55.10559249800008],[26.271291500000075,55.07756399600004],[26.208658,54.99742649800004],[25.779577500000073,54.85483950000008],[25.733722500000056,54.80223499700003],[25.762614,54.57689449900005],[25.531922500000064,54.34203149800004],[25.685688500000026,54.318358997000075],[25.774569,54.23725899900006],[25.779838,54.160037],[25.530006,54.15841049800008],[25.59022250000004,54.23255849800006],[25.49710950000008,54.309776498000076],[25.214800500000024,54.25603099600005],[25.07084750000007,54.13387849800006],[24.835949500000027,54.149027498000066],[24.793882500000052,54.11539049700008],[24.822516500000063,54.04345699900006],[24.73895550000003,53.96586449700004],[24.62589550000007,54.01048899800003],[24.435035500000026,53.90100299900007],[24.201624500000037,53.969946496000034],[23.51465,53.956559997000056],[23.482689,53.99711999900006],[23.52863750000006,54.06579799800005],[23.485877500000072,54.15324399700006],[23.321498500000075,54.25332599700005],[23.092209,54.298474499000065],[23.010273,54.38280599900003],[22.792095500000073,54.363358996000045],[22.704324,54.467884498000046],[22.683843500000023,54.58593649900007],[22.749024,54.631435999000075],[22.74368437800007,54.72925697200003],[22.74355550000007,54.73161799800005],[22.876243809000073,54.82263057800003],[22.879939500000035,54.825165499000036],[22.846403,54.89729850000003],[22.74843463600007,54.93279726900005],[22.642446500000062,54.97120199700004],[22.588968,55.07025149700007],[22.066402912000058,55.027111999000056],[21.882659155000056,55.09474270700008],[21.65107099900007,55.17998350000005],[21.65061459700007,55.18012854600005],[21.64969377700004,55.18042118500006],[21.432753630000036,55.24936541000005],[21.432520500000066,55.24943949900006],[21.273739482000053,55.245753008000065],[21.264349500000037,55.24553499700005],[21.297724,55.29712149900007],[21.18927750000006,55.34100249900007],[21.246628500000043,55.49004350000007],[21.129503500000055,55.70281349700008],[21.08229,55.72983749800005],[21.064238,56.06913699800003],[21.199444,56.07678999700005],[21.230745,56.16227299700006],[21.579046,56.31965999700003],[21.978209,56.38514299800005],[22.635563500000046,56.368167999000036],[22.920517,56.39914299800006],[23.045231,56.32003399700005],[23.765474,56.373081],[24.1518,56.25334850000007],[24.579554,56.288315],[24.702389,56.40388899800007],[24.91819,56.44086499900004]]],[[[20.986621,55.283702997000034],[20.982104,55.27790699700006],[20.954067,55.28084249900007],[21.089427,55.502812997000035],[21.106483500000024,55.614936998000076],[21.094303,55.72240349600003],[21.13515,55.678510998000036],[21.11057150000005,55.42052799900006],[20.986621,55.283702997000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT001","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Alytaus apskritis","NUTS_NAME":"Alytaus apskritis"},"geometry":{"type":"Polygon","coordinates":[[[24.414311,54.53326899800004],[24.438120500000025,54.42777749900006],[25.01897450000007,54.441532],[24.91258650000003,54.361139],[24.933917500000064,54.294920998000066],[24.756055,54.22396049900004],[24.835949500000027,54.149027498000066],[24.793882500000052,54.11539049700008],[24.822516500000063,54.04345699900006],[24.73895550000003,53.96586449700004],[24.62589550000007,54.01048899800003],[24.435035500000026,53.90100299900007],[24.201624500000037,53.969946496000034],[23.51465,53.956559997000056],[23.482689,53.99711999900006],[23.52863750000006,54.06579799800005],[23.485877500000072,54.15324399700006],[23.321498500000075,54.25332599700005],[23.347492,54.346874498000034],[23.444794500000057,54.365841500000045],[23.51009650000003,54.46554649800004],[23.755408500000044,54.468732],[24.029867,54.55080799700005],[24.414311,54.53326899800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT002","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Kauno apskritis","NUTS_NAME":"Kauno apskritis"},"geometry":{"type":"Polygon","coordinates":[[[23.879444,55.55816249600008],[24.038382,55.44357749900007],[24.42978850000003,55.39088649800004],[24.391269500000078,55.29687849900006],[24.646419,55.04964999800006],[24.55363150000005,54.98992449800005],[24.811022500000036,54.90341749700008],[24.761334500000032,54.82327699900003],[24.619247,54.829968],[24.503864500000077,54.729119998000044],[24.414311,54.53326899800004],[24.029867,54.55080799700005],[23.755408500000044,54.468732],[23.79371050000003,54.50312950000006],[23.65815199900004,54.58269750000005],[23.625283500000023,54.84023549700004],[23.394117,54.94859550000007],[23.584982500000024,55.03265299900005],[23.41193750000008,55.073508996000044],[23.504275,55.193331999000065],[22.996637500000077,55.25330549700004],[22.98098550000003,55.31412149600004],[22.81870250000003,55.32182899700007],[22.675759,55.471065499000076],[23.080698,55.503066999000055],[23.097397,55.564629],[23.571644500000048,55.47304199700005],[23.879444,55.55816249600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT003","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Klaipėdos apskritis","NUTS_NAME":"Klaipėdos apskritis"},"geometry":{"type":"MultiPolygon","coordinates":[[[[21.854010500000072,55.277349997000044],[21.86641050000003,55.20315049800007],[21.65107099900007,55.17998350000005],[21.65061459700007,55.18012854600005],[21.64969377700004,55.18042118500006],[21.432753630000036,55.24936541000005],[21.432520500000066,55.24943949900006],[21.273739482000053,55.245753008000065],[21.264349500000037,55.24553499700005],[21.297724,55.29712149900007],[21.18927750000006,55.34100249900007],[21.246628500000043,55.49004350000007],[21.129503500000055,55.70281349700008],[21.08229,55.72983749800005],[21.064238,56.06913699800003],[21.199444,56.07678999700005],[21.230745,56.16227299700006],[21.579046,56.31965999700003],[21.978209,56.38514299800005],[21.932097500000054,56.318853500000046],[22.01329450000003,56.19609199800004],[21.70398,56.07721],[21.512083500000074,55.80882749700004],[21.729068,55.74981799700004],[21.879833500000075,55.59701799900006],[21.808149,55.54883549700003],[21.871759,55.42450899900007],[22.020298,55.38490349800003],[21.854010500000072,55.277349997000044]]],[[[20.986621,55.283702997000034],[20.982104,55.27790699700006],[20.954067,55.28084249900007],[21.089427,55.502812997000035],[21.106483500000024,55.614936998000076],[21.094303,55.72240349600003],[21.13515,55.678510998000036],[21.11057150000005,55.42052799900006],[20.986621,55.283702997000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"LT004","LEVL_CODE":3,"CNTR_CODE":"LT","NAME_LATN":"Marijampolės apskritis","NUTS_NAME":"Marijampolės apskritis"},"geometry":{"type":"Polygon","coordinates":[[[23.755408500000044,54.468732],[23.51009650000003,54.46554649800004],[23.444794500000057,54.365841500000045],[23.347492,54.346874498000034],[23.321498500000075,54.25332599700005],[23.092209,54.298474499000065],[23.010273,54.38280599900003],[22.792095500000073,54.363358996000045],[22.704324,54.467884498000046],[22.683843500000023,54.58593649900007],[22.749024,54.631435999000075],[22.74368437800007,54.72925697200003],[22.74355550000007,54.73161799800005],[22.876243809000073,54.82263057800003],[22.879939500000035,54.825165499000036],[22.846403,54.89729850000003],[22.74843463600007,54.93279726900005],[22.642446500000062,54.97120199700004],[22.588968,55.07025149700007],[23.41193750000008,55.073508996000044],[23.584982500000024,55.03265299900005],[23.394117,54.94859550000007],[23.625283500000023,54.84023549700004],[23.65815199900004,54.58269750000005],[23.79371050000003,54.50312950000006],[23.755408500000044,54.468732]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO07","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Nord-Norge","NUTS_NAME":"Nord-Norge"},"geometry":{"type":"MultiPolygon","coordinates":[[[[28.73865071000006,70.15556738900005],[28.575159,70.11865250000005],[29.660585500000025,69.96182999900003],[29.350151,69.86261749900007],[30.030296500000077,69.82659149800008],[29.699119499000062,69.75655349900006],[29.767906536000055,69.82349093300007],[29.459511,69.648964],[30.076742500000023,69.76901999900008],[29.896385,69.70440649900007],[29.951505500000053,69.58993549900003],[29.916139500000043,69.69289399800005],[30.199936,69.77079749800004],[30.130657,69.852539],[30.185869,69.879448],[30.370466,69.844124],[30.421198,69.689911],[30.293487,69.66351299800004],[30.452156,69.680496],[30.43191550000006,69.81462799900004],[30.839926,69.775808],[30.95453850000007,69.63243199900006],[30.813123,69.52871399900005],[30.08532450000007,69.65810450000004],[30.188423500000056,69.56845749900003],[30.00879450000008,69.415908],[29.288473,69.29618099800007],[29.313494500000047,69.22850299800007],[29.242261,69.11306250000007],[28.92968,69.05190499900004],[28.805792500000052,69.11114699900008],[28.831907,69.22435249800003],[29.336974,69.47831049900003],[29.13434250000006,69.69532949900008],[28.404522,69.81841799900008],[27.984522,70.01396549800006],[27.959615,70.09209449800005],[27.037634500000024,69.91038799900008],[26.490588,69.94501949900007],[25.950607,69.69655049900007],[25.89156,69.66550199800008],[25.976791500000047,69.61067350000008],[25.702101,69.25366150000008],[25.777502500000026,69.018279],[25.157167,68.79989199900007],[25.116428,68.63958749900007],[24.903199500000028,68.55459099900008],[23.871476,68.83651899800003],[23.167591,68.62851849800006],[22.374526,68.71666699900004],[22.340785,68.82722549900006],[21.983611500000052,69.07289349900003],[21.627087500000073,69.27658799800008],[21.27882150000005,69.31188399800004],[20.987576500000046,69.191927],[21.108676,69.10392949900006],[21.057545,69.03628949800003],[20.548636500000043,69.05996850000008],[20.060047500000053,69.045759],[20.306589,68.92618199900005],[20.33587250000005,68.80231249900004],[19.93750250000005,68.55793949900004],[20.226541500000053,68.49081250000006],[19.921397,68.356013],[18.405688500000053,68.58187699900003],[18.125925,68.53651599800008],[18.101093001000038,68.406045],[18.151354,68.19878999900004],[17.899761500000068,67.96937199900003],[17.281524,68.11881499900005],[16.738120500000036,67.91420949800005],[16.407672,67.53409999900003],[16.15800150000007,67.519159],[16.089829,67.43527850000004],[16.404099501000076,67.20497449900006],[16.387759,67.04546249900005],[15.377226500000063,66.48430399600005],[15.453994500000022,66.34523350000006],[15.484732,66.28245849800004],[14.516289,66.13257899900003],[14.625476500000048,65.81180849900005],[14.49877250000003,65.52129749600005],[14.50683250000003,65.30972899800008],[14.378797500000076,65.24762049800006],[14.325985,65.118916],[13.085395500000061,65.10996799900005],[12.768324500000062,64.940775],[12.182329,64.98138099900007],[12.214979,65.01567099900006],[12.171146500000077,65.04820999800006],[12.460707500000069,65.07824699900004],[12.495921500000065,65.105568],[12.399293,65.098755],[12.964295500000048,65.32212049900005],[12.474547500000028,65.13399499900004],[12.350988500000028,65.15914899900008],[12.510954,65.19638799800003],[12.336974,65.23342099800004],[12.46169,65.25754549900006],[12.31042749900007,65.25910949900003],[12.471449500000062,65.37045849900005],[12.041082500000073,65.21619399900004],[12.140350500000068,65.28169249900003],[12.085761,65.30515299900003],[12.146955499000057,65.37075799700006],[12.276143,65.39269249800003],[12.158635,65.38862599800007],[12.353930500000047,65.46737649800008],[12.183487999000022,65.44551849900006],[12.292716,65.58846299700008],[12.630449,65.41567249600007],[12.781976500000042,65.46252449900004],[12.401527500000043,65.537445],[12.347975,65.62732699900005],[12.494364,65.73480999900005],[12.630283314000053,65.69639763800006],[12.537199,65.75340999900004],[12.646986,65.81443799600004],[12.718809,65.77714549900003],[12.72851,65.80428299800008],[12.597691500000053,65.82546249900008],[12.666879418000065,65.92070801000006],[12.380134500000054,65.89411099900008],[13.053273500000046,66.07566850000006],[12.657043500000043,66.04336549900006],[12.830141,66.12840199900006],[13.346362,66.22772149900004],[13.609442500000057,66.20105749800007],[13.548763,66.10309549800007],[13.819278,66.137657],[13.599074500000029,66.22496049800003],[14.147304500000075,66.33046699900007],[13.009953500000051,66.19128399800007],[13.562423500000023,66.30467999600006],[13.027488,66.32476049900004],[12.990386999000066,66.34877799900005],[13.159712500000069,66.40702049800007],[13.172011500000053,66.46587349900005],[12.978532,66.529541],[13.256189500000062,66.446121],[13.425744,66.47429649800006],[13.262984500000073,66.52902149900007],[13.672662,66.51731099900007],[13.470007,66.565529],[13.733414500000038,66.60917650000005],[13.185444,66.56871799900006],[13.413904,66.62222299900003],[13.16772,66.65842449900003],[13.566408,66.64874249800005],[13.207289500000059,66.71688099900007],[14.000651,66.79788950000005],[13.610096,66.84452799900004],[13.68425047900007,66.86556359300005],[13.492176,66.94919649900004],[14.013185500000077,66.96194450000007],[14.06197050000003,66.99144749900006],[14.019997500000045,67.04334999900004],[14.052112500000021,67.08169550000008],[14.173709975000065,67.01579152300008],[14.124613,67.07179250000007],[14.008136,67.09213299900006],[14.067618500000037,67.16149150000007],[14.302209,67.18673699800007],[14.216777,66.99249250000008],[14.42914905200007,67.08494146600003],[14.297962,67.10489649800007],[14.342555,67.17453],[14.602729091000072,67.19427639000003],[14.511739500000033,67.20764899900007],[15.006282,67.23958599900004],[14.88538550000004,67.16931899800005],[14.968609,67.11983499800004],[14.999890500000049,67.12169649800006],[14.917812499000036,67.16667199800008],[15.088397999000051,67.23653399900007],[15.433605,67.10498049900008],[15.481187,67.18446349900006],[15.349717,67.22185449900007],[15.430528500000037,67.26048299900003],[15.037823500000059,67.27246150000008],[15.162948499000038,67.32976499900008],[14.308716,67.26377099900003],[14.637446500000067,67.43815649800007],[14.943926,67.44770749900005],[14.651400500000022,67.47698199900003],[15.001051811000025,67.57711863800006],[15.080581499000061,67.52973949800008],[14.923469500000067,67.48969249900006],[15.4537,67.49208049800006],[15.582617,67.37529749900006],[15.633104446000061,67.41328633800003],[15.515371499000025,67.44784549800005],[15.650709213000027,67.53825820800006],[15.160784500000034,67.63269799900007],[15.339144,67.69324499800007],[15.247291500000074,67.702095],[15.311404,67.73804449900007],[15.654401,67.68859849900008],[15.428495500000054,67.75768299800006],[15.855372500000044,67.70989999900007],[15.43736750000005,67.809578],[14.734998,67.67552199900007],[15.14555050000007,67.76268799800005],[14.75716,67.80435949900004],[15.136513500000035,67.868988],[14.82857799900006,67.860016],[14.889670500000022,67.88421649900005],[16.00116550000007,67.99621600200004],[15.526616,68.06521600100007],[15.395943500000044,68.03652199800007],[15.440778500000022,68.02494799900006],[15.414783500000055,67.99591049900005],[15.275926500000025,68.05061299900007],[15.406996500000048,68.14228050200006],[15.67317,68.18434150000007],[15.725988500000028,68.11899549600008],[15.597519,68.11167149900007],[15.74319950000006,68.087616],[16.062347499000055,68.255936],[16.089401,68.20642099900004],[16.019762,68.18904099800005],[16.140629,68.18688199900004],[16.074756500000035,68.05342850000005],[16.22637750000007,67.96387499900004],[16.203220500000043,67.898308],[16.49381850000003,67.81392649800006],[16.237539500000025,67.902992],[16.282963,67.98599950000005],[16.189962500000036,68.02142349900004],[16.284780500000068,68.026779],[16.459911500000032,67.91499349900005],[16.316387,68.027191],[16.52378250000004,67.96525550000007],[16.354823500000066,68.04674549800006],[16.740179,68.08623499900006],[16.339367,68.094261],[16.24048250000004,68.137703],[16.60742,68.18205249800008],[16.140442500000063,68.29213550000003],[16.819980500000042,68.15521999800006],[16.27568050000002,68.37313849900005],[17.170841,68.393112],[17.33622,68.28037999800006],[17.318298500000026,68.19080349900008],[17.407579500000054,68.26064299800004],[17.249164,68.394539],[17.900738,68.41780799900005],[17.471044500000062,68.46656049900008],[17.573683,68.53850549900005],[17.54451,68.55082699800005],[17.211914,68.45962499800004],[17.01371,68.45533750000004],[17.105293500000073,68.484131],[16.995945,68.52622999900007],[16.557794,68.44740249900008],[16.501896,68.44715099800004],[16.530721500000027,68.48244499800006],[16.48231019900004,68.50168901000006],[16.515053,68.46984099800005],[16.075683500000025,68.414055],[16.232881500000076,68.54445649800005],[16.579589500000054,68.54136699900005],[16.628774500000077,68.53780349900006],[16.564274500000067,68.58321399900007],[16.614227500000027,68.64302099800005],[16.964586500000053,68.71334099900008],[17.685274,68.67411049900005],[17.251898,68.75971199800006],[17.805559,68.74999249900003],[17.474293,68.83225999900003],[17.853133999000022,68.86698899900006],[17.447077,68.90790550000008],[17.696062,69.08660849800003],[17.636711,69.10508749900004],[18.166633499000056,69.15019999900005],[17.96954549900005,69.22856150000007],[18.23868550000003,69.48264299800007],[18.499069,69.367111],[18.474834500000043,69.20949549900007],[18.542671,69.32290649800007],[19.00698650000004,69.28672049900007],[18.442138500000056,69.49892449900005],[18.828508500000055,69.56273650000003],[18.987579500000038,69.38976299800004],[19.349617,69.31079849800005],[19.231395500000076,69.235595],[19.547487500000045,69.21487449800003],[19.351875500000062,69.36683649800005],[19.073311,69.392891],[18.96809950000005,69.46321849900005],[19.07519426600004,69.54529346400005],[18.934316500000023,69.599945],[19.27431850000005,69.77920549900006],[19.732448500000032,69.80428249900007],[19.672811500000023,69.48411549900004],[19.437654500000065,69.39199049900003],[19.508491500000048,69.39473699900003],[19.732485,69.48689249800003],[19.73015,69.607582],[20.14599050000004,69.58264149900003],[19.76941850000003,69.67984749800007],[20.134111500000074,69.93837],[20.394979500000034,69.89038799800005],[20.25130850000005,69.68190749900003],[20.349264,69.62082649800004],[19.900448,69.27372],[19.930265500000075,69.26666250000005],[20.268589,69.396126],[20.398016,69.58557149800004],[20.821378500000037,69.49770350000006],[20.4823,69.61501299800005],[20.525961,69.72260299800007],[20.459503,69.76207749800005],[20.710785,69.79328899900008],[20.791124500000024,69.84407799800005],[20.749731,69.86638650000003],[21.027044500000045,69.95645150000007],[21.082113500000048,69.95459749800006],[20.844524500000034,69.85420250000004],[20.983516500000064,69.77506249800007],[21.325062,69.90923299800005],[21.18428250000005,69.93567649900007],[21.241577,70.00890349900004],[22.140845,69.74519350000003],[21.88821050000007,69.835617],[22.026851500000078,69.84824349900003],[21.862396,69.93569949900007],[22.091429,70.02889999800004],[21.77614,70.03570499800003],[22.06965850000006,70.12821199900003],[21.727436,70.05191049900003],[21.19089,70.22124049900003],[21.424858,70.222824],[21.478477276000035,70.20036185200007],[21.493496359000062,70.19927205500005],[21.47370150000006,70.29925549900008],[21.56427,70.323387],[21.832088500000054,70.16875450000003],[21.78227050000004,70.25872049900005],[21.983658,70.32945249800008],[22.011963,70.31783299800003],[21.948728500000072,70.28130349800006],[22.298141500000042,70.27687850000007],[22.269542500000057,70.13492599800003],[22.541012,70.113289],[22.290935500000046,70.16215499800006],[22.349849499000072,70.25868299800004],[22.96682,70.189369],[22.848073456000066,70.11755874200003],[23.002601500000026,70.13343799900008],[22.952436499000044,70.040924],[23.142261500000075,69.97520449800004],[23.00179650000007,69.91750349800003],[23.407093,69.96468349900005],[23.52833,70.01591499900007],[23.132837,70.08557150000007],[23.390547,70.12302399900005],[23.179638,70.21624750000007],[23.507255500000042,70.24166049900003],[23.246681500000022,70.24819199900003],[23.605201500000078,70.29695149800006],[23.429792500000076,70.31327050000004],[23.802875,70.47930149800004],[24.329584500000067,70.44633499900004],[24.09067550000003,70.54149649800007],[24.298256,70.68613450000004],[24.696213,70.63238549800008],[24.236952,70.83589149900007],[24.708558500000038,70.87100999900008],[24.553296500000044,70.97564699800006],[25.020571,70.96941399900004],[25.048740500000065,70.950287],[24.918836500000054,70.88396450000005],[25.115078,70.93007650000004],[25.232065,70.79688249900005],[25.395689,70.88725299900005],[25.298832,70.95856499800004],[25.338186500000063,70.979912],[25.547411,70.91419199900008],[25.39456,70.855156],[25.44389350000006,70.84555049900007],[25.91175450000003,70.87564849900008],[25.086530500000038,70.51461049800008],[25.241384500000038,70.520706],[25.056989500000043,70.48918150000003],[25.22985650000004,70.46246349900008],[25.039865500000076,70.35273749900006],[25.183798,70.32165549900003],[24.936825,70.23127],[24.912155,70.09609199900007],[25.153381500000023,70.06489549900004],[25.538208,70.31125649900008],[25.382824,70.32743849900004],[25.440464,70.36740049800005],[26.57236850000004,70.95436850000004],[26.70399499900003,70.96577449800003],[26.730856,70.90872950000005],[26.654697499000065,70.83110799900004],[26.733936500000027,70.82489799800004],[26.286564,70.65195449900006],[26.640539,70.646553],[26.475652500000024,70.35486599900008],[26.518347,70.34874749900007],[27.03247650000003,70.48385599800008],[26.951067,70.54679099900005],[27.025823500000058,70.61801150000008],[27.280205,70.58753949800007],[27.06658150000004,70.70458999900006],[27.355761,70.75415799900003],[27.285433,70.83248149900004],[27.702987500000063,70.800621],[27.101142500000037,70.93241899800006],[27.35516399900007,70.94692999900008],[27.200126500000067,71.01464850000008],[27.275039500000048,71.04461650000007],[27.550026,70.95600899800007],[27.574959,71.01279449900005],[27.487985500000036,71.06152349800004],[27.66123,71.126999],[28.548399,70.96723950000006],[28.240961,70.79145799800006],[27.790540500000077,70.79557049800007],[27.83346750000004,70.78317250000003],[27.753228999000044,70.75068649800005],[28.142963500000064,70.73532849800006],[27.63097,70.62425249800003],[28.305256,70.687088],[27.812294,70.486458],[27.855804500000033,70.43014549900005],[28.510006,70.44792999800006],[28.55335050000008,70.50961299800008],[28.397211,70.542122],[28.627989,70.67738350000008],[28.516052,70.72720349900004],[28.96657349900005,70.88494099900004],[29.421539,70.732119],[29.244068,70.670433],[29.27265,70.65177149800007],[30.202276,70.67542249800005],[30.107942500000036,70.64891799900005],[30.333885,70.60903149800004],[29.997783499000036,70.53657549900004],[30.618822,70.55101],[30.58019050000007,70.51128349800007],[30.96376550000008,70.44050599900004],[31.060808,70.28778850000003],[29.90366750000004,70.06065349900007],[28.73865071000006,70.15556738900005]]],[[[26.045212,71.11144250000007],[25.894638,71.010208],[26.225906500000065,71.06076799800007],[25.55792450000007,70.93201449900005],[25.391018,70.99469],[25.545727,71.03257750000006],[25.24226550000003,71.05068999900004],[25.739667999000062,71.07686599900006],[25.528230500000063,71.15545649800004],[25.673977,71.17353799800003],[26.045212,71.11144250000007]]],[[[24.12865849900004,70.60249349800006],[23.773941,70.50978849800003],[23.623031500000025,70.69841750000006],[23.815597500000024,70.72293849900007],[23.781578,70.695778],[24.12865849900004,70.60249349800006]]],[[[23.642444,70.48944849900005],[23.146014500000035,70.27236950000008],[22.815429,70.41147699800007],[23.43688750000007,70.56320949900004],[23.354719,70.617737],[23.56452,70.62199399800005],[23.529517500000054,70.52244549900007],[23.642444,70.48944849900005]]],[[[23.45733850000005,70.76512149900003],[22.798172,70.513825],[22.087011500000074,70.47561649800008],[22.310595,70.60153949900007],[21.928592500000036,70.64286049900005],[22.516258,70.72119899900008],[22.719646500000067,70.61257150000006],[22.769359500000064,70.678589],[22.679491,70.74093649800005],[22.997843,70.667839],[23.11976450000003,70.74383549900006],[23.015039500000057,70.79207599800003],[23.053814,70.81461349900007],[23.50584149900004,70.78686499900004],[23.369827500000042,70.78671249800004],[23.45733850000005,70.76512149900003]]],[[[22.956299,70.32978049900004],[23.009489,70.24874899800005],[22.342153499000062,70.33776850000004],[22.956299,70.32978049900004]]],[[[20.748051,70.09787749800006],[20.800907,70.056259],[20.370167,70.085861],[20.752233500000045,70.23168949800004],[20.80942550000003,70.20028699800008],[20.748051,70.09787749800006]]],[[[20.12632750000006,70.10135650000007],[19.742870500000038,70.056778],[19.50480250000004,70.23216249900008],[19.729519,70.22838599900007],[20.12632750000006,70.10135650000007]]],[[[19.674003500000026,69.995697],[19.12628550000005,69.787239],[18.704648999000028,69.92350749800005],[19.303821499000037,70.08371749800006],[19.269611500000053,70.03578949900003],[19.674003500000026,69.995697]]],[[[19.054355500000042,69.77339149900007],[18.761686500000053,69.67343150000005],[18.72437099900003,69.63607050000007],[18.837930500000027,69.60294349900005],[18.73374749900006,69.55794549900008],[18.006336,69.56680999800005],[18.27369409000005,69.62659492800003],[18.401043,69.63463599900007],[18.187591500000053,69.70993049800006],[18.620876500000065,69.69638049800005],[18.301355500000057,69.76936350000005],[18.639533499000038,69.804634],[18.645475500000032,69.690963],[18.748409500000037,69.68723299800007],[18.678631,69.701866],[18.732897,69.80738849900007],[18.643114,69.86262499900005],[19.054355500000042,69.77339149900007]]],[[[17.482522500000073,69.60085249800005],[17.658531,69.48767099800006],[17.66749,69.59092699900003],[17.880442,69.579735],[17.919758,69.51934799800006],[17.815377999000077,69.45812249900007],[17.966917,69.51940149900008],[18.06773799900003,69.48912049800003],[17.93697,69.42363749800006],[18.080169500000068,69.426903],[17.84903550000007,69.22781349900004],[18.009228,69.19287849800008],[17.961891,69.177925],[17.460866999000075,69.20072899800005],[17.153118,69.01339749900006],[17.029537,69.03549950000007],[17.1658,69.08332799800007],[16.782012999000074,69.05759449900006],[17.211347500000045,69.19654099900004],[16.887241500000073,69.20046249900008],[17.184337500000026,69.28690349800007],[16.896857500000067,69.313469],[17.186859,69.351883],[16.86576850000006,69.376091],[17.49771499900004,69.42509449900007],[17.196076,69.496956],[17.40388850000005,69.48494699900004],[17.310571500000037,69.54144299900008],[17.68260950000007,69.46909350000004],[17.482522500000073,69.60085249800005]]],[[[15.959016500000075,68.88904549800003],[15.813184,68.7472],[16.005573,68.75553849900007],[15.72271150000006,68.536804],[16.076727,68.74844349800003],[16.302828,68.715187],[16.08692,68.82158650000008],[16.123302500000023,68.84416199800006],[16.409397,68.85786449800008],[16.604114500000037,68.79657750000007],[16.594164,68.67980199800007],[16.633566,68.67171499900007],[16.45695199900007,68.55735799800004],[16.208227,68.55595449900005],[15.990130500000078,68.41610699900008],[16.011379,68.412079],[15.86366350000003,68.38358999900004],[15.874511,68.50976549900008],[15.661911,68.31165299900005],[15.326634500000068,68.35961149800005],[15.56163871700005,68.49955121900007],[14.980621499000051,68.27394849900003],[15.319273,68.46930699800004],[15.231805,68.51376350000004],[15.366633500000034,68.50199150000003],[15.177616,68.53307349800008],[15.205324999000027,68.57371499900006],[15.631016500000044,68.617607],[15.438093,68.71696449900003],[15.716285,68.693619],[15.464602500000069,68.753418],[15.682824,68.81733650000007],[15.551854,68.86223599900006],[15.619679500000075,68.95270549900005],[15.959016500000075,68.88904549800003]]],[[[15.601443500000073,68.97971349800008],[15.416072500000041,68.90097799900008],[15.597289,69.11744699900004],[16.179676,69.28639999900008],[15.873977500000024,69.02712249900003],[15.539591,69.00573749800003],[15.601443500000073,68.97971349800008]]],[[[15.317963500000076,68.91722099900005],[15.244818500000065,68.82391349800008],[15.392325500000027,68.84328449800006],[15.253855,68.75589],[15.41872,68.70305649900007],[15.033466500000031,68.57808699900005],[14.785534,68.63922899900007],[15.150149500000055,68.75284599900004],[14.52193950000003,68.60803999800004],[14.36857599900003,68.68449399900004],[14.583309,68.73529050000008],[14.436676,68.76052849900003],[14.549454500000024,68.81937450000004],[14.704345500000045,68.83254249800007],[14.68254,68.76516699900003],[14.703834499000038,68.74841299900004],[14.939134500000023,68.785446],[14.900189500000067,68.84649650000006],[14.968376,68.86647049800007],[15.152725,68.812866],[15.022504,68.94998949800004],[15.128936,69.02320850000007],[15.188699500000041,69.00209099900007],[15.17250450000006,68.931679],[15.317963500000076,68.91722099900005]]],[[[15.165045500000076,68.44262699800004],[14.206444500000032,68.17415599800006],[14.327897,68.33840199800005],[14.509889500000043,68.33101649900004],[14.416385500000047,68.39362349800007],[15.19480150000004,68.47393050000005],[15.165045500000076,68.44262699800004]]],[[[14.169512,68.25863699800004],[13.460458,68.08541099900003],[13.478425,68.20449049800004],[13.661848,68.23725899900006],[13.511562500000025,68.24778],[13.688828500000056,68.30903649800007],[14.272526,68.29914850000006],[14.14168701400007,68.27466567700003],[14.169512,68.25863699800004]]],[[[13.04957950000005,68.03257750000006],[13.187348500000041,67.976677],[13.014825,67.99353799800008],[12.97647,67.94109349800004],[13.08421050000004,67.92895499800005],[12.796074,67.84101850000008],[12.958006,68.02886199900007],[13.19181450000002,68.101097],[13.04957950000005,68.03257750000006]]],[[[12.667357500000037,66.103691],[12.298794500000042,66.01961499800007],[12.589514500000064,66.22785199900005],[12.629095,66.156784],[12.551321,66.13022599900006],[12.667357500000037,66.103691]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL","LEVL_CODE":0,"CNTR_CODE":"PL","NAME_LATN":"POLSKA","NUTS_NAME":"POLSKA"},"geometry":{"type":"Polygon","coordinates":[[[18.35961,54.81719149600008],[18.82896850000003,54.60770799800008],[18.423325500000033,54.78867299800004],[18.39677450000005,54.747280499000055],[18.54172650000004,54.58447999900005],[18.591159,54.430185496000036],[18.950029500000028,54.35831050000007],[19.33136950000005,54.366705498000044],[19.639032,54.45829449800004],[19.648523,54.45333899900004],[19.224721500000044,54.335841998000035],[19.256953,54.27846049900006],[19.398813,54.26036799600007],[19.803768500000047,54.44241049900006],[20.313503,54.402202],[21.559322,54.32250399900005],[22.792095500000073,54.363358996000045],[23.010273,54.38280599900003],[23.092209,54.298474499000065],[23.321498500000075,54.25332599700005],[23.485877500000072,54.15324399700006],[23.52863750000006,54.06579799800005],[23.482689,53.99711999900006],[23.51465,53.956559997000056],[23.588702,53.69592849600008],[23.798979,53.274188],[23.918259,53.15762199800008],[23.872502,53.083870997000076],[23.925377,53.024629],[23.916239,52.90481199900006],[23.93864,52.712916],[23.466628,52.54944799700007],[23.178338,52.28314099700003],[23.653733,52.07249499800008],[23.689023,51.99191],[23.612061,51.91877099800007],[23.641429,51.80080299900004],[23.529021,51.73175099800005],[23.564615,51.535325998000076],[23.617665999000053,51.50761399800007],[23.702295500000048,51.40404099800003],[23.646396500000037,51.29227849800003],[23.709221,51.27764849700003],[23.861774500000024,51.15733350000005],[23.970489500000042,50.95061849600006],[24.145782500000053,50.869376998000064],[23.957708500000024,50.79466599800003],[24.071474,50.720646001000034],[24.098669,50.59952549800005],[24.034557500000062,50.44484350000005],[23.727020500000037,50.388003998000045],[23.547642,50.25160199900006],[22.686149500000056,49.57316549600006],[22.640724,49.52996449900007],[22.700411,49.48843399800006],[22.750946,49.31658549700006],[22.70755,49.174300997000046],[22.86286750000005,49.09999850000003],[22.894087,49.01669299800005],[22.56684,49.08837749800006],[22.034885,49.22246399800008],[21.966094,49.34511199900004],[21.840236,49.391505996000035],[21.39771250000007,49.433793998000056],[21.055269,49.42130599800004],[21.104102499000078,49.376393998000026],[20.923722500000054,49.29623449800005],[20.743115500000044,49.415132999000036],[20.614939500000048,49.41783299800005],[20.322434500000043,49.40184599600008],[20.146103500000038,49.317959],[20.088046,49.17958099800006],[19.883929500000022,49.20417699800004],[19.758416,49.215991],[19.799801,49.295825498000056],[19.790551,49.41113149900008],[19.631736,49.40882099600003],[19.467386500000032,49.61376699900006],[19.260708,49.53122299800003],[19.153403,49.403777],[18.979868,49.395217],[18.972818,49.50261199800008],[18.851551,49.517189],[18.804583,49.67888899900004],[18.627212,49.72232999600004],[18.575724,49.91042299900005],[18.321821,49.915819999000064],[18.035060999000052,50.06577199800006],[17.868675,49.972545996000065],[17.592736,50.16001399900006],[17.758479,50.20656799900007],[17.71840399900003,50.32094999900005],[17.429605,50.254513],[16.907924500000036,50.449454],[16.860915,50.40808099700007],[17.028323,50.22999699800005],[16.86327,50.19812299800003],[16.70553,50.09651149800004],[16.58029,50.14278799700003],[16.548217,50.22978399800007],[16.360988500000076,50.37923649600003],[16.195729,50.432135],[16.443536,50.586257499000055],[16.322517,50.66645299700008],[16.107318,50.66207299800004],[16.024848,50.598625],[15.986371,50.61474699900003],[15.990918,50.683419],[15.860921,50.67491299800008],[15.816207,50.755322],[15.535267500000032,50.779375998000035],[15.370452,50.780894999000054],[15.277071,50.891020999000034],[15.274085,50.97954299700007],[15.032691,51.021315998000034],[14.968252,50.990008999000054],[15.019613,50.96659899900004],[15.00204,50.869024],[14.823362,50.87055049600008],[15.037271,51.243749998000055],[14.974183,51.36394999600003],[14.948293,51.47176899800007],[14.735769,51.52595199900003],[14.729862,51.581776998000066],[14.752694,51.66709599700005],[14.590146,51.82102599800004],[14.693557,51.901287996000065],[14.716716,52.001188],[14.755227,52.070024997000075],[14.682174,52.11468899800008],[14.716044,52.235458],[14.600891500000046,52.27205199800005],[14.534361999000055,52.39500799900003],[14.635373,52.49636849900003],[14.60426,52.53348949800005],[14.639691,52.57157449700003],[14.565063,52.62449699900003],[14.436438,52.67990049800005],[14.127624,52.83166249900006],[14.156692,52.895590998000046],[14.143658,52.96136849900006],[14.357842,53.06766699800005],[14.377268,53.20175199800008],[14.441223,53.25435999900003],[14.412157,53.32963599700008],[14.267542,53.697806496000055],[14.267567084000063,53.697836718000076],[14.302757,53.736885],[14.595727,53.591762499000026],[14.619581500000038,53.64978399800003],[14.539019500000052,53.70261],[14.610576500000036,53.76369099800007],[14.573513,53.84919699800008],[14.270650390000071,53.880213135000076],[14.28143566700004,53.85638231300004],[14.286778500000025,53.84457399900003],[14.336749,53.80701849800005],[14.213077500000054,53.86647949800005],[14.226302,53.92865299700003],[14.767616,54.02293799900008],[14.69569,53.99610899600003],[14.620618148000062,53.85057817500007],[14.757152500000075,53.925437497000075],[14.80588350000005,54.02959049800006],[14.727757500000052,54.022860997000066],[15.388074500000073,54.15890699900007],[16.102422500000046,54.27442849900007],[16.472283,54.51224349900008],[16.699085,54.569247],[17.666607,54.78323149900007],[18.35961,54.81719149600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO071","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Nordland","NUTS_NAME":"Nordland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[15.650709213000027,67.53825820800006],[15.160784500000034,67.63269799900007],[15.339144,67.69324499800007],[15.247291500000074,67.702095],[15.311404,67.73804449900007],[15.654401,67.68859849900008],[15.428495500000054,67.75768299800006],[15.855372500000044,67.70989999900007],[15.43736750000005,67.809578],[14.734998,67.67552199900007],[15.14555050000007,67.76268799800005],[14.75716,67.80435949900004],[15.136513500000035,67.868988],[14.82857799900006,67.860016],[14.889670500000022,67.88421649900005],[16.00116550000007,67.99621600200004],[15.526616,68.06521600100007],[15.395943500000044,68.03652199800007],[15.440778500000022,68.02494799900006],[15.414783500000055,67.99591049900005],[15.275926500000025,68.05061299900007],[15.406996500000048,68.14228050200006],[15.67317,68.18434150000007],[15.725988500000028,68.11899549600008],[15.597519,68.11167149900007],[15.74319950000006,68.087616],[16.062347499000055,68.255936],[16.089401,68.20642099900004],[16.019762,68.18904099800005],[16.140629,68.18688199900004],[16.074756500000035,68.05342850000005],[16.22637750000007,67.96387499900004],[16.203220500000043,67.898308],[16.49381850000003,67.81392649800006],[16.237539500000025,67.902992],[16.282963,67.98599950000005],[16.189962500000036,68.02142349900004],[16.284780500000068,68.026779],[16.459911500000032,67.91499349900005],[16.316387,68.027191],[16.52378250000004,67.96525550000007],[16.354823500000066,68.04674549800006],[16.740179,68.08623499900006],[16.339367,68.094261],[16.24048250000004,68.137703],[16.60742,68.18205249800008],[16.140442500000063,68.29213550000003],[16.819980500000042,68.15521999800006],[16.27568050000002,68.37313849900005],[17.170841,68.393112],[17.33622,68.28037999800006],[17.318298500000026,68.19080349900008],[17.407579500000054,68.26064299800004],[17.249164,68.394539],[17.900738,68.41780799900005],[17.471044500000062,68.46656049900008],[17.573683,68.53850549900005],[17.54451,68.55082699800005],[17.211914,68.45962499800004],[17.01371,68.45533750000004],[17.105293500000073,68.484131],[16.995945,68.52622999900007],[16.557794,68.44740249900008],[16.501896,68.44715099800004],[16.530721500000027,68.48244499800006],[16.48231019900004,68.50168901000006],[16.515053,68.46984099800005],[16.075683500000025,68.414055],[16.232881500000076,68.54445649800005],[16.579589500000054,68.54136699900005],[16.556564,68.52424199800004],[18.02328650000004,68.60845450000005],[18.125925,68.53651599800008],[18.101093001000038,68.406045],[18.151354,68.19878999900004],[17.899761500000068,67.96937199900003],[17.281524,68.11881499900005],[16.738120500000036,67.91420949800005],[16.407672,67.53409999900003],[16.15800150000007,67.519159],[16.089829,67.43527850000004],[16.404099501000076,67.20497449900006],[16.387759,67.04546249900005],[15.377226500000063,66.48430399600005],[15.453994500000022,66.34523350000006],[15.484732,66.28245849800004],[14.516289,66.13257899900003],[14.625476500000048,65.81180849900005],[14.49877250000003,65.52129749600005],[14.50683250000003,65.30972899800008],[14.378797500000076,65.24762049800006],[14.325985,65.118916],[13.085395500000061,65.10996799900005],[12.768324500000062,64.940775],[12.182329,64.98138099900007],[12.214979,65.01567099900006],[12.171146500000077,65.04820999800006],[12.460707500000069,65.07824699900004],[12.495921500000065,65.105568],[12.399293,65.098755],[12.964295500000048,65.32212049900005],[12.474547500000028,65.13399499900004],[12.350988500000028,65.15914899900008],[12.510954,65.19638799800003],[12.336974,65.23342099800004],[12.46169,65.25754549900006],[12.31042749900007,65.25910949900003],[12.471449500000062,65.37045849900005],[12.041082500000073,65.21619399900004],[12.140350500000068,65.28169249900003],[12.085761,65.30515299900003],[12.146955499000057,65.37075799700006],[12.276143,65.39269249800003],[12.158635,65.38862599800007],[12.353930500000047,65.46737649800008],[12.183487999000022,65.44551849900006],[12.292716,65.58846299700008],[12.630449,65.41567249600007],[12.781976500000042,65.46252449900004],[12.401527500000043,65.537445],[12.347975,65.62732699900005],[12.494364,65.73480999900005],[12.630283314000053,65.69639763800006],[12.537199,65.75340999900004],[12.646986,65.81443799600004],[12.718809,65.77714549900003],[12.72851,65.80428299800008],[12.597691500000053,65.82546249900008],[12.666879418000065,65.92070801000006],[12.380134500000054,65.89411099900008],[13.053273500000046,66.07566850000006],[12.657043500000043,66.04336549900006],[12.830141,66.12840199900006],[13.346362,66.22772149900004],[13.609442500000057,66.20105749800007],[13.548763,66.10309549800007],[13.819278,66.137657],[13.599074500000029,66.22496049800003],[14.147304500000075,66.33046699900007],[13.009953500000051,66.19128399800007],[13.562423500000023,66.30467999600006],[13.027488,66.32476049900004],[12.990386999000066,66.34877799900005],[13.159712500000069,66.40702049800007],[13.172011500000053,66.46587349900005],[12.978532,66.529541],[13.256189500000062,66.446121],[13.425744,66.47429649800006],[13.262984500000073,66.52902149900007],[13.672662,66.51731099900007],[13.470007,66.565529],[13.733414500000038,66.60917650000005],[13.185444,66.56871799900006],[13.413904,66.62222299900003],[13.16772,66.65842449900003],[13.566408,66.64874249800005],[13.207289500000059,66.71688099900007],[14.000651,66.79788950000005],[13.610096,66.84452799900004],[13.68425047900007,66.86556359300005],[13.492176,66.94919649900004],[14.013185500000077,66.96194450000007],[14.06197050000003,66.99144749900006],[14.019997500000045,67.04334999900004],[14.052112500000021,67.08169550000008],[14.173709975000065,67.01579152300008],[14.124613,67.07179250000007],[14.008136,67.09213299900006],[14.067618500000037,67.16149150000007],[14.302209,67.18673699800007],[14.216777,66.99249250000008],[14.42914905200007,67.08494146600003],[14.297962,67.10489649800007],[14.342555,67.17453],[14.602729091000072,67.19427639000003],[14.511739500000033,67.20764899900007],[15.006282,67.23958599900004],[14.88538550000004,67.16931899800005],[14.968609,67.11983499800004],[14.999890500000049,67.12169649800006],[14.917812499000036,67.16667199800008],[15.088397999000051,67.23653399900007],[15.433605,67.10498049900008],[15.481187,67.18446349900006],[15.349717,67.22185449900007],[15.430528500000037,67.26048299900003],[15.037823500000059,67.27246150000008],[15.162948499000038,67.32976499900008],[14.308716,67.26377099900003],[14.637446500000067,67.43815649800007],[14.943926,67.44770749900005],[14.651400500000022,67.47698199900003],[15.001051811000025,67.57711863800006],[15.080581499000061,67.52973949800008],[14.923469500000067,67.48969249900006],[15.4537,67.49208049800006],[15.582617,67.37529749900006],[15.633104446000061,67.41328633800003],[15.515371499000025,67.44784549800005],[15.650709213000027,67.53825820800006]]],[[[15.959016500000075,68.88904549800003],[15.813184,68.7472],[16.005573,68.75553849900007],[15.600355,68.56696249900006],[15.65824,68.454875],[15.965886500000067,68.61235150000005],[16.45695199900007,68.55735799800004],[16.208227,68.55595449900005],[15.990130500000078,68.41610699900008],[16.011379,68.412079],[15.86366350000003,68.38358999900004],[15.874511,68.50976549900008],[15.661911,68.31165299900005],[15.326634500000068,68.35961149800005],[15.56163871700005,68.49955121900007],[14.980621499000051,68.27394849900003],[15.319273,68.46930699800004],[15.231805,68.51376350000004],[15.366633500000034,68.50199150000003],[15.177616,68.53307349800008],[15.205324999000027,68.57371499900006],[15.631016500000044,68.617607],[15.438093,68.71696449900003],[15.716285,68.693619],[15.464602500000069,68.753418],[15.682824,68.81733650000007],[15.551854,68.86223599900006],[15.619679500000075,68.95270549900005],[15.959016500000075,68.88904549800003]]],[[[15.601443500000073,68.97971349800008],[15.416072500000041,68.90097799900008],[15.597289,69.11744699900004],[16.179676,69.28639999900008],[15.873977500000024,69.02712249900003],[15.539591,69.00573749800003],[15.601443500000073,68.97971349800008]]],[[[15.317963500000076,68.91722099900005],[15.244818500000065,68.82391349800008],[15.392325500000027,68.84328449800006],[15.253855,68.75589],[15.41872,68.70305649900007],[15.033466500000031,68.57808699900005],[14.785534,68.63922899900007],[15.150149500000055,68.75284599900004],[14.52193950000003,68.60803999800004],[14.36857599900003,68.68449399900004],[14.583309,68.73529050000008],[14.436676,68.76052849900003],[14.549454500000024,68.81937450000004],[14.704345500000045,68.83254249800007],[14.68254,68.76516699900003],[14.703834499000038,68.74841299900004],[14.939134500000023,68.785446],[14.900189500000067,68.84649650000006],[14.968376,68.86647049800007],[15.152725,68.812866],[15.022504,68.94998949800004],[15.128936,69.02320850000007],[15.188699500000041,69.00209099900007],[15.17250450000006,68.931679],[15.317963500000076,68.91722099900005]]],[[[15.165045500000076,68.44262699800004],[14.206444500000032,68.17415599800006],[14.327897,68.33840199800005],[14.509889500000043,68.33101649900004],[14.416385500000047,68.39362349800007],[15.19480150000004,68.47393050000005],[15.165045500000076,68.44262699800004]]],[[[14.169512,68.25863699800004],[13.460458,68.08541099900003],[13.478425,68.20449049800004],[13.661848,68.23725899900006],[13.511562500000025,68.24778],[13.688828500000056,68.30903649800007],[14.272526,68.29914850000006],[14.14168701400007,68.27466567700003],[14.169512,68.25863699800004]]],[[[13.04957950000005,68.03257750000006],[13.187348500000041,67.976677],[13.014825,67.99353799800008],[12.97647,67.94109349800004],[13.08421050000004,67.92895499800005],[12.796074,67.84101850000008],[12.958006,68.02886199900007],[13.19181450000002,68.101097],[13.04957950000005,68.03257750000006]]],[[[12.667357500000037,66.103691],[12.298794500000042,66.01961499800007],[12.589514500000064,66.22785199900005],[12.629095,66.156784],[12.551321,66.13022599900006],[12.667357500000037,66.103691]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO072","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Troms","NUTS_NAME":"Troms"},"geometry":{"type":"MultiPolygon","coordinates":[[[[21.608518,70.17871549800003],[21.829201500000067,70.10079799900006],[22.119250500000078,70.170473],[22.243237500000077,69.96088799800003],[22.52674650000006,69.901041],[22.44857250000007,69.873356],[22.546645500000068,69.84602749900006],[22.451471500000025,69.79070249900008],[22.570415,69.69699099900004],[22.894466,69.64739199900004],[22.722003,69.508474],[22.27318850000006,69.47042449900005],[22.529805500000066,69.23830549900003],[22.478635,69.10992149900005],[21.983611500000052,69.07289349900003],[21.627087500000073,69.27658799800008],[21.27882150000005,69.31188399800004],[20.987576500000046,69.191927],[21.108676,69.10392949900006],[21.057545,69.03628949800003],[20.548636500000043,69.05996850000008],[20.060047500000053,69.045759],[20.306589,68.92618199900005],[20.33587250000005,68.80231249900004],[19.93750250000005,68.55793949900004],[20.226541500000053,68.49081250000006],[19.921397,68.356013],[18.405688500000053,68.58187699900003],[18.125925,68.53651599800008],[18.02328650000004,68.60845450000005],[16.556564,68.52424199800004],[16.579589500000054,68.54136699900005],[16.628774500000077,68.53780349900006],[16.564274500000067,68.58321399900007],[16.614227500000027,68.64302099800005],[16.964586500000053,68.71334099900008],[17.685274,68.67411049900005],[17.251898,68.75971199800006],[17.805559,68.74999249900003],[17.474293,68.83225999900003],[17.853133999000022,68.86698899900006],[17.447077,68.90790550000008],[17.696062,69.08660849800003],[17.636711,69.10508749900004],[18.166633499000056,69.15019999900005],[17.96954549900005,69.22856150000007],[18.23868550000003,69.48264299800007],[18.499069,69.367111],[18.474834500000043,69.20949549900007],[18.542671,69.32290649800007],[19.00698650000004,69.28672049900007],[18.442138500000056,69.49892449900005],[18.828508500000055,69.56273650000003],[18.987579500000038,69.38976299800004],[19.349617,69.31079849800005],[19.231395500000076,69.235595],[19.547487500000045,69.21487449800003],[19.351875500000062,69.36683649800005],[19.073311,69.392891],[18.96809950000005,69.46321849900005],[19.07519426600004,69.54529346400005],[18.934316500000023,69.599945],[19.27431850000005,69.77920549900006],[19.732448500000032,69.80428249900007],[19.672811500000023,69.48411549900004],[19.437654500000065,69.39199049900003],[19.508491500000048,69.39473699900003],[19.732485,69.48689249800003],[19.73015,69.607582],[20.14599050000004,69.58264149900003],[19.76941850000003,69.67984749800007],[20.134111500000074,69.93837],[20.394979500000034,69.89038799800005],[20.25130850000005,69.68190749900003],[20.349264,69.62082649800004],[19.900448,69.27372],[19.930265500000075,69.26666250000005],[20.268589,69.396126],[20.398016,69.58557149800004],[20.821378500000037,69.49770350000006],[20.4823,69.61501299800005],[20.525961,69.72260299800007],[20.459503,69.76207749800005],[20.710785,69.79328899900008],[20.791124500000024,69.84407799800005],[20.749731,69.86638650000003],[21.027044500000045,69.95645150000007],[21.082113500000048,69.95459749800006],[20.844524500000034,69.85420250000004],[20.983516500000064,69.77506249800007],[21.325062,69.90923299800005],[21.18428250000005,69.93567649900007],[21.241577,70.00890349900004],[22.140845,69.74519350000003],[21.88821050000007,69.835617],[22.026851500000078,69.84824349900003],[21.862396,69.93569949900007],[22.091429,70.02889999800004],[21.77614,70.03570499800003],[22.06965850000006,70.12821199900003],[21.727436,70.05191049900003],[21.19089,70.22124049900003],[21.424858,70.222824],[21.478477276000035,70.20036185200007],[21.493496359000062,70.19927205500005],[21.608518,70.17871549800003]]],[[[20.748051,70.09787749800006],[20.800907,70.056259],[20.370167,70.085861],[20.752233500000045,70.23168949800004],[20.80942550000003,70.20028699800008],[20.748051,70.09787749800006]]],[[[20.12632750000006,70.10135650000007],[19.742870500000038,70.056778],[19.50480250000004,70.23216249900008],[19.729519,70.22838599900007],[20.12632750000006,70.10135650000007]]],[[[19.674003500000026,69.995697],[19.12628550000005,69.787239],[18.704648999000028,69.92350749800005],[19.303821499000037,70.08371749800006],[19.269611500000053,70.03578949900003],[19.674003500000026,69.995697]]],[[[19.054355500000042,69.77339149900007],[18.761686500000053,69.67343150000005],[18.72437099900003,69.63607050000007],[18.837930500000027,69.60294349900005],[18.73374749900006,69.55794549900008],[18.006336,69.56680999800005],[18.27369409000005,69.62659492800003],[18.401043,69.63463599900007],[18.187591500000053,69.70993049800006],[18.620876500000065,69.69638049800005],[18.301355500000057,69.76936350000005],[18.639533499000038,69.804634],[18.645475500000032,69.690963],[18.748409500000037,69.68723299800007],[18.678631,69.701866],[18.732897,69.80738849900007],[18.643114,69.86262499900005],[19.054355500000042,69.77339149900007]]],[[[17.482522500000073,69.60085249800005],[17.658531,69.48767099800006],[17.66749,69.59092699900003],[17.880442,69.579735],[17.919758,69.51934799800006],[17.815377999000077,69.45812249900007],[17.966917,69.51940149900008],[18.06773799900003,69.48912049800003],[17.93697,69.42363749800006],[18.080169500000068,69.426903],[17.84903550000007,69.22781349900004],[18.009228,69.19287849800008],[17.961891,69.177925],[17.460866999000075,69.20072899800005],[17.153118,69.01339749900006],[17.029537,69.03549950000007],[17.1658,69.08332799800007],[16.782012999000074,69.05759449900006],[17.211347500000045,69.19654099900004],[16.887241500000073,69.20046249900008],[17.184337500000026,69.28690349800007],[16.896857500000067,69.313469],[17.186859,69.351883],[16.86576850000006,69.376091],[17.49771499900004,69.42509449900007],[17.196076,69.496956],[17.40388850000005,69.48494699900004],[17.310571500000037,69.54144299900008],[17.68260950000007,69.46909350000004],[17.482522500000073,69.60085249800005]]],[[[16.604114500000037,68.79657750000007],[16.594164,68.67980199800007],[16.633566,68.67171499900007],[16.45695199900007,68.55735799800004],[15.965886500000067,68.61235150000005],[15.65824,68.454875],[15.600355,68.56696249900006],[16.005573,68.75553849900007],[15.72271150000006,68.536804],[16.076727,68.74844349800003],[16.302828,68.715187],[16.08692,68.82158650000008],[16.123302500000023,68.84416199800006],[16.409397,68.85786449800008],[16.604114500000037,68.79657750000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO073","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Finnmark","NUTS_NAME":"Finnmark"},"geometry":{"type":"MultiPolygon","coordinates":[[[[28.73865071000006,70.15556738900005],[28.575159,70.11865250000005],[29.660585500000025,69.96182999900003],[29.350151,69.86261749900007],[30.030296500000077,69.82659149800008],[29.699119499000062,69.75655349900006],[29.767906536000055,69.82349093300007],[29.459511,69.648964],[30.076742500000023,69.76901999900008],[29.896385,69.70440649900007],[29.951505500000053,69.58993549900003],[29.916139500000043,69.69289399800005],[30.199936,69.77079749800004],[30.130657,69.852539],[30.185869,69.879448],[30.370466,69.844124],[30.421198,69.689911],[30.293487,69.66351299800004],[30.452156,69.680496],[30.43191550000006,69.81462799900004],[30.839926,69.775808],[30.95453850000007,69.63243199900006],[30.813123,69.52871399900005],[30.08532450000007,69.65810450000004],[30.188423500000056,69.56845749900003],[30.00879450000008,69.415908],[29.288473,69.29618099800007],[29.313494500000047,69.22850299800007],[29.242261,69.11306250000007],[28.92968,69.05190499900004],[28.805792500000052,69.11114699900008],[28.831907,69.22435249800003],[29.336974,69.47831049900003],[29.13434250000006,69.69532949900008],[28.404522,69.81841799900008],[27.984522,70.01396549800006],[27.959615,70.09209449800005],[27.037634500000024,69.91038799900008],[26.490588,69.94501949900007],[25.950607,69.69655049900007],[25.89156,69.66550199800008],[25.976791500000047,69.61067350000008],[25.702101,69.25366150000008],[25.777502500000026,69.018279],[25.157167,68.79989199900007],[25.116428,68.63958749900007],[24.903199500000028,68.55459099900008],[23.871476,68.83651899800003],[23.167591,68.62851849800006],[22.374526,68.71666699900004],[22.340785,68.82722549900006],[21.983611500000052,69.07289349900003],[22.478635,69.10992149900005],[22.529805500000066,69.23830549900003],[22.27318850000006,69.47042449900005],[22.722003,69.508474],[22.894466,69.64739199900004],[22.570415,69.69699099900004],[22.451471500000025,69.79070249900008],[22.546645500000068,69.84602749900006],[22.44857250000007,69.873356],[22.52674650000006,69.901041],[22.243237500000077,69.96088799800003],[22.119250500000078,70.170473],[21.829201500000067,70.10079799900006],[21.608518,70.17871549800003],[21.493496359000062,70.19927205500005],[21.47370150000006,70.29925549900008],[21.56427,70.323387],[21.832088500000054,70.16875450000003],[21.78227050000004,70.25872049900005],[21.983658,70.32945249800008],[22.011963,70.31783299800003],[21.948728500000072,70.28130349800006],[22.298141500000042,70.27687850000007],[22.269542500000057,70.13492599800003],[22.541012,70.113289],[22.290935500000046,70.16215499800006],[22.349849499000072,70.25868299800004],[22.96682,70.189369],[22.848073456000066,70.11755874200003],[23.002601500000026,70.13343799900008],[22.952436499000044,70.040924],[23.142261500000075,69.97520449800004],[23.00179650000007,69.91750349800003],[23.407093,69.96468349900005],[23.52833,70.01591499900007],[23.132837,70.08557150000007],[23.390547,70.12302399900005],[23.179638,70.21624750000007],[23.507255500000042,70.24166049900003],[23.246681500000022,70.24819199900003],[23.605201500000078,70.29695149800006],[23.429792500000076,70.31327050000004],[23.802875,70.47930149800004],[24.329584500000067,70.44633499900004],[24.09067550000003,70.54149649800007],[24.298256,70.68613450000004],[24.696213,70.63238549800008],[24.236952,70.83589149900007],[24.708558500000038,70.87100999900008],[24.553296500000044,70.97564699800006],[25.020571,70.96941399900004],[25.048740500000065,70.950287],[24.918836500000054,70.88396450000005],[25.115078,70.93007650000004],[25.232065,70.79688249900005],[25.395689,70.88725299900005],[25.298832,70.95856499800004],[25.338186500000063,70.979912],[25.547411,70.91419199900008],[25.39456,70.855156],[25.44389350000006,70.84555049900007],[25.91175450000003,70.87564849900008],[25.086530500000038,70.51461049800008],[25.241384500000038,70.520706],[25.056989500000043,70.48918150000003],[25.22985650000004,70.46246349900008],[25.039865500000076,70.35273749900006],[25.183798,70.32165549900003],[24.936825,70.23127],[24.912155,70.09609199900007],[25.153381500000023,70.06489549900004],[25.538208,70.31125649900008],[25.382824,70.32743849900004],[25.440464,70.36740049800005],[26.57236850000004,70.95436850000004],[26.70399499900003,70.96577449800003],[26.730856,70.90872950000005],[26.654697499000065,70.83110799900004],[26.733936500000027,70.82489799800004],[26.286564,70.65195449900006],[26.640539,70.646553],[26.475652500000024,70.35486599900008],[26.518347,70.34874749900007],[27.03247650000003,70.48385599800008],[26.951067,70.54679099900005],[27.025823500000058,70.61801150000008],[27.280205,70.58753949800007],[27.06658150000004,70.70458999900006],[27.355761,70.75415799900003],[27.285433,70.83248149900004],[27.702987500000063,70.800621],[27.101142500000037,70.93241899800006],[27.35516399900007,70.94692999900008],[27.200126500000067,71.01464850000008],[27.275039500000048,71.04461650000007],[27.550026,70.95600899800007],[27.574959,71.01279449900005],[27.487985500000036,71.06152349800004],[27.66123,71.126999],[28.548399,70.96723950000006],[28.240961,70.79145799800006],[27.790540500000077,70.79557049800007],[27.83346750000004,70.78317250000003],[27.753228999000044,70.75068649800005],[28.142963500000064,70.73532849800006],[27.63097,70.62425249800003],[28.305256,70.687088],[27.812294,70.486458],[27.855804500000033,70.43014549900005],[28.510006,70.44792999800006],[28.55335050000008,70.50961299800008],[28.397211,70.542122],[28.627989,70.67738350000008],[28.516052,70.72720349900004],[28.96657349900005,70.88494099900004],[29.421539,70.732119],[29.244068,70.670433],[29.27265,70.65177149800007],[30.202276,70.67542249800005],[30.107942500000036,70.64891799900005],[30.333885,70.60903149800004],[29.997783499000036,70.53657549900004],[30.618822,70.55101],[30.58019050000007,70.51128349800007],[30.96376550000008,70.44050599900004],[31.060808,70.28778850000003],[29.90366750000004,70.06065349900007],[28.73865071000006,70.15556738900005]]],[[[26.045212,71.11144250000007],[25.894638,71.010208],[26.225906500000065,71.06076799800007],[25.55792450000007,70.93201449900005],[25.391018,70.99469],[25.545727,71.03257750000006],[25.24226550000003,71.05068999900004],[25.739667999000062,71.07686599900006],[25.528230500000063,71.15545649800004],[25.673977,71.17353799800003],[26.045212,71.11144250000007]]],[[[24.12865849900004,70.60249349800006],[23.773941,70.50978849800003],[23.623031500000025,70.69841750000006],[23.815597500000024,70.72293849900007],[23.781578,70.695778],[24.12865849900004,70.60249349800006]]],[[[23.642444,70.48944849900005],[23.146014500000035,70.27236950000008],[22.815429,70.41147699800007],[23.43688750000007,70.56320949900004],[23.354719,70.617737],[23.56452,70.62199399800005],[23.529517500000054,70.52244549900007],[23.642444,70.48944849900005]]],[[[23.45733850000005,70.76512149900003],[22.798172,70.513825],[22.087011500000074,70.47561649800008],[22.310595,70.60153949900007],[21.928592500000036,70.64286049900005],[22.516258,70.72119899900008],[22.719646500000067,70.61257150000006],[22.769359500000064,70.678589],[22.679491,70.74093649800005],[22.997843,70.667839],[23.11976450000003,70.74383549900006],[23.015039500000057,70.79207599800003],[23.053814,70.81461349900007],[23.50584149900004,70.78686499900004],[23.369827500000042,70.78671249800004],[23.45733850000005,70.76512149900003]]],[[[22.956299,70.32978049900004],[23.009489,70.24874899800005],[22.342153499000062,70.33776850000004],[22.956299,70.32978049900004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL1","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION CENTRALNY","NUTS_NAME":"REGION CENTRALNY"},"geometry":{"type":"Polygon","coordinates":[[[21.645086,53.28348049700003],[21.694427500000074,53.138093997000055],[22.001760500000046,52.972328997000034],[22.00393150000008,52.87786599800006],[22.279745500000047,52.88493649900005],[22.30506450000007,52.83934799700006],[22.262360500000057,52.81536199900006],[22.33441399900005,52.74046399800005],[22.453771500000073,52.788238499000045],[22.435035999000036,52.69693949900005],[22.473925,52.64772549800006],[22.408589,52.60968999800008],[22.525242500000047,52.522721999000055],[22.580413500000077,52.39315799600007],[22.929204500000026,52.37359349800005],[23.128409,52.28784149900008],[22.902187500000025,52.063525499000036],[22.698293500000034,52.10849999800007],[22.622901499000022,52.01874449600007],[22.483290500000066,52.067227],[22.344350500000075,51.99600399900004],[21.889174500000024,51.97318399900007],[21.866380500000048,51.94342599700008],[21.91285,51.88653099800007],[21.871248500000036,51.83553249900007],[21.95288850000003,51.79557499800006],[21.835289,51.74444899900004],[21.879981,51.69363149800006],[21.61554250000006,51.61756199900003],[21.82858,51.55884849700004],[21.873176001000047,51.474913498000035],[21.761347500000056,51.40577899800007],[21.85377949900004,51.35174849900005],[21.819536,51.299933500000066],[21.860111,51.274702500000046],[21.783307,51.17904949800004],[21.802998,51.07207899800005],[21.630164500000035,51.06338899700006],[21.463566999000022,51.013111998000056],[21.355528,51.085262499000066],[21.152750500000025,51.08028599700003],[21.070030501000076,51.21142849800003],[21.056556,51.15711399600008],[20.920719500000075,51.19533749800007],[20.813019001000043,51.14583749800005],[20.54365750000005,51.231178999000065],[20.432815,51.33940499800008],[20.366456500000027,51.31061599600008],[20.371114,51.23663199600003],[19.993847,51.183954],[19.98177050000004,51.07315549900005],[20.056235999000023,51.05774949800008],[20.03947050000005,50.99020649600004],[19.882372,51.04766299700003],[19.845536,50.93020449800008],[19.74706,50.86597],[19.489786499000047,50.878930498000045],[19.32268750000003,51.046190498000044],[19.243142500000033,51.03684449600007],[19.204254500000047,50.98241849800007],[18.922990499000036,51.098453997000036],[18.67295449900007,51.056902500000035],[18.163681500000052,51.17251699600007],[18.17038,51.267563996000035],[18.110033,51.260214],[18.07452150000006,51.349915],[18.195381500000053,51.39011499600008],[18.211442500000032,51.451917998000056],[18.326309500000036,51.419816999000034],[18.382492,51.47685750000005],[18.355307500000038,51.66052499700004],[18.471965500000067,51.85100549800006],[18.685665500000027,51.82100299700005],[18.767948,51.92784399800007],[18.71785950000003,52.065705998000055],[18.827809,52.064188],[18.943973500000027,52.10391899900003],[18.924092,52.19626799800005],[19.08975,52.20681199900008],[19.100311500000032,52.26592099800007],[19.03116650000004,52.269952998000065],[19.04712,52.33280399800003],[19.289184500000033,52.39271249700005],[19.363607500000057,52.503754498000035],[19.34111450000006,52.56705499800006],[19.426104500000065,52.61144649900007],[19.367898500000024,52.628333498000075],[19.52274299900006,52.74920049700006],[19.437642500000038,52.81479499800008],[19.474977,52.86535349800005],[19.444664,52.93904249900004],[19.684806,52.96303899700007],[19.634234,53.11056699900007],[19.761595500000055,53.15179649700008],[19.84106950000006,53.14099349800006],[19.893881500000077,53.212788499000055],[19.978165500000046,53.14391449900006],[20.30506950000006,53.140077],[20.411137,53.21416499900005],[20.675953999000058,53.269529497000065],[21.55169,53.478128],[21.59819991300003,53.48018197000005],[21.73542750000007,53.31272799800007],[21.645086,53.28348049700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL11","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Lódzkie","NUTS_NAME":"Lódzkie"},"geometry":{"type":"Polygon","coordinates":[[[19.289184500000033,52.39271249700005],[19.64831350000003,52.25367999800005],[19.936101,52.29973349800008],[20.080496,52.23304899800007],[20.090903500000024,52.15262699900006],[20.268853,52.10067450000008],[20.205895,52.02799599900004],[20.294028500000024,51.96713999800005],[20.242669,51.93451549800005],[20.569322,51.88775349800005],[20.582557500000064,51.747998497000026],[20.658493,51.724147998000035],[20.621591500000022,51.657863500000076],[20.46718899900003,51.69281299900007],[20.385006,51.64161],[20.42489050000006,51.61161949900003],[20.450549500000022,51.50680049700003],[20.52884750000004,51.462917996000044],[20.432815,51.33940499800008],[20.366456500000027,51.31061599600008],[20.371114,51.23663199600003],[19.993847,51.183954],[19.98177050000004,51.07315549900005],[20.056235999000023,51.05774949800008],[20.03947050000005,50.99020649600004],[19.882372,51.04766299700003],[19.845536,50.93020449800008],[19.74706,50.86597],[19.489786499000047,50.878930498000045],[19.32268750000003,51.046190498000044],[19.243142500000033,51.03684449600007],[19.204254500000047,50.98241849800007],[18.922990499000036,51.098453997000036],[18.67295449900007,51.056902500000035],[18.163681500000052,51.17251699600007],[18.17038,51.267563996000035],[18.110033,51.260214],[18.07452150000006,51.349915],[18.195381500000053,51.39011499600008],[18.211442500000032,51.451917998000056],[18.326309500000036,51.419816999000034],[18.382492,51.47685750000005],[18.355307500000038,51.66052499700004],[18.471965500000067,51.85100549800006],[18.685665500000027,51.82100299700005],[18.767948,51.92784399800007],[18.71785950000003,52.065705998000055],[18.827809,52.064188],[18.943973500000027,52.10391899900003],[18.924092,52.19626799800005],[19.08975,52.20681199900008],[19.100311500000032,52.26592099800007],[19.03116650000004,52.269952998000065],[19.04712,52.33280399800003],[19.289184500000033,52.39271249700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL113","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Lódz","NUTS_NAME":"Miasto Lódz"},"geometry":{"type":"Polygon","coordinates":[[[19.629511,51.72348499800006],[19.43181050000004,51.686604998000064],[19.341930500000046,51.717733999000075],[19.335549,51.80804049800008],[19.514372,51.84977699800004],[19.616460500000073,51.81440349600007],[19.629511,51.72348499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL114","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Lódzki","NUTS_NAME":"Lódzki"},"geometry":{"type":"Polygon","coordinates":[[[20.01624,51.77346799800006],[19.91288650000007,51.66444049800003],[19.751878,51.710236500000065],[19.594182,51.581146999000055],[19.598455500000057,51.50801199600005],[19.27340950000007,51.55517599700005],[19.147825,51.62635249900006],[19.206089500000076,51.713074],[18.998891,51.819907],[19.182974,51.87860199800008],[19.128616500000078,52.00053199800004],[19.59094,52.02988899800005],[19.831912,51.976907],[19.875913500000024,51.87445649800003],[20.014308500000027,51.82685249900004],[19.987137500000074,51.780778],[20.01624,51.77346799800006]],[[19.629511,51.72348499800006],[19.616460500000073,51.81440349600007],[19.514372,51.84977699800004],[19.335549,51.80804049800008],[19.341930500000046,51.717733999000075],[19.43181050000004,51.686604998000064],[19.629511,51.72348499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL115","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Piotrkowski","NUTS_NAME":"Piotrkowski"},"geometry":{"type":"Polygon","coordinates":[[[20.385006,51.64161],[20.42489050000006,51.61161949900003],[20.450549500000022,51.50680049700003],[20.52884750000004,51.462917996000044],[20.432815,51.33940499800008],[20.366456500000027,51.31061599600008],[20.371114,51.23663199600003],[19.993847,51.183954],[19.98177050000004,51.07315549900005],[20.056235999000023,51.05774949800008],[20.03947050000005,50.99020649600004],[19.882372,51.04766299700003],[19.845536,50.93020449800008],[19.74706,50.86597],[19.489786499000047,50.878930498000045],[19.32268750000003,51.046190498000044],[19.243142500000033,51.03684449600007],[19.20454250000006,51.045936],[19.26269650000006,51.11030799800005],[19.239287500000046,51.17446799800007],[19.312515500000075,51.184221500000035],[19.080324,51.29908849800006],[18.855769500000065,51.290557],[18.89357399900007,51.353296998000076],[19.047190500000056,51.35366049800007],[19.074317500000063,51.467711500000064],[19.244209,51.49566049600003],[19.27340950000007,51.55517599700005],[19.598455500000057,51.50801199600005],[19.594182,51.581146999000055],[19.751878,51.710236500000065],[19.91288650000007,51.66444049800003],[20.01624,51.77346799800006],[20.385006,51.64161]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL116","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Sieradzki","NUTS_NAME":"Sieradzki"},"geometry":{"type":"Polygon","coordinates":[[[18.827809,52.064188],[18.870699,52.00345049900005],[19.128616500000078,52.00053199800004],[19.182974,51.87860199800008],[18.998891,51.819907],[19.206089500000076,51.713074],[19.147825,51.62635249900006],[19.27340950000007,51.55517599700005],[19.244209,51.49566049600003],[19.074317500000063,51.467711500000064],[19.047190500000056,51.35366049800007],[18.89357399900007,51.353296998000076],[18.855769500000065,51.290557],[19.080324,51.29908849800006],[19.312515500000075,51.184221500000035],[19.239287500000046,51.17446799800007],[19.26269650000006,51.11030799800005],[19.20454250000006,51.045936],[19.243142500000033,51.03684449600007],[19.204254500000047,50.98241849800007],[18.922990499000036,51.098453997000036],[18.67295449900007,51.056902500000035],[18.163681500000052,51.17251699600007],[18.17038,51.267563996000035],[18.110033,51.260214],[18.07452150000006,51.349915],[18.195381500000053,51.39011499600008],[18.211442500000032,51.451917998000056],[18.326309500000036,51.419816999000034],[18.382492,51.47685750000005],[18.355307500000038,51.66052499700004],[18.471965500000067,51.85100549800006],[18.685665500000027,51.82100299700005],[18.767948,51.92784399800007],[18.71785950000003,52.065705998000055],[18.827809,52.064188]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL117","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Skierniewicki","NUTS_NAME":"Skierniewicki"},"geometry":{"type":"Polygon","coordinates":[[[19.936101,52.29973349800008],[20.080496,52.23304899800007],[20.090903500000024,52.15262699900006],[20.268853,52.10067450000008],[20.205895,52.02799599900004],[20.294028500000024,51.96713999800005],[20.242669,51.93451549800005],[20.569322,51.88775349800005],[20.582557500000064,51.747998497000026],[20.658493,51.724147998000035],[20.621591500000022,51.657863500000076],[20.46718899900003,51.69281299900007],[20.385006,51.64161],[20.01624,51.77346799800006],[19.987137500000074,51.780778],[20.014308500000027,51.82685249900004],[19.875913500000024,51.87445649800003],[19.831912,51.976907],[19.59094,52.02988899800005],[19.128616500000078,52.00053199800004],[18.870699,52.00345049900005],[18.827809,52.064188],[18.943973500000027,52.10391899900003],[18.924092,52.19626799800005],[19.08975,52.20681199900008],[19.100311500000032,52.26592099800007],[19.03116650000004,52.269952998000065],[19.04712,52.33280399800003],[19.289184500000033,52.39271249700005],[19.64831350000003,52.25367999800005],[19.936101,52.29973349800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Mazowieckie","NUTS_NAME":"Mazowieckie"},"geometry":{"type":"Polygon","coordinates":[[[21.645086,53.28348049700003],[21.694427500000074,53.138093997000055],[22.001760500000046,52.972328997000034],[22.00393150000008,52.87786599800006],[22.279745500000047,52.88493649900005],[22.30506450000007,52.83934799700006],[22.262360500000057,52.81536199900006],[22.33441399900005,52.74046399800005],[22.453771500000073,52.788238499000045],[22.435035999000036,52.69693949900005],[22.473925,52.64772549800006],[22.408589,52.60968999800008],[22.525242500000047,52.522721999000055],[22.580413500000077,52.39315799600007],[22.929204500000026,52.37359349800005],[23.128409,52.28784149900008],[22.902187500000025,52.063525499000036],[22.698293500000034,52.10849999800007],[22.622901499000022,52.01874449600007],[22.483290500000066,52.067227],[22.344350500000075,51.99600399900004],[21.889174500000024,51.97318399900007],[21.866380500000048,51.94342599700008],[21.91285,51.88653099800007],[21.871248500000036,51.83553249900007],[21.95288850000003,51.79557499800006],[21.835289,51.74444899900004],[21.879981,51.69363149800006],[21.61554250000006,51.61756199900003],[21.82858,51.55884849700004],[21.873176001000047,51.474913498000035],[21.761347500000056,51.40577899800007],[21.85377949900004,51.35174849900005],[21.819536,51.299933500000066],[21.860111,51.274702500000046],[21.783307,51.17904949800004],[21.802998,51.07207899800005],[21.630164500000035,51.06338899700006],[21.463566999000022,51.013111998000056],[21.355528,51.085262499000066],[21.152750500000025,51.08028599700003],[21.070030501000076,51.21142849800003],[21.056556,51.15711399600008],[20.920719500000075,51.19533749800007],[20.813019001000043,51.14583749800005],[20.54365750000005,51.231178999000065],[20.432815,51.33940499800008],[20.52884750000004,51.462917996000044],[20.450549500000022,51.50680049700003],[20.42489050000006,51.61161949900003],[20.385006,51.64161],[20.46718899900003,51.69281299900007],[20.621591500000022,51.657863500000076],[20.658493,51.724147998000035],[20.582557500000064,51.747998497000026],[20.569322,51.88775349800005],[20.242669,51.93451549800005],[20.294028500000024,51.96713999800005],[20.205895,52.02799599900004],[20.268853,52.10067450000008],[20.090903500000024,52.15262699900006],[20.080496,52.23304899800007],[19.936101,52.29973349800008],[19.64831350000003,52.25367999800005],[19.289184500000033,52.39271249700005],[19.363607500000057,52.503754498000035],[19.34111450000006,52.56705499800006],[19.426104500000065,52.61144649900007],[19.367898500000024,52.628333498000075],[19.52274299900006,52.74920049700006],[19.437642500000038,52.81479499800008],[19.474977,52.86535349800005],[19.444664,52.93904249900004],[19.684806,52.96303899700007],[19.634234,53.11056699900007],[19.761595500000055,53.15179649700008],[19.84106950000006,53.14099349800006],[19.893881500000077,53.212788499000055],[19.978165500000046,53.14391449900006],[20.30506950000006,53.140077],[20.411137,53.21416499900005],[20.675953999000058,53.269529497000065],[21.55169,53.478128],[21.59819991300003,53.48018197000005],[21.73542750000007,53.31272799800007],[21.645086,53.28348049700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL127","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Warszawa","NUTS_NAME":"Miasto Warszawa"},"geometry":{"type":"Polygon","coordinates":[[[21.164953,52.14472850000004],[20.986859,52.103902496000046],[20.856946,52.19042599800008],[20.915136,52.35401999800007],[21.073809,52.36728799700006],[21.10500950000005,52.27399399900003],[21.250465500000075,52.26268949900003],[21.27079850000007,52.16851299900003],[21.164953,52.14472850000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL128","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Radomski","NUTS_NAME":"Radomski"},"geometry":{"type":"Polygon","coordinates":[[[21.61554250000006,51.61756199900003],[21.82858,51.55884849700004],[21.873176001000047,51.474913498000035],[21.761347500000056,51.40577899800007],[21.85377949900004,51.35174849900005],[21.819536,51.299933500000066],[21.860111,51.274702500000046],[21.783307,51.17904949800004],[21.802998,51.07207899800005],[21.630164500000035,51.06338899700006],[21.463566999000022,51.013111998000056],[21.355528,51.085262499000066],[21.152750500000025,51.08028599700003],[21.070030501000076,51.21142849800003],[21.056556,51.15711399600008],[20.920719500000075,51.19533749800007],[20.813019001000043,51.14583749800005],[20.54365750000005,51.231178999000065],[20.432815,51.33940499800008],[20.52884750000004,51.462917996000044],[20.450549500000022,51.50680049700003],[20.42489050000006,51.61161949900003],[20.639366,51.56970399800008],[20.814174500000036,51.63828249800008],[20.76290799900005,51.69020599900006],[20.907882,51.75223599800006],[21.045849,51.682645500000035],[21.28640850000005,51.86167349800007],[21.425035500000035,51.79360399800004],[21.47282450000006,51.66484199800004],[21.61554250000006,51.61756199900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL129","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Warszawski wschodni","NUTS_NAME":"Warszawski wschodni"},"geometry":{"type":"Polygon","coordinates":[[[21.121797,52.55358799800007],[21.426734500000066,52.462888],[21.596811500000058,52.55323149900005],[21.704566,52.51310349600004],[21.733551500000033,52.454755497000065],[21.665407,52.46626999700004],[21.803576500000077,52.34312749600008],[21.72377,52.32684050000006],[21.94179450000007,52.21643799800006],[21.900324499000078,52.16117199900003],[21.978105,52.09076349800006],[21.867955,52.03754850000007],[21.889174500000024,51.97318399900007],[21.866380500000048,51.94342599700008],[21.91285,51.88653099800007],[21.871248500000036,51.83553249900007],[21.95288850000003,51.79557499800006],[21.835289,51.74444899900004],[21.879981,51.69363149800006],[21.61554250000006,51.61756199900003],[21.47282450000006,51.66484199800004],[21.425035500000035,51.79360399800004],[21.28640850000005,51.86167349800007],[21.235858500000063,51.90780249900007],[21.279390500000034,51.96348349700003],[21.164953,52.14472850000004],[21.27079850000007,52.16851299900003],[21.250465500000075,52.26268949900003],[21.10500950000005,52.27399399900003],[21.073809,52.36728799700006],[20.915136,52.35401999800007],[20.471228,52.285082500000044],[20.404814,52.38434249900007],[20.604309,52.56631899900003],[20.793318500000055,52.638041497000074],[20.976901,52.53798750000004],[21.121797,52.55358799800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12A","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Warszawski zachodni","NUTS_NAME":"Warszawski zachodni"},"geometry":{"type":"Polygon","coordinates":[[[20.251022,52.388087],[20.404814,52.38434249900007],[20.471228,52.285082500000044],[20.915136,52.35401999800007],[20.856946,52.19042599800008],[20.986859,52.103902496000046],[21.164953,52.14472850000004],[21.279390500000034,51.96348349700003],[21.235858500000063,51.90780249900007],[21.28640850000005,51.86167349800007],[21.045849,51.682645500000035],[20.907882,51.75223599800006],[20.76290799900005,51.69020599900006],[20.814174500000036,51.63828249800008],[20.639366,51.56970399800008],[20.42489050000006,51.61161949900003],[20.385006,51.64161],[20.46718899900003,51.69281299900007],[20.621591500000022,51.657863500000076],[20.658493,51.724147998000035],[20.582557500000064,51.747998497000026],[20.569322,51.88775349800005],[20.242669,51.93451549800005],[20.294028500000024,51.96713999800005],[20.205895,52.02799599900004],[20.268853,52.10067450000008],[20.090903500000024,52.15262699900006],[20.080496,52.23304899800007],[19.936101,52.29973349800008],[20.002255500000047,52.369001998000044],[19.961718,52.41634699900004],[20.251022,52.388087]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12B","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Ciechanowski","NUTS_NAME":"Ciechanowski"},"geometry":{"type":"Polygon","coordinates":[[[21.121797,52.55358799800007],[20.976901,52.53798750000004],[20.793318500000055,52.638041497000074],[20.604309,52.56631899900003],[20.404814,52.38434249900007],[20.251022,52.388087],[20.241141500000026,52.48781699800003],[20.148784500000033,52.60731099800006],[20.161863500000038,52.671996498000055],[19.968221500000027,52.77511149800006],[19.963786999000035,52.883865498000034],[19.684806,52.96303899700007],[19.634234,53.11056699900007],[19.761595500000055,53.15179649700008],[19.84106950000006,53.14099349800006],[19.893881500000077,53.212788499000055],[19.978165500000046,53.14391449900006],[20.30506950000006,53.140077],[20.411137,53.21416499900005],[20.675953999000058,53.269529497000065],[20.72007650000006,53.18290249800003],[20.668768499000066,53.02581749800004],[20.69907550000005,52.98785599900003],[20.80102450000004,52.98003349800007],[20.880497,52.843532],[21.00530850000007,52.767783],[21.13456350000007,52.803177997000034],[21.353451,52.72593499800007],[21.232031500000062,52.573193],[21.121797,52.55358799800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12C","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Plocki","NUTS_NAME":"Plocki"},"geometry":{"type":"Polygon","coordinates":[[[19.684806,52.96303899700007],[19.963786999000035,52.883865498000034],[19.968221500000027,52.77511149800006],[20.161863500000038,52.671996498000055],[20.148784500000033,52.60731099800006],[20.241141500000026,52.48781699800003],[20.251022,52.388087],[19.961718,52.41634699900004],[20.002255500000047,52.369001998000044],[19.936101,52.29973349800008],[19.64831350000003,52.25367999800005],[19.289184500000033,52.39271249700005],[19.363607500000057,52.503754498000035],[19.34111450000006,52.56705499800006],[19.426104500000065,52.61144649900007],[19.367898500000024,52.628333498000075],[19.52274299900006,52.74920049700006],[19.437642500000038,52.81479499800008],[19.474977,52.86535349800005],[19.444664,52.93904249900004],[19.684806,52.96303899700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12D","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Ostrolecki","NUTS_NAME":"Ostrolecki"},"geometry":{"type":"Polygon","coordinates":[[[22.408589,52.60968999800008],[22.159246,52.70248499600007],[21.99976,52.62192549800005],[21.90535249900006,52.69464899700006],[21.567467,52.60151399800003],[21.596811500000058,52.55323149900005],[21.426734500000066,52.462888],[21.121797,52.55358799800007],[21.232031500000062,52.573193],[21.353451,52.72593499800007],[21.13456350000007,52.803177997000034],[21.00530850000007,52.767783],[20.880497,52.843532],[20.80102450000004,52.98003349800007],[20.69907550000005,52.98785599900003],[20.668768499000066,53.02581749800004],[20.72007650000006,53.18290249800003],[20.675953999000058,53.269529497000065],[21.55169,53.478128],[21.59819991300003,53.48018197000005],[21.73542750000007,53.31272799800007],[21.645086,53.28348049700003],[21.694427500000074,53.138093997000055],[22.001760500000046,52.972328997000034],[22.00393150000008,52.87786599800006],[22.279745500000047,52.88493649900005],[22.30506450000007,52.83934799700006],[22.262360500000057,52.81536199900006],[22.33441399900005,52.74046399800005],[22.453771500000073,52.788238499000045],[22.435035999000036,52.69693949900005],[22.473925,52.64772549800006],[22.408589,52.60968999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL12E","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Siedlecki","NUTS_NAME":"Siedlecki"},"geometry":{"type":"Polygon","coordinates":[[[22.408589,52.60968999800008],[22.525242500000047,52.522721999000055],[22.580413500000077,52.39315799600007],[22.929204500000026,52.37359349800005],[23.128409,52.28784149900008],[22.902187500000025,52.063525499000036],[22.698293500000034,52.10849999800007],[22.622901499000022,52.01874449600007],[22.483290500000066,52.067227],[22.344350500000075,51.99600399900004],[21.889174500000024,51.97318399900007],[21.867955,52.03754850000007],[21.978105,52.09076349800006],[21.900324499000078,52.16117199900003],[21.94179450000007,52.21643799800006],[21.72377,52.32684050000006],[21.803576500000077,52.34312749600008],[21.665407,52.46626999700004],[21.733551500000033,52.454755497000065],[21.704566,52.51310349600004],[21.596811500000058,52.55323149900005],[21.567467,52.60151399800003],[21.90535249900006,52.69464899700006],[21.99976,52.62192549800005],[22.159246,52.70248499600007],[22.408589,52.60968999800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL2","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION POLUDNIOWY","NUTS_NAME":"REGION POLUDNIOWY"},"geometry":{"type":"Polygon","coordinates":[[[18.922990499000036,51.098453997000036],[19.204254500000047,50.98241849800007],[19.243142500000033,51.03684449600007],[19.32268750000003,51.046190498000044],[19.489786499000047,50.878930498000045],[19.74706,50.86597],[19.832163,50.81970099700004],[19.712949500000036,50.729156],[19.871657500000026,50.69304550000004],[19.831074,50.654359],[19.922185499000022,50.639417999000045],[19.803217,50.56555399900003],[19.949966500000073,50.50478250000003],[20.272868,50.46994499700003],[20.41552050000007,50.19123749600004],[20.681539,50.20587549600003],[20.786962500000072,50.28838849700003],[21.20883150000003,50.35489749800007],[21.145727,50.25712050000004],[21.146154500000023,50.13003999700004],[21.19502250000005,50.119844999000065],[21.150439,49.97648449900004],[21.28472,49.924064498000064],[21.224959,49.85259849800008],[21.297149500000046,49.84286399900003],[21.34895399900006,49.81496449800005],[21.241772,49.77610449900004],[21.347128500000053,49.690366],[21.326661500000057,49.593654498000035],[21.39771250000007,49.433793998000056],[21.055269,49.42130599800004],[21.104102499000078,49.376393998000026],[20.923722500000054,49.29623449800005],[20.743115500000044,49.415132999000036],[20.614939500000048,49.41783299800005],[20.322434500000043,49.40184599600008],[20.146103500000038,49.317959],[20.088046,49.17958099800006],[19.883929500000022,49.20417699800004],[19.758416,49.215991],[19.799801,49.295825498000056],[19.790551,49.41113149900008],[19.631736,49.40882099600003],[19.467386500000032,49.61376699900006],[19.260708,49.53122299800003],[19.153403,49.403777],[18.979868,49.395217],[18.972818,49.50261199800008],[18.851551,49.517189],[18.804583,49.67888899900004],[18.627212,49.72232999600004],[18.575724,49.91042299900005],[18.321821,49.915819999000064],[18.035060999000052,50.06577199800006],[18.059780500000045,50.17465249900005],[18.42586650000004,50.24896549700003],[18.379609,50.48174249900006],[18.440043,50.48072699800008],[18.438194500000066,50.54658399900006],[18.607473500000026,50.55001099900005],[18.478263500000025,50.703099],[18.580970500000035,50.752530998000054],[18.506283,50.801472998000065],[18.615877500000067,50.85358750000006],[18.654821,50.91595999800006],[18.612386,50.955137999000044],[18.695487,51.01577549800004],[18.67295449900007,51.056902500000035],[18.922990499000036,51.098453997000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL21","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Malopolskie","NUTS_NAME":"Malopolskie"},"geometry":{"type":"Polygon","coordinates":[[[20.272868,50.46994499700003],[20.41552050000007,50.19123749600004],[20.681539,50.20587549600003],[20.786962500000072,50.28838849700003],[21.20883150000003,50.35489749800007],[21.145727,50.25712050000004],[21.146154500000023,50.13003999700004],[21.19502250000005,50.119844999000065],[21.150439,49.97648449900004],[21.28472,49.924064498000064],[21.224959,49.85259849800008],[21.297149500000046,49.84286399900003],[21.34895399900006,49.81496449800005],[21.241772,49.77610449900004],[21.347128500000053,49.690366],[21.326661500000057,49.593654498000035],[21.39771250000007,49.433793998000056],[21.055269,49.42130599800004],[21.104102499000078,49.376393998000026],[20.923722500000054,49.29623449800005],[20.743115500000044,49.415132999000036],[20.614939500000048,49.41783299800005],[20.322434500000043,49.40184599600008],[20.146103500000038,49.317959],[20.088046,49.17958099800006],[19.883929500000022,49.20417699800004],[19.758416,49.215991],[19.799801,49.295825498000056],[19.790551,49.41113149900008],[19.631736,49.40882099600003],[19.467386500000032,49.61376699900006],[19.45943550000004,49.67643349600007],[19.369976500000064,49.69492599900008],[19.444958,49.73403499800003],[19.41495150000003,49.77526499900006],[19.31678450000004,49.777886497000054],[19.290000500000076,49.85055299800007],[19.196614500000067,49.86008249900004],[19.184922413000038,49.94999287700006],[19.09321,49.956658996000044],[19.253810500000043,50.13419299900005],[19.44197550000007,50.22657449800005],[19.323423,50.25914499800007],[19.483944999000073,50.32359699800003],[19.48904550000003,50.397027499000046],[19.844537500000058,50.434195499000054],[19.949966500000073,50.50478250000003],[20.272868,50.46994499700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL213","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Kraków","NUTS_NAME":"Miasto Kraków"},"geometry":{"type":"Polygon","coordinates":[[[20.159060500000066,50.121605497000076],[20.217346500000076,50.05343449800006],[19.963435,49.968627499000036],[19.79566650000004,49.995284],[19.801697,50.069210499000064],[19.880745500000046,50.120454],[20.159060500000066,50.121605497000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO","LEVL_CODE":0,"CNTR_CODE":"NO","NAME_LATN":"NORGE","NUTS_NAME":"NORGE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.115073512000038,60.965715414000044],[5.026263,61.01004050000006],[5.986885,61.128764997000076],[6.411997500000041,61.05647299700007],[6.645289500000047,61.17484649900007],[6.850829500000032,61.14572549700006],[7.035421,60.99983999800003],[6.84106550000007,60.882595],[7.044801500000062,60.985458496000035],[7.154517500000054,60.93813299900006],[7.121905500000025,60.86051199800005],[7.197031500000037,60.92475499900007],[7.026663500000041,61.08794399900006],[7.478136500000062,61.098621500000036],[7.412843,61.13557799700004],[7.712671,61.23199450000004],[7.432381,61.199321499000064],[7.312189,61.29982749700008],[7.585154,61.49429699900003],[7.284697,61.39589299700003],[7.348169,61.34857549900005],[7.247618,61.30030450000004],[7.395241500000054,61.18244450000003],[6.992821,61.10582799900004],[6.927117500000065,61.14121999900004],[7.17012,61.19993199800007],[6.630224500000054,61.21004499900005],[6.753228500000034,61.41973099900008],[6.473639500000047,61.115188499000055],[6.006493,61.162735],[6.050285,61.19412249700008],[5.394584,61.069965496000066],[5.076056,61.179718],[5.312890500000037,61.18884299900003],[4.94716950000003,61.25884999900006],[5.209688823000022,61.33452840500007],[5.026415500000041,61.37598399900003],[5.167576,61.42851999900006],[4.977364500000022,61.41730899700008],[5.240735500000028,61.47474299800007],[5.588792499000022,61.44279099800008],[5.751739500000042,61.492759499000044],[5.843049,61.45913699700003],[5.712819,61.50769399900008],[5.481126,61.454650998000034],[5.442402,61.46443949700006],[5.461188499000059,61.48801799600005],[5.177923999000029,61.51424049900004],[5.39946,61.551673996000034],[5.241315666000048,61.58393322900008],[4.994847999000058,61.591152],[5.414846500000067,61.646262999000044],[4.991695,61.62740699700004],[4.969778,61.72112249600008],[5.21751449900006,61.72261449900003],[5.151150500000028,61.75493249700003],[5.296247,61.77143499700003],[5.092194,61.76363749700005],[5.382925,61.84609199600004],[5.300301500000046,61.85786450000006],[5.434949,61.917640497000036],[5.780999500000064,61.852504500000066],[5.642197500000066,61.81269849900008],[5.985106500000029,61.84537099900007],[5.913162500000055,61.731254498000055],[6.010346500000026,61.842471999000054],[6.178206500000044,61.76468349700008],[6.214309,61.77514649900007],[6.079197500000021,61.84675599900004],[6.483179,61.80439750000005],[6.613900500000057,61.85201649600003],[6.586079,61.86867150000006],[6.846349,61.870303996000075],[6.630262,61.89056],[6.429300500000068,61.830268999000054],[5.911377999000024,61.882831499000076],[5.808180500000049,61.85716599700004],[5.731962499000076,61.89316199700005],[5.988245500000062,61.905822999000065],[5.458462500000053,61.93934999900006],[5.178933,61.895335998000064],[4.990589,61.99135599600004],[5.123834,62.032745498000054],[5.142885471000056,61.95648816400006],[5.288158,61.94990899900006],[5.426334,62.018695999000045],[5.089433,62.16728949900005],[5.185808,62.21553399700008],[5.491458,62.01478749900008],[5.623765500000047,62.06705450000004],[5.405087500000036,62.12742949900007],[5.485529,62.20001999900006],[5.73594,62.08772650000003],[5.671969,62.174431],[5.851422500000069,62.20307499900008],[5.95038099900006,62.156822],[5.847182,62.00476449700005],[5.981478500000037,62.142124],[6.325065500000051,62.06060049900003],[5.931158,62.21788399700006],[6.117795,62.18908299700007],[5.96181050000007,62.24787899900008],[6.328697,62.37550749700006],[6.580707500000074,62.14685449900003],[6.545071,62.104973],[6.610225999000022,62.13547149900006],[6.555007,62.19385549600008],[6.653805500000033,62.193794497000056],[6.402287500000057,62.357982496000034],[6.723693500000024,62.45203799700005],[6.914581,62.41138099800003],[6.884847500000035,62.38205349600008],[7.007536,62.283473997000044],[7.012420999000028,62.17502599900007],[6.873665,62.08268749700005],[7.20569050000006,62.10144799900007],[6.96163,62.11530299700007],[7.038890500000036,62.17032549700008],[7.047532499000056,62.27912899900008],[7.428215500000022,62.25268149900006],[6.987393,62.31317899900006],[6.941239500000052,62.42668149900004],[6.772653,62.46071199600004],[6.811594500000069,62.482394999000064],[6.220586,62.46456149800008],[6.717913,62.49196599600003],[6.461608500000068,62.51434349800007],[6.573409500000025,62.542159999000035],[6.256442500000048,62.52973549900008],[6.334281500000031,62.61331949700008],[6.568112500000041,62.62427899700003],[6.611302500000022,62.55686549600006],[6.660633500000074,62.632385500000055],[7.045805,62.65523149900008],[7.138921500000038,62.52282349800004],[7.128197500000056,62.60631549900006],[7.166881500000045,62.63157249800008],[7.55996,62.53118499900006],[7.786336,62.57716750000003],[7.396082,62.628738499000065],[7.970172500000047,62.74502199700004],[8.146601500000031,62.688270500000044],[8.019416,62.751445999000055],[8.06985,62.77651599900008],[7.349656500000037,62.69644949900004],[7.722468500000048,62.79231649900004],[6.98348149900005,62.724308],[6.969040500000062,62.77918599700007],[7.018433,62.81851599700008],[7.240911500000038,62.80947899900008],[6.898579,62.90984750000007],[7.301491500000054,63.010261496000055],[7.450778500000069,62.90481949700006],[7.685032,62.96850599900006],[7.791710500000022,62.95953349600006],[7.677089500000022,62.898002500000075],[8.036038500000075,62.96656399800008],[8.11747,62.921828997000034],[8.13680850000003,62.822223500000064],[8.530576,62.66896449600006],[8.55466750000005,62.717293],[8.190963,62.829601499000034],[8.136116,62.94702149900007],[7.988732,63.01142899800004],[7.880380500000058,62.98959749800008],[8.107145499000069,63.10542299900004],[8.142638,63.07119349900006],[8.132346,63.034172],[8.333088,62.96408099800004],[8.337154500000054,62.891792500000065],[8.700226500000042,62.81935499900004],[8.421752999000034,62.94767399800003],[8.663505499000053,62.97121799800004],[8.470663,62.97100050000006],[8.594899,63.02206049900008],[8.365225,62.96965399900006],[8.322330500000021,62.99421299900007],[8.49464050000006,63.01892850000007],[8.22018349900003,63.029011],[8.156400500000075,63.113617],[8.764103,63.184509999000056],[8.985986500000024,63.20794299800008],[8.832189,63.20198],[8.490692,63.27751149900007],[8.765663999000026,63.34213649800006],[8.650367,63.40087899900004],[8.755111500000055,63.42434099900004],[8.904203500000051,63.38735199800004],[9.01071,63.46447],[9.177391,63.37377950000007],[9.092917500000055,63.28715899900004],[9.501772,63.39708699700003],[9.249117,63.36782849600007],[9.176959,63.41464599900007],[9.297754401000077,63.448047849000034],[9.147209,63.48814],[9.585823,63.528888496000036],[9.491575,63.57952499900006],[9.75341050000003,63.645359],[9.71493350000003,63.615505],[9.977688,63.44161599700004],[9.823175500000048,63.31224450000008],[10.208758500000044,63.322869],[10.059104,63.38721099900005],[10.189022,63.45037849900007],[10.85359,63.439093496000055],[10.912328500000058,63.46505349900008],[10.756933,63.520218],[10.941581,63.56428899900004],[10.627927,63.54674150000005],[11.458975,63.80233749900003],[11.072477500000048,63.858989500000064],[11.501289500000041,64.00521850000007],[11.222313,64.074219],[10.591203500000063,63.80435549900005],[10.962953500000026,63.90656649700003],[11.070417500000076,63.841949499000066],[10.955013501000053,63.74560949900007],[10.170315,63.526134496000054],[9.948584500000038,63.495971498000074],[9.805798500000037,63.62687299900006],[10.130084,63.75410449900005],[9.52912150000003,63.71239849600005],[9.803867500000024,63.766426],[9.559135500000025,63.769199],[9.678071,63.84419649900008],[10.224334,63.927921498000046],[9.946816500000068,63.90663199800008],[10.226982,64.00061799900004],[9.951425500000028,63.94597250000004],[10.010268949000022,64.04565076200004],[9.932503500000053,64.060982],[10.016330268000047,64.06744965300004],[10.234300500000074,64.10188299600003],[10.120101,64.121704],[10.212196500000061,64.20630649900005],[10.374837,64.17877199600008],[10.274836500000049,64.23419149700004],[10.444528500000047,64.35304249800004],[10.68307,64.36034399900007],[10.45395,64.40396099800006],[10.634974130000046,64.39100241600005],[10.548171,64.44136050000003],[10.83981250000005,64.368828],[10.654607,64.45243049900006],[10.948439500000063,64.50749949600004],[10.976382,64.53115849900007],[10.906779500000027,64.57653050000005],[10.973181500000067,64.60691050000008],[11.340755500000057,64.43691999700007],[11.225456,64.311905],[11.407744500000035,64.36854549600008],[11.352719500000035,64.41844949900008],[11.641216500000041,64.47501099800007],[11.42755150000005,64.47552499900007],[11.725962500000037,64.58479299900006],[11.515246500000046,64.56695549800003],[11.379855,64.66954049900005],[11.86674750000003,64.79622649700008],[11.603879,64.814125],[11.817444,64.86047349600005],[11.653197500000033,64.83556349900005],[11.694047001000058,64.87286399600004],[12.136008884000034,64.96383255800004],[11.615549,64.86773699900004],[11.65877050000006,64.862915],[11.392232,64.75153349900006],[11.182071500000063,64.74090599900006],[11.446765999000036,64.809616],[11.240288,64.81357549900008],[11.277119500000026,64.845711],[11.514355244000058,64.87855370700004],[11.278464500000041,64.85742949800004],[11.960979500000064,65.07714799700005],[12.101823,65.11344899900007],[12.121571500000073,65.18907949900006],[12.337297500000034,65.11171699800008],[11.981972,65.07081549600008],[12.046106500000064,65.06244999900008],[12.119634500000075,65.05026249700006],[12.092824,65.03982549900007],[12.187348500000041,65.00011149700003],[12.182329,64.98138099900007],[12.214979,65.01567099900006],[12.171146500000077,65.04820999800006],[12.460707500000069,65.07824699900004],[12.495921500000065,65.105568],[12.399293,65.098755],[12.964295500000048,65.32212049900005],[12.474547500000028,65.13399499900004],[12.350988500000028,65.15914899900008],[12.510954,65.19638799800003],[12.336974,65.23342099800004],[12.46169,65.25754549900006],[12.31042749900007,65.25910949900003],[12.471449500000062,65.37045849900005],[12.041082500000073,65.21619399900004],[12.140350500000068,65.28169249900003],[12.085761,65.30515299900003],[12.146955499000057,65.37075799700006],[12.276143,65.39269249800003],[12.158635,65.38862599800007],[12.353930500000047,65.46737649800008],[12.183487999000022,65.44551849900006],[12.292716,65.58846299700008],[12.630449,65.41567249600007],[12.781976500000042,65.46252449900004],[12.401527500000043,65.537445],[12.347975,65.62732699900005],[12.494364,65.73480999900005],[12.630283314000053,65.69639763800006],[12.537199,65.75340999900004],[12.646986,65.81443799600004],[12.718809,65.77714549900003],[12.72851,65.80428299800008],[12.597691500000053,65.82546249900008],[12.666879418000065,65.92070801000006],[12.380134500000054,65.89411099900008],[13.053273500000046,66.07566850000006],[12.657043500000043,66.04336549900006],[12.830141,66.12840199900006],[13.346362,66.22772149900004],[13.609442500000057,66.20105749800007],[13.548763,66.10309549800007],[13.819278,66.137657],[13.599074500000029,66.22496049800003],[14.147304500000075,66.33046699900007],[13.009953500000051,66.19128399800007],[13.562423500000023,66.30467999600006],[13.027488,66.32476049900004],[12.990386999000066,66.34877799900005],[13.159712500000069,66.40702049800007],[13.172011500000053,66.46587349900005],[12.978532,66.529541],[13.256189500000062,66.446121],[13.425744,66.47429649800006],[13.262984500000073,66.52902149900007],[13.672662,66.51731099900007],[13.470007,66.565529],[13.733414500000038,66.60917650000005],[13.185444,66.56871799900006],[13.413904,66.62222299900003],[13.16772,66.65842449900003],[13.566408,66.64874249800005],[13.207289500000059,66.71688099900007],[14.000651,66.79788950000005],[13.610096,66.84452799900004],[13.68425047900007,66.86556359300005],[13.492176,66.94919649900004],[14.013185500000077,66.96194450000007],[14.06197050000003,66.99144749900006],[14.019997500000045,67.04334999900004],[14.052112500000021,67.08169550000008],[14.173709975000065,67.01579152300008],[14.124613,67.07179250000007],[14.008136,67.09213299900006],[14.067618500000037,67.16149150000007],[14.302209,67.18673699800007],[14.216777,66.99249250000008],[14.42914905200007,67.08494146600003],[14.297962,67.10489649800007],[14.342555,67.17453],[14.602729091000072,67.19427639000003],[14.511739500000033,67.20764899900007],[15.006282,67.23958599900004],[14.88538550000004,67.16931899800005],[14.968609,67.11983499800004],[14.999890500000049,67.12169649800006],[14.917812499000036,67.16667199800008],[15.088397999000051,67.23653399900007],[15.433605,67.10498049900008],[15.481187,67.18446349900006],[15.349717,67.22185449900007],[15.430528500000037,67.26048299900003],[15.037823500000059,67.27246150000008],[15.162948499000038,67.32976499900008],[14.308716,67.26377099900003],[14.637446500000067,67.43815649800007],[14.943926,67.44770749900005],[14.651400500000022,67.47698199900003],[15.001051811000025,67.57711863800006],[15.080581499000061,67.52973949800008],[14.923469500000067,67.48969249900006],[15.4537,67.49208049800006],[15.582617,67.37529749900006],[15.633104446000061,67.41328633800003],[15.515371499000025,67.44784549800005],[15.650709213000027,67.53825820800006],[15.160784500000034,67.63269799900007],[15.339144,67.69324499800007],[15.247291500000074,67.702095],[15.311404,67.73804449900007],[15.654401,67.68859849900008],[15.428495500000054,67.75768299800006],[15.855372500000044,67.70989999900007],[15.43736750000005,67.809578],[14.734998,67.67552199900007],[15.14555050000007,67.76268799800005],[14.75716,67.80435949900004],[15.136513500000035,67.868988],[14.82857799900006,67.860016],[14.889670500000022,67.88421649900005],[16.00116550000007,67.99621600200004],[15.526616,68.06521600100007],[15.395943500000044,68.03652199800007],[15.440778500000022,68.02494799900006],[15.414783500000055,67.99591049900005],[15.275926500000025,68.05061299900007],[15.406996500000048,68.14228050200006],[15.67317,68.18434150000007],[15.725988500000028,68.11899549600008],[15.597519,68.11167149900007],[15.74319950000006,68.087616],[16.062347499000055,68.255936],[16.089401,68.20642099900004],[16.019762,68.18904099800005],[16.140629,68.18688199900004],[16.074756500000035,68.05342850000005],[16.22637750000007,67.96387499900004],[16.203220500000043,67.898308],[16.49381850000003,67.81392649800006],[16.237539500000025,67.902992],[16.282963,67.98599950000005],[16.189962500000036,68.02142349900004],[16.284780500000068,68.026779],[16.459911500000032,67.91499349900005],[16.316387,68.027191],[16.52378250000004,67.96525550000007],[16.354823500000066,68.04674549800006],[16.740179,68.08623499900006],[16.339367,68.094261],[16.24048250000004,68.137703],[16.60742,68.18205249800008],[16.140442500000063,68.29213550000003],[16.819980500000042,68.15521999800006],[16.27568050000002,68.37313849900005],[17.170841,68.393112],[17.33622,68.28037999800006],[17.318298500000026,68.19080349900008],[17.407579500000054,68.26064299800004],[17.249164,68.394539],[17.900738,68.41780799900005],[17.471044500000062,68.46656049900008],[17.573683,68.53850549900005],[17.54451,68.55082699800005],[17.211914,68.45962499800004],[17.01371,68.45533750000004],[17.105293500000073,68.484131],[16.995945,68.52622999900007],[16.557794,68.44740249900008],[16.501896,68.44715099800004],[16.530721500000027,68.48244499800006],[16.48231019900004,68.50168901000006],[16.515053,68.46984099800005],[16.075683500000025,68.414055],[16.232881500000076,68.54445649800005],[16.579589500000054,68.54136699900005],[16.628774500000077,68.53780349900006],[16.564274500000067,68.58321399900007],[16.614227500000027,68.64302099800005],[16.964586500000053,68.71334099900008],[17.685274,68.67411049900005],[17.251898,68.75971199800006],[17.805559,68.74999249900003],[17.474293,68.83225999900003],[17.853133999000022,68.86698899900006],[17.447077,68.90790550000008],[17.696062,69.08660849800003],[17.636711,69.10508749900004],[18.166633499000056,69.15019999900005],[17.96954549900005,69.22856150000007],[18.23868550000003,69.48264299800007],[18.499069,69.367111],[18.474834500000043,69.20949549900007],[18.542671,69.32290649800007],[19.00698650000004,69.28672049900007],[18.442138500000056,69.49892449900005],[18.828508500000055,69.56273650000003],[18.987579500000038,69.38976299800004],[19.349617,69.31079849800005],[19.231395500000076,69.235595],[19.547487500000045,69.21487449800003],[19.351875500000062,69.36683649800005],[19.073311,69.392891],[18.96809950000005,69.46321849900005],[19.07519426600004,69.54529346400005],[18.934316500000023,69.599945],[19.27431850000005,69.77920549900006],[19.732448500000032,69.80428249900007],[19.672811500000023,69.48411549900004],[19.437654500000065,69.39199049900003],[19.508491500000048,69.39473699900003],[19.732485,69.48689249800003],[19.73015,69.607582],[20.14599050000004,69.58264149900003],[19.76941850000003,69.67984749800007],[20.134111500000074,69.93837],[20.394979500000034,69.89038799800005],[20.25130850000005,69.68190749900003],[20.349264,69.62082649800004],[19.900448,69.27372],[19.930265500000075,69.26666250000005],[20.268589,69.396126],[20.398016,69.58557149800004],[20.821378500000037,69.49770350000006],[20.4823,69.61501299800005],[20.525961,69.72260299800007],[20.459503,69.76207749800005],[20.710785,69.79328899900008],[20.791124500000024,69.84407799800005],[20.749731,69.86638650000003],[21.027044500000045,69.95645150000007],[21.082113500000048,69.95459749800006],[20.844524500000034,69.85420250000004],[20.983516500000064,69.77506249800007],[21.325062,69.90923299800005],[21.18428250000005,69.93567649900007],[21.241577,70.00890349900004],[22.140845,69.74519350000003],[21.88821050000007,69.835617],[22.026851500000078,69.84824349900003],[21.862396,69.93569949900007],[22.091429,70.02889999800004],[21.77614,70.03570499800003],[22.06965850000006,70.12821199900003],[21.727436,70.05191049900003],[21.19089,70.22124049900003],[21.424858,70.222824],[21.478477276000035,70.20036185200007],[21.493496359000062,70.19927205500005],[21.47370150000006,70.29925549900008],[21.56427,70.323387],[21.832088500000054,70.16875450000003],[21.78227050000004,70.25872049900005],[21.983658,70.32945249800008],[22.011963,70.31783299800003],[21.948728500000072,70.28130349800006],[22.298141500000042,70.27687850000007],[22.269542500000057,70.13492599800003],[22.541012,70.113289],[22.290935500000046,70.16215499800006],[22.349849499000072,70.25868299800004],[22.96682,70.189369],[22.848073456000066,70.11755874200003],[23.002601500000026,70.13343799900008],[22.952436499000044,70.040924],[23.142261500000075,69.97520449800004],[23.00179650000007,69.91750349800003],[23.407093,69.96468349900005],[23.52833,70.01591499900007],[23.132837,70.08557150000007],[23.390547,70.12302399900005],[23.179638,70.21624750000007],[23.507255500000042,70.24166049900003],[23.246681500000022,70.24819199900003],[23.605201500000078,70.29695149800006],[23.429792500000076,70.31327050000004],[23.802875,70.47930149800004],[24.329584500000067,70.44633499900004],[24.09067550000003,70.54149649800007],[24.298256,70.68613450000004],[24.696213,70.63238549800008],[24.236952,70.83589149900007],[24.708558500000038,70.87100999900008],[24.553296500000044,70.97564699800006],[25.020571,70.96941399900004],[25.048740500000065,70.950287],[24.918836500000054,70.88396450000005],[25.115078,70.93007650000004],[25.232065,70.79688249900005],[25.395689,70.88725299900005],[25.298832,70.95856499800004],[25.338186500000063,70.979912],[25.547411,70.91419199900008],[25.39456,70.855156],[25.44389350000006,70.84555049900007],[25.91175450000003,70.87564849900008],[25.086530500000038,70.51461049800008],[25.241384500000038,70.520706],[25.056989500000043,70.48918150000003],[25.22985650000004,70.46246349900008],[25.039865500000076,70.35273749900006],[25.183798,70.32165549900003],[24.936825,70.23127],[24.912155,70.09609199900007],[25.153381500000023,70.06489549900004],[25.538208,70.31125649900008],[25.382824,70.32743849900004],[25.440464,70.36740049800005],[26.57236850000004,70.95436850000004],[26.70399499900003,70.96577449800003],[26.730856,70.90872950000005],[26.654697499000065,70.83110799900004],[26.733936500000027,70.82489799800004],[26.286564,70.65195449900006],[26.640539,70.646553],[26.475652500000024,70.35486599900008],[26.518347,70.34874749900007],[27.03247650000003,70.48385599800008],[26.951067,70.54679099900005],[27.025823500000058,70.61801150000008],[27.280205,70.58753949800007],[27.06658150000004,70.70458999900006],[27.355761,70.75415799900003],[27.285433,70.83248149900004],[27.702987500000063,70.800621],[27.101142500000037,70.93241899800006],[27.35516399900007,70.94692999900008],[27.200126500000067,71.01464850000008],[27.275039500000048,71.04461650000007],[27.550026,70.95600899800007],[27.574959,71.01279449900005],[27.487985500000036,71.06152349800004],[27.66123,71.126999],[28.548399,70.96723950000006],[28.240961,70.79145799800006],[27.790540500000077,70.79557049800007],[27.83346750000004,70.78317250000003],[27.753228999000044,70.75068649800005],[28.142963500000064,70.73532849800006],[27.63097,70.62425249800003],[28.305256,70.687088],[27.812294,70.486458],[27.855804500000033,70.43014549900005],[28.510006,70.44792999800006],[28.55335050000008,70.50961299800008],[28.397211,70.542122],[28.627989,70.67738350000008],[28.516052,70.72720349900004],[28.96657349900005,70.88494099900004],[29.421539,70.732119],[29.244068,70.670433],[29.27265,70.65177149800007],[30.202276,70.67542249800005],[30.107942500000036,70.64891799900005],[30.333885,70.60903149800004],[29.997783499000036,70.53657549900004],[30.618822,70.55101],[30.58019050000007,70.51128349800007],[30.96376550000008,70.44050599900004],[31.060808,70.28778850000003],[29.90366750000004,70.06065349900007],[28.73865071000006,70.15556738900005],[28.575159,70.11865250000005],[29.660585500000025,69.96182999900003],[29.350151,69.86261749900007],[30.030296500000077,69.82659149800008],[29.699119499000062,69.75655349900006],[29.767906536000055,69.82349093300007],[29.459511,69.648964],[30.076742500000023,69.76901999900008],[29.896385,69.70440649900007],[29.951505500000053,69.58993549900003],[29.916139500000043,69.69289399800005],[30.199936,69.77079749800004],[30.130657,69.852539],[30.185869,69.879448],[30.370466,69.844124],[30.421198,69.689911],[30.293487,69.66351299800004],[30.452156,69.680496],[30.43191550000006,69.81462799900004],[30.839926,69.775808],[30.95453850000007,69.63243199900006],[30.813123,69.52871399900005],[30.08532450000007,69.65810450000004],[30.188423500000056,69.56845749900003],[30.00879450000008,69.415908],[29.288473,69.29618099800007],[29.313494500000047,69.22850299800007],[29.242261,69.11306250000007],[28.92968,69.05190499900004],[28.805792500000052,69.11114699900008],[28.831907,69.22435249800003],[29.336974,69.47831049900003],[29.13434250000006,69.69532949900008],[28.404522,69.81841799900008],[27.984522,70.01396549800006],[27.959615,70.09209449800005],[27.037634500000024,69.91038799900008],[26.490588,69.94501949900007],[25.950607,69.69655049900007],[25.89156,69.66550199800008],[25.976791500000047,69.61067350000008],[25.702101,69.25366150000008],[25.777502500000026,69.018279],[25.157167,68.79989199900007],[25.116428,68.63958749900007],[24.903199500000028,68.55459099900008],[23.871476,68.83651899800003],[23.167591,68.62851849800006],[22.374526,68.71666699900004],[22.340785,68.82722549900006],[21.983611500000052,69.07289349900003],[21.627087500000073,69.27658799800008],[21.27882150000005,69.31188399800004],[20.987576500000046,69.191927],[21.108676,69.10392949900006],[21.057545,69.03628949800003],[20.548636500000043,69.05996850000008],[20.060047500000053,69.045759],[20.306589,68.92618199900005],[20.33587250000005,68.80231249900004],[19.93750250000005,68.55793949900004],[20.226541500000053,68.49081250000006],[19.921397,68.356013],[18.405688500000053,68.58187699900003],[18.125925,68.53651599800008],[18.101093001000038,68.406045],[18.151354,68.19878999900004],[17.899761500000068,67.96937199900003],[17.281524,68.11881499900005],[16.738120500000036,67.91420949800005],[16.407672,67.53409999900003],[16.15800150000007,67.519159],[16.089829,67.43527850000004],[16.404099501000076,67.20497449900006],[16.387759,67.04546249900005],[15.377226500000063,66.48430399600005],[15.453994500000022,66.34523350000006],[15.484732,66.28245849800004],[14.516289,66.13257899900003],[14.625476500000048,65.81180849900005],[14.49877250000003,65.52129749600005],[14.50683250000003,65.30972899800008],[14.378797500000076,65.24762049800006],[14.325985,65.118916],[13.654257500000028,64.58034049800006],[14.113869500000021,64.46248450000007],[14.157109,64.19505449600007],[13.967524500000025,64.00796999900007],[13.211107500000026,64.09536949900007],[12.683565,63.97422349900006],[12.149767,63.593946998000035],[12.212877,63.47859199900006],[11.974581,63.269227999000066],[12.052455500000065,63.183444499000075],[12.218231,63.00033249900008],[12.074687999000048,62.902544997000064],[12.136381500000027,62.74791649700006],[12.056142,62.61191849800008],[12.25465950000006,62.331024496000055],[12.29937,62.26749399900007],[12.137665,61.72381699700003],[12.419609,61.56298099900005],[12.569316,61.568746496000074],[12.870847500000025,61.35649499700003],[12.670176500000025,61.055976497000074],[12.22399,61.013078],[12.606881500000043,60.512742496000044],[12.606053,60.405925499000034],[12.498795500000028,60.323652],[12.541906500000039,60.19337849700008],[12.499544999000022,60.09765699700006],[12.174292,59.88980850000007],[11.839729,59.84076699900004],[11.926970500000039,59.79047599800003],[11.939878500000077,59.69458099700006],[11.691129,59.58954749900005],[11.826196500000037,59.23784999700007],[11.632557,58.90830650000004],[11.457448500000055,58.888435497000046],[11.460827,58.98865849800006],[11.476778,58.98445499800005],[11.374898,59.09957849700004],[11.384853146000069,59.117312139000035],[11.385243500000058,59.118007498000054],[11.340374784000062,59.11173018900007],[11.330404740000063,59.11033547200003],[10.825053,59.15331250000003],[10.723924499000077,59.26906599700004],[10.814114500000073,59.32770549600008],[10.660101999000062,59.33565899800004],[10.686118,59.48940699800005],[10.581817500000056,59.756759500000044],[10.657540500000039,59.87100199900004],[10.764788,59.829544499000065],[10.759619,59.906375996000065],[10.64198,59.911508499000035],[10.506262,59.87787550000007],[10.497314500000073,59.788338499000076],[10.617811,59.57157149800008],[10.421314,59.525081500000056],[10.439196500000037,59.66585149800005],[10.212189500000022,59.73728199900006],[10.319881500000065,59.69134350000007],[10.374606,59.67696749900006],[10.410937500000045,59.622604497000054],[10.389866999000049,59.537372496000046],[10.23252750000006,59.56869149700003],[10.384464500000036,59.438155996000035],[10.493912500000022,59.43849199700003],[10.461864500000047,59.37163549700006],[10.517841500000031,59.293377],[10.386558500000035,59.27502049700007],[10.273399500000039,59.04173649900008],[10.017867,59.048832],[10.040186,58.97815299800004],[9.969417500000077,58.964656998000066],[9.83919700100006,59.044178997000074],[9.680050218000076,59.055061142000056],[9.752187500000048,58.988582500000064],[9.289463,58.83913049500006],[9.467710500000067,58.829845497000065],[9.365971,58.771348999000054],[9.082190499000035,58.74871449600005],[9.057591500000058,58.72521599800007],[9.244501,58.72533049900005],[9.076271,58.686345998000036],[9.229282500000068,58.688499499000045],[8.96709965100007,58.59442495500008],[8.992434500000059,58.55048349700007],[8.713581,58.436935497000036],[8.728853,58.39339849900006],[8.516029500000059,58.25892999800004],[8.391825500000039,58.25569199800003],[8.24362161700003,58.15584307300003],[8.256075,58.13458650100006],[8.185152,58.142941499000074],[8.061826500000052,58.11951050000005],[8.087569,58.18608849600008],[8.031234,58.23595050000006],[8.048012,58.16465399500004],[7.951262500000041,58.075958498000034],[7.513357499000051,58.00359750000007],[7.179073999000025,58.05886099600008],[7.05503,57.97974799800005],[7.038199,58.02142699800004],[7.146521,58.07286450000004],[6.98239650000005,58.03102099900008],[7.061141,58.119189997000035],[6.938757,58.05059799700007],[6.813322500000027,58.09625249700008],[7.051818,58.14051799900005],[6.798205500000051,58.18960549900004],[6.867762,58.132240498000044],[6.78057,58.11681349500003],[6.747517500000072,58.191993497000055],[6.73773650000004,58.11087049900004],[6.819480500000054,58.064850497000066],[6.553461500000026,58.117786498000044],[6.89118,58.27436049900007],[6.719107999000073,58.21998599800003],[6.672523,58.28101349800005],[6.380909,58.275402],[6.438839,58.290819499000065],[6.040288,58.379020498000045],[5.981829851000043,58.454346376000046],[5.874972,58.426685500000076],[5.921966174000033,58.463206064000076],[5.654306500000075,58.560295],[5.491752,58.75477999900005],[5.575934500000074,58.958621995000044],[5.686279,58.92847499900006],[5.574271500000066,59.03034249900003],[5.769078500000035,58.96493149600008],[5.746338499000046,58.85974899800004],[5.893873500000041,58.967807995000044],[6.176128500000061,58.83488099900006],[6.279346,58.839367],[6.073616,58.903625496000075],[5.995267500000068,58.968890997000074],[6.091457500000047,58.985473496000054],[5.889861500000052,59.07323049900003],[6.112402,59.140621],[6.022733,59.143718496000076],[6.217452500000036,59.26629649700004],[6.561808500000041,59.32115949800004],[6.155891500000052,59.260627498000076],[6.113465,59.301426],[6.241003500000033,59.34484099900004],[6.020808500000044,59.34091949900005],[6.110799426000028,59.40670361000008],[5.941965500000038,59.361309],[5.914231,59.44894449600008],[6.19901150000004,59.47964099900008],[5.909635,59.47428899700003],[5.856092500000045,59.54454799700005],[5.85419150000007,59.464008499000045],[5.667560500000036,59.39082349700004],[5.842963,59.437320499000066],[5.890834999000049,59.35116949700006],[5.646485,59.25997549600004],[5.63145,59.50061799900004],[5.577508500000022,59.50020199700003],[5.462600500000065,59.434536500000036],[5.592431500000032,59.474860997000064],[5.614074,59.33062349800008],[5.550886,59.27162149900005],[5.295012442000029,59.37086988600004],[5.330110500000046,59.27053449500005],[5.298477,59.14537050000007],[5.175062500000024,59.17837449700005],[5.186944500000038,59.35496150000006],[5.285783754000022,59.38118380700007],[5.222025500000029,59.49856949700006],[5.302875500000027,59.481168],[5.22452450000003,59.527431497000066],[5.374809500000026,59.64634699800007],[5.472243,59.61444099900007],[5.413851,59.663357],[5.534834,59.73207849900007],[5.506673157000023,59.52538185800006],[5.567902999000069,59.55945950000006],[5.555418,59.646022997000046],[5.597233500000073,59.68478799800005],[5.794665721000058,59.65094391400004],[6.381009,59.87469099900005],[6.018712,59.75312050000008],[5.675114,59.848640498000066],[5.994348,59.954562999000075],[6.038543,60.067142500000045],[6.320311500000059,60.127182],[6.142478500000038,60.10751699600007],[6.082894500000066,60.19144799900005],[6.353219500000023,60.37049899700003],[6.669563,60.40578099900006],[6.565501,60.24366750000007],[6.543831500000067,60.06967549900003],[6.685616,60.38555149600006],[6.829082747000029,60.47144575700008],[6.630605500000058,60.46667849900007],[6.716949,60.52021799600004],[6.284545,60.39194850000007],[6.209440500000028,60.46901699800003],[6.278606500000024,60.38286949900004],[6.172586,60.36284999900005],[6.195897500000058,60.28046799600003],[6.152649500000052,60.242542498000034],[6.01512850000006,60.269164999000054],[5.924047,60.16648499900003],[5.914957500000071,60.033649497000056],[5.817266,59.983325999000044],[5.726609,60.00392149900006],[5.720516,60.17061999900005],[5.575043500000049,60.15387349800005],[5.747276,60.23856349600004],[5.584078500000032,60.25258249900003],[5.756293,60.399604999000076],[5.647027500000036,60.37364199600006],[5.446748500000069,60.156437],[5.245271500000058,60.20990050000006],[5.333692499000051,60.257888997000066],[5.236371,60.24559000100004],[5.201010500000052,60.290233501000046],[5.249560163000069,60.32177458800004],[5.170499500000062,60.37267699700004],[5.309116500000073,60.38820650100007],[5.262637500000039,60.50583249600004],[5.521925,60.42126850100004],[5.738453500000048,60.45921349900004],[5.735689,60.657379],[5.794102011000064,60.650521713000046],[5.698336426000026,60.69965533100003],[5.734445500000049,60.673576499000035],[5.708317500000021,60.47111150000006],[5.53338750000006,60.42834849600007],[5.339822,60.54158799900006],[5.591289500000073,60.62145999900008],[5.706546,60.75936149700004],[5.280158,60.54076],[5.207727999000042,60.621887],[5.114059,60.65854249900008],[5.281049,60.63255299900004],[4.94567,60.809383499000035],[5.469385500000044,60.65925199900005],[5.266979500000048,60.76729949800006],[5.324973579000073,60.823131946000046],[5.140299,60.84025249900003],[5.002784999000028,60.95379249800004],[5.115073512000038,60.965715414000044]],[[5.241315666000048,61.58393322900008],[5.461267500000076,61.589287],[5.181713,61.596099999000046],[5.241315666000048,61.58393322900008]],[[5.115073512000038,60.965715414000044],[5.116729500000076,60.96488949700006],[5.313839500000029,60.98683149700008],[5.115073512000038,60.965715414000044]]],[[[25.673977,71.17353799800003],[26.045212,71.11144250000007],[25.894638,71.010208],[26.225906500000065,71.06076799800007],[25.55792450000007,70.93201449900005],[25.391018,70.99469],[25.545727,71.03257750000006],[25.24226550000003,71.05068999900004],[25.739667999000062,71.07686599900006],[25.528230500000063,71.15545649800004],[25.673977,71.17353799800003]]],[[[23.781578,70.695778],[24.12865849900004,70.60249349800006],[23.773941,70.50978849800003],[23.623031500000025,70.69841750000006],[23.815597500000024,70.72293849900007],[23.781578,70.695778]]],[[[23.529517500000054,70.52244549900007],[23.642444,70.48944849900005],[23.146014500000035,70.27236950000008],[22.815429,70.41147699800007],[23.43688750000007,70.56320949900004],[23.354719,70.617737],[23.56452,70.62199399800005],[23.529517500000054,70.52244549900007]]],[[[23.369827500000042,70.78671249800004],[23.45733850000005,70.76512149900003],[22.798172,70.513825],[22.087011500000074,70.47561649800008],[22.310595,70.60153949900007],[21.928592500000036,70.64286049900005],[22.516258,70.72119899900008],[22.719646500000067,70.61257150000006],[22.769359500000064,70.678589],[22.679491,70.74093649800005],[22.997843,70.667839],[23.11976450000003,70.74383549900006],[23.015039500000057,70.79207599800003],[23.053814,70.81461349900007],[23.50584149900004,70.78686499900004],[23.369827500000042,70.78671249800004]]],[[[22.342153499000062,70.33776850000004],[22.956299,70.32978049900004],[23.009489,70.24874899800005],[22.342153499000062,70.33776850000004]]],[[[20.748051,70.09787749800006],[20.800907,70.056259],[20.370167,70.085861],[20.752233500000045,70.23168949800004],[20.80942550000003,70.20028699800008],[20.748051,70.09787749800006]]],[[[19.729519,70.22838599900007],[20.12632750000006,70.10135650000007],[19.742870500000038,70.056778],[19.50480250000004,70.23216249900008],[19.729519,70.22838599900007]]],[[[19.269611500000053,70.03578949900003],[19.674003500000026,69.995697],[19.12628550000005,69.787239],[18.704648999000028,69.92350749800005],[19.303821499000037,70.08371749800006],[19.269611500000053,70.03578949900003]]],[[[18.643114,69.86262499900005],[19.054355500000042,69.77339149900007],[18.761686500000053,69.67343150000005],[18.72437099900003,69.63607050000007],[18.837930500000027,69.60294349900005],[18.73374749900006,69.55794549900008],[18.006336,69.56680999800005],[18.27369409000005,69.62659492800003],[18.401043,69.63463599900007],[18.187591500000053,69.70993049800006],[18.620876500000065,69.69638049800005],[18.301355500000057,69.76936350000005],[18.639533499000038,69.804634],[18.645475500000032,69.690963],[18.748409500000037,69.68723299800007],[18.678631,69.701866],[18.732897,69.80738849900007],[18.643114,69.86262499900005]]],[[[17.482522500000073,69.60085249800005],[17.658531,69.48767099800006],[17.66749,69.59092699900003],[17.880442,69.579735],[17.919758,69.51934799800006],[17.815377999000077,69.45812249900007],[17.966917,69.51940149900008],[18.06773799900003,69.48912049800003],[17.93697,69.42363749800006],[18.080169500000068,69.426903],[17.84903550000007,69.22781349900004],[18.009228,69.19287849800008],[17.961891,69.177925],[17.460866999000075,69.20072899800005],[17.153118,69.01339749900006],[17.029537,69.03549950000007],[17.1658,69.08332799800007],[16.782012999000074,69.05759449900006],[17.211347500000045,69.19654099900004],[16.887241500000073,69.20046249900008],[17.184337500000026,69.28690349800007],[16.896857500000067,69.313469],[17.186859,69.351883],[16.86576850000006,69.376091],[17.49771499900004,69.42509449900007],[17.196076,69.496956],[17.40388850000005,69.48494699900004],[17.310571500000037,69.54144299900008],[17.68260950000007,69.46909350000004],[17.482522500000073,69.60085249800005]]],[[[15.619679500000075,68.95270549900005],[15.959016500000075,68.88904549800003],[15.813184,68.7472],[16.005573,68.75553849900007],[15.72271150000006,68.536804],[16.076727,68.74844349800003],[16.302828,68.715187],[16.08692,68.82158650000008],[16.123302500000023,68.84416199800006],[16.409397,68.85786449800008],[16.604114500000037,68.79657750000007],[16.594164,68.67980199800007],[16.633566,68.67171499900007],[16.45695199900007,68.55735799800004],[16.208227,68.55595449900005],[15.990130500000078,68.41610699900008],[16.011379,68.412079],[15.86366350000003,68.38358999900004],[15.874511,68.50976549900008],[15.661911,68.31165299900005],[15.326634500000068,68.35961149800005],[15.56163871700005,68.49955121900007],[14.980621499000051,68.27394849900003],[15.319273,68.46930699800004],[15.231805,68.51376350000004],[15.366633500000034,68.50199150000003],[15.177616,68.53307349800008],[15.205324999000027,68.57371499900006],[15.631016500000044,68.617607],[15.438093,68.71696449900003],[15.716285,68.693619],[15.464602500000069,68.753418],[15.682824,68.81733650000007],[15.551854,68.86223599900006],[15.619679500000075,68.95270549900005]]],[[[15.539591,69.00573749800003],[15.601443500000073,68.97971349800008],[15.416072500000041,68.90097799900008],[15.597289,69.11744699900004],[16.179676,69.28639999900008],[15.873977500000024,69.02712249900003],[15.539591,69.00573749800003]]],[[[15.17250450000006,68.931679],[15.317963500000076,68.91722099900005],[15.244818500000065,68.82391349800008],[15.392325500000027,68.84328449800006],[15.253855,68.75589],[15.41872,68.70305649900007],[15.033466500000031,68.57808699900005],[14.785534,68.63922899900007],[15.150149500000055,68.75284599900004],[14.52193950000003,68.60803999800004],[14.36857599900003,68.68449399900004],[14.583309,68.73529050000008],[14.436676,68.76052849900003],[14.549454500000024,68.81937450000004],[14.704345500000045,68.83254249800007],[14.68254,68.76516699900003],[14.703834499000038,68.74841299900004],[14.939134500000023,68.785446],[14.900189500000067,68.84649650000006],[14.968376,68.86647049800007],[15.152725,68.812866],[15.022504,68.94998949800004],[15.128936,69.02320850000007],[15.188699500000041,69.00209099900007],[15.17250450000006,68.931679]]],[[[15.165045500000076,68.44262699800004],[14.206444500000032,68.17415599800006],[14.327897,68.33840199800005],[14.509889500000043,68.33101649900004],[14.416385500000047,68.39362349800007],[15.19480150000004,68.47393050000005],[15.165045500000076,68.44262699800004]]],[[[14.14168701400007,68.27466567700003],[14.169512,68.25863699800004],[13.460458,68.08541099900003],[13.478425,68.20449049800004],[13.661848,68.23725899900006],[13.511562500000025,68.24778],[13.688828500000056,68.30903649800007],[14.272526,68.29914850000006],[14.14168701400007,68.27466567700003]]],[[[13.04957950000005,68.03257750000006],[13.187348500000041,67.976677],[13.014825,67.99353799800008],[12.97647,67.94109349800004],[13.08421050000004,67.92895499800005],[12.796074,67.84101850000008],[12.958006,68.02886199900007],[13.19181450000002,68.101097],[13.04957950000005,68.03257750000006]]],[[[12.551321,66.13022599900006],[12.667357500000037,66.103691],[12.298794500000042,66.01961499800007],[12.589514500000064,66.22785199900005],[12.629095,66.156784],[12.551321,66.13022599900006]]],[[[10.725237,64.86402149900005],[11.120891,64.98121649600006],[11.05366454500006,64.93660237400007],[11.367821500000048,64.94194049900005],[11.143732,64.82415749700004],[11.077328500000021,64.89194499900003],[10.832070500000043,64.84275799700004],[11.031305850000024,64.92831044300004],[10.725237,64.86402149900005]]],[[[8.587555,63.610359],[9.201086,63.565173998000034],[8.527718,63.42691399600005],[8.296058500000072,63.47917950000004],[8.74709050000007,63.56145850000007],[8.544486,63.59365850000006],[8.587555,63.610359]]],[[[8.447085499000025,63.28402349900006],[8.579752,63.18283099700005],[8.358337500000061,63.160995498000034],[8.240482500000041,63.264587497000036],[8.447085499000025,63.28402349900006]]],[[[8.098034,63.468765500000075],[8.21287650000005,63.37200950000005],[7.998843,63.32286449600008],[7.932767,63.344970500000045],[8.043354,63.33882150000005],[7.787214,63.408584500000075],[8.098034,63.468765500000075]]],[[[7.617825,63.11251049900005],[7.727075,63.006405],[7.352569500000072,62.98101399900008],[7.462165,63.05691899900006],[7.626714,63.06288899900005],[7.643898,63.07329199700007],[7.597258,63.10822299900008],[7.638170500000058,63.11698899900006],[7.617825,63.11251049900005]]],[[[5.897257500000023,62.421013],[6.11445150000003,62.352317998000046],[5.764179500000068,62.203823500000055],[5.529337500000054,62.25860199600004],[5.831122256000072,62.28298388800005],[5.857613,62.31460150000004],[5.764841,62.323269],[5.897257500000023,62.421013]]],[[[5.699188,60.058669999000074],[5.703725,59.99919499500004],[5.624819499000068,59.89454249900007],[5.340132500000038,60.02786999500006],[5.425635367000041,60.07315416400007],[5.699188,60.058669999000074]]],[[[5.499698500000022,59.91142249600006],[5.536162,59.791172],[5.403593500000056,59.75078599800008],[5.274518500000056,59.88779049900006],[5.364507,59.969269],[5.499698500000022,59.91142249600006]]],[[[5.05945950000006,59.85404599700007],[5.179249,59.87766649800005],[5.39621949900004,59.71174250000007],[5.298333,59.674747499000034],[5.346907500000043,59.698738],[5.241508500000066,59.743931],[5.249395,59.60264950000004],[5.183555500000068,59.57461550000005],[5.151623,59.67523550000004],[5.245591999000055,59.75310149900008],[5.105754,59.76964949900008],[5.130981284000029,59.82973899600006],[5.05945950000006,59.85404599700007]]],[[[4.953383,61.90943899700005],[5.257675500000062,61.83687199700006],[4.959802500000023,61.77524199900006],[4.903413500000056,61.81495299900007],[5.001847500000054,61.836486999000044],[4.818475500000034,61.85469449900006],[4.953383,61.90943899700005]]],[[[5.21904550000005,60.48925],[5.244192,60.414787499000056],[5.173365499000056,60.38851149900006],[4.961089500000071,60.56077599900004],[5.21904550000005,60.48925]]],[[[5.000170500000024,60.42187899900006],[5.133132500000045,60.33346549700008],[5.163972,60.21688850000004],[5.079828500000076,60.16420749700006],[4.948168,60.276031499000055],[5.067186,60.31352250000003],[5.015766500000041,60.30218499700004],[4.956776500000046,60.42578149800005],[4.986847,60.458605996000074],[5.01680250000004,60.42941299600005],[5.000170500000024,60.42187899900006]]],[[[5.000149,61.19904350000007],[5.023131,61.11022949900007],[4.869070710000074,61.07303522800004],[4.906099500000039,61.04799249700005],[4.752019500000074,61.099673996000035],[4.820931225000038,61.08130329000005],[4.883923500000037,61.21157849800005],[5.000149,61.19904350000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO0","LEVL_CODE":1,"CNTR_CODE":"NO","NAME_LATN":"NORGE","NUTS_NAME":"NORGE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.115073512000038,60.965715414000044],[5.026263,61.01004050000006],[5.986885,61.128764997000076],[6.411997500000041,61.05647299700007],[6.645289500000047,61.17484649900007],[6.850829500000032,61.14572549700006],[7.035421,60.99983999800003],[6.84106550000007,60.882595],[7.044801500000062,60.985458496000035],[7.154517500000054,60.93813299900006],[7.121905500000025,60.86051199800005],[7.197031500000037,60.92475499900007],[7.026663500000041,61.08794399900006],[7.478136500000062,61.098621500000036],[7.412843,61.13557799700004],[7.712671,61.23199450000004],[7.432381,61.199321499000064],[7.312189,61.29982749700008],[7.585154,61.49429699900003],[7.284697,61.39589299700003],[7.348169,61.34857549900005],[7.247618,61.30030450000004],[7.395241500000054,61.18244450000003],[6.992821,61.10582799900004],[6.927117500000065,61.14121999900004],[7.17012,61.19993199800007],[6.630224500000054,61.21004499900005],[6.753228500000034,61.41973099900008],[6.473639500000047,61.115188499000055],[6.006493,61.162735],[6.050285,61.19412249700008],[5.394584,61.069965496000066],[5.076056,61.179718],[5.312890500000037,61.18884299900003],[4.94716950000003,61.25884999900006],[5.209688823000022,61.33452840500007],[5.026415500000041,61.37598399900003],[5.167576,61.42851999900006],[4.977364500000022,61.41730899700008],[5.240735500000028,61.47474299800007],[5.588792499000022,61.44279099800008],[5.751739500000042,61.492759499000044],[5.843049,61.45913699700003],[5.712819,61.50769399900008],[5.481126,61.454650998000034],[5.442402,61.46443949700006],[5.461188499000059,61.48801799600005],[5.177923999000029,61.51424049900004],[5.39946,61.551673996000034],[5.241315666000048,61.58393322900008],[4.994847999000058,61.591152],[5.414846500000067,61.646262999000044],[4.991695,61.62740699700004],[4.969778,61.72112249600008],[5.21751449900006,61.72261449900003],[5.151150500000028,61.75493249700003],[5.296247,61.77143499700003],[5.092194,61.76363749700005],[5.382925,61.84609199600004],[5.300301500000046,61.85786450000006],[5.434949,61.917640497000036],[5.780999500000064,61.852504500000066],[5.642197500000066,61.81269849900008],[5.985106500000029,61.84537099900007],[5.913162500000055,61.731254498000055],[6.010346500000026,61.842471999000054],[6.178206500000044,61.76468349700008],[6.214309,61.77514649900007],[6.079197500000021,61.84675599900004],[6.483179,61.80439750000005],[6.613900500000057,61.85201649600003],[6.586079,61.86867150000006],[6.846349,61.870303996000075],[6.630262,61.89056],[6.429300500000068,61.830268999000054],[5.911377999000024,61.882831499000076],[5.808180500000049,61.85716599700004],[5.731962499000076,61.89316199700005],[5.988245500000062,61.905822999000065],[5.458462500000053,61.93934999900006],[5.178933,61.895335998000064],[4.990589,61.99135599600004],[5.123834,62.032745498000054],[5.142885471000056,61.95648816400006],[5.288158,61.94990899900006],[5.426334,62.018695999000045],[5.089433,62.16728949900005],[5.185808,62.21553399700008],[5.491458,62.01478749900008],[5.623765500000047,62.06705450000004],[5.405087500000036,62.12742949900007],[5.485529,62.20001999900006],[5.73594,62.08772650000003],[5.671969,62.174431],[5.851422500000069,62.20307499900008],[5.95038099900006,62.156822],[5.847182,62.00476449700005],[5.981478500000037,62.142124],[6.325065500000051,62.06060049900003],[5.931158,62.21788399700006],[6.117795,62.18908299700007],[5.96181050000007,62.24787899900008],[6.328697,62.37550749700006],[6.580707500000074,62.14685449900003],[6.545071,62.104973],[6.610225999000022,62.13547149900006],[6.555007,62.19385549600008],[6.653805500000033,62.193794497000056],[6.402287500000057,62.357982496000034],[6.723693500000024,62.45203799700005],[6.914581,62.41138099800003],[6.884847500000035,62.38205349600008],[7.007536,62.283473997000044],[7.012420999000028,62.17502599900007],[6.873665,62.08268749700005],[7.20569050000006,62.10144799900007],[6.96163,62.11530299700007],[7.038890500000036,62.17032549700008],[7.047532499000056,62.27912899900008],[7.428215500000022,62.25268149900006],[6.987393,62.31317899900006],[6.941239500000052,62.42668149900004],[6.772653,62.46071199600004],[6.811594500000069,62.482394999000064],[6.220586,62.46456149800008],[6.717913,62.49196599600003],[6.461608500000068,62.51434349800007],[6.573409500000025,62.542159999000035],[6.256442500000048,62.52973549900008],[6.334281500000031,62.61331949700008],[6.568112500000041,62.62427899700003],[6.611302500000022,62.55686549600006],[6.660633500000074,62.632385500000055],[7.045805,62.65523149900008],[7.138921500000038,62.52282349800004],[7.128197500000056,62.60631549900006],[7.166881500000045,62.63157249800008],[7.55996,62.53118499900006],[7.786336,62.57716750000003],[7.396082,62.628738499000065],[7.970172500000047,62.74502199700004],[8.146601500000031,62.688270500000044],[8.019416,62.751445999000055],[8.06985,62.77651599900008],[7.349656500000037,62.69644949900004],[7.722468500000048,62.79231649900004],[6.98348149900005,62.724308],[6.969040500000062,62.77918599700007],[7.018433,62.81851599700008],[7.240911500000038,62.80947899900008],[6.898579,62.90984750000007],[7.301491500000054,63.010261496000055],[7.450778500000069,62.90481949700006],[7.685032,62.96850599900006],[7.791710500000022,62.95953349600006],[7.677089500000022,62.898002500000075],[8.036038500000075,62.96656399800008],[8.11747,62.921828997000034],[8.13680850000003,62.822223500000064],[8.530576,62.66896449600006],[8.55466750000005,62.717293],[8.190963,62.829601499000034],[8.136116,62.94702149900007],[7.988732,63.01142899800004],[7.880380500000058,62.98959749800008],[8.107145499000069,63.10542299900004],[8.142638,63.07119349900006],[8.132346,63.034172],[8.333088,62.96408099800004],[8.337154500000054,62.891792500000065],[8.700226500000042,62.81935499900004],[8.421752999000034,62.94767399800003],[8.663505499000053,62.97121799800004],[8.470663,62.97100050000006],[8.594899,63.02206049900008],[8.365225,62.96965399900006],[8.322330500000021,62.99421299900007],[8.49464050000006,63.01892850000007],[8.22018349900003,63.029011],[8.156400500000075,63.113617],[8.764103,63.184509999000056],[8.985986500000024,63.20794299800008],[8.832189,63.20198],[8.490692,63.27751149900007],[8.765663999000026,63.34213649800006],[8.650367,63.40087899900004],[8.755111500000055,63.42434099900004],[8.904203500000051,63.38735199800004],[9.01071,63.46447],[9.177391,63.37377950000007],[9.092917500000055,63.28715899900004],[9.501772,63.39708699700003],[9.249117,63.36782849600007],[9.176959,63.41464599900007],[9.297754401000077,63.448047849000034],[9.147209,63.48814],[9.585823,63.528888496000036],[9.491575,63.57952499900006],[9.75341050000003,63.645359],[9.71493350000003,63.615505],[9.977688,63.44161599700004],[9.823175500000048,63.31224450000008],[10.208758500000044,63.322869],[10.059104,63.38721099900005],[10.189022,63.45037849900007],[10.85359,63.439093496000055],[10.912328500000058,63.46505349900008],[10.756933,63.520218],[10.941581,63.56428899900004],[10.627927,63.54674150000005],[11.458975,63.80233749900003],[11.072477500000048,63.858989500000064],[11.501289500000041,64.00521850000007],[11.222313,64.074219],[10.591203500000063,63.80435549900005],[10.962953500000026,63.90656649700003],[11.070417500000076,63.841949499000066],[10.955013501000053,63.74560949900007],[10.170315,63.526134496000054],[9.948584500000038,63.495971498000074],[9.805798500000037,63.62687299900006],[10.130084,63.75410449900005],[9.52912150000003,63.71239849600005],[9.803867500000024,63.766426],[9.559135500000025,63.769199],[9.678071,63.84419649900008],[10.224334,63.927921498000046],[9.946816500000068,63.90663199800008],[10.226982,64.00061799900004],[9.951425500000028,63.94597250000004],[10.010268949000022,64.04565076200004],[9.932503500000053,64.060982],[10.016330268000047,64.06744965300004],[10.234300500000074,64.10188299600003],[10.120101,64.121704],[10.212196500000061,64.20630649900005],[10.374837,64.17877199600008],[10.274836500000049,64.23419149700004],[10.444528500000047,64.35304249800004],[10.68307,64.36034399900007],[10.45395,64.40396099800006],[10.634974130000046,64.39100241600005],[10.548171,64.44136050000003],[10.83981250000005,64.368828],[10.654607,64.45243049900006],[10.948439500000063,64.50749949600004],[10.976382,64.53115849900007],[10.906779500000027,64.57653050000005],[10.973181500000067,64.60691050000008],[11.340755500000057,64.43691999700007],[11.225456,64.311905],[11.407744500000035,64.36854549600008],[11.352719500000035,64.41844949900008],[11.641216500000041,64.47501099800007],[11.42755150000005,64.47552499900007],[11.725962500000037,64.58479299900006],[11.515246500000046,64.56695549800003],[11.379855,64.66954049900005],[11.86674750000003,64.79622649700008],[11.603879,64.814125],[11.817444,64.86047349600005],[11.653197500000033,64.83556349900005],[11.694047001000058,64.87286399600004],[12.136008884000034,64.96383255800004],[11.615549,64.86773699900004],[11.65877050000006,64.862915],[11.392232,64.75153349900006],[11.182071500000063,64.74090599900006],[11.446765999000036,64.809616],[11.240288,64.81357549900008],[11.277119500000026,64.845711],[11.514355244000058,64.87855370700004],[11.278464500000041,64.85742949800004],[11.960979500000064,65.07714799700005],[12.101823,65.11344899900007],[12.121571500000073,65.18907949900006],[12.337297500000034,65.11171699800008],[11.981972,65.07081549600008],[12.046106500000064,65.06244999900008],[12.119634500000075,65.05026249700006],[12.092824,65.03982549900007],[12.187348500000041,65.00011149700003],[12.182329,64.98138099900007],[12.214979,65.01567099900006],[12.171146500000077,65.04820999800006],[12.460707500000069,65.07824699900004],[12.495921500000065,65.105568],[12.399293,65.098755],[12.964295500000048,65.32212049900005],[12.474547500000028,65.13399499900004],[12.350988500000028,65.15914899900008],[12.510954,65.19638799800003],[12.336974,65.23342099800004],[12.46169,65.25754549900006],[12.31042749900007,65.25910949900003],[12.471449500000062,65.37045849900005],[12.041082500000073,65.21619399900004],[12.140350500000068,65.28169249900003],[12.085761,65.30515299900003],[12.146955499000057,65.37075799700006],[12.276143,65.39269249800003],[12.158635,65.38862599800007],[12.353930500000047,65.46737649800008],[12.183487999000022,65.44551849900006],[12.292716,65.58846299700008],[12.630449,65.41567249600007],[12.781976500000042,65.46252449900004],[12.401527500000043,65.537445],[12.347975,65.62732699900005],[12.494364,65.73480999900005],[12.630283314000053,65.69639763800006],[12.537199,65.75340999900004],[12.646986,65.81443799600004],[12.718809,65.77714549900003],[12.72851,65.80428299800008],[12.597691500000053,65.82546249900008],[12.666879418000065,65.92070801000006],[12.380134500000054,65.89411099900008],[13.053273500000046,66.07566850000006],[12.657043500000043,66.04336549900006],[12.830141,66.12840199900006],[13.346362,66.22772149900004],[13.609442500000057,66.20105749800007],[13.548763,66.10309549800007],[13.819278,66.137657],[13.599074500000029,66.22496049800003],[14.147304500000075,66.33046699900007],[13.009953500000051,66.19128399800007],[13.562423500000023,66.30467999600006],[13.027488,66.32476049900004],[12.990386999000066,66.34877799900005],[13.159712500000069,66.40702049800007],[13.172011500000053,66.46587349900005],[12.978532,66.529541],[13.256189500000062,66.446121],[13.425744,66.47429649800006],[13.262984500000073,66.52902149900007],[13.672662,66.51731099900007],[13.470007,66.565529],[13.733414500000038,66.60917650000005],[13.185444,66.56871799900006],[13.413904,66.62222299900003],[13.16772,66.65842449900003],[13.566408,66.64874249800005],[13.207289500000059,66.71688099900007],[14.000651,66.79788950000005],[13.610096,66.84452799900004],[13.68425047900007,66.86556359300005],[13.492176,66.94919649900004],[14.013185500000077,66.96194450000007],[14.06197050000003,66.99144749900006],[14.019997500000045,67.04334999900004],[14.052112500000021,67.08169550000008],[14.173709975000065,67.01579152300008],[14.124613,67.07179250000007],[14.008136,67.09213299900006],[14.067618500000037,67.16149150000007],[14.302209,67.18673699800007],[14.216777,66.99249250000008],[14.42914905200007,67.08494146600003],[14.297962,67.10489649800007],[14.342555,67.17453],[14.602729091000072,67.19427639000003],[14.511739500000033,67.20764899900007],[15.006282,67.23958599900004],[14.88538550000004,67.16931899800005],[14.968609,67.11983499800004],[14.999890500000049,67.12169649800006],[14.917812499000036,67.16667199800008],[15.088397999000051,67.23653399900007],[15.433605,67.10498049900008],[15.481187,67.18446349900006],[15.349717,67.22185449900007],[15.430528500000037,67.26048299900003],[15.037823500000059,67.27246150000008],[15.162948499000038,67.32976499900008],[14.308716,67.26377099900003],[14.637446500000067,67.43815649800007],[14.943926,67.44770749900005],[14.651400500000022,67.47698199900003],[15.001051811000025,67.57711863800006],[15.080581499000061,67.52973949800008],[14.923469500000067,67.48969249900006],[15.4537,67.49208049800006],[15.582617,67.37529749900006],[15.633104446000061,67.41328633800003],[15.515371499000025,67.44784549800005],[15.650709213000027,67.53825820800006],[15.160784500000034,67.63269799900007],[15.339144,67.69324499800007],[15.247291500000074,67.702095],[15.311404,67.73804449900007],[15.654401,67.68859849900008],[15.428495500000054,67.75768299800006],[15.855372500000044,67.70989999900007],[15.43736750000005,67.809578],[14.734998,67.67552199900007],[15.14555050000007,67.76268799800005],[14.75716,67.80435949900004],[15.136513500000035,67.868988],[14.82857799900006,67.860016],[14.889670500000022,67.88421649900005],[16.00116550000007,67.99621600200004],[15.526616,68.06521600100007],[15.395943500000044,68.03652199800007],[15.440778500000022,68.02494799900006],[15.414783500000055,67.99591049900005],[15.275926500000025,68.05061299900007],[15.406996500000048,68.14228050200006],[15.67317,68.18434150000007],[15.725988500000028,68.11899549600008],[15.597519,68.11167149900007],[15.74319950000006,68.087616],[16.062347499000055,68.255936],[16.089401,68.20642099900004],[16.019762,68.18904099800005],[16.140629,68.18688199900004],[16.074756500000035,68.05342850000005],[16.22637750000007,67.96387499900004],[16.203220500000043,67.898308],[16.49381850000003,67.81392649800006],[16.237539500000025,67.902992],[16.282963,67.98599950000005],[16.189962500000036,68.02142349900004],[16.284780500000068,68.026779],[16.459911500000032,67.91499349900005],[16.316387,68.027191],[16.52378250000004,67.96525550000007],[16.354823500000066,68.04674549800006],[16.740179,68.08623499900006],[16.339367,68.094261],[16.24048250000004,68.137703],[16.60742,68.18205249800008],[16.140442500000063,68.29213550000003],[16.819980500000042,68.15521999800006],[16.27568050000002,68.37313849900005],[17.170841,68.393112],[17.33622,68.28037999800006],[17.318298500000026,68.19080349900008],[17.407579500000054,68.26064299800004],[17.249164,68.394539],[17.900738,68.41780799900005],[17.471044500000062,68.46656049900008],[17.573683,68.53850549900005],[17.54451,68.55082699800005],[17.211914,68.45962499800004],[17.01371,68.45533750000004],[17.105293500000073,68.484131],[16.995945,68.52622999900007],[16.557794,68.44740249900008],[16.501896,68.44715099800004],[16.530721500000027,68.48244499800006],[16.48231019900004,68.50168901000006],[16.515053,68.46984099800005],[16.075683500000025,68.414055],[16.232881500000076,68.54445649800005],[16.579589500000054,68.54136699900005],[16.628774500000077,68.53780349900006],[16.564274500000067,68.58321399900007],[16.614227500000027,68.64302099800005],[16.964586500000053,68.71334099900008],[17.685274,68.67411049900005],[17.251898,68.75971199800006],[17.805559,68.74999249900003],[17.474293,68.83225999900003],[17.853133999000022,68.86698899900006],[17.447077,68.90790550000008],[17.696062,69.08660849800003],[17.636711,69.10508749900004],[18.166633499000056,69.15019999900005],[17.96954549900005,69.22856150000007],[18.23868550000003,69.48264299800007],[18.499069,69.367111],[18.474834500000043,69.20949549900007],[18.542671,69.32290649800007],[19.00698650000004,69.28672049900007],[18.442138500000056,69.49892449900005],[18.828508500000055,69.56273650000003],[18.987579500000038,69.38976299800004],[19.349617,69.31079849800005],[19.231395500000076,69.235595],[19.547487500000045,69.21487449800003],[19.351875500000062,69.36683649800005],[19.073311,69.392891],[18.96809950000005,69.46321849900005],[19.07519426600004,69.54529346400005],[18.934316500000023,69.599945],[19.27431850000005,69.77920549900006],[19.732448500000032,69.80428249900007],[19.672811500000023,69.48411549900004],[19.437654500000065,69.39199049900003],[19.508491500000048,69.39473699900003],[19.732485,69.48689249800003],[19.73015,69.607582],[20.14599050000004,69.58264149900003],[19.76941850000003,69.67984749800007],[20.134111500000074,69.93837],[20.394979500000034,69.89038799800005],[20.25130850000005,69.68190749900003],[20.349264,69.62082649800004],[19.900448,69.27372],[19.930265500000075,69.26666250000005],[20.268589,69.396126],[20.398016,69.58557149800004],[20.821378500000037,69.49770350000006],[20.4823,69.61501299800005],[20.525961,69.72260299800007],[20.459503,69.76207749800005],[20.710785,69.79328899900008],[20.791124500000024,69.84407799800005],[20.749731,69.86638650000003],[21.027044500000045,69.95645150000007],[21.082113500000048,69.95459749800006],[20.844524500000034,69.85420250000004],[20.983516500000064,69.77506249800007],[21.325062,69.90923299800005],[21.18428250000005,69.93567649900007],[21.241577,70.00890349900004],[22.140845,69.74519350000003],[21.88821050000007,69.835617],[22.026851500000078,69.84824349900003],[21.862396,69.93569949900007],[22.091429,70.02889999800004],[21.77614,70.03570499800003],[22.06965850000006,70.12821199900003],[21.727436,70.05191049900003],[21.19089,70.22124049900003],[21.424858,70.222824],[21.478477276000035,70.20036185200007],[21.493496359000062,70.19927205500005],[21.47370150000006,70.29925549900008],[21.56427,70.323387],[21.832088500000054,70.16875450000003],[21.78227050000004,70.25872049900005],[21.983658,70.32945249800008],[22.011963,70.31783299800003],[21.948728500000072,70.28130349800006],[22.298141500000042,70.27687850000007],[22.269542500000057,70.13492599800003],[22.541012,70.113289],[22.290935500000046,70.16215499800006],[22.349849499000072,70.25868299800004],[22.96682,70.189369],[22.848073456000066,70.11755874200003],[23.002601500000026,70.13343799900008],[22.952436499000044,70.040924],[23.142261500000075,69.97520449800004],[23.00179650000007,69.91750349800003],[23.407093,69.96468349900005],[23.52833,70.01591499900007],[23.132837,70.08557150000007],[23.390547,70.12302399900005],[23.179638,70.21624750000007],[23.507255500000042,70.24166049900003],[23.246681500000022,70.24819199900003],[23.605201500000078,70.29695149800006],[23.429792500000076,70.31327050000004],[23.802875,70.47930149800004],[24.329584500000067,70.44633499900004],[24.09067550000003,70.54149649800007],[24.298256,70.68613450000004],[24.696213,70.63238549800008],[24.236952,70.83589149900007],[24.708558500000038,70.87100999900008],[24.553296500000044,70.97564699800006],[25.020571,70.96941399900004],[25.048740500000065,70.950287],[24.918836500000054,70.88396450000005],[25.115078,70.93007650000004],[25.232065,70.79688249900005],[25.395689,70.88725299900005],[25.298832,70.95856499800004],[25.338186500000063,70.979912],[25.547411,70.91419199900008],[25.39456,70.855156],[25.44389350000006,70.84555049900007],[25.91175450000003,70.87564849900008],[25.086530500000038,70.51461049800008],[25.241384500000038,70.520706],[25.056989500000043,70.48918150000003],[25.22985650000004,70.46246349900008],[25.039865500000076,70.35273749900006],[25.183798,70.32165549900003],[24.936825,70.23127],[24.912155,70.09609199900007],[25.153381500000023,70.06489549900004],[25.538208,70.31125649900008],[25.382824,70.32743849900004],[25.440464,70.36740049800005],[26.57236850000004,70.95436850000004],[26.70399499900003,70.96577449800003],[26.730856,70.90872950000005],[26.654697499000065,70.83110799900004],[26.733936500000027,70.82489799800004],[26.286564,70.65195449900006],[26.640539,70.646553],[26.475652500000024,70.35486599900008],[26.518347,70.34874749900007],[27.03247650000003,70.48385599800008],[26.951067,70.54679099900005],[27.025823500000058,70.61801150000008],[27.280205,70.58753949800007],[27.06658150000004,70.70458999900006],[27.355761,70.75415799900003],[27.285433,70.83248149900004],[27.702987500000063,70.800621],[27.101142500000037,70.93241899800006],[27.35516399900007,70.94692999900008],[27.200126500000067,71.01464850000008],[27.275039500000048,71.04461650000007],[27.550026,70.95600899800007],[27.574959,71.01279449900005],[27.487985500000036,71.06152349800004],[27.66123,71.126999],[28.548399,70.96723950000006],[28.240961,70.79145799800006],[27.790540500000077,70.79557049800007],[27.83346750000004,70.78317250000003],[27.753228999000044,70.75068649800005],[28.142963500000064,70.73532849800006],[27.63097,70.62425249800003],[28.305256,70.687088],[27.812294,70.486458],[27.855804500000033,70.43014549900005],[28.510006,70.44792999800006],[28.55335050000008,70.50961299800008],[28.397211,70.542122],[28.627989,70.67738350000008],[28.516052,70.72720349900004],[28.96657349900005,70.88494099900004],[29.421539,70.732119],[29.244068,70.670433],[29.27265,70.65177149800007],[30.202276,70.67542249800005],[30.107942500000036,70.64891799900005],[30.333885,70.60903149800004],[29.997783499000036,70.53657549900004],[30.618822,70.55101],[30.58019050000007,70.51128349800007],[30.96376550000008,70.44050599900004],[31.060808,70.28778850000003],[29.90366750000004,70.06065349900007],[28.73865071000006,70.15556738900005],[28.575159,70.11865250000005],[29.660585500000025,69.96182999900003],[29.350151,69.86261749900007],[30.030296500000077,69.82659149800008],[29.699119499000062,69.75655349900006],[29.767906536000055,69.82349093300007],[29.459511,69.648964],[30.076742500000023,69.76901999900008],[29.896385,69.70440649900007],[29.951505500000053,69.58993549900003],[29.916139500000043,69.69289399800005],[30.199936,69.77079749800004],[30.130657,69.852539],[30.185869,69.879448],[30.370466,69.844124],[30.421198,69.689911],[30.293487,69.66351299800004],[30.452156,69.680496],[30.43191550000006,69.81462799900004],[30.839926,69.775808],[30.95453850000007,69.63243199900006],[30.813123,69.52871399900005],[30.08532450000007,69.65810450000004],[30.188423500000056,69.56845749900003],[30.00879450000008,69.415908],[29.288473,69.29618099800007],[29.313494500000047,69.22850299800007],[29.242261,69.11306250000007],[28.92968,69.05190499900004],[28.805792500000052,69.11114699900008],[28.831907,69.22435249800003],[29.336974,69.47831049900003],[29.13434250000006,69.69532949900008],[28.404522,69.81841799900008],[27.984522,70.01396549800006],[27.959615,70.09209449800005],[27.037634500000024,69.91038799900008],[26.490588,69.94501949900007],[25.950607,69.69655049900007],[25.89156,69.66550199800008],[25.976791500000047,69.61067350000008],[25.702101,69.25366150000008],[25.777502500000026,69.018279],[25.157167,68.79989199900007],[25.116428,68.63958749900007],[24.903199500000028,68.55459099900008],[23.871476,68.83651899800003],[23.167591,68.62851849800006],[22.374526,68.71666699900004],[22.340785,68.82722549900006],[21.983611500000052,69.07289349900003],[21.627087500000073,69.27658799800008],[21.27882150000005,69.31188399800004],[20.987576500000046,69.191927],[21.108676,69.10392949900006],[21.057545,69.03628949800003],[20.548636500000043,69.05996850000008],[20.060047500000053,69.045759],[20.306589,68.92618199900005],[20.33587250000005,68.80231249900004],[19.93750250000005,68.55793949900004],[20.226541500000053,68.49081250000006],[19.921397,68.356013],[18.405688500000053,68.58187699900003],[18.125925,68.53651599800008],[18.101093001000038,68.406045],[18.151354,68.19878999900004],[17.899761500000068,67.96937199900003],[17.281524,68.11881499900005],[16.738120500000036,67.91420949800005],[16.407672,67.53409999900003],[16.15800150000007,67.519159],[16.089829,67.43527850000004],[16.404099501000076,67.20497449900006],[16.387759,67.04546249900005],[15.377226500000063,66.48430399600005],[15.453994500000022,66.34523350000006],[15.484732,66.28245849800004],[14.516289,66.13257899900003],[14.625476500000048,65.81180849900005],[14.49877250000003,65.52129749600005],[14.50683250000003,65.30972899800008],[14.378797500000076,65.24762049800006],[14.325985,65.118916],[13.654257500000028,64.58034049800006],[14.113869500000021,64.46248450000007],[14.157109,64.19505449600007],[13.967524500000025,64.00796999900007],[13.211107500000026,64.09536949900007],[12.683565,63.97422349900006],[12.149767,63.593946998000035],[12.212877,63.47859199900006],[11.974581,63.269227999000066],[12.052455500000065,63.183444499000075],[12.218231,63.00033249900008],[12.074687999000048,62.902544997000064],[12.136381500000027,62.74791649700006],[12.056142,62.61191849800008],[12.25465950000006,62.331024496000055],[12.29937,62.26749399900007],[12.137665,61.72381699700003],[12.419609,61.56298099900005],[12.569316,61.568746496000074],[12.870847500000025,61.35649499700003],[12.670176500000025,61.055976497000074],[12.22399,61.013078],[12.606881500000043,60.512742496000044],[12.606053,60.405925499000034],[12.498795500000028,60.323652],[12.541906500000039,60.19337849700008],[12.499544999000022,60.09765699700006],[12.174292,59.88980850000007],[11.839729,59.84076699900004],[11.926970500000039,59.79047599800003],[11.939878500000077,59.69458099700006],[11.691129,59.58954749900005],[11.826196500000037,59.23784999700007],[11.632557,58.90830650000004],[11.457448500000055,58.888435497000046],[11.460827,58.98865849800006],[11.476778,58.98445499800005],[11.374898,59.09957849700004],[11.384853146000069,59.117312139000035],[11.385243500000058,59.118007498000054],[11.340374784000062,59.11173018900007],[11.330404740000063,59.11033547200003],[10.825053,59.15331250000003],[10.723924499000077,59.26906599700004],[10.814114500000073,59.32770549600008],[10.660101999000062,59.33565899800004],[10.686118,59.48940699800005],[10.581817500000056,59.756759500000044],[10.657540500000039,59.87100199900004],[10.764788,59.829544499000065],[10.759619,59.906375996000065],[10.64198,59.911508499000035],[10.506262,59.87787550000007],[10.497314500000073,59.788338499000076],[10.617811,59.57157149800008],[10.421314,59.525081500000056],[10.439196500000037,59.66585149800005],[10.212189500000022,59.73728199900006],[10.319881500000065,59.69134350000007],[10.374606,59.67696749900006],[10.410937500000045,59.622604497000054],[10.389866999000049,59.537372496000046],[10.23252750000006,59.56869149700003],[10.384464500000036,59.438155996000035],[10.493912500000022,59.43849199700003],[10.461864500000047,59.37163549700006],[10.517841500000031,59.293377],[10.386558500000035,59.27502049700007],[10.273399500000039,59.04173649900008],[10.017867,59.048832],[10.040186,58.97815299800004],[9.969417500000077,58.964656998000066],[9.83919700100006,59.044178997000074],[9.680050218000076,59.055061142000056],[9.752187500000048,58.988582500000064],[9.289463,58.83913049500006],[9.467710500000067,58.829845497000065],[9.365971,58.771348999000054],[9.082190499000035,58.74871449600005],[9.057591500000058,58.72521599800007],[9.244501,58.72533049900005],[9.076271,58.686345998000036],[9.229282500000068,58.688499499000045],[8.96709965100007,58.59442495500008],[8.992434500000059,58.55048349700007],[8.713581,58.436935497000036],[8.728853,58.39339849900006],[8.516029500000059,58.25892999800004],[8.391825500000039,58.25569199800003],[8.24362161700003,58.15584307300003],[8.256075,58.13458650100006],[8.185152,58.142941499000074],[8.061826500000052,58.11951050000005],[8.087569,58.18608849600008],[8.031234,58.23595050000006],[8.048012,58.16465399500004],[7.951262500000041,58.075958498000034],[7.513357499000051,58.00359750000007],[7.179073999000025,58.05886099600008],[7.05503,57.97974799800005],[7.038199,58.02142699800004],[7.146521,58.07286450000004],[6.98239650000005,58.03102099900008],[7.061141,58.119189997000035],[6.938757,58.05059799700007],[6.813322500000027,58.09625249700008],[7.051818,58.14051799900005],[6.798205500000051,58.18960549900004],[6.867762,58.132240498000044],[6.78057,58.11681349500003],[6.747517500000072,58.191993497000055],[6.73773650000004,58.11087049900004],[6.819480500000054,58.064850497000066],[6.553461500000026,58.117786498000044],[6.89118,58.27436049900007],[6.719107999000073,58.21998599800003],[6.672523,58.28101349800005],[6.380909,58.275402],[6.438839,58.290819499000065],[6.040288,58.379020498000045],[5.981829851000043,58.454346376000046],[5.874972,58.426685500000076],[5.921966174000033,58.463206064000076],[5.654306500000075,58.560295],[5.491752,58.75477999900005],[5.575934500000074,58.958621995000044],[5.686279,58.92847499900006],[5.574271500000066,59.03034249900003],[5.769078500000035,58.96493149600008],[5.746338499000046,58.85974899800004],[5.893873500000041,58.967807995000044],[6.176128500000061,58.83488099900006],[6.279346,58.839367],[6.073616,58.903625496000075],[5.995267500000068,58.968890997000074],[6.091457500000047,58.985473496000054],[5.889861500000052,59.07323049900003],[6.112402,59.140621],[6.022733,59.143718496000076],[6.217452500000036,59.26629649700004],[6.561808500000041,59.32115949800004],[6.155891500000052,59.260627498000076],[6.113465,59.301426],[6.241003500000033,59.34484099900004],[6.020808500000044,59.34091949900005],[6.110799426000028,59.40670361000008],[5.941965500000038,59.361309],[5.914231,59.44894449600008],[6.19901150000004,59.47964099900008],[5.909635,59.47428899700003],[5.856092500000045,59.54454799700005],[5.85419150000007,59.464008499000045],[5.667560500000036,59.39082349700004],[5.842963,59.437320499000066],[5.890834999000049,59.35116949700006],[5.646485,59.25997549600004],[5.63145,59.50061799900004],[5.577508500000022,59.50020199700003],[5.462600500000065,59.434536500000036],[5.592431500000032,59.474860997000064],[5.614074,59.33062349800008],[5.550886,59.27162149900005],[5.295012442000029,59.37086988600004],[5.330110500000046,59.27053449500005],[5.298477,59.14537050000007],[5.175062500000024,59.17837449700005],[5.186944500000038,59.35496150000006],[5.285783754000022,59.38118380700007],[5.222025500000029,59.49856949700006],[5.302875500000027,59.481168],[5.22452450000003,59.527431497000066],[5.374809500000026,59.64634699800007],[5.472243,59.61444099900007],[5.413851,59.663357],[5.534834,59.73207849900007],[5.506673157000023,59.52538185800006],[5.567902999000069,59.55945950000006],[5.555418,59.646022997000046],[5.597233500000073,59.68478799800005],[5.794665721000058,59.65094391400004],[6.381009,59.87469099900005],[6.018712,59.75312050000008],[5.675114,59.848640498000066],[5.994348,59.954562999000075],[6.038543,60.067142500000045],[6.320311500000059,60.127182],[6.142478500000038,60.10751699600007],[6.082894500000066,60.19144799900005],[6.353219500000023,60.37049899700003],[6.669563,60.40578099900006],[6.565501,60.24366750000007],[6.543831500000067,60.06967549900003],[6.685616,60.38555149600006],[6.829082747000029,60.47144575700008],[6.630605500000058,60.46667849900007],[6.716949,60.52021799600004],[6.284545,60.39194850000007],[6.209440500000028,60.46901699800003],[6.278606500000024,60.38286949900004],[6.172586,60.36284999900005],[6.195897500000058,60.28046799600003],[6.152649500000052,60.242542498000034],[6.01512850000006,60.269164999000054],[5.924047,60.16648499900003],[5.914957500000071,60.033649497000056],[5.817266,59.983325999000044],[5.726609,60.00392149900006],[5.720516,60.17061999900005],[5.575043500000049,60.15387349800005],[5.747276,60.23856349600004],[5.584078500000032,60.25258249900003],[5.756293,60.399604999000076],[5.647027500000036,60.37364199600006],[5.446748500000069,60.156437],[5.245271500000058,60.20990050000006],[5.333692499000051,60.257888997000066],[5.236371,60.24559000100004],[5.201010500000052,60.290233501000046],[5.249560163000069,60.32177458800004],[5.170499500000062,60.37267699700004],[5.309116500000073,60.38820650100007],[5.262637500000039,60.50583249600004],[5.521925,60.42126850100004],[5.738453500000048,60.45921349900004],[5.735689,60.657379],[5.794102011000064,60.650521713000046],[5.698336426000026,60.69965533100003],[5.734445500000049,60.673576499000035],[5.708317500000021,60.47111150000006],[5.53338750000006,60.42834849600007],[5.339822,60.54158799900006],[5.591289500000073,60.62145999900008],[5.706546,60.75936149700004],[5.280158,60.54076],[5.207727999000042,60.621887],[5.114059,60.65854249900008],[5.281049,60.63255299900004],[4.94567,60.809383499000035],[5.469385500000044,60.65925199900005],[5.266979500000048,60.76729949800006],[5.324973579000073,60.823131946000046],[5.140299,60.84025249900003],[5.002784999000028,60.95379249800004],[5.115073512000038,60.965715414000044]],[[5.241315666000048,61.58393322900008],[5.461267500000076,61.589287],[5.181713,61.596099999000046],[5.241315666000048,61.58393322900008]],[[5.115073512000038,60.965715414000044],[5.116729500000076,60.96488949700006],[5.313839500000029,60.98683149700008],[5.115073512000038,60.965715414000044]]],[[[25.673977,71.17353799800003],[26.045212,71.11144250000007],[25.894638,71.010208],[26.225906500000065,71.06076799800007],[25.55792450000007,70.93201449900005],[25.391018,70.99469],[25.545727,71.03257750000006],[25.24226550000003,71.05068999900004],[25.739667999000062,71.07686599900006],[25.528230500000063,71.15545649800004],[25.673977,71.17353799800003]]],[[[23.781578,70.695778],[24.12865849900004,70.60249349800006],[23.773941,70.50978849800003],[23.623031500000025,70.69841750000006],[23.815597500000024,70.72293849900007],[23.781578,70.695778]]],[[[23.529517500000054,70.52244549900007],[23.642444,70.48944849900005],[23.146014500000035,70.27236950000008],[22.815429,70.41147699800007],[23.43688750000007,70.56320949900004],[23.354719,70.617737],[23.56452,70.62199399800005],[23.529517500000054,70.52244549900007]]],[[[23.369827500000042,70.78671249800004],[23.45733850000005,70.76512149900003],[22.798172,70.513825],[22.087011500000074,70.47561649800008],[22.310595,70.60153949900007],[21.928592500000036,70.64286049900005],[22.516258,70.72119899900008],[22.719646500000067,70.61257150000006],[22.769359500000064,70.678589],[22.679491,70.74093649800005],[22.997843,70.667839],[23.11976450000003,70.74383549900006],[23.015039500000057,70.79207599800003],[23.053814,70.81461349900007],[23.50584149900004,70.78686499900004],[23.369827500000042,70.78671249800004]]],[[[22.342153499000062,70.33776850000004],[22.956299,70.32978049900004],[23.009489,70.24874899800005],[22.342153499000062,70.33776850000004]]],[[[20.748051,70.09787749800006],[20.800907,70.056259],[20.370167,70.085861],[20.752233500000045,70.23168949800004],[20.80942550000003,70.20028699800008],[20.748051,70.09787749800006]]],[[[19.729519,70.22838599900007],[20.12632750000006,70.10135650000007],[19.742870500000038,70.056778],[19.50480250000004,70.23216249900008],[19.729519,70.22838599900007]]],[[[19.269611500000053,70.03578949900003],[19.674003500000026,69.995697],[19.12628550000005,69.787239],[18.704648999000028,69.92350749800005],[19.303821499000037,70.08371749800006],[19.269611500000053,70.03578949900003]]],[[[18.643114,69.86262499900005],[19.054355500000042,69.77339149900007],[18.761686500000053,69.67343150000005],[18.72437099900003,69.63607050000007],[18.837930500000027,69.60294349900005],[18.73374749900006,69.55794549900008],[18.006336,69.56680999800005],[18.27369409000005,69.62659492800003],[18.401043,69.63463599900007],[18.187591500000053,69.70993049800006],[18.620876500000065,69.69638049800005],[18.301355500000057,69.76936350000005],[18.639533499000038,69.804634],[18.645475500000032,69.690963],[18.748409500000037,69.68723299800007],[18.678631,69.701866],[18.732897,69.80738849900007],[18.643114,69.86262499900005]]],[[[17.482522500000073,69.60085249800005],[17.658531,69.48767099800006],[17.66749,69.59092699900003],[17.880442,69.579735],[17.919758,69.51934799800006],[17.815377999000077,69.45812249900007],[17.966917,69.51940149900008],[18.06773799900003,69.48912049800003],[17.93697,69.42363749800006],[18.080169500000068,69.426903],[17.84903550000007,69.22781349900004],[18.009228,69.19287849800008],[17.961891,69.177925],[17.460866999000075,69.20072899800005],[17.153118,69.01339749900006],[17.029537,69.03549950000007],[17.1658,69.08332799800007],[16.782012999000074,69.05759449900006],[17.211347500000045,69.19654099900004],[16.887241500000073,69.20046249900008],[17.184337500000026,69.28690349800007],[16.896857500000067,69.313469],[17.186859,69.351883],[16.86576850000006,69.376091],[17.49771499900004,69.42509449900007],[17.196076,69.496956],[17.40388850000005,69.48494699900004],[17.310571500000037,69.54144299900008],[17.68260950000007,69.46909350000004],[17.482522500000073,69.60085249800005]]],[[[15.619679500000075,68.95270549900005],[15.959016500000075,68.88904549800003],[15.813184,68.7472],[16.005573,68.75553849900007],[15.72271150000006,68.536804],[16.076727,68.74844349800003],[16.302828,68.715187],[16.08692,68.82158650000008],[16.123302500000023,68.84416199800006],[16.409397,68.85786449800008],[16.604114500000037,68.79657750000007],[16.594164,68.67980199800007],[16.633566,68.67171499900007],[16.45695199900007,68.55735799800004],[16.208227,68.55595449900005],[15.990130500000078,68.41610699900008],[16.011379,68.412079],[15.86366350000003,68.38358999900004],[15.874511,68.50976549900008],[15.661911,68.31165299900005],[15.326634500000068,68.35961149800005],[15.56163871700005,68.49955121900007],[14.980621499000051,68.27394849900003],[15.319273,68.46930699800004],[15.231805,68.51376350000004],[15.366633500000034,68.50199150000003],[15.177616,68.53307349800008],[15.205324999000027,68.57371499900006],[15.631016500000044,68.617607],[15.438093,68.71696449900003],[15.716285,68.693619],[15.464602500000069,68.753418],[15.682824,68.81733650000007],[15.551854,68.86223599900006],[15.619679500000075,68.95270549900005]]],[[[15.539591,69.00573749800003],[15.601443500000073,68.97971349800008],[15.416072500000041,68.90097799900008],[15.597289,69.11744699900004],[16.179676,69.28639999900008],[15.873977500000024,69.02712249900003],[15.539591,69.00573749800003]]],[[[15.17250450000006,68.931679],[15.317963500000076,68.91722099900005],[15.244818500000065,68.82391349800008],[15.392325500000027,68.84328449800006],[15.253855,68.75589],[15.41872,68.70305649900007],[15.033466500000031,68.57808699900005],[14.785534,68.63922899900007],[15.150149500000055,68.75284599900004],[14.52193950000003,68.60803999800004],[14.36857599900003,68.68449399900004],[14.583309,68.73529050000008],[14.436676,68.76052849900003],[14.549454500000024,68.81937450000004],[14.704345500000045,68.83254249800007],[14.68254,68.76516699900003],[14.703834499000038,68.74841299900004],[14.939134500000023,68.785446],[14.900189500000067,68.84649650000006],[14.968376,68.86647049800007],[15.152725,68.812866],[15.022504,68.94998949800004],[15.128936,69.02320850000007],[15.188699500000041,69.00209099900007],[15.17250450000006,68.931679]]],[[[15.165045500000076,68.44262699800004],[14.206444500000032,68.17415599800006],[14.327897,68.33840199800005],[14.509889500000043,68.33101649900004],[14.416385500000047,68.39362349800007],[15.19480150000004,68.47393050000005],[15.165045500000076,68.44262699800004]]],[[[14.14168701400007,68.27466567700003],[14.169512,68.25863699800004],[13.460458,68.08541099900003],[13.478425,68.20449049800004],[13.661848,68.23725899900006],[13.511562500000025,68.24778],[13.688828500000056,68.30903649800007],[14.272526,68.29914850000006],[14.14168701400007,68.27466567700003]]],[[[13.04957950000005,68.03257750000006],[13.187348500000041,67.976677],[13.014825,67.99353799800008],[12.97647,67.94109349800004],[13.08421050000004,67.92895499800005],[12.796074,67.84101850000008],[12.958006,68.02886199900007],[13.19181450000002,68.101097],[13.04957950000005,68.03257750000006]]],[[[12.551321,66.13022599900006],[12.667357500000037,66.103691],[12.298794500000042,66.01961499800007],[12.589514500000064,66.22785199900005],[12.629095,66.156784],[12.551321,66.13022599900006]]],[[[10.725237,64.86402149900005],[11.120891,64.98121649600006],[11.05366454500006,64.93660237400007],[11.367821500000048,64.94194049900005],[11.143732,64.82415749700004],[11.077328500000021,64.89194499900003],[10.832070500000043,64.84275799700004],[11.031305850000024,64.92831044300004],[10.725237,64.86402149900005]]],[[[8.587555,63.610359],[9.201086,63.565173998000034],[8.527718,63.42691399600005],[8.296058500000072,63.47917950000004],[8.74709050000007,63.56145850000007],[8.544486,63.59365850000006],[8.587555,63.610359]]],[[[8.447085499000025,63.28402349900006],[8.579752,63.18283099700005],[8.358337500000061,63.160995498000034],[8.240482500000041,63.264587497000036],[8.447085499000025,63.28402349900006]]],[[[8.098034,63.468765500000075],[8.21287650000005,63.37200950000005],[7.998843,63.32286449600008],[7.932767,63.344970500000045],[8.043354,63.33882150000005],[7.787214,63.408584500000075],[8.098034,63.468765500000075]]],[[[7.617825,63.11251049900005],[7.727075,63.006405],[7.352569500000072,62.98101399900008],[7.462165,63.05691899900006],[7.626714,63.06288899900005],[7.643898,63.07329199700007],[7.597258,63.10822299900008],[7.638170500000058,63.11698899900006],[7.617825,63.11251049900005]]],[[[5.897257500000023,62.421013],[6.11445150000003,62.352317998000046],[5.764179500000068,62.203823500000055],[5.529337500000054,62.25860199600004],[5.831122256000072,62.28298388800005],[5.857613,62.31460150000004],[5.764841,62.323269],[5.897257500000023,62.421013]]],[[[5.699188,60.058669999000074],[5.703725,59.99919499500004],[5.624819499000068,59.89454249900007],[5.340132500000038,60.02786999500006],[5.425635367000041,60.07315416400007],[5.699188,60.058669999000074]]],[[[5.499698500000022,59.91142249600006],[5.536162,59.791172],[5.403593500000056,59.75078599800008],[5.274518500000056,59.88779049900006],[5.364507,59.969269],[5.499698500000022,59.91142249600006]]],[[[5.05945950000006,59.85404599700007],[5.179249,59.87766649800005],[5.39621949900004,59.71174250000007],[5.298333,59.674747499000034],[5.346907500000043,59.698738],[5.241508500000066,59.743931],[5.249395,59.60264950000004],[5.183555500000068,59.57461550000005],[5.151623,59.67523550000004],[5.245591999000055,59.75310149900008],[5.105754,59.76964949900008],[5.130981284000029,59.82973899600006],[5.05945950000006,59.85404599700007]]],[[[4.953383,61.90943899700005],[5.257675500000062,61.83687199700006],[4.959802500000023,61.77524199900006],[4.903413500000056,61.81495299900007],[5.001847500000054,61.836486999000044],[4.818475500000034,61.85469449900006],[4.953383,61.90943899700005]]],[[[5.21904550000005,60.48925],[5.244192,60.414787499000056],[5.173365499000056,60.38851149900006],[4.961089500000071,60.56077599900004],[5.21904550000005,60.48925]]],[[[5.000170500000024,60.42187899900006],[5.133132500000045,60.33346549700008],[5.163972,60.21688850000004],[5.079828500000076,60.16420749700006],[4.948168,60.276031499000055],[5.067186,60.31352250000003],[5.015766500000041,60.30218499700004],[4.956776500000046,60.42578149800005],[4.986847,60.458605996000074],[5.01680250000004,60.42941299600005],[5.000170500000024,60.42187899900006]]],[[[5.000149,61.19904350000007],[5.023131,61.11022949900007],[4.869070710000074,61.07303522800004],[4.906099500000039,61.04799249700005],[4.752019500000074,61.099673996000035],[4.820931225000038,61.08130329000005],[4.883923500000037,61.21157849800005],[5.000149,61.19904350000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO01","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Oslo og Akershus","NUTS_NAME":"Oslo og Akershus"},"geometry":{"type":"Polygon","coordinates":[[[11.209564500000056,60.50491449700007],[11.600878500000022,60.26627899600004],[11.562372,60.17596899900008],[11.822262,60.05214499700003],[11.793048,59.95645949900006],[11.839729,59.84076699900004],[11.926970500000039,59.79047599800003],[11.679633500000023,59.741835500000036],[11.707625,59.72001549600003],[11.492860500000063,59.59364649500003],[11.343559500000026,59.75459899900005],[11.255791500000043,59.77013449800006],[11.23028,59.682740996000064],[10.970043500000031,59.70487349800004],[10.983207500000049,59.657335999000054],[10.777705,59.495240497000054],[10.686118,59.48940699800005],[10.581817500000056,59.756759500000044],[10.657540500000039,59.87100199900004],[10.764788,59.829544499000065],[10.759619,59.906375996000065],[10.64198,59.911508499000035],[10.506262,59.87787550000007],[10.497314500000073,59.788338499000076],[10.342610500000035,59.79401399500006],[10.386372,59.981385],[10.489164999000025,60.01725999900003],[10.600720500000023,60.131609999000034],[10.680319499000063,60.133530499000074],[10.933902500000045,60.34669199700005],[10.671977500000025,60.43385699700008],[10.71330850000004,60.52416499600008],[10.922567,60.47058199600008],[11.153986,60.605148],[11.209564500000056,60.50491449700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO011","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Oslo","NUTS_NAME":"Oslo"},"geometry":{"type":"Polygon","coordinates":[[[10.764788,59.829544499000065],[10.759619,59.906375996000065],[10.64198,59.911508499000035],[10.489164999000025,60.01725999900003],[10.600720500000023,60.131609999000034],[10.680319499000063,60.133530499000074],[10.945241,59.94764699700005],[10.907628,59.88495750000004],[10.936527499000022,59.83166949900004],[10.764788,59.829544499000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO012","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Akershus","NUTS_NAME":"Akershus"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.209564500000056,60.50491449700007],[11.600878500000022,60.26627899600004],[11.562372,60.17596899900008],[11.822262,60.05214499700003],[11.793048,59.95645949900006],[11.839729,59.84076699900004],[11.926970500000039,59.79047599800003],[11.679633500000023,59.741835500000036],[11.707625,59.72001549600003],[11.492860500000063,59.59364649500003],[11.343559500000026,59.75459899900005],[11.255791500000043,59.77013449800006],[11.23028,59.682740996000064],[10.970043500000031,59.70487349800004],[10.983207500000049,59.657335999000054],[10.777705,59.495240497000054],[10.686118,59.48940699800005],[10.581817500000056,59.756759500000044],[10.657540500000039,59.87100199900004],[10.764788,59.829544499000065],[10.936527499000022,59.83166949900004],[10.907628,59.88495750000004],[10.945241,59.94764699700005],[10.680319499000063,60.133530499000074],[10.933902500000045,60.34669199700005],[10.671977500000025,60.43385699700008],[10.71330850000004,60.52416499600008],[10.922567,60.47058199600008],[11.153986,60.605148],[11.209564500000056,60.50491449700007]]],[[[10.506262,59.87787550000007],[10.497314500000073,59.788338499000076],[10.342610500000035,59.79401399500006],[10.386372,59.981385],[10.489164999000025,60.01725999900003],[10.64198,59.911508499000035],[10.506262,59.87787550000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO02","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Hedmark og Oppland","NUTS_NAME":"Hedmark og Oppland"},"geometry":{"type":"Polygon","coordinates":[[[12.25465950000006,62.331024496000055],[12.29937,62.26749399900007],[12.137665,61.72381699700003],[12.419609,61.56298099900005],[12.569316,61.568746496000074],[12.870847500000025,61.35649499700003],[12.670176500000025,61.055976497000074],[12.22399,61.013078],[12.606881500000043,60.512742496000044],[12.606053,60.405925499000034],[12.498795500000028,60.323652],[12.541906500000039,60.19337849700008],[12.499544999000022,60.09765699700006],[12.174292,59.88980850000007],[11.839729,59.84076699900004],[11.793048,59.95645949900006],[11.822262,60.05214499700003],[11.562372,60.17596899900008],[11.600878500000022,60.26627899600004],[11.209564500000056,60.50491449700007],[11.153986,60.605148],[10.922567,60.47058199600008],[10.71330850000004,60.52416499600008],[10.671977500000025,60.43385699700008],[10.933902500000045,60.34669199700005],[10.680319499000063,60.133530499000074],[10.600720500000023,60.131609999000034],[10.309107,60.246387998000046],[10.03533,60.632055],[9.903135,60.63427850000005],[9.952266500000064,60.61089550000008],[9.817202,60.52582350000006],[9.8305,60.456421997000064],[9.478695,60.530145497000035],[9.291303500000026,60.76186299900007],[8.251701,61.07394399600008],[8.051171,61.23277650000006],[8.243193500000075,61.32109399900003],[8.150360500000033,61.42674149900006],[8.322053,61.455885497000054],[8.26233250000007,61.533979500000044],[7.947312500000066,61.55668049900004],[7.895651,61.64364649700008],[7.90821550000004,61.72167899900006],[7.841840500000046,61.743131997000035],[7.513823500000058,61.72736449900003],[7.349109,62.00768199600003],[7.734595500000069,62.18463299900003],[7.98361,62.177458999000066],[8.205852,62.331743997000046],[9.062062,62.372608497000044],[9.792455,62.287932],[10.097583,62.42780749600007],[10.00608150000005,62.55823499600007],[10.1969,62.687693499000034],[10.928922,62.69609050000008],[11.160122,62.64737899800008],[11.556107,62.37158599600008],[12.25465950000006,62.331024496000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO021","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Hedmark","NUTS_NAME":"Hedmark"},"geometry":{"type":"Polygon","coordinates":[[[12.25465950000006,62.331024496000055],[12.29937,62.26749399900007],[12.137665,61.72381699700003],[12.419609,61.56298099900005],[12.569316,61.568746496000074],[12.870847500000025,61.35649499700003],[12.670176500000025,61.055976497000074],[12.22399,61.013078],[12.606881500000043,60.512742496000044],[12.606053,60.405925499000034],[12.498795500000028,60.323652],[12.541906500000039,60.19337849700008],[12.499544999000022,60.09765699700006],[12.174292,59.88980850000007],[11.839729,59.84076699900004],[11.793048,59.95645949900006],[11.822262,60.05214499700003],[11.562372,60.17596899900008],[11.600878500000022,60.26627899600004],[11.209564500000056,60.50491449700007],[11.153986,60.605148],[10.74476450000003,60.77494099900008],[10.650989,60.95580549700003],[10.442951,61.06229649900007],[10.849054500000022,61.26601549900005],[10.680535,61.47159299600003],[10.705044500000042,61.50139249700004],[10.639634500000057,61.70786899600006],[10.404653,61.668138500000055],[10.052638499000068,61.751767],[10.16482250000007,61.881924999000034],[9.848242500000026,61.90381049600006],[9.838812500000074,62.01339499900007],[9.92817350000007,62.04819299900004],[9.583827,62.18462099900006],[9.792455,62.287932],[10.097583,62.42780749600007],[10.00608150000005,62.55823499600007],[10.1969,62.687693499000034],[10.928922,62.69609050000008],[11.160122,62.64737899800008],[11.556107,62.37158599600008],[12.25465950000006,62.331024496000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO022","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Oppland","NUTS_NAME":"Oppland"},"geometry":{"type":"Polygon","coordinates":[[[9.062062,62.372608497000044],[9.792455,62.287932],[9.583827,62.18462099900006],[9.92817350000007,62.04819299900004],[9.838812500000074,62.01339499900007],[9.848242500000026,61.90381049600006],[10.16482250000007,61.881924999000034],[10.052638499000068,61.751767],[10.404653,61.668138500000055],[10.639634500000057,61.70786899600006],[10.705044500000042,61.50139249700004],[10.680535,61.47159299600003],[10.849054500000022,61.26601549900005],[10.442951,61.06229649900007],[10.650989,60.95580549700003],[10.74476450000003,60.77494099900008],[11.153986,60.605148],[10.922567,60.47058199600008],[10.71330850000004,60.52416499600008],[10.671977500000025,60.43385699700008],[10.933902500000045,60.34669199700005],[10.680319499000063,60.133530499000074],[10.600720500000023,60.131609999000034],[10.309107,60.246387998000046],[10.03533,60.632055],[9.903135,60.63427850000005],[9.952266500000064,60.61089550000008],[9.817202,60.52582350000006],[9.8305,60.456421997000064],[9.478695,60.530145497000035],[9.291303500000026,60.76186299900007],[8.251701,61.07394399600008],[8.051171,61.23277650000006],[8.243193500000075,61.32109399900003],[8.150360500000033,61.42674149900006],[8.322053,61.455885497000054],[8.26233250000007,61.533979500000044],[7.947312500000066,61.55668049900004],[7.895651,61.64364649700008],[7.90821550000004,61.72167899900006],[7.841840500000046,61.743131997000035],[7.513823500000058,61.72736449900003],[7.349109,62.00768199600003],[7.734595500000069,62.18463299900003],[7.98361,62.177458999000066],[8.205852,62.331743997000046],[9.062062,62.372608497000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO03","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Sør-Østlandet","NUTS_NAME":"Sør-Østlandet"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.939878500000077,59.69458099700006],[11.691129,59.58954749900005],[11.826196500000037,59.23784999700007],[11.632557,58.90830650000004],[11.457448500000055,58.888435497000046],[11.460827,58.98865849800006],[11.476778,58.98445499800005],[11.374898,59.09957849700004],[11.384853146000069,59.117312139000035],[11.385243500000058,59.118007498000054],[11.340374784000062,59.11173018900007],[11.330404740000063,59.11033547200003],[10.825053,59.15331250000003],[10.723924499000077,59.26906599700004],[10.814114500000073,59.32770549600008],[10.660101999000062,59.33565899800004],[10.686118,59.48940699800005],[10.777705,59.495240497000054],[10.983207500000049,59.657335999000054],[10.970043500000031,59.70487349800004],[11.23028,59.682740996000064],[11.255791500000043,59.77013449800006],[11.343559500000026,59.75459899900005],[11.492860500000063,59.59364649500003],[11.707625,59.72001549600003],[11.679633500000023,59.741835500000036],[11.926970500000039,59.79047599800003],[11.939878500000077,59.69458099700006]]],[[[9.478695,60.530145497000035],[9.8305,60.456421997000064],[9.817202,60.52582350000006],[9.952266500000064,60.61089550000008],[9.903135,60.63427850000005],[10.03533,60.632055],[10.309107,60.246387998000046],[10.600720500000023,60.131609999000034],[10.489164999000025,60.01725999900003],[10.386372,59.981385],[10.342610500000035,59.79401399500006],[10.497314500000073,59.788338499000076],[10.617811,59.57157149800008],[10.421314,59.525081500000056],[10.439196500000037,59.66585149800005],[10.212189500000022,59.73728199900006],[10.319881500000065,59.69134350000007],[10.374606,59.67696749900006],[10.410937500000045,59.622604497000054],[10.389866999000049,59.537372496000046],[10.23252750000006,59.56869149700003],[10.384464500000036,59.438155996000035],[10.493912500000022,59.43849199700003],[10.461864500000047,59.37163549700006],[10.517841500000031,59.293377],[10.386558500000035,59.27502049700007],[10.273399500000039,59.04173649900008],[10.017867,59.048832],[10.040186,58.97815299800004],[9.969417500000077,58.964656998000066],[9.83919700100006,59.044178997000074],[9.680050218000076,59.055061142000056],[9.752187500000048,58.988582500000064],[9.289463,58.83913049500006],[9.467710500000067,58.829845497000065],[9.365971,58.771348999000054],[9.196343,58.803446],[8.996341500000028,58.991956498000036],[8.761995500000069,58.948491],[8.830738,58.91257399700004],[8.743719,58.83139799800006],[8.575454,58.93143899800003],[7.966328500000031,58.968420999000045],[7.742193,59.143892495000046],[7.757826,59.22843499700008],[7.696158999000033,59.32674799700004],[7.539958500000068,59.398819],[7.625633,59.461783497000056],[7.506116001000066,59.60780099900006],[7.214666500000021,59.672686999000064],[7.11008,59.70060349900007],[7.096287500000074,59.78277699900008],[7.232911001000048,59.962714497000036],[7.488040500000068,60.09883849700003],[7.673709,60.29433499900006],[7.732114500000023,60.52097049900004],[7.639577,60.55958799700005],[7.63585050000006,60.632906999000056],[7.438842500000021,60.66003949900005],[7.46964,60.67554749700008],[7.704408,60.739120497000044],[7.851515500000062,60.91992299600008],[8.223110500000075,60.96808899700005],[8.282661500000074,61.033330497000065],[8.214433500000041,61.05391050000003],[8.251701,61.07394399600008],[9.291303500000026,60.76186299900007],[9.478695,60.530145497000035]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO031","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Østfold","NUTS_NAME":"Østfold"},"geometry":{"type":"Polygon","coordinates":[[[11.939878500000077,59.69458099700006],[11.691129,59.58954749900005],[11.826196500000037,59.23784999700007],[11.632557,58.90830650000004],[11.457448500000055,58.888435497000046],[11.460827,58.98865849800006],[11.476778,58.98445499800005],[11.374898,59.09957849700004],[11.384853146000069,59.117312139000035],[11.385243500000058,59.118007498000054],[11.340374784000062,59.11173018900007],[11.330404740000063,59.11033547200003],[10.825053,59.15331250000003],[10.723924499000077,59.26906599700004],[10.814114500000073,59.32770549600008],[10.660101999000062,59.33565899800004],[10.686118,59.48940699800005],[10.777705,59.495240497000054],[10.983207500000049,59.657335999000054],[10.970043500000031,59.70487349800004],[11.23028,59.682740996000064],[11.255791500000043,59.77013449800006],[11.343559500000026,59.75459899900005],[11.492860500000063,59.59364649500003],[11.707625,59.72001549600003],[11.679633500000023,59.741835500000036],[11.926970500000039,59.79047599800003],[11.939878500000077,59.69458099700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO032","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Buskerud","NUTS_NAME":"Buskerud"},"geometry":{"type":"Polygon","coordinates":[[[10.600720500000023,60.131609999000034],[10.489164999000025,60.01725999900003],[10.386372,59.981385],[10.342610500000035,59.79401399500006],[10.497314500000073,59.788338499000076],[10.617811,59.57157149800008],[10.421314,59.525081500000056],[10.439196500000037,59.66585149800005],[10.212189500000022,59.73728199900006],[10.319881500000065,59.69134350000007],[9.998168,59.663823500000035],[9.91543850000005,59.582539498000074],[10.059967500000027,59.473028999000064],[9.758501999000032,59.45827849900007],[9.778087500000026,59.412002],[9.471744501000046,59.48221149800003],[9.51526350000006,59.53811499700004],[9.405283999000062,59.684534995000035],[9.33329150000003,59.69277599700007],[9.367755,59.78133899900007],[9.18608850000004,59.82698749900004],[9.192594500000041,59.916813998000066],[8.992795,59.98077150000006],[8.957869500000072,60.09806049900004],[8.722863500000074,60.183107],[7.488040500000068,60.09883849700003],[7.673709,60.29433499900006],[7.732114500000023,60.52097049900004],[7.639577,60.55958799700005],[7.63585050000006,60.632906999000056],[7.438842500000021,60.66003949900005],[7.46964,60.67554749700008],[7.704408,60.739120497000044],[7.851515500000062,60.91992299600008],[8.223110500000075,60.96808899700005],[8.282661500000074,61.033330497000065],[8.214433500000041,61.05391050000003],[8.251701,61.07394399600008],[9.291303500000026,60.76186299900007],[9.478695,60.530145497000035],[9.8305,60.456421997000064],[9.817202,60.52582350000006],[9.952266500000064,60.61089550000008],[9.903135,60.63427850000005],[10.03533,60.632055],[10.309107,60.246387998000046],[10.600720500000023,60.131609999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO033","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Vestfold","NUTS_NAME":"Vestfold"},"geometry":{"type":"Polygon","coordinates":[[[10.374606,59.67696749900006],[10.410937500000045,59.622604497000054],[10.389866999000049,59.537372496000046],[10.23252750000006,59.56869149700003],[10.384464500000036,59.438155996000035],[10.493912500000022,59.43849199700003],[10.461864500000047,59.37163549700006],[10.517841500000031,59.293377],[10.386558500000035,59.27502049700007],[10.273399500000039,59.04173649900008],[10.017867,59.048832],[10.040186,58.97815299800004],[9.969417500000077,58.964656998000066],[9.83919700100006,59.044178997000074],[9.824771500000054,59.157609498000056],[9.896416501000033,59.177002],[9.776792500000056,59.316107500000044],[9.778087500000026,59.412002],[9.758501999000032,59.45827849900007],[10.059967500000027,59.473028999000064],[9.91543850000005,59.582539498000074],[9.998168,59.663823500000035],[10.319881500000065,59.69134350000007],[10.374606,59.67696749900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO034","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Telemark","NUTS_NAME":"Telemark"},"geometry":{"type":"Polygon","coordinates":[[[9.405283999000062,59.684534995000035],[9.51526350000006,59.53811499700004],[9.471744501000046,59.48221149800003],[9.778087500000026,59.412002],[9.776792500000056,59.316107500000044],[9.896416501000033,59.177002],[9.824771500000054,59.157609498000056],[9.83919700100006,59.044178997000074],[9.680050218000076,59.055061142000056],[9.752187500000048,58.988582500000064],[9.289463,58.83913049500006],[9.467710500000067,58.829845497000065],[9.365971,58.771348999000054],[9.196343,58.803446],[8.996341500000028,58.991956498000036],[8.761995500000069,58.948491],[8.830738,58.91257399700004],[8.743719,58.83139799800006],[8.575454,58.93143899800003],[7.966328500000031,58.968420999000045],[7.742193,59.143892495000046],[7.757826,59.22843499700008],[7.696158999000033,59.32674799700004],[7.539958500000068,59.398819],[7.625633,59.461783497000056],[7.506116001000066,59.60780099900006],[7.214666500000021,59.672686999000064],[7.11008,59.70060349900007],[7.096287500000074,59.78277699900008],[7.232911001000048,59.962714497000036],[7.488040500000068,60.09883849700003],[8.722863500000074,60.183107],[8.957869500000072,60.09806049900004],[8.992795,59.98077150000006],[9.192594500000041,59.916813998000066],[9.18608850000004,59.82698749900004],[9.367755,59.78133899900007],[9.33329150000003,59.69277599700007],[9.405283999000062,59.684534995000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO04","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Agder og Rogaland","NUTS_NAME":"Agder og Rogaland"},"geometry":{"type":"Polygon","coordinates":[[[6.685322,59.83117999700005],[6.651835,59.71141299900006],[7.096287500000074,59.78277699900008],[7.11008,59.70060349900007],[7.214666500000021,59.672686999000064],[7.506116001000066,59.60780099900006],[7.625633,59.461783497000056],[7.539958500000068,59.398819],[7.696158999000033,59.32674799700004],[7.757826,59.22843499700008],[7.742193,59.143892495000046],[7.966328500000031,58.968420999000045],[8.575454,58.93143899800003],[8.743719,58.83139799800006],[8.830738,58.91257399700004],[8.761995500000069,58.948491],[8.996341500000028,58.991956498000036],[9.196343,58.803446],[9.365971,58.771348999000054],[9.082190499000035,58.74871449600005],[9.057591500000058,58.72521599800007],[9.244501,58.72533049900005],[9.076271,58.686345998000036],[9.229282500000068,58.688499499000045],[8.96709965100007,58.59442495500008],[8.992434500000059,58.55048349700007],[8.713581,58.436935497000036],[8.728853,58.39339849900006],[8.516029500000059,58.25892999800004],[8.391825500000039,58.25569199800003],[8.24362161700003,58.15584307300003],[8.256075,58.13458650100006],[8.185152,58.142941499000074],[8.061826500000052,58.11951050000005],[8.087569,58.18608849600008],[8.031234,58.23595050000006],[8.048012,58.16465399500004],[7.951262500000041,58.075958498000034],[7.513357499000051,58.00359750000007],[7.179073999000025,58.05886099600008],[7.05503,57.97974799800005],[7.038199,58.02142699800004],[7.146521,58.07286450000004],[6.98239650000005,58.03102099900008],[7.061141,58.119189997000035],[6.938757,58.05059799700007],[6.813322500000027,58.09625249700008],[7.051818,58.14051799900005],[6.798205500000051,58.18960549900004],[6.867762,58.132240498000044],[6.78057,58.11681349500003],[6.747517500000072,58.191993497000055],[6.73773650000004,58.11087049900004],[6.819480500000054,58.064850497000066],[6.553461500000026,58.117786498000044],[6.89118,58.27436049900007],[6.719107999000073,58.21998599800003],[6.672523,58.28101349800005],[6.380909,58.275402],[6.438839,58.290819499000065],[6.040288,58.379020498000045],[5.981829851000043,58.454346376000046],[5.874972,58.426685500000076],[5.921966174000033,58.463206064000076],[5.654306500000075,58.560295],[5.491752,58.75477999900005],[5.575934500000074,58.958621995000044],[5.686279,58.92847499900006],[5.574271500000066,59.03034249900003],[5.769078500000035,58.96493149600008],[5.746338499000046,58.85974899800004],[5.893873500000041,58.967807995000044],[6.176128500000061,58.83488099900006],[6.279346,58.839367],[6.073616,58.903625496000075],[5.995267500000068,58.968890997000074],[6.091457500000047,58.985473496000054],[5.889861500000052,59.07323049900003],[6.112402,59.140621],[6.022733,59.143718496000076],[6.217452500000036,59.26629649700004],[6.561808500000041,59.32115949800004],[6.155891500000052,59.260627498000076],[6.113465,59.301426],[6.241003500000033,59.34484099900004],[6.020808500000044,59.34091949900005],[6.110799426000028,59.40670361000008],[5.941965500000038,59.361309],[5.914231,59.44894449600008],[6.19901150000004,59.47964099900008],[5.909635,59.47428899700003],[5.856092500000045,59.54454799700005],[5.85419150000007,59.464008499000045],[5.667560500000036,59.39082349700004],[5.842963,59.437320499000066],[5.890834999000049,59.35116949700006],[5.646485,59.25997549600004],[5.63145,59.50061799900004],[5.577508500000022,59.50020199700003],[5.462600500000065,59.434536500000036],[5.592431500000032,59.474860997000064],[5.614074,59.33062349800008],[5.550886,59.27162149900005],[5.295012442000029,59.37086988600004],[5.330110500000046,59.27053449500005],[5.298477,59.14537050000007],[5.175062500000024,59.17837449700005],[5.186944500000038,59.35496150000006],[5.285783754000022,59.38118380700007],[5.222025500000029,59.49856949700006],[5.302875500000027,59.481168],[5.506673157000023,59.52538185800006],[5.567902999000069,59.55945950000006],[5.555418,59.646022997000046],[5.597233500000073,59.68478799800005],[5.794665721000058,59.65094391400004],[5.803792709000049,59.649380173000054],[5.823972500000025,59.64592299700007],[6.183041500000058,59.58606599900003],[6.302432500000066,59.767431498000064],[6.538059,59.842846500000064],[6.685322,59.83117999700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO041","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Aust-Agder","NUTS_NAME":"Aust-Agder"},"geometry":{"type":"Polygon","coordinates":[[[7.214666500000021,59.672686999000064],[7.506116001000066,59.60780099900006],[7.625633,59.461783497000056],[7.539958500000068,59.398819],[7.696158999000033,59.32674799700004],[7.757826,59.22843499700008],[7.742193,59.143892495000046],[7.966328500000031,58.968420999000045],[8.575454,58.93143899800003],[8.743719,58.83139799800006],[8.830738,58.91257399700004],[8.761995500000069,58.948491],[8.996341500000028,58.991956498000036],[9.196343,58.803446],[9.365971,58.771348999000054],[9.082190499000035,58.74871449600005],[9.057591500000058,58.72521599800007],[9.244501,58.72533049900005],[9.076271,58.686345998000036],[9.229282500000068,58.688499499000045],[8.96709965100007,58.59442495500008],[8.992434500000059,58.55048349700007],[8.713581,58.436935497000036],[8.728853,58.39339849900006],[8.516029500000059,58.25892999800004],[8.391825500000039,58.25569199800003],[8.24362161700003,58.15584307300003],[8.256075,58.13458650100006],[8.185152,58.142941499000074],[8.144522,58.17002899800008],[8.209832500000061,58.24187449900006],[8.041807500000061,58.28226699900006],[8.079482,58.30627099900005],[8.048810500000059,58.37253449900004],[7.946464499000058,58.31648099800003],[7.766404,58.455104],[7.521502,58.481416497000055],[7.61318250000005,58.593604999000036],[7.541388,58.62289450000003],[7.547956,58.68358499800007],[7.626257500000065,58.67918099700006],[7.667,58.76100299800004],[7.577408500000047,58.856960498000035],[7.032587500000034,58.82148749900006],[7.159529,58.91677949700005],[7.107054500000061,58.995772997000074],[7.179352,59.01071699900007],[7.141959500000041,59.043026498000074],[7.197169,59.15148050000005],[6.939172500000041,59.18191549900007],[6.837700500000039,59.192589499000064],[6.831575500000042,59.31733049700006],[6.882832500000063,59.42929749800004],[7.02288,59.48120149700003],[6.96296,59.51645099900003],[6.99184199900003,59.59702899800004],[7.214666500000021,59.672686999000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO042","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Vest-Agder","NUTS_NAME":"Vest-Agder"},"geometry":{"type":"Polygon","coordinates":[[[7.159529,58.91677949700005],[7.032587500000034,58.82148749900006],[7.577408500000047,58.856960498000035],[7.667,58.76100299800004],[7.626257500000065,58.67918099700006],[7.547956,58.68358499800007],[7.541388,58.62289450000003],[7.61318250000005,58.593604999000036],[7.521502,58.481416497000055],[7.766404,58.455104],[7.946464499000058,58.31648099800003],[8.048810500000059,58.37253449900004],[8.079482,58.30627099900005],[8.041807500000061,58.28226699900006],[8.209832500000061,58.24187449900006],[8.144522,58.17002899800008],[8.185152,58.142941499000074],[8.061826500000052,58.11951050000005],[8.087569,58.18608849600008],[8.031234,58.23595050000006],[8.048012,58.16465399500004],[7.951262500000041,58.075958498000034],[7.513357499000051,58.00359750000007],[7.179073999000025,58.05886099600008],[7.05503,57.97974799800005],[7.038199,58.02142699800004],[7.146521,58.07286450000004],[6.98239650000005,58.03102099900008],[7.061141,58.119189997000035],[6.938757,58.05059799700007],[6.813322500000027,58.09625249700008],[7.051818,58.14051799900005],[6.798205500000051,58.18960549900004],[6.867762,58.132240498000044],[6.78057,58.11681349500003],[6.747517500000072,58.191993497000055],[6.73773650000004,58.11087049900004],[6.819480500000054,58.064850497000066],[6.553461500000026,58.117786498000044],[6.89118,58.27436049900007],[6.719107999000073,58.21998599800003],[6.672523,58.28101349800005],[6.380909,58.275402],[6.438839,58.290819499000065],[6.608524500000044,58.372175],[6.609769,58.57860950000003],[6.436069500000031,58.64932799700006],[6.622347500000046,58.696680997000044],[6.51809650000007,58.767412997000065],[6.560838,58.909951497000066],[6.939172500000041,59.18191549900007],[7.197169,59.15148050000005],[7.141959500000041,59.043026498000074],[7.179352,59.01071699900007],[7.107054500000061,58.995772997000074],[7.159529,58.91677949700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO043","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Rogaland","NUTS_NAME":"Rogaland"},"geometry":{"type":"Polygon","coordinates":[[[6.651835,59.71141299900006],[7.096287500000074,59.78277699900008],[7.11008,59.70060349900007],[7.214666500000021,59.672686999000064],[6.99184199900003,59.59702899800004],[6.96296,59.51645099900003],[7.02288,59.48120149700003],[6.882832500000063,59.42929749800004],[6.831575500000042,59.31733049700006],[6.837700500000039,59.192589499000064],[6.939172500000041,59.18191549900007],[6.560838,58.909951497000066],[6.51809650000007,58.767412997000065],[6.622347500000046,58.696680997000044],[6.436069500000031,58.64932799700006],[6.609769,58.57860950000003],[6.608524500000044,58.372175],[6.438839,58.290819499000065],[6.040288,58.379020498000045],[5.981829851000043,58.454346376000046],[5.874972,58.426685500000076],[5.921966174000033,58.463206064000076],[5.654306500000075,58.560295],[5.491752,58.75477999900005],[5.575934500000074,58.958621995000044],[5.686279,58.92847499900006],[5.574271500000066,59.03034249900003],[5.769078500000035,58.96493149600008],[5.746338499000046,58.85974899800004],[5.893873500000041,58.967807995000044],[6.176128500000061,58.83488099900006],[6.279346,58.839367],[6.073616,58.903625496000075],[5.995267500000068,58.968890997000074],[6.091457500000047,58.985473496000054],[5.889861500000052,59.07323049900003],[6.112402,59.140621],[6.022733,59.143718496000076],[6.217452500000036,59.26629649700004],[6.561808500000041,59.32115949800004],[6.155891500000052,59.260627498000076],[6.113465,59.301426],[6.241003500000033,59.34484099900004],[6.020808500000044,59.34091949900005],[6.110799426000028,59.40670361000008],[5.941965500000038,59.361309],[5.914231,59.44894449600008],[6.19901150000004,59.47964099900008],[5.909635,59.47428899700003],[5.856092500000045,59.54454799700005],[5.85419150000007,59.464008499000045],[5.667560500000036,59.39082349700004],[5.842963,59.437320499000066],[5.890834999000049,59.35116949700006],[5.646485,59.25997549600004],[5.63145,59.50061799900004],[5.577508500000022,59.50020199700003],[5.462600500000065,59.434536500000036],[5.592431500000032,59.474860997000064],[5.614074,59.33062349800008],[5.550886,59.27162149900005],[5.295012442000029,59.37086988600004],[5.330110500000046,59.27053449500005],[5.298477,59.14537050000007],[5.175062500000024,59.17837449700005],[5.186944500000038,59.35496150000006],[5.285783754000022,59.38118380700007],[5.222025500000029,59.49856949700006],[5.302875500000027,59.481168],[5.506673157000023,59.52538185800006],[5.567902999000069,59.55945950000006],[5.555418,59.646022997000046],[5.597233500000073,59.68478799800005],[5.794665721000058,59.65094391400004],[5.803792709000049,59.649380173000054],[5.823972500000025,59.64592299700007],[6.183041500000058,59.58606599900003],[6.302432500000066,59.767431498000064],[6.538059,59.842846500000064],[6.685322,59.83117999700005],[6.651835,59.71141299900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO05","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Vestlandet","NUTS_NAME":"Vestlandet"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.115073512000038,60.965715414000044],[5.026263,61.01004050000006],[5.986885,61.128764997000076],[6.411997500000041,61.05647299700007],[6.645289500000047,61.17484649900007],[6.850829500000032,61.14572549700006],[7.035421,60.99983999800003],[6.84106550000007,60.882595],[7.044801500000062,60.985458496000035],[7.154517500000054,60.93813299900006],[7.121905500000025,60.86051199800005],[7.197031500000037,60.92475499900007],[7.026663500000041,61.08794399900006],[7.478136500000062,61.098621500000036],[7.412843,61.13557799700004],[7.712671,61.23199450000004],[7.432381,61.199321499000064],[7.312189,61.29982749700008],[7.585154,61.49429699900003],[7.284697,61.39589299700003],[7.348169,61.34857549900005],[7.247618,61.30030450000004],[7.395241500000054,61.18244450000003],[6.992821,61.10582799900004],[6.927117500000065,61.14121999900004],[7.17012,61.19993199800007],[6.630224500000054,61.21004499900005],[6.753228500000034,61.41973099900008],[6.473639500000047,61.115188499000055],[6.006493,61.162735],[6.050285,61.19412249700008],[5.394584,61.069965496000066],[5.076056,61.179718],[5.312890500000037,61.18884299900003],[4.94716950000003,61.25884999900006],[5.209688823000022,61.33452840500007],[5.026415500000041,61.37598399900003],[5.167576,61.42851999900006],[4.977364500000022,61.41730899700008],[5.240735500000028,61.47474299800007],[5.588792499000022,61.44279099800008],[5.751739500000042,61.492759499000044],[5.843049,61.45913699700003],[5.712819,61.50769399900008],[5.481126,61.454650998000034],[5.442402,61.46443949700006],[5.461188499000059,61.48801799600005],[5.177923999000029,61.51424049900004],[5.39946,61.551673996000034],[5.241315666000048,61.58393322900008],[4.994847999000058,61.591152],[5.414846500000067,61.646262999000044],[4.991695,61.62740699700004],[4.969778,61.72112249600008],[5.21751449900006,61.72261449900003],[5.151150500000028,61.75493249700003],[5.296247,61.77143499700003],[5.092194,61.76363749700005],[5.382925,61.84609199600004],[5.300301500000046,61.85786450000006],[5.434949,61.917640497000036],[5.780999500000064,61.852504500000066],[5.642197500000066,61.81269849900008],[5.985106500000029,61.84537099900007],[5.913162500000055,61.731254498000055],[6.010346500000026,61.842471999000054],[6.178206500000044,61.76468349700008],[6.214309,61.77514649900007],[6.079197500000021,61.84675599900004],[6.483179,61.80439750000005],[6.613900500000057,61.85201649600003],[6.586079,61.86867150000006],[6.846349,61.870303996000075],[6.630262,61.89056],[6.429300500000068,61.830268999000054],[5.911377999000024,61.882831499000076],[5.808180500000049,61.85716599700004],[5.731962499000076,61.89316199700005],[5.988245500000062,61.905822999000065],[5.458462500000053,61.93934999900006],[5.178933,61.895335998000064],[4.990589,61.99135599600004],[5.123834,62.032745498000054],[5.142885471000056,61.95648816400006],[5.288158,61.94990899900006],[5.426334,62.018695999000045],[5.089433,62.16728949900005],[5.185808,62.21553399700008],[5.491458,62.01478749900008],[5.623765500000047,62.06705450000004],[5.405087500000036,62.12742949900007],[5.485529,62.20001999900006],[5.73594,62.08772650000003],[5.671969,62.174431],[5.851422500000069,62.20307499900008],[5.95038099900006,62.156822],[5.847182,62.00476449700005],[5.981478500000037,62.142124],[6.325065500000051,62.06060049900003],[5.931158,62.21788399700006],[6.117795,62.18908299700007],[5.96181050000007,62.24787899900008],[6.328697,62.37550749700006],[6.580707500000074,62.14685449900003],[6.545071,62.104973],[6.610225999000022,62.13547149900006],[6.555007,62.19385549600008],[6.653805500000033,62.193794497000056],[6.402287500000057,62.357982496000034],[6.723693500000024,62.45203799700005],[6.914581,62.41138099800003],[6.884847500000035,62.38205349600008],[7.007536,62.283473997000044],[7.012420999000028,62.17502599900007],[6.873665,62.08268749700005],[7.20569050000006,62.10144799900007],[6.96163,62.11530299700007],[7.038890500000036,62.17032549700008],[7.047532499000056,62.27912899900008],[7.428215500000022,62.25268149900006],[6.987393,62.31317899900006],[6.941239500000052,62.42668149900004],[6.772653,62.46071199600004],[6.811594500000069,62.482394999000064],[6.220586,62.46456149800008],[6.717913,62.49196599600003],[6.461608500000068,62.51434349800007],[6.573409500000025,62.542159999000035],[6.256442500000048,62.52973549900008],[6.334281500000031,62.61331949700008],[6.568112500000041,62.62427899700003],[6.611302500000022,62.55686549600006],[6.660633500000074,62.632385500000055],[7.045805,62.65523149900008],[7.138921500000038,62.52282349800004],[7.128197500000056,62.60631549900006],[7.166881500000045,62.63157249800008],[7.55996,62.53118499900006],[7.786336,62.57716750000003],[7.396082,62.628738499000065],[7.970172500000047,62.74502199700004],[8.146601500000031,62.688270500000044],[8.019416,62.751445999000055],[8.06985,62.77651599900008],[7.349656500000037,62.69644949900004],[7.722468500000048,62.79231649900004],[6.98348149900005,62.724308],[6.969040500000062,62.77918599700007],[7.018433,62.81851599700008],[7.240911500000038,62.80947899900008],[6.898579,62.90984750000007],[7.301491500000054,63.010261496000055],[7.450778500000069,62.90481949700006],[7.685032,62.96850599900006],[7.791710500000022,62.95953349600006],[7.677089500000022,62.898002500000075],[8.036038500000075,62.96656399800008],[8.11747,62.921828997000034],[8.13680850000003,62.822223500000064],[8.530576,62.66896449600006],[8.55466750000005,62.717293],[8.190963,62.829601499000034],[8.136116,62.94702149900007],[7.988732,63.01142899800004],[7.880380500000058,62.98959749800008],[8.107145499000069,63.10542299900004],[8.142638,63.07119349900006],[8.132346,63.034172],[8.333088,62.96408099800004],[8.337154500000054,62.891792500000065],[8.700226500000042,62.81935499900004],[8.421752999000034,62.94767399800003],[8.663505499000053,62.97121799800004],[8.470663,62.97100050000006],[8.594899,63.02206049900008],[8.365225,62.96965399900006],[8.322330500000021,62.99421299900007],[8.49464050000006,63.01892850000007],[8.22018349900003,63.029011],[8.156400500000075,63.113617],[8.764103,63.184509999000056],[8.917685,63.115575997000064],[9.468545500000062,63.177874],[9.431773500000077,63.04879999800005],[9.584212,63.009086499000034],[9.42994,62.954142499000056],[9.46301950000003,62.848407],[9.42156650000004,62.81605],[8.919962,62.711770999000066],[9.24373,62.556077998000035],[9.183625500000062,62.42664550000006],[9.062062,62.372608497000044],[8.205852,62.331743997000046],[7.98361,62.177458999000066],[7.734595500000069,62.18463299900003],[7.349109,62.00768199600003],[7.513823500000058,61.72736449900003],[7.841840500000046,61.743131997000035],[7.90821550000004,61.72167899900006],[7.895651,61.64364649700008],[7.947312500000066,61.55668049900004],[8.26233250000007,61.533979500000044],[8.322053,61.455885497000054],[8.150360500000033,61.42674149900006],[8.243193500000075,61.32109399900003],[8.051171,61.23277650000006],[8.251701,61.07394399600008],[8.214433500000041,61.05391050000003],[8.282661500000074,61.033330497000065],[8.223110500000075,60.96808899700005],[7.851515500000062,60.91992299600008],[7.704408,60.739120497000044],[7.46964,60.67554749700008],[7.438842500000021,60.66003949900005],[7.63585050000006,60.632906999000056],[7.639577,60.55958799700005],[7.732114500000023,60.52097049900004],[7.673709,60.29433499900006],[7.488040500000068,60.09883849700003],[7.232911001000048,59.962714497000036],[7.096287500000074,59.78277699900008],[6.651835,59.71141299900006],[6.685322,59.83117999700005],[6.538059,59.842846500000064],[6.302432500000066,59.767431498000064],[6.183041500000058,59.58606599900003],[5.823972500000025,59.64592299700007],[5.803792709000049,59.649380173000054],[5.794665721000058,59.65094391400004],[6.381009,59.87469099900005],[6.018712,59.75312050000008],[5.675114,59.848640498000066],[5.994348,59.954562999000075],[6.038543,60.067142500000045],[6.320311500000059,60.127182],[6.142478500000038,60.10751699600007],[6.082894500000066,60.19144799900005],[6.353219500000023,60.37049899700003],[6.669563,60.40578099900006],[6.565501,60.24366750000007],[6.543831500000067,60.06967549900003],[6.685616,60.38555149600006],[6.829082747000029,60.47144575700008],[6.630605500000058,60.46667849900007],[6.716949,60.52021799600004],[6.284545,60.39194850000007],[6.209440500000028,60.46901699800003],[6.278606500000024,60.38286949900004],[6.172586,60.36284999900005],[6.195897500000058,60.28046799600003],[6.152649500000052,60.242542498000034],[6.01512850000006,60.269164999000054],[5.924047,60.16648499900003],[5.914957500000071,60.033649497000056],[5.817266,59.983325999000044],[5.726609,60.00392149900006],[5.720516,60.17061999900005],[5.575043500000049,60.15387349800005],[5.747276,60.23856349600004],[5.584078500000032,60.25258249900003],[5.756293,60.399604999000076],[5.647027500000036,60.37364199600006],[5.446748500000069,60.156437],[5.245271500000058,60.20990050000006],[5.333692499000051,60.257888997000066],[5.236371,60.24559000100004],[5.201010500000052,60.290233501000046],[5.249560163000069,60.32177458800004],[5.170499500000062,60.37267699700004],[5.309116500000073,60.38820650100007],[5.262637500000039,60.50583249600004],[5.521925,60.42126850100004],[5.738453500000048,60.45921349900004],[5.735689,60.657379],[5.794102011000064,60.650521713000046],[5.698336426000026,60.69965533100003],[5.734445500000049,60.673576499000035],[5.708317500000021,60.47111150000006],[5.53338750000006,60.42834849600007],[5.339822,60.54158799900006],[5.591289500000073,60.62145999900008],[5.706546,60.75936149700004],[5.280158,60.54076],[5.207727999000042,60.621887],[5.114059,60.65854249900008],[5.281049,60.63255299900004],[4.94567,60.809383499000035],[5.469385500000044,60.65925199900005],[5.266979500000048,60.76729949800006],[5.324973579000073,60.823131946000046],[5.140299,60.84025249900003],[5.002784999000028,60.95379249800004],[5.115073512000038,60.965715414000044]],[[5.241315666000048,61.58393322900008],[5.461267500000076,61.589287],[5.181713,61.596099999000046],[5.241315666000048,61.58393322900008]],[[5.115073512000038,60.965715414000044],[5.116729500000076,60.96488949700006],[5.313839500000029,60.98683149700008],[5.115073512000038,60.965715414000044]]],[[[8.81574850000004,63.26612650000004],[8.832189,63.20198],[8.490692,63.27751149900007],[8.765663999000026,63.34213649800006],[8.650367,63.40087899900004],[8.755111500000055,63.42434099900004],[8.892114500000048,63.35351049900004],[8.81574850000004,63.26612650000004]]],[[[8.447085499000025,63.28402349900006],[8.579752,63.18283099700005],[8.358337500000061,63.160995498000034],[8.240482500000041,63.264587497000036],[8.447085499000025,63.28402349900006]]],[[[8.098034,63.468765500000075],[8.21287650000005,63.37200950000005],[7.998843,63.32286449600008],[7.932767,63.344970500000045],[8.043354,63.33882150000005],[7.787214,63.408584500000075],[8.098034,63.468765500000075]]],[[[7.617825,63.11251049900005],[7.727075,63.006405],[7.352569500000072,62.98101399900008],[7.462165,63.05691899900006],[7.626714,63.06288899900005],[7.643898,63.07329199700007],[7.597258,63.10822299900008],[7.638170500000058,63.11698899900006],[7.617825,63.11251049900005]]],[[[5.897257500000023,62.421013],[6.11445150000003,62.352317998000046],[5.764179500000068,62.203823500000055],[5.529337500000054,62.25860199600004],[5.831122256000072,62.28298388800005],[5.857613,62.31460150000004],[5.764841,62.323269],[5.897257500000023,62.421013]]],[[[5.699188,60.058669999000074],[5.703725,59.99919499500004],[5.624819499000068,59.89454249900007],[5.340132500000038,60.02786999500006],[5.425635367000041,60.07315416400007],[5.699188,60.058669999000074]]],[[[5.499698500000022,59.91142249600006],[5.536162,59.791172],[5.403593500000056,59.75078599800008],[5.274518500000056,59.88779049900006],[5.364507,59.969269],[5.499698500000022,59.91142249600006]]],[[[5.413851,59.663357],[5.534834,59.73207849900007],[5.506673157000023,59.52538185800006],[5.302875500000027,59.481168],[5.22452450000003,59.527431497000066],[5.374809500000026,59.64634699800007],[5.472243,59.61444099900007],[5.413851,59.663357]]],[[[5.05945950000006,59.85404599700007],[5.179249,59.87766649800005],[5.39621949900004,59.71174250000007],[5.298333,59.674747499000034],[5.346907500000043,59.698738],[5.241508500000066,59.743931],[5.249395,59.60264950000004],[5.183555500000068,59.57461550000005],[5.151623,59.67523550000004],[5.245591999000055,59.75310149900008],[5.105754,59.76964949900008],[5.130981284000029,59.82973899600006],[5.05945950000006,59.85404599700007]]],[[[4.953383,61.90943899700005],[5.257675500000062,61.83687199700006],[4.959802500000023,61.77524199900006],[4.903413500000056,61.81495299900007],[5.001847500000054,61.836486999000044],[4.818475500000034,61.85469449900006],[4.953383,61.90943899700005]]],[[[5.21904550000005,60.48925],[5.244192,60.414787499000056],[5.173365499000056,60.38851149900006],[4.961089500000071,60.56077599900004],[5.21904550000005,60.48925]]],[[[5.000170500000024,60.42187899900006],[5.133132500000045,60.33346549700008],[5.163972,60.21688850000004],[5.079828500000076,60.16420749700006],[4.948168,60.276031499000055],[5.067186,60.31352250000003],[5.015766500000041,60.30218499700004],[4.956776500000046,60.42578149800005],[4.986847,60.458605996000074],[5.01680250000004,60.42941299600005],[5.000170500000024,60.42187899900006]]],[[[5.000149,61.19904350000007],[5.023131,61.11022949900007],[4.869070710000074,61.07303522800004],[4.906099500000039,61.04799249700005],[4.752019500000074,61.099673996000035],[4.820931225000038,61.08130329000005],[4.883923500000037,61.21157849800005],[5.000149,61.19904350000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO051","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Hordaland","NUTS_NAME":"Hordaland"},"geometry":{"type":"MultiPolygon","coordinates":[[[[6.14084,60.971599997000055],[6.304287,60.88073499700005],[6.800689,60.92312949600006],[6.826853500000027,60.89811049900004],[6.721219,60.85250650000006],[6.948781,60.84117599800004],[6.871434,60.76811749600006],[6.986408,60.71918249700008],[7.46964,60.67554749700008],[7.438842500000021,60.66003949900005],[7.63585050000006,60.632906999000056],[7.639577,60.55958799700005],[7.732114500000023,60.52097049900004],[7.673709,60.29433499900006],[7.488040500000068,60.09883849700003],[7.232911001000048,59.962714497000036],[7.096287500000074,59.78277699900008],[6.651835,59.71141299900006],[6.685322,59.83117999700005],[6.538059,59.842846500000064],[6.302432500000066,59.767431498000064],[6.183041500000058,59.58606599900003],[5.823972500000025,59.64592299700007],[5.803792709000049,59.649380173000054],[5.794665721000058,59.65094391400004],[6.381009,59.87469099900005],[6.018712,59.75312050000008],[5.675114,59.848640498000066],[5.994348,59.954562999000075],[6.038543,60.067142500000045],[6.320311500000059,60.127182],[6.142478500000038,60.10751699600007],[6.082894500000066,60.19144799900005],[6.353219500000023,60.37049899700003],[6.669563,60.40578099900006],[6.565501,60.24366750000007],[6.543831500000067,60.06967549900003],[6.685616,60.38555149600006],[6.829082747000029,60.47144575700008],[6.630605500000058,60.46667849900007],[6.716949,60.52021799600004],[6.284545,60.39194850000007],[6.209440500000028,60.46901699800003],[6.278606500000024,60.38286949900004],[6.172586,60.36284999900005],[6.195897500000058,60.28046799600003],[6.152649500000052,60.242542498000034],[6.01512850000006,60.269164999000054],[5.924047,60.16648499900003],[5.914957500000071,60.033649497000056],[5.817266,59.983325999000044],[5.726609,60.00392149900006],[5.720516,60.17061999900005],[5.575043500000049,60.15387349800005],[5.747276,60.23856349600004],[5.584078500000032,60.25258249900003],[5.756293,60.399604999000076],[5.647027500000036,60.37364199600006],[5.446748500000069,60.156437],[5.245271500000058,60.20990050000006],[5.333692499000051,60.257888997000066],[5.236371,60.24559000100004],[5.201010500000052,60.290233501000046],[5.249560163000069,60.32177458800004],[5.170499500000062,60.37267699700004],[5.309116500000073,60.38820650100007],[5.262637500000039,60.50583249600004],[5.521925,60.42126850100004],[5.738453500000048,60.45921349900004],[5.735689,60.657379],[5.794102011000064,60.650521713000046],[5.698336426000026,60.69965533100003],[5.734445500000049,60.673576499000035],[5.708317500000021,60.47111150000006],[5.53338750000006,60.42834849600007],[5.339822,60.54158799900006],[5.591289500000073,60.62145999900008],[5.706546,60.75936149700004],[5.280158,60.54076],[5.207727999000042,60.621887],[5.114059,60.65854249900008],[5.281049,60.63255299900004],[4.94567,60.809383499000035],[5.469385500000044,60.65925199900005],[5.266979500000048,60.76729949800006],[5.324973579000073,60.823131946000046],[5.140299,60.84025249900003],[5.226700500000049,60.92587799900008],[5.886013,60.951585496000064],[6.017261499000028,61.03523799900006],[6.14084,60.971599997000055]]],[[[5.703725,59.99919499500004],[5.624819499000068,59.89454249900007],[5.340132500000038,60.02786999500006],[5.425635367000041,60.07315416400007],[5.699188,60.058669999000074],[5.703725,59.99919499500004]]],[[[5.536162,59.791172],[5.403593500000056,59.75078599800008],[5.274518500000056,59.88779049900006],[5.364507,59.969269],[5.499698500000022,59.91142249600006],[5.536162,59.791172]]],[[[5.534834,59.73207849900007],[5.506673157000023,59.52538185800006],[5.302875500000027,59.481168],[5.22452450000003,59.527431497000066],[5.374809500000026,59.64634699800007],[5.472243,59.61444099900007],[5.413851,59.663357],[5.534834,59.73207849900007]]],[[[5.179249,59.87766649800005],[5.39621949900004,59.71174250000007],[5.298333,59.674747499000034],[5.346907500000043,59.698738],[5.241508500000066,59.743931],[5.249395,59.60264950000004],[5.183555500000068,59.57461550000005],[5.151623,59.67523550000004],[5.245591999000055,59.75310149900008],[5.105754,59.76964949900008],[5.130981284000029,59.82973899600006],[5.05945950000006,59.85404599700007],[5.179249,59.87766649800005]]],[[[5.244192,60.414787499000056],[5.173365499000056,60.38851149900006],[4.961089500000071,60.56077599900004],[5.21904550000005,60.48925],[5.244192,60.414787499000056]]],[[[5.133132500000045,60.33346549700008],[5.163972,60.21688850000004],[5.079828500000076,60.16420749700006],[4.948168,60.276031499000055],[5.067186,60.31352250000003],[5.015766500000041,60.30218499700004],[4.956776500000046,60.42578149800005],[4.986847,60.458605996000074],[5.01680250000004,60.42941299600005],[5.000170500000024,60.42187899900006],[5.133132500000045,60.33346549700008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO052","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Sogn og Fjordane","NUTS_NAME":"Sogn og Fjordane"},"geometry":{"type":"MultiPolygon","coordinates":[[[[5.115073512000038,60.965715414000044],[5.026263,61.01004050000006],[5.986885,61.128764997000076],[6.411997500000041,61.05647299700007],[6.645289500000047,61.17484649900007],[6.850829500000032,61.14572549700006],[7.035421,60.99983999800003],[6.84106550000007,60.882595],[7.044801500000062,60.985458496000035],[7.154517500000054,60.93813299900006],[7.121905500000025,60.86051199800005],[7.197031500000037,60.92475499900007],[7.026663500000041,61.08794399900006],[7.478136500000062,61.098621500000036],[7.412843,61.13557799700004],[7.712671,61.23199450000004],[7.432381,61.199321499000064],[7.312189,61.29982749700008],[7.585154,61.49429699900003],[7.284697,61.39589299700003],[7.348169,61.34857549900005],[7.247618,61.30030450000004],[7.395241500000054,61.18244450000003],[6.992821,61.10582799900004],[6.927117500000065,61.14121999900004],[7.17012,61.19993199800007],[6.630224500000054,61.21004499900005],[6.753228500000034,61.41973099900008],[6.473639500000047,61.115188499000055],[6.006493,61.162735],[6.050285,61.19412249700008],[5.394584,61.069965496000066],[5.076056,61.179718],[5.312890500000037,61.18884299900003],[4.94716950000003,61.25884999900006],[5.209688823000022,61.33452840500007],[5.026415500000041,61.37598399900003],[5.167576,61.42851999900006],[4.977364500000022,61.41730899700008],[5.240735500000028,61.47474299800007],[5.588792499000022,61.44279099800008],[5.751739500000042,61.492759499000044],[5.843049,61.45913699700003],[5.712819,61.50769399900008],[5.481126,61.454650998000034],[5.442402,61.46443949700006],[5.461188499000059,61.48801799600005],[5.177923999000029,61.51424049900004],[5.39946,61.551673996000034],[5.241315666000048,61.58393322900008],[4.994847999000058,61.591152],[5.414846500000067,61.646262999000044],[4.991695,61.62740699700004],[4.969778,61.72112249600008],[5.21751449900006,61.72261449900003],[5.151150500000028,61.75493249700003],[5.296247,61.77143499700003],[5.092194,61.76363749700005],[5.382925,61.84609199600004],[5.300301500000046,61.85786450000006],[5.434949,61.917640497000036],[5.780999500000064,61.852504500000066],[5.642197500000066,61.81269849900008],[5.985106500000029,61.84537099900007],[5.913162500000055,61.731254498000055],[6.010346500000026,61.842471999000054],[6.178206500000044,61.76468349700008],[6.214309,61.77514649900007],[6.079197500000021,61.84675599900004],[6.483179,61.80439750000005],[6.613900500000057,61.85201649600003],[6.586079,61.86867150000006],[6.846349,61.870303996000075],[6.630262,61.89056],[6.429300500000068,61.830268999000054],[5.911377999000024,61.882831499000076],[5.808180500000049,61.85716599700004],[5.731962499000076,61.89316199700005],[5.988245500000062,61.905822999000065],[5.458462500000053,61.93934999900006],[5.178933,61.895335998000064],[4.990589,61.99135599600004],[5.123834,62.032745498000054],[5.142885471000056,61.95648816400006],[5.288158,61.94990899900006],[5.426334,62.018695999000045],[5.089433,62.16728949900005],[5.185808,62.21553399700008],[5.491458,62.01478749900008],[6.311349,61.95865249800005],[6.657573500000069,62.07185399700006],[6.840866,61.963547],[7.349109,62.00768199600003],[7.513823500000058,61.72736449900003],[7.841840500000046,61.743131997000035],[7.90821550000004,61.72167899900006],[7.895651,61.64364649700008],[7.947312500000066,61.55668049900004],[8.26233250000007,61.533979500000044],[8.322053,61.455885497000054],[8.150360500000033,61.42674149900006],[8.243193500000075,61.32109399900003],[8.051171,61.23277650000006],[8.251701,61.07394399600008],[8.214433500000041,61.05391050000003],[8.282661500000074,61.033330497000065],[8.223110500000075,60.96808899700005],[7.851515500000062,60.91992299600008],[7.704408,60.739120497000044],[7.46964,60.67554749700008],[6.986408,60.71918249700008],[6.871434,60.76811749600006],[6.948781,60.84117599800004],[6.721219,60.85250650000006],[6.826853500000027,60.89811049900004],[6.800689,60.92312949600006],[6.304287,60.88073499700005],[6.14084,60.971599997000055],[6.017261499000028,61.03523799900006],[5.886013,60.951585496000064],[5.226700500000049,60.92587799900008],[5.140299,60.84025249900003],[5.002784999000028,60.95379249800004],[5.115073512000038,60.965715414000044]],[[5.241315666000048,61.58393322900008],[5.461267500000076,61.589287],[5.181713,61.596099999000046],[5.241315666000048,61.58393322900008]],[[5.115073512000038,60.965715414000044],[5.116729500000076,60.96488949700006],[5.313839500000029,60.98683149700008],[5.115073512000038,60.965715414000044]]],[[[4.953383,61.90943899700005],[5.257675500000062,61.83687199700006],[4.959802500000023,61.77524199900006],[4.903413500000056,61.81495299900007],[5.001847500000054,61.836486999000044],[4.818475500000034,61.85469449900006],[4.953383,61.90943899700005]]],[[[5.000149,61.19904350000007],[5.023131,61.11022949900007],[4.869070710000074,61.07303522800004],[4.906099500000039,61.04799249700005],[4.752019500000074,61.099673996000035],[4.820931225000038,61.08130329000005],[4.883923500000037,61.21157849800005],[5.000149,61.19904350000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO053","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Møre og Romsdal","NUTS_NAME":"Møre og Romsdal"},"geometry":{"type":"MultiPolygon","coordinates":[[[[8.764103,63.184509999000056],[8.917685,63.115575997000064],[9.468545500000062,63.177874],[9.431773500000077,63.04879999800005],[9.584212,63.009086499000034],[9.42994,62.954142499000056],[9.46301950000003,62.848407],[9.42156650000004,62.81605],[8.919962,62.711770999000066],[9.24373,62.556077998000035],[9.183625500000062,62.42664550000006],[9.062062,62.372608497000044],[8.205852,62.331743997000046],[7.98361,62.177458999000066],[7.734595500000069,62.18463299900003],[7.349109,62.00768199600003],[6.840866,61.963547],[6.657573500000069,62.07185399700006],[6.311349,61.95865249800005],[5.491458,62.01478749900008],[5.623765500000047,62.06705450000004],[5.405087500000036,62.12742949900007],[5.485529,62.20001999900006],[5.73594,62.08772650000003],[5.671969,62.174431],[5.851422500000069,62.20307499900008],[5.95038099900006,62.156822],[5.847182,62.00476449700005],[5.981478500000037,62.142124],[6.325065500000051,62.06060049900003],[5.931158,62.21788399700006],[6.117795,62.18908299700007],[5.96181050000007,62.24787899900008],[6.328697,62.37550749700006],[6.580707500000074,62.14685449900003],[6.545071,62.104973],[6.610225999000022,62.13547149900006],[6.555007,62.19385549600008],[6.653805500000033,62.193794497000056],[6.402287500000057,62.357982496000034],[6.723693500000024,62.45203799700005],[6.914581,62.41138099800003],[6.884847500000035,62.38205349600008],[7.007536,62.283473997000044],[7.012420999000028,62.17502599900007],[6.873665,62.08268749700005],[7.20569050000006,62.10144799900007],[6.96163,62.11530299700007],[7.038890500000036,62.17032549700008],[7.047532499000056,62.27912899900008],[7.428215500000022,62.25268149900006],[6.987393,62.31317899900006],[6.941239500000052,62.42668149900004],[6.772653,62.46071199600004],[6.811594500000069,62.482394999000064],[6.220586,62.46456149800008],[6.717913,62.49196599600003],[6.461608500000068,62.51434349800007],[6.573409500000025,62.542159999000035],[6.256442500000048,62.52973549900008],[6.334281500000031,62.61331949700008],[6.568112500000041,62.62427899700003],[6.611302500000022,62.55686549600006],[6.660633500000074,62.632385500000055],[7.045805,62.65523149900008],[7.138921500000038,62.52282349800004],[7.128197500000056,62.60631549900006],[7.166881500000045,62.63157249800008],[7.55996,62.53118499900006],[7.786336,62.57716750000003],[7.396082,62.628738499000065],[7.970172500000047,62.74502199700004],[8.146601500000031,62.688270500000044],[8.019416,62.751445999000055],[8.06985,62.77651599900008],[7.349656500000037,62.69644949900004],[7.722468500000048,62.79231649900004],[6.98348149900005,62.724308],[6.969040500000062,62.77918599700007],[7.018433,62.81851599700008],[7.240911500000038,62.80947899900008],[6.898579,62.90984750000007],[7.301491500000054,63.010261496000055],[7.450778500000069,62.90481949700006],[7.685032,62.96850599900006],[7.791710500000022,62.95953349600006],[7.677089500000022,62.898002500000075],[8.036038500000075,62.96656399800008],[8.11747,62.921828997000034],[8.13680850000003,62.822223500000064],[8.530576,62.66896449600006],[8.55466750000005,62.717293],[8.190963,62.829601499000034],[8.136116,62.94702149900007],[7.988732,63.01142899800004],[7.880380500000058,62.98959749800008],[8.107145499000069,63.10542299900004],[8.142638,63.07119349900006],[8.132346,63.034172],[8.333088,62.96408099800004],[8.337154500000054,62.891792500000065],[8.700226500000042,62.81935499900004],[8.421752999000034,62.94767399800003],[8.663505499000053,62.97121799800004],[8.470663,62.97100050000006],[8.594899,63.02206049900008],[8.365225,62.96965399900006],[8.322330500000021,62.99421299900007],[8.49464050000006,63.01892850000007],[8.22018349900003,63.029011],[8.156400500000075,63.113617],[8.764103,63.184509999000056]]],[[[8.81574850000004,63.26612650000004],[8.832189,63.20198],[8.490692,63.27751149900007],[8.765663999000026,63.34213649800006],[8.650367,63.40087899900004],[8.755111500000055,63.42434099900004],[8.892114500000048,63.35351049900004],[8.81574850000004,63.26612650000004]]],[[[8.579752,63.18283099700005],[8.358337500000061,63.160995498000034],[8.240482500000041,63.264587497000036],[8.447085499000025,63.28402349900006],[8.579752,63.18283099700005]]],[[[8.21287650000005,63.37200950000005],[7.998843,63.32286449600008],[7.932767,63.344970500000045],[8.043354,63.33882150000005],[7.787214,63.408584500000075],[8.098034,63.468765500000075],[8.21287650000005,63.37200950000005]]],[[[7.727075,63.006405],[7.352569500000072,62.98101399900008],[7.462165,63.05691899900006],[7.626714,63.06288899900005],[7.643898,63.07329199700007],[7.597258,63.10822299900008],[7.638170500000058,63.11698899900006],[7.617825,63.11251049900005],[7.727075,63.006405]]],[[[6.11445150000003,62.352317998000046],[5.764179500000068,62.203823500000055],[5.529337500000054,62.25860199600004],[5.831122256000072,62.28298388800005],[5.857613,62.31460150000004],[5.764841,62.323269],[5.897257500000023,62.421013],[6.11445150000003,62.352317998000046]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO06","LEVL_CODE":2,"CNTR_CODE":"NO","NAME_LATN":"Trøndelag","NUTS_NAME":"Trøndelag"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.514355244000058,64.87855370700004],[11.278464500000041,64.85742949800004],[11.960979500000064,65.07714799700005],[12.101823,65.11344899900007],[12.121571500000073,65.18907949900006],[12.337297500000034,65.11171699800008],[11.981972,65.07081549600008],[12.046106500000064,65.06244999900008],[12.119634500000075,65.05026249700006],[12.092824,65.03982549900007],[12.187348500000041,65.00011149700003],[12.182329,64.98138099900007],[12.768324500000062,64.940775],[13.085395500000061,65.10996799900005],[14.325985,65.118916],[13.654257500000028,64.58034049800006],[14.113869500000021,64.46248450000007],[14.157109,64.19505449600007],[13.967524500000025,64.00796999900007],[13.211107500000026,64.09536949900007],[12.683565,63.97422349900006],[12.149767,63.593946998000035],[12.212877,63.47859199900006],[11.974581,63.269227999000066],[12.052455500000065,63.183444499000075],[12.218231,63.00033249900008],[12.074687999000048,62.902544997000064],[12.136381500000027,62.74791649700006],[12.056142,62.61191849800008],[12.25465950000006,62.331024496000055],[11.556107,62.37158599600008],[11.160122,62.64737899800008],[10.928922,62.69609050000008],[10.1969,62.687693499000034],[10.00608150000005,62.55823499600007],[10.097583,62.42780749600007],[9.792455,62.287932],[9.062062,62.372608497000044],[9.183625500000062,62.42664550000006],[9.24373,62.556077998000035],[8.919962,62.711770999000066],[9.42156650000004,62.81605],[9.46301950000003,62.848407],[9.42994,62.954142499000056],[9.584212,63.009086499000034],[9.431773500000077,63.04879999800005],[9.468545500000062,63.177874],[8.917685,63.115575997000064],[8.764103,63.184509999000056],[8.985986500000024,63.20794299800008],[8.832189,63.20198],[8.81574850000004,63.26612650000004],[8.892114500000048,63.35351049900004],[8.755111500000055,63.42434099900004],[8.904203500000051,63.38735199800004],[9.01071,63.46447],[9.177391,63.37377950000007],[9.092917500000055,63.28715899900004],[9.501772,63.39708699700003],[9.249117,63.36782849600007],[9.176959,63.41464599900007],[9.297754401000077,63.448047849000034],[9.147209,63.48814],[9.585823,63.528888496000036],[9.491575,63.57952499900006],[9.75341050000003,63.645359],[9.71493350000003,63.615505],[9.977688,63.44161599700004],[9.823175500000048,63.31224450000008],[10.208758500000044,63.322869],[10.059104,63.38721099900005],[10.189022,63.45037849900007],[10.85359,63.439093496000055],[10.912328500000058,63.46505349900008],[10.756933,63.520218],[10.941581,63.56428899900004],[10.627927,63.54674150000005],[11.458975,63.80233749900003],[11.072477500000048,63.858989500000064],[11.501289500000041,64.00521850000007],[11.222313,64.074219],[10.591203500000063,63.80435549900005],[10.962953500000026,63.90656649700003],[11.070417500000076,63.841949499000066],[10.955013501000053,63.74560949900007],[10.170315,63.526134496000054],[9.948584500000038,63.495971498000074],[9.805798500000037,63.62687299900006],[10.130084,63.75410449900005],[9.52912150000003,63.71239849600005],[9.803867500000024,63.766426],[9.559135500000025,63.769199],[9.678071,63.84419649900008],[10.224334,63.927921498000046],[9.946816500000068,63.90663199800008],[10.226982,64.00061799900004],[9.951425500000028,63.94597250000004],[10.010268949000022,64.04565076200004],[9.932503500000053,64.060982],[10.016330268000047,64.06744965300004],[10.234300500000074,64.10188299600003],[10.120101,64.121704],[10.212196500000061,64.20630649900005],[10.374837,64.17877199600008],[10.274836500000049,64.23419149700004],[10.444528500000047,64.35304249800004],[10.68307,64.36034399900007],[10.45395,64.40396099800006],[10.634974130000046,64.39100241600005],[10.548171,64.44136050000003],[10.83981250000005,64.368828],[10.654607,64.45243049900006],[10.948439500000063,64.50749949600004],[10.976382,64.53115849900007],[10.906779500000027,64.57653050000005],[10.973181500000067,64.60691050000008],[11.340755500000057,64.43691999700007],[11.225456,64.311905],[11.407744500000035,64.36854549600008],[11.352719500000035,64.41844949900008],[11.641216500000041,64.47501099800007],[11.42755150000005,64.47552499900007],[11.725962500000037,64.58479299900006],[11.515246500000046,64.56695549800003],[11.379855,64.66954049900005],[11.86674750000003,64.79622649700008],[11.603879,64.814125],[11.817444,64.86047349600005],[11.653197500000033,64.83556349900005],[11.694047001000058,64.87286399600004],[12.136008884000034,64.96383255800004],[11.615549,64.86773699900004],[11.65877050000006,64.862915],[11.392232,64.75153349900006],[11.182071500000063,64.74090599900006],[11.446765999000036,64.809616],[11.240288,64.81357549900008],[11.277119500000026,64.845711],[11.514355244000058,64.87855370700004]]],[[[11.120891,64.98121649600006],[11.05366454500006,64.93660237400007],[11.367821500000048,64.94194049900005],[11.143732,64.82415749700004],[11.077328500000021,64.89194499900003],[10.832070500000043,64.84275799700004],[11.031305850000024,64.92831044300004],[10.725237,64.86402149900005],[11.120891,64.98121649600006]]],[[[9.201086,63.565173998000034],[8.527718,63.42691399600005],[8.296058500000072,63.47917950000004],[8.74709050000007,63.56145850000007],[8.544486,63.59365850000006],[8.587555,63.610359],[9.201086,63.565173998000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO061","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Sør-Trøndelag","NUTS_NAME":"Sør-Trøndelag"},"geometry":{"type":"MultiPolygon","coordinates":[[[[9.71493350000003,63.615505],[9.977688,63.44161599700004],[9.823175500000048,63.31224450000008],[10.208758500000044,63.322869],[10.059104,63.38721099900005],[10.189022,63.45037849900007],[10.85359,63.439093496000055],[10.854599500000063,63.34418949900004],[11.454561500000068,63.32623350000006],[11.630786500000056,63.180687499000044],[12.052455500000065,63.183444499000075],[12.218231,63.00033249900008],[12.074687999000048,62.902544997000064],[12.136381500000027,62.74791649700006],[12.056142,62.61191849800008],[12.25465950000006,62.331024496000055],[11.556107,62.37158599600008],[11.160122,62.64737899800008],[10.928922,62.69609050000008],[10.1969,62.687693499000034],[10.00608150000005,62.55823499600007],[10.097583,62.42780749600007],[9.792455,62.287932],[9.062062,62.372608497000044],[9.183625500000062,62.42664550000006],[9.24373,62.556077998000035],[8.919962,62.711770999000066],[9.42156650000004,62.81605],[9.46301950000003,62.848407],[9.42994,62.954142499000056],[9.584212,63.009086499000034],[9.431773500000077,63.04879999800005],[9.468545500000062,63.177874],[8.917685,63.115575997000064],[8.764103,63.184509999000056],[8.985986500000024,63.20794299800008],[8.832189,63.20198],[8.81574850000004,63.26612650000004],[8.892114500000048,63.35351049900004],[8.755111500000055,63.42434099900004],[8.904203500000051,63.38735199800004],[9.01071,63.46447],[9.177391,63.37377950000007],[9.092917500000055,63.28715899900004],[9.501772,63.39708699700003],[9.249117,63.36782849600007],[9.176959,63.41464599900007],[9.297754401000077,63.448047849000034],[9.147209,63.48814],[9.585823,63.528888496000036],[9.491575,63.57952499900006],[9.75341050000003,63.645359],[9.71493350000003,63.615505]]],[[[10.927325500000052,64.28025499900008],[10.74079450000005,64.17706049900005],[10.880387,64.09788149600007],[10.773019,63.98698999800007],[10.134664500000042,63.588658999000074],[10.170315,63.526134496000054],[9.948584500000038,63.495971498000074],[9.805798500000037,63.62687299900006],[10.130084,63.75410449900005],[9.52912150000003,63.71239849600005],[9.803867500000024,63.766426],[9.559135500000025,63.769199],[9.678071,63.84419649900008],[10.224334,63.927921498000046],[9.946816500000068,63.90663199800008],[10.226982,64.00061799900004],[9.951425500000028,63.94597250000004],[10.010268949000022,64.04565076200004],[9.932503500000053,64.060982],[10.016330268000047,64.06744965300004],[10.234300500000074,64.10188299600003],[10.120101,64.121704],[10.212196500000061,64.20630649900005],[10.374837,64.17877199600008],[10.274836500000049,64.23419149700004],[10.444528500000047,64.35304249800004],[10.68307,64.36034399900007],[10.45395,64.40396099800006],[10.634974130000046,64.39100241600005],[10.927325500000052,64.28025499900008]]],[[[9.201086,63.565173998000034],[8.527718,63.42691399600005],[8.296058500000072,63.47917950000004],[8.74709050000007,63.56145850000007],[8.544486,63.59365850000006],[8.587555,63.610359],[9.201086,63.565173998000034]]]]}},{"type":"Feature","properties":{"NUTS_ID":"NO062","LEVL_CODE":3,"CNTR_CODE":"NO","NAME_LATN":"Nord-Trøndelag","NUTS_NAME":"Nord-Trøndelag"},"geometry":{"type":"MultiPolygon","coordinates":[[[[11.514355244000058,64.87855370700004],[11.278464500000041,64.85742949800004],[11.960979500000064,65.07714799700005],[12.101823,65.11344899900007],[12.121571500000073,65.18907949900006],[12.337297500000034,65.11171699800008],[11.981972,65.07081549600008],[12.046106500000064,65.06244999900008],[12.119634500000075,65.05026249700006],[12.092824,65.03982549900007],[12.187348500000041,65.00011149700003],[12.182329,64.98138099900007],[12.768324500000062,64.940775],[13.085395500000061,65.10996799900005],[14.325985,65.118916],[13.654257500000028,64.58034049800006],[14.113869500000021,64.46248450000007],[14.157109,64.19505449600007],[13.967524500000025,64.00796999900007],[13.211107500000026,64.09536949900007],[12.683565,63.97422349900006],[12.149767,63.593946998000035],[12.212877,63.47859199900006],[11.974581,63.269227999000066],[12.052455500000065,63.183444499000075],[11.630786500000056,63.180687499000044],[11.454561500000068,63.32623350000006],[10.854599500000063,63.34418949900004],[10.85359,63.439093496000055],[10.912328500000058,63.46505349900008],[10.756933,63.520218],[10.941581,63.56428899900004],[10.627927,63.54674150000005],[11.458975,63.80233749900003],[11.072477500000048,63.858989500000064],[11.501289500000041,64.00521850000007],[11.222313,64.074219],[10.591203500000063,63.80435549900005],[10.962953500000026,63.90656649700003],[11.070417500000076,63.841949499000066],[10.955013501000053,63.74560949900007],[10.170315,63.526134496000054],[10.134664500000042,63.588658999000074],[10.773019,63.98698999800007],[10.880387,64.09788149600007],[10.74079450000005,64.17706049900005],[10.927325500000052,64.28025499900008],[10.634974130000046,64.39100241600005],[10.548171,64.44136050000003],[10.83981250000005,64.368828],[10.654607,64.45243049900006],[10.948439500000063,64.50749949600004],[10.976382,64.53115849900007],[10.906779500000027,64.57653050000005],[10.973181500000067,64.60691050000008],[11.340755500000057,64.43691999700007],[11.225456,64.311905],[11.407744500000035,64.36854549600008],[11.352719500000035,64.41844949900008],[11.641216500000041,64.47501099800007],[11.42755150000005,64.47552499900007],[11.725962500000037,64.58479299900006],[11.515246500000046,64.56695549800003],[11.379855,64.66954049900005],[11.86674750000003,64.79622649700008],[11.603879,64.814125],[11.817444,64.86047349600005],[11.653197500000033,64.83556349900005],[11.694047001000058,64.87286399600004],[12.136008884000034,64.96383255800004],[11.615549,64.86773699900004],[11.65877050000006,64.862915],[11.392232,64.75153349900006],[11.182071500000063,64.74090599900006],[11.446765999000036,64.809616],[11.240288,64.81357549900008],[11.277119500000026,64.845711],[11.514355244000058,64.87855370700004]]],[[[11.120891,64.98121649600006],[11.05366454500006,64.93660237400007],[11.367821500000048,64.94194049900005],[11.143732,64.82415749700004],[11.077328500000021,64.89194499900003],[10.832070500000043,64.84275799700004],[11.031305850000024,64.92831044300004],[10.725237,64.86402149900005],[11.120891,64.98121649600006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL214","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Krakowski","NUTS_NAME":"Krakowski"},"geometry":{"type":"Polygon","coordinates":[[[20.681539,50.20587549600003],[20.500396500000022,50.13041049800006],[20.632079,50.056317998000054],[20.523501,49.93995499700003],[20.578520500000025,49.83790399900005],[20.518082,49.79824399800003],[20.244272,49.829503499000054],[20.040205,49.74650399800004],[19.99187950000004,49.65354299900008],[19.91968550000007,49.62293649900005],[19.88672450000007,49.680743998000025],[19.912116500000025,49.713539497000056],[19.795933,49.70547699900004],[19.757629,49.83010349700004],[19.810345,49.885189],[19.577438500000028,49.990349998000056],[19.629030500000056,50.03419249900003],[19.520387999000036,50.18864849900007],[19.648142,50.19995299800007],[19.704288,50.29951949900004],[19.893586500000026,50.265539998000065],[19.92094650000007,50.298537999000075],[19.82529050000005,50.32740749900006],[19.844537500000058,50.434195499000054],[19.949966500000073,50.50478250000003],[20.272868,50.46994499700003],[20.41552050000007,50.19123749600004],[20.681539,50.20587549600003]],[[20.159060500000066,50.121605497000076],[19.880745500000046,50.120454],[19.801697,50.069210499000064],[19.79566650000004,49.995284],[19.963435,49.968627499000036],[20.217346500000076,50.05343449800006],[20.159060500000066,50.121605497000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL217","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Tarnowski","NUTS_NAME":"Tarnowski"},"geometry":{"type":"Polygon","coordinates":[[[21.297149500000046,49.84286399900003],[21.34895399900006,49.81496449800005],[21.241772,49.77610449900004],[20.93836950000008,49.72880599800004],[20.518082,49.79824399800003],[20.578520500000025,49.83790399900005],[20.523501,49.93995499700003],[20.632079,50.056317998000054],[20.500396500000022,50.13041049800006],[20.681539,50.20587549600003],[20.786962500000072,50.28838849700003],[21.20883150000003,50.35489749800007],[21.145727,50.25712050000004],[21.146154500000023,50.13003999700004],[21.19502250000005,50.119844999000065],[21.150439,49.97648449900004],[21.28472,49.924064498000064],[21.224959,49.85259849800008],[21.297149500000046,49.84286399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL218","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Nowosadecki","NUTS_NAME":"Nowosadecki"},"geometry":{"type":"Polygon","coordinates":[[[20.518082,49.79824399800003],[20.93836950000008,49.72880599800004],[21.241772,49.77610449900004],[21.347128500000053,49.690366],[21.326661500000057,49.593654498000035],[21.39771250000007,49.433793998000056],[21.055269,49.42130599800004],[21.104102499000078,49.376393998000026],[20.923722500000054,49.29623449800005],[20.743115500000044,49.415132999000036],[20.614939500000048,49.41783299800005],[20.297749,49.571567998000035],[20.039495500000044,49.55997350000007],[19.99187950000004,49.65354299900008],[20.040205,49.74650399800004],[20.244272,49.829503499000054],[20.518082,49.79824399800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL219","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Nowotarski","NUTS_NAME":"Nowotarski"},"geometry":{"type":"Polygon","coordinates":[[[19.99187950000004,49.65354299900008],[20.039495500000044,49.55997350000007],[20.297749,49.571567998000035],[20.614939500000048,49.41783299800005],[20.322434500000043,49.40184599600008],[20.146103500000038,49.317959],[20.088046,49.17958099800006],[19.883929500000022,49.20417699800004],[19.758416,49.215991],[19.799801,49.295825498000056],[19.790551,49.41113149900008],[19.631736,49.40882099600003],[19.467386500000032,49.61376699900006],[19.45943550000004,49.67643349600007],[19.369976500000064,49.69492599900008],[19.444958,49.73403499800003],[19.41495150000003,49.77526499900006],[19.757629,49.83010349700004],[19.795933,49.70547699900004],[19.912116500000025,49.713539497000056],[19.88672450000007,49.680743998000025],[19.91968550000007,49.62293649900005],[19.99187950000004,49.65354299900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL21A","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Oswiecimski","NUTS_NAME":"Oswiecimski"},"geometry":{"type":"Polygon","coordinates":[[[19.844537500000058,50.434195499000054],[19.82529050000005,50.32740749900006],[19.92094650000007,50.298537999000075],[19.893586500000026,50.265539998000065],[19.704288,50.29951949900004],[19.648142,50.19995299800007],[19.520387999000036,50.18864849900007],[19.629030500000056,50.03419249900003],[19.577438500000028,49.990349998000056],[19.810345,49.885189],[19.757629,49.83010349700004],[19.41495150000003,49.77526499900006],[19.31678450000004,49.777886497000054],[19.290000500000076,49.85055299800007],[19.196614500000067,49.86008249900004],[19.184922413000038,49.94999287700006],[19.09321,49.956658996000044],[19.253810500000043,50.13419299900005],[19.44197550000007,50.22657449800005],[19.323423,50.25914499800007],[19.483944999000073,50.32359699800003],[19.48904550000003,50.397027499000046],[19.844537500000058,50.434195499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL22","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Slaskie","NUTS_NAME":"Slaskie"},"geometry":{"type":"Polygon","coordinates":[[[18.922990499000036,51.098453997000036],[19.204254500000047,50.98241849800007],[19.243142500000033,51.03684449600007],[19.32268750000003,51.046190498000044],[19.489786499000047,50.878930498000045],[19.74706,50.86597],[19.832163,50.81970099700004],[19.712949500000036,50.729156],[19.871657500000026,50.69304550000004],[19.831074,50.654359],[19.922185499000022,50.639417999000045],[19.803217,50.56555399900003],[19.949966500000073,50.50478250000003],[19.844537500000058,50.434195499000054],[19.48904550000003,50.397027499000046],[19.483944999000073,50.32359699800003],[19.323423,50.25914499800007],[19.44197550000007,50.22657449800005],[19.253810500000043,50.13419299900005],[19.09321,49.956658996000044],[19.184922413000038,49.94999287700006],[19.196614500000067,49.86008249900004],[19.290000500000076,49.85055299800007],[19.31678450000004,49.777886497000054],[19.41495150000003,49.77526499900006],[19.444958,49.73403499800003],[19.369976500000064,49.69492599900008],[19.45943550000004,49.67643349600007],[19.467386500000032,49.61376699900006],[19.260708,49.53122299800003],[19.153403,49.403777],[18.979868,49.395217],[18.972818,49.50261199800008],[18.851551,49.517189],[18.804583,49.67888899900004],[18.627212,49.72232999600004],[18.575724,49.91042299900005],[18.321821,49.915819999000064],[18.035060999000052,50.06577199800006],[18.059780500000045,50.17465249900005],[18.42586650000004,50.24896549700003],[18.379609,50.48174249900006],[18.440043,50.48072699800008],[18.438194500000066,50.54658399900006],[18.607473500000026,50.55001099900005],[18.478263500000025,50.703099],[18.580970500000035,50.752530998000054],[18.506283,50.801472998000065],[18.615877500000067,50.85358750000006],[18.654821,50.91595999800006],[18.612386,50.955137999000044],[18.695487,51.01577549800004],[18.67295449900007,51.056902500000035],[18.922990499000036,51.098453997000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL224","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Czestochowski","NUTS_NAME":"Czestochowski"},"geometry":{"type":"Polygon","coordinates":[[[19.243142500000033,51.03684449600007],[19.32268750000003,51.046190498000044],[19.489786499000047,50.878930498000045],[19.74706,50.86597],[19.832163,50.81970099700004],[19.712949500000036,50.729156],[19.76885550000003,50.676738998000076],[19.381448,50.588036499000054],[19.35964350000006,50.50950049700003],[19.133614,50.52254499800006],[19.079062500000077,50.50790999800006],[19.095733,50.643523498000036],[18.984682,50.662728498000035],[18.917673500000035,50.765220500000055],[18.615877500000067,50.85358750000006],[18.654821,50.91595999800006],[18.612386,50.955137999000044],[18.695487,51.01577549800004],[18.67295449900007,51.056902500000035],[18.922990499000036,51.098453997000036],[19.204254500000047,50.98241849800007],[19.243142500000033,51.03684449600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL225","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Bielski","NUTS_NAME":"Bielski"},"geometry":{"type":"Polygon","coordinates":[[[19.41495150000003,49.77526499900006],[19.444958,49.73403499800003],[19.369976500000064,49.69492599900008],[19.45943550000004,49.67643349600007],[19.467386500000032,49.61376699900006],[19.260708,49.53122299800003],[19.153403,49.403777],[18.979868,49.395217],[18.972818,49.50261199800008],[18.851551,49.517189],[18.804583,49.67888899900004],[18.627212,49.72232999600004],[18.575724,49.91042299900005],[18.629584,49.89987399800003],[19.09321,49.956658996000044],[19.184922413000038,49.94999287700006],[19.196614500000067,49.86008249900004],[19.290000500000076,49.85055299800007],[19.31678450000004,49.777886497000054],[19.41495150000003,49.77526499900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL227","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Rybnicki","NUTS_NAME":"Rybnicki"},"geometry":{"type":"Polygon","coordinates":[[[18.42586650000004,50.24896549700003],[18.511086,50.17459499900008],[18.719745500000045,50.20011549900005],[18.773562500000025,50.00093449900004],[18.667867500000057,50.009532498000056],[18.629584,49.89987399800003],[18.575724,49.91042299900005],[18.321821,49.915819999000064],[18.035060999000052,50.06577199800006],[18.059780500000045,50.17465249900005],[18.42586650000004,50.24896549700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL228","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Bytomski","NUTS_NAME":"Bytomski"},"geometry":{"type":"Polygon","coordinates":[[[19.133614,50.52254499800006],[18.95777,50.377375],[19.041529500000024,50.34876849800003],[18.86234,50.32809899800003],[18.808099,50.38259899900004],[18.675037,50.33720649900005],[18.665846500000043,50.535438999000064],[18.607473500000026,50.55001099900005],[18.478263500000025,50.703099],[18.580970500000035,50.752530998000054],[18.506283,50.801472998000065],[18.615877500000067,50.85358750000006],[18.917673500000035,50.765220500000055],[18.984682,50.662728498000035],[19.095733,50.643523498000036],[19.079062500000077,50.50790999800006],[19.133614,50.52254499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL229","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Gliwicki","NUTS_NAME":"Gliwicki"},"geometry":{"type":"Polygon","coordinates":[[[18.86234,50.32809899800003],[18.796864,50.24626499800007],[18.719745500000045,50.20011549900005],[18.511086,50.17459499900008],[18.42586650000004,50.24896549700003],[18.379609,50.48174249900006],[18.440043,50.48072699800008],[18.438194500000066,50.54658399900006],[18.607473500000026,50.55001099900005],[18.665846500000043,50.535438999000064],[18.675037,50.33720649900005],[18.808099,50.38259899900004],[18.86234,50.32809899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL22A","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Katowicki","NUTS_NAME":"Katowicki"},"geometry":{"type":"Polygon","coordinates":[[[19.241773500000022,50.16126649800003],[19.074581500000022,50.133951499000034],[18.796864,50.24626499800007],[18.86234,50.32809899800003],[19.041529500000024,50.34876849800003],[19.241773500000022,50.16126649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL22B","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Sosnowiecki","NUTS_NAME":"Sosnowiecki"},"geometry":{"type":"Polygon","coordinates":[[[19.949966500000073,50.50478250000003],[19.844537500000058,50.434195499000054],[19.48904550000003,50.397027499000046],[19.483944999000073,50.32359699800003],[19.323423,50.25914499800007],[19.44197550000007,50.22657449800005],[19.253810500000043,50.13419299900005],[19.241773500000022,50.16126649800003],[19.041529500000024,50.34876849800003],[18.95777,50.377375],[19.133614,50.52254499800006],[19.35964350000006,50.50950049700003],[19.381448,50.588036499000054],[19.76885550000003,50.676738998000076],[19.712949500000036,50.729156],[19.871657500000026,50.69304550000004],[19.831074,50.654359],[19.922185499000022,50.639417999000045],[19.803217,50.56555399900003],[19.949966500000073,50.50478250000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL22C","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Tyski","NUTS_NAME":"Tyski"},"geometry":{"type":"Polygon","coordinates":[[[19.241773500000022,50.16126649800003],[19.253810500000043,50.13419299900005],[19.09321,49.956658996000044],[18.629584,49.89987399800003],[18.667867500000057,50.009532498000056],[18.773562500000025,50.00093449900004],[18.719745500000045,50.20011549900005],[18.796864,50.24626499800007],[19.074581500000022,50.133951499000034],[19.241773500000022,50.16126649800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL3","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION WSCHODNI","NUTS_NAME":"REGION WSCHODNI"},"geometry":{"type":"Polygon","coordinates":[[[23.588702,53.69592849600008],[23.798979,53.274188],[23.918259,53.15762199800008],[23.872502,53.083870997000076],[23.925377,53.024629],[23.916239,52.90481199900006],[23.93864,52.712916],[23.466628,52.54944799700007],[23.178338,52.28314099700003],[23.653733,52.07249499800008],[23.689023,51.99191],[23.612061,51.91877099800007],[23.641429,51.80080299900004],[23.529021,51.73175099800005],[23.564615,51.535325998000076],[23.617665999000053,51.50761399800007],[23.702295500000048,51.40404099800003],[23.646396500000037,51.29227849800003],[23.709221,51.27764849700003],[23.861774500000024,51.15733350000005],[23.970489500000042,50.95061849600006],[24.145782500000053,50.869376998000064],[23.957708500000024,50.79466599800003],[24.071474,50.720646001000034],[24.098669,50.59952549800005],[24.034557500000062,50.44484350000005],[23.727020500000037,50.388003998000045],[23.547642,50.25160199900006],[22.686149500000056,49.57316549600006],[22.640724,49.52996449900007],[22.700411,49.48843399800006],[22.750946,49.31658549700006],[22.70755,49.174300997000046],[22.86286750000005,49.09999850000003],[22.894087,49.01669299800005],[22.56684,49.08837749800006],[22.034885,49.22246399800008],[21.966094,49.34511199900004],[21.840236,49.391505996000035],[21.39771250000007,49.433793998000056],[21.326661500000057,49.593654498000035],[21.347128500000053,49.690366],[21.241772,49.77610449900004],[21.34895399900006,49.81496449800005],[21.297149500000046,49.84286399900003],[21.224959,49.85259849800008],[21.28472,49.924064498000064],[21.150439,49.97648449900004],[21.19502250000005,50.119844999000065],[21.146154500000023,50.13003999700004],[21.145727,50.25712050000004],[21.20883150000003,50.35489749800007],[20.786962500000072,50.28838849700003],[20.681539,50.20587549600003],[20.41552050000007,50.19123749600004],[20.272868,50.46994499700003],[19.949966500000073,50.50478250000003],[19.803217,50.56555399900003],[19.922185499000022,50.639417999000045],[19.831074,50.654359],[19.871657500000026,50.69304550000004],[19.712949500000036,50.729156],[19.832163,50.81970099700004],[19.74706,50.86597],[19.845536,50.93020449800008],[19.882372,51.04766299700003],[20.03947050000005,50.99020649600004],[20.056235999000023,51.05774949800008],[19.98177050000004,51.07315549900005],[19.993847,51.183954],[20.371114,51.23663199600003],[20.366456500000027,51.31061599600008],[20.432815,51.33940499800008],[20.54365750000005,51.231178999000065],[20.813019001000043,51.14583749800005],[20.920719500000075,51.19533749800007],[21.056556,51.15711399600008],[21.070030501000076,51.21142849800003],[21.152750500000025,51.08028599700003],[21.355528,51.085262499000066],[21.463566999000022,51.013111998000056],[21.630164500000035,51.06338899700006],[21.802998,51.07207899800005],[21.783307,51.17904949800004],[21.860111,51.274702500000046],[21.819536,51.299933500000066],[21.85377949900004,51.35174849900005],[21.761347500000056,51.40577899800007],[21.873176001000047,51.474913498000035],[21.82858,51.55884849700004],[21.61554250000006,51.61756199900003],[21.879981,51.69363149800006],[21.835289,51.74444899900004],[21.95288850000003,51.79557499800006],[21.871248500000036,51.83553249900007],[21.91285,51.88653099800007],[21.866380500000048,51.94342599700008],[21.889174500000024,51.97318399900007],[22.344350500000075,51.99600399900004],[22.483290500000066,52.067227],[22.622901499000022,52.01874449600007],[22.698293500000034,52.10849999800007],[22.902187500000025,52.063525499000036],[23.128409,52.28784149900008],[22.929204500000026,52.37359349800005],[22.580413500000077,52.39315799600007],[22.525242500000047,52.522721999000055],[22.408589,52.60968999800008],[22.473925,52.64772549800006],[22.435035999000036,52.69693949900005],[22.453771500000073,52.788238499000045],[22.33441399900005,52.74046399800005],[22.262360500000057,52.81536199900006],[22.30506450000007,52.83934799700006],[22.279745500000047,52.88493649900005],[22.00393150000008,52.87786599800006],[22.001760500000046,52.972328997000034],[21.694427500000074,53.138093997000055],[21.645086,53.28348049700003],[21.73542750000007,53.31272799800007],[21.59819991300003,53.48018197000005],[21.88084150000003,53.46127949800007],[22.135728,53.54453999700007],[22.70350250000007,53.76735449600005],[22.782806,53.915489498000056],[22.476556500000072,54.201305500000046],[22.768004,54.28944349900007],[22.792095500000073,54.363358996000045],[23.010273,54.38280599900003],[23.092209,54.298474499000065],[23.321498500000075,54.25332599700005],[23.485877500000072,54.15324399700006],[23.52863750000006,54.06579799800005],[23.482689,53.99711999900006],[23.51465,53.956559997000056],[23.588702,53.69592849600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL31","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Lubelskie","NUTS_NAME":"Lubelskie"},"geometry":{"type":"Polygon","coordinates":[[[23.653733,52.07249499800008],[23.689023,51.99191],[23.612061,51.91877099800007],[23.641429,51.80080299900004],[23.529021,51.73175099800005],[23.564615,51.535325998000076],[23.617665999000053,51.50761399800007],[23.702295500000048,51.40404099800003],[23.646396500000037,51.29227849800003],[23.709221,51.27764849700003],[23.861774500000024,51.15733350000005],[23.970489500000042,50.95061849600006],[24.145782500000053,50.869376998000064],[23.957708500000024,50.79466599800003],[24.071474,50.720646001000034],[24.098669,50.59952549800005],[24.034557500000062,50.44484350000005],[23.727020500000037,50.388003998000045],[23.547642,50.25160199900006],[23.41320150000007,50.307480500000054],[23.436153,50.344782999000074],[23.387574500000028,50.40539299800008],[23.179142,50.394578996000064],[23.028099500000053,50.288860998000075],[22.638222,50.30307749600007],[22.606575,50.325451],[22.687964500000078,50.356915498000035],[22.50713250000007,50.34399149800004],[22.437193,50.39971749800003],[22.443922500000042,50.46144449800005],[22.5862,50.47734849700004],[22.519738,50.58323349800003],[22.141785500000026,50.66969149700003],[22.205586,50.754013],[22.16296150000005,50.79725149800004],[21.864303500000062,50.80270649800008],[21.802998,51.07207899800005],[21.783307,51.17904949800004],[21.860111,51.274702500000046],[21.819536,51.299933500000066],[21.85377949900004,51.35174849900005],[21.761347500000056,51.40577899800007],[21.873176001000047,51.474913498000035],[21.82858,51.55884849700004],[21.61554250000006,51.61756199900003],[21.879981,51.69363149800006],[21.835289,51.74444899900004],[21.95288850000003,51.79557499800006],[21.871248500000036,51.83553249900007],[21.91285,51.88653099800007],[21.866380500000048,51.94342599700008],[21.889174500000024,51.97318399900007],[22.344350500000075,51.99600399900004],[22.483290500000066,52.067227],[22.622901499000022,52.01874449600007],[22.698293500000034,52.10849999800007],[22.902187500000025,52.063525499000036],[23.128409,52.28784149900008],[23.178338,52.28314099700003],[23.653733,52.07249499800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL311","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Bialski","NUTS_NAME":"Bialski"},"geometry":{"type":"Polygon","coordinates":[[[23.178338,52.28314099700003],[23.653733,52.07249499800008],[23.689023,51.99191],[23.612061,51.91877099800007],[23.641429,51.80080299900004],[23.529021,51.73175099800005],[23.564615,51.535325998000076],[23.617665999000053,51.50761399800007],[23.702295500000048,51.40404099800003],[23.646396500000037,51.29227849800003],[23.709221,51.27764849700003],[23.605156,51.26557749700004],[23.507937,51.35771499800006],[23.26920350000006,51.32980299900004],[22.981220500000063,51.42386349800006],[23.0074,51.52060649800006],[22.785253,51.52081849600006],[22.398407,51.713810496000065],[22.43664750000005,51.746208497000055],[22.35832,51.85980849900005],[22.56180850000004,51.88863699800004],[22.622901499000022,52.01874449600007],[22.698293500000034,52.10849999800007],[22.902187500000025,52.063525499000036],[23.128409,52.28784149900008],[23.178338,52.28314099700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL312","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Chelmsko-zamojski","NUTS_NAME":"Chelmsko-zamojski"},"geometry":{"type":"Polygon","coordinates":[[[23.709221,51.27764849700003],[23.861774500000024,51.15733350000005],[23.970489500000042,50.95061849600006],[24.145782500000053,50.869376998000064],[23.957708500000024,50.79466599800003],[24.071474,50.720646001000034],[24.098669,50.59952549800005],[24.034557500000062,50.44484350000005],[23.727020500000037,50.388003998000045],[23.547642,50.25160199900006],[23.41320150000007,50.307480500000054],[23.436153,50.344782999000074],[23.387574500000028,50.40539299800008],[23.179142,50.394578996000064],[23.028099500000053,50.288860998000075],[22.638222,50.30307749600007],[22.606575,50.325451],[22.687964500000078,50.356915498000035],[22.50713250000007,50.34399149800004],[22.437193,50.39971749800003],[22.443922500000042,50.46144449800005],[22.5862,50.47734849700004],[22.519738,50.58323349800003],[22.660653500000024,50.63716199600003],[22.676479,50.76671449600008],[22.578085499000053,50.83838049800005],[22.918015,50.976534498000035],[22.953422500000045,51.00799649600003],[22.901551,51.02429],[22.917797,51.11037749900004],[23.064719,51.09731499800006],[23.054343500000073,51.23261099900003],[23.226443500000073,51.258142500000076],[23.26920350000006,51.32980299900004],[23.507937,51.35771499800006],[23.605156,51.26557749700004],[23.709221,51.27764849700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL314","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Lubelski","NUTS_NAME":"Lubelski"},"geometry":{"type":"Polygon","coordinates":[[[23.26920350000006,51.32980299900004],[23.226443500000073,51.258142500000076],[23.054343500000073,51.23261099900003],[23.064719,51.09731499800006],[22.917797,51.11037749900004],[22.901551,51.02429],[22.953422500000045,51.00799649600003],[22.918015,50.976534498000035],[22.578085499000053,50.83838049800005],[22.358686,51.048977997000065],[22.177355,51.03555799900005],[22.235192,51.08514899900007],[22.12929,51.22408999800007],[22.314171,51.27773299800003],[22.251027,51.33340349800005],[22.308247,51.424666],[22.189831500000025,51.487558499000045],[22.266743,51.51531799800006],[22.187790500000062,51.58437699900003],[22.19330750000006,51.65969399900007],[22.340148,51.64202899800006],[22.398407,51.713810496000065],[22.785253,51.52081849600006],[23.0074,51.52060649800006],[22.981220500000063,51.42386349800006],[23.26920350000006,51.32980299900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL315","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Pulawski","NUTS_NAME":"Pulawski"},"geometry":{"type":"Polygon","coordinates":[[[22.622901499000022,52.01874449600007],[22.56180850000004,51.88863699800004],[22.35832,51.85980849900005],[22.43664750000005,51.746208497000055],[22.398407,51.713810496000065],[22.340148,51.64202899800006],[22.19330750000006,51.65969399900007],[22.187790500000062,51.58437699900003],[22.266743,51.51531799800006],[22.189831500000025,51.487558499000045],[22.308247,51.424666],[22.251027,51.33340349800005],[22.314171,51.27773299800003],[22.12929,51.22408999800007],[22.235192,51.08514899900007],[22.177355,51.03555799900005],[22.358686,51.048977997000065],[22.578085499000053,50.83838049800005],[22.676479,50.76671449600008],[22.660653500000024,50.63716199600003],[22.519738,50.58323349800003],[22.141785500000026,50.66969149700003],[22.205586,50.754013],[22.16296150000005,50.79725149800004],[21.864303500000062,50.80270649800008],[21.802998,51.07207899800005],[21.783307,51.17904949800004],[21.860111,51.274702500000046],[21.819536,51.299933500000066],[21.85377949900004,51.35174849900005],[21.761347500000056,51.40577899800007],[21.873176001000047,51.474913498000035],[21.82858,51.55884849700004],[21.61554250000006,51.61756199900003],[21.879981,51.69363149800006],[21.835289,51.74444899900004],[21.95288850000003,51.79557499800006],[21.871248500000036,51.83553249900007],[21.91285,51.88653099800007],[21.866380500000048,51.94342599700008],[21.889174500000024,51.97318399900007],[22.344350500000075,51.99600399900004],[22.483290500000066,52.067227],[22.622901499000022,52.01874449600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL32","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Podkarpackie","NUTS_NAME":"Podkarpackie"},"geometry":{"type":"Polygon","coordinates":[[[22.437193,50.39971749800003],[22.50713250000007,50.34399149800004],[22.687964500000078,50.356915498000035],[22.606575,50.325451],[22.638222,50.30307749600007],[23.028099500000053,50.288860998000075],[23.179142,50.394578996000064],[23.387574500000028,50.40539299800008],[23.436153,50.344782999000074],[23.41320150000007,50.307480500000054],[23.547642,50.25160199900006],[22.686149500000056,49.57316549600006],[22.640724,49.52996449900007],[22.700411,49.48843399800006],[22.750946,49.31658549700006],[22.70755,49.174300997000046],[22.86286750000005,49.09999850000003],[22.894087,49.01669299800005],[22.56684,49.08837749800006],[22.034885,49.22246399800008],[21.966094,49.34511199900004],[21.840236,49.391505996000035],[21.39771250000007,49.433793998000056],[21.326661500000057,49.593654498000035],[21.347128500000053,49.690366],[21.241772,49.77610449900004],[21.34895399900006,49.81496449800005],[21.297149500000046,49.84286399900003],[21.224959,49.85259849800008],[21.28472,49.924064498000064],[21.150439,49.97648449900004],[21.19502250000005,50.119844999000065],[21.146154500000023,50.13003999700004],[21.145727,50.25712050000004],[21.20883150000003,50.35489749800007],[21.60119650000007,50.51969549900008],[21.82088650000003,50.69158649900004],[21.864303500000062,50.80270649800008],[22.16296150000005,50.79725149800004],[22.205586,50.754013],[22.141785500000026,50.66969149700003],[22.519738,50.58323349800003],[22.5862,50.47734849700004],[22.443922500000042,50.46144449800005],[22.437193,50.39971749800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL323","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Krosnienski","NUTS_NAME":"Krosnienski"},"geometry":{"type":"Polygon","coordinates":[[[21.47736750000007,49.840546],[21.559855500000026,49.78611199800008],[21.66080050000005,49.84368949700007],[21.89479650000004,49.78109349600004],[22.096148,49.83576799800005],[22.294474500000035,49.74457999800006],[22.253974500000027,49.709535998000035],[22.427904,49.59317250000004],[22.58085250000005,49.670919996000066],[22.686149500000056,49.57316549600006],[22.640724,49.52996449900007],[22.700411,49.48843399800006],[22.750946,49.31658549700006],[22.70755,49.174300997000046],[22.86286750000005,49.09999850000003],[22.894087,49.01669299800005],[22.56684,49.08837749800006],[22.034885,49.22246399800008],[21.966094,49.34511199900004],[21.840236,49.391505996000035],[21.39771250000007,49.433793998000056],[21.326661500000057,49.593654498000035],[21.347128500000053,49.690366],[21.241772,49.77610449900004],[21.34895399900006,49.81496449800005],[21.297149500000046,49.84286399900003],[21.47736750000007,49.840546]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL324","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Przemyski","NUTS_NAME":"Przemyski"},"geometry":{"type":"Polygon","coordinates":[[[23.547642,50.25160199900006],[22.686149500000056,49.57316549600006],[22.58085250000005,49.670919996000066],[22.427904,49.59317250000004],[22.253974500000027,49.709535998000035],[22.294474500000035,49.74457999800006],[22.310238500000025,49.869822999000064],[22.23422750000003,49.93153899600003],[22.29612650000007,49.93373249900003],[22.447906,50.12307549700006],[22.523282,50.129020498000045],[22.638222,50.30307749600007],[23.028099500000053,50.288860998000075],[23.179142,50.394578996000064],[23.387574500000028,50.40539299800008],[23.436153,50.344782999000074],[23.41320150000007,50.307480500000054],[23.547642,50.25160199900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL42","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Zachodniopomorskie","NUTS_NAME":"Zachodniopomorskie"},"geometry":{"type":"Polygon","coordinates":[[[16.804609500000026,54.11607299800005],[16.792764500000033,53.985550499000055],[16.982053499000074,53.90490949900004],[16.872407,53.86866299900004],[16.945877499000062,53.802610997000045],[16.857653500000026,53.74709499800008],[16.892248,53.655868998000074],[16.758229500000027,53.62412199600004],[16.645440999000073,53.48463299600007],[16.45421,53.48847550000005],[16.475250500000072,53.387784999000075],[16.696911500000056,53.30104599900005],[16.405317500000024,53.158510499000045],[16.3173,53.043952499000056],[15.96249750000004,53.04138099600004],[15.979279,53.11031049600007],[15.84818,53.11523650000004],[15.721065999000075,52.98659799900008],[15.286838500000044,52.95719749700004],[15.33263450000004,52.89896699800005],[15.071762500000034,52.83058150000005],[14.904187,52.88390949800004],[14.747612,52.647111497000026],[14.565063,52.62449699900003],[14.436438,52.67990049800005],[14.127624,52.83166249900006],[14.156692,52.895590998000046],[14.143658,52.96136849900006],[14.357842,53.06766699800005],[14.377268,53.20175199800008],[14.441223,53.25435999900003],[14.412157,53.32963599700008],[14.267542,53.697806496000055],[14.267567084000063,53.697836718000076],[14.302757,53.736885],[14.595727,53.591762499000026],[14.619581500000038,53.64978399800003],[14.539019500000052,53.70261],[14.610576500000036,53.76369099800007],[14.573513,53.84919699800008],[14.270650390000071,53.880213135000076],[14.28143566700004,53.85638231300004],[14.286778500000025,53.84457399900003],[14.336749,53.80701849800005],[14.213077500000054,53.86647949800005],[14.226302,53.92865299700003],[14.767616,54.02293799900008],[14.69569,53.99610899600003],[14.620618148000062,53.85057817500007],[14.757152500000075,53.925437497000075],[14.80588350000005,54.02959049800006],[14.727757500000052,54.022860997000066],[15.388074500000073,54.15890699900007],[16.102422500000046,54.27442849900007],[16.472283,54.51224349900008],[16.699085,54.569247],[16.85862050000003,54.38257649700006],[16.813893500000063,54.318491],[16.865116499000067,54.260984998000026],[16.711381,54.21400299900006],[16.804609500000026,54.11607299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL424","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Szczecin","NUTS_NAME":"Miasto Szczecin"},"geometry":{"type":"Polygon","coordinates":[[[14.791054,53.39610999900003],[14.759483500000044,53.36164299700005],[14.803832,53.32736249800007],[14.575121,53.33349949800004],[14.484466500000053,53.40716449800004],[14.460598,53.476447998000026],[14.658552,53.53593549800007],[14.719865500000026,53.40755999900006],[14.791054,53.39610999900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL426","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Koszalinski","NUTS_NAME":"Koszalinski"},"geometry":{"type":"Polygon","coordinates":[[[16.792764500000033,53.985550499000055],[16.690343,53.949861498000075],[16.731993,53.889985],[16.366108999000062,53.916899998000076],[16.184234,53.820266],[16.14402750000005,53.878476997000064],[15.786874,53.89056499800006],[15.673763,53.97020150000003],[15.479201500000045,53.877115],[15.417937,53.957251],[15.452467500000068,54.05510950000007],[15.359935,54.07933599800003],[15.388074500000073,54.15890699900007],[16.102422500000046,54.27442849900007],[16.472283,54.51224349900008],[16.699085,54.569247],[16.85862050000003,54.38257649700006],[16.813893500000063,54.318491],[16.865116499000067,54.260984998000026],[16.711381,54.21400299900006],[16.804609500000026,54.11607299800005],[16.792764500000033,53.985550499000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL427","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Szczecinecko-pyrzycki","NUTS_NAME":"Szczecinecko-pyrzycki"},"geometry":{"type":"Polygon","coordinates":[[[16.982053499000074,53.90490949900004],[16.872407,53.86866299900004],[16.945877499000062,53.802610997000045],[16.857653500000026,53.74709499800008],[16.892248,53.655868998000074],[16.758229500000027,53.62412199600004],[16.645440999000073,53.48463299600007],[16.45421,53.48847550000005],[16.475250500000072,53.387784999000075],[16.696911500000056,53.30104599900005],[16.405317500000024,53.158510499000045],[16.3173,53.043952499000056],[15.96249750000004,53.04138099600004],[15.979279,53.11031049600007],[15.84818,53.11523650000004],[15.721065999000075,52.98659799900008],[15.286838500000044,52.95719749700004],[15.33263450000004,52.89896699800005],[15.071762500000034,52.83058150000005],[14.904187,52.88390949800004],[14.747612,52.647111497000026],[14.565063,52.62449699900003],[14.436438,52.67990049800005],[14.58253650000006,52.75458449800004],[14.696804,52.92546249800006],[14.695868999000027,53.00705099700008],[14.757231500000046,53.06865299900005],[14.729951999000036,53.158727499000065],[14.668887880000057,53.16670521400005],[14.715873,53.27351949800004],[15.012533,53.275815496000064],[15.068943,53.23291149600004],[15.044194,53.18998749600007],[15.181107500000053,53.17824599800008],[15.220107,53.09495549900004],[15.367815,53.16607599900004],[15.25405550000005,53.218335498000044],[15.28259950000006,53.25262399800005],[15.499671,53.24386349900004],[15.476232500000037,53.31126049800008],[15.584765,53.343331997000064],[15.688461,53.484514998000066],[15.145223,53.56131299800006],[15.32699799900007,53.62772599900006],[15.222529,53.726792],[15.479201500000045,53.877115],[15.673763,53.97020150000003],[15.786874,53.89056499800006],[16.14402750000005,53.878476997000064],[16.184234,53.820266],[16.366108999000062,53.916899998000076],[16.731993,53.889985],[16.690343,53.949861498000075],[16.792764500000033,53.985550499000055],[16.982053499000074,53.90490949900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL428","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Szczecinski","NUTS_NAME":"Szczecinski"},"geometry":{"type":"Polygon","coordinates":[[[15.452467500000068,54.05510950000007],[15.417937,53.957251],[15.479201500000045,53.877115],[15.222529,53.726792],[15.32699799900007,53.62772599900006],[15.145223,53.56131299800006],[15.688461,53.484514998000066],[15.584765,53.343331997000064],[15.476232500000037,53.31126049800008],[15.499671,53.24386349900004],[15.28259950000006,53.25262399800005],[15.25405550000005,53.218335498000044],[15.367815,53.16607599900004],[15.220107,53.09495549900004],[15.181107500000053,53.17824599800008],[15.044194,53.18998749600007],[15.068943,53.23291149600004],[15.012533,53.275815496000064],[14.715873,53.27351949800004],[14.668887880000057,53.16670521400005],[14.729951999000036,53.158727499000065],[14.757231500000046,53.06865299900005],[14.695868999000027,53.00705099700008],[14.696804,52.92546249800006],[14.58253650000006,52.75458449800004],[14.436438,52.67990049800005],[14.127624,52.83166249900006],[14.156692,52.895590998000046],[14.143658,52.96136849900006],[14.357842,53.06766699800005],[14.377268,53.20175199800008],[14.441223,53.25435999900003],[14.412157,53.32963599700008],[14.267542,53.697806496000055],[14.267567084000063,53.697836718000076],[14.302757,53.736885],[14.595727,53.591762499000026],[14.619581500000038,53.64978399800003],[14.539019500000052,53.70261],[14.610576500000036,53.76369099800007],[14.573513,53.84919699800008],[14.270650390000071,53.880213135000076],[14.28143566700004,53.85638231300004],[14.286778500000025,53.84457399900003],[14.336749,53.80701849800005],[14.213077500000054,53.86647949800005],[14.226302,53.92865299700003],[14.767616,54.02293799900008],[14.69569,53.99610899600003],[14.620618148000062,53.85057817500007],[14.757152500000075,53.925437497000075],[14.80588350000005,54.02959049800006],[14.727757500000052,54.022860997000066],[15.388074500000073,54.15890699900007],[15.359935,54.07933599800003],[15.452467500000068,54.05510950000007]],[[14.484466500000053,53.40716449800004],[14.575121,53.33349949800004],[14.803832,53.32736249800007],[14.759483500000044,53.36164299700005],[14.791054,53.39610999900003],[14.719865500000026,53.40755999900006],[14.658552,53.53593549800007],[14.460598,53.476447998000026],[14.484466500000053,53.40716449800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL43","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Lubuskie","NUTS_NAME":"Lubuskie"},"geometry":{"type":"Polygon","coordinates":[[[15.923608500000057,52.83117850000008],[15.946907,52.75489599700006],[15.793263500000023,52.70692799600005],[15.776346500000045,52.63786950000008],[15.891571500000055,52.47315549800004],[15.804233,52.44419749700006],[15.900337500000035,52.39086949600005],[15.880810499000063,52.29042949800004],[15.833296,52.111123],[15.967463500000065,52.08194850000007],[16.010617,51.98142249800003],[16.122916,51.99312899800003],[16.137304500000027,51.92695199700006],[16.09856750000006,51.89378650000003],[16.284856,51.90001549600004],[16.416191,51.784864999000035],[16.269073,51.68447149800005],[15.979245,51.802309],[15.963185,51.733663997000065],[15.869526500000063,51.71966199800005],[15.823620500000061,51.57274349700003],[15.71403250000003,51.51739699600006],[15.599760500000059,51.44567449800007],[15.406762,51.52037099800003],[15.324932500000045,51.41849599900007],[15.190489,51.46392199600007],[14.974183,51.36394999600003],[14.948293,51.47176899800007],[14.735769,51.52595199900003],[14.729862,51.581776998000066],[14.752694,51.66709599700005],[14.590146,51.82102599800004],[14.693557,51.901287996000065],[14.716716,52.001188],[14.755227,52.070024997000075],[14.682174,52.11468899800008],[14.716044,52.235458],[14.600891500000046,52.27205199800005],[14.534361999000055,52.39500799900003],[14.635373,52.49636849900003],[14.60426,52.53348949800005],[14.639691,52.57157449700003],[14.565063,52.62449699900003],[14.747612,52.647111497000026],[14.904187,52.88390949800004],[15.071762500000034,52.83058150000005],[15.33263450000004,52.89896699800005],[15.286838500000044,52.95719749700004],[15.721065999000075,52.98659799900008],[15.84818,53.11523650000004],[15.979279,53.11031049600007],[15.96249750000004,53.04138099600004],[15.979305,52.87763099600005],[15.923608500000057,52.83117850000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL431","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Gorzowski","NUTS_NAME":"Gorzowski"},"geometry":{"type":"Polygon","coordinates":[[[15.96249750000004,53.04138099600004],[15.979305,52.87763099600005],[15.923608500000057,52.83117850000008],[15.946907,52.75489599700006],[15.793263500000023,52.70692799600005],[15.776346500000045,52.63786950000008],[15.891571500000055,52.47315549800004],[15.804233,52.44419749700006],[15.900337500000035,52.39086949600005],[15.880810499000063,52.29042949800004],[15.257388500000047,52.37161099800005],[15.149259,52.23377949700006],[14.889526,52.20676849700004],[14.755227,52.070024997000075],[14.682174,52.11468899800008],[14.716044,52.235458],[14.600891500000046,52.27205199800005],[14.534361999000055,52.39500799900003],[14.635373,52.49636849900003],[14.60426,52.53348949800005],[14.639691,52.57157449700003],[14.565063,52.62449699900003],[14.747612,52.647111497000026],[14.904187,52.88390949800004],[15.071762500000034,52.83058150000005],[15.33263450000004,52.89896699800005],[15.286838500000044,52.95719749700004],[15.721065999000075,52.98659799900008],[15.84818,53.11523650000004],[15.979279,53.11031049600007],[15.96249750000004,53.04138099600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL432","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Zielonogórski","NUTS_NAME":"Zielonogórski"},"geometry":{"type":"Polygon","coordinates":[[[15.880810499000063,52.29042949800004],[15.833296,52.111123],[15.967463500000065,52.08194850000007],[16.010617,51.98142249800003],[16.122916,51.99312899800003],[16.137304500000027,51.92695199700006],[16.09856750000006,51.89378650000003],[16.284856,51.90001549600004],[16.416191,51.784864999000035],[16.269073,51.68447149800005],[15.979245,51.802309],[15.963185,51.733663997000065],[15.869526500000063,51.71966199800005],[15.823620500000061,51.57274349700003],[15.71403250000003,51.51739699600006],[15.599760500000059,51.44567449800007],[15.406762,51.52037099800003],[15.324932500000045,51.41849599900007],[15.190489,51.46392199600007],[14.974183,51.36394999600003],[14.948293,51.47176899800007],[14.735769,51.52595199900003],[14.729862,51.581776998000066],[14.752694,51.66709599700005],[14.590146,51.82102599800004],[14.693557,51.901287996000065],[14.716716,52.001188],[14.755227,52.070024997000075],[14.889526,52.20676849700004],[15.149259,52.23377949700006],[15.257388500000047,52.37161099800005],[15.880810499000063,52.29042949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL5","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION POLUDNIOWO-ZACHODNI","NUTS_NAME":"REGION POLUDNIOWO-ZACHODNI"},"geometry":{"type":"Polygon","coordinates":[[[15.979245,51.802309],[16.269073,51.68447149800005],[16.416191,51.784864999000035],[16.647939,51.74416199800004],[16.639154500000075,51.66602949900005],[16.783478500000058,51.64098699900006],[16.828372,51.57218799800006],[17.125499,51.56531750000005],[17.257433,51.642843],[17.556245,51.58430149800006],[17.576286,51.539581498000075],[17.51216,51.50550399700006],[17.540565999000023,51.421283],[17.729088,51.39570149700006],[17.70442299900003,51.36684949600004],[17.764996,51.31638749800004],[17.744111,51.24296350000003],[17.795269500000074,51.194146],[17.851184,51.17243899700003],[17.82347850000002,51.117894498000055],[17.939456,51.10886499800006],[18.163681500000052,51.17251699600007],[18.67295449900007,51.056902500000035],[18.695487,51.01577549800004],[18.612386,50.955137999000044],[18.654821,50.91595999800006],[18.615877500000067,50.85358750000006],[18.506283,50.801472998000065],[18.580970500000035,50.752530998000054],[18.478263500000025,50.703099],[18.607473500000026,50.55001099900005],[18.438194500000066,50.54658399900006],[18.440043,50.48072699800008],[18.379609,50.48174249900006],[18.42586650000004,50.24896549700003],[18.059780500000045,50.17465249900005],[18.035060999000052,50.06577199800006],[17.868675,49.972545996000065],[17.592736,50.16001399900006],[17.758479,50.20656799900007],[17.71840399900003,50.32094999900005],[17.429605,50.254513],[16.907924500000036,50.449454],[16.860915,50.40808099700007],[17.028323,50.22999699800005],[16.86327,50.19812299800003],[16.70553,50.09651149800004],[16.58029,50.14278799700003],[16.548217,50.22978399800007],[16.360988500000076,50.37923649600003],[16.195729,50.432135],[16.443536,50.586257499000055],[16.322517,50.66645299700008],[16.107318,50.66207299800004],[16.024848,50.598625],[15.986371,50.61474699900003],[15.990918,50.683419],[15.860921,50.67491299800008],[15.816207,50.755322],[15.535267500000032,50.779375998000035],[15.370452,50.780894999000054],[15.277071,50.891020999000034],[15.274085,50.97954299700007],[15.032691,51.021315998000034],[14.968252,50.990008999000054],[15.019613,50.96659899900004],[15.00204,50.869024],[14.823362,50.87055049600008],[15.037271,51.243749998000055],[14.974183,51.36394999600003],[15.190489,51.46392199600007],[15.324932500000045,51.41849599900007],[15.406762,51.52037099800003],[15.599760500000059,51.44567449800007],[15.71403250000003,51.51739699600006],[15.823620500000061,51.57274349700003],[15.869526500000063,51.71966199800005],[15.963185,51.733663997000065],[15.979245,51.802309]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL51","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Dolnoslaskie","NUTS_NAME":"Dolnoslaskie"},"geometry":{"type":"Polygon","coordinates":[[[15.979245,51.802309],[16.269073,51.68447149800005],[16.416191,51.784864999000035],[16.647939,51.74416199800004],[16.639154500000075,51.66602949900005],[16.783478500000058,51.64098699900006],[16.828372,51.57218799800006],[17.125499,51.56531750000005],[17.257433,51.642843],[17.556245,51.58430149800006],[17.576286,51.539581498000075],[17.51216,51.50550399700006],[17.540565999000023,51.421283],[17.729088,51.39570149700006],[17.70442299900003,51.36684949600004],[17.764996,51.31638749800004],[17.744111,51.24296350000003],[17.795269500000074,51.194146],[17.578243500000042,51.16224099900006],[17.582133500000054,51.08184249800007],[17.518703,51.05881499900005],[17.542534500000045,50.99120049800007],[17.43139750000006,50.967085498000074],[17.328201500000034,50.850595498000075],[17.354658500000028,50.79479999900008],[17.22078750000003,50.71370299900008],[17.244772499000078,50.61972049900004],[17.164896,50.613462998000045],[16.907924500000036,50.449454],[16.860915,50.40808099700007],[17.028323,50.22999699800005],[16.86327,50.19812299800003],[16.70553,50.09651149800004],[16.58029,50.14278799700003],[16.548217,50.22978399800007],[16.360988500000076,50.37923649600003],[16.195729,50.432135],[16.443536,50.586257499000055],[16.322517,50.66645299700008],[16.107318,50.66207299800004],[16.024848,50.598625],[15.986371,50.61474699900003],[15.990918,50.683419],[15.860921,50.67491299800008],[15.816207,50.755322],[15.535267500000032,50.779375998000035],[15.370452,50.780894999000054],[15.277071,50.891020999000034],[15.274085,50.97954299700007],[15.032691,51.021315998000034],[14.968252,50.990008999000054],[15.019613,50.96659899900004],[15.00204,50.869024],[14.823362,50.87055049600008],[15.037271,51.243749998000055],[14.974183,51.36394999600003],[15.190489,51.46392199600007],[15.324932500000045,51.41849599900007],[15.406762,51.52037099800003],[15.599760500000059,51.44567449800007],[15.71403250000003,51.51739699600006],[15.823620500000061,51.57274349700003],[15.869526500000063,51.71966199800005],[15.963185,51.733663997000065],[15.979245,51.802309]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL514","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Wroclaw","NUTS_NAME":"Miasto Wroclaw"},"geometry":{"type":"Polygon","coordinates":[[[17.175163,51.10767399800005],[17.029004500000042,51.04273599800007],[16.80733950000007,51.13894749700006],[16.881741,51.202521997000076],[17.156084,51.17536349900007],[17.175163,51.10767399800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL515","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Jeleniogórski","NUTS_NAME":"Jeleniogórski"},"geometry":{"type":"Polygon","coordinates":[[[15.71403250000003,51.51739699600006],[15.858353,51.45656849800008],[15.822154,51.45070849800004],[15.850278500000059,51.39014499800004],[15.75219050000004,51.224986999000066],[16.05520850000005,51.223069],[16.017363500000044,51.159422498000026],[16.092471500000045,51.09936449800006],[16.441790999000034,51.12612849800007],[16.480191,51.08592599800005],[16.337212500000078,51.02903249800005],[16.19863249900004,51.01672099700005],[16.159403,50.95068949800003],[16.197590500000047,50.87152249700006],[16.084206,50.86033150000003],[16.075597,50.77964749900008],[16.153476500000068,50.68650849800008],[16.107318,50.66207299800004],[16.024848,50.598625],[15.986371,50.61474699900003],[15.990918,50.683419],[15.860921,50.67491299800008],[15.816207,50.755322],[15.535267500000032,50.779375998000035],[15.370452,50.780894999000054],[15.277071,50.891020999000034],[15.274085,50.97954299700007],[15.032691,51.021315998000034],[14.968252,50.990008999000054],[15.019613,50.96659899900004],[15.00204,50.869024],[14.823362,50.87055049600008],[15.037271,51.243749998000055],[14.974183,51.36394999600003],[15.190489,51.46392199600007],[15.324932500000045,51.41849599900007],[15.406762,51.52037099800003],[15.599760500000059,51.44567449800007],[15.71403250000003,51.51739699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL516","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Legnicko-Glogowski","NUTS_NAME":"Legnicko-Glogowski"},"geometry":{"type":"Polygon","coordinates":[[[16.416191,51.784864999000035],[16.647939,51.74416199800004],[16.639154500000075,51.66602949900005],[16.783478500000058,51.64098699900006],[16.828372,51.57218799800006],[16.764173500000027,51.48540749800003],[16.45458750000006,51.493618999000034],[16.477188500000068,51.36689399900007],[16.435190500000033,51.275581],[16.441790999000034,51.12612849800007],[16.092471500000045,51.09936449800006],[16.017363500000044,51.159422498000026],[16.05520850000005,51.223069],[15.75219050000004,51.224986999000066],[15.850278500000059,51.39014499800004],[15.822154,51.45070849800004],[15.858353,51.45656849800008],[15.71403250000003,51.51739699600006],[15.823620500000061,51.57274349700003],[15.869526500000063,51.71966199800005],[15.963185,51.733663997000065],[15.979245,51.802309],[16.269073,51.68447149800005],[16.416191,51.784864999000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL517","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Walbrzyski","NUTS_NAME":"Walbrzyski"},"geometry":{"type":"Polygon","coordinates":[[[17.164896,50.613462998000045],[16.907924500000036,50.449454],[16.860915,50.40808099700007],[17.028323,50.22999699800005],[16.86327,50.19812299800003],[16.70553,50.09651149800004],[16.58029,50.14278799700003],[16.548217,50.22978399800007],[16.360988500000076,50.37923649600003],[16.195729,50.432135],[16.443536,50.586257499000055],[16.322517,50.66645299700008],[16.107318,50.66207299800004],[16.153476500000068,50.68650849800008],[16.075597,50.77964749900008],[16.084206,50.86033150000003],[16.197590500000047,50.87152249700006],[16.159403,50.95068949800003],[16.19863249900004,51.01672099700005],[16.337212500000078,51.02903249800005],[16.602655,51.00179799800003],[16.63025,50.98034299900007],[16.575754,50.91993549800003],[16.690228,50.93937649800006],[16.687232,50.840294],[16.88078450000006,50.82884949800007],[16.857894,50.72141649900004],[17.055497500000058,50.69757549700006],[17.164896,50.613462998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL518","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Wroclawski","NUTS_NAME":"Wroclawski"},"geometry":{"type":"Polygon","coordinates":[[[17.795269500000074,51.194146],[17.578243500000042,51.16224099900006],[17.582133500000054,51.08184249800007],[17.518703,51.05881499900005],[17.542534500000045,50.99120049800007],[17.43139750000006,50.967085498000074],[17.328201500000034,50.850595498000075],[17.354658500000028,50.79479999900008],[17.22078750000003,50.71370299900008],[17.244772499000078,50.61972049900004],[17.164896,50.613462998000045],[17.055497500000058,50.69757549700006],[16.857894,50.72141649900004],[16.88078450000006,50.82884949800007],[16.687232,50.840294],[16.690228,50.93937649800006],[16.575754,50.91993549800003],[16.63025,50.98034299900007],[16.602655,51.00179799800003],[16.337212500000078,51.02903249800005],[16.480191,51.08592599800005],[16.441790999000034,51.12612849800007],[16.435190500000033,51.275581],[16.477188500000068,51.36689399900007],[16.45458750000006,51.493618999000034],[16.764173500000027,51.48540749800003],[16.828372,51.57218799800006],[17.125499,51.56531750000005],[17.257433,51.642843],[17.556245,51.58430149800006],[17.576286,51.539581498000075],[17.51216,51.50550399700006],[17.540565999000023,51.421283],[17.729088,51.39570149700006],[17.70442299900003,51.36684949600004],[17.764996,51.31638749800004],[17.744111,51.24296350000003],[17.795269500000074,51.194146]],[[17.175163,51.10767399800005],[17.156084,51.17536349900007],[16.881741,51.202521997000076],[16.80733950000007,51.13894749700006],[17.029004500000042,51.04273599800007],[17.175163,51.10767399800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL52","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Opolskie","NUTS_NAME":"Opolskie"},"geometry":{"type":"Polygon","coordinates":[[[17.82347850000002,51.117894498000055],[17.939456,51.10886499800006],[18.163681500000052,51.17251699600007],[18.67295449900007,51.056902500000035],[18.695487,51.01577549800004],[18.612386,50.955137999000044],[18.654821,50.91595999800006],[18.615877500000067,50.85358750000006],[18.506283,50.801472998000065],[18.580970500000035,50.752530998000054],[18.478263500000025,50.703099],[18.607473500000026,50.55001099900005],[18.438194500000066,50.54658399900006],[18.440043,50.48072699800008],[18.379609,50.48174249900006],[18.42586650000004,50.24896549700003],[18.059780500000045,50.17465249900005],[18.035060999000052,50.06577199800006],[17.868675,49.972545996000065],[17.592736,50.16001399900006],[17.758479,50.20656799900007],[17.71840399900003,50.32094999900005],[17.429605,50.254513],[16.907924500000036,50.449454],[17.164896,50.613462998000045],[17.244772499000078,50.61972049900004],[17.22078750000003,50.71370299900008],[17.354658500000028,50.79479999900008],[17.328201500000034,50.850595498000075],[17.43139750000006,50.967085498000074],[17.542534500000045,50.99120049800007],[17.518703,51.05881499900005],[17.582133500000054,51.08184249800007],[17.578243500000042,51.16224099900006],[17.795269500000074,51.194146],[17.851184,51.17243899700003],[17.82347850000002,51.117894498000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL523","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Nyski","NUTS_NAME":"Nyski"},"geometry":{"type":"Polygon","coordinates":[[[17.939456,51.10886499800006],[17.934084,50.98285449900004],[18.011709,50.96359099800003],[17.889229,50.861420499000076],[17.659677500000043,50.899318496000035],[17.59267650000004,50.854100997000046],[17.760032,50.780366498000035],[17.745017,50.72628399800004],[17.539681,50.72066949800006],[17.467142500000023,50.60036899900007],[17.813818500000025,50.53017549900005],[17.753450500000042,50.42116599800005],[18.021691,50.35793649800007],[17.911334500000066,50.30372699700007],[17.91832850000003,50.227349496000045],[18.059780500000045,50.17465249900005],[18.035060999000052,50.06577199800006],[17.868675,49.972545996000065],[17.592736,50.16001399900006],[17.758479,50.20656799900007],[17.71840399900003,50.32094999900005],[17.429605,50.254513],[16.907924500000036,50.449454],[17.164896,50.613462998000045],[17.244772499000078,50.61972049900004],[17.22078750000003,50.71370299900008],[17.354658500000028,50.79479999900008],[17.328201500000034,50.850595498000075],[17.43139750000006,50.967085498000074],[17.542534500000045,50.99120049800007],[17.518703,51.05881499900005],[17.582133500000054,51.08184249800007],[17.578243500000042,51.16224099900006],[17.795269500000074,51.194146],[17.851184,51.17243899700003],[17.82347850000002,51.117894498000055],[17.939456,51.10886499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL524","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Opolski","NUTS_NAME":"Opolski"},"geometry":{"type":"Polygon","coordinates":[[[18.67295449900007,51.056902500000035],[18.695487,51.01577549800004],[18.612386,50.955137999000044],[18.654821,50.91595999800006],[18.615877500000067,50.85358750000006],[18.506283,50.801472998000065],[18.580970500000035,50.752530998000054],[18.478263500000025,50.703099],[18.607473500000026,50.55001099900005],[18.438194500000066,50.54658399900006],[18.440043,50.48072699800008],[18.379609,50.48174249900006],[18.42586650000004,50.24896549700003],[18.059780500000045,50.17465249900005],[17.91832850000003,50.227349496000045],[17.911334500000066,50.30372699700007],[18.021691,50.35793649800007],[17.753450500000042,50.42116599800005],[17.813818500000025,50.53017549900005],[17.467142500000023,50.60036899900007],[17.539681,50.72066949800006],[17.745017,50.72628399800004],[17.760032,50.780366498000035],[17.59267650000004,50.854100997000046],[17.659677500000043,50.899318496000035],[17.889229,50.861420499000076],[18.011709,50.96359099800003],[17.934084,50.98285449900004],[17.939456,51.10886499800006],[18.163681500000052,51.17251699600007],[18.67295449900007,51.056902500000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL6","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION PÓLNOCNY","NUTS_NAME":"REGION PÓLNOCNY"},"geometry":{"type":"Polygon","coordinates":[[[18.35961,54.81719149600008],[18.82896850000003,54.60770799800008],[18.423325500000033,54.78867299800004],[18.39677450000005,54.747280499000055],[18.54172650000004,54.58447999900005],[18.591159,54.430185496000036],[18.950029500000028,54.35831050000007],[19.33136950000005,54.366705498000044],[19.639032,54.45829449800004],[19.648523,54.45333899900004],[19.224721500000044,54.335841998000035],[19.256953,54.27846049900006],[19.398813,54.26036799600007],[19.803768500000047,54.44241049900006],[20.313503,54.402202],[21.559322,54.32250399900005],[22.792095500000073,54.363358996000045],[22.768004,54.28944349900007],[22.476556500000072,54.201305500000046],[22.782806,53.915489498000056],[22.70350250000007,53.76735449600005],[22.135728,53.54453999700007],[21.88084150000003,53.46127949800007],[21.59819991300003,53.48018197000005],[21.55169,53.478128],[20.675953999000058,53.269529497000065],[20.411137,53.21416499900005],[20.30506950000006,53.140077],[19.978165500000046,53.14391449900006],[19.893881500000077,53.212788499000055],[19.84106950000006,53.14099349800006],[19.761595500000055,53.15179649700008],[19.634234,53.11056699900007],[19.684806,52.96303899700007],[19.444664,52.93904249900004],[19.474977,52.86535349800005],[19.437642500000038,52.81479499800008],[19.52274299900006,52.74920049700006],[19.367898500000024,52.628333498000075],[19.426104500000065,52.61144649900007],[19.34111450000006,52.56705499800006],[19.363607500000057,52.503754498000035],[19.289184500000033,52.39271249700005],[19.04712,52.33280399800003],[18.928907,52.37807699800004],[18.755498500000044,52.33276950000004],[18.650178500000038,52.45026549800008],[18.391668500000037,52.470221999000046],[18.377150500000027,52.53746350000006],[18.204579,52.48316999900004],[17.786666500000024,52.65151549800004],[17.678062,52.60239749800007],[17.627635499000064,52.70817999900004],[17.467201500000044,52.68038399800008],[17.489669500000048,52.72729399800005],[17.458523,52.738948],[17.40436550000004,52.756718],[17.503342,52.78672549600003],[17.509582500000022,52.917767996000066],[17.30155650000006,52.99480299900006],[17.392802500000073,53.14195699900006],[17.321557,53.216272497000034],[17.43879350000003,53.26751450000006],[17.247740500000077,53.379893999000046],[17.390653,53.49096399900003],[16.999648999000044,53.51806250000004],[16.892248,53.655868998000074],[16.857653500000026,53.74709499800008],[16.945877499000062,53.802610997000045],[16.872407,53.86866299900004],[16.982053499000074,53.90490949900004],[16.792764500000033,53.985550499000055],[16.804609500000026,54.11607299800005],[16.711381,54.21400299900006],[16.865116499000067,54.260984998000026],[16.813893500000063,54.318491],[16.85862050000003,54.38257649700006],[16.699085,54.569247],[17.666607,54.78323149900007],[18.35961,54.81719149600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL61","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Kujawsko-Pomorskie","NUTS_NAME":"Kujawsko-Pomorskie"},"geometry":{"type":"Polygon","coordinates":[[[18.072036500000024,53.781074997000076],[18.550634500000058,53.655921498000055],[18.78897150000006,53.70210499900003],[18.739787,53.64637599900004],[18.761577500000044,53.604881997000064],[19.12927650000006,53.58826049800007],[19.199701,53.570058998000036],[19.259198500000025,53.396483],[19.69011050000006,53.33629149700005],[19.718123,53.30206849700005],[19.689079,53.23605849900008],[19.761595500000055,53.15179649700008],[19.634234,53.11056699900007],[19.684806,52.96303899700007],[19.444664,52.93904249900004],[19.474977,52.86535349800005],[19.437642500000038,52.81479499800008],[19.52274299900006,52.74920049700006],[19.367898500000024,52.628333498000075],[19.426104500000065,52.61144649900007],[19.34111450000006,52.56705499800006],[19.363607500000057,52.503754498000035],[19.289184500000033,52.39271249700005],[19.04712,52.33280399800003],[18.928907,52.37807699800004],[18.755498500000044,52.33276950000004],[18.650178500000038,52.45026549800008],[18.391668500000037,52.470221999000046],[18.377150500000027,52.53746350000006],[18.204579,52.48316999900004],[17.786666500000024,52.65151549800004],[17.678062,52.60239749800007],[17.627635499000064,52.70817999900004],[17.467201500000044,52.68038399800008],[17.489669500000048,52.72729399800005],[17.458523,52.738948],[17.40436550000004,52.756718],[17.503342,52.78672549600003],[17.509582500000022,52.917767996000066],[17.30155650000006,52.99480299900006],[17.392802500000073,53.14195699900006],[17.321557,53.216272497000034],[17.43879350000003,53.26751450000006],[17.247740500000077,53.379893999000046],[17.390653,53.49096399900003],[17.448825,53.60201999800006],[17.730447500000025,53.59290499700006],[17.740691,53.67797549800008],[17.85171299900003,53.687752497000076],[17.889393,53.72768099900003],[17.860104,53.754955],[18.031934500000034,53.724946499000055],[18.072036500000024,53.781074997000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL613","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Bydgosko-Torunski","NUTS_NAME":"Bydgosko-Torunski"},"geometry":{"type":"Polygon","coordinates":[[[18.27966850000007,53.25307749600006],[18.42367550000006,53.17655749800008],[18.726667500000076,53.26162149800007],[18.803537,53.21977499900004],[18.727403500000037,53.15277399900003],[19.094164500000034,52.96896349800005],[19.009569500000055,52.96459899800004],[19.064049500000067,52.88265349900007],[19.023134,52.84571899800005],[18.722577,52.93341049700007],[18.564748500000064,52.87898199600005],[18.33899450000007,52.98533299800005],[18.008841500000074,52.96281249800006],[17.668997,53.22832049900006],[17.70242150000007,53.285882498000035],[17.730805,53.33517350000005],[17.68734549900006,53.341566998000076],[17.73753,53.396919998000044],[17.927021,53.43374849800006],[18.27966850000007,53.25307749600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL616","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Grudziadzki","NUTS_NAME":"Grudziadzki"},"geometry":{"type":"Polygon","coordinates":[[[19.12927650000006,53.58826049800007],[19.199701,53.570058998000036],[19.259198500000025,53.396483],[19.69011050000006,53.33629149700005],[19.718123,53.30206849700005],[19.689079,53.23605849900008],[19.761595500000055,53.15179649700008],[19.634234,53.11056699900007],[19.684806,52.96303899700007],[19.444664,52.93904249900004],[19.094164500000034,52.96896349800005],[18.727403500000037,53.15277399900003],[18.803537,53.21977499900004],[18.726667500000076,53.26162149800007],[18.42367550000006,53.17655749800008],[18.27966850000007,53.25307749600006],[18.425586,53.38104749900003],[18.704015499000036,53.45166499900006],[18.761577500000044,53.604881997000064],[19.12927650000006,53.58826049800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL617","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Inowroclawski","NUTS_NAME":"Inowroclawski"},"geometry":{"type":"Polygon","coordinates":[[[18.564748500000064,52.87898199600005],[18.542179,52.81993949800005],[18.62514999900003,52.782947498000055],[18.499955500000056,52.74806799800007],[18.45789350000007,52.64681199800003],[18.480917500000032,52.585572497000044],[18.377150500000027,52.53746350000006],[18.204579,52.48316999900004],[17.786666500000024,52.65151549800004],[17.678062,52.60239749800007],[17.627635499000064,52.70817999900004],[17.467201500000044,52.68038399800008],[17.489669500000048,52.72729399800005],[17.458523,52.738948],[17.40436550000004,52.756718],[17.503342,52.78672549600003],[17.509582500000022,52.917767996000066],[17.30155650000006,52.99480299900006],[17.392802500000073,53.14195699900006],[17.321557,53.216272497000034],[17.43879350000003,53.26751450000006],[17.70242150000007,53.285882498000035],[17.668997,53.22832049900006],[18.008841500000074,52.96281249800006],[18.33899450000007,52.98533299800005],[18.564748500000064,52.87898199600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL618","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Swiecki","NUTS_NAME":"Swiecki"},"geometry":{"type":"Polygon","coordinates":[[[18.761577500000044,53.604881997000064],[18.704015499000036,53.45166499900006],[18.425586,53.38104749900003],[18.27966850000007,53.25307749600006],[17.927021,53.43374849800006],[17.73753,53.396919998000044],[17.68734549900006,53.341566998000076],[17.730805,53.33517350000005],[17.70242150000007,53.285882498000035],[17.43879350000003,53.26751450000006],[17.247740500000077,53.379893999000046],[17.390653,53.49096399900003],[17.448825,53.60201999800006],[17.730447500000025,53.59290499700006],[17.740691,53.67797549800008],[17.85171299900003,53.687752497000076],[17.889393,53.72768099900003],[17.860104,53.754955],[18.031934500000034,53.724946499000055],[18.072036500000024,53.781074997000076],[18.550634500000058,53.655921498000055],[18.78897150000006,53.70210499900003],[18.739787,53.64637599900004],[18.761577500000044,53.604881997000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL619","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Wloclawski","NUTS_NAME":"Wloclawski"},"geometry":{"type":"Polygon","coordinates":[[[19.444664,52.93904249900004],[19.474977,52.86535349800005],[19.437642500000038,52.81479499800008],[19.52274299900006,52.74920049700006],[19.367898500000024,52.628333498000075],[19.426104500000065,52.61144649900007],[19.34111450000006,52.56705499800006],[19.363607500000057,52.503754498000035],[19.289184500000033,52.39271249700005],[19.04712,52.33280399800003],[18.928907,52.37807699800004],[18.755498500000044,52.33276950000004],[18.650178500000038,52.45026549800008],[18.391668500000037,52.470221999000046],[18.377150500000027,52.53746350000006],[18.480917500000032,52.585572497000044],[18.45789350000007,52.64681199800003],[18.499955500000056,52.74806799800007],[18.62514999900003,52.782947498000055],[18.542179,52.81993949800005],[18.564748500000064,52.87898199600005],[18.722577,52.93341049700007],[19.023134,52.84571899800005],[19.064049500000067,52.88265349900007],[19.009569500000055,52.96459899800004],[19.094164500000034,52.96896349800005],[19.444664,52.93904249900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL62","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Warminsko-Mazurskie","NUTS_NAME":"Warminsko-Mazurskie"},"geometry":{"type":"Polygon","coordinates":[[[20.313503,54.402202],[21.559322,54.32250399900005],[22.792095500000073,54.363358996000045],[22.768004,54.28944349900007],[22.476556500000072,54.201305500000046],[22.782806,53.915489498000056],[22.70350250000007,53.76735449600005],[22.135728,53.54453999700007],[21.88084150000003,53.46127949800007],[21.59819991300003,53.48018197000005],[21.55169,53.478128],[20.675953999000058,53.269529497000065],[20.411137,53.21416499900005],[20.30506950000006,53.140077],[19.978165500000046,53.14391449900006],[19.893881500000077,53.212788499000055],[19.84106950000006,53.14099349800006],[19.761595500000055,53.15179649700008],[19.689079,53.23605849900008],[19.718123,53.30206849700005],[19.69011050000006,53.33629149700005],[19.259198500000025,53.396483],[19.199701,53.570058998000036],[19.12927650000006,53.58826049800007],[19.322978,53.81394449900006],[19.456246,53.79695049800006],[19.581343,53.92706299900004],[19.368867,53.93803149900003],[19.377739500000075,53.984958997000035],[19.217688500000065,54.11612149600006],[19.326466500000038,54.203823499000066],[19.256953,54.27846049900006],[19.398813,54.26036799600007],[19.803768500000047,54.44241049900006],[20.313503,54.402202]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL621","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Elblaski","NUTS_NAME":"Elblaski"},"geometry":{"type":"Polygon","coordinates":[[[20.313503,54.402202],[20.33389150000005,54.36736949700003],[20.260984,54.27500199700006],[20.393591,54.221866498000054],[19.944209500000056,54.149877],[20.151507,54.02904099800003],[20.106628,53.91394449900008],[20.177932,53.88219599700005],[20.123257500000022,53.837228],[20.182224500000075,53.77727949900003],[20.111928500000033,53.76643149900008],[20.149159,53.65847249800004],[20.11289,53.62146549700003],[20.272394500000075,53.49388699800005],[20.185718,53.45704249900007],[20.180119,53.330427],[20.257374,53.28489599900007],[20.247601,53.23825349600003],[20.411137,53.21416499900005],[20.30506950000006,53.140077],[19.978165500000046,53.14391449900006],[19.893881500000077,53.212788499000055],[19.84106950000006,53.14099349800006],[19.761595500000055,53.15179649700008],[19.689079,53.23605849900008],[19.718123,53.30206849700005],[19.69011050000006,53.33629149700005],[19.259198500000025,53.396483],[19.199701,53.570058998000036],[19.12927650000006,53.58826049800007],[19.322978,53.81394449900006],[19.456246,53.79695049800006],[19.581343,53.92706299900004],[19.368867,53.93803149900003],[19.377739500000075,53.984958997000035],[19.217688500000065,54.11612149600006],[19.326466500000038,54.203823499000066],[19.256953,54.27846049900006],[19.398813,54.26036799600007],[19.803768500000047,54.44241049900006],[20.313503,54.402202]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL622","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Olsztynski","NUTS_NAME":"Olsztynski"},"geometry":{"type":"Polygon","coordinates":[[[21.559322,54.32250399900005],[21.536763500000063,54.24605899800008],[21.602312,54.22006749700006],[21.531085,54.12274299900008],[21.551478,53.99727849900006],[21.429664,53.94287449700005],[21.52620550000006,53.85460750000004],[21.74378850000005,53.89186649900006],[21.680247,53.722339],[21.46218,53.72190549700008],[21.47598750000003,53.632002998000075],[21.410007,53.607792999000026],[21.455978,53.50928799800005],[21.55169,53.478128],[20.675953999000058,53.269529497000065],[20.411137,53.21416499900005],[20.247601,53.23825349600003],[20.257374,53.28489599900007],[20.180119,53.330427],[20.185718,53.45704249900007],[20.272394500000075,53.49388699800005],[20.11289,53.62146549700003],[20.149159,53.65847249800004],[20.111928500000033,53.76643149900008],[20.182224500000075,53.77727949900003],[20.123257500000022,53.837228],[20.177932,53.88219599700005],[20.106628,53.91394449900008],[20.151507,54.02904099800003],[19.944209500000056,54.149877],[20.393591,54.221866498000054],[20.260984,54.27500199700006],[20.33389150000005,54.36736949700003],[20.313503,54.402202],[21.559322,54.32250399900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL623","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Elcki","NUTS_NAME":"Elcki"},"geometry":{"type":"Polygon","coordinates":[[[22.135728,53.54453999700007],[21.88084150000003,53.46127949800007],[21.59819991300003,53.48018197000005],[21.55169,53.478128],[21.455978,53.50928799800005],[21.410007,53.607792999000026],[21.47598750000003,53.632002998000075],[21.46218,53.72190549700008],[21.680247,53.722339],[21.74378850000005,53.89186649900006],[21.52620550000006,53.85460750000004],[21.429664,53.94287449700005],[21.551478,53.99727849900006],[21.531085,54.12274299900008],[21.602312,54.22006749700006],[21.536763500000063,54.24605899800008],[21.559322,54.32250399900005],[22.792095500000073,54.363358996000045],[22.768004,54.28944349900007],[22.476556500000072,54.201305500000046],[22.782806,53.915489498000056],[22.70350250000007,53.76735449600005],[22.135728,53.54453999700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL63","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Pomorskie","NUTS_NAME":"Pomorskie"},"geometry":{"type":"Polygon","coordinates":[[[18.35961,54.81719149600008],[18.82896850000003,54.60770799800008],[18.423325500000033,54.78867299800004],[18.39677450000005,54.747280499000055],[18.54172650000004,54.58447999900005],[18.591159,54.430185496000036],[18.950029500000028,54.35831050000007],[19.33136950000005,54.366705498000044],[19.639032,54.45829449800004],[19.648523,54.45333899900004],[19.224721500000044,54.335841998000035],[19.256953,54.27846049900006],[19.326466500000038,54.203823499000066],[19.217688500000065,54.11612149600006],[19.377739500000075,53.984958997000035],[19.368867,53.93803149900003],[19.581343,53.92706299900004],[19.456246,53.79695049800006],[19.322978,53.81394449900006],[19.12927650000006,53.58826049800007],[18.761577500000044,53.604881997000064],[18.739787,53.64637599900004],[18.78897150000006,53.70210499900003],[18.550634500000058,53.655921498000055],[18.072036500000024,53.781074997000076],[18.031934500000034,53.724946499000055],[17.860104,53.754955],[17.889393,53.72768099900003],[17.85171299900003,53.687752497000076],[17.740691,53.67797549800008],[17.730447500000025,53.59290499700006],[17.448825,53.60201999800006],[17.390653,53.49096399900003],[16.999648999000044,53.51806250000004],[16.892248,53.655868998000074],[16.857653500000026,53.74709499800008],[16.945877499000062,53.802610997000045],[16.872407,53.86866299900004],[16.982053499000074,53.90490949900004],[16.792764500000033,53.985550499000055],[16.804609500000026,54.11607299800005],[16.711381,54.21400299900006],[16.865116499000067,54.260984998000026],[16.813893500000063,54.318491],[16.85862050000003,54.38257649700006],[16.699085,54.569247],[17.666607,54.78323149900007],[18.35961,54.81719149600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL633","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Trójmiejski","NUTS_NAME":"Trójmiejski"},"geometry":{"type":"Polygon","coordinates":[[[18.950029500000028,54.35831050000007],[18.602853500000037,54.28548949900005],[18.456920500000024,54.33227399800006],[18.429497,54.38504749800006],[18.45794150000006,54.43603999700008],[18.358159,54.493491498000026],[18.54172650000004,54.58447999900005],[18.591159,54.430185496000036],[18.950029500000028,54.35831050000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL634","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Gdanski","NUTS_NAME":"Gdanski"},"geometry":{"type":"Polygon","coordinates":[[[18.54172650000004,54.58447999900005],[18.358159,54.493491498000026],[18.45794150000006,54.43603999700008],[18.429497,54.38504749800006],[18.456920500000024,54.33227399800006],[18.602853500000037,54.28548949900005],[18.950029500000028,54.35831050000007],[19.33136950000005,54.366705498000044],[19.639032,54.45829449800004],[19.648523,54.45333899900004],[19.224721500000044,54.335841998000035],[19.256953,54.27846049900006],[19.326466500000038,54.203823499000066],[19.217688500000065,54.11612149600006],[19.04808,54.17181349900005],[18.580178500000045,54.103316],[18.327238500000078,54.14108549600007],[17.800194,54.17817249700005],[17.69747250000006,54.23669199700004],[17.709683,54.31247199900008],[17.886816,54.47931349900006],[17.781354,54.59690249700003],[17.827587,54.665852],[17.666607,54.78323149900007],[18.35961,54.81719149600008],[18.82896850000003,54.60770799800008],[18.423325500000033,54.78867299800004],[18.39677450000005,54.747280499000055],[18.54172650000004,54.58447999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL636","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Slupski","NUTS_NAME":"Slupski"},"geometry":{"type":"Polygon","coordinates":[[[17.800194,54.17817249700005],[17.491311,53.922256497000035],[17.364152,53.940090498000075],[17.29575150000005,53.87112649900007],[17.231730499000037,53.981730999000035],[17.107128500000044,54.008212498000034],[16.982053499000074,53.90490949900004],[16.792764500000033,53.985550499000055],[16.804609500000026,54.11607299800005],[16.711381,54.21400299900006],[16.865116499000067,54.260984998000026],[16.813893500000063,54.318491],[16.85862050000003,54.38257649700006],[16.699085,54.569247],[17.666607,54.78323149900007],[17.827587,54.665852],[17.781354,54.59690249700003],[17.886816,54.47931349900006],[17.709683,54.31247199900008],[17.69747250000006,54.23669199700004],[17.800194,54.17817249700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL637","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Chojnicki","NUTS_NAME":"Chojnicki"},"geometry":{"type":"Polygon","coordinates":[[[18.327238500000078,54.14108549600007],[18.358194,54.06097650000004],[18.278068500000074,53.99663049800006],[18.349536,53.97745899900008],[18.135067500000048,53.947448500000064],[18.070848,53.87387399800008],[18.072036500000024,53.781074997000076],[18.031934500000034,53.724946499000055],[17.860104,53.754955],[17.889393,53.72768099900003],[17.85171299900003,53.687752497000076],[17.740691,53.67797549800008],[17.730447500000025,53.59290499700006],[17.448825,53.60201999800006],[17.390653,53.49096399900003],[16.999648999000044,53.51806250000004],[16.892248,53.655868998000074],[16.857653500000026,53.74709499800008],[16.945877499000062,53.802610997000045],[16.872407,53.86866299900004],[16.982053499000074,53.90490949900004],[17.107128500000044,54.008212498000034],[17.231730499000037,53.981730999000035],[17.29575150000005,53.87112649900007],[17.364152,53.940090498000075],[17.491311,53.922256497000035],[17.800194,54.17817249700005],[18.327238500000078,54.14108549600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL638","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Starogardzki","NUTS_NAME":"Starogardzki"},"geometry":{"type":"Polygon","coordinates":[[[19.217688500000065,54.11612149600006],[19.377739500000075,53.984958997000035],[19.368867,53.93803149900003],[19.581343,53.92706299900004],[19.456246,53.79695049800006],[19.322978,53.81394449900006],[19.12927650000006,53.58826049800007],[18.761577500000044,53.604881997000064],[18.739787,53.64637599900004],[18.78897150000006,53.70210499900003],[18.550634500000058,53.655921498000055],[18.072036500000024,53.781074997000076],[18.070848,53.87387399800008],[18.135067500000048,53.947448500000064],[18.349536,53.97745899900008],[18.278068500000074,53.99663049800006],[18.358194,54.06097650000004],[18.327238500000078,54.14108549600007],[18.580178500000045,54.103316],[19.04808,54.17181349900005],[19.217688500000065,54.11612149600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT","LEVL_CODE":0,"CNTR_CODE":"PT","NAME_LATN":"PORTUGAL","NUTS_NAME":"PORTUGAL"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-8.216984,41.91365499600005],[-8.165075499999944,41.81830199900003],[-8.05186249999997,41.820613997000066],[-7.905759499999931,41.92737949700006],[-7.876458499999956,41.84949499600003],[-7.697163499999931,41.907523998000045],[-7.588074,41.87964999600007],[-7.613568,41.82929599800008],[-7.452781,41.865161998000076],[-7.427273499999956,41.80749949800003],[-7.200464499999953,41.879749497000034],[-7.17188,41.97989049600005],[-6.983513499999958,41.97290399800005],[-6.588461499999937,41.967761997000025],[-6.549125499999946,41.944511496000075],[-6.571177499999976,41.88363249900004],[-6.51576149999994,41.870739],[-6.568986499999937,41.743514997000034],[-6.548242499999958,41.68563849800006],[-6.302936,41.66290249800005],[-6.189352,41.57504649800006],[-6.318309499999941,41.38614249600005],[-6.391473,41.38374699800005],[-6.479713,41.29437999800007],[-6.689786,41.20524149700003],[-6.808375,41.036526],[-6.929903499999966,41.02946649800003],[-6.801935,40.861045997000076],[-6.831229499999949,40.75269699800003],[-6.793930499999931,40.66174349600004],[-6.844862,40.56593699700005],[-6.795071,40.51197799800008],[-6.848108,40.45544049700004],[-6.781047499999943,40.36473099700004],[-6.865144,40.27069449800007],[-6.95129849999995,40.25744599800004],[-7.012223,40.22654349700008],[-7.01196049999993,40.12693399900007],[-6.864203,40.011867497000026],[-6.903161499999953,39.87268849800006],[-6.986185499999976,39.81074499700003],[-7.015405,39.670856498000035],[-7.534902449999947,39.66198689000004],[-7.294676,39.455993499000044],[-7.310608,39.34102249800003],[-7.231467,39.27843099900008],[-7.24126,39.20853049800007],[-7.136354499999925,39.17224899800004],[-7.143882,39.10932149800004],[-7.02995,39.11653149600005],[-6.951391499999943,39.02407049800007],[-7.052114,38.90723399700005],[-7.039417499999956,38.86484549700003],[-7.203135,38.75101749700008],[-7.255615,38.72397599800007],[-7.316636,38.43987649800005],[-7.107952499999953,38.18812149900003],[-6.931738499999938,38.20837799700007],[-7.002483499999926,38.02271649900007],[-7.108363,38.040984998000056],[-7.263284499999941,37.979907999000034],[-7.279641,37.876762496000026],[-7.418441,37.74499499800004],[-7.51269149999996,37.526256498000066],[-7.463588499999958,37.485454498000024],[-7.40191649999997,37.174827497000024],[-7.569612,37.14237499800004],[-7.882587499999943,36.96182299800006],[-8.182106499999975,37.09027499800004],[-8.592472,37.12205449900006],[-8.981500499999981,37.02743849700005],[-8.796315499999935,37.44294799900007],[-8.79764849999998,37.89460749700004],[-8.887846499999966,37.95198499700007],[-8.796723,38.12135599800007],[-8.777948,38.27864449800006],[-8.820624,38.41169899700003],[-8.91185,38.49040399800003],[-8.693546,38.41568299900007],[-8.73506649999996,38.516034998000066],[-8.747424499999966,38.55086049800008],[-8.79943011399996,38.53490125900004],[-8.755707499999971,38.498098998000046],[-8.783712499999979,38.48785049700007],[-8.890975000999958,38.52048549800003],[-9.222857499999975,38.413747998000076],[-9.182559,38.524313498000026],[-9.260362499999928,38.662664998000025],[-9.146172499999977,38.68822249800007],[-9.076919000999965,38.64121249800007],[-8.957384,38.69262549700005],[-9.044651,38.716027498000074],[-8.924883499999964,38.758676499000046],[-8.968469,38.82776249800003],[-9.03491,38.860948],[-9.13207449999993,38.706739998000046],[-9.47746849999993,38.70183849700004],[-9.50053,38.780704996000054],[-9.41547449999996,38.94505349700006],[-9.416459499999974,39.054692998000064],[-9.340562499999976,39.22482049800004],[-9.36595050099993,39.348377998000046],[-9.074842,39.59182399700006],[-9.040266000999964,39.741422],[-8.894938,40.045502998000075],[-8.863048499999934,40.14007599800004],[-8.909242500999937,40.186537498000064],[-8.784033,40.520366498000044],[-8.653119499999946,40.964778496000065],[-8.776427499999954,41.471979496000074],[-8.811573,41.61157699600005],[-8.880883000999972,41.75180899700007],[-8.863186,41.87206649800004],[-8.630127,42.050002997000036],[-8.331857499999956,42.084220997000045],[-8.199000499999954,42.15441899800004],[-8.181348499999956,42.06683349700006],[-8.083840000999942,42.03905099800005],[-8.216984,41.91365499600005]]],[[[-17.193386499999974,32.87055649800004],[-17.102119499999958,32.81972899900006],[-16.903707,32.83755799800008],[-16.746594499999958,32.73416999900007],[-16.831212499999936,32.64036199800006],[-16.944882500999938,32.63293549800005],[-17.212603499999943,32.73696999800006],[-17.265786,32.81489249700007],[-17.193386499999974,32.87055649800004]]],[[[-25.765763,37.906070998000075],[-25.689569,37.84193950000008],[-25.193974,37.85964199800003],[-25.134163499999943,37.807617497000024],[-25.205413,37.735234999000056],[-25.511525499999948,37.708371498000076],[-25.68617949999998,37.73279899800008],[-25.85274,37.85310549800005],[-25.765763,37.906070998000075]]],[[[-27.042569499999956,38.696639997000034],[-27.079417,38.64398649800006],[-27.276853,38.65224649700008],[-27.379397499999982,38.74690699800004],[-27.256799,38.80333449700004],[-27.105539,38.78568549800008],[-27.042569499999956,38.696639997000034]]],[[[-28.31683449999997,38.756650498000056],[-27.77179,38.53695349800006],[-28.19083349999994,38.66695149800006],[-28.31683449999997,38.756650498000056]]],[[[-28.029527499999972,38.41165249900007],[-28.178494499999942,38.40929349800007],[-28.445251,38.42045999700008],[-28.539842499999963,38.48665149800007],[-28.519377,38.54547049800004],[-28.389973,38.55832099700007],[-28.029527499999972,38.41165249900007]]],[[[-28.62946949999997,38.524696997000035],[-28.719994,38.517318997000075],[-28.834771,38.599279498000044],[-28.714248,38.643756997000025],[-28.620574,38.61104949800006],[-28.62946949999997,38.524696997000035]]],[[[-31.136211499999945,39.46318949800008],[-31.170449,39.377477998000074],[-31.25928049999993,39.399148997000054],[-31.23694249999994,39.52081050000004],[-31.136211499999945,39.46318949800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT1","LEVL_CODE":1,"CNTR_CODE":"PT","NAME_LATN":"CONTINENTE","NUTS_NAME":"CONTINENTE"},"geometry":{"type":"Polygon","coordinates":[[[-8.216984,41.91365499600005],[-8.165075499999944,41.81830199900003],[-8.05186249999997,41.820613997000066],[-7.905759499999931,41.92737949700006],[-7.876458499999956,41.84949499600003],[-7.697163499999931,41.907523998000045],[-7.588074,41.87964999600007],[-7.613568,41.82929599800008],[-7.452781,41.865161998000076],[-7.427273499999956,41.80749949800003],[-7.200464499999953,41.879749497000034],[-7.17188,41.97989049600005],[-6.983513499999958,41.97290399800005],[-6.588461499999937,41.967761997000025],[-6.549125499999946,41.944511496000075],[-6.571177499999976,41.88363249900004],[-6.51576149999994,41.870739],[-6.568986499999937,41.743514997000034],[-6.548242499999958,41.68563849800006],[-6.302936,41.66290249800005],[-6.189352,41.57504649800006],[-6.318309499999941,41.38614249600005],[-6.391473,41.38374699800005],[-6.479713,41.29437999800007],[-6.689786,41.20524149700003],[-6.808375,41.036526],[-6.929903499999966,41.02946649800003],[-6.801935,40.861045997000076],[-6.831229499999949,40.75269699800003],[-6.793930499999931,40.66174349600004],[-6.844862,40.56593699700005],[-6.795071,40.51197799800008],[-6.848108,40.45544049700004],[-6.781047499999943,40.36473099700004],[-6.865144,40.27069449800007],[-6.95129849999995,40.25744599800004],[-7.012223,40.22654349700008],[-7.01196049999993,40.12693399900007],[-6.864203,40.011867497000026],[-6.903161499999953,39.87268849800006],[-6.986185499999976,39.81074499700003],[-7.015405,39.670856498000035],[-7.534902449999947,39.66198689000004],[-7.294676,39.455993499000044],[-7.310608,39.34102249800003],[-7.231467,39.27843099900008],[-7.24126,39.20853049800007],[-7.136354499999925,39.17224899800004],[-7.143882,39.10932149800004],[-7.02995,39.11653149600005],[-6.951391499999943,39.02407049800007],[-7.052114,38.90723399700005],[-7.039417499999956,38.86484549700003],[-7.203135,38.75101749700008],[-7.255615,38.72397599800007],[-7.316636,38.43987649800005],[-7.107952499999953,38.18812149900003],[-6.931738499999938,38.20837799700007],[-7.002483499999926,38.02271649900007],[-7.108363,38.040984998000056],[-7.263284499999941,37.979907999000034],[-7.279641,37.876762496000026],[-7.418441,37.74499499800004],[-7.51269149999996,37.526256498000066],[-7.463588499999958,37.485454498000024],[-7.40191649999997,37.174827497000024],[-7.569612,37.14237499800004],[-7.882587499999943,36.96182299800006],[-8.182106499999975,37.09027499800004],[-8.592472,37.12205449900006],[-8.981500499999981,37.02743849700005],[-8.796315499999935,37.44294799900007],[-8.79764849999998,37.89460749700004],[-8.887846499999966,37.95198499700007],[-8.796723,38.12135599800007],[-8.777948,38.27864449800006],[-8.820624,38.41169899700003],[-8.91185,38.49040399800003],[-8.693546,38.41568299900007],[-8.73506649999996,38.516034998000066],[-8.747424499999966,38.55086049800008],[-8.79943011399996,38.53490125900004],[-8.755707499999971,38.498098998000046],[-8.783712499999979,38.48785049700007],[-8.890975000999958,38.52048549800003],[-9.222857499999975,38.413747998000076],[-9.182559,38.524313498000026],[-9.260362499999928,38.662664998000025],[-9.146172499999977,38.68822249800007],[-9.076919000999965,38.64121249800007],[-8.957384,38.69262549700005],[-9.044651,38.716027498000074],[-8.924883499999964,38.758676499000046],[-8.968469,38.82776249800003],[-9.03491,38.860948],[-9.13207449999993,38.706739998000046],[-9.47746849999993,38.70183849700004],[-9.50053,38.780704996000054],[-9.41547449999996,38.94505349700006],[-9.416459499999974,39.054692998000064],[-9.340562499999976,39.22482049800004],[-9.36595050099993,39.348377998000046],[-9.074842,39.59182399700006],[-9.040266000999964,39.741422],[-8.894938,40.045502998000075],[-8.863048499999934,40.14007599800004],[-8.909242500999937,40.186537498000064],[-8.784033,40.520366498000044],[-8.653119499999946,40.964778496000065],[-8.776427499999954,41.471979496000074],[-8.811573,41.61157699600005],[-8.880883000999972,41.75180899700007],[-8.863186,41.87206649800004],[-8.630127,42.050002997000036],[-8.331857499999956,42.084220997000045],[-8.199000499999954,42.15441899800004],[-8.181348499999956,42.06683349700006],[-8.083840000999942,42.03905099800005],[-8.216984,41.91365499600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Norte","NUTS_NAME":"Norte"},"geometry":{"type":"Polygon","coordinates":[[[-8.216984,41.91365499600005],[-8.165075499999944,41.81830199900003],[-8.05186249999997,41.820613997000066],[-7.905759499999931,41.92737949700006],[-7.876458499999956,41.84949499600003],[-7.697163499999931,41.907523998000045],[-7.588074,41.87964999600007],[-7.613568,41.82929599800008],[-7.452781,41.865161998000076],[-7.427273499999956,41.80749949800003],[-7.200464499999953,41.879749497000034],[-7.17188,41.97989049600005],[-6.983513499999958,41.97290399800005],[-6.588461499999937,41.967761997000025],[-6.549125499999946,41.944511496000075],[-6.571177499999976,41.88363249900004],[-6.51576149999994,41.870739],[-6.568986499999937,41.743514997000034],[-6.548242499999958,41.68563849800006],[-6.302936,41.66290249800005],[-6.189352,41.57504649800006],[-6.318309499999941,41.38614249600005],[-6.391473,41.38374699800005],[-6.479713,41.29437999800007],[-6.689786,41.20524149700003],[-6.808375,41.036526],[-6.929903499999966,41.02946649800003],[-7.135693499999945,40.92315749900007],[-7.185556,40.93581949700007],[-7.178086,41.03022249700007],[-7.379045,41.013359998000055],[-7.32527,40.971959998000045],[-7.380430823999973,40.90520315300006],[-7.429548499999953,40.90528549800007],[-7.401235449999945,40.88094133000004],[-7.454203499999949,40.81260999800003],[-7.529297499999927,40.87304549700008],[-7.687923,40.85438499700007],[-7.69249149999996,40.94231549800003],[-7.915638499999943,41.020001997000065],[-7.970363499999962,40.97217149900007],[-8.089171499999964,40.98755749700007],[-8.13151849999997,40.950611500000036],[-8.106421499999954,40.85516699800007],[-8.206337499999961,40.848535998000045],[-8.273369,40.763173496000036],[-8.549126499999943,40.79180699600005],[-8.600873499999977,40.96358249800005],[-8.653119499999946,40.964778496000065],[-8.776427499999954,41.471979496000074],[-8.811573,41.61157699600005],[-8.880883000999972,41.75180899700007],[-8.863186,41.87206649800004],[-8.630127,42.050002997000036],[-8.331857499999956,42.084220997000045],[-8.199000499999954,42.15441899800004],[-8.181348499999956,42.06683349700006],[-8.083840000999942,42.03905099800005],[-8.216984,41.91365499600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT111","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Alto Minho","NUTS_NAME":"Alto Minho"},"geometry":{"type":"Polygon","coordinates":[[[-8.165075499999944,41.81830199900003],[-8.514550499999928,41.71592099700007],[-8.5628,41.62367949800006],[-8.811573,41.61157699600005],[-8.880883000999972,41.75180899700007],[-8.863186,41.87206649800004],[-8.630127,42.050002997000036],[-8.331857499999956,42.084220997000045],[-8.199000499999954,42.15441899800004],[-8.181348499999956,42.06683349700006],[-8.083840000999942,42.03905099800005],[-8.216984,41.91365499600005],[-8.165075499999944,41.81830199900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT112","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Cávado","NUTS_NAME":"Cávado"},"geometry":{"type":"Polygon","coordinates":[[[-8.119459,41.68924499700006],[-8.312807,41.645294997000065],[-8.344194499999958,41.59105599700007],[-8.305511,41.54323049900006],[-8.622716499999967,41.41488099800006],[-8.776427499999954,41.471979496000074],[-8.811573,41.61157699600005],[-8.5628,41.62367949800006],[-8.514550499999928,41.71592099700007],[-8.165075499999944,41.81830199900003],[-8.05186249999997,41.820613997000066],[-8.119459,41.68924499700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT119","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Ave","NUTS_NAME":"Ave"},"geometry":{"type":"Polygon","coordinates":[[[-7.788682500999926,41.41141399900005],[-7.907637499999964,41.302670498000055],[-7.980876,41.395185997000056],[-7.938686,41.44705049700008],[-8.063192499999957,41.48584949800005],[-8.116946,41.37891649900007],[-8.213451,41.41184149600008],[-8.316447499999981,41.35126649800003],[-8.603665000999968,41.338585497000054],[-8.622716499999967,41.41488099800006],[-8.305511,41.54323049900006],[-8.344194499999958,41.59105599700007],[-8.312807,41.645294997000065],[-8.119459,41.68924499700006],[-8.023403,41.69686349800003],[-7.99669549999993,41.60314999700006],[-7.824516499999959,41.595991499000036],[-7.909410499999979,41.46588399800004],[-7.788682500999926,41.41141399900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11A","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Área Metropolitana do Porto","NUTS_NAME":"Área Metropolitana do Porto"},"geometry":{"type":"Polygon","coordinates":[[[-8.622716499999967,41.41488099800006],[-8.603665000999968,41.338585497000054],[-8.316447499999981,41.35126649800003],[-8.450333,41.26203299900004],[-8.296194,41.23081549900007],[-8.392568499999982,41.12675599900007],[-8.384186,41.003656998000054],[-8.309572000999935,40.96226299800003],[-8.240793500999928,40.963629998000044],[-8.223744,41.01606049800006],[-8.089171499999964,40.98755749700007],[-8.13151849999997,40.950611500000036],[-8.106421499999954,40.85516699800007],[-8.206337499999961,40.848535998000045],[-8.273369,40.763173496000036],[-8.549126499999943,40.79180699600005],[-8.600873499999977,40.96358249800005],[-8.653119499999946,40.964778496000065],[-8.776427499999954,41.471979496000074],[-8.622716499999967,41.41488099800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11B","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Alto Tâmega","NUTS_NAME":"Alto Tâmega"},"geometry":{"type":"Polygon","coordinates":[[[-7.200464499999953,41.879749497000034],[-7.172026499999959,41.74498950000003],[-7.258002,41.59249699800006],[-7.229112499999928,41.52279249700007],[-7.365316499999949,41.44042449800003],[-7.42092,41.52786699700005],[-7.61071900099995,41.39109199600006],[-7.788682500999926,41.41141399900005],[-7.909410499999979,41.46588399800004],[-7.824516499999959,41.595991499000036],[-7.99669549999993,41.60314999700006],[-8.023403,41.69686349800003],[-8.119459,41.68924499700006],[-8.05186249999997,41.820613997000066],[-7.905759499999931,41.92737949700006],[-7.876458499999956,41.84949499600003],[-7.697163499999931,41.907523998000045],[-7.588074,41.87964999600007],[-7.613568,41.82929599800008],[-7.452781,41.865161998000076],[-7.427273499999956,41.80749949800003],[-7.200464499999953,41.879749497000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11C","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Tâmega e Sousa","NUTS_NAME":"Tâmega e Sousa"},"geometry":{"type":"Polygon","coordinates":[[[-7.907637499999964,41.302670498000055],[-7.875595500999964,41.24202249700005],[-7.89738449999993,41.13188199600006],[-7.850698,41.11111299700008],[-7.915638499999943,41.020001997000065],[-7.970363499999962,40.97217149900007],[-8.089171499999964,40.98755749700007],[-8.223744,41.01606049800006],[-8.240793500999928,40.963629998000044],[-8.309572000999935,40.96226299800003],[-8.384186,41.003656998000054],[-8.392568499999982,41.12675599900007],[-8.296194,41.23081549900007],[-8.450333,41.26203299900004],[-8.316447499999981,41.35126649800003],[-8.213451,41.41184149600008],[-8.116946,41.37891649900007],[-8.063192499999957,41.48584949800005],[-7.938686,41.44705049700008],[-7.980876,41.395185997000056],[-7.907637499999964,41.302670498000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11D","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Douro","NUTS_NAME":"Douro"},"geometry":{"type":"Polygon","coordinates":[[[-7.365316499999949,41.44042449800003],[-7.362331499999925,41.366682998000044],[-7.270540500999971,41.332797998000046],[-7.230929,41.230141497000034],[-7.097192,41.23436899700005],[-7.069013499999926,41.309141999000076],[-6.811689499999943,41.193577498000025],[-6.736976,41.24860149800003],[-6.689786,41.20524149700003],[-6.808375,41.036526],[-6.929903499999966,41.02946649800003],[-7.135693499999945,40.92315749900007],[-7.185556,40.93581949700007],[-7.178086,41.03022249700007],[-7.379045,41.013359998000055],[-7.32527,40.971959998000045],[-7.380430823999973,40.90520315300006],[-7.429548499999953,40.90528549800007],[-7.401235449999945,40.88094133000004],[-7.454203499999949,40.81260999800003],[-7.529297499999927,40.87304549700008],[-7.687923,40.85438499700007],[-7.69249149999996,40.94231549800003],[-7.915638499999943,41.020001997000065],[-7.850698,41.11111299700008],[-7.89738449999993,41.13188199600006],[-7.875595500999964,41.24202249700005],[-7.907637499999964,41.302670498000055],[-7.788682500999926,41.41141399900005],[-7.61071900099995,41.39109199600006],[-7.42092,41.52786699700005],[-7.365316499999949,41.44042449800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT11E","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Terras de Trás-os-Montes","NUTS_NAME":"Terras de Trás-os-Montes"},"geometry":{"type":"Polygon","coordinates":[[[-6.479713,41.29437999800007],[-6.689786,41.20524149700003],[-6.736976,41.24860149800003],[-6.811689499999943,41.193577498000025],[-7.069013499999926,41.309141999000076],[-7.097192,41.23436899700005],[-7.230929,41.230141497000034],[-7.270540500999971,41.332797998000046],[-7.362331499999925,41.366682998000044],[-7.365316499999949,41.44042449800003],[-7.229112499999928,41.52279249700007],[-7.258002,41.59249699800006],[-7.172026499999959,41.74498950000003],[-7.200464499999953,41.879749497000034],[-7.17188,41.97989049600005],[-6.983513499999958,41.97290399800005],[-6.588461499999937,41.967761997000025],[-6.549125499999946,41.944511496000075],[-6.571177499999976,41.88363249900004],[-6.51576149999994,41.870739],[-6.568986499999937,41.743514997000034],[-6.548242499999958,41.68563849800006],[-6.302936,41.66290249800005],[-6.189352,41.57504649800006],[-6.318309499999941,41.38614249600005],[-6.391473,41.38374699800005],[-6.479713,41.29437999800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT15","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Algarve","NUTS_NAME":"Algarve"},"geometry":{"type":"Polygon","coordinates":[[[-7.40191649999997,37.174827497000024],[-7.569612,37.14237499800004],[-7.882587499999943,36.96182299800006],[-8.182106499999975,37.09027499800004],[-8.592472,37.12205449900006],[-8.981500499999981,37.02743849700005],[-8.796315499999935,37.44294799900007],[-8.733497499999942,37.39518549700006],[-8.454535,37.37370599700006],[-8.377928499999939,37.42724599600007],[-8.284828499999946,37.43091149800006],[-8.187234499999931,37.341102996000075],[-8.065689499999962,37.31896999700007],[-8.000372499999969,37.39640049800005],[-7.51269149999996,37.526256498000066],[-7.463588499999958,37.485454498000024],[-7.40191649999997,37.174827497000024]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT150","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Algarve","NUTS_NAME":"Algarve"},"geometry":{"type":"Polygon","coordinates":[[[-7.40191649999997,37.174827497000024],[-7.569612,37.14237499800004],[-7.882587499999943,36.96182299800006],[-8.182106499999975,37.09027499800004],[-8.592472,37.12205449900006],[-8.981500499999981,37.02743849700005],[-8.796315499999935,37.44294799900007],[-8.733497499999942,37.39518549700006],[-8.454535,37.37370599700006],[-8.377928499999939,37.42724599600007],[-8.284828499999946,37.43091149800006],[-8.187234499999931,37.341102996000075],[-8.065689499999962,37.31896999700007],[-8.000372499999969,37.39640049800005],[-7.51269149999996,37.526256498000066],[-7.463588499999958,37.485454498000024],[-7.40191649999997,37.174827497000024]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Centro (PT)","NUTS_NAME":"Centro (PT)"},"geometry":{"type":"Polygon","coordinates":[[[-7.185556,40.93581949700007],[-7.135693499999945,40.92315749900007],[-6.929903499999966,41.02946649800003],[-6.801935,40.861045997000076],[-6.831229499999949,40.75269699800003],[-6.793930499999931,40.66174349600004],[-6.844862,40.56593699700005],[-6.795071,40.51197799800008],[-6.848108,40.45544049700004],[-6.781047499999943,40.36473099700004],[-6.865144,40.27069449800007],[-6.95129849999995,40.25744599800004],[-7.012223,40.22654349700008],[-7.01196049999993,40.12693399900007],[-6.864203,40.011867497000026],[-6.903161499999953,39.87268849800006],[-6.986185499999976,39.81074499700003],[-7.015405,39.670856498000035],[-7.534902449999947,39.66198689000004],[-7.70324,39.63661549700004],[-7.825108499999942,39.537363998000046],[-7.959676,39.56094949800007],[-8.008888499999955,39.473171500000035],[-7.939462499999934,39.40995249700006],[-8.17260349999998,39.23331399600005],[-8.261504,39.29215549700007],[-8.338524,39.46152449700003],[-8.537092,39.39722099800008],[-8.759282000999974,39.40008099800008],[-8.759391,39.47568699800007],[-8.903426,39.46600549800007],[-9.001579,39.35463499800005],[-8.975229,39.28713449700007],[-9.004715499999975,39.18638249800006],[-8.918676,39.10605499700006],[-8.93028049999998,39.01802249800005],[-9.025205,39.01710799800003],[-9.025622,38.960199497000076],[-9.105246,38.92204099700007],[-9.240566,38.94842249800007],[-9.205836499999975,38.969239498000036],[-9.220932499999947,39.01962099900004],[-9.333296,39.02805749600003],[-9.416459499999974,39.054692998000064],[-9.340562499999976,39.22482049800004],[-9.36595050099993,39.348377998000046],[-9.074842,39.59182399700006],[-9.040266000999964,39.741422],[-8.894938,40.045502998000075],[-8.863048499999934,40.14007599800004],[-8.909242500999937,40.186537498000064],[-8.784033,40.520366498000044],[-8.653119499999946,40.964778496000065],[-8.600873499999977,40.96358249800005],[-8.549126499999943,40.79180699600005],[-8.273369,40.763173496000036],[-8.206337499999961,40.848535998000045],[-8.106421499999954,40.85516699800007],[-8.13151849999997,40.950611500000036],[-8.089171499999964,40.98755749700007],[-7.970363499999962,40.97217149900007],[-7.915638499999943,41.020001997000065],[-7.69249149999996,40.94231549800003],[-7.687923,40.85438499700007],[-7.529297499999927,40.87304549700008],[-7.454203499999949,40.81260999800003],[-7.401235449999945,40.88094133000004],[-7.429548499999953,40.90528549800007],[-7.380430823999973,40.90520315300006],[-7.32527,40.971959998000045],[-7.379045,41.013359998000055],[-7.178086,41.03022249700007],[-7.185556,40.93581949700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16B","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Oeste","NUTS_NAME":"Oeste"},"geometry":{"type":"Polygon","coordinates":[[[-8.903426,39.46600549800007],[-9.001579,39.35463499800005],[-8.975229,39.28713449700007],[-9.004715499999975,39.18638249800006],[-8.918676,39.10605499700006],[-8.93028049999998,39.01802249800005],[-9.025205,39.01710799800003],[-9.025622,38.960199497000076],[-9.105246,38.92204099700007],[-9.240566,38.94842249800007],[-9.205836499999975,38.969239498000036],[-9.220932499999947,39.01962099900004],[-9.333296,39.02805749600003],[-9.416459499999974,39.054692998000064],[-9.340562499999976,39.22482049800004],[-9.36595050099993,39.348377998000046],[-9.074842,39.59182399700006],[-9.040266000999964,39.741422],[-8.914885,39.667515],[-8.903426,39.46600549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16D","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Região de Aveiro","NUTS_NAME":"Região de Aveiro"},"geometry":{"type":"Polygon","coordinates":[[[-8.273369,40.763173496000036],[-8.33358949899997,40.73700299800004],[-8.250663499999973,40.51489249800005],[-8.333551499999942,40.487142497000036],[-8.35415,40.39560849800006],[-8.572446,40.40029149700007],[-8.61459,40.485441],[-8.635692499999948,40.42563849800007],[-8.784033,40.520366498000044],[-8.653119499999946,40.964778496000065],[-8.600873499999977,40.96358249800005],[-8.549126499999943,40.79180699600005],[-8.273369,40.763173496000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16E","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Região de Coimbra","NUTS_NAME":"Região de Coimbra"},"geometry":{"type":"Polygon","coordinates":[[[-8.250663499999973,40.51489249800005],[-8.15786849999995,40.44924049800005],[-8.17718149999996,40.33677549600003],[-7.828317,40.498545498000055],[-7.77901049999997,40.31844099800003],[-7.809048,40.181775498000036],[-7.73412849999994,40.108243497000046],[-7.808315499999935,40.035735498000065],[-7.859842000999947,40.05904849700005],[-8.086902499999951,39.94323049800005],[-8.118683499999975,39.93408949800005],[-8.179102,40.08973549900003],[-8.388375,39.92453249700003],[-8.426018499999941,40.02655699700006],[-8.538816499999939,39.94884449700004],[-8.56262,40.028174997000065],[-8.64335749999998,39.99067599800003],[-8.894938,40.045502998000075],[-8.863048499999934,40.14007599800004],[-8.909242500999937,40.186537498000064],[-8.784033,40.520366498000044],[-8.635692499999948,40.42563849800007],[-8.61459,40.485441],[-8.572446,40.40029149700007],[-8.35415,40.39560849800006],[-8.333551499999942,40.487142497000036],[-8.250663499999973,40.51489249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16F","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Região de Leiria","NUTS_NAME":"Região de Leiria"},"geometry":{"type":"Polygon","coordinates":[[[-8.118683499999975,39.93408949800005],[-8.452754,39.73088949700008],[-8.47797049999997,39.838696997000056],[-8.675488499999972,39.77436299900006],[-8.673758499999963,39.709810496000046],[-8.632080499999972,39.695528496000065],[-8.702189499999974,39.63741749800005],[-8.671424499999944,39.56218199800003],[-8.759391,39.47568699800007],[-8.903426,39.46600549800007],[-8.914885,39.667515],[-9.040266000999964,39.741422],[-8.894938,40.045502998000075],[-8.64335749999998,39.99067599800003],[-8.56262,40.028174997000065],[-8.538816499999939,39.94884449700004],[-8.426018499999941,40.02655699700006],[-8.388375,39.92453249700003],[-8.179102,40.08973549900003],[-8.118683499999975,39.93408949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16G","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Viseu Dão Lafões","NUTS_NAME":"Viseu Dão Lafões"},"geometry":{"type":"Polygon","coordinates":[[[-7.454203499999949,40.81260999800003],[-7.424471499999981,40.758091498000056],[-7.551368499999967,40.67709849700003],[-7.583545499999957,40.571729497000035],[-7.828317,40.498545498000055],[-8.17718149999996,40.33677549600003],[-8.15786849999995,40.44924049800005],[-8.250663499999973,40.51489249800005],[-8.33358949899997,40.73700299800004],[-8.273369,40.763173496000036],[-8.206337499999961,40.848535998000045],[-8.106421499999954,40.85516699800007],[-8.13151849999997,40.950611500000036],[-8.089171499999964,40.98755749700007],[-7.970363499999962,40.97217149900007],[-7.915638499999943,41.020001997000065],[-7.69249149999996,40.94231549800003],[-7.687923,40.85438499700007],[-7.529297499999927,40.87304549700008],[-7.454203499999949,40.81260999800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16H","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Beira Baixa","NUTS_NAME":"Beira Baixa"},"geometry":{"type":"Polygon","coordinates":[[[-6.95129849999995,40.25744599800004],[-7.012223,40.22654349700008],[-7.01196049999993,40.12693399900007],[-6.864203,40.011867497000026],[-6.903161499999953,39.87268849800006],[-6.986185499999976,39.81074499700003],[-7.015405,39.670856498000035],[-7.534902449999947,39.66198689000004],[-7.70324,39.63661549700004],[-7.825108499999942,39.537363998000046],[-7.847883499999966,39.623035498000036],[-7.959101,39.66421749700004],[-7.960394,39.74258999800003],[-8.035756,39.746923999000046],[-7.916521499999931,39.85356949800007],[-8.086902499999951,39.94323049800005],[-7.859842000999947,40.05904849700005],[-7.808315499999935,40.035735498000065],[-7.52515649999998,40.080616],[-7.483266500999946,39.99678149700003],[-7.357263500999977,40.020200997000074],[-7.251906,40.201238498000066],[-7.273076,40.265704999000036],[-7.098035,40.315052997000066],[-6.95129849999995,40.25744599800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16I","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Médio Tejo","NUTS_NAME":"Médio Tejo"},"geometry":{"type":"Polygon","coordinates":[[[-7.825108499999942,39.537363998000046],[-7.959676,39.56094949800007],[-8.008888499999955,39.473171500000035],[-7.939462499999934,39.40995249700006],[-8.17260349999998,39.23331399600005],[-8.261504,39.29215549700007],[-8.338524,39.46152449700003],[-8.537092,39.39722099800008],[-8.759282000999974,39.40008099800008],[-8.759391,39.47568699800007],[-8.671424499999944,39.56218199800003],[-8.702189499999974,39.63741749800005],[-8.632080499999972,39.695528496000065],[-8.673758499999963,39.709810496000046],[-8.675488499999972,39.77436299900006],[-8.47797049999997,39.838696997000056],[-8.452754,39.73088949700008],[-8.118683499999975,39.93408949800005],[-8.086902499999951,39.94323049800005],[-7.916521499999931,39.85356949800007],[-8.035756,39.746923999000046],[-7.960394,39.74258999800003],[-7.959101,39.66421749700004],[-7.847883499999966,39.623035498000036],[-7.825108499999942,39.537363998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT16J","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Beiras e Serra da Estrela","NUTS_NAME":"Beiras e Serra da Estrela"},"geometry":{"type":"Polygon","coordinates":[[[-7.401235449999945,40.88094133000004],[-7.429548499999953,40.90528549800007],[-7.380430823999973,40.90520315300006],[-7.32527,40.971959998000045],[-7.379045,41.013359998000055],[-7.178086,41.03022249700007],[-7.185556,40.93581949700007],[-7.135693499999945,40.92315749900007],[-6.929903499999966,41.02946649800003],[-6.801935,40.861045997000076],[-6.831229499999949,40.75269699800003],[-6.793930499999931,40.66174349600004],[-6.844862,40.56593699700005],[-6.795071,40.51197799800008],[-6.848108,40.45544049700004],[-6.781047499999943,40.36473099700004],[-6.865144,40.27069449800007],[-6.95129849999995,40.25744599800004],[-7.098035,40.315052997000066],[-7.273076,40.265704999000036],[-7.251906,40.201238498000066],[-7.357263500999977,40.020200997000074],[-7.483266500999946,39.99678149700003],[-7.52515649999998,40.080616],[-7.808315499999935,40.035735498000065],[-7.73412849999994,40.108243497000046],[-7.809048,40.181775498000036],[-7.77901049999997,40.31844099800003],[-7.828317,40.498545498000055],[-7.583545499999957,40.571729497000035],[-7.551368499999967,40.67709849700003],[-7.424471499999981,40.758091498000056],[-7.454203499999949,40.81260999800003],[-7.401235449999945,40.88094133000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT17","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Área Metropolitana de Lisboa","NUTS_NAME":"Área Metropolitana de Lisboa"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-8.490975,38.76131049800006],[-8.653472,38.612269999000034],[-8.639115499999946,38.54948899800007],[-8.73506649999996,38.516034998000066],[-8.747424499999966,38.55086049800008],[-8.79943011399996,38.53490125900004],[-8.755707499999971,38.498098998000046],[-8.783712499999979,38.48785049700007],[-8.890975000999958,38.52048549800003],[-9.222857499999975,38.413747998000076],[-9.182559,38.524313498000026],[-9.260362499999928,38.662664998000025],[-9.146172499999977,38.68822249800007],[-9.076919000999965,38.64121249800007],[-8.957384,38.69262549700005],[-9.044651,38.716027498000074],[-8.924883499999964,38.758676499000046],[-8.779183499999931,38.73389199800005],[-8.708034,38.84542049700008],[-8.54680049999996,38.76343399700005],[-8.490975,38.76131049800006]]],[[[-9.240566,38.94842249800007],[-9.105246,38.92204099700007],[-9.025622,38.960199497000076],[-9.025205,39.01710799800003],[-8.93028049999998,39.01802249800005],[-8.84154449999994,39.01734449800006],[-8.968469,38.82776249800003],[-9.03491,38.860948],[-9.13207449999993,38.706739998000046],[-9.47746849999993,38.70183849700004],[-9.50053,38.780704996000054],[-9.41547449999996,38.94505349700006],[-9.416459499999974,39.054692998000064],[-9.333296,39.02805749600003],[-9.220932499999947,39.01962099900004],[-9.205836499999975,38.969239498000036],[-9.240566,38.94842249800007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT170","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Área Metropolitana de Lisboa","NUTS_NAME":"Área Metropolitana de Lisboa"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-8.490975,38.76131049800006],[-8.653472,38.612269999000034],[-8.639115499999946,38.54948899800007],[-8.73506649999996,38.516034998000066],[-8.747424499999966,38.55086049800008],[-8.79943011399996,38.53490125900004],[-8.755707499999971,38.498098998000046],[-8.783712499999979,38.48785049700007],[-8.890975000999958,38.52048549800003],[-9.222857499999975,38.413747998000076],[-9.182559,38.524313498000026],[-9.260362499999928,38.662664998000025],[-9.146172499999977,38.68822249800007],[-9.076919000999965,38.64121249800007],[-8.957384,38.69262549700005],[-9.044651,38.716027498000074],[-8.924883499999964,38.758676499000046],[-8.779183499999931,38.73389199800005],[-8.708034,38.84542049700008],[-8.54680049999996,38.76343399700005],[-8.490975,38.76131049800006]]],[[[-9.240566,38.94842249800007],[-9.105246,38.92204099700007],[-9.025622,38.960199497000076],[-9.025205,39.01710799800003],[-8.93028049999998,39.01802249800005],[-8.84154449999994,39.01734449800006],[-8.968469,38.82776249800003],[-9.03491,38.860948],[-9.13207449999993,38.706739998000046],[-9.47746849999993,38.70183849700004],[-9.50053,38.780704996000054],[-9.41547449999996,38.94505349700006],[-9.416459499999974,39.054692998000064],[-9.333296,39.02805749600003],[-9.220932499999947,39.01962099900004],[-9.205836499999975,38.969239498000036],[-9.240566,38.94842249800007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT18","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Alentejo","NUTS_NAME":"Alentejo"},"geometry":{"type":"Polygon","coordinates":[[[-7.534902449999947,39.66198689000004],[-7.294676,39.455993499000044],[-7.310608,39.34102249800003],[-7.231467,39.27843099900008],[-7.24126,39.20853049800007],[-7.136354499999925,39.17224899800004],[-7.143882,39.10932149800004],[-7.02995,39.11653149600005],[-6.951391499999943,39.02407049800007],[-7.052114,38.90723399700005],[-7.039417499999956,38.86484549700003],[-7.203135,38.75101749700008],[-7.255615,38.72397599800007],[-7.316636,38.43987649800005],[-7.107952499999953,38.18812149900003],[-6.931738499999938,38.20837799700007],[-7.002483499999926,38.02271649900007],[-7.108363,38.040984998000056],[-7.263284499999941,37.979907999000034],[-7.279641,37.876762496000026],[-7.418441,37.74499499800004],[-7.51269149999996,37.526256498000066],[-8.000372499999969,37.39640049800005],[-8.065689499999962,37.31896999700007],[-8.187234499999931,37.341102996000075],[-8.284828499999946,37.43091149800006],[-8.377928499999939,37.42724599600007],[-8.454535,37.37370599700006],[-8.733497499999942,37.39518549700006],[-8.796315499999935,37.44294799900007],[-8.79764849999998,37.89460749700004],[-8.887846499999966,37.95198499700007],[-8.796723,38.12135599800007],[-8.777948,38.27864449800006],[-8.820624,38.41169899700003],[-8.91185,38.49040399800003],[-8.693546,38.41568299900007],[-8.73506649999996,38.516034998000066],[-8.639115499999946,38.54948899800007],[-8.653472,38.612269999000034],[-8.490975,38.76131049800006],[-8.54680049999996,38.76343399700005],[-8.708034,38.84542049700008],[-8.779183499999931,38.73389199800005],[-8.924883499999964,38.758676499000046],[-8.968469,38.82776249800003],[-8.84154449999994,39.01734449800006],[-8.93028049999998,39.01802249800005],[-8.918676,39.10605499700006],[-9.004715499999975,39.18638249800006],[-8.975229,39.28713449700007],[-9.001579,39.35463499800005],[-8.903426,39.46600549800007],[-8.759391,39.47568699800007],[-8.759282000999974,39.40008099800008],[-8.537092,39.39722099800008],[-8.338524,39.46152449700003],[-8.261504,39.29215549700007],[-8.17260349999998,39.23331399600005],[-7.939462499999934,39.40995249700006],[-8.008888499999955,39.473171500000035],[-7.959676,39.56094949800007],[-7.825108499999942,39.537363998000046],[-7.70324,39.63661549700004],[-7.534902449999947,39.66198689000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT181","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Alentejo Litoral","NUTS_NAME":"Alentejo Litoral"},"geometry":{"type":"Polygon","coordinates":[[[-8.164370499999961,38.319843],[-8.129799,38.22395299800007],[-8.28432749999996,38.186577499000066],[-8.40956449999993,38.07299599700008],[-8.293608499999948,37.99332399800005],[-8.33074449999998,37.84701549700003],[-8.40091,37.86269549900004],[-8.37427649999995,37.76893449600004],[-8.466763,37.707438497000055],[-8.340664,37.66428499700004],[-8.314352499999927,37.56649999700005],[-8.385618499999964,37.51768649600007],[-8.377928499999939,37.42724599600007],[-8.454535,37.37370599700006],[-8.733497499999942,37.39518549700006],[-8.796315499999935,37.44294799900007],[-8.79764849999998,37.89460749700004],[-8.887846499999966,37.95198499700007],[-8.796723,38.12135599800007],[-8.777948,38.27864449800006],[-8.820624,38.41169899700003],[-8.91185,38.49040399800003],[-8.693546,38.41568299900007],[-8.73506649999996,38.516034998000066],[-8.639115499999946,38.54948899800007],[-8.369122499999946,38.54574249600006],[-8.285075499999948,38.424620497000035],[-8.322571,38.37218349700004],[-8.164370499999961,38.319843]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT184","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Baixo Alentejo","NUTS_NAME":"Baixo Alentejo"},"geometry":{"type":"Polygon","coordinates":[[[-7.107952499999953,38.18812149900003],[-6.931738499999938,38.20837799700007],[-7.002483499999926,38.02271649900007],[-7.108363,38.040984998000056],[-7.263284499999941,37.979907999000034],[-7.279641,37.876762496000026],[-7.418441,37.74499499800004],[-7.51269149999996,37.526256498000066],[-8.000372499999969,37.39640049800005],[-8.065689499999962,37.31896999700007],[-8.187234499999931,37.341102996000075],[-8.284828499999946,37.43091149800006],[-8.377928499999939,37.42724599600007],[-8.385618499999964,37.51768649600007],[-8.314352499999927,37.56649999700005],[-8.340664,37.66428499700004],[-8.466763,37.707438497000055],[-8.37427649999995,37.76893449600004],[-8.40091,37.86269549900004],[-8.33074449999998,37.84701549700003],[-8.293608499999948,37.99332399800005],[-8.40956449999993,38.07299599700008],[-8.28432749999996,38.186577499000066],[-8.129799,38.22395299800007],[-8.164370499999961,38.319843],[-7.497894499999973,38.18846599700004],[-7.397366,38.28957249800004],[-7.276417,38.32642999700005],[-7.159556,38.15989499800003],[-7.107952499999953,38.18812149900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT185","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Lezíria do Tejo","NUTS_NAME":"Lezíria do Tejo"},"geometry":{"type":"Polygon","coordinates":[[[-8.17260349999998,39.23331399600005],[-8.343478,39.109924998000054],[-8.22791849999993,38.98738549800004],[-8.272191499999963,38.90738999800004],[-8.158744,38.86639649700004],[-8.187920499999962,38.83812999700007],[-8.124314,38.80496750000003],[-8.141565,38.772977499000035],[-8.301621,38.86059299800007],[-8.54680049999996,38.76343399700005],[-8.708034,38.84542049700008],[-8.779183499999931,38.73389199800005],[-8.924883499999964,38.758676499000046],[-8.968469,38.82776249800003],[-8.84154449999994,39.01734449800006],[-8.93028049999998,39.01802249800005],[-8.918676,39.10605499700006],[-9.004715499999975,39.18638249800006],[-8.975229,39.28713449700007],[-9.001579,39.35463499800005],[-8.903426,39.46600549800007],[-8.759391,39.47568699800007],[-8.759282000999974,39.40008099800008],[-8.537092,39.39722099800008],[-8.338524,39.46152449700003],[-8.261504,39.29215549700007],[-8.17260349999998,39.23331399600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT186","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Alto Alentejo","NUTS_NAME":"Alto Alentejo"},"geometry":{"type":"Polygon","coordinates":[[[-7.534902449999947,39.66198689000004],[-7.294676,39.455993499000044],[-7.310608,39.34102249800003],[-7.231467,39.27843099900008],[-7.24126,39.20853049800007],[-7.136354499999925,39.17224899800004],[-7.143882,39.10932149800004],[-7.02995,39.11653149600005],[-6.951391499999943,39.02407049800007],[-7.052114,38.90723399700005],[-7.039417499999956,38.86484549700003],[-7.203135,38.75101749700008],[-7.244899,38.847780998000076],[-7.391644,38.80162949900006],[-7.452719499999944,38.963097498000025],[-7.533353499999976,39.008439999000075],[-7.564648500999965,38.95245349700008],[-7.789951499999972,38.89579299800005],[-7.924353499999938,38.97449549700008],[-8.02225,38.940972499000054],[-8.068534,39.01507299700006],[-8.10316450099998,38.96915699600004],[-8.145509,39.02329099700006],[-8.22791849999993,38.98738549800004],[-8.343478,39.109924998000054],[-8.17260349999998,39.23331399600005],[-7.939462499999934,39.40995249700006],[-8.008888499999955,39.473171500000035],[-7.959676,39.56094949800007],[-7.825108499999942,39.537363998000046],[-7.70324,39.63661549700004],[-7.534902449999947,39.66198689000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT187","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Alentejo Central","NUTS_NAME":"Alentejo Central"},"geometry":{"type":"Polygon","coordinates":[[[-7.203135,38.75101749700008],[-7.255615,38.72397599800007],[-7.316636,38.43987649800005],[-7.107952499999953,38.18812149900003],[-7.159556,38.15989499800003],[-7.276417,38.32642999700005],[-7.397366,38.28957249800004],[-7.497894499999973,38.18846599700004],[-8.164370499999961,38.319843],[-8.322571,38.37218349700004],[-8.285075499999948,38.424620497000035],[-8.369122499999946,38.54574249600006],[-8.639115499999946,38.54948899800007],[-8.653472,38.612269999000034],[-8.490975,38.76131049800006],[-8.54680049999996,38.76343399700005],[-8.301621,38.86059299800007],[-8.141565,38.772977499000035],[-8.124314,38.80496750000003],[-8.187920499999962,38.83812999700007],[-8.158744,38.86639649700004],[-8.272191499999963,38.90738999800004],[-8.22791849999993,38.98738549800004],[-8.145509,39.02329099700006],[-8.10316450099998,38.96915699600004],[-8.068534,39.01507299700006],[-8.02225,38.940972499000054],[-7.924353499999938,38.97449549700008],[-7.789951499999972,38.89579299800005],[-7.564648500999965,38.95245349700008],[-7.533353499999976,39.008439999000075],[-7.452719499999944,38.963097498000025],[-7.391644,38.80162949900006],[-7.244899,38.847780998000076],[-7.203135,38.75101749700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT2","LEVL_CODE":1,"CNTR_CODE":"PT","NAME_LATN":"REGIÃO AUTÓNOMA DOS AÇORES","NUTS_NAME":"REGIÃO AUTÓNOMA DOS AÇORES"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-25.765763,37.906070998000075],[-25.689569,37.84193950000008],[-25.193974,37.85964199800003],[-25.134163499999943,37.807617497000024],[-25.205413,37.735234999000056],[-25.511525499999948,37.708371498000076],[-25.68617949999998,37.73279899800008],[-25.85274,37.85310549800005],[-25.765763,37.906070998000075]]],[[[-27.042569499999956,38.696639997000034],[-27.079417,38.64398649800006],[-27.276853,38.65224649700008],[-27.379397499999982,38.74690699800004],[-27.256799,38.80333449700004],[-27.105539,38.78568549800008],[-27.042569499999956,38.696639997000034]]],[[[-28.31683449999997,38.756650498000056],[-27.77179,38.53695349800006],[-28.19083349999994,38.66695149800006],[-28.31683449999997,38.756650498000056]]],[[[-28.029527499999972,38.41165249900007],[-28.178494499999942,38.40929349800007],[-28.445251,38.42045999700008],[-28.539842499999963,38.48665149800007],[-28.519377,38.54547049800004],[-28.389973,38.55832099700007],[-28.029527499999972,38.41165249900007]]],[[[-28.62946949999997,38.524696997000035],[-28.719994,38.517318997000075],[-28.834771,38.599279498000044],[-28.714248,38.643756997000025],[-28.620574,38.61104949800006],[-28.62946949999997,38.524696997000035]]],[[[-31.136211499999945,39.46318949800008],[-31.170449,39.377477998000074],[-31.25928049999993,39.399148997000054],[-31.23694249999994,39.52081050000004],[-31.136211499999945,39.46318949800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT20","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Região Autónoma dos Açores","NUTS_NAME":"Região Autónoma dos Açores"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-25.765763,37.906070998000075],[-25.689569,37.84193950000008],[-25.193974,37.85964199800003],[-25.134163499999943,37.807617497000024],[-25.205413,37.735234999000056],[-25.511525499999948,37.708371498000076],[-25.68617949999998,37.73279899800008],[-25.85274,37.85310549800005],[-25.765763,37.906070998000075]]],[[[-27.042569499999956,38.696639997000034],[-27.079417,38.64398649800006],[-27.276853,38.65224649700008],[-27.379397499999982,38.74690699800004],[-27.256799,38.80333449700004],[-27.105539,38.78568549800008],[-27.042569499999956,38.696639997000034]]],[[[-28.31683449999997,38.756650498000056],[-27.77179,38.53695349800006],[-28.19083349999994,38.66695149800006],[-28.31683449999997,38.756650498000056]]],[[[-28.029527499999972,38.41165249900007],[-28.178494499999942,38.40929349800007],[-28.445251,38.42045999700008],[-28.539842499999963,38.48665149800007],[-28.519377,38.54547049800004],[-28.389973,38.55832099700007],[-28.029527499999972,38.41165249900007]]],[[[-28.62946949999997,38.524696997000035],[-28.719994,38.517318997000075],[-28.834771,38.599279498000044],[-28.714248,38.643756997000025],[-28.620574,38.61104949800006],[-28.62946949999997,38.524696997000035]]],[[[-31.136211499999945,39.46318949800008],[-31.170449,39.377477998000074],[-31.25928049999993,39.399148997000054],[-31.23694249999994,39.52081050000004],[-31.136211499999945,39.46318949800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT200","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Região Autónoma dos Açores","NUTS_NAME":"Região Autónoma dos Açores"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-25.765763,37.906070998000075],[-25.689569,37.84193950000008],[-25.193974,37.85964199800003],[-25.134163499999943,37.807617497000024],[-25.205413,37.735234999000056],[-25.511525499999948,37.708371498000076],[-25.68617949999998,37.73279899800008],[-25.85274,37.85310549800005],[-25.765763,37.906070998000075]]],[[[-27.042569499999956,38.696639997000034],[-27.079417,38.64398649800006],[-27.276853,38.65224649700008],[-27.379397499999982,38.74690699800004],[-27.256799,38.80333449700004],[-27.105539,38.78568549800008],[-27.042569499999956,38.696639997000034]]],[[[-28.31683449999997,38.756650498000056],[-27.77179,38.53695349800006],[-28.19083349999994,38.66695149800006],[-28.31683449999997,38.756650498000056]]],[[[-28.029527499999972,38.41165249900007],[-28.178494499999942,38.40929349800007],[-28.445251,38.42045999700008],[-28.539842499999963,38.48665149800007],[-28.519377,38.54547049800004],[-28.389973,38.55832099700007],[-28.029527499999972,38.41165249900007]]],[[[-28.62946949999997,38.524696997000035],[-28.719994,38.517318997000075],[-28.834771,38.599279498000044],[-28.714248,38.643756997000025],[-28.620574,38.61104949800006],[-28.62946949999997,38.524696997000035]]],[[[-31.136211499999945,39.46318949800008],[-31.170449,39.377477998000074],[-31.25928049999993,39.399148997000054],[-31.23694249999994,39.52081050000004],[-31.136211499999945,39.46318949800008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT3","LEVL_CODE":1,"CNTR_CODE":"PT","NAME_LATN":"REGIÃO AUTÓNOMA DA MADEIRA","NUTS_NAME":"REGIÃO AUTÓNOMA DA MADEIRA"},"geometry":{"type":"Polygon","coordinates":[[[-17.193386499999974,32.87055649800004],[-17.102119499999958,32.81972899900006],[-16.903707,32.83755799800008],[-16.746594499999958,32.73416999900007],[-16.831212499999936,32.64036199800006],[-16.944882500999938,32.63293549800005],[-17.212603499999943,32.73696999800006],[-17.265786,32.81489249700007],[-17.193386499999974,32.87055649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT30","LEVL_CODE":2,"CNTR_CODE":"PT","NAME_LATN":"Região Autónoma da Madeira","NUTS_NAME":"Região Autónoma da Madeira"},"geometry":{"type":"Polygon","coordinates":[[[-17.193386499999974,32.87055649800004],[-17.102119499999958,32.81972899900006],[-16.903707,32.83755799800008],[-16.746594499999958,32.73416999900007],[-16.831212499999936,32.64036199800006],[-16.944882500999938,32.63293549800005],[-17.212603499999943,32.73696999800006],[-17.265786,32.81489249700007],[-17.193386499999974,32.87055649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PT300","LEVL_CODE":3,"CNTR_CODE":"PT","NAME_LATN":"Região Autónoma da Madeira","NUTS_NAME":"Região Autónoma da Madeira"},"geometry":{"type":"Polygon","coordinates":[[[-17.193386499999974,32.87055649800004],[-17.102119499999958,32.81972899900006],[-16.903707,32.83755799800008],[-16.746594499999958,32.73416999900007],[-16.831212499999936,32.64036199800006],[-16.944882500999938,32.63293549800005],[-17.212603499999943,32.73696999800006],[-17.265786,32.81489249700007],[-17.193386499999974,32.87055649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO","LEVL_CODE":0,"CNTR_CODE":"RO","NAME_LATN":"ROMÂNIA","NUTS_NAME":"ROMÂNIA"},"geometry":{"type":"Polygon","coordinates":[[[27.213884499000073,47.82219949800003],[27.391166500000054,47.58939749600006],[27.559885,47.46286199800005],[27.603923,47.328363498000044],[27.73602550000004,47.28789749600003],[27.915807,47.06719250000003],[28.09621,46.977461996000045],[28.11380550000007,46.838411],[28.252111,46.61181099700008],[28.221767500000055,46.50262349600007],[28.260886,46.43713949800008],[28.109676,46.242388498000025],[28.14290550000004,46.16414949800003],[28.115802,46.10782699600003],[28.08719750000006,46.00968099800008],[28.170279,45.67490599800004],[28.088561500000026,45.60611249800007],[28.21136,45.46727999800004],[28.28599,45.43063],[28.34137,45.325049998000054],[28.717,45.224379998000074],[28.793709999000043,45.24397999800004],[28.79881,45.335729997000044],[28.9442,45.28135],[29.21645,45.419339998000055],[29.42759,45.44223999800005],[29.654986500000064,45.33608449600007],[29.679639500000064,45.21183799900007],[29.631290500000034,45.20977],[29.71331950000007,45.16507299600005],[29.653199500000028,45.09557799600003],[29.600269,44.83955449800004],[29.185992500000054,44.78466350000008],[28.994085,44.679625497000075],[28.625982500000077,44.29703199700003],[28.667546,43.98677249800005],[28.578884,43.73873899900008],[28.236143,43.75852199800005],[27.994264500000043,43.842964497000025],[27.916834,44.00827399700006],[27.737999,43.95341499700004],[27.69541,43.98734299900008],[27.643537500000036,44.047972998000034],[27.399397500000077,44.012369998000054],[27.271344999000064,44.12633649800006],[26.906678500000055,44.133331998000074],[26.379968,44.04295099800004],[26.358359999000072,44.03841550000004],[26.114924,43.97408949700008],[25.783091,43.71223699700005],[25.67186250000003,43.691339998000046],[25.54464050000007,43.64298149600006],[25.29374050000007,43.65429399900006],[24.641605999000035,43.73341299800006],[24.501014500000053,43.762396996000064],[24.324132,43.69956799600004],[24.112719,43.699566497000035],[23.63008,43.791259997000054],[23.267777,43.846812],[22.99715450000008,43.80762699600007],[22.838719,43.877852],[22.878725,43.98156750000004],[23.045898,44.06374949900004],[22.966399500000023,44.09852499600004],[22.67516150000006,44.21566299600005],[22.677534500000036,44.280286498000066],[22.560019500000067,44.30736649600004],[22.457399500000065,44.46746649900007],[22.548219500000073,44.48124899800007],[22.58179350000006,44.548156996000046],[22.762785,44.55141149700006],[22.70595650000007,44.60321649800005],[22.467334,44.71474549900006],[22.332951,44.67587199800005],[22.159579500000063,44.47177749800005],[22.016132500000026,44.59920249800007],[22.012350500000025,44.602318498000045],[21.640041500000052,44.659280498000044],[21.587786,44.75841599800003],[21.35847,44.82161199600006],[21.367058,44.86448149600005],[21.560126500000024,44.88900649900006],[21.36023349900006,45.02043299800005],[21.456053500000053,45.04039949600008],[21.527420500000062,45.13786699600007],[21.479178500000046,45.19302750000003],[21.218988,45.250575998000045],[21.176224500000046,45.32522649800006],[21.016575,45.32462749700005],[20.86616199900004,45.46523399800003],[20.771111500000075,45.47996649700008],[20.832577,45.53589299600003],[20.765725,45.61256849700004],[20.823079999000072,45.77506449900005],[20.70064650000006,45.75056199900007],[20.662833,45.79411599800005],[20.569299,45.91013849800004],[20.348325500000044,45.99962699800005],[20.264296,46.12637349900007],[20.501525,46.18993599800007],[20.6406,46.12720699800008],[20.70530350000007,46.16093749800007],[20.7756,46.27590999800003],[21.103170500000033,46.26259049600003],[21.206304,46.40323099600005],[21.296291,46.40669299800004],[21.317476,46.45065599600008],[21.260401500000057,46.501945996000075],[21.314121999000065,46.617556],[21.441398,46.651466999000036],[21.43006950000006,46.693812998000055],[21.52933950000005,46.72074199900004],[21.483475500000054,46.76463549700003],[21.60309650000005,46.86840799600003],[21.658955,47.02213149700003],[21.792838500000073,47.10724299900005],[21.938521500000036,47.37301099800004],[22.011991,47.37579649800006],[22.012631,47.50135449800007],[22.12832,47.59808949600006],[22.180837500000052,47.60009449800003],[22.31726,47.76592199600003],[22.680243999000027,47.78774099800006],[22.89627050000007,47.954120500000045],[22.945598,47.96832099800008],[22.926977500000078,48.01914899700006],[23.089905,48.00488799900006],[23.11396650000006,48.086210996000034],[23.188535500000057,48.10868799800005],[23.493605,47.96781149800006],[23.771808500000077,47.99492699800004],[23.864122,47.93173899800007],[24.00904950000006,47.967242500000054],[24.228265,47.896465498000055],[24.583293500000025,47.96485149600005],[24.88314,47.72435349700004],[24.947100500000033,47.72912399900008],[25.115598999000042,47.754823498000064],[25.310894500000074,47.91432050000003],[26.098827500000027,47.97879599600003],[26.188041,47.99546449800005],[26.33267550000005,48.18580449600006],[26.63056,48.259749998000075],[26.852818,48.23365449600004],[27.164651500000048,47.99468349800003],[27.213884499000073,47.82219949800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO1","LEVL_CODE":1,"CNTR_CODE":"RO","NAME_LATN":"MACROREGIUNEA UNU","NUTS_NAME":"MACROREGIUNEA UNU"},"geometry":{"type":"Polygon","coordinates":[[[23.188535500000057,48.10868799800005],[23.493605,47.96781149800006],[23.771808500000077,47.99492699800004],[23.864122,47.93173899800007],[24.00904950000006,47.967242500000054],[24.228265,47.896465498000055],[24.583293500000025,47.96485149600005],[24.88314,47.72435349700004],[24.947100500000033,47.72912399900008],[25.022151500000064,47.66637799800003],[24.960837500000025,47.596850998000036],[25.095044,47.516200996000066],[25.058275,47.41653199800004],[24.982833500000027,47.42401949900005],[25.069417,47.36992249700006],[25.07738849900005,47.31147850000008],[25.028061500000035,47.286654499000065],[25.07958699900007,47.25862549800007],[25.063383,47.13653849700006],[25.246710500000063,47.09791899900006],[25.305628,47.16538899600005],[25.425438,47.17119699800003],[25.539915,47.082194996000055],[25.661582499000076,47.091646499000035],[25.86196750000005,46.92338349700003],[25.765381500000046,46.88445299700004],[25.79531,46.71911749800006],[25.867373,46.649418997000055],[25.975159500000075,46.69731249800003],[26.08134750000005,46.51633349800005],[26.006634,46.48163849800005],[25.995239999000034,46.40674749800007],[26.165532,46.384739997000054],[26.15608050000003,46.31515849900006],[26.29125350000004,46.33603499800006],[26.263439,46.24633849600008],[26.324927,46.23116549700006],[26.439984500000037,46.038876498000036],[26.37564499900003,45.90731349600003],[26.39209450000004,45.802389496000046],[26.310291500000062,45.606565997000075],[26.212819,45.621830997000075],[26.093334500000026,45.51630599600003],[26.072718500000065,45.50578799800007],[25.899684,45.49885549800007],[25.86003850000003,45.421914998000034],[25.680995,45.50061949800005],[25.452539,45.44134199800004],[25.321577500000046,45.38108899700006],[25.25536850000003,45.42249349800005],[25.229541,45.517015997000044],[25.103214,45.58519399800008],[24.684925500000077,45.60411099600003],[24.512834500000054,45.586824998000054],[23.703613,45.496772],[23.597270500000036,45.47347149800004],[23.41857199900005,45.59558899800004],[23.31906,45.90771499600004],[23.234864,46.030501996000055],[23.087312,46.06646149800008],[23.055348500000036,46.140611496000076],[23.095532500000047,46.18230050000005],[22.96113450000007,46.23682549600005],[22.925720500000068,46.330588996000074],[22.748716,46.351207499000054],[22.67657550000007,46.405825496000034],[22.424138,46.39332049600006],[22.32517050000007,46.51450699700007],[22.197508,46.51382549600004],[22.168398,46.60386349800007],[22.026914,46.64379299600006],[21.441398,46.651466999000036],[21.43006950000006,46.693812998000055],[21.52933950000005,46.72074199900004],[21.483475500000054,46.76463549700003],[21.60309650000005,46.86840799600003],[21.658955,47.02213149700003],[21.792838500000073,47.10724299900005],[21.938521500000036,47.37301099800004],[22.011991,47.37579649800006],[22.012631,47.50135449800007],[22.12832,47.59808949600006],[22.180837500000052,47.60009449800003],[22.31726,47.76592199600003],[22.680243999000027,47.78774099800006],[22.89627050000007,47.954120500000045],[22.945598,47.96832099800008],[22.926977500000078,48.01914899700006],[23.089905,48.00488799900006],[23.11396650000006,48.086210996000034],[23.188535500000057,48.10868799800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO11","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Nord-Vest","NUTS_NAME":"Nord-Vest"},"geometry":{"type":"Polygon","coordinates":[[[23.188535500000057,48.10868799800005],[23.493605,47.96781149800006],[23.771808500000077,47.99492699800004],[23.864122,47.93173899800007],[24.00904950000006,47.967242500000054],[24.228265,47.896465498000055],[24.583293500000025,47.96485149600005],[24.88314,47.72435349700004],[24.947100500000033,47.72912399900008],[25.022151500000064,47.66637799800003],[24.960837500000025,47.596850998000036],[25.095044,47.516200996000066],[25.058275,47.41653199800004],[24.982833500000027,47.42401949900005],[25.069417,47.36992249700006],[25.07738849900005,47.31147850000008],[25.028061500000035,47.286654499000065],[25.07958699900007,47.25862549800007],[25.063383,47.13653849700006],[24.853731,47.09559999800007],[24.721708,46.93237399800006],[24.74557150000004,46.905763998000054],[24.59522450000003,46.94562549600005],[24.400994999000034,46.75053549800003],[24.18309310500007,46.802148025000065],[24.066509500000052,46.73344149800005],[24.123644,46.69899299800005],[24.020229500000028,46.62786199900006],[24.069059500000037,46.606018498000026],[24.045371500000044,46.54505],[23.95904850000005,46.51538749800005],[23.985275,46.43091799800004],[23.702531,46.45742799800007],[23.660915500000044,46.396698],[23.591964,46.43607399600006],[23.611035,46.48346599900003],[23.384006,46.54046550000004],[23.052890500000046,46.466147499000044],[22.999922999000034,46.55257499900006],[22.811950500000023,46.56885950000003],[22.676956,46.52328949900004],[22.719911499000034,46.43091049800006],[22.67657550000007,46.405825496000034],[22.424138,46.39332049600006],[22.32517050000007,46.51450699700007],[22.197508,46.51382549600004],[22.168398,46.60386349800007],[22.026914,46.64379299600006],[21.441398,46.651466999000036],[21.43006950000006,46.693812998000055],[21.52933950000005,46.72074199900004],[21.483475500000054,46.76463549700003],[21.60309650000005,46.86840799600003],[21.658955,47.02213149700003],[21.792838500000073,47.10724299900005],[21.938521500000036,47.37301099800004],[22.011991,47.37579649800006],[22.012631,47.50135449800007],[22.12832,47.59808949600006],[22.180837500000052,47.60009449800003],[22.31726,47.76592199600003],[22.680243999000027,47.78774099800006],[22.89627050000007,47.954120500000045],[22.945598,47.96832099800008],[22.926977500000078,48.01914899700006],[23.089905,48.00488799900006],[23.11396650000006,48.086210996000034],[23.188535500000057,48.10868799800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO111","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Bihor","NUTS_NAME":"Bihor"},"geometry":{"type":"Polygon","coordinates":[[[22.61267950000007,47.33474149800003],[22.496962,47.18815999800006],[22.554477500000075,47.12614149800004],[22.51430350000004,47.10912899900006],[22.72817850000007,47.00749099600006],[22.677120500000058,46.949077998000064],[22.754126,46.87621249800003],[22.637293500000055,46.799401498000066],[22.709879,46.76473649800005],[22.811950500000023,46.56885950000003],[22.676956,46.52328949900004],[22.719911499000034,46.43091049800006],[22.67657550000007,46.405825496000034],[22.424138,46.39332049600006],[22.32517050000007,46.51450699700007],[22.197508,46.51382549600004],[22.168398,46.60386349800007],[22.026914,46.64379299600006],[21.441398,46.651466999000036],[21.43006950000006,46.693812998000055],[21.52933950000005,46.72074199900004],[21.483475500000054,46.76463549700003],[21.60309650000005,46.86840799600003],[21.658955,47.02213149700003],[21.792838500000073,47.10724299900005],[21.938521500000036,47.37301099800004],[22.011991,47.37579649800006],[22.012631,47.50135449800007],[22.12832,47.59808949600006],[22.180837500000052,47.60009449800003],[22.352439,47.433572],[22.529385,47.41859949700006],[22.61267950000007,47.33474149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO112","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Bistrita-Nasaud","NUTS_NAME":"Bistrita-Nasaud"},"geometry":{"type":"Polygon","coordinates":[[[24.960837500000025,47.596850998000036],[25.095044,47.516200996000066],[25.058275,47.41653199800004],[24.982833500000027,47.42401949900005],[25.069417,47.36992249700006],[25.07738849900005,47.31147850000008],[25.028061500000035,47.286654499000065],[25.07958699900007,47.25862549800007],[25.063383,47.13653849700006],[24.853731,47.09559999800007],[24.721708,46.93237399800006],[24.74557150000004,46.905763998000054],[24.59522450000003,46.94562549600005],[24.400994999000034,46.75053549800003],[24.18309310500007,46.802148025000065],[24.227452500000027,46.888119998000036],[24.113925,46.96054099700007],[24.135193,47.08687749900008],[23.93603,47.19927899900006],[23.928280500000028,47.30065049900003],[24.022714500000063,47.361948],[24.15342450000003,47.50273999900003],[24.388667,47.60661149900005],[24.65958450000005,47.553972498000064],[24.960837500000025,47.596850998000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO113","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Cluj","NUTS_NAME":"Cluj"},"geometry":{"type":"Polygon","coordinates":[[[24.022714500000063,47.361948],[23.928280500000028,47.30065049900003],[23.93603,47.19927899900006],[24.135193,47.08687749900008],[24.113925,46.96054099700007],[24.227452500000027,46.888119998000036],[24.18309310500007,46.802148025000065],[24.066509500000052,46.73344149800005],[24.123644,46.69899299800005],[24.020229500000028,46.62786199900006],[24.069059500000037,46.606018498000026],[24.045371500000044,46.54505],[23.95904850000005,46.51538749800005],[23.985275,46.43091799800004],[23.702531,46.45742799800007],[23.660915500000044,46.396698],[23.591964,46.43607399600006],[23.611035,46.48346599900003],[23.384006,46.54046550000004],[23.052890500000046,46.466147499000044],[22.999922999000034,46.55257499900006],[22.811950500000023,46.56885950000003],[22.709879,46.76473649800005],[22.637293500000055,46.799401498000066],[22.754126,46.87621249800003],[22.677120500000058,46.949077998000064],[22.72817850000007,47.00749099600006],[23.103373500000032,46.85385149600006],[23.222600500000055,46.892614999000045],[23.229597,46.96258799900005],[23.338755500000048,46.908509999000046],[23.47406950000004,47.01350149600006],[23.521386,47.15547749600006],[23.752597500000036,47.24196049900007],[23.841384,47.340007498000034],[24.022714500000063,47.361948]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO114","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Maramures","NUTS_NAME":"Maramures"},"geometry":{"type":"Polygon","coordinates":[[[24.947100500000033,47.72912399900008],[25.022151500000064,47.66637799800003],[24.960837500000025,47.596850998000036],[24.65958450000005,47.553972498000064],[24.388667,47.60661149900005],[24.15342450000003,47.50273999900003],[24.022714500000063,47.361948],[23.841384,47.340007498000034],[23.106107,47.442358],[22.964224500000057,47.439354999000045],[22.963248500000077,47.48644999900006],[23.142585,47.624257998000076],[23.258154500000046,47.55085699800003],[23.387573,47.68631299800006],[23.24427550000007,47.72270749900008],[23.224826,47.76639299800007],[23.261835,47.79732199800003],[23.450439,47.761952],[23.639845,47.864457998000034],[23.493605,47.96781149800006],[23.771808500000077,47.99492699800004],[23.864122,47.93173899800007],[24.00904950000006,47.967242500000054],[24.228265,47.896465498000055],[24.583293500000025,47.96485149600005],[24.88314,47.72435349700004],[24.947100500000033,47.72912399900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO115","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Satu Mare","NUTS_NAME":"Satu Mare"},"geometry":{"type":"Polygon","coordinates":[[[23.493605,47.96781149800006],[23.639845,47.864457998000034],[23.450439,47.761952],[23.261835,47.79732199800003],[23.224826,47.76639299800007],[23.24427550000007,47.72270749900008],[23.387573,47.68631299800006],[23.258154500000046,47.55085699800003],[23.142585,47.624257998000076],[22.963248500000077,47.48644999900006],[22.964224500000057,47.439354999000045],[23.106107,47.442358],[23.119117999000025,47.39563449800005],[23.054975,47.352898],[22.847762,47.42215],[22.61267950000007,47.33474149800003],[22.529385,47.41859949700006],[22.352439,47.433572],[22.180837500000052,47.60009449800003],[22.31726,47.76592199600003],[22.680243999000027,47.78774099800006],[22.89627050000007,47.954120500000045],[22.945598,47.96832099800008],[22.926977500000078,48.01914899700006],[23.089905,48.00488799900006],[23.11396650000006,48.086210996000034],[23.188535500000057,48.10868799800005],[23.493605,47.96781149800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO116","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Salaj","NUTS_NAME":"Salaj"},"geometry":{"type":"Polygon","coordinates":[[[23.841384,47.340007498000034],[23.752597500000036,47.24196049900007],[23.521386,47.15547749600006],[23.47406950000004,47.01350149600006],[23.338755500000048,46.908509999000046],[23.229597,46.96258799900005],[23.222600500000055,46.892614999000045],[23.103373500000032,46.85385149600006],[22.72817850000007,47.00749099600006],[22.51430350000004,47.10912899900006],[22.554477500000075,47.12614149800004],[22.496962,47.18815999800006],[22.61267950000007,47.33474149800003],[22.847762,47.42215],[23.054975,47.352898],[23.119117999000025,47.39563449800005],[23.106107,47.442358],[23.841384,47.340007498000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO12","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Centru","NUTS_NAME":"Centru"},"geometry":{"type":"Polygon","coordinates":[[[25.425438,47.17119699800003],[25.539915,47.082194996000055],[25.661582499000076,47.091646499000035],[25.86196750000005,46.92338349700003],[25.765381500000046,46.88445299700004],[25.79531,46.71911749800006],[25.867373,46.649418997000055],[25.975159500000075,46.69731249800003],[26.08134750000005,46.51633349800005],[26.006634,46.48163849800005],[25.995239999000034,46.40674749800007],[26.165532,46.384739997000054],[26.15608050000003,46.31515849900006],[26.29125350000004,46.33603499800006],[26.263439,46.24633849600008],[26.324927,46.23116549700006],[26.439984500000037,46.038876498000036],[26.37564499900003,45.90731349600003],[26.39209450000004,45.802389496000046],[26.310291500000062,45.606565997000075],[26.212819,45.621830997000075],[26.093334500000026,45.51630599600003],[26.072718500000065,45.50578799800007],[25.899684,45.49885549800007],[25.86003850000003,45.421914998000034],[25.680995,45.50061949800005],[25.452539,45.44134199800004],[25.321577500000046,45.38108899700006],[25.25536850000003,45.42249349800005],[25.229541,45.517015997000044],[25.103214,45.58519399800008],[24.684925500000077,45.60411099600003],[24.512834500000054,45.586824998000054],[23.703613,45.496772],[23.597270500000036,45.47347149800004],[23.41857199900005,45.59558899800004],[23.31906,45.90771499600004],[23.234864,46.030501996000055],[23.087312,46.06646149800008],[23.055348500000036,46.140611496000076],[23.095532500000047,46.18230050000005],[22.96113450000007,46.23682549600005],[22.925720500000068,46.330588996000074],[22.748716,46.351207499000054],[22.67657550000007,46.405825496000034],[22.719911499000034,46.43091049800006],[22.676956,46.52328949900004],[22.811950500000023,46.56885950000003],[22.999922999000034,46.55257499900006],[23.052890500000046,46.466147499000044],[23.384006,46.54046550000004],[23.611035,46.48346599900003],[23.591964,46.43607399600006],[23.660915500000044,46.396698],[23.702531,46.45742799800007],[23.985275,46.43091799800004],[23.95904850000005,46.51538749800005],[24.045371500000044,46.54505],[24.069059500000037,46.606018498000026],[24.020229500000028,46.62786199900006],[24.123644,46.69899299800005],[24.066509500000052,46.73344149800005],[24.18309310500007,46.802148025000065],[24.400994999000034,46.75053549800003],[24.59522450000003,46.94562549600005],[24.74557150000004,46.905763998000054],[24.721708,46.93237399800006],[24.853731,47.09559999800007],[25.063383,47.13653849700006],[25.246710500000063,47.09791899900006],[25.305628,47.16538899600005],[25.425438,47.17119699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO121","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Alba","NUTS_NAME":"Alba"},"geometry":{"type":"Polygon","coordinates":[[[22.811950500000023,46.56885950000003],[22.999922999000034,46.55257499900006],[23.052890500000046,46.466147499000044],[23.384006,46.54046550000004],[23.611035,46.48346599900003],[23.591964,46.43607399600006],[23.660915500000044,46.396698],[23.702531,46.45742799800007],[23.985275,46.43091799800004],[24.032309500000054,46.457075],[23.997585,46.370426499000075],[24.23131150000006,46.25963999800007],[24.21325550000006,46.154210497000065],[24.09027150000003,46.11654599800005],[23.999912,45.99833749900006],[23.89133350000003,46.04470499900003],[23.816467,45.92328749600006],[23.734621,45.91516349600005],[23.687518,45.80263349900008],[23.61655650000006,45.801904998000055],[23.674468,45.74945899800008],[23.58890899900007,45.70272799800006],[23.650547,45.60187949800007],[23.610621,45.563638999000034],[23.703613,45.496772],[23.597270500000036,45.47347149800004],[23.41857199900005,45.59558899800004],[23.31906,45.90771499600004],[23.234864,46.030501996000055],[23.087312,46.06646149800008],[23.055348500000036,46.140611496000076],[23.095532500000047,46.18230050000005],[22.96113450000007,46.23682549600005],[22.925720500000068,46.330588996000074],[22.748716,46.351207499000054],[22.67657550000007,46.405825496000034],[22.719911499000034,46.43091049800006],[22.676956,46.52328949900004],[22.811950500000023,46.56885950000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO122","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Brasov","NUTS_NAME":"Brasov"},"geometry":{"type":"Polygon","coordinates":[[[25.451241500000037,46.10409449800005],[25.580992500000036,46.02654699800007],[25.541224,45.982996498000034],[25.567211,45.83655099900005],[25.971584,45.68758249900003],[26.09514,45.570042999000066],[26.093334500000026,45.51630599600003],[26.072718500000065,45.50578799800007],[25.899684,45.49885549800007],[25.86003850000003,45.421914998000034],[25.680995,45.50061949800005],[25.452539,45.44134199800004],[25.321577500000046,45.38108899700006],[25.25536850000003,45.42249349800005],[25.229541,45.517015997000044],[25.103214,45.58519399800008],[24.684925500000077,45.60411099600003],[24.639231,45.795109498000045],[24.77614150000005,45.86125749900003],[24.702945,45.94018549800006],[24.946548500000063,46.07910899600006],[25.173251,46.16551949900003],[25.250545,46.14156649800003],[25.273424,46.18721099800007],[25.451241500000037,46.10409449800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO123","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Covasna","NUTS_NAME":"Covasna"},"geometry":{"type":"Polygon","coordinates":[[[26.263439,46.24633849600008],[26.324927,46.23116549700006],[26.439984500000037,46.038876498000036],[26.37564499900003,45.90731349600003],[26.39209450000004,45.802389496000046],[26.310291500000062,45.606565997000075],[26.212819,45.621830997000075],[26.093334500000026,45.51630599600003],[26.09514,45.570042999000066],[25.971584,45.68758249900003],[25.567211,45.83655099900005],[25.541224,45.982996498000034],[25.580992500000036,46.02654699800007],[25.451241500000037,46.10409449800005],[25.505654999000058,46.185333497000045],[25.725981,46.27456549800007],[25.859878500000036,46.119689],[26.172338999000033,46.160329500000046],[26.263439,46.24633849600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO124","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Harghita","NUTS_NAME":"Harghita"},"geometry":{"type":"Polygon","coordinates":[[[25.661582499000076,47.091646499000035],[25.86196750000005,46.92338349700003],[25.765381500000046,46.88445299700004],[25.79531,46.71911749800006],[25.867373,46.649418997000055],[25.975159500000075,46.69731249800003],[26.08134750000005,46.51633349800005],[26.006634,46.48163849800005],[25.995239999000034,46.40674749800007],[26.165532,46.384739997000054],[26.15608050000003,46.31515849900006],[26.29125350000004,46.33603499800006],[26.263439,46.24633849600008],[26.172338999000033,46.160329500000046],[25.859878500000036,46.119689],[25.725981,46.27456549800007],[25.505654999000058,46.185333497000045],[25.451241500000037,46.10409449800005],[25.273424,46.18721099800007],[25.250545,46.14156649800003],[25.173251,46.16551949900003],[24.868171,46.298955998000054],[24.86323,46.34605899700006],[24.979179,46.38464449800006],[24.93590850000004,46.42108049800004],[25.017587500000047,46.464049998000064],[25.022374,46.54110950000006],[25.294925,46.688334499000064],[25.26108350000004,46.74979949800007],[25.318422500000054,46.77302099900004],[25.251292,46.84125049800008],[25.289613,46.982797996000045],[25.246710500000063,47.09791899900006],[25.305628,47.16538899600005],[25.425438,47.17119699800003],[25.539915,47.082194996000055],[25.661582499000076,47.091646499000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO125","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Mures","NUTS_NAME":"Mures"},"geometry":{"type":"Polygon","coordinates":[[[25.246710500000063,47.09791899900006],[25.289613,46.982797996000045],[25.251292,46.84125049800008],[25.318422500000054,46.77302099900004],[25.26108350000004,46.74979949800007],[25.294925,46.688334499000064],[25.022374,46.54110950000006],[25.017587500000047,46.464049998000064],[24.93590850000004,46.42108049800004],[24.979179,46.38464449800006],[24.86323,46.34605899700006],[24.868171,46.298955998000054],[25.173251,46.16551949900003],[24.946548500000063,46.07910899600006],[24.69624199900005,46.08335949800005],[24.664698999000052,46.28095449800003],[24.23131150000006,46.25963999800007],[23.997585,46.370426499000075],[24.032309500000054,46.457075],[23.985275,46.43091799800004],[23.95904850000005,46.51538749800005],[24.045371500000044,46.54505],[24.069059500000037,46.606018498000026],[24.020229500000028,46.62786199900006],[24.123644,46.69899299800005],[24.066509500000052,46.73344149800005],[24.18309310500007,46.802148025000065],[24.400994999000034,46.75053549800003],[24.59522450000003,46.94562549600005],[24.74557150000004,46.905763998000054],[24.721708,46.93237399800006],[24.853731,47.09559999800007],[25.063383,47.13653849700006],[25.246710500000063,47.09791899900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO126","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Sibiu","NUTS_NAME":"Sibiu"},"geometry":{"type":"Polygon","coordinates":[[[24.946548500000063,46.07910899600006],[24.702945,45.94018549800006],[24.77614150000005,45.86125749900003],[24.639231,45.795109498000045],[24.684925500000077,45.60411099600003],[24.512834500000054,45.586824998000054],[23.703613,45.496772],[23.610621,45.563638999000034],[23.650547,45.60187949800007],[23.58890899900007,45.70272799800006],[23.674468,45.74945899800008],[23.61655650000006,45.801904998000055],[23.687518,45.80263349900008],[23.734621,45.91516349600005],[23.816467,45.92328749600006],[23.89133350000003,46.04470499900003],[23.999912,45.99833749900006],[24.09027150000003,46.11654599800005],[24.21325550000006,46.154210497000065],[24.23131150000006,46.25963999800007],[24.664698999000052,46.28095449800003],[24.69624199900005,46.08335949800005],[24.946548500000063,46.07910899600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO2","LEVL_CODE":1,"CNTR_CODE":"RO","NAME_LATN":"MACROREGIUNEA DOI","NUTS_NAME":"MACROREGIUNEA DOI"},"geometry":{"type":"Polygon","coordinates":[[[27.559885,47.46286199800005],[27.603923,47.328363498000044],[27.73602550000004,47.28789749600003],[27.915807,47.06719250000003],[28.09621,46.977461996000045],[28.11380550000007,46.838411],[28.252111,46.61181099700008],[28.221767500000055,46.50262349600007],[28.260886,46.43713949800008],[28.109676,46.242388498000025],[28.14290550000004,46.16414949800003],[28.115802,46.10782699600003],[28.08719750000006,46.00968099800008],[28.170279,45.67490599800004],[28.088561500000026,45.60611249800007],[28.21136,45.46727999800004],[28.28599,45.43063],[28.34137,45.325049998000054],[28.717,45.224379998000074],[28.793709999000043,45.24397999800004],[28.79881,45.335729997000044],[28.9442,45.28135],[29.21645,45.419339998000055],[29.42759,45.44223999800005],[29.654986500000064,45.33608449600007],[29.679639500000064,45.21183799900007],[29.631290500000034,45.20977],[29.71331950000007,45.16507299600005],[29.653199500000028,45.09557799600003],[29.600269,44.83955449800004],[29.185992500000054,44.78466350000008],[28.994085,44.679625497000075],[28.625982500000077,44.29703199700003],[28.667546,43.98677249800005],[28.578884,43.73873899900008],[28.236143,43.75852199800005],[27.994264500000043,43.842964497000025],[27.916834,44.00827399700006],[27.737999,43.95341499700004],[27.69541,43.98734299900008],[27.643537500000036,44.047972998000034],[27.399397500000077,44.012369998000054],[27.271344999000064,44.12633649800006],[27.926198500000055,44.248121498000046],[28.01737399900003,44.34024899600007],[28.110168500000043,44.439703500000064],[28.016514,44.56481799900007],[28.031737500000077,44.62529149800008],[27.882344,44.707761498000025],[27.881007,44.76307649800003],[27.818804,44.82033499700003],[27.721871,44.765049],[27.203458500000067,44.78707399800004],[27.134057,44.73553449700006],[27.007946,44.82446649600007],[26.810180500000058,44.79534150000006],[26.605551500000047,44.856993997000075],[26.530521500000077,44.937532496000074],[26.599910500000078,44.95584449900008],[26.451353,45.08330349800008],[26.46035,45.19809199700006],[26.376487500000053,45.158456999000066],[26.249952499000074,45.212642996000056],[26.18549850000005,45.27803449600003],[26.233643,45.314494499000034],[26.153438,45.37709149600005],[26.14896550000003,45.45715549800008],[26.072718500000065,45.50578799800007],[26.093334500000026,45.51630599600003],[26.212819,45.621830997000075],[26.310291500000062,45.606565997000075],[26.39209450000004,45.802389496000046],[26.37564499900003,45.90731349600003],[26.439984500000037,46.038876498000036],[26.324927,46.23116549700006],[26.263439,46.24633849600008],[26.29125350000004,46.33603499800006],[26.15608050000003,46.31515849900006],[26.165532,46.384739997000054],[25.995239999000034,46.40674749800007],[26.006634,46.48163849800005],[26.08134750000005,46.51633349800005],[25.975159500000075,46.69731249800003],[25.867373,46.649418997000055],[25.79531,46.71911749800006],[25.765381500000046,46.88445299700004],[25.86196750000005,46.92338349700003],[25.661582499000076,47.091646499000035],[25.539915,47.082194996000055],[25.425438,47.17119699800003],[25.305628,47.16538899600005],[25.246710500000063,47.09791899900006],[25.063383,47.13653849700006],[25.07958699900007,47.25862549800007],[25.028061500000035,47.286654499000065],[25.07738849900005,47.31147850000008],[25.069417,47.36992249700006],[24.982833500000027,47.42401949900005],[25.058275,47.41653199800004],[25.095044,47.516200996000066],[24.960837500000025,47.596850998000036],[25.022151500000064,47.66637799800003],[24.947100500000033,47.72912399900008],[25.115598999000042,47.754823498000064],[25.310894500000074,47.91432050000003],[26.098827500000027,47.97879599600003],[26.188041,47.99546449800005],[26.33267550000005,48.18580449600006],[26.63056,48.259749998000075],[26.852818,48.23365449600004],[27.164651500000048,47.99468349800003],[27.213884499000073,47.82219949800003],[27.391166500000054,47.58939749600006],[27.559885,47.46286199800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO21","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Nord-Est","NUTS_NAME":"Nord-Est"},"geometry":{"type":"Polygon","coordinates":[[[27.559885,47.46286199800005],[27.603923,47.328363498000044],[27.73602550000004,47.28789749600003],[27.915807,47.06719250000003],[28.09621,46.977461996000045],[28.11380550000007,46.838411],[28.252111,46.61181099700008],[28.221767500000055,46.50262349600007],[28.260886,46.43713949800008],[28.109676,46.242388498000025],[28.14290550000004,46.16414949800003],[28.115802,46.10782699600003],[27.610243,46.13624849800004],[27.62768250000005,46.06138249800006],[27.562554500000033,46.002876997000044],[27.503053,46.139620998000055],[27.492818500000055,46.159212998000044],[27.38783250000006,46.19381849600006],[27.350005500000066,46.15270549600007],[27.306986,46.20606399900004],[27.123597,46.147470998000074],[27.10561850000005,46.08868949600003],[26.439984500000037,46.038876498000036],[26.324927,46.23116549700006],[26.263439,46.24633849600008],[26.29125350000004,46.33603499800006],[26.15608050000003,46.31515849900006],[26.165532,46.384739997000054],[25.995239999000034,46.40674749800007],[26.006634,46.48163849800005],[26.08134750000005,46.51633349800005],[25.975159500000075,46.69731249800003],[25.867373,46.649418997000055],[25.79531,46.71911749800006],[25.765381500000046,46.88445299700004],[25.86196750000005,46.92338349700003],[25.661582499000076,47.091646499000035],[25.539915,47.082194996000055],[25.425438,47.17119699800003],[25.305628,47.16538899600005],[25.246710500000063,47.09791899900006],[25.063383,47.13653849700006],[25.07958699900007,47.25862549800007],[25.028061500000035,47.286654499000065],[25.07738849900005,47.31147850000008],[25.069417,47.36992249700006],[24.982833500000027,47.42401949900005],[25.058275,47.41653199800004],[25.095044,47.516200996000066],[24.960837500000025,47.596850998000036],[25.022151500000064,47.66637799800003],[24.947100500000033,47.72912399900008],[25.115598999000042,47.754823498000064],[25.310894500000074,47.91432050000003],[26.098827500000027,47.97879599600003],[26.188041,47.99546449800005],[26.33267550000005,48.18580449600006],[26.63056,48.259749998000075],[26.852818,48.23365449600004],[27.164651500000048,47.99468349800003],[27.213884499000073,47.82219949800003],[27.391166500000054,47.58939749600006],[27.559885,47.46286199800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO211","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Bacau","NUTS_NAME":"Bacau"},"geometry":{"type":"Polygon","coordinates":[[[27.206219,46.769848498000044],[27.326546500000063,46.69796099600006],[27.276358,46.62526549800003],[27.4644,46.370655498000076],[27.492818500000055,46.159212998000044],[27.38783250000006,46.19381849600006],[27.350005500000066,46.15270549600007],[27.306986,46.20606399900004],[27.123597,46.147470998000074],[27.10561850000005,46.08868949600003],[26.439984500000037,46.038876498000036],[26.324927,46.23116549700006],[26.263439,46.24633849600008],[26.29125350000004,46.33603499800006],[26.15608050000003,46.31515849900006],[26.165532,46.384739997000054],[25.995239999000034,46.40674749800007],[26.006634,46.48163849800005],[26.08134750000005,46.51633349800005],[25.975159500000075,46.69731249800003],[26.390468,46.65100449900007],[26.570231,46.678727998000056],[26.675947500000063,46.76768999800004],[26.786345,46.716898996000054],[26.909392500000024,46.82724349600005],[26.97865549900007,46.762641],[27.206219,46.769848498000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO212","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Botosani","NUTS_NAME":"Botosani"},"geometry":{"type":"Polygon","coordinates":[[[27.391166500000054,47.58939749600006],[27.24214550000005,47.52198099800006],[27.100016500000038,47.547901498000044],[27.04187950000005,47.438239500000066],[26.977343500000075,47.49620050000004],[26.700572,47.471711998000046],[26.58586850000006,47.46387099800006],[26.54375,47.64399349800004],[26.098827500000027,47.97879599600003],[26.188041,47.99546449800005],[26.33267550000005,48.18580449600006],[26.63056,48.259749998000075],[26.852818,48.23365449600004],[27.164651500000048,47.99468349800003],[27.213884499000073,47.82219949800003],[27.391166500000054,47.58939749600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO213","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Iasi","NUTS_NAME":"Iasi"},"geometry":{"type":"Polygon","coordinates":[[[28.11380550000007,46.838411],[27.941983,46.85516099800003],[27.895161,46.807225499000026],[27.790382500000078,46.94211449800008],[27.768514,46.876247500000034],[27.623251,46.99070499800007],[27.64409050000006,46.89765149800007],[27.585105500000054,46.87764599900004],[27.202391,46.88240999800007],[27.121949,46.95072699900004],[27.239941,47.011503496000046],[27.155912,47.08269199800003],[26.902529,47.02796299900007],[26.636329,47.09242599600003],[26.60708,47.19858499800006],[26.507789500000058,47.27169349700006],[26.550504,47.30983949800003],[26.492965500000025,47.39232399700006],[26.681763,47.36168599800004],[26.700572,47.471711998000046],[26.977343500000075,47.49620050000004],[27.04187950000005,47.438239500000066],[27.100016500000038,47.547901498000044],[27.24214550000005,47.52198099800006],[27.391166500000054,47.58939749600006],[27.559885,47.46286199800005],[27.603923,47.328363498000044],[27.73602550000004,47.28789749600003],[27.915807,47.06719250000003],[28.09621,46.977461996000045],[28.11380550000007,46.838411]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO214","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Neamt","NUTS_NAME":"Neamt"},"geometry":{"type":"Polygon","coordinates":[[[26.507789500000058,47.27169349700006],[26.60708,47.19858499800006],[26.636329,47.09242599600003],[26.902529,47.02796299900007],[27.155912,47.08269199800003],[27.239941,47.011503496000046],[27.121949,46.95072699900004],[27.202391,46.88240999800007],[27.206219,46.769848498000044],[26.97865549900007,46.762641],[26.909392500000024,46.82724349600005],[26.786345,46.716898996000054],[26.675947500000063,46.76768999800004],[26.570231,46.678727998000056],[26.390468,46.65100449900007],[25.975159500000075,46.69731249800003],[25.867373,46.649418997000055],[25.79531,46.71911749800006],[25.765381500000046,46.88445299700004],[25.86196750000005,46.92338349700003],[25.661582499000076,47.091646499000035],[25.811813500000028,47.259295],[25.764054,47.31779449800007],[26.053532,47.27487599800003],[26.436448,47.32730649800004],[26.507789500000058,47.27169349700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO215","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Suceava","NUTS_NAME":"Suceava"},"geometry":{"type":"Polygon","coordinates":[[[26.700572,47.471711998000046],[26.681763,47.36168599800004],[26.492965500000025,47.39232399700006],[26.550504,47.30983949800003],[26.507789500000058,47.27169349700006],[26.436448,47.32730649800004],[26.053532,47.27487599800003],[25.764054,47.31779449800007],[25.811813500000028,47.259295],[25.661582499000076,47.091646499000035],[25.539915,47.082194996000055],[25.425438,47.17119699800003],[25.305628,47.16538899600005],[25.246710500000063,47.09791899900006],[25.063383,47.13653849700006],[25.07958699900007,47.25862549800007],[25.028061500000035,47.286654499000065],[25.07738849900005,47.31147850000008],[25.069417,47.36992249700006],[24.982833500000027,47.42401949900005],[25.058275,47.41653199800004],[25.095044,47.516200996000066],[24.960837500000025,47.596850998000036],[25.022151500000064,47.66637799800003],[24.947100500000033,47.72912399900008],[25.115598999000042,47.754823498000064],[25.310894500000074,47.91432050000003],[26.098827500000027,47.97879599600003],[26.54375,47.64399349800004],[26.58586850000006,47.46387099800006],[26.700572,47.471711998000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO216","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Vaslui","NUTS_NAME":"Vaslui"},"geometry":{"type":"Polygon","coordinates":[[[28.11380550000007,46.838411],[28.252111,46.61181099700008],[28.221767500000055,46.50262349600007],[28.260886,46.43713949800008],[28.109676,46.242388498000025],[28.14290550000004,46.16414949800003],[28.115802,46.10782699600003],[27.610243,46.13624849800004],[27.62768250000005,46.06138249800006],[27.562554500000033,46.002876997000044],[27.503053,46.139620998000055],[27.492818500000055,46.159212998000044],[27.4644,46.370655498000076],[27.276358,46.62526549800003],[27.326546500000063,46.69796099600006],[27.206219,46.769848498000044],[27.202391,46.88240999800007],[27.585105500000054,46.87764599900004],[27.64409050000006,46.89765149800007],[27.623251,46.99070499800007],[27.768514,46.876247500000034],[27.790382500000078,46.94211449800008],[27.895161,46.807225499000026],[27.941983,46.85516099800003],[28.11380550000007,46.838411]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL325","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Rzeszowski","NUTS_NAME":"Rzeszowski"},"geometry":{"type":"Polygon","coordinates":[[[22.447906,50.12307549700006],[22.29612650000007,49.93373249900003],[22.23422750000003,49.93153899600003],[22.310238500000025,49.869822999000064],[22.294474500000035,49.74457999800006],[22.096148,49.83576799800005],[21.89479650000004,49.78109349600004],[21.66080050000005,49.84368949700007],[21.559855500000026,49.78611199800008],[21.47736750000007,49.840546],[21.523523,49.89291499800004],[21.464288,49.99362599800003],[21.567961,50.12979749600004],[21.511249,50.16153299600006],[21.611306,50.17852349900005],[21.610332,50.24922649600006],[21.530777500000056,50.293228498000076],[21.670649500000025,50.43696299900006],[21.75817950000004,50.38792399700003],[21.79726,50.405981498000074],[21.776439,50.449234997000076],[21.882174500000076,50.47288449800004],[21.851936,50.462288],[21.881088,50.39192049600007],[21.973452500000064,50.39032499800004],[21.998631500000045,50.323852498000065],[22.173827500000073,50.34865749800008],[22.19626,50.27338149800005],[22.447906,50.12307549700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL326","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Tarnobrzeski","NUTS_NAME":"Tarnobrzeski"},"geometry":{"type":"Polygon","coordinates":[[[22.519738,50.58323349800003],[22.5862,50.47734849700004],[22.443922500000042,50.46144449800005],[22.437193,50.39971749800003],[22.50713250000007,50.34399149800004],[22.687964500000078,50.356915498000035],[22.606575,50.325451],[22.638222,50.30307749600007],[22.523282,50.129020498000045],[22.447906,50.12307549700006],[22.19626,50.27338149800005],[22.173827500000073,50.34865749800008],[21.998631500000045,50.323852498000065],[21.973452500000064,50.39032499800004],[21.881088,50.39192049600007],[21.851936,50.462288],[21.882174500000076,50.47288449800004],[21.776439,50.449234997000076],[21.79726,50.405981498000074],[21.75817950000004,50.38792399700003],[21.670649500000025,50.43696299900006],[21.530777500000056,50.293228498000076],[21.610332,50.24922649600006],[21.611306,50.17852349900005],[21.511249,50.16153299600006],[21.567961,50.12979749600004],[21.464288,49.99362599800003],[21.523523,49.89291499800004],[21.47736750000007,49.840546],[21.297149500000046,49.84286399900003],[21.224959,49.85259849800008],[21.28472,49.924064498000064],[21.150439,49.97648449900004],[21.19502250000005,50.119844999000065],[21.146154500000023,50.13003999700004],[21.145727,50.25712050000004],[21.20883150000003,50.35489749800007],[21.60119650000007,50.51969549900008],[21.82088650000003,50.69158649900004],[21.864303500000062,50.80270649800008],[22.16296150000005,50.79725149800004],[22.205586,50.754013],[22.141785500000026,50.66969149700003],[22.519738,50.58323349800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL33","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Swietokrzyskie","NUTS_NAME":"Swietokrzyskie"},"geometry":{"type":"Polygon","coordinates":[[[20.54365750000005,51.231178999000065],[20.813019001000043,51.14583749800005],[20.920719500000075,51.19533749800007],[21.056556,51.15711399600008],[21.070030501000076,51.21142849800003],[21.152750500000025,51.08028599700003],[21.355528,51.085262499000066],[21.463566999000022,51.013111998000056],[21.630164500000035,51.06338899700006],[21.802998,51.07207899800005],[21.864303500000062,50.80270649800008],[21.82088650000003,50.69158649900004],[21.60119650000007,50.51969549900008],[21.20883150000003,50.35489749800007],[20.786962500000072,50.28838849700003],[20.681539,50.20587549600003],[20.41552050000007,50.19123749600004],[20.272868,50.46994499700003],[19.949966500000073,50.50478250000003],[19.803217,50.56555399900003],[19.922185499000022,50.639417999000045],[19.831074,50.654359],[19.871657500000026,50.69304550000004],[19.712949500000036,50.729156],[19.832163,50.81970099700004],[19.74706,50.86597],[19.845536,50.93020449800008],[19.882372,51.04766299700003],[20.03947050000005,50.99020649600004],[20.056235999000023,51.05774949800008],[19.98177050000004,51.07315549900005],[19.993847,51.183954],[20.371114,51.23663199600003],[20.366456500000027,51.31061599600008],[20.432815,51.33940499800008],[20.54365750000005,51.231178999000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL331","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Kielecki","NUTS_NAME":"Kielecki"},"geometry":{"type":"Polygon","coordinates":[[[21.630164500000035,51.06338899700006],[21.659561,51.00044199800004],[21.52105,50.83257149900004],[21.399207,50.889327998000056],[21.132894,50.82215149900003],[21.157554,50.647332498000026],[20.878705,50.66159949900003],[20.77452050000005,50.53648399900004],[20.641291500000023,50.55667049700003],[20.647868999000025,50.66097099900003],[20.03947050000005,50.99020649600004],[20.056235999000023,51.05774949800008],[19.98177050000004,51.07315549900005],[19.993847,51.183954],[20.371114,51.23663199600003],[20.366456500000027,51.31061599600008],[20.432815,51.33940499800008],[20.54365750000005,51.231178999000065],[20.813019001000043,51.14583749800005],[20.920719500000075,51.19533749800007],[21.056556,51.15711399600008],[21.070030501000076,51.21142849800003],[21.152750500000025,51.08028599700003],[21.355528,51.085262499000066],[21.463566999000022,51.013111998000056],[21.630164500000035,51.06338899700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL332","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Sandomiersko-jedrzejowski","NUTS_NAME":"Sandomiersko-jedrzejowski"},"geometry":{"type":"Polygon","coordinates":[[[21.864303500000062,50.80270649800008],[21.82088650000003,50.69158649900004],[21.60119650000007,50.51969549900008],[21.20883150000003,50.35489749800007],[20.786962500000072,50.28838849700003],[20.681539,50.20587549600003],[20.41552050000007,50.19123749600004],[20.272868,50.46994499700003],[19.949966500000073,50.50478250000003],[19.803217,50.56555399900003],[19.922185499000022,50.639417999000045],[19.831074,50.654359],[19.871657500000026,50.69304550000004],[19.712949500000036,50.729156],[19.832163,50.81970099700004],[19.74706,50.86597],[19.845536,50.93020449800008],[19.882372,51.04766299700003],[20.03947050000005,50.99020649600004],[20.647868999000025,50.66097099900003],[20.641291500000023,50.55667049700003],[20.77452050000005,50.53648399900004],[20.878705,50.66159949900003],[21.157554,50.647332498000026],[21.132894,50.82215149900003],[21.399207,50.889327998000056],[21.52105,50.83257149900004],[21.659561,51.00044199800004],[21.630164500000035,51.06338899700006],[21.802998,51.07207899800005],[21.864303500000062,50.80270649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL34","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Podlaskie","NUTS_NAME":"Podlaskie"},"geometry":{"type":"Polygon","coordinates":[[[23.588702,53.69592849600008],[23.798979,53.274188],[23.918259,53.15762199800008],[23.872502,53.083870997000076],[23.925377,53.024629],[23.916239,52.90481199900006],[23.93864,52.712916],[23.466628,52.54944799700007],[23.178338,52.28314099700003],[23.128409,52.28784149900008],[22.929204500000026,52.37359349800005],[22.580413500000077,52.39315799600007],[22.525242500000047,52.522721999000055],[22.408589,52.60968999800008],[22.473925,52.64772549800006],[22.435035999000036,52.69693949900005],[22.453771500000073,52.788238499000045],[22.33441399900005,52.74046399800005],[22.262360500000057,52.81536199900006],[22.30506450000007,52.83934799700006],[22.279745500000047,52.88493649900005],[22.00393150000008,52.87786599800006],[22.001760500000046,52.972328997000034],[21.694427500000074,53.138093997000055],[21.645086,53.28348049700003],[21.73542750000007,53.31272799800007],[21.59819991300003,53.48018197000005],[21.88084150000003,53.46127949800007],[22.135728,53.54453999700007],[22.70350250000007,53.76735449600005],[22.782806,53.915489498000056],[22.476556500000072,54.201305500000046],[22.768004,54.28944349900007],[22.792095500000073,54.363358996000045],[23.010273,54.38280599900003],[23.092209,54.298474499000065],[23.321498500000075,54.25332599700005],[23.485877500000072,54.15324399700006],[23.52863750000006,54.06579799800005],[23.482689,53.99711999900006],[23.51465,53.956559997000056],[23.588702,53.69592849600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL343","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Bialostocki","NUTS_NAME":"Bialostocki"},"geometry":{"type":"Polygon","coordinates":[[[23.588702,53.69592849600008],[23.798979,53.274188],[23.918259,53.15762199800008],[23.872502,53.083870997000076],[23.925377,53.024629],[23.916239,52.90481199900006],[23.49340550000005,52.99020899700008],[23.374987,52.965084500000046],[23.36849950000004,52.893124499000066],[22.937102,52.92508649800004],[22.758567500000026,52.84989649900007],[22.733524500000044,52.90323849600003],[22.788024,52.965940998000065],[22.761199500000032,53.00660249800006],[22.866527,53.07654350000007],[22.658378,53.14378299900005],[22.584047,53.09510049800008],[22.47971750000005,53.21343249900008],[22.70966150000004,53.277183998000055],[22.773812500000076,53.21388850000005],[23.109699,53.319621],[23.10929150000004,53.397635996000076],[23.05822550000005,53.40189249600007],[23.05778950000007,53.53545799700004],[22.957745500000044,53.60019699600008],[23.22647050000006,53.718437],[23.588702,53.69592849600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL344","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Lomzynski","NUTS_NAME":"Lomzynski"},"geometry":{"type":"Polygon","coordinates":[[[22.47971750000005,53.21343249900008],[22.584047,53.09510049800008],[22.658378,53.14378299900005],[22.866527,53.07654350000007],[22.761199500000032,53.00660249800006],[22.788024,52.965940998000065],[22.733524500000044,52.90323849600003],[22.758567500000026,52.84989649900007],[22.937102,52.92508649800004],[23.36849950000004,52.893124499000066],[23.374987,52.965084500000046],[23.49340550000005,52.99020899700008],[23.916239,52.90481199900006],[23.93864,52.712916],[23.466628,52.54944799700007],[23.178338,52.28314099700003],[23.128409,52.28784149900008],[22.929204500000026,52.37359349800005],[22.580413500000077,52.39315799600007],[22.525242500000047,52.522721999000055],[22.408589,52.60968999800008],[22.473925,52.64772549800006],[22.435035999000036,52.69693949900005],[22.453771500000073,52.788238499000045],[22.33441399900005,52.74046399800005],[22.262360500000057,52.81536199900006],[22.30506450000007,52.83934799700006],[22.279745500000047,52.88493649900005],[22.00393150000008,52.87786599800006],[22.001760500000046,52.972328997000034],[21.694427500000074,53.138093997000055],[21.645086,53.28348049700003],[21.73542750000007,53.31272799800007],[21.59819991300003,53.48018197000005],[21.88084150000003,53.46127949800007],[22.135728,53.54453999700007],[22.23909,53.53975399800004],[22.266728,53.502485499000045],[22.230944,53.440783999000075],[22.296445,53.40082599800007],[22.497577500000034,53.34859049900007],[22.430343,53.21771399700003],[22.47971750000005,53.21343249900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL345","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Suwalski","NUTS_NAME":"Suwalski"},"geometry":{"type":"Polygon","coordinates":[[[23.321498500000075,54.25332599700005],[23.485877500000072,54.15324399700006],[23.52863750000006,54.06579799800005],[23.482689,53.99711999900006],[23.51465,53.956559997000056],[23.588702,53.69592849600008],[23.22647050000006,53.718437],[22.957745500000044,53.60019699600008],[23.05778950000007,53.53545799700004],[23.05822550000005,53.40189249600007],[23.10929150000004,53.397635996000076],[23.109699,53.319621],[22.773812500000076,53.21388850000005],[22.70966150000004,53.277183998000055],[22.47971750000005,53.21343249900008],[22.430343,53.21771399700003],[22.497577500000034,53.34859049900007],[22.296445,53.40082599800007],[22.230944,53.440783999000075],[22.266728,53.502485499000045],[22.23909,53.53975399800004],[22.135728,53.54453999700007],[22.70350250000007,53.76735449600005],[22.782806,53.915489498000056],[22.476556500000072,54.201305500000046],[22.768004,54.28944349900007],[22.792095500000073,54.363358996000045],[23.010273,54.38280599900003],[23.092209,54.298474499000065],[23.321498500000075,54.25332599700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL4","LEVL_CODE":1,"CNTR_CODE":"PL","NAME_LATN":"REGION PÓLNOCNO-ZACHODNI","NUTS_NAME":"REGION PÓLNOCNO-ZACHODNI"},"geometry":{"type":"Polygon","coordinates":[[[16.804609500000026,54.11607299800005],[16.792764500000033,53.985550499000055],[16.982053499000074,53.90490949900004],[16.872407,53.86866299900004],[16.945877499000062,53.802610997000045],[16.857653500000026,53.74709499800008],[16.892248,53.655868998000074],[16.999648999000044,53.51806250000004],[17.390653,53.49096399900003],[17.247740500000077,53.379893999000046],[17.43879350000003,53.26751450000006],[17.321557,53.216272497000034],[17.392802500000073,53.14195699900006],[17.30155650000006,52.99480299900006],[17.509582500000022,52.917767996000066],[17.503342,52.78672549600003],[17.40436550000004,52.756718],[17.458523,52.738948],[17.489669500000048,52.72729399800005],[17.467201500000044,52.68038399800008],[17.627635499000064,52.70817999900004],[17.678062,52.60239749800007],[17.786666500000024,52.65151549800004],[18.204579,52.48316999900004],[18.377150500000027,52.53746350000006],[18.391668500000037,52.470221999000046],[18.650178500000038,52.45026549800008],[18.755498500000044,52.33276950000004],[18.928907,52.37807699800004],[19.04712,52.33280399800003],[19.03116650000004,52.269952998000065],[19.100311500000032,52.26592099800007],[19.08975,52.20681199900008],[18.924092,52.19626799800005],[18.943973500000027,52.10391899900003],[18.827809,52.064188],[18.71785950000003,52.065705998000055],[18.767948,51.92784399800007],[18.685665500000027,51.82100299700005],[18.471965500000067,51.85100549800006],[18.355307500000038,51.66052499700004],[18.382492,51.47685750000005],[18.326309500000036,51.419816999000034],[18.211442500000032,51.451917998000056],[18.195381500000053,51.39011499600008],[18.07452150000006,51.349915],[18.110033,51.260214],[18.17038,51.267563996000035],[18.163681500000052,51.17251699600007],[17.939456,51.10886499800006],[17.82347850000002,51.117894498000055],[17.851184,51.17243899700003],[17.795269500000074,51.194146],[17.744111,51.24296350000003],[17.764996,51.31638749800004],[17.70442299900003,51.36684949600004],[17.729088,51.39570149700006],[17.540565999000023,51.421283],[17.51216,51.50550399700006],[17.576286,51.539581498000075],[17.556245,51.58430149800006],[17.257433,51.642843],[17.125499,51.56531750000005],[16.828372,51.57218799800006],[16.783478500000058,51.64098699900006],[16.639154500000075,51.66602949900005],[16.647939,51.74416199800004],[16.416191,51.784864999000035],[16.269073,51.68447149800005],[15.979245,51.802309],[15.963185,51.733663997000065],[15.869526500000063,51.71966199800005],[15.823620500000061,51.57274349700003],[15.71403250000003,51.51739699600006],[15.599760500000059,51.44567449800007],[15.406762,51.52037099800003],[15.324932500000045,51.41849599900007],[15.190489,51.46392199600007],[14.974183,51.36394999600003],[14.948293,51.47176899800007],[14.735769,51.52595199900003],[14.729862,51.581776998000066],[14.752694,51.66709599700005],[14.590146,51.82102599800004],[14.693557,51.901287996000065],[14.716716,52.001188],[14.755227,52.070024997000075],[14.682174,52.11468899800008],[14.716044,52.235458],[14.600891500000046,52.27205199800005],[14.534361999000055,52.39500799900003],[14.635373,52.49636849900003],[14.60426,52.53348949800005],[14.639691,52.57157449700003],[14.565063,52.62449699900003],[14.436438,52.67990049800005],[14.127624,52.83166249900006],[14.156692,52.895590998000046],[14.143658,52.96136849900006],[14.357842,53.06766699800005],[14.377268,53.20175199800008],[14.441223,53.25435999900003],[14.412157,53.32963599700008],[14.267542,53.697806496000055],[14.267567084000063,53.697836718000076],[14.302757,53.736885],[14.595727,53.591762499000026],[14.619581500000038,53.64978399800003],[14.539019500000052,53.70261],[14.610576500000036,53.76369099800007],[14.573513,53.84919699800008],[14.270650390000071,53.880213135000076],[14.28143566700004,53.85638231300004],[14.286778500000025,53.84457399900003],[14.336749,53.80701849800005],[14.213077500000054,53.86647949800005],[14.226302,53.92865299700003],[14.767616,54.02293799900008],[14.69569,53.99610899600003],[14.620618148000062,53.85057817500007],[14.757152500000075,53.925437497000075],[14.80588350000005,54.02959049800006],[14.727757500000052,54.022860997000066],[15.388074500000073,54.15890699900007],[16.102422500000046,54.27442849900007],[16.472283,54.51224349900008],[16.699085,54.569247],[16.85862050000003,54.38257649700006],[16.813893500000063,54.318491],[16.865116499000067,54.260984998000026],[16.711381,54.21400299900006],[16.804609500000026,54.11607299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL41","LEVL_CODE":2,"CNTR_CODE":"PL","NAME_LATN":"Wielkopolskie","NUTS_NAME":"Wielkopolskie"},"geometry":{"type":"Polygon","coordinates":[[[17.390653,53.49096399900003],[17.247740500000077,53.379893999000046],[17.43879350000003,53.26751450000006],[17.321557,53.216272497000034],[17.392802500000073,53.14195699900006],[17.30155650000006,52.99480299900006],[17.509582500000022,52.917767996000066],[17.503342,52.78672549600003],[17.40436550000004,52.756718],[17.458523,52.738948],[17.489669500000048,52.72729399800005],[17.467201500000044,52.68038399800008],[17.627635499000064,52.70817999900004],[17.678062,52.60239749800007],[17.786666500000024,52.65151549800004],[18.204579,52.48316999900004],[18.377150500000027,52.53746350000006],[18.391668500000037,52.470221999000046],[18.650178500000038,52.45026549800008],[18.755498500000044,52.33276950000004],[18.928907,52.37807699800004],[19.04712,52.33280399800003],[19.03116650000004,52.269952998000065],[19.100311500000032,52.26592099800007],[19.08975,52.20681199900008],[18.924092,52.19626799800005],[18.943973500000027,52.10391899900003],[18.827809,52.064188],[18.71785950000003,52.065705998000055],[18.767948,51.92784399800007],[18.685665500000027,51.82100299700005],[18.471965500000067,51.85100549800006],[18.355307500000038,51.66052499700004],[18.382492,51.47685750000005],[18.326309500000036,51.419816999000034],[18.211442500000032,51.451917998000056],[18.195381500000053,51.39011499600008],[18.07452150000006,51.349915],[18.110033,51.260214],[18.17038,51.267563996000035],[18.163681500000052,51.17251699600007],[17.939456,51.10886499800006],[17.82347850000002,51.117894498000055],[17.851184,51.17243899700003],[17.795269500000074,51.194146],[17.744111,51.24296350000003],[17.764996,51.31638749800004],[17.70442299900003,51.36684949600004],[17.729088,51.39570149700006],[17.540565999000023,51.421283],[17.51216,51.50550399700006],[17.576286,51.539581498000075],[17.556245,51.58430149800006],[17.257433,51.642843],[17.125499,51.56531750000005],[16.828372,51.57218799800006],[16.783478500000058,51.64098699900006],[16.639154500000075,51.66602949900005],[16.647939,51.74416199800004],[16.416191,51.784864999000035],[16.284856,51.90001549600004],[16.09856750000006,51.89378650000003],[16.137304500000027,51.92695199700006],[16.122916,51.99312899800003],[16.010617,51.98142249800003],[15.967463500000065,52.08194850000007],[15.833296,52.111123],[15.880810499000063,52.29042949800004],[15.900337500000035,52.39086949600005],[15.804233,52.44419749700006],[15.891571500000055,52.47315549800004],[15.776346500000045,52.63786950000008],[15.793263500000023,52.70692799600005],[15.946907,52.75489599700006],[15.923608500000057,52.83117850000008],[15.979305,52.87763099600005],[15.96249750000004,53.04138099600004],[16.3173,53.043952499000056],[16.405317500000024,53.158510499000045],[16.696911500000056,53.30104599900005],[16.475250500000072,53.387784999000075],[16.45421,53.48847550000005],[16.645440999000073,53.48463299600007],[16.758229500000027,53.62412199600004],[16.892248,53.655868998000074],[16.999648999000044,53.51806250000004],[17.390653,53.49096399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL411","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Pilski","NUTS_NAME":"Pilski"},"geometry":{"type":"Polygon","coordinates":[[[17.390653,53.49096399900003],[17.247740500000077,53.379893999000046],[17.43879350000003,53.26751450000006],[17.321557,53.216272497000034],[17.392802500000073,53.14195699900006],[17.30155650000006,52.99480299900006],[17.509582500000022,52.917767996000066],[17.503342,52.78672549600003],[17.40436550000004,52.756718],[17.458523,52.738948],[17.15143850000004,52.57397199800005],[17.031056500000034,52.67916849800008],[17.105797,52.68863699900004],[17.106119,52.78098249800007],[16.789004,52.882438498000056],[16.701527500000054,52.854201998000065],[16.798223,52.795091496000055],[16.738486,52.72643999600007],[16.18502650000005,52.79405699700004],[16.013770500000078,52.73472799800004],[15.946907,52.75489599700006],[15.923608500000057,52.83117850000008],[15.979305,52.87763099600005],[15.96249750000004,53.04138099600004],[16.3173,53.043952499000056],[16.405317500000024,53.158510499000045],[16.696911500000056,53.30104599900005],[16.475250500000072,53.387784999000075],[16.45421,53.48847550000005],[16.645440999000073,53.48463299600007],[16.758229500000027,53.62412199600004],[16.892248,53.655868998000074],[16.999648999000044,53.51806250000004],[17.390653,53.49096399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL414","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Koninski","NUTS_NAME":"Koninski"},"geometry":{"type":"Polygon","coordinates":[[[18.377150500000027,52.53746350000006],[18.391668500000037,52.470221999000046],[18.650178500000038,52.45026549800008],[18.755498500000044,52.33276950000004],[18.928907,52.37807699800004],[19.04712,52.33280399800003],[19.03116650000004,52.269952998000065],[19.100311500000032,52.26592099800007],[19.08975,52.20681199900008],[18.924092,52.19626799800005],[18.943973500000027,52.10391899900003],[18.827809,52.064188],[18.71785950000003,52.065705998000055],[18.767948,51.92784399800007],[18.685665500000027,51.82100299700005],[18.471965500000067,51.85100549800006],[18.254606,52.018327499000065],[18.105045499000028,51.97573999900004],[17.941301499000076,52.001273998000045],[17.96738299900005,52.075743997000075],[17.514385,52.12413650000008],[17.425605500000074,52.176485498000034],[17.488138500000048,52.26827999800008],[17.364883500000076,52.35150199800006],[17.373466,52.44971],[17.320399500000065,52.50372499800005],[17.35702850000007,52.530999500000064],[17.15143850000004,52.57397199800005],[17.458523,52.738948],[17.489669500000048,52.72729399800005],[17.467201500000044,52.68038399800008],[17.627635499000064,52.70817999900004],[17.678062,52.60239749800007],[17.786666500000024,52.65151549800004],[18.204579,52.48316999900004],[18.377150500000027,52.53746350000006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL415","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Miasto Poznan","NUTS_NAME":"Miasto Poznan"},"geometry":{"type":"Polygon","coordinates":[[[17.04917350000005,52.34201899800007],[16.995869500000026,52.29192549900006],[16.841993500000058,52.34423449900004],[16.778012043000047,52.46415434000005],[16.970175,52.508523998000044],[16.975661,52.441485500000056],[17.068946,52.42513199800004],[17.04917350000005,52.34201899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL416","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Kaliski","NUTS_NAME":"Kaliski"},"geometry":{"type":"Polygon","coordinates":[[[18.471965500000067,51.85100549800006],[18.355307500000038,51.66052499700004],[18.382492,51.47685750000005],[18.326309500000036,51.419816999000034],[18.211442500000032,51.451917998000056],[18.195381500000053,51.39011499600008],[18.07452150000006,51.349915],[18.110033,51.260214],[18.17038,51.267563996000035],[18.163681500000052,51.17251699600007],[17.939456,51.10886499800006],[17.82347850000002,51.117894498000055],[17.851184,51.17243899700003],[17.795269500000074,51.194146],[17.744111,51.24296350000003],[17.764996,51.31638749800004],[17.70442299900003,51.36684949600004],[17.729088,51.39570149700006],[17.540565999000023,51.421283],[17.51216,51.50550399700006],[17.576286,51.539581498000075],[17.556245,51.58430149800006],[17.257433,51.642843],[17.128032500000074,51.716147497000065],[17.18052350000005,51.71727049800006],[17.152748,51.74595549800006],[17.18029350000006,51.79144799800008],[17.322547,51.761276499000076],[17.336786500000073,51.88560399800008],[17.188619,51.96749949800005],[17.462667500000066,52.03418749800005],[17.514385,52.12413650000008],[17.96738299900005,52.075743997000075],[17.941301499000076,52.001273998000045],[18.105045499000028,51.97573999900004],[18.254606,52.018327499000065],[18.471965500000067,51.85100549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL417","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Leszczynski","NUTS_NAME":"Leszczynski"},"geometry":{"type":"Polygon","coordinates":[[[16.013770500000078,52.73472799800004],[16.319151,52.64880199600003],[16.186702500000024,52.49928599800006],[16.438341500000035,52.40573599700008],[16.513602,52.274482498000054],[16.602638,52.272140498000056],[16.602994,52.171081],[16.779712,52.20428799800004],[16.790251,52.140795999000034],[17.036542500000053,51.94164999800006],[17.188619,51.96749949800005],[17.336786500000073,51.88560399800008],[17.322547,51.761276499000076],[17.18029350000006,51.79144799800008],[17.152748,51.74595549800006],[17.18052350000005,51.71727049800006],[17.128032500000074,51.716147497000065],[17.257433,51.642843],[17.125499,51.56531750000005],[16.828372,51.57218799800006],[16.783478500000058,51.64098699900006],[16.639154500000075,51.66602949900005],[16.647939,51.74416199800004],[16.416191,51.784864999000035],[16.284856,51.90001549600004],[16.09856750000006,51.89378650000003],[16.137304500000027,51.92695199700006],[16.122916,51.99312899800003],[16.010617,51.98142249800003],[15.967463500000065,52.08194850000007],[15.833296,52.111123],[15.880810499000063,52.29042949800004],[15.900337500000035,52.39086949600005],[15.804233,52.44419749700006],[15.891571500000055,52.47315549800004],[15.776346500000045,52.63786950000008],[15.793263500000023,52.70692799600005],[15.946907,52.75489599700006],[16.013770500000078,52.73472799800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"PL418","LEVL_CODE":3,"CNTR_CODE":"PL","NAME_LATN":"Poznanski","NUTS_NAME":"Poznanski"},"geometry":{"type":"Polygon","coordinates":[[[17.15143850000004,52.57397199800005],[17.35702850000007,52.530999500000064],[17.320399500000065,52.50372499800005],[17.373466,52.44971],[17.364883500000076,52.35150199800006],[17.488138500000048,52.26827999800008],[17.425605500000074,52.176485498000034],[17.514385,52.12413650000008],[17.462667500000066,52.03418749800005],[17.188619,51.96749949800005],[17.036542500000053,51.94164999800006],[16.790251,52.140795999000034],[16.779712,52.20428799800004],[16.602994,52.171081],[16.602638,52.272140498000056],[16.513602,52.274482498000054],[16.438341500000035,52.40573599700008],[16.186702500000024,52.49928599800006],[16.319151,52.64880199600003],[16.013770500000078,52.73472799800004],[16.18502650000005,52.79405699700004],[16.738486,52.72643999600007],[16.798223,52.795091496000055],[16.701527500000054,52.854201998000065],[16.789004,52.882438498000056],[17.106119,52.78098249800007],[17.105797,52.68863699900004],[17.031056500000034,52.67916849800008],[17.15143850000004,52.57397199800005]],[[17.04917350000005,52.34201899800007],[17.068946,52.42513199800004],[16.975661,52.441485500000056],[16.970175,52.508523998000044],[16.778012043000047,52.46415434000005],[16.841993500000058,52.34423449900004],[16.995869500000026,52.29192549900006],[17.04917350000005,52.34201899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE33","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Övre Norrland","NUTS_NAME":"Övre Norrland"},"geometry":{"type":"Polygon","coordinates":[[[22.844611,68.38334150000003],[23.143492500000036,68.24682799900006],[23.15471750100005,68.136291],[23.652348999000026,67.958793],[23.494326001000047,67.86402599900003],[23.477203,67.70166250000005],[23.55897800100007,67.61996149800007],[23.39402350000006,67.48582099800007],[23.764582500000074,67.428214],[23.722462,67.28252899800003],[23.560987999000076,67.26085349800007],[23.55127200100003,67.17146199900003],[23.995160501000043,66.81970899800007],[23.879574500000047,66.76324399900005],[23.907815500000027,66.72142649900007],[23.856645,66.55690399900004],[23.650217501000043,66.45498399900004],[23.64559950000006,66.30140849900005],[23.918210499000054,66.16141],[24.155129,65.81602699900003],[23.47538650100006,65.84518749800003],[23.402833500000042,65.75004549800008],[23.217332500000055,65.82339349900008],[23.099601500000063,65.71327249700005],[22.671455,65.90403199800005],[22.617905,65.90125599900006],[22.690188500000033,65.86531349800003],[22.699252999000066,65.75771999800003],[22.340766001000077,65.86896049800004],[22.310333,65.78256899900003],[22.369043,65.66880349700006],[22.232683500000064,65.65303649700007],[22.37649,65.63100049800005],[22.294381500000043,65.60182449900003],[22.402720499000054,65.53735549600003],[21.831895001000078,65.52277549900003],[22.035621500000047,65.46488549900005],[21.768649500000038,65.40010249900007],[21.82642050000004,65.34806849800003],[21.44599050000005,65.38320899900003],[21.63175399900007,65.33305449900007],[21.634512,65.24373449800004],[21.750894,65.19125699900007],[21.524723359000063,65.23180558500007],[21.589963501000057,65.15028449900007],[21.506678500000078,65.14285999800006],[21.54360450000007,65.06305899700004],[21.23190150000005,64.94453999900003],[21.25731150000007,64.92216599700004],[21.156021,64.81548449900004],[20.997129,64.83223249600007],[21.297873,64.75350349800004],[21.21348050000006,64.73940199900005],[21.28227,64.66163399700008],[21.17361950000003,64.69389299900007],[21.60536250000007,64.43396099900008],[21.00002850000004,64.159882],[20.765231999000036,63.86661399900004],[20.560563,63.80467549700006],[20.574595500000044,63.72209999800003],[20.494180999000037,63.836441499000045],[20.396852,63.67203849900005],[20.305153500000074,63.78511399700005],[20.25163851700006,63.704674295000075],[20.300421500000027,63.74921249700003],[20.31427,63.67404250000004],[19.897695500000054,63.61702299800004],[19.683038999000075,63.434309],[19.449868500000036,63.57309199900004],[19.422997500000065,63.521906999000066],[19.486668500000064,63.41935599900006],[19.286468,63.469719499000064],[18.95877050000007,63.761131498000054],[18.485727500000053,63.84251799700007],[18.406906,63.92540749700004],[18.473282,63.96616399700008],[18.405653,63.99566249900005],[17.731568999000046,63.879935999000054],[17.261841,63.91410049600006],[16.718954,64.02787799800006],[16.219421500000067,64.26332049900003],[16.040148,64.24841299600007],[16.023498499000027,64.33295450000003],[15.740346,64.40313699900008],[15.704471500000068,64.53781899900008],[15.578810500000031,64.44361849900008],[15.239639,64.63615399900004],[15.334211500000038,64.66961650000007],[14.325985,65.118916],[14.378797500000076,65.24762049800006],[14.50683250000003,65.30972899800008],[14.49877250000003,65.52129749600005],[14.625476500000048,65.81180849900005],[14.516289,66.13257899900003],[15.484732,66.28245849800004],[15.453994500000022,66.34523350000006],[15.377226500000063,66.48430399600005],[16.387759,67.04546249900005],[16.404099501000076,67.20497449900006],[16.089829,67.43527850000004],[16.15800150000007,67.519159],[16.407672,67.53409999900003],[16.738120500000036,67.91420949800005],[17.281524,68.11881499900005],[17.899761500000068,67.96937199900003],[18.151354,68.19878999900004],[18.101093001000038,68.406045],[18.125925,68.53651599800008],[18.405688500000053,68.58187699900003],[19.921397,68.356013],[20.226541500000053,68.49081250000006],[19.93750250000005,68.55793949900004],[20.33587250000005,68.80231249900004],[20.306589,68.92618199900005],[20.060047500000053,69.045759],[20.548636500000043,69.05996850000008],[21.88894300100003,68.58438349900007],[22.844611,68.38334150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE331","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Västerbottens län","NUTS_NAME":"Västerbottens län"},"geometry":{"type":"Polygon","coordinates":[[[19.243149,65.21990949900004],[19.39203850100006,65.14297499900005],[19.894382499000073,65.38066849900008],[21.54360450000007,65.06305899700004],[21.23190150000005,64.94453999900003],[21.25731150000007,64.92216599700004],[21.156021,64.81548449900004],[20.997129,64.83223249600007],[21.297873,64.75350349800004],[21.21348050000006,64.73940199900005],[21.28227,64.66163399700008],[21.17361950000003,64.69389299900007],[21.60536250000007,64.43396099900008],[21.00002850000004,64.159882],[20.765231999000036,63.86661399900004],[20.560563,63.80467549700006],[20.574595500000044,63.72209999800003],[20.494180999000037,63.836441499000045],[20.396852,63.67203849900005],[20.305153500000074,63.78511399700005],[20.25163851700006,63.704674295000075],[20.300421500000027,63.74921249700003],[20.31427,63.67404250000004],[19.897695500000054,63.61702299800004],[19.683038999000075,63.434309],[19.449868500000036,63.57309199900004],[19.422997500000065,63.521906999000066],[19.486668500000064,63.41935599900006],[19.286468,63.469719499000064],[18.95877050000007,63.761131498000054],[18.485727500000053,63.84251799700007],[18.406906,63.92540749700004],[18.473282,63.96616399700008],[18.405653,63.99566249900005],[17.731568999000046,63.879935999000054],[17.261841,63.91410049600006],[16.718954,64.02787799800006],[16.219421500000067,64.26332049900003],[16.040148,64.24841299600007],[16.023498499000027,64.33295450000003],[15.740346,64.40313699900008],[15.704471500000068,64.53781899900008],[15.578810500000031,64.44361849900008],[15.239639,64.63615399900004],[15.334211500000038,64.66961650000007],[14.325985,65.118916],[14.378797500000076,65.24762049800006],[14.50683250000003,65.30972899800008],[14.49877250000003,65.52129749600005],[14.625476500000048,65.81180849900005],[14.516289,66.13257899900003],[15.484732,66.28245849800004],[15.453994500000022,66.34523350000006],[16.773557500000038,66.03124999900007],[17.674654,65.657028],[19.273641500000053,65.24147049900006],[19.243149,65.21990949900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE332","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Norrbottens län","NUTS_NAME":"Norrbottens län"},"geometry":{"type":"Polygon","coordinates":[[[21.88894300100003,68.58438349900007],[22.844611,68.38334150000003],[23.143492500000036,68.24682799900006],[23.15471750100005,68.136291],[23.652348999000026,67.958793],[23.494326001000047,67.86402599900003],[23.477203,67.70166250000005],[23.55897800100007,67.61996149800007],[23.39402350000006,67.48582099800007],[23.764582500000074,67.428214],[23.722462,67.28252899800003],[23.560987999000076,67.26085349800007],[23.55127200100003,67.17146199900003],[23.995160501000043,66.81970899800007],[23.879574500000047,66.76324399900005],[23.907815500000027,66.72142649900007],[23.856645,66.55690399900004],[23.650217501000043,66.45498399900004],[23.64559950000006,66.30140849900005],[23.918210499000054,66.16141],[24.155129,65.81602699900003],[23.47538650100006,65.84518749800003],[23.402833500000042,65.75004549800008],[23.217332500000055,65.82339349900008],[23.099601500000063,65.71327249700005],[22.671455,65.90403199800005],[22.617905,65.90125599900006],[22.690188500000033,65.86531349800003],[22.699252999000066,65.75771999800003],[22.340766001000077,65.86896049800004],[22.310333,65.78256899900003],[22.369043,65.66880349700006],[22.232683500000064,65.65303649700007],[22.37649,65.63100049800005],[22.294381500000043,65.60182449900003],[22.402720499000054,65.53735549600003],[21.831895001000078,65.52277549900003],[22.035621500000047,65.46488549900005],[21.768649500000038,65.40010249900007],[21.82642050000004,65.34806849800003],[21.44599050000005,65.38320899900003],[21.63175399900007,65.33305449900007],[21.634512,65.24373449800004],[21.750894,65.19125699900007],[21.524723359000063,65.23180558500007],[21.589963501000057,65.15028449900007],[21.506678500000078,65.14285999800006],[21.54360450000007,65.06305899700004],[19.894382499000073,65.38066849900008],[19.39203850100006,65.14297499900005],[19.243149,65.21990949900004],[19.273641500000053,65.24147049900006],[17.674654,65.657028],[16.773557500000038,66.03124999900007],[15.453994500000022,66.34523350000006],[15.377226500000063,66.48430399600005],[16.387759,67.04546249900005],[16.404099501000076,67.20497449900006],[16.089829,67.43527850000004],[16.15800150000007,67.519159],[16.407672,67.53409999900003],[16.738120500000036,67.91420949800005],[17.281524,68.11881499900005],[17.899761500000068,67.96937199900003],[18.151354,68.19878999900004],[18.101093001000038,68.406045],[18.125925,68.53651599800008],[18.405688500000053,68.58187699900003],[19.921397,68.356013],[20.226541500000053,68.49081250000006],[19.93750250000005,68.55793949900004],[20.33587250000005,68.80231249900004],[20.306589,68.92618199900005],[20.060047500000053,69.045759],[20.548636500000043,69.05996850000008],[21.88894300100003,68.58438349900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI","LEVL_CODE":0,"CNTR_CODE":"SI","NAME_LATN":"SLOVENIJA","NUTS_NAME":"SLOVENIJA"},"geometry":{"type":"Polygon","coordinates":[[[16.596805,46.47590249800004],[16.38205350000004,46.551329999000075],[16.24214850000004,46.49007549600003],[16.301549,46.37828799600004],[16.088382999000032,46.388544499000034],[15.876868500000057,46.28005549800008],[15.791475500000047,46.259327496000026],[15.775107500000047,46.211418998000056],[15.647185500000035,46.21717399700003],[15.59612450000003,46.14635599800005],[15.627262,46.08595349800004],[15.732483,46.04452549900003],[15.706404500000076,45.97534299800003],[15.67752,45.86888799900004],[15.708988500000032,45.84641299900005],[15.404425,45.79271699600008],[15.331290500000023,45.76285499800008],[15.256749,45.718848999000045],[15.319616,45.674262498000076],[15.357104,45.71397099700005],[15.406294,45.647968],[15.27705050000003,45.604461498000035],[15.385532,45.48658049800008],[15.226437500000031,45.42708799700006],[14.932349,45.529082],[14.823365500000023,45.46070999800003],[14.570012500000075,45.67294449600007],[14.436836499000037,45.51059299900004],[14.118196500000067,45.48123899600006],[14.109059,45.48245849600005],[14.011006500000065,45.51988049900007],[13.962175500000058,45.50601349600004],[13.984909500000072,45.45602199800004],[13.887097500000039,45.425872998000045],[13.583067,45.47740999600006],[13.601972,45.537027496000064],[13.746271999000044,45.56440299800005],[13.722823500000061,45.59472549600008],[13.850761500000033,45.58493249600008],[13.918656500000054,45.63351749800006],[13.795683500000052,45.74320449900006],[13.596243,45.80793750000004],[13.597145,45.81952250000006],[13.575802500000066,45.847736498000074],[13.643036,45.98270199800004],[13.526538,45.97231849800005],[13.476164999000048,46.00608149800007],[13.496938999000065,46.05133499800007],[13.664347500000076,46.177549998000075],[13.423357,46.206053997000026],[13.375492500000064,46.29823249800006],[13.684032,46.43747250000007],[13.714184999000054,46.522703499000045],[14.43450050000007,46.44294350000007],[14.565175500000066,46.37245349600005],[14.674580500000047,46.450687499000026],[14.818606,46.50938299900008],[14.879666,46.61124799800007],[15.065120500000035,46.65211249600003],[15.40197949900005,46.65354849800008],[15.508846500000061,46.616227998000056],[15.649988,46.70575699900007],[15.786422,46.70746949900007],[16.038086,46.65614549600008],[15.984746500000028,46.752207],[15.996236,46.835398499000064],[16.113849,46.86906799800005],[16.291385500000047,46.87283350000007],[16.347822500000063,46.84051750000003],[16.31216150000006,46.778002498000035],[16.370793500000048,46.72224349800007],[16.42654550000003,46.69493699800006],[16.39033150000006,46.63820750000008],[16.596805,46.47590249800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI0","LEVL_CODE":1,"CNTR_CODE":"SI","NAME_LATN":"SLOVENIJA","NUTS_NAME":"SLOVENIJA"},"geometry":{"type":"Polygon","coordinates":[[[16.42654550000003,46.69493699800006],[16.39033150000006,46.63820750000008],[16.596805,46.47590249800004],[16.38205350000004,46.551329999000075],[16.24214850000004,46.49007549600003],[16.301549,46.37828799600004],[16.088382999000032,46.388544499000034],[15.876868500000057,46.28005549800008],[15.791475500000047,46.259327496000026],[15.775107500000047,46.211418998000056],[15.647185500000035,46.21717399700003],[15.59612450000003,46.14635599800005],[15.627262,46.08595349800004],[15.732483,46.04452549900003],[15.706404500000076,45.97534299800003],[15.67752,45.86888799900004],[15.708988500000032,45.84641299900005],[15.404425,45.79271699600008],[15.331290500000023,45.76285499800008],[15.256749,45.718848999000045],[15.319616,45.674262498000076],[15.357104,45.71397099700005],[15.406294,45.647968],[15.27705050000003,45.604461498000035],[15.385532,45.48658049800008],[15.226437500000031,45.42708799700006],[14.932349,45.529082],[14.823365500000023,45.46070999800003],[14.570012500000075,45.67294449600007],[14.436836499000037,45.51059299900004],[14.118196500000067,45.48123899600006],[14.109059,45.48245849600005],[14.011006500000065,45.51988049900007],[13.962175500000058,45.50601349600004],[13.984909500000072,45.45602199800004],[13.887097500000039,45.425872998000045],[13.583067,45.47740999600006],[13.601972,45.537027496000064],[13.746271999000044,45.56440299800005],[13.722823500000061,45.59472549600008],[13.850761500000033,45.58493249600008],[13.918656500000054,45.63351749800006],[13.795683500000052,45.74320449900006],[13.596243,45.80793750000004],[13.597145,45.81952250000006],[13.575802500000066,45.847736498000074],[13.643036,45.98270199800004],[13.526538,45.97231849800005],[13.476164999000048,46.00608149800007],[13.496938999000065,46.05133499800007],[13.664347500000076,46.177549998000075],[13.423357,46.206053997000026],[13.375492500000064,46.29823249800006],[13.684032,46.43747250000007],[13.714184999000054,46.522703499000045],[14.43450050000007,46.44294350000007],[14.565175500000066,46.37245349600005],[14.674580500000047,46.450687499000026],[14.818606,46.50938299900008],[14.879666,46.61124799800007],[15.065120500000035,46.65211249600003],[15.40197949900005,46.65354849800008],[15.508846500000061,46.616227998000056],[15.649988,46.70575699900007],[15.786422,46.70746949900007],[16.038086,46.65614549600008],[15.984746500000028,46.752207],[15.996236,46.835398499000064],[16.113849,46.86906799800005],[16.291385500000047,46.87283350000007],[16.347822500000063,46.84051750000003],[16.31216150000006,46.778002498000035],[16.370793500000048,46.72224349800007],[16.42654550000003,46.69493699800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI03","LEVL_CODE":2,"CNTR_CODE":"SI","NAME_LATN":"Vzhodna Slovenija","NUTS_NAME":"Vzhodna Slovenija"},"geometry":{"type":"Polygon","coordinates":[[[16.42654550000003,46.69493699800006],[16.39033150000006,46.63820750000008],[16.596805,46.47590249800004],[16.38205350000004,46.551329999000075],[16.24214850000004,46.49007549600003],[16.301549,46.37828799600004],[16.088382999000032,46.388544499000034],[15.876868500000057,46.28005549800008],[15.791475500000047,46.259327496000026],[15.775107500000047,46.211418998000056],[15.647185500000035,46.21717399700003],[15.59612450000003,46.14635599800005],[15.627262,46.08595349800004],[15.732483,46.04452549900003],[15.706404500000076,45.97534299800003],[15.67752,45.86888799900004],[15.708988500000032,45.84641299900005],[15.404425,45.79271699600008],[15.331290500000023,45.76285499800008],[15.256749,45.718848999000045],[15.319616,45.674262498000076],[15.357104,45.71397099700005],[15.406294,45.647968],[15.27705050000003,45.604461498000035],[15.385532,45.48658049800008],[15.226437500000031,45.42708799700006],[14.932349,45.529082],[14.823365500000023,45.46070999800003],[14.570012500000075,45.67294449600007],[14.436836499000037,45.51059299900004],[14.118196500000067,45.48123899600006],[14.067097500000045,45.68623199800004],[14.09933,45.71626799900008],[14.042371500000058,45.76135899800005],[14.049503500000071,45.83733749600003],[14.129577500000039,45.86949899800004],[14.490795,45.88227999800006],[14.564807500000029,45.791461998000045],[14.688184,45.819037999000045],[14.823844500000064,45.726579497000046],[14.804826500000047,45.79839949600006],[14.858963,45.80290449600005],[14.883571,45.94606350000004],[14.948575,45.978610496000044],[14.886626500000034,46.06226349800005],[14.711757500000033,46.07792999800006],[14.908764500000075,46.207565997000074],[14.704025500000057,46.25826899600003],[14.637874500000066,46.364228498000045],[14.565212,46.36769699800004],[14.565175500000066,46.37245349600005],[14.674580500000047,46.450687499000026],[14.818606,46.50938299900008],[14.879666,46.61124799800007],[15.065120500000035,46.65211249600003],[15.40197949900005,46.65354849800008],[15.508846500000061,46.616227998000056],[15.649988,46.70575699900007],[15.786422,46.70746949900007],[16.038086,46.65614549600008],[15.984746500000028,46.752207],[15.996236,46.835398499000064],[16.113849,46.86906799800005],[16.291385500000047,46.87283350000007],[16.347822500000063,46.84051750000003],[16.31216150000006,46.778002498000035],[16.370793500000048,46.72224349800007],[16.42654550000003,46.69493699800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI031","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Pomurska","NUTS_NAME":"Pomurska"},"geometry":{"type":"Polygon","coordinates":[[[16.370793500000048,46.72224349800007],[16.42654550000003,46.69493699800006],[16.39033150000006,46.63820750000008],[16.596805,46.47590249800004],[16.38205350000004,46.551329999000075],[16.24214850000004,46.49007549600003],[16.011612,46.51020199900006],[15.912551,46.606250998000064],[15.931629499000053,46.63944749900003],[15.786422,46.70746949900007],[16.038086,46.65614549600008],[15.984746500000028,46.752207],[15.996236,46.835398499000064],[16.113849,46.86906799800005],[16.291385500000047,46.87283350000007],[16.347822500000063,46.84051750000003],[16.31216150000006,46.778002498000035],[16.370793500000048,46.72224349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI032","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Podravska","NUTS_NAME":"Podravska"},"geometry":{"type":"Polygon","coordinates":[[[16.24214850000004,46.49007549600003],[16.301549,46.37828799600004],[16.088382999000032,46.388544499000034],[15.876868500000057,46.28005549800008],[15.791475500000047,46.259327496000026],[15.566286500000047,46.28084899800007],[15.535353,46.350648998000054],[15.396701,46.382213498000056],[15.419257,46.42434799800003],[15.314141500000062,46.479625998000074],[15.370126500000026,46.563056999000025],[15.439412,46.567551998000056],[15.40197949900005,46.65354849800008],[15.508846500000061,46.616227998000056],[15.649988,46.70575699900007],[15.786422,46.70746949900007],[15.931629499000053,46.63944749900003],[15.912551,46.606250998000064],[16.011612,46.51020199900006],[16.24214850000004,46.49007549600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI033","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Koroška","NUTS_NAME":"Koroška"},"geometry":{"type":"Polygon","coordinates":[[[15.40197949900005,46.65354849800008],[15.439412,46.567551998000056],[15.370126500000026,46.563056999000025],[15.314141500000062,46.479625998000074],[15.187700500000062,46.38657950000004],[14.674580500000047,46.450687499000026],[14.818606,46.50938299900008],[14.879666,46.61124799800007],[15.065120500000035,46.65211249600003],[15.40197949900005,46.65354849800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI034","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Savinjska","NUTS_NAME":"Savinjska"},"geometry":{"type":"Polygon","coordinates":[[[15.791475500000047,46.259327496000026],[15.775107500000047,46.211418998000056],[15.647185500000035,46.21717399700003],[15.59612450000003,46.14635599800005],[15.627262,46.08595349800004],[15.599213500000076,46.054295],[15.651795500000048,46.03062999600007],[15.578302500000063,45.99616099800005],[15.515037,46.06469399900004],[15.154900500000053,46.093410498000026],[15.188929500000029,46.109960998000076],[15.107508500000051,46.18751150000003],[14.908764500000075,46.207565997000074],[14.704025500000057,46.25826899600003],[14.637874500000066,46.364228498000045],[14.565212,46.36769699800004],[14.565175500000066,46.37245349600005],[14.674580500000047,46.450687499000026],[15.187700500000062,46.38657950000004],[15.314141500000062,46.479625998000074],[15.419257,46.42434799800003],[15.396701,46.382213498000056],[15.535353,46.350648998000054],[15.566286500000047,46.28084899800007],[15.791475500000047,46.259327496000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI035","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Zasavska","NUTS_NAME":"Zasavska"},"geometry":{"type":"Polygon","coordinates":[[[15.154900500000053,46.093410498000026],[15.06730650000003,46.065522498000064],[15.09940750000004,46.01895899900006],[14.948575,45.978610496000044],[14.886626500000034,46.06226349800005],[14.711757500000033,46.07792999800006],[14.908764500000075,46.207565997000074],[15.107508500000051,46.18751150000003],[15.188929500000029,46.109960998000076],[15.154900500000053,46.093410498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI036","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Posavska","NUTS_NAME":"Posavska"},"geometry":{"type":"Polygon","coordinates":[[[15.627262,46.08595349800004],[15.732483,46.04452549900003],[15.706404500000076,45.97534299800003],[15.67752,45.86888799900004],[15.708988500000032,45.84641299900005],[15.404425,45.79271699600008],[15.32688550000006,45.95711799600008],[15.194664500000044,45.936113499000044],[15.09940750000004,46.01895899900006],[15.06730650000003,46.065522498000064],[15.154900500000053,46.093410498000026],[15.515037,46.06469399900004],[15.578302500000063,45.99616099800005],[15.651795500000048,46.03062999600007],[15.599213500000076,46.054295],[15.627262,46.08595349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI037","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Jugovzhodna Slovenija","NUTS_NAME":"Jugovzhodna Slovenija"},"geometry":{"type":"Polygon","coordinates":[[[15.404425,45.79271699600008],[15.331290500000023,45.76285499800008],[15.256749,45.718848999000045],[15.319616,45.674262498000076],[15.357104,45.71397099700005],[15.406294,45.647968],[15.27705050000003,45.604461498000035],[15.385532,45.48658049800008],[15.226437500000031,45.42708799700006],[14.932349,45.529082],[14.823365500000023,45.46070999800003],[14.570012500000075,45.67294449600007],[14.537504,45.739324],[14.578675500000031,45.73761650000006],[14.564807500000029,45.791461998000045],[14.688184,45.819037999000045],[14.823844500000064,45.726579497000046],[14.804826500000047,45.79839949600006],[14.858963,45.80290449600005],[14.883571,45.94606350000004],[14.948575,45.978610496000044],[15.09940750000004,46.01895899900006],[15.194664500000044,45.936113499000044],[15.32688550000006,45.95711799600008],[15.404425,45.79271699600008]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI038","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Primorsko-notranjska","NUTS_NAME":"Primorsko-notranjska"},"geometry":{"type":"Polygon","coordinates":[[[14.564807500000029,45.791461998000045],[14.578675500000031,45.73761650000006],[14.537504,45.739324],[14.570012500000075,45.67294449600007],[14.436836499000037,45.51059299900004],[14.118196500000067,45.48123899600006],[14.067097500000045,45.68623199800004],[14.09933,45.71626799900008],[14.042371500000058,45.76135899800005],[14.049503500000071,45.83733749600003],[14.129577500000039,45.86949899800004],[14.490795,45.88227999800006],[14.564807500000029,45.791461998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI04","LEVL_CODE":2,"CNTR_CODE":"SI","NAME_LATN":"Zahodna Slovenija","NUTS_NAME":"Zahodna Slovenija"},"geometry":{"type":"Polygon","coordinates":[[[14.886626500000034,46.06226349800005],[14.948575,45.978610496000044],[14.883571,45.94606350000004],[14.858963,45.80290449600005],[14.804826500000047,45.79839949600006],[14.823844500000064,45.726579497000046],[14.688184,45.819037999000045],[14.564807500000029,45.791461998000045],[14.490795,45.88227999800006],[14.129577500000039,45.86949899800004],[14.049503500000071,45.83733749600003],[14.042371500000058,45.76135899800005],[14.09933,45.71626799900008],[14.067097500000045,45.68623199800004],[14.118196500000067,45.48123899600006],[14.109059,45.48245849600005],[14.011006500000065,45.51988049900007],[13.962175500000058,45.50601349600004],[13.984909500000072,45.45602199800004],[13.887097500000039,45.425872998000045],[13.583067,45.47740999600006],[13.601972,45.537027496000064],[13.746271999000044,45.56440299800005],[13.722823500000061,45.59472549600008],[13.850761500000033,45.58493249600008],[13.918656500000054,45.63351749800006],[13.795683500000052,45.74320449900006],[13.596243,45.80793750000004],[13.597145,45.81952250000006],[13.575802500000066,45.847736498000074],[13.643036,45.98270199800004],[13.526538,45.97231849800005],[13.476164999000048,46.00608149800007],[13.496938999000065,46.05133499800007],[13.664347500000076,46.177549998000075],[13.423357,46.206053997000026],[13.375492500000064,46.29823249800006],[13.684032,46.43747250000007],[13.714184999000054,46.522703499000045],[14.43450050000007,46.44294350000007],[14.565175500000066,46.37245349600005],[14.565212,46.36769699800004],[14.637874500000066,46.364228498000045],[14.704025500000057,46.25826899600003],[14.908764500000075,46.207565997000074],[14.711757500000033,46.07792999800006],[14.886626500000034,46.06226349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI041","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Osrednjeslovenska","NUTS_NAME":"Osrednjeslovenska"},"geometry":{"type":"Polygon","coordinates":[[[14.908764500000075,46.207565997000074],[14.711757500000033,46.07792999800006],[14.886626500000034,46.06226349800005],[14.948575,45.978610496000044],[14.883571,45.94606350000004],[14.858963,45.80290449600005],[14.804826500000047,45.79839949600006],[14.823844500000064,45.726579497000046],[14.688184,45.819037999000045],[14.564807500000029,45.791461998000045],[14.490795,45.88227999800006],[14.129577500000039,45.86949899800004],[14.140111500000046,46.004658498000026],[14.225433500000065,46.09656049900008],[14.331476,46.09744399700003],[14.405381,46.190030999000044],[14.566047500000025,46.24840199600004],[14.535457499000074,46.35739649800007],[14.565212,46.36769699800004],[14.637874500000066,46.364228498000045],[14.704025500000057,46.25826899600003],[14.908764500000075,46.207565997000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI042","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Gorenjska","NUTS_NAME":"Gorenjska"},"geometry":{"type":"Polygon","coordinates":[[[14.43450050000007,46.44294350000007],[14.565175500000066,46.37245349600005],[14.565212,46.36769699800004],[14.535457499000074,46.35739649800007],[14.566047500000025,46.24840199600004],[14.405381,46.190030999000044],[14.331476,46.09744399700003],[14.225433500000065,46.09656049900008],[14.140111500000046,46.004658498000026],[14.018399,46.088404998000044],[14.073708,46.165686996000034],[13.73722250000003,46.276252500000055],[13.746101500000066,46.337485496000056],[13.835832,46.38134249900003],[13.684032,46.43747250000007],[13.714184999000054,46.522703499000045],[14.43450050000007,46.44294350000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI043","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Goriška","NUTS_NAME":"Goriška"},"geometry":{"type":"Polygon","coordinates":[[[13.684032,46.43747250000007],[13.835832,46.38134249900003],[13.746101500000066,46.337485496000056],[13.73722250000003,46.276252500000055],[14.073708,46.165686996000034],[14.018399,46.088404998000044],[14.140111500000046,46.004658498000026],[14.129577500000039,45.86949899800004],[14.049503500000071,45.83733749600003],[14.042371500000058,45.76135899800005],[13.715857,45.86351399900008],[13.597145,45.81952250000006],[13.575802500000066,45.847736498000074],[13.643036,45.98270199800004],[13.526538,45.97231849800005],[13.476164999000048,46.00608149800007],[13.496938999000065,46.05133499800007],[13.664347500000076,46.177549998000075],[13.423357,46.206053997000026],[13.375492500000064,46.29823249800006],[13.684032,46.43747250000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SI044","LEVL_CODE":3,"CNTR_CODE":"SI","NAME_LATN":"Obalno-kraška","NUTS_NAME":"Obalno-kraška"},"geometry":{"type":"Polygon","coordinates":[[[14.042371500000058,45.76135899800005],[14.09933,45.71626799900008],[14.067097500000045,45.68623199800004],[14.118196500000067,45.48123899600006],[14.109059,45.48245849600005],[14.011006500000065,45.51988049900007],[13.962175500000058,45.50601349600004],[13.984909500000072,45.45602199800004],[13.887097500000039,45.425872998000045],[13.583067,45.47740999600006],[13.601972,45.537027496000064],[13.746271999000044,45.56440299800005],[13.722823500000061,45.59472549600008],[13.850761500000033,45.58493249600008],[13.918656500000054,45.63351749800006],[13.795683500000052,45.74320449900006],[13.596243,45.80793750000004],[13.597145,45.81952250000006],[13.715857,45.86351399900008],[14.042371500000058,45.76135899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK","LEVL_CODE":0,"CNTR_CODE":"SK","NAME_LATN":"SLOVENSKO","NUTS_NAME":"SLOVENSKO"},"geometry":{"type":"Polygon","coordinates":[[[22.56684,49.08837749800006],[22.54507650000005,49.00630699800007],[22.382817500000044,48.86226399800006],[22.33966950000007,48.684236499000065],[22.160994500000072,48.567245496000055],[22.155306,48.40339649800006],[22.12107750000007,48.37831149800007],[21.721957,48.35105049800006],[21.61392150000006,48.50941299900006],[21.440056,48.58523299800004],[21.11743750000005,48.49105349700005],[20.850424,48.581631499000025],[20.506496,48.53441350000003],[20.463937,48.463966999000036],[20.325701500000037,48.27279849800004],[20.133181500000035,48.253985998000076],[20.051879,48.16770399600006],[19.899319,48.124797998000076],[19.644519,48.239175499000055],[19.52604,48.203133997000066],[19.451139,48.094441998000036],[19.01432250000005,48.077736498000036],[18.928392,48.05683249800006],[18.754815500000063,47.97508249800006],[18.768924,47.868569498000056],[18.848478,47.818228],[18.65544150000005,47.76008150000007],[17.893923,47.73945699800004],[17.705436500000076,47.758992498000055],[17.24742750000007,48.012008998000056],[17.160797500000058,48.00665650000008],[17.06741450000004,48.03143750000004],[17.066741,48.11868149800006],[16.976203,48.172244497000065],[16.957182,48.26347999600006],[16.842372500000067,48.35332699800006],[16.851106,48.438635],[16.949778,48.535791998000036],[16.940278,48.617245498000045],[17.201662500000054,48.87802899600007],[17.39672550000006,48.813349997000046],[17.64693,48.85426599700003],[17.884914500000036,48.92775350000005],[17.924501,49.02002299700007],[18.069837500000062,49.037547999000026],[18.14669250000003,49.247281],[18.322436,49.31505899900003],[18.40359550000005,49.39674549800003],[18.564790500000072,49.50693449700003],[18.851551,49.517189],[18.972818,49.50261199800008],[18.979868,49.395217],[19.153403,49.403777],[19.260708,49.53122299800003],[19.467386500000032,49.61376699900006],[19.631736,49.40882099600003],[19.790551,49.41113149900008],[19.799801,49.295825498000056],[19.758416,49.215991],[19.883929500000022,49.20417699800004],[20.088046,49.17958099800006],[20.146103500000038,49.317959],[20.322434500000043,49.40184599600008],[20.614939500000048,49.41783299800005],[20.743115500000044,49.415132999000036],[20.923722500000054,49.29623449800005],[21.104102499000078,49.376393998000026],[21.055269,49.42130599800004],[21.39771250000007,49.433793998000056],[21.840236,49.391505996000035],[21.966094,49.34511199900004],[22.034885,49.22246399800008],[22.56684,49.08837749800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK0","LEVL_CODE":1,"CNTR_CODE":"SK","NAME_LATN":"SLOVENSKO","NUTS_NAME":"SLOVENSKO"},"geometry":{"type":"Polygon","coordinates":[[[19.467386500000032,49.61376699900006],[19.631736,49.40882099600003],[19.790551,49.41113149900008],[19.799801,49.295825498000056],[19.758416,49.215991],[19.883929500000022,49.20417699800004],[20.088046,49.17958099800006],[20.146103500000038,49.317959],[20.322434500000043,49.40184599600008],[20.614939500000048,49.41783299800005],[20.743115500000044,49.415132999000036],[20.923722500000054,49.29623449800005],[21.104102499000078,49.376393998000026],[21.055269,49.42130599800004],[21.39771250000007,49.433793998000056],[21.840236,49.391505996000035],[21.966094,49.34511199900004],[22.034885,49.22246399800008],[22.56684,49.08837749800006],[22.54507650000005,49.00630699800007],[22.382817500000044,48.86226399800006],[22.33966950000007,48.684236499000065],[22.160994500000072,48.567245496000055],[22.155306,48.40339649800006],[22.12107750000007,48.37831149800007],[21.721957,48.35105049800006],[21.61392150000006,48.50941299900006],[21.440056,48.58523299800004],[21.11743750000005,48.49105349700005],[20.850424,48.581631499000025],[20.506496,48.53441350000003],[20.463937,48.463966999000036],[20.325701500000037,48.27279849800004],[20.133181500000035,48.253985998000076],[20.051879,48.16770399600006],[19.899319,48.124797998000076],[19.644519,48.239175499000055],[19.52604,48.203133997000066],[19.451139,48.094441998000036],[19.01432250000005,48.077736498000036],[18.928392,48.05683249800006],[18.754815500000063,47.97508249800006],[18.768924,47.868569498000056],[18.848478,47.818228],[18.65544150000005,47.76008150000007],[17.893923,47.73945699800004],[17.705436500000076,47.758992498000055],[17.24742750000007,48.012008998000056],[17.160797500000058,48.00665650000008],[17.06741450000004,48.03143750000004],[17.066741,48.11868149800006],[16.976203,48.172244497000065],[16.957182,48.26347999600006],[16.842372500000067,48.35332699800006],[16.851106,48.438635],[16.949778,48.535791998000036],[16.940278,48.617245498000045],[17.201662500000054,48.87802899600007],[17.39672550000006,48.813349997000046],[17.64693,48.85426599700003],[17.884914500000036,48.92775350000005],[17.924501,49.02002299700007],[18.069837500000062,49.037547999000026],[18.14669250000003,49.247281],[18.322436,49.31505899900003],[18.40359550000005,49.39674549800003],[18.564790500000072,49.50693449700003],[18.851551,49.517189],[18.972818,49.50261199800008],[18.979868,49.395217],[19.153403,49.403777],[19.260708,49.53122299800003],[19.467386500000032,49.61376699900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK01","LEVL_CODE":2,"CNTR_CODE":"SK","NAME_LATN":"Bratislavský kraj","NUTS_NAME":"Bratislavský kraj"},"geometry":{"type":"Polygon","coordinates":[[[17.24742750000007,48.012008998000056],[17.160797500000058,48.00665650000008],[17.06741450000004,48.03143750000004],[17.066741,48.11868149800006],[16.976203,48.172244497000065],[16.957182,48.26347999600006],[16.842372500000067,48.35332699800006],[16.851106,48.438635],[16.949778,48.535791998000036],[17.235799,48.56740950000005],[17.32763999900004,48.65402749800006],[17.376616,48.612037497000074],[17.27990440600007,48.54338659300004],[17.326222001000076,48.514675999000076],[17.302191,48.46593949800007],[17.477537,48.35257849900006],[17.449279,48.31961249900007],[17.513756,48.27907699600007],[17.47329,48.235082497000064],[17.529213,48.19649749900003],[17.34048350000006,48.11844499600005],[17.24742750000007,48.012008998000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK010","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Bratislavský kraj","NUTS_NAME":"Bratislavský kraj"},"geometry":{"type":"Polygon","coordinates":[[[17.24742750000007,48.012008998000056],[17.160797500000058,48.00665650000008],[17.06741450000004,48.03143750000004],[17.066741,48.11868149800006],[16.976203,48.172244497000065],[16.957182,48.26347999600006],[16.842372500000067,48.35332699800006],[16.851106,48.438635],[16.949778,48.535791998000036],[17.235799,48.56740950000005],[17.32763999900004,48.65402749800006],[17.376616,48.612037497000074],[17.27990440600007,48.54338659300004],[17.326222001000076,48.514675999000076],[17.302191,48.46593949800007],[17.477537,48.35257849900006],[17.449279,48.31961249900007],[17.513756,48.27907699600007],[17.47329,48.235082497000064],[17.529213,48.19649749900003],[17.34048350000006,48.11844499600005],[17.24742750000007,48.012008998000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK02","LEVL_CODE":2,"CNTR_CODE":"SK","NAME_LATN":"Západné Slovensko","NUTS_NAME":"Západné Slovensko"},"geometry":{"type":"Polygon","coordinates":[[[18.600387,49.02848899900005],[18.46424250000007,48.95002999800005],[18.69456,48.959177],[18.69807650000007,48.847709499000075],[18.826351,48.74405899800007],[18.716381500000068,48.626464499000065],[18.478677,48.54860399700004],[18.537486,48.50315099800008],[18.575207,48.34789599800007],[18.73144250000007,48.41677849800004],[18.809206500000073,48.37073649800004],[18.85251150000005,48.30583749900006],[18.808854,48.164832997000076],[19.062313,48.18836149900005],[19.01432250000005,48.077736498000036],[18.928392,48.05683249800006],[18.754815500000063,47.97508249800006],[18.768924,47.868569498000056],[18.848478,47.818228],[18.65544150000005,47.76008150000007],[17.893923,47.73945699800004],[17.705436500000076,47.758992498000055],[17.24742750000007,48.012008998000056],[17.34048350000006,48.11844499600005],[17.529213,48.19649749900003],[17.47329,48.235082497000064],[17.513756,48.27907699600007],[17.449279,48.31961249900007],[17.477537,48.35257849900006],[17.302191,48.46593949800007],[17.326222001000076,48.514675999000076],[17.27990440600007,48.54338659300004],[17.376616,48.612037497000074],[17.32763999900004,48.65402749800006],[17.235799,48.56740950000005],[16.949778,48.535791998000036],[16.940278,48.617245498000045],[17.201662500000054,48.87802899600007],[17.39672550000006,48.813349997000046],[17.64693,48.85426599700003],[17.884914500000036,48.92775350000005],[17.924501,49.02002299700007],[18.069837500000062,49.037547999000026],[18.14669250000003,49.247281],[18.322436,49.31505899900003],[18.590456500000073,49.11776899700004],[18.600387,49.02848899900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK021","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Trnavský kraj","NUTS_NAME":"Trnavský kraj"},"geometry":{"type":"Polygon","coordinates":[[[17.39672550000006,48.813349997000046],[17.353044500000067,48.777521497000066],[17.472818,48.76083399800007],[17.466150500000026,48.69993499800006],[17.533132500000022,48.62661599900008],[17.70167250000003,48.68251549900003],[17.979113,48.61253099800007],[17.845544500000074,48.52156599800003],[17.964675,48.46479949800005],[17.833375999000054,48.29675199800005],[17.882103,48.20860150000004],[17.777683500000023,48.13955949900003],[17.88045850000003,48.046347498000046],[17.887778,47.91929249900005],[17.705436500000076,47.758992498000055],[17.24742750000007,48.012008998000056],[17.34048350000006,48.11844499600005],[17.529213,48.19649749900003],[17.47329,48.235082497000064],[17.513756,48.27907699600007],[17.449279,48.31961249900007],[17.477537,48.35257849900006],[17.302191,48.46593949800007],[17.326222001000076,48.514675999000076],[17.27990440600007,48.54338659300004],[17.376616,48.612037497000074],[17.32763999900004,48.65402749800006],[17.235799,48.56740950000005],[16.949778,48.535791998000036],[16.940278,48.617245498000045],[17.201662500000054,48.87802899600007],[17.39672550000006,48.813349997000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK022","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Trenciansky kraj","NUTS_NAME":"Trenciansky kraj"},"geometry":{"type":"Polygon","coordinates":[[[18.826351,48.74405899800007],[18.716381500000068,48.626464499000065],[18.478677,48.54860399700004],[18.311377,48.49688899800003],[18.214969500000052,48.57415849800003],[18.25610050000006,48.609357497000076],[18.021289,48.706713999000044],[17.979113,48.61253099800007],[17.70167250000003,48.68251549900003],[17.533132500000022,48.62661599900008],[17.466150500000026,48.69993499800006],[17.472818,48.76083399800007],[17.353044500000067,48.777521497000066],[17.39672550000006,48.813349997000046],[17.64693,48.85426599700003],[17.884914500000036,48.92775350000005],[17.924501,49.02002299700007],[18.069837500000062,49.037547999000026],[18.14669250000003,49.247281],[18.322436,49.31505899900003],[18.590456500000073,49.11776899700004],[18.600387,49.02848899900005],[18.46424250000007,48.95002999800005],[18.69456,48.959177],[18.69807650000007,48.847709499000075],[18.826351,48.74405899800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK023","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Nitriansky kraj","NUTS_NAME":"Nitriansky kraj"},"geometry":{"type":"Polygon","coordinates":[[[18.478677,48.54860399700004],[18.537486,48.50315099800008],[18.575207,48.34789599800007],[18.73144250000007,48.41677849800004],[18.809206500000073,48.37073649800004],[18.85251150000005,48.30583749900006],[18.808854,48.164832997000076],[19.062313,48.18836149900005],[19.01432250000005,48.077736498000036],[18.928392,48.05683249800006],[18.754815500000063,47.97508249800006],[18.768924,47.868569498000056],[18.848478,47.818228],[18.65544150000005,47.76008150000007],[17.893923,47.73945699800004],[17.705436500000076,47.758992498000055],[17.887778,47.91929249900005],[17.88045850000003,48.046347498000046],[17.777683500000023,48.13955949900003],[17.882103,48.20860150000004],[17.833375999000054,48.29675199800005],[17.964675,48.46479949800005],[17.845544500000074,48.52156599800003],[17.979113,48.61253099800007],[18.021289,48.706713999000044],[18.25610050000006,48.609357497000076],[18.214969500000052,48.57415849800003],[18.311377,48.49688899800003],[18.478677,48.54860399700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK03","LEVL_CODE":2,"CNTR_CODE":"SK","NAME_LATN":"Stredné Slovensko","NUTS_NAME":"Stredné Slovensko"},"geometry":{"type":"Polygon","coordinates":[[[19.467386500000032,49.61376699900006],[19.631736,49.40882099600003],[19.790551,49.41113149900008],[19.799801,49.295825498000056],[19.758416,49.215991],[19.883929500000022,49.20417699800004],[19.952423,49.11006149700006],[20.037746,49.108310998000036],[20.015285,49.015974999000036],[19.963713,49.009667998000054],[20.060430500000052,48.93230299800007],[19.99225449900007,48.90749999800005],[20.267450500000052,48.874073],[20.197176,48.818583998000065],[20.182295500000066,48.728224999000076],[20.35597850000005,48.59883899700003],[20.306414,48.479673999000056],[20.463937,48.463966999000036],[20.325701500000037,48.27279849800004],[20.133181500000035,48.253985998000076],[20.051879,48.16770399600006],[19.899319,48.124797998000076],[19.644519,48.239175499000055],[19.52604,48.203133997000066],[19.451139,48.094441998000036],[19.01432250000005,48.077736498000036],[19.062313,48.18836149900005],[18.808854,48.164832997000076],[18.85251150000005,48.30583749900006],[18.809206500000073,48.37073649800004],[18.73144250000007,48.41677849800004],[18.575207,48.34789599800007],[18.537486,48.50315099800008],[18.478677,48.54860399700004],[18.716381500000068,48.626464499000065],[18.826351,48.74405899800007],[18.69807650000007,48.847709499000075],[18.69456,48.959177],[18.46424250000007,48.95002999800005],[18.600387,49.02848899900005],[18.590456500000073,49.11776899700004],[18.322436,49.31505899900003],[18.40359550000005,49.39674549800003],[18.564790500000072,49.50693449700003],[18.851551,49.517189],[18.972818,49.50261199800008],[18.979868,49.395217],[19.153403,49.403777],[19.260708,49.53122299800003],[19.467386500000032,49.61376699900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK031","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Žilinský kraj","NUTS_NAME":"Žilinský kraj"},"geometry":{"type":"Polygon","coordinates":[[[19.883929500000022,49.20417699800004],[19.952423,49.11006149700006],[20.037746,49.108310998000036],[20.015285,49.015974999000036],[19.963713,49.009667998000054],[20.060430500000052,48.93230299800007],[19.99225449900007,48.90749999800005],[19.546426500000052,48.94196099800007],[19.26648,48.86497899900007],[19.04708050000005,48.88368699700004],[18.994554,48.847297497000056],[18.99831450000005,48.748995998000055],[18.826351,48.74405899800007],[18.69807650000007,48.847709499000075],[18.69456,48.959177],[18.46424250000007,48.95002999800005],[18.600387,49.02848899900005],[18.590456500000073,49.11776899700004],[18.322436,49.31505899900003],[18.40359550000005,49.39674549800003],[18.564790500000072,49.50693449700003],[18.851551,49.517189],[18.972818,49.50261199800008],[18.979868,49.395217],[19.153403,49.403777],[19.260708,49.53122299800003],[19.467386500000032,49.61376699900006],[19.631736,49.40882099600003],[19.790551,49.41113149900008],[19.799801,49.295825498000056],[19.758416,49.215991],[19.883929500000022,49.20417699800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK032","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Banskobystrický kraj","NUTS_NAME":"Banskobystrický kraj"},"geometry":{"type":"Polygon","coordinates":[[[19.99225449900007,48.90749999800005],[20.267450500000052,48.874073],[20.197176,48.818583998000065],[20.182295500000066,48.728224999000076],[20.35597850000005,48.59883899700003],[20.306414,48.479673999000056],[20.463937,48.463966999000036],[20.325701500000037,48.27279849800004],[20.133181500000035,48.253985998000076],[20.051879,48.16770399600006],[19.899319,48.124797998000076],[19.644519,48.239175499000055],[19.52604,48.203133997000066],[19.451139,48.094441998000036],[19.01432250000005,48.077736498000036],[19.062313,48.18836149900005],[18.808854,48.164832997000076],[18.85251150000005,48.30583749900006],[18.809206500000073,48.37073649800004],[18.73144250000007,48.41677849800004],[18.575207,48.34789599800007],[18.537486,48.50315099800008],[18.478677,48.54860399700004],[18.716381500000068,48.626464499000065],[18.826351,48.74405899800007],[18.99831450000005,48.748995998000055],[18.994554,48.847297497000056],[19.04708050000005,48.88368699700004],[19.26648,48.86497899900007],[19.546426500000052,48.94196099800007],[19.99225449900007,48.90749999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK04","LEVL_CODE":2,"CNTR_CODE":"SK","NAME_LATN":"Východné Slovensko","NUTS_NAME":"Východné Slovensko"},"geometry":{"type":"Polygon","coordinates":[[[21.966094,49.34511199900004],[22.034885,49.22246399800008],[22.56684,49.08837749800006],[22.54507650000005,49.00630699800007],[22.382817500000044,48.86226399800006],[22.33966950000007,48.684236499000065],[22.160994500000072,48.567245496000055],[22.155306,48.40339649800006],[22.12107750000007,48.37831149800007],[21.721957,48.35105049800006],[21.61392150000006,48.50941299900006],[21.440056,48.58523299800004],[21.11743750000005,48.49105349700005],[20.850424,48.581631499000025],[20.506496,48.53441350000003],[20.463937,48.463966999000036],[20.306414,48.479673999000056],[20.35597850000005,48.59883899700003],[20.182295500000066,48.728224999000076],[20.197176,48.818583998000065],[20.267450500000052,48.874073],[19.99225449900007,48.90749999800005],[20.060430500000052,48.93230299800007],[19.963713,49.009667998000054],[20.015285,49.015974999000036],[20.037746,49.108310998000036],[19.952423,49.11006149700006],[19.883929500000022,49.20417699800004],[20.088046,49.17958099800006],[20.146103500000038,49.317959],[20.322434500000043,49.40184599600008],[20.614939500000048,49.41783299800005],[20.743115500000044,49.415132999000036],[20.923722500000054,49.29623449800005],[21.104102499000078,49.376393998000026],[21.055269,49.42130599800004],[21.39771250000007,49.433793998000056],[21.840236,49.391505996000035],[21.966094,49.34511199900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK041","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Prešovský kraj","NUTS_NAME":"Prešovský kraj"},"geometry":{"type":"Polygon","coordinates":[[[21.39771250000007,49.433793998000056],[21.840236,49.391505996000035],[21.966094,49.34511199900004],[22.034885,49.22246399800008],[22.56684,49.08837749800006],[22.54507650000005,49.00630699800007],[22.382817500000044,48.86226399800006],[22.257577500000025,48.865778499000044],[22.213631,48.92970149800004],[22.131583,48.844694],[21.799625,48.90137899900003],[21.80233,48.79053250000004],[21.68646750000005,48.762760496000055],[21.541557,48.79407199800005],[21.483930500000042,48.88226399800004],[21.27505,48.81181799700005],[21.252751500000045,48.867897998000046],[20.95502350000004,48.95459949800005],[20.776541,48.99882699800003],[20.632302500000037,48.95223849700005],[20.533948500000065,49.01146349800007],[20.331405,48.96643149800008],[20.297825500000044,48.939959498000064],[20.339325,48.91591649800006],[20.267450500000052,48.874073],[19.99225449900007,48.90749999800005],[20.060430500000052,48.93230299800007],[19.963713,49.009667998000054],[20.015285,49.015974999000036],[20.037746,49.108310998000036],[19.952423,49.11006149700006],[19.883929500000022,49.20417699800004],[20.088046,49.17958099800006],[20.146103500000038,49.317959],[20.322434500000043,49.40184599600008],[20.614939500000048,49.41783299800005],[20.743115500000044,49.415132999000036],[20.923722500000054,49.29623449800005],[21.104102499000078,49.376393998000026],[21.055269,49.42130599800004],[21.39771250000007,49.433793998000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"SK042","LEVL_CODE":3,"CNTR_CODE":"SK","NAME_LATN":"Košický kraj","NUTS_NAME":"Košický kraj"},"geometry":{"type":"Polygon","coordinates":[[[22.382817500000044,48.86226399800006],[22.33966950000007,48.684236499000065],[22.160994500000072,48.567245496000055],[22.155306,48.40339649800006],[22.12107750000007,48.37831149800007],[21.721957,48.35105049800006],[21.61392150000006,48.50941299900006],[21.440056,48.58523299800004],[21.11743750000005,48.49105349700005],[20.850424,48.581631499000025],[20.506496,48.53441350000003],[20.463937,48.463966999000036],[20.306414,48.479673999000056],[20.35597850000005,48.59883899700003],[20.182295500000066,48.728224999000076],[20.197176,48.818583998000065],[20.267450500000052,48.874073],[20.339325,48.91591649800006],[20.297825500000044,48.939959498000064],[20.331405,48.96643149800008],[20.533948500000065,49.01146349800007],[20.632302500000037,48.95223849700005],[20.776541,48.99882699800003],[20.95502350000004,48.95459949800005],[21.252751500000045,48.867897998000046],[21.27505,48.81181799700005],[21.483930500000042,48.88226399800004],[21.541557,48.79407199800005],[21.68646750000005,48.762760496000055],[21.80233,48.79053250000004],[21.799625,48.90137899900003],[22.131583,48.844694],[22.213631,48.92970149800004],[22.257577500000025,48.865778499000044],[22.382817500000044,48.86226399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR","LEVL_CODE":0,"CNTR_CODE":"TR","NAME_LATN":"TÜRKIYE","NUTS_NAME":"TÜRKIYE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[34.943724,42.096982998000044],[35.08171250000004,42.02601799800004],[35.193533,42.03946899700003],[35.095869,41.965749997000046],[35.12668050000008,41.863176499000076],[35.276678,41.715734498000074],[35.51369950000003,41.63597999800004],[35.96095850000006,41.73469299900006],[36.11516750000004,41.625360500000056],[36.132280500000036,41.46124449800004],[36.38541250000003,41.25627349800004],[36.532507,41.26788149600003],[36.651739,41.38340199800007],[36.99572950000004,41.284005997000065],[37.03171350000008,41.18925299800003],[37.15234850000007,41.148413497000035],[37.292886500000066,41.14552499800004],[37.53754250000003,41.020898498000065],[37.674974500000076,41.128019497000025],[37.786104,41.116929998000046],[37.77697949900005,41.06200599600004],[37.87777150000005,40.98752399800003],[38.10999250000003,40.95851799700006],[38.529001,40.91645999700006],[39.17930850000005,41.074095998000075],[39.48634549900004,41.091859998000075],[39.646242,40.99686599700004],[40.12019150000003,40.912206498000046],[40.328287500000044,40.987883500000066],[40.721079,41.08785849700007],[41.2519,41.32993349800006],[41.401453,41.380735499000025],[41.54713350000003,41.52037549700003],[41.82492750000006,41.431911497000044],[41.971884,41.521492],[42.515174932000036,41.43828241500006],[42.601024,41.584979496000074],[42.836095,41.584442497000055],[42.803529500000025,41.494068998000046],[43.21288050000004,41.30604149800007],[43.130417,41.25584549800004],[43.19456150000008,41.25395999800003],[43.23022650000007,41.17379899900004],[43.376619,41.20249449600004],[43.473827,41.123297499000046],[43.469638,41.05734499700003],[43.67249,40.926521998000055],[43.750409,40.74499899800003],[43.717934,40.620918999000025],[43.583054,40.45111099800005],[43.715371,40.15718099700007],[43.653923,40.13037749800003],[43.933884,40.01361099800005],[44.351662,40.02222099700003],[44.487778,39.96276899700007],[44.76841,39.714094497000076],[44.812042500000075,39.63174349700006],[44.608464,39.77915599800008],[44.50209950000004,39.71699349900007],[44.401101,39.41651599800008],[44.076863,39.411656],[44.04995581000003,39.362815426000054],[44.21645,39.13651699600007],[44.173324999000044,38.96804099800005],[44.30026699900003,38.84262799800007],[44.261796,38.72429199800007],[44.317215,38.61304499800008],[44.30526,38.40053499700008],[44.436726,38.39480999800003],[44.482518,38.34129999700008],[44.223969,37.89914999800004],[44.46285493800008,37.809568168000055],[44.617768,37.71797599800004],[44.588852,37.443092997000065],[44.801659,37.32166299800008],[44.79314750000003,37.17628399800003],[44.582768,37.16318499800008],[44.349709,37.03832299700008],[44.319161,36.97123799900004],[44.257077,36.98568799800006],[44.195965,37.09637799700005],[44.262597,37.163467997000055],[44.262146,37.23651799800007],[44.118882,37.31568599900004],[44.009015,37.31776799800008],[43.91298,37.22658899800007],[43.424563,37.27533449900005],[43.156937,37.37387799800007],[42.924164,37.32776699800007],[42.786801,37.38367499800006],[42.57950599900005,37.15367499800004],[42.35568250000006,37.10799350000008],[42.351521,37.23116999900003],[42.180832,37.29054199800004],[42.05693099900003,37.19277099900006],[41.663342,37.102903498000046],[41.216382,37.07248999800004],[40.770821,37.11804999800006],[40.226033,36.901373498000055],[39.818055,36.75360899800006],[39.221524,36.665340996000054],[38.725266,36.70360599800006],[38.386376,36.89832999600003],[38.181107,36.90582299700003],[38.04758950000007,36.845554496000034],[37.58551650000004,36.703768498000045],[37.127495,36.65915699900006],[36.98832,36.757419999000035],[36.678777500000024,36.83266099700006],[36.66001039400004,36.83323153500004],[36.549576,36.48776999700004],[36.690133,36.28624799900007],[36.683648,36.23678349800008],[36.39222,36.21332599900006],[36.37497,35.99791399800006],[36.223607,35.962219997000034],[36.168469,35.819717997000055],[35.917953,35.928695498000025],[35.978777,36.01944149800005],[35.779623,36.31855599700003],[36.037126500000056,36.53373499700007],[36.188917,36.593206500000065],[36.21573050000006,36.65963549800006],[36.205831500000045,36.78273199600005],[36.14857650000005,36.85506649800004],[36.018686,36.92843449800006],[35.963096,36.902986998000074],[35.792162,36.76891499900006],[35.56386,36.71301049700003],[35.723783500000025,36.71192749800008],[35.56584,36.56479449900007],[35.420896500000026,36.59061599900008],[35.339365,36.53852649800007],[34.899245,36.73828099800005],[34.74605750000006,36.81102949900003],[34.56072050000006,36.76816349600006],[34.085783,36.41823749800005],[34.04285250000004,36.29496949800006],[33.94976999900007,36.26343749600005],[33.866678,36.31495849800007],[33.68421,36.13340199800007],[33.64763050000005,36.19356349700007],[33.56112850000005,36.125440499000035],[33.14687549900003,36.13780399800004],[33.08139250000005,36.08072449800005],[32.950445,36.103795998000066],[32.805395,36.02311499800004],[32.57634,36.093232998000076],[32.295229,36.234987498000066],[32.029032,36.53820249800003],[31.334676500000057,36.80768799800006],[30.698167500000068,36.88451949700004],[30.57494150000008,36.794511998000075],[30.556848500000058,36.619237998000074],[30.59149,36.58946799700004],[30.481527500000027,36.433797998000045],[30.48763450000007,36.27913099700004],[30.405119,36.20929149700004],[30.32793250000003,36.29695299800005],[30.188627,36.31390950000008],[30.115274500000055,36.24686649800003],[29.922674,36.23392699600004],[29.683466,36.130262498000036],[29.644518,36.19632899900006],[29.408991,36.261224997000056],[29.351629500000058,36.230226499000025],[29.26056150000005,36.30439899700008],[29.102026,36.386720497000056],[29.125913500000024,36.542272499000035],[29.02238650000004,36.58621399800006],[29.115694,36.62598249700005],[29.103811,36.67034349700003],[28.928788999000062,36.754858],[28.851038,36.66208449800007],[28.878309,36.62911799800003],[28.846163,36.58836949600004],[28.790045,36.685636500000044],[28.618429,36.70327949800003],[28.630129,36.781706],[28.558664500000077,36.831861497000034],[28.53590599900008,36.78577999600003],[28.270406500000036,36.852479997000046],[28.233087500000067,36.80461299800004],[28.279173,36.734334998000065],[28.163698,36.68377899800004],[28.089052500000037,36.582017998000026],[27.97934550000008,36.553571497000064],[27.956487500000037,36.60048499800007],[28.095728,36.64494499700004],[27.963636500000064,36.683786498000075],[28.10157950000007,36.705804999000065],[28.131860500000073,36.79396650000007],[27.730821500000047,36.763143497000044],[27.67334150000005,36.67207149700005],[27.478128500000025,36.64709299800006],[27.374422,36.68548399900004],[27.428606,36.748281500000076],[27.642294,36.80760399800005],[28.032835,36.78775999800007],[28.059328,36.878801500000066],[28.014459500000044,36.90612599800005],[28.159700500000042,36.925954998000066],[28.330560499000057,37.03339549700007],[27.565542,36.97728549800007],[27.42487150000005,37.03574199700006],[27.264269,36.964120997000066],[27.225256,37.05810349700005],[27.32292,37.15848349700008],[27.470903500000077,37.08077799800003],[27.546667,37.12506699900007],[27.526208999000062,37.193677996000076],[27.611025,37.25751299900003],[27.411882500000047,37.30456749900003],[27.404363,37.36809499900005],[27.424582461000057,37.40700347200004],[27.419004802000075,37.41190175500003],[27.202436500000033,37.350824499000055],[27.21665,37.59111199800003],[27.003069,37.65979599800005],[27.234331,37.72418399800006],[27.264569,37.87504849700008],[27.24043450000005,37.98498349800008],[26.980882500000064,38.07474699800008],[26.864545500000077,38.02855499900005],[26.757936500000028,38.221479499000054],[26.632337500000062,38.19990350000006],[26.5912,38.102750497000045],[26.281446500000072,38.26644349800006],[26.284891,38.376329498000075],[26.379034,38.309190998000076],[26.51573,38.42750749600003],[26.37163350000003,38.53015349800006],[26.353018,38.63760549800003],[26.397398,38.66724949800005],[26.47363850000005,38.674909497000044],[26.62529550000005,38.52710550000006],[26.611200500000052,38.42235349900005],[26.67402050000004,38.311986999000055],[26.707414499000038,38.43452649900007],[26.808508,38.35518449700004],[27.167742,38.44041999800004],[26.951025,38.443891499000074],[26.733785500000067,38.653470998000046],[26.740319999000064,38.741529497000045],[26.89434650000004,38.73359099800007],[26.939085,38.76427649800007],[26.892832,38.824922498000035],[27.066274500000077,38.87722599700004],[27.043180500000062,38.94094649800007],[26.80241599900006,38.95018599800005],[26.799127500000054,39.032182498000054],[26.886392500000056,39.07982449700006],[26.763921,39.17520999800007],[26.71832450000005,39.25861949800003],[26.60920150000004,39.27264199800004],[26.622667500000034,39.30575349800006],[26.671144500000025,39.27934849600007],[26.858988,39.47728899800006],[26.931722500000035,39.48557849800005],[26.950548500000025,39.559968498000046],[26.618685,39.54787949800004],[26.204252500000052,39.46097399800004],[26.062643,39.479497998000056],[26.163519,39.659459996000066],[26.137392,39.84089499800007],[26.179243,39.99143399700006],[26.321268,40.01747699600003],[26.397505,40.145826498000076],[26.403608500000075,40.19677149900008],[26.524309,40.21770649700005],[26.756845500000054,40.40379899700008],[27.033163,40.38958949900007],[27.286051,40.475088],[27.319921500000078,40.43341249700006],[27.315832,40.37848099800004],[27.50628050000006,40.304677998000045],[27.7619,40.30641749600005],[27.879064500000027,40.372956997000074],[27.686708500000066,40.47253599800007],[27.753523,40.52842149700007],[28.03245,40.480451499000026],[28.01680950000008,40.44744299900003],[27.896269,40.38821199900008],[27.954508,40.35312049700008],[28.164779,40.395612999000036],[28.980173,40.355935997000074],[29.154478,40.42537499800005],[28.991232500000024,40.46627349700003],[28.780161,40.53619199700006],[28.996286500000053,40.645887498000036],[29.517332,40.72897149700003],[29.546993,40.69543599900004],[29.942186500000048,40.750489998000035],[29.353948500000058,40.75541099800006],[29.342517500000042,40.807657498000026],[29.03146950000007,40.967084999000065],[29.006174,41.01020249800007],[29.095785,41.11750999800006],[29.087270500000045,41.178521998000065],[29.265264,41.230551999000056],[29.865624500000024,41.14344749700007],[30.35468650000007,41.18419449800007],[30.982055500000058,41.07260250000007],[31.295656500000064,41.11631999900004],[31.409174,41.21326249800006],[31.403204,41.31739249700007],[32.062578,41.58030599800003],[32.28410150000008,41.719801],[32.73610350000007,41.84900949800004],[33.31961650000005,42.01599299800006],[34.228878499000075,41.95495899800005],[34.797708500000056,41.95460299700005],[34.943724,42.096982998000044]]],[[[27.25369450000005,42.09926199800003],[27.559395,41.904781496000055],[27.859884500000078,42.005878497000026],[28.03551250000004,41.98307949800005],[28.05775250000005,41.883687999000074],[27.997202,41.88834949900007],[27.984159500000032,41.81855199600005],[28.152973,41.57896249600003],[28.181659499000034,41.564058998000064],[28.63096450000006,41.35011499800004],[29.11588450000005,41.22891049700007],[29.037527,41.15583999800003],[29.07263350000005,41.124540498000044],[28.985017500000026,41.00490349800003],[28.824152,40.954229498000075],[28.170191,41.073117997000054],[27.991102500000068,41.01834499700004],[27.955469,40.96380050000005],[27.763170500000058,41.013876],[27.512853500000062,40.974729496000066],[27.454728999000054,40.84480099700005],[27.309553,40.702420999000026],[26.968651500000078,40.55336899800005],[26.694452500000068,40.45057849800003],[26.592272,40.322187496000026],[26.358301,40.20253199900003],[26.38106350000004,40.143331498000066],[26.218744,40.04709449600006],[26.165872500000035,40.052560999000036],[26.27830699900005,40.21555899800006],[26.218675500000074,40.319448496000064],[26.838197500000035,40.58582499800008],[26.80476550000003,40.65426449600005],[26.734886500000073,40.64293299700006],[26.151155500000073,40.589315499000065],[26.07517450000006,40.61760899800004],[26.032758,40.73025699800007],[26.12489,40.742109998000046],[26.29101550000007,40.93188349800005],[26.359039,40.94717649800003],[26.31295750000004,41.07548049700006],[26.321652,41.250406498000075],[26.628431500000033,41.345533498000066],[26.60012050000006,41.60119899700004],[26.357879,41.711104498000054],[26.328071499000032,41.74837899700003],[26.369060500000046,41.82068249800005],[26.538608500000066,41.823482496000054],[26.584593,41.902652998000065],[26.561544500000025,41.92627349800006],[26.94922850000006,42.00021349800005],[27.059650500000032,42.088336997000056],[27.25369450000005,42.09926199800003]]],[[[25.97707950000006,40.21614249700008],[25.965894500000047,40.15049950000008],[26.013651,40.16370949700007],[25.98987,40.13219250000003],[25.738722,40.09072699800004],[25.665178500000025,40.12435349700007],[25.78412050000003,40.20969949600004],[25.97707950000006,40.21614249700008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR1","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"ISTANBUL","NUTS_NAME":"ISTANBUL"},"geometry":{"type":"MultiPolygon","coordinates":[[[[29.407457,40.95459],[29.342517500000042,40.807657498000026],[29.03146950000007,40.967084999000065],[29.006174,41.01020249800007],[29.095785,41.11750999800006],[29.087270500000045,41.178521998000065],[29.265264,41.230551999000056],[29.865624500000024,41.14344749700007],[29.849114,41.064824497000075],[29.407457,40.95459]]],[[[29.07263350000005,41.124540498000044],[28.985017500000026,41.00490349800003],[28.824152,40.954229498000075],[28.170191,41.073117997000054],[27.991102500000068,41.01834499700004],[27.964958,41.14141499800007],[28.138288,41.42519649800005],[28.08682950000008,41.50417799600007],[28.181659499000034,41.564058998000064],[28.63096450000006,41.35011499800004],[29.11588450000005,41.22891049700007],[29.037527,41.15583999800003],[29.07263350000005,41.124540498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR10","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Istanbul","NUTS_NAME":"Istanbul"},"geometry":{"type":"MultiPolygon","coordinates":[[[[29.407457,40.95459],[29.342517500000042,40.807657498000026],[29.03146950000007,40.967084999000065],[29.006174,41.01020249800007],[29.095785,41.11750999800006],[29.087270500000045,41.178521998000065],[29.265264,41.230551999000056],[29.865624500000024,41.14344749700007],[29.849114,41.064824497000075],[29.407457,40.95459]]],[[[29.07263350000005,41.124540498000044],[28.985017500000026,41.00490349800003],[28.824152,40.954229498000075],[28.170191,41.073117997000054],[27.991102500000068,41.01834499700004],[27.964958,41.14141499800007],[28.138288,41.42519649800005],[28.08682950000008,41.50417799600007],[28.181659499000034,41.564058998000064],[28.63096450000006,41.35011499800004],[29.11588450000005,41.22891049700007],[29.037527,41.15583999800003],[29.07263350000005,41.124540498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR100","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Istanbul","NUTS_NAME":"Istanbul"},"geometry":{"type":"MultiPolygon","coordinates":[[[[29.407457,40.95459],[29.342517500000042,40.807657498000026],[29.03146950000007,40.967084999000065],[29.006174,41.01020249800007],[29.095785,41.11750999800006],[29.087270500000045,41.178521998000065],[29.265264,41.230551999000056],[29.865624500000024,41.14344749700007],[29.849114,41.064824497000075],[29.407457,40.95459]]],[[[29.07263350000005,41.124540498000044],[28.985017500000026,41.00490349800003],[28.824152,40.954229498000075],[28.170191,41.073117997000054],[27.991102500000068,41.01834499700004],[27.964958,41.14141499800007],[28.138288,41.42519649800005],[28.08682950000008,41.50417799600007],[28.181659499000034,41.564058998000064],[28.63096450000006,41.35011499800004],[29.11588450000005,41.22891049700007],[29.037527,41.15583999800003],[29.07263350000005,41.124540498000044]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR2","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"BATI MARMARA","NUTS_NAME":"BATI MARMARA"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.896269,40.38821199900008],[27.954508,40.35312049700008],[28.164779,40.395612999000036],[28.06755050000004,40.25313399800007],[28.265037500000062,39.87023399700007],[28.515012,39.73130899700004],[28.69535250000007,39.757907498000066],[28.793313500000068,39.61746949700006],[28.968574,39.60067749800004],[28.885427,39.372048997000036],[28.597988,39.280635999000026],[28.652546500000028,39.14736849800005],[28.15897,39.05698399800008],[27.969201,39.195721],[27.900652,39.348802997000064],[27.800632,39.31310799900007],[27.462255500000026,39.39673249800006],[27.381316,39.35274650000008],[27.064404500000023,39.36679849800004],[26.763921,39.17520999800007],[26.71832450000005,39.25861949800003],[26.60920150000004,39.27264199800004],[26.622667500000034,39.30575349800006],[26.671144500000025,39.27934849600007],[26.858988,39.47728899800006],[26.931722500000035,39.48557849800005],[26.950548500000025,39.559968498000046],[26.618685,39.54787949800004],[26.204252500000052,39.46097399800004],[26.062643,39.479497998000056],[26.163519,39.659459996000066],[26.137392,39.84089499800007],[26.179243,39.99143399700006],[26.321268,40.01747699600003],[26.397505,40.145826498000076],[26.403608500000075,40.19677149900008],[26.524309,40.21770649700005],[26.756845500000054,40.40379899700008],[27.033163,40.38958949900007],[27.286051,40.475088],[27.319921500000078,40.43341249700006],[27.315832,40.37848099800004],[27.50628050000006,40.304677998000045],[27.7619,40.30641749600005],[27.879064500000027,40.372956997000074],[27.686708500000066,40.47253599800007],[27.753523,40.52842149700007],[28.03245,40.480451499000026],[28.01680950000008,40.44744299900003],[27.896269,40.38821199900008]]],[[[27.25369450000005,42.09926199800003],[27.559395,41.904781496000055],[27.859884500000078,42.005878497000026],[28.03551250000004,41.98307949800005],[28.05775250000005,41.883687999000074],[27.997202,41.88834949900007],[27.984159500000032,41.81855199600005],[28.152973,41.57896249600003],[28.181659499000034,41.564058998000064],[28.08682950000008,41.50417799600007],[28.138288,41.42519649800005],[27.964958,41.14141499800007],[27.991102500000068,41.01834499700004],[27.955469,40.96380050000005],[27.763170500000058,41.013876],[27.512853500000062,40.974729496000066],[27.454728999000054,40.84480099700005],[27.309553,40.702420999000026],[26.968651500000078,40.55336899800005],[26.694452500000068,40.45057849800003],[26.592272,40.322187496000026],[26.358301,40.20253199900003],[26.38106350000004,40.143331498000066],[26.218744,40.04709449600006],[26.165872500000035,40.052560999000036],[26.27830699900005,40.21555899800006],[26.218675500000074,40.319448496000064],[26.838197500000035,40.58582499800008],[26.80476550000003,40.65426449600005],[26.734886500000073,40.64293299700006],[26.151155500000073,40.589315499000065],[26.07517450000006,40.61760899800004],[26.032758,40.73025699800007],[26.12489,40.742109998000046],[26.29101550000007,40.93188349800005],[26.359039,40.94717649800003],[26.31295750000004,41.07548049700006],[26.321652,41.250406498000075],[26.628431500000033,41.345533498000066],[26.60012050000006,41.60119899700004],[26.357879,41.711104498000054],[26.328071499000032,41.74837899700003],[26.369060500000046,41.82068249800005],[26.538608500000066,41.823482496000054],[26.584593,41.902652998000065],[26.561544500000025,41.92627349800006],[26.94922850000006,42.00021349800005],[27.059650500000032,42.088336997000056],[27.25369450000005,42.09926199800003]]],[[[25.97707950000006,40.21614249700008],[25.965894500000047,40.15049950000008],[26.013651,40.16370949700007],[25.98987,40.13219250000003],[25.738722,40.09072699800004],[25.665178500000025,40.12435349700007],[25.78412050000003,40.20969949600004],[25.97707950000006,40.21614249700008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR21","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Tekirdag, Edirne, Kirklareli","NUTS_NAME":"Tekirdag, Edirne, Kirklareli"},"geometry":{"type":"Polygon","coordinates":[[[27.25369450000005,42.09926199800003],[27.559395,41.904781496000055],[27.859884500000078,42.005878497000026],[28.03551250000004,41.98307949800005],[28.05775250000005,41.883687999000074],[27.997202,41.88834949900007],[27.984159500000032,41.81855199600005],[28.152973,41.57896249600003],[28.181659499000034,41.564058998000064],[28.08682950000008,41.50417799600007],[28.138288,41.42519649800005],[27.964958,41.14141499800007],[27.991102500000068,41.01834499700004],[27.955469,40.96380050000005],[27.763170500000058,41.013876],[27.512853500000062,40.974729496000066],[27.454728999000054,40.84480099700005],[27.309553,40.702420999000026],[26.968651500000078,40.55336899800005],[26.908618,40.600146497000026],[26.962811,40.66703799800007],[26.967178,40.73651299800008],[26.773845,40.741130998000074],[26.734886500000073,40.64293299700006],[26.151155500000073,40.589315499000065],[26.07517450000006,40.61760899800004],[26.032758,40.73025699800007],[26.12489,40.742109998000046],[26.29101550000007,40.93188349800005],[26.359039,40.94717649800003],[26.31295750000004,41.07548049700006],[26.321652,41.250406498000075],[26.628431500000033,41.345533498000066],[26.60012050000006,41.60119899700004],[26.357879,41.711104498000054],[26.328071499000032,41.74837899700003],[26.369060500000046,41.82068249800005],[26.538608500000066,41.823482496000054],[26.584593,41.902652998000065],[26.561544500000025,41.92627349800006],[26.94922850000006,42.00021349800005],[27.059650500000032,42.088336997000056],[27.25369450000005,42.09926199800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR211","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Tekirdag","NUTS_NAME":"Tekirdag"},"geometry":{"type":"Polygon","coordinates":[[[28.181659499000034,41.564058998000064],[28.08682950000008,41.50417799600007],[28.138288,41.42519649800005],[27.964958,41.14141499800007],[27.991102500000068,41.01834499700004],[27.955469,40.96380050000005],[27.763170500000058,41.013876],[27.512853500000062,40.974729496000066],[27.454728999000054,40.84480099700005],[27.309553,40.702420999000026],[26.968651500000078,40.55336899800005],[26.908618,40.600146497000026],[26.962811,40.66703799800007],[26.967178,40.73651299800008],[26.773845,40.741130998000074],[26.706048500000065,40.82317199800008],[26.717087500000048,41.00452649700003],[26.669465,41.03496899800007],[26.915826,41.17484349800003],[26.932587,41.251641498000026],[26.881272,41.32253050000003],[27.01635150000004,41.34809899900006],[27.510124500000074,41.24701849600007],[27.56750850000003,41.270872498000074],[27.579741,41.366367998000044],[27.644470500000068,41.437103998000055],[28.152973,41.57896249600003],[28.181659499000034,41.564058998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR212","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Edirne","NUTS_NAME":"Edirne"},"geometry":{"type":"Polygon","coordinates":[[[26.881272,41.32253050000003],[26.932587,41.251641498000026],[26.915826,41.17484349800003],[26.669465,41.03496899800007],[26.717087500000048,41.00452649700003],[26.706048500000065,40.82317199800008],[26.773845,40.741130998000074],[26.734886500000073,40.64293299700006],[26.151155500000073,40.589315499000065],[26.07517450000006,40.61760899800004],[26.032758,40.73025699800007],[26.12489,40.742109998000046],[26.29101550000007,40.93188349800005],[26.359039,40.94717649800003],[26.31295750000004,41.07548049700006],[26.321652,41.250406498000075],[26.628431500000033,41.345533498000066],[26.60012050000006,41.60119899700004],[26.357879,41.711104498000054],[26.328071499000032,41.74837899700003],[26.369060500000046,41.82068249800005],[26.538608500000066,41.823482496000054],[26.584593,41.902652998000065],[26.561544500000025,41.92627349800006],[26.94922850000006,42.00021349800005],[26.981257500000027,41.76139899800006],[26.89475,41.63524699900006],[26.97930450000007,41.605826998000055],[26.866486,41.410487499000055],[26.881272,41.32253050000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR213","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kirklareli","NUTS_NAME":"Kirklareli"},"geometry":{"type":"Polygon","coordinates":[[[28.03551250000004,41.98307949800005],[28.05775250000005,41.883687999000074],[27.997202,41.88834949900007],[27.984159500000032,41.81855199600005],[28.152973,41.57896249600003],[27.644470500000068,41.437103998000055],[27.579741,41.366367998000044],[27.56750850000003,41.270872498000074],[27.510124500000074,41.24701849600007],[27.01635150000004,41.34809899900006],[26.881272,41.32253050000003],[26.866486,41.410487499000055],[26.97930450000007,41.605826998000055],[26.89475,41.63524699900006],[26.981257500000027,41.76139899800006],[26.94922850000006,42.00021349800005],[27.059650500000032,42.088336997000056],[27.25369450000005,42.09926199800003],[27.559395,41.904781496000055],[27.859884500000078,42.005878497000026],[28.03551250000004,41.98307949800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR22","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Balikesir, Çanakkale","NUTS_NAME":"Balikesir, Çanakkale"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.896269,40.38821199900008],[27.954508,40.35312049700008],[28.164779,40.395612999000036],[28.06755050000004,40.25313399800007],[28.265037500000062,39.87023399700007],[28.515012,39.73130899700004],[28.69535250000007,39.757907498000066],[28.793313500000068,39.61746949700006],[28.968574,39.60067749800004],[28.885427,39.372048997000036],[28.597988,39.280635999000026],[28.652546500000028,39.14736849800005],[28.15897,39.05698399800008],[27.969201,39.195721],[27.900652,39.348802997000064],[27.800632,39.31310799900007],[27.462255500000026,39.39673249800006],[27.381316,39.35274650000008],[27.064404500000023,39.36679849800004],[26.763921,39.17520999800007],[26.71832450000005,39.25861949800003],[26.60920150000004,39.27264199800004],[26.622667500000034,39.30575349800006],[26.671144500000025,39.27934849600007],[26.858988,39.47728899800006],[26.931722500000035,39.48557849800005],[26.950548500000025,39.559968498000046],[26.618685,39.54787949800004],[26.204252500000052,39.46097399800004],[26.062643,39.479497998000056],[26.163519,39.659459996000066],[26.137392,39.84089499800007],[26.179243,39.99143399700006],[26.321268,40.01747699600003],[26.397505,40.145826498000076],[26.403608500000075,40.19677149900008],[26.524309,40.21770649700005],[26.756845500000054,40.40379899700008],[27.033163,40.38958949900007],[27.286051,40.475088],[27.319921500000078,40.43341249700006],[27.315832,40.37848099800004],[27.50628050000006,40.304677998000045],[27.7619,40.30641749600005],[27.879064500000027,40.372956997000074],[27.686708500000066,40.47253599800007],[27.753523,40.52842149700007],[28.03245,40.480451499000026],[28.01680950000008,40.44744299900003],[27.896269,40.38821199900008]]],[[[26.962811,40.66703799800007],[26.908618,40.600146497000026],[26.968651500000078,40.55336899800005],[26.694452500000068,40.45057849800003],[26.592272,40.322187496000026],[26.358301,40.20253199900003],[26.38106350000004,40.143331498000066],[26.218744,40.04709449600006],[26.165872500000035,40.052560999000036],[26.27830699900005,40.21555899800006],[26.218675500000074,40.319448496000064],[26.838197500000035,40.58582499800008],[26.80476550000003,40.65426449600005],[26.734886500000073,40.64293299700006],[26.773845,40.741130998000074],[26.967178,40.73651299800008],[26.962811,40.66703799800007]]],[[[25.97707950000006,40.21614249700008],[25.965894500000047,40.15049950000008],[26.013651,40.16370949700007],[25.98987,40.13219250000003],[25.738722,40.09072699800004],[25.665178500000025,40.12435349700007],[25.78412050000003,40.20969949600004],[25.97707950000006,40.21614249700008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR221","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Balikesir","NUTS_NAME":"Balikesir"},"geometry":{"type":"Polygon","coordinates":[[[28.01680950000008,40.44744299900003],[27.896269,40.38821199900008],[27.954508,40.35312049700008],[28.164779,40.395612999000036],[28.06755050000004,40.25313399800007],[28.265037500000062,39.87023399700007],[28.515012,39.73130899700004],[28.69535250000007,39.757907498000066],[28.793313500000068,39.61746949700006],[28.968574,39.60067749800004],[28.885427,39.372048997000036],[28.597988,39.280635999000026],[28.652546500000028,39.14736849800005],[28.15897,39.05698399800008],[27.969201,39.195721],[27.900652,39.348802997000064],[27.800632,39.31310799900007],[27.462255500000026,39.39673249800006],[27.381316,39.35274650000008],[27.064404500000023,39.36679849800004],[26.763921,39.17520999800007],[26.71832450000005,39.25861949800003],[26.60920150000004,39.27264199800004],[26.622667500000034,39.30575349800006],[26.671144500000025,39.27934849600007],[26.858988,39.47728899800006],[26.931722500000035,39.48557849800005],[26.950548500000025,39.559968498000046],[26.618685,39.54787949800004],[26.713539500000024,39.70721849800003],[26.967601500000058,39.76314799700003],[27.23951550000004,39.68220249900003],[27.430025500000056,39.77245499700007],[27.489155500000038,39.89518149700007],[27.44819750000005,40.04782],[27.363447499000074,40.146663998000065],[27.50628050000006,40.304677998000045],[27.7619,40.30641749600005],[27.879064500000027,40.372956997000074],[27.686708500000066,40.47253599800007],[27.753523,40.52842149700007],[28.03245,40.480451499000026],[28.01680950000008,40.44744299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO22","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Sud-Est","NUTS_NAME":"Sud-Est"},"geometry":{"type":"Polygon","coordinates":[[[27.306986,46.20606399900004],[27.350005500000066,46.15270549600007],[27.38783250000006,46.19381849600006],[27.492818500000055,46.159212998000044],[27.503053,46.139620998000055],[27.562554500000033,46.002876997000044],[27.62768250000005,46.06138249800006],[27.610243,46.13624849800004],[28.115802,46.10782699600003],[28.08719750000006,46.00968099800008],[28.170279,45.67490599800004],[28.088561500000026,45.60611249800007],[28.21136,45.46727999800004],[28.28599,45.43063],[28.34137,45.325049998000054],[28.717,45.224379998000074],[28.793709999000043,45.24397999800004],[28.79881,45.335729997000044],[28.9442,45.28135],[29.21645,45.419339998000055],[29.42759,45.44223999800005],[29.654986500000064,45.33608449600007],[29.679639500000064,45.21183799900007],[29.631290500000034,45.20977],[29.71331950000007,45.16507299600005],[29.653199500000028,45.09557799600003],[29.600269,44.83955449800004],[29.185992500000054,44.78466350000008],[28.994085,44.679625497000075],[28.625982500000077,44.29703199700003],[28.667546,43.98677249800005],[28.578884,43.73873899900008],[28.236143,43.75852199800005],[27.994264500000043,43.842964497000025],[27.916834,44.00827399700006],[27.737999,43.95341499700004],[27.69541,43.98734299900008],[27.643537500000036,44.047972998000034],[27.399397500000077,44.012369998000054],[27.271344999000064,44.12633649800006],[27.926198500000055,44.248121498000046],[28.01737399900003,44.34024899600007],[28.110168500000043,44.439703500000064],[28.016514,44.56481799900007],[28.031737500000077,44.62529149800008],[27.882344,44.707761498000025],[27.881007,44.76307649800003],[27.818804,44.82033499700003],[27.721871,44.765049],[27.203458500000067,44.78707399800004],[27.134057,44.73553449700006],[27.007946,44.82446649600007],[26.810180500000058,44.79534150000006],[26.605551500000047,44.856993997000075],[26.530521500000077,44.937532496000074],[26.599910500000078,44.95584449900008],[26.451353,45.08330349800008],[26.46035,45.19809199700006],[26.376487500000053,45.158456999000066],[26.249952499000074,45.212642996000056],[26.18549850000005,45.27803449600003],[26.233643,45.314494499000034],[26.153438,45.37709149600005],[26.14896550000003,45.45715549800008],[26.072718500000065,45.50578799800007],[26.093334500000026,45.51630599600003],[26.212819,45.621830997000075],[26.310291500000062,45.606565997000075],[26.39209450000004,45.802389496000046],[26.37564499900003,45.90731349600003],[26.439984500000037,46.038876498000036],[27.10561850000005,46.08868949600003],[27.123597,46.147470998000074],[27.306986,46.20606399900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO221","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Braila","NUTS_NAME":"Braila"},"geometry":{"type":"Polygon","coordinates":[[[28.03051750000003,45.40432799800004],[27.99481050000003,45.286741996000046],[28.11604650000004,45.25799999700007],[28.091365499000062,45.19024850000005],[28.19764850000007,45.136866498000074],[28.136162,45.10268749900007],[28.128102,44.902157997000074],[28.07410850000008,44.80388699800005],[27.881007,44.76307649800003],[27.818804,44.82033499700003],[27.721871,44.765049],[27.203458500000067,44.78707399800004],[27.280107,44.997640999000055],[27.197532,45.02803649900005],[27.149556500000074,45.17121350000008],[27.066489500000046,45.158743498000035],[27.175062,45.249329998000064],[27.35006149900005,45.21221799600005],[27.312494,45.29218899800003],[27.390138,45.286676499000066],[27.434726500000068,45.39470199800007],[27.558515500000055,45.48837549700005],[27.664150500000062,45.48978999600007],[27.741207,45.41420399700007],[28.03051750000003,45.40432799800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO222","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Buzau","NUTS_NAME":"Buzau"},"geometry":{"type":"Polygon","coordinates":[[[27.434726500000068,45.39470199800007],[27.390138,45.286676499000066],[27.312494,45.29218899800003],[27.35006149900005,45.21221799600005],[27.175062,45.249329998000064],[27.066489500000046,45.158743498000035],[27.149556500000074,45.17121350000008],[27.197532,45.02803649900005],[27.280107,44.997640999000055],[27.203458500000067,44.78707399800004],[27.134057,44.73553449700006],[27.007946,44.82446649600007],[26.810180500000058,44.79534150000006],[26.605551500000047,44.856993997000075],[26.530521500000077,44.937532496000074],[26.599910500000078,44.95584449900008],[26.451353,45.08330349800008],[26.46035,45.19809199700006],[26.376487500000053,45.158456999000066],[26.249952499000074,45.212642996000056],[26.18549850000005,45.27803449600003],[26.233643,45.314494499000034],[26.153438,45.37709149600005],[26.14896550000003,45.45715549800008],[26.072718500000065,45.50578799800007],[26.093334500000026,45.51630599600003],[26.212819,45.621830997000075],[26.310291500000062,45.606565997000075],[26.39209450000004,45.802389496000046],[26.517906,45.65124149800005],[26.748325,45.54866899700005],[27.15825250000006,45.46520049800006],[27.115936,45.409234499000036],[27.194376,45.45336499800004],[27.33659250000005,45.37132799900007],[27.434726500000068,45.39470199800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO223","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Constanta","NUTS_NAME":"Constanta"},"geometry":{"type":"Polygon","coordinates":[[[28.994085,44.679625497000075],[28.625982500000077,44.29703199700003],[28.667546,43.98677249800005],[28.578884,43.73873899900008],[28.236143,43.75852199800005],[27.994264500000043,43.842964497000025],[27.916834,44.00827399700006],[27.737999,43.95341499700004],[27.69541,43.98734299900008],[27.643537500000036,44.047972998000034],[27.399397500000077,44.012369998000054],[27.271344999000064,44.12633649800006],[27.926198500000055,44.248121498000046],[28.01737399900003,44.34024899600007],[28.110168500000043,44.439703500000064],[28.016514,44.56481799900007],[28.031737500000077,44.62529149800008],[27.882344,44.707761498000025],[27.881007,44.76307649800003],[28.07410850000008,44.80388699800005],[28.239861,44.76468550000004],[28.23283,44.70903249600008],[28.365502500000048,44.69159699800008],[28.421229,44.61598649700005],[28.994085,44.679625497000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO224","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Galati","NUTS_NAME":"Galati"},"geometry":{"type":"Polygon","coordinates":[[[28.115802,46.10782699600003],[28.08719750000006,46.00968099800008],[28.170279,45.67490599800004],[28.088561500000026,45.60611249800007],[28.21136,45.46727999800004],[28.19216350000005,45.41532099900007],[28.03051750000003,45.40432799800004],[27.741207,45.41420399700007],[27.664150500000062,45.48978999600007],[27.558515500000055,45.48837549700005],[27.51306050000005,45.62099450000005],[27.362351500000045,45.72936999800004],[27.227019,46.00912649900005],[27.296287,46.08445999700007],[27.503053,46.139620998000055],[27.562554500000033,46.002876997000044],[27.62768250000005,46.06138249800006],[27.610243,46.13624849800004],[28.115802,46.10782699600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO225","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Tulcea","NUTS_NAME":"Tulcea"},"geometry":{"type":"Polygon","coordinates":[[[28.28599,45.43063],[28.34137,45.325049998000054],[28.717,45.224379998000074],[28.793709999000043,45.24397999800004],[28.79881,45.335729997000044],[28.9442,45.28135],[29.21645,45.419339998000055],[29.42759,45.44223999800005],[29.654986500000064,45.33608449600007],[29.679639500000064,45.21183799900007],[29.631290500000034,45.20977],[29.71331950000007,45.16507299600005],[29.653199500000028,45.09557799600003],[29.600269,44.83955449800004],[29.185992500000054,44.78466350000008],[28.994085,44.679625497000075],[28.421229,44.61598649700005],[28.365502500000048,44.69159699800008],[28.23283,44.70903249600008],[28.239861,44.76468550000004],[28.07410850000008,44.80388699800005],[28.128102,44.902157997000074],[28.136162,45.10268749900007],[28.19764850000007,45.136866498000074],[28.091365499000062,45.19024850000005],[28.11604650000004,45.25799999700007],[27.99481050000003,45.286741996000046],[28.03051750000003,45.40432799800004],[28.19216350000005,45.41532099900007],[28.21136,45.46727999800004],[28.28599,45.43063]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO226","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Vrancea","NUTS_NAME":"Vrancea"},"geometry":{"type":"Polygon","coordinates":[[[27.492818500000055,46.159212998000044],[27.503053,46.139620998000055],[27.296287,46.08445999700007],[27.227019,46.00912649900005],[27.362351500000045,45.72936999800004],[27.51306050000005,45.62099450000005],[27.558515500000055,45.48837549700005],[27.434726500000068,45.39470199800007],[27.33659250000005,45.37132799900007],[27.194376,45.45336499800004],[27.115936,45.409234499000036],[27.15825250000006,45.46520049800006],[26.748325,45.54866899700005],[26.517906,45.65124149800005],[26.39209450000004,45.802389496000046],[26.37564499900003,45.90731349600003],[26.439984500000037,46.038876498000036],[27.10561850000005,46.08868949600003],[27.123597,46.147470998000074],[27.306986,46.20606399900004],[27.350005500000066,46.15270549600007],[27.38783250000006,46.19381849600006],[27.492818500000055,46.159212998000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO3","LEVL_CODE":1,"CNTR_CODE":"RO","NAME_LATN":"MACROREGIUNEA TREI","NUTS_NAME":"MACROREGIUNEA TREI"},"geometry":{"type":"Polygon","coordinates":[[[25.25536850000003,45.42249349800005],[25.321577500000046,45.38108899700006],[25.452539,45.44134199800004],[25.680995,45.50061949800005],[25.86003850000003,45.421914998000034],[25.899684,45.49885549800007],[26.072718500000065,45.50578799800007],[26.14896550000003,45.45715549800008],[26.153438,45.37709149600005],[26.233643,45.314494499000034],[26.18549850000005,45.27803449600003],[26.249952499000074,45.212642996000056],[26.376487500000053,45.158456999000066],[26.46035,45.19809199700006],[26.451353,45.08330349800008],[26.599910500000078,44.95584449900008],[26.530521500000077,44.937532496000074],[26.605551500000047,44.856993997000075],[26.810180500000058,44.79534150000006],[27.007946,44.82446649600007],[27.134057,44.73553449700006],[27.203458500000067,44.78707399800004],[27.721871,44.765049],[27.818804,44.82033499700003],[27.881007,44.76307649800003],[27.882344,44.707761498000025],[28.031737500000077,44.62529149800008],[28.016514,44.56481799900007],[28.110168500000043,44.439703500000064],[28.01737399900003,44.34024899600007],[27.926198500000055,44.248121498000046],[27.271344999000064,44.12633649800006],[26.906678500000055,44.133331998000074],[26.379968,44.04295099800004],[26.358359999000072,44.03841550000004],[26.114924,43.97408949700008],[25.783091,43.71223699700005],[25.67186250000003,43.691339998000046],[25.54464050000007,43.64298149600006],[25.29374050000007,43.65429399900006],[24.641605999000035,43.73341299800006],[24.755096,43.817601498000045],[24.602269,43.962112999000055],[24.61309249900006,44.014155997000046],[24.855496,44.091871500000025],[24.793566,44.191254498000035],[24.817085500000076,44.361135998000066],[24.88401950000008,44.38263649600003],[24.744521,44.395482997000045],[24.68812,44.69849499800006],[24.721729500000038,44.71459699800005],[24.593776500000047,44.75087149600006],[24.540722500000072,44.89103999800005],[24.480183,44.82174099800005],[24.438121,44.84537549800007],[24.520454500000028,45.048977498000056],[24.480691499000045,45.449102],[24.540518500000076,45.53569699600007],[24.512834500000054,45.586824998000054],[24.684925500000077,45.60411099600003],[25.103214,45.58519399800008],[25.229541,45.517015997000044],[25.25536850000003,45.42249349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO31","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Sud - Muntenia","NUTS_NAME":"Sud - Muntenia"},"geometry":{"type":"Polygon","coordinates":[[[25.25536850000003,45.42249349800005],[25.321577500000046,45.38108899700006],[25.452539,45.44134199800004],[25.680995,45.50061949800005],[25.86003850000003,45.421914998000034],[25.899684,45.49885549800007],[26.072718500000065,45.50578799800007],[26.14896550000003,45.45715549800008],[26.153438,45.37709149600005],[26.233643,45.314494499000034],[26.18549850000005,45.27803449600003],[26.249952499000074,45.212642996000056],[26.376487500000053,45.158456999000066],[26.46035,45.19809199700006],[26.451353,45.08330349800008],[26.599910500000078,44.95584449900008],[26.530521500000077,44.937532496000074],[26.605551500000047,44.856993997000075],[26.810180500000058,44.79534150000006],[27.007946,44.82446649600007],[27.134057,44.73553449700006],[27.203458500000067,44.78707399800004],[27.721871,44.765049],[27.818804,44.82033499700003],[27.881007,44.76307649800003],[27.882344,44.707761498000025],[28.031737500000077,44.62529149800008],[28.016514,44.56481799900007],[28.110168500000043,44.439703500000064],[28.01737399900003,44.34024899600007],[27.926198500000055,44.248121498000046],[27.271344999000064,44.12633649800006],[26.906678500000055,44.133331998000074],[26.379968,44.04295099800004],[26.358359999000072,44.03841550000004],[26.114924,43.97408949700008],[25.783091,43.71223699700005],[25.67186250000003,43.691339998000046],[25.54464050000007,43.64298149600006],[25.29374050000007,43.65429399900006],[24.641605999000035,43.73341299800006],[24.755096,43.817601498000045],[24.602269,43.962112999000055],[24.61309249900006,44.014155997000046],[24.855496,44.091871500000025],[24.793566,44.191254498000035],[24.817085500000076,44.361135998000066],[24.88401950000008,44.38263649600003],[24.744521,44.395482997000045],[24.68812,44.69849499800006],[24.721729500000038,44.71459699800005],[24.593776500000047,44.75087149600006],[24.540722500000072,44.89103999800005],[24.480183,44.82174099800005],[24.438121,44.84537549800007],[24.520454500000028,45.048977498000056],[24.480691499000045,45.449102],[24.540518500000076,45.53569699600007],[24.512834500000054,45.586824998000054],[24.684925500000077,45.60411099600003],[25.103214,45.58519399800008],[25.229541,45.517015997000044],[25.25536850000003,45.42249349800005]],[[25.87001850000007,44.35740049800006],[26.145374,44.234496997000065],[26.259956500000044,44.29498599800007],[26.33095650000007,44.37169149600004],[26.26002950000003,44.409247],[26.451718,44.44428049800007],[26.360542500000065,44.48609999800004],[26.41606050000007,44.521682998000074],[26.337175,44.54750949800007],[26.401260500000035,44.611001],[26.301818500000024,44.76887499900005],[25.970744,44.71040850000003],[25.99199150000004,44.61231249800005],[25.890739500000052,44.54095299800008],[25.91392,44.47650699800005],[25.823709500000064,44.398084],[25.87001850000007,44.35740049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO311","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Arges","NUTS_NAME":"Arges"},"geometry":{"type":"Polygon","coordinates":[[[25.321577500000046,45.38108899700006],[25.29370650000004,45.23208149800007],[25.211846500000036,45.15661249800007],[25.15638350000006,45.20164649800006],[25.124680500000068,45.07050799800004],[25.194397,45.023382498000046],[25.161313500000063,44.921003496000026],[25.197548,44.90327849600004],[25.200511,44.740812498000025],[25.237266500000032,44.69475349800007],[25.205002500000035,44.63285799800008],[25.292579999000054,44.55144499800008],[25.279671500000063,44.44445499600005],[25.234749500000078,44.424251998000045],[24.88401950000008,44.38263649600003],[24.744521,44.395482997000045],[24.68812,44.69849499800006],[24.721729500000038,44.71459699800005],[24.593776500000047,44.75087149600006],[24.540722500000072,44.89103999800005],[24.480183,44.82174099800005],[24.438121,44.84537549800007],[24.520454500000028,45.048977498000056],[24.480691499000045,45.449102],[24.540518500000076,45.53569699600007],[24.512834500000054,45.586824998000054],[24.684925500000077,45.60411099600003],[25.103214,45.58519399800008],[25.229541,45.517015997000044],[25.25536850000003,45.42249349800005],[25.321577500000046,45.38108899700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO312","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Calarasi","NUTS_NAME":"Calarasi"},"geometry":{"type":"Polygon","coordinates":[[[28.01737399900003,44.34024899600007],[27.926198500000055,44.248121498000046],[27.271344999000064,44.12633649800006],[26.906678500000055,44.133331998000074],[26.379968,44.04295099800004],[26.35876,44.10739499600004],[26.428482,44.14700849600007],[26.400423499000055,44.224207500000034],[26.259956500000044,44.29498599800007],[26.33095650000007,44.37169149600004],[26.26002950000003,44.409247],[26.451718,44.44428049800007],[26.360542500000065,44.48609999800004],[26.41606050000007,44.521682998000074],[26.479835499000046,44.57650349600004],[26.599343500000032,44.523067996000066],[26.683809,44.591050996000035],[26.779263,44.52140699800003],[27.13549,44.48302049800003],[27.518504,44.497145997000075],[27.57194750000008,44.55907399600005],[27.570191500000078,44.467806996000036],[27.732860500000072,44.417695999000046],[27.762474,44.347751996000056],[28.01737399900003,44.34024899600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO313","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Dâmbovita","NUTS_NAME":"Dâmbovita"},"geometry":{"type":"Polygon","coordinates":[[[25.970744,44.71040850000003],[25.99199150000004,44.61231249800005],[25.890739500000052,44.54095299800008],[25.495685999000045,44.51968149800007],[25.480489,44.451997997000035],[25.395094,44.51813099900005],[25.302263500000038,44.42370899600007],[25.250729,44.403355497000064],[25.234749500000078,44.424251998000045],[25.279671500000063,44.44445499600005],[25.292579999000054,44.55144499800008],[25.205002500000035,44.63285799800008],[25.237266500000032,44.69475349800007],[25.200511,44.740812498000025],[25.197548,44.90327849600004],[25.161313500000063,44.921003496000026],[25.194397,45.023382498000046],[25.124680500000068,45.07050799800004],[25.15638350000006,45.20164649800006],[25.211846500000036,45.15661249800007],[25.29370650000004,45.23208149800007],[25.321577500000046,45.38108899700006],[25.452539,45.44134199800004],[25.475116,45.32741499800005],[25.554831500000034,45.270550498000034],[25.51093650000007,45.22224649800006],[25.701677,45.04496099800008],[25.68808450000006,44.94729949800006],[25.791381500000057,44.94468899800006],[25.825313,44.897138],[25.77762050000007,44.82464299600008],[25.870943,44.83172649900007],[25.970744,44.71040850000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO314","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Giurgiu","NUTS_NAME":"Giurgiu"},"geometry":{"type":"Polygon","coordinates":[[[25.890739500000052,44.54095299800008],[25.91392,44.47650699800005],[25.823709500000064,44.398084],[25.87001850000007,44.35740049800006],[26.145374,44.234496997000065],[26.259956500000044,44.29498599800007],[26.400423499000055,44.224207500000034],[26.428482,44.14700849600007],[26.35876,44.10739499600004],[26.379968,44.04295099800004],[26.358359999000072,44.03841550000004],[26.114924,43.97408949700008],[25.783091,43.71223699700005],[25.67186250000003,43.691339998000046],[25.65993799900008,43.820491996000044],[25.60689,43.84447049800008],[25.584797,43.93758899800008],[25.598721500000067,44.03768949900007],[25.678987,44.12664949700007],[25.68401250000005,44.238902],[25.468115,44.32524599700008],[25.480489,44.451997997000035],[25.495685999000045,44.51968149800007],[25.890739500000052,44.54095299800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO315","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Ialomita","NUTS_NAME":"Ialomita"},"geometry":{"type":"Polygon","coordinates":[[[27.203458500000067,44.78707399800004],[27.721871,44.765049],[27.818804,44.82033499700003],[27.881007,44.76307649800003],[27.882344,44.707761498000025],[28.031737500000077,44.62529149800008],[28.016514,44.56481799900007],[28.110168500000043,44.439703500000064],[28.01737399900003,44.34024899600007],[27.762474,44.347751996000056],[27.732860500000072,44.417695999000046],[27.570191500000078,44.467806996000036],[27.57194750000008,44.55907399600005],[27.518504,44.497145997000075],[27.13549,44.48302049800003],[26.779263,44.52140699800003],[26.683809,44.591050996000035],[26.599343500000032,44.523067996000066],[26.479835499000046,44.57650349600004],[26.41606050000007,44.521682998000074],[26.337175,44.54750949800007],[26.401260500000035,44.611001],[26.301818500000024,44.76887499900005],[26.605551500000047,44.856993997000075],[26.810180500000058,44.79534150000006],[27.007946,44.82446649600007],[27.134057,44.73553449700006],[27.203458500000067,44.78707399800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO316","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Prahova","NUTS_NAME":"Prahova"},"geometry":{"type":"Polygon","coordinates":[[[26.072718500000065,45.50578799800007],[26.14896550000003,45.45715549800008],[26.153438,45.37709149600005],[26.233643,45.314494499000034],[26.18549850000005,45.27803449600003],[26.249952499000074,45.212642996000056],[26.376487500000053,45.158456999000066],[26.46035,45.19809199700006],[26.451353,45.08330349800008],[26.599910500000078,44.95584449900008],[26.530521500000077,44.937532496000074],[26.605551500000047,44.856993997000075],[26.301818500000024,44.76887499900005],[25.970744,44.71040850000003],[25.870943,44.83172649900007],[25.77762050000007,44.82464299600008],[25.825313,44.897138],[25.791381500000057,44.94468899800006],[25.68808450000006,44.94729949800006],[25.701677,45.04496099800008],[25.51093650000007,45.22224649800006],[25.554831500000034,45.270550498000034],[25.475116,45.32741499800005],[25.452539,45.44134199800004],[25.680995,45.50061949800005],[25.86003850000003,45.421914998000034],[25.899684,45.49885549800007],[26.072718500000065,45.50578799800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO317","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Teleorman","NUTS_NAME":"Teleorman"},"geometry":{"type":"Polygon","coordinates":[[[25.480489,44.451997997000035],[25.468115,44.32524599700008],[25.68401250000005,44.238902],[25.678987,44.12664949700007],[25.598721500000067,44.03768949900007],[25.584797,43.93758899800008],[25.60689,43.84447049800008],[25.65993799900008,43.820491996000044],[25.67186250000003,43.691339998000046],[25.54464050000007,43.64298149600006],[25.29374050000007,43.65429399900006],[24.641605999000035,43.73341299800006],[24.755096,43.817601498000045],[24.602269,43.962112999000055],[24.61309249900006,44.014155997000046],[24.855496,44.091871500000025],[24.793566,44.191254498000035],[24.817085500000076,44.361135998000066],[24.88401950000008,44.38263649600003],[25.234749500000078,44.424251998000045],[25.250729,44.403355497000064],[25.302263500000038,44.42370899600007],[25.395094,44.51813099900005],[25.480489,44.451997997000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO32","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Bucuresti - Ilfov","NUTS_NAME":"Bucuresti - Ilfov"},"geometry":{"type":"Polygon","coordinates":[[[26.259956500000044,44.29498599800007],[26.145374,44.234496997000065],[25.87001850000007,44.35740049800006],[25.823709500000064,44.398084],[25.91392,44.47650699800005],[25.890739500000052,44.54095299800008],[25.99199150000004,44.61231249800005],[25.970744,44.71040850000003],[26.301818500000024,44.76887499900005],[26.401260500000035,44.611001],[26.337175,44.54750949800007],[26.41606050000007,44.521682998000074],[26.360542500000065,44.48609999800004],[26.451718,44.44428049800007],[26.26002950000003,44.409247],[26.33095650000007,44.37169149600004],[26.259956500000044,44.29498599800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO321","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Bucuresti","NUTS_NAME":"Bucuresti"},"geometry":{"type":"Polygon","coordinates":[[[26.225549500000056,44.43240699800003],[26.151923,44.33424749900007],[25.966651999000078,44.44072549800006],[26.06857449900008,44.53484949800003],[26.225549500000056,44.43240699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO322","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Ilfov","NUTS_NAME":"Ilfov"},"geometry":{"type":"Polygon","coordinates":[[[26.41606050000007,44.521682998000074],[26.360542500000065,44.48609999800004],[26.451718,44.44428049800007],[26.26002950000003,44.409247],[26.33095650000007,44.37169149600004],[26.259956500000044,44.29498599800007],[26.145374,44.234496997000065],[25.87001850000007,44.35740049800006],[25.823709500000064,44.398084],[25.91392,44.47650699800005],[25.890739500000052,44.54095299800008],[25.99199150000004,44.61231249800005],[25.970744,44.71040850000003],[26.301818500000024,44.76887499900005],[26.401260500000035,44.611001],[26.337175,44.54750949800007],[26.41606050000007,44.521682998000074]],[[26.225549500000056,44.43240699800003],[26.06857449900008,44.53484949800003],[25.966651999000078,44.44072549800006],[26.151923,44.33424749900007],[26.225549500000056,44.43240699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO4","LEVL_CODE":1,"CNTR_CODE":"RO","NAME_LATN":"MACROREGIUNEA PATRU","NUTS_NAME":"MACROREGIUNEA PATRU"},"geometry":{"type":"Polygon","coordinates":[[[22.168398,46.60386349800007],[22.197508,46.51382549600004],[22.32517050000007,46.51450699700007],[22.424138,46.39332049600006],[22.67657550000007,46.405825496000034],[22.748716,46.351207499000054],[22.925720500000068,46.330588996000074],[22.96113450000007,46.23682549600005],[23.095532500000047,46.18230050000005],[23.055348500000036,46.140611496000076],[23.087312,46.06646149800008],[23.234864,46.030501996000055],[23.31906,45.90771499600004],[23.41857199900005,45.59558899800004],[23.597270500000036,45.47347149800004],[23.703613,45.496772],[24.512834500000054,45.586824998000054],[24.540518500000076,45.53569699600007],[24.480691499000045,45.449102],[24.520454500000028,45.048977498000056],[24.438121,44.84537549800007],[24.480183,44.82174099800005],[24.540722500000072,44.89103999800005],[24.593776500000047,44.75087149600006],[24.721729500000038,44.71459699800005],[24.68812,44.69849499800006],[24.744521,44.395482997000045],[24.88401950000008,44.38263649600003],[24.817085500000076,44.361135998000066],[24.793566,44.191254498000035],[24.855496,44.091871500000025],[24.61309249900006,44.014155997000046],[24.602269,43.962112999000055],[24.755096,43.817601498000045],[24.641605999000035,43.73341299800006],[24.501014500000053,43.762396996000064],[24.324132,43.69956799600004],[24.112719,43.699566497000035],[23.63008,43.791259997000054],[23.267777,43.846812],[22.99715450000008,43.80762699600007],[22.838719,43.877852],[22.878725,43.98156750000004],[23.045898,44.06374949900004],[22.966399500000023,44.09852499600004],[22.67516150000006,44.21566299600005],[22.677534500000036,44.280286498000066],[22.560019500000067,44.30736649600004],[22.457399500000065,44.46746649900007],[22.548219500000073,44.48124899800007],[22.58179350000006,44.548156996000046],[22.762785,44.55141149700006],[22.70595650000007,44.60321649800005],[22.467334,44.71474549900006],[22.332951,44.67587199800005],[22.159579500000063,44.47177749800005],[22.016132500000026,44.59920249800007],[22.012350500000025,44.602318498000045],[21.640041500000052,44.659280498000044],[21.587786,44.75841599800003],[21.35847,44.82161199600006],[21.367058,44.86448149600005],[21.560126500000024,44.88900649900006],[21.36023349900006,45.02043299800005],[21.456053500000053,45.04039949600008],[21.527420500000062,45.13786699600007],[21.479178500000046,45.19302750000003],[21.218988,45.250575998000045],[21.176224500000046,45.32522649800006],[21.016575,45.32462749700005],[20.86616199900004,45.46523399800003],[20.771111500000075,45.47996649700008],[20.832577,45.53589299600003],[20.765725,45.61256849700004],[20.823079999000072,45.77506449900005],[20.70064650000006,45.75056199900007],[20.662833,45.79411599800005],[20.569299,45.91013849800004],[20.348325500000044,45.99962699800005],[20.264296,46.12637349900007],[20.501525,46.18993599800007],[20.6406,46.12720699800008],[20.70530350000007,46.16093749800007],[20.7756,46.27590999800003],[21.103170500000033,46.26259049600003],[21.206304,46.40323099600005],[21.296291,46.40669299800004],[21.317476,46.45065599600008],[21.260401500000057,46.501945996000075],[21.314121999000065,46.617556],[21.441398,46.651466999000036],[22.026914,46.64379299600006],[22.168398,46.60386349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO41","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Sud-Vest Oltenia","NUTS_NAME":"Sud-Vest Oltenia"},"geometry":{"type":"Polygon","coordinates":[[[24.480691499000045,45.449102],[24.520454500000028,45.048977498000056],[24.438121,44.84537549800007],[24.480183,44.82174099800005],[24.540722500000072,44.89103999800005],[24.593776500000047,44.75087149600006],[24.721729500000038,44.71459699800005],[24.68812,44.69849499800006],[24.744521,44.395482997000045],[24.88401950000008,44.38263649600003],[24.817085500000076,44.361135998000066],[24.793566,44.191254498000035],[24.855496,44.091871500000025],[24.61309249900006,44.014155997000046],[24.602269,43.962112999000055],[24.755096,43.817601498000045],[24.641605999000035,43.73341299800006],[24.501014500000053,43.762396996000064],[24.324132,43.69956799600004],[24.112719,43.699566497000035],[23.63008,43.791259997000054],[23.267777,43.846812],[22.99715450000008,43.80762699600007],[22.838719,43.877852],[22.878725,43.98156750000004],[23.045898,44.06374949900004],[22.966399500000023,44.09852499600004],[22.67516150000006,44.21566299600005],[22.677534500000036,44.280286498000066],[22.560019500000067,44.30736649600004],[22.457399500000065,44.46746649900007],[22.548219500000073,44.48124899800007],[22.58179350000006,44.548156996000046],[22.762785,44.55141149700006],[22.70595650000007,44.60321649800005],[22.467334,44.71474549900006],[22.332951,44.67587199800005],[22.159579500000063,44.47177749800005],[22.016132500000026,44.59920249800007],[22.154162,44.59519949600008],[22.146892500000035,44.70075699800003],[22.207799,44.81687849600007],[22.418032,44.756857498000045],[22.492156,44.97327249700004],[22.656632,45.10816699600008],[22.559166,45.15686449800006],[22.68624650000004,45.257960998000044],[23.009152,45.253561998000066],[23.377566,45.33844749600007],[23.443675,45.29751599800005],[23.59064,45.353072498000074],[23.597270500000036,45.47347149800004],[23.703613,45.496772],[24.512834500000054,45.586824998000054],[24.540518500000076,45.53569699600007],[24.480691499000045,45.449102]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO411","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Dolj","NUTS_NAME":"Dolj"},"geometry":{"type":"Polygon","coordinates":[[[23.78049949900003,44.71398650000003],[23.928197,44.53022549600007],[23.885288,44.51205149900005],[23.89047850000003,44.41279099800005],[24.226886999000044,44.22991749600004],[24.222607500000038,44.069267996000065],[24.181231,44.068599996000046],[24.173538500000063,44.00629699900003],[24.26249850000005,43.987319496000055],[24.226797,43.85174899800006],[24.155538500000034,43.81371099900008],[24.112719,43.699566497000035],[23.63008,43.791259997000054],[23.267777,43.846812],[22.99715450000008,43.80762699600007],[22.838719,43.877852],[22.878725,43.98156750000004],[23.045898,44.06374949900004],[22.966399500000023,44.09852499600004],[23.00536,44.157920500000046],[23.109875,44.15014799700003],[23.09368550000005,44.26211899800006],[23.218123,44.37553649800003],[23.166413500000033,44.41213799800005],[23.279467500000067,44.50152849800003],[23.25519,44.530020498000056],[23.455035500000065,44.56956149800004],[23.707695500000057,44.585086997000076],[23.712497,44.71469649800008],[23.78049949900003,44.71398650000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO412","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Gorj","NUTS_NAME":"Gorj"},"geometry":{"type":"Polygon","coordinates":[[[23.78049949900003,44.71398650000003],[23.712497,44.71469649800008],[23.707695500000057,44.585086997000076],[23.455035500000065,44.56956149800004],[23.00708750000007,44.74333999900006],[22.908749500000056,44.82196699900004],[22.831356,45.02353849800005],[22.656632,45.10816699600008],[22.559166,45.15686449800006],[22.68624650000004,45.257960998000044],[23.009152,45.253561998000066],[23.377566,45.33844749600007],[23.443675,45.29751599800005],[23.59064,45.353072498000074],[23.841606,45.33209449800006],[23.796259,45.26100949800008],[23.854660500000023,45.152386998000054],[23.758082,45.005660996000074],[23.78049949900003,44.71398650000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO413","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Mehedinti","NUTS_NAME":"Mehedinti"},"geometry":{"type":"Polygon","coordinates":[[[23.455035500000065,44.56956149800004],[23.25519,44.530020498000056],[23.279467500000067,44.50152849800003],[23.166413500000033,44.41213799800005],[23.218123,44.37553649800003],[23.09368550000005,44.26211899800006],[23.109875,44.15014799700003],[23.00536,44.157920500000046],[22.966399500000023,44.09852499600004],[22.67516150000006,44.21566299600005],[22.677534500000036,44.280286498000066],[22.560019500000067,44.30736649600004],[22.457399500000065,44.46746649900007],[22.548219500000073,44.48124899800007],[22.58179350000006,44.548156996000046],[22.762785,44.55141149700006],[22.70595650000007,44.60321649800005],[22.467334,44.71474549900006],[22.332951,44.67587199800005],[22.159579500000063,44.47177749800005],[22.016132500000026,44.59920249800007],[22.154162,44.59519949600008],[22.146892500000035,44.70075699800003],[22.207799,44.81687849600007],[22.418032,44.756857498000045],[22.492156,44.97327249700004],[22.656632,45.10816699600008],[22.831356,45.02353849800005],[22.908749500000056,44.82196699900004],[23.00708750000007,44.74333999900006],[23.455035500000065,44.56956149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO414","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Olt","NUTS_NAME":"Olt"},"geometry":{"type":"Polygon","coordinates":[[[24.438121,44.84537549800007],[24.480183,44.82174099800005],[24.540722500000072,44.89103999800005],[24.593776500000047,44.75087149600006],[24.721729500000038,44.71459699800005],[24.68812,44.69849499800006],[24.744521,44.395482997000045],[24.88401950000008,44.38263649600003],[24.817085500000076,44.361135998000066],[24.793566,44.191254498000035],[24.855496,44.091871500000025],[24.61309249900006,44.014155997000046],[24.602269,43.962112999000055],[24.755096,43.817601498000045],[24.641605999000035,43.73341299800006],[24.501014500000053,43.762396996000064],[24.324132,43.69956799600004],[24.112719,43.699566497000035],[24.155538500000034,43.81371099900008],[24.226797,43.85174899800006],[24.26249850000005,43.987319496000055],[24.173538500000063,44.00629699900003],[24.181231,44.068599996000046],[24.222607500000038,44.069267996000065],[24.226886999000044,44.22991749600004],[23.89047850000003,44.41279099800005],[23.885288,44.51205149900005],[23.928197,44.53022549600007],[24.077848,44.49025999600008],[24.155444500000044,44.5585],[24.324896500000023,44.56085199700004],[24.274618,44.76877999800007],[24.38893950000005,44.90146149800006],[24.438121,44.84537549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO415","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Vâlcea","NUTS_NAME":"Vâlcea"},"geometry":{"type":"Polygon","coordinates":[[[24.438121,44.84537549800007],[24.38893950000005,44.90146149800006],[24.274618,44.76877999800007],[24.324896500000023,44.56085199700004],[24.155444500000044,44.5585],[24.077848,44.49025999600008],[23.928197,44.53022549600007],[23.78049949900003,44.71398650000003],[23.758082,45.005660996000074],[23.854660500000023,45.152386998000054],[23.796259,45.26100949800008],[23.841606,45.33209449800006],[23.59064,45.353072498000074],[23.597270500000036,45.47347149800004],[23.703613,45.496772],[24.512834500000054,45.586824998000054],[24.540518500000076,45.53569699600007],[24.480691499000045,45.449102],[24.520454500000028,45.048977498000056],[24.438121,44.84537549800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO42","LEVL_CODE":2,"CNTR_CODE":"RO","NAME_LATN":"Vest","NUTS_NAME":"Vest"},"geometry":{"type":"Polygon","coordinates":[[[22.168398,46.60386349800007],[22.197508,46.51382549600004],[22.32517050000007,46.51450699700007],[22.424138,46.39332049600006],[22.67657550000007,46.405825496000034],[22.748716,46.351207499000054],[22.925720500000068,46.330588996000074],[22.96113450000007,46.23682549600005],[23.095532500000047,46.18230050000005],[23.055348500000036,46.140611496000076],[23.087312,46.06646149800008],[23.234864,46.030501996000055],[23.31906,45.90771499600004],[23.41857199900005,45.59558899800004],[23.597270500000036,45.47347149800004],[23.59064,45.353072498000074],[23.443675,45.29751599800005],[23.377566,45.33844749600007],[23.009152,45.253561998000066],[22.68624650000004,45.257960998000044],[22.559166,45.15686449800006],[22.656632,45.10816699600008],[22.492156,44.97327249700004],[22.418032,44.756857498000045],[22.207799,44.81687849600007],[22.146892500000035,44.70075699800003],[22.154162,44.59519949600008],[22.016132500000026,44.59920249800007],[22.012350500000025,44.602318498000045],[21.640041500000052,44.659280498000044],[21.587786,44.75841599800003],[21.35847,44.82161199600006],[21.367058,44.86448149600005],[21.560126500000024,44.88900649900006],[21.36023349900006,45.02043299800005],[21.456053500000053,45.04039949600008],[21.527420500000062,45.13786699600007],[21.479178500000046,45.19302750000003],[21.218988,45.250575998000045],[21.176224500000046,45.32522649800006],[21.016575,45.32462749700005],[20.86616199900004,45.46523399800003],[20.771111500000075,45.47996649700008],[20.832577,45.53589299600003],[20.765725,45.61256849700004],[20.823079999000072,45.77506449900005],[20.70064650000006,45.75056199900007],[20.662833,45.79411599800005],[20.569299,45.91013849800004],[20.348325500000044,45.99962699800005],[20.264296,46.12637349900007],[20.501525,46.18993599800007],[20.6406,46.12720699800008],[20.70530350000007,46.16093749800007],[20.7756,46.27590999800003],[21.103170500000033,46.26259049600003],[21.206304,46.40323099600005],[21.296291,46.40669299800004],[21.317476,46.45065599600008],[21.260401500000057,46.501945996000075],[21.314121999000065,46.617556],[21.441398,46.651466999000036],[22.026914,46.64379299600006],[22.168398,46.60386349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO421","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Arad","NUTS_NAME":"Arad"},"geometry":{"type":"Polygon","coordinates":[[[22.67657550000007,46.405825496000034],[22.748716,46.351207499000054],[22.677868,46.321352998000066],[22.649039,46.228789498000026],[22.43679750000007,46.207945996000035],[22.434002500000076,46.010563998000066],[22.352794499000026,45.97268649800003],[22.217798,45.912645],[21.966398,45.99425799800008],[21.799437500000067,45.925269996000054],[21.737031,45.98928299800008],[21.66750250000007,45.906291],[21.641931500000055,45.97400449600008],[21.524238,46.01032399600007],[21.516022500000076,46.06750399900005],[21.110209500000053,45.97706799600007],[21.06874250000004,46.03976449800007],[20.94065450000005,46.037481498000034],[20.70530350000007,46.16093749800007],[20.7756,46.27590999800003],[21.103170500000033,46.26259049600003],[21.206304,46.40323099600005],[21.296291,46.40669299800004],[21.317476,46.45065599600008],[21.260401500000057,46.501945996000075],[21.314121999000065,46.617556],[21.441398,46.651466999000036],[22.026914,46.64379299600006],[22.168398,46.60386349800007],[22.197508,46.51382549600004],[22.32517050000007,46.51450699700007],[22.424138,46.39332049600006],[22.67657550000007,46.405825496000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO422","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Caras-Severin","NUTS_NAME":"Caras-Severin"},"geometry":{"type":"Polygon","coordinates":[[[22.68624650000004,45.257960998000044],[22.559166,45.15686449800006],[22.656632,45.10816699600008],[22.492156,44.97327249700004],[22.418032,44.756857498000045],[22.207799,44.81687849600007],[22.146892500000035,44.70075699800003],[22.154162,44.59519949600008],[22.016132500000026,44.59920249800007],[22.012350500000025,44.602318498000045],[21.640041500000052,44.659280498000044],[21.587786,44.75841599800003],[21.35847,44.82161199600006],[21.367058,44.86448149600005],[21.560126500000024,44.88900649900006],[21.36023349900006,45.02043299800005],[21.456053500000053,45.04039949600008],[21.527420500000062,45.13786699600007],[21.479178500000046,45.19302750000003],[21.556103500000063,45.27242999600003],[21.44508950000005,45.390378998000074],[21.458706,45.47676649700003],[21.566182,45.496437496000055],[21.599883,45.57106699800005],[21.831358,45.528571498000076],[21.92352150000005,45.566872998000065],[21.94536750000003,45.50597699800005],[22.042823,45.49828399900008],[22.11490150000003,45.590862498000035],[22.270097,45.584837],[22.317528,45.660649497000065],[22.426479,45.66888399800007],[22.63659,45.57107749800008],[22.689725,45.49267199900004],[22.712771,45.415677998000035],[22.656594,45.371338500000036],[22.68624650000004,45.257960998000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO423","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Hunedoara","NUTS_NAME":"Hunedoara"},"geometry":{"type":"Polygon","coordinates":[[[23.597270500000036,45.47347149800004],[23.59064,45.353072498000074],[23.443675,45.29751599800005],[23.377566,45.33844749600007],[23.009152,45.253561998000066],[22.68624650000004,45.257960998000044],[22.656594,45.371338500000036],[22.712771,45.415677998000035],[22.689725,45.49267199900004],[22.63659,45.57107749800008],[22.426479,45.66888399800007],[22.551454,45.77863399700004],[22.410611499000026,45.85456399600008],[22.41777,45.93850849900008],[22.352794499000026,45.97268649800003],[22.434002500000076,46.010563998000066],[22.43679750000007,46.207945996000035],[22.649039,46.228789498000026],[22.677868,46.321352998000066],[22.748716,46.351207499000054],[22.925720500000068,46.330588996000074],[22.96113450000007,46.23682549600005],[23.095532500000047,46.18230050000005],[23.055348500000036,46.140611496000076],[23.087312,46.06646149800008],[23.234864,46.030501996000055],[23.31906,45.90771499600004],[23.41857199900005,45.59558899800004],[23.597270500000036,45.47347149800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"RO424","LEVL_CODE":3,"CNTR_CODE":"RO","NAME_LATN":"Timis","NUTS_NAME":"Timis"},"geometry":{"type":"Polygon","coordinates":[[[20.70530350000007,46.16093749800007],[20.94065450000005,46.037481498000034],[21.06874250000004,46.03976449800007],[21.110209500000053,45.97706799600007],[21.516022500000076,46.06750399900005],[21.524238,46.01032399600007],[21.641931500000055,45.97400449600008],[21.66750250000007,45.906291],[21.737031,45.98928299800008],[21.799437500000067,45.925269996000054],[21.966398,45.99425799800008],[22.217798,45.912645],[22.352794499000026,45.97268649800003],[22.41777,45.93850849900008],[22.410611499000026,45.85456399600008],[22.551454,45.77863399700004],[22.426479,45.66888399800007],[22.317528,45.660649497000065],[22.270097,45.584837],[22.11490150000003,45.590862498000035],[22.042823,45.49828399900008],[21.94536750000003,45.50597699800005],[21.92352150000005,45.566872998000065],[21.831358,45.528571498000076],[21.599883,45.57106699800005],[21.566182,45.496437496000055],[21.458706,45.47676649700003],[21.44508950000005,45.390378998000074],[21.556103500000063,45.27242999600003],[21.479178500000046,45.19302750000003],[21.218988,45.250575998000045],[21.176224500000046,45.32522649800006],[21.016575,45.32462749700005],[20.86616199900004,45.46523399800003],[20.771111500000075,45.47996649700008],[20.832577,45.53589299600003],[20.765725,45.61256849700004],[20.823079999000072,45.77506449900005],[20.70064650000006,45.75056199900007],[20.662833,45.79411599800005],[20.569299,45.91013849800004],[20.348325500000044,45.99962699800005],[20.264296,46.12637349900007],[20.501525,46.18993599800007],[20.6406,46.12720699800008],[20.70530350000007,46.16093749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE","LEVL_CODE":0,"CNTR_CODE":"SE","NAME_LATN":"SVERIGE","NUTS_NAME":"SVERIGE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[18.138552267000023,59.33231602300003],[18.452799500000026,59.33505699800003],[18.450584,59.39106699700005],[18.414662500000077,59.41990099700007],[18.481714500000066,59.420649498000046],[18.627228,59.36869299700004],[18.6492,59.34953349800003],[18.539283500000067,59.357112],[18.667466,59.31371349800003],[18.55595850000003,59.28586149600005],[18.708546500000068,59.29002999700003],[18.497404,59.214245997000035],[18.328565,59.28458399700003],[18.53894936000006,59.272426050000036],[18.313185,59.32400099900008],[18.275957,59.29948999900006],[18.330841,59.26752999900003],[18.252245500000072,59.268267997000066],[18.469045,59.17130649900008],[18.021724,59.044556499000066],[17.991549570000075,58.96697998900004],[18.026501,58.983126999000035],[17.901606172000072,58.851799827000036],[17.923739500000067,58.80439099800003],[17.86863524200004,58.84706318500008],[17.850585,58.90182599800005],[17.82753427800003,58.884624761000055],[17.732177,58.92898899800008],[17.74335350000007,59.05995249700004],[17.687003,59.104753498000036],[17.61468250000007,59.08311850000007],[17.619061500000043,58.950139999000044],[17.572396,58.95085249700003],[17.577561,58.83197449600004],[17.45246650100006,58.88982449500003],[17.50531150100005,58.78197999500003],[16.939922,58.67114749700005],[17.023572,58.62613799800005],[16.750663501000076,58.626932],[16.218891636000023,58.62509637000005],[16.773165,58.60499249900005],[16.948542,58.47978149900007],[16.738593,58.42019749700006],[16.414471,58.47595449800008],[16.927406,58.335789497000064],[16.627013500000032,58.35635249900008],[16.82255250000003,58.31112999800007],[16.73041850000004,58.25969050000003],[16.854670499000065,58.172553496000035],[16.803516484000056,58.11861524500006],[16.84388750000005,58.09650499500003],[16.60343450000005,58.202163],[16.813647,58.063552997000045],[16.666683500000033,57.99607649700005],[16.62113449900005,57.986361],[16.813461,57.99590199900007],[16.771834500000068,57.858126997000056],[16.469091500000047,57.890911],[16.70012950000006,57.75089299800004],[16.407790500000033,57.88058949900005],[16.732922,57.69605450000006],[16.63175650000005,57.682930999000064],[16.652136480000024,57.582051179000075],[16.727928,57.60177099600003],[16.668210500000043,57.559473499000035],[16.497907,57.63715349800003],[16.727634500000022,57.44355299700004],[16.464502500000037,57.26499449700003],[16.466294,57.15372149700005],[16.580416,57.045376],[16.45088050000004,57.04275549700003],[16.433601,56.98088549600004],[16.472051500000077,56.940429],[16.41184750000002,56.79190099900006],[16.484478,56.78741899700003],[16.359856500000035,56.76269149900003],[16.378437999000028,56.658735500000034],[16.248298500000033,56.634299498000075],[16.04987150000005,56.32188850000006],[15.99619750000005,56.199460499000054],[15.851547999000047,56.08510249700004],[15.581619,56.162849],[15.592694,56.21186899700007],[15.348007,56.12492250000008],[15.047866500000055,56.19555599900008],[14.687203,56.16711699900003],[14.78186850000003,56.03437999800008],[14.721438500000033,55.993557],[14.546399500000064,56.06131749900004],[14.220109500000035,55.83045749900003],[14.204630500000064,55.700960999000074],[14.360844,55.55425949700003],[14.194909,55.384035],[13.903504,55.43513799800007],[13.359049500000026,55.336853],[12.809839500000066,55.37825649900003],[12.968462,55.43839999700003],[12.889695,55.56459999800006],[13.063593,55.670109496000066],[12.44596050000007,56.30268],[12.783795,56.21956799700007],[12.836667500000033,56.27053949800006],[12.61962950000003,56.42242299800006],[12.899313,56.44900449900007],[12.936727500000075,56.58707699900003],[12.730945,56.65627649900006],[12.607982,56.820760997000036],[12.360219,56.926837997000064],[12.214189,57.13360750000004],[12.211207,57.22473899600004],[12.093919,57.23457049700005],[12.166869999000028,57.30980449900005],[12.064521499000023,57.33840049600008],[12.127203,57.378446],[12.108000999000069,57.434753500000056],[12.059552500000052,57.46236299600008],[12.004654500000072,57.340674498000055],[11.913888,57.40103749700006],[11.922900500000026,57.56222299900003],[11.927425001000074,57.62475799900005],[11.839630499000066,57.66350149800007],[11.990241500000025,57.72145350000005],[11.700709500000073,57.705336997000074],[11.835160500000029,57.794209],[11.673720500000059,57.84875499800006],[11.81427750000006,57.99311499800007],[11.844677282000077,58.16014767200005],[11.553980500000023,58.08578250000005],[11.406957,58.14387399900005],[11.678524500000037,58.23903999700008],[11.682809072000055,58.292475712000055],[11.501040999000054,58.250593998000056],[11.702689,58.43364799900007],[11.435055001000023,58.26864899700007],[11.358601,58.34339350000005],[11.43373850100005,58.43250349900006],[11.271686,58.35302849900006],[11.220071499000028,58.40892949700003],[11.309479500000066,58.47959049500008],[11.256905,58.677688],[11.174135001000025,58.71829249700005],[11.239031500000067,58.83897149900008],[11.113543499000059,58.99826349700004],[11.318191540000043,59.10402379800007],[11.338415674000032,59.10458747300004],[11.34712780600006,59.10483029300008],[11.460827,58.98865849800006],[11.457448500000055,58.888435497000046],[11.632557,58.90830650000004],[11.826196500000037,59.23784999700007],[11.691129,59.58954749900005],[11.939878500000077,59.69458099700006],[11.926970500000039,59.79047599800003],[11.839729,59.84076699900004],[12.174292,59.88980850000007],[12.499544999000022,60.09765699700006],[12.541906500000039,60.19337849700008],[12.498795500000028,60.323652],[12.606053,60.405925499000034],[12.606881500000043,60.512742496000044],[12.22399,61.013078],[12.670176500000025,61.055976497000074],[12.870847500000025,61.35649499700003],[12.569316,61.568746496000074],[12.419609,61.56298099900005],[12.137665,61.72381699700003],[12.29937,62.26749399900007],[12.25465950000006,62.331024496000055],[12.056142,62.61191849800008],[12.136381500000027,62.74791649700006],[12.074687999000048,62.902544997000064],[12.218231,63.00033249900008],[12.052455500000065,63.183444499000075],[11.974581,63.269227999000066],[12.212877,63.47859199900006],[12.149767,63.593946998000035],[12.683565,63.97422349900006],[13.211107500000026,64.09536949900007],[13.967524500000025,64.00796999900007],[14.157109,64.19505449600007],[14.113869500000021,64.46248450000007],[13.654257500000028,64.58034049800006],[14.325985,65.118916],[14.378797500000076,65.24762049800006],[14.50683250000003,65.30972899800008],[14.49877250000003,65.52129749600005],[14.625476500000048,65.81180849900005],[14.516289,66.13257899900003],[15.484732,66.28245849800004],[15.453994500000022,66.34523350000006],[15.377226500000063,66.48430399600005],[16.387759,67.04546249900005],[16.404099501000076,67.20497449900006],[16.089829,67.43527850000004],[16.15800150000007,67.519159],[16.407672,67.53409999900003],[16.738120500000036,67.91420949800005],[17.281524,68.11881499900005],[17.899761500000068,67.96937199900003],[18.151354,68.19878999900004],[18.101093001000038,68.406045],[18.125925,68.53651599800008],[18.405688500000053,68.58187699900003],[19.921397,68.356013],[20.226541500000053,68.49081250000006],[19.93750250000005,68.55793949900004],[20.33587250000005,68.80231249900004],[20.306589,68.92618199900005],[20.060047500000053,69.045759],[20.548636500000043,69.05996850000008],[21.88894300100003,68.58438349900007],[22.844611,68.38334150000003],[23.143492500000036,68.24682799900006],[23.15471750100005,68.136291],[23.652348999000026,67.958793],[23.494326001000047,67.86402599900003],[23.477203,67.70166250000005],[23.55897800100007,67.61996149800007],[23.39402350000006,67.48582099800007],[23.764582500000074,67.428214],[23.722462,67.28252899800003],[23.560987999000076,67.26085349800007],[23.55127200100003,67.17146199900003],[23.995160501000043,66.81970899800007],[23.879574500000047,66.76324399900005],[23.907815500000027,66.72142649900007],[23.856645,66.55690399900004],[23.650217501000043,66.45498399900004],[23.64559950000006,66.30140849900005],[23.918210499000054,66.16141],[24.155129,65.81602699900003],[23.47538650100006,65.84518749800003],[23.402833500000042,65.75004549800008],[23.217332500000055,65.82339349900008],[23.099601500000063,65.71327249700005],[22.671455,65.90403199800005],[22.617905,65.90125599900006],[22.690188500000033,65.86531349800003],[22.699252999000066,65.75771999800003],[22.340766001000077,65.86896049800004],[22.310333,65.78256899900003],[22.369043,65.66880349700006],[22.232683500000064,65.65303649700007],[22.37649,65.63100049800005],[22.294381500000043,65.60182449900003],[22.402720499000054,65.53735549600003],[21.831895001000078,65.52277549900003],[22.035621500000047,65.46488549900005],[21.768649500000038,65.40010249900007],[21.82642050000004,65.34806849800003],[21.44599050000005,65.38320899900003],[21.63175399900007,65.33305449900007],[21.634512,65.24373449800004],[21.750894,65.19125699900007],[21.524723359000063,65.23180558500007],[21.589963501000057,65.15028449900007],[21.506678500000078,65.14285999800006],[21.54360450000007,65.06305899700004],[21.23190150000005,64.94453999900003],[21.25731150000007,64.92216599700004],[21.156021,64.81548449900004],[20.997129,64.83223249600007],[21.297873,64.75350349800004],[21.21348050000006,64.73940199900005],[21.28227,64.66163399700008],[21.17361950000003,64.69389299900007],[21.60536250000007,64.43396099900008],[21.00002850000004,64.159882],[20.765231999000036,63.86661399900004],[20.560563,63.80467549700006],[20.574595500000044,63.72209999800003],[20.494180999000037,63.836441499000045],[20.396852,63.67203849900005],[20.305153500000074,63.78511399700005],[20.25163851700006,63.704674295000075],[20.300421500000027,63.74921249700003],[20.31427,63.67404250000004],[19.897695500000054,63.61702299800004],[19.683038999000075,63.434309],[19.449868500000036,63.57309199900004],[19.422997500000065,63.521906999000066],[19.486668500000064,63.41935599900006],[19.286468,63.469719499000064],[19.254426500000022,63.33568899900007],[19.038011,63.27557649600004],[19.08771550000006,63.22014999800007],[19.03094,63.187909999000055],[18.869752,63.294495],[18.710203,63.287126999000066],[18.901825,63.230789997000045],[18.74743650000005,63.17651649600003],[18.67888750000003,63.249727],[18.724288500000057,63.161021997000034],[18.359343500000023,63.05831],[18.436673,63.021690499000044],[18.331517,63.05352650000003],[18.287163500000077,63.00699950000006],[18.557591501000047,62.96259649900003],[18.382934001000024,62.959036998000045],[18.510375,62.93184449800003],[18.345507499000064,62.83480999800008],[18.201886001000048,62.88821449900007],[18.286522,62.851045998000075],[18.221234500000037,62.79667],[17.949062,62.79962999900005],[17.81687450000004,62.96930199800005],[17.90028650000005,62.96116599800007],[17.769321001000037,62.99744699900003],[17.98876350000006,62.702602],[17.885818500000028,62.66721799600003],[18.0637,62.61007099900007],[17.65642,62.467043499000056],[17.73663350000004,62.410059],[17.401207500000055,62.549360498000055],[17.433393500000022,62.506204499000035],[17.358609500000057,62.492508497000074],[17.37630991700007,62.448571625000056],[17.504983,62.44795099700008],[17.536147500000027,62.36078199900004],[17.359868,62.39108099600003],[17.392239500000073,62.323179],[17.659471500000052,62.23300349900006],[17.489299500000072,62.13695049800003],[17.460933,62.00146549700008],[17.339091500000052,61.95643749800007],[17.307419500000037,61.85283999600006],[17.40242950000004,61.72239749900007],[17.52387550000003,61.70176699900003],[17.487514,61.62040649700003],[17.312563499000078,61.72397549900006],[17.34512249900007,61.66663099900006],[17.094999,61.70968000100004],[17.255855500000052,61.656099],[17.086958500000037,61.63812899700008],[17.232869,61.593874999000036],[17.058534500000064,61.614304497000035],[17.142338,61.58367899900003],[17.03733950000003,61.574806496000065],[17.221907500000043,61.55158649900005],[17.040934999000058,61.54727999900007],[17.223366,61.42427749900003],[17.08862350000004,61.39873750000004],[17.28051350000004,61.275707],[17.115180500000065,61.20674949600004],[17.19188450000007,61.178396500000076],[17.136091500000077,61.00020549900006],[17.248538499000063,61.00658050000004],[17.070646,60.89967199800003],[17.330891500000064,60.88606499800005],[17.239781,60.81133199900006],[17.352966,60.752157502000045],[17.225903,60.68897899900003],[17.370218,60.654461499000035],[17.670093,60.625987],[17.587032,60.58429349900007],[17.675121999000055,60.50564799600005],[17.98928550000005,60.60410949900006],[18.012909499000045,60.50216349900006],[18.150248,60.411684499000046],[18.617636,60.23238249900004],[18.31014650000003,60.316625499000054],[18.513265500000045,60.14808899700006],[18.74861050000004,60.11804649700008],[18.810672500000067,59.975247498000044],[18.766777,60.10365249700004],[18.838097,60.110722500000065],[18.899117500000045,59.930443497000056],[19.08419850000007,59.87595199700007],[19.049067500000035,59.81700899700007],[18.852333500000043,59.93937899900004],[18.989798,59.78332149900007],[18.711557499000037,59.74776499900008],[19.08541850000006,59.75826449600004],[18.76564,59.68565549900006],[18.74923750000005,59.61774449700005],[18.28815650000007,59.43202199800004],[18.331780500000036,59.381805],[18.14285,59.447774],[18.058382209000058,59.37904236500003],[18.138552267000023,59.33231602300003]]],[[[19.169407500000034,57.88666799900005],[19.122862,57.84081599900003],[19.034330500000067,57.906982998000046],[19.183279500000026,57.990034999000045],[19.350247,57.95936599700008],[19.14739,57.92498299700003],[19.169407500000034,57.88666799900005]]],[[[18.734817500000077,57.92931150000004],[18.77719250000007,57.867417],[19.003448500000047,57.915782999000044],[19.097435,57.819173],[18.810876,57.70597550000008],[18.761467500000037,57.46970350000004],[18.91781650000007,57.40334749800007],[18.660018,57.29484649900007],[18.711624,57.24468649700003],[18.39982549900003,57.13976399900008],[18.340005500000075,57.07232799800005],[18.366750999000033,56.984774996000056],[18.312185,56.93918450000007],[18.129537,56.90817750000008],[18.301114,57.09550850000005],[18.19604350000003,57.06748150000004],[18.21981450000004,57.14083849900004],[18.091156,57.259374],[18.185531,57.390713497000036],[18.11695099900004,57.44050450000003],[18.106193500000074,57.53464150000008],[18.460552500000063,57.819143],[18.734817500000077,57.92931150000004]]],[[[16.56806050000006,56.36792549900008],[16.400671,56.195806996000044],[16.406908500000043,56.41264649700008],[16.370938500000022,56.46075499700004],[16.413673,56.58997449900005],[16.631586500000026,56.87503949900008],[16.720757,56.88387499700008],[16.959014,57.291363499000056],[17.076804309000067,57.35587951500003],[17.073719999000048,57.179024],[16.907787,57.04047799700004],[16.865505,56.845187999000075],[16.774367,56.787356497000076],[16.56806050000006,56.36792549900008]]],[[[11.755686500000024,58.006059],[11.586167499000055,57.932169499000054],[11.502457,58.047987996000074],[11.708466,58.08634899900005],[11.682340500000066,58.06584699700005],[11.769487500000025,58.059569],[11.755686500000024,58.006059]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE1","LEVL_CODE":1,"CNTR_CODE":"SE","NAME_LATN":"ÖSTRA SVERIGE","NUTS_NAME":"ÖSTRA SVERIGE"},"geometry":{"type":"Polygon","coordinates":[[[18.138552267000023,59.33231602300003],[18.452799500000026,59.33505699800003],[18.450584,59.39106699700005],[18.414662500000077,59.41990099700007],[18.481714500000066,59.420649498000046],[18.627228,59.36869299700004],[18.6492,59.34953349800003],[18.539283500000067,59.357112],[18.667466,59.31371349800003],[18.55595850000003,59.28586149600005],[18.708546500000068,59.29002999700003],[18.497404,59.214245997000035],[18.328565,59.28458399700003],[18.53894936000006,59.272426050000036],[18.313185,59.32400099900008],[18.275957,59.29948999900006],[18.330841,59.26752999900003],[18.252245500000072,59.268267997000066],[18.469045,59.17130649900008],[18.021724,59.044556499000066],[17.991549570000075,58.96697998900004],[18.026501,58.983126999000035],[17.901606172000072,58.851799827000036],[17.923739500000067,58.80439099800003],[17.86863524200004,58.84706318500008],[17.850585,58.90182599800005],[17.82753427800003,58.884624761000055],[17.732177,58.92898899800008],[17.74335350000007,59.05995249700004],[17.687003,59.104753498000036],[17.61468250000007,59.08311850000007],[17.619061500000043,58.950139999000044],[17.572396,58.95085249700003],[17.577561,58.83197449600004],[17.45246650100006,58.88982449500003],[17.50531150100005,58.78197999500003],[16.939922,58.67114749700005],[17.023572,58.62613799800005],[16.750663501000076,58.626932],[16.218891636000023,58.62509637000005],[16.773165,58.60499249900005],[16.948542,58.47978149900007],[16.738593,58.42019749700006],[16.414471,58.47595449800008],[16.927406,58.335789497000064],[16.627013500000032,58.35635249900008],[16.82255250000003,58.31112999800007],[16.73041850000004,58.25969050000003],[16.854670499000065,58.172553496000035],[16.803516484000056,58.11861524500006],[16.84388750000005,58.09650499500003],[16.60343450000005,58.202163],[16.813647,58.063552997000045],[16.666683500000033,57.99607649700005],[16.531376,58.11576849900007],[16.239588,58.134933499000056],[16.178619500000025,58.064018496000074],[15.986948,58.074669],[16.06543350000004,57.87426750000003],[15.942313999000078,57.80328749700004],[15.599087500000053,57.85389349700006],[15.421051,57.704879497000036],[15.128358500000047,57.71631250000007],[14.994365500000072,57.90547549900003],[15.009529,57.98624799900006],[15.081543,57.998931998000046],[14.982714,58.14911249900007],[14.808759500000065,58.08264549900008],[14.41671,58.18724049600007],[14.778936499000054,58.645992500000034],[14.691086,58.73967749700006],[14.538840500000049,58.675800999000046],[14.286332,58.94845249900004],[14.295995,59.01277950000008],[14.339636,59.08344249800007],[14.289464,59.157100497000044],[14.305491500000073,59.284824496000056],[14.43705650000004,59.462387],[14.391029500000059,59.51383599700006],[14.475906500000065,59.52899149700005],[14.401095500000054,59.74196999600008],[14.434949500000073,59.77083949900003],[14.42161550000003,59.90319449600008],[14.332584500000053,59.95231650000005],[14.456956,59.97297649900008],[14.437155500000074,60.026160996000044],[14.753791,59.99715799900008],[14.737407,60.08113499900003],[14.85119050000003,60.10405350000008],[15.422092500000076,59.85474749900004],[15.529911,59.956668999000044],[15.691971,59.947555495000074],[15.801567,60.179103498000075],[16.28451350000006,60.07316199700006],[16.703831,60.19572049700008],[17.192478,60.30071249900004],[17.192915,60.407017],[17.287695,60.446842],[17.370218,60.654461499000035],[17.670093,60.625987],[17.587032,60.58429349900007],[17.675121999000055,60.50564799600005],[17.98928550000005,60.60410949900006],[18.012909499000045,60.50216349900006],[18.150248,60.411684499000046],[18.617636,60.23238249900004],[18.31014650000003,60.316625499000054],[18.513265500000045,60.14808899700006],[18.74861050000004,60.11804649700008],[18.810672500000067,59.975247498000044],[18.766777,60.10365249700004],[18.838097,60.110722500000065],[18.899117500000045,59.930443497000056],[19.08419850000007,59.87595199700007],[19.049067500000035,59.81700899700007],[18.852333500000043,59.93937899900004],[18.989798,59.78332149900007],[18.711557499000037,59.74776499900008],[19.08541850000006,59.75826449600004],[18.76564,59.68565549900006],[18.74923750000005,59.61774449700005],[18.28815650000007,59.43202199800004],[18.331780500000036,59.381805],[18.14285,59.447774],[18.058382209000058,59.37904236500003],[18.138552267000023,59.33231602300003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE11","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Stockholm","NUTS_NAME":"Stockholm"},"geometry":{"type":"Polygon","coordinates":[[[18.138552267000023,59.33231602300003],[18.452799500000026,59.33505699800003],[18.450584,59.39106699700005],[18.414662500000077,59.41990099700007],[18.481714500000066,59.420649498000046],[18.627228,59.36869299700004],[18.6492,59.34953349800003],[18.539283500000067,59.357112],[18.667466,59.31371349800003],[18.55595850000003,59.28586149600005],[18.708546500000068,59.29002999700003],[18.497404,59.214245997000035],[18.328565,59.28458399700003],[18.53894936000006,59.272426050000036],[18.313185,59.32400099900008],[18.275957,59.29948999900006],[18.330841,59.26752999900003],[18.252245500000072,59.268267997000066],[18.469045,59.17130649900008],[18.021724,59.044556499000066],[17.991549570000075,58.96697998900004],[18.026501,58.983126999000035],[17.901606172000072,58.851799827000036],[17.923739500000067,58.80439099800003],[17.86863524200004,58.84706318500008],[17.850585,58.90182599800005],[17.82753427800003,58.884624761000055],[17.732177,58.92898899800008],[17.74335350000007,59.05995249700004],[17.687003,59.104753498000036],[17.61468250000007,59.08311850000007],[17.619061500000043,58.950139999000044],[17.572396,58.95085249700003],[17.376837,58.98299799600005],[17.245373001000075,59.204196997000054],[17.416126500000075,59.384139997000034],[17.581468500000028,59.56073749900003],[17.513893,59.62264250000004],[17.629102500000045,59.69300449500008],[18.08449550000006,59.739021],[18.133012499000074,59.818924],[18.36479950000006,59.86438749600006],[18.47500050000002,60.00445149600006],[18.388929500000074,60.042473],[18.513265500000045,60.14808899700006],[18.74861050000004,60.11804649700008],[18.810672500000067,59.975247498000044],[18.766777,60.10365249700004],[18.838097,60.110722500000065],[18.899117500000045,59.930443497000056],[19.08419850000007,59.87595199700007],[19.049067500000035,59.81700899700007],[18.852333500000043,59.93937899900004],[18.989798,59.78332149900007],[18.711557499000037,59.74776499900008],[19.08541850000006,59.75826449600004],[18.76564,59.68565549900006],[18.74923750000005,59.61774449700005],[18.28815650000007,59.43202199800004],[18.331780500000036,59.381805],[18.14285,59.447774],[18.058382209000058,59.37904236500003],[18.138552267000023,59.33231602300003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE110","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Stockholms län","NUTS_NAME":"Stockholms län"},"geometry":{"type":"Polygon","coordinates":[[[18.138552267000023,59.33231602300003],[18.452799500000026,59.33505699800003],[18.450584,59.39106699700005],[18.414662500000077,59.41990099700007],[18.481714500000066,59.420649498000046],[18.627228,59.36869299700004],[18.6492,59.34953349800003],[18.539283500000067,59.357112],[18.667466,59.31371349800003],[18.55595850000003,59.28586149600005],[18.708546500000068,59.29002999700003],[18.497404,59.214245997000035],[18.328565,59.28458399700003],[18.53894936000006,59.272426050000036],[18.313185,59.32400099900008],[18.275957,59.29948999900006],[18.330841,59.26752999900003],[18.252245500000072,59.268267997000066],[18.469045,59.17130649900008],[18.021724,59.044556499000066],[17.991549570000075,58.96697998900004],[18.026501,58.983126999000035],[17.901606172000072,58.851799827000036],[17.923739500000067,58.80439099800003],[17.86863524200004,58.84706318500008],[17.850585,58.90182599800005],[17.82753427800003,58.884624761000055],[17.732177,58.92898899800008],[17.74335350000007,59.05995249700004],[17.687003,59.104753498000036],[17.61468250000007,59.08311850000007],[17.619061500000043,58.950139999000044],[17.572396,58.95085249700003],[17.376837,58.98299799600005],[17.245373001000075,59.204196997000054],[17.416126500000075,59.384139997000034],[17.581468500000028,59.56073749900003],[17.513893,59.62264250000004],[17.629102500000045,59.69300449500008],[18.08449550000006,59.739021],[18.133012499000074,59.818924],[18.36479950000006,59.86438749600006],[18.47500050000002,60.00445149600006],[18.388929500000074,60.042473],[18.513265500000045,60.14808899700006],[18.74861050000004,60.11804649700008],[18.810672500000067,59.975247498000044],[18.766777,60.10365249700004],[18.838097,60.110722500000065],[18.899117500000045,59.930443497000056],[19.08419850000007,59.87595199700007],[19.049067500000035,59.81700899700007],[18.852333500000043,59.93937899900004],[18.989798,59.78332149900007],[18.711557499000037,59.74776499900008],[19.08541850000006,59.75826449600004],[18.76564,59.68565549900006],[18.74923750000005,59.61774449700005],[18.28815650000007,59.43202199800004],[18.331780500000036,59.381805],[18.14285,59.447774],[18.058382209000058,59.37904236500003],[18.138552267000023,59.33231602300003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE12","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Östra Mellansverige","NUTS_NAME":"Östra Mellansverige"},"geometry":{"type":"Polygon","coordinates":[[[17.587032,60.58429349900007],[17.675121999000055,60.50564799600005],[17.98928550000005,60.60410949900006],[18.012909499000045,60.50216349900006],[18.150248,60.411684499000046],[18.617636,60.23238249900004],[18.31014650000003,60.316625499000054],[18.513265500000045,60.14808899700006],[18.388929500000074,60.042473],[18.47500050000002,60.00445149600006],[18.36479950000006,59.86438749600006],[18.133012499000074,59.818924],[18.08449550000006,59.739021],[17.629102500000045,59.69300449500008],[17.513893,59.62264250000004],[17.581468500000028,59.56073749900003],[17.416126500000075,59.384139997000034],[17.245373001000075,59.204196997000054],[17.376837,58.98299799600005],[17.572396,58.95085249700003],[17.577561,58.83197449600004],[17.45246650100006,58.88982449500003],[17.50531150100005,58.78197999500003],[16.939922,58.67114749700005],[17.023572,58.62613799800005],[16.750663501000076,58.626932],[16.218891636000023,58.62509637000005],[16.773165,58.60499249900005],[16.948542,58.47978149900007],[16.738593,58.42019749700006],[16.414471,58.47595449800008],[16.927406,58.335789497000064],[16.627013500000032,58.35635249900008],[16.82255250000003,58.31112999800007],[16.73041850000004,58.25969050000003],[16.854670499000065,58.172553496000035],[16.803516484000056,58.11861524500006],[16.84388750000005,58.09650499500003],[16.60343450000005,58.202163],[16.813647,58.063552997000045],[16.666683500000033,57.99607649700005],[16.531376,58.11576849900007],[16.239588,58.134933499000056],[16.178619500000025,58.064018496000074],[15.986948,58.074669],[16.06543350000004,57.87426750000003],[15.942313999000078,57.80328749700004],[15.599087500000053,57.85389349700006],[15.421051,57.704879497000036],[15.128358500000047,57.71631250000007],[14.994365500000072,57.90547549900003],[15.009529,57.98624799900006],[15.081543,57.998931998000046],[14.982714,58.14911249900007],[14.808759500000065,58.08264549900008],[14.41671,58.18724049600007],[14.778936499000054,58.645992500000034],[14.691086,58.73967749700006],[14.538840500000049,58.675800999000046],[14.286332,58.94845249900004],[14.295995,59.01277950000008],[14.339636,59.08344249800007],[14.289464,59.157100497000044],[14.305491500000073,59.284824496000056],[14.43705650000004,59.462387],[14.391029500000059,59.51383599700006],[14.475906500000065,59.52899149700005],[14.401095500000054,59.74196999600008],[14.434949500000073,59.77083949900003],[14.42161550000003,59.90319449600008],[14.332584500000053,59.95231650000005],[14.456956,59.97297649900008],[14.437155500000074,60.026160996000044],[14.753791,59.99715799900008],[14.737407,60.08113499900003],[14.85119050000003,60.10405350000008],[15.422092500000076,59.85474749900004],[15.529911,59.956668999000044],[15.691971,59.947555495000074],[15.801567,60.179103498000075],[16.28451350000006,60.07316199700006],[16.703831,60.19572049700008],[17.192478,60.30071249900004],[17.192915,60.407017],[17.287695,60.446842],[17.370218,60.654461499000035],[17.670093,60.625987],[17.587032,60.58429349900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE121","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Uppsala län","NUTS_NAME":"Uppsala län"},"geometry":{"type":"Polygon","coordinates":[[[17.675121999000055,60.50564799600005],[17.98928550000005,60.60410949900006],[18.012909499000045,60.50216349900006],[18.150248,60.411684499000046],[18.617636,60.23238249900004],[18.31014650000003,60.316625499000054],[18.513265500000045,60.14808899700006],[18.388929500000074,60.042473],[18.47500050000002,60.00445149600006],[18.36479950000006,59.86438749600006],[18.133012499000074,59.818924],[18.08449550000006,59.739021],[17.629102500000045,59.69300449500008],[17.513893,59.62264250000004],[17.581468500000028,59.56073749900003],[17.416126500000075,59.384139997000034],[16.96491050000003,59.52132399900006],[16.727358,59.77533699900005],[16.83678050000003,59.86002749600004],[16.691070500000023,60.08332050000007],[16.745502500000043,60.13538349800007],[16.703831,60.19572049700008],[17.192478,60.30071249900004],[17.192915,60.407017],[17.287695,60.446842],[17.370218,60.654461499000035],[17.670093,60.625987],[17.587032,60.58429349900007],[17.675121999000055,60.50564799600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE122","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Södermanlands län","NUTS_NAME":"Södermanlands län"},"geometry":{"type":"Polygon","coordinates":[[[15.991972,59.37535449900008],[16.159219501000052,59.34619599600006],[16.306436500000075,59.398899],[16.25500100100004,59.45220949600008],[16.96491050000003,59.52132399900006],[17.416126500000075,59.384139997000034],[17.245373001000075,59.204196997000054],[17.376837,58.98299799600005],[17.572396,58.95085249700003],[17.577561,58.83197449600004],[17.45246650100006,58.88982449500003],[17.50531150100005,58.78197999500003],[16.939922,58.67114749700005],[17.023572,58.62613799800005],[16.750663501000076,58.626932],[16.38710400100007,58.70158],[16.263183500000025,58.832165],[15.91061650000006,58.96247099900006],[15.686749500000076,58.98191849600005],[15.617399,59.02344499900005],[15.793618,59.10988599500007],[15.761593,59.19794849700003],[15.893567999000027,59.230377],[15.875536,59.278323999000065],[15.991972,59.37535449900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE123","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Östergötlands län","NUTS_NAME":"Östergötlands län"},"geometry":{"type":"Polygon","coordinates":[[[15.91061650000006,58.96247099900006],[16.263183500000025,58.832165],[16.38710400100007,58.70158],[16.750663501000076,58.626932],[16.218891636000023,58.62509637000005],[16.773165,58.60499249900005],[16.948542,58.47978149900007],[16.738593,58.42019749700006],[16.414471,58.47595449800008],[16.927406,58.335789497000064],[16.627013500000032,58.35635249900008],[16.82255250000003,58.31112999800007],[16.73041850000004,58.25969050000003],[16.854670499000065,58.172553496000035],[16.803516484000056,58.11861524500006],[16.84388750000005,58.09650499500003],[16.60343450000005,58.202163],[16.813647,58.063552997000045],[16.666683500000033,57.99607649700005],[16.531376,58.11576849900007],[16.239588,58.134933499000056],[16.178619500000025,58.064018496000074],[15.986948,58.074669],[16.06543350000004,57.87426750000003],[15.942313999000078,57.80328749700004],[15.599087500000053,57.85389349700006],[15.421051,57.704879497000036],[15.128358500000047,57.71631250000007],[14.994365500000072,57.90547549900003],[15.009529,57.98624799900006],[15.081543,57.998931998000046],[14.982714,58.14911249900007],[14.808759500000065,58.08264549900008],[14.41671,58.18724049600007],[14.778936499000054,58.645992500000034],[15.034304500000076,58.70448299800006],[15.263016,58.86462399900006],[15.554029,58.86807599500003],[15.594214500000078,58.90625749700007],[15.559378500000037,58.95655799900004],[15.686749500000076,58.98191849600005],[15.91061650000006,58.96247099900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE124","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Örebro län","NUTS_NAME":"Örebro län"},"geometry":{"type":"Polygon","coordinates":[[[15.422092500000076,59.85474749900004],[15.549673,59.72681799800006],[15.520207500000026,59.634589997000035],[15.71260650000005,59.52587899900004],[15.730538500000023,59.40852749600003],[15.60462350000006,59.39687749800004],[15.626599,59.30958399800005],[15.798456,59.23087299700006],[15.761593,59.19794849700003],[15.793618,59.10988599500007],[15.617399,59.02344499900005],[15.686749500000076,58.98191849600005],[15.559378500000037,58.95655799900004],[15.594214500000078,58.90625749700007],[15.554029,58.86807599500003],[15.263016,58.86462399900006],[15.034304500000076,58.70448299800006],[14.778936499000054,58.645992500000034],[14.691086,58.73967749700006],[14.538840500000049,58.675800999000046],[14.286332,58.94845249900004],[14.295995,59.01277950000008],[14.339636,59.08344249800007],[14.289464,59.157100497000044],[14.305491500000073,59.284824496000056],[14.43705650000004,59.462387],[14.391029500000059,59.51383599700006],[14.475906500000065,59.52899149700005],[14.401095500000054,59.74196999600008],[14.434949500000073,59.77083949900003],[14.42161550000003,59.90319449600008],[14.332584500000053,59.95231650000005],[14.456956,59.97297649900008],[14.437155500000074,60.026160996000044],[14.753791,59.99715799900008],[14.737407,60.08113499900003],[14.85119050000003,60.10405350000008],[15.422092500000076,59.85474749900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE125","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Västmanlands län","NUTS_NAME":"Västmanlands län"},"geometry":{"type":"Polygon","coordinates":[[[16.691070500000023,60.08332050000007],[16.83678050000003,59.86002749600004],[16.727358,59.77533699900005],[16.96491050000003,59.52132399900006],[16.25500100100004,59.45220949600008],[16.306436500000075,59.398899],[16.159219501000052,59.34619599600006],[15.991972,59.37535449900008],[15.875536,59.278323999000065],[15.893567999000027,59.230377],[15.761593,59.19794849700003],[15.798456,59.23087299700006],[15.626599,59.30958399800005],[15.60462350000006,59.39687749800004],[15.730538500000023,59.40852749600003],[15.71260650000005,59.52587899900004],[15.520207500000026,59.634589997000035],[15.549673,59.72681799800006],[15.422092500000076,59.85474749900004],[15.529911,59.956668999000044],[15.691971,59.947555495000074],[15.801567,60.179103498000075],[16.28451350000006,60.07316199700006],[16.703831,60.19572049700008],[16.745502500000043,60.13538349800007],[16.691070500000023,60.08332050000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE2","LEVL_CODE":1,"CNTR_CODE":"SE","NAME_LATN":"SÖDRA SVERIGE","NUTS_NAME":"SÖDRA SVERIGE"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.169407500000034,57.88666799900005],[19.122862,57.84081599900003],[19.034330500000067,57.906982998000046],[19.183279500000026,57.990034999000045],[19.350247,57.95936599700008],[19.14739,57.92498299700003],[19.169407500000034,57.88666799900005]]],[[[18.734817500000077,57.92931150000004],[18.77719250000007,57.867417],[19.003448500000047,57.915782999000044],[19.097435,57.819173],[18.810876,57.70597550000008],[18.761467500000037,57.46970350000004],[18.91781650000007,57.40334749800007],[18.660018,57.29484649900007],[18.711624,57.24468649700003],[18.39982549900003,57.13976399900008],[18.340005500000075,57.07232799800005],[18.366750999000033,56.984774996000056],[18.312185,56.93918450000007],[18.129537,56.90817750000008],[18.301114,57.09550850000005],[18.19604350000003,57.06748150000004],[18.21981450000004,57.14083849900004],[18.091156,57.259374],[18.185531,57.390713497000036],[18.11695099900004,57.44050450000003],[18.106193500000074,57.53464150000008],[18.460552500000063,57.819143],[18.734817500000077,57.92931150000004]]],[[[16.56806050000006,56.36792549900008],[16.400671,56.195806996000044],[16.406908500000043,56.41264649700008],[16.370938500000022,56.46075499700004],[16.413673,56.58997449900005],[16.631586500000026,56.87503949900008],[16.720757,56.88387499700008],[16.959014,57.291363499000056],[17.076804309000067,57.35587951500003],[17.073719999000048,57.179024],[16.907787,57.04047799700004],[16.865505,56.845187999000075],[16.774367,56.787356497000076],[16.56806050000006,56.36792549900008]]],[[[12.22861449900006,59.25669849800005],[12.48174,59.126941498000065],[12.695175,59.170833498000036],[12.898777,58.814361],[13.254575500000044,58.72561999700008],[13.425621,58.74576949900006],[13.338156,58.813301],[13.423325500000033,58.957587998000065],[13.589992,59.05944449900005],[14.295995,59.01277950000008],[14.286332,58.94845249900004],[14.538840500000049,58.675800999000046],[14.691086,58.73967749700006],[14.778936499000054,58.645992500000034],[14.41671,58.18724049600007],[14.808759500000065,58.08264549900008],[14.982714,58.14911249900007],[15.081543,57.998931998000046],[15.009529,57.98624799900006],[14.994365500000072,57.90547549900003],[15.128358500000047,57.71631250000007],[15.421051,57.704879497000036],[15.599087500000053,57.85389349700006],[15.942313999000078,57.80328749700004],[16.06543350000004,57.87426750000003],[15.986948,58.074669],[16.178619500000025,58.064018496000074],[16.239588,58.134933499000056],[16.531376,58.11576849900007],[16.666683500000033,57.99607649700005],[16.62113449900005,57.986361],[16.813461,57.99590199900007],[16.771834500000068,57.858126997000056],[16.469091500000047,57.890911],[16.70012950000006,57.75089299800004],[16.407790500000033,57.88058949900005],[16.732922,57.69605450000006],[16.63175650000005,57.682930999000064],[16.652136480000024,57.582051179000075],[16.727928,57.60177099600003],[16.668210500000043,57.559473499000035],[16.497907,57.63715349800003],[16.727634500000022,57.44355299700004],[16.464502500000037,57.26499449700003],[16.466294,57.15372149700005],[16.580416,57.045376],[16.45088050000004,57.04275549700003],[16.433601,56.98088549600004],[16.472051500000077,56.940429],[16.41184750000002,56.79190099900006],[16.484478,56.78741899700003],[16.359856500000035,56.76269149900003],[16.378437999000028,56.658735500000034],[16.248298500000033,56.634299498000075],[16.04987150000005,56.32188850000006],[15.99619750000005,56.199460499000054],[15.851547999000047,56.08510249700004],[15.581619,56.162849],[15.592694,56.21186899700007],[15.348007,56.12492250000008],[15.047866500000055,56.19555599900008],[14.687203,56.16711699900003],[14.78186850000003,56.03437999800008],[14.721438500000033,55.993557],[14.546399500000064,56.06131749900004],[14.220109500000035,55.83045749900003],[14.204630500000064,55.700960999000074],[14.360844,55.55425949700003],[14.194909,55.384035],[13.903504,55.43513799800007],[13.359049500000026,55.336853],[12.809839500000066,55.37825649900003],[12.968462,55.43839999700003],[12.889695,55.56459999800006],[13.063593,55.670109496000066],[12.44596050000007,56.30268],[12.783795,56.21956799700007],[12.836667500000033,56.27053949800006],[12.61962950000003,56.42242299800006],[12.899313,56.44900449900007],[12.936727500000075,56.58707699900003],[12.730945,56.65627649900006],[12.607982,56.820760997000036],[12.360219,56.926837997000064],[12.214189,57.13360750000004],[12.211207,57.22473899600004],[12.093919,57.23457049700005],[12.166869999000028,57.30980449900005],[12.064521499000023,57.33840049600008],[12.127203,57.378446],[12.108000999000069,57.434753500000056],[12.059552500000052,57.46236299600008],[12.004654500000072,57.340674498000055],[11.913888,57.40103749700006],[11.922900500000026,57.56222299900003],[11.927425001000074,57.62475799900005],[11.839630499000066,57.66350149800007],[11.990241500000025,57.72145350000005],[11.700709500000073,57.705336997000074],[11.835160500000029,57.794209],[11.673720500000059,57.84875499800006],[11.81427750000006,57.99311499800007],[11.844677282000077,58.16014767200005],[11.553980500000023,58.08578250000005],[11.406957,58.14387399900005],[11.678524500000037,58.23903999700008],[11.682809072000055,58.292475712000055],[11.501040999000054,58.250593998000056],[11.702689,58.43364799900007],[11.435055001000023,58.26864899700007],[11.358601,58.34339350000005],[11.43373850100005,58.43250349900006],[11.271686,58.35302849900006],[11.220071499000028,58.40892949700003],[11.309479500000066,58.47959049500008],[11.256905,58.677688],[11.174135001000025,58.71829249700005],[11.239031500000067,58.83897149900008],[11.113543499000059,58.99826349700004],[11.318191540000043,59.10402379800007],[11.338415674000032,59.10458747300004],[11.34712780600006,59.10483029300008],[11.460827,58.98865849800006],[11.457448500000055,58.888435497000046],[11.632557,58.90830650000004],[11.826196500000037,59.23784999700007],[12.084637500000042,59.17670799900003],[12.08104650000007,59.257522496000036],[12.22861449900006,59.25669849800005]]],[[[11.755686500000024,58.006059],[11.586167499000055,57.932169499000054],[11.502457,58.047987996000074],[11.708466,58.08634899900005],[11.682340500000066,58.06584699700005],[11.769487500000025,58.059569],[11.755686500000024,58.006059]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE21","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Småland med öarna","NUTS_NAME":"Småland med öarna"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.169407500000034,57.88666799900005],[19.122862,57.84081599900003],[19.034330500000067,57.906982998000046],[19.183279500000026,57.990034999000045],[19.350247,57.95936599700008],[19.14739,57.92498299700003],[19.169407500000034,57.88666799900005]]],[[[18.734817500000077,57.92931150000004],[18.77719250000007,57.867417],[19.003448500000047,57.915782999000044],[19.097435,57.819173],[18.810876,57.70597550000008],[18.761467500000037,57.46970350000004],[18.91781650000007,57.40334749800007],[18.660018,57.29484649900007],[18.711624,57.24468649700003],[18.39982549900003,57.13976399900008],[18.340005500000075,57.07232799800005],[18.366750999000033,56.984774996000056],[18.312185,56.93918450000007],[18.129537,56.90817750000008],[18.301114,57.09550850000005],[18.19604350000003,57.06748150000004],[18.21981450000004,57.14083849900004],[18.091156,57.259374],[18.185531,57.390713497000036],[18.11695099900004,57.44050450000003],[18.106193500000074,57.53464150000008],[18.460552500000063,57.819143],[18.734817500000077,57.92931150000004]]],[[[16.56806050000006,56.36792549900008],[16.400671,56.195806996000044],[16.406908500000043,56.41264649700008],[16.370938500000022,56.46075499700004],[16.413673,56.58997449900005],[16.631586500000026,56.87503949900008],[16.720757,56.88387499700008],[16.959014,57.291363499000056],[17.076804309000067,57.35587951500003],[17.073719999000048,57.179024],[16.907787,57.04047799700004],[16.865505,56.845187999000075],[16.774367,56.787356497000076],[16.56806050000006,56.36792549900008]]],[[[14.41671,58.18724049600007],[14.808759500000065,58.08264549900008],[14.982714,58.14911249900007],[15.081543,57.998931998000046],[15.009529,57.98624799900006],[14.994365500000072,57.90547549900003],[15.128358500000047,57.71631250000007],[15.421051,57.704879497000036],[15.599087500000053,57.85389349700006],[15.942313999000078,57.80328749700004],[16.06543350000004,57.87426750000003],[15.986948,58.074669],[16.178619500000025,58.064018496000074],[16.239588,58.134933499000056],[16.531376,58.11576849900007],[16.666683500000033,57.99607649700005],[16.62113449900005,57.986361],[16.813461,57.99590199900007],[16.771834500000068,57.858126997000056],[16.469091500000047,57.890911],[16.70012950000006,57.75089299800004],[16.407790500000033,57.88058949900005],[16.732922,57.69605450000006],[16.63175650000005,57.682930999000064],[16.652136480000024,57.582051179000075],[16.727928,57.60177099600003],[16.668210500000043,57.559473499000035],[16.497907,57.63715349800003],[16.727634500000022,57.44355299700004],[16.464502500000037,57.26499449700003],[16.466294,57.15372149700005],[16.580416,57.045376],[16.45088050000004,57.04275549700003],[16.433601,56.98088549600004],[16.472051500000077,56.940429],[16.41184750000002,56.79190099900006],[16.484478,56.78741899700003],[16.359856500000035,56.76269149900003],[16.378437999000028,56.658735500000034],[16.248298500000033,56.634299498000075],[16.04987150000005,56.32188850000006],[15.838573500000052,56.32371499900006],[15.586373500000036,56.48293999900005],[15.363503500000036,56.48237599600003],[15.011258999000063,56.44783799800007],[14.858296500000051,56.359028],[14.514669500000025,56.46003699700003],[14.099523499000043,56.53189449700005],[13.87668750000006,56.43731699600005],[13.46263650000003,56.42818850000003],[13.458932,56.52870949800007],[13.328169,56.687850998000044],[13.298661,56.823604500000044],[13.662645499000064,56.91290299700006],[13.681201,56.97211450000003],[13.488431,57.112910999000064],[13.122336500000074,57.025776],[13.080038,57.098521997000034],[13.100760500000035,57.145515498000066],[13.688058,57.558208500000035],[13.761729,57.89426449700005],[13.712541500000043,57.96596549700007],[13.772147,58.04856099600005],[14.068728500000077,58.02514250000007],[14.122593,58.10822299900008],[14.41671,58.18724049600007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE211","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Jönköpings län","NUTS_NAME":"Jönköpings län"},"geometry":{"type":"Polygon","coordinates":[[[15.421051,57.704879497000036],[15.630748500000038,57.60473249900008],[15.654259500000023,57.50207149900007],[15.490846500000032,57.43219749800005],[15.527632500000038,57.406826],[15.512460500000032,57.306854498000064],[15.457144500000027,57.27023299800004],[15.51653,57.201450496000064],[15.132341,57.23878849600004],[14.904491500000063,57.13467399800004],[14.752879,57.22707349800004],[14.363357500000063,57.15093599800008],[14.329294,57.023307998000064],[14.395977500000072,56.962913497000045],[14.298977,56.878970999000046],[14.071796500000062,57.04540250000008],[13.733304,57.035747498000035],[13.681201,56.97211450000003],[13.488431,57.112910999000064],[13.122336500000074,57.025776],[13.080038,57.098521997000034],[13.100760500000035,57.145515498000066],[13.688058,57.558208500000035],[13.761729,57.89426449700005],[13.712541500000043,57.96596549700007],[13.772147,58.04856099600005],[14.068728500000077,58.02514250000007],[14.122593,58.10822299900008],[14.41671,58.18724049600007],[14.808759500000065,58.08264549900008],[14.982714,58.14911249900007],[15.081543,57.998931998000046],[15.009529,57.98624799900006],[14.994365500000072,57.90547549900003],[15.128358500000047,57.71631250000007],[15.421051,57.704879497000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE212","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Kronobergs län","NUTS_NAME":"Kronobergs län"},"geometry":{"type":"Polygon","coordinates":[[[15.51653,57.201450496000064],[15.834081500000025,56.94460299700006],[15.541595500000028,56.92714699600003],[15.537199,56.80534],[15.36791050000005,56.78101749700005],[15.418052500000044,56.63969050000003],[15.366969,56.60236],[15.363503500000036,56.48237599600003],[15.011258999000063,56.44783799800007],[14.858296500000051,56.359028],[14.514669500000025,56.46003699700003],[14.099523499000043,56.53189449700005],[13.87668750000006,56.43731699600005],[13.46263650000003,56.42818850000003],[13.458932,56.52870949800007],[13.328169,56.687850998000044],[13.298661,56.823604500000044],[13.662645499000064,56.91290299700006],[13.681201,56.97211450000003],[13.733304,57.035747498000035],[14.071796500000062,57.04540250000008],[14.298977,56.878970999000046],[14.395977500000072,56.962913497000045],[14.329294,57.023307998000064],[14.363357500000063,57.15093599800008],[14.752879,57.22707349800004],[14.904491500000063,57.13467399800004],[15.132341,57.23878849600004],[15.51653,57.201450496000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE213","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Kalmar län","NUTS_NAME":"Kalmar län"},"geometry":{"type":"MultiPolygon","coordinates":[[[[16.56806050000006,56.36792549900008],[16.400671,56.195806996000044],[16.406908500000043,56.41264649700008],[16.370938500000022,56.46075499700004],[16.413673,56.58997449900005],[16.631586500000026,56.87503949900008],[16.720757,56.88387499700008],[16.959014,57.291363499000056],[17.076804309000067,57.35587951500003],[17.073719999000048,57.179024],[16.907787,57.04047799700004],[16.865505,56.845187999000075],[16.774367,56.787356497000076],[16.56806050000006,56.36792549900008]]],[[[16.531376,58.11576849900007],[16.666683500000033,57.99607649700005],[16.62113449900005,57.986361],[16.813461,57.99590199900007],[16.771834500000068,57.858126997000056],[16.469091500000047,57.890911],[16.70012950000006,57.75089299800004],[16.407790500000033,57.88058949900005],[16.732922,57.69605450000006],[16.63175650000005,57.682930999000064],[16.652136480000024,57.582051179000075],[16.727928,57.60177099600003],[16.668210500000043,57.559473499000035],[16.497907,57.63715349800003],[16.727634500000022,57.44355299700004],[16.464502500000037,57.26499449700003],[16.466294,57.15372149700005],[16.580416,57.045376],[16.45088050000004,57.04275549700003],[16.433601,56.98088549600004],[16.472051500000077,56.940429],[16.41184750000002,56.79190099900006],[16.484478,56.78741899700003],[16.359856500000035,56.76269149900003],[16.378437999000028,56.658735500000034],[16.248298500000033,56.634299498000075],[16.04987150000005,56.32188850000006],[15.838573500000052,56.32371499900006],[15.586373500000036,56.48293999900005],[15.363503500000036,56.48237599600003],[15.366969,56.60236],[15.418052500000044,56.63969050000003],[15.36791050000005,56.78101749700005],[15.537199,56.80534],[15.541595500000028,56.92714699600003],[15.834081500000025,56.94460299700006],[15.51653,57.201450496000064],[15.457144500000027,57.27023299800004],[15.512460500000032,57.306854498000064],[15.527632500000038,57.406826],[15.490846500000032,57.43219749800005],[15.654259500000023,57.50207149900007],[15.630748500000038,57.60473249900008],[15.421051,57.704879497000036],[15.599087500000053,57.85389349700006],[15.942313999000078,57.80328749700004],[16.06543350000004,57.87426750000003],[15.986948,58.074669],[16.178619500000025,58.064018496000074],[16.239588,58.134933499000056],[16.531376,58.11576849900007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE214","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Gotlands län","NUTS_NAME":"Gotlands län"},"geometry":{"type":"MultiPolygon","coordinates":[[[[19.169407500000034,57.88666799900005],[19.122862,57.84081599900003],[19.034330500000067,57.906982998000046],[19.183279500000026,57.990034999000045],[19.350247,57.95936599700008],[19.14739,57.92498299700003],[19.169407500000034,57.88666799900005]]],[[[18.734817500000077,57.92931150000004],[18.77719250000007,57.867417],[19.003448500000047,57.915782999000044],[19.097435,57.819173],[18.810876,57.70597550000008],[18.761467500000037,57.46970350000004],[18.91781650000007,57.40334749800007],[18.660018,57.29484649900007],[18.711624,57.24468649700003],[18.39982549900003,57.13976399900008],[18.340005500000075,57.07232799800005],[18.366750999000033,56.984774996000056],[18.312185,56.93918450000007],[18.129537,56.90817750000008],[18.301114,57.09550850000005],[18.19604350000003,57.06748150000004],[18.21981450000004,57.14083849900004],[18.091156,57.259374],[18.185531,57.390713497000036],[18.11695099900004,57.44050450000003],[18.106193500000074,57.53464150000008],[18.460552500000063,57.819143],[18.734817500000077,57.92931150000004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE22","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Sydsverige","NUTS_NAME":"Sydsverige"},"geometry":{"type":"Polygon","coordinates":[[[14.514669500000025,56.46003699700003],[14.858296500000051,56.359028],[15.011258999000063,56.44783799800007],[15.363503500000036,56.48237599600003],[15.586373500000036,56.48293999900005],[15.838573500000052,56.32371499900006],[16.04987150000005,56.32188850000006],[15.99619750000005,56.199460499000054],[15.851547999000047,56.08510249700004],[15.581619,56.162849],[15.592694,56.21186899700007],[15.348007,56.12492250000008],[15.047866500000055,56.19555599900008],[14.687203,56.16711699900003],[14.78186850000003,56.03437999800008],[14.721438500000033,55.993557],[14.546399500000064,56.06131749900004],[14.220109500000035,55.83045749900003],[14.204630500000064,55.700960999000074],[14.360844,55.55425949700003],[14.194909,55.384035],[13.903504,55.43513799800007],[13.359049500000026,55.336853],[12.809839500000066,55.37825649900003],[12.968462,55.43839999700003],[12.889695,55.56459999800006],[13.063593,55.670109496000066],[12.44596050000007,56.30268],[12.783795,56.21956799700007],[12.836667500000033,56.27053949800006],[12.61962950000003,56.42242299800006],[12.899313,56.44900449900007],[12.965949,56.42768099700004],[12.950138,56.36926250000005],[13.195944,56.32356999700005],[13.46263650000003,56.42818850000003],[13.87668750000006,56.43731699600005],[14.099523499000043,56.53189449700005],[14.514669500000025,56.46003699700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE221","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Blekinge län","NUTS_NAME":"Blekinge län"},"geometry":{"type":"Polygon","coordinates":[[[15.586373500000036,56.48293999900005],[15.838573500000052,56.32371499900006],[16.04987150000005,56.32188850000006],[15.99619750000005,56.199460499000054],[15.851547999000047,56.08510249700004],[15.581619,56.162849],[15.592694,56.21186899700007],[15.348007,56.12492250000008],[15.047866500000055,56.19555599900008],[14.687203,56.16711699900003],[14.78186850000003,56.03437999800008],[14.721438500000033,55.993557],[14.546399500000064,56.06131749900004],[14.585922,56.129901999000026],[14.56695250000007,56.22462449800008],[14.442976,56.228980998000054],[14.390576500000066,56.301818999000034],[14.414189500000077,56.40056999800004],[14.47277,56.40805799900005],[14.514669500000025,56.46003699700003],[14.858296500000051,56.359028],[15.011258999000063,56.44783799800007],[15.363503500000036,56.48237599600003],[15.586373500000036,56.48293999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE224","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Skåne län","NUTS_NAME":"Skåne län"},"geometry":{"type":"Polygon","coordinates":[[[14.47277,56.40805799900005],[14.414189500000077,56.40056999800004],[14.390576500000066,56.301818999000034],[14.442976,56.228980998000054],[14.56695250000007,56.22462449800008],[14.585922,56.129901999000026],[14.546399500000064,56.06131749900004],[14.220109500000035,55.83045749900003],[14.204630500000064,55.700960999000074],[14.360844,55.55425949700003],[14.194909,55.384035],[13.903504,55.43513799800007],[13.359049500000026,55.336853],[12.809839500000066,55.37825649900003],[12.968462,55.43839999700003],[12.889695,55.56459999800006],[13.063593,55.670109496000066],[12.44596050000007,56.30268],[12.783795,56.21956799700007],[12.836667500000033,56.27053949800006],[12.61962950000003,56.42242299800006],[12.899313,56.44900449900007],[12.965949,56.42768099700004],[12.950138,56.36926250000005],[13.195944,56.32356999700005],[13.46263650000003,56.42818850000003],[13.87668750000006,56.43731699600005],[14.099523499000043,56.53189449700005],[14.514669500000025,56.46003699700003],[14.47277,56.40805799900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE23","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Västsverige","NUTS_NAME":"Västsverige"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.22861449900006,59.25669849800005],[12.48174,59.126941498000065],[12.695175,59.170833498000036],[12.898777,58.814361],[13.254575500000044,58.72561999700008],[13.425621,58.74576949900006],[13.338156,58.813301],[13.423325500000033,58.957587998000065],[13.589992,59.05944449900005],[14.295995,59.01277950000008],[14.286332,58.94845249900004],[14.538840500000049,58.675800999000046],[14.691086,58.73967749700006],[14.778936499000054,58.645992500000034],[14.41671,58.18724049600007],[14.122593,58.10822299900008],[14.068728500000077,58.02514250000007],[13.772147,58.04856099600005],[13.712541500000043,57.96596549700007],[13.761729,57.89426449700005],[13.688058,57.558208500000035],[13.100760500000035,57.145515498000066],[13.080038,57.098521997000034],[13.122336500000074,57.025776],[13.488431,57.112910999000064],[13.681201,56.97211450000003],[13.662645499000064,56.91290299700006],[13.298661,56.823604500000044],[13.328169,56.687850998000044],[13.458932,56.52870949800007],[13.46263650000003,56.42818850000003],[13.195944,56.32356999700005],[12.950138,56.36926250000005],[12.965949,56.42768099700004],[12.899313,56.44900449900007],[12.936727500000075,56.58707699900003],[12.730945,56.65627649900006],[12.607982,56.820760997000036],[12.360219,56.926837997000064],[12.214189,57.13360750000004],[12.211207,57.22473899600004],[12.093919,57.23457049700005],[12.166869999000028,57.30980449900005],[12.064521499000023,57.33840049600008],[12.127203,57.378446],[12.108000999000069,57.434753500000056],[12.059552500000052,57.46236299600008],[12.004654500000072,57.340674498000055],[11.913888,57.40103749700006],[11.922900500000026,57.56222299900003],[11.927425001000074,57.62475799900005],[11.839630499000066,57.66350149800007],[11.990241500000025,57.72145350000005],[11.700709500000073,57.705336997000074],[11.835160500000029,57.794209],[11.673720500000059,57.84875499800006],[11.81427750000006,57.99311499800007],[11.844677282000077,58.16014767200005],[11.553980500000023,58.08578250000005],[11.406957,58.14387399900005],[11.678524500000037,58.23903999700008],[11.682809072000055,58.292475712000055],[11.501040999000054,58.250593998000056],[11.702689,58.43364799900007],[11.435055001000023,58.26864899700007],[11.358601,58.34339350000005],[11.43373850100005,58.43250349900006],[11.271686,58.35302849900006],[11.220071499000028,58.40892949700003],[11.309479500000066,58.47959049500008],[11.256905,58.677688],[11.174135001000025,58.71829249700005],[11.239031500000067,58.83897149900008],[11.113543499000059,58.99826349700004],[11.318191540000043,59.10402379800007],[11.338415674000032,59.10458747300004],[11.34712780600006,59.10483029300008],[11.460827,58.98865849800006],[11.457448500000055,58.888435497000046],[11.632557,58.90830650000004],[11.826196500000037,59.23784999700007],[12.084637500000042,59.17670799900003],[12.08104650000007,59.257522496000036],[12.22861449900006,59.25669849800005]]],[[[11.755686500000024,58.006059],[11.586167499000055,57.932169499000054],[11.502457,58.047987996000074],[11.708466,58.08634899900005],[11.682340500000066,58.06584699700005],[11.769487500000025,58.059569],[11.755686500000024,58.006059]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE231","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Hallands län","NUTS_NAME":"Hallands län"},"geometry":{"type":"Polygon","coordinates":[[[13.100760500000035,57.145515498000066],[13.080038,57.098521997000034],[13.122336500000074,57.025776],[13.488431,57.112910999000064],[13.681201,56.97211450000003],[13.662645499000064,56.91290299700006],[13.298661,56.823604500000044],[13.328169,56.687850998000044],[13.458932,56.52870949800007],[13.46263650000003,56.42818850000003],[13.195944,56.32356999700005],[12.950138,56.36926250000005],[12.965949,56.42768099700004],[12.899313,56.44900449900007],[12.936727500000075,56.58707699900003],[12.730945,56.65627649900006],[12.607982,56.820760997000036],[12.360219,56.926837997000064],[12.214189,57.13360750000004],[12.211207,57.22473899600004],[12.093919,57.23457049700005],[12.166869999000028,57.30980449900005],[12.064521499000023,57.33840049600008],[12.127203,57.378446],[12.108000999000069,57.434753500000056],[12.059552500000052,57.46236299600008],[12.004654500000072,57.340674498000055],[11.913888,57.40103749700006],[11.922900500000026,57.56222299900003],[12.277231001000075,57.596812999000065],[12.340670500000044,57.56116099700006],[12.323105001000044,57.46278749600003],[12.420299,57.39985249600005],[12.384351,57.34392949900007],[12.430993,57.26909999700007],[12.657167999000023,57.341560496000056],[13.100760500000035,57.145515498000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE232","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Västra Götalands län","NUTS_NAME":"Västra Götalands län"},"geometry":{"type":"MultiPolygon","coordinates":[[[[12.22861449900006,59.25669849800005],[12.48174,59.126941498000065],[12.695175,59.170833498000036],[12.898777,58.814361],[13.254575500000044,58.72561999700008],[13.425621,58.74576949900006],[13.338156,58.813301],[13.423325500000033,58.957587998000065],[13.589992,59.05944449900005],[14.295995,59.01277950000008],[14.286332,58.94845249900004],[14.538840500000049,58.675800999000046],[14.691086,58.73967749700006],[14.778936499000054,58.645992500000034],[14.41671,58.18724049600007],[14.122593,58.10822299900008],[14.068728500000077,58.02514250000007],[13.772147,58.04856099600005],[13.712541500000043,57.96596549700007],[13.761729,57.89426449700005],[13.688058,57.558208500000035],[13.100760500000035,57.145515498000066],[12.657167999000023,57.341560496000056],[12.430993,57.26909999700007],[12.384351,57.34392949900007],[12.420299,57.39985249600005],[12.323105001000044,57.46278749600003],[12.340670500000044,57.56116099700006],[12.277231001000075,57.596812999000065],[11.922900500000026,57.56222299900003],[11.927425001000074,57.62475799900005],[11.839630499000066,57.66350149800007],[11.990241500000025,57.72145350000005],[11.700709500000073,57.705336997000074],[11.835160500000029,57.794209],[11.673720500000059,57.84875499800006],[11.81427750000006,57.99311499800007],[11.844677282000077,58.16014767200005],[11.553980500000023,58.08578250000005],[11.406957,58.14387399900005],[11.678524500000037,58.23903999700008],[11.682809072000055,58.292475712000055],[11.501040999000054,58.250593998000056],[11.702689,58.43364799900007],[11.435055001000023,58.26864899700007],[11.358601,58.34339350000005],[11.43373850100005,58.43250349900006],[11.271686,58.35302849900006],[11.220071499000028,58.40892949700003],[11.309479500000066,58.47959049500008],[11.256905,58.677688],[11.174135001000025,58.71829249700005],[11.239031500000067,58.83897149900008],[11.113543499000059,58.99826349700004],[11.318191540000043,59.10402379800007],[11.338415674000032,59.10458747300004],[11.34712780600006,59.10483029300008],[11.460827,58.98865849800006],[11.457448500000055,58.888435497000046],[11.632557,58.90830650000004],[11.826196500000037,59.23784999700007],[12.084637500000042,59.17670799900003],[12.08104650000007,59.257522496000036],[12.22861449900006,59.25669849800005]]],[[[11.755686500000024,58.006059],[11.586167499000055,57.932169499000054],[11.502457,58.047987996000074],[11.708466,58.08634899900005],[11.682340500000066,58.06584699700005],[11.769487500000025,58.059569],[11.755686500000024,58.006059]]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE3","LEVL_CODE":1,"CNTR_CODE":"SE","NAME_LATN":"NORRA SVERIGE","NUTS_NAME":"NORRA SVERIGE"},"geometry":{"type":"Polygon","coordinates":[[[22.844611,68.38334150000003],[23.143492500000036,68.24682799900006],[23.15471750100005,68.136291],[23.652348999000026,67.958793],[23.494326001000047,67.86402599900003],[23.477203,67.70166250000005],[23.55897800100007,67.61996149800007],[23.39402350000006,67.48582099800007],[23.764582500000074,67.428214],[23.722462,67.28252899800003],[23.560987999000076,67.26085349800007],[23.55127200100003,67.17146199900003],[23.995160501000043,66.81970899800007],[23.879574500000047,66.76324399900005],[23.907815500000027,66.72142649900007],[23.856645,66.55690399900004],[23.650217501000043,66.45498399900004],[23.64559950000006,66.30140849900005],[23.918210499000054,66.16141],[24.155129,65.81602699900003],[23.47538650100006,65.84518749800003],[23.402833500000042,65.75004549800008],[23.217332500000055,65.82339349900008],[23.099601500000063,65.71327249700005],[22.671455,65.90403199800005],[22.617905,65.90125599900006],[22.690188500000033,65.86531349800003],[22.699252999000066,65.75771999800003],[22.340766001000077,65.86896049800004],[22.310333,65.78256899900003],[22.369043,65.66880349700006],[22.232683500000064,65.65303649700007],[22.37649,65.63100049800005],[22.294381500000043,65.60182449900003],[22.402720499000054,65.53735549600003],[21.831895001000078,65.52277549900003],[22.035621500000047,65.46488549900005],[21.768649500000038,65.40010249900007],[21.82642050000004,65.34806849800003],[21.44599050000005,65.38320899900003],[21.63175399900007,65.33305449900007],[21.634512,65.24373449800004],[21.750894,65.19125699900007],[21.524723359000063,65.23180558500007],[21.589963501000057,65.15028449900007],[21.506678500000078,65.14285999800006],[21.54360450000007,65.06305899700004],[21.23190150000005,64.94453999900003],[21.25731150000007,64.92216599700004],[21.156021,64.81548449900004],[20.997129,64.83223249600007],[21.297873,64.75350349800004],[21.21348050000006,64.73940199900005],[21.28227,64.66163399700008],[21.17361950000003,64.69389299900007],[21.60536250000007,64.43396099900008],[21.00002850000004,64.159882],[20.765231999000036,63.86661399900004],[20.560563,63.80467549700006],[20.574595500000044,63.72209999800003],[20.494180999000037,63.836441499000045],[20.396852,63.67203849900005],[20.305153500000074,63.78511399700005],[20.25163851700006,63.704674295000075],[20.300421500000027,63.74921249700003],[20.31427,63.67404250000004],[19.897695500000054,63.61702299800004],[19.683038999000075,63.434309],[19.449868500000036,63.57309199900004],[19.422997500000065,63.521906999000066],[19.486668500000064,63.41935599900006],[19.286468,63.469719499000064],[19.254426500000022,63.33568899900007],[19.038011,63.27557649600004],[19.08771550000006,63.22014999800007],[19.03094,63.187909999000055],[18.869752,63.294495],[18.710203,63.287126999000066],[18.901825,63.230789997000045],[18.74743650000005,63.17651649600003],[18.67888750000003,63.249727],[18.724288500000057,63.161021997000034],[18.359343500000023,63.05831],[18.436673,63.021690499000044],[18.331517,63.05352650000003],[18.287163500000077,63.00699950000006],[18.557591501000047,62.96259649900003],[18.382934001000024,62.959036998000045],[18.510375,62.93184449800003],[18.345507499000064,62.83480999800008],[18.201886001000048,62.88821449900007],[18.286522,62.851045998000075],[18.221234500000037,62.79667],[17.949062,62.79962999900005],[17.81687450000004,62.96930199800005],[17.90028650000005,62.96116599800007],[17.769321001000037,62.99744699900003],[17.98876350000006,62.702602],[17.885818500000028,62.66721799600003],[18.0637,62.61007099900007],[17.65642,62.467043499000056],[17.73663350000004,62.410059],[17.401207500000055,62.549360498000055],[17.433393500000022,62.506204499000035],[17.358609500000057,62.492508497000074],[17.37630991700007,62.448571625000056],[17.504983,62.44795099700008],[17.536147500000027,62.36078199900004],[17.359868,62.39108099600003],[17.392239500000073,62.323179],[17.659471500000052,62.23300349900006],[17.489299500000072,62.13695049800003],[17.460933,62.00146549700008],[17.339091500000052,61.95643749800007],[17.307419500000037,61.85283999600006],[17.40242950000004,61.72239749900007],[17.52387550000003,61.70176699900003],[17.487514,61.62040649700003],[17.312563499000078,61.72397549900006],[17.34512249900007,61.66663099900006],[17.094999,61.70968000100004],[17.255855500000052,61.656099],[17.086958500000037,61.63812899700008],[17.232869,61.593874999000036],[17.058534500000064,61.614304497000035],[17.142338,61.58367899900003],[17.03733950000003,61.574806496000065],[17.221907500000043,61.55158649900005],[17.040934999000058,61.54727999900007],[17.223366,61.42427749900003],[17.08862350000004,61.39873750000004],[17.28051350000004,61.275707],[17.115180500000065,61.20674949600004],[17.19188450000007,61.178396500000076],[17.136091500000077,61.00020549900006],[17.248538499000063,61.00658050000004],[17.070646,60.89967199800003],[17.330891500000064,60.88606499800005],[17.239781,60.81133199900006],[17.352966,60.752157502000045],[17.225903,60.68897899900003],[17.370218,60.654461499000035],[17.287695,60.446842],[17.192915,60.407017],[17.192478,60.30071249900004],[16.703831,60.19572049700008],[16.28451350000006,60.07316199700006],[15.801567,60.179103498000075],[15.691971,59.947555495000074],[15.529911,59.956668999000044],[15.422092500000076,59.85474749900004],[14.85119050000003,60.10405350000008],[14.737407,60.08113499900003],[14.753791,59.99715799900008],[14.437155500000074,60.026160996000044],[14.456956,59.97297649900008],[14.332584500000053,59.95231650000005],[14.42161550000003,59.90319449600008],[14.434949500000073,59.77083949900003],[14.401095500000054,59.74196999600008],[14.475906500000065,59.52899149700005],[14.391029500000059,59.51383599700006],[14.43705650000004,59.462387],[14.305491500000073,59.284824496000056],[14.289464,59.157100497000044],[14.339636,59.08344249800007],[14.295995,59.01277950000008],[13.589992,59.05944449900005],[13.423325500000033,58.957587998000065],[13.338156,58.813301],[13.425621,58.74576949900006],[13.254575500000044,58.72561999700008],[12.898777,58.814361],[12.695175,59.170833498000036],[12.48174,59.126941498000065],[12.22861449900006,59.25669849800005],[12.08104650000007,59.257522496000036],[12.084637500000042,59.17670799900003],[11.826196500000037,59.23784999700007],[11.691129,59.58954749900005],[11.939878500000077,59.69458099700006],[11.926970500000039,59.79047599800003],[11.839729,59.84076699900004],[12.174292,59.88980850000007],[12.499544999000022,60.09765699700006],[12.541906500000039,60.19337849700008],[12.498795500000028,60.323652],[12.606053,60.405925499000034],[12.606881500000043,60.512742496000044],[12.22399,61.013078],[12.670176500000025,61.055976497000074],[12.870847500000025,61.35649499700003],[12.569316,61.568746496000074],[12.419609,61.56298099900005],[12.137665,61.72381699700003],[12.29937,62.26749399900007],[12.25465950000006,62.331024496000055],[12.056142,62.61191849800008],[12.136381500000027,62.74791649700006],[12.074687999000048,62.902544997000064],[12.218231,63.00033249900008],[12.052455500000065,63.183444499000075],[11.974581,63.269227999000066],[12.212877,63.47859199900006],[12.149767,63.593946998000035],[12.683565,63.97422349900006],[13.211107500000026,64.09536949900007],[13.967524500000025,64.00796999900007],[14.157109,64.19505449600007],[14.113869500000021,64.46248450000007],[13.654257500000028,64.58034049800006],[14.325985,65.118916],[14.378797500000076,65.24762049800006],[14.50683250000003,65.30972899800008],[14.49877250000003,65.52129749600005],[14.625476500000048,65.81180849900005],[14.516289,66.13257899900003],[15.484732,66.28245849800004],[15.453994500000022,66.34523350000006],[15.377226500000063,66.48430399600005],[16.387759,67.04546249900005],[16.404099501000076,67.20497449900006],[16.089829,67.43527850000004],[16.15800150000007,67.519159],[16.407672,67.53409999900003],[16.738120500000036,67.91420949800005],[17.281524,68.11881499900005],[17.899761500000068,67.96937199900003],[18.151354,68.19878999900004],[18.101093001000038,68.406045],[18.125925,68.53651599800008],[18.405688500000053,68.58187699900003],[19.921397,68.356013],[20.226541500000053,68.49081250000006],[19.93750250000005,68.55793949900004],[20.33587250000005,68.80231249900004],[20.306589,68.92618199900005],[20.060047500000053,69.045759],[20.548636500000043,69.05996850000008],[21.88894300100003,68.58438349900007],[22.844611,68.38334150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE31","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Norra Mellansverige","NUTS_NAME":"Norra Mellansverige"},"geometry":{"type":"Polygon","coordinates":[[[17.307419500000037,61.85283999600006],[17.40242950000004,61.72239749900007],[17.52387550000003,61.70176699900003],[17.487514,61.62040649700003],[17.312563499000078,61.72397549900006],[17.34512249900007,61.66663099900006],[17.094999,61.70968000100004],[17.255855500000052,61.656099],[17.086958500000037,61.63812899700008],[17.232869,61.593874999000036],[17.058534500000064,61.614304497000035],[17.142338,61.58367899900003],[17.03733950000003,61.574806496000065],[17.221907500000043,61.55158649900005],[17.040934999000058,61.54727999900007],[17.223366,61.42427749900003],[17.08862350000004,61.39873750000004],[17.28051350000004,61.275707],[17.115180500000065,61.20674949600004],[17.19188450000007,61.178396500000076],[17.136091500000077,61.00020549900006],[17.248538499000063,61.00658050000004],[17.070646,60.89967199800003],[17.330891500000064,60.88606499800005],[17.239781,60.81133199900006],[17.352966,60.752157502000045],[17.225903,60.68897899900003],[17.370218,60.654461499000035],[17.287695,60.446842],[17.192915,60.407017],[17.192478,60.30071249900004],[16.703831,60.19572049700008],[16.28451350000006,60.07316199700006],[15.801567,60.179103498000075],[15.691971,59.947555495000074],[15.529911,59.956668999000044],[15.422092500000076,59.85474749900004],[14.85119050000003,60.10405350000008],[14.737407,60.08113499900003],[14.753791,59.99715799900008],[14.437155500000074,60.026160996000044],[14.456956,59.97297649900008],[14.332584500000053,59.95231650000005],[14.42161550000003,59.90319449600008],[14.434949500000073,59.77083949900003],[14.401095500000054,59.74196999600008],[14.475906500000065,59.52899149700005],[14.391029500000059,59.51383599700006],[14.43705650000004,59.462387],[14.305491500000073,59.284824496000056],[14.289464,59.157100497000044],[14.339636,59.08344249800007],[14.295995,59.01277950000008],[13.589992,59.05944449900005],[13.423325500000033,58.957587998000065],[13.338156,58.813301],[13.425621,58.74576949900006],[13.254575500000044,58.72561999700008],[12.898777,58.814361],[12.695175,59.170833498000036],[12.48174,59.126941498000065],[12.22861449900006,59.25669849800005],[12.08104650000007,59.257522496000036],[12.084637500000042,59.17670799900003],[11.826196500000037,59.23784999700007],[11.691129,59.58954749900005],[11.939878500000077,59.69458099700006],[11.926970500000039,59.79047599800003],[11.839729,59.84076699900004],[12.174292,59.88980850000007],[12.499544999000022,60.09765699700006],[12.541906500000039,60.19337849700008],[12.498795500000028,60.323652],[12.606053,60.405925499000034],[12.606881500000043,60.512742496000044],[12.22399,61.013078],[12.670176500000025,61.055976497000074],[12.870847500000025,61.35649499700003],[12.569316,61.568746496000074],[12.419609,61.56298099900005],[12.137665,61.72381699700003],[12.29937,62.26749399900007],[12.806175500000052,62.21990599800006],[13.158978500000046,62.01970299900006],[13.286972,62.05792249900003],[13.39365,62.000683],[13.20156650000007,61.928974],[13.558676,61.64341299900008],[14.447561500000063,61.594948],[14.498601,61.62541949600006],[14.531112500000063,61.78393149900006],[14.685895,61.90166849600007],[14.856937499000026,61.78346699800005],[14.759173499000042,61.86290749800003],[15.091695,61.81742849800008],[15.092639,61.997261],[15.430450500000063,62.14597249800005],[15.32733950000005,62.272083500000065],[15.614001500000029,62.34337249600003],[17.489299500000072,62.13695049800003],[17.460933,62.00146549700008],[17.339091500000052,61.95643749800007],[17.307419500000037,61.85283999600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE311","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Värmlands län","NUTS_NAME":"Värmlands län"},"geometry":{"type":"Polygon","coordinates":[[[14.437155500000074,60.026160996000044],[14.456956,59.97297649900008],[14.332584500000053,59.95231650000005],[14.42161550000003,59.90319449600008],[14.434949500000073,59.77083949900003],[14.401095500000054,59.74196999600008],[14.475906500000065,59.52899149700005],[14.391029500000059,59.51383599700006],[14.43705650000004,59.462387],[14.305491500000073,59.284824496000056],[14.289464,59.157100497000044],[14.339636,59.08344249800007],[14.295995,59.01277950000008],[13.589992,59.05944449900005],[13.423325500000033,58.957587998000065],[13.338156,58.813301],[13.425621,58.74576949900006],[13.254575500000044,58.72561999700008],[12.898777,58.814361],[12.695175,59.170833498000036],[12.48174,59.126941498000065],[12.22861449900006,59.25669849800005],[12.08104650000007,59.257522496000036],[12.084637500000042,59.17670799900003],[11.826196500000037,59.23784999700007],[11.691129,59.58954749900005],[11.939878500000077,59.69458099700006],[11.926970500000039,59.79047599800003],[11.839729,59.84076699900004],[12.174292,59.88980850000007],[12.499544999000022,60.09765699700006],[12.541906500000039,60.19337849700008],[12.498795500000028,60.323652],[12.606053,60.405925499000034],[12.606881500000043,60.512742496000044],[12.22399,61.013078],[12.670176500000025,61.055976497000074],[13.160275500000068,60.75510799700004],[13.467756500000064,60.45967099900008],[13.670397,60.40298099900008],[13.965625,60.17492299600008],[14.01171,60.19066249700006],[13.95123,60.25118649900003],[14.149735500000077,60.23045349900008],[14.21306050000004,60.09530599700008],[14.437155500000074,60.026160996000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE312","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Dalarnas län","NUTS_NAME":"Dalarnas län"},"geometry":{"type":"Polygon","coordinates":[[[14.447561500000063,61.594948],[14.676026499000045,61.591179],[14.680693500000075,61.485820998000065],[15.091183999000066,61.484031497000046],[15.151608500000066,61.596194997000055],[15.657477500000027,61.21643049900007],[15.763084500000048,61.04999150000003],[16.131170500000053,60.99192049900006],[16.386562500000025,60.786064],[16.237739500000032,60.622925499000075],[16.135956,60.60968],[16.17071550000003,60.55904399800005],[16.703831,60.19572049700008],[16.28451350000006,60.07316199700006],[15.801567,60.179103498000075],[15.691971,59.947555495000074],[15.529911,59.956668999000044],[15.422092500000076,59.85474749900004],[14.85119050000003,60.10405350000008],[14.737407,60.08113499900003],[14.753791,59.99715799900008],[14.437155500000074,60.026160996000044],[14.21306050000004,60.09530599700008],[14.149735500000077,60.23045349900008],[13.95123,60.25118649900003],[14.01171,60.19066249700006],[13.965625,60.17492299600008],[13.670397,60.40298099900008],[13.467756500000064,60.45967099900008],[13.160275500000068,60.75510799700004],[12.670176500000025,61.055976497000074],[12.870847500000025,61.35649499700003],[12.569316,61.568746496000074],[12.419609,61.56298099900005],[12.137665,61.72381699700003],[12.29937,62.26749399900007],[12.806175500000052,62.21990599800006],[13.158978500000046,62.01970299900006],[13.286972,62.05792249900003],[13.39365,62.000683],[13.20156650000007,61.928974],[13.558676,61.64341299900008],[14.447561500000063,61.594948]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE313","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Gävleborgs län","NUTS_NAME":"Gävleborgs län"},"geometry":{"type":"Polygon","coordinates":[[[17.307419500000037,61.85283999600006],[17.40242950000004,61.72239749900007],[17.52387550000003,61.70176699900003],[17.487514,61.62040649700003],[17.312563499000078,61.72397549900006],[17.34512249900007,61.66663099900006],[17.094999,61.70968000100004],[17.255855500000052,61.656099],[17.086958500000037,61.63812899700008],[17.232869,61.593874999000036],[17.058534500000064,61.614304497000035],[17.142338,61.58367899900003],[17.03733950000003,61.574806496000065],[17.221907500000043,61.55158649900005],[17.040934999000058,61.54727999900007],[17.223366,61.42427749900003],[17.08862350000004,61.39873750000004],[17.28051350000004,61.275707],[17.115180500000065,61.20674949600004],[17.19188450000007,61.178396500000076],[17.136091500000077,61.00020549900006],[17.248538499000063,61.00658050000004],[17.070646,60.89967199800003],[17.330891500000064,60.88606499800005],[17.239781,60.81133199900006],[17.352966,60.752157502000045],[17.225903,60.68897899900003],[17.370218,60.654461499000035],[17.287695,60.446842],[17.192915,60.407017],[17.192478,60.30071249900004],[16.703831,60.19572049700008],[16.17071550000003,60.55904399800005],[16.135956,60.60968],[16.237739500000032,60.622925499000075],[16.386562500000025,60.786064],[16.131170500000053,60.99192049900006],[15.763084500000048,61.04999150000003],[15.657477500000027,61.21643049900007],[15.151608500000066,61.596194997000055],[15.091183999000066,61.484031497000046],[14.680693500000075,61.485820998000065],[14.676026499000045,61.591179],[14.447561500000063,61.594948],[14.498601,61.62541949600006],[14.531112500000063,61.78393149900006],[14.685895,61.90166849600007],[14.856937499000026,61.78346699800005],[14.759173499000042,61.86290749800003],[15.091695,61.81742849800008],[15.092639,61.997261],[15.430450500000063,62.14597249800005],[15.32733950000005,62.272083500000065],[15.614001500000029,62.34337249600003],[17.489299500000072,62.13695049800003],[17.460933,62.00146549700008],[17.339091500000052,61.95643749800007],[17.307419500000037,61.85283999600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE32","LEVL_CODE":2,"CNTR_CODE":"SE","NAME_LATN":"Mellersta Norrland","NUTS_NAME":"Mellersta Norrland"},"geometry":{"type":"Polygon","coordinates":[[[15.239639,64.63615399900004],[15.578810500000031,64.44361849900008],[15.704471500000068,64.53781899900008],[15.740346,64.40313699900008],[16.023498499000027,64.33295450000003],[16.040148,64.24841299600007],[16.219421500000067,64.26332049900003],[16.718954,64.02787799800006],[17.261841,63.91410049600006],[17.731568999000046,63.879935999000054],[18.405653,63.99566249900005],[18.473282,63.96616399700008],[18.406906,63.92540749700004],[18.485727500000053,63.84251799700007],[18.95877050000007,63.761131498000054],[19.286468,63.469719499000064],[19.254426500000022,63.33568899900007],[19.038011,63.27557649600004],[19.08771550000006,63.22014999800007],[19.03094,63.187909999000055],[18.869752,63.294495],[18.710203,63.287126999000066],[18.901825,63.230789997000045],[18.74743650000005,63.17651649600003],[18.67888750000003,63.249727],[18.724288500000057,63.161021997000034],[18.359343500000023,63.05831],[18.436673,63.021690499000044],[18.331517,63.05352650000003],[18.287163500000077,63.00699950000006],[18.557591501000047,62.96259649900003],[18.382934001000024,62.959036998000045],[18.510375,62.93184449800003],[18.345507499000064,62.83480999800008],[18.201886001000048,62.88821449900007],[18.286522,62.851045998000075],[18.221234500000037,62.79667],[17.949062,62.79962999900005],[17.81687450000004,62.96930199800005],[17.90028650000005,62.96116599800007],[17.769321001000037,62.99744699900003],[17.98876350000006,62.702602],[17.885818500000028,62.66721799600003],[18.0637,62.61007099900007],[17.65642,62.467043499000056],[17.73663350000004,62.410059],[17.401207500000055,62.549360498000055],[17.433393500000022,62.506204499000035],[17.358609500000057,62.492508497000074],[17.37630991700007,62.448571625000056],[17.504983,62.44795099700008],[17.536147500000027,62.36078199900004],[17.359868,62.39108099600003],[17.392239500000073,62.323179],[17.659471500000052,62.23300349900006],[17.489299500000072,62.13695049800003],[15.614001500000029,62.34337249600003],[15.32733950000005,62.272083500000065],[15.430450500000063,62.14597249800005],[15.092639,61.997261],[15.091695,61.81742849800008],[14.759173499000042,61.86290749800003],[14.856937499000026,61.78346699800005],[14.685895,61.90166849600007],[14.531112500000063,61.78393149900006],[14.498601,61.62541949600006],[14.447561500000063,61.594948],[13.558676,61.64341299900008],[13.20156650000007,61.928974],[13.39365,62.000683],[13.286972,62.05792249900003],[13.158978500000046,62.01970299900006],[12.806175500000052,62.21990599800006],[12.29937,62.26749399900007],[12.25465950000006,62.331024496000055],[12.056142,62.61191849800008],[12.136381500000027,62.74791649700006],[12.074687999000048,62.902544997000064],[12.218231,63.00033249900008],[12.052455500000065,63.183444499000075],[11.974581,63.269227999000066],[12.212877,63.47859199900006],[12.149767,63.593946998000035],[12.683565,63.97422349900006],[13.211107500000026,64.09536949900007],[13.967524500000025,64.00796999900007],[14.157109,64.19505449600007],[14.113869500000021,64.46248450000007],[13.654257500000028,64.58034049800006],[14.325985,65.118916],[15.334211500000038,64.66961650000007],[15.239639,64.63615399900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE321","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Västernorrlands län","NUTS_NAME":"Västernorrlands län"},"geometry":{"type":"Polygon","coordinates":[[[17.261841,63.91410049600006],[17.731568999000046,63.879935999000054],[18.405653,63.99566249900005],[18.473282,63.96616399700008],[18.406906,63.92540749700004],[18.485727500000053,63.84251799700007],[18.95877050000007,63.761131498000054],[19.286468,63.469719499000064],[19.254426500000022,63.33568899900007],[19.038011,63.27557649600004],[19.08771550000006,63.22014999800007],[19.03094,63.187909999000055],[18.869752,63.294495],[18.710203,63.287126999000066],[18.901825,63.230789997000045],[18.74743650000005,63.17651649600003],[18.67888750000003,63.249727],[18.724288500000057,63.161021997000034],[18.359343500000023,63.05831],[18.436673,63.021690499000044],[18.331517,63.05352650000003],[18.287163500000077,63.00699950000006],[18.557591501000047,62.96259649900003],[18.382934001000024,62.959036998000045],[18.510375,62.93184449800003],[18.345507499000064,62.83480999800008],[18.201886001000048,62.88821449900007],[18.286522,62.851045998000075],[18.221234500000037,62.79667],[17.949062,62.79962999900005],[17.81687450000004,62.96930199800005],[17.90028650000005,62.96116599800007],[17.769321001000037,62.99744699900003],[17.98876350000006,62.702602],[17.885818500000028,62.66721799600003],[18.0637,62.61007099900007],[17.65642,62.467043499000056],[17.73663350000004,62.410059],[17.401207500000055,62.549360498000055],[17.433393500000022,62.506204499000035],[17.358609500000057,62.492508497000074],[17.37630991700007,62.448571625000056],[17.504983,62.44795099700008],[17.536147500000027,62.36078199900004],[17.359868,62.39108099600003],[17.392239500000073,62.323179],[17.659471500000052,62.23300349900006],[17.489299500000072,62.13695049800003],[15.614001500000029,62.34337249600003],[15.32733950000005,62.272083500000065],[14.940039500000069,62.32027799700006],[14.800273,62.404278],[14.849283,62.467677999000045],[14.780597,62.59052649600005],[16.369146500000056,62.695632997000075],[16.671231,62.916775],[16.999031500000058,62.94648299900007],[16.096855,63.36851099900008],[16.140632500000038,63.42062799900003],[15.96642,63.443732999000076],[15.999298,63.54999899900008],[15.791387500000042,63.65770749600006],[16.613163,63.71351599900004],[16.563432500000033,63.823108500000046],[16.718954,64.02787799800006],[17.261841,63.91410049600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"SE322","LEVL_CODE":3,"CNTR_CODE":"SE","NAME_LATN":"Jämtlands län","NUTS_NAME":"Jämtlands län"},"geometry":{"type":"Polygon","coordinates":[[[16.718954,64.02787799800006],[16.563432500000033,63.823108500000046],[16.613163,63.71351599900004],[15.791387500000042,63.65770749600006],[15.999298,63.54999899900008],[15.96642,63.443732999000076],[16.140632500000038,63.42062799900003],[16.096855,63.36851099900008],[16.999031500000058,62.94648299900007],[16.671231,62.916775],[16.369146500000056,62.695632997000075],[14.780597,62.59052649600005],[14.849283,62.467677999000045],[14.800273,62.404278],[14.940039500000069,62.32027799700006],[15.32733950000005,62.272083500000065],[15.430450500000063,62.14597249800005],[15.092639,61.997261],[15.091695,61.81742849800008],[14.759173499000042,61.86290749800003],[14.856937499000026,61.78346699800005],[14.685895,61.90166849600007],[14.531112500000063,61.78393149900006],[14.498601,61.62541949600006],[14.447561500000063,61.594948],[13.558676,61.64341299900008],[13.20156650000007,61.928974],[13.39365,62.000683],[13.286972,62.05792249900003],[13.158978500000046,62.01970299900006],[12.806175500000052,62.21990599800006],[12.29937,62.26749399900007],[12.25465950000006,62.331024496000055],[12.056142,62.61191849800008],[12.136381500000027,62.74791649700006],[12.074687999000048,62.902544997000064],[12.218231,63.00033249900008],[12.052455500000065,63.183444499000075],[11.974581,63.269227999000066],[12.212877,63.47859199900006],[12.149767,63.593946998000035],[12.683565,63.97422349900006],[13.211107500000026,64.09536949900007],[13.967524500000025,64.00796999900007],[14.157109,64.19505449600007],[14.113869500000021,64.46248450000007],[13.654257500000028,64.58034049800006],[14.325985,65.118916],[15.334211500000038,64.66961650000007],[15.239639,64.63615399900004],[15.578810500000031,64.44361849900008],[15.704471500000068,64.53781899900008],[15.740346,64.40313699900008],[16.023498499000027,64.33295450000003],[16.040148,64.24841299600007],[16.219421500000067,64.26332049900003],[16.718954,64.02787799800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR222","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Çanakkale","NUTS_NAME":"Çanakkale"},"geometry":{"type":"MultiPolygon","coordinates":[[[[27.319921500000078,40.43341249700006],[27.315832,40.37848099800004],[27.50628050000006,40.304677998000045],[27.363447499000074,40.146663998000065],[27.44819750000005,40.04782],[27.489155500000038,39.89518149700007],[27.430025500000056,39.77245499700007],[27.23951550000004,39.68220249900003],[26.967601500000058,39.76314799700003],[26.713539500000024,39.70721849800003],[26.618685,39.54787949800004],[26.204252500000052,39.46097399800004],[26.062643,39.479497998000056],[26.163519,39.659459996000066],[26.137392,39.84089499800007],[26.179243,39.99143399700006],[26.321268,40.01747699600003],[26.397505,40.145826498000076],[26.403608500000075,40.19677149900008],[26.524309,40.21770649700005],[26.756845500000054,40.40379899700008],[27.033163,40.38958949900007],[27.286051,40.475088],[27.319921500000078,40.43341249700006]]],[[[26.962811,40.66703799800007],[26.908618,40.600146497000026],[26.968651500000078,40.55336899800005],[26.694452500000068,40.45057849800003],[26.592272,40.322187496000026],[26.358301,40.20253199900003],[26.38106350000004,40.143331498000066],[26.218744,40.04709449600006],[26.165872500000035,40.052560999000036],[26.27830699900005,40.21555899800006],[26.218675500000074,40.319448496000064],[26.838197500000035,40.58582499800008],[26.80476550000003,40.65426449600005],[26.734886500000073,40.64293299700006],[26.773845,40.741130998000074],[26.967178,40.73651299800008],[26.962811,40.66703799800007]]],[[[25.97707950000006,40.21614249700008],[25.965894500000047,40.15049950000008],[26.013651,40.16370949700007],[25.98987,40.13219250000003],[25.738722,40.09072699800004],[25.665178500000025,40.12435349700007],[25.78412050000003,40.20969949600004],[25.97707950000006,40.21614249700008]]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR3","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"EGE","NUTS_NAME":"EGE"},"geometry":{"type":"Polygon","coordinates":[[[29.760535,39.71012349700004],[30.06013450000006,39.658640999000056],[30.193454,39.65880449900004],[30.230593500000055,39.57547949800005],[30.355477,39.49900499800003],[30.36653,39.40263399600008],[30.315111,39.36209499800003],[30.42949150000004,39.21761249800005],[30.74145550000003,39.126772998000035],[31.192543,39.27911049700003],[31.303426,39.26033799800007],[31.389171,39.14805799700008],[31.619732500000055,39.10305549800006],[31.616613,39.00463699800008],[31.721062500000073,38.94946949700005],[31.62634950000006,38.92188599800005],[31.635725500000035,38.82079299700007],[31.52451650000006,38.69974099800004],[31.621331,38.619947497000055],[31.23334950000003,38.40997299700007],[31.13277,38.48890349800007],[30.986741,38.46641449800006],[30.70472249900007,38.265575],[30.42972050000003,38.15599899800003],[30.327528,38.03099949800003],[30.174609,37.95408949800003],[30.039979500000072,37.752413998000065],[29.852119,37.75228549800005],[29.500381,37.61887199700004],[29.514886,37.460002],[29.606530500000076,37.39536099700007],[29.341447,37.00684899700008],[29.422852500000033,36.956733996000025],[29.386635500000068,36.85942699800006],[29.60908950000004,36.98030049800008],[29.713264499000047,36.95630949800005],[29.751884,36.900562998000055],[29.637207,36.669686],[29.317792,36.44839999900006],[29.26056150000005,36.30439899700008],[29.102026,36.386720497000056],[29.125913500000024,36.542272499000035],[29.02238650000004,36.58621399800006],[29.115694,36.62598249700005],[29.103811,36.67034349700003],[28.928788999000062,36.754858],[28.851038,36.66208449800007],[28.878309,36.62911799800003],[28.846163,36.58836949600004],[28.790045,36.685636500000044],[28.618429,36.70327949800003],[28.630129,36.781706],[28.558664500000077,36.831861497000034],[28.53590599900008,36.78577999600003],[28.270406500000036,36.852479997000046],[28.233087500000067,36.80461299800004],[28.279173,36.734334998000065],[28.163698,36.68377899800004],[28.089052500000037,36.582017998000026],[27.97934550000008,36.553571497000064],[27.956487500000037,36.60048499800007],[28.095728,36.64494499700004],[27.963636500000064,36.683786498000075],[28.10157950000007,36.705804999000065],[28.131860500000073,36.79396650000007],[27.730821500000047,36.763143497000044],[27.67334150000005,36.67207149700005],[27.478128500000025,36.64709299800006],[27.374422,36.68548399900004],[27.428606,36.748281500000076],[27.642294,36.80760399800005],[28.032835,36.78775999800007],[28.059328,36.878801500000066],[28.014459500000044,36.90612599800005],[28.159700500000042,36.925954998000066],[28.330560499000057,37.03339549700007],[27.565542,36.97728549800007],[27.42487150000005,37.03574199700006],[27.264269,36.964120997000066],[27.225256,37.05810349700005],[27.32292,37.15848349700008],[27.470903500000077,37.08077799800003],[27.546667,37.12506699900007],[27.526208999000062,37.193677996000076],[27.611025,37.25751299900003],[27.411882500000047,37.30456749900003],[27.404363,37.36809499900005],[27.424582461000057,37.40700347200004],[27.419004802000075,37.41190175500003],[27.202436500000033,37.350824499000055],[27.21665,37.59111199800003],[27.003069,37.65979599800005],[27.234331,37.72418399800006],[27.264569,37.87504849700008],[27.24043450000005,37.98498349800008],[26.980882500000064,38.07474699800008],[26.864545500000077,38.02855499900005],[26.757936500000028,38.221479499000054],[26.632337500000062,38.19990350000006],[26.5912,38.102750497000045],[26.281446500000072,38.26644349800006],[26.284891,38.376329498000075],[26.379034,38.309190998000076],[26.51573,38.42750749600003],[26.37163350000003,38.53015349800006],[26.353018,38.63760549800003],[26.397398,38.66724949800005],[26.47363850000005,38.674909497000044],[26.62529550000005,38.52710550000006],[26.611200500000052,38.42235349900005],[26.67402050000004,38.311986999000055],[26.707414499000038,38.43452649900007],[26.808508,38.35518449700004],[27.167742,38.44041999800004],[26.951025,38.443891499000074],[26.733785500000067,38.653470998000046],[26.740319999000064,38.741529497000045],[26.89434650000004,38.73359099800007],[26.939085,38.76427649800007],[26.892832,38.824922498000035],[27.066274500000077,38.87722599700004],[27.043180500000062,38.94094649800007],[26.80241599900006,38.95018599800005],[26.799127500000054,39.032182498000054],[26.886392500000056,39.07982449700006],[26.763921,39.17520999800007],[27.064404500000023,39.36679849800004],[27.381316,39.35274650000008],[27.462255500000026,39.39673249800006],[27.800632,39.31310799900007],[27.900652,39.348802997000064],[27.969201,39.195721],[28.15897,39.05698399800008],[28.652546500000028,39.14736849800005],[28.597988,39.280635999000026],[28.885427,39.372048997000036],[28.968574,39.60067749800004],[29.194093500000065,39.600573997000026],[29.28185,39.73377549800006],[29.405343,39.809691996000026],[29.396933,39.901012499000046],[29.742183500000067,39.87133399900006],[29.760535,39.71012349700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR31","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Izmir","NUTS_NAME":"Izmir"},"geometry":{"type":"Polygon","coordinates":[[[27.42629050000005,38.94496649800004],[27.144471,38.88617799900004],[27.10131349900007,38.77824549700006],[27.260101500000076,38.54846750000007],[27.680527,38.42624499800007],[27.816327,38.29641849600006],[27.889356,38.411711499000035],[28.06295750000004,38.39836149700005],[28.137481999000045,38.31729899700008],[28.28641950000008,38.31971599800005],[28.452412500000037,38.22420999700006],[28.44260950000006,38.10466149800004],[28.285376,38.110739498000044],[28.185534500000074,38.00091449800004],[27.714490500000068,37.949471998000035],[27.556524500000023,37.99826899800007],[27.36633450000005,37.86620799800005],[27.264569,37.87504849700008],[27.24043450000005,37.98498349800008],[26.980882500000064,38.07474699800008],[26.864545500000077,38.02855499900005],[26.757936500000028,38.221479499000054],[26.632337500000062,38.19990350000006],[26.5912,38.102750497000045],[26.281446500000072,38.26644349800006],[26.284891,38.376329498000075],[26.379034,38.309190998000076],[26.51573,38.42750749600003],[26.37163350000003,38.53015349800006],[26.353018,38.63760549800003],[26.397398,38.66724949800005],[26.47363850000005,38.674909497000044],[26.62529550000005,38.52710550000006],[26.611200500000052,38.42235349900005],[26.67402050000004,38.311986999000055],[26.707414499000038,38.43452649900007],[26.808508,38.35518449700004],[27.167742,38.44041999800004],[26.951025,38.443891499000074],[26.733785500000067,38.653470998000046],[26.740319999000064,38.741529497000045],[26.89434650000004,38.73359099800007],[26.939085,38.76427649800007],[26.892832,38.824922498000035],[27.066274500000077,38.87722599700004],[27.043180500000062,38.94094649800007],[26.80241599900006,38.95018599800005],[26.799127500000054,39.032182498000054],[26.886392500000056,39.07982449700006],[26.763921,39.17520999800007],[27.064404500000023,39.36679849800004],[27.381316,39.35274650000008],[27.452409999000054,39.25615049700008],[27.48838150000006,39.06407850000005],[27.42629050000005,38.94496649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR310","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Izmir","NUTS_NAME":"Izmir"},"geometry":{"type":"Polygon","coordinates":[[[27.42629050000005,38.94496649800004],[27.144471,38.88617799900004],[27.10131349900007,38.77824549700006],[27.260101500000076,38.54846750000007],[27.680527,38.42624499800007],[27.816327,38.29641849600006],[27.889356,38.411711499000035],[28.06295750000004,38.39836149700005],[28.137481999000045,38.31729899700008],[28.28641950000008,38.31971599800005],[28.452412500000037,38.22420999700006],[28.44260950000006,38.10466149800004],[28.285376,38.110739498000044],[28.185534500000074,38.00091449800004],[27.714490500000068,37.949471998000035],[27.556524500000023,37.99826899800007],[27.36633450000005,37.86620799800005],[27.264569,37.87504849700008],[27.24043450000005,37.98498349800008],[26.980882500000064,38.07474699800008],[26.864545500000077,38.02855499900005],[26.757936500000028,38.221479499000054],[26.632337500000062,38.19990350000006],[26.5912,38.102750497000045],[26.281446500000072,38.26644349800006],[26.284891,38.376329498000075],[26.379034,38.309190998000076],[26.51573,38.42750749600003],[26.37163350000003,38.53015349800006],[26.353018,38.63760549800003],[26.397398,38.66724949800005],[26.47363850000005,38.674909497000044],[26.62529550000005,38.52710550000006],[26.611200500000052,38.42235349900005],[26.67402050000004,38.311986999000055],[26.707414499000038,38.43452649900007],[26.808508,38.35518449700004],[27.167742,38.44041999800004],[26.951025,38.443891499000074],[26.733785500000067,38.653470998000046],[26.740319999000064,38.741529497000045],[26.89434650000004,38.73359099800007],[26.939085,38.76427649800007],[26.892832,38.824922498000035],[27.066274500000077,38.87722599700004],[27.043180500000062,38.94094649800007],[26.80241599900006,38.95018599800005],[26.799127500000054,39.032182498000054],[26.886392500000056,39.07982449700006],[26.763921,39.17520999800007],[27.064404500000023,39.36679849800004],[27.381316,39.35274650000008],[27.452409999000054,39.25615049700008],[27.48838150000006,39.06407850000005],[27.42629050000005,38.94496649800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR32","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Aydin, Denizli, Mugla","NUTS_NAME":"Aydin, Denizli, Mugla"},"geometry":{"type":"Polygon","coordinates":[[[29.946554,38.39677449900006],[30.02725350000003,38.214679499000056],[29.65392250000008,37.96017549800007],[29.663986,37.864153997000074],[29.852119,37.75228549800005],[29.500381,37.61887199700004],[29.514886,37.460002],[29.606530500000076,37.39536099700007],[29.341447,37.00684899700008],[29.422852500000033,36.956733996000025],[29.386635500000068,36.85942699800006],[29.60908950000004,36.98030049800008],[29.713264499000047,36.95630949800005],[29.751884,36.900562998000055],[29.637207,36.669686],[29.317792,36.44839999900006],[29.26056150000005,36.30439899700008],[29.102026,36.386720497000056],[29.125913500000024,36.542272499000035],[29.02238650000004,36.58621399800006],[29.115694,36.62598249700005],[29.103811,36.67034349700003],[28.928788999000062,36.754858],[28.851038,36.66208449800007],[28.878309,36.62911799800003],[28.846163,36.58836949600004],[28.790045,36.685636500000044],[28.618429,36.70327949800003],[28.630129,36.781706],[28.558664500000077,36.831861497000034],[28.53590599900008,36.78577999600003],[28.270406500000036,36.852479997000046],[28.233087500000067,36.80461299800004],[28.279173,36.734334998000065],[28.163698,36.68377899800004],[28.089052500000037,36.582017998000026],[27.97934550000008,36.553571497000064],[27.956487500000037,36.60048499800007],[28.095728,36.64494499700004],[27.963636500000064,36.683786498000075],[28.10157950000007,36.705804999000065],[28.131860500000073,36.79396650000007],[27.730821500000047,36.763143497000044],[27.67334150000005,36.67207149700005],[27.478128500000025,36.64709299800006],[27.374422,36.68548399900004],[27.428606,36.748281500000076],[27.642294,36.80760399800005],[28.032835,36.78775999800007],[28.059328,36.878801500000066],[28.014459500000044,36.90612599800005],[28.159700500000042,36.925954998000066],[28.330560499000057,37.03339549700007],[27.565542,36.97728549800007],[27.42487150000005,37.03574199700006],[27.264269,36.964120997000066],[27.225256,37.05810349700005],[27.32292,37.15848349700008],[27.470903500000077,37.08077799800003],[27.546667,37.12506699900007],[27.526208999000062,37.193677996000076],[27.611025,37.25751299900003],[27.411882500000047,37.30456749900003],[27.404363,37.36809499900005],[27.424582461000057,37.40700347200004],[27.419004802000075,37.41190175500003],[27.202436500000033,37.350824499000055],[27.21665,37.59111199800003],[27.003069,37.65979599800005],[27.234331,37.72418399800006],[27.264569,37.87504849700008],[27.36633450000005,37.86620799800005],[27.556524500000023,37.99826899800007],[27.714490500000068,37.949471998000035],[28.185534500000074,38.00091449800004],[28.285376,38.110739498000044],[28.44260950000006,38.10466149800004],[28.616089,38.089585998000075],[28.847341500000027,38.240428500000064],[29.23033350000003,38.23764349900006],[29.379754,38.30714599800007],[29.580195,38.246578498000076],[29.61755850000003,38.403098998000075],[29.75570350000004,38.48430749600004],[29.946554,38.39677449900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR321","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Aydin","NUTS_NAME":"Aydin"},"geometry":{"type":"Polygon","coordinates":[[[28.285376,38.110739498000044],[28.44260950000006,38.10466149800004],[28.616089,38.089585998000075],[28.793492,37.98045349800003],[28.70325150000008,37.881838498000036],[28.741235500000073,37.78772349900004],[28.83511,37.73129999900004],[28.793877,37.652881498000056],[28.55216850000005,37.53501349700008],[28.588908,37.45993949800004],[28.254244500000027,37.53932899700004],[28.088255,37.46083149800006],[27.748681500000032,37.459423998000034],[27.618107500000065,37.51877049700005],[27.44771650000007,37.53993649800003],[27.38111250000003,37.445186997000064],[27.419004802000075,37.41190175500003],[27.202436500000033,37.350824499000055],[27.21665,37.59111199800003],[27.003069,37.65979599800005],[27.234331,37.72418399800006],[27.264569,37.87504849700008],[27.36633450000005,37.86620799800005],[27.556524500000023,37.99826899800007],[27.714490500000068,37.949471998000035],[28.185534500000074,38.00091449800004],[28.285376,38.110739498000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR322","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Denizli","NUTS_NAME":"Denizli"},"geometry":{"type":"Polygon","coordinates":[[[29.852119,37.75228549800005],[29.500381,37.61887199700004],[29.514886,37.460002],[29.606530500000076,37.39536099700007],[29.341447,37.00684899700008],[29.278776,36.87297399700003],[29.155344,36.88800249600007],[29.08627350000006,36.948807498000065],[29.028546,37.108749498000066],[28.743742500000053,37.24758449700005],[28.588908,37.45993949800004],[28.55216850000005,37.53501349700008],[28.793877,37.652881498000056],[28.83511,37.73129999900004],[28.741235500000073,37.78772349900004],[28.70325150000008,37.881838498000036],[28.793492,37.98045349800003],[28.616089,38.089585998000075],[28.847341500000027,38.240428500000064],[29.23033350000003,38.23764349900006],[29.379754,38.30714599800007],[29.580195,38.246578498000076],[29.61755850000003,38.403098998000075],[29.75570350000004,38.48430749600004],[29.946554,38.39677449900006],[30.02725350000003,38.214679499000056],[29.65392250000008,37.96017549800007],[29.663986,37.864153997000074],[29.852119,37.75228549800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR323","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Mugla","NUTS_NAME":"Mugla"},"geometry":{"type":"Polygon","coordinates":[[[27.618107500000065,37.51877049700005],[27.748681500000032,37.459423998000034],[28.088255,37.46083149800006],[28.254244500000027,37.53932899700004],[28.588908,37.45993949800004],[28.743742500000053,37.24758449700005],[29.028546,37.108749498000066],[29.08627350000006,36.948807498000065],[29.155344,36.88800249600007],[29.278776,36.87297399700003],[29.341447,37.00684899700008],[29.422852500000033,36.956733996000025],[29.386635500000068,36.85942699800006],[29.60908950000004,36.98030049800008],[29.713264499000047,36.95630949800005],[29.751884,36.900562998000055],[29.637207,36.669686],[29.317792,36.44839999900006],[29.26056150000005,36.30439899700008],[29.102026,36.386720497000056],[29.125913500000024,36.542272499000035],[29.02238650000004,36.58621399800006],[29.115694,36.62598249700005],[29.103811,36.67034349700003],[28.928788999000062,36.754858],[28.851038,36.66208449800007],[28.878309,36.62911799800003],[28.846163,36.58836949600004],[28.790045,36.685636500000044],[28.618429,36.70327949800003],[28.630129,36.781706],[28.558664500000077,36.831861497000034],[28.53590599900008,36.78577999600003],[28.270406500000036,36.852479997000046],[28.233087500000067,36.80461299800004],[28.279173,36.734334998000065],[28.163698,36.68377899800004],[28.089052500000037,36.582017998000026],[27.97934550000008,36.553571497000064],[27.956487500000037,36.60048499800007],[28.095728,36.64494499700004],[27.963636500000064,36.683786498000075],[28.10157950000007,36.705804999000065],[28.131860500000073,36.79396650000007],[27.730821500000047,36.763143497000044],[27.67334150000005,36.67207149700005],[27.478128500000025,36.64709299800006],[27.374422,36.68548399900004],[27.428606,36.748281500000076],[27.642294,36.80760399800005],[28.032835,36.78775999800007],[28.059328,36.878801500000066],[28.014459500000044,36.90612599800005],[28.159700500000042,36.925954998000066],[28.330560499000057,37.03339549700007],[27.565542,36.97728549800007],[27.42487150000005,37.03574199700006],[27.264269,36.964120997000066],[27.225256,37.05810349700005],[27.32292,37.15848349700008],[27.470903500000077,37.08077799800003],[27.546667,37.12506699900007],[27.526208999000062,37.193677996000076],[27.611025,37.25751299900003],[27.411882500000047,37.30456749900003],[27.404363,37.36809499900005],[27.424582461000057,37.40700347200004],[27.419004802000075,37.41190175500003],[27.38111250000003,37.445186997000064],[27.44771650000007,37.53993649800003],[27.618107500000065,37.51877049700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR33","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Manisa, Afyonkarahisar, Kütahya, Usak","NUTS_NAME":"Manisa, Afyonkarahisar, Kütahya, Usak"},"geometry":{"type":"Polygon","coordinates":[[[29.760535,39.71012349700004],[30.06013450000006,39.658640999000056],[30.193454,39.65880449900004],[30.230593500000055,39.57547949800005],[30.355477,39.49900499800003],[30.36653,39.40263399600008],[30.315111,39.36209499800003],[30.42949150000004,39.21761249800005],[30.74145550000003,39.126772998000035],[31.192543,39.27911049700003],[31.303426,39.26033799800007],[31.389171,39.14805799700008],[31.619732500000055,39.10305549800006],[31.616613,39.00463699800008],[31.721062500000073,38.94946949700005],[31.62634950000006,38.92188599800005],[31.635725500000035,38.82079299700007],[31.52451650000006,38.69974099800004],[31.621331,38.619947497000055],[31.23334950000003,38.40997299700007],[31.13277,38.48890349800007],[30.986741,38.46641449800006],[30.70472249900007,38.265575],[30.42972050000003,38.15599899800003],[30.327528,38.03099949800003],[30.174609,37.95408949800003],[30.039979500000072,37.752413998000065],[29.852119,37.75228549800005],[29.663986,37.864153997000074],[29.65392250000008,37.96017549800007],[30.02725350000003,38.214679499000056],[29.946554,38.39677449900006],[29.75570350000004,38.48430749600004],[29.61755850000003,38.403098998000075],[29.580195,38.246578498000076],[29.379754,38.30714599800007],[29.23033350000003,38.23764349900006],[28.847341500000027,38.240428500000064],[28.616089,38.089585998000075],[28.44260950000006,38.10466149800004],[28.452412500000037,38.22420999700006],[28.28641950000008,38.31971599800005],[28.137481999000045,38.31729899700008],[28.06295750000004,38.39836149700005],[27.889356,38.411711499000035],[27.816327,38.29641849600006],[27.680527,38.42624499800007],[27.260101500000076,38.54846750000007],[27.10131349900007,38.77824549700006],[27.144471,38.88617799900004],[27.42629050000005,38.94496649800004],[27.48838150000006,39.06407850000005],[27.452409999000054,39.25615049700008],[27.381316,39.35274650000008],[27.462255500000026,39.39673249800006],[27.800632,39.31310799900007],[27.900652,39.348802997000064],[27.969201,39.195721],[28.15897,39.05698399800008],[28.652546500000028,39.14736849800005],[28.597988,39.280635999000026],[28.885427,39.372048997000036],[28.968574,39.60067749800004],[29.194093500000065,39.600573997000026],[29.28185,39.73377549800006],[29.405343,39.809691996000026],[29.396933,39.901012499000046],[29.742183500000067,39.87133399900006],[29.760535,39.71012349700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR331","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Manisa","NUTS_NAME":"Manisa"},"geometry":{"type":"Polygon","coordinates":[[[28.652546500000028,39.14736849800005],[28.69472550000006,39.07236849800006],[28.88383650000003,39.047397998000065],[28.900279500000067,38.95834199800004],[28.97054650000007,38.92482799800007],[28.957545500000037,38.87183799700006],[29.043041500000072,38.789480997000055],[28.993915,38.70958899900006],[28.79814650000003,38.618915497000046],[28.74994950000007,38.36621599800003],[28.847341500000027,38.240428500000064],[28.616089,38.089585998000075],[28.44260950000006,38.10466149800004],[28.452412500000037,38.22420999700006],[28.28641950000008,38.31971599800005],[28.137481999000045,38.31729899700008],[28.06295750000004,38.39836149700005],[27.889356,38.411711499000035],[27.816327,38.29641849600006],[27.680527,38.42624499800007],[27.260101500000076,38.54846750000007],[27.10131349900007,38.77824549700006],[27.144471,38.88617799900004],[27.42629050000005,38.94496649800004],[27.48838150000006,39.06407850000005],[27.452409999000054,39.25615049700008],[27.381316,39.35274650000008],[27.462255500000026,39.39673249800006],[27.800632,39.31310799900007],[27.900652,39.348802997000064],[27.969201,39.195721],[28.15897,39.05698399800008],[28.652546500000028,39.14736849800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR332","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Afyon","NUTS_NAME":"Afyon"},"geometry":{"type":"Polygon","coordinates":[[[31.619732500000055,39.10305549800006],[31.616613,39.00463699800008],[31.721062500000073,38.94946949700005],[31.62634950000006,38.92188599800005],[31.635725500000035,38.82079299700007],[31.52451650000006,38.69974099800004],[31.621331,38.619947497000055],[31.23334950000003,38.40997299700007],[31.13277,38.48890349800007],[30.986741,38.46641449800006],[30.70472249900007,38.265575],[30.42972050000003,38.15599899800003],[30.327528,38.03099949800003],[30.174609,37.95408949800003],[30.039979500000072,37.752413998000065],[29.852119,37.75228549800005],[29.663986,37.864153997000074],[29.65392250000008,37.96017549800007],[30.02725350000003,38.214679499000056],[29.946554,38.39677449900006],[29.75570350000004,38.48430749600004],[29.868075,38.58099449800005],[29.858238500000027,38.669573498000034],[29.969909,38.750473498000076],[30.191478500000073,38.92936999700004],[30.254659,39.10732249600005],[30.42949150000004,39.21761249800005],[30.74145550000003,39.126772998000035],[31.192543,39.27911049700003],[31.303426,39.26033799800007],[31.389171,39.14805799700008],[31.619732500000055,39.10305549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR333","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kütahya","NUTS_NAME":"Kütahya"},"geometry":{"type":"Polygon","coordinates":[[[29.742183500000067,39.87133399900006],[29.760535,39.71012349700004],[30.06013450000006,39.658640999000056],[30.193454,39.65880449900004],[30.230593500000055,39.57547949800005],[30.355477,39.49900499800003],[30.36653,39.40263399600008],[30.315111,39.36209499800003],[30.42949150000004,39.21761249800005],[30.254659,39.10732249600005],[30.191478500000073,38.92936999700004],[29.969909,38.750473498000076],[29.929225,38.84005749800008],[29.854158,38.84979999800004],[29.910295500000075,38.90821599900005],[29.616070500000035,38.91890099800003],[29.222732999000073,38.77487149700005],[29.043041500000072,38.789480997000055],[28.957545500000037,38.87183799700006],[28.97054650000007,38.92482799800007],[28.900279500000067,38.95834199800004],[28.88383650000003,39.047397998000065],[28.69472550000006,39.07236849800006],[28.652546500000028,39.14736849800005],[28.597988,39.280635999000026],[28.885427,39.372048997000036],[28.968574,39.60067749800004],[29.194093500000065,39.600573997000026],[29.28185,39.73377549800006],[29.405343,39.809691996000026],[29.396933,39.901012499000046],[29.742183500000067,39.87133399900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR334","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Usak","NUTS_NAME":"Usak"},"geometry":{"type":"Polygon","coordinates":[[[29.969909,38.750473498000076],[29.858238500000027,38.669573498000034],[29.868075,38.58099449800005],[29.75570350000004,38.48430749600004],[29.61755850000003,38.403098998000075],[29.580195,38.246578498000076],[29.379754,38.30714599800007],[29.23033350000003,38.23764349900006],[28.847341500000027,38.240428500000064],[28.74994950000007,38.36621599800003],[28.79814650000003,38.618915497000046],[28.993915,38.70958899900006],[29.043041500000072,38.789480997000055],[29.222732999000073,38.77487149700005],[29.616070500000035,38.91890099800003],[29.910295500000075,38.90821599900005],[29.854158,38.84979999800004],[29.929225,38.84005749800008],[29.969909,38.750473498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR4","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"DOGU MARMARA","NUTS_NAME":"DOGU MARMARA"},"geometry":{"type":"Polygon","coordinates":[[[30.35468650000007,41.18419449800007],[30.982055500000058,41.07260250000007],[31.295656500000064,41.11631999900004],[31.424844,41.02525499600006],[31.75363,41.005629998000074],[32.134024,41.030703999000025],[32.274522,41.024024498000074],[32.325888500000076,40.88752399800006],[32.560043,40.807477498000026],[32.554816,40.69194399700007],[32.37547550000005,40.663500999000064],[32.396747,40.561117996000064],[31.911622,40.32874049700007],[31.590715500000044,40.27586449800003],[31.437988,40.34499449700007],[31.297481,40.31101649800007],[31.128547500000025,40.36527999800006],[31.065705,40.225254997000036],[30.82533,40.12578449800003],[30.921542,40.02736699600007],[31.676844,40.03524050000004],[31.78699050000006,39.88126549900005],[31.880758,39.87372349800006],[31.850136,39.84329899700003],[31.915188,39.82739399800005],[31.822506499000042,39.74352399800006],[32.030849,39.481336998000074],[31.994242,39.35153349800004],[32.069824,39.27607099700003],[31.81531750000005,39.128693498000075],[31.750522,39.07900349800008],[31.619732500000055,39.10305549800006],[31.389171,39.14805799700008],[31.303426,39.26033799800007],[31.192543,39.27911049700003],[30.74145550000003,39.126772998000035],[30.42949150000004,39.21761249800005],[30.315111,39.36209499800003],[30.36653,39.40263399600008],[30.355477,39.49900499800003],[30.230593500000055,39.57547949800005],[30.193454,39.65880449900004],[30.06013450000006,39.658640999000056],[29.760535,39.71012349700004],[29.742183500000067,39.87133399900006],[29.396933,39.901012499000046],[29.405343,39.809691996000026],[29.28185,39.73377549800006],[29.194093500000065,39.600573997000026],[28.968574,39.60067749800004],[28.793313500000068,39.61746949700006],[28.69535250000007,39.757907498000066],[28.515012,39.73130899700004],[28.265037500000062,39.87023399700007],[28.06755050000004,40.25313399800007],[28.164779,40.395612999000036],[28.980173,40.355935997000074],[29.154478,40.42537499800005],[28.991232500000024,40.46627349700003],[28.780161,40.53619199700006],[28.996286500000053,40.645887498000036],[29.517332,40.72897149700003],[29.546993,40.69543599900004],[29.942186500000048,40.750489998000035],[29.353948500000058,40.75541099800006],[29.342517500000042,40.807657498000026],[29.407457,40.95459],[29.849114,41.064824497000075],[29.865624500000024,41.14344749700007],[30.35468650000007,41.18419449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR61","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Antalya, Isparta, Burdur","NUTS_NAME":"Antalya, Isparta, Burdur"},"geometry":{"type":"Polygon","coordinates":[[[31.300975,37.404258997000056],[31.453391500000066,37.33383299800005],[31.740267500000073,37.35582599800006],[32.005236500000024,37.064884998000025],[32.298969,36.91551849700005],[32.457231500000034,36.738470998000025],[32.51975250000004,36.54548949900004],[32.50581550000004,36.39687199700006],[32.573227,36.357623999000054],[32.612618,36.25166149900008],[32.57634,36.093232998000076],[32.295229,36.234987498000066],[32.029032,36.53820249800003],[31.334676500000057,36.80768799800006],[30.698167500000068,36.88451949700004],[30.57494150000008,36.794511998000075],[30.556848500000058,36.619237998000074],[30.59149,36.58946799700004],[30.481527500000027,36.433797998000045],[30.48763450000007,36.27913099700004],[30.405119,36.20929149700004],[30.32793250000003,36.29695299800005],[30.188627,36.31390950000008],[30.115274500000055,36.24686649800003],[29.922674,36.23392699600004],[29.683466,36.130262498000036],[29.644518,36.19632899900006],[29.408991,36.261224997000056],[29.351629500000058,36.230226499000025],[29.26056150000005,36.30439899700008],[29.317792,36.44839999900006],[29.637207,36.669686],[29.751884,36.900562998000055],[29.713264499000047,36.95630949800005],[29.60908950000004,36.98030049800008],[29.386635500000068,36.85942699800006],[29.422852500000033,36.956733996000025],[29.341447,37.00684899700008],[29.606530500000076,37.39536099700007],[29.514886,37.460002],[29.500381,37.61887199700004],[29.852119,37.75228549800005],[30.039979500000072,37.752413998000065],[30.174609,37.95408949800003],[30.327528,38.03099949800003],[30.42972050000003,38.15599899800003],[30.70472249900007,38.265575],[30.986741,38.46641449800006],[31.13277,38.48890349800007],[31.23334950000003,38.40997299700007],[31.298652,38.284827998000026],[31.59753050000006,38.05608549900006],[31.419393,37.97350799600008],[31.448459,37.729105497000035],[31.321775,37.628682997000055],[31.300975,37.404258997000056]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR611","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Antalya","NUTS_NAME":"Antalya"},"geometry":{"type":"Polygon","coordinates":[[[31.453391500000066,37.33383299800005],[31.740267500000073,37.35582599800006],[32.005236500000024,37.064884998000025],[32.298969,36.91551849700005],[32.457231500000034,36.738470998000025],[32.51975250000004,36.54548949900004],[32.50581550000004,36.39687199700006],[32.573227,36.357623999000054],[32.612618,36.25166149900008],[32.57634,36.093232998000076],[32.295229,36.234987498000066],[32.029032,36.53820249800003],[31.334676500000057,36.80768799800006],[30.698167500000068,36.88451949700004],[30.57494150000008,36.794511998000075],[30.556848500000058,36.619237998000074],[30.59149,36.58946799700004],[30.481527500000027,36.433797998000045],[30.48763450000007,36.27913099700004],[30.405119,36.20929149700004],[30.32793250000003,36.29695299800005],[30.188627,36.31390950000008],[30.115274500000055,36.24686649800003],[29.922674,36.23392699600004],[29.683466,36.130262498000036],[29.644518,36.19632899900006],[29.408991,36.261224997000056],[29.351629500000058,36.230226499000025],[29.26056150000005,36.30439899700008],[29.317792,36.44839999900006],[29.637207,36.669686],[29.751884,36.900562998000055],[29.713264499000047,36.95630949800005],[29.814961500000038,37.010068998000065],[29.829011499000046,37.14257199800005],[30.161246,37.30115999800006],[30.78288249900004,37.230462498000065],[30.901673,37.355087998000045],[30.96199450000006,37.314773],[31.300975,37.404258997000056],[31.453391500000066,37.33383299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR612","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Isparta","NUTS_NAME":"Isparta"},"geometry":{"type":"Polygon","coordinates":[[[31.23334950000003,38.40997299700007],[31.298652,38.284827998000026],[31.59753050000006,38.05608549900006],[31.419393,37.97350799600008],[31.448459,37.729105497000035],[31.321775,37.628682997000055],[31.300975,37.404258997000056],[30.96199450000006,37.314773],[30.901673,37.355087998000045],[30.738611,37.54212599900006],[30.720983500000045,37.62062099800005],[30.480434,37.69576999700007],[30.352391500000067,37.83771249900008],[30.120921500000065,37.78033099900006],[30.072524500000043,37.723858997000036],[30.039979500000072,37.752413998000065],[30.174609,37.95408949800003],[30.327528,38.03099949800003],[30.42972050000003,38.15599899800003],[30.70472249900007,38.265575],[30.986741,38.46641449800006],[31.13277,38.48890349800007],[31.23334950000003,38.40997299700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR613","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Burdur","NUTS_NAME":"Burdur"},"geometry":{"type":"Polygon","coordinates":[[[30.901673,37.355087998000045],[30.78288249900004,37.230462498000065],[30.161246,37.30115999800006],[29.829011499000046,37.14257199800005],[29.814961500000038,37.010068998000065],[29.713264499000047,36.95630949800005],[29.60908950000004,36.98030049800008],[29.386635500000068,36.85942699800006],[29.422852500000033,36.956733996000025],[29.341447,37.00684899700008],[29.606530500000076,37.39536099700007],[29.514886,37.460002],[29.500381,37.61887199700004],[29.852119,37.75228549800005],[30.039979500000072,37.752413998000065],[30.072524500000043,37.723858997000036],[30.120921500000065,37.78033099900006],[30.352391500000067,37.83771249900008],[30.480434,37.69576999700007],[30.720983500000045,37.62062099800005],[30.738611,37.54212599900006],[30.901673,37.355087998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR62","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Adana, Mersin","NUTS_NAME":"Adana, Mersin"},"geometry":{"type":"Polygon","coordinates":[[[36.35362850000007,37.86235099800007],[36.303865,37.734576498000024],[36.04048350000005,37.66030499800007],[35.88494249900003,37.34412649800004],[35.856385,37.16971299700003],[36.045457500000055,37.18898299800003],[36.05271,37.018507998000075],[35.963096,36.902986998000074],[35.792162,36.76891499900006],[35.56386,36.71301049700003],[35.723783500000025,36.71192749800008],[35.56584,36.56479449900007],[35.420896500000026,36.59061599900008],[35.339365,36.53852649800007],[34.899245,36.73828099800005],[34.74605750000006,36.81102949900003],[34.56072050000006,36.76816349600006],[34.085783,36.41823749800005],[34.04285250000004,36.29496949800006],[33.94976999900007,36.26343749600005],[33.866678,36.31495849800007],[33.68421,36.13340199800007],[33.64763050000005,36.19356349700007],[33.56112850000005,36.125440499000035],[33.14687549900003,36.13780399800004],[33.08139250000005,36.08072449800005],[32.950445,36.103795998000066],[32.805395,36.02311499800004],[32.57634,36.093232998000076],[32.612618,36.25166149900008],[32.573227,36.357623999000054],[33.023219,36.458324997000034],[33.10775699900006,36.551714498000024],[33.23895550000003,36.520920998000065],[33.008288,36.67464049700004],[32.95091050000008,36.83134499800008],[33.107767,36.851857497000026],[33.15150550000004,36.812453498000025],[33.22189049900004,36.904015998000034],[33.659066,37.015296497000065],[33.80370450000004,36.98732599700003],[33.997476,37.09766399700004],[34.409057500000074,37.23256999700004],[34.428914500000076,37.316384998000046],[34.748,37.40930399800004],[34.87491350000005,37.492744],[34.88540450000005,37.67054299800003],[35.218708,37.75922449800004],[35.574764,37.73715849800004],[35.575068,37.911404497000035],[35.61442150000005,37.96211099800007],[35.925933,38.084282],[36.10676950000004,38.31260099700006],[36.253814,38.37942299800005],[36.36776750000007,38.32651049800006],[36.440496,38.21943249800006],[36.37051549900008,38.12090849700007],[36.31578650000006,37.91049349900004],[36.35362850000007,37.86235099800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR621","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Adana","NUTS_NAME":"Adana"},"geometry":{"type":"Polygon","coordinates":[[[36.440496,38.21943249800006],[36.37051549900008,38.12090849700007],[36.31578650000006,37.91049349900004],[36.35362850000007,37.86235099800007],[36.303865,37.734576498000024],[36.04048350000005,37.66030499800007],[35.88494249900003,37.34412649800004],[35.856385,37.16971299700003],[36.045457500000055,37.18898299800003],[36.05271,37.018507998000075],[35.963096,36.902986998000074],[35.792162,36.76891499900006],[35.56386,36.71301049700003],[35.723783500000025,36.71192749800008],[35.56584,36.56479449900007],[35.420896500000026,36.59061599900008],[35.339365,36.53852649800007],[34.899245,36.73828099800005],[35.153671,36.807659],[35.083552,36.93684349700004],[35.120977,37.029791498000066],[34.949234500000046,37.056808998000065],[34.929497,37.17205299800003],[34.97267,37.26057749700004],[34.789160999000046,37.315841996000074],[34.748,37.40930399800004],[34.87491350000005,37.492744],[34.88540450000005,37.67054299800003],[35.218708,37.75922449800004],[35.574764,37.73715849800004],[35.575068,37.911404497000035],[35.61442150000005,37.96211099800007],[35.925933,38.084282],[36.10676950000004,38.31260099700006],[36.253814,38.37942299800005],[36.36776750000007,38.32651049800006],[36.440496,38.21943249800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR622","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Içel","NUTS_NAME":"Içel"},"geometry":{"type":"Polygon","coordinates":[[[34.899245,36.73828099800005],[34.74605750000006,36.81102949900003],[34.56072050000006,36.76816349600006],[34.085783,36.41823749800005],[34.04285250000004,36.29496949800006],[33.94976999900007,36.26343749600005],[33.866678,36.31495849800007],[33.68421,36.13340199800007],[33.64763050000005,36.19356349700007],[33.56112850000005,36.125440499000035],[33.14687549900003,36.13780399800004],[33.08139250000005,36.08072449800005],[32.950445,36.103795998000066],[32.805395,36.02311499800004],[32.57634,36.093232998000076],[32.612618,36.25166149900008],[32.573227,36.357623999000054],[33.023219,36.458324997000034],[33.10775699900006,36.551714498000024],[33.23895550000003,36.520920998000065],[33.008288,36.67464049700004],[32.95091050000008,36.83134499800008],[33.107767,36.851857497000026],[33.15150550000004,36.812453498000025],[33.22189049900004,36.904015998000034],[33.659066,37.015296497000065],[33.80370450000004,36.98732599700003],[33.997476,37.09766399700004],[34.409057500000074,37.23256999700004],[34.428914500000076,37.316384998000046],[34.748,37.40930399800004],[34.789160999000046,37.315841996000074],[34.97267,37.26057749700004],[34.929497,37.17205299800003],[34.949234500000046,37.056808998000065],[35.120977,37.029791498000066],[35.083552,36.93684349700004],[35.153671,36.807659],[34.899245,36.73828099800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR63","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Hatay, Kahramanmaras, Osmaniye","NUTS_NAME":"Hatay, Kahramanmaras, Osmaniye"},"geometry":{"type":"Polygon","coordinates":[[[37.379789,37.35999050000004],[37.08278,37.17567599900008],[36.901796,37.33366049800003],[36.697388500000045,37.21814249600004],[36.52120250000007,37.08523999800008],[36.465673,36.94866749800008],[36.66001039400004,36.83323153500004],[36.549576,36.48776999700004],[36.690133,36.28624799900007],[36.683648,36.23678349800008],[36.39222,36.21332599900006],[36.37497,35.99791399800006],[36.223607,35.962219997000034],[36.168469,35.819717997000055],[35.917953,35.928695498000025],[35.978777,36.01944149800005],[35.779623,36.31855599700003],[36.037126500000056,36.53373499700007],[36.188917,36.593206500000065],[36.21573050000006,36.65963549800006],[36.205831500000045,36.78273199600005],[36.14857650000005,36.85506649800004],[36.018686,36.92843449800006],[35.963096,36.902986998000074],[36.05271,37.018507998000075],[36.045457500000055,37.18898299800003],[35.856385,37.16971299700003],[35.88494249900003,37.34412649800004],[36.04048350000005,37.66030499800007],[36.303865,37.734576498000024],[36.35362850000007,37.86235099800007],[36.31578650000006,37.91049349900004],[36.37051549900008,38.12090849700007],[36.440496,38.21943249800006],[36.59920550000004,38.39936399900006],[36.723043500000074,38.427591998000025],[36.69959350000005,38.503656496000076],[36.76984550000003,38.54643599800005],[37.341487,38.593515498000045],[37.25953450000003,38.479910999000026],[37.64785550000005,38.34777349700005],[37.76024650000005,38.20305099700005],[37.74833550000005,38.14038149900006],[37.614140500000076,38.03833199700006],[37.638667500000054,37.93116699900003],[37.61636,37.85288349900003],[37.502716,37.792868498000075],[37.432401,37.64170899900006],[37.56929149900003,37.58780599700003],[37.62321650000007,37.51165249600007],[37.55469750000003,37.382446999000024],[37.379789,37.35999050000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR631","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Hatay","NUTS_NAME":"Hatay"},"geometry":{"type":"Polygon","coordinates":[[[36.465673,36.94866749800008],[36.66001039400004,36.83323153500004],[36.549576,36.48776999700004],[36.690133,36.28624799900007],[36.683648,36.23678349800008],[36.39222,36.21332599900006],[36.37497,35.99791399800006],[36.223607,35.962219997000034],[36.168469,35.819717997000055],[35.917953,35.928695498000025],[35.978777,36.01944149800005],[35.779623,36.31855599700003],[36.037126500000056,36.53373499700007],[36.188917,36.593206500000065],[36.21573050000006,36.65963549800006],[36.205831500000045,36.78273199600005],[36.14857650000005,36.85506649800004],[36.018686,36.92843449800006],[35.963096,36.902986998000074],[36.05271,37.018507998000075],[36.465673,36.94866749800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR632","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kahramanmaras","NUTS_NAME":"Kahramanmaras"},"geometry":{"type":"Polygon","coordinates":[[[37.638667500000054,37.93116699900003],[37.61636,37.85288349900003],[37.502716,37.792868498000075],[37.432401,37.64170899900006],[37.56929149900003,37.58780599700003],[37.62321650000007,37.51165249600007],[37.55469750000003,37.382446999000024],[37.379789,37.35999050000004],[37.08278,37.17567599900008],[36.901796,37.33366049800003],[36.697388500000045,37.21814249600004],[36.582839500000034,37.36956549800004],[36.25541350000003,37.35487949800006],[36.27497250000005,37.603245997000045],[36.200632,37.64916099800007],[36.303865,37.734576498000024],[36.35362850000007,37.86235099800007],[36.31578650000006,37.91049349900004],[36.37051549900008,38.12090849700007],[36.440496,38.21943249800006],[36.59920550000004,38.39936399900006],[36.723043500000074,38.427591998000025],[36.69959350000005,38.503656496000076],[36.76984550000003,38.54643599800005],[37.341487,38.593515498000045],[37.25953450000003,38.479910999000026],[37.64785550000005,38.34777349700005],[37.76024650000005,38.20305099700005],[37.74833550000005,38.14038149900006],[37.614140500000076,38.03833199700006],[37.638667500000054,37.93116699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR633","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Osmaniye","NUTS_NAME":"Osmaniye"},"geometry":{"type":"Polygon","coordinates":[[[36.697388500000045,37.21814249600004],[36.52120250000007,37.08523999800008],[36.465673,36.94866749800008],[36.05271,37.018507998000075],[36.045457500000055,37.18898299800003],[35.856385,37.16971299700003],[35.88494249900003,37.34412649800004],[36.04048350000005,37.66030499800007],[36.303865,37.734576498000024],[36.200632,37.64916099800007],[36.27497250000005,37.603245997000045],[36.25541350000003,37.35487949800006],[36.582839500000034,37.36956549800004],[36.697388500000045,37.21814249600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR7","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"ORTA ANADOLU","NUTS_NAME":"ORTA ANADOLU"},"geometry":{"type":"Polygon","coordinates":[[[38.148452500000076,40.52421849700005],[38.22174250000006,40.34620349800008],[38.213064,40.21331249800005],[38.78355,40.083271498000045],[38.745546999000055,39.98660549700003],[38.436214,39.993923497000026],[38.366779500000064,39.93940199700006],[38.477686,39.85301899800004],[38.326654,39.65799949800004],[38.36187150000006,39.604872998000076],[38.318415,39.55150599800004],[38.335844500000064,39.494996498000035],[38.422460999000066,39.427839497000036],[38.35196150000007,39.35526299800006],[38.349975,39.14994349600005],[37.825541,39.09395049800003],[37.750074,38.978260498000054],[37.51233950000005,39.031703],[37.507155,38.890611499000045],[37.574179,38.78082549800007],[37.42422350000004,38.70474199800003],[37.341487,38.593515498000045],[36.76984550000003,38.54643599800005],[36.69959350000005,38.503656496000076],[36.723043500000074,38.427591998000025],[36.59920550000004,38.39936399900006],[36.440496,38.21943249800006],[36.36776750000007,38.32651049800006],[36.253814,38.37942299800005],[36.10676950000004,38.31260099700006],[35.925933,38.084282],[35.61442150000005,37.96211099800007],[35.575068,37.911404497000035],[35.574764,37.73715849800004],[35.218708,37.75922449800004],[34.88540450000005,37.67054299800003],[34.87491350000005,37.492744],[34.748,37.40930399800004],[34.428914500000076,37.316384998000046],[34.33503450000006,37.47940749600008],[34.328714,37.586077999000054],[34.40377649900006,37.64982299700006],[34.40042650000004,37.751674998000055],[34.10607150000004,37.996095498000045],[34.033573500000045,38.00481099700005],[33.394453500000054,37.97126099700006],[33.203369500000065,38.278494],[33.25558650000005,38.42087150000003],[33.464405,38.63659149800003],[33.70547450000004,38.680822499000044],[33.795347,38.83806799700005],[33.722786,38.94294799700003],[33.886019,39.043412498000066],[33.541194,39.215465497000025],[33.441716,39.351449],[33.39039450000007,39.41628549700005],[33.39862950000003,39.51610499800006],[33.24737250000004,39.64108699700006],[33.364065,39.972091997000064],[33.457947500000046,39.96101749700006],[33.603764,40.044869996000045],[33.557854500000076,40.16114249700007],[33.695422,40.33224149800003],[33.91622550000005,40.26303799900006],[34.044084,40.18185199700008],[34.093781500000034,40.02593349800003],[34.175507,39.94043399900005],[34.384643,40.00414999700007],[34.57078750000005,39.960256998000034],[34.86692750000003,40.036017498000035],[34.93486450000006,39.98463749800004],[35.072878,40.01205649800005],[35.135924,40.036527498000055],[35.131494,40.11003999800005],[35.17860250000007,40.143898498000055],[35.15563950000006,40.226112999000065],[35.35115,40.24338049900007],[35.442588,40.23402249800006],[35.596853500000066,40.09647649800007],[35.903141,40.10542649800004],[36.003423,39.95451049700006],[36.60641950000007,39.980525998000076],[36.693430500000034,40.183547],[36.80592750000005,40.22843699800006],[37.09173550000003,40.249413],[37.450466,40.17053549800005],[37.468184,40.254590998000026],[37.605382500000076,40.33798799900006],[37.584064,40.424386998000045],[37.732457500000066,40.34250949900007],[37.75732850000003,40.405527996000046],[37.928967,40.38215749600005],[37.98682850000006,40.49290799900007],[38.148452500000076,40.52421849700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR71","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Kirikkale, Aksaray, Nigde, Nevsehir, Kirsehir","NUTS_NAME":"Kirikkale, Aksaray, Nigde, Nevsehir, Kirsehir"},"geometry":{"type":"Polygon","coordinates":[[[34.093781500000034,40.02593349800003],[34.175507,39.94043399900005],[34.03558950000007,39.901025],[34.108037,39.79300249700003],[34.414304499000025,39.68603749700003],[34.530465,39.56261999700007],[34.48273450000005,39.50843549800004],[34.64418150000006,39.41150399800006],[34.91474150000005,39.28363249700004],[34.90792250000004,39.18828849800008],[35.032442,39.081664999000054],[35.034188,39.00488099700004],[34.96803350000005,38.788846998000054],[35.03261450000008,38.72953549700003],[35.083858,38.57053549700004],[34.88845750000007,38.42024999700004],[34.885322,38.34468849700005],[34.972307,38.32934249700003],[35.04687450000006,38.176261999000076],[35.26566850000006,38.10601999700003],[35.28990150000004,37.885253499000044],[35.218708,37.75922449800004],[34.88540450000005,37.67054299800003],[34.87491350000005,37.492744],[34.748,37.40930399800004],[34.428914500000076,37.316384998000046],[34.33503450000006,37.47940749600008],[34.328714,37.586077999000054],[34.40377649900006,37.64982299700006],[34.40042650000004,37.751674998000055],[34.10607150000004,37.996095498000045],[34.033573500000045,38.00481099700005],[33.394453500000054,37.97126099700006],[33.203369500000065,38.278494],[33.25558650000005,38.42087150000003],[33.464405,38.63659149800003],[33.70547450000004,38.680822499000044],[33.795347,38.83806799700005],[33.722786,38.94294799700003],[33.886019,39.043412498000066],[33.541194,39.215465497000025],[33.441716,39.351449],[33.39039450000007,39.41628549700005],[33.39862950000003,39.51610499800006],[33.24737250000004,39.64108699700006],[33.364065,39.972091997000064],[33.457947500000046,39.96101749700006],[33.603764,40.044869996000045],[33.557854500000076,40.16114249700007],[33.695422,40.33224149800003],[33.91622550000005,40.26303799900006],[34.044084,40.18185199700008],[34.093781500000034,40.02593349800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR711","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kirikkale","NUTS_NAME":"Kirikkale"},"geometry":{"type":"Polygon","coordinates":[[[33.91622550000005,40.26303799900006],[34.044084,40.18185199700008],[34.093781500000034,40.02593349800003],[34.175507,39.94043399900005],[34.03558950000007,39.901025],[34.108037,39.79300249700003],[33.94097849900004,39.71776349900006],[33.885559,39.60151499700004],[33.74106099900007,39.48557249800007],[33.648039,39.486789499000054],[33.441716,39.351449],[33.39039450000007,39.41628549700005],[33.39862950000003,39.51610499800006],[33.24737250000004,39.64108699700006],[33.364065,39.972091997000064],[33.457947500000046,39.96101749700006],[33.603764,40.044869996000045],[33.557854500000076,40.16114249700007],[33.695422,40.33224149800003],[33.91622550000005,40.26303799900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR712","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Aksaray","NUTS_NAME":"Aksaray"},"geometry":{"type":"Polygon","coordinates":[[[34.16938,38.81103749600004],[34.255555,38.771600498000055],[34.25689,38.629969998000036],[34.42397950000003,38.53243249800005],[34.383297500000026,38.414585497000076],[34.412443,38.24429799700005],[34.033573500000045,38.00481099700005],[33.394453500000054,37.97126099700006],[33.203369500000065,38.278494],[33.25558650000005,38.42087150000003],[33.464405,38.63659149800003],[33.70547450000004,38.680822499000044],[33.795347,38.83806799700005],[33.722786,38.94294799700003],[33.886019,39.043412498000066],[33.948095,39.044802498000024],[34.16938,38.81103749600004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR713","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Nigde","NUTS_NAME":"Nigde"},"geometry":{"type":"Polygon","coordinates":[[[34.885322,38.34468849700005],[34.972307,38.32934249700003],[35.04687450000006,38.176261999000076],[35.26566850000006,38.10601999700003],[35.28990150000004,37.885253499000044],[35.218708,37.75922449800004],[34.88540450000005,37.67054299800003],[34.87491350000005,37.492744],[34.748,37.40930399800004],[34.428914500000076,37.316384998000046],[34.33503450000006,37.47940749600008],[34.328714,37.586077999000054],[34.40377649900006,37.64982299700006],[34.40042650000004,37.751674998000055],[34.10607150000004,37.996095498000045],[34.033573500000045,38.00481099700005],[34.412443,38.24429799700005],[34.383297500000026,38.414585497000076],[34.566268500000035,38.32127749700004],[34.885322,38.34468849700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR714","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Nevsehir","NUTS_NAME":"Nevsehir"},"geometry":{"type":"Polygon","coordinates":[[[35.034188,39.00488099700004],[34.96803350000005,38.788846998000054],[35.03261450000008,38.72953549700003],[35.083858,38.57053549700004],[34.88845750000007,38.42024999700004],[34.885322,38.34468849700005],[34.566268500000035,38.32127749700004],[34.383297500000026,38.414585497000076],[34.42397950000003,38.53243249800005],[34.25689,38.629969998000036],[34.255555,38.771600498000055],[34.16938,38.81103749600004],[34.250381,38.89323599800008],[34.36884950000007,38.866085498000075],[34.52247250000005,38.99421749800007],[34.50798150000003,39.053476999000054],[34.58813549900003,39.04981749700005],[34.57288850000003,39.137433498000064],[34.719268500000055,39.28010799700007],[34.566031,39.28121099800006],[34.64418150000006,39.41150399800006],[34.91474150000005,39.28363249700004],[34.90792250000004,39.18828849800008],[35.032442,39.081664999000054],[35.034188,39.00488099700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR715","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kirsehir","NUTS_NAME":"Kirsehir"},"geometry":{"type":"Polygon","coordinates":[[[34.64418150000006,39.41150399800006],[34.566031,39.28121099800006],[34.719268500000055,39.28010799700007],[34.57288850000003,39.137433498000064],[34.58813549900003,39.04981749700005],[34.50798150000003,39.053476999000054],[34.52247250000005,38.99421749800007],[34.36884950000007,38.866085498000075],[34.250381,38.89323599800008],[34.16938,38.81103749600004],[33.948095,39.044802498000024],[33.886019,39.043412498000066],[33.541194,39.215465497000025],[33.441716,39.351449],[33.648039,39.486789499000054],[33.74106099900007,39.48557249800007],[33.885559,39.60151499700004],[33.94097849900004,39.71776349900006],[34.108037,39.79300249700003],[34.414304499000025,39.68603749700003],[34.530465,39.56261999700007],[34.48273450000005,39.50843549800004],[34.64418150000006,39.41150399800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR72","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Kayseri, Sivas, Yozgat","NUTS_NAME":"Kayseri, Sivas, Yozgat"},"geometry":{"type":"Polygon","coordinates":[[[38.148452500000076,40.52421849700005],[38.22174250000006,40.34620349800008],[38.213064,40.21331249800005],[38.78355,40.083271498000045],[38.745546999000055,39.98660549700003],[38.436214,39.993923497000026],[38.366779500000064,39.93940199700006],[38.477686,39.85301899800004],[38.326654,39.65799949800004],[38.36187150000006,39.604872998000076],[38.318415,39.55150599800004],[38.335844500000064,39.494996498000035],[38.422460999000066,39.427839497000036],[38.35196150000007,39.35526299800006],[38.349975,39.14994349600005],[37.825541,39.09395049800003],[37.750074,38.978260498000054],[37.51233950000005,39.031703],[37.507155,38.890611499000045],[37.574179,38.78082549800007],[37.42422350000004,38.70474199800003],[37.341487,38.593515498000045],[36.76984550000003,38.54643599800005],[36.69959350000005,38.503656496000076],[36.723043500000074,38.427591998000025],[36.59920550000004,38.39936399900006],[36.440496,38.21943249800006],[36.36776750000007,38.32651049800006],[36.253814,38.37942299800005],[36.10676950000004,38.31260099700006],[35.925933,38.084282],[35.61442150000005,37.96211099800007],[35.575068,37.911404497000035],[35.574764,37.73715849800004],[35.218708,37.75922449800004],[35.28990150000004,37.885253499000044],[35.26566850000006,38.10601999700003],[35.04687450000006,38.176261999000076],[34.972307,38.32934249700003],[34.885322,38.34468849700005],[34.88845750000007,38.42024999700004],[35.083858,38.57053549700004],[35.03261450000008,38.72953549700003],[34.96803350000005,38.788846998000054],[35.034188,39.00488099700004],[35.032442,39.081664999000054],[34.90792250000004,39.18828849800008],[34.91474150000005,39.28363249700004],[34.64418150000006,39.41150399800006],[34.48273450000005,39.50843549800004],[34.530465,39.56261999700007],[34.414304499000025,39.68603749700003],[34.108037,39.79300249700003],[34.03558950000007,39.901025],[34.175507,39.94043399900005],[34.384643,40.00414999700007],[34.57078750000005,39.960256998000034],[34.86692750000003,40.036017498000035],[34.93486450000006,39.98463749800004],[35.072878,40.01205649800005],[35.135924,40.036527498000055],[35.131494,40.11003999800005],[35.17860250000007,40.143898498000055],[35.15563950000006,40.226112999000065],[35.35115,40.24338049900007],[35.442588,40.23402249800006],[35.596853500000066,40.09647649800007],[35.903141,40.10542649800004],[36.003423,39.95451049700006],[36.60641950000007,39.980525998000076],[36.693430500000034,40.183547],[36.80592750000005,40.22843699800006],[37.09173550000003,40.249413],[37.450466,40.17053549800005],[37.468184,40.254590998000026],[37.605382500000076,40.33798799900006],[37.584064,40.424386998000045],[37.732457500000066,40.34250949900007],[37.75732850000003,40.405527996000046],[37.928967,40.38215749600005],[37.98682850000006,40.49290799900007],[38.148452500000076,40.52421849700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR721","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kayseri","NUTS_NAME":"Kayseri"},"geometry":{"type":"Polygon","coordinates":[[[36.76984550000003,38.54643599800005],[36.69959350000005,38.503656496000076],[36.723043500000074,38.427591998000025],[36.59920550000004,38.39936399900006],[36.440496,38.21943249800006],[36.36776750000007,38.32651049800006],[36.253814,38.37942299800005],[36.10676950000004,38.31260099700006],[35.925933,38.084282],[35.61442150000005,37.96211099800007],[35.575068,37.911404497000035],[35.574764,37.73715849800004],[35.218708,37.75922449800004],[35.28990150000004,37.885253499000044],[35.26566850000006,38.10601999700003],[35.04687450000006,38.176261999000076],[34.972307,38.32934249700003],[34.885322,38.34468849700005],[34.88845750000007,38.42024999700004],[35.083858,38.57053549700004],[35.03261450000008,38.72953549700003],[34.96803350000005,38.788846998000054],[35.034188,39.00488099700004],[35.31257050000005,38.97070850000006],[35.516476,39.08297649800005],[35.482601,39.164083498000025],[35.669683,39.19712799800004],[35.847428,39.30426099700003],[35.927069,39.220848499000056],[35.869762,39.17494099800007],[36.077994,39.10971949700007],[36.49336,39.097139500000026],[36.738537500000064,39.167993497000055],[36.814329,39.07699849800008],[36.958394,39.025157997000065],[36.95740050000006,38.95083249700008],[36.716504,38.71976649800007],[36.76984550000003,38.54643599800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR722","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Sivas","NUTS_NAME":"Sivas"},"geometry":{"type":"Polygon","coordinates":[[[38.78355,40.083271498000045],[38.745546999000055,39.98660549700003],[38.436214,39.993923497000026],[38.366779500000064,39.93940199700006],[38.477686,39.85301899800004],[38.326654,39.65799949800004],[38.36187150000006,39.604872998000076],[38.318415,39.55150599800004],[38.335844500000064,39.494996498000035],[38.422460999000066,39.427839497000036],[38.35196150000007,39.35526299800006],[38.349975,39.14994349600005],[37.825541,39.09395049800003],[37.750074,38.978260498000054],[37.51233950000005,39.031703],[37.507155,38.890611499000045],[37.574179,38.78082549800007],[37.42422350000004,38.70474199800003],[37.341487,38.593515498000045],[36.76984550000003,38.54643599800005],[36.716504,38.71976649800007],[36.95740050000006,38.95083249700008],[36.958394,39.025157997000065],[36.814329,39.07699849800008],[36.738537500000064,39.167993497000055],[36.49336,39.097139500000026],[36.077994,39.10971949700007],[35.869762,39.17494099800007],[35.927069,39.220848499000056],[35.847428,39.30426099700003],[36.028864500000054,39.40748599700004],[36.146201500000075,39.59388949600003],[36.12575099900005,39.73541349800007],[35.970516,39.90485749900006],[36.003423,39.95451049700006],[36.60641950000007,39.980525998000076],[36.693430500000034,40.183547],[36.80592750000005,40.22843699800006],[37.09173550000003,40.249413],[37.450466,40.17053549800005],[37.468184,40.254590998000026],[37.605382500000076,40.33798799900006],[37.584064,40.424386998000045],[37.732457500000066,40.34250949900007],[37.75732850000003,40.405527996000046],[37.928967,40.38215749600005],[37.98682850000006,40.49290799900007],[38.148452500000076,40.52421849700005],[38.22174250000006,40.34620349800008],[38.213064,40.21331249800005],[38.78355,40.083271498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR723","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Yozgat","NUTS_NAME":"Yozgat"},"geometry":{"type":"Polygon","coordinates":[[[35.442588,40.23402249800006],[35.596853500000066,40.09647649800007],[35.903141,40.10542649800004],[36.003423,39.95451049700006],[35.970516,39.90485749900006],[36.12575099900005,39.73541349800007],[36.146201500000075,39.59388949600003],[36.028864500000054,39.40748599700004],[35.847428,39.30426099700003],[35.669683,39.19712799800004],[35.482601,39.164083498000025],[35.516476,39.08297649800005],[35.31257050000005,38.97070850000006],[35.034188,39.00488099700004],[35.032442,39.081664999000054],[34.90792250000004,39.18828849800008],[34.91474150000005,39.28363249700004],[34.64418150000006,39.41150399800006],[34.48273450000005,39.50843549800004],[34.530465,39.56261999700007],[34.414304499000025,39.68603749700003],[34.108037,39.79300249700003],[34.03558950000007,39.901025],[34.175507,39.94043399900005],[34.384643,40.00414999700007],[34.57078750000005,39.960256998000034],[34.86692750000003,40.036017498000035],[34.93486450000006,39.98463749800004],[35.072878,40.01205649800005],[35.135924,40.036527498000055],[35.131494,40.11003999800005],[35.17860250000007,40.143898498000055],[35.15563950000006,40.226112999000065],[35.35115,40.24338049900007],[35.442588,40.23402249800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR8","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"BATI KARADENIZ","NUTS_NAME":"BATI KARADENIZ"},"geometry":{"type":"Polygon","coordinates":[[[34.943724,42.096982998000044],[35.08171250000004,42.02601799800004],[35.193533,42.03946899700003],[35.095869,41.965749997000046],[35.12668050000008,41.863176499000076],[35.276678,41.715734498000074],[35.51369950000003,41.63597999800004],[35.96095850000006,41.73469299900006],[36.11516750000004,41.625360500000056],[36.132280500000036,41.46124449800004],[36.38541250000003,41.25627349800004],[36.532507,41.26788149600003],[36.651739,41.38340199800007],[36.99572950000004,41.284005997000065],[37.03171350000008,41.18925299800003],[37.15234850000007,41.148413497000035],[36.986465,41.09718299700006],[36.880096500000036,40.960860497000056],[36.671261,40.915629498000044],[36.878349,40.74424849600007],[37.203664,40.704057498000054],[37.312427,40.57292049800003],[37.632352,40.54368199700008],[37.584064,40.424386998000045],[37.605382500000076,40.33798799900006],[37.468184,40.254590998000026],[37.450466,40.17053549800005],[37.09173550000003,40.249413],[36.80592750000005,40.22843699800006],[36.693430500000034,40.183547],[36.60641950000007,39.980525998000076],[36.003423,39.95451049700006],[35.903141,40.10542649800004],[35.596853500000066,40.09647649800007],[35.442588,40.23402249800006],[35.35115,40.24338049900007],[35.15563950000006,40.226112999000065],[35.17860250000007,40.143898498000055],[35.131494,40.11003999800005],[35.135924,40.036527498000055],[35.072878,40.01205649800005],[34.93486450000006,39.98463749800004],[34.86692750000003,40.036017498000035],[34.57078750000005,39.960256998000034],[34.384643,40.00414999700007],[34.175507,39.94043399900005],[34.093781500000034,40.02593349800003],[34.044084,40.18185199700008],[33.91622550000005,40.26303799900006],[33.695422,40.33224149800003],[33.34562450000004,40.398914999000056],[33.220758999000054,40.320784497000034],[32.944935,40.527210997000054],[32.972095499000034,40.61589350000003],[32.554816,40.69194399700007],[32.560043,40.807477498000026],[32.325888500000076,40.88752399800006],[32.274522,41.024024498000074],[32.134024,41.030703999000025],[31.75363,41.005629998000074],[31.424844,41.02525499600006],[31.295656500000064,41.11631999900004],[31.409174,41.21326249800006],[31.403204,41.31739249700007],[32.062578,41.58030599800003],[32.28410150000008,41.719801],[32.73610350000007,41.84900949800004],[33.31961650000005,42.01599299800006],[34.228878499000075,41.95495899800005],[34.797708500000056,41.95460299700005],[34.943724,42.096982998000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR81","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Zonguldak, Karabük, Bartin","NUTS_NAME":"Zonguldak, Karabük, Bartin"},"geometry":{"type":"Polygon","coordinates":[[[33.042014,41.06934599700003],[33.00407450000006,40.98937999700007],[32.796901,40.98866450000003],[32.73201050000006,40.89565949800004],[32.560043,40.807477498000026],[32.325888500000076,40.88752399800006],[32.274522,41.024024498000074],[32.134024,41.030703999000025],[31.75363,41.005629998000074],[31.424844,41.02525499600006],[31.295656500000064,41.11631999900004],[31.409174,41.21326249800006],[31.403204,41.31739249700007],[32.062578,41.58030599800003],[32.28410150000008,41.719801],[32.73610350000007,41.84900949800004],[32.84092950000007,41.689565498000036],[32.776286,41.58835299800006],[32.884223500000076,41.575608998000064],[33.014991,41.573578499000064],[33.127837,41.414296999000044],[32.946342,41.33131349700005],[32.89125450000006,41.17311499900006],[33.042014,41.06934599700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR811","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Zonguldak","NUTS_NAME":"Zonguldak"},"geometry":{"type":"Polygon","coordinates":[[[32.299087,41.305068497000036],[32.137025,41.219763998000076],[32.10346550000003,41.084481],[32.134024,41.030703999000025],[31.75363,41.005629998000074],[31.424844,41.02525499600006],[31.295656500000064,41.11631999900004],[31.409174,41.21326249800006],[31.403204,41.31739249700007],[32.062578,41.58030599800003],[32.21393749900005,41.52467349800003],[32.25201750000008,41.346753498000055],[32.299087,41.305068497000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR812","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Karabük","NUTS_NAME":"Karabük"},"geometry":{"type":"Polygon","coordinates":[[[33.042014,41.06934599700003],[33.00407450000006,40.98937999700007],[32.796901,40.98866450000003],[32.73201050000006,40.89565949800004],[32.560043,40.807477498000026],[32.325888500000076,40.88752399800006],[32.274522,41.024024498000074],[32.134024,41.030703999000025],[32.10346550000003,41.084481],[32.137025,41.219763998000076],[32.299087,41.305068497000036],[32.884223500000076,41.575608998000064],[33.014991,41.573578499000064],[33.127837,41.414296999000044],[32.946342,41.33131349700005],[32.89125450000006,41.17311499900006],[33.042014,41.06934599700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR813","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bartin","NUTS_NAME":"Bartin"},"geometry":{"type":"Polygon","coordinates":[[[32.884223500000076,41.575608998000064],[32.299087,41.305068497000036],[32.25201750000008,41.346753498000055],[32.21393749900005,41.52467349800003],[32.062578,41.58030599800003],[32.28410150000008,41.719801],[32.73610350000007,41.84900949800004],[32.84092950000007,41.689565498000036],[32.776286,41.58835299800006],[32.884223500000076,41.575608998000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR82","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Kastamonu, Çankiri, Sinop","NUTS_NAME":"Kastamonu, Çankiri, Sinop"},"geometry":{"type":"Polygon","coordinates":[[[34.943724,42.096982998000044],[35.08171250000004,42.02601799800004],[35.193533,42.03946899700003],[35.095869,41.965749997000046],[35.12668050000008,41.863176499000076],[35.276678,41.715734498000074],[35.51369950000003,41.63597999800004],[35.397116,41.461418998000056],[35.43774,41.35957599900007],[35.364086,41.27055649600004],[35.21965450000005,41.25416699600004],[35.10201650000005,41.400094498000044],[34.961706,41.34702899800004],[34.85675199900004,41.21563049800005],[34.449347,41.32738399800007],[34.301015,41.28488699600007],[34.177718,41.146645499000044],[34.165606,41.07271049800005],[34.24280950000008,41.003693],[34.263036500000055,40.89032049800005],[33.929878,40.827810499000066],[34.093371500000046,40.67939199800003],[34.076888,40.593744497000046],[34.134909,40.56427449800003],[34.097647,40.47327099800003],[34.14361950000006,40.41869599800003],[34.05706350000003,40.31695149900003],[33.91622550000005,40.26303799900006],[33.695422,40.33224149800003],[33.34562450000004,40.398914999000056],[33.220758999000054,40.320784497000034],[32.944935,40.527210997000054],[32.972095499000034,40.61589350000003],[32.554816,40.69194399700007],[32.560043,40.807477498000026],[32.73201050000006,40.89565949800004],[32.796901,40.98866450000003],[33.00407450000006,40.98937999700007],[33.042014,41.06934599700003],[32.89125450000006,41.17311499900006],[32.946342,41.33131349700005],[33.127837,41.414296999000044],[33.014991,41.573578499000064],[32.884223500000076,41.575608998000064],[32.776286,41.58835299800006],[32.84092950000007,41.689565498000036],[32.73610350000007,41.84900949800004],[33.31961650000005,42.01599299800006],[34.228878499000075,41.95495899800005],[34.797708500000056,41.95460299700005],[34.943724,42.096982998000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR821","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kastamonu","NUTS_NAME":"Kastamonu"},"geometry":{"type":"Polygon","coordinates":[[[34.228878499000075,41.95495899800005],[34.288809,41.810005498000066],[34.27204950000004,41.721594],[34.59725650000007,41.68005249700008],[34.57619,41.609762998000065],[34.436591,41.537417996000045],[34.449347,41.32738399800007],[34.301015,41.28488699600007],[34.177718,41.146645499000044],[34.165606,41.07271049800005],[34.24280950000008,41.003693],[34.263036500000055,40.89032049800005],[33.929878,40.827810499000066],[33.75175350000006,40.839949],[33.76123450000006,41.04970549900003],[33.446588500000075,41.04181149800007],[33.341282500000034,40.95854449900003],[33.257408,41.056318],[33.042014,41.06934599700003],[32.89125450000006,41.17311499900006],[32.946342,41.33131349700005],[33.127837,41.414296999000044],[33.014991,41.573578499000064],[32.884223500000076,41.575608998000064],[32.776286,41.58835299800006],[32.84092950000007,41.689565498000036],[32.73610350000007,41.84900949800004],[33.31961650000005,42.01599299800006],[34.228878499000075,41.95495899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR822","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Çankiri","NUTS_NAME":"Çankiri"},"geometry":{"type":"Polygon","coordinates":[[[33.929878,40.827810499000066],[34.093371500000046,40.67939199800003],[34.076888,40.593744497000046],[34.134909,40.56427449800003],[34.097647,40.47327099800003],[34.14361950000006,40.41869599800003],[34.05706350000003,40.31695149900003],[33.91622550000005,40.26303799900006],[33.695422,40.33224149800003],[33.34562450000004,40.398914999000056],[33.220758999000054,40.320784497000034],[32.944935,40.527210997000054],[32.972095499000034,40.61589350000003],[32.554816,40.69194399700007],[32.560043,40.807477498000026],[32.73201050000006,40.89565949800004],[32.796901,40.98866450000003],[33.00407450000006,40.98937999700007],[33.042014,41.06934599700003],[33.257408,41.056318],[33.341282500000034,40.95854449900003],[33.446588500000075,41.04181149800007],[33.76123450000006,41.04970549900003],[33.75175350000006,40.839949],[33.929878,40.827810499000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR823","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Sinop","NUTS_NAME":"Sinop"},"geometry":{"type":"Polygon","coordinates":[[[35.51369950000003,41.63597999800004],[35.397116,41.461418998000056],[35.43774,41.35957599900007],[35.364086,41.27055649600004],[35.21965450000005,41.25416699600004],[35.10201650000005,41.400094498000044],[34.961706,41.34702899800004],[34.85675199900004,41.21563049800005],[34.449347,41.32738399800007],[34.436591,41.537417996000045],[34.57619,41.609762998000065],[34.59725650000007,41.68005249700008],[34.27204950000004,41.721594],[34.288809,41.810005498000066],[34.228878499000075,41.95495899800005],[34.797708500000056,41.95460299700005],[34.943724,42.096982998000044],[35.08171250000004,42.02601799800004],[35.193533,42.03946899700003],[35.095869,41.965749997000046],[35.12668050000008,41.863176499000076],[35.276678,41.715734498000074],[35.51369950000003,41.63597999800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR83","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Samsun, Tokat, Çorum, Amasya","NUTS_NAME":"Samsun, Tokat, Çorum, Amasya"},"geometry":{"type":"Polygon","coordinates":[[[36.132280500000036,41.46124449800004],[36.38541250000003,41.25627349800004],[36.532507,41.26788149600003],[36.651739,41.38340199800007],[36.99572950000004,41.284005997000065],[37.03171350000008,41.18925299800003],[37.15234850000007,41.148413497000035],[36.986465,41.09718299700006],[36.880096500000036,40.960860497000056],[36.671261,40.915629498000044],[36.878349,40.74424849600007],[37.203664,40.704057498000054],[37.312427,40.57292049800003],[37.632352,40.54368199700008],[37.584064,40.424386998000045],[37.605382500000076,40.33798799900006],[37.468184,40.254590998000026],[37.450466,40.17053549800005],[37.09173550000003,40.249413],[36.80592750000005,40.22843699800006],[36.693430500000034,40.183547],[36.60641950000007,39.980525998000076],[36.003423,39.95451049700006],[35.903141,40.10542649800004],[35.596853500000066,40.09647649800007],[35.442588,40.23402249800006],[35.35115,40.24338049900007],[35.15563950000006,40.226112999000065],[35.17860250000007,40.143898498000055],[35.131494,40.11003999800005],[35.135924,40.036527498000055],[35.072878,40.01205649800005],[34.93486450000006,39.98463749800004],[34.86692750000003,40.036017498000035],[34.57078750000005,39.960256998000034],[34.384643,40.00414999700007],[34.175507,39.94043399900005],[34.093781500000034,40.02593349800003],[34.044084,40.18185199700008],[33.91622550000005,40.26303799900006],[34.05706350000003,40.31695149900003],[34.14361950000006,40.41869599800003],[34.097647,40.47327099800003],[34.134909,40.56427449800003],[34.076888,40.593744497000046],[34.093371500000046,40.67939199800003],[33.929878,40.827810499000066],[34.263036500000055,40.89032049800005],[34.24280950000008,41.003693],[34.165606,41.07271049800005],[34.177718,41.146645499000044],[34.301015,41.28488699600007],[34.449347,41.32738399800007],[34.85675199900004,41.21563049800005],[34.961706,41.34702899800004],[35.10201650000005,41.400094498000044],[35.21965450000005,41.25416699600004],[35.364086,41.27055649600004],[35.43774,41.35957599900007],[35.397116,41.461418998000056],[35.51369950000003,41.63597999800004],[35.96095850000006,41.73469299900006],[36.11516750000004,41.625360500000056],[36.132280500000036,41.46124449800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR831","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Samsun","NUTS_NAME":"Samsun"},"geometry":{"type":"Polygon","coordinates":[[[37.15234850000007,41.148413497000035],[36.986465,41.09718299700006],[36.880096500000036,40.960860497000056],[36.671261,40.915629498000044],[36.40945650000003,40.88810099800003],[36.32181450000007,40.92678299800008],[36.355107,40.97716599800003],[36.211812500000065,40.99699099800006],[36.059354,40.85417399900007],[35.700208,40.89044850000005],[35.474747500000035,40.96725349700006],[35.47908750000005,41.03699699800006],[35.22381050000007,41.01282999700004],[35.06223750000004,41.085923998000055],[34.92832750000008,41.12317199800003],[34.85675199900004,41.21563049800005],[34.961706,41.34702899800004],[35.10201650000005,41.400094498000044],[35.21965450000005,41.25416699600004],[35.364086,41.27055649600004],[35.43774,41.35957599900007],[35.397116,41.461418998000056],[35.51369950000003,41.63597999800004],[35.96095850000006,41.73469299900006],[36.11516750000004,41.625360500000056],[36.132280500000036,41.46124449800004],[36.38541250000003,41.25627349800004],[36.532507,41.26788149600003],[36.651739,41.38340199800007],[36.99572950000004,41.284005997000065],[37.03171350000008,41.18925299800003],[37.15234850000007,41.148413497000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR832","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Tokat","NUTS_NAME":"Tokat"},"geometry":{"type":"Polygon","coordinates":[[[37.584064,40.424386998000045],[37.605382500000076,40.33798799900006],[37.468184,40.254590998000026],[37.450466,40.17053549800005],[37.09173550000003,40.249413],[36.80592750000005,40.22843699800006],[36.693430500000034,40.183547],[36.60641950000007,39.980525998000076],[36.003423,39.95451049700006],[35.903141,40.10542649800004],[35.596853500000066,40.09647649800007],[35.442588,40.23402249800006],[35.499166,40.320726997000065],[35.791906,40.43604449900005],[35.826942,40.510967497000024],[36.132001,40.47069849700006],[36.26935150000003,40.53088549900008],[36.2575,40.61322699700003],[36.473253500000055,40.58042649600003],[36.41066150000006,40.72031949800004],[36.504773500000056,40.79812099700007],[36.418114,40.83286849600006],[36.40945650000003,40.88810099800003],[36.671261,40.915629498000044],[36.878349,40.74424849600007],[37.203664,40.704057498000054],[37.312427,40.57292049800003],[37.632352,40.54368199700008],[37.584064,40.424386998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR833","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Çorum","NUTS_NAME":"Çorum"},"geometry":{"type":"Polygon","coordinates":[[[34.85675199900004,41.21563049800005],[34.92832750000008,41.12317199800003],[35.06223750000004,41.085923998000055],[34.981332,40.838295998000035],[35.025363,40.72851499800004],[35.30619150000007,40.69104699800005],[35.321765500000026,40.636255998000024],[35.412757,40.63961549700008],[35.525584,40.559030998000026],[35.367863500000055,40.37973499700007],[35.4002,40.33634399700003],[35.35115,40.24338049900007],[35.15563950000006,40.226112999000065],[35.17860250000007,40.143898498000055],[35.131494,40.11003999800005],[35.135924,40.036527498000055],[35.072878,40.01205649800005],[34.93486450000006,39.98463749800004],[34.86692750000003,40.036017498000035],[34.57078750000005,39.960256998000034],[34.384643,40.00414999700007],[34.175507,39.94043399900005],[34.093781500000034,40.02593349800003],[34.044084,40.18185199700008],[33.91622550000005,40.26303799900006],[34.05706350000003,40.31695149900003],[34.14361950000006,40.41869599800003],[34.097647,40.47327099800003],[34.134909,40.56427449800003],[34.076888,40.593744497000046],[34.093371500000046,40.67939199800003],[33.929878,40.827810499000066],[34.263036500000055,40.89032049800005],[34.24280950000008,41.003693],[34.165606,41.07271049800005],[34.177718,41.146645499000044],[34.301015,41.28488699600007],[34.449347,41.32738399800007],[34.85675199900004,41.21563049800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR834","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Amasya","NUTS_NAME":"Amasya"},"geometry":{"type":"Polygon","coordinates":[[[36.40945650000003,40.88810099800003],[36.418114,40.83286849600006],[36.504773500000056,40.79812099700007],[36.41066150000006,40.72031949800004],[36.473253500000055,40.58042649600003],[36.2575,40.61322699700003],[36.26935150000003,40.53088549900008],[36.132001,40.47069849700006],[35.826942,40.510967497000024],[35.791906,40.43604449900005],[35.499166,40.320726997000065],[35.442588,40.23402249800006],[35.35115,40.24338049900007],[35.4002,40.33634399700003],[35.367863500000055,40.37973499700007],[35.525584,40.559030998000026],[35.412757,40.63961549700008],[35.321765500000026,40.636255998000024],[35.30619150000007,40.69104699800005],[35.025363,40.72851499800004],[34.981332,40.838295998000035],[35.06223750000004,41.085923998000055],[35.22381050000007,41.01282999700004],[35.47908750000005,41.03699699800006],[35.474747500000035,40.96725349700006],[35.700208,40.89044850000005],[36.059354,40.85417399900007],[36.211812500000065,40.99699099800006],[36.355107,40.97716599800003],[36.32181450000007,40.92678299800008],[36.40945650000003,40.88810099800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR9","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"DOGU KARADENIZ","NUTS_NAME":"DOGU KARADENIZ"},"geometry":{"type":"Polygon","coordinates":[[[42.515174932000036,41.43828241500006],[42.596096,41.271092499000076],[42.28524950000008,40.91643799800005],[41.94587350000006,40.950454996000076],[41.88722850000005,40.90449349800008],[41.870706,40.77003749900007],[41.80985750000008,40.68467349900004],[41.565097500000036,40.678368498000054],[41.38464550000003,40.56791199700007],[41.34072099900004,40.61019649600007],[41.34302750000006,40.739782999000056],[41.16401949900006,40.83493949800004],[40.60606150000007,40.53599999900007],[40.461455,40.52868399700003],[40.089906,40.57209499800007],[40.08720950000003,40.49855699600005],[39.93122050000005,40.50413949700004],[39.845806,40.430441],[39.813348,40.32007549800005],[39.84552650000006,40.22664399700005],[39.64333650000003,40.095837998000036],[39.65515050000005,40.01851849600007],[39.789502,39.94014599900004],[39.624406,39.86310449600006],[39.43621850000005,39.878557999000066],[38.956678500000066,40.068511998000076],[38.78355,40.083271498000045],[38.213064,40.21331249800005],[38.22174250000006,40.34620349800008],[38.148452500000076,40.52421849700005],[37.98682850000006,40.49290799900007],[37.928967,40.38215749600005],[37.75732850000003,40.405527996000046],[37.732457500000066,40.34250949900007],[37.584064,40.424386998000045],[37.632352,40.54368199700008],[37.312427,40.57292049800003],[37.203664,40.704057498000054],[36.878349,40.74424849600007],[36.671261,40.915629498000044],[36.880096500000036,40.960860497000056],[36.986465,41.09718299700006],[37.15234850000007,41.148413497000035],[37.292886500000066,41.14552499800004],[37.53754250000003,41.020898498000065],[37.674974500000076,41.128019497000025],[37.786104,41.116929998000046],[37.77697949900005,41.06200599600004],[37.87777150000005,40.98752399800003],[38.10999250000003,40.95851799700006],[38.529001,40.91645999700006],[39.17930850000005,41.074095998000075],[39.48634549900004,41.091859998000075],[39.646242,40.99686599700004],[40.12019150000003,40.912206498000046],[40.328287500000044,40.987883500000066],[40.721079,41.08785849700007],[41.2519,41.32993349800006],[41.401453,41.380735499000025],[41.54713350000003,41.52037549700003],[41.82492750000006,41.431911497000044],[41.971884,41.521492],[42.515174932000036,41.43828241500006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR90","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Trabzon","NUTS_NAME":"Trabzon"},"geometry":{"type":"Polygon","coordinates":[[[42.515174932000036,41.43828241500006],[42.596096,41.271092499000076],[42.28524950000008,40.91643799800005],[41.94587350000006,40.950454996000076],[41.88722850000005,40.90449349800008],[41.870706,40.77003749900007],[41.80985750000008,40.68467349900004],[41.565097500000036,40.678368498000054],[41.38464550000003,40.56791199700007],[41.34072099900004,40.61019649600007],[41.34302750000006,40.739782999000056],[41.16401949900006,40.83493949800004],[40.60606150000007,40.53599999900007],[40.461455,40.52868399700003],[40.089906,40.57209499800007],[40.08720950000003,40.49855699600005],[39.93122050000005,40.50413949700004],[39.845806,40.430441],[39.813348,40.32007549800005],[39.84552650000006,40.22664399700005],[39.64333650000003,40.095837998000036],[39.65515050000005,40.01851849600007],[39.789502,39.94014599900004],[39.624406,39.86310449600006],[39.43621850000005,39.878557999000066],[38.956678500000066,40.068511998000076],[38.78355,40.083271498000045],[38.213064,40.21331249800005],[38.22174250000006,40.34620349800008],[38.148452500000076,40.52421849700005],[37.98682850000006,40.49290799900007],[37.928967,40.38215749600005],[37.75732850000003,40.405527996000046],[37.732457500000066,40.34250949900007],[37.584064,40.424386998000045],[37.632352,40.54368199700008],[37.312427,40.57292049800003],[37.203664,40.704057498000054],[36.878349,40.74424849600007],[36.671261,40.915629498000044],[36.880096500000036,40.960860497000056],[36.986465,41.09718299700006],[37.15234850000007,41.148413497000035],[37.292886500000066,41.14552499800004],[37.53754250000003,41.020898498000065],[37.674974500000076,41.128019497000025],[37.786104,41.116929998000046],[37.77697949900005,41.06200599600004],[37.87777150000005,40.98752399800003],[38.10999250000003,40.95851799700006],[38.529001,40.91645999700006],[39.17930850000005,41.074095998000075],[39.48634549900004,41.091859998000075],[39.646242,40.99686599700004],[40.12019150000003,40.912206498000046],[40.328287500000044,40.987883500000066],[40.721079,41.08785849700007],[41.2519,41.32993349800006],[41.401453,41.380735499000025],[41.54713350000003,41.52037549700003],[41.82492750000006,41.431911497000044],[41.971884,41.521492],[42.515174932000036,41.43828241500006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR901","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Trabzon","NUTS_NAME":"Trabzon"},"geometry":{"type":"Polygon","coordinates":[[[40.328287500000044,40.987883500000066],[40.437976,40.893152],[40.48983550000003,40.73574049600006],[40.441989500000034,40.61126149800003],[40.461455,40.52868399700003],[40.089906,40.57209499800007],[40.00230350000004,40.65530799600003],[39.966579,40.64199549700004],[39.976475,40.58140349800004],[39.916164500000036,40.579353500000025],[39.82366250000007,40.633624498000074],[39.839495500000055,40.699610998000026],[39.79249050000004,40.72487549900006],[39.747298500000056,40.606919997000034],[39.650933500000065,40.55815949600003],[39.182076,40.79319099800006],[39.12265050000008,40.89709299800006],[39.17930850000005,41.074095998000075],[39.48634549900004,41.091859998000075],[39.646242,40.99686599700004],[40.12019150000003,40.912206498000046],[40.328287500000044,40.987883500000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR902","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Ordu","NUTS_NAME":"Ordu"},"geometry":{"type":"Polygon","coordinates":[[[38.10999250000003,40.95851799700006],[37.97504950000007,40.72798],[38.04872650000004,40.60285799600007],[38.164141,40.58740749900005],[38.148452500000076,40.52421849700005],[37.98682850000006,40.49290799900007],[37.928967,40.38215749600005],[37.75732850000003,40.405527996000046],[37.732457500000066,40.34250949900007],[37.584064,40.424386998000045],[37.632352,40.54368199700008],[37.312427,40.57292049800003],[37.203664,40.704057498000054],[36.878349,40.74424849600007],[36.671261,40.915629498000044],[36.880096500000036,40.960860497000056],[36.986465,41.09718299700006],[37.15234850000007,41.148413497000035],[37.292886500000066,41.14552499800004],[37.53754250000003,41.020898498000065],[37.674974500000076,41.128019497000025],[37.786104,41.116929998000046],[37.77697949900005,41.06200599600004],[37.87777150000005,40.98752399800003],[38.10999250000003,40.95851799700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR903","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Giresun","NUTS_NAME":"Giresun"},"geometry":{"type":"Polygon","coordinates":[[[39.17930850000005,41.074095998000075],[39.12265050000008,40.89709299800006],[39.182076,40.79319099800006],[38.981499,40.76249349700004],[39.007946,40.72213899800005],[38.78840950000006,40.53722849800005],[39.07812050000007,40.36806549800008],[38.954929,40.314482998000074],[38.88936950000004,40.21103399900005],[38.89397950000006,40.13299999700007],[38.956678500000066,40.068511998000076],[38.78355,40.083271498000045],[38.213064,40.21331249800005],[38.22174250000006,40.34620349800008],[38.148452500000076,40.52421849700005],[38.164141,40.58740749900005],[38.04872650000004,40.60285799600007],[37.97504950000007,40.72798],[38.10999250000003,40.95851799700006],[38.529001,40.91645999700006],[39.17930850000005,41.074095998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR904","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Rize","NUTS_NAME":"Rize"},"geometry":{"type":"Polygon","coordinates":[[[41.16401949900006,40.83493949800004],[40.60606150000007,40.53599999900007],[40.461455,40.52868399700003],[40.441989500000034,40.61126149800003],[40.48983550000003,40.73574049600006],[40.437976,40.893152],[40.328287500000044,40.987883500000066],[40.721079,41.08785849700007],[41.2519,41.32993349800006],[41.44371350000006,41.11401999700007],[41.19877850000006,40.92335249700005],[41.16401949900006,40.83493949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR905","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Artvin","NUTS_NAME":"Artvin"},"geometry":{"type":"Polygon","coordinates":[[[42.515174932000036,41.43828241500006],[42.596096,41.271092499000076],[42.28524950000008,40.91643799800005],[41.94587350000006,40.950454996000076],[41.88722850000005,40.90449349800008],[41.870706,40.77003749900007],[41.80985750000008,40.68467349900004],[41.565097500000036,40.678368498000054],[41.38464550000003,40.56791199700007],[41.34072099900004,40.61019649600007],[41.34302750000006,40.739782999000056],[41.16401949900006,40.83493949800004],[41.19877850000006,40.92335249700005],[41.44371350000006,41.11401999700007],[41.2519,41.32993349800006],[41.401453,41.380735499000025],[41.54713350000003,41.52037549700003],[41.82492750000006,41.431911497000044],[41.971884,41.521492],[42.515174932000036,41.43828241500006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR906","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Gümüshane","NUTS_NAME":"Gümüshane"},"geometry":{"type":"Polygon","coordinates":[[[40.089906,40.57209499800007],[40.08720950000003,40.49855699600005],[39.93122050000005,40.50413949700004],[39.845806,40.430441],[39.813348,40.32007549800005],[39.84552650000006,40.22664399700005],[39.64333650000003,40.095837998000036],[39.65515050000005,40.01851849600007],[39.789502,39.94014599900004],[39.624406,39.86310449600006],[39.43621850000005,39.878557999000066],[38.956678500000066,40.068511998000076],[38.89397950000006,40.13299999700007],[38.88936950000004,40.21103399900005],[38.954929,40.314482998000074],[39.07812050000007,40.36806549800008],[38.78840950000006,40.53722849800005],[39.007946,40.72213899800005],[38.981499,40.76249349700004],[39.182076,40.79319099800006],[39.650933500000065,40.55815949600003],[39.747298500000056,40.606919997000034],[39.79249050000004,40.72487549900006],[39.839495500000055,40.699610998000026],[39.82366250000007,40.633624498000074],[39.916164500000036,40.579353500000025],[39.976475,40.58140349800004],[39.966579,40.64199549700004],[40.00230350000004,40.65530799600003],[40.089906,40.57209499800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"KUZEYDOGU ANADOLU","NUTS_NAME":"KUZEYDOGU ANADOLU"},"geometry":{"type":"Polygon","coordinates":[[[43.19456150000008,41.25395999800003],[43.23022650000007,41.17379899900004],[43.376619,41.20249449600004],[43.473827,41.123297499000046],[43.469638,41.05734499700003],[43.67249,40.926521998000055],[43.750409,40.74499899800003],[43.717934,40.620918999000025],[43.583054,40.45111099800005],[43.715371,40.15718099700007],[43.653923,40.13037749800003],[43.933884,40.01361099800005],[44.351662,40.02222099700003],[44.487778,39.96276899700007],[44.76841,39.714094497000076],[44.812042500000075,39.63174349700006],[44.608464,39.77915599800008],[44.50209950000004,39.71699349900007],[44.401101,39.41651599800008],[44.076863,39.411656],[44.04995581000003,39.362815426000054],[43.834416,39.36310199800005],[43.716901,39.20951249900003],[43.412439,39.39359249800003],[43.14188850000005,39.31062699600005],[43.19581,39.259757498000056],[43.184449,39.198752497000044],[42.979636,39.01662349700007],[42.904852,38.93495549800008],[42.75008,38.92519099800006],[42.69884150000007,39.00302949700006],[42.671909,39.21350849700008],[42.51140449900004,39.427257498000074],[42.406795,39.46710349800003],[42.28747450000003,39.42019299900005],[42.125841500000035,39.21914499800005],[41.801216,39.14470749800006],[41.651629,39.15016250000008],[41.449497500000064,39.34245799800004],[41.20683,39.353699],[41.041464500000075,39.45200749800006],[40.633344,39.42827249700008],[40.605570500000056,39.46346699800006],[40.651635,39.522910498000044],[40.44844050000006,39.52215549700003],[40.345303,39.58817049800007],[40.00130350000006,39.553154998000025],[39.82890150000003,39.60165249700003],[39.607748,39.451070998000034],[39.551698,39.46044849700007],[39.536539,39.52085999700006],[39.072714,39.43730049800007],[38.93431299900004,39.326328498000066],[38.81324950000004,39.33135049800006],[38.801491500000054,39.28032399800003],[38.910778,39.237662998000076],[38.72951350000005,39.13715549600005],[38.768138,39.00626299700008],[38.68405350000006,39.02384299800008],[38.618035500000076,39.11645299700007],[38.349975,39.14994349600005],[38.35196150000007,39.35526299800006],[38.422460999000066,39.427839497000036],[38.335844500000064,39.494996498000035],[38.318415,39.55150599800004],[38.36187150000006,39.604872998000076],[38.326654,39.65799949800004],[38.477686,39.85301899800004],[38.366779500000064,39.93940199700006],[38.436214,39.993923497000026],[38.745546999000055,39.98660549700003],[38.78355,40.083271498000045],[38.956678500000066,40.068511998000076],[39.43621850000005,39.878557999000066],[39.624406,39.86310449600006],[39.789502,39.94014599900004],[39.65515050000005,40.01851849600007],[39.64333650000003,40.095837998000036],[39.84552650000006,40.22664399700005],[39.813348,40.32007549800005],[39.845806,40.430441],[39.93122050000005,40.50413949700004],[40.08720950000003,40.49855699600005],[40.089906,40.57209499800007],[40.461455,40.52868399700003],[40.60606150000007,40.53599999900007],[41.16401949900006,40.83493949800004],[41.34302750000006,40.739782999000056],[41.34072099900004,40.61019649600007],[41.38464550000003,40.56791199700007],[41.565097500000036,40.678368498000054],[41.80985750000008,40.68467349900004],[41.870706,40.77003749900007],[41.88722850000005,40.90449349800008],[41.94587350000006,40.950454996000076],[42.28524950000008,40.91643799800005],[42.596096,41.271092499000076],[42.515174932000036,41.43828241500006],[42.601024,41.584979496000074],[42.836095,41.584442497000055],[42.803529500000025,41.494068998000046],[43.21288050000004,41.30604149800007],[43.130417,41.25584549800004],[43.19456150000008,41.25395999800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA1","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Erzurum, Erzincan, Bayburt","NUTS_NAME":"Erzurum, Erzincan, Bayburt"},"geometry":{"type":"Polygon","coordinates":[[[42.57464950000008,40.66907699800004],[42.530173500000046,40.435869498000045],[42.128152,40.292448998000054],[42.203975,40.20729099700003],[42.373398500000064,40.15416150000004],[42.575026999000045,39.953925998000045],[42.300699500000064,39.841677498000024],[42.53097050000008,39.630930499000044],[42.406795,39.46710349800003],[42.28747450000003,39.42019299900005],[42.125841500000035,39.21914499800005],[41.801216,39.14470749800006],[41.651629,39.15016250000008],[41.449497500000064,39.34245799800004],[41.20683,39.353699],[41.041464500000075,39.45200749800006],[40.633344,39.42827249700008],[40.605570500000056,39.46346699800006],[40.651635,39.522910498000044],[40.44844050000006,39.52215549700003],[40.345303,39.58817049800007],[40.00130350000006,39.553154998000025],[39.82890150000003,39.60165249700003],[39.607748,39.451070998000034],[39.551698,39.46044849700007],[39.536539,39.52085999700006],[39.072714,39.43730049800007],[38.93431299900004,39.326328498000066],[38.81324950000004,39.33135049800006],[38.801491500000054,39.28032399800003],[38.910778,39.237662998000076],[38.72951350000005,39.13715549600005],[38.768138,39.00626299700008],[38.68405350000006,39.02384299800008],[38.618035500000076,39.11645299700007],[38.349975,39.14994349600005],[38.35196150000007,39.35526299800006],[38.422460999000066,39.427839497000036],[38.335844500000064,39.494996498000035],[38.318415,39.55150599800004],[38.36187150000006,39.604872998000076],[38.326654,39.65799949800004],[38.477686,39.85301899800004],[38.366779500000064,39.93940199700006],[38.436214,39.993923497000026],[38.745546999000055,39.98660549700003],[38.78355,40.083271498000045],[38.956678500000066,40.068511998000076],[39.43621850000005,39.878557999000066],[39.624406,39.86310449600006],[39.789502,39.94014599900004],[39.65515050000005,40.01851849600007],[39.64333650000003,40.095837998000036],[39.84552650000006,40.22664399700005],[39.813348,40.32007549800005],[39.845806,40.430441],[39.93122050000005,40.50413949700004],[40.08720950000003,40.49855699600005],[40.089906,40.57209499800007],[40.461455,40.52868399700003],[40.60606150000007,40.53599999900007],[41.16401949900006,40.83493949800004],[41.34302750000006,40.739782999000056],[41.34072099900004,40.61019649600007],[41.38464550000003,40.56791199700007],[41.565097500000036,40.678368498000054],[41.80985750000008,40.68467349900004],[41.870706,40.77003749900007],[41.88722850000005,40.90449349800008],[41.94587350000006,40.950454996000076],[42.28524950000008,40.91643799800005],[42.558164,40.74960150000004],[42.57464950000008,40.66907699800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA11","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Erzurum","NUTS_NAME":"Erzurum"},"geometry":{"type":"Polygon","coordinates":[[[42.28524950000008,40.91643799800005],[42.558164,40.74960150000004],[42.57464950000008,40.66907699800004],[42.530173500000046,40.435869498000045],[42.128152,40.292448998000054],[42.203975,40.20729099700003],[42.373398500000064,40.15416150000004],[42.575026999000045,39.953925998000045],[42.300699500000064,39.841677498000024],[42.53097050000008,39.630930499000044],[42.406795,39.46710349800003],[42.28747450000003,39.42019299900005],[42.125841500000035,39.21914499800005],[41.801216,39.14470749800006],[41.651629,39.15016250000008],[41.449497500000064,39.34245799800004],[41.20683,39.353699],[41.041464500000075,39.45200749800006],[40.633344,39.42827249700008],[40.605570500000056,39.46346699800006],[40.651635,39.522910498000044],[40.7787,39.54357],[40.697197,39.57090199800007],[40.55962899900004,39.75106499700007],[40.53842050000003,39.864175498000066],[40.281214,39.95203199700006],[40.36181450000004,40.02067799800005],[40.48088350000006,40.027695497000025],[40.748841500000026,40.214354497000045],[40.536043,40.24934799700003],[40.494345500000065,40.30989249900006],[40.589721,40.368134498000074],[40.529625500000066,40.429178498000056],[40.60606150000007,40.53599999900007],[41.16401949900006,40.83493949800004],[41.34302750000006,40.739782999000056],[41.34072099900004,40.61019649600007],[41.38464550000003,40.56791199700007],[41.565097500000036,40.678368498000054],[41.80985750000008,40.68467349900004],[41.870706,40.77003749900007],[41.88722850000005,40.90449349800008],[41.94587350000006,40.950454996000076],[42.28524950000008,40.91643799800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA12","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Erzincan","NUTS_NAME":"Erzincan"},"geometry":{"type":"Polygon","coordinates":[[[38.956678500000066,40.068511998000076],[39.43621850000005,39.878557999000066],[39.624406,39.86310449600006],[39.789502,39.94014599900004],[39.965553500000055,40.04410899800007],[40.36181450000004,40.02067799800005],[40.281214,39.95203199700006],[40.53842050000003,39.864175498000066],[40.55962899900004,39.75106499700007],[40.697197,39.57090199800007],[40.7787,39.54357],[40.651635,39.522910498000044],[40.44844050000006,39.52215549700003],[40.345303,39.58817049800007],[40.00130350000006,39.553154998000025],[39.82890150000003,39.60165249700003],[39.607748,39.451070998000034],[39.551698,39.46044849700007],[39.536539,39.52085999700006],[39.072714,39.43730049800007],[38.93431299900004,39.326328498000066],[38.81324950000004,39.33135049800006],[38.801491500000054,39.28032399800003],[38.910778,39.237662998000076],[38.72951350000005,39.13715549600005],[38.768138,39.00626299700008],[38.68405350000006,39.02384299800008],[38.618035500000076,39.11645299700007],[38.349975,39.14994349600005],[38.35196150000007,39.35526299800006],[38.422460999000066,39.427839497000036],[38.335844500000064,39.494996498000035],[38.318415,39.55150599800004],[38.36187150000006,39.604872998000076],[38.326654,39.65799949800004],[38.477686,39.85301899800004],[38.366779500000064,39.93940199700006],[38.436214,39.993923497000026],[38.745546999000055,39.98660549700003],[38.78355,40.083271498000045],[38.956678500000066,40.068511998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA13","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bayburt","NUTS_NAME":"Bayburt"},"geometry":{"type":"Polygon","coordinates":[[[40.461455,40.52868399700003],[40.60606150000007,40.53599999900007],[40.529625500000066,40.429178498000056],[40.589721,40.368134498000074],[40.494345500000065,40.30989249900006],[40.536043,40.24934799700003],[40.748841500000026,40.214354497000045],[40.48088350000006,40.027695497000025],[40.36181450000004,40.02067799800005],[39.965553500000055,40.04410899800007],[39.789502,39.94014599900004],[39.65515050000005,40.01851849600007],[39.64333650000003,40.095837998000036],[39.84552650000006,40.22664399700005],[39.813348,40.32007549800005],[39.845806,40.430441],[39.93122050000005,40.50413949700004],[40.08720950000003,40.49855699600005],[40.089906,40.57209499800007],[40.461455,40.52868399700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA2","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Agri, Kars, Igdir, Ardahan","NUTS_NAME":"Agri, Kars, Igdir, Ardahan"},"geometry":{"type":"Polygon","coordinates":[[[43.19456150000008,41.25395999800003],[43.23022650000007,41.17379899900004],[43.376619,41.20249449600004],[43.473827,41.123297499000046],[43.469638,41.05734499700003],[43.67249,40.926521998000055],[43.750409,40.74499899800003],[43.717934,40.620918999000025],[43.583054,40.45111099800005],[43.715371,40.15718099700007],[43.653923,40.13037749800003],[43.933884,40.01361099800005],[44.351662,40.02222099700003],[44.487778,39.96276899700007],[44.76841,39.714094497000076],[44.812042500000075,39.63174349700006],[44.608464,39.77915599800008],[44.50209950000004,39.71699349900007],[44.401101,39.41651599800008],[44.076863,39.411656],[44.04995581000003,39.362815426000054],[43.834416,39.36310199800005],[43.716901,39.20951249900003],[43.412439,39.39359249800003],[43.14188850000005,39.31062699600005],[43.19581,39.259757498000056],[43.184449,39.198752497000044],[42.979636,39.01662349700007],[42.904852,38.93495549800008],[42.75008,38.92519099800006],[42.69884150000007,39.00302949700006],[42.671909,39.21350849700008],[42.51140449900004,39.427257498000074],[42.406795,39.46710349800003],[42.53097050000008,39.630930499000044],[42.300699500000064,39.841677498000024],[42.575026999000045,39.953925998000045],[42.373398500000064,40.15416150000004],[42.203975,40.20729099700003],[42.128152,40.292448998000054],[42.530173500000046,40.435869498000045],[42.57464950000008,40.66907699800004],[42.558164,40.74960150000004],[42.28524950000008,40.91643799800005],[42.596096,41.271092499000076],[42.515174932000036,41.43828241500006],[42.601024,41.584979496000074],[42.836095,41.584442497000055],[42.803529500000025,41.494068998000046],[43.21288050000004,41.30604149800007],[43.130417,41.25584549800004],[43.19456150000008,41.25395999800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA21","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Agri","NUTS_NAME":"Agri"},"geometry":{"type":"Polygon","coordinates":[[[43.277592500000026,40.00681699800003],[43.436209,39.93152150000003],[43.427736,39.79388749700007],[43.574034,39.84365549700004],[43.96240350000005,39.717989],[44.50209950000004,39.71699349900007],[44.401101,39.41651599800008],[44.076863,39.411656],[44.04995581000003,39.362815426000054],[43.834416,39.36310199800005],[43.716901,39.20951249900003],[43.412439,39.39359249800003],[43.14188850000005,39.31062699600005],[43.19581,39.259757498000056],[43.184449,39.198752497000044],[42.979636,39.01662349700007],[42.904852,38.93495549800008],[42.75008,38.92519099800006],[42.69884150000007,39.00302949700006],[42.671909,39.21350849700008],[42.51140449900004,39.427257498000074],[42.406795,39.46710349800003],[42.53097050000008,39.630930499000044],[42.300699500000064,39.841677498000024],[42.575026999000045,39.953925998000045],[43.277592500000026,40.00681699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA22","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kars","NUTS_NAME":"Kars"},"geometry":{"type":"Polygon","coordinates":[[[43.469638,41.05734499700003],[43.67249,40.926521998000055],[43.750409,40.74499899800003],[43.717934,40.620918999000025],[43.583054,40.45111099800005],[43.715371,40.15718099700007],[43.653923,40.13037749800003],[43.37059750000003,40.127677998000024],[43.354865,40.060521497000025],[43.26325450000007,40.057434998000076],[43.277592500000026,40.00681699800003],[42.575026999000045,39.953925998000045],[42.373398500000064,40.15416150000004],[42.203975,40.20729099700003],[42.128152,40.292448998000054],[42.530173500000046,40.435869498000045],[42.57464950000008,40.66907699800004],[42.85228450000005,40.677241998000056],[42.930322,40.89911999800006],[43.26569150000006,40.977818],[43.312347,41.09227649800005],[43.469638,41.05734499700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA23","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Igdir","NUTS_NAME":"Igdir"},"geometry":{"type":"Polygon","coordinates":[[[44.76841,39.714094497000076],[44.812042500000075,39.63174349700006],[44.608464,39.77915599800008],[44.50209950000004,39.71699349900007],[43.96240350000005,39.717989],[43.574034,39.84365549700004],[43.427736,39.79388749700007],[43.436209,39.93152150000003],[43.277592500000026,40.00681699800003],[43.26325450000007,40.057434998000076],[43.354865,40.060521497000025],[43.37059750000003,40.127677998000024],[43.653923,40.13037749800003],[43.933884,40.01361099800005],[44.351662,40.02222099700003],[44.487778,39.96276899700007],[44.76841,39.714094497000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRA24","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Ardahan","NUTS_NAME":"Ardahan"},"geometry":{"type":"Polygon","coordinates":[[[43.473827,41.123297499000046],[43.469638,41.05734499700003],[43.312347,41.09227649800005],[43.26569150000006,40.977818],[42.930322,40.89911999800006],[42.85228450000005,40.677241998000056],[42.57464950000008,40.66907699800004],[42.558164,40.74960150000004],[42.28524950000008,40.91643799800005],[42.596096,41.271092499000076],[42.515174932000036,41.43828241500006],[42.601024,41.584979496000074],[42.836095,41.584442497000055],[42.803529500000025,41.494068998000046],[43.21288050000004,41.30604149800007],[43.130417,41.25584549800004],[43.19456150000008,41.25395999800003],[43.23022650000007,41.17379899900004],[43.376619,41.20249449600004],[43.473827,41.123297499000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"ORTADOGU ANADOLU","NUTS_NAME":"ORTADOGU ANADOLU"},"geometry":{"type":"Polygon","coordinates":[[[39.82890150000003,39.60165249700003],[40.00130350000006,39.553154998000025],[40.345303,39.58817049800007],[40.44844050000006,39.52215549700003],[40.651635,39.522910498000044],[40.605570500000056,39.46346699800006],[40.633344,39.42827249700008],[41.041464500000075,39.45200749800006],[41.20683,39.353699],[41.449497500000064,39.34245799800004],[41.651629,39.15016250000008],[41.801216,39.14470749800006],[42.125841500000035,39.21914499800005],[42.28747450000003,39.42019299900005],[42.406795,39.46710349800003],[42.51140449900004,39.427257498000074],[42.671909,39.21350849700008],[42.69884150000007,39.00302949700006],[42.75008,38.92519099800006],[42.904852,38.93495549800008],[42.979636,39.01662349700007],[43.184449,39.198752497000044],[43.19581,39.259757498000056],[43.14188850000005,39.31062699600005],[43.412439,39.39359249800003],[43.716901,39.20951249900003],[43.834416,39.36310199800005],[44.04995581000003,39.362815426000054],[44.21645,39.13651699600007],[44.173324999000044,38.96804099800005],[44.30026699900003,38.84262799800007],[44.261796,38.72429199800007],[44.317215,38.61304499800008],[44.30526,38.40053499700008],[44.436726,38.39480999800003],[44.482518,38.34129999700008],[44.223969,37.89914999800004],[44.46285493800008,37.809568168000055],[44.617768,37.71797599800004],[44.588852,37.443092997000065],[44.801659,37.32166299800008],[44.79314750000003,37.17628399800003],[44.582768,37.16318499800008],[44.349709,37.03832299700008],[44.319161,36.97123799900004],[44.257077,36.98568799800006],[44.195965,37.09637799700005],[44.262597,37.163467997000055],[44.262146,37.23651799800007],[44.118882,37.31568599900004],[44.009015,37.31776799800008],[43.91298,37.22658899800007],[43.424563,37.27533449900005],[43.307242,37.43826649700003],[43.362341,37.47377049800008],[43.358446,37.55748599800006],[43.494746,37.64243299800006],[43.50539050000003,37.71352999700008],[42.96620850000005,37.76213299900007],[42.896664,37.91730899800007],[42.767993500000046,37.913629497000045],[42.672305,37.862262498000064],[42.20442950000006,38.18850499800004],[41.998573,38.12600249800005],[41.702295,38.24654299800005],[41.648455,38.49419399800007],[41.506182,38.564977998000074],[41.38054999900004,38.492253497000036],[41.320564500000046,38.661234997000065],[41.176564,38.71672599900006],[40.98091350000004,38.606696498000076],[40.751045,38.56377199600007],[40.45941650000003,38.62262349800005],[40.38600950000006,38.472117997000055],[40.31407,38.465824498000075],[39.90497250000004,38.472211498000036],[39.870629499000074,38.380431498000064],[39.55518650000005,38.28074799900003],[39.315360500000054,38.336847],[39.162863,38.304434497000045],[39.10101350000008,38.20811699700005],[39.122747,38.183312998000076],[39.03216050000003,38.089630998000075],[38.816859,38.01220549800007],[38.32978250000008,38.20437699800004],[38.09520250000003,38.108330497000054],[38.15518,38.00250749700007],[38.089428500000054,37.95417299800005],[37.843465500000036,37.89613299800004],[37.638667500000054,37.93116699900003],[37.614140500000076,38.03833199700006],[37.74833550000005,38.14038149900006],[37.76024650000005,38.20305099700005],[37.64785550000005,38.34777349700005],[37.25953450000003,38.479910999000026],[37.341487,38.593515498000045],[37.42422350000004,38.70474199800003],[37.574179,38.78082549800007],[37.507155,38.890611499000045],[37.51233950000005,39.031703],[37.750074,38.978260498000054],[37.825541,39.09395049800003],[38.349975,39.14994349600005],[38.618035500000076,39.11645299700007],[38.68405350000006,39.02384299800008],[38.768138,39.00626299700008],[38.72951350000005,39.13715549600005],[38.910778,39.237662998000076],[38.801491500000054,39.28032399800003],[38.81324950000004,39.33135049800006],[38.93431299900004,39.326328498000066],[39.072714,39.43730049800007],[39.536539,39.52085999700006],[39.551698,39.46044849700007],[39.607748,39.451070998000034],[39.82890150000003,39.60165249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB1","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Malatya, Elazig, Bingöl, Tunceli","NUTS_NAME":"Malatya, Elazig, Bingöl, Tunceli"},"geometry":{"type":"Polygon","coordinates":[[[39.82890150000003,39.60165249700003],[40.00130350000006,39.553154998000025],[40.345303,39.58817049800007],[40.44844050000006,39.52215549700003],[40.651635,39.522910498000044],[40.605570500000056,39.46346699800006],[40.633344,39.42827249700008],[41.041464500000075,39.45200749800006],[41.20683,39.353699],[41.116833,39.273619997000026],[41.18267550000007,39.15644499800004],[41.32904950000005,39.04896249800004],[41.20420850000005,38.974273],[41.13685950000007,38.807050497000034],[41.176564,38.71672599900006],[40.98091350000004,38.606696498000076],[40.751045,38.56377199600007],[40.45941650000003,38.62262349800005],[40.38600950000006,38.472117997000055],[40.31407,38.465824498000075],[39.90497250000004,38.472211498000036],[39.870629499000074,38.380431498000064],[39.55518650000005,38.28074799900003],[39.315360500000054,38.336847],[39.162863,38.304434497000045],[39.10101350000008,38.20811699700005],[39.122747,38.183312998000076],[39.03216050000003,38.089630998000075],[38.816859,38.01220549800007],[38.32978250000008,38.20437699800004],[38.09520250000003,38.108330497000054],[38.15518,38.00250749700007],[38.089428500000054,37.95417299800005],[37.843465500000036,37.89613299800004],[37.638667500000054,37.93116699900003],[37.614140500000076,38.03833199700006],[37.74833550000005,38.14038149900006],[37.76024650000005,38.20305099700005],[37.64785550000005,38.34777349700005],[37.25953450000003,38.479910999000026],[37.341487,38.593515498000045],[37.42422350000004,38.70474199800003],[37.574179,38.78082549800007],[37.507155,38.890611499000045],[37.51233950000005,39.031703],[37.750074,38.978260498000054],[37.825541,39.09395049800003],[38.349975,39.14994349600005],[38.618035500000076,39.11645299700007],[38.68405350000006,39.02384299800008],[38.768138,39.00626299700008],[38.72951350000005,39.13715549600005],[38.910778,39.237662998000076],[38.801491500000054,39.28032399800003],[38.81324950000004,39.33135049800006],[38.93431299900004,39.326328498000066],[39.072714,39.43730049800007],[39.536539,39.52085999700006],[39.551698,39.46044849700007],[39.607748,39.451070998000034],[39.82890150000003,39.60165249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB11","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Malatya","NUTS_NAME":"Malatya"},"geometry":{"type":"Polygon","coordinates":[[[38.349975,39.14994349600005],[38.618035500000076,39.11645299700007],[38.68405350000006,39.02384299800008],[38.590775,38.94054399800007],[38.674176,38.76825249800004],[38.511610500000074,38.76980099700006],[38.375019500000064,38.630722],[38.367964,38.54359599700007],[38.446075,38.47405449900003],[38.727572500000065,38.407560498000066],[38.840506,38.44316199800005],[38.96106399900003,38.34211249700007],[39.091883,38.361440497000046],[39.162863,38.304434497000045],[39.10101350000008,38.20811699700005],[39.122747,38.183312998000076],[39.03216050000003,38.089630998000075],[38.816859,38.01220549800007],[38.32978250000008,38.20437699800004],[38.09520250000003,38.108330497000054],[38.15518,38.00250749700007],[38.089428500000054,37.95417299800005],[37.843465500000036,37.89613299800004],[37.638667500000054,37.93116699900003],[37.614140500000076,38.03833199700006],[37.74833550000005,38.14038149900006],[37.76024650000005,38.20305099700005],[37.64785550000005,38.34777349700005],[37.25953450000003,38.479910999000026],[37.341487,38.593515498000045],[37.42422350000004,38.70474199800003],[37.574179,38.78082549800007],[37.507155,38.890611499000045],[37.51233950000005,39.031703],[37.750074,38.978260498000054],[37.825541,39.09395049800003],[38.349975,39.14994349600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB12","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Elazig","NUTS_NAME":"Elazig"},"geometry":{"type":"Polygon","coordinates":[[[40.31407,38.465824498000075],[39.90497250000004,38.472211498000036],[39.870629499000074,38.380431498000064],[39.55518650000005,38.28074799900003],[39.315360500000054,38.336847],[39.162863,38.304434497000045],[39.091883,38.361440497000046],[38.96106399900003,38.34211249700007],[38.840506,38.44316199800005],[38.727572500000065,38.407560498000066],[38.446075,38.47405449900003],[38.367964,38.54359599700007],[38.375019500000064,38.630722],[38.511610500000074,38.76980099700006],[38.674176,38.76825249800004],[38.590775,38.94054399800007],[38.68405350000006,39.02384299800008],[38.768138,39.00626299700008],[38.781938,38.886246999000036],[39.252985,38.86466749900006],[39.49491,38.765093497000066],[39.60264,38.84575799800007],[39.816908,38.85708899800005],[39.89849850000007,38.95713049600005],[39.851196,39.07288349800007],[39.93973650000004,39.11405849800008],[40.037517500000035,39.08262549700004],[40.209618,39.18305299800005],[40.287477,39.17187149800003],[40.25677250000007,38.87099499900006],[40.367115,38.78831249800004],[40.155632,38.62051749600005],[40.31407,38.465824498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB13","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bingöl","NUTS_NAME":"Bingöl"},"geometry":{"type":"Polygon","coordinates":[[[41.20683,39.353699],[41.116833,39.273619997000026],[41.18267550000007,39.15644499800004],[41.32904950000005,39.04896249800004],[41.20420850000005,38.974273],[41.13685950000007,38.807050497000034],[41.176564,38.71672599900006],[40.98091350000004,38.606696498000076],[40.751045,38.56377199600007],[40.45941650000003,38.62262349800005],[40.38600950000006,38.472117997000055],[40.31407,38.465824498000075],[40.155632,38.62051749600005],[40.367115,38.78831249800004],[40.25677250000007,38.87099499900006],[40.287477,39.17187149800003],[40.209618,39.18305299800005],[40.037517500000035,39.08262549700004],[39.93973650000004,39.11405849800008],[40.00148,39.31645049800005],[40.099464,39.29433499900006],[40.23974150000004,39.35718550000007],[40.27382250000005,39.385785498000075],[40.232269,39.41772749900008],[40.44844050000006,39.52215549700003],[40.651635,39.522910498000044],[40.605570500000056,39.46346699800006],[40.633344,39.42827249700008],[41.041464500000075,39.45200749800006],[41.20683,39.353699]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB14","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Tunceli","NUTS_NAME":"Tunceli"},"geometry":{"type":"Polygon","coordinates":[[[40.44844050000006,39.52215549700003],[40.232269,39.41772749900008],[40.27382250000005,39.385785498000075],[40.23974150000004,39.35718550000007],[40.099464,39.29433499900006],[40.00148,39.31645049800005],[39.93973650000004,39.11405849800008],[39.851196,39.07288349800007],[39.89849850000007,38.95713049600005],[39.816908,38.85708899800005],[39.60264,38.84575799800007],[39.49491,38.765093497000066],[39.252985,38.86466749900006],[38.781938,38.886246999000036],[38.768138,39.00626299700008],[38.72951350000005,39.13715549600005],[38.910778,39.237662998000076],[38.801491500000054,39.28032399800003],[38.81324950000004,39.33135049800006],[38.93431299900004,39.326328498000066],[39.072714,39.43730049800007],[39.536539,39.52085999700006],[39.551698,39.46044849700007],[39.607748,39.451070998000034],[39.82890150000003,39.60165249700003],[40.00130350000006,39.553154998000025],[40.345303,39.58817049800007],[40.44844050000006,39.52215549700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB2","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Van, Mus, Bitlis, Hakkari","NUTS_NAME":"Van, Mus, Bitlis, Hakkari"},"geometry":{"type":"Polygon","coordinates":[[[42.671909,39.21350849700008],[42.69884150000007,39.00302949700006],[42.75008,38.92519099800006],[42.904852,38.93495549800008],[42.979636,39.01662349700007],[43.184449,39.198752497000044],[43.19581,39.259757498000056],[43.14188850000005,39.31062699600005],[43.412439,39.39359249800003],[43.716901,39.20951249900003],[43.834416,39.36310199800005],[44.04995581000003,39.362815426000054],[44.21645,39.13651699600007],[44.173324999000044,38.96804099800005],[44.30026699900003,38.84262799800007],[44.261796,38.72429199800007],[44.317215,38.61304499800008],[44.30526,38.40053499700008],[44.436726,38.39480999800003],[44.482518,38.34129999700008],[44.223969,37.89914999800004],[44.46285493800008,37.809568168000055],[44.617768,37.71797599800004],[44.588852,37.443092997000065],[44.801659,37.32166299800008],[44.79314750000003,37.17628399800003],[44.582768,37.16318499800008],[44.349709,37.03832299700008],[44.319161,36.97123799900004],[44.257077,36.98568799800006],[44.195965,37.09637799700005],[44.262597,37.163467997000055],[44.262146,37.23651799800007],[44.118882,37.31568599900004],[44.009015,37.31776799800008],[43.91298,37.22658899800007],[43.424563,37.27533449900005],[43.307242,37.43826649700003],[43.362341,37.47377049800008],[43.358446,37.55748599800006],[43.494746,37.64243299800006],[43.50539050000003,37.71352999700008],[42.96620850000005,37.76213299900007],[42.896664,37.91730899800007],[42.767993500000046,37.913629497000045],[42.672305,37.862262498000064],[42.20442950000006,38.18850499800004],[41.998573,38.12600249800005],[41.702295,38.24654299800005],[41.648455,38.49419399800007],[41.506182,38.564977998000074],[41.38054999900004,38.492253497000036],[41.320564500000046,38.661234997000065],[41.176564,38.71672599900006],[41.13685950000007,38.807050497000034],[41.20420850000005,38.974273],[41.32904950000005,39.04896249800004],[41.18267550000007,39.15644499800004],[41.116833,39.273619997000026],[41.20683,39.353699],[41.449497500000064,39.34245799800004],[41.651629,39.15016250000008],[41.801216,39.14470749800006],[42.125841500000035,39.21914499800005],[42.28747450000003,39.42019299900005],[42.406795,39.46710349800003],[42.51140449900004,39.427257498000074],[42.671909,39.21350849700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB21","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Van","NUTS_NAME":"Van"},"geometry":{"type":"Polygon","coordinates":[[[44.04995581000003,39.362815426000054],[44.21645,39.13651699600007],[44.173324999000044,38.96804099800005],[44.30026699900003,38.84262799800007],[44.261796,38.72429199800007],[44.317215,38.61304499800008],[44.30526,38.40053499700008],[44.436726,38.39480999800003],[44.482518,38.34129999700008],[44.223969,37.89914999800004],[44.46285493800008,37.809568168000055],[44.346455,37.73681349800006],[44.109362,37.703663998000025],[44.134818,37.82305499700004],[44.07464850000008,37.82818799700004],[43.857082,37.74410049800008],[43.50539050000003,37.71352999700008],[42.96620850000005,37.76213299900007],[42.896664,37.91730899800007],[42.767993500000046,37.913629497000045],[42.79074450000007,38.13433999800003],[42.711182,38.19332699800003],[42.795731500000045,38.23351299600006],[42.755496,38.25786499800006],[42.75635450000004,38.50179399600006],[42.91132650000003,38.55022299700005],[43.233557,38.85092949800003],[43.220898,38.90294699800006],[42.979636,39.01662349700007],[43.184449,39.198752497000044],[43.19581,39.259757498000056],[43.14188850000005,39.31062699600005],[43.412439,39.39359249800003],[43.716901,39.20951249900003],[43.834416,39.36310199800005],[44.04995581000003,39.362815426000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB22","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Mus","NUTS_NAME":"Mus"},"geometry":{"type":"Polygon","coordinates":[[[42.75008,38.92519099800006],[42.216686,38.88752499800006],[42.152504,38.80760349800005],[42.00543,38.78774599800005],[42.077666,38.72828549600007],[41.980509,38.60559299600004],[41.506182,38.564977998000074],[41.38054999900004,38.492253497000036],[41.320564500000046,38.661234997000065],[41.176564,38.71672599900006],[41.13685950000007,38.807050497000034],[41.20420850000005,38.974273],[41.32904950000005,39.04896249800004],[41.18267550000007,39.15644499800004],[41.116833,39.273619997000026],[41.20683,39.353699],[41.449497500000064,39.34245799800004],[41.651629,39.15016250000008],[41.801216,39.14470749800006],[42.125841500000035,39.21914499800005],[42.28747450000003,39.42019299900005],[42.406795,39.46710349800003],[42.51140449900004,39.427257498000074],[42.671909,39.21350849700008],[42.69884150000007,39.00302949700006],[42.75008,38.92519099800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB23","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bitlis","NUTS_NAME":"Bitlis"},"geometry":{"type":"Polygon","coordinates":[[[42.767993500000046,37.913629497000045],[42.672305,37.862262498000064],[42.20442950000006,38.18850499800004],[41.998573,38.12600249800005],[41.702295,38.24654299800005],[41.648455,38.49419399800007],[41.506182,38.564977998000074],[41.980509,38.60559299600004],[42.077666,38.72828549600007],[42.00543,38.78774599800005],[42.152504,38.80760349800005],[42.216686,38.88752499800006],[42.75008,38.92519099800006],[42.904852,38.93495549800008],[42.979636,39.01662349700007],[43.220898,38.90294699800006],[43.233557,38.85092949800003],[42.91132650000003,38.55022299700005],[42.75635450000004,38.50179399600006],[42.755496,38.25786499800006],[42.795731500000045,38.23351299600006],[42.711182,38.19332699800003],[42.79074450000007,38.13433999800003],[42.767993500000046,37.913629497000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRB24","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Hakkari","NUTS_NAME":"Hakkari"},"geometry":{"type":"Polygon","coordinates":[[[44.46285493800008,37.809568168000055],[44.617768,37.71797599800004],[44.588852,37.443092997000065],[44.801659,37.32166299800008],[44.79314750000003,37.17628399800003],[44.582768,37.16318499800008],[44.349709,37.03832299700008],[44.319161,36.97123799900004],[44.257077,36.98568799800006],[44.195965,37.09637799700005],[44.262597,37.163467997000055],[44.262146,37.23651799800007],[44.118882,37.31568599900004],[44.009015,37.31776799800008],[43.91298,37.22658899800007],[43.424563,37.27533449900005],[43.307242,37.43826649700003],[43.362341,37.47377049800008],[43.358446,37.55748599800006],[43.494746,37.64243299800006],[43.50539050000003,37.71352999700008],[43.857082,37.74410049800008],[44.07464850000008,37.82818799700004],[44.134818,37.82305499700004],[44.109362,37.703663998000025],[44.346455,37.73681349800006],[44.46285493800008,37.809568168000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"GÜNEYDOGU ANADOLU","NUTS_NAME":"GÜNEYDOGU ANADOLU"},"geometry":{"type":"Polygon","coordinates":[[[41.320564500000046,38.661234997000065],[41.38054999900004,38.492253497000036],[41.506182,38.564977998000074],[41.648455,38.49419399800007],[41.702295,38.24654299800005],[41.998573,38.12600249800005],[42.20442950000006,38.18850499800004],[42.672305,37.862262498000064],[42.767993500000046,37.913629497000045],[42.896664,37.91730899800007],[42.96620850000005,37.76213299900007],[43.50539050000003,37.71352999700008],[43.494746,37.64243299800006],[43.358446,37.55748599800006],[43.362341,37.47377049800008],[43.307242,37.43826649700003],[43.424563,37.27533449900005],[43.156937,37.37387799800007],[42.924164,37.32776699800007],[42.786801,37.38367499800006],[42.57950599900005,37.15367499800004],[42.35568250000006,37.10799350000008],[42.351521,37.23116999900003],[42.180832,37.29054199800004],[42.05693099900003,37.19277099900006],[41.663342,37.102903498000046],[41.216382,37.07248999800004],[40.770821,37.11804999800006],[40.226033,36.901373498000055],[39.818055,36.75360899800006],[39.221524,36.665340996000054],[38.725266,36.70360599800006],[38.386376,36.89832999600003],[38.181107,36.90582299700003],[38.04758950000007,36.845554496000034],[37.58551650000004,36.703768498000045],[37.127495,36.65915699900006],[36.98832,36.757419999000035],[36.678777500000024,36.83266099700006],[36.66001039400004,36.83323153500004],[36.465673,36.94866749800008],[36.52120250000007,37.08523999800008],[36.697388500000045,37.21814249600004],[36.901796,37.33366049800003],[37.08278,37.17567599900008],[37.379789,37.35999050000004],[37.55469750000003,37.382446999000024],[37.62321650000007,37.51165249600007],[37.56929149900003,37.58780599700003],[37.432401,37.64170899900006],[37.502716,37.792868498000075],[37.61636,37.85288349900003],[37.638667500000054,37.93116699900003],[37.843465500000036,37.89613299800004],[38.089428500000054,37.95417299800005],[38.15518,38.00250749700007],[38.09520250000003,38.108330497000054],[38.32978250000008,38.20437699800004],[38.816859,38.01220549800007],[39.03216050000003,38.089630998000075],[39.122747,38.183312998000076],[39.10101350000008,38.20811699700005],[39.162863,38.304434497000045],[39.315360500000054,38.336847],[39.55518650000005,38.28074799900003],[39.870629499000074,38.380431498000064],[39.90497250000004,38.472211498000036],[40.31407,38.465824498000075],[40.38600950000006,38.472117997000055],[40.45941650000003,38.62262349800005],[40.751045,38.56377199600007],[40.98091350000004,38.606696498000076],[41.176564,38.71672599900006],[41.320564500000046,38.661234997000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC1","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Gaziantep, Adiyaman, Kilis","NUTS_NAME":"Gaziantep, Adiyaman, Kilis"},"geometry":{"type":"Polygon","coordinates":[[[38.32978250000008,38.20437699800004],[38.816859,38.01220549800007],[39.03216050000003,38.089630998000075],[39.122747,38.183312998000076],[39.22818150000006,38.20150099800003],[39.262784,38.14612599700007],[39.176390500000025,38.02607199800008],[39.099832,38.012478498000064],[38.978756,37.82477049800008],[38.900089,37.807860997000034],[38.969933500000025,37.76203299700006],[38.822572500000035,37.69401449800006],[38.851450500000055,37.649203998000075],[38.17676650000004,37.422139998000034],[38.008049999000036,37.454740997000044],[37.89713750000004,37.348729],[37.83555750000005,37.165888497000026],[38.04758950000007,36.845554496000034],[37.58551650000004,36.703768498000045],[37.127495,36.65915699900006],[36.98832,36.757419999000035],[36.678777500000024,36.83266099700006],[36.66001039400004,36.83323153500004],[36.465673,36.94866749800008],[36.52120250000007,37.08523999800008],[36.697388500000045,37.21814249600004],[36.901796,37.33366049800003],[37.08278,37.17567599900008],[37.379789,37.35999050000004],[37.55469750000003,37.382446999000024],[37.62321650000007,37.51165249600007],[37.56929149900003,37.58780599700003],[37.432401,37.64170899900006],[37.502716,37.792868498000075],[37.61636,37.85288349900003],[37.638667500000054,37.93116699900003],[37.843465500000036,37.89613299800004],[38.089428500000054,37.95417299800005],[38.15518,38.00250749700007],[38.09520250000003,38.108330497000054],[38.32978250000008,38.20437699800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC11","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Gaziantep","NUTS_NAME":"Gaziantep"},"geometry":{"type":"Polygon","coordinates":[[[38.008049999000036,37.454740997000044],[37.89713750000004,37.348729],[37.83555750000005,37.165888497000026],[38.04758950000007,36.845554496000034],[37.58551650000004,36.703768498000045],[37.39410850000007,36.78877399700008],[37.329394,36.73435949700007],[37.247576,36.86680049800003],[37.059747,36.92276749800004],[36.99025,37.05038699800008],[36.912925,37.037874497000075],[36.89773150000008,36.951103997000075],[36.678777500000024,36.83266099700006],[36.66001039400004,36.83323153500004],[36.465673,36.94866749800008],[36.52120250000007,37.08523999800008],[36.697388500000045,37.21814249600004],[36.901796,37.33366049800003],[37.08278,37.17567599900008],[37.379789,37.35999050000004],[37.55469750000003,37.382446999000024],[37.62321650000007,37.51165249600007],[37.746002,37.528160498000034],[38.008049999000036,37.454740997000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC12","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Adiyaman","NUTS_NAME":"Adiyaman"},"geometry":{"type":"Polygon","coordinates":[[[39.122747,38.183312998000076],[39.22818150000006,38.20150099800003],[39.262784,38.14612599700007],[39.176390500000025,38.02607199800008],[39.099832,38.012478498000064],[38.978756,37.82477049800008],[38.900089,37.807860997000034],[38.969933500000025,37.76203299700006],[38.822572500000035,37.69401449800006],[38.851450500000055,37.649203998000075],[38.17676650000004,37.422139998000034],[38.008049999000036,37.454740997000044],[37.746002,37.528160498000034],[37.62321650000007,37.51165249600007],[37.56929149900003,37.58780599700003],[37.432401,37.64170899900006],[37.502716,37.792868498000075],[37.61636,37.85288349900003],[37.638667500000054,37.93116699900003],[37.843465500000036,37.89613299800004],[38.089428500000054,37.95417299800005],[38.15518,38.00250749700007],[38.09520250000003,38.108330497000054],[38.32978250000008,38.20437699800004],[38.816859,38.01220549800007],[39.03216050000003,38.089630998000075],[39.122747,38.183312998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC13","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kilis","NUTS_NAME":"Kilis"},"geometry":{"type":"Polygon","coordinates":[[[37.58551650000004,36.703768498000045],[37.127495,36.65915699900006],[36.98832,36.757419999000035],[36.678777500000024,36.83266099700006],[36.89773150000008,36.951103997000075],[36.912925,37.037874497000075],[36.99025,37.05038699800008],[37.059747,36.92276749800004],[37.247576,36.86680049800003],[37.329394,36.73435949700007],[37.39410850000007,36.78877399700008],[37.58551650000004,36.703768498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC2","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Sanliurfa, Diyarbakir","NUTS_NAME":"Sanliurfa, Diyarbakir"},"geometry":{"type":"Polygon","coordinates":[[[41.320564500000046,38.661234997000065],[41.38054999900004,38.492253497000036],[41.189474,38.31202699700003],[41.180091,38.25693699800007],[41.25448,38.13951949900007],[40.995091,37.82686299900007],[41.08888950000005,37.758909998000036],[41.045006,37.71503049800003],[40.89438750000005,37.65617199800005],[40.68667850000003,37.73295399800003],[40.639687,37.62242899800003],[40.464743,37.61741849800006],[40.331074999000066,37.52148299700008],[39.85554950000005,37.531011997000064],[40.11830850000007,37.119650998000054],[40.109505,37.002485498000055],[40.226033,36.901373498000055],[39.818055,36.75360899800006],[39.221524,36.665340996000054],[38.725266,36.70360599800006],[38.386376,36.89832999600003],[38.181107,36.90582299700003],[38.04758950000007,36.845554496000034],[37.83555750000005,37.165888497000026],[37.89713750000004,37.348729],[38.008049999000036,37.454740997000044],[38.17676650000004,37.422139998000034],[38.851450500000055,37.649203998000075],[38.822572500000035,37.69401449800006],[38.969933500000025,37.76203299700006],[38.900089,37.807860997000034],[38.978756,37.82477049800008],[39.099832,38.012478498000064],[39.176390500000025,38.02607199800008],[39.262784,38.14612599700007],[39.22818150000006,38.20150099800003],[39.122747,38.183312998000076],[39.10101350000008,38.20811699700005],[39.162863,38.304434497000045],[39.315360500000054,38.336847],[39.55518650000005,38.28074799900003],[39.870629499000074,38.380431498000064],[39.90497250000004,38.472211498000036],[40.31407,38.465824498000075],[40.38600950000006,38.472117997000055],[40.45941650000003,38.62262349800005],[40.751045,38.56377199600007],[40.98091350000004,38.606696498000076],[41.176564,38.71672599900006],[41.320564500000046,38.661234997000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC21","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Sanliurfa","NUTS_NAME":"Sanliurfa"},"geometry":{"type":"Polygon","coordinates":[[[39.85554950000005,37.531011997000064],[40.11830850000007,37.119650998000054],[40.109505,37.002485498000055],[40.226033,36.901373498000055],[39.818055,36.75360899800006],[39.221524,36.665340996000054],[38.725266,36.70360599800006],[38.386376,36.89832999600003],[38.181107,36.90582299700003],[38.04758950000007,36.845554496000034],[37.83555750000005,37.165888497000026],[37.89713750000004,37.348729],[38.008049999000036,37.454740997000044],[38.17676650000004,37.422139998000034],[38.851450500000055,37.649203998000075],[38.822572500000035,37.69401449800006],[38.969933500000025,37.76203299700006],[38.900089,37.807860997000034],[38.978756,37.82477049800008],[39.099832,38.012478498000064],[39.41677150000004,37.95038049800007],[39.63252250000005,37.80828],[39.82965050000007,37.845090998000046],[39.85554950000005,37.531011997000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC22","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Diyarbakir","NUTS_NAME":"Diyarbakir"},"geometry":{"type":"Polygon","coordinates":[[[41.38054999900004,38.492253497000036],[41.189474,38.31202699700003],[41.180091,38.25693699800007],[41.25448,38.13951949900007],[40.995091,37.82686299900007],[41.08888950000005,37.758909998000036],[41.045006,37.71503049800003],[40.89438750000005,37.65617199800005],[40.68667850000003,37.73295399800003],[40.639687,37.62242899800003],[40.464743,37.61741849800006],[40.331074999000066,37.52148299700008],[39.85554950000005,37.531011997000064],[39.82965050000007,37.845090998000046],[39.63252250000005,37.80828],[39.41677150000004,37.95038049800007],[39.099832,38.012478498000064],[39.176390500000025,38.02607199800008],[39.262784,38.14612599700007],[39.22818150000006,38.20150099800003],[39.122747,38.183312998000076],[39.10101350000008,38.20811699700005],[39.162863,38.304434497000045],[39.315360500000054,38.336847],[39.55518650000005,38.28074799900003],[39.870629499000074,38.380431498000064],[39.90497250000004,38.472211498000036],[40.31407,38.465824498000075],[40.38600950000006,38.472117997000055],[40.45941650000003,38.62262349800005],[40.751045,38.56377199600007],[40.98091350000004,38.606696498000076],[41.176564,38.71672599900006],[41.320564500000046,38.661234997000065],[41.38054999900004,38.492253497000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC3","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Mardin, Batman, Sirnak, Siirt","NUTS_NAME":"Mardin, Batman, Sirnak, Siirt"},"geometry":{"type":"Polygon","coordinates":[[[41.702295,38.24654299800005],[41.998573,38.12600249800005],[42.20442950000006,38.18850499800004],[42.672305,37.862262498000064],[42.767993500000046,37.913629497000045],[42.896664,37.91730899800007],[42.96620850000005,37.76213299900007],[43.50539050000003,37.71352999700008],[43.494746,37.64243299800006],[43.358446,37.55748599800006],[43.362341,37.47377049800008],[43.307242,37.43826649700003],[43.424563,37.27533449900005],[43.156937,37.37387799800007],[42.924164,37.32776699800007],[42.786801,37.38367499800006],[42.57950599900005,37.15367499800004],[42.35568250000006,37.10799350000008],[42.351521,37.23116999900003],[42.180832,37.29054199800004],[42.05693099900003,37.19277099900006],[41.663342,37.102903498000046],[41.216382,37.07248999800004],[40.770821,37.11804999800006],[40.226033,36.901373498000055],[40.109505,37.002485498000055],[40.11830850000007,37.119650998000054],[39.85554950000005,37.531011997000064],[40.331074999000066,37.52148299700008],[40.464743,37.61741849800006],[40.639687,37.62242899800003],[40.68667850000003,37.73295399800003],[40.89438750000005,37.65617199800005],[41.045006,37.71503049800003],[41.08888950000005,37.758909998000036],[40.995091,37.82686299900007],[41.25448,38.13951949900007],[41.180091,38.25693699800007],[41.189474,38.31202699700003],[41.38054999900004,38.492253497000036],[41.506182,38.564977998000074],[41.648455,38.49419399800007],[41.702295,38.24654299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC31","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Mardin","NUTS_NAME":"Mardin"},"geometry":{"type":"Polygon","coordinates":[[[41.045006,37.71503049800003],[41.15804550000007,37.49862449700004],[41.478503,37.501679496000065],[41.851014500000076,37.66463249700007],[41.88327950000007,37.57231899800007],[41.836755,37.465827998000066],[41.61087149900004,37.415159499000026],[41.572054500000036,37.22637399900003],[41.671568,37.23702049700006],[41.663342,37.102903498000046],[41.216382,37.07248999800004],[40.770821,37.11804999800006],[40.226033,36.901373498000055],[40.109505,37.002485498000055],[40.11830850000007,37.119650998000054],[39.85554950000005,37.531011997000064],[40.331074999000066,37.52148299700008],[40.464743,37.61741849800006],[40.639687,37.62242899800003],[40.68667850000003,37.73295399800003],[40.89438750000005,37.65617199800005],[41.045006,37.71503049800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC32","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Batman","NUTS_NAME":"Batman"},"geometry":{"type":"Polygon","coordinates":[[[41.702295,38.24654299800005],[41.64290450000004,38.12968499800007],[41.48672850000003,38.066767998000046],[41.39499650000005,37.963971997000044],[41.70631,37.85089799800005],[41.691440500000056,37.72687149700005],[41.821212,37.727032499000074],[41.851014500000076,37.66463249700007],[41.478503,37.501679496000065],[41.15804550000007,37.49862449700004],[41.045006,37.71503049800003],[41.08888950000005,37.758909998000036],[40.995091,37.82686299900007],[41.25448,38.13951949900007],[41.180091,38.25693699800007],[41.189474,38.31202699700003],[41.38054999900004,38.492253497000036],[41.506182,38.564977998000074],[41.648455,38.49419399800007],[41.702295,38.24654299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC33","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Sirnak","NUTS_NAME":"Sirnak"},"geometry":{"type":"Polygon","coordinates":[[[43.50539050000003,37.71352999700008],[43.494746,37.64243299800006],[43.358446,37.55748599800006],[43.362341,37.47377049800008],[43.307242,37.43826649700003],[43.424563,37.27533449900005],[43.156937,37.37387799800007],[42.924164,37.32776699800007],[42.786801,37.38367499800006],[42.57950599900005,37.15367499800004],[42.35568250000006,37.10799350000008],[42.351521,37.23116999900003],[42.180832,37.29054199800004],[42.05693099900003,37.19277099900006],[41.663342,37.102903498000046],[41.671568,37.23702049700006],[41.572054500000036,37.22637399900003],[41.61087149900004,37.415159499000026],[41.836755,37.465827998000066],[41.88327950000007,37.57231899800007],[41.851014500000076,37.66463249700007],[41.941865,37.59998299800003],[42.26554550000003,37.58815949700005],[42.43476750000008,37.71360750000008],[42.59270850000007,37.747483998000064],[42.86155449900008,37.63603999800006],[42.96620850000005,37.76213299900007],[43.50539050000003,37.71352999700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"TRC34","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Siirt","NUTS_NAME":"Siirt"},"geometry":{"type":"Polygon","coordinates":[[[42.767993500000046,37.913629497000045],[42.896664,37.91730899800007],[42.96620850000005,37.76213299900007],[42.86155449900008,37.63603999800006],[42.59270850000007,37.747483998000064],[42.43476750000008,37.71360750000008],[42.26554550000003,37.58815949700005],[41.941865,37.59998299800003],[41.851014500000076,37.66463249700007],[41.821212,37.727032499000074],[41.691440500000056,37.72687149700005],[41.70631,37.85089799800005],[41.39499650000005,37.963971997000044],[41.48672850000003,38.066767998000046],[41.64290450000004,38.12968499800007],[41.702295,38.24654299800005],[41.998573,38.12600249800005],[42.20442950000006,38.18850499800004],[42.672305,37.862262498000064],[42.767993500000046,37.913629497000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"UK","LEVL_CODE":0,"CNTR_CODE":"UK","NAME_LATN":"UNITED KINGDOM","NUTS_NAME":"UNITED KINGDOM"},"geometry":{"type":"MultiPolygon","coordinates":[[[[0.854097374000048,51.601599640000074],[0.958179500000028,51.61968999900006],[0.821244,51.54071449700007],[0.626791,51.53217299800008],[0.51370350000002,51.53118499700008],[0.54146,51.51271449800004],[0.401860592000048,51.45663058400004],[0.458900500000027,51.45493699800005],[0.720437,51.45972799800006],[0.539261022000062,51.40701982600007],[0.626894500000049,51.37520999900005],[0.817462036000052,51.37265732800006],[0.724821,51.41263199700006],[0.743699,51.44689949800005],[0.951056,51.373244998000075],[0.855278181000074,51.36499536100007],[0.950297500000033,51.34574899900008],[1.44974650000006,51.37717799700005],[1.37071,51.31177899700003],[1.404974500000037,51.22982049700005],[1.379638500000056,51.14215849900006],[1.034675,51.04708849800005],[0.977051500000073,51.002594],[0.98057299900006,50.914032],[0.854784,50.92370999800005],[0.377511,50.820377499000074],[0.240893,50.734935998000026],[-0.038176499999963,50.799503498000036],[-0.216009,50.82756399600004],[-0.787529499999948,50.72245399900004],[-0.916800499999965,50.780898997000065],[-0.932831,50.843150999000045],[-1.021779506999962,50.83572701200006],[-1.09147,50.77824399600007],[-1.116579,50.842498997000064],[-1.156856,50.838321499000074],[-1.111877499999935,50.789558499000066],[-1.138110499999925,50.773792498000034],[-1.365159499999947,50.88006949600003],[-1.477037499999938,50.92395799600007],[-1.306856,50.81985449800004],[-1.577171499999963,50.716056999000045],[-1.692049499999939,50.736625497000034],[-1.74072549999994,50.721538499000076],[-2.0406,50.71996299800003],[-2.080504,50.68798449900004],[-1.967877,50.66637399600006],[-1.953566000999956,50.59413149800008],[-2.423359499999947,50.63589499900007],[-2.470142499999952,50.58435449900003],[-2.415669499999979,50.54917899800006],[-2.456731499999933,50.51285949900006],[-2.450997,50.56172549900003],[-2.652679499999977,50.669887499000026],[-2.947815000999981,50.71830750000004],[-3.38546,50.606719998000074],[-3.469847,50.68223549700008],[-3.443591499999968,50.602722],[-3.509030499999938,50.516514],[-3.488737499999957,50.46379099600006],[-3.559090499999968,50.42766199700003],[-3.482395499999939,50.39991799900008],[-3.507436,50.378989998000065],[-3.723704,50.20151899900003],[-4.12313249999994,50.34674049700004],[-4.177532637999946,50.36402146800003],[-4.187758,50.31752399800007],[-4.386875,50.36486449900008],[-4.76328,50.32623649700008],[-4.813982,50.22931649700007],[-5.008761499999935,50.13920949800007],[-5.032680849999963,50.20807098900008],[-5.076777,50.13928599700006],[-5.05757049999994,50.052741997000055],[-5.185075499999925,49.963165496000045],[-5.29696649999994,50.07032799800004],[-5.479827,50.126014498000075],[-5.577235499999972,50.05170799900003],[-5.71682149999998,50.06912599800006],[-5.670151499999974,50.167129498000065],[-5.434389499999952,50.19229899700008],[-5.15565,50.34718299600007],[-5.04049,50.44879549700005],[-5.026513,50.541625999000075],[-4.845801,50.52435950000006],[-4.936013499999945,50.587012998000034],[-4.796273499999927,50.59804899800008],[-4.562391499999933,50.78140999900006],[-4.545959499999981,50.92835249800004],[-4.526117499999941,51.02189249600008],[-4.345269,50.988857497000026],[-4.139783840999939,51.077219299000035],[-4.21529449999997,51.07535549600004],[-4.200037000999941,51.20093899900007],[-3.720778499999938,51.23309349800007],[-3.313210500999958,51.179561498000055],[-3.019771499999933,51.21881099700005],[-3.05087,51.18717599900003],[-2.999351,51.23696899600003],[-3.035921499999972,51.32854849600005],[-2.992948,51.32031999800006],[-2.980789,51.38841999600004],[-2.797451499999966,51.48601149700005],[-2.679612,51.48021299800007],[-2.716312499999958,51.50710699900003],[-2.674005499999964,51.54427349800005],[-2.534737,51.677245998000046],[-2.379470499999968,51.767123998000045],[-2.660656919999951,51.62766319500008],[-3.082688,51.50190749800004],[-3.16959049999997,51.40624999900007],[-3.340482,51.38127499600006],[-3.558956,51.401328998000054],[-3.637855499999944,51.46984099700006],[-3.88623,51.61738599700004],[-4.209631,51.53722749600007],[-4.307776,51.61030199800007],[-4.05255,51.703281497000035],[-4.287212,51.66770149900003],[-4.382688,51.72815699700004],[-4.369891792999965,51.77438571400006],[-4.661121499999979,51.73165499800007],[-4.711614500999929,51.65161149800008],[-4.929975499999955,51.59847650000006],[-5.12474549999996,51.68109499900004],[-4.898221312999965,51.71056675600005],[-5.187262000999965,51.70885449800005],[-5.254024,51.73765199800005],[-5.11745,51.768268498000054],[-5.139965500999949,51.86383049600005],[-5.320248499999934,51.86160299800008],[-5.086008,51.968773],[-5.07227649999993,52.03031299700007],[-4.878756,52.019611499000064],[-4.20750450099996,52.26382049700004],[-4.040780499999926,52.52494449800008],[-3.931013499999949,52.553611999000054],[-4.070163,52.54323949900004],[-4.128679,52.61278149900005],[-3.938924000999975,52.74887849900006],[-4.060166,52.720512496000026],[-4.150184499999966,52.80757149600004],[-4.058637499999975,52.920723],[-4.407304000999943,52.89085749700007],[-4.531266499999958,52.77798449800008],[-4.767578499999956,52.79495999900007],[-4.353505,53.03351599900003],[-4.333707,53.11389899800008],[-4.216110416999925,53.19851737000005],[-4.329113,53.12664049700004],[-4.69663,53.30674349800006],[-4.575122,53.28789149800008],[-4.574172,53.40379349700004],[-4.404923,53.42721949600008],[-4.275177500999973,53.403098998000075],[-4.213657,53.29817199800004],[-4.040188,53.31065349800008],[-4.196792116999973,53.21081699300004],[-4.007376,53.246928998000044],[-3.845432499999959,53.29659249900004],[-3.865022,53.34220099700008],[-3.609629499999926,53.29049299600007],[-3.363392499999975,53.35202799900003],[-3.019001,53.21184549800006],[-3.084193,53.256122497000035],[-3.110706,53.29631799800006],[-3.20036750099996,53.38752749900004],[-3.040308,53.44292849800007],[-2.928556499999956,53.30827699900004],[-2.752411499999937,53.31475449900006],[-2.675166499999932,53.35448449900008],[-2.693358,53.36183549700007],[-2.826677,53.33167250000008],[-3.008742,53.43841149800005],[-3.105458499999941,53.551548],[-2.956203,53.697529],[-2.833743,53.72212999800007],[-2.757161000999929,53.75646949900005],[-3.057371499999931,53.77648149800007],[-3.047940499999925,53.875774498000055],[-2.853505,53.96575949800007],[-2.924294499999974,54.031154496000056],[-2.794437,54.12753699800004],[-2.86947,54.17673849800008],[-2.789148499999953,54.22734049700006],[-3.016447,54.13262549700005],[-3.03217,54.245723499000064],[-3.149062,54.093600999000046],[-3.241605000999925,54.10834499900005],[-3.256430499999965,54.16888450000005],[-3.204951499999936,54.21513],[-3.223693608999952,54.249058382000044],[-3.330026499999974,54.19639199900007],[-3.613598,54.52509699800004],[-3.578772499999957,54.651966],[-3.433302,54.76629649800003],[-3.398636499999952,54.86912549800007],[-3.251950499999964,54.90313699900008],[-3.314172499999927,54.91571799900004],[-3.285158,54.94161599800003],[-3.11915,54.927886998000076],[-3.00619949999998,54.94408049700007],[-3.131486,54.94579299700007],[-3.091623992999928,54.97554629100006],[-3.560086499999954,54.97845099800003],[-3.594912499999964,54.872833498000034],[-3.850849,54.86328899700004],[-3.824464499999976,54.82371149800008],[-3.981397,54.76880249700008],[-4.06509049999994,54.78116249700008],[-4.050716,54.849548499000036],[-4.105112499999962,54.76763549900005],[-4.425736,54.926769499000045],[-4.413721499999951,54.82943750000004],[-4.341246499999954,54.79918649600006],[-4.347857499999975,54.71093749700003],[-4.405439499999943,54.67750950000004],[-4.860413,54.86895749900003],[-4.960747,54.80627449700006],[-4.876391499999954,54.63921349900005],[-4.964987,54.66406249900007],[-4.957434499999977,54.72864899900003],[-5.183450000999926,54.912364998000044],[-5.161495,55.00796149900003],[-5.098928000999933,55.01810849900005],[-5.060561,54.92361050000005],[-4.990791499999943,54.92166149600007],[-5.040228499999955,54.997726500000056],[-5.048733499999969,55.05751799600006],[-4.859926,55.227607499000044],[-4.768847499999936,55.400829498000064],[-4.650890499999946,55.441147],[-4.620535,55.49978649600007],[-4.658272499999953,55.57028949800008],[-4.905256,55.69921899800005],[-4.855803499999979,55.760661998000046],[-4.888966499999981,55.87478999600006],[-4.878924499999926,55.94240199700005],[-4.765130499999941,55.957771499000046],[-4.392671740999958,55.88981584100003],[-4.609648,55.94667449700006],[-4.828885,56.07900999700007],[-4.766975,55.98970399800004],[-4.853408499999944,55.99036049800003],[-4.880223,56.058253998000055],[-4.752816000999928,56.20385749700006],[-4.862619499999937,56.100319],[-4.917712,56.164127499000074],[-4.896133499999962,55.98607999600006],[-4.979976,55.86157999800008],[-5.120735,56.00808349700003],[-5.074320499999942,55.90140149900003],[-5.190108,55.94739149900005],[-5.242238,55.89447799800007],[-5.205963,55.82628650000004],[-5.351127,55.90980549800008],[-5.34193149999993,55.995186],[-5.127448198999957,56.166530160000036],[-5.380715,56.00484449900006],[-5.440771499999926,56.03461450000003],[-5.419614,55.896728497000026],[-5.314025,55.783824999000046],[-5.448947,55.707809498000074],[-5.490266,55.52957549800004],[-5.568165000999954,55.42831399700003],[-5.52000449999997,55.36192299700008],[-5.799178,55.299698],[-5.66694,55.66451649900006],[-5.436028,55.85756299800005],[-5.613714,55.76017749700003],[-5.666273499999932,55.800583],[-5.571037499999932,55.93432999800007],[-5.688422499999945,55.911018496000054],[-5.566499,56.04157249900004],[-5.705351,55.93992999800008],[-5.503709499999957,56.18886549800004],[-5.613348,56.133239497000034],[-5.567006,56.23656099900006],[-5.487101000999928,56.256931500000064],[-5.654321,56.297649496000076],[-5.441643,56.365916997000056],[-5.589511,56.38047799600008],[-5.441085499999929,56.455798998000034],[-5.259811117999959,56.450957274000075],[-5.473969,56.481708499000035],[-5.384345607999933,56.53174223900004],[-5.421163,56.54778649900004],[-5.325605499999938,56.648285],[-4.969490499999949,56.714679498000066],[-5.247245499999963,56.70311749700005],[-5.092236,56.83002449600008],[-5.334743499999945,56.859508497000036],[-5.122688000999972,56.827274498000065],[-5.684919,56.497295499000074],[-6.006905,56.645126497000035],[-5.867277643999955,56.675698871000066],[-6.226232,56.710991],[-5.751703000999953,56.78216549700005],[-5.887583499999948,56.792953999000076],[-5.722717500999977,56.889038],[-5.924258,56.89156349700005],[-5.820707,57.01260749700003],[-5.50935149999998,56.99639499700004],[-5.797323,57.06098549700005],[-5.72895,57.10147849700007],[-5.509437499999933,57.097774498000035],[-5.681389,57.15053049900007],[-5.670602562999932,57.20843875300005],[-6.017994,57.01794799800007],[-6.004551,57.114955999000074],[-5.908913,57.17123799900003],[-6.32237,57.160293498000044],[-6.483495,57.310973999000055],[-6.307433,57.299278497000046],[-6.789714499999945,57.42120749800006],[-6.716572,57.51391599900006],[-6.582203,57.423621996000065],[-6.637269499999945,57.50403199800007],[-6.562680499999942,57.504444],[-6.638996499999962,57.549937998000075],[-6.635986499999944,57.60881799900005],[-6.406945,57.523063498000056],[-6.427763499999969,57.64247149900007],[-6.296708,57.70756149700003],[-6.146597000999975,57.57447449700004],[-6.130341,57.316936498000075],[-5.88541349999997,57.238167],[-5.776867,57.27114499700008],[-5.64761249999998,57.25529499700008],[-5.664395153999976,57.219288446000064],[-5.415392,57.23015599900003],[-5.73379,57.283454996000046],[-5.441254,57.418952997000076],[-5.82202,57.363647496000056],[-5.872894,57.47562050000005],[-5.841507499999977,57.57624799900003],[-5.513192499999946,57.54019549900005],[-5.81922,57.639786],[-5.673280499999976,57.70169849900003],[-5.813466499999947,57.75067150000007],[-5.81416349999995,57.85871099900004],[-5.683442,57.86545199900007],[-5.619121,57.766895497000064],[-5.577558499999952,57.79293449800008],[-5.655929499999957,57.87793349800006],[-5.618142,57.92462899800006],[-5.217599,57.84221649700004],[-5.406790499999943,57.92776849900008],[-5.360826000999964,57.93917449800006],[-5.071033000999932,57.81991949700006],[-5.458675,58.074409495000054],[-5.268833,58.10244799800006],[-5.403668,58.23530949700006],[-5.375456,58.26508699900006],[-4.924184000999958,58.25644699700007],[-5.134424,58.27807599700003],[-5.182866499999932,58.35124949800007],[-5.033927499999947,58.39294799700008],[-5.114617499999952,58.415272],[-5.072923,58.452243999000075],[-4.988059499999963,58.425987],[-5.126430499999969,58.48945599700005],[-5.007117499999936,58.625992],[-4.65146249999998,58.534366498000054],[-4.739715499999932,58.44739149800006],[-4.589487499999962,58.57794549700003],[-4.409354499999949,58.523773],[-4.475344,58.44438949700003],[-4.357074,58.536857498000074],[-3.024449,58.644264],[-3.130931,58.519863],[-3.046968499999934,58.47255350000006],[-3.116977499999962,58.368579999000076],[-4.082477499999925,57.949943500000074],[-4.001344,57.92654399800006],[-4.016409,57.85973750000005],[-4.400494,57.918987497000046],[-4.04890249999994,57.81504049600005],[-3.771491499999968,57.867034998000065],[-3.975255499999946,57.69451899900008],[-4.301044,57.67506799700004],[-4.428933,57.574283499000046],[-3.993433499999981,57.67645649800005],[-4.234324499999957,57.500965],[-4.432052499999941,57.494354],[-4.191771,57.48384099900005],[-4.019451499999946,57.598220999000034],[-3.347729,57.72444899900006],[-2.801509,57.69524749900006],[-1.920823499999926,57.67439649600004],[-1.78944,57.503555497000036],[-2.042654,57.243911498000045],[-2.077448499999946,57.17697899800004],[-2.05034749999993,57.128338],[-2.206539500999952,56.96594999700005],[-2.214603,56.88209899700007],[-2.425346,56.755187996000075],[-2.538641,56.566062997000074],[-2.733122499999979,56.46479799900004],[-3.051892500999941,56.45849999700005],[-3.345321499999955,56.37197099600007],[-3.276975499999935,56.35053250000004],[-2.803087,56.43967799700005],[-2.838920499999972,56.35307299600004],[-2.620387,56.26118849900007],[-2.955285,56.21049499700007],[-3.39145149999996,56.00593199700006],[-3.866839499999969,56.12066249900005],[-3.820153,56.099117500000034],[-3.515724,56.00225850000004],[-3.425330499999973,55.99403],[-3.07764549999996,55.94689549800006],[-2.815697499999942,56.062404496000056],[-2.366623499999946,55.946113498000045],[-2.13755649999996,55.91727849600005],[-2.034329,55.81116499900003],[-1.838949499999956,55.64233399900007],[-1.639348,55.57831949700005],[-1.570851,55.275554500000055],[-1.503029,55.196361499000034],[-1.526808499999959,55.13655849700007],[-1.461693,55.07439049900006],[-1.36390149999994,54.944183499000076],[-1.347372499999949,54.86069099800005],[-1.242229499999951,54.722594997000044],[-1.17516749999993,54.696644],[-1.158751,54.617881999000076],[-1.261252,54.57190299700005],[-1.118690499999957,54.62907799900006],[-0.790906499999949,54.55948249900007],[-0.520867,54.44644949900004],[-0.396518499999956,54.27443699600008],[-0.212449499999934,54.157627],[-0.076264499999979,54.11638649900004],[-0.198329499999943,54.077949498000066],[-0.208244,53.99762349800005],[0.141740262000042,53.61067700600006],[-0.103462,53.63536049800007],[-0.250096,53.73331850000005],[-0.419136500999969,53.71961999600006],[-0.617187,53.73120499800007],[-0.718628381999963,53.702292244000034],[-0.698366,53.684653998000044],[-0.694836,53.69454949900006],[-0.295713499999977,53.71386699800007],[0.017378,53.52536750000007],[0.163623,53.482135999000036],[0.355712,53.19206249800004],[0.339612,53.09699249800008],[0.043715500000076,52.90388099900008],[0.26596,52.810116],[0.367146500000047,52.79617299600005],[0.490182,52.947913999000036],[0.695918,52.98762899700006],[1.343277,52.92004399700005],[1.675478,52.74269099800006],[1.743963,52.62939049700003],[1.740543500000058,52.53210049900008],[1.756430500000022,52.47172549700008],[1.596105927000053,52.14379643900003],[1.595382,52.14189149900005],[1.391530924000051,51.98931566500005],[1.317633500000056,51.94887150000005],[1.160663500000055,52.04616549900004],[1.27915249900002,51.96037649800007],[1.05638,51.95149249700006],[1.280599,51.940426],[1.291964500000063,51.87065099900008],[1.124435,51.77542499700007],[0.844628,51.781318498000076],[0.894665,51.76629249700005],[0.688036500000067,51.731712498000036],[0.71307,51.71509149900004],[0.756304,51.691741999000044],[0.94589650000006,51.71621299700007],[0.926602500000058,51.628940500000056],[0.776491999000029,51.63602449800004],[0.764340716000049,51.63674747400006],[0.854097374000048,51.601599640000074]]],[[[-0.75837,60.81646749600003],[-0.831153,60.685222499000076],[-0.981795499999976,60.685482],[-0.865963499999964,60.83370199700005],[-0.75837,60.81646749600003]]],[[[-1.003055499999959,60.58155799700006],[-1.028645,60.49721549700007],[-1.167948499999966,60.49954999900007],[-1.202889,60.607105499000056],[-1.074501,60.73317349700005],[-1.008528,60.71493899900008],[-0.981463500999951,60.637199500000065],[-1.031584500999941,60.59758399900005],[-1.003055499999959,60.58155799700006]]],[[[-1.304159,60.62399649900004],[-1.319979499999931,60.451770999000075],[-1.383588,60.397674496000036],[-1.237423000999968,60.49381999700006],[-1.049048,60.43949149600007],[-1.155833499999972,60.33803549900006],[-1.079679499999941,60.32329199900005],[-1.15929,60.281048],[-1.095383499999969,60.26348899900006],[-1.170235,60.23842599900007],[-1.155375499999934,60.15899649800008],[-1.203963500999976,60.130611499000054],[-1.277545,59.85259999700003],[-1.391431500999943,59.91602349900006],[-1.276649,60.099434],[-1.32783749999993,60.16605749900003],[-1.293532499999969,60.251404],[-1.462364499999978,60.14639299900006],[-1.702932,60.255359498000075],[-1.339904500999978,60.359771497000054],[-1.632932,60.48316199900006],[-1.499491318999958,60.543659871000045],[-1.489197784999931,60.54412077600006],[-1.304159,60.62399649900004]]],[[[-1.069896,50.683647],[-1.303737499999954,50.57604599700005],[-1.586375,50.66316599600003],[-1.292164499999956,50.75799949800006],[-1.069896,50.683647]]],[[[-2.848329500999967,58.958244499000045],[-2.788509000999966,58.915839995000056],[-2.704751499999929,58.961914],[-2.712558499999943,58.92079149700004],[-2.897167053999965,58.89524031000008],[-3.368161499999928,58.99882899900007],[-3.325062,59.13616199900008],[-3.196686,59.15369050000004],[-3.004050499999948,59.07222],[-3.11450150099995,59.00661450000007],[-2.911924499999941,59.006061498000065],[-2.801162499999975,58.99209999800007],[-2.848329500999967,58.958244499000045]]],[[[-3.259673,58.78805149900006],[-3.219864,58.775715],[-3.293498,58.77659249900006],[-3.434554499999933,58.87152850000007],[-3.32561149999998,58.92784899900005],[-3.170331,58.82104099800006],[-3.259673,58.78805149900006]]],[[[-5.082880499999931,55.55278799800004],[-5.083631,55.45461249700003],[-5.313261,55.46509149600007],[-5.39711000099993,55.61743149700004],[-5.288532499999974,55.71808599700006],[-5.201502499999947,55.702484],[-5.082880499999931,55.55278799800004]]],[[[-5.976527499999975,55.056598500000064],[-5.992633499999954,54.989261998000075],[-5.721779,54.77285249600004],[-5.720909000999939,54.772636498000054],[-5.780185,54.858516500000064],[-5.691673,54.80959299800003],[-5.690664000999959,54.769462496000074],[-5.912931499999956,54.64803699700008],[-5.855336,54.63376999900004],[-5.575003,54.67019650000003],[-5.432802,54.48764399700008],[-5.499689499999931,54.338054498000076],[-5.579466499999967,54.40056599800005],[-5.536291499999948,54.431365999000036],[-5.561936,54.53355049900006],[-5.679648499999928,54.58318649800003],[-5.724728499999969,54.53908899800007],[-5.643023,54.51126499700007],[-5.65442,54.381484998000076],[-5.542285,54.32002999800005],[-5.874409,54.184848500000044],[-5.896720499999958,54.10493099900003],[-6.062905,54.02288050000004],[-6.268015499999933,54.102336999000045],[-6.623778499999958,54.03654849600008],[-6.668772499999932,54.07254799900005],[-6.643909,54.180019497000046],[-6.79513849999995,54.20911399800008],[-6.910962499999926,54.37472549900008],[-7.02863549999995,54.421306497000046],[-7.211874499999965,54.299679],[-7.145823499999949,54.22494499900006],[-7.248977,54.20340750000008],[-7.279176499999949,54.12441650000005],[-7.290923499999963,54.17190949600007],[-7.565956,54.12651449800006],[-7.855390499999942,54.22252249600007],[-7.872462499999926,54.29415499800007],[-8.177718,54.46497350000004],[-7.703411500999948,54.60828799800004],[-7.921372,54.69654449800004],[-7.534506,54.747139],[-7.548892,54.789589498000055],[-7.44311,54.87147149800006],[-7.396395499999926,55.016722997000045],[-7.256068499999969,55.067034997000064],[-7.072085,55.046199999000066],[-6.960326,55.17870349900005],[-6.455277,55.23930349900007],[-6.139729,55.22702049900005],[-6.027018,55.16146449800004],[-6.057595499999934,55.06350699700005],[-5.976527499999975,55.056598500000064]]],[[[-5.646332,56.44055950000006],[-5.834676,56.310558496000056],[-6.385666500999946,56.28878799700004],[-6.35029,56.34788899800003],[-6.190255,56.332347998000046],[-5.979565499999978,56.38647099800005],[-6.194926499999951,56.358978499000045],[-6.20478049999997,56.38530749700004],[-6.129291,56.448417498000026],[-5.996329499999945,56.48778549900004],[-6.340745,56.538177500000074],[-6.323570500999949,56.60616299700007],[-6.112694000999966,56.64403899900003],[-5.950804500999936,56.518039498000064],[-5.799656500999959,56.51579649700005],[-5.646332,56.44055950000006]]],[[[-5.793228499999941,56.012721999000064],[-5.96267649899994,55.794749997000054],[-6.09684249999998,55.871860498000046],[-5.924639498999966,55.962383499000055],[-6.006308,55.973919],[-5.964173,56.027405],[-5.687118998999949,56.12949350000008],[-5.793228499999941,56.012721999000064]]],[[[-6.101226499999939,55.84050350000007],[-6.01987,55.68493249900007],[-6.266864000999931,55.579871996000065],[-6.335771499999964,55.58950049800006],[-6.260210499999971,55.659263500000066],[-6.341544499999941,55.71799849700005],[-6.260346499999969,55.781940500000076],[-6.526495499999953,55.693542499000046],[-6.45583,55.85268399800003],[-6.120203,55.934597],[-6.101226499999939,55.84050350000007]]],[[[-6.167010499999947,58.34677099900006],[-6.344816,58.234204496000075],[-6.135358,58.25944149700007],[-6.601655500999925,58.08697899600003],[-6.368577499999958,58.07648849700007],[-6.388316,57.999859],[-6.573711,58.004272498000034],[-6.454871,57.99351499900007],[-6.469132499999944,57.939422499000045],[-6.652290499999935,57.91831599800008],[-6.712528,58.013172],[-6.588023500999952,58.05318850000003],[-6.69152,58.05827699800005],[-6.739541,57.991840496000066],[-6.663898,57.88521599900008],[-6.812832499999956,57.880672496000045],[-6.744717,57.82280749800003],[-6.967598,57.727894],[-7.133195499999943,57.836654500000066],[-6.81601349999994,57.900836998000045],[-7.079999499999929,57.967232],[-7.082464,58.02035899700007],[-6.907952,58.049675],[-7.103295000999935,58.07354749900003],[-7.108741499999951,58.177036498000064],[-7.027442,58.24435399600003],[-6.746137499999975,58.18470399700004],[-6.828675,58.284096],[-6.25831,58.516716],[-6.180591,58.46701049900008],[-6.217891499999951,58.36830499600006],[-6.167010499999947,58.34677099900006]]],[[[-7.182542499999954,57.65663150000006],[-7.062166,57.64057150000008],[-7.214868499999966,57.64065949800005],[-7.10083149999997,57.59365449900008],[-7.299847,57.553420997000046],[-7.133844,57.552966998000045],[-7.148515,57.50972349700004],[-7.549144499999954,57.60290549900003],[-7.152460499999961,57.665553997000075],[-7.182542499999954,57.65663150000006]]],[[[-7.35166,57.15094749700006],[-7.211596499999928,57.11778249900004],[-7.412508499999944,57.149811],[-7.412302499999953,57.38740150000007],[-7.19152,57.29863750000004],[-7.298812670999951,57.22601419200004],[-7.246141,57.16512699800006],[-7.35166,57.15094749700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"NORTH EAST (ENGLAND)","NUTS_NAME":"NORTH EAST (ENGLAND)"},"geometry":{"type":"Polygon","coordinates":[[[-1.158751,54.617881999000076],[-1.261252,54.57190299700005],[-1.118690499999957,54.62907799900006],[-0.790906499999949,54.55948249900007],[-0.952505,54.488063999000076],[-1.234791,54.51036849700006],[-1.434778,54.48751449800005],[-1.696865,54.536060500000076],[-2.170166499999937,54.45825599900007],[-2.172482499999944,54.53248199700005],[-2.351850499999955,54.68591699600006],[-2.31204300099995,54.79108049900003],[-2.567809,54.82364249900007],[-2.605391499999939,54.88444899800004],[-2.545466499999975,54.92846299900003],[-2.601320500999975,54.97138999600003],[-2.48629649999998,55.08311849800003],[-2.689751,55.18906399700006],[-2.496131499999933,55.346595999000044],[-2.165465499999925,55.46846750000003],[-2.335961,55.63214499800006],[-2.034329,55.81116499900003],[-1.838949499999956,55.64233399900007],[-1.639348,55.57831949700005],[-1.570851,55.275554500000055],[-1.503029,55.196361499000034],[-1.526808499999959,55.13655849700007],[-1.461693,55.07439049900006],[-1.36390149999994,54.944183499000076],[-1.347372499999949,54.86069099800005],[-1.242229499999951,54.722594997000044],[-1.17516749999993,54.696644],[-1.158751,54.617881999000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Tees Valley and Durham","NUTS_NAME":"Tees Valley and Durham"},"geometry":{"type":"Polygon","coordinates":[[[-1.559341500999949,54.88210299800005],[-1.493438,54.80006049800005],[-1.347372499999949,54.86069099800005],[-1.242229499999951,54.722594997000044],[-1.17516749999993,54.696644],[-1.158751,54.617881999000076],[-1.261252,54.57190299700005],[-1.118690499999957,54.62907799900006],[-0.790906499999949,54.55948249900007],[-0.952505,54.488063999000076],[-1.234791,54.51036849700006],[-1.434778,54.48751449800005],[-1.696865,54.536060500000076],[-2.170166499999937,54.45825599900007],[-2.172482499999944,54.53248199700005],[-2.351850499999955,54.68591699600006],[-2.31204300099995,54.79108049900003],[-1.820951,54.90572749900008],[-1.559341500999949,54.88210299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Hartlepool and Stockton-on-Tees","NUTS_NAME":"Hartlepool and Stockton-on-Tees"},"geometry":{"type":"Polygon","coordinates":[[[-1.261252,54.57190299700005],[-1.284184499999981,54.528625497000064],[-1.234791,54.51036849700006],[-1.434778,54.48751449800005],[-1.407287499999939,54.593818498000076],[-1.438286499999947,54.59513449900004],[-1.242229499999951,54.722594997000044],[-1.17516749999993,54.696644],[-1.158751,54.617881999000076],[-1.261252,54.57190299700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South Teesside","NUTS_NAME":"South Teesside"},"geometry":{"type":"Polygon","coordinates":[[[-0.790906499999949,54.55948249900007],[-0.952505,54.488063999000076],[-1.234791,54.51036849700006],[-1.284184499999981,54.528625497000064],[-1.261252,54.57190299700005],[-1.118690499999957,54.62907799900006],[-0.790906499999949,54.55948249900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Darlington","NUTS_NAME":"Darlington"},"geometry":{"type":"Polygon","coordinates":[[[-1.438286499999947,54.59513449900004],[-1.407287499999939,54.593818498000076],[-1.434778,54.48751449800005],[-1.696865,54.536060500000076],[-1.682361000999947,54.61782850000003],[-1.438286499999947,54.59513449900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Durham CC","NUTS_NAME":"Durham CC"},"geometry":{"type":"Polygon","coordinates":[[[-1.559341500999949,54.88210299800005],[-1.493438,54.80006049800005],[-1.347372499999949,54.86069099800005],[-1.242229499999951,54.722594997000044],[-1.438286499999947,54.59513449900004],[-1.682361000999947,54.61782850000003],[-1.696865,54.536060500000076],[-2.170166499999937,54.45825599900007],[-2.172482499999944,54.53248199700005],[-2.351850499999955,54.68591699600006],[-2.31204300099995,54.79108049900003],[-1.820951,54.90572749900008],[-1.559341500999949,54.88210299800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Northumberland and Tyne and Wear","NUTS_NAME":"Northumberland and Tyne and Wear"},"geometry":{"type":"Polygon","coordinates":[[[-1.36390149999994,54.944183499000076],[-1.347372499999949,54.86069099800005],[-1.493438,54.80006049800005],[-1.559341500999949,54.88210299800005],[-1.820951,54.90572749900008],[-2.31204300099995,54.79108049900003],[-2.567809,54.82364249900007],[-2.605391499999939,54.88444899800004],[-2.545466499999975,54.92846299900003],[-2.601320500999975,54.97138999600003],[-2.48629649999998,55.08311849800003],[-2.689751,55.18906399700006],[-2.496131499999933,55.346595999000044],[-2.165465499999925,55.46846750000003],[-2.335961,55.63214499800006],[-2.034329,55.81116499900003],[-1.838949499999956,55.64233399900007],[-1.639348,55.57831949700005],[-1.570851,55.275554500000055],[-1.503029,55.196361499000034],[-1.526808499999959,55.13655849700007],[-1.461693,55.07439049900006],[-1.36390149999994,54.944183499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR41","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Bursa, Eskisehir, Bilecik","NUTS_NAME":"Bursa, Eskisehir, Bilecik"},"geometry":{"type":"Polygon","coordinates":[[[29.924952,40.551626498000076],[29.964417,40.52208449800003],[30.014735,40.41825399800007],[30.186187500000074,40.36268099700004],[30.419072500000027,40.378578498000024],[30.421594500000026,40.32574999800005],[30.521932,40.29434150000003],[30.650090500000033,40.144735998000044],[30.82533,40.12578449800003],[30.921542,40.02736699600007],[31.676844,40.03524050000004],[31.78699050000006,39.88126549900005],[31.880758,39.87372349800006],[31.850136,39.84329899700003],[31.915188,39.82739399800005],[31.822506499000042,39.74352399800006],[32.030849,39.481336998000074],[31.994242,39.35153349800004],[32.069824,39.27607099700003],[31.81531750000005,39.128693498000075],[31.750522,39.07900349800008],[31.619732500000055,39.10305549800006],[31.389171,39.14805799700008],[31.303426,39.26033799800007],[31.192543,39.27911049700003],[30.74145550000003,39.126772998000035],[30.42949150000004,39.21761249800005],[30.315111,39.36209499800003],[30.36653,39.40263399600008],[30.355477,39.49900499800003],[30.230593500000055,39.57547949800005],[30.193454,39.65880449900004],[30.06013450000006,39.658640999000056],[29.760535,39.71012349700004],[29.742183500000067,39.87133399900006],[29.396933,39.901012499000046],[29.405343,39.809691996000026],[29.28185,39.73377549800006],[29.194093500000065,39.600573997000026],[28.968574,39.60067749800004],[28.793313500000068,39.61746949700006],[28.69535250000007,39.757907498000066],[28.515012,39.73130899700004],[28.265037500000062,39.87023399700007],[28.06755050000004,40.25313399800007],[28.164779,40.395612999000036],[28.980173,40.355935997000074],[29.154478,40.42537499800005],[28.991232500000024,40.46627349700003],[29.06757250000004,40.54533149800005],[29.410220500000037,40.550392498000065],[29.562798499000053,40.542847496000036],[29.710609500000032,40.60812549700006],[29.924952,40.551626498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR411","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bursa","NUTS_NAME":"Bursa"},"geometry":{"type":"Polygon","coordinates":[[[29.924952,40.551626498000076],[29.964417,40.52208449800003],[29.822871,40.43237249900005],[29.813636,40.26580899800007],[29.697423,40.13085799900006],[29.742183500000067,39.87133399900006],[29.396933,39.901012499000046],[29.405343,39.809691996000026],[29.28185,39.73377549800006],[29.194093500000065,39.600573997000026],[28.968574,39.60067749800004],[28.793313500000068,39.61746949700006],[28.69535250000007,39.757907498000066],[28.515012,39.73130899700004],[28.265037500000062,39.87023399700007],[28.06755050000004,40.25313399800007],[28.164779,40.395612999000036],[28.980173,40.355935997000074],[29.154478,40.42537499800005],[28.991232500000024,40.46627349700003],[29.06757250000004,40.54533149800005],[29.410220500000037,40.550392498000065],[29.562798499000053,40.542847496000036],[29.710609500000032,40.60812549700006],[29.924952,40.551626498000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR412","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Eskisehir","NUTS_NAME":"Eskisehir"},"geometry":{"type":"Polygon","coordinates":[[[30.82533,40.12578449800003],[30.921542,40.02736699600007],[31.676844,40.03524050000004],[31.78699050000006,39.88126549900005],[31.880758,39.87372349800006],[31.850136,39.84329899700003],[31.915188,39.82739399800005],[31.822506499000042,39.74352399800006],[32.030849,39.481336998000074],[31.994242,39.35153349800004],[32.069824,39.27607099700003],[31.81531750000005,39.128693498000075],[31.750522,39.07900349800008],[31.619732500000055,39.10305549800006],[31.389171,39.14805799700008],[31.303426,39.26033799800007],[31.192543,39.27911049700003],[30.74145550000003,39.126772998000035],[30.42949150000004,39.21761249800005],[30.315111,39.36209499800003],[30.36653,39.40263399600008],[30.355477,39.49900499800003],[30.230593500000055,39.57547949800005],[30.193454,39.65880449900004],[30.06013450000006,39.658640999000056],[29.994148,39.72334299900007],[30.036582500000065,39.80522799900007],[30.255979500000024,39.84272649900004],[30.42575750000003,39.95823549800008],[30.435256,40.030706999000074],[30.518736,40.10713849600006],[30.650090500000033,40.144735998000044],[30.82533,40.12578449800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR413","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bilecik","NUTS_NAME":"Bilecik"},"geometry":{"type":"Polygon","coordinates":[[[30.521932,40.29434150000003],[30.650090500000033,40.144735998000044],[30.518736,40.10713849600006],[30.435256,40.030706999000074],[30.42575750000003,39.95823549800008],[30.255979500000024,39.84272649900004],[30.036582500000065,39.80522799900007],[29.994148,39.72334299900007],[30.06013450000006,39.658640999000056],[29.760535,39.71012349700004],[29.742183500000067,39.87133399900006],[29.697423,40.13085799900006],[29.813636,40.26580899800007],[29.822871,40.43237249900005],[29.964417,40.52208449800003],[30.014735,40.41825399800007],[30.186187500000074,40.36268099700004],[30.419072500000027,40.378578498000024],[30.421594500000026,40.32574999800005],[30.521932,40.29434150000003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR42","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Kocaeli, Sakarya, Düzce, Bolu, Yalova","NUTS_NAME":"Kocaeli, Sakarya, Düzce, Bolu, Yalova"},"geometry":{"type":"Polygon","coordinates":[[[30.35468650000007,41.18419449800007],[30.982055500000058,41.07260250000007],[31.295656500000064,41.11631999900004],[31.424844,41.02525499600006],[31.75363,41.005629998000074],[32.134024,41.030703999000025],[32.274522,41.024024498000074],[32.325888500000076,40.88752399800006],[32.560043,40.807477498000026],[32.554816,40.69194399700007],[32.37547550000005,40.663500999000064],[32.396747,40.561117996000064],[31.911622,40.32874049700007],[31.590715500000044,40.27586449800003],[31.437988,40.34499449700007],[31.297481,40.31101649800007],[31.128547500000025,40.36527999800006],[31.065705,40.225254997000036],[30.82533,40.12578449800003],[30.650090500000033,40.144735998000044],[30.521932,40.29434150000003],[30.421594500000026,40.32574999800005],[30.419072500000027,40.378578498000024],[30.186187500000074,40.36268099700004],[30.014735,40.41825399800007],[29.964417,40.52208449800003],[29.924952,40.551626498000076],[29.710609500000032,40.60812549700006],[29.562798499000053,40.542847496000036],[29.410220500000037,40.550392498000065],[29.06757250000004,40.54533149800005],[28.991232500000024,40.46627349700003],[28.780161,40.53619199700006],[28.996286500000053,40.645887498000036],[29.517332,40.72897149700003],[29.546993,40.69543599900004],[29.942186500000048,40.750489998000035],[29.353948500000058,40.75541099800006],[29.342517500000042,40.807657498000026],[29.407457,40.95459],[29.849114,41.064824497000075],[29.865624500000024,41.14344749700007],[30.35468650000007,41.18419449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR421","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Kocaeli","NUTS_NAME":"Kocaeli"},"geometry":{"type":"Polygon","coordinates":[[[30.35468650000007,41.18419449800007],[30.254108,41.05421049600005],[30.312039,40.87832599800004],[30.234823499000072,40.83167899600005],[30.24338750000004,40.750796997000066],[30.19492,40.673701496000035],[29.924952,40.551626498000076],[29.710609500000032,40.60812549700006],[29.562798499000053,40.542847496000036],[29.410220500000037,40.550392498000065],[29.467295,40.662832999000045],[29.546993,40.69543599900004],[29.942186500000048,40.750489998000035],[29.353948500000058,40.75541099800006],[29.342517500000042,40.807657498000026],[29.407457,40.95459],[29.849114,41.064824497000075],[29.865624500000024,41.14344749700007],[30.35468650000007,41.18419449800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR422","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Sakarya","NUTS_NAME":"Sakarya"},"geometry":{"type":"Polygon","coordinates":[[[30.982055500000058,41.07260250000007],[30.967463500000065,40.96352899800007],[30.831471500000077,40.81762949700004],[30.831305,40.767607498000075],[30.934537,40.70447],[30.897045500000047,40.66165799800007],[30.866952,40.52088299900004],[30.628357,40.49151999700007],[30.559878,40.403381997000054],[30.580095,40.33931249700004],[30.521932,40.29434150000003],[30.421594500000026,40.32574999800005],[30.419072500000027,40.378578498000024],[30.186187500000074,40.36268099700004],[30.014735,40.41825399800007],[29.964417,40.52208449800003],[29.924952,40.551626498000076],[30.19492,40.673701496000035],[30.24338750000004,40.750796997000066],[30.234823499000072,40.83167899600005],[30.312039,40.87832599800004],[30.254108,41.05421049600005],[30.35468650000007,41.18419449800007],[30.982055500000058,41.07260250000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR423","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Düzce","NUTS_NAME":"Düzce"},"geometry":{"type":"Polygon","coordinates":[[[31.75363,41.005629998000074],[31.714458500000035,40.89269349700004],[31.441533500000048,40.83426149600007],[31.391271500000073,40.76563249700007],[31.390153500000054,40.655491496000025],[31.22459250000003,40.61657199700005],[30.897045500000047,40.66165799800007],[30.934537,40.70447],[30.831305,40.767607498000075],[30.831471500000077,40.81762949700004],[30.967463500000065,40.96352899800007],[30.982055500000058,41.07260250000007],[31.295656500000064,41.11631999900004],[31.424844,41.02525499600006],[31.75363,41.005629998000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR424","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Bolu","NUTS_NAME":"Bolu"},"geometry":{"type":"Polygon","coordinates":[[[32.560043,40.807477498000026],[32.554816,40.69194399700007],[32.37547550000005,40.663500999000064],[32.396747,40.561117996000064],[31.911622,40.32874049700007],[31.590715500000044,40.27586449800003],[31.437988,40.34499449700007],[31.297481,40.31101649800007],[31.128547500000025,40.36527999800006],[31.065705,40.225254997000036],[30.82533,40.12578449800003],[30.650090500000033,40.144735998000044],[30.521932,40.29434150000003],[30.580095,40.33931249700004],[30.559878,40.403381997000054],[30.628357,40.49151999700007],[30.866952,40.52088299900004],[30.897045500000047,40.66165799800007],[31.22459250000003,40.61657199700005],[31.390153500000054,40.655491496000025],[31.391271500000073,40.76563249700007],[31.441533500000048,40.83426149600007],[31.714458500000035,40.89269349700004],[31.75363,41.005629998000074],[32.134024,41.030703999000025],[32.274522,41.024024498000074],[32.325888500000076,40.88752399800006],[32.560043,40.807477498000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR425","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Yalova","NUTS_NAME":"Yalova"},"geometry":{"type":"Polygon","coordinates":[[[29.546993,40.69543599900004],[29.467295,40.662832999000045],[29.410220500000037,40.550392498000065],[29.06757250000004,40.54533149800005],[28.991232500000024,40.46627349700003],[28.780161,40.53619199700006],[28.996286500000053,40.645887498000036],[29.517332,40.72897149700003],[29.546993,40.69543599900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR5","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"BATI ANADOLU","NUTS_NAME":"BATI ANADOLU"},"geometry":{"type":"Polygon","coordinates":[[[32.944935,40.527210997000054],[33.220758999000054,40.320784497000034],[33.34562450000004,40.398914999000056],[33.695422,40.33224149800003],[33.557854500000076,40.16114249700007],[33.603764,40.044869996000045],[33.457947500000046,39.96101749700006],[33.364065,39.972091997000064],[33.24737250000004,39.64108699700006],[33.39862950000003,39.51610499800006],[33.39039450000007,39.41628549700005],[33.441716,39.351449],[33.541194,39.215465497000025],[33.886019,39.043412498000066],[33.722786,38.94294799700003],[33.795347,38.83806799700005],[33.70547450000004,38.680822499000044],[33.464405,38.63659149800003],[33.25558650000005,38.42087150000003],[33.203369500000065,38.278494],[33.394453500000054,37.97126099700006],[34.033573500000045,38.00481099700005],[34.10607150000004,37.996095498000045],[34.40042650000004,37.751674998000055],[34.40377649900006,37.64982299700006],[34.328714,37.586077999000054],[34.33503450000006,37.47940749600008],[34.428914500000076,37.316384998000046],[34.409057500000074,37.23256999700004],[33.997476,37.09766399700004],[33.80370450000004,36.98732599700003],[33.659066,37.015296497000065],[33.22189049900004,36.904015998000034],[33.15150550000004,36.812453498000025],[33.107767,36.851857497000026],[32.95091050000008,36.83134499800008],[33.008288,36.67464049700004],[33.23895550000003,36.520920998000065],[33.10775699900006,36.551714498000024],[33.023219,36.458324997000034],[32.573227,36.357623999000054],[32.50581550000004,36.39687199700006],[32.51975250000004,36.54548949900004],[32.457231500000034,36.738470998000025],[32.298969,36.91551849700005],[32.005236500000024,37.064884998000025],[31.740267500000073,37.35582599800006],[31.453391500000066,37.33383299800005],[31.300975,37.404258997000056],[31.321775,37.628682997000055],[31.448459,37.729105497000035],[31.419393,37.97350799600008],[31.59753050000006,38.05608549900006],[31.298652,38.284827998000026],[31.23334950000003,38.40997299700007],[31.621331,38.619947497000055],[31.52451650000006,38.69974099800004],[31.635725500000035,38.82079299700007],[31.62634950000006,38.92188599800005],[31.721062500000073,38.94946949700005],[31.616613,39.00463699800008],[31.619732500000055,39.10305549800006],[31.750522,39.07900349800008],[31.81531750000005,39.128693498000075],[32.069824,39.27607099700003],[31.994242,39.35153349800004],[32.030849,39.481336998000074],[31.822506499000042,39.74352399800006],[31.915188,39.82739399800005],[31.850136,39.84329899700003],[31.880758,39.87372349800006],[31.78699050000006,39.88126549900005],[31.676844,40.03524050000004],[30.921542,40.02736699600007],[30.82533,40.12578449800003],[31.065705,40.225254997000036],[31.128547500000025,40.36527999800006],[31.297481,40.31101649800007],[31.437988,40.34499449700007],[31.590715500000044,40.27586449800003],[31.911622,40.32874049700007],[32.396747,40.561117996000064],[32.37547550000005,40.663500999000064],[32.554816,40.69194399700007],[32.972095499000034,40.61589350000003],[32.944935,40.527210997000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR51","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Ankara","NUTS_NAME":"Ankara"},"geometry":{"type":"Polygon","coordinates":[[[33.695422,40.33224149800003],[33.557854500000076,40.16114249700007],[33.603764,40.044869996000045],[33.457947500000046,39.96101749700006],[33.364065,39.972091997000064],[33.24737250000004,39.64108699700006],[33.39862950000003,39.51610499800006],[33.39039450000007,39.41628549700005],[33.441716,39.351449],[33.541194,39.215465497000025],[33.886019,39.043412498000066],[33.722786,38.94294799700003],[33.795347,38.83806799700005],[33.70547450000004,38.680822499000044],[33.464405,38.63659149800003],[33.360580500000026,38.80198599700003],[33.374464,38.97509350000007],[33.337978,39.050211997000076],[32.99239550000004,39.244097],[32.826072,39.20321099700004],[32.72033350000004,39.069154499000035],[32.44286,38.95891299800007],[32.22421,39.01559449800004],[31.990929,38.98034749800007],[31.81531750000005,39.128693498000075],[32.069824,39.27607099700003],[31.994242,39.35153349800004],[32.030849,39.481336998000074],[31.822506499000042,39.74352399800006],[31.915188,39.82739399800005],[31.850136,39.84329899700003],[31.880758,39.87372349800006],[31.78699050000006,39.88126549900005],[31.676844,40.03524050000004],[30.921542,40.02736699600007],[30.82533,40.12578449800003],[31.065705,40.225254997000036],[31.128547500000025,40.36527999800006],[31.297481,40.31101649800007],[31.437988,40.34499449700007],[31.590715500000044,40.27586449800003],[31.911622,40.32874049700007],[32.396747,40.561117996000064],[32.37547550000005,40.663500999000064],[32.554816,40.69194399700007],[32.972095499000034,40.61589350000003],[32.944935,40.527210997000054],[33.220758999000054,40.320784497000034],[33.34562450000004,40.398914999000056],[33.695422,40.33224149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR510","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Ankara","NUTS_NAME":"Ankara"},"geometry":{"type":"Polygon","coordinates":[[[33.695422,40.33224149800003],[33.557854500000076,40.16114249700007],[33.603764,40.044869996000045],[33.457947500000046,39.96101749700006],[33.364065,39.972091997000064],[33.24737250000004,39.64108699700006],[33.39862950000003,39.51610499800006],[33.39039450000007,39.41628549700005],[33.441716,39.351449],[33.541194,39.215465497000025],[33.886019,39.043412498000066],[33.722786,38.94294799700003],[33.795347,38.83806799700005],[33.70547450000004,38.680822499000044],[33.464405,38.63659149800003],[33.360580500000026,38.80198599700003],[33.374464,38.97509350000007],[33.337978,39.050211997000076],[32.99239550000004,39.244097],[32.826072,39.20321099700004],[32.72033350000004,39.069154499000035],[32.44286,38.95891299800007],[32.22421,39.01559449800004],[31.990929,38.98034749800007],[31.81531750000005,39.128693498000075],[32.069824,39.27607099700003],[31.994242,39.35153349800004],[32.030849,39.481336998000074],[31.822506499000042,39.74352399800006],[31.915188,39.82739399800005],[31.850136,39.84329899700003],[31.880758,39.87372349800006],[31.78699050000006,39.88126549900005],[31.676844,40.03524050000004],[30.921542,40.02736699600007],[30.82533,40.12578449800003],[31.065705,40.225254997000036],[31.128547500000025,40.36527999800006],[31.297481,40.31101649800007],[31.437988,40.34499449700007],[31.590715500000044,40.27586449800003],[31.911622,40.32874049700007],[32.396747,40.561117996000064],[32.37547550000005,40.663500999000064],[32.554816,40.69194399700007],[32.972095499000034,40.61589350000003],[32.944935,40.527210997000054],[33.220758999000054,40.320784497000034],[33.34562450000004,40.398914999000056],[33.695422,40.33224149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR52","LEVL_CODE":2,"CNTR_CODE":"TR","NAME_LATN":"Konya, Karaman","NUTS_NAME":"Konya, Karaman"},"geometry":{"type":"Polygon","coordinates":[[[33.374464,38.97509350000007],[33.360580500000026,38.80198599700003],[33.464405,38.63659149800003],[33.25558650000005,38.42087150000003],[33.203369500000065,38.278494],[33.394453500000054,37.97126099700006],[34.033573500000045,38.00481099700005],[34.10607150000004,37.996095498000045],[34.40042650000004,37.751674998000055],[34.40377649900006,37.64982299700006],[34.328714,37.586077999000054],[34.33503450000006,37.47940749600008],[34.428914500000076,37.316384998000046],[34.409057500000074,37.23256999700004],[33.997476,37.09766399700004],[33.80370450000004,36.98732599700003],[33.659066,37.015296497000065],[33.22189049900004,36.904015998000034],[33.15150550000004,36.812453498000025],[33.107767,36.851857497000026],[32.95091050000008,36.83134499800008],[33.008288,36.67464049700004],[33.23895550000003,36.520920998000065],[33.10775699900006,36.551714498000024],[33.023219,36.458324997000034],[32.573227,36.357623999000054],[32.50581550000004,36.39687199700006],[32.51975250000004,36.54548949900004],[32.457231500000034,36.738470998000025],[32.298969,36.91551849700005],[32.005236500000024,37.064884998000025],[31.740267500000073,37.35582599800006],[31.453391500000066,37.33383299800005],[31.300975,37.404258997000056],[31.321775,37.628682997000055],[31.448459,37.729105497000035],[31.419393,37.97350799600008],[31.59753050000006,38.05608549900006],[31.298652,38.284827998000026],[31.23334950000003,38.40997299700007],[31.621331,38.619947497000055],[31.52451650000006,38.69974099800004],[31.635725500000035,38.82079299700007],[31.62634950000006,38.92188599800005],[31.721062500000073,38.94946949700005],[31.616613,39.00463699800008],[31.619732500000055,39.10305549800006],[31.750522,39.07900349800008],[31.81531750000005,39.128693498000075],[31.990929,38.98034749800007],[32.22421,39.01559449800004],[32.44286,38.95891299800007],[32.72033350000004,39.069154499000035],[32.826072,39.20321099700004],[32.99239550000004,39.244097],[33.337978,39.050211997000076],[33.374464,38.97509350000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR521","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Konya","NUTS_NAME":"Konya"},"geometry":{"type":"Polygon","coordinates":[[[33.464405,38.63659149800003],[33.25558650000005,38.42087150000003],[33.203369500000065,38.278494],[33.394453500000054,37.97126099700006],[34.033573500000045,38.00481099700005],[34.10607150000004,37.996095498000045],[34.40042650000004,37.751674998000055],[34.40377649900006,37.64982299700006],[34.328714,37.586077999000054],[34.33503450000006,37.47940749600008],[34.428914500000076,37.316384998000046],[34.409057500000074,37.23256999700004],[33.997476,37.09766399700004],[34.023874,37.331639497000026],[33.847965,37.40618749600003],[33.718868,37.657969997000066],[33.52491250000003,37.615049998000075],[33.36355850000007,37.451306998000064],[33.21474550000005,37.47245449800005],[33.170546,37.52954949800005],[32.95570250000003,37.34636849700007],[32.80869750000005,37.32775449800005],[32.806067,37.25219099800006],[32.67774950000006,37.11176199700003],[32.710696,37.03646249700006],[32.86762250000004,36.95162099800007],[32.684321,36.81712699700006],[32.457231500000034,36.738470998000025],[32.298969,36.91551849700005],[32.005236500000024,37.064884998000025],[31.740267500000073,37.35582599800006],[31.453391500000066,37.33383299800005],[31.300975,37.404258997000056],[31.321775,37.628682997000055],[31.448459,37.729105497000035],[31.419393,37.97350799600008],[31.59753050000006,38.05608549900006],[31.298652,38.284827998000026],[31.23334950000003,38.40997299700007],[31.621331,38.619947497000055],[31.52451650000006,38.69974099800004],[31.635725500000035,38.82079299700007],[31.62634950000006,38.92188599800005],[31.721062500000073,38.94946949700005],[31.616613,39.00463699800008],[31.619732500000055,39.10305549800006],[31.750522,39.07900349800008],[31.81531750000005,39.128693498000075],[31.990929,38.98034749800007],[32.22421,39.01559449800004],[32.44286,38.95891299800007],[32.72033350000004,39.069154499000035],[32.826072,39.20321099700004],[32.99239550000004,39.244097],[33.337978,39.050211997000076],[33.374464,38.97509350000007],[33.360580500000026,38.80198599700003],[33.464405,38.63659149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR522","LEVL_CODE":3,"CNTR_CODE":"TR","NAME_LATN":"Karaman","NUTS_NAME":"Karaman"},"geometry":{"type":"Polygon","coordinates":[[[33.997476,37.09766399700004],[33.80370450000004,36.98732599700003],[33.659066,37.015296497000065],[33.22189049900004,36.904015998000034],[33.15150550000004,36.812453498000025],[33.107767,36.851857497000026],[32.95091050000008,36.83134499800008],[33.008288,36.67464049700004],[33.23895550000003,36.520920998000065],[33.10775699900006,36.551714498000024],[33.023219,36.458324997000034],[32.573227,36.357623999000054],[32.50581550000004,36.39687199700006],[32.51975250000004,36.54548949900004],[32.457231500000034,36.738470998000025],[32.684321,36.81712699700006],[32.86762250000004,36.95162099800007],[32.710696,37.03646249700006],[32.67774950000006,37.11176199700003],[32.806067,37.25219099800006],[32.80869750000005,37.32775449800005],[32.95570250000003,37.34636849700007],[33.170546,37.52954949800005],[33.21474550000005,37.47245449800005],[33.36355850000007,37.451306998000064],[33.52491250000003,37.615049998000075],[33.718868,37.657969997000066],[33.847965,37.40618749600003],[34.023874,37.331639497000026],[33.997476,37.09766399700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"TR6","LEVL_CODE":1,"CNTR_CODE":"TR","NAME_LATN":"AKDENIZ","NUTS_NAME":"AKDENIZ"},"geometry":{"type":"Polygon","coordinates":[[[37.64785550000005,38.34777349700005],[37.76024650000005,38.20305099700005],[37.74833550000005,38.14038149900006],[37.614140500000076,38.03833199700006],[37.638667500000054,37.93116699900003],[37.61636,37.85288349900003],[37.502716,37.792868498000075],[37.432401,37.64170899900006],[37.56929149900003,37.58780599700003],[37.62321650000007,37.51165249600007],[37.55469750000003,37.382446999000024],[37.379789,37.35999050000004],[37.08278,37.17567599900008],[36.901796,37.33366049800003],[36.697388500000045,37.21814249600004],[36.52120250000007,37.08523999800008],[36.465673,36.94866749800008],[36.66001039400004,36.83323153500004],[36.549576,36.48776999700004],[36.690133,36.28624799900007],[36.683648,36.23678349800008],[36.39222,36.21332599900006],[36.37497,35.99791399800006],[36.223607,35.962219997000034],[36.168469,35.819717997000055],[35.917953,35.928695498000025],[35.978777,36.01944149800005],[35.779623,36.31855599700003],[36.037126500000056,36.53373499700007],[36.188917,36.593206500000065],[36.21573050000006,36.65963549800006],[36.205831500000045,36.78273199600005],[36.14857650000005,36.85506649800004],[36.018686,36.92843449800006],[35.963096,36.902986998000074],[35.792162,36.76891499900006],[35.56386,36.71301049700003],[35.723783500000025,36.71192749800008],[35.56584,36.56479449900007],[35.420896500000026,36.59061599900008],[35.339365,36.53852649800007],[34.899245,36.73828099800005],[34.74605750000006,36.81102949900003],[34.56072050000006,36.76816349600006],[34.085783,36.41823749800005],[34.04285250000004,36.29496949800006],[33.94976999900007,36.26343749600005],[33.866678,36.31495849800007],[33.68421,36.13340199800007],[33.64763050000005,36.19356349700007],[33.56112850000005,36.125440499000035],[33.14687549900003,36.13780399800004],[33.08139250000005,36.08072449800005],[32.950445,36.103795998000066],[32.805395,36.02311499800004],[32.57634,36.093232998000076],[32.295229,36.234987498000066],[32.029032,36.53820249800003],[31.334676500000057,36.80768799800006],[30.698167500000068,36.88451949700004],[30.57494150000008,36.794511998000075],[30.556848500000058,36.619237998000074],[30.59149,36.58946799700004],[30.481527500000027,36.433797998000045],[30.48763450000007,36.27913099700004],[30.405119,36.20929149700004],[30.32793250000003,36.29695299800005],[30.188627,36.31390950000008],[30.115274500000055,36.24686649800003],[29.922674,36.23392699600004],[29.683466,36.130262498000036],[29.644518,36.19632899900006],[29.408991,36.261224997000056],[29.351629500000058,36.230226499000025],[29.26056150000005,36.30439899700008],[29.317792,36.44839999900006],[29.637207,36.669686],[29.751884,36.900562998000055],[29.713264499000047,36.95630949800005],[29.60908950000004,36.98030049800008],[29.386635500000068,36.85942699800006],[29.422852500000033,36.956733996000025],[29.341447,37.00684899700008],[29.606530500000076,37.39536099700007],[29.514886,37.460002],[29.500381,37.61887199700004],[29.852119,37.75228549800005],[30.039979500000072,37.752413998000065],[30.174609,37.95408949800003],[30.327528,38.03099949800003],[30.42972050000003,38.15599899800003],[30.70472249900007,38.265575],[30.986741,38.46641449800006],[31.13277,38.48890349800007],[31.23334950000003,38.40997299700007],[31.298652,38.284827998000026],[31.59753050000006,38.05608549900006],[31.419393,37.97350799600008],[31.448459,37.729105497000035],[31.321775,37.628682997000055],[31.300975,37.404258997000056],[31.453391500000066,37.33383299800005],[31.740267500000073,37.35582599800006],[32.005236500000024,37.064884998000025],[32.298969,36.91551849700005],[32.457231500000034,36.738470998000025],[32.51975250000004,36.54548949900004],[32.50581550000004,36.39687199700006],[32.573227,36.357623999000054],[33.023219,36.458324997000034],[33.10775699900006,36.551714498000024],[33.23895550000003,36.520920998000065],[33.008288,36.67464049700004],[32.95091050000008,36.83134499800008],[33.107767,36.851857497000026],[33.15150550000004,36.812453498000025],[33.22189049900004,36.904015998000034],[33.659066,37.015296497000065],[33.80370450000004,36.98732599700003],[33.997476,37.09766399700004],[34.409057500000074,37.23256999700004],[34.428914500000076,37.316384998000046],[34.748,37.40930399800004],[34.87491350000005,37.492744],[34.88540450000005,37.67054299800003],[35.218708,37.75922449800004],[35.574764,37.73715849800004],[35.575068,37.911404497000035],[35.61442150000005,37.96211099800007],[35.925933,38.084282],[36.10676950000004,38.31260099700006],[36.253814,38.37942299800005],[36.36776750000007,38.32651049800006],[36.440496,38.21943249800006],[36.59920550000004,38.39936399900006],[36.723043500000074,38.427591998000025],[36.69959350000005,38.503656496000076],[36.76984550000003,38.54643599800005],[37.341487,38.593515498000045],[37.25953450000003,38.479910999000026],[37.64785550000005,38.34777349700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK4","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Devon","NUTS_NAME":"Devon"},"geometry":{"type":"Polygon","coordinates":[[[-4.139783840999939,51.077219299000035],[-4.21529449999997,51.07535549600004],[-4.200037000999941,51.20093899900007],[-3.720778499999938,51.23309349800007],[-3.723824,51.18072899900005],[-3.834736,51.14138399900003],[-3.594467499999951,51.05519499900004],[-3.609741,51.007984],[-3.426810499999931,51.03140649900007],[-3.163939,50.94597249800006],[-3.187833500999943,50.910362],[-3.152869,50.89379899800008],[-3.052417,50.90826399900004],[-2.954316,50.82117449800006],[-2.886639499999944,50.79480749800007],[-2.954328,50.76771149600006],[-2.947815000999981,50.71830750000004],[-3.38546,50.606719998000074],[-3.469847,50.68223549700008],[-3.443591499999968,50.602722],[-3.509030499999938,50.516514],[-3.488737499999957,50.46379099600006],[-3.559090499999968,50.42766199700003],[-3.482395499999939,50.39991799900008],[-3.507436,50.378989998000065],[-3.723704,50.20151899900003],[-4.12313249999994,50.34674049700004],[-4.177532637999946,50.36402146800003],[-4.203700022999953,50.39825618900005],[-4.23509422099994,50.46025278400003],[-4.180500499999937,50.50337999800007],[-4.307998,50.57887649800006],[-4.386616,50.76392750000008],[-4.493454499999928,50.792831498000055],[-4.42092,50.86619549900007],[-4.456273500999941,50.92902349800005],[-4.545959499999981,50.92835249800004],[-4.526117499999941,51.02189249600008],[-4.345269,50.988857497000026],[-4.139783840999939,51.077219299000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK41","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Plymouth","NUTS_NAME":"Plymouth"},"geometry":{"type":"Polygon","coordinates":[[[-4.12313249999994,50.34674049700004],[-4.177532637999946,50.36402146800003],[-4.203700022999953,50.39825618900005],[-4.23509422099994,50.46025278400003],[-4.187040499999966,50.42440049700008],[-4.025316,50.381782498000064],[-4.12313249999994,50.34674049700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK42","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Torbay","NUTS_NAME":"Torbay"},"geometry":{"type":"Polygon","coordinates":[[[-3.507436,50.378989998000065],[-3.62841,50.434322500000064],[-3.509030499999938,50.516514],[-3.488737499999957,50.46379099600006],[-3.559090499999968,50.42766199700003],[-3.482395499999939,50.39991799900008],[-3.507436,50.378989998000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK43","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Devon CC","NUTS_NAME":"Devon CC"},"geometry":{"type":"Polygon","coordinates":[[[-2.954316,50.82117449800006],[-2.886639499999944,50.79480749800007],[-2.954328,50.76771149600006],[-2.947815000999981,50.71830750000004],[-3.38546,50.606719998000074],[-3.469847,50.68223549700008],[-3.443591499999968,50.602722],[-3.509030499999938,50.516514],[-3.62841,50.434322500000064],[-3.507436,50.378989998000065],[-3.723704,50.20151899900003],[-4.12313249999994,50.34674049700004],[-4.025316,50.381782498000064],[-4.187040499999966,50.42440049700008],[-4.23509422099994,50.46025278400003],[-4.180500499999937,50.50337999800007],[-4.307998,50.57887649800006],[-4.386616,50.76392750000008],[-4.493454499999928,50.792831498000055],[-4.42092,50.86619549900007],[-4.456273500999941,50.92902349800005],[-4.545959499999981,50.92835249800004],[-4.526117499999941,51.02189249600008],[-4.345269,50.988857497000026],[-4.139783840999939,51.077219299000035],[-4.21529449999997,51.07535549600004],[-4.200037000999941,51.20093899900007],[-3.720778499999938,51.23309349800007],[-3.723824,51.18072899900005],[-3.834736,51.14138399900003],[-3.594467499999951,51.05519499900004],[-3.609741,51.007984],[-3.426810499999931,51.03140649900007],[-3.163939,50.94597249800006],[-3.187833500999943,50.910362],[-3.152869,50.89379899800008],[-3.052417,50.90826399900004],[-2.954316,50.82117449800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"WALES","NUTS_NAME":"WALES"},"geometry":{"type":"Polygon","coordinates":[[[-4.275177500999973,53.403098998000075],[-4.213657,53.29817199800004],[-4.040188,53.31065349800008],[-4.196792116999973,53.21081699300004],[-4.007376,53.246928998000044],[-3.845432499999959,53.29659249900004],[-3.865022,53.34220099700008],[-3.609629499999926,53.29049299600007],[-3.363392499999975,53.35202799900003],[-3.019001,53.21184549800006],[-3.084193,53.256122497000035],[-2.922283,53.18931599600006],[-2.99507449999993,53.154220499000076],[-2.834814,52.998698999000055],[-2.726823499999966,52.983295499000064],[-2.728526,52.925197498000045],[-2.80052349999994,52.89548099900003],[-2.982011,52.959217],[-3.14748,52.89015599700008],[-3.161091499999941,52.79612349800004],[-2.977511000999925,52.72648250000003],[-3.139272499999947,52.586006],[-3.137092,52.533389998000075],[-2.994171499999936,52.55295950000004],[-3.235562,52.44255049800006],[-2.954719,52.34925849900003],[-3.141911499999935,52.127876498000035],[-3.067357499999957,51.98315050000008],[-2.65021349999995,51.826152998000055],[-2.686266000999979,51.662364996000065],[-2.660656919999951,51.62766319500008],[-3.082688,51.50190749800004],[-3.16959049999997,51.40624999900007],[-3.340482,51.38127499600006],[-3.558956,51.401328998000054],[-3.637855499999944,51.46984099700006],[-3.88623,51.61738599700004],[-4.209631,51.53722749600007],[-4.307776,51.61030199800007],[-4.05255,51.703281497000035],[-4.287212,51.66770149900003],[-4.382688,51.72815699700004],[-4.369891792999965,51.77438571400006],[-4.661121499999979,51.73165499800007],[-4.711614500999929,51.65161149800008],[-4.929975499999955,51.59847650000006],[-5.12474549999996,51.68109499900004],[-4.898221312999965,51.71056675600005],[-5.187262000999965,51.70885449800005],[-5.254024,51.73765199800005],[-5.11745,51.768268498000054],[-5.139965500999949,51.86383049600005],[-5.320248499999934,51.86160299800008],[-5.086008,51.968773],[-5.07227649999993,52.03031299700007],[-4.878756,52.019611499000064],[-4.20750450099996,52.26382049700004],[-4.040780499999926,52.52494449800008],[-3.931013499999949,52.553611999000054],[-4.070163,52.54323949900004],[-4.128679,52.61278149900005],[-3.938924000999975,52.74887849900006],[-4.060166,52.720512496000026],[-4.150184499999966,52.80757149600004],[-4.058637499999975,52.920723],[-4.407304000999943,52.89085749700007],[-4.531266499999958,52.77798449800008],[-4.767578499999956,52.79495999900007],[-4.353505,53.03351599900003],[-4.333707,53.11389899800008],[-4.216110416999925,53.19851737000005],[-4.329113,53.12664049700004],[-4.69663,53.30674349800006],[-4.575122,53.28789149800008],[-4.574172,53.40379349700004],[-4.404923,53.42721949600008],[-4.275177500999973,53.403098998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"West Wales and The Valleys","NUTS_NAME":"West Wales and The Valleys"},"geometry":{"type":"Polygon","coordinates":[[[-4.275177500999973,53.403098998000075],[-4.213657,53.29817199800004],[-4.040188,53.31065349800008],[-4.196792116999973,53.21081699300004],[-4.007376,53.246928998000044],[-3.845432499999959,53.29659249900004],[-3.865022,53.34220099700008],[-3.609629499999926,53.29049299600007],[-3.363392499999975,53.35202799900003],[-3.400592,53.31385399900006],[-3.271256,53.157535497000026],[-3.185847499999966,53.141968],[-3.127353500999959,53.06857699700004],[-3.148757499999931,52.99859999900008],[-3.090286499999934,52.971641498000054],[-3.375009499999976,52.89247499700008],[-3.48307,52.86553949800003],[-3.566589499999964,52.83011999800004],[-3.601979,52.705105],[-3.817276,52.67310699900003],[-3.926611,52.560783498000035],[-3.733258499999977,52.50660299900005],[-3.771615499999939,52.442165499000055],[-3.658389,52.34775899600004],[-3.744970499999965,52.317470496000055],[-3.758101,52.128242500000056],[-3.647133,52.03879949600008],[-3.80665,51.78792949800004],[-3.591288499999962,51.75461949700008],[-3.402411,51.82632049800003],[-3.334463,51.79040899800003],[-3.157343499999968,51.816058998000074],[-3.043488500999956,51.77045799700005],[-2.958938,51.628749998000046],[-3.118871499999955,51.54570749800007],[-3.237515,51.55262749800005],[-3.497506499999929,51.51264549600006],[-3.637855499999944,51.46984099700006],[-3.88623,51.61738599700004],[-4.209631,51.53722749600007],[-4.307776,51.61030199800007],[-4.05255,51.703281497000035],[-4.287212,51.66770149900003],[-4.382688,51.72815699700004],[-4.369891792999965,51.77438571400006],[-4.661121499999979,51.73165499800007],[-4.711614500999929,51.65161149800008],[-4.929975499999955,51.59847650000006],[-5.12474549999996,51.68109499900004],[-4.898221312999965,51.71056675600005],[-5.187262000999965,51.70885449800005],[-5.254024,51.73765199800005],[-5.11745,51.768268498000054],[-5.139965500999949,51.86383049600005],[-5.320248499999934,51.86160299800008],[-5.086008,51.968773],[-5.07227649999993,52.03031299700007],[-4.878756,52.019611499000064],[-4.20750450099996,52.26382049700004],[-4.040780499999926,52.52494449800008],[-3.931013499999949,52.553611999000054],[-4.070163,52.54323949900004],[-4.128679,52.61278149900005],[-3.938924000999975,52.74887849900006],[-4.060166,52.720512496000026],[-4.150184499999966,52.80757149600004],[-4.058637499999975,52.920723],[-4.407304000999943,52.89085749700007],[-4.531266499999958,52.77798449800008],[-4.767578499999956,52.79495999900007],[-4.353505,53.03351599900003],[-4.333707,53.11389899800008],[-4.216110416999925,53.19851737000005],[-4.329113,53.12664049700004],[-4.69663,53.30674349800006],[-4.575122,53.28789149800008],[-4.574172,53.40379349700004],[-4.404923,53.42721949600008],[-4.275177500999973,53.403098998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Isle of Anglesey","NUTS_NAME":"Isle of Anglesey"},"geometry":{"type":"Polygon","coordinates":[[[-4.196792116999973,53.21081699300004],[-4.216110416999925,53.19851737000005],[-4.329113,53.12664049700004],[-4.69663,53.30674349800006],[-4.575122,53.28789149800008],[-4.574172,53.40379349700004],[-4.404923,53.42721949600008],[-4.275177500999973,53.403098998000075],[-4.213657,53.29817199800004],[-4.040188,53.31065349800008],[-4.196792116999973,53.21081699300004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Gwynedd","NUTS_NAME":"Gwynedd"},"geometry":{"type":"Polygon","coordinates":[[[-3.48307,52.86553949800003],[-3.566589499999964,52.83011999800004],[-3.601979,52.705105],[-3.817276,52.67310699900003],[-3.926611,52.560783498000035],[-3.931013499999949,52.553611999000054],[-4.070163,52.54323949900004],[-4.128679,52.61278149900005],[-3.938924000999975,52.74887849900006],[-4.060166,52.720512496000026],[-4.150184499999966,52.80757149600004],[-4.058637499999975,52.920723],[-4.407304000999943,52.89085749700007],[-4.531266499999958,52.77798449800008],[-4.767578499999956,52.79495999900007],[-4.353505,53.03351599900003],[-4.333707,53.11389899800008],[-4.216110416999925,53.19851737000005],[-4.196792116999973,53.21081699300004],[-4.007376,53.246928998000044],[-3.935375,53.214881999000056],[-4.031342000999928,53.10567099700006],[-3.977065499999981,53.06895849700004],[-3.990336,53.013668],[-3.813224,52.949394],[-3.436763,52.98085799900008],[-3.49139,52.93379199900005],[-3.48307,52.86553949800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Conwy and Denbighshire","NUTS_NAME":"Conwy and Denbighshire"},"geometry":{"type":"Polygon","coordinates":[[[-3.375009499999976,52.89247499700008],[-3.48307,52.86553949800003],[-3.49139,52.93379199900005],[-3.436763,52.98085799900008],[-3.813224,52.949394],[-3.990336,53.013668],[-3.977065499999981,53.06895849700004],[-4.031342000999928,53.10567099700006],[-3.935375,53.214881999000056],[-4.007376,53.246928998000044],[-3.845432499999959,53.29659249900004],[-3.865022,53.34220099700008],[-3.609629499999926,53.29049299600007],[-3.363392499999975,53.35202799900003],[-3.400592,53.31385399900006],[-3.271256,53.157535497000026],[-3.185847499999966,53.141968],[-3.127353500999959,53.06857699700004],[-3.148757499999931,52.99859999900008],[-3.090286499999934,52.971641498000054],[-3.375009499999976,52.89247499700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South West Wales","NUTS_NAME":"South West Wales"},"geometry":{"type":"Polygon","coordinates":[[[-3.80665,51.78792949800004],[-3.936332,51.77134699800007],[-4.05255,51.703281497000035],[-4.287212,51.66770149900003],[-4.382688,51.72815699700004],[-4.369891792999965,51.77438571400006],[-4.661121499999979,51.73165499800007],[-4.711614500999929,51.65161149800008],[-4.929975499999955,51.59847650000006],[-5.12474549999996,51.68109499900004],[-4.898221312999965,51.71056675600005],[-5.187262000999965,51.70885449800005],[-5.254024,51.73765199800005],[-5.11745,51.768268498000054],[-5.139965500999949,51.86383049600005],[-5.320248499999934,51.86160299800008],[-5.086008,51.968773],[-5.07227649999993,52.03031299700007],[-4.878756,52.019611499000064],[-4.20750450099996,52.26382049700004],[-4.040780499999926,52.52494449800008],[-3.931013499999949,52.553611999000054],[-3.926611,52.560783498000035],[-3.733258499999977,52.50660299900005],[-3.771615499999939,52.442165499000055],[-3.658389,52.34775899600004],[-3.744970499999965,52.317470496000055],[-3.758101,52.128242500000056],[-3.647133,52.03879949600008],[-3.80665,51.78792949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL15","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Central Valleys","NUTS_NAME":"Central Valleys"},"geometry":{"type":"Polygon","coordinates":[[[-3.334463,51.79040899800003],[-3.27409849999998,51.68225099700004],[-3.302681499999949,51.614978998000026],[-3.237515,51.55262749800005],[-3.497506499999929,51.51264549600006],[-3.471669499999962,51.59859099800008],[-3.589626499999952,51.67931],[-3.591288499999962,51.75461949700008],[-3.402411,51.82632049800003],[-3.334463,51.79040899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL16","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Gwent Valleys","NUTS_NAME":"Gwent Valleys"},"geometry":{"type":"Polygon","coordinates":[[[-3.118871499999955,51.54570749800007],[-3.237515,51.55262749800005],[-3.302681499999949,51.614978998000026],[-3.27409849999998,51.68225099700004],[-3.334463,51.79040899800003],[-3.157343499999968,51.816058998000074],[-3.043488500999956,51.77045799700005],[-2.958938,51.628749998000046],[-3.118871499999955,51.54570749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL17","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bridgend and Neath Port Talbot","NUTS_NAME":"Bridgend and Neath Port Talbot"},"geometry":{"type":"Polygon","coordinates":[[[-3.80665,51.78792949800004],[-3.591288499999962,51.75461949700008],[-3.589626499999952,51.67931],[-3.471669499999962,51.59859099800008],[-3.497506499999929,51.51264549600006],[-3.637855499999944,51.46984099700006],[-3.88623,51.61738599700004],[-3.855920499999968,51.697459998000056],[-3.936332,51.77134699800007],[-3.80665,51.78792949800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL18","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Swansea","NUTS_NAME":"Swansea"},"geometry":{"type":"Polygon","coordinates":[[[-3.936332,51.77134699800007],[-3.855920499999968,51.697459998000056],[-3.88623,51.61738599700004],[-4.209631,51.53722749600007],[-4.307776,51.61030199800007],[-4.05255,51.703281497000035],[-3.936332,51.77134699800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"East Wales","NUTS_NAME":"East Wales"},"geometry":{"type":"Polygon","coordinates":[[[-3.363392499999975,53.35202799900003],[-3.019001,53.21184549800006],[-3.084193,53.256122497000035],[-2.922283,53.18931599600006],[-2.99507449999993,53.154220499000076],[-2.834814,52.998698999000055],[-2.726823499999966,52.983295499000064],[-2.728526,52.925197498000045],[-2.80052349999994,52.89548099900003],[-2.982011,52.959217],[-3.14748,52.89015599700008],[-3.161091499999941,52.79612349800004],[-2.977511000999925,52.72648250000003],[-3.139272499999947,52.586006],[-3.137092,52.533389998000075],[-2.994171499999936,52.55295950000004],[-3.235562,52.44255049800006],[-2.954719,52.34925849900003],[-3.141911499999935,52.127876498000035],[-3.067357499999957,51.98315050000008],[-2.65021349999995,51.826152998000055],[-2.686266000999979,51.662364996000065],[-2.660656919999951,51.62766319500008],[-3.082688,51.50190749800004],[-3.16959049999997,51.40624999900007],[-3.340482,51.38127499600006],[-3.558956,51.401328998000054],[-3.637855499999944,51.46984099700006],[-3.497506499999929,51.51264549600006],[-3.237515,51.55262749800005],[-3.118871499999955,51.54570749800007],[-2.958938,51.628749998000046],[-3.043488500999956,51.77045799700005],[-3.157343499999968,51.816058998000074],[-3.334463,51.79040899800003],[-3.402411,51.82632049800003],[-3.591288499999962,51.75461949700008],[-3.80665,51.78792949800004],[-3.647133,52.03879949600008],[-3.758101,52.128242500000056],[-3.744970499999965,52.317470496000055],[-3.658389,52.34775899600004],[-3.771615499999939,52.442165499000055],[-3.733258499999977,52.50660299900005],[-3.926611,52.560783498000035],[-3.817276,52.67310699900003],[-3.601979,52.705105],[-3.566589499999964,52.83011999800004],[-3.48307,52.86553949800003],[-3.375009499999976,52.89247499700008],[-3.090286499999934,52.971641498000054],[-3.148757499999931,52.99859999900008],[-3.127353500999959,53.06857699700004],[-3.185847499999966,53.141968],[-3.271256,53.157535497000026],[-3.400592,53.31385399900006],[-3.363392499999975,53.35202799900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Monmouthshire and Newport","NUTS_NAME":"Monmouthshire and Newport"},"geometry":{"type":"Polygon","coordinates":[[[-2.65021349999995,51.826152998000055],[-2.686266000999979,51.662364996000065],[-2.660656919999951,51.62766319500008],[-3.082688,51.50190749800004],[-3.118871499999955,51.54570749800007],[-2.958938,51.628749998000046],[-3.043488500999956,51.77045799700005],[-3.157343499999968,51.816058998000074],[-3.039992000999973,51.884753997000075],[-3.090104,51.950393500000075],[-3.067357499999957,51.98315050000008],[-2.65021349999995,51.826152998000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Cardiff and Vale of Glamorgan","NUTS_NAME":"Cardiff and Vale of Glamorgan"},"geometry":{"type":"Polygon","coordinates":[[[-3.118871499999955,51.54570749800007],[-3.082688,51.50190749800004],[-3.16959049999997,51.40624999900007],[-3.340482,51.38127499600006],[-3.558956,51.401328998000054],[-3.637855499999944,51.46984099700006],[-3.497506499999929,51.51264549600006],[-3.237515,51.55262749800005],[-3.118871499999955,51.54570749800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Flintshire and Wrexham","NUTS_NAME":"Flintshire and Wrexham"},"geometry":{"type":"Polygon","coordinates":[[[-3.084193,53.256122497000035],[-2.922283,53.18931599600006],[-2.99507449999993,53.154220499000076],[-2.834814,52.998698999000055],[-2.726823499999966,52.983295499000064],[-2.728526,52.925197498000045],[-2.80052349999994,52.89548099900003],[-2.982011,52.959217],[-3.14748,52.89015599700008],[-3.375009499999976,52.89247499700008],[-3.090286499999934,52.971641498000054],[-3.148757499999931,52.99859999900008],[-3.127353500999959,53.06857699700004],[-3.185847499999966,53.141968],[-3.271256,53.157535497000026],[-3.400592,53.31385399900006],[-3.363392499999975,53.35202799900003],[-3.019001,53.21184549800006],[-3.084193,53.256122497000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKL24","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Powys","NUTS_NAME":"Powys"},"geometry":{"type":"Polygon","coordinates":[[[-3.14748,52.89015599700008],[-3.161091499999941,52.79612349800004],[-2.977511000999925,52.72648250000003],[-3.139272499999947,52.586006],[-3.137092,52.533389998000075],[-2.994171499999936,52.55295950000004],[-3.235562,52.44255049800006],[-2.954719,52.34925849900003],[-3.141911499999935,52.127876498000035],[-3.067357499999957,51.98315050000008],[-3.090104,51.950393500000075],[-3.039992000999973,51.884753997000075],[-3.157343499999968,51.816058998000074],[-3.334463,51.79040899800003],[-3.402411,51.82632049800003],[-3.591288499999962,51.75461949700008],[-3.80665,51.78792949800004],[-3.647133,52.03879949600008],[-3.758101,52.128242500000056],[-3.744970499999965,52.317470496000055],[-3.658389,52.34775899600004],[-3.771615499999939,52.442165499000055],[-3.733258499999977,52.50660299900005],[-3.926611,52.560783498000035],[-3.817276,52.67310699900003],[-3.601979,52.705105],[-3.566589499999964,52.83011999800004],[-3.48307,52.86553949800003],[-3.375009499999976,52.89247499700008],[-3.14748,52.89015599700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"SCOTLAND","NUTS_NAME":"SCOTLAND"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.75837,60.81646749600003],[-0.831153,60.685222499000076],[-0.981795499999976,60.685482],[-0.865963499999964,60.83370199700005],[-0.75837,60.81646749600003]]],[[[-1.003055499999959,60.58155799700006],[-1.028645,60.49721549700007],[-1.167948499999966,60.49954999900007],[-1.202889,60.607105499000056],[-1.074501,60.73317349700005],[-1.008528,60.71493899900008],[-0.981463500999951,60.637199500000065],[-1.031584500999941,60.59758399900005],[-1.003055499999959,60.58155799700006]]],[[[-1.304159,60.62399649900004],[-1.319979499999931,60.451770999000075],[-1.383588,60.397674496000036],[-1.237423000999968,60.49381999700006],[-1.049048,60.43949149600007],[-1.155833499999972,60.33803549900006],[-1.079679499999941,60.32329199900005],[-1.15929,60.281048],[-1.095383499999969,60.26348899900006],[-1.170235,60.23842599900007],[-1.155375499999934,60.15899649800008],[-1.203963500999976,60.130611499000054],[-1.277545,59.85259999700003],[-1.391431500999943,59.91602349900006],[-1.276649,60.099434],[-1.32783749999993,60.16605749900003],[-1.293532499999969,60.251404],[-1.462364499999978,60.14639299900006],[-1.702932,60.255359498000075],[-1.339904500999978,60.359771497000054],[-1.632932,60.48316199900006],[-1.499491318999958,60.543659871000045],[-1.489197784999931,60.54412077600006],[-1.304159,60.62399649900004]]],[[[-5.384345607999933,56.53174223900004],[-5.421163,56.54778649900004],[-5.325605499999938,56.648285],[-4.969490499999949,56.714679498000066],[-5.247245499999963,56.70311749700005],[-5.092236,56.83002449600008],[-5.334743499999945,56.859508497000036],[-5.122688000999972,56.827274498000065],[-5.684919,56.497295499000074],[-6.006905,56.645126497000035],[-5.867277643999955,56.675698871000066],[-6.226232,56.710991],[-5.751703000999953,56.78216549700005],[-5.887583499999948,56.792953999000076],[-5.722717500999977,56.889038],[-5.924258,56.89156349700005],[-5.820707,57.01260749700003],[-5.50935149999998,56.99639499700004],[-5.797323,57.06098549700005],[-5.72895,57.10147849700007],[-5.509437499999933,57.097774498000035],[-5.681389,57.15053049900007],[-5.670602562999932,57.20843875300005],[-6.017994,57.01794799800007],[-6.004551,57.114955999000074],[-5.908913,57.17123799900003],[-6.32237,57.160293498000044],[-6.483495,57.310973999000055],[-6.307433,57.299278497000046],[-6.789714499999945,57.42120749800006],[-6.716572,57.51391599900006],[-6.582203,57.423621996000065],[-6.637269499999945,57.50403199800007],[-6.562680499999942,57.504444],[-6.638996499999962,57.549937998000075],[-6.635986499999944,57.60881799900005],[-6.406945,57.523063498000056],[-6.427763499999969,57.64247149900007],[-6.296708,57.70756149700003],[-6.146597000999975,57.57447449700004],[-6.130341,57.316936498000075],[-5.88541349999997,57.238167],[-5.776867,57.27114499700008],[-5.64761249999998,57.25529499700008],[-5.664395153999976,57.219288446000064],[-5.415392,57.23015599900003],[-5.73379,57.283454996000046],[-5.441254,57.418952997000076],[-5.82202,57.363647496000056],[-5.872894,57.47562050000005],[-5.841507499999977,57.57624799900003],[-5.513192499999946,57.54019549900005],[-5.81922,57.639786],[-5.673280499999976,57.70169849900003],[-5.813466499999947,57.75067150000007],[-5.81416349999995,57.85871099900004],[-5.683442,57.86545199900007],[-5.619121,57.766895497000064],[-5.577558499999952,57.79293449800008],[-5.655929499999957,57.87793349800006],[-5.618142,57.92462899800006],[-5.217599,57.84221649700004],[-5.406790499999943,57.92776849900008],[-5.360826000999964,57.93917449800006],[-5.071033000999932,57.81991949700006],[-5.458675,58.074409495000054],[-5.268833,58.10244799800006],[-5.403668,58.23530949700006],[-5.375456,58.26508699900006],[-4.924184000999958,58.25644699700007],[-5.134424,58.27807599700003],[-5.182866499999932,58.35124949800007],[-5.033927499999947,58.39294799700008],[-5.114617499999952,58.415272],[-5.072923,58.452243999000075],[-4.988059499999963,58.425987],[-5.126430499999969,58.48945599700005],[-5.007117499999936,58.625992],[-4.65146249999998,58.534366498000054],[-4.739715499999932,58.44739149800006],[-4.589487499999962,58.57794549700003],[-4.409354499999949,58.523773],[-4.475344,58.44438949700003],[-4.357074,58.536857498000074],[-3.024449,58.644264],[-3.130931,58.519863],[-3.046968499999934,58.47255350000006],[-3.116977499999962,58.368579999000076],[-4.082477499999925,57.949943500000074],[-4.001344,57.92654399800006],[-4.016409,57.85973750000005],[-4.400494,57.918987497000046],[-4.04890249999994,57.81504049600005],[-3.771491499999968,57.867034998000065],[-3.975255499999946,57.69451899900008],[-4.301044,57.67506799700004],[-4.428933,57.574283499000046],[-3.993433499999981,57.67645649800005],[-4.234324499999957,57.500965],[-4.432052499999941,57.494354],[-4.191771,57.48384099900005],[-4.019451499999946,57.598220999000034],[-3.347729,57.72444899900006],[-2.801509,57.69524749900006],[-1.920823499999926,57.67439649600004],[-1.78944,57.503555497000036],[-2.042654,57.243911498000045],[-2.077448499999946,57.17697899800004],[-2.05034749999993,57.128338],[-2.206539500999952,56.96594999700005],[-2.214603,56.88209899700007],[-2.425346,56.755187996000075],[-2.538641,56.566062997000074],[-2.733122499999979,56.46479799900004],[-3.051892500999941,56.45849999700005],[-3.345321499999955,56.37197099600007],[-3.276975499999935,56.35053250000004],[-2.803087,56.43967799700005],[-2.838920499999972,56.35307299600004],[-2.620387,56.26118849900007],[-2.955285,56.21049499700007],[-3.39145149999996,56.00593199700006],[-3.866839499999969,56.12066249900005],[-3.820153,56.099117500000034],[-3.515724,56.00225850000004],[-3.425330499999973,55.99403],[-3.07764549999996,55.94689549800006],[-2.815697499999942,56.062404496000056],[-2.366623499999946,55.946113498000045],[-2.13755649999996,55.91727849600005],[-2.034329,55.81116499900003],[-2.335961,55.63214499800006],[-2.165465499999925,55.46846750000003],[-2.496131499999933,55.346595999000044],[-2.689751,55.18906399700006],[-2.858508,55.10842499800003],[-3.051281499999959,55.05262349800006],[-3.091623992999928,54.97554629100006],[-3.560086499999954,54.97845099800003],[-3.594912499999964,54.872833498000034],[-3.850849,54.86328899700004],[-3.824464499999976,54.82371149800008],[-3.981397,54.76880249700008],[-4.06509049999994,54.78116249700008],[-4.050716,54.849548499000036],[-4.105112499999962,54.76763549900005],[-4.425736,54.926769499000045],[-4.413721499999951,54.82943750000004],[-4.341246499999954,54.79918649600006],[-4.347857499999975,54.71093749700003],[-4.405439499999943,54.67750950000004],[-4.860413,54.86895749900003],[-4.960747,54.80627449700006],[-4.876391499999954,54.63921349900005],[-4.964987,54.66406249900007],[-4.957434499999977,54.72864899900003],[-5.183450000999926,54.912364998000044],[-5.161495,55.00796149900003],[-5.098928000999933,55.01810849900005],[-5.060561,54.92361050000005],[-4.990791499999943,54.92166149600007],[-5.040228499999955,54.997726500000056],[-5.048733499999969,55.05751799600006],[-4.859926,55.227607499000044],[-4.768847499999936,55.400829498000064],[-4.650890499999946,55.441147],[-4.620535,55.49978649600007],[-4.658272499999953,55.57028949800008],[-4.905256,55.69921899800005],[-4.855803499999979,55.760661998000046],[-4.888966499999981,55.87478999600006],[-4.878924499999926,55.94240199700005],[-4.765130499999941,55.957771499000046],[-4.392671740999958,55.88981584100003],[-4.609648,55.94667449700006],[-4.828885,56.07900999700007],[-4.766975,55.98970399800004],[-4.853408499999944,55.99036049800003],[-4.880223,56.058253998000055],[-4.752816000999928,56.20385749700006],[-4.862619499999937,56.100319],[-4.917712,56.164127499000074],[-4.896133499999962,55.98607999600006],[-4.979976,55.86157999800008],[-5.120735,56.00808349700003],[-5.074320499999942,55.90140149900003],[-5.190108,55.94739149900005],[-5.242238,55.89447799800007],[-5.205963,55.82628650000004],[-5.351127,55.90980549800008],[-5.34193149999993,55.995186],[-5.127448198999957,56.166530160000036],[-5.380715,56.00484449900006],[-5.440771499999926,56.03461450000003],[-5.419614,55.896728497000026],[-5.314025,55.783824999000046],[-5.448947,55.707809498000074],[-5.490266,55.52957549800004],[-5.568165000999954,55.42831399700003],[-5.52000449999997,55.36192299700008],[-5.799178,55.299698],[-5.66694,55.66451649900006],[-5.436028,55.85756299800005],[-5.613714,55.76017749700003],[-5.666273499999932,55.800583],[-5.571037499999932,55.93432999800007],[-5.688422499999945,55.911018496000054],[-5.566499,56.04157249900004],[-5.705351,55.93992999800008],[-5.503709499999957,56.18886549800004],[-5.613348,56.133239497000034],[-5.567006,56.23656099900006],[-5.487101000999928,56.256931500000064],[-5.654321,56.297649496000076],[-5.441643,56.365916997000056],[-5.589511,56.38047799600008],[-5.441085499999929,56.455798998000034],[-5.259811117999959,56.450957274000075],[-5.473969,56.481708499000035],[-5.384345607999933,56.53174223900004]]],[[[-2.848329500999967,58.958244499000045],[-2.788509000999966,58.915839995000056],[-2.704751499999929,58.961914],[-2.712558499999943,58.92079149700004],[-2.897167053999965,58.89524031000008],[-3.368161499999928,58.99882899900007],[-3.325062,59.13616199900008],[-3.196686,59.15369050000004],[-3.004050499999948,59.07222],[-3.11450150099995,59.00661450000007],[-2.911924499999941,59.006061498000065],[-2.801162499999975,58.99209999800007],[-2.848329500999967,58.958244499000045]]],[[[-3.259673,58.78805149900006],[-3.219864,58.775715],[-3.293498,58.77659249900006],[-3.434554499999933,58.87152850000007],[-3.32561149999998,58.92784899900005],[-3.170331,58.82104099800006],[-3.259673,58.78805149900006]]],[[[-5.082880499999931,55.55278799800004],[-5.083631,55.45461249700003],[-5.313261,55.46509149600007],[-5.39711000099993,55.61743149700004],[-5.288532499999974,55.71808599700006],[-5.201502499999947,55.702484],[-5.082880499999931,55.55278799800004]]],[[[-5.646332,56.44055950000006],[-5.834676,56.310558496000056],[-6.385666500999946,56.28878799700004],[-6.35029,56.34788899800003],[-6.190255,56.332347998000046],[-5.979565499999978,56.38647099800005],[-6.194926499999951,56.358978499000045],[-6.20478049999997,56.38530749700004],[-6.129291,56.448417498000026],[-5.996329499999945,56.48778549900004],[-6.340745,56.538177500000074],[-6.323570500999949,56.60616299700007],[-6.112694000999966,56.64403899900003],[-5.950804500999936,56.518039498000064],[-5.799656500999959,56.51579649700005],[-5.646332,56.44055950000006]]],[[[-5.793228499999941,56.012721999000064],[-5.96267649899994,55.794749997000054],[-6.09684249999998,55.871860498000046],[-5.924639498999966,55.962383499000055],[-6.006308,55.973919],[-5.964173,56.027405],[-5.687118998999949,56.12949350000008],[-5.793228499999941,56.012721999000064]]],[[[-6.101226499999939,55.84050350000007],[-6.01987,55.68493249900007],[-6.266864000999931,55.579871996000065],[-6.335771499999964,55.58950049800006],[-6.260210499999971,55.659263500000066],[-6.341544499999941,55.71799849700005],[-6.260346499999969,55.781940500000076],[-6.526495499999953,55.693542499000046],[-6.45583,55.85268399800003],[-6.120203,55.934597],[-6.101226499999939,55.84050350000007]]],[[[-6.167010499999947,58.34677099900006],[-6.344816,58.234204496000075],[-6.135358,58.25944149700007],[-6.601655500999925,58.08697899600003],[-6.368577499999958,58.07648849700007],[-6.388316,57.999859],[-6.573711,58.004272498000034],[-6.454871,57.99351499900007],[-6.469132499999944,57.939422499000045],[-6.652290499999935,57.91831599800008],[-6.712528,58.013172],[-6.588023500999952,58.05318850000003],[-6.69152,58.05827699800005],[-6.739541,57.991840496000066],[-6.663898,57.88521599900008],[-6.812832499999956,57.880672496000045],[-6.744717,57.82280749800003],[-6.967598,57.727894],[-7.133195499999943,57.836654500000066],[-6.81601349999994,57.900836998000045],[-7.079999499999929,57.967232],[-7.082464,58.02035899700007],[-6.907952,58.049675],[-7.103295000999935,58.07354749900003],[-7.108741499999951,58.177036498000064],[-7.027442,58.24435399600003],[-6.746137499999975,58.18470399700004],[-6.828675,58.284096],[-6.25831,58.516716],[-6.180591,58.46701049900008],[-6.217891499999951,58.36830499600006],[-6.167010499999947,58.34677099900006]]],[[[-7.182542499999954,57.65663150000006],[-7.062166,57.64057150000008],[-7.214868499999966,57.64065949800005],[-7.10083149999997,57.59365449900008],[-7.299847,57.553420997000046],[-7.133844,57.552966998000045],[-7.148515,57.50972349700004],[-7.549144499999954,57.60290549900003],[-7.152460499999961,57.665553997000075],[-7.182542499999954,57.65663150000006]]],[[[-7.35166,57.15094749700006],[-7.211596499999928,57.11778249900004],[-7.412508499999944,57.149811],[-7.412302499999953,57.38740150000007],[-7.19152,57.29863750000004],[-7.298812670999951,57.22601419200004],[-7.246141,57.16512699800006],[-7.35166,57.15094749700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Angus and Dundee City","NUTS_NAME":"Angus and Dundee City"},"geometry":{"type":"Polygon","coordinates":[[[-2.425346,56.755187996000075],[-2.538641,56.566062997000074],[-2.733122499999979,56.46479799900004],[-3.051892500999941,56.45849999700005],[-3.212033,56.524406498000076],[-3.088117000999944,56.61330049800006],[-3.363916500999949,56.73824299800003],[-3.37210749999997,56.87474449700005],[-2.851041000999942,56.97269799800006],[-2.681236499999955,56.91047299600007],[-2.616337499999929,56.78367250000008],[-2.425346,56.755187996000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Clackmannanshire and Fife","NUTS_NAME":"Clackmannanshire and Fife"},"geometry":{"type":"Polygon","coordinates":[[[-3.866839499999969,56.12066249900005],[-3.780869,56.21733849700007],[-3.571391499999947,56.19304649900005],[-3.639821499999925,56.133686],[-3.369888500999934,56.145950500000026],[-3.260916499999951,56.19618599900008],[-3.383621499999947,56.26887149800007],[-3.276975499999935,56.35053250000004],[-2.803087,56.43967799700005],[-2.838920499999972,56.35307299600004],[-2.620387,56.26118849900007],[-2.955285,56.21049499700007],[-3.39145149999996,56.00593199700006],[-3.866839499999969,56.12066249900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Lothian and Midlothian","NUTS_NAME":"East Lothian and Midlothian"},"geometry":{"type":"Polygon","coordinates":[[[-2.366623499999946,55.946113498000045],[-2.562669000999961,55.90673449800005],[-2.55483449999997,55.83918750000004],[-2.949831,55.81887799900005],[-3.103345499999932,55.737220997000065],[-3.369016500999976,55.82420750000006],[-3.07764549999996,55.94689549800006],[-2.815697499999942,56.062404496000056],[-2.366623499999946,55.946113498000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM24","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Scottish Borders","NUTS_NAME":"Scottish Borders"},"geometry":{"type":"Polygon","coordinates":[[[-2.034329,55.81116499900003],[-2.335961,55.63214499800006],[-2.165465499999925,55.46846750000003],[-2.496131499999933,55.346595999000044],[-2.689751,55.18906399700006],[-2.858508,55.10842499800003],[-2.904721,55.17374799800007],[-2.863809499999945,55.236431],[-2.917027,55.27749249800007],[-3.316549,55.34824],[-3.243565000999979,55.427772497000035],[-3.310962,55.444916],[-3.507365,55.41234949600005],[-3.539553,55.44326],[-3.486589,55.51714299900004],[-3.529996499999925,55.611484500000074],[-3.396386,55.716407996000044],[-3.471619,55.771068497000044],[-3.394744499999945,55.81983950000006],[-3.369016500999976,55.82420750000006],[-3.103345499999932,55.737220997000065],[-2.949831,55.81887799900005],[-2.55483449999997,55.83918750000004],[-2.562669000999961,55.90673449800005],[-2.366623499999946,55.946113498000045],[-2.13755649999996,55.91727849600005],[-2.034329,55.81116499900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM25","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Edinburgh, City of","NUTS_NAME":"Edinburgh, City of"},"geometry":{"type":"Polygon","coordinates":[[[-3.07764549999996,55.94689549800006],[-3.369016500999976,55.82420750000006],[-3.394744499999945,55.81983950000006],[-3.386966000999962,55.89801799900005],[-3.44418,55.90784449800003],[-3.425330499999973,55.99403],[-3.07764549999996,55.94689549800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM26","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Falkirk","NUTS_NAME":"Falkirk"},"geometry":{"type":"Polygon","coordinates":[[[-3.515724,56.00225850000004],[-3.822552,55.896595],[-4.020121,56.028145],[-3.797901,56.06551349700004],[-3.820153,56.099117500000034],[-3.515724,56.00225850000004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM27","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Perth & Kinross and Stirling","NUTS_NAME":"Perth & Kinross and Stirling"},"geometry":{"type":"Polygon","coordinates":[[[-3.37210749999997,56.87474449700005],[-3.363916500999949,56.73824299800003],[-3.088117000999944,56.61330049800006],[-3.212033,56.524406498000076],[-3.051892500999941,56.45849999700005],[-3.345321499999955,56.37197099600007],[-3.276975499999935,56.35053250000004],[-3.383621499999947,56.26887149800007],[-3.260916499999951,56.19618599900008],[-3.369888500999934,56.145950500000026],[-3.639821499999925,56.133686],[-3.571391499999947,56.19304649900005],[-3.780869,56.21733849700007],[-3.866839499999969,56.12066249900005],[-3.820153,56.099117500000034],[-3.797901,56.06551349700004],[-4.020121,56.028145],[-4.152381499999933,56.00814449900008],[-4.47081,56.002395500000034],[-4.598209,56.08433899800008],[-4.695389499999976,56.19928349600008],[-4.676723,56.31299199600005],[-4.854503499999964,56.370532998000044],[-4.63908349999997,56.47616199700008],[-4.697694499999955,56.549896],[-4.561758499999939,56.57027049900006],[-4.720448,56.65656299700004],[-4.532142499999964,56.795907998000075],[-3.801627499999938,56.936012497000036],[-3.37210749999997,56.87474449700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM28","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Lothian","NUTS_NAME":"West Lothian"},"geometry":{"type":"Polygon","coordinates":[[[-3.425330499999973,55.99403],[-3.44418,55.90784449800003],[-3.386966000999962,55.89801799900005],[-3.394744499999945,55.81983950000006],[-3.471619,55.771068497000044],[-3.744007499999952,55.78210849700008],[-3.717458499999964,55.88576149700003],[-3.822552,55.896595],[-3.515724,56.00225850000004],[-3.425330499999973,55.99403]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Dumfries & Galloway","NUTS_NAME":"Dumfries & Galloway"},"geometry":{"type":"Polygon","coordinates":[[[-3.507365,55.41234949600005],[-3.310962,55.444916],[-3.243565000999979,55.427772497000035],[-3.316549,55.34824],[-2.917027,55.27749249800007],[-2.863809499999945,55.236431],[-2.904721,55.17374799800007],[-2.858508,55.10842499800003],[-3.051281499999959,55.05262349800006],[-3.091623992999928,54.97554629100006],[-3.560086499999954,54.97845099800003],[-3.594912499999964,54.872833498000034],[-3.850849,54.86328899700004],[-3.824464499999976,54.82371149800008],[-3.981397,54.76880249700008],[-4.06509049999994,54.78116249700008],[-4.050716,54.849548499000036],[-4.105112499999962,54.76763549900005],[-4.425736,54.926769499000045],[-4.413721499999951,54.82943750000004],[-4.341246499999954,54.79918649600006],[-4.347857499999975,54.71093749700003],[-4.405439499999943,54.67750950000004],[-4.860413,54.86895749900003],[-4.960747,54.80627449700006],[-4.876391499999954,54.63921349900005],[-4.964987,54.66406249900007],[-4.957434499999977,54.72864899900003],[-5.183450000999926,54.912364998000044],[-5.161495,55.00796149900003],[-5.098928000999933,55.01810849900005],[-5.060561,54.92361050000005],[-4.990791499999943,54.92166149600007],[-5.040228499999955,54.997726500000056],[-4.646661,55.05647299900005],[-4.663614499999937,55.11446749800007],[-4.626393499999949,55.13945749800007],[-4.461868,55.170299499000066],[-4.300525,55.310744998000075],[-4.139340499999946,55.28467549800007],[-4.090828,55.41217399900006],[-3.98623,55.46414199700007],[-3.825497499999926,55.44450399900006],[-3.663608,55.291832],[-3.507365,55.41234949600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM33","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Ayrshire and North Ayrshire mainland","NUTS_NAME":"East Ayrshire and North Ayrshire mainland"},"geometry":{"type":"Polygon","coordinates":[[[-4.799727,55.89271549800003],[-4.671259499999962,55.77867149800005],[-4.246906499999966,55.67914950000005],[-4.175039499999968,55.60496499800007],[-4.228142,55.552272998000035],[-3.957049499999926,55.55584350000004],[-4.025692,55.49254250000007],[-3.98623,55.46414199700007],[-4.090828,55.41217399900006],[-4.139340499999946,55.28467549800007],[-4.300525,55.310744998000075],[-4.461868,55.170299499000066],[-4.437971499999946,55.30772799700003],[-4.603401,55.39524099800008],[-4.45037149999996,55.412273500000026],[-4.493813499999931,55.46902449700008],[-4.409498499999927,55.55337899700004],[-4.658272499999953,55.57028949800008],[-4.905256,55.69921899800005],[-4.855803499999979,55.760661998000046],[-4.888966499999981,55.87478999600006],[-4.799727,55.89271549800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM34","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Glasgow City","NUTS_NAME":"Glasgow City"},"geometry":{"type":"Polygon","coordinates":[[[-4.07842549999998,55.880641999000034],[-4.107039499999928,55.83475499900004],[-4.250752499999976,55.78498849700003],[-4.378092,55.800003],[-4.354563,55.873252998000055],[-4.391739821999977,55.88943051700005],[-4.392409,55.88967899900007],[-4.392671740999958,55.88981584100003],[-4.179462499999943,55.90214549600006],[-4.07842549999998,55.880641999000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM35","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Inverclyde, East Renfrewshire and Renfrewshire","NUTS_NAME":"Inverclyde, East Renfrewshire and Renfrewshire"},"geometry":{"type":"Polygon","coordinates":[[[-4.765130499999941,55.957771499000046],[-4.392671740999958,55.88981584100003],[-4.392409,55.88967899900007],[-4.391739821999977,55.88943051700005],[-4.354563,55.873252998000055],[-4.378092,55.800003],[-4.250752499999976,55.78498849700003],[-4.282508499999949,55.76940149900008],[-4.224057499999958,55.72930899800008],[-4.246906499999966,55.67914950000005],[-4.671259499999962,55.77867149800005],[-4.799727,55.89271549800003],[-4.888966499999981,55.87478999600006],[-4.878924499999926,55.94240199700005],[-4.765130499999941,55.957771499000046]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM36","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North Lanarkshire","NUTS_NAME":"North Lanarkshire"},"geometry":{"type":"Polygon","coordinates":[[[-3.822552,55.896595],[-3.717458499999964,55.88576149700003],[-3.744007499999952,55.78210849700008],[-3.919395,55.73510349900005],[-4.107039499999928,55.83475499900004],[-4.07842549999998,55.880641999000034],[-4.179462499999943,55.90214549600006],[-4.047734499999933,55.95128649800006],[-4.152381499999933,56.00814449900008],[-4.020121,56.028145],[-3.822552,55.896595]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM37","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South Ayrshire","NUTS_NAME":"South Ayrshire"},"geometry":{"type":"Polygon","coordinates":[[[-4.461868,55.170299499000066],[-4.626393499999949,55.13945749800007],[-4.663614499999937,55.11446749800007],[-4.646661,55.05647299900005],[-5.040228499999955,54.997726500000056],[-5.048733499999969,55.05751799600006],[-4.859926,55.227607499000044],[-4.768847499999936,55.400829498000064],[-4.650890499999946,55.441147],[-4.620535,55.49978649600007],[-4.658272499999953,55.57028949800008],[-4.409498499999927,55.55337899700004],[-4.493813499999931,55.46902449700008],[-4.45037149999996,55.412273500000026],[-4.603401,55.39524099800008],[-4.437971499999946,55.30772799700003],[-4.461868,55.170299499000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM38","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South Lanarkshire","NUTS_NAME":"South Lanarkshire"},"geometry":{"type":"Polygon","coordinates":[[[-4.107039499999928,55.83475499900004],[-3.919395,55.73510349900005],[-3.744007499999952,55.78210849700008],[-3.471619,55.771068497000044],[-3.396386,55.716407996000044],[-3.529996499999925,55.611484500000074],[-3.486589,55.51714299900004],[-3.539553,55.44326],[-3.507365,55.41234949600005],[-3.663608,55.291832],[-3.825497499999926,55.44450399900006],[-3.98623,55.46414199700007],[-4.025692,55.49254250000007],[-3.957049499999926,55.55584350000004],[-4.228142,55.552272998000035],[-4.175039499999968,55.60496499800007],[-4.246906499999966,55.67914950000005],[-4.224057499999958,55.72930899800008],[-4.282508499999949,55.76940149900008],[-4.250752499999976,55.78498849700003],[-4.107039499999928,55.83475499900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM50","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Aberdeen City and Aberdeenshire","NUTS_NAME":"Aberdeen City and Aberdeenshire"},"geometry":{"type":"Polygon","coordinates":[[[-2.425346,56.755187996000075],[-2.616337499999929,56.78367250000008],[-2.681236499999955,56.91047299600007],[-2.851041000999942,56.97269799800006],[-3.37210749999997,56.87474449700005],[-3.801627499999938,56.936012497000036],[-3.749711499999933,57.07641999900005],[-3.368641,57.10041799700008],[-3.322594,57.13163749900008],[-3.353487499999972,57.17829899600008],[-3.208485499999938,57.25625999600004],[-2.984190499999954,57.27787799600003],[-2.954439,57.37796],[-3.019933499999979,57.44965749800008],[-2.971998,57.49148199800004],[-2.649769,57.52970099700008],[-2.810592,57.61008449700006],[-2.801509,57.69524749900006],[-1.920823499999926,57.67439649600004],[-1.78944,57.503555497000036],[-2.042654,57.243911498000045],[-2.077448499999946,57.17697899800004],[-2.05034749999993,57.128338],[-2.206539500999952,56.96594999700005],[-2.214603,56.88209899700007],[-2.425346,56.755187996000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM64","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Eilean Siar (Western Isles)","NUTS_NAME":"Eilean Siar (Western Isles)"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-6.167010499999947,58.34677099900006],[-6.344816,58.234204496000075],[-6.135358,58.25944149700007],[-6.601655500999925,58.08697899600003],[-6.368577499999958,58.07648849700007],[-6.388316,57.999859],[-6.573711,58.004272498000034],[-6.454871,57.99351499900007],[-6.469132499999944,57.939422499000045],[-6.652290499999935,57.91831599800008],[-6.712528,58.013172],[-6.588023500999952,58.05318850000003],[-6.69152,58.05827699800005],[-6.739541,57.991840496000066],[-6.663898,57.88521599900008],[-6.812832499999956,57.880672496000045],[-6.744717,57.82280749800003],[-6.967598,57.727894],[-7.133195499999943,57.836654500000066],[-6.81601349999994,57.900836998000045],[-7.079999499999929,57.967232],[-7.082464,58.02035899700007],[-6.907952,58.049675],[-7.103295000999935,58.07354749900003],[-7.108741499999951,58.177036498000064],[-7.027442,58.24435399600003],[-6.746137499999975,58.18470399700004],[-6.828675,58.284096],[-6.25831,58.516716],[-6.180591,58.46701049900008],[-6.217891499999951,58.36830499600006],[-6.167010499999947,58.34677099900006]]],[[[-7.182542499999954,57.65663150000006],[-7.062166,57.64057150000008],[-7.214868499999966,57.64065949800005],[-7.10083149999997,57.59365449900008],[-7.299847,57.553420997000046],[-7.133844,57.552966998000045],[-7.148515,57.50972349700004],[-7.549144499999954,57.60290549900003],[-7.152460499999961,57.665553997000075],[-7.182542499999954,57.65663150000006]]],[[[-7.35166,57.15094749700006],[-7.211596499999928,57.11778249900004],[-7.412508499999944,57.149811],[-7.412302499999953,57.38740150000007],[-7.19152,57.29863750000004],[-7.298812670999951,57.22601419200004],[-7.246141,57.16512699800006],[-7.35166,57.15094749700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM65","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Orkney Islands","NUTS_NAME":"Orkney Islands"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-2.848329500999967,58.958244499000045],[-2.788509000999966,58.915839995000056],[-2.704751499999929,58.961914],[-2.712558499999943,58.92079149700004],[-2.897167053999965,58.89524031000008],[-3.368161499999928,58.99882899900007],[-3.325062,59.13616199900008],[-3.196686,59.15369050000004],[-3.004050499999948,59.07222],[-3.11450150099995,59.00661450000007],[-2.911924499999941,59.006061498000065],[-2.801162499999975,58.99209999800007],[-2.848329500999967,58.958244499000045]]],[[[-3.259673,58.78805149900006],[-3.219864,58.775715],[-3.293498,58.77659249900006],[-3.434554499999933,58.87152850000007],[-3.32561149999998,58.92784899900005],[-3.170331,58.82104099800006],[-3.259673,58.78805149900006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM66","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Shetland Islands","NUTS_NAME":"Shetland Islands"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.75837,60.81646749600003],[-0.831153,60.685222499000076],[-0.981795499999976,60.685482],[-0.865963499999964,60.83370199700005],[-0.75837,60.81646749600003]]],[[[-1.003055499999959,60.58155799700006],[-1.028645,60.49721549700007],[-1.167948499999966,60.49954999900007],[-1.202889,60.607105499000056],[-1.074501,60.73317349700005],[-1.008528,60.71493899900008],[-0.981463500999951,60.637199500000065],[-1.031584500999941,60.59758399900005],[-1.003055499999959,60.58155799700006]]],[[[-1.304159,60.62399649900004],[-1.319979499999931,60.451770999000075],[-1.383588,60.397674496000036],[-1.237423000999968,60.49381999700006],[-1.049048,60.43949149600007],[-1.155833499999972,60.33803549900006],[-1.079679499999941,60.32329199900005],[-1.15929,60.281048],[-1.095383499999969,60.26348899900006],[-1.170235,60.23842599900007],[-1.155375499999934,60.15899649800008],[-1.203963500999976,60.130611499000054],[-1.277545,59.85259999700003],[-1.391431500999943,59.91602349900006],[-1.276649,60.099434],[-1.32783749999993,60.16605749900003],[-1.293532499999969,60.251404],[-1.462364499999978,60.14639299900006],[-1.702932,60.255359498000075],[-1.339904500999978,60.359771497000054],[-1.632932,60.48316199900006],[-1.499491318999958,60.543659871000045],[-1.489197784999931,60.54412077600006],[-1.304159,60.62399649900004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"NORTHERN IRELAND","NUTS_NAME":"NORTHERN IRELAND"},"geometry":{"type":"Polygon","coordinates":[[[-5.976527499999975,55.056598500000064],[-5.992633499999954,54.989261998000075],[-5.721779,54.77285249600004],[-5.720909000999939,54.772636498000054],[-5.780185,54.858516500000064],[-5.691673,54.80959299800003],[-5.690664000999959,54.769462496000074],[-5.912931499999956,54.64803699700008],[-5.855336,54.63376999900004],[-5.575003,54.67019650000003],[-5.432802,54.48764399700008],[-5.499689499999931,54.338054498000076],[-5.579466499999967,54.40056599800005],[-5.536291499999948,54.431365999000036],[-5.561936,54.53355049900006],[-5.679648499999928,54.58318649800003],[-5.724728499999969,54.53908899800007],[-5.643023,54.51126499700007],[-5.65442,54.381484998000076],[-5.542285,54.32002999800005],[-5.874409,54.184848500000044],[-5.896720499999958,54.10493099900003],[-6.062905,54.02288050000004],[-6.268015499999933,54.102336999000045],[-6.623778499999958,54.03654849600008],[-6.668772499999932,54.07254799900005],[-6.643909,54.180019497000046],[-6.79513849999995,54.20911399800008],[-6.910962499999926,54.37472549900008],[-7.02863549999995,54.421306497000046],[-7.211874499999965,54.299679],[-7.145823499999949,54.22494499900006],[-7.248977,54.20340750000008],[-7.279176499999949,54.12441650000005],[-7.290923499999963,54.17190949600007],[-7.565956,54.12651449800006],[-7.855390499999942,54.22252249600007],[-7.872462499999926,54.29415499800007],[-8.177718,54.46497350000004],[-7.703411500999948,54.60828799800004],[-7.921372,54.69654449800004],[-7.534506,54.747139],[-7.548892,54.789589498000055],[-7.44311,54.87147149800006],[-7.396395499999926,55.016722997000045],[-7.256068499999969,55.067034997000064],[-7.072085,55.046199999000066],[-6.960326,55.17870349900005],[-6.455277,55.23930349900007],[-6.139729,55.22702049900005],[-6.027018,55.16146449800004],[-6.057595499999934,55.06350699700005],[-5.976527499999975,55.056598500000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN0","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Northern Ireland","NUTS_NAME":"Northern Ireland"},"geometry":{"type":"Polygon","coordinates":[[[-5.976527499999975,55.056598500000064],[-5.992633499999954,54.989261998000075],[-5.721779,54.77285249600004],[-5.720909000999939,54.772636498000054],[-5.780185,54.858516500000064],[-5.691673,54.80959299800003],[-5.690664000999959,54.769462496000074],[-5.912931499999956,54.64803699700008],[-5.855336,54.63376999900004],[-5.575003,54.67019650000003],[-5.432802,54.48764399700008],[-5.499689499999931,54.338054498000076],[-5.579466499999967,54.40056599800005],[-5.536291499999948,54.431365999000036],[-5.561936,54.53355049900006],[-5.679648499999928,54.58318649800003],[-5.724728499999969,54.53908899800007],[-5.643023,54.51126499700007],[-5.65442,54.381484998000076],[-5.542285,54.32002999800005],[-5.874409,54.184848500000044],[-5.896720499999958,54.10493099900003],[-6.062905,54.02288050000004],[-6.268015499999933,54.102336999000045],[-6.623778499999958,54.03654849600008],[-6.668772499999932,54.07254799900005],[-6.643909,54.180019497000046],[-6.79513849999995,54.20911399800008],[-6.910962499999926,54.37472549900008],[-7.02863549999995,54.421306497000046],[-7.211874499999965,54.299679],[-7.145823499999949,54.22494499900006],[-7.248977,54.20340750000008],[-7.279176499999949,54.12441650000005],[-7.290923499999963,54.17190949600007],[-7.565956,54.12651449800006],[-7.855390499999942,54.22252249600007],[-7.872462499999926,54.29415499800007],[-8.177718,54.46497350000004],[-7.703411500999948,54.60828799800004],[-7.921372,54.69654449800004],[-7.534506,54.747139],[-7.548892,54.789589498000055],[-7.44311,54.87147149800006],[-7.396395499999926,55.016722997000045],[-7.256068499999969,55.067034997000064],[-7.072085,55.046199999000066],[-6.960326,55.17870349900005],[-6.455277,55.23930349900007],[-6.139729,55.22702049900005],[-6.027018,55.16146449800004],[-6.057595499999934,55.06350699700005],[-5.976527499999975,55.056598500000064]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN01","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Belfast","NUTS_NAME":"Belfast"},"geometry":{"type":"Polygon","coordinates":[[[-5.912931499999956,54.64803699700008],[-5.855336,54.63376999900004],[-5.809016,54.61549749800008],[-5.976565500999925,54.53077299700004],[-6.045803499999977,54.60599499800003],[-5.986533,54.659297999000046],[-5.912931499999956,54.64803699700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN02","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Outer Belfast","NUTS_NAME":"Outer Belfast"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-5.770635499999969,54.61195350000003],[-5.950804,54.37976049800005],[-6.210429,54.43279649800007],[-6.304308499999934,54.572796],[-6.045803499999977,54.60599499800003],[-5.976565500999925,54.53077299700004],[-5.809016,54.61549749800008],[-5.855336,54.63376999900004],[-5.575003,54.67019650000003],[-5.770635499999969,54.61195350000003]]],[[[-5.691673,54.80959299800003],[-5.690664000999959,54.769462496000074],[-5.912931499999956,54.64803699700008],[-5.986533,54.659297999000046],[-6.05192349999993,54.66431049800008],[-6.102755,54.792628998000055],[-5.721779,54.77285249600004],[-5.720909000999939,54.772636498000054],[-5.780185,54.858516500000064],[-5.691673,54.80959299800003]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN03","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East of Northern Ireland","NUTS_NAME":"East of Northern Ireland"},"geometry":{"type":"Polygon","coordinates":[[[-5.992633499999954,54.989261998000075],[-5.721779,54.77285249600004],[-6.102755,54.792628998000055],[-6.05192349999993,54.66431049800008],[-5.986533,54.659297999000046],[-6.045803499999977,54.60599499800003],[-6.304308499999934,54.572796],[-6.210429,54.43279649800007],[-5.950804,54.37976049800005],[-5.770635499999969,54.61195350000003],[-5.575003,54.67019650000003],[-5.432802,54.48764399700008],[-5.499689499999931,54.338054498000076],[-5.579466499999967,54.40056599800005],[-5.536291499999948,54.431365999000036],[-5.561936,54.53355049900006],[-5.679648499999928,54.58318649800003],[-5.724728499999969,54.53908899800007],[-5.643023,54.51126499700007],[-5.65442,54.381484998000076],[-5.542285,54.32002999800005],[-5.874409,54.184848500000044],[-6.330851,54.26377849900007],[-6.401486499999976,54.39864749900005],[-6.566696499999978,54.46605299600003],[-6.588373,54.507907998000064],[-6.426119,54.568114999000045],[-6.40702349999998,54.64842999900003],[-6.496078499999953,54.713814],[-6.457087499999943,54.82454699800007],[-6.500022,54.91871249800005],[-5.976527499999975,55.056598500000064],[-5.992633499999954,54.989261998000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN04","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North of Northern Ireland","NUTS_NAME":"North of Northern Ireland"},"geometry":{"type":"Polygon","coordinates":[[[-6.455277,55.23930349900007],[-6.139729,55.22702049900005],[-6.027018,55.16146449800004],[-6.057595499999934,55.06350699700005],[-5.976527499999975,55.056598500000064],[-6.500022,54.91871249800005],[-6.775439,54.91451649800007],[-6.924585499999978,54.77289199900008],[-7.703411500999948,54.60828799800004],[-7.921372,54.69654449800004],[-7.534506,54.747139],[-7.548892,54.789589498000055],[-7.44311,54.87147149800006],[-7.396395499999926,55.016722997000045],[-7.256068499999969,55.067034997000064],[-7.072085,55.046199999000066],[-6.960326,55.17870349900005],[-6.455277,55.23930349900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKN05","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West and South of Northern Ireland","NUTS_NAME":"West and South of Northern Ireland"},"geometry":{"type":"Polygon","coordinates":[[[-6.500022,54.91871249800005],[-6.457087499999943,54.82454699800007],[-6.496078499999953,54.713814],[-6.40702349999998,54.64842999900003],[-6.426119,54.568114999000045],[-6.588373,54.507907998000064],[-6.566696499999978,54.46605299600003],[-6.401486499999976,54.39864749900005],[-6.330851,54.26377849900007],[-5.874409,54.184848500000044],[-5.896720499999958,54.10493099900003],[-6.062905,54.02288050000004],[-6.268015499999933,54.102336999000045],[-6.623778499999958,54.03654849600008],[-6.668772499999932,54.07254799900005],[-6.643909,54.180019497000046],[-6.79513849999995,54.20911399800008],[-6.910962499999926,54.37472549900008],[-7.02863549999995,54.421306497000046],[-7.211874499999965,54.299679],[-7.145823499999949,54.22494499900006],[-7.248977,54.20340750000008],[-7.279176499999949,54.12441650000005],[-7.290923499999963,54.17190949600007],[-7.565956,54.12651449800006],[-7.855390499999942,54.22252249600007],[-7.872462499999926,54.29415499800007],[-8.177718,54.46497350000004],[-7.703411500999948,54.60828799800004],[-6.924585499999978,54.77289199900008],[-6.775439,54.91451649800007],[-6.500022,54.91871249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM63","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Lochaber, Skye & Lochalsh, Arran & Cumbrae and Argyll & Bute","NUTS_NAME":"Lochaber, Skye & Lochalsh, Arran & Cumbrae and Argyll & Bute"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-6.146597000999975,57.57447449700004],[-6.130341,57.316936498000075],[-5.88541349999997,57.238167],[-5.776867,57.27114499700008],[-5.64761249999998,57.25529499700008],[-5.664395153999976,57.219288446000064],[-5.415392,57.23015599900003],[-5.73379,57.283454996000046],[-5.552481264999926,57.367434305000074],[-5.385484499999961,57.364482999000074],[-5.294564,57.43513849900006],[-5.217820499999959,57.43837749900007],[-5.096129,57.41197599900005],[-5.01045449999998,57.349815499000044],[-5.048863499999925,57.299747499000034],[-5.283571,57.22115699900007],[-5.067874499999959,57.16604599900006],[-5.08938349999994,57.12227999900006],[-5.04175,57.10871099900004],[-4.832698,57.131961999000055],[-4.62738,57.043246999000075],[-4.569325,56.84575649900006],[-4.532142499999964,56.795907998000075],[-4.720448,56.65656299700004],[-4.561758499999939,56.57027049900006],[-4.697694499999955,56.549896],[-4.63908349999997,56.47616199700008],[-4.854503499999964,56.370532998000044],[-4.771805749999942,56.343766781000056],[-4.836395499999981,56.291446499000074],[-4.732895499999927,56.23231549900004],[-4.752816000999928,56.20385749700006],[-4.862619499999937,56.100319],[-4.917712,56.164127499000074],[-4.896133499999962,55.98607999600006],[-4.979976,55.86157999800008],[-5.120735,56.00808349700003],[-5.074320499999942,55.90140149900003],[-5.190108,55.94739149900005],[-5.242238,55.89447799800007],[-5.205963,55.82628650000004],[-5.351127,55.90980549800008],[-5.34193149999993,55.995186],[-5.127448198999957,56.166530160000036],[-5.380715,56.00484449900006],[-5.440771499999926,56.03461450000003],[-5.419614,55.896728497000026],[-5.314025,55.783824999000046],[-5.448947,55.707809498000074],[-5.490266,55.52957549800004],[-5.568165000999954,55.42831399700003],[-5.52000449999997,55.36192299700008],[-5.799178,55.299698],[-5.66694,55.66451649900006],[-5.436028,55.85756299800005],[-5.613714,55.76017749700003],[-5.666273499999932,55.800583],[-5.571037499999932,55.93432999800007],[-5.688422499999945,55.911018496000054],[-5.566499,56.04157249900004],[-5.705351,55.93992999800008],[-5.503709499999957,56.18886549800004],[-5.613348,56.133239497000034],[-5.567006,56.23656099900006],[-5.487101000999928,56.256931500000064],[-5.654321,56.297649496000076],[-5.441643,56.365916997000056],[-5.589511,56.38047799600008],[-5.441085499999929,56.455798998000034],[-5.259811117999959,56.450957274000075],[-5.473969,56.481708499000035],[-5.384345607999933,56.53174223900004],[-5.421163,56.54778649900004],[-5.325605499999938,56.648285],[-4.969490499999949,56.714679498000066],[-5.247245499999963,56.70311749700005],[-5.092236,56.83002449600008],[-5.334743499999945,56.859508497000036],[-5.122688000999972,56.827274498000065],[-5.684919,56.497295499000074],[-6.006905,56.645126497000035],[-5.867277643999955,56.675698871000066],[-6.226232,56.710991],[-5.751703000999953,56.78216549700005],[-5.887583499999948,56.792953999000076],[-5.722717500999977,56.889038],[-5.924258,56.89156349700005],[-5.820707,57.01260749700003],[-5.50935149999998,56.99639499700004],[-5.797323,57.06098549700005],[-5.72895,57.10147849700007],[-5.509437499999933,57.097774498000035],[-5.681389,57.15053049900007],[-5.670602562999932,57.20843875300005],[-6.017994,57.01794799800007],[-6.004551,57.114955999000074],[-5.908913,57.17123799900003],[-6.32237,57.160293498000044],[-6.483495,57.310973999000055],[-6.307433,57.299278497000046],[-6.789714499999945,57.42120749800006],[-6.716572,57.51391599900006],[-6.582203,57.423621996000065],[-6.637269499999945,57.50403199800007],[-6.562680499999942,57.504444],[-6.638996499999962,57.549937998000075],[-6.635986499999944,57.60881799900005],[-6.406945,57.523063498000056],[-6.427763499999969,57.64247149900007],[-6.296708,57.70756149700003],[-6.146597000999975,57.57447449700004]]],[[[-5.082880499999931,55.55278799800004],[-5.083631,55.45461249700003],[-5.313261,55.46509149600007],[-5.39711000099993,55.61743149700004],[-5.288532499999974,55.71808599700006],[-5.201502499999947,55.702484],[-5.082880499999931,55.55278799800004]]],[[[-5.646332,56.44055950000006],[-5.834676,56.310558496000056],[-6.385666500999946,56.28878799700004],[-6.35029,56.34788899800003],[-6.190255,56.332347998000046],[-5.979565499999978,56.38647099800005],[-6.194926499999951,56.358978499000045],[-6.20478049999997,56.38530749700004],[-6.129291,56.448417498000026],[-5.996329499999945,56.48778549900004],[-6.340745,56.538177500000074],[-6.323570500999949,56.60616299700007],[-6.112694000999966,56.64403899900003],[-5.950804500999936,56.518039498000064],[-5.799656500999959,56.51579649700005],[-5.646332,56.44055950000006]]],[[[-5.793228499999941,56.012721999000064],[-5.96267649899994,55.794749997000054],[-6.09684249999998,55.871860498000046],[-5.924639498999966,55.962383499000055],[-6.006308,55.973919],[-5.964173,56.027405],[-5.687118998999949,56.12949350000008],[-5.793228499999941,56.012721999000064]]],[[[-6.101226499999939,55.84050350000007],[-6.01987,55.68493249900007],[-6.266864000999931,55.579871996000065],[-6.335771499999964,55.58950049800006],[-6.260210499999971,55.659263500000066],[-6.341544499999941,55.71799849700005],[-6.260346499999969,55.781940500000076],[-6.526495499999953,55.693542499000046],[-6.45583,55.85268399800003],[-6.120203,55.934597],[-6.101226499999939,55.84050350000007]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Dunbartonshire, West Dunbartonshire and Helensburgh & Lomond","NUTS_NAME":"East Dunbartonshire, West Dunbartonshire and Helensburgh & Lomond"},"geometry":{"type":"Polygon","coordinates":[[[-4.676723,56.31299199600005],[-4.695389499999976,56.19928349600008],[-4.598209,56.08433899800008],[-4.47081,56.002395500000034],[-4.152381499999933,56.00814449900008],[-4.047734499999933,55.95128649800006],[-4.179462499999943,55.90214549600006],[-4.392671740999958,55.88981584100003],[-4.609648,55.94667449700006],[-4.828885,56.07900999700007],[-4.766975,55.98970399800004],[-4.853408499999944,55.99036049800003],[-4.880223,56.058253998000055],[-4.752816000999928,56.20385749700006],[-4.732895499999927,56.23231549900004],[-4.836395499999981,56.291446499000074],[-4.771805749999942,56.343766781000056],[-4.676723,56.31299199600005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM62","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Inverness & Nairn and Moray, Badenoch & Strathspey","NUTS_NAME":"Inverness & Nairn and Moray, Badenoch & Strathspey"},"geometry":{"type":"Polygon","coordinates":[[[-2.801509,57.69524749900006],[-2.810592,57.61008449700006],[-2.649769,57.52970099700008],[-2.971998,57.49148199800004],[-3.019933499999979,57.44965749800008],[-2.954439,57.37796],[-2.984190499999954,57.27787799600003],[-3.208485499999938,57.25625999600004],[-3.353487499999972,57.17829899600008],[-3.322594,57.13163749900008],[-3.368641,57.10041799700008],[-3.749711499999933,57.07641999900005],[-3.801627499999938,56.936012497000036],[-4.532142499999964,56.795907998000075],[-4.569325,56.84575649900006],[-4.62738,57.043246999000075],[-4.832698,57.131961999000055],[-5.04175,57.10871099900004],[-5.08938349999994,57.12227999900006],[-5.067874499999959,57.16604599900006],[-5.283571,57.22115699900007],[-5.048863499999925,57.299747499000034],[-5.01045449999998,57.349815499000044],[-5.096129,57.41197599900005],[-4.663278499999933,57.497073999000065],[-4.432052499999941,57.494354],[-4.191771,57.48384099900005],[-4.019451499999946,57.598220999000034],[-3.347729,57.72444899900006],[-2.801509,57.69524749900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM61","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Caithness & Sutherland and Ross & Cromarty","NUTS_NAME":"Caithness & Sutherland and Ross & Cromarty"},"geometry":{"type":"Polygon","coordinates":[[[-3.130931,58.519863],[-3.046968499999934,58.47255350000006],[-3.116977499999962,58.368579999000076],[-4.082477499999925,57.949943500000074],[-4.001344,57.92654399800006],[-4.016409,57.85973750000005],[-4.400494,57.918987497000046],[-4.04890249999994,57.81504049600005],[-3.771491499999968,57.867034998000065],[-3.975255499999946,57.69451899900008],[-4.301044,57.67506799700004],[-4.428933,57.574283499000046],[-3.993433499999981,57.67645649800005],[-4.234324499999957,57.500965],[-4.432052499999941,57.494354],[-4.663278499999933,57.497073999000065],[-5.096129,57.41197599900005],[-5.217820499999959,57.43837749900007],[-5.294564,57.43513849900006],[-5.385484499999961,57.364482999000074],[-5.552481264999926,57.367434305000074],[-5.441254,57.418952997000076],[-5.82202,57.363647496000056],[-5.872894,57.47562050000005],[-5.841507499999977,57.57624799900003],[-5.513192499999946,57.54019549900005],[-5.81922,57.639786],[-5.673280499999976,57.70169849900003],[-5.813466499999947,57.75067150000007],[-5.81416349999995,57.85871099900004],[-5.683442,57.86545199900007],[-5.619121,57.766895497000064],[-5.577558499999952,57.79293449800008],[-5.655929499999957,57.87793349800006],[-5.618142,57.92462899800006],[-5.217599,57.84221649700004],[-5.406790499999943,57.92776849900008],[-5.360826000999964,57.93917449800006],[-5.071033000999932,57.81991949700006],[-5.458675,58.074409495000054],[-5.268833,58.10244799800006],[-5.403668,58.23530949700006],[-5.375456,58.26508699900006],[-4.924184000999958,58.25644699700007],[-5.134424,58.27807599700003],[-5.182866499999932,58.35124949800007],[-5.033927499999947,58.39294799700008],[-5.114617499999952,58.415272],[-5.072923,58.452243999000075],[-4.988059499999963,58.425987],[-5.126430499999969,58.48945599700005],[-5.007117499999936,58.625992],[-4.65146249999998,58.534366498000054],[-4.739715499999932,58.44739149800006],[-4.589487499999962,58.57794549700003],[-4.409354499999949,58.523773],[-4.475344,58.44438949700003],[-4.357074,58.536857498000074],[-3.024449,58.644264],[-3.130931,58.519863]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"South Western Scotland","NUTS_NAME":"South Western Scotland"},"geometry":{"type":"Polygon","coordinates":[[[-4.598209,56.08433899800008],[-4.47081,56.002395500000034],[-4.152381499999933,56.00814449900008],[-4.020121,56.028145],[-3.822552,55.896595],[-3.717458499999964,55.88576149700003],[-3.744007499999952,55.78210849700008],[-3.471619,55.771068497000044],[-3.396386,55.716407996000044],[-3.529996499999925,55.611484500000074],[-3.486589,55.51714299900004],[-3.539553,55.44326],[-3.507365,55.41234949600005],[-3.310962,55.444916],[-3.243565000999979,55.427772497000035],[-3.316549,55.34824],[-2.917027,55.27749249800007],[-2.863809499999945,55.236431],[-2.904721,55.17374799800007],[-2.858508,55.10842499800003],[-3.051281499999959,55.05262349800006],[-3.091623992999928,54.97554629100006],[-3.560086499999954,54.97845099800003],[-3.594912499999964,54.872833498000034],[-3.850849,54.86328899700004],[-3.824464499999976,54.82371149800008],[-3.981397,54.76880249700008],[-4.06509049999994,54.78116249700008],[-4.050716,54.849548499000036],[-4.105112499999962,54.76763549900005],[-4.425736,54.926769499000045],[-4.413721499999951,54.82943750000004],[-4.341246499999954,54.79918649600006],[-4.347857499999975,54.71093749700003],[-4.405439499999943,54.67750950000004],[-4.860413,54.86895749900003],[-4.960747,54.80627449700006],[-4.876391499999954,54.63921349900005],[-4.964987,54.66406249900007],[-4.957434499999977,54.72864899900003],[-5.183450000999926,54.912364998000044],[-5.161495,55.00796149900003],[-5.098928000999933,55.01810849900005],[-5.060561,54.92361050000005],[-4.990791499999943,54.92166149600007],[-5.040228499999955,54.997726500000056],[-5.048733499999969,55.05751799600006],[-4.859926,55.227607499000044],[-4.768847499999936,55.400829498000064],[-4.650890499999946,55.441147],[-4.620535,55.49978649600007],[-4.658272499999953,55.57028949800008],[-4.905256,55.69921899800005],[-4.855803499999979,55.760661998000046],[-4.888966499999981,55.87478999600006],[-4.878924499999926,55.94240199700005],[-4.765130499999941,55.957771499000046],[-4.392671740999958,55.88981584100003],[-4.609648,55.94667449700006],[-4.828885,56.07900999700007],[-4.766975,55.98970399800004],[-4.853408499999944,55.99036049800003],[-4.880223,56.058253998000055],[-4.752816000999928,56.20385749700006],[-4.732895499999927,56.23231549900004],[-4.836395499999981,56.291446499000074],[-4.771805749999942,56.343766781000056],[-4.676723,56.31299199600005],[-4.695389499999976,56.19928349600008],[-4.598209,56.08433899800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM5","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"North Eastern Scotland","NUTS_NAME":"North Eastern Scotland"},"geometry":{"type":"Polygon","coordinates":[[[-2.425346,56.755187996000075],[-2.616337499999929,56.78367250000008],[-2.681236499999955,56.91047299600007],[-2.851041000999942,56.97269799800006],[-3.37210749999997,56.87474449700005],[-3.801627499999938,56.936012497000036],[-3.749711499999933,57.07641999900005],[-3.368641,57.10041799700008],[-3.322594,57.13163749900008],[-3.353487499999972,57.17829899600008],[-3.208485499999938,57.25625999600004],[-2.984190499999954,57.27787799600003],[-2.954439,57.37796],[-3.019933499999979,57.44965749800008],[-2.971998,57.49148199800004],[-2.649769,57.52970099700008],[-2.810592,57.61008449700006],[-2.801509,57.69524749900006],[-1.920823499999926,57.67439649600004],[-1.78944,57.503555497000036],[-2.042654,57.243911498000045],[-2.077448499999946,57.17697899800004],[-2.05034749999993,57.128338],[-2.206539500999952,56.96594999700005],[-2.214603,56.88209899700007],[-2.425346,56.755187996000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Northumberland","NUTS_NAME":"Northumberland"},"geometry":{"type":"Polygon","coordinates":[[[-1.461693,55.07439049900006],[-1.713691,55.065552],[-1.852595,54.91747649700005],[-1.820951,54.90572749900008],[-2.31204300099995,54.79108049900003],[-2.567809,54.82364249900007],[-2.605391499999939,54.88444899800004],[-2.545466499999975,54.92846299900003],[-2.601320500999975,54.97138999600003],[-2.48629649999998,55.08311849800003],[-2.689751,55.18906399700006],[-2.496131499999933,55.346595999000044],[-2.165465499999925,55.46846750000003],[-2.335961,55.63214499800006],[-2.034329,55.81116499900003],[-1.838949499999956,55.64233399900007],[-1.639348,55.57831949700005],[-1.570851,55.275554500000055],[-1.503029,55.196361499000034],[-1.526808499999959,55.13655849700007],[-1.461693,55.07439049900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Tyneside","NUTS_NAME":"Tyneside"},"geometry":{"type":"Polygon","coordinates":[[[-1.36390149999994,54.944183499000076],[-1.559341500999949,54.88210299800005],[-1.820951,54.90572749900008],[-1.852595,54.91747649700005],[-1.713691,55.065552],[-1.461693,55.07439049900006],[-1.36390149999994,54.944183499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKC23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Sunderland","NUTS_NAME":"Sunderland"},"geometry":{"type":"Polygon","coordinates":[[[-1.347372499999949,54.86069099800005],[-1.493438,54.80006049800005],[-1.559341500999949,54.88210299800005],[-1.36390149999994,54.944183499000076],[-1.347372499999949,54.86069099800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"NORTH WEST (ENGLAND)","NUTS_NAME":"NORTH WEST (ENGLAND)"},"geometry":{"type":"Polygon","coordinates":[[[-2.545466499999975,54.92846299900003],[-2.605391499999939,54.88444899800004],[-2.567809,54.82364249900007],[-2.31204300099995,54.79108049900003],[-2.351850499999955,54.68591699600006],[-2.172482499999944,54.53248199700005],[-2.170166499999937,54.45825599900007],[-2.367673,54.35612499800004],[-2.309801499999935,54.32436749900006],[-2.322618,54.245509998000045],[-2.460827499999937,54.22676449700003],[-2.56470749999994,54.12711349800003],[-2.469517,54.04625699700006],[-2.184482499999945,53.952304998000045],[-2.046089,53.85017799900004],[-2.06121,53.825671998000075],[-2.128320499999973,53.799068498000054],[-2.173259,53.72304549900008],[-2.146293000999947,53.68226599900004],[-2.048864499999979,53.680259500000034],[-1.909583,53.53842149800005],[-1.963350499999933,53.50985349600006],[-2.031023499999947,53.370289],[-1.987376,53.21360799900003],[-2.380991,53.05268499700003],[-2.380768,52.998428499000056],[-2.699274500999934,52.995460500000036],[-2.726823499999966,52.983295499000064],[-2.834814,52.998698999000055],[-2.99507449999993,53.154220499000076],[-2.922283,53.18931599600006],[-3.084193,53.256122497000035],[-3.110706,53.29631799800006],[-3.20036750099996,53.38752749900004],[-3.040308,53.44292849800007],[-2.928556499999956,53.30827699900004],[-2.752411499999937,53.31475449900006],[-2.675166499999932,53.35448449900008],[-2.693358,53.36183549700007],[-2.826677,53.33167250000008],[-3.008742,53.43841149800005],[-3.105458499999941,53.551548],[-2.956203,53.697529],[-2.833743,53.72212999800007],[-2.757161000999929,53.75646949900005],[-3.057371499999931,53.77648149800007],[-3.047940499999925,53.875774498000055],[-2.853505,53.96575949800007],[-2.924294499999974,54.031154496000056],[-2.794437,54.12753699800004],[-2.86947,54.17673849800008],[-2.789148499999953,54.22734049700006],[-3.016447,54.13262549700005],[-3.03217,54.245723499000064],[-3.149062,54.093600999000046],[-3.241605000999925,54.10834499900005],[-3.256430499999965,54.16888450000005],[-3.204951499999936,54.21513],[-3.223693608999952,54.249058382000044],[-3.330026499999974,54.19639199900007],[-3.613598,54.52509699800004],[-3.578772499999957,54.651966],[-3.433302,54.76629649800003],[-3.398636499999952,54.86912549800007],[-3.251950499999964,54.90313699900008],[-3.314172499999927,54.91571799900004],[-3.285158,54.94161599800003],[-3.11915,54.927886998000076],[-3.00619949999998,54.94408049700007],[-3.131486,54.94579299700007],[-3.091623992999928,54.97554629100006],[-3.051281499999959,55.05262349800006],[-2.858508,55.10842499800003],[-2.689751,55.18906399700006],[-2.48629649999998,55.08311849800003],[-2.601320500999975,54.97138999600003],[-2.545466499999975,54.92846299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Cumbria","NUTS_NAME":"Cumbria"},"geometry":{"type":"Polygon","coordinates":[[[-2.545466499999975,54.92846299900003],[-2.605391499999939,54.88444899800004],[-2.567809,54.82364249900007],[-2.31204300099995,54.79108049900003],[-2.351850499999955,54.68591699600006],[-2.172482499999944,54.53248199700005],[-2.170166499999937,54.45825599900007],[-2.367673,54.35612499800004],[-2.309801499999935,54.32436749900006],[-2.322618,54.245509998000045],[-2.460827499999937,54.22676449700003],[-2.86947,54.17673849800008],[-2.789148499999953,54.22734049700006],[-3.016447,54.13262549700005],[-3.03217,54.245723499000064],[-3.149062,54.093600999000046],[-3.241605000999925,54.10834499900005],[-3.256430499999965,54.16888450000005],[-3.204951499999936,54.21513],[-3.223693608999952,54.249058382000044],[-3.330026499999974,54.19639199900007],[-3.613598,54.52509699800004],[-3.578772499999957,54.651966],[-3.433302,54.76629649800003],[-3.398636499999952,54.86912549800007],[-3.251950499999964,54.90313699900008],[-3.314172499999927,54.91571799900004],[-3.285158,54.94161599800003],[-3.11915,54.927886998000076],[-3.00619949999998,54.94408049700007],[-3.131486,54.94579299700007],[-3.091623992999928,54.97554629100006],[-3.051281499999959,55.05262349800006],[-2.858508,55.10842499800003],[-2.689751,55.18906399700006],[-2.48629649999998,55.08311849800003],[-2.601320500999975,54.97138999600003],[-2.545466499999975,54.92846299900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Cumbria","NUTS_NAME":"West Cumbria"},"geometry":{"type":"Polygon","coordinates":[[[-3.223693608999952,54.249058382000044],[-3.330026499999974,54.19639199900007],[-3.613598,54.52509699800004],[-3.578772499999957,54.651966],[-3.433302,54.76629649800003],[-3.398636499999952,54.86912549800007],[-3.251950499999964,54.90313699900008],[-3.314172499999927,54.91571799900004],[-3.285158,54.94161599800003],[-3.11915,54.927886998000076],[-2.983040499999959,54.78607199900006],[-2.994024499999966,54.69737249800005],[-3.092676499999925,54.65443049700008],[-3.007665,54.595489497000074],[-3.011364499999956,54.50744250000008],[-3.167118499999958,54.454169999000044],[-3.241853,54.32358549800006],[-3.223693608999952,54.249058382000044]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Cumbria","NUTS_NAME":"East Cumbria"},"geometry":{"type":"Polygon","coordinates":[[[-2.309801499999935,54.32436749900006],[-2.322618,54.245509998000045],[-2.460827499999937,54.22676449700003],[-2.86947,54.17673849800008],[-2.789148499999953,54.22734049700006],[-3.016447,54.13262549700005],[-3.03217,54.245723499000064],[-3.149062,54.093600999000046],[-3.241605000999925,54.10834499900005],[-3.256430499999965,54.16888450000005],[-3.204951499999936,54.21513],[-3.223693608999952,54.249058382000044],[-3.241853,54.32358549800006],[-3.167118499999958,54.454169999000044],[-3.011364499999956,54.50744250000008],[-3.007665,54.595489497000074],[-3.092676499999925,54.65443049700008],[-2.994024499999966,54.69737249800005],[-2.983040499999959,54.78607199900006],[-3.11915,54.927886998000076],[-3.00619949999998,54.94408049700007],[-3.131486,54.94579299700007],[-3.091623992999928,54.97554629100006],[-3.051281499999959,55.05262349800006],[-2.858508,55.10842499800003],[-2.689751,55.18906399700006],[-2.48629649999998,55.08311849800003],[-2.601320500999975,54.97138999600003],[-2.545466499999975,54.92846299900003],[-2.605391499999939,54.88444899800004],[-2.567809,54.82364249900007],[-2.31204300099995,54.79108049900003],[-2.351850499999955,54.68591699600006],[-2.172482499999944,54.53248199700005],[-2.170166499999937,54.45825599900007],[-2.367673,54.35612499800004],[-2.309801499999935,54.32436749900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Greater Manchester","NUTS_NAME":"Greater Manchester"},"geometry":{"type":"Polygon","coordinates":[[[-1.963350499999933,53.50985349600006],[-2.031023499999947,53.370289],[-2.240760499999965,53.35959999800008],[-2.313970499999925,53.35745249900003],[-2.426563499999929,53.38748949700005],[-2.489705499999957,53.460170497000036],[-2.576719,53.446087],[-2.73049950099994,53.52062999700007],[-2.689291,53.60433599600003],[-2.511298,53.62702950000005],[-2.379104,53.63090150000005],[-2.371208,53.66711450000008],[-2.146293000999947,53.68226599900004],[-2.048864499999979,53.680259500000034],[-1.909583,53.53842149800005],[-1.963350499999933,53.50985349600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD33","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Manchester","NUTS_NAME":"Manchester"},"geometry":{"type":"Polygon","coordinates":[[[-2.163022499999954,53.49289699800005],[-2.240760499999965,53.35959999800008],[-2.313970499999925,53.35745249900003],[-2.256896,53.517971],[-2.163022499999954,53.49289699800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD34","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Greater Manchester South West","NUTS_NAME":"Greater Manchester South West"},"geometry":{"type":"Polygon","coordinates":[[[-2.338208,53.533675998000035],[-2.256896,53.517971],[-2.313970499999925,53.35745249900003],[-2.426563499999929,53.38748949700005],[-2.489705499999957,53.460170497000036],[-2.415322,53.512073497000074],[-2.454087,53.52625249700003],[-2.338208,53.533675998000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD35","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Greater Manchester South East","NUTS_NAME":"Greater Manchester South East"},"geometry":{"type":"Polygon","coordinates":[[[-1.963350499999933,53.50985349600006],[-2.031023499999947,53.370289],[-2.240760499999965,53.35959999800008],[-2.163022499999954,53.49289699800005],[-1.963350499999933,53.50985349600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD36","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Greater Manchester North West","NUTS_NAME":"Greater Manchester North West"},"geometry":{"type":"Polygon","coordinates":[[[-2.379104,53.63090150000005],[-2.338208,53.533675998000035],[-2.454087,53.52625249700003],[-2.415322,53.512073497000074],[-2.489705499999957,53.460170497000036],[-2.576719,53.446087],[-2.73049950099994,53.52062999700007],[-2.689291,53.60433599600003],[-2.511298,53.62702950000005],[-2.379104,53.63090150000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD37","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Greater Manchester North East","NUTS_NAME":"Greater Manchester North East"},"geometry":{"type":"Polygon","coordinates":[[[-1.909583,53.53842149800005],[-1.963350499999933,53.50985349600006],[-2.163022499999954,53.49289699800005],[-2.256896,53.517971],[-2.338208,53.533675998000035],[-2.379104,53.63090150000005],[-2.371208,53.66711450000008],[-2.146293000999947,53.68226599900004],[-2.048864499999979,53.680259500000034],[-1.909583,53.53842149800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD4","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Lancashire","NUTS_NAME":"Lancashire"},"geometry":{"type":"Polygon","coordinates":[[[-2.184482499999945,53.952304998000045],[-2.046089,53.85017799900004],[-2.06121,53.825671998000075],[-2.128320499999973,53.799068498000054],[-2.173259,53.72304549900008],[-2.146293000999947,53.68226599900004],[-2.371208,53.66711450000008],[-2.379104,53.63090150000005],[-2.511298,53.62702950000005],[-2.689291,53.60433599600003],[-2.73049950099994,53.52062999700007],[-2.887978,53.50386049900004],[-3.046686499999964,53.54299149800005],[-2.940127,53.658645498000055],[-2.956203,53.697529],[-2.833743,53.72212999800007],[-2.757161000999929,53.75646949900005],[-3.057371499999931,53.77648149800007],[-3.047940499999925,53.875774498000055],[-2.853505,53.96575949800007],[-2.924294499999974,54.031154496000056],[-2.794437,54.12753699800004],[-2.86947,54.17673849800008],[-2.460827499999937,54.22676449700003],[-2.56470749999994,54.12711349800003],[-2.469517,54.04625699700006],[-2.184482499999945,53.952304998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD41","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Blackburn with Darwen","NUTS_NAME":"Blackburn with Darwen"},"geometry":{"type":"Polygon","coordinates":[[[-2.371208,53.66711450000008],[-2.379104,53.63090150000005],[-2.511298,53.62702950000005],[-2.561577,53.746936997000034],[-2.465780499999937,53.780849499000055],[-2.371208,53.66711450000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD42","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Blackpool","NUTS_NAME":"Blackpool"},"geometry":{"type":"Polygon","coordinates":[[[-3.010641,53.82623299800008],[-2.996546499999965,53.774486496000065],[-3.057371499999931,53.77648149800007],[-3.047940499999925,53.875774498000055],[-3.010641,53.82623299800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD44","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Lancaster & Wyre","NUTS_NAME":"Lancaster & Wyre"},"geometry":{"type":"Polygon","coordinates":[[[-2.469517,54.04625699700006],[-2.709795000999975,53.853679500000055],[-2.94254,53.85083399600006],[-2.976814,53.86206799800004],[-3.010641,53.82623299800008],[-3.047940499999925,53.875774498000055],[-2.853505,53.96575949800007],[-2.924294499999974,54.031154496000056],[-2.794437,54.12753699800004],[-2.86947,54.17673849800008],[-2.460827499999937,54.22676449700003],[-2.56470749999994,54.12711349800003],[-2.469517,54.04625699700006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD45","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Mid Lancashire","NUTS_NAME":"Mid Lancashire"},"geometry":{"type":"Polygon","coordinates":[[[-2.184482499999945,53.952304998000045],[-2.336189499999932,53.79548650000004],[-2.465780499999937,53.780849499000055],[-2.561577,53.746936997000034],[-2.658895499999971,53.680656498000076],[-2.833743,53.72212999800007],[-2.757161000999929,53.75646949900005],[-3.057371499999931,53.77648149800007],[-2.996546499999965,53.774486496000065],[-3.010641,53.82623299800008],[-2.976814,53.86206799800004],[-2.94254,53.85083399600006],[-2.709795000999975,53.853679500000055],[-2.469517,54.04625699700006],[-2.184482499999945,53.952304998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD46","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Lancashire","NUTS_NAME":"East Lancashire"},"geometry":{"type":"Polygon","coordinates":[[[-2.046089,53.85017799900004],[-2.06121,53.825671998000075],[-2.128320499999973,53.799068498000054],[-2.173259,53.72304549900008],[-2.146293000999947,53.68226599900004],[-2.371208,53.66711450000008],[-2.465780499999937,53.780849499000055],[-2.336189499999932,53.79548650000004],[-2.184482499999945,53.952304998000045],[-2.046089,53.85017799900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD47","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Chorley & West Lancashire","NUTS_NAME":"Chorley & West Lancashire"},"geometry":{"type":"Polygon","coordinates":[[[-2.511298,53.62702950000005],[-2.689291,53.60433599600003],[-2.73049950099994,53.52062999700007],[-2.887978,53.50386049900004],[-3.046686499999964,53.54299149800005],[-2.940127,53.658645498000055],[-2.956203,53.697529],[-2.833743,53.72212999800007],[-2.658895499999971,53.680656498000076],[-2.561577,53.746936997000034],[-2.511298,53.62702950000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD6","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Cheshire","NUTS_NAME":"Cheshire"},"geometry":{"type":"Polygon","coordinates":[[[-2.426563499999929,53.38748949700005],[-2.313970499999925,53.35745249900003],[-2.240760499999965,53.35959999800008],[-2.031023499999947,53.370289],[-1.987376,53.21360799900003],[-2.380991,53.05268499700003],[-2.380768,52.998428499000056],[-2.699274500999934,52.995460500000036],[-2.726823499999966,52.983295499000064],[-2.834814,52.998698999000055],[-2.99507449999993,53.154220499000076],[-2.922283,53.18931599600006],[-3.084193,53.256122497000035],[-3.110706,53.29631799800006],[-2.928556499999956,53.30827699900004],[-2.752411499999937,53.31475449900006],[-2.675166499999932,53.35448449900008],[-2.693358,53.36183549700007],[-2.668757500999959,53.44113549800005],[-2.576719,53.446087],[-2.489705499999957,53.460170497000036],[-2.426563499999929,53.38748949700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD61","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Warrington","NUTS_NAME":"Warrington"},"geometry":{"type":"Polygon","coordinates":[[[-2.489705499999957,53.460170497000036],[-2.426563499999929,53.38748949700005],[-2.518369499999949,53.342418500000065],[-2.595201499999973,53.322483],[-2.675166499999932,53.35448449900008],[-2.693358,53.36183549700007],[-2.668757500999959,53.44113549800005],[-2.576719,53.446087],[-2.489705499999957,53.460170497000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD62","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Cheshire East","NUTS_NAME":"Cheshire East"},"geometry":{"type":"Polygon","coordinates":[[[-2.313970499999925,53.35745249900003],[-2.240760499999965,53.35959999800008],[-2.031023499999947,53.370289],[-1.987376,53.21360799900003],[-2.380991,53.05268499700003],[-2.380768,52.998428499000056],[-2.699274500999934,52.995460500000036],[-2.678663,53.032710999000074],[-2.752911,53.069248],[-2.7059,53.11849599800007],[-2.40655449999997,53.17415999800005],[-2.348985,53.248698997000076],[-2.518369499999949,53.342418500000065],[-2.426563499999929,53.38748949700005],[-2.313970499999925,53.35745249900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD63","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Cheshire West and Chester","NUTS_NAME":"Cheshire West and Chester"},"geometry":{"type":"Polygon","coordinates":[[[-2.675166499999932,53.35448449900008],[-2.595201499999973,53.322483],[-2.518369499999949,53.342418500000065],[-2.348985,53.248698997000076],[-2.40655449999997,53.17415999800005],[-2.7059,53.11849599800007],[-2.752911,53.069248],[-2.678663,53.032710999000074],[-2.699274500999934,52.995460500000036],[-2.726823499999966,52.983295499000064],[-2.834814,52.998698999000055],[-2.99507449999993,53.154220499000076],[-2.922283,53.18931599600006],[-3.084193,53.256122497000035],[-3.110706,53.29631799800006],[-2.928556499999956,53.30827699900004],[-2.752411499999937,53.31475449900006],[-2.675166499999932,53.35448449900008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD7","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Merseyside","NUTS_NAME":"Merseyside"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-3.046686499999964,53.54299149800005],[-2.887978,53.50386049900004],[-2.73049950099994,53.52062999700007],[-2.576719,53.446087],[-2.668757500999959,53.44113549800005],[-2.693358,53.36183549700007],[-2.826677,53.33167250000008],[-3.008742,53.43841149800005],[-3.105458499999941,53.551548],[-2.956203,53.697529],[-2.940127,53.658645498000055],[-3.046686499999964,53.54299149800005]]],[[[-2.928556499999956,53.30827699900004],[-3.110706,53.29631799800006],[-3.20036750099996,53.38752749900004],[-3.040308,53.44292849800007],[-2.928556499999956,53.30827699900004]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD71","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Merseyside","NUTS_NAME":"East Merseyside"},"geometry":{"type":"Polygon","coordinates":[[[-2.73049950099994,53.52062999700007],[-2.576719,53.446087],[-2.668757500999959,53.44113549800005],[-2.693358,53.36183549700007],[-2.826677,53.33167250000008],[-2.92255149999994,53.47502899800003],[-2.887978,53.50386049900004],[-2.73049950099994,53.52062999700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD72","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Liverpool","NUTS_NAME":"Liverpool"},"geometry":{"type":"Polygon","coordinates":[[[-2.826677,53.33167250000008],[-3.008742,53.43841149800005],[-2.92255149999994,53.47502899800003],[-2.826677,53.33167250000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD73","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Sefton","NUTS_NAME":"Sefton"},"geometry":{"type":"Polygon","coordinates":[[[-2.887978,53.50386049900004],[-2.92255149999994,53.47502899800003],[-3.008742,53.43841149800005],[-3.105458499999941,53.551548],[-2.956203,53.697529],[-2.940127,53.658645498000055],[-3.046686499999964,53.54299149800005],[-2.887978,53.50386049900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKD74","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Wirral","NUTS_NAME":"Wirral"},"geometry":{"type":"Polygon","coordinates":[[[-2.928556499999956,53.30827699900004],[-3.110706,53.29631799800006],[-3.20036750099996,53.38752749900004],[-3.040308,53.44292849800007],[-2.928556499999956,53.30827699900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"YORKSHIRE AND THE HUMBER","NUTS_NAME":"YORKSHIRE AND THE HUMBER"},"geometry":{"type":"Polygon","coordinates":[[[-0.790906499999949,54.55948249900007],[-0.520867,54.44644949900004],[-0.396518499999956,54.27443699600008],[-0.212449499999934,54.157627],[-0.076264499999979,54.11638649900004],[-0.198329499999943,54.077949498000066],[-0.208244,53.99762349800005],[0.141740262000042,53.61067700600006],[-0.103462,53.63536049800007],[-0.250096,53.73331850000005],[-0.419136500999969,53.71961999600006],[-0.617187,53.73120499800007],[-0.718628381999963,53.702292244000034],[-0.698366,53.684653998000044],[-0.694836,53.69454949900006],[-0.295713499999977,53.71386699800007],[0.017378,53.52536750000007],[-0.131727499999954,53.435950999000056],[-0.300850499999967,53.61640949900004],[-0.551721499999928,53.45960599800003],[-0.738485499999967,53.519866998000055],[-0.741809831999944,53.516955831000075],[-0.797417499999938,53.455085997000026],[-0.935518,53.50252149800008],[-1.199686499999927,53.311454996000066],[-1.324675,53.32885349700007],[-1.599034499999959,53.31140149700008],[-1.801430499999981,53.481018],[-1.822188499999925,53.52111799700003],[-1.909583,53.53842149800005],[-2.048864499999979,53.680259500000034],[-2.146293000999947,53.68226599900004],[-2.173259,53.72304549900008],[-2.128320499999973,53.799068498000054],[-2.06121,53.825671998000075],[-2.046089,53.85017799900004],[-2.184482499999945,53.952304998000045],[-2.469517,54.04625699700006],[-2.56470749999994,54.12711349800003],[-2.460827499999937,54.22676449700003],[-2.322618,54.245509998000045],[-2.309801499999935,54.32436749900006],[-2.367673,54.35612499800004],[-2.170166499999937,54.45825599900007],[-1.696865,54.536060500000076],[-1.434778,54.48751449800005],[-1.234791,54.51036849700006],[-0.952505,54.488063999000076],[-0.790906499999949,54.55948249900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"East Yorkshire and Northern Lincolnshire","NUTS_NAME":"East Yorkshire and Northern Lincolnshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.208244,53.99762349800005],[0.141740262000042,53.61067700600006],[-0.103462,53.63536049800007],[-0.250096,53.73331850000005],[-0.419136500999969,53.71961999600006],[-0.617187,53.73120499800007],[-0.718628381999963,53.702292244000034],[-0.698366,53.684653998000044],[-0.694836,53.69454949900006],[-0.295713499999977,53.71386699800007],[0.017378,53.52536750000007],[-0.131727499999954,53.435950999000056],[-0.300850499999967,53.61640949900004],[-0.551721499999928,53.45960599800003],[-0.738485499999967,53.519866998000055],[-0.741809831999944,53.516955831000075],[-0.797417499999938,53.455085997000026],[-0.935518,53.50252149800008],[-0.865270499999951,53.637733499000035],[-1.0486,53.65608199800005],[-1.103509499999973,53.66958249900006],[-0.905781511999976,53.72576118500007],[-0.911604,53.72841249900006],[-0.973540499999956,53.74990449900008],[-0.928301499999975,53.765358],[-0.923447499999952,53.880794500000036],[-0.925222,53.99155050000007],[-0.681425499999932,54.01087199700004],[-0.418159000999935,54.17409899900008],[-0.212449499999934,54.157627],[-0.076264499999979,54.11638649900004],[-0.198329499999943,54.077949498000066],[-0.208244,53.99762349800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Kingston upon Hull, City of","NUTS_NAME":"Kingston upon Hull, City of"},"geometry":{"type":"Polygon","coordinates":[[[-0.250096,53.73331850000005],[-0.419136500999969,53.71961999600006],[-0.31605049999996,53.813297498000054],[-0.250096,53.73331850000005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Riding of Yorkshire","NUTS_NAME":"East Riding of Yorkshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.212449499999934,54.157627],[-0.076264499999979,54.11638649900004],[-0.198329499999943,54.077949498000066],[-0.208244,53.99762349800005],[0.141740262000042,53.61067700600006],[-0.103462,53.63536049800007],[-0.250096,53.73331850000005],[-0.31605049999996,53.813297498000054],[-0.419136500999969,53.71961999600006],[-0.617187,53.73120499800007],[-0.718628381999963,53.702292244000034],[-0.698366,53.684653998000044],[-0.865270499999951,53.637733499000035],[-1.0486,53.65608199800005],[-1.103509499999973,53.66958249900006],[-0.905781511999976,53.72576118500007],[-0.911604,53.72841249900006],[-0.973540499999956,53.74990449900008],[-0.928301499999975,53.765358],[-0.923447499999952,53.880794500000036],[-0.925222,53.99155050000007],[-0.681425499999932,54.01087199700004],[-0.418159000999935,54.17409899900008],[-0.212449499999934,54.157627]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North and North East Lincolnshire","NUTS_NAME":"North and North East Lincolnshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.295713499999977,53.71386699800007],[0.017378,53.52536750000007],[-0.131727499999954,53.435950999000056],[-0.300850499999967,53.61640949900004],[-0.551721499999928,53.45960599800003],[-0.738485499999967,53.519866998000055],[-0.741809831999944,53.516955831000075],[-0.797417499999938,53.455085997000026],[-0.935518,53.50252149800008],[-0.865270499999951,53.637733499000035],[-0.698366,53.684653998000044],[-0.694836,53.69454949900006],[-0.295713499999977,53.71386699800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"North Yorkshire","NUTS_NAME":"North Yorkshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.790906499999949,54.55948249900007],[-0.520867,54.44644949900004],[-0.396518499999956,54.27443699600008],[-0.212449499999934,54.157627],[-0.418159000999935,54.17409899900008],[-0.681425499999932,54.01087199700004],[-0.925222,53.99155050000007],[-0.923447499999952,53.880794500000036],[-0.928301499999975,53.765358],[-0.973540499999956,53.74990449900008],[-0.911604,53.72841249900006],[-0.905781511999976,53.72576118500007],[-1.103509499999973,53.66958249900006],[-1.0486,53.65608199800005],[-1.232786499999975,53.62113949900004],[-1.244358499999976,53.69245899900005],[-1.198758,53.700759997000034],[-1.301985499999944,53.741756499000076],[-1.294119,53.92709749900007],[-1.727166499999953,53.910236498000074],[-1.966130499999963,53.951603],[-1.952523,53.90357199600004],[-2.046089,53.85017799900004],[-2.184482499999945,53.952304998000045],[-2.469517,54.04625699700006],[-2.56470749999994,54.12711349800003],[-2.460827499999937,54.22676449700003],[-2.322618,54.245509998000045],[-2.309801499999935,54.32436749900006],[-2.367673,54.35612499800004],[-2.170166499999937,54.45825599900007],[-1.696865,54.536060500000076],[-1.434778,54.48751449800005],[-1.234791,54.51036849700006],[-0.952505,54.488063999000076],[-0.790906499999949,54.55948249900007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"York","NUTS_NAME":"York"},"geometry":{"type":"Polygon","coordinates":[[[-0.925222,53.99155050000007],[-0.923447499999952,53.880794500000036],[-1.10503749999998,53.87576299800003],[-1.219755,53.963806],[-1.047386,54.054481500000065],[-0.925222,53.99155050000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North Yorkshire CC","NUTS_NAME":"North Yorkshire CC"},"geometry":{"type":"Polygon","coordinates":[[[-0.212449499999934,54.157627],[-0.418159000999935,54.17409899900008],[-0.681425499999932,54.01087199700004],[-0.925222,53.99155050000007],[-1.047386,54.054481500000065],[-1.219755,53.963806],[-1.10503749999998,53.87576299800003],[-0.923447499999952,53.880794500000036],[-0.928301499999975,53.765358],[-0.973540499999956,53.74990449900008],[-0.911604,53.72841249900006],[-0.905781511999976,53.72576118500007],[-1.103509499999973,53.66958249900006],[-1.0486,53.65608199800005],[-1.232786499999975,53.62113949900004],[-1.244358499999976,53.69245899900005],[-1.198758,53.700759997000034],[-1.301985499999944,53.741756499000076],[-1.294119,53.92709749900007],[-1.727166499999953,53.910236498000074],[-1.966130499999963,53.951603],[-1.952523,53.90357199600004],[-2.046089,53.85017799900004],[-2.184482499999945,53.952304998000045],[-2.469517,54.04625699700006],[-2.56470749999994,54.12711349800003],[-2.460827499999937,54.22676449700003],[-2.322618,54.245509998000045],[-2.309801499999935,54.32436749900006],[-2.367673,54.35612499800004],[-2.170166499999937,54.45825599900007],[-1.696865,54.536060500000076],[-1.434778,54.48751449800005],[-1.234791,54.51036849700006],[-0.952505,54.488063999000076],[-0.790906499999949,54.55948249900007],[-0.520867,54.44644949900004],[-0.396518499999956,54.27443699600008],[-0.212449499999934,54.157627]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"South Yorkshire","NUTS_NAME":"South Yorkshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.935518,53.50252149800008],[-1.199686499999927,53.311454996000066],[-1.324675,53.32885349700007],[-1.599034499999959,53.31140149700008],[-1.801430499999981,53.481018],[-1.822188499999925,53.52111799700003],[-1.58640349999996,53.607204497000055],[-1.232786499999975,53.62113949900004],[-1.0486,53.65608199800005],[-0.865270499999951,53.637733499000035],[-0.935518,53.50252149800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Barnsley, Doncaster and Rotherham","NUTS_NAME":"Barnsley, Doncaster and Rotherham"},"geometry":{"type":"Polygon","coordinates":[[[-0.865270499999951,53.637733499000035],[-0.935518,53.50252149800008],[-1.199686499999927,53.311454996000066],[-1.324675,53.32885349700007],[-1.455154499999935,53.471790500000054],[-1.801430499999981,53.481018],[-1.822188499999925,53.52111799700003],[-1.58640349999996,53.607204497000055],[-1.232786499999975,53.62113949900004],[-1.0486,53.65608199800005],[-0.865270499999951,53.637733499000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Sheffield","NUTS_NAME":"Sheffield"},"geometry":{"type":"Polygon","coordinates":[[[-1.324675,53.32885349700007],[-1.599034499999959,53.31140149700008],[-1.801430499999981,53.481018],[-1.455154499999935,53.471790500000054],[-1.324675,53.32885349700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE4","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"West Yorkshire","NUTS_NAME":"West Yorkshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.966130499999963,53.951603],[-1.727166499999953,53.910236498000074],[-1.294119,53.92709749900007],[-1.301985499999944,53.741756499000076],[-1.198758,53.700759997000034],[-1.244358499999976,53.69245899900005],[-1.232786499999975,53.62113949900004],[-1.58640349999996,53.607204497000055],[-1.822188499999925,53.52111799700003],[-1.909583,53.53842149800005],[-2.048864499999979,53.680259500000034],[-2.146293000999947,53.68226599900004],[-2.173259,53.72304549900008],[-2.128320499999973,53.799068498000054],[-2.06121,53.825671998000075],[-2.046089,53.85017799900004],[-1.952523,53.90357199600004],[-1.966130499999963,53.951603]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE41","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bradford","NUTS_NAME":"Bradford"},"geometry":{"type":"Polygon","coordinates":[[[-1.727166499999953,53.910236498000074],[-1.800315,53.88595199900004],[-1.640479,53.77996449600005],[-1.681632,53.75652699700004],[-2.06121,53.825671998000075],[-2.046089,53.85017799900004],[-1.952523,53.90357199600004],[-1.966130499999963,53.951603],[-1.727166499999953,53.910236498000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE42","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Leeds","NUTS_NAME":"Leeds"},"geometry":{"type":"Polygon","coordinates":[[[-1.301985499999944,53.741756499000076],[-1.571081499999934,53.70646299900005],[-1.681632,53.75652699700004],[-1.640479,53.77996449600005],[-1.800315,53.88595199900004],[-1.727166499999953,53.910236498000074],[-1.294119,53.92709749900007],[-1.301985499999944,53.741756499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE44","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Calderdale and Kirklees","NUTS_NAME":"Calderdale and Kirklees"},"geometry":{"type":"Polygon","coordinates":[[[-1.681632,53.75652699700004],[-1.571081499999934,53.70646299900005],[-1.624156,53.654109996000045],[-1.58640349999996,53.607204497000055],[-1.822188499999925,53.52111799700003],[-1.909583,53.53842149800005],[-2.048864499999979,53.680259500000034],[-2.146293000999947,53.68226599900004],[-2.173259,53.72304549900008],[-2.128320499999973,53.799068498000054],[-2.06121,53.825671998000075],[-1.681632,53.75652699700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKE45","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Wakefield","NUTS_NAME":"Wakefield"},"geometry":{"type":"Polygon","coordinates":[[[-1.232786499999975,53.62113949900004],[-1.58640349999996,53.607204497000055],[-1.624156,53.654109996000045],[-1.571081499999934,53.70646299900005],[-1.301985499999944,53.741756499000076],[-1.198758,53.700759997000034],[-1.244358499999976,53.69245899900005],[-1.232786499999975,53.62113949900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"EAST MIDLANDS (ENGLAND)","NUTS_NAME":"EAST MIDLANDS (ENGLAND)"},"geometry":{"type":"Polygon","coordinates":[[[-0.300850499999967,53.61640949900004],[-0.131727499999954,53.435950999000056],[0.017378,53.52536750000007],[0.163623,53.482135999000036],[0.355712,53.19206249800004],[0.339612,53.09699249800008],[0.043715500000076,52.90388099900008],[0.26596,52.810116],[0.171689,52.73803699800004],[-0.031214,52.66153699800003],[-0.494721499999969,52.64032],[-0.491674499999931,52.58129099800004],[-0.41533,52.57874699600006],[-0.341543,52.46694549800003],[-0.495901499999945,52.379516497000054],[-0.465321499999959,52.322956],[-0.653614499999946,52.268291496000074],[-0.668097,52.19503399800004],[-0.705417499999953,52.19157049700004],[-0.88068549999997,52.12633149800007],[-0.831384499999956,52.07194899700005],[-0.871292499999925,52.040252499000076],[-1.042297,52.07371149900007],[-1.118058,52.01542649700008],[-1.248983499999952,51.985714],[-1.320798,52.087630999000055],[-1.247932,52.09577550000006],[-1.331868499999928,52.16848749900004],[-1.21705,52.26267999800007],[-1.20923000099998,52.31471249800006],[-1.265234499999963,52.32838049800006],[-1.17209349999996,52.36130499700005],[-1.20158,52.396735998000054],[-1.560725,52.596164499000054],[-1.551736499999947,52.66746499800007],[-1.589611,52.68727099800003],[-1.597507,52.700431998000056],[-1.704478499999937,52.731067497000026],[-1.585425500999975,52.831641999000055],[-1.832636499999978,52.88741699700006],[-1.85291849999993,52.93172049800006],[-1.757684000999973,53.01716250000004],[-1.831996000999936,53.17280949900004],[-1.987376,53.21360799900003],[-2.031023499999947,53.370289],[-1.963350499999933,53.50985349600006],[-1.909583,53.53842149800005],[-1.822188499999925,53.52111799700003],[-1.801430499999981,53.481018],[-1.599034499999959,53.31140149700008],[-1.324675,53.32885349700007],[-1.199686499999927,53.311454996000066],[-0.935518,53.50252149800008],[-0.797417499999938,53.455085997000026],[-0.741809831999944,53.516955831000075],[-0.738485499999967,53.519866998000055],[-0.551721499999928,53.45960599800003],[-0.300850499999967,53.61640949900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Derbyshire and Nottinghamshire","NUTS_NAME":"Derbyshire and Nottinghamshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.801430499999981,53.481018],[-1.599034499999959,53.31140149700008],[-1.324675,53.32885349700007],[-1.199686499999927,53.311454996000066],[-0.935518,53.50252149800008],[-0.797417499999938,53.455085997000026],[-0.747089499999959,53.30446249700003],[-0.777013499999953,53.24692149800006],[-0.666140499999926,53.23412299800003],[-0.762053499999979,53.18822849800006],[-0.7181,53.17730349800007],[-0.706091,53.08338949700004],[-0.804156,53.01229099600005],[-0.778224499999965,52.97693249800005],[-0.819968499999959,52.96047199800006],[-0.988956499999972,52.818256500000075],[-1.228364,52.793117497000026],[-1.267846,52.87337899700003],[-1.597507,52.700431998000056],[-1.704478499999937,52.731067497000026],[-1.585425500999975,52.831641999000055],[-1.832636499999978,52.88741699700006],[-1.85291849999993,52.93172049800006],[-1.757684000999973,53.01716250000004],[-1.831996000999936,53.17280949900004],[-1.987376,53.21360799900003],[-2.031023499999947,53.370289],[-1.963350499999933,53.50985349600006],[-1.909583,53.53842149800005],[-1.822188499999925,53.52111799700003],[-1.801430499999981,53.481018]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Derby","NUTS_NAME":"Derby"},"geometry":{"type":"Polygon","coordinates":[[[-1.383031499999959,52.92594549700004],[-1.444838,52.862163500000065],[-1.555072,52.90606299900003],[-1.496431,52.964981],[-1.383031499999959,52.92594549700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Derbyshire","NUTS_NAME":"East Derbyshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.324675,53.32885349700007],[-1.199686499999927,53.311454996000066],[-1.166435499999977,53.27767199900006],[-1.21240850099997,53.24560149900003],[-1.198751,53.18310549900008],[-1.323399,53.162361],[-1.300650499999961,53.101692],[-1.331589,53.08100149900008],[-1.486745,53.12291699900004],[-1.599034499999959,53.31140149700008],[-1.324675,53.32885349700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South and West Derbyshire","NUTS_NAME":"South and West Derbyshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.822188499999925,53.52111799700003],[-1.801430499999981,53.481018],[-1.599034499999959,53.31140149700008],[-1.486745,53.12291699900004],[-1.331589,53.08100149900008],[-1.335822499999949,53.05297850000005],[-1.239661499999954,52.89229949800006],[-1.267846,52.87337899700003],[-1.597507,52.700431998000056],[-1.704478499999937,52.731067497000026],[-1.585425500999975,52.831641999000055],[-1.832636499999978,52.88741699700006],[-1.85291849999993,52.93172049800006],[-1.757684000999973,53.01716250000004],[-1.831996000999936,53.17280949900004],[-1.987376,53.21360799900003],[-2.031023499999947,53.370289],[-1.963350499999933,53.50985349600006],[-1.909583,53.53842149800005],[-1.822188499999925,53.52111799700003]],[[-1.383031499999959,52.92594549700004],[-1.496431,52.964981],[-1.555072,52.90606299900003],[-1.444838,52.862163500000065],[-1.383031499999959,52.92594549700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Nottingham","NUTS_NAME":"Nottingham"},"geometry":{"type":"Polygon","coordinates":[[[-1.2251,53.008109998000066],[-1.182698499999958,53.01858149700007],[-1.086066,52.948581500000046],[-1.214104000999953,52.903320499000074],[-1.246776000999944,52.95310949800006],[-1.2251,53.008109998000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF15","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North Nottinghamshire","NUTS_NAME":"North Nottinghamshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.797417499999938,53.455085997000026],[-0.747089499999959,53.30446249700003],[-0.777013499999953,53.24692149800006],[-0.666140499999926,53.23412299800003],[-0.762053499999979,53.18822849800006],[-0.7181,53.17730349800007],[-0.706091,53.08338949700004],[-0.804156,53.01229099600005],[-0.778224499999965,52.97693249800005],[-0.819968499999959,52.96047199800006],[-0.910068,53.035938498000064],[-0.975910499999941,52.98633599600004],[-1.139175499999965,53.099674],[-1.226261499999964,53.08105849800006],[-1.182698499999958,53.01858149700007],[-1.2251,53.008109998000066],[-1.335822499999949,53.05297850000005],[-1.331589,53.08100149900008],[-1.300650499999961,53.101692],[-1.323399,53.162361],[-1.198751,53.18310549900008],[-1.21240850099997,53.24560149900003],[-1.166435499999977,53.27767199900006],[-1.199686499999927,53.311454996000066],[-0.935518,53.50252149800008],[-0.797417499999938,53.455085997000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF16","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South Nottinghamshire","NUTS_NAME":"South Nottinghamshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.819968499999959,52.96047199800006],[-0.988956499999972,52.818256500000075],[-1.228364,52.793117497000026],[-1.267846,52.87337899700003],[-1.239661499999954,52.89229949800006],[-1.335822499999949,53.05297850000005],[-1.2251,53.008109998000066],[-1.246776000999944,52.95310949800006],[-1.214104000999953,52.903320499000074],[-1.086066,52.948581500000046],[-1.182698499999958,53.01858149700007],[-1.226261499999964,53.08105849800006],[-1.139175499999965,53.099674],[-0.975910499999941,52.98633599600004],[-0.910068,53.035938498000064],[-0.819968499999959,52.96047199800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Leicestershire, Rutland and Northamptonshire","NUTS_NAME":"Leicestershire, Rutland and Northamptonshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.778224499999965,52.97693249800005],[-0.664050499999973,52.75671749900005],[-0.430389,52.70542549900006],[-0.494721499999969,52.64032],[-0.491674499999931,52.58129099800004],[-0.41533,52.57874699600006],[-0.341543,52.46694549800003],[-0.495901499999945,52.379516497000054],[-0.465321499999959,52.322956],[-0.653614499999946,52.268291496000074],[-0.668097,52.19503399800004],[-0.705417499999953,52.19157049700004],[-0.88068549999997,52.12633149800007],[-0.831384499999956,52.07194899700005],[-0.871292499999925,52.040252499000076],[-1.042297,52.07371149900007],[-1.118058,52.01542649700008],[-1.248983499999952,51.985714],[-1.320798,52.087630999000055],[-1.247932,52.09577550000006],[-1.331868499999928,52.16848749900004],[-1.21705,52.26267999800007],[-1.20923000099998,52.31471249800006],[-1.265234499999963,52.32838049800006],[-1.17209349999996,52.36130499700005],[-1.20158,52.396735998000054],[-1.560725,52.596164499000054],[-1.551736499999947,52.66746499800007],[-1.589611,52.68727099800003],[-1.597507,52.700431998000056],[-1.267846,52.87337899700003],[-1.228364,52.793117497000026],[-0.988956499999972,52.818256500000075],[-0.819968499999959,52.96047199800006],[-0.778224499999965,52.97693249800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Leicester","NUTS_NAME":"Leicester"},"geometry":{"type":"Polygon","coordinates":[[[-1.046919499999944,52.63462449600007],[-1.159288499999946,52.58167250000008],[-1.190614,52.64039999600004],[-1.140737,52.684390998000026],[-1.046919499999944,52.63462449600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Leicestershire CC and Rutland","NUTS_NAME":"Leicestershire CC and Rutland"},"geometry":{"type":"Polygon","coordinates":[[[-0.494973499999958,52.640239500000064],[-0.716561499999955,52.52301399700008],[-0.868715499999951,52.526985],[-0.901241,52.45975899900003],[-1.20158,52.396735998000054],[-1.560725,52.596164499000054],[-1.551736499999947,52.66746499800007],[-1.589611,52.68727099800003],[-1.597507,52.700431998000056],[-1.267846,52.87337899700003],[-1.228364,52.793117497000026],[-0.988956499999972,52.818256500000075],[-0.819968499999959,52.96047199800006],[-0.778224499999965,52.97693249800005],[-0.664050499999973,52.75671749900005],[-0.430389,52.70542549900006],[-0.494721499999969,52.64032],[-0.494973499999958,52.640239500000064]],[[-1.046919499999944,52.63462449600007],[-1.140737,52.684390998000026],[-1.190614,52.64039999600004],[-1.159288499999946,52.58167250000008],[-1.046919499999944,52.63462449600007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF24","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Northamptonshire","NUTS_NAME":"West Northamptonshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.901241,52.45975899900003],[-0.791163500999971,52.35597249800003],[-0.801433,52.252674],[-0.705417499999953,52.19157049700004],[-0.88068549999997,52.12633149800007],[-0.831384499999956,52.07194899700005],[-0.871292499999925,52.040252499000076],[-1.042297,52.07371149900007],[-1.118058,52.01542649700008],[-1.248983499999952,51.985714],[-1.320798,52.087630999000055],[-1.247932,52.09577550000006],[-1.331868499999928,52.16848749900004],[-1.21705,52.26267999800007],[-1.20923000099998,52.31471249800006],[-1.265234499999963,52.32838049800006],[-1.17209349999996,52.36130499700005],[-1.20158,52.396735998000054],[-0.901241,52.45975899900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF25","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North Northamptonshire","NUTS_NAME":"North Northamptonshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.41533,52.57874699600006],[-0.341543,52.46694549800003],[-0.495901499999945,52.379516497000054],[-0.465321499999959,52.322956],[-0.653614499999946,52.268291496000074],[-0.668097,52.19503399800004],[-0.705417499999953,52.19157049700004],[-0.801433,52.252674],[-0.791163500999971,52.35597249800003],[-0.901241,52.45975899900003],[-0.868715499999951,52.526985],[-0.716561499999955,52.52301399700008],[-0.494973499999958,52.640239500000064],[-0.494721499999969,52.64032],[-0.491674499999931,52.58129099800004],[-0.41533,52.57874699600006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Lincolnshire","NUTS_NAME":"Lincolnshire"},"geometry":{"type":"Polygon","coordinates":[[[0.017378,53.52536750000007],[0.163623,53.482135999000036],[0.355712,53.19206249800004],[0.339612,53.09699249800008],[0.043715500000076,52.90388099900008],[0.26596,52.810116],[0.171689,52.73803699800004],[-0.031214,52.66153699800003],[-0.494721499999969,52.64032],[-0.430389,52.70542549900006],[-0.664050499999973,52.75671749900005],[-0.778224499999965,52.97693249800005],[-0.804156,53.01229099600005],[-0.706091,53.08338949700004],[-0.7181,53.17730349800007],[-0.762053499999979,53.18822849800006],[-0.666140499999926,53.23412299800003],[-0.777013499999953,53.24692149800006],[-0.747089499999959,53.30446249700003],[-0.797417499999938,53.455085997000026],[-0.741809831999944,53.516955831000075],[-0.738485499999967,53.519866998000055],[-0.551721499999928,53.45960599800003],[-0.300850499999967,53.61640949900004],[-0.131727499999954,53.435950999000056],[0.017378,53.52536750000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKF30","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Lincolnshire","NUTS_NAME":"Lincolnshire"},"geometry":{"type":"Polygon","coordinates":[[[0.017378,53.52536750000007],[0.163623,53.482135999000036],[0.355712,53.19206249800004],[0.339612,53.09699249800008],[0.043715500000076,52.90388099900008],[0.26596,52.810116],[0.171689,52.73803699800004],[-0.031214,52.66153699800003],[-0.494721499999969,52.64032],[-0.430389,52.70542549900006],[-0.664050499999973,52.75671749900005],[-0.778224499999965,52.97693249800005],[-0.804156,53.01229099600005],[-0.706091,53.08338949700004],[-0.7181,53.17730349800007],[-0.762053499999979,53.18822849800006],[-0.666140499999926,53.23412299800003],[-0.777013499999953,53.24692149800006],[-0.747089499999959,53.30446249700003],[-0.797417499999938,53.455085997000026],[-0.741809831999944,53.516955831000075],[-0.738485499999967,53.519866998000055],[-0.551721499999928,53.45960599800003],[-0.300850499999967,53.61640949900004],[-0.131727499999954,53.435950999000056],[0.017378,53.52536750000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"WEST MIDLANDS (ENGLAND)","NUTS_NAME":"WEST MIDLANDS (ENGLAND)"},"geometry":{"type":"Polygon","coordinates":[[[-1.85291849999993,52.93172049800006],[-1.832636499999978,52.88741699700006],[-1.585425500999975,52.831641999000055],[-1.704478499999937,52.731067497000026],[-1.597507,52.700431998000056],[-1.589611,52.68727099800003],[-1.551736499999947,52.66746499800007],[-1.560725,52.596164499000054],[-1.20158,52.396735998000054],[-1.17209349999996,52.36130499700005],[-1.265234499999963,52.32838049800006],[-1.20923000099998,52.31471249800006],[-1.21705,52.26267999800007],[-1.331868499999928,52.16848749900004],[-1.357743,52.10131050000007],[-1.487778,52.09395999600008],[-1.527906499999972,51.991756499000076],[-1.665732499999933,51.987491497000065],[-1.767627000999937,52.11259449800008],[-1.853368,52.07237250000003],[-1.839030000999969,52.00678650000003],[-2.158117499999946,52.050209],[-2.300416499999926,51.96678150000008],[-2.351362,52.02136599800008],[-2.478917,52.02253699900007],[-2.500891,51.96061349800004],[-2.445992499999932,51.893996998000034],[-2.65021349999995,51.826152998000055],[-3.067357499999957,51.98315050000008],[-3.141911499999935,52.127876498000035],[-2.954719,52.34925849900003],[-3.235562,52.44255049800006],[-2.994171499999936,52.55295950000004],[-3.137092,52.533389998000075],[-3.139272499999947,52.586006],[-2.977511000999925,52.72648250000003],[-3.161091499999941,52.79612349800004],[-3.14748,52.89015599700008],[-2.982011,52.959217],[-2.80052349999994,52.89548099900003],[-2.728526,52.925197498000045],[-2.726823499999966,52.983295499000064],[-2.699274500999934,52.995460500000036],[-2.380768,52.998428499000056],[-2.380991,53.05268499700003],[-1.987376,53.21360799900003],[-1.831996000999936,53.17280949900004],[-1.757684000999973,53.01716250000004],[-1.85291849999993,52.93172049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Herefordshire, Worcestershire and Warwickshire","NUTS_NAME":"Herefordshire, Worcestershire and Warwickshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.560725,52.596164499000054],[-1.20158,52.396735998000054],[-1.17209349999996,52.36130499700005],[-1.265234499999963,52.32838049800006],[-1.20923000099998,52.31471249800006],[-1.21705,52.26267999800007],[-1.331868499999928,52.16848749900004],[-1.357743,52.10131050000007],[-1.487778,52.09395999600008],[-1.527906499999972,51.991756499000076],[-1.665732499999933,51.987491497000065],[-1.767627000999937,52.11259449800008],[-1.853368,52.07237250000003],[-1.839030000999969,52.00678650000003],[-2.158117499999946,52.050209],[-2.300416499999926,51.96678150000008],[-2.351362,52.02136599800008],[-2.478917,52.02253699900007],[-2.500891,51.96061349800004],[-2.445992499999932,51.893996998000034],[-2.65021349999995,51.826152998000055],[-3.067357499999957,51.98315050000008],[-3.141911499999935,52.127876498000035],[-2.954719,52.34925849900003],[-2.805421,52.38826350000005],[-2.618022,52.30696499700008],[-2.287363,52.455325998000035],[-2.164830000999928,52.43021399700007],[-2.016967499999964,52.43268949800006],[-1.997301,52.381073],[-1.868721499999936,52.404742998000074],[-1.87201,52.36760349700006],[-1.601023,52.38932399600003],[-1.462974,52.37635399800007],[-1.424152499999934,52.43407449800003],[-1.595457499999952,52.45592499700007],[-1.677125499999931,52.43636299800005],[-1.753495,52.512973998000064],[-1.788053,52.58787149800003],[-1.651547499999936,52.59842699600006],[-1.589611,52.68727099800003],[-1.551736499999947,52.66746499800007],[-1.560725,52.596164499000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Herefordshire, County of","NUTS_NAME":"Herefordshire, County of"},"geometry":{"type":"Polygon","coordinates":[[[-2.618022,52.30696499700008],[-2.663053499999933,52.30397399700007],[-2.628857499999981,52.24035649700005],[-2.451386499999956,52.28509149800004],[-2.474686,52.241367497000056],[-2.385893499999952,52.22645549900005],[-2.435159,52.168224500000065],[-2.344591,52.13259899800005],[-2.351362,52.02136599800008],[-2.478917,52.02253699900007],[-2.500891,51.96061349800004],[-2.445992499999932,51.893996998000034],[-2.65021349999995,51.826152998000055],[-3.067357499999957,51.98315050000008],[-3.141911499999935,52.127876498000035],[-2.954719,52.34925849900003],[-2.805421,52.38826350000005],[-2.618022,52.30696499700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Worcestershire","NUTS_NAME":"Worcestershire"},"geometry":{"type":"Polygon","coordinates":[[[-2.164830000999928,52.43021399700007],[-2.016967499999964,52.43268949800006],[-1.997301,52.381073],[-1.868721499999936,52.404742998000074],[-1.87201,52.36760349700006],[-1.944553499999927,52.174770496000065],[-1.767627000999937,52.11259449800008],[-1.853368,52.07237250000003],[-1.839030000999969,52.00678650000003],[-2.158117499999946,52.050209],[-2.300416499999926,51.96678150000008],[-2.351362,52.02136599800008],[-2.344591,52.13259899800005],[-2.435159,52.168224500000065],[-2.385893499999952,52.22645549900005],[-2.474686,52.241367497000056],[-2.451386499999956,52.28509149800004],[-2.628857499999981,52.24035649700005],[-2.663053499999933,52.30397399700007],[-2.618022,52.30696499700008],[-2.287363,52.455325998000035],[-2.164830000999928,52.43021399700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Warwickshire","NUTS_NAME":"Warwickshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.20158,52.396735998000054],[-1.17209349999996,52.36130499700005],[-1.265234499999963,52.32838049800006],[-1.20923000099998,52.31471249800006],[-1.21705,52.26267999800007],[-1.331868499999928,52.16848749900004],[-1.357743,52.10131050000007],[-1.487778,52.09395999600008],[-1.527906499999972,51.991756499000076],[-1.665732499999933,51.987491497000065],[-1.767627000999937,52.11259449800008],[-1.944553499999927,52.174770496000065],[-1.87201,52.36760349700006],[-1.601023,52.38932399600003],[-1.462974,52.37635399800007],[-1.424152499999934,52.43407449800003],[-1.595457499999952,52.45592499700007],[-1.677125499999931,52.43636299800005],[-1.753495,52.512973998000064],[-1.788053,52.58787149800003],[-1.651547499999936,52.59842699600006],[-1.589611,52.68727099800003],[-1.551736499999947,52.66746499800007],[-1.560725,52.596164499000054],[-1.20158,52.396735998000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Shropshire and Staffordshire","NUTS_NAME":"Shropshire and Staffordshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.85291849999993,52.93172049800006],[-1.832636499999978,52.88741699700006],[-1.585425500999975,52.831641999000055],[-1.704478499999937,52.731067497000026],[-1.597507,52.700431998000056],[-1.589611,52.68727099800003],[-1.651547499999936,52.59842699600006],[-1.788053,52.58787149800003],[-1.872537499999964,52.58495349700007],[-1.912816,52.658245],[-2.050694000999954,52.620533],[-2.201781499999925,52.57879250000008],[-2.133465499999943,52.55407699800003],[-2.191728,52.50340649800006],[-2.164830000999928,52.43021399700007],[-2.287363,52.455325998000035],[-2.618022,52.30696499700008],[-2.805421,52.38826350000005],[-2.954719,52.34925849900003],[-3.235562,52.44255049800006],[-2.994171499999936,52.55295950000004],[-3.137092,52.533389998000075],[-3.139272499999947,52.586006],[-2.977511000999925,52.72648250000003],[-3.161091499999941,52.79612349800004],[-3.14748,52.89015599700008],[-2.982011,52.959217],[-2.80052349999994,52.89548099900003],[-2.728526,52.925197498000045],[-2.726823499999966,52.983295499000064],[-2.699274500999934,52.995460500000036],[-2.380768,52.998428499000056],[-2.380991,53.05268499700003],[-1.987376,53.21360799900003],[-1.831996000999936,53.17280949900004],[-1.757684000999973,53.01716250000004],[-1.85291849999993,52.93172049800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Telford and Wrekin","NUTS_NAME":"Telford and Wrekin"},"geometry":{"type":"Polygon","coordinates":[[[-2.315553499999965,52.73296349800006],[-2.371852,52.73854449800007],[-2.438020499999936,52.61461999800008],[-2.664106,52.73630499800004],[-2.416324499999973,52.82702249700003],[-2.315553499999965,52.73296349800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Shropshire CC","NUTS_NAME":"Shropshire CC"},"geometry":{"type":"Polygon","coordinates":[[[-2.416324499999973,52.82702249700003],[-2.664106,52.73630499800004],[-2.438020499999936,52.61461999800008],[-2.371852,52.73854449800007],[-2.315553499999965,52.73296349800006],[-2.232872,52.64770499700006],[-2.312729499999932,52.48921950000005],[-2.287363,52.455325998000035],[-2.618022,52.30696499700008],[-2.805421,52.38826350000005],[-2.954719,52.34925849900003],[-3.235562,52.44255049800006],[-2.994171499999936,52.55295950000004],[-3.137092,52.533389998000075],[-3.139272499999947,52.586006],[-2.977511000999925,52.72648250000003],[-3.161091499999941,52.79612349800004],[-3.14748,52.89015599700008],[-2.982011,52.959217],[-2.80052349999994,52.89548099900003],[-2.728526,52.925197498000045],[-2.726823499999966,52.983295499000064],[-2.699274500999934,52.995460500000036],[-2.380768,52.998428499000056],[-2.470819,52.90588749900007],[-2.375355000999946,52.89149099800005],[-2.416324499999973,52.82702249700003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Stoke-on-Trent","NUTS_NAME":"Stoke-on-Trent"},"geometry":{"type":"Polygon","coordinates":[[[-2.081188499999939,52.96692299800003],[-2.181580499999939,52.946224],[-2.238732499999969,53.073360498000056],[-2.188881,53.09010299800008],[-2.081188499999939,52.96692299800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG24","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Staffordshire CC","NUTS_NAME":"Staffordshire CC"},"geometry":{"type":"Polygon","coordinates":[[[-1.597507,52.700431998000056],[-1.589611,52.68727099800003],[-1.651547499999936,52.59842699600006],[-1.788053,52.58787149800003],[-1.872537499999964,52.58495349700007],[-1.912816,52.658245],[-2.050694000999954,52.620533],[-2.201781499999925,52.57879250000008],[-2.133465499999943,52.55407699800003],[-2.191728,52.50340649800006],[-2.164830000999928,52.43021399700007],[-2.287363,52.455325998000035],[-2.312729499999932,52.48921950000005],[-2.232872,52.64770499700006],[-2.315553499999965,52.73296349800006],[-2.416324499999973,52.82702249700003],[-2.375355000999946,52.89149099800005],[-2.470819,52.90588749900007],[-2.380768,52.998428499000056],[-2.380991,53.05268499700003],[-1.987376,53.21360799900003],[-1.831996000999936,53.17280949900004],[-1.757684000999973,53.01716250000004],[-1.85291849999993,52.93172049800006],[-1.832636499999978,52.88741699700006],[-1.585425500999975,52.831641999000055],[-1.704478499999937,52.731067497000026],[-1.597507,52.700431998000056]],[[-2.081188499999939,52.96692299800003],[-2.188881,53.09010299800008],[-2.238732499999969,53.073360498000056],[-2.181580499999939,52.946224],[-2.081188499999939,52.96692299800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"West Midlands","NUTS_NAME":"West Midlands"},"geometry":{"type":"Polygon","coordinates":[[[-1.912816,52.658245],[-1.872537499999964,52.58495349700007],[-1.788053,52.58787149800003],[-1.753495,52.512973998000064],[-1.677125499999931,52.43636299800005],[-1.595457499999952,52.45592499700007],[-1.424152499999934,52.43407449800003],[-1.462974,52.37635399800007],[-1.601023,52.38932399600003],[-1.87201,52.36760349700006],[-1.868721499999936,52.404742998000074],[-1.997301,52.381073],[-2.016967499999964,52.43268949800006],[-2.164830000999928,52.43021399700007],[-2.191728,52.50340649800006],[-2.133465499999943,52.55407699800003],[-2.201781499999925,52.57879250000008],[-2.050694000999954,52.620533],[-1.912816,52.658245]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Birmingham","NUTS_NAME":"Birmingham"},"geometry":{"type":"Polygon","coordinates":[[[-1.788053,52.58787149800003],[-1.753495,52.512973998000064],[-1.794473,52.502864998000064],[-1.75502,52.45630650000004],[-1.868721499999936,52.404742998000074],[-1.997301,52.381073],[-2.016967499999964,52.43268949800006],[-2.01314949999994,52.462192498000036],[-1.918131,52.54731749900003],[-1.872537499999964,52.58495349700007],[-1.788053,52.58787149800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Solihull","NUTS_NAME":"Solihull"},"geometry":{"type":"Polygon","coordinates":[[[-1.595457499999952,52.45592499700007],[-1.601023,52.38932399600003],[-1.87201,52.36760349700006],[-1.868721499999936,52.404742998000074],[-1.75502,52.45630650000004],[-1.794473,52.502864998000064],[-1.753495,52.512973998000064],[-1.677125499999931,52.43636299800005],[-1.595457499999952,52.45592499700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG33","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Coventry","NUTS_NAME":"Coventry"},"geometry":{"type":"Polygon","coordinates":[[[-1.601023,52.38932399600003],[-1.595457499999952,52.45592499700007],[-1.424152499999934,52.43407449800003],[-1.462974,52.37635399800007],[-1.601023,52.38932399600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG36","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Dudley","NUTS_NAME":"Dudley"},"geometry":{"type":"Polygon","coordinates":[[[-2.073918499999934,52.54988849700004],[-2.097052499999961,52.468402997000055],[-2.01314949999994,52.462192498000036],[-2.016967499999964,52.43268949800006],[-2.164830000999928,52.43021399700007],[-2.191728,52.50340649800006],[-2.133465499999943,52.55407699800003],[-2.073918499999934,52.54988849700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG37","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Sandwell","NUTS_NAME":"Sandwell"},"geometry":{"type":"Polygon","coordinates":[[[-1.918131,52.54731749900003],[-2.01314949999994,52.462192498000036],[-2.097052499999961,52.468402997000055],[-2.073918499999934,52.54988849700004],[-2.050958,52.552737998000055],[-1.918131,52.54731749900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG38","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Walsall","NUTS_NAME":"Walsall"},"geometry":{"type":"Polygon","coordinates":[[[-1.872537499999964,52.58495349700007],[-1.918131,52.54731749900003],[-2.050958,52.552737998000055],[-2.050694000999954,52.620533],[-1.912816,52.658245],[-1.872537499999964,52.58495349700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKG39","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Wolverhampton","NUTS_NAME":"Wolverhampton"},"geometry":{"type":"Polygon","coordinates":[[[-2.050694000999954,52.620533],[-2.050958,52.552737998000055],[-2.073918499999934,52.54988849700004],[-2.133465499999943,52.55407699800003],[-2.201781499999925,52.57879250000008],[-2.050694000999954,52.620533]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"EAST OF ENGLAND","NUTS_NAME":"EAST OF ENGLAND"},"geometry":{"type":"Polygon","coordinates":[[[0.854097374000048,51.601599640000074],[0.958179500000028,51.61968999900006],[0.821244,51.54071449700007],[0.626791,51.53217299800008],[0.51370350000002,51.53118499700008],[0.54146,51.51271449800004],[0.401860592000048,51.45663058400004],[0.379662895000024,51.45728967800005],[0.217437,51.480071999000074],[0.210514,51.49003599800005],[0.313079500000072,51.565814998000064],[0.200354,51.62493149800008],[0.138225500000033,51.62354299900005],[-0.012219499999958,51.64622899700004],[-0.011878499999966,51.680877499000076],[-0.182047499999953,51.66860199600006],[-0.304423,51.636348498000075],[-0.500561499999947,51.59968949700004],[-0.586348499999929,51.752113499000075],[-0.745649500999946,51.84209449800005],[-0.553598,51.82670949900006],[-0.70212749999996,51.909118498000055],[-0.652946499999928,51.96923050000004],[-0.668942,52.04871749800003],[-0.591772499999934,52.11069099900004],[-0.668097,52.19503399800004],[-0.653614499999946,52.268291496000074],[-0.465321499999959,52.322956],[-0.495901499999945,52.379516497000054],[-0.341543,52.46694549800003],[-0.41533,52.57874699600006],[-0.491674499999931,52.58129099800004],[-0.494721499999969,52.64032],[-0.031214,52.66153699800003],[0.171689,52.73803699800004],[0.26596,52.810116],[0.367146500000047,52.79617299600005],[0.490182,52.947913999000036],[0.695918,52.98762899700006],[1.343277,52.92004399700005],[1.675478,52.74269099800006],[1.743963,52.62939049700003],[1.740543500000058,52.53210049900008],[1.756430500000022,52.47172549700008],[1.596105927000053,52.14379643900003],[1.595382,52.14189149900005],[1.391530924000051,51.98931566500005],[1.317633500000056,51.94887150000005],[1.160663500000055,52.04616549900004],[1.27915249900002,51.96037649800007],[1.05638,51.95149249700006],[1.280599,51.940426],[1.291964500000063,51.87065099900008],[1.124435,51.77542499700007],[0.844628,51.781318498000076],[0.894665,51.76629249700005],[0.688036500000067,51.731712498000036],[0.71307,51.71509149900004],[0.756304,51.691741999000044],[0.94589650000006,51.71621299700007],[0.926602500000058,51.628940500000056],[0.776491999000029,51.63602449800004],[0.764340716000049,51.63674747400006],[0.854097374000048,51.601599640000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"East Anglia","NUTS_NAME":"East Anglia"},"geometry":{"type":"Polygon","coordinates":[[[1.343277,52.92004399700005],[1.675478,52.74269099800006],[1.743963,52.62939049700003],[1.740543500000058,52.53210049900008],[1.756430500000022,52.47172549700008],[1.596105927000053,52.14379643900003],[1.595382,52.14189149900005],[1.391530924000051,51.98931566500005],[1.317633500000056,51.94887150000005],[1.160663500000055,52.04616549900004],[1.27915249900002,51.96037649800007],[1.05638,51.95149249700006],[0.781522500000051,51.96650699800006],[0.705025500000033,52.062668],[0.404686500000025,52.06549849800007],[0.389743,52.03646450000008],[0.235253500000056,52.088996999000074],[0.068169,52.005786998000076],[-0.016232,52.06300749800005],[-0.126529000999938,52.02091999900006],[-0.157241000999932,52.080547497000055],[-0.144351,52.13746249800005],[-0.249742,52.184372],[-0.465321499999959,52.322956],[-0.495901499999945,52.379516497000054],[-0.341543,52.46694549800003],[-0.41533,52.57874699600006],[-0.491674499999931,52.58129099800004],[-0.494721499999969,52.64032],[-0.031214,52.66153699800003],[0.171689,52.73803699800004],[0.26596,52.810116],[0.367146500000047,52.79617299600005],[0.490182,52.947913999000036],[0.695918,52.98762899700006],[1.343277,52.92004399700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Peterborough","NUTS_NAME":"Peterborough"},"geometry":{"type":"Polygon","coordinates":[[[-0.031214,52.66153699800003],[-0.012696499999947,52.59424949900006],[-0.284589499999925,52.50611499900003],[-0.41533,52.57874699600006],[-0.491674499999931,52.58129099800004],[-0.494721499999969,52.64032],[-0.031214,52.66153699800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Cambridgeshire CC","NUTS_NAME":"Cambridgeshire CC"},"geometry":{"type":"Polygon","coordinates":[[[0.429402499000048,52.436424498000065],[0.37483450000002,52.40972899900004],[0.498856,52.27301799700007],[0.360789001000057,52.29780950000003],[0.342734,52.242339996000055],[0.51365850000002,52.227843998000026],[0.417325500000061,52.159301999000036],[0.381851,52.10269949900004],[0.404686500000025,52.06549849800007],[0.389743,52.03646450000008],[0.235253500000056,52.088996999000074],[0.068169,52.005786998000076],[-0.016232,52.06300749800005],[-0.126529000999938,52.02091999900006],[-0.157241000999932,52.080547497000055],[-0.144351,52.13746249800005],[-0.249742,52.184372],[-0.465321499999959,52.322956],[-0.495901499999945,52.379516497000054],[-0.341543,52.46694549800003],[-0.41533,52.57874699600006],[-0.284589499999925,52.50611499900003],[-0.012696499999947,52.59424949900006],[-0.031214,52.66153699800003],[0.171689,52.73803699800004],[0.153706,52.68217850000008],[0.219362,52.62181449800005],[0.214998500000036,52.532050999000035],[0.429402499000048,52.436424498000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Suffolk","NUTS_NAME":"Suffolk"},"geometry":{"type":"Polygon","coordinates":[[[1.756430500000022,52.47172549700008],[1.596105927000053,52.14379643900003],[1.595382,52.14189149900005],[1.391530924000051,51.98931566500005],[1.317633500000056,51.94887150000005],[1.160663500000055,52.04616549900004],[1.27915249900002,51.96037649800007],[1.05638,51.95149249700006],[0.781522500000051,51.96650699800006],[0.705025500000033,52.062668],[0.404686500000025,52.06549849800007],[0.381851,52.10269949900004],[0.417325500000061,52.159301999000036],[0.51365850000002,52.227843998000026],[0.342734,52.242339996000055],[0.360789001000057,52.29780950000003],[0.498856,52.27301799700007],[0.37483450000002,52.40972899900004],[0.429402499000048,52.436424498000065],[0.582561500000054,52.451858498000036],[1.214148,52.355403996000064],[1.682458,52.49521249900005],[1.628665500000068,52.52600849800007],[1.740543500000058,52.53210049900008],[1.756430500000022,52.47172549700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH15","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Norwich & East Norfolk","NUTS_NAME":"Norwich & East Norfolk"},"geometry":{"type":"Polygon","coordinates":[[[1.675478,52.74269099800006],[1.743963,52.62939049700003],[1.740543500000058,52.53210049900008],[1.628665500000068,52.52600849800007],[1.432651501000066,52.611995499000045],[1.257864,52.598140500000056],[1.221771,52.67316449800006],[1.097769499000037,52.64974999800006],[1.105279,52.714687498000046],[0.993672500000059,52.79835499700005],[1.308967,52.805259496000076],[1.512110500000063,52.68167499900005],[1.675478,52.74269099800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH16","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North & West Norfolk","NUTS_NAME":"North & West Norfolk"},"geometry":{"type":"Polygon","coordinates":[[[1.675478,52.74269099800006],[1.512110500000063,52.68167499900005],[1.308967,52.805259496000076],[0.993672500000059,52.79835499700005],[0.702071,52.786670498000035],[0.668205500000056,52.74055499900004],[0.699088500000073,52.70140850000007],[0.551049,52.680519],[0.61343,52.53163149900007],[0.582561500000054,52.451858498000036],[0.429402499000048,52.436424498000065],[0.214998500000036,52.532050999000035],[0.219362,52.62181449800005],[0.153706,52.68217850000008],[0.171689,52.73803699800004],[0.26596,52.810116],[0.367146500000047,52.79617299600005],[0.490182,52.947913999000036],[0.695918,52.98762899700006],[1.343277,52.92004399700005],[1.675478,52.74269099800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH17","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Breckland & South Norfolk","NUTS_NAME":"Breckland & South Norfolk"},"geometry":{"type":"Polygon","coordinates":[[[0.993672500000059,52.79835499700005],[1.105279,52.714687498000046],[1.097769499000037,52.64974999800006],[1.221771,52.67316449800006],[1.257864,52.598140500000056],[1.432651501000066,52.611995499000045],[1.628665500000068,52.52600849800007],[1.682458,52.49521249900005],[1.214148,52.355403996000064],[0.582561500000054,52.451858498000036],[0.61343,52.53163149900007],[0.551049,52.680519],[0.699088500000073,52.70140850000007],[0.668205500000056,52.74055499900004],[0.702071,52.786670498000035],[0.993672500000059,52.79835499700005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Bedfordshire and Hertfordshire","NUTS_NAME":"Bedfordshire and Hertfordshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.157241000999932,52.080547497000055],[-0.126529000999938,52.02091999900006],[-0.016232,52.06300749800005],[0.068169,52.005786998000076],[0.124805500000036,51.88458249700005],[0.195639,51.86808399600005],[-0.012939,51.742748498000026],[-0.011878499999966,51.680877499000076],[-0.182047499999953,51.66860199600006],[-0.304423,51.636348498000075],[-0.500561499999947,51.59968949700004],[-0.586348499999929,51.752113499000075],[-0.745649500999946,51.84209449800005],[-0.553598,51.82670949900006],[-0.70212749999996,51.909118498000055],[-0.652946499999928,51.96923050000004],[-0.668942,52.04871749800003],[-0.591772499999934,52.11069099900004],[-0.668097,52.19503399800004],[-0.653614499999946,52.268291496000074],[-0.465321499999959,52.322956],[-0.249742,52.184372],[-0.144351,52.13746249800005],[-0.157241000999932,52.080547497000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Luton","NUTS_NAME":"Luton"},"geometry":{"type":"Polygon","coordinates":[[[-0.385580499999946,51.91568],[-0.354797499999961,51.87401199800007],[-0.499775499999942,51.897831],[-0.385580499999946,51.91568]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Hertfordshire","NUTS_NAME":"Hertfordshire"},"geometry":{"type":"Polygon","coordinates":[[[0.068169,52.005786998000076],[0.124805500000036,51.88458249700005],[0.195639,51.86808399600005],[-0.012939,51.742748498000026],[-0.011878499999966,51.680877499000076],[-0.182047499999953,51.66860199600006],[-0.304423,51.636348498000075],[-0.500561499999947,51.59968949700004],[-0.586348499999929,51.752113499000075],[-0.745649500999946,51.84209449800005],[-0.553598,51.82670949900006],[-0.354797499999961,51.87401199800007],[-0.385580499999946,51.91568],[-0.391268,51.97702049900005],[-0.247853000999953,51.985172498000054],[-0.157241000999932,52.080547497000055],[-0.126529000999938,52.02091999900006],[-0.016232,52.06300749800005],[0.068169,52.005786998000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH24","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bedford","NUTS_NAME":"Bedford"},"geometry":{"type":"Polygon","coordinates":[[[-0.249742,52.184372],[-0.396979,52.071003],[-0.591772499999934,52.11069099900004],[-0.668097,52.19503399800004],[-0.653614499999946,52.268291496000074],[-0.465321499999959,52.322956],[-0.249742,52.184372]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH25","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Central Bedfordshire","NUTS_NAME":"Central Bedfordshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.157241000999932,52.080547497000055],[-0.247853000999953,51.985172498000054],[-0.391268,51.97702049900005],[-0.385580499999946,51.91568],[-0.499775499999942,51.897831],[-0.354797499999961,51.87401199800007],[-0.553598,51.82670949900006],[-0.70212749999996,51.909118498000055],[-0.652946499999928,51.96923050000004],[-0.668942,52.04871749800003],[-0.591772499999934,52.11069099900004],[-0.396979,52.071003],[-0.249742,52.184372],[-0.144351,52.13746249800005],[-0.157241000999932,52.080547497000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Essex","NUTS_NAME":"Essex"},"geometry":{"type":"Polygon","coordinates":[[[0.854097374000048,51.601599640000074],[0.958179500000028,51.61968999900006],[0.821244,51.54071449700007],[0.626791,51.53217299800008],[0.51370350000002,51.53118499700008],[0.54146,51.51271449800004],[0.401860592000048,51.45663058400004],[0.379662895000024,51.45728967800005],[0.217437,51.480071999000074],[0.210514,51.49003599800005],[0.313079500000072,51.565814998000064],[0.200354,51.62493149800008],[0.138225500000033,51.62354299900005],[-0.012219499999958,51.64622899700004],[-0.011878499999966,51.680877499000076],[-0.012939,51.742748498000026],[0.195639,51.86808399600005],[0.124805500000036,51.88458249700005],[0.068169,52.005786998000076],[0.235253500000056,52.088996999000074],[0.389743,52.03646450000008],[0.404686500000025,52.06549849800007],[0.705025500000033,52.062668],[0.781522500000051,51.96650699800006],[1.05638,51.95149249700006],[1.280599,51.940426],[1.291964500000063,51.87065099900008],[1.124435,51.77542499700007],[0.844628,51.781318498000076],[0.894665,51.76629249700005],[0.688036500000067,51.731712498000036],[0.71307,51.71509149900004],[0.756304,51.691741999000044],[0.94589650000006,51.71621299700007],[0.926602500000058,51.628940500000056],[0.776491999000029,51.63602449800004],[0.764340716000049,51.63674747400006],[0.854097374000048,51.601599640000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Southend-on-Sea","NUTS_NAME":"Southend-on-Sea"},"geometry":{"type":"Polygon","coordinates":[[[0.821244,51.54071449700007],[0.626791,51.53217299800008],[0.634696,51.57493199800007],[0.821244,51.54071449700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Thurrock","NUTS_NAME":"Thurrock"},"geometry":{"type":"Polygon","coordinates":[[[0.51370350000002,51.53118499700008],[0.54146,51.51271449800004],[0.401860592000048,51.45663058400004],[0.379662895000024,51.45728967800005],[0.217437,51.480071999000074],[0.210514,51.49003599800005],[0.313079500000072,51.565814998000064],[0.38264950000007,51.565818996000075],[0.51370350000002,51.53118499700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH34","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Essex Haven Gateway","NUTS_NAME":"Essex Haven Gateway"},"geometry":{"type":"Polygon","coordinates":[[[0.705025500000033,52.062668],[0.781522500000051,51.96650699800006],[1.05638,51.95149249700006],[1.280599,51.940426],[1.291964500000063,51.87065099900008],[1.124435,51.77542499700007],[0.844628,51.781318498000076],[0.698827,51.82841099700005],[0.572150500000021,51.751895998000066],[0.519312500000069,51.85044499800006],[0.421441,51.92629999800005],[0.431438,52.03892899700003],[0.389743,52.03646450000008],[0.404686500000025,52.06549849800007],[0.705025500000033,52.062668]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH35","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Essex","NUTS_NAME":"West Essex"},"geometry":{"type":"Polygon","coordinates":[[[0.389743,52.03646450000008],[0.431438,52.03892899700003],[0.421441,51.92629999800005],[0.519312500000069,51.85044499800006],[0.413043500000072,51.85631949800006],[0.302171,51.68096150000008],[0.200354,51.62493149800008],[0.138225500000033,51.62354299900005],[-0.012219499999958,51.64622899700004],[-0.011878499999966,51.680877499000076],[-0.012939,51.742748498000026],[0.195639,51.86808399600005],[0.124805500000036,51.88458249700005],[0.068169,52.005786998000076],[0.235253500000056,52.088996999000074],[0.389743,52.03646450000008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH36","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Heart of Essex","NUTS_NAME":"Heart of Essex"},"geometry":{"type":"Polygon","coordinates":[[[0.519312500000069,51.85044499800006],[0.572150500000021,51.751895998000066],[0.698827,51.82841099700005],[0.844628,51.781318498000076],[0.894665,51.76629249700005],[0.688036500000067,51.731712498000036],[0.71307,51.71509149900004],[0.756304,51.691741999000044],[0.94589650000006,51.71621299700007],[0.926602500000058,51.628940500000056],[0.776491999000029,51.63602449800004],[0.764340716000049,51.63674747400006],[0.762833917000023,51.636837126000046],[0.711096999000063,51.63991549800005],[0.590769500000022,51.63064949800008],[0.386124500000051,51.634341996000046],[0.38264950000007,51.565818996000075],[0.313079500000072,51.565814998000064],[0.200354,51.62493149800008],[0.302171,51.68096150000008],[0.413043500000072,51.85631949800006],[0.519312500000069,51.85044499800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKH37","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Essex Thames Gateway","NUTS_NAME":"Essex Thames Gateway"},"geometry":{"type":"Polygon","coordinates":[[[0.590769500000022,51.63064949800008],[0.711096999000063,51.63991549800005],[0.762833917000023,51.636837126000046],[0.764340716000049,51.63674747400006],[0.854097374000048,51.601599640000074],[0.958179500000028,51.61968999900006],[0.821244,51.54071449700007],[0.634696,51.57493199800007],[0.626791,51.53217299800008],[0.51370350000002,51.53118499700008],[0.38264950000007,51.565818996000075],[0.386124500000051,51.634341996000046],[0.590769500000022,51.63064949800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"LONDON","NUTS_NAME":"LONDON"},"geometry":{"type":"Polygon","coordinates":[[[-0.012219499999958,51.64622899700004],[0.138225500000033,51.62354299900005],[0.200354,51.62493149800008],[0.313079500000072,51.565814998000064],[0.210514,51.49003599800005],[0.217437,51.480071999000074],[0.152974,51.40870299800008],[0.148871,51.40850449900006],[0.042433500000072,51.29266749700008],[0.002330500000028,51.329131999000026],[-0.156508,51.321506499000066],[-0.244644499999936,51.36905649800008],[-0.330622,51.329006],[-0.317662,51.393665498000075],[-0.458605499999976,51.45631399900003],[-0.50966650099997,51.46917349900008],[-0.489989499999979,51.49474699900003],[-0.500561499999947,51.59968949700004],[-0.304423,51.636348498000075],[-0.182047499999953,51.66860199600006],[-0.011878499999966,51.680877499000076],[-0.012219499999958,51.64622899700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Inner London - West","NUTS_NAME":"Inner London - West"},"geometry":{"type":"Polygon","coordinates":[[[-0.171223,51.572429500000055],[-0.085154,51.52033249800007],[-0.078408,51.52150750000004],[-0.079542452999931,51.50774022300004],[-0.079878637999968,51.500596306000034],[-0.108863,51.50844949800006],[-0.128365499999973,51.48505799800006],[-0.140344,51.41924649600003],[-0.253975499999967,51.43723699900005],[-0.222829,51.47181699600003],[-0.244502,51.48870099700008],[-0.253024,51.50139999900006],[-0.246334499999932,51.53279899800003],[-0.215968,51.527927498000054],[-0.191422,51.53628899800003],[-0.213401000999966,51.55514899800005],[-0.171223,51.572429500000055]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Camden & City of London","NUTS_NAME":"Camden & City of London"},"geometry":{"type":"Polygon","coordinates":[[[-0.085154,51.52033249800007],[-0.078408,51.52150750000004],[-0.079542452999931,51.50774022300004],[-0.079878637999968,51.500596306000034],[-0.108863,51.50844949800006],[-0.111511,51.51075349800004],[-0.191422,51.53628899800003],[-0.213401000999966,51.55514899800005],[-0.171223,51.572429500000055],[-0.085154,51.52033249800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Westminster","NUTS_NAME":"Westminster"},"geometry":{"type":"Polygon","coordinates":[[[-0.111511,51.51075349800004],[-0.108863,51.50844949800006],[-0.128365499999973,51.48505799800006],[-0.149948499999937,51.48551549800004],[-0.215968,51.527927498000054],[-0.191422,51.53628899800003],[-0.111511,51.51075349800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI33","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Kensington and Chelsea & Hammersmith and Fulham","NUTS_NAME":"Kensington and Chelsea & Hammersmith and Fulham"},"geometry":{"type":"Polygon","coordinates":[[[-0.215968,51.527927498000054],[-0.149948499999937,51.48551549800004],[-0.128365499999973,51.48505799800006],[-0.161824425999953,51.48036760600007],[-0.213546233999978,51.47311799800008],[-0.222829,51.47181699600003],[-0.244502,51.48870099700008],[-0.253024,51.50139999900006],[-0.246334499999932,51.53279899800003],[-0.215968,51.527927498000054]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI34","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Wandsworth","NUTS_NAME":"Wandsworth"},"geometry":{"type":"Polygon","coordinates":[[[-0.140344,51.41924649600003],[-0.253975499999967,51.43723699900005],[-0.222829,51.47181699600003],[-0.213546233999978,51.47311799800008],[-0.161824425999953,51.48036760600007],[-0.128365499999973,51.48505799800006],[-0.140344,51.41924649600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI4","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Inner London - East","NUTS_NAME":"Inner London - East"},"geometry":{"type":"Polygon","coordinates":[[[0.068394,51.54441450000007],[0.096999499000049,51.515670998000076],[0.073823467000068,51.50462927600006],[0.048330898000074,51.498521839000034],[-0.024711499999967,51.485649],[0.028960500000039,51.44162349800007],[-0.078261,51.42050549900006],[-0.078611500999955,51.41982249800003],[-0.127744,51.412315498000055],[-0.140344,51.41924649600003],[-0.128365499999973,51.48505799800006],[-0.108863,51.50844949800006],[-0.079878637999968,51.500596306000034],[-0.079542452999931,51.50774022300004],[-0.078408,51.52150750000004],[-0.085154,51.52033249800007],[-0.171223,51.572429500000055],[-0.138722499999972,51.61019149700007],[-0.041382499999941,51.60563649800008],[-0.061119,51.577785498000026],[0.068394,51.54441450000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI41","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Hackney & Newham","NUTS_NAME":"Hackney & Newham"},"geometry":{"type":"Polygon","coordinates":[[[0.068394,51.54441450000007],[0.096999499000049,51.515670998000076],[0.073823467000068,51.50462927600006],[0.048330898000074,51.498521839000034],[0.00941750000004,51.50709899800006],[-0.078408,51.52150750000004],[-0.085154,51.52033249800007],[-0.104447000999926,51.56470499900007],[-0.061119,51.577785498000026],[0.068394,51.54441450000007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI42","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Tower Hamlets","NUTS_NAME":"Tower Hamlets"},"geometry":{"type":"Polygon","coordinates":[[[0.00941750000004,51.50709899800006],[0.048330898000074,51.498521839000034],[-0.024711499999967,51.485649],[-0.079878637999968,51.500596306000034],[-0.079542452999931,51.50774022300004],[-0.078408,51.52150750000004],[0.00941750000004,51.50709899800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI43","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Haringey & Islington","NUTS_NAME":"Haringey & Islington"},"geometry":{"type":"Polygon","coordinates":[[[-0.041382499999941,51.60563649800008],[-0.061119,51.577785498000026],[-0.104447000999926,51.56470499900007],[-0.085154,51.52033249800007],[-0.171223,51.572429500000055],[-0.138722499999972,51.61019149700007],[-0.041382499999941,51.60563649800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI44","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Lewisham & Southwark","NUTS_NAME":"Lewisham & Southwark"},"geometry":{"type":"Polygon","coordinates":[[[-0.024711499999967,51.485649],[0.028960500000039,51.44162349800007],[-0.078261,51.42050549900006],[-0.108863,51.50844949800006],[-0.079878637999968,51.500596306000034],[-0.024711499999967,51.485649]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI45","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Lambeth","NUTS_NAME":"Lambeth"},"geometry":{"type":"Polygon","coordinates":[[[-0.078261,51.42050549900006],[-0.078611500999955,51.41982249800003],[-0.127744,51.412315498000055],[-0.140344,51.41924649600003],[-0.128365499999973,51.48505799800006],[-0.108863,51.50844949800006],[-0.078261,51.42050549900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI5","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Outer London - East and North East","NUTS_NAME":"Outer London - East and North East"},"geometry":{"type":"Polygon","coordinates":[[[-0.012219499999958,51.64622899700004],[0.138225500000033,51.62354299900005],[0.200354,51.62493149800008],[0.313079500000072,51.565814998000064],[0.210514,51.49003599800005],[0.217437,51.480071999000074],[0.152974,51.40870299800008],[0.148871,51.40850449900006],[0.028960500000039,51.44162349800007],[-0.024711499999967,51.485649],[0.048330898000074,51.498521839000034],[0.073823467000068,51.50462927600006],[0.096999499000049,51.515670998000076],[0.068394,51.54441450000007],[-0.061119,51.577785498000026],[-0.041382499999941,51.60563649800008],[-0.138722499999972,51.61019149700007],[-0.182047499999953,51.66860199600006],[-0.011878499999966,51.680877499000076],[-0.012219499999958,51.64622899700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI51","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bexley & Greenwich","NUTS_NAME":"Bexley & Greenwich"},"geometry":{"type":"Polygon","coordinates":[[[0.152974,51.40870299800008],[0.148871,51.40850449900006],[0.028960500000039,51.44162349800007],[-0.024711499999967,51.485649],[0.048330898000074,51.498521839000034],[0.073823467000068,51.50462927600006],[0.096999499000049,51.515670998000076],[0.161074608000035,51.49919295900003],[0.195995402000051,51.48864586000008],[0.217437,51.480071999000074],[0.152974,51.40870299800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI52","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Barking and Dagenham & Havering","NUTS_NAME":"Barking and Dagenham & Havering"},"geometry":{"type":"Polygon","coordinates":[[[0.210514,51.49003599800005],[0.217437,51.480071999000074],[0.195995402000051,51.48864586000008],[0.161074608000035,51.49919295900003],[0.096999499000049,51.515670998000076],[0.068394,51.54441450000007],[0.138225500000033,51.62354299900005],[0.200354,51.62493149800008],[0.313079500000072,51.565814998000064],[0.210514,51.49003599800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI53","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Redbridge & Waltham Forest","NUTS_NAME":"Redbridge & Waltham Forest"},"geometry":{"type":"Polygon","coordinates":[[[0.138225500000033,51.62354299900005],[0.068394,51.54441450000007],[-0.061119,51.577785498000026],[-0.041382499999941,51.60563649800008],[-0.012219499999958,51.64622899700004],[0.138225500000033,51.62354299900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI54","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Enfield","NUTS_NAME":"Enfield"},"geometry":{"type":"Polygon","coordinates":[[[-0.012219499999958,51.64622899700004],[-0.041382499999941,51.60563649800008],[-0.138722499999972,51.61019149700007],[-0.182047499999953,51.66860199600006],[-0.011878499999966,51.680877499000076],[-0.012219499999958,51.64622899700004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI6","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Outer London - South","NUTS_NAME":"Outer London - South"},"geometry":{"type":"Polygon","coordinates":[[[0.028960500000039,51.44162349800007],[0.148871,51.40850449900006],[0.042433500000072,51.29266749700008],[0.002330500000028,51.329131999000026],[-0.156508,51.321506499000066],[-0.244644499999936,51.36905649800008],[-0.330622,51.329006],[-0.317662,51.393665498000075],[-0.253975499999967,51.43723699900005],[-0.140344,51.41924649600003],[-0.127744,51.412315498000055],[-0.078611500999955,51.41982249800003],[-0.078261,51.42050549900006],[0.028960500000039,51.44162349800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI61","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bromley","NUTS_NAME":"Bromley"},"geometry":{"type":"Polygon","coordinates":[[[0.148871,51.40850449900006],[0.042433500000072,51.29266749700008],[0.002330500000028,51.329131999000026],[-0.078611500999955,51.41982249800003],[-0.078261,51.42050549900006],[0.028960500000039,51.44162349800007],[0.148871,51.40850449900006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI62","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Croydon","NUTS_NAME":"Croydon"},"geometry":{"type":"Polygon","coordinates":[[[0.002330500000028,51.329131999000026],[-0.156508,51.321506499000066],[-0.117250500999944,51.345676498000046],[-0.127744,51.412315498000055],[-0.078611500999955,51.41982249800003],[0.002330500000028,51.329131999000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI63","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Merton, Kingston upon Thames & Sutton","NUTS_NAME":"Merton, Kingston upon Thames & Sutton"},"geometry":{"type":"Polygon","coordinates":[[[-0.140344,51.41924649600003],[-0.127744,51.412315498000055],[-0.117250500999944,51.345676498000046],[-0.156508,51.321506499000066],[-0.244644499999936,51.36905649800008],[-0.330622,51.329006],[-0.317662,51.393665498000075],[-0.253975499999967,51.43723699900005],[-0.140344,51.41924649600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI7","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Outer London - West and North West","NUTS_NAME":"Outer London - West and North West"},"geometry":{"type":"Polygon","coordinates":[[[-0.191422,51.53628899800003],[-0.215968,51.527927498000054],[-0.246334499999932,51.53279899800003],[-0.253024,51.50139999900006],[-0.244502,51.48870099700008],[-0.222829,51.47181699600003],[-0.253975499999967,51.43723699900005],[-0.317662,51.393665498000075],[-0.458605499999976,51.45631399900003],[-0.50966650099997,51.46917349900008],[-0.489989499999979,51.49474699900003],[-0.500561499999947,51.59968949700004],[-0.304423,51.636348498000075],[-0.182047499999953,51.66860199600006],[-0.138722499999972,51.61019149700007],[-0.171223,51.572429500000055],[-0.213401000999966,51.55514899800005],[-0.191422,51.53628899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI71","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Barnet","NUTS_NAME":"Barnet"},"geometry":{"type":"Polygon","coordinates":[[[-0.138722499999972,51.61019149700007],[-0.171223,51.572429500000055],[-0.213401000999966,51.55514899800005],[-0.267095500999972,51.60037249800007],[-0.304423,51.636348498000075],[-0.182047499999953,51.66860199600006],[-0.138722499999972,51.61019149700007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI72","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Brent","NUTS_NAME":"Brent"},"geometry":{"type":"Polygon","coordinates":[[[-0.213401000999966,51.55514899800005],[-0.191422,51.53628899800003],[-0.215968,51.527927498000054],[-0.246334499999932,51.53279899800003],[-0.33552549999996,51.556583498000066],[-0.267095500999972,51.60037249800007],[-0.213401000999966,51.55514899800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI73","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Ealing","NUTS_NAME":"Ealing"},"geometry":{"type":"Polygon","coordinates":[[[-0.246334499999932,51.53279899800003],[-0.253024,51.50139999900006],[-0.406848500999956,51.49969099600003],[-0.376323,51.528839],[-0.418477,51.54024899800004],[-0.33552549999996,51.556583498000066],[-0.246334499999932,51.53279899800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI74","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Harrow & Hillingdon","NUTS_NAME":"Harrow & Hillingdon"},"geometry":{"type":"Polygon","coordinates":[[[-0.267095500999972,51.60037249800007],[-0.33552549999996,51.556583498000066],[-0.418477,51.54024899800004],[-0.376323,51.528839],[-0.406848500999956,51.49969099600003],[-0.458605499999976,51.45631399900003],[-0.50966650099997,51.46917349900008],[-0.489989499999979,51.49474699900003],[-0.500561499999947,51.59968949700004],[-0.304423,51.636348498000075],[-0.267095500999972,51.60037249800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKI75","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Hounslow & Richmond upon Thames","NUTS_NAME":"Hounslow & Richmond upon Thames"},"geometry":{"type":"Polygon","coordinates":[[[-0.244502,51.48870099700008],[-0.222829,51.47181699600003],[-0.253975499999967,51.43723699900005],[-0.317662,51.393665498000075],[-0.458605499999976,51.45631399900003],[-0.406848500999956,51.49969099600003],[-0.253024,51.50139999900006],[-0.244502,51.48870099700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"SOUTH EAST (ENGLAND)","NUTS_NAME":"SOUTH EAST (ENGLAND)"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.668942,52.04871749800003],[-0.652946499999928,51.96923050000004],[-0.70212749999996,51.909118498000055],[-0.553598,51.82670949900006],[-0.745649500999946,51.84209449800005],[-0.586348499999929,51.752113499000075],[-0.500561499999947,51.59968949700004],[-0.489989499999979,51.49474699900003],[-0.50966650099997,51.46917349900008],[-0.458605499999976,51.45631399900003],[-0.317662,51.393665498000075],[-0.330622,51.329006],[-0.244644499999936,51.36905649800008],[-0.156508,51.321506499000066],[0.002330500000028,51.329131999000026],[0.042433500000072,51.29266749700008],[0.148871,51.40850449900006],[0.152974,51.40870299800008],[0.217437,51.480071999000074],[0.379662895000024,51.45728967800005],[0.401860592000048,51.45663058400004],[0.458900500000027,51.45493699800005],[0.720437,51.45972799800006],[0.539261022000062,51.40701982600007],[0.626894500000049,51.37520999900005],[0.817462036000052,51.37265732800006],[0.724821,51.41263199700006],[0.743699,51.44689949800005],[0.951056,51.373244998000075],[0.855278181000074,51.36499536100007],[0.950297500000033,51.34574899900008],[1.44974650000006,51.37717799700005],[1.37071,51.31177899700003],[1.404974500000037,51.22982049700005],[1.379638500000056,51.14215849900006],[1.034675,51.04708849800005],[0.977051500000073,51.002594],[0.98057299900006,50.914032],[0.854784,50.92370999800005],[0.377511,50.820377499000074],[0.240893,50.734935998000026],[-0.038176499999963,50.799503498000036],[-0.216009,50.82756399600004],[-0.787529499999948,50.72245399900004],[-0.916800499999965,50.780898997000065],[-0.932831,50.843150999000045],[-1.021779506999962,50.83572701200006],[-1.09147,50.77824399600007],[-1.116579,50.842498997000064],[-1.156856,50.838321499000074],[-1.111877499999935,50.789558499000066],[-1.138110499999925,50.773792498000034],[-1.365159499999947,50.88006949600003],[-1.477037499999938,50.92395799600007],[-1.306856,50.81985449800004],[-1.577171499999963,50.716056999000045],[-1.692049499999939,50.736625497000034],[-1.788733,50.76716249800006],[-1.829502499999933,50.85531599700005],[-1.810535,50.92680749900006],[-1.956807,50.98982999800006],[-1.623381499999937,50.95463550000005],[-1.59938,51.023727499000074],[-1.668818499999929,51.190772999000046],[-1.535935,51.24861899700005],[-1.49828,51.329375999000035],[-1.571147499999938,51.416465996000056],[-1.545737499999973,51.458991996000066],[-1.584689499999968,51.524913998000045],[-1.602793499999962,51.51829549800004],[-1.691093,51.58352649700004],[-1.683040499999947,51.69011299600004],[-1.648235499999942,51.68408949800005],[-1.719469499999946,51.78321450000004],[-1.645167,51.91452050000004],[-1.665732499999933,51.987491497000065],[-1.527906499999972,51.991756499000076],[-1.487778,52.09395999600008],[-1.357743,52.10131050000007],[-1.331868499999928,52.16848749900004],[-1.247932,52.09577550000006],[-1.320798,52.087630999000055],[-1.248983499999952,51.985714],[-1.118058,52.01542649700008],[-1.042297,52.07371149900007],[-0.871292499999925,52.040252499000076],[-0.831384499999956,52.07194899700005],[-0.88068549999997,52.12633149800007],[-0.705417499999953,52.19157049700004],[-0.668097,52.19503399800004],[-0.591772499999934,52.11069099900004],[-0.668942,52.04871749800003]]],[[[-1.069896,50.683647],[-1.303737499999954,50.57604599700005],[-1.586375,50.66316599600003],[-1.292164499999956,50.75799949800006],[-1.069896,50.683647]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Berkshire, Buckinghamshire and Oxfordshire","NUTS_NAME":"Berkshire, Buckinghamshire and Oxfordshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.668942,52.04871749800003],[-0.652946499999928,51.96923050000004],[-0.70212749999996,51.909118498000055],[-0.553598,51.82670949900006],[-0.745649500999946,51.84209449800005],[-0.586348499999929,51.752113499000075],[-0.500561499999947,51.59968949700004],[-0.489989499999979,51.49474699900003],[-0.50966650099997,51.46917349900008],[-0.775435499999958,51.331954996000036],[-1.075324499999965,51.379493496000066],[-1.429674499999976,51.33653249900004],[-1.49828,51.329375999000035],[-1.571147499999938,51.416465996000056],[-1.545737499999973,51.458991996000066],[-1.584689499999968,51.524913998000045],[-1.602793499999962,51.51829549800004],[-1.691093,51.58352649700004],[-1.683040499999947,51.69011299600004],[-1.648235499999942,51.68408949800005],[-1.719469499999946,51.78321450000004],[-1.645167,51.91452050000004],[-1.665732499999933,51.987491497000065],[-1.527906499999972,51.991756499000076],[-1.487778,52.09395999600008],[-1.357743,52.10131050000007],[-1.331868499999928,52.16848749900004],[-1.247932,52.09577550000006],[-1.320798,52.087630999000055],[-1.248983499999952,51.985714],[-1.118058,52.01542649700008],[-1.042297,52.07371149900007],[-0.871292499999925,52.040252499000076],[-0.831384499999956,52.07194899700005],[-0.88068549999997,52.12633149800007],[-0.705417499999953,52.19157049700004],[-0.668097,52.19503399800004],[-0.591772499999934,52.11069099900004],[-0.668942,52.04871749800003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Berkshire","NUTS_NAME":"Berkshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.489989499999979,51.49474699900003],[-0.50966650099997,51.46917349900008],[-0.775435499999958,51.331954996000036],[-1.075324499999965,51.379493496000066],[-1.429674499999976,51.33653249900004],[-1.49828,51.329375999000035],[-1.571147499999938,51.416465996000056],[-1.545737499999973,51.458991996000066],[-1.584689499999968,51.524913998000045],[-1.3287,51.563712999000074],[-0.949142499999937,51.45951099700005],[-0.870028,51.50411999800008],[-0.89685450099995,51.544860999000036],[-0.71223,51.57223150000004],[-0.66225,51.487922498000046],[-0.624735499999929,51.53800199800003],[-0.489989499999979,51.49474699900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Milton Keynes","NUTS_NAME":"Milton Keynes"},"geometry":{"type":"Polygon","coordinates":[[[-0.591772499999934,52.11069099900004],[-0.668942,52.04871749800003],[-0.652946499999928,51.96923050000004],[-0.871292499999925,52.040252499000076],[-0.831384499999956,52.07194899700005],[-0.88068549999997,52.12633149800007],[-0.705417499999953,52.19157049700004],[-0.668097,52.19503399800004],[-0.591772499999934,52.11069099900004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Buckinghamshire CC","NUTS_NAME":"Buckinghamshire CC"},"geometry":{"type":"Polygon","coordinates":[[[-0.871292499999925,52.040252499000076],[-0.652946499999928,51.96923050000004],[-0.70212749999996,51.909118498000055],[-0.553598,51.82670949900006],[-0.745649500999946,51.84209449800005],[-0.586348499999929,51.752113499000075],[-0.500561499999947,51.59968949700004],[-0.489989499999979,51.49474699900003],[-0.624735499999929,51.53800199800003],[-0.66225,51.487922498000046],[-0.71223,51.57223150000004],[-0.89685450099995,51.544860999000036],[-0.95000349999998,51.62890649700006],[-0.87768,51.66838450000006],[-0.887656499999935,51.71834549700003],[-1.12192,51.787082499000064],[-1.140652499999931,51.834689999000034],[-1.04731049999998,51.839709999000036],[-1.118058,52.01542649700008],[-1.042297,52.07371149900007],[-0.871292499999925,52.040252499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Oxfordshire","NUTS_NAME":"Oxfordshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.118058,52.01542649700008],[-1.04731049999998,51.839709999000036],[-1.140652499999931,51.834689999000034],[-1.12192,51.787082499000064],[-0.887656499999935,51.71834549700003],[-0.87768,51.66838450000006],[-0.95000349999998,51.62890649700006],[-0.89685450099995,51.544860999000036],[-0.870028,51.50411999800008],[-0.949142499999937,51.45951099700005],[-1.3287,51.563712999000074],[-1.584689499999968,51.524913998000045],[-1.602793499999962,51.51829549800004],[-1.691093,51.58352649700004],[-1.683040499999947,51.69011299600004],[-1.648235499999942,51.68408949800005],[-1.719469499999946,51.78321450000004],[-1.645167,51.91452050000004],[-1.665732499999933,51.987491497000065],[-1.527906499999972,51.991756499000076],[-1.487778,52.09395999600008],[-1.357743,52.10131050000007],[-1.331868499999928,52.16848749900004],[-1.247932,52.09577550000006],[-1.320798,52.087630999000055],[-1.248983499999952,51.985714],[-1.118058,52.01542649700008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Surrey, East and West Sussex","NUTS_NAME":"Surrey, East and West Sussex"},"geometry":{"type":"Polygon","coordinates":[[[-0.317662,51.393665498000075],[-0.330622,51.329006],[-0.244644499999936,51.36905649800008],[-0.156508,51.321506499000066],[0.002330500000028,51.329131999000026],[0.042433500000072,51.29266749700008],[0.050053500000047,51.14264299900003],[0.325307500000065,51.12287150000003],[0.605435,51.012058499000034],[0.779000999000061,50.98947899900003],[0.854784,50.92370999800005],[0.377511,50.820377499000074],[0.240893,50.734935998000026],[-0.038176499999963,50.799503498000036],[-0.216009,50.82756399600004],[-0.787529499999948,50.72245399900004],[-0.916800499999965,50.780898997000065],[-0.932831,50.843150999000045],[-0.938582,50.87371849800007],[-0.894479499999932,51.023491],[-0.753453499999978,51.08645249800003],[-0.848883499999943,51.21071249900007],[-0.738007,51.24179849800004],[-0.775435499999958,51.331954996000036],[-0.50966650099997,51.46917349900008],[-0.458605499999976,51.45631399900003],[-0.317662,51.393665498000075]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Brighton and Hove","NUTS_NAME":"Brighton and Hove"},"geometry":{"type":"Polygon","coordinates":[[[-0.038176499999963,50.799503498000036],[-0.216009,50.82756399600004],[-0.244942499999979,50.86379999800005],[-0.134979499999929,50.886626998000054],[-0.038176499999963,50.799503498000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Sussex CC","NUTS_NAME":"East Sussex CC"},"geometry":{"type":"Polygon","coordinates":[[[0.605435,51.012058499000034],[0.779000999000061,50.98947899900003],[0.854784,50.92370999800005],[0.377511,50.820377499000074],[0.240893,50.734935998000026],[-0.038176499999963,50.799503498000036],[-0.134979499999929,50.886626998000054],[-0.116074499999968,50.97035999800005],[-0.018739499999924,50.98895649800005],[-0.036162499999932,51.09344849800004],[0.027397499000074,51.13984299800006],[0.050053500000047,51.14264299900003],[0.325307500000065,51.12287150000003],[0.605435,51.012058499000034]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ25","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Surrey","NUTS_NAME":"West Surrey"},"geometry":{"type":"Polygon","coordinates":[[[-0.458605499999976,51.45631399900003],[-0.317662,51.393665498000075],[-0.330622,51.329006],[-0.434616499999947,51.23990249700006],[-0.41917449999994,51.105198],[-0.492099,51.09447850000004],[-0.753453499999978,51.08645249800003],[-0.848883499999943,51.21071249900007],[-0.738007,51.24179849800004],[-0.775435499999958,51.331954996000036],[-0.50966650099997,51.46917349900008],[-0.458605499999976,51.45631399900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ26","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Surrey","NUTS_NAME":"East Surrey"},"geometry":{"type":"Polygon","coordinates":[[[-0.156508,51.321506499000066],[0.002330500000028,51.329131999000026],[0.042433500000072,51.29266749700008],[0.050053500000047,51.14264299900003],[0.027397499000074,51.13984299800006],[-0.41917449999994,51.105198],[-0.434616499999947,51.23990249700006],[-0.330622,51.329006],[-0.244644499999936,51.36905649800008],[-0.156508,51.321506499000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ27","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Sussex (South West)","NUTS_NAME":"West Sussex (South West)"},"geometry":{"type":"Polygon","coordinates":[[[-0.244942499999979,50.86379999800005],[-0.216009,50.82756399600004],[-0.787529499999948,50.72245399900004],[-0.916800499999965,50.780898997000065],[-0.932831,50.843150999000045],[-0.938582,50.87371849800007],[-0.894479499999932,51.023491],[-0.753453499999978,51.08645249800003],[-0.492099,51.09447850000004],[-0.475866499999938,51.02087799800006],[-0.568738499999938,50.942409498000075],[-0.555431,50.88193499800008],[-0.244942499999979,50.86379999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ28","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Sussex (North East)","NUTS_NAME":"West Sussex (North East)"},"geometry":{"type":"Polygon","coordinates":[[[0.027397499000074,51.13984299800006],[-0.036162499999932,51.09344849800004],[-0.018739499999924,50.98895649800005],[-0.116074499999968,50.97035999800005],[-0.134979499999929,50.886626998000054],[-0.244942499999979,50.86379999800005],[-0.555431,50.88193499800008],[-0.568738499999938,50.942409498000075],[-0.475866499999938,51.02087799800006],[-0.492099,51.09447850000004],[-0.41917449999994,51.105198],[0.027397499000074,51.13984299800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Hampshire and Isle of Wight","NUTS_NAME":"Hampshire and Isle of Wight"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.894479499999932,51.023491],[-0.938582,50.87371849800007],[-0.932831,50.843150999000045],[-1.021779506999962,50.83572701200006],[-1.09147,50.77824399600007],[-1.116579,50.842498997000064],[-1.156856,50.838321499000074],[-1.111877499999935,50.789558499000066],[-1.138110499999925,50.773792498000034],[-1.365159499999947,50.88006949600003],[-1.477037499999938,50.92395799600007],[-1.306856,50.81985449800004],[-1.577171499999963,50.716056999000045],[-1.692049499999939,50.736625497000034],[-1.788733,50.76716249800006],[-1.829502499999933,50.85531599700005],[-1.810535,50.92680749900006],[-1.956807,50.98982999800006],[-1.623381499999937,50.95463550000005],[-1.59938,51.023727499000074],[-1.668818499999929,51.190772999000046],[-1.535935,51.24861899700005],[-1.49828,51.329375999000035],[-1.429674499999976,51.33653249900004],[-1.075324499999965,51.379493496000066],[-0.775435499999958,51.331954996000036],[-0.738007,51.24179849800004],[-0.848883499999943,51.21071249900007],[-0.753453499999978,51.08645249800003],[-0.894479499999932,51.023491]]],[[[-1.069896,50.683647],[-1.303737499999954,50.57604599700005],[-1.586375,50.66316599600003],[-1.292164499999956,50.75799949800006],[-1.069896,50.683647]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ31","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Portsmouth","NUTS_NAME":"Portsmouth"},"geometry":{"type":"Polygon","coordinates":[[[-1.053991,50.85654849800005],[-1.021779506999962,50.83572701200006],[-1.09147,50.77824399600007],[-1.116579,50.842498997000064],[-1.115824,50.858268499000076],[-1.053991,50.85654849800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ32","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Southampton","NUTS_NAME":"Southampton"},"geometry":{"type":"Polygon","coordinates":[[[-1.365159499999947,50.88006949600003],[-1.477037499999938,50.92395799600007],[-1.379869,50.94751749800008],[-1.322064,50.90092849800004],[-1.365159499999947,50.88006949600003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ34","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Isle of Wight","NUTS_NAME":"Isle of Wight"},"geometry":{"type":"Polygon","coordinates":[[[-1.069896,50.683647],[-1.303737499999954,50.57604599700005],[-1.586375,50.66316599600003],[-1.292164499999956,50.75799949800006],[-1.069896,50.683647]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ35","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"South Hampshire","NUTS_NAME":"South Hampshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.273667499999931,50.935699499000066],[-1.253474499999925,50.87238299900008],[-1.115824,50.858268499000076],[-1.116579,50.842498997000064],[-1.156856,50.838321499000074],[-1.111877499999935,50.789558499000066],[-1.138110499999925,50.773792498000034],[-1.365159499999947,50.88006949600003],[-1.322064,50.90092849800004],[-1.379869,50.94751749800008],[-1.392952,51.00435999800004],[-1.258131,50.97833999900007],[-1.273667499999931,50.935699499000066]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ36","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Central Hampshire","NUTS_NAME":"Central Hampshire"},"geometry":{"type":"Polygon","coordinates":[[[-1.213375499999927,51.151946999000074],[-1.178113,51.13405249900006],[-0.848883499999943,51.21071249900007],[-0.753453499999978,51.08645249800003],[-0.894479499999932,51.023491],[-0.938582,50.87371849800007],[-0.932831,50.843150999000045],[-1.021779506999962,50.83572701200006],[-1.053991,50.85654849800005],[-1.115824,50.858268499000076],[-1.253474499999925,50.87238299900008],[-1.273667499999931,50.935699499000066],[-1.258131,50.97833999900007],[-1.392952,51.00435999800004],[-1.379869,50.94751749800008],[-1.477037499999938,50.92395799600007],[-1.306856,50.81985449800004],[-1.577171499999963,50.716056999000045],[-1.692049499999939,50.736625497000034],[-1.788733,50.76716249800006],[-1.829502499999933,50.85531599700005],[-1.810535,50.92680749900006],[-1.956807,50.98982999800006],[-1.623381499999937,50.95463550000005],[-1.59938,51.023727499000074],[-1.668818499999929,51.190772999000046],[-1.535935,51.24861899700005],[-1.49828,51.329375999000035],[-1.429674499999976,51.33653249900004],[-1.416829499999949,51.28287499800007],[-1.460132499999929,51.255889996000064],[-1.428075,51.225036498000065],[-1.189572,51.18240749700004],[-1.213375499999927,51.151946999000074]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ37","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"North Hampshire","NUTS_NAME":"North Hampshire"},"geometry":{"type":"Polygon","coordinates":[[[-0.775435499999958,51.331954996000036],[-0.738007,51.24179849800004],[-0.848883499999943,51.21071249900007],[-1.178113,51.13405249900006],[-1.213375499999927,51.151946999000074],[-1.189572,51.18240749700004],[-1.428075,51.225036498000065],[-1.460132499999929,51.255889996000064],[-1.416829499999949,51.28287499800007],[-1.429674499999976,51.33653249900004],[-1.075324499999965,51.379493496000066],[-0.775435499999958,51.331954996000036]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ4","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Kent","NUTS_NAME":"Kent"},"geometry":{"type":"Polygon","coordinates":[[[0.401860592000048,51.45663058400004],[0.458900500000027,51.45493699800005],[0.720437,51.45972799800006],[0.539261022000062,51.40701982600007],[0.626894500000049,51.37520999900005],[0.817462036000052,51.37265732800006],[0.724821,51.41263199700006],[0.743699,51.44689949800005],[0.951056,51.373244998000075],[0.855278181000074,51.36499536100007],[0.950297500000033,51.34574899900008],[1.44974650000006,51.37717799700005],[1.37071,51.31177899700003],[1.404974500000037,51.22982049700005],[1.379638500000056,51.14215849900006],[1.034675,51.04708849800005],[0.977051500000073,51.002594],[0.98057299900006,50.914032],[0.854784,50.92370999800005],[0.779000999000061,50.98947899900003],[0.605435,51.012058499000034],[0.325307500000065,51.12287150000003],[0.050053500000047,51.14264299900003],[0.042433500000072,51.29266749700008],[0.148871,51.40850449900006],[0.152974,51.40870299800008],[0.217437,51.480071999000074],[0.379662895000024,51.45728967800005],[0.401860592000048,51.45663058400004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ41","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Medway","NUTS_NAME":"Medway"},"geometry":{"type":"Polygon","coordinates":[[[0.626894500000049,51.37520999900005],[0.601451500000053,51.333293998000045],[0.525687,51.33470899900004],[0.468097323000052,51.362647552000055],[0.401326500000039,51.35294699800005],[0.489282,51.415321498000026],[0.458900500000027,51.45493699800005],[0.720437,51.45972799800006],[0.539261022000062,51.40701982600007],[0.626894500000049,51.37520999900005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ43","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Kent Thames Gateway","NUTS_NAME":"Kent Thames Gateway"},"geometry":{"type":"MultiPolygon","coordinates":[[[[0.951056,51.373244998000075],[0.855278181000074,51.36499536100007],[0.950297500000033,51.34574899900008],[1.015095500000029,51.31676850000008],[0.973332500000026,51.26858149900005],[0.821158500000024,51.224418498000034],[0.601451500000053,51.333293998000045],[0.626894500000049,51.37520999900005],[0.817462036000052,51.37265732800006],[0.724821,51.41263199700006],[0.743699,51.44689949800005],[0.951056,51.373244998000075]]],[[[0.401326500000039,51.35294699800005],[0.341124,51.325001],[0.336721500000067,51.386539500000026],[0.152974,51.40870299800008],[0.217437,51.480071999000074],[0.379662895000024,51.45728967800005],[0.401860592000048,51.45663058400004],[0.458900500000027,51.45493699800005],[0.489282,51.415321498000026],[0.401326500000039,51.35294699800005]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ44","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"East Kent","NUTS_NAME":"East Kent"},"geometry":{"type":"Polygon","coordinates":[[[0.854784,50.92370999800005],[0.779000999000061,50.98947899900003],[1.029294500000049,51.13252249700008],[0.973332500000026,51.26858149900005],[1.015095500000029,51.31676850000008],[0.950297500000033,51.34574899900008],[1.44974650000006,51.37717799700005],[1.37071,51.31177899700003],[1.404974500000037,51.22982049700005],[1.379638500000056,51.14215849900006],[1.034675,51.04708849800005],[0.977051500000073,51.002594],[0.98057299900006,50.914032],[0.854784,50.92370999800005]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ45","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Mid Kent","NUTS_NAME":"Mid Kent"},"geometry":{"type":"Polygon","coordinates":[[[0.601451500000053,51.333293998000045],[0.821158500000024,51.224418498000034],[0.973332500000026,51.26858149900005],[1.029294500000049,51.13252249700008],[0.779000999000061,50.98947899900003],[0.605435,51.012058499000034],[0.59041250000007,51.111327998000036],[0.640386,51.152465996000046],[0.468969500000071,51.14324950000008],[0.378219,51.23341349900005],[0.525687,51.33470899900004],[0.601451500000053,51.333293998000045]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKJ46","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"West Kent","NUTS_NAME":"West Kent"},"geometry":{"type":"Polygon","coordinates":[[[0.336721500000067,51.386539500000026],[0.341124,51.325001],[0.401326500000039,51.35294699800005],[0.468097323000052,51.362647552000055],[0.525687,51.33470899900004],[0.378219,51.23341349900005],[0.468969500000071,51.14324950000008],[0.640386,51.152465996000046],[0.59041250000007,51.111327998000036],[0.605435,51.012058499000034],[0.325307500000065,51.12287150000003],[0.050053500000047,51.14264299900003],[0.042433500000072,51.29266749700008],[0.148871,51.40850449900006],[0.152974,51.40870299800008],[0.336721500000067,51.386539500000026]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK","LEVL_CODE":1,"CNTR_CODE":"UK","NAME_LATN":"SOUTH WEST (ENGLAND)","NUTS_NAME":"SOUTH WEST (ENGLAND)"},"geometry":{"type":"Polygon","coordinates":[[[-4.139783840999939,51.077219299000035],[-4.21529449999997,51.07535549600004],[-4.200037000999941,51.20093899900007],[-3.720778499999938,51.23309349800007],[-3.313210500999958,51.179561498000055],[-3.019771499999933,51.21881099700005],[-3.05087,51.18717599900003],[-2.999351,51.23696899600003],[-3.035921499999972,51.32854849600005],[-2.992948,51.32031999800006],[-2.980789,51.38841999600004],[-2.797451499999966,51.48601149700005],[-2.679612,51.48021299800007],[-2.716312499999958,51.50710699900003],[-2.674005499999964,51.54427349800005],[-2.534737,51.677245998000046],[-2.379470499999968,51.767123998000045],[-2.660656919999951,51.62766319500008],[-2.686266000999979,51.662364996000065],[-2.65021349999995,51.826152998000055],[-2.445992499999932,51.893996998000034],[-2.500891,51.96061349800004],[-2.478917,52.02253699900007],[-2.351362,52.02136599800008],[-2.300416499999926,51.96678150000008],[-2.158117499999946,52.050209],[-1.839030000999969,52.00678650000003],[-1.853368,52.07237250000003],[-1.767627000999937,52.11259449800008],[-1.665732499999933,51.987491497000065],[-1.645167,51.91452050000004],[-1.719469499999946,51.78321450000004],[-1.648235499999942,51.68408949800005],[-1.683040499999947,51.69011299600004],[-1.691093,51.58352649700004],[-1.602793499999962,51.51829549800004],[-1.584689499999968,51.524913998000045],[-1.545737499999973,51.458991996000066],[-1.571147499999938,51.416465996000056],[-1.49828,51.329375999000035],[-1.535935,51.24861899700005],[-1.668818499999929,51.190772999000046],[-1.59938,51.023727499000074],[-1.623381499999937,50.95463550000005],[-1.956807,50.98982999800006],[-1.810535,50.92680749900006],[-1.829502499999933,50.85531599700005],[-1.788733,50.76716249800006],[-1.692049499999939,50.736625497000034],[-1.74072549999994,50.721538499000076],[-2.0406,50.71996299800003],[-2.080504,50.68798449900004],[-1.967877,50.66637399600006],[-1.953566000999956,50.59413149800008],[-2.423359499999947,50.63589499900007],[-2.470142499999952,50.58435449900003],[-2.415669499999979,50.54917899800006],[-2.456731499999933,50.51285949900006],[-2.450997,50.56172549900003],[-2.652679499999977,50.669887499000026],[-2.947815000999981,50.71830750000004],[-3.38546,50.606719998000074],[-3.469847,50.68223549700008],[-3.443591499999968,50.602722],[-3.509030499999938,50.516514],[-3.488737499999957,50.46379099600006],[-3.559090499999968,50.42766199700003],[-3.482395499999939,50.39991799900008],[-3.507436,50.378989998000065],[-3.723704,50.20151899900003],[-4.12313249999994,50.34674049700004],[-4.177532637999946,50.36402146800003],[-4.187758,50.31752399800007],[-4.386875,50.36486449900008],[-4.76328,50.32623649700008],[-4.813982,50.22931649700007],[-5.008761499999935,50.13920949800007],[-5.032680849999963,50.20807098900008],[-5.076777,50.13928599700006],[-5.05757049999994,50.052741997000055],[-5.185075499999925,49.963165496000045],[-5.29696649999994,50.07032799800004],[-5.479827,50.126014498000075],[-5.577235499999972,50.05170799900003],[-5.71682149999998,50.06912599800006],[-5.670151499999974,50.167129498000065],[-5.434389499999952,50.19229899700008],[-5.15565,50.34718299600007],[-5.04049,50.44879549700005],[-5.026513,50.541625999000075],[-4.845801,50.52435950000006],[-4.936013499999945,50.587012998000034],[-4.796273499999927,50.59804899800008],[-4.562391499999933,50.78140999900006],[-4.545959499999981,50.92835249800004],[-4.526117499999941,51.02189249600008],[-4.345269,50.988857497000026],[-4.139783840999939,51.077219299000035]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK1","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Gloucestershire, Wiltshire and Bristol/Bath area","NUTS_NAME":"Gloucestershire, Wiltshire and Bristol/Bath area"},"geometry":{"type":"Polygon","coordinates":[[[-1.767627000999937,52.11259449800008],[-1.665732499999933,51.987491497000065],[-1.645167,51.91452050000004],[-1.719469499999946,51.78321450000004],[-1.648235499999942,51.68408949800005],[-1.683040499999947,51.69011299600004],[-1.691093,51.58352649700004],[-1.602793499999962,51.51829549800004],[-1.584689499999968,51.524913998000045],[-1.545737499999973,51.458991996000066],[-1.571147499999938,51.416465996000056],[-1.49828,51.329375999000035],[-1.535935,51.24861899700005],[-1.668818499999929,51.190772999000046],[-1.59938,51.023727499000074],[-1.623381499999937,50.95463550000005],[-1.956807,50.98982999800006],[-2.101780499999961,50.94498450000003],[-2.325841499999967,51.07967749900007],[-2.364515499999925,51.118880999000055],[-2.245328,51.25387949800006],[-2.289077,51.32527550000003],[-2.477988499999981,51.273102],[-2.992948,51.32031999800006],[-2.980789,51.38841999600004],[-2.797451499999966,51.48601149700005],[-2.679612,51.48021299800007],[-2.716312499999958,51.50710699900003],[-2.674005499999964,51.54427349800005],[-2.534737,51.677245998000046],[-2.379470499999968,51.767123998000045],[-2.660656919999951,51.62766319500008],[-2.686266000999979,51.662364996000065],[-2.65021349999995,51.826152998000055],[-2.445992499999932,51.893996998000034],[-2.500891,51.96061349800004],[-2.478917,52.02253699900007],[-2.351362,52.02136599800008],[-2.300416499999926,51.96678150000008],[-2.158117499999946,52.050209],[-1.839030000999969,52.00678650000003],[-1.853368,52.07237250000003],[-1.767627000999937,52.11259449800008]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK11","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bristol, City of","NUTS_NAME":"Bristol, City of"},"geometry":{"type":"Polygon","coordinates":[[[-2.679612,51.48021299800007],[-2.716312499999958,51.50710699900003],[-2.674005499999964,51.54427349800005],[-2.512135499999943,51.463062498000056],[-2.630455499999925,51.40211499900005],[-2.679612,51.48021299800007]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK12","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bath and North East Somerset, North Somerset and South Gloucestershire","NUTS_NAME":"Bath and North East Somerset, North Somerset and South Gloucestershire"},"geometry":{"type":"Polygon","coordinates":[[[-2.272544499999981,51.577595],[-2.321626499999979,51.49826050000007],[-2.291306,51.48737350000005],[-2.289077,51.32527550000003],[-2.477988499999981,51.273102],[-2.992948,51.32031999800006],[-2.980789,51.38841999600004],[-2.797451499999966,51.48601149700005],[-2.679612,51.48021299800007],[-2.630455499999925,51.40211499900005],[-2.512135499999943,51.463062498000056],[-2.674005499999964,51.54427349800005],[-2.534737,51.677245998000046],[-2.272544499999981,51.577595]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK13","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Gloucestershire","NUTS_NAME":"Gloucestershire"},"geometry":{"type":"Polygon","coordinates":[[[-1.665732499999933,51.987491497000065],[-1.645167,51.91452050000004],[-1.719469499999946,51.78321450000004],[-1.648235499999942,51.68408949800005],[-1.683040499999947,51.69011299600004],[-1.788599499999975,51.66700349900003],[-2.05715749999996,51.672446998000055],[-2.272544499999981,51.577595],[-2.534737,51.677245998000046],[-2.379470499999968,51.767123998000045],[-2.660656919999951,51.62766319500008],[-2.686266000999979,51.662364996000065],[-2.65021349999995,51.826152998000055],[-2.445992499999932,51.893996998000034],[-2.500891,51.96061349800004],[-2.478917,52.02253699900007],[-2.351362,52.02136599800008],[-2.300416499999926,51.96678150000008],[-2.158117499999946,52.050209],[-1.839030000999969,52.00678650000003],[-1.853368,52.07237250000003],[-1.767627000999937,52.11259449800008],[-1.665732499999933,51.987491497000065]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK14","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Swindon","NUTS_NAME":"Swindon"},"geometry":{"type":"Polygon","coordinates":[[[-1.602793499999962,51.51829549800004],[-1.83756649999998,51.50083899600003],[-1.857567499999959,51.56331250000005],[-1.788599499999975,51.66700349900003],[-1.683040499999947,51.69011299600004],[-1.691093,51.58352649700004],[-1.602793499999962,51.51829549800004]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK15","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Wiltshire CC","NUTS_NAME":"Wiltshire CC"},"geometry":{"type":"Polygon","coordinates":[[[-1.788599499999975,51.66700349900003],[-1.857567499999959,51.56331250000005],[-1.83756649999998,51.50083899600003],[-1.602793499999962,51.51829549800004],[-1.584689499999968,51.524913998000045],[-1.545737499999973,51.458991996000066],[-1.571147499999938,51.416465996000056],[-1.49828,51.329375999000035],[-1.535935,51.24861899700005],[-1.668818499999929,51.190772999000046],[-1.59938,51.023727499000074],[-1.623381499999937,50.95463550000005],[-1.956807,50.98982999800006],[-2.101780499999961,50.94498450000003],[-2.325841499999967,51.07967749900007],[-2.364515499999925,51.118880999000055],[-2.245328,51.25387949800006],[-2.289077,51.32527550000003],[-2.291306,51.48737350000005],[-2.321626499999979,51.49826050000007],[-2.272544499999981,51.577595],[-2.05715749999996,51.672446998000055],[-1.788599499999975,51.66700349900003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Dorset and Somerset","NUTS_NAME":"Dorset and Somerset"},"geometry":{"type":"Polygon","coordinates":[[[-2.992948,51.32031999800006],[-2.477988499999981,51.273102],[-2.289077,51.32527550000003],[-2.245328,51.25387949800006],[-2.364515499999925,51.118880999000055],[-2.325841499999967,51.07967749900007],[-2.101780499999961,50.94498450000003],[-1.956807,50.98982999800006],[-1.810535,50.92680749900006],[-1.829502499999933,50.85531599700005],[-1.788733,50.76716249800006],[-1.692049499999939,50.736625497000034],[-1.74072549999994,50.721538499000076],[-2.0406,50.71996299800003],[-2.080504,50.68798449900004],[-1.967877,50.66637399600006],[-1.953566000999956,50.59413149800008],[-2.423359499999947,50.63589499900007],[-2.470142499999952,50.58435449900003],[-2.415669499999979,50.54917899800006],[-2.456731499999933,50.51285949900006],[-2.450997,50.56172549900003],[-2.652679499999977,50.669887499000026],[-2.947815000999981,50.71830750000004],[-2.954328,50.76771149600006],[-2.886639499999944,50.79480749800007],[-2.954316,50.82117449800006],[-3.052417,50.90826399900004],[-3.152869,50.89379899800008],[-3.187833500999943,50.910362],[-3.163939,50.94597249800006],[-3.426810499999931,51.03140649900007],[-3.609741,51.007984],[-3.594467499999951,51.05519499900004],[-3.834736,51.14138399900003],[-3.723824,51.18072899900005],[-3.720778499999938,51.23309349800007],[-3.313210500999958,51.179561498000055],[-3.019771499999933,51.21881099700005],[-3.05087,51.18717599900003],[-2.999351,51.23696899600003],[-3.035921499999972,51.32854849600005],[-2.992948,51.32031999800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK21","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Bournemouth and Poole","NUTS_NAME":"Bournemouth and Poole"},"geometry":{"type":"Polygon","coordinates":[[[-1.74072549999994,50.721538499000076],[-2.0406,50.71996299800003],[-2.005862,50.79092399900003],[-1.935599,50.792091500000026],[-1.74072549999994,50.721538499000076]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK22","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Dorset CC","NUTS_NAME":"Dorset CC"},"geometry":{"type":"Polygon","coordinates":[[[-1.956807,50.98982999800006],[-1.810535,50.92680749900006],[-1.829502499999933,50.85531599700005],[-1.788733,50.76716249800006],[-1.692049499999939,50.736625497000034],[-1.74072549999994,50.721538499000076],[-1.935599,50.792091500000026],[-2.005862,50.79092399900003],[-2.0406,50.71996299800003],[-2.080504,50.68798449900004],[-1.967877,50.66637399600006],[-1.953566000999956,50.59413149800008],[-2.423359499999947,50.63589499900007],[-2.470142499999952,50.58435449900003],[-2.415669499999979,50.54917899800006],[-2.456731499999933,50.51285949900006],[-2.450997,50.56172549900003],[-2.652679499999977,50.669887499000026],[-2.947815000999981,50.71830750000004],[-2.954328,50.76771149600006],[-2.886639499999944,50.79480749800007],[-2.954316,50.82117449800006],[-2.627953,50.88630699600003],[-2.603377499999965,50.976345],[-2.544765,50.991577],[-2.344547,50.97882849700005],[-2.381747,51.001945498000055],[-2.325841499999967,51.07967749900007],[-2.101780499999961,50.94498450000003],[-1.956807,50.98982999800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK23","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Somerset","NUTS_NAME":"Somerset"},"geometry":{"type":"Polygon","coordinates":[[[-2.992948,51.32031999800006],[-2.477988499999981,51.273102],[-2.289077,51.32527550000003],[-2.245328,51.25387949800006],[-2.364515499999925,51.118880999000055],[-2.325841499999967,51.07967749900007],[-2.381747,51.001945498000055],[-2.344547,50.97882849700005],[-2.544765,50.991577],[-2.603377499999965,50.976345],[-2.627953,50.88630699600003],[-2.954316,50.82117449800006],[-3.052417,50.90826399900004],[-3.152869,50.89379899800008],[-3.187833500999943,50.910362],[-3.163939,50.94597249800006],[-3.426810499999931,51.03140649900007],[-3.609741,51.007984],[-3.594467499999951,51.05519499900004],[-3.834736,51.14138399900003],[-3.723824,51.18072899900005],[-3.720778499999938,51.23309349800007],[-3.313210500999958,51.179561498000055],[-3.019771499999933,51.21881099700005],[-3.05087,51.18717599900003],[-2.999351,51.23696899600003],[-3.035921499999972,51.32854849600005],[-2.992948,51.32031999800006]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK3","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Cornwall and Isles of Scilly","NUTS_NAME":"Cornwall and Isles of Scilly"},"geometry":{"type":"Polygon","coordinates":[[[-4.23509422099994,50.46025278400003],[-4.203700022999953,50.39825618900005],[-4.177532637999946,50.36402146800003],[-4.187758,50.31752399800007],[-4.386875,50.36486449900008],[-4.76328,50.32623649700008],[-4.813982,50.22931649700007],[-5.008761499999935,50.13920949800007],[-5.032680849999963,50.20807098900008],[-5.076777,50.13928599700006],[-5.05757049999994,50.052741997000055],[-5.185075499999925,49.963165496000045],[-5.29696649999994,50.07032799800004],[-5.479827,50.126014498000075],[-5.577235499999972,50.05170799900003],[-5.71682149999998,50.06912599800006],[-5.670151499999974,50.167129498000065],[-5.434389499999952,50.19229899700008],[-5.15565,50.34718299600007],[-5.04049,50.44879549700005],[-5.026513,50.541625999000075],[-4.845801,50.52435950000006],[-4.936013499999945,50.587012998000034],[-4.796273499999927,50.59804899800008],[-4.562391499999933,50.78140999900006],[-4.545959499999981,50.92835249800004],[-4.456273500999941,50.92902349800005],[-4.42092,50.86619549900007],[-4.493454499999928,50.792831498000055],[-4.386616,50.76392750000008],[-4.307998,50.57887649800006],[-4.180500499999937,50.50337999800007],[-4.23509422099994,50.46025278400003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKK30","LEVL_CODE":3,"CNTR_CODE":"UK","NAME_LATN":"Cornwall and Isles of Scilly","NUTS_NAME":"Cornwall and Isles of Scilly"},"geometry":{"type":"Polygon","coordinates":[[[-4.23509422099994,50.46025278400003],[-4.203700022999953,50.39825618900005],[-4.177532637999946,50.36402146800003],[-4.187758,50.31752399800007],[-4.386875,50.36486449900008],[-4.76328,50.32623649700008],[-4.813982,50.22931649700007],[-5.008761499999935,50.13920949800007],[-5.032680849999963,50.20807098900008],[-5.076777,50.13928599700006],[-5.05757049999994,50.052741997000055],[-5.185075499999925,49.963165496000045],[-5.29696649999994,50.07032799800004],[-5.479827,50.126014498000075],[-5.577235499999972,50.05170799900003],[-5.71682149999998,50.06912599800006],[-5.670151499999974,50.167129498000065],[-5.434389499999952,50.19229899700008],[-5.15565,50.34718299600007],[-5.04049,50.44879549700005],[-5.026513,50.541625999000075],[-4.845801,50.52435950000006],[-4.936013499999945,50.587012998000034],[-4.796273499999927,50.59804899800008],[-4.562391499999933,50.78140999900006],[-4.545959499999981,50.92835249800004],[-4.456273500999941,50.92902349800005],[-4.42092,50.86619549900007],[-4.493454499999928,50.792831498000055],[-4.386616,50.76392750000008],[-4.307998,50.57887649800006],[-4.180500499999937,50.50337999800007],[-4.23509422099994,50.46025278400003]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM6","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Highlands and Islands","NUTS_NAME":"Highlands and Islands"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-0.75837,60.81646749600003],[-0.831153,60.685222499000076],[-0.981795499999976,60.685482],[-0.865963499999964,60.83370199700005],[-0.75837,60.81646749600003]]],[[[-1.003055499999959,60.58155799700006],[-1.028645,60.49721549700007],[-1.167948499999966,60.49954999900007],[-1.202889,60.607105499000056],[-1.074501,60.73317349700005],[-1.008528,60.71493899900008],[-0.981463500999951,60.637199500000065],[-1.031584500999941,60.59758399900005],[-1.003055499999959,60.58155799700006]]],[[[-1.319979499999931,60.451770999000075],[-1.383588,60.397674496000036],[-1.237423000999968,60.49381999700006],[-1.049048,60.43949149600007],[-1.155833499999972,60.33803549900006],[-1.079679499999941,60.32329199900005],[-1.15929,60.281048],[-1.095383499999969,60.26348899900006],[-1.170235,60.23842599900007],[-1.155375499999934,60.15899649800008],[-1.203963500999976,60.130611499000054],[-1.277545,59.85259999700003],[-1.391431500999943,59.91602349900006],[-1.276649,60.099434],[-1.32783749999993,60.16605749900003],[-1.293532499999969,60.251404],[-1.462364499999978,60.14639299900006],[-1.702932,60.255359498000075],[-1.339904500999978,60.359771497000054],[-1.632932,60.48316199900006],[-1.499491318999958,60.543659871000045],[-1.489197784999931,60.54412077600006],[-1.304159,60.62399649900004],[-1.319979499999931,60.451770999000075]]],[[[-3.046968499999934,58.47255350000006],[-3.116977499999962,58.368579999000076],[-4.082477499999925,57.949943500000074],[-4.001344,57.92654399800006],[-4.016409,57.85973750000005],[-4.400494,57.918987497000046],[-4.04890249999994,57.81504049600005],[-3.771491499999968,57.867034998000065],[-3.975255499999946,57.69451899900008],[-4.301044,57.67506799700004],[-4.428933,57.574283499000046],[-3.993433499999981,57.67645649800005],[-4.234324499999957,57.500965],[-4.432052499999941,57.494354],[-4.191771,57.48384099900005],[-4.019451499999946,57.598220999000034],[-3.347729,57.72444899900006],[-2.801509,57.69524749900006],[-2.810592,57.61008449700006],[-2.649769,57.52970099700008],[-2.971998,57.49148199800004],[-3.019933499999979,57.44965749800008],[-2.954439,57.37796],[-2.984190499999954,57.27787799600003],[-3.208485499999938,57.25625999600004],[-3.353487499999972,57.17829899600008],[-3.322594,57.13163749900008],[-3.368641,57.10041799700008],[-3.749711499999933,57.07641999900005],[-3.801627499999938,56.936012497000036],[-4.532142499999964,56.795907998000075],[-4.720448,56.65656299700004],[-4.561758499999939,56.57027049900006],[-4.697694499999955,56.549896],[-4.63908349999997,56.47616199700008],[-4.854503499999964,56.370532998000044],[-4.771805749999942,56.343766781000056],[-4.836395499999981,56.291446499000074],[-4.732895499999927,56.23231549900004],[-4.752816000999928,56.20385749700006],[-4.862619499999937,56.100319],[-4.917712,56.164127499000074],[-4.896133499999962,55.98607999600006],[-4.979976,55.86157999800008],[-5.120735,56.00808349700003],[-5.074320499999942,55.90140149900003],[-5.190108,55.94739149900005],[-5.242238,55.89447799800007],[-5.205963,55.82628650000004],[-5.351127,55.90980549800008],[-5.34193149999993,55.995186],[-5.127448198999957,56.166530160000036],[-5.380715,56.00484449900006],[-5.440771499999926,56.03461450000003],[-5.419614,55.896728497000026],[-5.314025,55.783824999000046],[-5.448947,55.707809498000074],[-5.490266,55.52957549800004],[-5.568165000999954,55.42831399700003],[-5.52000449999997,55.36192299700008],[-5.799178,55.299698],[-5.66694,55.66451649900006],[-5.436028,55.85756299800005],[-5.613714,55.76017749700003],[-5.666273499999932,55.800583],[-5.571037499999932,55.93432999800007],[-5.688422499999945,55.911018496000054],[-5.566499,56.04157249900004],[-5.705351,55.93992999800008],[-5.503709499999957,56.18886549800004],[-5.613348,56.133239497000034],[-5.567006,56.23656099900006],[-5.487101000999928,56.256931500000064],[-5.654321,56.297649496000076],[-5.441643,56.365916997000056],[-5.589511,56.38047799600008],[-5.441085499999929,56.455798998000034],[-5.259811117999959,56.450957274000075],[-5.473969,56.481708499000035],[-5.384345607999933,56.53174223900004],[-5.421163,56.54778649900004],[-5.325605499999938,56.648285],[-4.969490499999949,56.714679498000066],[-5.247245499999963,56.70311749700005],[-5.092236,56.83002449600008],[-5.334743499999945,56.859508497000036],[-5.122688000999972,56.827274498000065],[-5.684919,56.497295499000074],[-6.006905,56.645126497000035],[-5.867277643999955,56.675698871000066],[-6.226232,56.710991],[-5.751703000999953,56.78216549700005],[-5.887583499999948,56.792953999000076],[-5.722717500999977,56.889038],[-5.924258,56.89156349700005],[-5.820707,57.01260749700003],[-5.50935149999998,56.99639499700004],[-5.797323,57.06098549700005],[-5.72895,57.10147849700007],[-5.509437499999933,57.097774498000035],[-5.681389,57.15053049900007],[-5.670602562999932,57.20843875300005],[-6.017994,57.01794799800007],[-6.004551,57.114955999000074],[-5.908913,57.17123799900003],[-6.32237,57.160293498000044],[-6.483495,57.310973999000055],[-6.307433,57.299278497000046],[-6.789714499999945,57.42120749800006],[-6.716572,57.51391599900006],[-6.582203,57.423621996000065],[-6.637269499999945,57.50403199800007],[-6.562680499999942,57.504444],[-6.638996499999962,57.549937998000075],[-6.635986499999944,57.60881799900005],[-6.406945,57.523063498000056],[-6.427763499999969,57.64247149900007],[-6.296708,57.70756149700003],[-6.146597000999975,57.57447449700004],[-6.130341,57.316936498000075],[-5.88541349999997,57.238167],[-5.776867,57.27114499700008],[-5.64761249999998,57.25529499700008],[-5.664395153999976,57.219288446000064],[-5.415392,57.23015599900003],[-5.73379,57.283454996000046],[-5.552481264999926,57.367434305000074],[-5.441254,57.418952997000076],[-5.82202,57.363647496000056],[-5.872894,57.47562050000005],[-5.841507499999977,57.57624799900003],[-5.513192499999946,57.54019549900005],[-5.81922,57.639786],[-5.673280499999976,57.70169849900003],[-5.813466499999947,57.75067150000007],[-5.81416349999995,57.85871099900004],[-5.683442,57.86545199900007],[-5.619121,57.766895497000064],[-5.577558499999952,57.79293449800008],[-5.655929499999957,57.87793349800006],[-5.618142,57.92462899800006],[-5.217599,57.84221649700004],[-5.406790499999943,57.92776849900008],[-5.360826000999964,57.93917449800006],[-5.071033000999932,57.81991949700006],[-5.458675,58.074409495000054],[-5.268833,58.10244799800006],[-5.403668,58.23530949700006],[-5.375456,58.26508699900006],[-4.924184000999958,58.25644699700007],[-5.134424,58.27807599700003],[-5.182866499999932,58.35124949800007],[-5.033927499999947,58.39294799700008],[-5.114617499999952,58.415272],[-5.072923,58.452243999000075],[-4.988059499999963,58.425987],[-5.126430499999969,58.48945599700005],[-5.007117499999936,58.625992],[-4.65146249999998,58.534366498000054],[-4.739715499999932,58.44739149800006],[-4.589487499999962,58.57794549700003],[-4.409354499999949,58.523773],[-4.475344,58.44438949700003],[-4.357074,58.536857498000074],[-3.024449,58.644264],[-3.130931,58.519863],[-3.046968499999934,58.47255350000006]]],[[[-2.848329500999967,58.958244499000045],[-2.788509000999966,58.915839995000056],[-2.704751499999929,58.961914],[-2.712558499999943,58.92079149700004],[-2.897167053999965,58.89524031000008],[-3.368161499999928,58.99882899900007],[-3.325062,59.13616199900008],[-3.196686,59.15369050000004],[-3.004050499999948,59.07222],[-3.11450150099995,59.00661450000007],[-2.911924499999941,59.006061498000065],[-2.801162499999975,58.99209999800007],[-2.848329500999967,58.958244499000045]]],[[[-3.259673,58.78805149900006],[-3.219864,58.775715],[-3.293498,58.77659249900006],[-3.434554499999933,58.87152850000007],[-3.32561149999998,58.92784899900005],[-3.170331,58.82104099800006],[-3.259673,58.78805149900006]]],[[[-5.082880499999931,55.55278799800004],[-5.083631,55.45461249700003],[-5.313261,55.46509149600007],[-5.39711000099993,55.61743149700004],[-5.288532499999974,55.71808599700006],[-5.201502499999947,55.702484],[-5.082880499999931,55.55278799800004]]],[[[-5.646332,56.44055950000006],[-5.834676,56.310558496000056],[-6.385666500999946,56.28878799700004],[-6.35029,56.34788899800003],[-6.190255,56.332347998000046],[-5.979565499999978,56.38647099800005],[-6.194926499999951,56.358978499000045],[-6.20478049999997,56.38530749700004],[-6.129291,56.448417498000026],[-5.996329499999945,56.48778549900004],[-6.340745,56.538177500000074],[-6.323570500999949,56.60616299700007],[-6.112694000999966,56.64403899900003],[-5.950804500999936,56.518039498000064],[-5.799656500999959,56.51579649700005],[-5.646332,56.44055950000006]]],[[[-5.793228499999941,56.012721999000064],[-5.96267649899994,55.794749997000054],[-6.09684249999998,55.871860498000046],[-5.924639498999966,55.962383499000055],[-6.006308,55.973919],[-5.964173,56.027405],[-5.687118998999949,56.12949350000008],[-5.793228499999941,56.012721999000064]]],[[[-6.101226499999939,55.84050350000007],[-6.01987,55.68493249900007],[-6.266864000999931,55.579871996000065],[-6.335771499999964,55.58950049800006],[-6.260210499999971,55.659263500000066],[-6.341544499999941,55.71799849700005],[-6.260346499999969,55.781940500000076],[-6.526495499999953,55.693542499000046],[-6.45583,55.85268399800003],[-6.120203,55.934597],[-6.101226499999939,55.84050350000007]]],[[[-6.167010499999947,58.34677099900006],[-6.344816,58.234204496000075],[-6.135358,58.25944149700007],[-6.601655500999925,58.08697899600003],[-6.368577499999958,58.07648849700007],[-6.388316,57.999859],[-6.573711,58.004272498000034],[-6.454871,57.99351499900007],[-6.469132499999944,57.939422499000045],[-6.652290499999935,57.91831599800008],[-6.712528,58.013172],[-6.588023500999952,58.05318850000003],[-6.69152,58.05827699800005],[-6.739541,57.991840496000066],[-6.663898,57.88521599900008],[-6.812832499999956,57.880672496000045],[-6.744717,57.82280749800003],[-6.967598,57.727894],[-7.133195499999943,57.836654500000066],[-6.81601349999994,57.900836998000045],[-7.079999499999929,57.967232],[-7.082464,58.02035899700007],[-6.907952,58.049675],[-7.103295000999935,58.07354749900003],[-7.108741499999951,58.177036498000064],[-7.027442,58.24435399600003],[-6.746137499999975,58.18470399700004],[-6.828675,58.284096],[-6.25831,58.516716],[-6.180591,58.46701049900008],[-6.217891499999951,58.36830499600006],[-6.167010499999947,58.34677099900006]]],[[[-7.182542499999954,57.65663150000006],[-7.062166,57.64057150000008],[-7.214868499999966,57.64065949800005],[-7.10083149999997,57.59365449900008],[-7.299847,57.553420997000046],[-7.133844,57.552966998000045],[-7.148515,57.50972349700004],[-7.549144499999954,57.60290549900003],[-7.152460499999961,57.665553997000075],[-7.182542499999954,57.65663150000006]]],[[[-7.35166,57.15094749700006],[-7.211596499999928,57.11778249900004],[-7.412508499999944,57.149811],[-7.412302499999953,57.38740150000007],[-7.19152,57.29863750000004],[-7.298812670999951,57.22601419200004],[-7.246141,57.16512699800006],[-7.35166,57.15094749700006]]]]}},{"type":"Feature","properties":{"NUTS_ID":"UKM2","LEVL_CODE":2,"CNTR_CODE":"UK","NAME_LATN":"Eastern Scotland","NUTS_NAME":"Eastern Scotland"},"geometry":{"type":"Polygon","coordinates":[[[-2.681236499999955,56.91047299600007],[-2.616337499999929,56.78367250000008],[-2.425346,56.755187996000075],[-2.538641,56.566062997000074],[-2.733122499999979,56.46479799900004],[-3.051892500999941,56.45849999700005],[-3.345321499999955,56.37197099600007],[-3.276975499999935,56.35053250000004],[-2.803087,56.43967799700005],[-2.838920499999972,56.35307299600004],[-2.620387,56.26118849900007],[-2.955285,56.21049499700007],[-3.39145149999996,56.00593199700006],[-3.866839499999969,56.12066249900005],[-3.820153,56.099117500000034],[-3.515724,56.00225850000004],[-3.425330499999973,55.99403],[-3.07764549999996,55.94689549800006],[-2.815697499999942,56.062404496000056],[-2.366623499999946,55.946113498000045],[-2.13755649999996,55.91727849600005],[-2.034329,55.81116499900003],[-2.335961,55.63214499800006],[-2.165465499999925,55.46846750000003],[-2.496131499999933,55.346595999000044],[-2.689751,55.18906399700006],[-2.858508,55.10842499800003],[-2.904721,55.17374799800007],[-2.863809499999945,55.236431],[-2.917027,55.27749249800007],[-3.316549,55.34824],[-3.243565000999979,55.427772497000035],[-3.310962,55.444916],[-3.507365,55.41234949600005],[-3.539553,55.44326],[-3.486589,55.51714299900004],[-3.529996499999925,55.611484500000074],[-3.396386,55.716407996000044],[-3.471619,55.771068497000044],[-3.744007499999952,55.78210849700008],[-3.717458499999964,55.88576149700003],[-3.822552,55.896595],[-4.020121,56.028145],[-4.152381499999933,56.00814449900008],[-4.47081,56.002395500000034],[-4.598209,56.08433899800008],[-4.695389499999976,56.19928349600008],[-4.676723,56.31299199600005],[-4.854503499999964,56.370532998000044],[-4.63908349999997,56.47616199700008],[-4.697694499999955,56.549896],[-4.561758499999939,56.57027049900006],[-4.720448,56.65656299700004],[-4.532142499999964,56.795907998000075],[-3.801627499999938,56.936012497000036],[-3.37210749999997,56.87474449700005],[-2.851041000999942,56.97269799800006],[-2.681236499999955,56.91047299600007]]]}}]}},"visId":"gw_JDdk","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1,"timezoneDisplayOffset":0},"encodings":{"dimensions":[{"fid":"person_name","name":"person_name","basename":"person_name","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"nuts_code","name":"nuts_code","basename":"nuts_code","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"technology_field","name":"technology_field","basename":"technology_field","semanticType":"nominal","analyticType":"dimension","offset":0},{"fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"fid":"appln_count","name":"appln_count","basename":"appln_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","offset":0},{"fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[],"columns":[],"color":[],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":320,"height":200},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_5Zgs","name":"Chart 2"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["nuts_code"],"measures":[{"field":"appln_count","agg":"sum","asFieldKey":"appln_count_sum"}]}]}]},{"workflow":[{"type":"view","query":[{"op":"raw","fields":[]}]}]}],"version":"0.4.9.13"}"""
# pyg.walk(df, spec=vis_spec)

# Query Documentation: Patent Analysis by NUTS Level 3

## Purpose:
This query retrieves detailed information about patent applications in Germany, categorized by NUTS Level 3 regions (Landkreise). It is designed to provide insights into:
- Applicants (`person_name`) operating in specific regions.
- The technologies they focus on (`technology_field`), identified using CPC classifications.
- Filing trends over time (`filing_year`).
- Patent status, specifically whether the patent has been granted (`granted`).

## Key Features:
1. **Regional Insights**:
   - Leverages NUTS Level 3 codes to localize applicants within Germany.
   - Extracts NUTS Level 1 (Federal State) codes for higher-level aggregation.

2. **Technological Focus**:
   - Links patents to their Cooperative Patent Classification (CPC) codes, enabling analysis of specific technology fields.

3. **Filing Year Trends**:
   - Includes the filing year to track application trends over time.

4. **Grant Status**:
   - Determines whether patents in the dataset have been granted.

5. **Execution Timing**:
   - Measures and reports the query execution time to provide transparency on performance.

## Data Sources:
- **Applicants**: `TLS206_PERSON`
- **Patent Applications**: `TLS201_APPLN`
- **Technology Classification**: `TLS224_APPLN_CPC`
- **Geographic Information**: NUTS codes from `TLS206_PERSON`

## Intended Audience:
This query is specifically designed for patent information professionals seeking actionable insights into:
- Regional patent activity.
- Applicant profiles and their focus areas.
- Trends in patent filings and grants across Germany.

## How to Use:
1. Execute the query in a PATSTAT-connected environment.
2. Use the resulting DataFrame (`df`) for:
   - Exploratory data analysis.
   - Visualization with tools like Pygwalker.
   - Further filtering or aggregation based on your needs.

In [ ]:
# Define SQL queries using ORM for analysis
# Example: Query patent applicants and technology distribution
# Start the timer
start_time = time.time()

q = db.query(
    TLS206_PERSON.person_name,
    TLS206_PERSON.nuts.label('nuts_code'),
    literal_column("SUBSTR(nuts, 1, 3)").label('federal_state_code'),  # Federal state (NUTS Level 1)
    TLS224_APPLN_CPC.cpc_class_symbol.label('technology_field'),       # Technology field
    literal_column("SUBSTR(cpc_class_symbol, 1, 4)").label('cpc_mainclass'),
    TLS201_APPLN.appln_filing_year.label('filing_year'),               # Filing year
    TLS201_APPLN.granted.label('granted'),                             # Grant status
    func.count(TLS201_APPLN.appln_id).label('appln_count')             # Application count
).select_from(TLS206_PERSON).\
    join(TLS207_PERS_APPLN, TLS206_PERSON.person_id == TLS207_PERS_APPLN.person_id).\
    join(TLS201_APPLN, TLS207_PERS_APPLN.appln_id == TLS201_APPLN.appln_id).\
    join(TLS224_APPLN_CPC, TLS201_APPLN.appln_id == TLS224_APPLN_CPC.appln_id).\
    filter(
        TLS206_PERSON.nuts.startswith('DE'),   # Filter for Germany NUTS code
        TLS206_PERSON.nuts_level == 3          # Limit to NUTS level 3
    ).\
    group_by(
        TLS206_PERSON.nuts,                    # Group by NUTS Level 3 code
        literal_column("SUBSTR(nuts, 1, 3)"),  # Group by federal state code
        TLS224_APPLN_CPC.cpc_class_symbol,     # Group by technology field
        TLS206_PERSON.person_name,             # Group by person name
        TLS201_APPLN.appln_filing_year,        # Group by filing year
        TLS201_APPLN.granted                   # Group by grant status
    ).\
    order_by(TLS206_PERSON.nuts) #, TLS201_APPLN.appln_filing_year)

# Execute the query
df = patstat.df(q)

# Stop the timer
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Query execution time: {execution_time:.2f} seconds")



# Federal state mapping

federal_state_mapping = {
    "DE1": "Baden-Württemberg",
    "DE2": "Bayern",
    "DE3": "Berlin",
    "DE4": "Brandenburg",
    "DE5": "Bremen",
    "DE6": "Hamburg",
    "DE7": "Hessen",
    "DE8": "Mecklenburg-Vorpommern",
    "DE9": "Niedersachsen",
    "DEA": "Nordrhein-Westfalen",
    "DEB": "Rheinland-Pfalz",
    "DEC": "Saarland",
    "DED": "Sachsen",
    "DEE": "Sachsen-Anhalt",
    "DEF": "Schleswig-Holstein",
    "DEG": "Thüringen"
}

# execute the mapping
df["federal_state_name"] = df["federal_state_code"].map(federal_state_mapping)

# Display the first few rows of the updated DataFrame
df



In [65]:

# File path to the IPC XML
filename = "./EN_ipc_scheme_20210101.xml"

# Define the namespace and parser
ipc_namespace = '{http://www.wipo.int/classifications/ipc/masterfiles}'
ipcEntry = f"{ipc_namespace}ipcEntry"
text_body = f"{ipc_namespace}textBody"
title_part = f"{ipc_namespace}titlePart"
text = f"{ipc_namespace}text"
parser = ET.XMLParser(remove_blank_text=True)

# Parse the XML file
tree = ET.parse(filename, parser=parser)
root = tree.getroot()

# Initialize dictionary for sub-class mapping
sub_class_mapping = {}

# Start measuring time
start = time.time()

# Iterate through the XML to extract sub-class information
for element in root.iter(ipcEntry):
    if element.attrib.get("kind") == "u":  # Focus on sub-classes
        symbol = element.attrib.get("symbol")  # Extract sub-class symbol

        # Locate the title text within the nested structure
        text_element = element.find(f".//{text_body}//{title_part}//{text}")
        title = text_element.text.strip() if text_element is not None else "No Title"

        sub_class_mapping[symbol] = title

# Print execution time
print(f"Extracted {len(sub_class_mapping)} sub-classes in {(time.time() - start) * 1000:.0f} ms.")

# Print a sample of the extracted data
for symbol, title in list(sub_class_mapping.items())[:20]:
    print(f"{symbol}: {title}")



Extracted 646 sub-classes in 156 ms.
A01B: SOIL WORKING IN AGRICULTURE OR FORESTRY
A01C: PLANTING
A01D: HARVESTING
A01F: THRESHING
A01G: HORTICULTURE
A01H: NEW PLANTS OR PROCESSES FOR OBTAINING THEM
A01J: MANUFACTURE OF DAIRY PRODUCTS
A01K: ANIMAL HUSBANDRY
A01L: SHOEING OF ANIMALS
A01M: CATCHING, TRAPPING OR SCARING OF ANIMALS
A01N: PRESERVATION OF BODIES OF HUMANS OR ANIMALS OR PLANTS OR PARTS THEREOF
A01P: BIOCIDAL, PEST REPELLANT, PEST ATTRACTANT OR PLANT GROWTH REGULATORY ACTIVITY OF CHEMICAL COMPOUNDS OR PREPARATIONS
A21B: BAKERS' OVENS
A21C: MACHINES OR EQUIPMENT FOR MAKING OR PROCESSING DOUGHS
A21D: TREATMENT, e.g. PRESERVATION, OF FLOUR OR DOUGH FOR BAKING, e.g. BY ADDITION OF MATERIALS
A22B: SLAUGHTERING
A22C: PROCESSING MEAT, POULTRY, OR FISH
A23B: PRESERVING, e.g. BY CANNING, MEAT, FISH, EGGS, FRUIT, VEGETABLES, EDIBLE SEEDS
A23C: DAIRY PRODUCTS, e.g. MILK, BUTTER, CHEESE
A23D: EDIBLE OILS OR FATS, e.g. MARGARINES, SHORTENINGS, COOKING OILS


In [ ]:
# start the viusalization with pygwalker
pyg.walk(df)
